# Retrieve the filename of the latest dataset of NCBI Pathogen Isolates.

In [ ]:
%%time
from ftplib import FTP
import re

def get_version():
  version = None
  def get_ftp_version(ftp_filename):
    nonlocal version
    version = re.search('PDG000000004.([0-9]+).amr.metadata.tsv', ftp_filename).group(1)

  with FTP('ftp.ncbi.nlm.nih.gov') as ftp:
      ftp.login()
      ftp.cwd('pathogen/Results/Escherichia_coli_Shigella/latest_snps/AMR/')
      ftp.retrlines('NLST PDG000000004.*.amr.metadata.tsv', get_ftp_version)

  return version

# Store the isolates data into a Pandas dataframe.

In [ ]:
%%time
import pandas as pd

# set display options to show dataframes in full
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

url = 'https://ftp.ncbi.nlm.nih.gov/pathogen/Results/Escherichia_coli_Shigella/latest_snps/AMR/PDG000000004.'\
      + get_version() + '.amr.metadata.tsv'
df = pd.read_table(url, sep='\t')[['target_acc', 'target_creation_date', 'AMR_genotypes', 'stress_genotypes',
                                   'virulence_genotype']]

# Store a copy of the isolates dataframe into an in-memory database.

In [ ]:
%%time
from sqlalchemy import create_engine

db = create_engine('sqlite://', echo=False)
df.to_sql('isolate', con=db, if_exists='replace', index=False)

# Using SQL, report the total number of isolate records retrieved.

In [ ]:
%%time
print('There are', db.execute('SELECT COUNT(*) FROM isolate').fetchall()[0][0], 'records retrieved.')
print('Note: These', db.execute('SELECT COUNT(DISTINCT(target_acc)) FROM isolate').fetchall()[0][0],
      'are uniquely identified by the field `target_acc` (i.e., a unique key).')

### Using SQL, report the number of isolates missing any of the `AMR_genotypes`, `stress_genotypes`, or `virulence_genotype` data.

In [ ]:
print('Number of isolates missing any of the AMR_genotypes, stress_genotypes, or virulence_genotype data: ', db.execute('SELECT COUNT(*) FROM isolate WHERE AMR_genotypes IS NULL OR stress_genotypes IS NULL OR virulence_genotype IS NULL').fetchall()[0][0])

## Using SQL, remove the records missing any of the genotype data.

In [ ]:
# db before removing missing genotype data
print('There are', db.execute('SELECT COUNT(*) FROM isolate').fetchall()[0][0], 'records retrieved before removing missing records.')

In [ ]:
%%time
db.execute('DELETE FROM isolate WHERE AMR_genotypes IS NULL OR stress_genotypes IS NULL OR virulence_genotype IS NULL')

In [ ]:
# db after removing missing genotype data
print('There are', db.execute('SELECT COUNT(*) FROM isolate').fetchall()[0][0], 'records retrieved after removing missing records.')

# The AMR, stress, and virulence genes are stored by NCBI in their Reference Gene Catalog, which can be retrieved (in a simplified format, so that `gene_family` is the unique key) as follows. We will use it to map each gene (or allele in the gene) to its gene family.

Note: Unfortunately, updates to the catalog are lagging behind the genes found in the isolates dataset. We will manually update the catalog below, but, for the next task, assume the catalog is up-to-date.

In [ ]:
%%time
gurl = 'https://ftp.ncbi.nlm.nih.gov/pathogen/Antimicrobial_resistance/Data/latest/ReferenceGeneCatalog.txt'
gdf = pd.read_table(gurl, sep='\t')[['allele', 'gene_family', 'type']].drop_duplicates()
gdf = gdf.loc[gdf['gene_family'].notnull() & gdf['type'].notnull()]
gdf.to_sql('gene', con=db, if_exists='replace', index=False)

# ***
# Fix 1: Add entries with allele = gene_family (since some isolates record gene families instead of alleles),
#        whenever they are missing, then remove the entries with allele = NULL
db.execute('INSERT INTO gene (allele, gene_family, type) SELECT DISTINCT gene_family, gene_family, type'\
           + ' FROM gene t WHERE (SELECT COUNT(*) FROM gene WHERE allele = gene_family'\
           + ' AND gene_family = t.gene_family and type = t.type) = 0')
db.execute('DELETE FROM gene WHERE allele IS NULL')
# ***

print('There are', db.execute('SELECT COUNT(*) FROM gene').fetchall()[0][0], 'unique reference genes.')
print('There are', db.execute('SELECT COUNT(DISTINCT(gene_family)) FROM gene').fetchall()[0][0],
      'unique reference gene families.')
print('The set of unique types of genes are:',
      {t for ts in db.execute('SELECT DISTINCT(type) FROM gene').fetchall() for t in ts})
print('There are', db.execute('SELECT COUNT(*) FROM (SELECT allele, COUNT(DISTINCT(type)) FROM gene'\
                              + ' GROUP BY allele HAVING COUNT(DISTINCT(type)) > 1)').fetchall()[0][0],
      'genes of multiple types.')
genemap = {g[0]:g[1] for g in db.execute('SELECT allele, gene_family FROM gene').fetchall()}

# Retrieve the set of unique of `AMR_genotypes`, `stress_genotypes`, and `virulence_genotype` not containing the equals symbol (`=`) as a Python set.

In [ ]:
amr = {g for gss in db.execute('SELECT DISTINCT(AMR_genotypes) FROM isolate').fetchall()
         for gs in gss for g in gs.split(',') if '=' not in g}
print('AMR:      ', amr)
stress = {g for gss in db.execute('SELECT DISTINCT(stress_genotypes) FROM isolate').fetchall()
            for gs in gss for g in gs.split(',') if '=' not in g}
print('stress:   ', stress)
virulence = {g for gss in db.execute('SELECT DISTINCT(virulence_genotype) FROM isolate').fetchall()
               for gs in gss for g in gs.split(',') if '=' not in g}
print('virulence:', virulence)
print('It is', (amr.intersection(stress) == set() & amr.intersection(virulence) == set()
                & stress.intersection(virulence) == set()), 'that these sets are non-intersecting.')

# Update the mapping of AMR, stress, and virulence alleles to their gene families

In [ ]:
%%time
# ***
# Fix 2: Add mapping of "new" alleles to their own gene families
# ***
for g in amr.symmetric_difference(set(genemap.keys()).intersection(amr)):
  db.execute('INSERT INTO gene (allele, gene_family, type) VALUES ("' + g + '","' + g + '","AMR")')
genemap.update({g:g for g in amr.symmetric_difference(set(genemap.keys()).intersection(amr))})
for g in stress.symmetric_difference(set(genemap.keys()).intersection(stress)):
  db.execute('INSERT INTO gene (allele, gene_family, type) VALUES ("' + g + '","' + g + '","STRESS")')
genemap.update({g:g for g in stress.symmetric_difference(set(genemap.keys()).intersection(stress))})
for g in virulence.symmetric_difference(set(genemap.keys()).intersection(virulence)):
  db.execute('INSERT INTO gene (allele, gene_family, type) VALUES ("' + g + '","' + g + '","VIRULENCE")')
genemap.update({g:g for g in virulence.symmetric_difference(set(genemap.keys()).intersection(virulence))})

# ***
# Override: Change the mapping of `mcr-*` gene families to a single `mcr` family
# ***
genemap.update({g:'mcr' for g in set(genemap.keys()) if genemap.get(g).find('mcr') == 0})

# define the set of unique genes and gene families across AMR, stress, and virulence alleles
genes = amr.union(stress).union(virulence)
gene_families = {genemap.get(g) for g in genes}

Create a new `isolate` table in your in-memory database, `isolate_for_pattern_recognition`, where each row represents an isolate, uniquely identified by their `target_acc` (i.e., the `PRIMARY KEY`), and characterized by their `target_creation date` and `gene_family` presence-absence flag (i.e., presence or absense of a `gene_family` in that isolate) that can be analyzed for Knowledge Discovery and Machine Learning.

Note: Only the unique `gene_family` (i.e., **NOT** `allele`) columns present in any of the isolates are included. This new table is not *normalized* (in the database sense), but affords easy analyses for pattern recognition. Remember to execute "`DROP TABLE IF EXISTS isolate_for_pattern_recognition`" before creating the table.

In [ ]:

%%time
# creates table isolate_for_pattern_recognition

c_name = []
c_name.append('target_acc datatype VARCHAR')
c_name.append('target_creation_date datatype DATETIME')
for g in gene_families:
  c_name.append('"'+g+'" datatype NUMERIC')

db.execute('DROP TABLE IF EXISTS isolate_for_pattern_recognition')
db.execute('CREATE TABLE IF NOT EXISTS isolate_for_pattern_recognition(%s);' %", ".join(c for c in c_name))
target = pd.DataFrame(pd.read_sql_query('SELECT target_acc, target_creation_date, AMR_genotypes, stress_genotypes, virulence_genotype from isolate', con=db))
target['all_genes'] = target[['AMR_genotypes', 'stress_genotypes', 'virulence_genotype']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
target.head()
target_columns = ['target_acc', 'target_creation_date']
for i in range(len(target)):
  tar_acc = target['target_acc'].iloc[i]
  tar_date = target['target_creation_date'].iloc[i]
  all_genes = {genemap.get(g) for g in target['all_genes'].iloc[i].split(',') if '=' not in g}
  

  target_values = ['"'+tar_acc+'"', '"'+str(tar_date)+'"']
  genes_present = all_genes.intersection(gene_families) 
  genes_absent = gene_families - genes_present
  ones = [1 for ind in  range(len(genes_present))]
  zeros = [0 for ind in range(len(genes_absent))]
  for c in range(len(genes_present)):
    target_values.append(1)
  for z in range(len(genes_absent)):
    target_values.append(0)
  all_columns = target_columns+list(genes_present)+list(genes_absent)

  db.execute('INSERT INTO isolate_for_pattern_recognition('+", ".join('"'+c+'"' for c in all_columns)+') VALUES ('+",".join(str(v) for v in target_values)+')')
  

# Check that all isolates are recorded in the new table

In [ ]:
%%time
print('There are', db.execute('SELECT COUNT(*) FROM isolate').fetchall()[0][0], 'isolates in the `isolate`',
      'table.')
print('There are', db.execute('SELECT COUNT(*) FROM isolate_for_pattern_recognition').fetchall()[0][0],
      'isolates in the `isolate_for_pattern_recognition` table.')

In the subsequent Tasks, we will use KDD and ML methods to find patterns in the isolates associated with the *Mobile Colistin Resistance* AMR gene family `mcr`. Because panel-testing for `mcr` started after a subset of isolates have been collected and recorded in the data, we must remove those older isolates to avoid biased analyses. For simplicity, we will use the earliest date of `mcr` recorded in the data as the cutoff. Create a new table `isolate_for_mcr` from `isolate_for_pattern_recognition` containing the isolates from the date of first record of `mcr` presence (i.e., `mcr = 1`).

Note: Remember to execute "`DROP TABLE IF EXISTS isolate_for_mcr`" before creating the table.

In [ ]:
print('First recorded date of isolate with mcr: ', db.execute('SELECT MIN(target_creation_date) from isolate_for_pattern_recognition WHERE mcr = 1').fetchall()[0][0])

In [ ]:
%%time
# drops all records before the first recorded date of mcr 
db.execute('DROP TABLE IF EXISTS isolate_for_mcr')
db.execute('CREATE TABLE isolate_for_mcr AS SELECT * FROM isolate_for_pattern_recognition WHERE target_creation_date>=(SELECT MIN(target_creation_date) from isolate_for_pattern_recognition WHERE mcr = 1)')

In [ ]:
print('number of mcr 0 records: ', db.execute('SELECT COUNT(*) FROM isolate_for_mcr  WHERE mcr = 0').fetchall()[0][0])
print('number of mcr 1 records: ', db.execute('SELECT COUNT(*) FROM isolate_for_mcr  WHERE mcr = 1').fetchall()[0][0])

## Association Rules Mining

For this task, we seek to collect all association rules of the form `S => {'mcr'}`, where `S` is a subset of `gene_families` (excluding `mcr`). Display a table or dataframe of all such rules with `support > 0.00005` and `lift > 1` and having the following columns: `antecedent`, `consequent`, `support`, `confidence`, and `lift` in descending order of [`lift`, `support`].

For simplicity, while it is true that whenever `S => {'mcr'}` is a candidate rule, then so is `T => {'mcr'}`, for each `T` subset of `S`, the analysis is restricted to only those **full sets** `S` recorded to have co-occured at least once with `mcr`.

Note: Because we are interested in rules of a specific form and a single `conseqent` (`mcr`), it is best to generate the rules by custom-programming rather than using an association-rules-mining library, which would exhaust RAM and run for a long time enumerating all forms of rules on a large dataset.

---

Note: While the `support` and `lift` metrics are the same for rules `S => {'mcr'}` and `{'mcr'} => S`, their `confidence` metrics are different.

In [ ]:
import pandas as pd
isolate_mcr_df = pd.DataFrame(pd.read_sql_query('SELECT * FROM isolate_for_mcr where mcr==1', con=db))

# drop columns; target_acc and target_creation_date
mcr_df = isolate_mcr_df.drop(columns=['target_acc', 'target_creation_date'])

# delete all gene families which have not co-occured with mcr
for c in mcr_df.columns:
  if mcr_df[c].sum() == 0:
    mcr_df.drop(columns=c, inplace=True)

mcr_df.drop(columns='mcr', inplace=True)

In [ ]:
len_isolate = db.execute('SELECT COUNT(*) FROM isolate_for_mcr').fetchall()[0][0]
isolate_df =  pd.DataFrame(pd.read_sql_query('SELECT * FROM isolate_for_mcr', con=db))
len_mcr = len(mcr_df)
one_df = pd.DataFrame(columns=['antecedent', 'consequent', 'support', 'confidence', 'lift'])
support_consequent = len_mcr/len_isolate
for i in range(len(mcr_df)):
  itemset = []
  for c in mcr_df.columns:
    if mcr_df[c].iloc[i] == 1:
      itemset.append(c)
  # print(itemset)
  sup = {ind for ind in mcr_df.index}
  sup_ant = {ind for ind in isolate_df.index}
  for j in itemset:
    ind_mcr = {ind_m for ind_m in mcr_df.index[mcr_df[j]==1]}
    ind_ant = {ind_a for ind_a in isolate_df.index[isolate_df[j]==1]}
    sup = sup.intersection(ind_mcr)
    sup_ant = sup_ant.intersection(ind_ant)
     
  support = len(sup)/len_isolate
  support_antecedent = len(sup_ant)/len_isolate
  

  confidence = support / support_antecedent
  lift = support / (support_consequent * support_antecedent)
  if support > 0.00005 and lift>1:
    two_itemset = pd.DataFrame({'antecedent': ','.join(str(s) for s in itemset), 'consequent':'mcr', 'support':support_antecedent, 'confidence':confidence, 'lift':lift},  index=[0])
    one_df = one_df.append(two_itemset)
    # print('support: ', support, 'lift: ', lift)
  # break

# __Association Rules in descending order of ['lift', 'support]__


In [ ]:
one_df.drop_duplicates('antecedent').sort_values(['lift', 'support'], ascending=False)

In [ ]:
len(one_df.drop_duplicates('antecedent').sort_values(['lift', 'support'], ascending=False))

---
# Machine Learning (Classification using logistic regression on imbalanced data)
---
We create a logistic regression model to predict the presence or absence of `mcr` from all other `gene_family` columns of isolates. For simplicity, we ignore interactions of two or more `gene_family` columns. To handle the imbalance in the `mcr` column (i.e., only a small fraction of isolates with `mcr` present can be found in the `isolate_for_mcr` dataset), we use the following adjustments:
1.   Change the penalty of misclassifying `mcr = 0` to `penalty = 1` and misclassifying `mcr = 1` to `penalty = 2`, and;
2.   Change the classification threshold for classifying `mcr` to `0.375`.



In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Pull and Convert sql DB to pandas df
logistic_df = pd.DataFrame(pd.read_sql_query('SELECT * FROM isolate_for_mcr', con=db))

# training data
train_data = logistic_df.drop(columns=['target_acc', 'target_creation_date'])
X = train_data.drop(columns='mcr', axis=1)
y = train_data['mcr']

# logistic regression model
model = LogisticRegression(random_state=0, class_weight={0:1, 1:2}, max_iter = 100000).fit(X, y)

# probabilities for mcr=1
preds = model.predict_proba(X)[:,1]

# Change the prediction threshold for classifying `mcr` to `0.375`
predictions = []
for prob in preds:
  if prob>=0.375:
    predictions.append(1)
  else:
    predictions.append(0)


# __Precision and Recall__

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score
print('precision: ', precision_score(y, predictions))
print('recall: ', recall_score(y, predictions))

In [ ]:
# confusion matrix
labels = [1,0]
cm = confusion_matrix(y, predictions, labels=labels)
print(pd.DataFrame(cm, index=labels, columns=labels))

# __Precision-Recall Curve__

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

disp = plot_precision_recall_curve(model, X, y)
plt.axvline(x=recall_score(y, predictions))
plt.axhline(y=precision_score(y, predictions))

# __Odds__

In [ ]:
# oddsratio
import numpy as np
odds_df = pd.DataFrame({'gene_family': X.columns, 'odds':np.exp(model.coef_[0])})
print(odds_df.sort_values('odds', ascending=False))

# For the subsequent Tasks, we will reference the phylogenetic trees of the isolates and derive a modified dataset for survival analyses, where the *death* event is the aquisition of the `mcr` AMR `gene_family`.


In [ ]:
%%writefile tree.newick
 (('PDT000667730.2':0.366551,(('PDT000652466.1':0.313242,('PDT000661222.1':0.202829,'PDT000661229.1':0.225535)'':0.267952)'':0.000160796,(('PDT000297982.1':0.469042,'PDT000358769.1':0.470538)'':0.0184432,((('PDT000652248.1':0.268416,'PDT000483532.1':0.268441)'':0.219572,('PDT000691293.1':0.488259,('PDT000346806.1':0.472798,('PDT000321340.1':0.392952,('PDT000321330.1':0.375034,('PDT000151579.1':0.154709,'PDT000482605.1':0.157155)'':0.219066)'':0.0181568)'':0.0805489)'':0.0158969)'':0.00127472)'':0.00131841,(('PDT000069688.1':0.000774878,('PDT000069687.1':0.000349776,'PDT000069693.1':0.000350154)'':0.000424452)'':0.496319,('PDT000691300.1':0.497659,(('PDT000627348.1':0.498708,'PDT000395120.1':0.498894)'':0.00104101,('PDT000627352.1':0.497273,('PDT000150555.1':0.0521186,('PDT000148572.1':0.0187865,'PDT000150552.1':0.0187422)'':0.0333529)'':0.446299)'':0.00149814)'':0.00116522)'':0.000198786)'':0.00431735)'':0.00227341)'':0.0590061)'':0.0464899)'':-0.0494396,(((('PDT000322044.1':0.135761,'PDT000525472.2':0.13723)'':0.000762254,((('PDT000671874.1':0.111963,('PDT000583493.1':0.0999603,'PDT000239353.1':0.0990507)'':0.0116663)'':0.0109199,(('PDT000365887.1':0.11211,'PDT000564173.1':0.11057)'':0.00360082,('PDT000321573.1':0.0420174,'PDT000155550.2':0.0414787)'':0.0743746)'':0.00662153)'':0.00500047,(('PDT000384204.1':0.00809207,'PDT000389805.1':0.00809663)'':0.0890404,('PDT000385875.1':0.071017,(('PDT000542135.1':0.025881,'PDT000291296.1':0.0292269)'':0.029642,('PDT000239354.1':0.0496702,('PDT000400858.1':0.0402421,'PDT000583494.1':0.0401116)'':0.00939022)'':0.00496373)'':0.0169064)'':0.0254782)'':0.0306901)'':0.012137)'':0.00179996,(('PDT000063329.2':0.122484,('PDT000365893.2':0.0672742,(('PDT000040400.4':0.00645457,'PDT000367213.1':0.00630333)'':0.0100153,('PDT000384207.1':0.00978734,'PDT000384193.1':0.00974856)'':0.00741794)'':0.0503992)'':0.0565439)'':0.023981,((('PDT000316112.1':0.0576887,'PDT000583495.1':0.0568913)'':0.0315915,('PDT000110187.3':0.0846553,('PDT000539865.2':0.024295,('PDT000273196.3':0.00915054,('PDT000107954.3':0.00271291,('PDT000137660.3':0.00267412,'PDT000414218.1':0.00212108)'':0.00047757)'':0.00648977)'':0.0148277)'':0.0655601)'':0.000852096)'':0.0428235,((('PDT000619606.2':0.105989,'PDT000347094.2':0.106986)'':0.0231564,(('PDT000502931.1':0.0572357,'PDT000102227.1':0.0572983)'':0.0632272,('PDT000156599.2':0.0884439,('PDT000671875.1':0.0584589,('PDT000271615.2':0.0464989,('PDT000212608.2':0.0215184,'PDT000400859.1':0.0210558)'':0.02603)'':0.0122234)'':0.0282188)'':0.034759)'':0.00755087)'':-0.00339935,((('PDT000102223.1':0.0766717,'PDT000102224.1':0.0771103)'':0.0294363,('PDT000589651.1':0.04871,('PDT000160520.2':0.042181,('PDT000600204.1':0.0248353,'PDT000414217.1':0.0237704)'':0.0180563)'':0.00847817)'':0.0546933)'':0.0154037,('PDT000293555.1':0.121353,(('PDT000573324.2':0.0531577,('PDT000110189.3':0.0397909,'PDT000102222.1':0.0399157)'':0.0142575)'':0.0331305,('PDT000649260.1':0.0792649,('PDT000102237.1':0.077397,('PDT000583486.1':0.0501767,'PDT000671879.1':0.0516183)'':0.0273305)'':0.00315081)'':0.00492758)'':0.0361957)'':-0.00117997)'':0.00605642)'':0.00596453)'':0.0138812)'':-0.00314685)'':0.0770464,((('PDT000502937.1':0.0783664,('PDT000155588.2':0.0737357,(('PDT000212664.2':0.0555458,'PDT000271623.2':0.0571272)'':0.00924092,(('PDT000132226.2':0.0375089,'PDT000020133.1':0.0365944)'':0.0174609,('PDT000335808.1':0.0477632,('PDT000155132.2':0.027765,'PDT000616646.1':0.0288978)'':0.0187512)'':0.00923844)'':0.0109505)'':0.00739709)'':0.00555165)'':0.115311,(('PDT000619790.1':0.02933,('PDT000130676.2':0.0247586,('PDT000157038.2':0.0131672,'PDT000005521.2':0.0124662)'':0.0130116)'':0.00472668)'':0.162918,(((('PDT000392395.1':0.152095,'PDT000583489.1':0.15083)'':0.0113039,('PDT000102230.1':0.153871,'PDT000253682.1':0.155755)'':0.0120141)'':0.00356173,(('PDT000041492.7':0.0650186,('PDT000281488.1':0.0480638,'PDT000385879.1':0.0479814)'':0.0180643)'':0.0303318,('PDT000589652.1':0.0540506,('PDT000589649.1':0.0282926,'PDT000291293.1':0.0306868)'':0.0259817)'':0.0405055)'':0.0713475)'':0.00915968,(('PDT000414863.1':0.0459863,('PDT000161895.2':0.0200756,'PDT000271622.2':0.0207749)'':0.0258676)'':0.141787,('PDT000600071.1':0.174121,((('PDT000347095.2':0.0587665,('PDT000131845.2':0.0434685,('PDT000010909.2':0.00926379,('PDT000420980.1':0.00581447,'PDT000714286.1':0.00544273)'':0.00242541)'':0.0353066)'':0.0147847)'':0.107091,('PDT000662634.1':0.164936,('PDT000102215.1':0.15985,('PDT000102232.1':0.153023,('PDT000619772.1':0.146485,'PDT000502929.1':0.147283)'':0.00732749)'':0.00852367)'':0.00283021)'':0.00235119)'':0.00502723,((('PDT000502933.1':0.0564908,'PDT000102236.1':0.0590032)'':0.0460473,('PDT000502935.1':0.090929,'PDT000102233.1':0.091267)'':0.0146957)'':0.0127697,(('PDT000589648.1':0.0318527,('PDT000589650.1':0.0220809,('PDT000199182.2':0.0196556,'PDT000589653.1':0.0208199)'':0.00102683)'':0.0100664)'':0.0496602,(('PDT000404097.1':0.0567275,('PDT000357120.1':0.0315677,'PDT000600203.1':0.0315448)'':0.0273682)'':0.0141335,('PDT000484175.1':0.0143555,('PDT000063342.3':0.0106675,('PDT000442718.1':0.00397086,'PDT000347093.2':0.00391495)'':0.00641896)'':0.00363656)'':0.0569217)'':0.0196037)'':0.0260787)'':0.0545166)'':0.0115283)'':0.00408303)'':-0.00927377)'':0.0153173)'':7.38252e-05)'':0.00558282,((('PDT000304205.1':0.172034,('PDT000502934.1':0.156659,'PDT000291291.1':0.158388)'':0.014506)'':0.0127143,(('PDT000212647.2':0.0919002,('PDT000583491.1':0.0339254,('PDT000249002.3':0.028998,('PDT000502930.1':0.0246983,'PDT000502932.1':0.0240579)'':0.00405354)'':0.00570845)'':0.0553483)'':0.0902703,(((('PDT000355603.1':0.0495118,'PDT000417622.1':0.0478092)'':0.0800682,('PDT000102213.1':0.0877961,'PDT000291292.1':0.0918359)'':0.0422163)'':0.0382718,('PDT000271616.2':0.169142,('PDT000102229.1':0.102905,(('PDT000063328.2':0.0609557,'PDT000441813.1':0.0592973)'':0.0227672,(('PDT000212656.2':0.0150126,'PDT000083569.1':0.0151971)'':0.029611,('PDT000325239.1':0.0343419,'PDT000102214.1':0.0339102)'':0.01107)'':0.0365297)'':0.0160399)'':0.0680017)'':-0.00151012)'':0.00200545,((('PDT000010913.2':0.147452,('PDT000502939.1':0.134091,('PDT000102234.1':0.106579,('PDT000525469.1':0.100882,'PDT000671878.1':0.101965)'':0.00452348)'':0.030264)'':0.00912071)'':0.0176893,(('PDT000583487.1':0.146302,'PDT000205455.1':0.147473)'':0.0135674,('PDT000379490.1':0.110987,('PDT000583492.1':0.0554163,'PDT000327322.1':0.0578567)'':0.052122)'':0.0502894)'':0.00352611)'':0.00369637,(('PDT000502936.1':0.170654,('PDT000367783.2':0.109232,'PDT000102220.1':0.109016)'':0.0624252)'':-0.00165814,('PDT000583488.1':0.162351,('PDT000261050.1':0.0914767,(('PDT000213263.2':0.0670412,'PDT000327245.1':0.0663298)'':0.00396919,('PDT000316198.1':0.0452012,('PDT000102235.1':0.0246697,'PDT000380169.1':0.0235219)'':0.021703)'':0.0264894)'':0.0225166)'':0.0685977)'':0.00426635)'':-0.00119092)'':0.0023385)'':0.010405)'':0.00177381)'':0.00863022,(('PDT000063343.2':0.0122576,('PDT000212578.2':0.00920817,('PDT000271621.2':0.00725709,'PDT000115010.2':0.00710271)'':0.00297933)'':0.00208639)'':0.183144,(('PDT000327236.1':0.0934162,'PDT000352693.1':0.0828168)'':0.0924574,((('PDT000271618.2':0.118777,'PDT000102228.1':0.118526)'':0.0412592,('PDT000271608.2':0.0777135,('PDT000102218.1':0.0252384,('PDT000502928.1':0.0131379,'PDT000502938.1':0.0131101)'':0.0103367)'':0.0535456)'':0.0817928)'':0.0170247,(('PDT000102226.1':0.0563079,'PDT000291298.1':0.0571451)'':0.119593,('PDT000583490.1':0.186323,(('PDT000131848.2':0.162212,'PDT000553641.1':0.161446)'':0.0116242,((('PDT000563648.1':0.127408,'PDT000583485.1':0.128104)'':0.0339354,('PDT000671877.1':0.144043,('PDT000678426.1':0.0798174,'PDT000102225.1':0.0815826)'':0.0625431)'':0.0178892)'':0.00436243,('PDT000108009.2':0.170417,(('PDT000430674.1':0.0599501,'PDT000473802.1':0.0575149)'':0.0946079,(('PDT000063330.2':0.116548,(('PDT000355594.1':0.0664393,'PDT000102217.1':0.0646517)'':0.0269012,(('PDT000106119.1':0.0778907,('PDT000102231.1':0.0610759,('PDT000102216.1':0.0295764,'PDT000102238.1':0.0297506)'':0.0325196)'':0.0168726)'':-0.00466838,('PDT000619753.1':0.0682155,('PDT000287367.2':0.0423932,('PDT000291295.1':0.0169263,'PDT000291297.1':0.0157554)'':0.0262108)'':0.024646)'':0.00612611)'':0.0185411)'':0.0244614)'':-0.00119983,(('PDT000430315.1':0.0987836,('PDT000102221.1':0.079994,'PDT000291294.1':0.080702)'':0.0218034)'':0.00865292,('PDT000063181.2':0.095709,('PDT000539909.1':0.0934415,('PDT000277855.1':0.070253,('PDT000110188.3':0.065569,('PDT000600895.1':0.0443028,('PDT000671873.1':0.0361209,('PDT000156952.2':0.0191806,'PDT000102219.1':0.0196935)'':0.0168249)'':0.00679811)'':0.0215533)'':0.00677795)'':0.0204807)'':0.00690801)'':0.00838585)'':0.00750836)'':0.0412038)'':0.0137657)'':-0.00384649)'':0.00706839)'':0.0109128)'':-0.00701446)'':-0.000840348)'':0.0102708)'':0.0114522)'':-0.00695331)'':0.00749617)'':0.0263436)'':0.233414,('PDT000468227.1':0.399462,((((('PDT000510884.1':0.187215,('PDT000400852.1':0.0827532,'PDT000631955.1':0.0819618)'':0.103644)'':0.0479304,(('PDT000132146.2':0.197761,('PDT000015168.2':0.174518,'PDT000017821.2':0.176598)'':0.0199075)'':0.00453417,(('PDT000494565.1':0.010157,'PDT000509745.1':0.0101133)'':0.173903,(('PDT000343597.1':0.0180973,'PDT000510866.1':0.0176135)'':0.13408,(('PDT000322990.1':0.0469442,'PDT000238221.2':0.0491461)'':0.0142621,('PDT000405295.1':0.05595,'PDT000127569.1':0.054877)'':0.00861498)'':0.0912629)'':0.0269296)'':0.0193076)'':0.0297881)'':0.0827972,(('PDT000500749.1':0.170754,('PDT000137882.2':0.163547,('PDT000494521.1':0.164668,('PDT000430710.1':0.154178,'PDT000583886.1':0.157595)'':0.0075785)'':0.00199885)'':0.00494597)'':-0.0014794,(('PDT000438203.1':0.00651651,'PDT000024641.1':0.00585629)'':0.14973,(('PDT000626906.1':0.15276,('PDT000205451.1':0.0419553,('PDT000539295.1':0.0221016,'PDT000502881.1':0.0207753)'':0.0193347)'':0.106605)'':-0.00464245,('PDT000687922.1':0.073526,('PDT000510864.1':0.043739,('PDT000107696.1':0.0250492,'PDT000318987.1':0.0258297)'':0.0141722)'':0.0304538)'':0.0718451)'':0.0136406)'':0.0106344)'':0.137932)'':0.0929797,((('PDT000310639.2':0.391504,'PDT000487926.1':0.362561)'':0.0344354,('PDT000714684.1':0.174628,'PDT000718452.1':0.17373)'':0.239647)'':0.0104101,(('PDT000167711.2':0.0791889,'PDT000619781.1':0.0787541)'':0.0542503,('PDT000714030.1':0.14532,(('PDT000510898.1':0.13397,(('PDT000714017.1':0.11059,('PDT000161556.2':0.0487966,'PDT000161562.2':0.050936)'':0.0615963)'':0.00324173,('PDT000395166.1':0.108307,('PDT000714146.1':0.0986699,('PDT000661584.1':0.0936315,'PDT000362470.1':0.0951225)'':0.00494107)'':0.00843789)'':0.00812946)'':0.0148975)'':-0.00994113,(('PDT000129808.2':0.0809062,('PDT000354949.1':0.0792413,'PDT000641917.1':0.0791697)'':0.00163577)'':0.0410084,('PDT000167689.2':0.123308,(('PDT000354873.1':0.0696367,'PDT000372048.1':0.0685753)'':0.0453406,(('PDT000354871.1':0.0962378,'PDT000648054.1':0.0944972)'':0.0144295,(('PDT000310626.2':0.068689,'PDT000063366.2':0.066473)'':0.040729,(('PDT000470057.1':0.0537793,('PDT000354906.1':0.0238419,'PDT000413147.1':0.0245246)'':0.0264824)'':0.0551318,('PDT000619721.1':0.10433,('PDT000619566.1':0.0990011,('PDT000161970.2':0.0600388,('PDT000713918.1':0.0375903,('PDT000324093.1':0.0316552,'PDT000162002.2':0.0307467)'':0.00722081)'':0.0204839)'':0.0403058)'':0.00787009)'':-0.000141126)'':0.00134936)'':0.00198149)'':0.00409416)'':0.0147674)'':-0.00782375)'':-0.00213105)'':0.0209582)'':-0.000416731)'':0.315978)'':-0.00653649)'':0.00977955,(('PDT000538550.1':0.307975,((('PDT000620000.1':0.0542659,'PDT000620004.1':0.0544591)'':0.150032,('PDT000066126.2':0.17882,(('PDT000619999.1':0.137374,'PDT000052585.2':0.137222)'':0.00933219,('PDT000620006.1':0.0763679,('PDT000571827.1':0.0396204,'PDT000291290.1':0.0412967)'':0.037557)'':0.0692956)'':0.0301683)'':0.0240723)'':0.00359253,('PDT000417621.2':0.215631,(('PDT000620014.1':0.169709,(('PDT000290002.1':0.0779208,('PDT000329030.1':0.0578245,'PDT000667964.1':0.0577445)'':0.0185637)'':0.0302882,('PDT000417157.1':0.000310296,('PDT000417155.1':0.000200901,'PDT000417156.1':0.000199019)'':8.96493e-05)'':0.103312)'':0.0627991)'':0.024273,('PDT000620007.1':0.208917,(('PDT000088299.2':0.056745,'PDT000620005.1':0.05608)'':0.143343,((('PDT000088302.2':0.0815602,'PDT000290003.1':0.0853388)'':0.00706405,('PDT000619998.1':0.061625,'PDT000620018.1':0.060015)'':0.029745)'':0.101961,(('PDT000328662.1':0.18625,('PDT000328701.1':0.113983,'PDT000652879.1':0.111922)'':0.0768543)'':-0.000237159,(('PDT000373300.1':0.122291,'PDT000488328.1':0.12425)'':0.0658193,('PDT000620001.1':0.192991,(('PDT000417620.1':0.0792608,('PDT000620012.1':0.068051,'PDT000052586.2':0.064953)'':0.0127427)'':0.101916,(('PDT000010190.2':0.0728493,'PDT000291289.1':0.0760727)'':0.099868,((('PDT000212669.2':0.0476772,'PDT000620010.1':0.0468981)'':0.127653,('PDT000620015.1':0.1625,('PDT000066123.2':0.00222107,'PDT000571825.1':0.00247893)'':0.158115)'':0.0123742)'':-5.60973e-06,(('PDT000052584.2':0.168794,('PDT000620003.1':0.0722327,(('PDT000571826.1':0.00496022,('PDT000066121.2':0.00341101,'PDT000066127.2':0.0031824)'':0.0014105)'':0.0472782,('PDT000008596.2':0.0559576,('PDT000620017.1':0.0460023,('PDT000253348.2':0.0417056,'PDT000620021.1':0.0424604)'':0.0040215)'':0.0100443)'':-0.00197917)'':0.0191325)'':0.0938248)'':7.58106e-05,(('PDT000620008.1':0.000403761,'PDT000620020.1':0.000296099)'':0.17022,(('PDT000620002.1':0.145044,'PDT000620019.1':0.144653)'':0.0213475,(('PDT000620009.1':0.128961,'PDT000620016.1':0.128295)'':0.0301643,('PDT000620013.1':0.132944,('PDT000357564.1':0.00267152,(('PDT000066124.2':0.000703248,'PDT000066125.2':0.000695912)'':0.000324465,(('PDT000271613.2':0,'PDT000271624.2':0)'':0.000636195,('PDT000570873.1':0,'PDT000005193.2':0)'':0.000563565)'':0.000525687)'':0.00126029)'':0.132707)'':0.0224468)'':0.00897771)'':0.0025703)'':-0.00250272)'':0.00691719)'':0.00275495)'':0.0035689)'':0.0111636)'':-0.00186542)'':-0.00183356)'':0.00415211)'':0.00611086)'':0.0142651)'':-0.0169857)'':0.0167154)'':-0.00304731)'':0.143158)'':0.0949423,((((('PDT000029848.1':0.0992337,('PDT000618891.1':0.0915195,('PDT000125551.2':0.0495126,'PDT000140740.2':0.0529624)'':0.0371085)'':0.00967849)'':0.0465576,(('PDT000094202.2':0.0375016,'PDT000353179.2':0.0390275)'':0.0995502,((('PDT000305500.2':0.0732962,'PDT000460494.1':0.0767268)'':0.0152843,(('PDT000128809.2':0.0542209,'PDT000565872.1':0.0491251)'':0.0193879,(('PDT000397636.1':0.0436121,'PDT000566203.1':0.0437806)'':0.0119593,('PDT000550402.1':0.0248382,'PDT000644865.1':0.0252059)'':0.0317159)'':0.0189381)'':0.0158193)'':0.0183058,((('PDT000128793.2':0.0668859,'PDT000215766.2':0.0612831)'':0.00844131,('PDT000395414.1':0.0717409,('PDT000167695.2':0.0451241,('PDT000444819.1':0.0386906,'PDT000504755.1':0.0379289)'':0.00871853)'':0.0189876)'':0.00246071)'':0.0123338,(('PDT000652943.1':0.0456237,'PDT000661628.1':0.0433745)'':0.01834,(('PDT000283180.1':0.051324,('PDT000551876.1':0.0480361,('PDT000394975.1':0.0353411,'PDT000417486.1':0.0331268)'':0.011606)'':0.0120551)'':0.000361807,('PDT000710217.1':0.044313,('PDT000604845.1':0.0398892,('PDT000550364.1':0.0285367,'PDT000645128.1':0.0250242)'':0.0144433)'':0.00873975)'':0.0125018)'':0.0084851)'':0.0209893)'':0.0157397)'':0.0317976)'':0.0203889)'':0.089804,((('PDT000356786.1':0.126608,('PDT000538399.1':0.103455,'PDT000646906.1':0.0950995)'':0.0293053)'':0.0952017,('PDT000472553.1':0.230798,('PDT000713279.1':0.200651,(('PDT000129873.2':0.168165,'PDT000221036.1':0.161259)'':0.0213355,('PDT000032338.1':0.0354784,('PDT000332841.1':0.0326295,'PDT000484692.1':0.0320712)'':0.0056064)'':0.141086)'':0.0230707)'':0.0232406)'':0.00277304)'':-0.00410536,((('PDT000664962.1':0.0487136,'PDT000671868.1':0.0485192)'':0.0187937,('PDT000664977.1':0.0505465,('PDT000196656.2':0.041575,'PDT000661536.1':0.0426193)'':0.00384689)'':0.0197757)'':0.111052,((('PDT000156606.3':0.0208144,('PDT000305220.3':0.0135536,'PDT000313390.3':0.0150404)'':0.00293839)'':0.0711957,('PDT000616679.2':0.0598186,('PDT000682956.1':0.0145416,('PDT000213243.3':0.00913628,('PDT000120879.3':0.00559838,'PDT000120904.3':0.00524852)'':0.00278277)'':0.00452885)'':0.0448672)'':0.0428325)'':0.00595563,(('PDT000213202.2':0.0824262,(('PDT000129850.2':0.058032,('PDT000067867.1':0.05406,('PDT000498868.1':0.0371395,('PDT000387575.2':0.0342556,'PDT000097333.3':0.034074)'':0.00232673)'':0.0100033)'':0.0158378)'':-0.00429678,('PDT000133751.3':0.0638772,('PDT000300547.1':0.0522247,(('PDT000359236.2':0.0200353,('PDT000283342.3':0.0128036,'PDT000081123.4':0.0134314)'':0.00961787)'':0.00757546,('PDT000494271.2':0.021812,('PDT000235280.3':0.0222201,('PDT000254238.3':0.0180748,('PDT000414239.2':0.0142425,'PDT000569703.2':0.0156568)'':0.00280846)'':0.00132665)'':0.00473911)'':0.00528616)'':0.0234681)'':0.00838159)'':-0.000125796)'':0.0333219)'':-0.0113531,(('PDT000400867.2':0.0464674,'PDT000518589.2':0.0425963)'':0.0478004,(('PDT000408015.1':0.0499166,('PDT000340985.1':0.0376145,'PDT000104639.2':0.0351305)'':0.0138843)'':0.0248595,(('PDT000664978.1':0.0630499,'PDT000664987.1':0.0675531)'':0.0118286,(('PDT000218061.2':0.0712255,'PDT000557183.1':0.0635495)'':0.0110305,('PDT000003955.2':0.0742154,(('PDT000150311.2':0.0413433,'PDT000655799.1':0.0449511)'':0.00301505,(('PDT000266406.2':0.0369406,'PDT000653439.1':0.0396098)'':0.00465737,('PDT000241656.2':0.0362025,('PDT000455210.1':0.0257784,'PDT000668404.2':0.0289391)'':0.00564104)'':0.0129676)'':0.00311619)'':0.025076)'':-0.000316038)'':0.00118077)'':0.00665832)'':0.0159125)'':-0.00958051)'':0.0285712)'':0.0646834)'':0.038447)'':0.0130491)'':0.00284596,((((('PDT000025502.2':0.0498445,'PDT000497402.1':0.0414658)'':0.119087,('PDT000395359.1':0.080895,'PDT000399993.1':0.081722)'':0.0756115)'':0.0514618,('PDT000360978.1':0.221468,(('PDT000129848.2':0.0389594,'PDT000672203.1':0.0404348)'':0.159716,('PDT000672180.1':0.17216,('PDT000026907.2':0.165462,'PDT000046607.1':0.167062)'':0.0102353)'':0.022978)'':0.017792)'':-0.00607286)'':0.0174782,(('PDT000270492.1':0.0786246,('PDT000638798.2':0.0332852,'PDT000114996.3':0.0325235)'':0.0343096)'':0.0970408,(('PDT000081259.1':0.0631727,('PDT000273038.3':0.0244226,'PDT000498969.2':0.0265908)'':0.0422691)'':0.021157,(('PDT000501712.2':0.0535315,('PDT000500720.1':0.0530788,'PDT000672697.1':0.0475532)'':0.006307)'':0.0162669,(('PDT000619312.2':0.0307635,'PDT000008713.2':0.0292248)'':0.0355562,('PDT000240678.3':0.0706438,('PDT000008697.2':0.054575,('PDT000378566.2':0.0642475,('PDT000132549.1':0.0507978,('PDT000486507.1':0.0407865,('PDT000008595.2':0.036037,'PDT000252186.1':0.0402682)'':0.00261154)'':0.0071076)'':0.0101297)'':0.00154008)'':0.00564265)'':-0.00424532)'':0.00348917)'':0.0217094)'':0.0779265)'':0.0657465)'':0.00607613,((('PDT000617810.1':0.0870987,'PDT000457698.1':0.0964503)'':0.0140851,(('PDT000232213.2':0.0813812,'PDT000682548.1':0.0773718)'':0.0172183,('PDT000173938.3':0.0438538,('PDT000527189.2':0.0183521,'PDT000570308.2':0.0195988)'':0.0222017)'':0.053668)'':0.015153)'':0.00417435,(('PDT000584997.2':0.0302856,('PDT000366112.2':0.0257361,('PDT000234851.3':0.0144012,'PDT000344389.2':0.0116374)'':0.0142789)'':0.00992064)'':0.0862296,(('PDT000443386.1':0.0943388,('PDT000192039.3':0.0853461,('PDT000187874.3':0.0230675,('PDT000326081.2':0.0157081,'PDT000678189.1':0.0152706)'':0.00923649)'':0.0557082)'':0.0122852)'':0.0123179,((('PDT000557324.1':0.0784775,'PDT000616525.1':0.0761685)'':0.0199961,('PDT000306435.2':0.0895601,('PDT000565936.1':0.0678796,('PDT000351847.1':0.0261866,'PDT000522685.1':0.0260246)'':0.0379098)'':0.0177918)'':0.00396204)'':0.00531739,((('PDT000426710.1':0.0430839,('PDT000174362.1':0.0361533,('PDT000174377.1':0.0261118,('PDT000625511.1':0.0132407,('PDT000155660.2':0.0107258,'PDT000174311.1':0.0102601)'':0.00216935)'':0.00984174)'':0.00705514)'':0.0163036)'':0.0213086,('PDT000603066.2':0.0792577,(('PDT000174304.1':0.031883,'PDT000174380.1':0.0283697)'':0.00347848,('PDT000008592.2':0.0293229,('PDT000174324.1':0.0225919,('PDT000174308.1':0.014732,'PDT000174370.1':0.0124992)'':0.00829542)'':0.00335344)'':0.00299985)'':0.056047)'':-0.0135857)'':0.0179945,('PDT000690319.1':0.0785791,(('PDT000672643.1':0.028533,('PDT000494257.2':0.0291419,('PDT000709713.1':0.0233841,('PDT000137731.3':0.0144103,('PDT000162959.4':0.012283,'PDT000140213.3':0.0142753)'':0.00232444)'':0.00454497)'':0.00555936)'':0.00188889)'':0.035,(('PDT000051931.3':0.0262455,(('PDT000414366.2':0.0203767,'PDT000687646.1':0.0207675)'':0.00259039,('PDT000549342.2':0.0184998,'PDT000101048.3':0.019557)'':0.00226084)'':0.00189144)'':0.00221799,(('PDT000687890.1':0.0294628,('PDT000721697.1':0.0251501,('PDT000594851.2':0.0203208,(('PDT000492134.2':0.0132054,('PDT000492047.2':0.0112329,'PDT000124997.3':0.0119607)'':0.000464448)'':0.00170522,('PDT000190986.3':0.00902786,('PDT000156441.3':0.00853965,('PDT000296063.3':0.00779519,'PDT000101108.3':0.00780031)'':0.000479698)'':0.000461553)'':0.00552554)'':0.00610325)'':0.00250972)'':0.00380048)'':-0.0033091,(('PDT000498881.2':0.0232145,'PDT000098725.3':0.0201011)'':0.00134017,(('PDT000503998.2':0.021492,('PDT000548233.2':0.016096,('PDT000421978.2':0.0128553,'PDT000533999.2':0.0115616)'':0.00402025)'':0.00238031)'':-0.000527822,('PDT000631686.2':0.0281752,(('PDT000409278.2':0.013512,('PDT000635970.2':0.0063726,'PDT000132435.3':0.0072742)'':0.00688927)'':0.00264286,(('PDT000412835.2':0.020319,(('PDT000273855.3':0.00821357,'PDT000140544.3':0.00795703)'':0.00343276,('PDT000492129.2':0.0134696,('PDT000141121.3':0.0106501,('PDT000140535.3':0.0103695,('PDT000190143.3':0.00635477,'PDT000295861.3':0.00618883)'':0.00219603)'':0.00144231)'':0.000973676)'':-0.000435415)'':0.00768006)'':-0.00506728,(('PDT000360406.2':0.00833537,'PDT000590277.2':0.00922413)'':0.0049816,('PDT000499186.2':0.0133759,('PDT000627553.2':0.0124824,(('PDT000147483.3':0.00796256,'PDT000672393.1':0.00934224)'':0.00166894,('PDT000120847.3':0.00617992,('PDT000190664.3':0.00247633,'PDT000687901.1':0.0023184)'':0.00325031)'':0.00320557)'':0.00173973)'':0.00118903)'':0.000628237)'':0.00140717)'':0.00140156)'':0.00680047)'':-0.00355884)'':0.00454534)'':-0.00198657)'':0.00305888)'':0.037191)'':0.00571454)'':0.0165236)'':0.00282539)'':0.0146112)'':0.00226503)'':0.00305729)'':0.118035)'':0.0115692)'':0.0981518,((((((('PDT000005199.2':0.0521483,('PDT000358774.1':0.0466785,('PDT000155565.2':0.0483246,('PDT000250129.1':0.0376109,('PDT000354212.1':0.0347292,('PDT000448197.1':0.0333988,('PDT000397861.1':0.0312015,('PDT000279226.2':0.0261063,('PDT000354171.1':0.0263126,('PDT000713417.1':0.0182191,('PDT000017672.2':0.0180743,('PDT000438084.1':0.0171284,'PDT000249838.1':0.0167933)'':0.0011004)'':0.00168302)'':0.00289639)'':0.00469484)'':0.00212884)'':0.00209887)'':0.000583739)'':0.00234178)'':0.00595481)'':0.00370064)'':0.00856211)'':-0.0130685,('PDT000713222.1':0.0498341,((('PDT000444052.1':0.0238744,'PDT000588050.1':0.0209444)'':0.00945076,('PDT000426020.1':0.0227722,'PDT000380900.1':0.0221969)'':0.00946877)'':0.00443084,('PDT000042507.1':0.0412733,('PDT000333932.2':0.0361111,('PDT000661349.1':0.0297419,('PDT000125988.1':0.0281939,('PDT000443076.1':0.0253559,(('PDT000535139.1':0.015735,'PDT000588005.1':0.0144574)'':0.00367412,('PDT000447363.1':0.0188819,('PDT000132062.2':0.0127667,'PDT000438088.1':0.0152818)'':0.00365603)'':0.000932952)'':0.00722578)'':0.00178497)'':0.00212637)'':0.00485173)'':0.00120387)'':-0.00180963)'':0.0116484)'':-0.00686028)'':0.00681087,(('PDT000661398.1':0.04955,'PDT000362443.1':0.056765)'':0.00284857,('PDT000218360.1':0.058112,(('PDT000631785.1':0.0172048,('PDT000631765.1':0.0107152,'PDT000631767.1':0.01341)'':0.00474123)'':0.0282016,('PDT000017919.2':0.0580444,(('PDT000249782.1':0.0357204,('PDT000661803.1':0.0292145,'PDT000661812.1':0.0261335)'':0.00517581)'':0.00273719,((('PDT000251543.1':0.0312735,'PDT000278768.2':0.0307586)'':0.0040385,(('PDT000249771.1':0.0220875,'PDT000602893.1':0.0270928)'':0.00738141,('PDT000714046.1':0.0246395,('PDT000007691.2':0.0158483,('PDT000354172.1':0.0140767,'PDT000357612.1':0.0165758)'':0.000264267)'':0.00785207)'':0.00475536)'':0.00419277)'':0.0027835,(('PDT000427017.1':0.0292908,'PDT000673496.1':0.0277572)'':0.00652403,('PDT000636382.1':0.042908,(('PDT000397566.1':0.0249623,('PDT000438115.1':0.00870979,'PDT000042718.1':0.00781751)'':0.0159341)'':0.00494905,(('PDT000086181.2':0.0194198,'PDT000103070.1':0.020852)'':0.00598785,('PDT000427756.1':0.0210367,('PDT000427679.1':0.0178149,'PDT000694439.1':0.0165419)'':0.00485395)'':0.00409287)'':0.00150445)'':0.00655399)'':-0.0023963)'':0.0023255)'':0.00100096)'':0.0125256)'':-0.00634111)'':0.0065482)'':-0.00010102)'':-0.00299431)'':0.161213,(('PDT000718895.1':0.157338,(('PDT000015329.2':0.0132969,('PDT000088271.2':0.0116161,'PDT000088294.2':0.00967536)'':0.00653773)'':0.0887778,((('PDT000427543.1':0.034023,'PDT000357566.1':0.0353261)'':0.00664503,('PDT000092533.1':0.0251066,'PDT000495421.1':0.0283024)'':0.0151849)'':0.0255628,((('PDT000317727.1':0.0311474,'PDT000155246.2':0.0352991)'':0.0147743,('PDT000238192.2':0.0311046,('PDT000713472.1':0.019895,'PDT000015301.2':0.0211754)'':0.00342777)'':0.0215516)'':0.00436536,('PDT000557770.1':0.0562216,(('PDT000157101.2':0.0278376,'PDT000325723.1':0.0313038)'':0.0146881,('PDT000006419.2':0.0488348,('PDT000426014.1':0.0376399,('PDT000615965.1':0.0273966,'PDT000111112.1':0.0247688)'':0.00397807)'':0.00744983)'':0.00395162)'':0.0103622)'':-0.00416672)'':0.0198371)'':0.0393639)'':0.0256959)'':-0.0294191,((('PDT000357587.1':0.0443121,(('PDT000155924.2':0.012965,'PDT000312569.2':0.013147)'':0.00496813,('PDT000145735.1':0.0193147,('PDT000132268.2':0.0125489,'PDT000428316.1':0.0118099)'':0.00347347)'':0.00340278)'':0.0170844)'':0.029929,('PDT000115050.2':0.0740399,('PDT000707844.1':0.0625415,('PDT000213210.2':0.0625948,(('PDT000160693.2':0.0518581,(('PDT000160697.2':0.0403539,('PDT000160918.2':0.0289097,('PDT000160957.2':0.0130553,'PDT000161006.2':0.0141859)'':0.0104422)'':0.00645052)'':-0.00150194,('PDT000357582.1':0.0457542,(('PDT000534855.1':0.0186482,('PDT000155863.2':0.0177893,'PDT000015297.2':0.0184347)'':0.0007251)'':0.00314353,('PDT000667977.1':0.0263294,('PDT000157248.2':0.0193628,('PDT000160917.2':0.0161553,('PDT000161075.2':0.0118006,('PDT000160951.2':0.0122061,'PDT000161068.2':0.00783577)'':0.000930478)'':0.00441272)'':0.00508909)'':0.00179957)'':-0.00117551)'':0.0132673)'':-0.00333034)'':0.0161299)'':-0.013526,((('PDT000448209.1':0.0302061,('PDT000490347.1':0.0195337,('PDT000120955.2':0.0179564,'PDT000134074.2':0.0136434)'':0.00740476)'':0.00491694)'':0.00233516,('PDT000621333.1':0.0241953,('PDT000132084.2':0.0203848,('PDT000439272.1':0.0145986,('PDT000471171.1':0.0122711,'PDT000384168.1':0.0103092)'':0.00272802)'':0.00371111)'':0.0032349)'':0.0118373)'':0.00323147,(('PDT000519309.1':0.0383625,('PDT000687100.1':0.0175474,('PDT000132254.2':0.0191337,(('PDT000157716.2':0.0107342,'PDT000156771.2':0.00886187)'':0.00135734,('PDT000324089.1':0.00763196,'PDT000327939.1':0.00724874)'':0.00305861)'':0.00497712)'':0.00220598)'':0.0148535)'':-0.007427,('PDT000628605.1':0.032833,('PDT000327956.1':0.0337133,(('PDT000713950.1':0.0238182,('PDT000711119.1':0.013514,'PDT000661440.1':0.0130839)'':0.0133731)'':0.000728193,(('PDT000132091.2':0.0169749,'PDT000444104.1':0.0151845)'':0.00499759,('PDT000713612.1':0.0191686,'PDT000661696.1':0.0233804)'':0.000498206)'':0.0039466)'':0.00570462)'':0.00202501)'':-0.00454483)'':0.00497218)'':0.00308523)'':0.0200778)'':0.00175776)'':0.00689136)'':0.00286663)'':0.0329798,(('PDT000346794.1':0.078497,(('PDT000131824.2':0.0529774,(('PDT000427550.1':0.0325482,('PDT000713241.1':0.0235671,'PDT000262734.1':0.0255908)'':0.00673356)'':0.00690548,('PDT000380168.1':0.0428851,('PDT000501965.1':0.0221287,'PDT000354217.1':0.0228129)'':0.013333)'':-0.000313991)'':0.015287)'':-0.00803503,(('PDT000307420.2':0.0433628,'PDT000017693.2':0.046961)'':0.0070339,(('PDT000015302.2':0.0469041,('PDT000427803.1':0.0342794,('PDT000318850.1':0.0253358,('PDT000694502.1':0.0182008,('PDT000713582.1':0.0109336,'PDT000713939.1':0.0125993)'':0.00919386)'':0.0051359)'':0.00827409)'':0.00500699)'':0.00447275,(('PDT000011776.2':0.0316554,('PDT000380334.1':0.0365322,('PDT000713431.1':0.0346565,(('PDT000661534.1':0.0254005,('PDT000661558.1':0.0177312,'PDT000015323.2':0.0181171)'':0.00634863)'':0.00178647,('PDT000574937.1':0.0301224,('PDT000714000.1':0.0210469,('PDT000713798.1':0.0191874,'PDT000713865.1':0.0153261)'':0.00193819)'':0.00553994)'':-0.000523803)'':0.00882898)'':0.00238784)'':0.00382016)'':-0.000494655,(('PDT000672259.1':0.0297006,('PDT000466909.1':0.0221624,'PDT000360994.1':0.0260923)'':0.00363668)'':0.00795993,('PDT000661593.1':0.0452195,('PDT000324041.1':0.0345225,('PDT000571161.1':0.0343727,('PDT000362487.1':0.0269053,('PDT000713204.1':0.027422,('PDT000211039.2':0.0237355,('PDT000316487.1':0.0225475,'PDT000694454.1':0.0241504)'':0.00135183)'':0.00227188)'':0.00285666)'':0.0017436)'':0.00101235)'':0.00799724)'':-0.00229957)'':-0.000412455)'':0.00443353)'':0.00396978)'':2.21667e-06)'':0.0366402)'':-0.0274284,(('PDT000132553.1':0.0787429,(('PDT000619614.1':0.0415478,'PDT000017862.2':0.0399229)'':0.0082179,(('PDT000332740.2':0.0446323,('PDT000673644.1':0.0277498,'PDT000695517.1':0.0249591)'':0.0181923)'':-0.00132383,('PDT000655571.1':0.0441317,('PDT000357580.1':0.041753,('PDT000130952.2':0.0324049,'PDT000619556.1':0.0316021)'':0.00353872)'':0.00425431)'':0.00336765)'':0.00832475)'':0.0155622)'':-0.0141831,('PDT000443047.1':0.0748954,(('PDT000619613.1':0.0463255,'PDT000438169.1':0.0428777)'':0.00371721,(('PDT000161702.3':0.0426055,'PDT000354174.1':0.0376563)'':0.0103052,(('PDT000715866.1':0.0495899,('PDT000661387.1':0.0365007,'PDT000017887.2':0.0288408)'':0.00890591)'':0.000121486,('PDT000491593.1':0.0471774,('PDT000251490.1':0.0494807,('PDT000664879.1':0.0648129,('PDT000664881.1':0.0457646,('PDT000631866.1':0.0447551,('PDT000664808.1':0.0391593,(('PDT000664933.1':0.0138843,('PDT000631864.1':0.00914281,('PDT000631927.1':0.00684666,('PDT000631860.1':0.00504258,(('PDT000664713.1':0.00295028,('PDT000631926.1':0.00203695,'PDT000664721.1':0.00155874)'':0.00054341)'':0.000419811,('PDT000664937.1':0.00386812,('PDT000631928.1':0.00329273,'PDT000664932.1':0.0029878)'':0.000515699)'':8.27389e-05)'':0.00157439)'':0.000556838)'':0.00265325)'':0.00182209)'':-0.00207185,('PDT000664886.1':0.00707764,(('PDT000664921.1':0.00418521,'PDT000664923.1':0.0034887)'':0.00254192,(('PDT000664812.1':0.00326977,'PDT000664925.1':0.0026161)'':0.00210588,('PDT000664849.1':0.00493642,('PDT000664810.1':0.00528819,(('PDT000664847.1':0.00343848,'PDT000664922.1':0.00364381)'':0.00017901,('PDT000664844.1':0.00347223,('PDT000664884.1':0.00317936,('PDT000664934.1':0.00258257,('PDT000664882.1':0.00258431,'PDT000664883.1':0.002703)'':0.000408264)'':0.000139614)'':0.000259472)'':0.000587583)'':0.000756465)'':0.00027613)'':0.000239686)'':0.000754252)'':0.00268931)'':-5.92148e-06)'':0.0160786)'':0.00176561)'':0.00199657)'':0.00828752)'':0.00753398)'':0.00275802)'':-0.00708436)'':0.00662198)'':0.00419411)'':0.0266162)'':-0.0261324)'':0.00218545)'':0.0458764)'':0.00931275)'':0.10552)'':0.0182818,((((('PDT000248513.1':0.0337711,('PDT000249794.1':0.0237385,'PDT000250130.1':0.0272929)'':0.0112294)'':0.171811,('PDT000502184.1':0.168755,((('PDT000249945.1':0.0417492,('PDT000396818.1':0.0197368,'PDT000251385.1':0.0189806)'':0.0321366)'':0.0256544,('PDT000485554.1':0.0718563,(('PDT000246527.2':0.0560818,'PDT000460004.1':0.0513312)'':0.0136624,('PDT000645547.2':0.0658588,('PDT000424624.1':0.0512109,'PDT000359694.1':0.0431844)'':0.0125215)'':0.00637955)'':0.00929855)'':-0.00299458)'':0.0227362,('PDT000325752.1':0.116538,(('PDT000502062.1':0.0245757,'PDT000502210.1':0.0226924)'':0.053911,(('PDT000682545.1':0.0574203,'PDT000682547.1':0.0473007)'':0.0380994,('PDT000250032.1':0.0641607,('PDT000672420.1':0.0228757,('PDT000672407.1':0.0183138,'PDT000300583.1':0.0150096)'':0.00512791)'':0.0427477)'':0.0132472)'':0.00425794)'':0.0259858)'':-0.00890392)'':0.0789996)'':0.0370567)'':0.00186968,(('PDT000146555.3':0.00947468,'PDT000264692.3':0.00800822)'':0.202156,(('PDT000372523.1':0.0607065,('PDT000672719.1':0.0513732,('PDT000651899.1':0.0424904,'PDT000655798.1':0.0416326)'':0.0152873)'':0.0139426)'':0.142778,((('PDT000003809.2':0.173587,('PDT000309216.2':0.0588787,'PDT000557285.1':0.0623133)'':0.122336)'':0.0041079,(('PDT000473274.1':0.151704,'PDT000713452.1':0.147922)'':0.0109232,('PDT000536919.1':0.078457,('PDT000411083.1':0.0672598,('PDT000661647.1':0.042701,('PDT000395002.1':0.0365331,'PDT000710536.1':0.0338853)'':0.00742456)'':0.019033)'':0.0154982)'':0.0924546)'':0.00743187)'':0.0019148,((('PDT000077107.2':0.0481703,'PDT000397784.1':0.048821)'':0.118741,('PDT000497415.1':0.0509926,('PDT000708755.1':0.0496852,('PDT000395046.1':0.0336663,('PDT000025504.2':0.031995,'PDT000030569.1':0.0279599)'':0.00239835)'':0.0116598)'':0.00412255)'':0.107909)'':0.0170748,('PDT000498761.1':0.177252,('PDT000691092.1':0.0891325,('PDT000721991.1':0.0840536,(('PDT000397998.1':0.0463046,'PDT000557117.1':0.051384)'':0.0136863,(('PDT000289953.1':0.0485606,('PDT000167703.2':0.0470136,'PDT000228337.1':0.0490002)'':0.00828706)'':0.00886474,('PDT000149351.2':0.0401086,('PDT000394923.1':0.0213403,'PDT000559254.1':0.0251851)'':0.0100102)'':0.0262318)'':0.0016718)'':0.0126865)'':0.00260379)'':0.0975406)'':-0.00129229)'':0.00301968)'':0.0133609)'':0.0171948)'':0.00255764)'':0.00415899,((('PDT000485942.1':0.089545,('PDT000443164.1':0.0822962,('PDT000213273.2':0.0595925,'PDT000713359.1':0.0592195)'':0.00813476)'':0.0274611)'':0.121709,(('PDT000390620.1':0.134365,('PDT000235758.2':0.0548851,('PDT000502213.1':0.0102618,'PDT000520952.1':0.00841495)'':0.0360205)'':0.0865357)'':0.0416575,((('PDT000534953.1':0.084408,('PDT000535381.1':0.0494372,(('PDT000535251.1':0.0423992,'PDT000632353.1':0.0362187)'':0.00603655,('PDT000634193.1':0.0438932,('PDT000590015.1':0.0282871,('PDT000633175.1':0.0178283,'PDT000634021.1':0.015594)'':0.00923402)'':0.0184326)'':-0.00159401)'':0.0128387)'':0.00806187)'':-0.00638841,('PDT000425890.1':0.0347738,(('PDT000636278.1':0.0171074,('PDT000181694.2':0.0180427,'PDT000426345.1':0.0149167)'':0.00467431)'':0.0137594,('PDT000424018.1':0.0361184,('PDT000202040.2':0.0299548,('PDT000618778.1':0.0287522,('PDT000535222.1':0.0211452,'PDT000212808.1':0.0245147)'':0.00316634)'':0.00298337)'':0.000819926)'':0.00080617)'':0.00386388)'':0.0226253)'':0.0402227,(((('PDT000294556.1':0.0886342,'PDT000298072.1':0.0980538)'':0.0121214,('PDT000291279.1':0.0825777,('PDT000168715.1':0.054488,('PDT000168763.1':0.0214833,'PDT000168768.1':0.0141526)'':0.0386761)'':0.0391853)'':0.0123014)'':-0.00326581,((('PDT000417110.1':0.0624687,'PDT000346627.1':0.0683853)'':0.0100041,('PDT000325636.1':0.0728438,'PDT000457710.1':0.0686962)'':0.0186456)'':0.0156365,(('PDT000417092.1':0.0785091,('PDT000175834.2':0.0601826,'PDT000417107.1':0.0607144)'':0.00523138)'':0.00774855,('PDT000604978.1':0.0840319,('PDT000346626.1':0.0562088,('PDT000634127.1':0.0345377,'PDT000327854.1':0.0413009)'':0.0199894)'':0.0176974)'':0.000690759)'':0.0139138)'':0.00239665)'':0.0150754,((('PDT000520832.1':0.0620862,('PDT000426338.1':0.0462692,'PDT000124687.1':0.0457258)'':0.0186448)'':0.0141874,((('PDT000226327.1':0.0516758,('PDT000633889.1':0.0494965,('PDT000535304.1':0.0190153,'PDT000485987.1':0.0180472)'':0.0237852)'':0.0092969)'':0.0058405,(('PDT000535326.1':0.0112415,'PDT000535422.1':0.0164032)'':0.0418255,('PDT000589809.1':0.0152392,'PDT000632346.1':0.0177648)'':0.0412954)'':0.00812768)'':0.00758174,(('PDT000661725.1':0.0655749,('PDT000535022.1':0.0557143,'PDT000636134.1':0.0571607)'':0.00997612)'':0.00105752,('PDT000425825.1':0.0605018,((('PDT000521010.1':0.0301441,'PDT000233317.1':0.0284333)'':0.0130143,('PDT000363553.1':0.0373397,'PDT000019255.2':0.038257)'':0.00641136)'':0.00579993,('PDT000534915.1':0.0541393,('PDT000535033.1':0.0426644,('PDT000156763.2':0.0430052,('PDT000188116.2':0.0365461,('PDT000188119.2':0.0321315,'PDT000425596.1':0.038191)'':0.0038096)'':0.00324121)'':0.0029809)'':0.00210634)'':9.65421e-05)'':0.00536585)'':0.0125439)'':0.00113464)'':0.0146936)'':0.00150147,((('PDT000656965.1':0.103265,('PDT000409985.1':0.104015,('PDT000720466.1':0.100042,'PDT000135971.2':0.0877535)'':0.00405759)'':0.0182481)'':-0.00121723,(('PDT000716930.1':0.0842607,'PDT000354285.1':0.0663613)'':0.0219769,(('PDT000298019.1':0.0509306,'PDT000298086.1':0.0579074)'':0.0422975,(('PDT000426792.1':0.0749503,'PDT000081759.1':0.0723987)'':0.0107066,('PDT000520839.1':0.0766711,('PDT000274868.2':0.0545309,('PDT000644859.1':0.0436938,'PDT000704979.1':0.0393155)'':0.0075864)'':0.0244229)'':0.00799458)'':0.00304175)'':0.00963185)'':0.00890114)'':0.000831714,(('PDT000417176.1':0.0949795,(('PDT000510087.1':0.0329659,'PDT000321318.1':0.0394667)'':0.0372094,('PDT000042638.1':0.043382,'PDT000129581.1':0.0492682)'':0.0207394)'':0.0276474)'':-0.00232016,((('PDT000716826.1':0.0617636,'PDT000346033.1':0.0633304)'':0.0134182,('PDT000425588.1':0.0609304,('PDT000661612.1':0.0629317,('PDT000354229.1':0.0619991,(('PDT000425555.1':0.0225097,'PDT000521090.1':0.0258489)'':0.0106784,('PDT000253668.1':0.0383075,('PDT000425509.1':0.0368317,'PDT000427020.1':0.033784)'':0.0037423)'':0.000209144)'':0.0110741)'':0.00800691)'':0.00556963)'':0.0140035)'':0.0144609,((('PDT000233281.1':0.0375015,'PDT000346625.1':0.0384418)'':0.0392668,('PDT000714560.1':0.0619828,('PDT000175885.2':0.0577237,('PDT000433757.1':0.0338293,('PDT000632717.1':0.014135,'PDT000634057.1':0.0129508)'':0.0195774)'':0.0221111)'':0.00336981)'':0.00935905)'':0.00599848,(((('PDT000656987.1':0.070772,'PDT000708210.1':0.068689)'':0.00626325,('PDT000632458.1':0.0486556,'PDT000168876.1':0.0656804)'':0.0127317)'':0.0153747,((('PDT000535134.1':0.0448542,'PDT000069668.1':0.0390509)'':0.0285295,(('PDT000423862.1':0.0274506,'PDT000668921.1':0.034797)'':0.0323142,('PDT000652982.1':0.0661766,('PDT000622449.1':0.0486224,(('PDT000425674.1':0.0273831,'PDT000535400.1':0.0296672)'':0.0117886,('PDT000501873.1':0.0226314,('PDT000520843.1':0.0202418,'PDT000482855.1':0.0229258)'':0.00384664)'':0.0168225)'':0.00350894)'':0.012156)'':0.0138937)'':0.00361476)'':-0.00403069,('PDT000716901.1':0.0935153,(('PDT000718301.1':0.0628603,('PDT000671871.1':0.0601008,'PDT000231013.1':0.0540082)'':0.00676173)'':0.0148908,(('PDT000136092.2':0.0401501,'PDT000425897.1':0.0487431)'':0.0127047,('PDT000468879.1':0.0568078,(('PDT000176000.2':0.064099,('PDT000181840.2':0.0388226,('PDT000175907.2':0.0311655,'PDT000535146.1':0.0278908)'':0.00532086)'':0.0175981)'':-0.00453833,('PDT000520830.1':0.0434827,('PDT000602898.1':0.0448715,(('PDT000181811.2':0.0225956,'PDT000669006.1':0.0214916)'':0.0150099,('PDT000425828.1':0.0281687,'PDT000521046.1':0.0252551)'':0.0110093)'':0.00218447)'':0.00503219)'':-0.000120026)'':0.00557219)'':-0.00121404)'':0.0183422)'':0.0149784)'':-0.0156317)'':0.0132072)'':-0.0040047,(('PDT000716814.1':0.0853763,(('PDT000004992.2':0.0480847,('PDT000426998.1':0.046622,'PDT000427249.1':0.0490911)'':0.00392409)'':0.00688673,('PDT000425659.1':0.0705135,('PDT000221017.1':0.0480999,('PDT000425971.1':0.051429,('PDT000694490.1':0.0416464,('PDT000425653.1':0.0421697,(('PDT000557092.1':0.0191618,'PDT000622523.1':0.0211132)'':0.0117829,('PDT000426736.1':0.021,('PDT000719070.1':0.0224768,('PDT000175808.2':0.00609107,'PDT000448215.1':0.00899983)'':0.0125438)'':0.00418933)'':0.000867089)'':0.00905269)'':0.00214701)'':0.00566182)'':0.00483343)'':0.0136154)'':-0.00858956)'':0.0218674)'':-0.00763789,('PDT000432060.1':0.0872302,(('PDT000040735.2':0.0876075,('PDT000501892.1':0.0692712,('PDT000520822.1':0.0678746,('PDT000175812.2':0.0551651,'PDT000520977.1':0.0569229)'':0.00847244)'':0.00265904)'':0.0118326)'':-0.00104537,('PDT000226153.1':0.0956829,(('PDT000535514.1':0.0591713,('PDT000433482.1':0.0567256,(('PDT000175798.2':0.037132,'PDT000175807.2':0.0405697)'':0.00689584,('PDT000218127.2':0.0313611,'PDT000019247.2':0.024612)'':0.0186409)'':0.00993914)'':0.00521968)'':-0.00468313,('PDT000672316.1':0.0909939,('PDT000175921.2':0.0674262,((('PDT000103009.1':0.0203075,'PDT000103068.1':0.0277723)'':0.0288847,('PDT000385058.1':0.0447222,('PDT000433401.1':0.0418619,'PDT000616158.1':0.041836)'':0.00505978)'':0.00853188)'':0.000793586,(('PDT000143939.3':0.0472099,'PDT000520854.1':0.0415602)'':0.00224255,('PDT000694385.1':0.0385411,('PDT000181718.2':0.0237177,('PDT000175926.2':0.00284038,'PDT000616169.1':0.00365118)'':0.0169548)'':0.0151163)'':0.00760514)'':0.00316099)'':0.0125324)'':0.0111252)'':-0.0149954)'':0.0259102)'':-0.0147501)'':0.00713488)'':-0.00712711)'':0.0057413)'':0.0050311)'':0.00571464)'':0.002111)'':0.0027251)'':-0.00263139)'':0.00712222)'':0.00564895)'':0.0781196)'':0.0291985)'':-0.000680076,(((('PDT000502206.1':0.0501799,'PDT000714592.1':0.0422546)'':0.0221574,('PDT000633277.1':0.060897,'PDT000228341.1':0.058492)'':0.0134368)'':0.0100181,('PDT000339374.1':0.0818908,(('PDT000644871.1':0.0374587,'PDT000288185.1':0.0364253)'':0.0218652,('PDT000566157.1':0.03959,('PDT000397701.1':0.0321022,('PDT000566034.1':0.018284,('PDT000395056.1':0.0156469,'PDT000639526.1':0.0169521)'':0.00339118)'':0.0076296)'':0.0176826)'':0.00372952)'':0.0233954)'':0.0084305)'':0.124361,(('PDT000204870.2':0.18149,'PDT000131757.2':0.177949)'':0.0282739,(((('PDT000500549.1':0.0287132,'PDT000658563.1':0.0295156)'':0.0474971,(('PDT000633582.1':0.0163701,'PDT000633623.1':0.0171665)'':0.0291421,('PDT000633483.1':0.0517693,(('PDT000651184.1':0.033835,('PDT000424015.1':0.02363,'PDT000636292.1':0.0158358)'':0.0194708)'':0.00651952,(('PDT000535089.1':0.0242842,'PDT000535335.1':0.0306922)'':0.0129243,('PDT000535421.1':0.0310835,'PDT000716905.1':0.0343544)'':0.00575291)'':0.0018036)'':0.00630429)'':0.00195949)'':0.0272361)'':0.00273811,(('PDT000632291.1':0.0461368,('PDT000661684.1':0.0345065,('PDT000291151.2':0.0297174,('PDT000582125.1':0.0137373,'PDT000661601.1':0.0155841)'':0.0125286)'':0.00703368)'':0.00197488)'':0.0219207,(('PDT000502167.1':0.0504461,('PDT000535008.1':0.0380415,('PDT000633875.1':0.00658582,'PDT000651208.1':0.00826668)'':0.02604)'':0.017722)'':0.00156749,(('PDT000415749.1':0.0530048,('PDT000145162.2':0.0432974,'PDT000565994.1':0.0368314)'':0.00995655)'':-0.000555844,(('PDT000427685.1':0.0444298,('PDT000675053.1':0.027753,('PDT000177290.1':0.0171859,'PDT000364515.1':0.0144086)'':0.0130735)'':0.0172018)'':0.00110564,('PDT000520997.1':0.0433365,('PDT000270503.1':0.0313742,('PDT000426064.1':0.0212189,('PDT000713292.1':0.00974256,'PDT000716296.1':0.0114897)'':0.0109808)'':0.0105597)'':0.00896071)'':-0.000474488)'':0.00327364)'':0.00972375)'':0.00664244)'':0.0173137)'':0.0720256,((('PDT000490243.1':0.0470988,('PDT000218939.1':0.043013,('PDT000633250.1':0.0343437,('PDT000716921.1':0.0242357,('PDT000543280.1':0.0282552,'PDT000243042.1':0.02481)'':0.00272898)'':0.00249321)'':0.00484997)'':0.00440707)'':0.00541926,('PDT000318192.1':0.0728011,(('PDT000343505.1':0.0394959,('PDT000423893.1':0.0181698,'PDT000423954.1':0.0216282)'':0.0161282)'':0.00212658,('PDT000661424.1':0.0371812,('PDT000445031.1':0.0264129,'PDT000713703.1':0.0206273)'':0.0105067)'':0.0101173)'':0.0149781)'':-0.0077845)'':0.0931682,(((('PDT000274710.2':0.0848197,'PDT000252691.1':0.0870143)'':0.00904308,('PDT000003810.2':0.0390219,'PDT000397916.1':0.0440224)'':0.066614)'':0.00743645,('PDT000168712.1':0.128425,(('PDT000168828.1':0.0962939,('PDT000627360.1':0.0892617,('PDT000366729.1':0.0628973,('PDT000508913.1':0.0536733,('PDT000417106.1':0.0382262,('PDT000675080.1':0.0335435,'PDT000652385.1':0.0207123)'':0.00908981)'':0.0125475)'':0.0067879)'':0.0194351)'':0.00989859)'':-0.0099582,(('PDT000627031.1':0.0744208,'PDT000411294.1':0.0777892)'':0.00620788,((('PDT000486528.1':0.05301,('PDT000720447.1':0.0427164,('PDT000720453.1':0.0238722,('PDT000201670.2':0.0160091,'PDT000485937.1':0.0152039)'':0.0118977)'':0.00993341)'':0.014542)'':-0.0034168,(('PDT000417132.1':0.04203,'PDT000346621.1':0.0426443)'':0.0123337,('PDT000431184.1':0.0621204,('PDT000124686.1':0.0465324,('PDT000181736.2':0.0384362,('PDT000181681.2':0.0405804,'PDT000411306.1':0.0374256)'':0.00677561)'':0.00284817)'':0.00215975)'':0.00331522)'':0.0020029)'':0.0129784,('PDT000346623.1':0.0874734,((('PDT000329817.1':0.059601,('PDT000534796.1':0.0267661,'PDT000267151.1':0.0298082)'':0.0250659)'':0.00244463,('PDT000398342.1':0.0460242,('PDT000463602.1':0.0420683,('PDT000543086.1':0.0399495,('PDT000622448.1':0.0331762,('PDT000678608.1':0.0102716,'PDT000687486.1':0.0102194)'':0.0192605)'':0.00200152)'':0.00673123)'':0.00745264)'':0.00636309)'':0.00421352,(('PDT000499077.1':0.074778,('PDT000333201.1':0.0672616,('PDT000347024.1':0.0505057,('PDT000691333.1':0.0408045,('PDT000720286.1':0.0348711,'PDT000485986.1':0.0326859)'':0.00350832)'':0.00553684)'':0.0152746)'':0.00372419)'':-0.00224258,('PDT000358770.1':0.0828621,('PDT000633732.1':0.0637702,(('PDT000618895.1':0.0314901,'PDT000632945.1':0.0401973)'':0.0235684,(('PDT000107919.1':0.0508664,'PDT000422528.1':0.0493616)'':0.00446934,(('PDT000225240.1':0.0224275,'PDT000346624.1':0.0298556)'':0.0204976,(('PDT000434600.1':0.0185737,'PDT000537812.1':0.0199371)'':0.0115875,('PDT000694446.1':0.0197532,'PDT000244076.1':0.0253304)'':0.0109991)'':0.00825709)'':0.00957474)'':0.00409235)'':0.0075611)'':0.00416736)'':-0.00790582)'':0.00359991)'':0.0231801)'':-0.0225044)'':0.0134514)'':0.00494372)'':0.0241411)'':-0.00339772)'':0.00492475,(('PDT000496895.1':0.105465,(('PDT000178582.2':0.0784811,'PDT000303102.2':0.0864869)'':0.00274155,(('PDT000717585.1':0.0667832,('PDT000188078.2':0.0675928,('PDT000501997.1':0.0379706,'PDT000347052.1':0.0334024)'':0.0192177)'':0.00355207)'':-0.000486728,(('PDT000386752.1':0.0512579,('PDT000200123.2':0.0515659,'PDT000566126.1':0.0350988)'':0.0078593)'':0.0167576,('PDT000645008.1':0.0714252,('PDT000455199.1':0.0599638,('PDT000497365.1':0.0427318,'PDT000644834.1':0.0394917)'':0.00987551)'':0.00648532)'':0.000860064)'':0.00128092)'':0.0182606)'':0.0125485)'':-0.0173277,((('PDT000716904.1':0.0370291,('PDT000106254.2':0.0318798,'PDT000188370.1':0.0295033)'':0.00608652)'':0.0278763,(('PDT000501973.1':0.0410183,'PDT000008698.2':0.0524333)'':0.00989901,('PDT000204784.1':0.0635912,'PDT000422530.1':0.0545758)'':0.00784204)'':0.00917666)'':0.0148648,(((('PDT000656710.1':0.0480874,'PDT000502059.1':0.0494666)'':0.0147665,('PDT000509499.1':0.0466044,('PDT000136102.2':0.0404209,'PDT000719129.1':0.042257)'':0.00567223)'':0.0186846)'':0.00075625,('PDT000521100.1':0.0689044,('PDT000482826.1':0.060317,('PDT000502064.1':0.0375098,('PDT000271778.1':0.0386856,('PDT000310576.2':0.0361235,'PDT000543117.1':0.0360295)'':0.00295223)'':0.00454821)'':0.00897665)'':0.0100318)'':0.00180924)'':0.0031065,(('PDT000589880.1':0.0563825,('PDT000565851.1':0.0585337,('PDT000455365.1':0.033233,('PDT000549087.1':0.0181016,'PDT000683162.1':0.0226399)'':0.00938439)'':0.0163906)'':0.0125937)'':0.00555327,(('PDT000132124.2':0.0627036,('PDT000632570.1':0.0122102,'PDT000632781.1':0.0114579)'':0.0461054)'':-0.00250764,((('PDT000634218.1':0.0489227,'PDT000650988.1':0.0509441)'':0.00841446,(('PDT000129702.1':0.0248886,'PDT000399955.1':0.0233467)'':0.0205508,('PDT000535339.1':0.0411875,('PDT000534859.1':0.0285173,'PDT000651107.1':0.026152)'':0.00836501)'':0.00685954)'':0.00416806)'':0.00448126,(('PDT000716974.1':0.0527843,('PDT000535449.1':0.0462702,(('PDT000181738.2':0.0326973,'PDT000345911.1':0.0323892)'':0.00734845,('PDT000713540.1':0.0361569,('PDT000426099.1':0.0331598,'PDT000602930.1':0.0318281)'':0.00418576)'':0.00341318)'':0.00524643)'':0.00289063)'':-0.00315777,('PDT000651655.1':0.0421102,(('PDT000607502.1':0.0299864,('PDT000501999.1':0.0153318,'PDT000346714.1':0.0158193)'':0.00926502)'':0.00346803,('PDT000291157.2':0.0286584,('PDT000716836.1':0.0261328,('PDT000716956.1':0.0141676,('PDT000433466.1':0.00811286,'PDT000716910.1':0.0104167)'':0.00621619)'':0.00807417)'':0.00480182)'':0.00571582)'':0.00848071)'':-0.00016603)'':0.00798958)'':0.00493186)'':0.0111051)'':-0.00252385)'':0.012829)'':-0.00461864)'':0.027668)'':0.0394317)'':0.0104153)'':0.0487236)'':-0.00264873)'':0.00271261)'':0.00691287)'':0.00611494,((('PDT000720477.1':0.087909,('PDT000566170.1':0.0759477,'PDT000359681.1':0.0584323)'':0.0216815)'':0.110269,(((('PDT000431984.1':0.0364244,('PDT000535058.1':0.0154008,'PDT000633413.1':0.0143911)'':0.0108186)'':0.0259849,('PDT000622469.1':0.0593892,(('PDT000535069.1':0.0323856,('PDT000535168.1':0.00621604,'PDT000716936.1':0.00710406)'':0.0203095)'':0.00588608,('PDT000423926.1':0.0316207,('PDT000534984.1':0.0296477,('PDT000716816.1':0.0231922,('PDT000534883.1':0.0117774,'PDT000535223.1':0.013907)'':0.00857549)'':0.00531384)'':0.0100113)'':-0.000265319)'':0.0199739)'':-0.00260298)'':0.0522969,(('PDT000627643.1':0.105003,('PDT000718889.1':0.0857863,('PDT000228181.1':0.0790029,'PDT000252629.1':0.0767071)'':0.0105222)'':0.0107161)'':0.00394473,(('PDT000520783.1':0.0653387,'PDT000521054.1':0.0591313)'':0.0220905,((('PDT000645501.2':0.0327655,'PDT000441358.1':0.0301578)'':0.0307366,('PDT000189435.1':0.048768,('PDT000426588.1':0.0292969,'PDT000446004.1':0.0337184)'':0.0112836)'':0.0226209)'':0.00961518,('PDT000426070.1':0.0565494,(('PDT000427687.1':0.0239538,'PDT000520908.1':0.0231661)'':0.00632552,('PDT000425983.1':0.0292807,(('PDT000713338.1':0.0166558,'PDT000714074.1':0.0178985)'':0.00396517,('PDT000426272.1':0.0189711,('PDT000426217.1':0.00837724,'PDT000521078.1':0.0119132)'':0.00642859)'':0.0057244)'':0.00614577)'':-0.000374469)'':0.0223781)'':0.0104074)'':0.0158053)'':0.0262517)'':0.00885408)'':0.022679,('PDT000618995.1':0.148047,((('PDT000497211.1':0.0791558,('PDT000661656.1':0.0544252,('PDT000311243.2':0.0516841,('PDT000720459.1':0.0519848,('PDT000716005.1':0.031992,'PDT000591884.1':0.0351963)'':0.0112097)'':0.00411105)'':0.0059578)'':0.0198193)'':0.00826751,(('PDT000346564.1':0.0703943,('PDT000433445.1':0.0559316,('PDT000222628.1':0.0445128,('PDT000364524.2':0.0208295,'PDT000289899.1':0.0232012)'':0.0165416)'':0.0058311)'':0.0230111)'':-0.00171023,(('PDT000325100.1':0.0464843,'PDT000588842.1':0.038935)'':0.00404953,('PDT000712631.1':0.050561,('PDT000155642.2':0.0284708,('PDT000202153.2':0.0257902,'PDT000500571.1':0.0233869)'':0.00208111)'':0.0134422)'':0.00092298)'':0.0164059)'':0.0272385)'':0.00871716,('PDT000282207.2':0.107611,(('PDT000648061.1':0.0783827,('PDT000218930.1':0.0493228,('PDT000421803.1':0.0299805,'PDT000321351.1':0.0359342)'':0.0110241)'':0.0177144)'':0.0107888,(('PDT000424004.1':0.0966666,(('PDT000181752.2':0.0563819,'PDT000292639.1':0.0453701)'':0.00668536,(('PDT000421787.1':0.0389334,'PDT000188346.1':0.035653)'':0.0170732,('PDT000216279.2':0.0701322,('PDT000110787.2':0.0457733,'PDT000677122.1':0.0357388)'':0.0120753)'':0.00271584)'':0.0138889)'':0.0145792)'':-0.00923418,(('PDT000346622.1':0.0672916,('PDT000305497.2':0.0595876,('PDT000425629.1':0.0461445,'PDT000718942.1':0.0418938)'':0.00992778)'':0.00505795)'':0.00371006,(('PDT000636622.1':0.0430085,('PDT000218933.1':0.0260335,'PDT000250707.1':0.0237207)'':0.0187982)'':0.00967258,('PDT000309326.2':0.0587128,(('PDT000425922.1':0.0286288,'PDT000340016.1':0.0296972)'':0.00980907,('PDT000354870.1':0.035397,('PDT000467016.1':0.0120254,'PDT000673538.1':0.0117158)'':0.0168363)'':0.00808242)'':0.0141626)'':0.000763569)'':0.00525848)'':0.0105251)'':0.00401081)'':0.0201757)'':-0.00423193)'':0.0307569)'':0.00685256)'':0.0610274)'':-0.0431493,((((('PDT000645496.2':0.0470193,'PDT000645504.2':0.0545707)'':0.034678,(('PDT000176007.2':0.0585244,'PDT000621072.1':0.0675646)'':0.00301746,('PDT000534986.1':0.0527492,('PDT000417102.1':0.0435511,'PDT000253676.1':0.0449716)'':0.00571241)'':0.00776762)'':0.0124402)'':0.0250649,('PDT000218130.2':0.0908339,('PDT000251433.1':0.0843657,('PDT000718987.1':0.0764374,('PDT000342662.1':0.0212676,('PDT000485553.1':0.00897713,('PDT000620254.1':0.00172368,'PDT000620266.1':0.00197262)'':0.00717897)'':0.00974587)'':0.055097)'':0.00945089)'':0.00271043)'':0.0158345)'':0.00170367,(((('PDT000300822.2':0.0658502,'PDT000678000.1':0.0704918)'':0.0205131,(('PDT000212796.1':0.0703356,'PDT000251690.1':0.0678284)'':0.0201257,(('PDT000425847.1':0.0245802,'PDT000652880.1':0.0348739)'':0.0555984,(('PDT000394946.1':0.0583895,'PDT000719229.1':0.0625665)'':0.00618716,('PDT000494411.1':0.0565354,'PDT000188362.1':0.0522766)'':0.0133281)'':0.0123526)'':0.00147639)'':0.00987275)'':0.00862165,('PDT000384976.1':0.0991956,(('PDT000425737.1':0.0860346,('PDT000535122.1':0.0867854,('PDT000367297.1':0.0536475,('PDT000713586.1':0.0116463,'PDT000720267.1':0.0151866)'':0.0284017)'':0.0323293)'':0.00237913)'':-0.00104846,('PDT000259739.1':0.0804361,(('PDT000633445.1':0.0634393,'PDT000660661.1':0.0651237)'':0.00114246,(('PDT000417115.1':0.0694381,'PDT000645077.1':0.0603899)'':0.00572688,(('PDT000714171.1':0.0339328,'PDT000505167.1':0.0304802)'':0.00531171,('PDT000395162.1':0.0398357,('PDT000201774.1':0.0161463,('PDT000688693.1':0.0124293,('PDT000639512.1':0.00395199,'PDT000540522.1':0.00392046)'':0.00760617)'':0.00193022)'':0.0176297)'':-0.00272898)'':0.0168731)'':0.0173156)'':0.0128205)'':0.00624496)'':0.00816867)'':0.0053749)'':0.000653144,(((('PDT000248465.1':0.0651809,'PDT000346620.1':0.0615351)'':0.0180078,(('PDT000645016.1':0.0306006,'PDT000645031.1':0.0266521)'':0.0340534,('PDT000565878.1':0.0375183,('PDT000395507.1':0.0440256,'PDT000394912.1':0.0285045)'':0.0138582)'':0.0174763)'':0.00743745)'':0.0182912,('PDT000168798.1':0.108172,(('PDT000600994.1':0.0546677,'PDT000228387.1':0.0549253)'':0.0255894,(('PDT000425648.1':0.0660136,('PDT000129826.2':0.0620499,'PDT000539045.1':0.0622021)'':0.00256787)'':0.00599176,('PDT000557226.1':0.0621692,('PDT000619859.1':0.0371352,'PDT000619860.1':0.0393999)'':0.0275287)'':0.00650264)'':0.0114394)'':0.0168214)'':-0.0117583)'':0.00736177,(('PDT000423907.1':0.0363031,'PDT000424023.1':0.0420504)'':0.0398775,((('PDT000535116.1':0.0347263,'PDT000129924.1':0.0494925)'':0.0128829,('PDT000535038.1':0.0161591,('PDT000535174.1':0.0184449,'PDT000535192.1':0.0179062)'':0.00168001)'':0.0291835)'':0.00340211,(('PDT000534867.1':0.0335421,('PDT000535267.1':0.0129712,('PDT000535406.1':0.0078449,'PDT000535490.1':0.0093673)'':0.0048875)'':0.022389)'':0.0119296,(('PDT000534858.1':0.0213256,'PDT000411272.1':0.0250555)'':0.0306471,(('PDT000393062.1':0.0290549,'PDT000287614.1':0.0329161)'':0.00435803,('PDT000713600.1':0.0305015,'PDT000362493.1':0.0385617)'':0.00114582)'':0.00752326)'':0.00415304)'':0.00521221)'':0.0155142)'':0.0259295)'':0.00573579)'':0.00366925)'':0.0122607,(((('PDT000227983.1':0.0442444,('PDT000535126.1':0.0260861,'PDT000287619.1':0.0206521)'':0.0254225)'':0.0249356,('PDT000674866.1':0.035871,('PDT000675113.1':0.0214421,'PDT000325637.1':0.0208859)'':0.0123881)'':0.0397332)'':0.0172673,(('PDT000391860.1':0.0407184,('PDT000438356.1':0.00847168,'PDT000444870.1':0.00941032)'':0.0310694)'':0.0344472,((((('PDT000042579.1':0.0430817,'PDT000019610.1':0.0378872)'':0.00122394,('PDT000616047.1':0.03481,('PDT000616080.1':0.0194836,'PDT000408009.1':0.0200511)'':0.0140939)'':0.00460649)'':0.0121162,(('PDT000535183.1':0.0329327,'PDT000520853.1':0.0357298)'':0.010334,('PDT000668908.1':0.0354432,('PDT000535119.1':0.0292057,('PDT000175964.2':0.0159425,'PDT000615824.1':0.0153476)'':0.0135246)'':0.0069232)'':0.0117197)'':0.00166895)'':0.00802524,('PDT000619081.1':0.0787968,(('PDT000455749.1':0.0681802,(('PDT000431893.1':0.0136038,'PDT000632901.1':0.0119787)'':0.0244276,('PDT000438316.1':0.0322314,('PDT000656278.1':0.0257365,('PDT000175846.2':0.0203909,'PDT000175905.2':0.0211204)'':0.00492288)'':0.00109388)'':0.00832804)'':0.0217713)'':-0.00846559,(('PDT000535250.1':0.0326339,'PDT000520888.1':0.0413527)'':0.0115273,(('PDT000633206.1':0.0460532,('PDT000589760.1':0.0325899,'PDT000590084.1':0.0283223)'':0.0208985)'':0.00255329,('PDT000615805.1':0.0405573,(('PDT000426132.1':0.0299731,'PDT000633669.1':0.0326253)'':0.00639087,('PDT000287620.1':0.0392367,('PDT000615844.1':0.0305144,('PDT000535028.1':0.0190885,'PDT000250042.1':0.0245182)'':0.00785786)'':0.00602236)'':-0.00124041)'':0.00590918)'':0.00266252)'':0.00477073)'':0.00327639)'':0.0199988)'':-0.0119912)'':0.00921543,(('PDT000673413.1':0.0692983,(('PDT000107936.2':0.0284873,'PDT000535590.1':0.0250211)'':0.0187663,(('PDT000632979.1':0.00800128,('PDT000231015.1':0.0053148,'PDT000283179.1':0.0048446)'':0.00652512)'':0.0079688,('PDT000361026.1':0.021616,('PDT000181717.2':0.0117612,('PDT000616015.1':0.0107947,'PDT000716892.1':0.00983823)'':0.00152888)'':0.00724172)'':-3.48086e-05)'':0.0247183)'':0.0230498)'':-0.00140098,((('PDT000433305.1':0.0331492,'PDT000687492.1':0.0347066)'':0.0221414,('PDT000520732.1':0.046755,('PDT000633631.1':0.0209773,'PDT000682318.1':0.0240757)'':0.02444)'':0.00895204)'':0.0008251,(('PDT000538445.1':0.0576972,('PDT000537950.1':0.0386901,('PDT000426510.1':0.0391806,('PDT000538492.1':0.0254575,('PDT000426545.1':0.0214427,'PDT000536568.1':0.021703)'':0.000544144)'':0.0170183)'':0.0024811)'':0.00545326)'':0.000712166,(('PDT000202185.2':0.0360914,('PDT000426845.1':0.0229872,'PDT000711068.1':0.0223692)'':0.00630631)'':0.011408,('PDT000716933.1':0.051092,('PDT000636274.1':0.0559497,(('PDT000175946.1':0.0299893,'PDT000433250.1':0.0328896)'':0.00901026,('PDT000616031.1':0.0231347,('PDT000443106.1':0.0246152,'PDT000619075.1':0.0166783)'':0.00466441)'':0.0117072)'':0.0063323)'':0.00313053)'':-0.000871135)'':0.00150634)'':0.00770259)'':0.00418704)'':0.0104602)'':0.00383304)'':0.0173175)'':-0.00770731,(((('PDT000615813.1':0.0245177,('PDT000615891.1':0.00243035,'PDT000615910.1':0.00206246)'':0.0254212)'':0.029116,('PDT000427587.1':0.0547827,('PDT000661372.1':0.0383591,('PDT000434472.1':0.0227519,(('PDT000705063.1':0.0157785,('PDT000705062.1':0.00727868,'PDT000720305.1':0.00726462)'':0.00626561)'':-0.00042109,('PDT000259740.1':0.00955227,('PDT000538606.1':0.0100892,('PDT000200509.2':0.00577279,'PDT000634554.1':0.00768411)'':0.0017213)'':0.00120522)'':0.00297363)'':0.00876335)'':0.0118657)'':0.0196612)'':-0.00498107)'':0.0185798,(('PDT000336556.1':0.0522214,'PDT000241785.1':0.0573716)'':0.0336603,((('PDT000409998.1':0.0342416,('PDT000222834.2':0.031751,'PDT000191158.1':0.0289912)'':0.00928856)'':0.0258517,(('PDT000346739.1':0.0616948,(('PDT000705171.1':0.0284277,'PDT000572840.1':0.0340478)'':0.00516531,(('PDT000181734.2':0.0260173,'PDT000425673.1':0.0331351)'':0.00662737,('PDT000103035.1':0.0226418,'PDT000501876.1':0.0267032)'':0.00803083)'':0.00578794)'':0.0147931)'':-0.00662965,('PDT000616182.1':0.0334769,('PDT000101293.1':0.0308055,(('PDT000626803.1':0.0157038,'PDT000408003.1':0.0114172)'':0.00612685,('PDT000287615.1':0.0140448,('PDT000616177.1':0.0127868,('PDT000040736.2':0.00639272,'PDT000611904.1':0.00702008)'':0.00308341)'':0.00454452)'':-0.000141792)'':0.0163909)'':0.00255752)'':0.011231)'':0.0100025)'':0.00601574,(('PDT000673527.1':0.0676011,('PDT000018105.2':0.0218116,'PDT000020615.1':0.0196038)'':0.0379127)'':0.0205574,((('PDT000202747.2':0.0358539,('PDT000652583.1':0.0410634,('PDT000709808.1':0.0304806,('PDT000426143.1':0.0218662,('PDT000616046.1':0.00893623,'PDT000510089.1':0.0124696)'':0.00767393)'':0.00756907)'':0.00446789)'':0.00393114)'':0.0141841,(('PDT000675568.1':0.0406179,('PDT000674886.1':0.00978747,'PDT000046970.1':0.0132044)'':0.0367576)'':0.00775052,(('PDT000633513.1':0.0431438,'PDT000720446.1':0.0498096)'':0.0031375,('PDT000618774.1':0.0423824,(('PDT000535004.1':0.015585,'PDT000633537.1':0.0174581)'':0.0161949,('PDT000148571.1':0.0345069,'PDT000206605.1':0.025917)'':0.00532631)'':0.0125899)'':-0.000492275)'':0.00277239)'':0.00431136)'':0.00972306,(('PDT000426059.1':0.0729458,('PDT000485930.1':0.0449297,('PDT000433246.1':0.0422917,'PDT000279215.1':0.037843)'':0.00307997)'':0.0378228)'':-0.0102404,((('PDT000425485.1':0.0470378,'PDT000426683.1':0.042183)'':0.00282676,('PDT000651801.1':0.0327414,'PDT000287613.1':0.0322115)'':0.0143603)'':0.00911076,((('PDT000218109.2':0.0390431,'PDT000132115.2':0.0310738)'':0.0182937,('PDT000588035.1':0.0502126,('PDT000150954.2':0.0432746,('PDT000632870.1':0.0341863,('PDT000352504.1':0.0371226,('PDT000538458.1':0.0265264,'PDT000520861.1':0.0263263)'':0.00650294)'':0.0044361)'':0.00321328)'':0.00736541)'':0.000487413)'':0.000880024,(('PDT000524117.1':0.0473016,('PDT000044017.1':0.034558,('PDT000271348.2':0.0316929,'PDT000632817.1':0.0265776)'':0.000482736)'':0.012379)'':0.0131978,('PDT000685896.1':0.0793253,('PDT000426175.1':0.0538055,('PDT000352408.1':0.0494782,('PDT000173719.2':0.037976,('PDT000132554.1':0.039115,(('PDT000535136.1':0.0276061,('PDT000175892.2':0.0178247,'PDT000433424.1':0.0136792)'':0.0125064)'':0.00512629,('PDT000619862.1':0.0342147,('PDT000719032.1':0.0286241,(('PDT000425745.1':0.0164811,'PDT000622563.1':0.0176446)'':0.00416602,('PDT000636624.1':0.0217093,'PDT000520709.1':0.0213099)'':0.00137378)'':0.00412761)'':0.00381904)'':-0.00127448)'':0.00853369)'':0.0011167)'':0.00328825)'':0.00525752)'':0.0158288)'':-0.012997)'':-8.14662e-05)'':0.00396435)'':0.00778709)'':-0.0032332)'':0.0103137)'':-0.00847252)'':0.0342753)'':-0.0231852)'':0.00623368,(((('PDT000426022.1':0.0312842,'PDT000626801.1':0.0383847)'':0.020259,(('PDT000619865.1':0.043894,('PDT000618718.1':0.029974,('PDT000535057.1':0.0208084,'PDT000415840.1':0.02225)'':0.00470797)'':0.011974)'':0.0022505,(('PDT000633198.1':0.0280702,('PDT000535302.1':0.0241972,'PDT000633610.1':0.0247095)'':0.00712092)'':0.00806936,('PDT000678046.1':0.0278002,('PDT000633279.1':0.0196759,('PDT000534974.1':0.019024,('PDT000425876.1':0.0106461,'PDT000535209.1':0.00967515)'':0.00470188)'':0.00569847)'':0.00537843)'':0.0120892)'':0.00452521)'':0.0118777)'':0.00769585,('PDT000414216.1':0.112987,(('PDT000485933.1':0.0552544,((('PDT000632897.1':0.0395413,'PDT000632874.1':0.0361077)'':0.00454774,('PDT000714035.1':0.0400332,('PDT000673615.1':0.027038,('PDT000223027.2':0.0209084,'PDT000499082.1':0.0224416)'':0.0063056)'':0.00731365)'':0.00368831)'':0.00306155,(('PDT000626814.1':0.0580814,('PDT000618838.1':0.0205708,'PDT000720313.1':0.0213776)'':0.0262233)'':-0.00115887,('PDT000538591.1':0.0398523,('PDT000720309.1':0.0357001,('PDT000243742.2':0.0249641,('PDT000626703.1':0.0184456,'PDT000352402.1':0.0151328)'':0.00933984)'':0.00533795)'':0.00508026)'':0.00786599)'':0.00329168)'':0.00806669)'':-0.00559419,('PDT000581874.1':0.0686409,(('PDT000633311.1':0.0420528,('PDT000538440.1':0.0283622,('PDT000581630.1':0.0217298,('PDT000520831.1':0.00630902,'PDT000460545.1':0.00657038)'':0.0200022)'':0.00427214)'':0.0112044)'':0.0054748,('PDT000373303.1':0.0507612,('PDT000632861.1':0.0455441,(('PDT000346998.1':0.0414023,('PDT000705254.1':0.0389171,('PDT000705419.1':0.0300215,('PDT000277146.3':0.0274889,(('PDT000704999.1':0.0109229,'PDT000705250.1':0.0127086)'':0.00250263,('PDT000705010.1':0.00807691,'PDT000705263.1':0.0106694)'':0.0051855)'':0.00845377)'':0.00052313)'':0.0104722)'':0.00453919)'':-0.00703875,((('PDT000661594.1':0.0361798,'PDT000452803.1':0.0297853)'':0.00343357,('PDT000103016.1':0.0203793,('PDT000175940.2':0.0158629,'PDT000632734.1':0.0197668)'':0.00303326)'':0.00875247)'':0.00108207,((('PDT000716914.1':0.0141768,'PDT000717043.1':0.011345)'':0.0117068,('PDT000535350.1':0.0215186,('PDT000632227.1':0.00828044,'PDT000633196.1':0.0127418)'':0.0105734)'':0.00493868)'':0.00227382,('PDT000619085.1':0.0306297,('PDT000421786.1':0.0235576,('PDT000535489.1':0.02919,('PDT000582122.1':0.0199887,('PDT000202685.2':0.0155088,('PDT000425853.1':0.0140704,'PDT000426603.1':0.0129406)'':0.00221041)'':0.00371879)'':0.00421635)'':0.00176534)'':0.00283797)'':0.00434458)'':0.00339897)'':0.00372253)'':0.0101681)'':0.00617759)'':-0.00969753)'':0.020106)'':-0.0135948)'':0.0436031)'':-0.0304261)'':0.0158016,(('PDT000534794.1':0.0708883,('PDT000460509.1':0.0363049,('PDT000691334.1':0.0512403,(('PDT000691311.1':0.0256575,'PDT000691317.1':0.0207778)'':0.01133,('PDT000589788.1':0.0369384,('PDT000590023.1':0.0314185,('PDT000589766.1':0.031278,('PDT000433245.1':0.0252542,('PDT000202175.2':0.0160483,('PDT000136090.2':0.0121532,'PDT000632481.1':0.0113692)'':0.00142442)'':0.0111013)'':0.00222299)'':0.00371837)'':0.00351306)'':-0.00267512)'':0.010278)'':0.0085904)'':0.0156594)'':0.0108466,(((('PDT000604271.1':0.0511071,('PDT000633509.1':0.0233039,('PDT000023703.1':0.0278864,('PDT000030553.1':0.0229135,'PDT000175977.2':0.0245328)'':0.00222904)'':0.00250606)'':0.0185205)'':-0.00394199,(('PDT000619090.1':0.0175957,'PDT000538928.1':0.0150691)'':0.021667,('PDT000634073.1':0.0301033,(('PDT000423876.1':0.00853421,'PDT000581964.1':0.0115057)'':0.0057512,('PDT000705160.1':0.0160182,'PDT000668909.1':0.013759)'':0.00340523)'':0.0195305)'':0.00474173)'':0.00701177)'':0.0244347,(('PDT000123370.1':0.0503804,('PDT000425450.1':0.0316007,'PDT000521019.1':0.0311053)'':0.0129514)'':0.0203026,(('PDT000659285.1':0.0291547,'PDT000362488.1':0.0375582)'':0.0346777,(('PDT000175978.2':0.0473953,('PDT000633674.1':0.036013,('PDT000535097.1':0.0286936,'PDT000632289.1':0.0294357)'':0.00621682)'':0.0135964)'':-0.00143481,(('PDT000713589.1':0.0302302,('PDT000202164.2':0.0206193,'PDT000202597.2':0.0124311)'':0.00600264)'':0.00408603,('PDT000019902.3':0.0243249,('PDT000459531.2':0.0186967,'PDT000589952.1':0.0218226)'':0.0102912)'':0.000685076)'':0.0169986)'':0.0171847)'':-0.00116544)'':0.00526707)'':0.00155614,((('PDT000675083.1':0.0694059,(('PDT000320890.1':0.0364328,'PDT000136075.6':0.0466194)'':0.00294265,('PDT000202664.2':0.0258253,'PDT000425917.1':0.0192)'':0.0161389)'':0.0122195)'':0.00296272,(('PDT000426491.1':0.0438987,('PDT000616084.1':0.0306853,'PDT000704947.1':0.0276888)'':0.0138578)'':0.00983119,('PDT000716969.1':0.0377271,('PDT000705098.1':0.0380603,('PDT000136042.2':0.0319112,('PDT000423864.1':0.0276949,'PDT000705008.1':0.0272523)'':0.00250852)'':0.00660399)'':0.00412193)'':0.00762398)'':0.00505947)'':0.00643835,((('PDT000535355.1':0.0412744,'PDT000673629.1':0.0405964)'':0.00679211,(('PDT000431745.1':0.0230087,'PDT000618849.1':0.024671)'':0.0119168,('PDT000020191.1':0.0390902,('PDT000615928.1':0.0253984,'PDT000287616.1':0.0254606)'':0.00750144)'':0.00176192)'':0.0106343)'':0.0144497,(('PDT000101319.1':0.0296161,('PDT000635914.1':0.0128077,'PDT000501879.1':0.0104135)'':0.0215182)'':0.00480149,('PDT000202177.2':0.0396813,('PDT000253660.1':0.0168168,('PDT000287618.1':0.0197404,('PDT000202689.2':0.013553,'PDT000425541.1':0.0150355)'':0.003977)'':0.00158372)'':0.0189228)'':0.000954219)'':0.0154919)'':0.00612968)'':0.00739468)'':0.00282302)'':0.00151761)'':0.00513256)'':0.00622809)'':0.0287075)'':0.0325461)'':0.070277)'':0.00446104)'':0.0473838,((((('PDT000557330.1':0.162581,(('PDT000281732.2':0.0458547,'PDT000537440.2':0.0414967)'':0.101445,('PDT000502161.1':0.00920633,'PDT000627374.1':0.00873937)'':0.126492)'':0.0120367)'':0.0128853,('PDT000060658.2':0.130424,(('PDT000637170.1':0.0791931,('PDT000155952.2':0.0247456,'PDT000619702.1':0.0290952)'':0.048745)'':0.00876066,('PDT000535469.1':0.0444921,('PDT000242491.1':0.0327012,'PDT000361006.1':0.0367636)'':0.0178079)'':0.0482846)'':0.0141749)'':0.0683354)'':0.00699572,(('PDT000271749.2':0.0544433,('PDT000277163.3':0.0478666,('PDT000546142.2':0.0336028,('PDT000671358.1':0.0344703,('PDT000400834.2':0.0226149,('PDT000532403.2':0.0207304,'PDT000548700.2':0.0245051)'':0.00183164)'':0.00856884)'':0.000853499)'':0.0169302)'':0.0159875)'':0.0677947,(('PDT000233887.3':0.0440304,(('PDT000273390.3':0.0295104,'PDT000052443.3':0.019914)'':0.0106663,('PDT000300545.1':0.0304592,('PDT000498168.1':0.0306128,('PDT000334183.2':0.0204702,'PDT000047675.4':0.0221936)'':0.00327808)'':0.00504741)'':0.000861886)'':0.0144028)'':0.0860354,(('PDT000557162.1':0.0753665,('PDT000485244.1':0.011609,'PDT000661502.1':0.0111612)'':0.0626294)'':0.0232057,(('PDT000443249.2':0.0367651,'PDT000063267.3':0.0378992)'':0.0244734,((('PDT000316103.2':0.0414416,(('PDT000566856.2':0.0392574,('PDT000148170.3':0.0336038,'PDT000519330.2':0.0327317)'':0.00679893)'':0.00393922,(('PDT000498960.2':0.0135429,'PDT000011763.2':0.0155516)'':0.0193848,('PDT000621322.2':0.0235153,'PDT000040442.4':0.0200833)'':0.0132034)'':0.00870767)'':0.00337356)'':0.001576,(('PDT000651774.2':0.0422525,('PDT000160143.3':0.0195525,'PDT000469496.2':0.0185793)'':0.0272552)'':0.000885103,(('PDT000123474.3':0.028846,'PDT000172093.1':0.0276457)'':0.00230904,('PDT000645768.2':0.0321696,('PDT000671626.1':0.0253985,'PDT000362530.1':0.0288338)'':0.00856406)'':0.000197794)'':0.0130031)'':0.000674478)'':0.00106999,((('PDT000476904.2':0.0387926,'PDT000060992.3':0.0375212)'':0.00789479,(('PDT000069918.3':0.0337819,('PDT000387546.2':0.0236702,'PDT000479296.2':0.0236463)'':0.0113417)'':0.00200082,('PDT000324057.2':0.0408909,('PDT000534066.2':0.0326948,('PDT000157889.3':0.0309585,'PDT000566858.2':0.0309435)'':0.00624817)'':0.00366101)'':-0.000775332)'':0.00626061)'':0.00294082,('PDT000682781.1':0.0561322,((('PDT000491601.2':0.0225282,'PDT000641388.2':0.0247035)'':0.0119888,('PDT000318892.2':0.040845,('PDT000494507.2':0.0163008,('PDT000378624.2':0.014126,'PDT000626587.2':0.0140708)'':0.00304831)'':0.0183163)'':-0.000841401)'':0.0076955,('PDT000498123.1':0.0465288,((('PDT000341777.2':0.02607,'PDT000598472.2':0.0273063)'':0.00454925,('PDT000160259.3':0.0292175,('PDT000115433.3':0.0193747,('PDT000702594.1':0.0165923,'PDT000131883.3':0.016222)'':0.00164706)'':0.00765799)'':0.00289771)'':0.00157231,(('PDT000519367.2':0.0240933,('PDT000316557.2':0.0187978,'PDT000523448.2':0.0207847)'':0.00562508)'':0.00383551,(('PDT000316990.2':0.0179486,'PDT000534071.2':0.0156084)'':0.00457589,('PDT000440495.2':0.00439954,'PDT000586510.2':0.00498263)'':0.0168414)'':0.00403175)'':0.00362892)'':0.0178347)'':-0.0108004)'':0.0193037)'':-0.0121352)'':0.00201022)'':0.0172378)'':0.0599246)'':0.0249309)'':0.00398988)'':0.0457809)'':0.0344123,((('PDT000443104.1':0.100721,('PDT000346143.1':0.0828432,('PDT000520785.1':0.0739253,('PDT000178599.2':0.0166297,'PDT000397800.1':0.0201728)'':0.0586065)'':0.0101409)'':0.0171456)'':0.0111554,('PDT000251928.1':0.145896,(('PDT000354283.1':0.113643,('PDT000502489.1':0.0896357,(('PDT000397682.1':0.0650694,'PDT000346446.1':0.0740786)'':0.00131791,('PDT000426442.1':0.0556816,'PDT000652878.1':0.0600324)'':0.0162793)'':0.0135916)'':0.0122267)'':-0.00696157,(('PDT000538613.1':0.0430563,('PDT000202163.2':0.0368177,'PDT000633327.1':0.0347289)'':0.00943958)'':0.0144338,(('PDT000602013.2':0.0281966,'PDT000270466.1':0.0234457)'':0.0232994,(('PDT000502221.1':0.0302687,'PDT000218926.1':0.0296274)'':0.0144762,('PDT000720298.1':0.0400236,'PDT000352499.1':0.0391089)'':0.00250807)'':0.010191)'':0.0105937)'':0.0254905)'':0.0542208)'':-0.0318559)'':0.0735198,((('PDT000494523.1':0.157779,(('PDT000425719.1':0.0423633,'PDT000081722.1':0.0369491)'':0.0197071,('PDT000497237.1':0.00558959,'PDT000497261.1':0.00697651)'':0.0494464)'':0.100744)'':-0.00861885,((('PDT000146513.3':0.0394571,'PDT000042498.1':0.0400671)'':0.0162606,('PDT000589715.2':0.0301256,'PDT000712302.1':0.0270173)'':0.0287241)'':0.0627173,(('PDT000252457.1':0.101287,('PDT000538584.1':0.0908826,(('PDT000357717.1':0.0806646,'PDT000497356.1':0.0705924)'':0.00177221,('PDT000557673.2':0.0814834,('PDT000664989.1':0.0801654,(('PDT000651386.1':0.0680041,'PDT000289133.3':0.0700369)'':0.00131786,('PDT000502365.1':0.0743963,(('PDT000346237.1':0.0520483,('PDT000497353.1':0.0390897,'PDT000644926.1':0.0472722)'':0.0102295)'':0.010828,('PDT000359630.1':0.0695871,('PDT000325508.1':0.0565212,(('PDT000430733.1':0.0292228,'PDT000499266.2':0.034789)'':0.0188065,('PDT000354903.1':0.0372558,('PDT000312624.2':0.0339141,'PDT000444812.1':0.0314974)'':0.00395697)'':0.0108642)'':0.00163361)'':0.0119418)'':-0.00391334)'':0.00862744)'':-0.00139587)'':0.00944635)'':0.00184384)'':-0.00210747)'':0.00992137)'':0.0116459)'':-0.0185048,('PDT000490242.1':0.0874539,(('PDT000309683.3':0.052965,('PDT000161914.3':0.034789,('PDT000322754.2':0.0255178,'PDT000047677.4':0.0254767)'':0.00633002)'':0.0195625)'':0.00965327,('PDT000494177.2':0.068668,('PDT000317716.2':0.071661,(('PDT000143902.3':0.0374918,'PDT000288547.3':0.0340678)'':0.0254229,(('PDT000392399.2':0.0524551,(('PDT000340785.2':0.027542,'PDT000081059.1':0.0334085)'':0.00124909,('PDT000655523.2':0.0316544,('PDT000218426.3':0.0310876,('PDT000496166.2':0.0269444,('PDT000283374.3':0.0190414,(('PDT000494933.2':0.00230428,'PDT000646150.2':0.00198714)'':0.0127501,('PDT000307846.3':0.0105941,('PDT000233967.3':0.00977045,'PDT000394727.2':0.00898515)'':0.00205598)'':0.00425044)'':0.00169226)'':0.0054537)'':0.00182111)'':0.00308894)'':6.98834e-05)'':0.0194813)'':0.00422866,(('PDT000498087.1':0.0359037,('PDT000510688.2':0.029169,('PDT000532406.2':0.0334223,('PDT000103586.3':0.0226554,('PDT000279562.3':0.00814058,'PDT000128670.3':0.00725172)'':0.0146123)'':0.00328632)'':0.00108237)'':0.00549521)'':0.0120426,('PDT000378631.2':0.0448663,('PDT000396204.2':0.0387903,('PDT000325457.2':0.0262301,('PDT000490926.2':0.0192947,(('PDT000239272.3':0.0139337,'PDT000579285.2':0.013128)'':0.00666138,('PDT000336989.2':0.0214137,('PDT000143263.3':0.0242745,('PDT000499171.2':0.010122,('PDT000384008.2':0.007861,('PDT000443256.2':0.00645722,('PDT000300538.1':0.00302066,('PDT000190121.3':0.00271051,'PDT000225445.3':0.0024817)'':0.000368556)'':0.00270658)'':0.00145921)'':0.00392323)'':0.0061098)'':0.00140465)'':-0.00185811)'':0.00533392)'':0.0028114)'':0.0101967)'':0.00709569)'':0.00404644)'':0.00211639)'':0.00638844)'':0.0157468)'':0.00637591)'':-0.0147131)'':0.0219087)'':0.00114427)'':0.0194672)'':0.0480194)'':-0.0119768,((('PDT000274099.2':0.0595754,'PDT000088303.2':0.0720736)'':0.0262789,(('PDT000095817.2':0.0553275,('PDT000306429.2':0.0430923,(('PDT000399667.1':0.0156522,'PDT000468328.1':0.0198879)'':0.0111898,('PDT000497315.1':0.0161358,('PDT000479097.1':0.00752872,'PDT000395463.1':0.00706908)'':0.00970275)'':0.0104383)'':0.00714729)'':0.0206134)'':0.0146839,(('PDT000088291.2':0.0443905,('PDT000147461.3':0.0363467,('PDT000088277.2':0.0333388,'PDT000610390.1':0.0336125)'':0.00259109)'':0.00871498)'':0.00744222,('PDT000281211.2':0.0503905,('PDT000471027.1':0.0399764,('PDT000504799.1':0.0240192,('PDT000394865.1':0.0197366,'PDT000717112.1':0.0163973)'':0.00327072)'':0.0181954)'':0.00824469)'':0.00289083)'':0.0202004)'':0.00969851)'':0.0104511,((((('PDT000426246.1':0.0659774,('PDT000412512.1':0.0513203,('PDT000294651.2':0.0414062,'PDT000312636.2':0.0450563)'':0.0100774)'':0.0154071)'':0.00535672,('PDT000497311.1':0.0495264,('PDT000150251.2':0.0534063,('PDT000645114.1':0.0410129,('PDT000303329.2':0.0277105,'PDT000393091.1':0.0276985)'':0.00934305)'':0.00580306)'':0.00181506)'':0.0220577)'':0.00580099,((('PDT000466070.2':0.0363775,('PDT000213916.3':0.0154332,'PDT000708942.1':0.0160146)'':0.0100269)'':0.0157174,('PDT000524410.2':0.033177,('PDT000298484.3':0.0123642,('PDT000501775.2':0.00602413,'PDT000589629.2':0.00570667)'':0.00971448)'':0.0154626)'':0.0116881)'':0.0127537,('PDT000145396.3':0.0513862,((('PDT000357267.2':0.0385679,'PDT000085635.3':0.0359381)'':0.0068424,('PDT000404029.2':0.0394637,('PDT000490331.2':0.014366,('PDT000069898.3':0.00425547,'PDT000682720.1':0.00461349)'':0.0136291)'':0.0224864)'':0.00137635)'':0.00277031,(('PDT000163015.4':0.00827118,('PDT000574212.2':0.004815,('PDT000144453.3':0.00404217,'PDT000683594.1':0.0056404)'':0.00182175)'':0.00152337)'':0.0308517,('PDT000388874.2':0.0218962,('PDT000271277.3':0.0176341,('PDT000372100.2':0.0186805,('PDT000247443.3':0.00976046,'PDT000040397.4':0.013486)'':0.00449214)'':0.00202258)'':0.0063838)'':0.0143901)'':0.00849035)'':0.0063627)'':9.78598e-05)'':0.0249469)'':0.0129576,((((('PDT000209463.1':0.0575253,'PDT000252568.1':0.0546147)'':0.00514345,(('PDT000672270.1':0.0500762,('PDT000425537.1':0.0501643,'PDT000619643.1':0.043129)'':0.00327124)'':0.0100041,('PDT000517275.1':0.0599094,('PDT000209459.1':0.0447148,('PDT000362512.1':0.0324207,('PDT000425930.1':0.0330118,'PDT000426747.1':0.0278056)'':0.0057417)'':0.0165368)'':0.00829693)'':-0.000241029)'':0.00442641)'':0.00612305,(('PDT000426003.1':0.052514,('PDT000397569.1':0.0315078,'PDT000397617.1':0.0375999)'':0.0195617)'':0.0182656,(('PDT000399695.1':0.0457953,(('PDT000652966.1':0.0160818,'PDT000652999.1':0.0153654)'':0.0133709,(('PDT000644917.1':0.0162125,'PDT000656721.1':0.0174798)'':0.00180943,('PDT000300516.1':0.0113876,('PDT000167685.2':0.00715115,'PDT000566073.1':0.00865555)'':0.00546804)'':0.00729656)'':0.00583458)'':0.0153407)'':0.0182945,(('PDT000249913.1':0.0606598,('PDT000645095.1':0.0563347,('PDT000714177.1':0.0510236,('PDT000565923.1':0.0405482,('PDT000003951.2':0.035001,'PDT000136132.2':0.0358545)'':0.00229133)'':0.0109196)'':0.00871785)'':0.00331884)'':-0.00349981,('PDT000685880.1':0.0773599,('PDT000497224.1':0.056163,(('PDT000346123.1':0.0380357,'PDT000346215.1':0.0407873)'':0.0133388,(('PDT000661327.1':0.0386178,'PDT000167654.2':0.0377711)'':0.002502,(('PDT000102978.1':0.030122,'PDT000252609.1':0.0299655)'':0.00313137,('PDT000287998.1':0.0188824,'PDT000346178.1':0.0166939)'':0.0130976)'':0.00707997)'':0.00675043)'':0.00726651)'':0.00403193)'':-0.003302)'':0.00414054)'':0.00829099)'':-0.00150665)'':0.0125888,((('PDT000135987.2':0.0507246,'PDT000664985.1':0.0487589)'':0.0285698,('PDT000235761.3':0.0641735,'PDT000092876.1':0.0688625)'':0.0119937)'':0.00520503,('PDT000661570.1':0.128683,(('PDT000557216.1':0.0500887,'PDT000318980.1':0.0528653)'':0.012787,(('PDT000414215.1':0.0650342,(('PDT000492117.2':0.0263301,('PDT000649796.2':0.0228525,('PDT000326190.2':0.0175585,'PDT000379524.2':0.0201792)'':0.00364773)'':0.00371289)'':0.020336,('PDT000421821.1':0.0435771,(('PDT000131814.2':0.0277352,('PDT000167626.2':0.0201384,'PDT000471020.1':0.0235198)'':0.00355361)'':0.00905894,('PDT000244077.1':0.0520538,(('PDT000271326.2':0.0196048,'PDT000285393.2':0.0190606)'':0.000432572,('PDT000088304.2':0.0139484,'PDT000017688.2':0.0132766)'':0.00477568)'':0.0223916)'':-0.00501116)'':0.00404332)'':0.00286805)'':0.01999)'':-0.0158712,(('PDT000622534.1':0.0489879,'PDT000468842.1':0.0416301)'':0.0183149,(('PDT000209460.1':0.0448011,'PDT000411343.1':0.039617)'':0.00592891,('PDT000463682.1':0.0645937,(('PDT000497258.1':0.0430797,(('PDT000006907.2':0.0173821,'PDT000242977.1':0.0162239)'':0.0135312,('PDT000420678.1':0.0158567,('PDT000503432.1':0.0181081,'PDT000714108.1':0.0177395)'':0.00041638)'':0.00558628)'':0.0099219)'':-0.00372767,(('PDT000397977.1':0.0257162,'PDT000019597.1':0.0271933)'':0.00909716,('PDT000426607.1':0.0301024,('PDT000519805.1':0.0323442,('PDT000619559.1':0.0327465,('PDT000425493.1':0.0221567,('PDT000167659.2':0.0221306,('PDT000248474.1':0.0146064,'PDT000019617.1':0.0135403)'':0.00515565)'':0.00562943)'':0.0033112)'':0.00230705)'':0.00166635)'':0.00106105)'':0.00181413)'':0.0209626)'':-0.00713725)'':0.00885063)'':-0.00930886)'':0.0147636)'':0.0364236)'':-0.0233107)'':-0.00137336)'':0.00235609,((('PDT000200186.2':0.0734472,('PDT000167716.2':0.0395194,'PDT000300818.2':0.0316501)'':0.0135726)'':0.026854,('PDT000645595.2':0.102924,((('PDT000566162.1':0.0413858,('PDT000394953.1':0.0163095,'PDT000687494.1':0.0156117)'':0.0166597)'':0.0175393,('PDT000652997.1':0.0226845,('PDT000653011.1':0.0118134,'PDT000497221.1':0.0116311)'':0.0093804)'':0.0275396)'':0.00822106,('PDT000713263.1':0.069258,(('PDT000397544.1':0.0502831,'PDT000209462.1':0.0487438)'':0.0104605,('PDT000713451.1':0.0585001,('PDT000661432.1':0.046835,('PDT000644900.1':0.0327647,('PDT000184584.1':0.0367144,('PDT000394940.1':0.0356932,('PDT000616363.1':0.028437,('PDT000399661.1':0.00851609,'PDT000494426.1':0.00785461)'':0.0209609)'':0.00746532)'':0.00110949)'':0.00142777)'':0.00520829)'':0.00911139)'':0.000515583)'':0.00988351)'':-0.00766131)'':0.0326695)'':-0.0138262)'':0.00199409,(((('PDT000396748.1':0.0628984,'PDT000429817.1':0.0568336)'':0.0242283,('PDT000346852.1':0.0792672,('PDT000477087.1':0.0701736,'PDT000566013.1':0.0745994)'':0.00337027)'':0.00310574)'':0.00359028,('PDT000687915.1':0.0811389,((('PDT000427542.1':0.0410448,'PDT000652906.1':0.0435594)'':0.0132176,('PDT000510033.1':0.049741,(('PDT000430767.1':0.0179974,'PDT000468236.1':0.0189523)'':0.027703,('PDT000394885.1':0.0430464,('PDT000395063.1':0.0352897,'PDT000438091.1':0.0281254)'':0.0101931)'':0.00364261)'':0.00462988)'':0.00311422)'':0.00129539,('PDT000644857.1':0.033501,('PDT000639525.1':0.0405396,('PDT000565846.1':0.031301,('PDT000286931.2':0.0237553,('PDT000395109.1':0.0164034,('PDT000645123.1':0.0157805,('PDT000566067.1':0.0122944,'PDT000395288.1':0.0101987)'':0.00283162)'':0.000585218)'':0.00747965)'':0.00184784)'':0.007693)'':0.00398874)'':0.0168911)'':0.017768)'':0.00909377)'':-0.00732249,(('PDT000417183.1':0.0697089,('PDT000210714.1':0.0613586,'PDT000312662.1':0.0498064)'':0.0115031)'':0.0207434,((('PDT000397919.1':0.060531,'PDT000319006.1':0.051074)'':0.00675873,('PDT000395466.1':0.0554214,(('PDT000468907.1':0.0245387,'PDT000468919.1':0.016907)'':0.0207094,('PDT000354235.1':0.0445302,('PDT000354277.1':0.0425631,('PDT000235757.2':0.0294901,('PDT000313325.2':0.0168532,'PDT000550336.1':0.0196504)'':0.010885)'':0.00622591)'':0.00437751)'':-0.000121987)'':0.0115368)'':0.00240488)'':0.0129719,((('PDT000426555.1':0.063602,'PDT000497255.1':0.063895)'':0.0118501,((('PDT000492233.1':0.065065,('PDT000426106.1':0.0466649,'PDT000661389.1':0.0610421)'':0.00966845)'':0.00210985,(('PDT000188129.2':0.0497086,'PDT000385037.1':0.0627034)'':0.0053482,('PDT000645058.1':0.0129292,'PDT000395227.1':0.0119469)'':0.0475059)'':0.00271552)'':0.001996,((('PDT000051499.2':0.0298447,'PDT000566029.1':0.0358931)'':0.00956938,('PDT000352458.1':0.0286091,'PDT000395319.1':0.036557)'':0.00881609)'':0.0259449,(('PDT000171805.1':0.0447594,('PDT000407295.1':0.0229464,'PDT000430731.1':0.0199432)'':0.0258763)'':0.00891928,(('PDT000323538.1':0.0267158,'PDT000645024.1':0.0302419)'':0.00930425,('PDT000644976.1':0.0334834,('PDT000566075.1':0.0294617,'PDT000652922.1':0.0303674)'':0.00476)'':0.00385088)'':0.0257342)'':0.00329376)'':0.00213488)'':0.00458532)'':-0.0021009,('PDT000046599.1':0.0837505,(((('PDT000395080.1':0.0165408,'PDT000540524.1':0.011165)'':0.0311358,('PDT000474440.1':0.0512202,('PDT000276017.1':0.0380715,'PDT000468911.1':0.0379851)'':0.00966444)'':8.78393e-05)'':0.00160573,('PDT000645122.1':0.0484964,(('PDT000566149.1':0.0226498,'PDT000395435.1':0.0239006)'':0.00412075,('PDT000395441.1':0.0141151,('PDT000600651.1':0.0135098,'PDT000644887.1':0.0102166)'':0.0091613)'':0.00588105)'':0.0140273)'':0.00740464)'':0.00846307,(('PDT000465220.2':0.0296108,'PDT000081155.3':0.03027)'':0.0287287,(('PDT000397843.1':0.0320508,('PDT000566095.1':0.0221318,'PDT000644982.1':0.0222143)'':0.0117582)'':-0.00239144,(('PDT000497384.1':0.0190587,('PDT000565880.1':0.0144503,'PDT000645057.1':0.0165781)'':0.00130465)'':0.00861553,(('PDT000264730.2':0.00267385,'PDT000482529.1':0.00222028)'':0.020913,('PDT000504789.1':0.0136795,'PDT000550363.1':0.0131044)'':0.00898855)'':0.00294343)'':0.00889283)'':0.0244933)'':-0.00294186)'':0.0228933)'':-0.0143721)'':0.00239798)'':0.016764)'':-0.0106339)'':0.00329433)'':0.000283343)'':0.00652553)'':-3.58552e-05,(((('PDT000190871.1':0.0944384,('PDT000241774.1':0.0664931,('PDT000306655.2':0.0235233,'PDT000306673.2':0.018584)'':0.0432993)'':0.0245993)'':-0.00372543,('PDT000652881.1':0.0804279,('PDT000397840.1':0.0195148,('PDT000566026.1':0.0105039,('PDT000386723.1':0.0089707,'PDT000397688.1':0.0072708)'':0.00391382)'':0.00629172)'':0.0384488)'':0.0222992)'':0.00855689,((('PDT000664958.1':0.0636253,'PDT000209458.1':0.0556537)'':0.0246265,('PDT000095709.2':0.0553917,(('PDT000040376.4':0.0281835,('PDT000503052.2':0.00682791,'PDT000720112.1':0.00780189)'':0.0210622)'':0.00972471,('PDT000137336.3':0.0381686,('PDT000273344.3':0.0212783,('PDT000203681.1':0.00372395,'PDT000270498.1':0.00455866)'':0.0141079)'':0.0131441)'':0.00694454)'':0.0181181)'':0.025492)'':-6.11379e-05,((('PDT000298016.1':0.0593256,('PDT000167646.2':0.0188051,'PDT000124595.1':0.0230334)'':0.0282967)'':0.00616183,('PDT000046601.1':0.0695676,(('PDT000158799.2':0.0539318,'PDT000025499.2':0.0553992)'':0.00602915,('PDT000027904.2':0.051852,('PDT000024326.2':0.0314897,'PDT000542094.1':0.0339099)'':0.0105139)'':0.0133853)'':0.00739371)'':-0.00185503)'':-0.000913598,(('PDT000286091.1':0.0732041,(('PDT000557093.1':0.0267924,'PDT000352432.1':0.0273546)'':0.0309764,('PDT000252449.1':0.0626278,('PDT000251388.1':0.0473344,('PDT000250183.1':0.0135246,('PDT000718866.1':0.0132104,('PDT000432496.1':0.00584314,'PDT000066960.2':0.00604496)'':0.00370211)'':0.00383485)'':0.0261184)'':0.015679)'':0.0029294)'':0.0165874)'':-0.0137115,('PDT000395028.1':0.0754262,('PDT000274101.2':0.0614775,('PDT000399629.1':0.0486521,('PDT000575120.1':0.0493939,('PDT000394842.1':0.0344831,('PDT000397657.1':0.0261545,('PDT000645115.1':0.0217117,('PDT000644840.1':0.018924,('PDT000394886.1':0.013024,'PDT000395010.1':0.00881919)'':0.00566973)'':0.00426578)'':0.00720989)'':0.00368542)'':0.013953)'':0.00575989)'':0.00925861)'':0.0209738)'':-0.0155016)'':0.000365091)'':0.0219599)'':0.00908569)'':-0.00087012,(('PDT000417126.1':0.069658,('PDT000626016.1':0.0565989,('PDT000352440.1':0.0448458,(('PDT000352443.1':0.00429951,'PDT000354279.1':0.00556203)'':0.00785632,('PDT000354251.1':0.00806152,('PDT000188357.1':0.00244503,'PDT000188372.1':0.00254549)'':0.00714227)'':0.00730548)'':0.0171194)'':0.027226)'':0.0010607)'':0.0384918,((('PDT000188165.2':0.079943,(('PDT000538586.1':0.0728748,'PDT000290004.1':0.0684802)'':0.00513736,('PDT000417131.1':0.0616615,(('PDT000645545.2':0.0613124,'PDT000417196.1':0.0577576)'':0.00176368,('PDT000417108.1':0.0493664,'PDT000346046.1':0.0495106)'':0.00820632)'':0.0093085)'':0.00402824)'':0.0130165)'':-0.00238923,('PDT000354236.1':0.0801389,((('PDT000713545.1':0.035973,'PDT000067792.1':0.0343083)'':0.00989244,('PDT000618441.1':0.0388087,'PDT000017770.2':0.0384188)'':0.00444036)'':0.0173968,(('PDT000352437.1':0.0390008,'PDT000354289.1':0.0420048)'':0.0143265,('PDT000352460.1':0.0396396,(('PDT000352462.1':0.00296488,'PDT000352463.1':0.00281835)'':0.019725,('PDT000354237.1':0.0303275,('PDT000352444.1':0.0194344,('PDT000352448.1':0.00526095,'PDT000354246.1':0.00530205)'':0.00961931)'':0.00572752)'':0.00273154)'':0.0127058)'':0.000812227)'':0.0258282)'':0.0124414)'':0.00381878)'':0.00614741,(('PDT000124611.1':0.105794,('PDT000124602.1':0.0908124,(('PDT000167675.2':0.0852056,'PDT000168861.1':0.0859034)'':0.00377713,(('PDT000645571.2':0.0630023,'PDT000468833.1':0.0666257)'':0.01515,('PDT000354275.1':0.0829785,('PDT000136717.1':0.0840306,('PDT000664955.1':0.0664809,(('PDT000399713.1':0.0536429,('PDT000203070.2':0.0275063,'PDT000077132.2':0.0339127)'':0.0268046)'':0.00096485,('PDT000428420.1':0.0525101,('PDT000645557.2':0.0548826,('PDT000204904.2':0.0388615,'PDT000004346.2':0.0423813)'':0.0144456)'':0.00172564)'':0.0046897)'':0.0119359)'':0.00658816)'':0.0038249)'':-0.00178171)'':0.00680749)'':0.00475974)'':0.0138999)'':-0.00782328,(('PDT000687819.1':0.0719273,('PDT000003774.2':0.0567979,('PDT000395281.1':0.0497126,(('PDT000566708.1':0.0507036,'PDT000604477.1':0.0422069)'':0.00370634,('PDT000637168.1':0.0329726,'PDT000352453.1':0.0297157)'':0.0113722)'':0.00427173)'':0.00434112)'':0.0149915)'':0.018948,((('PDT000318884.1':0.055366,'PDT000262733.1':0.062987)'':0.0272597,('PDT000128849.2':0.0763262,'PDT000425538.1':0.0790688)'':0.0240968)'':0.00130472,((('PDT000293492.1':0.113251,(('PDT000718883.1':0.0583932,'PDT000718890.1':0.0603328)'':0.0221135,(('PDT000687404.1':0.0492263,'PDT000346618.1':0.0558277)'':0.0127523,('PDT000346439.1':0.0354579,('PDT000128788.2':0.0234524,'PDT000482835.1':0.0233261)'':0.00845031)'':0.0406935)'':0.0143275)'':0.00932179)'':-0.00503316,(('PDT000636325.1':0.103352,(('PDT000468880.1':0.0502155,('PDT000652857.1':0.0508998,'PDT000652862.1':0.0534442)'':0.00341498)'':0.00763675,('PDT000167628.2':0.0672227,('PDT000253654.1':0.0506933,('PDT000425458.1':0.0411087,'PDT000425809.1':0.0437451)'':0.0140218)'':0.0126974)'':-0.00117859)'':0.0396156)'':-0.0187749,(('PDT000400021.1':0.0958615,('PDT000264656.2':0.0351592,'PDT000382206.1':0.0341028)'':0.054057)'':0.00149207,(('PDT000181841.2':0.0798244,('PDT000181654.2':0.0721373,(('PDT000223304.1':0.0289953,'PDT000395168.1':0.0310554)'':0.0119989,('PDT000610199.1':0.0360961,(('PDT000497344.1':0.0131939,'PDT000497406.1':0.0114327)'':0.0114662,('PDT000397972.1':0.01996,('PDT000394838.1':0.00874129,'PDT000397773.1':0.0118701)'':0.0139201)'':0.00482845)'':0.0116042)'':-0.00152328)'':0.0217105)'':0.017948)'':-0.00689122,(('PDT000372313.1':0.0743647,('PDT000095675.2':0.0404567,'PDT000584738.1':0.0415772)'':0.0300693)'':0.00971731,(('PDT000250076.1':0.0837534,('PDT000619627.1':0.0489896,('PDT000060090.2':0.0241621,'PDT000566031.1':0.019071)'':0.0295816)'':0.023614)'':0.00105262,((('PDT000311255.2':0.0450399,('PDT000203098.4':0.0487023,'PDT000395233.1':0.0457363)'':0.00298365)'':0.0263777,(('PDT000399657.1':0.0589958,('PDT000307425.2':0.0301852,'PDT000397799.1':0.0357735)'':0.0155053)'':0.0166813,(('PDT000619666.1':0.0456827,('PDT000167637.2':0.0321776,'PDT000520710.1':0.0363849)'':0.0187851)'':0.00733203,('PDT000501988.1':0.0591117,('PDT000619578.1':0.0416767,'PDT000141600.1':0.0418735)'':0.0150357)'':-2.48317e-05)'':0.00817177)'':0.00396143)'':-0.0024445,('PDT000209461.1':0.0814562,(('PDT000251450.1':0.0633264,('PDT000588051.1':0.0687488,(('PDT000241776.1':0.054466,('PDT000384904.1':0.0339395,'PDT000600987.1':0.0299524)'':0.0135221)'':0.00255569,(('PDT000619607.1':0.0420981,'PDT000497239.1':0.0460525)'':0.00256993,('PDT000251434.1':0.035328,'PDT000252735.1':0.0400976)'':0.00615399)'':0.00396077)'':0.0165381)'':0.00172597)'':-0.00766341,(('PDT000124605.1':0.0626764,('PDT000618787.1':0.0498573,(('PDT000521367.1':0.0332894,('PDT000032363.1':0.0239409,'PDT000250162.1':0.0226488)'':0.0126646)'':0.00467177,(('PDT000709082.1':0.0348421,'PDT000502165.1':0.034713)'':0.00958104,('PDT000228284.1':0.0499059,(('PDT000652988.1':0.0297551,('PDT000397731.1':0.0187684,('PDT000652992.1':0.00450168,'PDT000346130.1':0.00436639)'':0.0177119)'':0.0102481)'':0.00313689,('PDT000468252.1':0.0317857,('PDT000395025.1':0.0264452,('PDT000027738.2':0.0233382,('PDT000167670.2':0.0117064,'PDT000220291.1':0.0146146)'':0.00826112)'':0.00372276)'':0.00289653)'':0.00152584)'':0.0115427)'':-0.00393895)'':0.0045617)'':0.0131611)'':0.0043052)'':-0.00889001,(('PDT000497257.1':0.047257,('PDT000466963.1':0.0301473,('PDT000343723.1':0.0219956,'PDT000252515.1':0.0218115)'':0.00920379)'':0.0150227)'':0.00346249,(('PDT000027893.2':0.0407145,('PDT000051504.2':0.0130505,'PDT000399689.1':0.0160584)'':0.0291555)'':0.00204851,(('PDT000399696.1':0.0404452,('PDT000497373.1':0.0279964,(('PDT000564937.1':0.0207037,'PDT000644997.1':0.0196855)'':0.00143421,('PDT000631553.1':0.00289449,('PDT000639548.1':0.00195503,'PDT000644954.1':0.00263946)'':0.00129853)'':0.0104403)'':0.00829382)'':0.0104222)'':0.0067909,('PDT000557150.1':0.0533634,(('PDT000566117.1':0.0378177,'PDT000395365.1':0.0411528)'':0.000172692,(('PDT000167717.2':0.0224997,('PDT000397935.1':0.0225895,('PDT000399681.1':0.0120778,'PDT000346444.1':0.0160856)'':0.00959436)'':0.00149055)'':0.00908044,('PDT000713398.1':0.0415279,('PDT000395290.1':0.025553,('PDT000397646.1':0.0088095,'PDT000397686.1':0.0075447)'':0.0135392)'':0.00825825)'':0.000192728)'':0.00895625)'':0.0126442)'':-0.0121558)'':0.00255447)'':0.00915884)'':0.00141135)'':0.00632097)'':0.0131425)'':-0.00779476)'':0.0091205)'':0.00481625)'':-0.00997754)'':0.0115144)'':-0.00298871)'':0.00968872)'':-0.0127853,(('PDT000251778.1':0.0844778,'PDT000011193.2':0.0907052)'':0.027448,('PDT000395220.1':0.105932,(('PDT000397882.1':0.0637752,'PDT000251863.1':0.0591688)'':0.0241043,(('PDT000720455.1':0.0833267,(('PDT000212788.1':0.0424864,'PDT000250259.1':0.0441506)'':0.0171681,(('PDT000394860.1':0.0372834,'PDT000519204.1':0.0386255)'':0.0101403,('PDT000395036.1':0.0469072,('PDT000394979.1':0.0346152,'PDT000550358.1':0.0359685)'':0.00652839)'':0.00210086)'':0.0127627)'':0.0107707)'':0.00607075,((('PDT000645680.2':0.0951967,('PDT000645022.1':0.0590284,('PDT000137839.2':0.0480309,('PDT000399654.1':0.046027,('PDT000426079.1':0.03677,'PDT000445047.1':0.0410574)'':0.00711719)'':0.0065118)'':0.00447463)'':0.0267277)'':-0.0159302,('PDT000565901.1':0.0417466,('PDT000518066.1':0.0494513,(('PDT000425592.1':0.0132013,'PDT000520970.1':0.0134575)'':0.00859231,('PDT000652942.1':0.0230772,'PDT000241777.1':0.0207514)'':0.00567934)'':0.019557)'':0.0044345)'':0.0204255)'':0.0038883,(((('PDT000718571.1':0.0372365,('PDT000248199.2':0.0361334,('PDT000468331.1':0.0216368,'PDT000468335.1':0.0173703)'':0.0147567)'':0.00769217)'':0.00409813,(('PDT000027912.2':0.036791,'PDT000395472.1':0.0322349)'':0.00585757,('PDT000399632.1':0.0157803,('PDT000399690.1':0.00702891,'PDT000399712.1':0.00676139)'':0.00767443)'':0.01895)'':0.0047587)'':0.0206018,(('PDT000364416.1':0.0615527,('PDT000034495.2':0.0502174,'PDT000251520.1':0.0534276)'':0.00950181)'':0.0123376,(('PDT000395500.1':0.0476406,('PDT000027891.2':0.0274876,(('PDT000394810.1':0.00936394,'PDT000645143.1':0.0124056)'':0.00560436,('PDT000468289.1':0.0134617,'PDT000471022.1':0.0118773)'':0.00697482)'':0.00643696)'':0.0263052)'':0.0128594,(('PDT000395473.1':0.0562938,('PDT000652994.1':0.0307071,'PDT000049098.1':0.0331622)'':0.0247676)'':0.00650715,((('PDT000660405.1':0.0147685,'PDT000394904.1':0.0153395)'':0.00699075,('PDT000060532.2':0.025464,('PDT000397706.1':0.0103315,'PDT000397715.1':0.011399)'':0.00981123)'':0.00362999)'':0.0136699,(('PDT000027908.2':0.0153066,('PDT000034463.2':0.00222221,'PDT000034480.2':0.00276831)'':0.00720291)'':0.0202306,('PDT000025495.2':0.0269087,('PDT000397543.1':0.0100263,'PDT000397690.1':0.0069161)'':0.0145696)'':0.00650951)'':0.00766181)'':0.0216974)'':0.00227355)'':0.00960269)'':-0.00631039)'':0.00170017,('PDT000346017.1':0.109598,(('PDT000653003.1':0.0394307,'PDT000395268.1':0.037072)'':0.0359816,(((('PDT000397841.1':0.0167857,'PDT000622445.1':0.0210871)'':0.0291625,('PDT000539350.1':0.0257097,('PDT000025244.2':0.017834,'PDT000395072.1':0.0181401)'':0.00447678)'':0.0301455)'':0.00315899,(('PDT000397829.1':0.0631314,'PDT000042682.1':0.0594116)'':0.00259853,(('PDT000145326.2':0.044364,('PDT000282981.2':0.0350551,'PDT000329062.1':0.0374016)'':0.00616684)'':0.00162094,('PDT000004398.2':0.0471686,('PDT000645138.1':0.0477861,('PDT000394899.1':0.0366997,('PDT000394811.1':0.00784376,'PDT000497411.1':0.00574804)'':0.0282252)'':0.00552435)'':0.00304283)'':0.00333385)'':0.0116052)'':-0.00168451)'':0.00381463,(('PDT000645521.2':0.0557253,('PDT000652834.1':0.0449171,(('PDT000652831.1':0.0217539,'PDT000652969.1':0.01713)'':0.00999485,('PDT000652844.1':0.0137947,'PDT000652921.1':0.0172384)'':0.0151882)'':0.0109809)'':0.00353149)'':0.0144502,((('PDT000566177.1':0.0227875,'PDT000652929.1':0.0201797)'':0.0274554,('PDT000645540.2':0.0548959,(('PDT000652913.1':0.0314035,'PDT000346110.1':0.0327401)'':0.0111673,('PDT000128835.2':0.0462611,('PDT000500526.1':0.0297311,('PDT000397991.1':0.0231509,('PDT000397830.1':0.0204743,'PDT000397907.1':0.0210574)'':0.00401901)'':0.00420914)'':0.00517953)'':0.0035695)'':0.0110875)'':-0.000843717)'':0.00274733,(('PDT000313324.2':0.0449607,('PDT000374997.1':0.051173,('PDT000397668.1':0.0246986,'PDT000652909.1':0.0271438)'':0.0113895)'':0.0118204)'':0.00506919,((('PDT000323526.1':0.0411332,'PDT000399621.1':0.0431786)'':0.00106375,('PDT000026892.2':0.0152574,'PDT000399684.1':0.0116484)'':0.0277823)'':0.00328806,(('PDT000556906.1':0.00872747,'PDT000718841.1':0.0118584)'':0.0332942,('PDT000019618.1':0.037463,('PDT000399680.1':0.0304353,'PDT000395492.1':0.0322162)'':0.00562499)'':0.00374744)'':0.00198387)'':0.0112403)'':-8.04933e-05)'':0.00833986)'':-0.00233842)'':0.0150725)'':0.0263326)'':-0.0375805)'':0.00364308)'':0.0119923)'':0.00508336)'':0.0174948)'':0.0125723)'':-0.0363789)'':0.0135018)'':0.000632024)'':-0.00439662)'':0.00097641)'':0.00632584)'':-0.00382377)'':-0.00183493)'':0.00987383)'':0.0425256)'':0.0436069)'':0.0291933)'':0.0422533,(((((('PDT000441815.1':0.0541978,'PDT000619638.1':0.0441089)'':0.127659,(('PDT000167652.2':0.024692,'PDT000362495.1':0.0228709)'':0.136531,((('PDT000719288.1':0.0862671,('PDT000319028.1':0.0514727,('PDT000425777.1':0.0379274,'PDT000713961.1':0.0335082)'':0.0181732)'':0.0331303)'':0.00434353,(('PDT000713304.1':0.0787188,('PDT000077101.2':0.0684748,'PDT000618445.1':0.0514942)'':0.0157297)'':0.0114415,('PDT000017909.2':0.0676963,('PDT000694400.1':0.0610814,('PDT000394841.1':0.0236816,'PDT000395128.1':0.0234393)'':0.0369942)'':0.00797084)'':0.0164349)'':0.00351091)'':0.0556575,(('PDT000719010.1':0.078467,('PDT000384906.1':0.0488874,'PDT000619587.1':0.051017)'':0.0249033)'':0.0398683,('PDT000006900.2':0.104678,('PDT000713694.1':0.0808284,('PDT000017752.2':0.0703609,('PDT000639760.1':0.0568914,(('PDT000543335.1':0.040613,('PDT000042554.1':0.0281727,'PDT000103066.1':0.0248427)'':0.00781525)'':0.0135374,(('PDT000716893.1':0.0486901,'PDT000251572.1':0.0492288)'':0.00289909,('PDT000426259.1':0.0532555,('PDT000713698.1':0.0487629,('PDT000008782.2':0.0305062,(('PDT000566852.1':0.0200304,'PDT000588054.1':0.0207438)'':0.00395687,('PDT000687926.1':0.0197218,'PDT000713793.1':0.0200911)'':0.00249203)'':0.00614813)'':0.00793694)'':0.00987145)'':-0.00217379)'':0.00140887)'':0.0108685)'':0.00384414)'':0.00580744)'':0.0158199)'':0.0238307)'':0.0351282)'':0.00848518)'':0.00209943)'':-0.000708949,(('PDT000228296.1':0.0768918,('PDT000258434.2':0.0222765,'PDT000252631.1':0.0203188)'':0.0606015)'':0.0993692,((('PDT000522682.1':0.0447616,('PDT000391673.1':0.0196527,'PDT000543111.1':0.0209131)'':0.0222817)'':0.0976297,(('PDT000309203.2':0.0897382,'PDT000251592.1':0.0967288)'':0.00356713,((('PDT000309344.2':0.0332449,'PDT000719120.1':0.0359606)'':0.0249653,('PDT000499447.1':0.0618418,'PDT000248781.1':0.0633812)'':0.00494535)'':0.0050357,(('PDT000719031.1':0.0047849,('PDT000343400.1':0.00147361,'PDT000343406.1':0.0019213)'':0.0054034)'':0.0415512,(('PDT000719050.1':0.0255942,('PDT000718834.1':0.0135667,'PDT000042495.1':0.0117245)'':0.00956999)'':0.00204255,('PDT000460003.1':0.0264835,('PDT000354895.1':0.0265549,('PDT000426730.1':0.0234486,('PDT000251590.1':0.0230267,('PDT000310632.2':0.0190744,'PDT000557111.1':0.0201759)'':0.00252483)'':0.00134211)'':0.00212679)'':0.00314618)'':0.00129435)'':0.0198292)'':0.0188428)'':0.0216737)'':0.0551346)'':0.00578127,((('PDT000661721.1':0.0313153,'PDT000501907.1':0.0253162)'':0.0759978,('PDT000535131.1':0.0958978,(('PDT000042447.1':0.0828952,('PDT000713741.1':0.0120735,'PDT000304219.1':0.0152937)'':0.0615032)'':-0.000682673,('PDT000403083.1':0.0876297,('PDT000459534.1':0.0664669,('PDT000502147.1':0.0522299,('PDT000647952.1':0.022211,'PDT000502116.1':0.0203883)'':0.0195323)'':0.0227555)'':0.00409081)'':0.00238416)'':0.0194589)'':0.00411586)'':0.00122929,((('PDT000276025.1':0.0373256,'PDT000346489.1':0.0326599)'':0.0223655,('PDT000354152.1':0.0440125,('PDT000661461.1':0.0104888,'PDT000346695.1':0.00969409)'':0.0391209)'':0.00872677)'':0.00859986,('PDT000042484.1':0.114344,('PDT000618889.1':0.0593923,(('PDT000674906.1':0.0632458,'PDT000502021.1':0.0551162)'':0.000767361,(('PDT000535080.1':0.0337101,('PDT000557335.1':0.0219822,('PDT000719189.1':0.0147889,'PDT000719190.1':0.0141437)'':0.00454142)'':0.0122598)'':-0.00278227,('PDT000249891.1':0.0229254,('PDT000713714.1':0.0173622,('PDT000717139.1':0.0148018,('PDT000714119.1':0.0114411,('PDT000042534.1':0.00903152,('PDT000719053.1':0.00654335,('PDT000175365.2':0.00512083,'PDT000549080.1':0.00656517)'':0.000559749)'':0.00131469)'':0.00300833)'':0.00305297)'':0.00219666)'':0.00306885)'':0.0103151)'':0.0222391)'':0.00487705)'':0.0236331)'':-0.0119892)'':0.0359245)'':0.0399136)'':0.0219657)'':-0.0122989)'':0.00214497,((('PDT000652470.1':0.0665937,('PDT000652472.1':0.0139411,'PDT000709900.1':0.0241067)'':0.0620249)'':0.0957179,((('PDT000502152.1':0.129655,('PDT000502094.1':0.0192201,'PDT000502109.1':0.0220516)'':0.110121)'':0.013883,(('PDT000004018.2':0.00583278,'PDT000395352.1':0.00505522)'':0.0754635,(('PDT000394796.1':0.0502433,'PDT000677987.1':0.0490057)'':0.00429215,('PDT000017848.2':0.0522918,('PDT000213220.2':0.0359663,('PDT000644949.1':0.0221413,'PDT000017993.2':0.0217366)'':0.0118801)'':0.017657)'':0.000756257)'':0.0191998)'':0.0680853)'':0.00248998,((('PDT000433817.1':0.0316319,('PDT000470073.1':0.0168785,'PDT000672384.1':0.0171934)'':0.0100183)'':0.0730499,('PDT000542728.1':0.100129,('PDT000017836.2':0.0707783,('PDT000285822.3':0.0596242,('PDT000710575.1':0.0384316,'PDT000604966.1':0.0377066)'':0.0115337)'':0.0144618)'':0.0402161)'':-0.000583913)'':0.0151285,((('PDT000426286.1':0.0268573,'PDT000160404.3':0.0287686)'':0.00417071,('PDT000442307.1':0.0226031,('PDT000713433.1':0.0182762,('PDT000137859.2':0.0122749,'PDT000011817.2':0.0114944)'':0.00506225)'':0.00153459)'':0.007647)'':0.0277571,(('PDT000394880.1':0.0450451,'PDT000566147.1':0.0463445)'':0.00693393,(('PDT000397884.1':0.0492261,('PDT000639524.1':0.0100431,'PDT000687775.2':0.00901781)'':0.027968)'':0.00624708,('PDT000444810.1':0.0374355,('PDT000627597.1':0.0324651,('PDT000188150.2':0.0182092,'PDT000069661.1':0.0213752)'':0.00908937)'':0.00322758)'':0.00845365)'':0.00487551)'':0.00756544)'':0.0579984)'':0.025661)'':0.0166071)'':-0.00828219,(('PDT000668487.2':0.0955316,(((('PDT000025250.2':0.0377572,'PDT000346456.1':0.0369516)'':0.0363701,(('PDT000557105.1':0.0528156,'PDT000426680.1':0.0505964)'':0.004268,('PDT000427247.1':0.0352484,'PDT000281485.1':0.0249615)'':0.0284665)'':0.00968966)'':0.00357583,('PDT000557253.1':0.0757105,(('PDT000200108.2':0.035002,'PDT000305090.2':0.0370762)'':0.0268519,('PDT000532151.1':0.0608888,(('PDT000656954.1':0.051613,'PDT000394909.1':0.0475621)'':0.00301806,('PDT000563743.2':0.0385999,('PDT000396189.1':0.0269532,'PDT000250011.1':0.0330567)'':0.00360629)'':0.00938921)'':0.0143907)'':-7.3422e-05)'':0.0093401)'':-0.00412926)'':0.000822813,((('PDT000604281.1':0.0490534,('PDT000248786.1':0.0360165,'PDT000251824.1':0.036383)'':0.00810149)'':0.0062585,(('PDT000397616.1':0.04039,'PDT000671872.1':0.0412034)'':0.00450132,('PDT000395145.1':0.0400938,('PDT000565935.1':0.0125192,'PDT000565947.1':0.0126194)'':0.0244791)'':0.00697211)'':0.00671052)'':0.00467211,('PDT000669461.1':0.0601067,('PDT000484008.2':0.0323689,('PDT000140222.3':0.0217792,('PDT000367726.2':0.0135445,(('PDT000510671.2':0.00521507,'PDT000120826.3':0.00387856)'':0.00450355,('PDT000604154.2':0.00651172,('PDT000366128.2':0.00327243,'PDT000386184.2':0.00271978)'':0.00195786)'':0.00249885)'':0.0040544)'':0.00569269)'':0.0140311)'':0.0236118)'':0.00448123)'':0.0118223)'':0.0177985)'':0.0520452,(('PDT000372053.1':0.137237,((('PDT000362504.1':0.030443,'PDT000448196.1':0.0324751)'':0.00773143,(('PDT000663848.1':0.0346274,'PDT000103019.1':0.0351856)'':0.00548734,('PDT000213195.2':0.0392854,('PDT000397892.1':0.0256452,('PDT000289948.1':0.0179663,('PDT000708879.1':0.0212894,('PDT000385044.1':0.0149075,('PDT000103041.1':0.0113961,('PDT000213205.2':0.00686289,'PDT000502067.1':0.00748691)'':0.00255719)'':0.00458074)'':0.00347885)'':0.00181485)'':0.00144553)'':0.00750735)'':0.0022336)'':0.00566317)'':0.0166647,(('PDT000181755.2':0.0434799,'PDT000713330.1':0.0370616)'':0.02274,((('PDT000397746.1':0.048305,'PDT000639539.1':0.054072)'':0.00678836,(('PDT000550328.1':0.0240412,'PDT000566093.1':0.0318502)'':0.0235603,('PDT000425819.1':0.0439798,('PDT000694410.1':0.0241718,'PDT000713389.1':0.0239089)'':0.0142467)'':0.00571833)'':0.00457978)'':-0.0024895,(('PDT000636337.1':0.0344379,('PDT000694530.1':0.0206647,'PDT000502032.1':0.0251723)'':0.00457407)'':0.0148133,(('PDT000636809.1':0.0461055,(('PDT000399650.1':0.0397248,'PDT000445991.1':0.0443031)'':0.0057831,(('PDT000312635.2':0.0272257,'PDT000385059.1':0.0297419)'':0.0137373,('PDT000645524.2':0.0288094,('PDT000354892.1':0.0210026,'PDT000042713.1':0.0228588)'':0.00390463)'':0.0136538)'':0.00309518)'':0.00380687)'':-0.00218299,(('PDT000502130.1':0.0326095,'PDT000618462.1':0.0274789)'':0.0100042,(('PDT000713825.1':0.0289578,('PDT000713514.1':0.020205,('PDT000479865.1':0.0145569,'PDT000574935.1':0.0135101)'':0.00803723)'':0.00327281)'':0.00506477,('PDT000188335.1':0.0385421,('PDT000694435.1':0.0363395,('PDT000713878.1':0.0333116,(('PDT000627370.1':0.00449076,'PDT000415823.1':0.00639404)'':0.0140812,('PDT000389385.2':0.0183244,('PDT000482859.1':0.0151984,('PDT000427233.1':0.0151076,('PDT000713381.1':0.0122636,'PDT000713901.1':0.0115129)'':0.00246237)'':0.00149007)'':0.00111256)'':0.00286977)'':0.00619847)'':0.00399236)'':0.00166579)'':-0.00167637)'':0.011644)'':0.00322365)'':0.00353713)'':0.00157995)'':0.0106053)'':-0.00593015)'':0.0805267)'':-0.00954433,((((('PDT000135933.2':0.0429366,'PDT000501903.1':0.0424246)'':0.0105076,('PDT000011789.2':0.0360618,('PDT000636249.1':0.0312496,'PDT000168466.1':0.0294364)'':0.0138288)'':0.00681417)'':0.00515943,(('PDT000234086.2':0.0398417,'PDT000502086.1':0.0453982)'':0.00930885,((('PDT000635710.1':0.035926,'PDT000103090.1':0.0380835)'':0.000719971,('PDT000565919.1':0.0375252,('PDT000098721.2':0.0203386,'PDT000618455.1':0.0225298)'':0.00948755)'':0.00252508)'':0.00509255,(('PDT000426382.1':0.0359368,('PDT000083137.2':0.0335884,(('PDT000327103.1':0.0214704,'PDT000135962.2':0.0243687)'':0.00432232,('PDT000713602.1':0.0232299,('PDT000661539.1':0.0180499,'PDT000011785.2':0.0150745)'':0.00449464)'':0.0029677)'':0.00387264)'':0.00466479)'':0.000945989,(('PDT000713608.1':0.0371362,('PDT000415813.1':0.0369294,('PDT000130620.2':0.0182029,'PDT000713885.1':0.0170877)'':0.0148714)'':0.000611939)'':0.00186327,('PDT000661309.1':0.0411366,(('PDT000694424.1':0.0247592,'PDT000713345.1':0.0232557)'':0.00923121,(('PDT000318594.1':0.0217465,('PDT000098720.2':0.0121605,'PDT000448200.1':0.0124061)'':0.00917677)'':0.00397925,('PDT000378058.1':0.0265007,(('PDT000011719.2':0.0211244,('PDT000131844.2':0.0218974,('PDT000135974.2':0.0163139,'PDT000426567.1':0.0177434)'':0.00324195)'':0.00341828)'':-0.000581682,('PDT000694506.1':0.0173051,('PDT000661661.1':0.0131662,('PDT000011832.2':0.0131386,('PDT000714143.1':0.00758611,('PDT000661548.1':0.0069861,'PDT000011786.2':0.0065328)'':0.00164664)'':0.00357958)'':0.0046292)'':0.00213207)'':0.00142629)'':0.0069605)'':-0.0043782)'':0.00827661)'':0.00778059)'':-0.00645243)'':-0.000365052)'':0.00773438)'':0.0117945)'':-0.00261649)'':0.00864857,(('PDT000717009.1':0.0181782,'PDT000661708.1':0.0209734)'':0.0548179,((('PDT000502001.1':0.0400077,(('PDT000502131.1':0.0230036,'PDT000346752.1':0.0256829)'':0.00212758,('PDT000188158.2':0.0245051,('PDT000502100.1':0.0198778,'PDT000502135.1':0.0189762)'':0.00520489)'':0.00142759)'':0.00750592)'':-0.00169529,(('PDT000501966.1':0.037852,'PDT000502035.1':0.0357676)'':0.00361746,(('PDT000502128.1':0.0198489,'PDT000346798.1':0.0216698)'':0.015245,((('PDT000501981.1':0.0189297,'PDT000502179.1':0.0154985)'':0.00412798,('PDT000502169.1':0.0176695,'PDT000346774.1':0.0167997)'':0.00289605)'':0.0100307,(('PDT000502207.1':0.0205453,'PDT000501904.1':0.0199381)'':0.00470149,(('PDT000501971.1':0.0163437,'PDT000501993.1':0.0200163)'':0.00727315,('PDT000501982.1':0.0261779,('PDT000502151.1':0.0236816,'PDT000502208.1':0.0236296)'':0.00224176)'':0.00137191)'':0.000510586)'':0.00146495)'':0.0039782)'':0.0039027)'':-0.00150799)'':-0.000148777,((('PDT000502125.1':0.0239307,'PDT000502238.1':0.0225147)'':0.00689456,('PDT000502012.1':0.0332161,(('PDT000502073.1':0.0210129,'PDT000502090.1':0.0198074)'':0.00697723,('PDT000502069.1':0.0238522,('PDT000502027.1':0.0245707,'PDT000502166.1':0.0234121)'':0.000960368)'':0.00439406)'':0.00361277)'':-0.000140917)'':0.00564369,(('PDT000502143.1':0.0290748,('PDT000346691.1':0.0135214,'PDT000346779.1':0.0122876)'':0.0181826)'':0.00713981,(('PDT000502101.1':0.0261163,(('PDT000501986.1':0.0267394,'PDT000502008.1':0.028302)'':0.00105171,('PDT000502198.1':0.0283263,('PDT000502097.1':0.0171086,('PDT000502088.1':0.0108989,'PDT000502110.1':0.00951135)'':0.00765374)'':0.00637935)'':3.5093e-05)'':0.00116473)'':0.000757071,(('PDT000502041.1':0.0232884,'PDT000501889.1':0.024354)'':0.0025274,(('PDT000502033.1':0.0156717,'PDT000502215.1':0.0140596)'':0.00384982,('PDT000501969.1':0.0162888,('PDT000502212.1':0.0135282,'PDT000502218.1':0.0112882)'':0.00312289)'':0.00276495)'':0.00479302)'':0.00213017)'':0.00683704)'':-0.000296882)'':0.000889083)'':0.0438916)'':-0.0183642)'':0.0478381,((('PDT000713306.1':0.0918062,(('PDT000253438.2':0.0386052,'PDT000488272.1':0.0411754)'':0.00542875,('PDT000427222.1':0.0447196,'PDT000713900.1':0.0365884)'':0.00934968)'':0.0318007)'':0.0311419,('PDT000550339.1':0.0590358,(('PDT000397593.1':0.0393694,('PDT000232217.2':0.0299973,'PDT000394960.1':0.0357806)'':0.00573939)'':0.0106011,(('PDT000394985.1':0.0146883,'PDT000395270.1':0.0138211)'':0.023053,('PDT000228257.1':0.0355644,('PDT000473278.1':0.0271077,'PDT000644906.1':0.0259455)'':0.0104581)'':0.000183823)'':0.00807811)'':0.00381678)'':0.0627072)'':0.000131823,((('PDT000534887.1':0.0635544,((('PDT000062969.2':0.043192,('PDT000051479.1':0.0306344,'PDT000103012.1':0.0245967)'':0.0117351)'':0.00413563,('PDT000137869.2':0.042929,('PDT000426834.1':0.0307985,'PDT000718953.1':0.0279962)'':0.00885931)'':0.000522057)'':0.00842481,('PDT000693780.1':0.0585501,('PDT000501978.1':0.0441559,('PDT000661312.1':0.0309972,('PDT000251754.1':0.035317,('PDT000466992.1':0.0245284,('PDT000024483.2':0.0220904,('PDT000557131.1':0.0135229,'PDT000252712.1':0.0165298)'':0.0104675)'':0.00652225)'':0.0027995)'':0.00438144)'':0.0122601)'':0.00845854)'':0.00119628)'':0.00998065)'':0.0119537,(('PDT000566135.1':0.040936,('PDT000397883.1':0.0407488,('PDT000639540.1':0.022136,('PDT000394814.1':0.0184857,'PDT000557064.1':0.017499)'':0.00750539)'':0.0141812)'':0.00410505)'':0.0122541,('PDT000304204.1':0.0538593,(('PDT000557071.1':0.028742,(('PDT000427604.1':0.0133013,'PDT000713985.1':0.0124841)'':0.00761925,('PDT000438144.1':0.0198529,('PDT000661609.1':0.0139751,('PDT000321608.1':0.00342074,'PDT000694429.1':0.00256549)'':0.00803068)'':0.00572689)'':5.73601e-05)'':0.0120102)'':-0.00219747,(('PDT000397893.1':0.0133136,'PDT000394846.1':0.0143293)'':0.0126662,('PDT000394972.1':0.0301547,('PDT000352433.1':0.0249363,(('PDT000565909.1':0.0103532,'PDT000394828.1':0.00984879)'':0.00206393,('PDT000644985.1':0.0155573,('PDT000557079.1':0.0103402,('PDT000135938.2':0.0105303,('PDT000397952.1':0.00804785,('PDT000167644.2':0.00312953,'PDT000644838.1':0.00315413)'':0.00314877)'':0.00173073)'':0.00130409)'':0.00175151)'':0.00116253)'':0.00804664)'':0.00570518)'':-0.00193112)'':-0.000758329)'':0.0258259)'':-0.00785382)'':0.0363214)'':0.0131721,(('PDT000215089.2':0.0673668,((('PDT000619386.1':0.0117227,'PDT000320254.1':0.0124094)'':0.0377467,(('PDT000167686.2':0.0116191,'PDT000497228.1':0.0136219)'':0.0338807,('PDT000397815.1':0.0433328,('PDT000497218.1':0.0312593,('PDT000458778.1':0.0277002,('PDT000301024.2':0.0259308,'PDT000610887.1':0.0251035)'':0.00275034)'':0.00357219)'':0.00382379)'':0.0040888)'':0.00465916)'':0.00557209,(('PDT000610888.1':0.045438,'PDT000288001.1':0.0468431)'':0.0167273,(('PDT000687841.1':0.0378706,('PDT000251518.1':0.0267456,('PDT000619615.1':0.0206753,'PDT000248765.1':0.0243237)'':0.0052236)'':0.00633053)'':0.00620813,('PDT000019232.2':0.0537626,('PDT000713580.1':0.0391513,(('PDT000713688.1':0.0187758,'PDT000011721.2':0.0173438)'':0.0128769,('PDT000713870.1':0.0290082,'PDT000287999.1':0.0238831)'':0.003439)'':0.00624158)'':0.00916575)'':-0.00215238)'':0.0094305)'':-0.00352775)'':0.0185903)'':-0.00566051,(('PDT000399609.1':0.05665,('PDT000661543.1':0.0575589,('PDT000181684.2':0.0470996,(('PDT000359135.1':0.0287188,'PDT000713677.1':0.0280408)'':0.00605003,('PDT000714092.1':0.0317696,('PDT000716810.1':0.0292529,('PDT000468152.1':0.0270348,('PDT000362508.1':0.0244664,('PDT000600661.1':0.0234773,('PDT000322770.1':0.017561,('PDT000167643.2':0.0143233,'PDT000713568.1':0.0136522)'':0.00376144)'':0.00427606)'':0.00150647)'':0.000957045)'':0.00277039)'':0.00136203)'':0.00457163)'':0.00789391)'':0.00315936)'':0.00531596)'':-0.00642472,((('PDT000568761.1':0.0290314,'PDT000395508.1':0.0316958)'':0.00526018,('PDT000004017.2':0.00939366,('PDT000644979.1':0.00476836,('PDT000395078.1':0.00207381,'PDT000395283.1':0.00221932)'':0.00401908)'':0.00606329)'':0.0219731)'':0.0135043,(('PDT000427736.1':0.0363886,'PDT000143593.1':0.0302323)'':0.0301955,('PDT000550353.1':0.0454773,(('PDT000497363.1':0.0187721,'PDT000566118.1':0.0181848)'':0.00386419,(('PDT000628178.1':0.0131287,'PDT000708568.1':0.0128428)'':0.00375403,('PDT000167712.2':0.014409,('PDT000497304.1':0.0125503,('PDT000644836.1':0.0120036,('PDT000566205.1':0.00865347,'PDT000653458.1':0.00981143)'':0.000944863)'':0.00382909)'':0.00235706)'':-0.000639451)'':0.00343713)'':0.0155562)'':0.0139475)'':-0.00561169)'':0.00394746)'':0.0164537)'':0.0246409)'':0.0257784)'':-0.00502871)'':0.0209109)'':0.0108849)'':0.012731)'':0.00658534)'':0.0148209,(((('PDT000538925.2':0.0117426,'PDT000538917.2':0.0138121)'':0.0502909,(('PDT000205202.2':0.0191091,'PDT000713443.1':0.0182398)'':0.0340222,(('PDT000502113.1':0.0465039,('PDT000694495.1':0.0347537,'PDT000215988.1':0.0371436)'':0.00698297)'':0.000837989,('PDT000205135.2':0.0403718,('PDT000205145.2':0.0303378,('PDT000279200.1':0.0301423,'PDT000415815.1':0.0282639)'':0.00433549)'':0.00658431)'':0.00451808)'':0.00353994)'':0.0140894)'':0.0012402,(('PDT000705161.1':0.0350502,('PDT000618964.1':0.0224834,('PDT000181725.2':0.0164222,'PDT000522781.1':0.0195476)'':0.00258097)'':0.0146899)'':0.0202161,((((('PDT000632352.1':0.0293681,'PDT000633100.1':0.0257407)'':0.00675767,('PDT000626794.1':0.0348434,(('PDT000346802.1':0.0287076,('PDT000534972.1':0.013535,'PDT000237096.3':0.0111723)'':0.013453)'':0.00419538,('PDT000144523.2':0.0195414,('PDT000368916.1':0.00966353,('PDT000537862.1':0.00328492,'PDT000368924.1':0.00270609)'':0.00794231)'':0.0101789)'':0.0111039)'':0.0029246)'':-0.00192297)'':0.0102679,(('PDT000502016.1':0.0476124,('PDT000188106.3':0.037909,'PDT000538545.1':0.0400873)'':0.00379713)'':0.0033288,(('PDT000713453.1':0.0413456,'PDT000661479.1':0.0512739)'':0.00498728,(('PDT000501994.1':0.0377017,('PDT000559712.1':0.0314671,('PDT000158920.2':0.0229029,'PDT000588002.1':0.0259064)'':0.00923189)'':0.00184219)'':0.00440939,('PDT000543285.1':0.0671568,(('PDT000618780.1':0.0420786,'PDT000276026.1':0.0388747)'':0.00222518,(('PDT000502011.1':0.0346722,('PDT000675162.1':0.0354491,('PDT000535503.1':0.0355077,('PDT000502000.1':0.0241662,('PDT000501885.1':0.0220704,('PDT000712682.1':0.0208481,('PDT000520835.1':0.0164475,('PDT000423905.1':0.00497959,'PDT000538548.1':0.00458883)'':0.0073941)'':0.00860189)'':0.00136166)'':7.00694e-05)'':0.00428661)'':0.00177919)'':0.00314303)'':-0.00203238,('PDT000011822.2':0.0440031,('PDT000188092.2':0.0336144,(('PDT000501913.1':0.0288389,('PDT000538448.1':0.0237271,('PDT000125949.1':0.0219373,('PDT000188133.2':0.0185815,'PDT000131812.2':0.0169465)'':0.00426159)'':0.00180735)'':0.00303169)'':-0.00123718,(('PDT000321349.1':0.0195438,'PDT000322915.1':0.0187016)'':0.00643965,('PDT000346799.1':0.0236423,('PDT000438174.1':0.0246022,'PDT000713397.1':0.020171)'':0.00171519)'':0.00103626)'':0.00241772)'':0.00621759)'':0.00666323)'':-0.00906535)'':0.00583858)'':0.0122633)'':-0.00944019)'':0.00817077)'':-0.00188941)'':-0.00350094)'':0.00422037,(('PDT000688007.1':0.0442051,(('PDT000535306.1':0.0309605,('PDT000618716.1':0.0304583,('PDT000714136.1':0.020468,('PDT000673159.1':0.0168858,'PDT000042646.1':0.0173926)'':0.00422117)'':0.00798573)'':0.000905484)'':0.000337323,(('PDT000226328.1':0.0259906,('PDT000651368.1':0.0196416,'PDT000668780.1':0.0156909)'':0.00879165)'':0.00385928,('PDT000685889.1':0.0337726,('PDT000433690.1':0.029436,(('PDT000188187.2':0.017563,('PDT000502139.1':0.0136586,'PDT000502150.1':0.0113538)'':0.00199145)'':0.00419728,('PDT000125204.1':0.025845,('PDT000618782.1':0.0210521,('PDT000651343.1':0.013855,('PDT000625964.1':0.0113444,('PDT000535063.1':0.0061667,'PDT000346930.1':0.007491)'':0.00445644)'':0.00169924)'':0.00537085)'':0.0015464)'':9.89305e-05)'':0.00551811)'':0.00348376)'':-0.00200539)'':0.00432923)'':0.0160202)'':-0.00328064,(('PDT000460583.1':0.0521256,('PDT000673568.1':0.045662,('PDT000136003.2':0.0324205,('PDT000534995.1':0.0325126,('PDT000427545.1':0.0195569,('PDT000433425.1':0.0203765,'PDT000588010.1':0.0205741)'':0.0023214)'':0.00403273)'':0.0068165)'':0.00547901)'':0.00366236)'':0.000535993,(('PDT000426042.1':0.0475353,'PDT000502195.1':0.045)'':0.00127911,(('PDT000633308.1':0.0482726,('PDT000588034.1':0.0451082,('PDT000205157.2':0.0336781,('PDT000534990.1':0.0286463,('PDT000619079.1':0.0304132,(('PDT000583395.1':0.012349,('PDT000636276.1':0.00966779,'PDT000011830.2':0.00736581)'':0.00327692)'':0.00334134,('PDT000069700.1':0.0200499,('PDT000534936.1':0.0076635,('PDT000616162.1':0.00491946,'PDT000583374.1':0.00465756)'':0.00304594)'':0.00599759)'':0.00324743)'':0.00811114)'':0.00227571)'':0.00494326)'':0.0113802)'':0.00376955)'':-0.00749729,((('PDT000438189.1':0.0434498,'PDT000438342.1':0.0416402)'':0.00745802,(('PDT000622399.1':0.0169014,'PDT000485916.1':0.0149565)'':0.0190189,('PDT000135934.2':0.0297394,('PDT000538570.1':0.0278655,'PDT000714173.1':0.0233738)'':0.00676456)'':0.00735791)'':0.00630714)'':-0.0014643,(('PDT000626815.1':0.0343171,('PDT000362502.1':0.0312144,(('PDT000205200.2':0.0109924,'PDT000368921.1':0.0120349)'':0.00783482,('PDT000570884.1':0.0161603,('PDT000713726.1':0.00996771,'PDT000107529.1':0.0104202)'':0.00448636)'':0.00143063)'':0.00815772)'':0.00303843)'':0.00329776,('PDT000368923.1':0.0506,(('PDT000636315.1':0.0390303,('PDT000717041.1':0.0301426,('PDT000425926.1':0.0154635,'PDT000626692.1':0.0196486)'':0.00700454)'':0.0104602)'':-0.00354508,('PDT000715919.1':0.029816,(('PDT000636343.1':0.0173022,'PDT000543121.1':0.0214406)'':0.00320767,('PDT000427492.1':0.0126499,('PDT000713838.1':0.00594007,'PDT000713861.1':0.00600383)'':0.00609031)'':0.00578263)'':0.00765995)'':0.000509516)'':0.00893792)'':-0.00512893)'':0.00450332)'':0.00034811)'':0.00569741)'':-0.000541821)'':0.000987832)'':-0.000102111)'':0.00375507,((('PDT000707818.1':0.0418801,('PDT000647674.1':0.0346969,('PDT000433646.1':0.0270016,('PDT000426621.1':0.016382,('PDT000535332.1':0.0125096,'PDT000483535.1':0.013261)'':0.00142106)'':0.0104608)'':0.00362213)'':0.0127852)'':0.001154,((('PDT000444935.1':0.0228954,'PDT000233336.1':0.0253567)'':0.0149755,(('PDT000425709.1':0.0246339,'PDT000216787.1':0.0256886)'':0.00531328,('PDT000538403.1':0.0164424,'PDT000538514.1':0.0183101)'':0.0132741)'':0.00873066)'':0.0031169,(('PDT000538525.1':0.0443229,'PDT000673498.1':0.0430661)'':0.00261045,(('PDT000713881.1':0.0284038,'PDT000422531.1':0.0304826)'':0.000929949,('PDT000534882.1':0.0248746,('PDT000616167.1':0.0208192,'PDT000626704.1':0.0196347)'':0.00185391)'':0.00491905)'':0.0136966)'':0.000176584)'':0.00092286)'':0.00586193,(('PDT000285336.1':0.0876325,(('PDT000225433.2':0.0211159,'PDT000415814.1':0.0234356)'':0.025995,('PDT000537880.1':0.054295,('PDT000640370.1':0.0379973,('PDT000132272.2':0.0327758,('PDT000409476.1':0.0296567,('PDT000327368.1':0.0258373,'PDT000618779.1':0.0262298)'':0.00581661)'':0.00251241)'':0.00462633)'':0.00541532)'':0.001125)'':0.0347957)'':-0.0207855,(('PDT000520668.1':0.0414186,('PDT000125205.2':0.0348288,(('PDT000713266.1':0.0231092,'PDT000713863.1':0.022149)'':0.0103347,(('PDT000636189.1':0.0227791,'PDT000661681.1':0.0157359)'':0.0136529,('PDT000205147.2':0.0388388,('PDT000125206.1':0.0267398,('PDT000673442.1':0.00670609,'PDT000107527.1':0.0104784)'':0.0144608)'':0.00776618)'':-0.000612062)'':0.00141782)'':0.00452362)'':0.00268775)'':0.0088787,(('PDT000618792.1':0.0579738,('PDT000425781.1':0.0586571,'PDT000426740.1':0.0458539)'':0.0057572)'':0.00299953,(((('PDT000661737.1':0.0376239,('PDT000433223.1':0.0377149,('PDT000433725.2':0.0333198,('PDT000580718.1':0.0268047,('PDT000484658.1':0.025041,('PDT000265428.2':0.0268638,(('PDT000713494.1':0.0209081,('PDT000581879.1':0.0160152,'PDT000716880.1':0.0155262)'':0.00324262)'':0.00181642,('PDT000225305.2':0.0161806,('PDT000640362.1':0.00997394,'PDT000661533.1':0.0110746)'':0.00270233)'':0.00547281)'':0.0031417)'':0.00207881)'':0.00184594)'':0.00353805)'':0.00185084)'':0.00686546)'':-0.00324873,('PDT000508908.1':0.0447364,(('PDT000636311.1':0.0342836,'PDT000673625.1':0.0358189)'':0.00319742,(('PDT000423942.1':0.0286053,('PDT000508901.1':0.0262815,('PDT000633129.1':0.0205026,('PDT000500569.1':0.0136292,'PDT000500570.1':0.00949681)'':0.00777917)'':0.00669068)'':0.00149074)'':0.00134784,('PDT000069645.1':0.0313975,('PDT000619100.1':0.0222256,('PDT000433657.1':0.0124639,('PDT000636182.1':0.0075119,'PDT000600531.1':0.0062468)'':0.00325319)'':0.00615593)'':0.0158508)'':-0.00377326)'':0.00503843)'':0.00400518)'':0.00305089)'':0.000858408,(('PDT000543106.1':0.0440319,('PDT000543309.1':0.0432448,('PDT000673548.1':0.0164552,'PDT000362519.1':0.0200196)'':0.0152992)'':0.00747057)'':0.00688819,(('PDT000661338.1':0.0450814,('PDT000314837.2':0.0261125,('PDT000362476.1':0.0240749,('PDT000652371.1':0.0206207,'PDT000661659.1':0.01873)'':0.00754068)'':0.000406594)'':0.019667)'':-0.00887833,('PDT000636255.1':0.0384019,((('PDT000661710.1':0.0238404,('PDT000661300.1':0.0186563,('PDT000713276.1':0.010323,'PDT000716942.1':0.00910216)'':0.00544633)'':0.00294854)'':-0.000235031,('PDT000129577.1':0.0182296,('PDT000713879.1':0.0113526,('PDT000713415.1':0.00791531,('PDT000694408.1':0.00206824,'PDT000714175.1':0.0020248)'':0.00608067)'':0.00438321)'':0.00483076)'':0.00181119)'':0.00228743,(('PDT000136059.2':0.0168965,'PDT000368912.1':0.0148903)'':0.00567271,('PDT000714135.1':0.0252587,(('PDT000502602.1':0.0133074,'PDT000627372.1':0.014768)'':0.000392932,(('PDT000324626.1':0.00876186,'PDT000337007.1':0.00945074)'':0.00366617,('PDT000636245.1':0.0137779,('PDT000714027.1':0.0104899,('PDT000427505.1':0.00970449,('PDT000205136.2':0.00697525,('PDT000694463.1':0.00217253,'PDT000713978.1':0.0021236)'':0.00270998)'':0.00351983)'':0.00143383)'':0.00237402)'':-0.000992241)'':0.0022912)'':0.00921175)'':-0.00150304)'':0.000739432)'':0.013203)'':-0.00409961)'':0.0101424)'':-0.00566723)'':0.00137797,(('PDT000173737.2':0.0260418,'PDT000618851.1':0.0310674)'':0.0204902,(('PDT000640369.1':0.0507997,('PDT000710979.1':0.0461548,('PDT000716951.1':0.0306367,('PDT000120928.2':0.0285236,('PDT000501439.1':0.021874,'PDT000543098.1':0.0237184)'':0.00497821)'':0.00263402)'':0.00909386)'':0.00452564)'':-0.00316134,((('PDT000072465.1':0.0652906,('PDT000501974.1':0.0248333,('PDT000502182.1':0.0220182,'PDT000007760.2':0.0148858)'':0.00613274)'':0.0231711)'':0.00141796,(('PDT000661478.1':0.0357565,(('PDT000427619.1':0.0129151,'PDT000714022.1':0.0147087)'':0.00956985,('PDT000069703.1':0.024924,('PDT000534878.1':0.0164557,'PDT000714087.1':0.0143105)'':0.00553583)'':0.00144792)'':0.00666101)'':-0.000767968,(('PDT000368945.1':0.0247569,'PDT000368946.1':0.0258476)'':0.0161224,('PDT000329823.1':0.040695,('PDT000661835.1':0.0344821,(('PDT000713347.1':0.023155,('PDT000042503.1':0.0184073,'PDT000346761.1':0.0187177)'':0.00291408)'':0.00267249,('PDT000661492.1':0.0317332,('PDT000661717.1':0.0277564,('PDT000145826.3':0.0256401,('PDT000433473.1':0.0266524,(('PDT000587987.1':0.0158924,'PDT000652370.1':0.0162448)'':0.00166761,('PDT000713790.1':0.0201237,('PDT000538638.1':0.0186193,(('PDT000583923.1':0.0116626,'PDT000640355.1':0.0135333)'':0.00219036,('PDT000502217.1':0.0186745,(('PDT000103064.1':0.00827635,'PDT000346690.1':0.00779475)'':0.00331677,('PDT000543287.1':0.0100498,('PDT000713823.1':0.00972755,('PDT000713293.1':0.00695022,('PDT000673559.1':0.00523535,'PDT000069647.1':0.00455095)'':0.00122253)'':0.00271485)'':0.000996016)'':0.000439226)'':0.00475379)'':-0.00131695)'':0.00258895)'':0.00208086)'':-0.00203129)'':0.00534634)'':0.00111288)'':0.00333317)'':0.00312226)'':-0.00416332)'':0.00742018)'':0.00554372)'':-0.00597223)'':-0.00318038)'':0.0185437)'':-0.0147306,((('PDT000398389.1':0.0427525,'PDT000520750.1':0.0375189)'':0.00425061,(('PDT000588043.1':0.0181837,('PDT000444981.1':0.00862727,'PDT000346775.1':0.00990413)'':0.0102619)'':0.0151408,(('PDT000425932.1':0.0263921,'PDT000636243.1':0.0294946)'':0.011987,('PDT000717036.1':0.0304768,(('PDT000248476.1':0.0306355,('PDT000521075.1':0.0208057,('PDT000425789.1':0.0163834,'PDT000110714.1':0.0168412)'':0.00247769)'':0.00610353)'':0.000913324,(('PDT000145835.3':0.0254956,'PDT000134615.1':0.0239775)'':0.00631015,('PDT000175864.2':0.0280251,(('PDT000102011.2':0.0203993,'PDT000103098.1':0.0225587)'':0.00243056,('PDT000042467.1':0.0221793,('PDT000433224.1':0.0175229,('PDT000011740.2':0.0167856,('PDT000426274.1':0.0158899,'PDT000713842.1':0.0159077)'':0.000457866)'':0.00189986)'':0.00248185)'':0.00342776)'':0.00633159)'':-0.00219631)'':0.00256094)'':0.00318654)'':0.00251393)'':0.00339898)'':0.00492056)'':-0.0062971,((('PDT000351873.1':0.0276838,'PDT000622557.1':0.0277797)'':0.00878098,(('PDT000176034.2':0.0243545,'PDT000425679.1':0.0254344)'':0.00312788,('PDT000421192.1':0.0255372,'PDT000673411.1':0.0253419)'':0.00117339)'':0.00359387)'':0.000843814,('PDT000537947.1':0.0726602,('PDT000352815.1':0.0359408,((('PDT000713595.1':0.0119323,'PDT000521045.1':0.0102523)'':0.0113167,('PDT000136130.2':0.0235359,('PDT000672254.1':0.0170001,('PDT000104798.2':0.0147147,'PDT000683608.1':0.0142121)'':0.00130637)'':0.00579456)'':0.000454022)'':0.000220432,('PDT000445082.1':0.0315865,('PDT000103096.1':0.0238618,('PDT000427230.1':0.0195741,(('PDT000590276.1':0.0129071,('PDT000570881.1':0.0116557,'PDT000661458.1':0.0116737)'':0.00149943)'':0.00190937,('PDT000036729.1':0.0190221,('PDT000702579.1':0.013923,('PDT000407976.1':0.0119251,('PDT000673565.1':0.00872743,'PDT000042517.1':0.00849167)'':0.00244505)'':0.00175735)'':0.00337997)'':-0.00105766)'':0.00518367)'':0.00323113)'':0.000866842)'':-0.000905984)'':0.0100377)'':0.0142991)'':-0.0133993)'':0.00052205)'':0.00201304)'':0.00401207)'':0.00817985)'':-0.0112514)'':0.0153095)'':-0.00502454)'':0.0019564)'':-0.000265277)'':-7.30795e-05)'':0.00798602)'':0.00451854)'':0.0429402,((((('PDT000479556.1':0.0700325,('PDT000477475.1':0.0490824,('PDT000003801.2':0.0337694,'PDT000718887.1':0.0279094)'':0.00933926)'':0.0204843)'':0.00835723,(('PDT000482222.1':0.0278498,'PDT000712932.1':0.0297449)'':0.0442207,(('PDT000251866.1':0.0554887,('PDT000520980.1':0.0481592,('PDT000619870.1':0.016956,'PDT000623227.1':0.016341)'':0.0280255)'':0.0137323)'':0.00568951,(('PDT000619871.1':0.056516,('PDT000484091.1':0.0105568,'PDT000352554.1':0.00736079)'':0.0410767)'':-0.00214923,('PDT000668350.1':0.0326886,(('PDT000536983.1':0.023069,'PDT000343318.1':0.0241073)'':0.00431324,('PDT000687364.1':0.0233783,('PDT000487575.1':0.0176661,('PDT000265968.4':0.0100006,('PDT000358607.1':0.00330353,'PDT000304203.1':0.00428433)'':0.00588758)'':0.00917026)'':0.00539854)'':-0.000706559)'':0.00870866)'':0.0120501)'':0.0167832)'':0.0121264)'':-0.00171043)'':0.00351942,((('PDT000713277.1':0.0363615,'PDT000502117.1':0.0365836)'':0.0143149,('PDT000664805.1':0.106259,('PDT000426076.1':0.0402358,('PDT000426742.1':0.0289393,('PDT000713629.1':0.0141541,('PDT000107539.1':0.0149618,(('PDT000664768.1':0.00833844,('PDT000713917.1':0.00702778,'PDT000716891.1':0.00614592)'':0.00206186)'':0.00152196,('PDT000572844.1':0.00937748,('PDT000107528.1':0.00401907,'PDT000424799.1':0.00365713)'':0.00431592)'':0.00265303)'':0.00136396)'':0.00163713)'':0.0129791)'':0.0140132)'':0.0222366)'':-0.0138043)'':0.00953371,(('PDT000535032.1':0.0693646,('PDT000346689.1':0.0424873,('PDT000103071.2':0.0317762,('PDT000346881.1':0.0200821,(('PDT000346759.1':0.0144207,'PDT000346770.1':0.0119813)'':0.0022372,('PDT000102178.1':0.0140683,('PDT000264293.1':0.0117289,'PDT000327859.1':0.0120853)'':0.00172129)'':0.00402356)'':0.0040148)'':0.00444545)'':0.0213477)'':0.021111)'':-0.00377267,(('PDT000321910.1':0.0247397,'PDT000426293.1':0.0207867)'':0.031765,('PDT000443033.1':0.0627383,((('PDT000502127.1':0.0208785,('PDT000346745.1':0.0183465,('PDT000713242.1':0.0185298,'PDT000129572.1':0.0183035)'':0.00227903)'':0.00260267)'':0.00918432,('PDT000713420.1':0.0395694,('PDT000205141.2':0.0384486,(('PDT000017784.2':0.0301349,('PDT000205168.2':0.0150248,('PDT000205167.2':0.00653159,('PDT000714001.1':0.00252184,'PDT000017696.2':0.0033705)'':0.00261554)'':0.00577405)'':0.0148342)'':-0.00200819,('PDT000205142.2':0.0269656,(('PDT000713955.1':0.017033,('PDT000205177.2':0.0166046,('PDT000205201.2':0.0150293,'PDT000139230.2':0.0139576)'':0.00186279)'':0.000939687)'':0.00199072,(('PDT000137850.2':0.0121553,'PDT000648066.1':0.0134323)'':0.00817007,('PDT000421766.1':0.0173155,(('PDT000205143.2':0.00882779,('PDT000713334.1':0.00272271,('PDT000205144.2':0.000977781,'PDT000205218.2':0.00102082)'':0.00247113)'':0.00406809)'':0.00228166,('PDT000017744.2':0.00833452,('PDT000107531.1':0.0100492,('PDT000501968.1':0.00470531,'PDT000110712.1':0.00378705)'':0.00327182)'':0.00148842)'':0.00133751)'':0.00624983)'':0.00173246)'':0.00381852)'':0.00501723)'':-0.00050193)'':0.00723773)'':0.0012941)'':-6.25357e-06)'':-0.00477927,('PDT000103053.2':0.0425348,(('PDT000615981.1':0.0336311,('PDT000710305.1':0.0223592,'PDT000131239.1':0.0206219)'':0.0109501)'':0.000136591,('PDT000713502.1':0.0455244,(('PDT000105132.2':0.0204334,'PDT000131932.2':0.0195313)'':0.00730946,('PDT000072463.1':0.0318481,('PDT000661582.1':0.0256356,('PDT000111515.1':0.0246522,(('PDT000316573.1':0.0140667,'PDT000042669.1':0.0126898)'':0.00486295,('PDT000205199.2':0.0248404,(('PDT000372692.1':0.0168748,'PDT000129578.1':0.019827)'':0.00238419,('PDT000616183.1':0.0247927,('PDT000713701.1':0.0262313,(('PDT000713708.1':0.0097251,'PDT000233091.1':0.0107581)'':0.00719379,(('PDT000103038.1':0.0122105,'PDT000352397.1':0.0096866)'':0.00274406,('PDT000063321.2':0.0171589,(('PDT000715969.1':0.0108107,('PDT000502089.1':0.00877789,'PDT000352420.1':0.00961541)'':0.00149943)'':0.00419698,('PDT000107535.1':0.0129201,(('PDT000188331.1':0.00835448,('PDT000107512.1':0.00334329,'PDT000011782.2':0.00474943)'':0.00398871)'':0.00207017,('PDT000694479.1':0.00899133,('PDT000132371.2':0.00638445,'PDT000694489.1':0.00596365)'':0.00191382)'':0.000566904)'':0.00234155)'':0.000799427)'':0.00224559)'':-0.00128994)'':0.00123861)'':0.00335703)'':0.00160915)'':-0.00122349)'':0.00220664)'':-0.00235788)'':0.00392382)'':0.00147908)'':0.00311598)'':0.00111847)'':0.0101553)'':-0.00574043)'':0.00647047)'':-0.00950208)'':0.022669)'':0.000278054)'':0.0119468)'':-0.00122554)'':0.0127737)'':0.00924098,((((('PDT000257699.2':0.0581606,'PDT000395332.1':0.0540914)'':0.0124434,(('PDT000394861.1':0.0383946,'PDT000644844.1':0.0365692)'':0.0275265,(('PDT000216146.2':0.0230882,'PDT000556945.1':0.029191)'':0.0179486,('PDT000107547.1':0.025613,'PDT000233038.1':0.0204364)'':0.0107929)'':0.0199646)'':0.00666284)'':0.0116226,(('PDT000305491.2':0.0596848,'PDT000251943.1':0.0532072)'':0.0229752,(('PDT000249890.1':0.0528031,('PDT000616122.1':0.056486,'PDT000414013.1':0.057215)'':0.00124443)'':0.0102586,(('PDT000396817.1':0.0555131,'PDT000191161.1':0.0539049)'':0.00786176,('PDT000660652.1':0.0433412,('PDT000395076.1':0.0224053,('PDT000660676.1':0.0175973,'PDT000714233.1':0.012966)'':0.00782401)'':0.0187079)'':0.0226085)'':0.00241766)'':0.0238447)'':-0.00186779)'':0.00326257,(('PDT000499658.1':0.0793005,('PDT000394926.1':0.0675555,('PDT000060763.2':0.0465449,'PDT000388470.1':0.0456312)'':0.0206944)'':0.00660197)'':0.0139138,((('PDT000305468.2':0.0758629,'PDT000305519.2':0.0712541)'':0.00649561,(('PDT000591956.1':0.0380649,'PDT000628867.1':0.0372505)'':0.0262687,((('PDT000425589.1':0.0426179,'PDT000536591.1':0.0383538)'':0.00912559,('PDT000661396.1':0.0171808,'PDT000191154.1':0.0164641)'':0.0282392)'':0.00944936,('PDT000501857.1':0.0646459,(('PDT000222837.2':0.0185234,'PDT000550392.1':0.014995)'':0.0226785,('PDT000309348.2':0.0481487,'PDT000557055.1':0.0474703)'':0.00176292)'':0.0154951)'':-0.000738695)'':0.00413427)'':0.0156246)'':-0.0101813,((('PDT000110713.1':0.0523189,('PDT000288610.2':0.0412673,('PDT000186890.2':0.0413728,('PDT000132198.2':0.0237628,'PDT000714548.1':0.0241132)'':0.00964111)'':0.00462476)'':0.0040368)'':0.00923317,((('PDT000494436.1':0.0141966,'PDT000565957.1':0.0112204)'':0.0358887,(('PDT000661333.1':0.0256119,'PDT000395052.1':0.0232116)'':0.0143543,(('PDT000178576.2':0.0132039,'PDT000497325.1':0.0175088)'':0.0309327,('PDT000125969.1':0.039859,(('PDT000713251.1':0.0194986,'PDT000468869.1':0.0219639)'':0.00789125,('PDT000427090.1':0.0269072,'PDT000025503.2':0.0303253)'':0.000346899)'':0.00942214)'':0.00460363)'':0.000385827)'':0.00114531)'':0.00588469,((('PDT000321334.1':0.0238511,'PDT000354286.1':0.0272094)'':0.00743181,('PDT000645133.1':0.0226516,('PDT000497230.1':0.0141224,'PDT000497243.1':0.0183127)'':0.00317062)'':0.0128919)'':0.011404,('PDT000133655.2':0.0472094,(('PDT000205205.2':0.0322228,'PDT000647167.1':0.0321528)'':0.0138532,('PDT000281738.2':0.0366142,('PDT000205191.2':0.0260132,'PDT000557303.1':0.0290201)'':0.0110106)'':0.00188762)'':0.00521043)'':-0.00221472)'':0.00201107)'':0.011609)'':-0.0027222,(('PDT000136716.1':0.0565578,('PDT000664953.1':0.0472668,('PDT000395187.1':0.0308637,('PDT000204881.2':0.0354715,'PDT000024329.2':0.0307179)'':0.0054502)'':0.00442483)'':0.00536877)'':0.0138818,(('PDT000673526.1':0.0471381,(('PDT000271340.2':0.0328626,'PDT000550368.1':0.0340501)'':0.0077784,('PDT000425551.1':0.0313983,'PDT000557067.1':0.0344501)'':0.00736475)'':0.00386582)'':0.00531246,(('PDT000394797.1':0.0433039,('PDT000616104.1':0.0303466,('PDT000616124.1':0.022596,'PDT000716812.1':0.028068)'':0.00703255)'':0.00888635)'':-0.0014199,(('PDT000639541.1':0.0116326,'PDT000661993.1':0.0151896)'':0.0228603,('PDT000250700.1':0.038761,(('PDT000380903.1':0.0249749,('PDT000025501.2':0.0140513,'PDT000233101.1':0.015637)'':0.00817908)'':0.00521197,(('PDT000502038.1':0.0273289,'PDT000347015.1':0.0234753)'':0.005623,(('PDT000327860.1':0.0183832,('PDT000202624.2':0.0161459,'PDT000358747.1':0.014328)'':0.00403657)'':0.0101042,('PDT000213215.2':0.0160911,('PDT000004387.2':0.0139089,'PDT000004399.2':0.0181957)'':0.00500255)'':0.00786787)'':0.00169554)'':0.00442449)'':0.0109609)'':-0.00148455)'':0.000205048)'':0.00139602)'':0.0067865)'':0.00985395)'':0.0116044)'':0.0120083)'':-0.00109225)'':0.00466014,(('PDT000135860.1':0.0895468,(('PDT000583949.1':0.0627623,('PDT000395055.1':0.0654732,'PDT000228219.1':0.0584738)'':0.00270374)'':0.00753449,('PDT000719138.1':0.0669901,('PDT000293514.1':0.0692656,(('PDT000713989.1':0.0525202,('PDT000137829.2':0.0393885,'PDT000694441.1':0.0361323)'':0.00555582)'':0.011993,('PDT000502106.1':0.0540586,('PDT000538912.2':0.0494465,('PDT000484089.1':0.0296811,('PDT000619777.1':0.0247687,('PDT000131240.1':0.0129393,'PDT000019969.2':0.0159544)'':0.00898405)'':0.0133209)'':0.0156619)'':0.00363612)'':0.00216417)'':0.00615423)'':0.00378122)'':-0.00173032)'':0.0169149)'':-0.00923584,((('PDT000205208.2':0.0340031,('PDT000713221.1':0.0264085,('PDT000205223.2':0.0206388,'PDT000276028.1':0.0278377)'':0.00590365)'':0.00541914)'':0.0118555,('PDT000209467.1':0.0632291,('PDT000521029.1':0.050335,(('PDT000202003.2':0.0333703,'PDT000205215.2':0.0299653)'':0.00338119,('PDT000098368.2':0.0431682,(('PDT000175811.2':0.0253744,'PDT000107521.1':0.0249376)'':0.00246772,('PDT000427240.1':0.0295004,('PDT000619640.1':0.0272556,('PDT000713996.1':0.0207789,('PDT000136160.2':0.0163398,'PDT000426025.1':0.0170889)'':0.00281564)'':0.00525501)'':0.00232775)'':0.00102433)'':0.00753582)'':-0.00148707)'':0.00896239)'':0.0114866)'':-0.0130722)'':0.0321499,(('PDT000208169.2':0.0555975,'PDT000716888.1':0.0616425)'':0.00732684,(('PDT000167708.2':0.0527615,'PDT000718443.1':0.0487265)'':0.0131975,((('PDT000521130.1':0.0391814,'PDT000168463.1':0.0336585)'':0.00539913,('PDT000708225.1':0.0281864,('PDT000645519.2':0.0212958,'PDT000242957.1':0.0214367)'':0.00500104)'':0.0148282)'':0.0151277,('PDT000426835.1':0.0644905,('PDT000645565.2':0.0682225,('PDT000675187.1':0.064909,(('PDT000389636.1':0.0463536,'PDT000672217.1':0.0462603)'':0.00595265,(('PDT000661740.1':0.0317917,(('PDT000317617.1':0.0205581,'PDT000339372.1':0.0160885)'':0.0104599,('PDT000645025.1':0.0285349,('PDT000518568.1':0.0142299,'PDT000714229.1':0.0135067)'':0.010558)'':0.00327565)'':0.0052438)'':0.00471052,(('PDT000129575.1':0.039289,'PDT000319017.1':0.0395153)'':0.00834727,('PDT000714002.1':0.0433726,('PDT000088272.2':0.0444396,('PDT000620537.1':0.0451242,('PDT000455223.1':0.0404731,(('PDT000343793.1':0.0216276,'PDT000395226.1':0.0249273)'':0.0126235,(('PDT000682348.1':0.0274251,('PDT000250699.1':0.0228092,('PDT000275537.2':0.0160271,'PDT000500478.1':0.0165484)'':0.00858681)'':0.00433381)'':-0.000277802,((('PDT000600653.1':0.0144723,'PDT000688703.1':0.0200242)'':0.0120014,('PDT000399534.1':0.0253265,('PDT000403085.1':0.0139062,'PDT000395126.1':0.0151894)'':0.0068469)'':0.0018267)'':0.00479252,(('PDT000426330.1':0.0256985,'PDT000427019.1':0.025496)'':0.010416,('PDT000565969.1':0.0455644,(('PDT000426378.1':0.0295748,('PDT000631530.1':0.0215221,('PDT000017738.2':0.0242132,('PDT000107533.1':0.0167727,'PDT000188352.1':0.0191566)'':0.00391046)'':0.00294053)'':0.00124845)'':-0.000513611,(('PDT000003948.2':0.0286668,('PDT000557066.1':0.025064,('PDT000563560.1':0.0231934,(('PDT000661562.1':0.0139887,'PDT000107520.1':0.0156117)'':0.000879988,('PDT000509489.1':0.0183341,('PDT000017643.2':0.00799829,'PDT000017733.2':0.00615501)'':0.00789211)'':0.000958141)'':0.0056463)'':0.000972471)'':0.00144299)'':-0.00122672,('PDT000131236.1':0.0218804,('PDT000285935.2':0.0253968,('PDT000477137.1':0.0229499,(('PDT000136148.2':0.0127627,'PDT000137844.2':0.0109853)'':0.00373801,('PDT000011724.2':0.0213511,('PDT000556987.1':0.0120035,('PDT000471005.1':0.0118586,'PDT000517944.1':0.0140035)'':0.00155403)'':0.00436991)'':-0.000530906)'':0.00307751)'':0.000624434)'':0.0030679)'':-0.00202697)'':0.00100365)'':0.00941736)'':-0.00299004)'':-0.00236069)'':0.00248158)'':0.00365841)'':0.00308228)'':0.00199602)'':0.00124113)'':0.00203854)'':0.00427761)'':-0.0120272)'':0.0131412)'':0.00466196)'':0.00411506)'':0.0044076)'':-0.0145929)'':0.0126891)'':0.00466766)'':0.01118)'':0.000209544)'':0.00627471)'':0.00469393)'':0.0127247,((((('PDT000636202.1':0.0313792,'PDT000636300.1':0.0327184)'':0.0105219,('PDT000443109.1':0.0400438,('PDT000125968.1':0.032929,('PDT000143962.3':0.0268302,'PDT000713846.1':0.0302819)'':0.00575867)'':0.00196882)'':0.00325275)'':0.00337404,('PDT000175948.2':0.0530409,(('PDT000129566.1':0.0278016,('PDT000136168.2':0.0232926,'PDT000714527.1':0.0223937)'':0.00258941)'':0.00840658,('PDT000094020.2':0.0379401,('PDT000713253.1':0.0188611,('PDT000200510.2':0.0132973,'PDT000132041.2':0.0112067)'':0.00515585)'':0.0145864)'':-2.61403e-05)'':0.0176173)'':-0.00811425)'':0.0175954,(('PDT000427553.1':0.0466755,'PDT000677013.1':0.0446351)'':0.0118442,(('PDT000443031.1':0.039373,('PDT000443101.1':0.0295192,'PDT000626702.1':0.0245138)'':0.0117236)'':0.0224109,(('PDT000443084.1':0.0505506,('PDT000443095.1':0.0353883,('PDT000443107.1':0.0346369,('PDT000032369.1':0.0175636,'PDT000033992.1':0.0189544)'':0.00799895)'':0.00770215)'':0.0184852)'':-0.00663348,('PDT000248515.1':0.0595146,('PDT000520758.1':0.0509448,('PDT000042483.1':0.0490981,('PDT000443057.1':0.0419076,('PDT000443029.1':0.0336407,(('PDT000136002.2':0.0252564,('PDT000175910.2':0.019734,'PDT000649527.1':0.0218467)'':0.00514662)'':0.00448345,('PDT000604151.1':0.0301254,(('PDT000443072.1':0.0117677,('PDT000443060.1':0.00930208,'PDT000443089.1':0.00657252)'':0.00207801)'':0.0109948,('PDT000366114.1':0.0216308,('PDT000636261.1':0.00852967,('PDT000433450.1':0.00677319,'PDT000672542.1':0.00826211)'':0.00244728)'':0.0091145)'':0.00196531)'':0.00991184)'':-0.000760828)'':0.0046244)'':0.00421989)'':0.00603057)'':0.00434421)'':0.0067734)'':-0.0107691)'':0.00191963)'':0.0137151)'':0.0025742)'':0.0236816,('PDT000600526.1':0.145468,(((('PDT000176040.2':0.0452469,('PDT000042637.1':0.0263452,'PDT000069695.1':0.0304763)'':0.0161269)'':0.00706369,((('PDT000615995.1':0.0374729,'PDT000125929.1':0.0338169)'':0.000696312,(('PDT000636148.1':0.0360612,('PDT000520866.1':0.0185953,'PDT000485940.1':0.0180431)'':0.0122694)'':0.000352367,(('PDT000231460.2':0.013761,'PDT000485973.1':0.0125912)'':0.00958056,(('PDT000673492.1':0.0190294,('PDT000651380.1':0.015956,'PDT000668346.1':0.0166559)'':0.00388451)'':-0.000770136,(('PDT000581569.1':0.0108007,'PDT000543237.1':0.0117774)'':0.00431332,('PDT000427001.1':0.0111808,'PDT000278771.1':0.0141596)'':0.000687452)'':0.00253518)'':0.00442066)'':0.00550085)'':0.00688135)'':-0.000798457,('PDT000248461.1':0.0485333,(('PDT000132344.2':0.0231884,'PDT000508882.1':0.0285784)'':0.00233076,('PDT000636361.1':0.0272768,(('PDT000358737.1':0.0200128,('PDT000131911.2':0.0105126,'PDT000702741.1':0.0114654)'':0.0052365)'':0.000841561,(('PDT000636215.1':0.0158796,'PDT000661798.1':0.0201137)'':0.000426126,('PDT000713232.1':0.0184552,('PDT000673517.1':0.0112814,('PDT000538466.1':0.00975049,('PDT000423909.1':0.00558208,'PDT000423940.1':0.00439099)'':0.00636173)'':0.00305687)'':0.0023953)'':0.00267263)'':0.000947561)'':0.004092)'':0.00302675)'':0.0156041)'':-0.00657825)'':0.0137445)'':-0.0117395,((('PDT000687963.1':0.0394826,('PDT000555539.1':0.0182759,'PDT000555540.1':0.0144528)'':0.0120816)'':0.00465752,(('PDT000433812.1':0.0281445,('PDT000202121.2':0.0156092,'PDT000288722.2':0.0142253)'':0.0130244)'':0.000585271,((('PDT000486446.1':0.0154556,'PDT000673186.1':0.0165357)'':0.00557971,('PDT000426756.1':0.0148697,('PDT000500539.1':0.0129471,'PDT000661444.1':0.0138236)'':0.00347555)'':0.00348026)'':0.0017156,('PDT000656279.1':0.0146172,('PDT000636355.1':0.0170656,(('PDT000622447.1':0.0114417,'PDT000100156.2':0.0156553)'':0.00288709,('PDT000425599.1':0.0108771,('PDT000538505.1':0.00565079,'PDT000615976.1':0.00569701)'':0.0044491)'':0.00379984)'':0.00389749)'':0.00156063)'':0.000656239)'':0.00547242)'':0.0030936)'':0.00874103,(('PDT000427029.1':0.0549769,('PDT000161898.2':0.0350523,((('PDT000343404.1':0.04357,('PDT000622577.1':0.018978,('PDT000343403.1':0.0202475,('PDT000343405.1':0.00842184,'PDT000343408.1':0.00580276)'':0.00812152)'':0.00347665)'':0.00968777)'':-0.00476952,(('PDT000636271.1':0.0235959,'PDT000495419.1':0.0210294)'':0.00273192,(('PDT000200512.3':0.0193704,('PDT000202191.2':0.0141425,('PDT000661483.1':0.00897867,'PDT000661508.1':0.0103531)'':0.00576833)'':0.00334585)'':0.000398434,(('PDT000661781.1':0.0120565,('PDT000535333.1':0.00764958,'PDT000661624.1':0.00766162)'':0.00698798)'':0.000247888,('PDT000276024.1':0.0102896,('PDT000445039.1':0.00986302,'PDT000709973.1':0.00649278)'':0.00352104)'':0.00367628)'':0.0034328)'':0.00317208)'':0.0013516)'':0.00248152,(('PDT000495426.1':0.0604428,(('PDT000433648.1':0.0262565,'PDT000543119.1':0.0245694)'':0.0028297,('PDT000433659.1':0.0222135,'PDT000534793.1':0.0227919)'':0.0060728)'':0.0125367)'':-0.0072911,((('PDT000538417.1':0.0261073,'PDT000713434.1':0.0219895)'':0.0055798,('PDT000433758.1':0.0213522,('PDT000535194.1':0.0159019,'PDT000634219.1':0.0175956)'':0.00728736)'':0.0061514)'':0.00156919,('PDT000618904.1':0.0378962,(('PDT000713516.1':0.0253316,('PDT000463606.1':0.0179415,('PDT000433624.1':0.0188604,('PDT000619101.1':0.0141522,('PDT000347069.1':0.00745581,('PDT000616049.1':0.00252311,'PDT000485965.1':0.00236809)'':0.00441449)'':0.00427547)'':0.002906)'':0.00329737)'':0.00457182)'':-0.00269348,('PDT000610214.1':0.0288926,(('PDT000661545.1':0.0250049,('PDT000499081.1':0.0239191,(('PDT000433620.1':0.0139848,('PDT000433687.1':0.0116868,('PDT000616044.1':0.00760856,'PDT000388469.1':0.00995634)'':0.00430418)'':0.00277705)'':-0.00184508,('PDT000426074.1':0.0215987,('PDT000654367.1':0.0182502,(('PDT000427721.1':0.00753255,'PDT000618899.1':0.00795285)'':0.00380834,('PDT000520875.1':0.0104865,('PDT000705111.1':0.00659819,('PDT000583402.1':0.00646334,('PDT000433705.1':0.00562211,('PDT000661731.1':0.00530113,('PDT000433428.1':0.00333943,('PDT000616025.1':0.00221329,'PDT000713609.1':0.00228221)'':0.00104436)'':0.00100046)'':0.0015696)'':0.00162876)'':0.000197069)'':0.00247)'':-0.00102037)'':0.00481035)'':0.00397688)'':-0.00497153)'':0.00783588)'':0.00198795)'':-0.00764993,(('PDT000433604.1':0.0110844,'PDT000433804.1':0.0146333)'':0.00619148,(('PDT000433611.1':0.0124067,'PDT000636327.1':0.0117326)'':0.00394977,('PDT000444170.1':0.0237233,((('PDT000640354.1':0.011848,'PDT000661585.1':0.00995167)'':0.00185805,('PDT000426741.1':0.00775266,'PDT000621693.1':0.00845984)'':0.0026477)'':0.00124963,(('PDT000538517.1':0.00964676,('PDT000433645.1':0.00383695,('PDT000372606.1':0.00222616,'PDT000491866.1':0.00206483)'':0.00195363)'':0.00490924)'':0.00138433,(('PDT000423922.1':0.0024609,'PDT000673424.1':0.00233335)'':0.00466812,('PDT000202017.2':0.00445554,('PDT000713508.1':0.00305817,'PDT000230871.1':0.00322863)'':0.00132839)'':0.00327539)'':0.00152826)'':0.00108897)'':0.00735959)'':-0.00148054)'':0.0020371)'':-0.000883117)'':0.0115064)'':-0.0063101)'':0.00948588)'':-0.00437363)'':0.00477984)'':-0.00404359)'':0.00371291)'':0.00603073)'':-0.0102605,((('PDT000298004.1':0.038696,('PDT000615932.1':0.0318157,'PDT000636628.1':0.030157)'':0.0134671)'':-0.00398544,(('PDT000661693.1':0.0219729,('PDT000202041.2':0.0115817,'PDT000705232.1':0.0163702)'':0.0049652)'':0.0034034,('PDT000582075.1':0.0242163,('PDT000086322.1':0.0234665,(('PDT000713895.1':0.0105214,'PDT000508890.1':0.0106688)'':0.0039671,('PDT000632127.1':0.0122659,('PDT000347060.1':0.011808,('PDT000427660.1':0.00776576,'PDT000538509.1':0.00830884)'':0.00206036)'':0.000606722)'':0.00193713)'':0.00626175)'':0.00100649)'':0.00259985)'':0.0124296)'':-0.0052932,((('PDT000423890.1':0.0149629,'PDT000705249.1':0.0173877)'':0.0148167,('PDT000651349.1':0.0349071,'PDT000661844.1':0.0309233)'':0.00372279)'':0.00207416,((('PDT000538443.1':0.0183463,('PDT000538488.1':0.0135481,('PDT000433710.1':0.0131255,'PDT000581872.1':0.0129761)'':0.00335416)'':0.00411464)'':0.00254011,('PDT000433788.1':0.0230672,(('PDT000538384.1':0.0145758,'PDT000583387.1':0.0166494)'':0.00243321,('PDT000673227.1':0.0175123,('PDT000326276.1':0.012118,('PDT000636136.1':0.00894091,('PDT000205181.2':0.00808237,'PDT000616023.1':0.00656053)'':0.00101414)'':0.00108836)'':0.00608676)'':-0.00103333)'':0.00892363)'':3.59813e-05)'':0.000260897,('PDT000433311.1':0.0651622,(('PDT000444102.1':0.0239288,'PDT000636169.1':0.0191824)'':0.0072878,(('PDT000536632.1':0.0276691,('PDT000427480.1':0.0284667,((('PDT000231446.2':0.0199561,'PDT000621691.1':0.0142378)'':0.000580453,('PDT000636283.1':0.0117022,('PDT000426386.1':0.0111625,'PDT000615933.1':0.00956959)'':0.00574844)'':0.0016999)'':0.000883081,(('PDT000616111.1':0.0114553,'PDT000661622.1':0.0117521)'':0.00358022,('PDT000636151.1':0.0103426,('PDT000433446.1':0.00531656,'PDT000640353.1':0.00716474)'':0.00523168)'':0.00457437)'':0.00402882)'':0.00576706)'':0.00525225)'':-0.00601193,((('PDT000251382.1':0.0180121,('PDT000522768.1':0.0166835,('PDT000202045.2':0.00486698,'PDT000522783.1':0.00726022)'':0.00680181)'':0.00365066)'':0.00674474,('PDT000661738.1':0.025062,(('PDT000720327.1':0.0183367,('PDT000444181.1':0.0146877,'PDT000508893.1':0.0167881)'':0.00259107)'':0.000445875,(('PDT000441830.1':0.0162931,'PDT000713378.1':0.0129461)'':0.00136133,(('PDT000502599.1':0.0168487,('PDT000615980.1':0.00926045,('PDT000720319.1':0.00675497,('PDT000176033.2':0.00413489,('PDT000107549.1':0.0030221,('PDT000615897.1':0.00296258,'PDT000347075.1':0.00232948)'':0.000866353)'':0.00132768)'':0.000948682)'':0.00249225)'':0.0074817)'':-0.00131969,('PDT000347055.1':0.0130344,('PDT000433689.1':0.0143873,('PDT000621588.1':0.0103728,(('PDT000520736.1':0.00406347,'PDT000520874.1':0.00360891)'':0.00499863,(('PDT000202002.2':0.00462358,'PDT000673223.1':0.00474457)'':0.00194935,('PDT000426676.1':0.00594344,'PDT000647675.1':0.00629416)'':0.0014887)'':0.000702983)'':0.00108704)'':0.00270818)'':0.000874237)'':-0.000655592)'':0.00148469)'':0.00441956)'':0.00934336)'':-0.00669714)'':-0.00177361,((('PDT000618950.1':0.0152504,'PDT000030604.1':0.0142867)'':0.00519375,('PDT000616032.1':0.0209572,('PDT000149561.2':0.0194146,('PDT000636187.1':0.014363,('PDT000581598.1':0.0131778,('PDT000423894.1':0.00807225,('PDT000426474.1':0.00588647,'PDT000615935.1':0.00574833)'':0.0027545)'':0.00259777)'':0.00206175)'':0.00180203)'':0.00523208)'':-0.000299385)'':-0.000738507,(('PDT000202748.2':0.0238203,('PDT000636344.1':0.0131703,('PDT000588020.1':0.00135352,'PDT000588022.1':0.00164378)'':0.0151641)'':0.0120265)'':-0.000531818,(('PDT000615986.1':0.0308048,'PDT000191151.1':0.0339378)'':0.000826786,(('PDT000615874.1':0.0198358,'PDT000661617.1':0.0182557)'':0.00437248,('PDT000097039.1':0.0227887,('PDT000651355.1':0.0186965,(('PDT000433791.1':0.00825765,('PDT000425950.1':0.00485662,'PDT000433752.1':0.00490336)'':0.00239856)'':0.00188622,('PDT000427026.1':0.0172765,(('PDT000714138.1':0.0115803,'PDT000716966.1':0.0121703)'':0.000582778,('PDT000636313.1':0.00806195,('PDT000508902.1':0.00983335,('PDT000616007.1':0.00523009,('PDT000347062.1':0.00549878,('PDT000426713.1':0.00531328,(('PDT000042711.1':0.00211096,'PDT000042737.1':0.00208317)'':0.00153932,('PDT000538512.1':0.00336183,('PDT000363560.1':0.00268208,('PDT000433744.1':0.0025935,('PDT000615967.1':0.0019093,'PDT000694430.1':0.00198447)'':0.000300679)'':0.000342494)'':0.00111946)'':-0.00018697)'':0.00146868)'':0.000684645)'':0.000158742)'':0.00258241)'':0.00116127)'':-5.81072e-05)'':0.00401105)'':-0.0026781)'':0.00510373)'':0.000385077)'':0.00109714)'':0.00543455)'':-0.00237485)'':0.000602647)'':0.000591793)'':0.000385154)'':0.00478255)'':0.0276489)'':-0.0286554)'':0.00789966)'':-0.00312498)'':-0.00291434)'':0.0107657)'':0.00181261)'':0.0154933,((('PDT000427690.1':0.0742421,('PDT000433734.1':0.02413,('PDT000661673.1':0.0271032,('PDT000433628.1':0.0198493,('PDT000713702.1':0.0116217,('PDT000572849.1':0.0071505,'PDT000208668.1':0.0053158)'':0.00555588)'':0.00534183)'':0.00552591)'':0.00262184)'':0.0258603)'':-0.0165716,(('PDT000661374.1':0.0229369,('PDT000324132.1':0.0247771,'PDT000407971.1':0.0195996)'':0.00561169)'':0.00149218,(('PDT000636163.1':0.0192773,('PDT000714077.1':0.012801,('PDT000487290.1':0.0104531,('PDT000438195.1':0.00935217,('PDT000132349.2':0.00202056,'PDT000661778.1':0.0017764)'':0.00561815)'':0.00182128)'':0.00188017)'':0.00280084)'':0.00389294,('PDT000604194.1':0.0355414,('PDT000604196.1':0.0300003,('PDT000427699.1':0.0277663,(('PDT000713919.1':0.0129962,'PDT000661480.1':0.0154478)'':0.00301341,('PDT000566591.1':0.020061,(('PDT000661658.1':0.00475368,'PDT000661698.1':0.00663602)'':0.00694462,('PDT000635575.1':0.0122943,('PDT000661514.1':0.0126375,('PDT000133643.2':0.00745274,('PDT000295831.2':0.00453225,'PDT000663843.1':0.00543784)'':0.00292092)'':0.00217056)'':0.000959537)'':-0.000307925)'':0.00411089)'':0.000789343)'':0.00636208)'':0.005838)'':0.000826148)'':-0.00791969)'':0.0063371)'':0.00337211)'':0.0396971,(('PDT000538919.2':0.0694751,('PDT000175882.2':0.00892525,('PDT000175805.2':0.00447051,('PDT000175826.2':0.00218817,'PDT000175839.2':0.00240173)'':0.00131773)'':0.00591479)'':0.063854)'':0.00569727,((((('PDT000520719.1':0.0529938,(('PDT000175898.2':0.0098298,'PDT000129576.1':0.0111853)'':0.0190194,(('PDT000634551.1':0.0249777,'PDT000587993.1':0.0298572)'':0.00564111,('PDT000615848.1':0.0266286,(('PDT000175810.2':0.0149498,'PDT000205188.2':0.0199661)'':0.0038025,('PDT000520718.1':0.00908638,'PDT000520746.1':0.0135502)'':0.0109343)'':0.00791672)'':0.000763534)'':0.00076499)'':0.0156801)'':-0.0107353,('PDT000375416.1':0.047116,(('PDT000103073.1':0.0290865,(('PDT000545931.1':0.0152898,'PDT000107551.1':0.0118474)'':0.00427502,('PDT000136122.2':0.01011,('PDT000615840.1':0.00364471,'PDT000615851.1':0.00492786)'':0.00354719)'':0.0116469)'':0.0130337)'':-0.00537026,((('PDT000425498.1':0.00195996,'PDT000426533.1':0.00173561)'':0.0249588,('PDT000426405.1':0.0194455,('PDT000485935.1':0.0170021,('PDT000423957.1':0.0126776,('PDT000143967.3':0.00813099,'PDT000714556.1':0.00927151)'':0.0059266)'':0.00413501)'':0.00215217)'':0.00469731)'':-0.000147983,('PDT000720323.1':0.0223038,(('PDT000615854.1':0.0135737,'PDT000615879.1':0.0127915)'':0.00331538,('PDT000423847.1':0.015194,(('PDT000426422.1':0.0057315,'PDT000588026.1':0.0075274)'':0.00242863,('PDT000713982.1':0.0103493,'PDT000720307.1':0.00907196)'':0.000862424)'':0.00276534)'':0.00257567)'':0.00484496)'':0.00292324)'':0.00239317)'':0.0134687)'':-0.00342414)'':0.012336,((('PDT000636279.1':0.0296783,(('PDT000538465.1':0.0168116,('PDT000433673.1':0.0141411,'PDT000445091.1':0.0130603)'':0.00228085)'':0.000740951,('PDT000618887.1':0.0226594,('PDT000423921.1':0.0179108,('PDT000616018.1':0.00837157,'PDT000636365.1':0.00830833)'':0.00517212)'':0.00202255)'':0.00104751)'':0.00801751)'':0.00659064,(('PDT000433286.1':0.031439,'PDT000448183.1':0.0324197)'':0.00886885,('PDT000588052.1':0.0327714,('PDT000433454.1':0.0338137,('PDT000423881.1':0.024287,('PDT000588015.1':0.0155518,(('PDT000205178.2':0.0208353,('PDT000094013.3':0.010162,'PDT000673512.1':0.0102543)'':0.00552713)'':0.000645846,('PDT000615837.1':0.0143098,('PDT000423843.1':0.00984436,'PDT000636162.1':0.00924494)'':0.00117224)'':0.00733611)'':0.00133716)'':0.00385379)'':0.00881434)'':0.00172138)'':0.000701901)'':-0.00145704)'':0.013244,(('PDT000615947.1':0.0511608,('PDT000279218.1':0.0475141,(('PDT000205159.2':0.0198101,('PDT000175842.2':0.0200727,('PDT000205193.2':0.00741632,'PDT000177243.1':0.00866948)'':0.00808381)'':0.00318929)'':0.00705163,(('PDT000704986.1':0.0243778,'PDT000668936.1':0.0264165)'':0.000980905,('PDT000205152.2':0.0267912,('PDT000175903.2':0.0227952,('PDT000615803.1':0.0143024,('PDT000176208.2':0.012215,'PDT000205185.2':0.0101834)'':0.00272607)'':0.00869056)'':0.00173143)'':0.00246819)'':0.0010164)'':0.0166116)'':0.00802905)'':-0.00484292,((('PDT000137816.2':0.0302194,('PDT000421784.1':0.0213804,('PDT000520809.1':0.0192902,(('PDT000441816.1':0.0102646,'PDT000175974.2':0.0106534)'':0.0010557,('PDT000347003.1':0.0126552,('PDT000444874.1':0.0104441,('PDT000107546.1':0.000640318,'PDT000129579.1':0.000758842)'':0.00970728)'':0.00342352)'':0.000694742)'':0.00458729)'':0.00347724)'':0.00420224)'':0.000274988,('PDT000426356.1':0.0392894,(('PDT000137815.2':0.0227867,'PDT000020972.1':0.0207728)'':0.00439437,('PDT000433328.1':0.0235911,(('PDT000615843.1':0.0172366,'PDT000616036.1':0.0176666)'':0.00121849,('PDT000407986.1':0.00723319,('PDT000175933.2':0.00424366,('PDT000175862.2':0.00640307,('PDT000175827.2':0.00149285,'PDT000176003.2':0.00110533)'':0.00223148)'':0.00107708)'':0.00229171)'':0.00646959)'':0.00868317)'':-0.00256909)'':0.0162009)'':-0.00571836)'':0.0057366,(('PDT000352423.1':0.0205603,(('PDT000626710.1':0.0142314,'PDT000694540.1':0.0116897)'':0.00931178,(('PDT000543090.1':0.0122802,'PDT000248468.1':0.010764)'':0.006019,('PDT000346987.1':0.0204294,('PDT000441356.1':0.0135012,('PDT000426727.1':0.0105688,('PDT000538496.1':0.00675192,'PDT000347031.1':0.00727798)'':0.00329665)'':0.000984294)'':0.0053305)'':-0.00110887)'':0.00252443)'':0.00229952)'':0.00549694,(('PDT000661363.1':0.0161376,'PDT000114939.1':0.0157227)'':0.0139374,('PDT000176015.2':0.0298768,(('PDT000713301.1':0.0161775,('PDT000713610.1':0.0106995,'PDT000543115.1':0.00887721)'':0.0054329)'':0.000595417,(('PDT000661321.1':0.0135224,'PDT000363559.1':0.0153713)'':0.000688757,('PDT000131862.2':0.0153955,('PDT000175835.2':0.00942523,'PDT000175976.2':0.0115156)'':0.00388213)'':0.0012186)'':0.00244532)'':0.00642192)'':0.00426004)'':0.00124411)'':0.00360031)'':0.00913469)'':0.00368297)'':0.000560137)'':-0.000498255,(((('PDT000535478.1':0.0397096,'PDT000720444.1':0.0464923)'':0.0121713,('PDT000633935.1':0.0394173,('PDT000618714.1':0.0227803,'PDT000129573.1':0.0203527)'':0.014742)'':0.0092115)'':0.0143321,(('PDT000423871.1':0.0401449,((('PDT000636160.1':0.0208539,'PDT000661474.1':0.0217035)'':0.0042145,('PDT000618720.1':0.0169267,('PDT000636207.1':0.0140083,'PDT000347048.1':0.0125054)'':0.0016415)'':0.0100545)'':0.0077281,(('PDT000535573.1':0.0172586,('PDT000285163.1':0.014046,('PDT000500559.1':0.0158706,('PDT000285165.1':0.0154755,('PDT000426688.1':0.00548969,'PDT000285159.1':0.00758681)'':0.00467989)'':0.00230672)'':0.00132558)'':0.00138515)'':0.00822024,('PDT000448188.1':0.0255217,('PDT000485939.1':0.0144199,('PDT000437973.1':0.0121771,('PDT000636625.1':0.0082464,('PDT000713775.1':0.00602246,'PDT000508887.1':0.00735214)'':0.00340785)'':0.00260807)'':0.00606495)'':0.00328892)'':0.00157612)'':0.00961324)'':0.00504187)'':0.0048858,(('PDT000444937.1':0.0547753,('PDT000490349.1':0.0436444,('PDT000705095.1':0.033259,(('PDT000426584.1':0.0269652,('PDT000137826.2':0.0223454,'PDT000433780.1':0.0179907)'':0.00476681)'':0.00162744,(('PDT000135951.2':0.0137212,'PDT000135977.2':0.0209268)'':0.00603743,('PDT000137841.2':0.0272995,('PDT000414844.1':0.0192589,('PDT000176215.2':0.0125307,'PDT000713482.1':0.0118885)'':0.00612282)'':0.00575104)'':-0.000699072)'':0.00278251)'':0.0036097)'':0.014623)'':0.00754489)'':-0.0165982,(('PDT000713944.1':0.0465848,(('PDT000520674.1':0.0214625,'PDT000106389.1':0.0189505)'':0.00367907,('PDT000425630.1':0.0215986,('PDT000713618.1':0.0169487,('PDT000175917.2':0.0131472,'PDT000257683.2':0.0144596)'':0.00430214)'':0.00482021)'':0.0011672)'':0.0183907)'':-0.0120021,('PDT000160484.2':0.0430267,('PDT000438184.1':0.0355229,(('PDT000158300.1':0.023112,'PDT000018104.2':0.0195685)'':0.00563461,('PDT000661638.1':0.0226763,('PDT000136008.2':0.0113813,(('PDT000694471.1':0.00197791,'PDT000704919.1':0.00211472)'':0.00082482,('PDT000175849.2':0.00313509,('PDT000713426.1':0.00150824,('PDT000135973.2':0.000377674,'PDT000136009.2':0.000422086)'':0.00158872)'':0.00059794)'':0.00138203)'':0.00947748)'':0.00915329)'':-0.00373704)'':0.0125174)'':0.002367)'':-0.00137788)'':0.00251304)'':0.00428006)'':0.0221363)'':-0.0121397,((('PDT000176039.2':0.0490366,('PDT000588012.1':0.0317453,('PDT000427786.1':0.020219,'PDT000673460.1':0.019093)'':0.0144212)'':0.0128451)'':-0.00096826,('PDT000425966.1':0.0535983,(('PDT000042603.1':0.028509,('PDT000427590.1':0.019253,('PDT000136126.2':0.0138878,('PDT000647658.1':0.0110127,('PDT000534863.1':0.00678056,'PDT000581595.1':0.00812994)'':0.00358797)'':0.0030963)'':0.0012645)'':0.00630153)'':0.0116373,(('PDT000713340.1':0.0255356,('PDT000520837.1':0.0227218,('PDT000175984.2':0.0115224,'PDT000425618.1':0.0102922)'':0.00761332)'':0.00943469)'':0.00566453,(('PDT000636181.1':0.0274284,('PDT000175936.2':0.0196341,('PDT000615914.1':0.0107915,'PDT000233070.1':0.00797101)'':0.00567349)'':0.00778326)'':0.00396483,(('PDT000522786.1':0.01052,('PDT000522764.1':0.00834913,('PDT000522785.1':0.00519909,'PDT000522787.1':0.00485571)'':0.00197287)'':0.00242213)'':0.00208742,('PDT000508889.1':0.0105329,('PDT000205233.2':0.0070664,('PDT000673142.1':0.00568483,('PDT000095195.3':0.00377705,('PDT000581462.1':0.00349324,'PDT000619076.1':0.00368666)'':0.000107961)'':0.00256638)'':0.00116422)'':0.00408404)'':-0.00135472)'':0.0178508)'':0.00793903)'':0.00201783)'':0.0112807)'':-0.00292823)'':0.00235941,((('PDT000433787.1':0.0340489,('PDT000526181.1':0.0336788,('PDT000202781.2':0.0343547,(('PDT000433448.1':0.0238525,('PDT000636348.1':0.0147974,('PDT000107536.1':0.00685387,('PDT000713581.1':0.00671327,'PDT000107542.1':0.00765513)'':0.00234668)'':0.00473058)'':0.00949564)'':0.000709995,(('PDT000107514.1':0.0258157,('PDT000175972.2':0.0186455,'PDT000615956.1':0.0148989)'':0.00739057)'':6.88451e-05,(('PDT000615833.1':0.0243572,('PDT000145845.3':0.0167538,('PDT000137825.2':0.0129147,('PDT000427040.1':0.0087358,'PDT000615829.1':0.0075518)'':0.00592499)'':0.00194052)'':0.00555192)'':-0.00200402,(('PDT000444965.1':0.0154499,'PDT000414009.1':0.0162163)'':0.00196998,('PDT000444963.1':0.0170378,('PDT000705261.1':0.0118682,'PDT000520885.1':0.0123413)'':0.00179701)'':0.00307025)'':0.00221021)'':0.00279836)'':-9.11059e-05)'':0.00749633)'':0.000117893)'':0.0077347)'':-0.00099707,((('PDT000536649.1':0.027592,'PDT000640363.1':0.0295761)'':0.00616867,(('PDT000538527.1':0.0289402,('PDT000485983.1':0.019087,('PDT000520989.1':0.019325,('PDT000636334.1':0.0109583,('PDT000239094.1':0.00927018,'PDT000252015.1':0.00766882)'':0.00154988)'':0.00487494)'':0.00489237)'':0.00668606)'':-0.00276056,('PDT000622417.1':0.037718,('PDT000135961.2':0.0254837,(('PDT000538486.1':0.0204931,(('PDT000615946.1':0.0116013,'PDT000636259.1':0.0121611)'':0.00791865,('PDT000616001.1':0.0184254,'PDT000042636.1':0.0163244)'':0.00203645)'':0.00215085)'':-0.000753602,(('PDT000543210.1':0.021329,('PDT000616017.1':0.0100246,'PDT000635915.1':0.00843849)'':0.00946621)'':0.00504549,('PDT000705061.1':0.0174464,('PDT000407988.1':0.0187848,(('PDT000132021.2':0.00992818,'PDT000538504.1':0.0117009)'':0.00174297,(('PDT000632232.1':0.0134823,('PDT000423900.1':0.00754761,'PDT000423959.1':0.00873839)'':0.00439083)'':3.11889e-05,('PDT000433470.1':0.00838182,('PDT000616085.1':0.00570218,'PDT000704910.1':0.00573262)'':0.00283438)'':0.00185274)'':0.00219396)'':0.0043354)'':0.00289705)'':0.000999173)'':9.86972e-05)'':0.00510124)'':0.00838525)'':-0.0106986)'':0.00704253)'':0.00274197,(('PDT000705422.1':0.0322349,('PDT000705415.1':0.026946,('PDT000616029.1':0.0252333,(('PDT000202166.2':0.0191159,'PDT000520929.1':0.0197572)'':0.00258858,(('PDT000705355.1':0.0239074,(('PDT000133656.2':0.00915148,'PDT000407974.1':0.0111632)'':0.00920873,(('PDT000520933.1':0.00788932,'PDT000520966.1':0.00984858)'':0.00804467,('PDT000705073.1':0.0114346,('PDT000704928.1':0.0122668,'PDT000705002.1':0.00995323)'':0.00170073)'':0.00308662)'':0.00192947)'':0.0085489)'':-0.00549569,(('PDT000485968.1':0.0164868,('PDT000175941.2':0.0121547,'PDT000508886.1':0.0125415)'':0.00397775)'':0.00168568,('PDT000520810.1':0.0190068,('PDT000615909.1':0.0164919,(('PDT000176224.2':0.0138423,'PDT000705069.1':0.0152695)'':0.00145471,(('PDT000218038.2':0.000564308,'PDT000143933.3':0.000634972)'':0.0116128,('PDT000720321.1':0.00949549,('PDT000520914.1':0.0101066,('PDT000615834.1':0.000761397,'PDT000615883.1':0.000638043)'':0.00837103)'':0.00289872)'':0.000915221)'':0.00244022)'':0.00109303)'':0.00366322)'':-0.00123753)'':0.00284095)'':0.00149318)'':0.00110055)'':0.00177815)'':0.00100472)'':0.00215275,(('PDT000521114.1':0.0271443,('PDT000520794.1':0.0160986,'PDT000618381.1':0.0174525)'':0.0093406)'':0.00906263,(('PDT000368933.1':0.0307027,(('PDT000425841.1':0.0137352,'PDT000636380.1':0.0136824)'':0.00478697,('PDT000368919.1':0.0164931,('PDT000231456.2':0.0097855,'PDT000616112.1':0.0076448)'':0.00692824)'':0.00357614)'':0.00544504)'':0.00272375,(('PDT000176028.2':0.023722,'PDT000205148.2':0.028486)'':0.0065157,((('PDT000574922.1':0.0200997,'PDT000490171.1':0.016458)'':0.0141058,(('PDT000538489.1':0.00357307,'PDT000233308.1':0.00390897)'':0.0151969,('PDT000661713.1':0.0240409,(('PDT000175995.2':0.0100212,'PDT000661290.1':0.010996)'':0.00780251,('PDT000285166.1':0.0247423,('PDT000131923.2':0.0216121,('PDT000581878.1':0.0134508,('PDT000485929.1':0.0112783,('PDT000650994.1':0.014293,('PDT000688539.1':0.011126,('PDT000484659.1':0.00988716,('PDT000632135.1':0.00860322,('PDT000647668.1':0.00594181,('PDT000426015.1':0.00382838,('PDT000647654.1':0.00405416,('PDT000508911.1':0.00264061,('PDT000285366.2':0.00224661,'PDT000285164.1':0.00254716)'':0.000251399)'':0.0010399)'':0.000145422)'':0.00139657)'':0.0020813)'':0.000740568)'':0.000604041)'':0.00274578)'':0.000859598)'':0.00134397)'':0.000988823)'':0.00143865)'':-0.00150604)'':0.00432631)'':-0.00163114)'':0.00752636)'':-0.00473718,(('PDT000426221.1':0.0234145,('PDT000615994.1':0.0194569,('PDT000426473.1':0.0135219,('PDT000427304.1':0.00891476,'PDT000444889.1':0.00856984)'':0.00526707)'':0.00576748)'':0.00190838)'':0.00271159,(('PDT000714050.1':0.0153065,('PDT000581870.1':0.0126914,'PDT000042601.1':0.0121134)'':0.00432467)'':0.00446528,('PDT000427482.1':0.0359223,('PDT000368944.1':0.0357779,(('PDT000205217.2':0.029664,('PDT000205221.2':0.00776227,'PDT000205222.2':0.00551383)'':0.0168568)'':0.000221157,('PDT000426843.1':0.023051,((('PDT000426476.1':0.0176084,'PDT000616035.1':0.0137871)'':0.00425781,('PDT000636627.1':0.0195811,(('PDT000538515.1':0.0100322,('PDT000202724.2':0.0069122,('PDT000427489.1':0.0020864,'PDT000615895.1':0.00260609)'':0.00526836)'':0.00269816)'':0.00316082,('PDT000433655.1':0.0139983,('PDT000427196.1':0.00865354,('PDT000647661.1':0.00989926,('PDT000615922.1':0.0066189,('PDT000636305.1':0.00587469,('PDT000647653.1':0.00473701,'PDT000521053.1':0.00522613)'':0.000900894)'':0.000569461)'':0.00214631)'':0.000822576)'':0.00185054)'':0.000717633)'':0.00413293)'':-0.000802741)'':0.000157828,(('PDT000444962.1':0.0171874,'PDT000616042.1':0.0128394)'':0.00319007,('PDT000705413.1':0.0205857,(('PDT000202202.2':0.0152769,'PDT000715986.1':0.0137343)'':0.00279281,((('PDT000020953.1':0.0230922,('PDT000705421.1':0.0147108,('PDT000538405.1':0.00853843,'PDT000672269.1':0.00891327)'':0.0050546)'':0.00569885)'':-0.00169672,('PDT000125970.1':0.0228141,(('PDT000202170.2':0.0122383,'PDT000673601.1':0.0145271)'':0.000212231,(('PDT000426007.1':0.0114312,'PDT000673461.1':0.00981164)'':0.00149797,('PDT000521092.1':0.0100266,(('PDT000407998.1':0.00480165,'PDT000459418.1':0.00397991)'':0.00243125,('PDT000543118.1':0.00534799,('PDT000175831.2':0.00484018,'PDT000175860.2':0.00493051)'':0.00130832)'':0.000659496)'':0.00375159)'':0.00213271)'':0.000474402)'':0.00609603)'':-0.00204414)'':-0.000748556,((('PDT000501914.1':0.00805599,'PDT000501919.1':0.00830471)'':0.0157785,('PDT000616026.1':0.0223409,('PDT000426703.1':0.0136526,('PDT000426483.1':0.0103222,'PDT000520994.1':0.0088051)'':0.00321672)'':0.0048802)'':0.00129894)'':-0.000220614,(('PDT000132061.2':0.0133264,('PDT000581461.1':0.00712478,'PDT000615899.1':0.00650302)'':0.00428735)'':0.00354582,(('PDT000635912.1':0.0161093,'PDT000520743.1':0.0130351)'':0.00378212,(('PDT000538471.1':0.0117744,('PDT000598277.1':0.00558271,'PDT000415828.1':0.00557379)'':0.00667374)'':0.00422963,(('PDT000407983.1':0.0100086,'PDT000408010.1':0.0117651)'':0.00375969,('PDT000705023.1':0.0160459,(('PDT000107555.1':0.0198877,('PDT000521020.1':0.00891457,(('PDT000161875.2':0.00623804,'PDT000460544.1':0.00672176)'':0.000963725,('PDT000019184.2':0.00705617,(('PDT000175924.2':0.00352081,'PDT000135969.2':0.00306996)'':0.00197015,('PDT000615898.1':0.00413849,('PDT000538491.1':0.00447928,('PDT000434566.1':0.00372171,'PDT000615809.1':0.00305525)'':0.00071928)'':0.000678843)'':0.000777398)'':0.00124501)'':-5.74699e-05)'':0.00235535)'':0.00503352)'':-0.00404489,('PDT000407999.1':0.0131915,(('PDT000202025.2':0.0134403,'PDT000713933.1':0.0106897)'':0.00253164,(('PDT000427504.1':0.00817473,'PDT000538507.1':0.00773457)'':0.000537692,('PDT000663749.1':0.00774678,(('PDT000202762.2':0.00559431,'PDT000202780.2':0.00653899)'':0.00065968,('PDT000521016.1':0.00500466,(('PDT000426033.1':0.00455941,'PDT000615890.1':0.0052181)'':0.000763687,('PDT000615999.1':0.00440905,'PDT000616120.1':0.00347361)'':0.00112012)'':0.000536672)'':0.00144225)'':0.00136523)'':-0.000320683)'':0.00325084)'':0.00138728)'':-0.0025465)'':0.0071986)'':-0.00336196)'':0.00142434)'':0.000654044)'':-0.00127212)'':0.00155113)'':-0.00064508)'':0.00276205)'':0.00268462)'':-0.0017286)'':-0.00299027)'':0.00340751)'':0.000499025)'':0.00419558)'':0.00531482)'':-0.00837378)'':-0.000437445)'':-0.000526285)'':0.00672389)'':-0.00270219)'':0.00933791)'':-0.00888653)'':0.00585481)'':0.00247095)'':0.00624084,(((('PDT000441359.1':0.0289348,('PDT000677118.1':0.018423,('PDT000205204.2':0.0171564,'PDT000721694.1':0.0149932)'':0.00190699)'':0.0111323)'':0.0157733,('PDT000602892.1':0.0476871,('PDT000694469.1':0.0261314,(('PDT000175861.2':0.00811637,'PDT000175935.2':0.00672723)'':0.0119273,('PDT000175896.2':0.0140603,'PDT000288723.2':0.0145817)'':0.00390149)'':0.0131845)'':0.00687988)'':0.00249663)'':0.00270924,('PDT000425864.1':0.0579811,('PDT000536585.1':0.0368795,((('PDT000144558.3':0.0232412,('PDT000704916.1':0.0164147,'PDT000257782.1':0.018856)'':0.00340692)'':0.00286916,('PDT000622391.1':0.0342858,('PDT000408023.1':0.0179331,('PDT000175918.2':0.0158113,'PDT000433724.1':0.0171764)'':0.000521976)'':0.008054)'':-0.000607849)'':0.00611612,(('PDT000426626.1':0.0249543,'PDT000615966.1':0.0256218)'':0.0103071,(('PDT000520956.1':0.0251571,('PDT000694503.1':0.0232598,'PDT000705113.1':0.0219678)'':0.00382632)'':0.00299242,(('PDT000668792.1':0.0268153,('PDT000175950.2':0.0131616,'PDT000248452.1':0.0170391)'':0.0112785)'':-0.000364455,('PDT000426573.1':0.0220244,('PDT000175874.2':0.0139988,'PDT000720315.1':0.0135478)'':0.00871101)'':0.00310942)'':0.00228052)'':0.00625755)'':0.00131603)'':0.00317016)'':0.00804273)'':-0.00592911)'':0.00201036,((('PDT000668777.1':0.0470652,('PDT000651350.1':0.0380726,('PDT000704924.1':0.022718,('PDT000704939.1':0.0144312,'PDT000347029.1':0.0127756)'':0.0101268)'':0.011711)'':0.00806603)'':-0.00513201,('PDT000427715.1':0.0560532,((('PDT000145885.3':0.0253754,('PDT000713421.1':0.0181722,('PDT000202767.2':0.0203904,'PDT000615985.1':0.0193109)'':0.00263013)'':0.00374038)'':0.00563725,('PDT000643767.1':0.0323799,('PDT000619086.1':0.0262705,('PDT000433444.1':0.0216146,'PDT000279221.1':0.0229063)'':0.00512607)'':0.00130325)'':0.00554646)'':0.00393252,(('PDT000425914.1':0.042396,'PDT000426606.1':0.0363681)'':0.00404328,((('PDT000186183.4':0.00968054,'PDT000030567.1':0.0137445)'':0.029107,(('PDT000427453.1':0.0292314,'PDT000427458.1':0.0323056)'':0.00746121,(('PDT000508905.1':0.0193528,'PDT000643766.1':0.0243647)'':0.0105572,('PDT000705020.1':0.0243288,('PDT000425704.1':0.0131235,'PDT000615808.1':0.0141131)'':0.0092001)'':0.00771539)'':0.00367429)'':0.00170947)'':-0.00116578,((('PDT000223031.2':0.0280079,('PDT000713475.1':0.0254642,('PDT000425568.1':0.0180143,'PDT000713375.1':0.0155859)'':0.00842129)'':0.00361749)'':0.00187725,(('PDT000143965.3':0.0301285,('PDT000217950.2':0.022919,('PDT000202738.2':0.0154369,('PDT000705247.1':0.00962436,'PDT000705264.1':0.00878544)'':0.0041756)'':0.0072829)'':0.00480552)'':-0.000763865,('PDT000423877.1':0.0396134,((('PDT000161908.2':0.0122091,'PDT000175866.2':0.0102884)'':0.00559663,('PDT000175925.2':0.0136641,'PDT000615835.1':0.0156924)'':0.00329037)'':0.00719389,('PDT000705245.1':0.024874,('PDT000640352.1':0.0157582,('PDT000705384.1':0.0166019,('PDT000615949.1':0.00663793,'PDT000705172.1':0.00543787)'':0.00713037)'':0.00316597)'':0.00262184)'':0.00314006)'':0.0101239)'':-0.0061407)'':0.000868521)'':0.00206449,((('PDT000694375.1':0.00196607,('PDT000694547.1':0.00107994,'PDT000694550.1':0.00101964)'':0.0014291)'':0.0345118,('PDT000661421.1':0.0326233,('PDT000250796.2':0.0264524,(('PDT000616013.1':0.0085602,'PDT000619616.1':0.0104353)'':0.00386747,(('PDT000615908.1':0.0112832,'PDT000191150.1':0.0124392)'':0.00300072,(('PDT000615818.1':0.00223413,'PDT000615978.1':0.00386404)'':0.00526688,('PDT000543228.1':0.00543141,('PDT000615959.1':0.00221097,'PDT000616030.1':0.0020813)'':0.00398722)'':0.000346211)'':0.00409059)'':0.00210759)'':0.00926842)'':0.00763158)'':0.00371567)'':-0.00253005,(('PDT000538393.1':0.0390807,'PDT000538478.1':0.0385312)'':0.00233931,(('PDT000137822.2':0.0420074,'PDT000536598.1':0.0440286)'':0.00216594,((('PDT000135950.2':0.0209917,'PDT000107925.1':0.0225727)'':0.00407191,('PDT000538518.1':0.0249985,('PDT000280670.2':0.0141302,'PDT000615907.1':0.0136119)'':0.00398844)'':0.00342866)'':0.00251272,(('PDT000175962.2':0.0127172,'PDT000443840.1':0.00924547)'':0.0233644,((('PDT000202062.2':0.0100179,'PDT000538476.1':0.00676698)'':0.0131951,('PDT000019900.3':0.0239695,((('PDT000175818.2':0.0107499,'PDT000175880.2':0.0110323)'':0.0067542,(('PDT000150128.2':0.010198,'PDT000107513.1':0.00812321)'':0.00368588,('PDT000615849.1':0.0128012,('PDT000181698.2':0.00912145,('PDT000615825.1':0.0068795,'PDT000705094.1':0.0070342)'':0.0012878)'':0.00423207)'':0.000453048)'':0.00160197)'':-0.000205407,('PDT000181825.2':0.0210146,('PDT000425860.1':0.0192101,('PDT000176019.2':0.0199812,('PDT000616105.1':0.00879708,('PDT000102177.1':0.0105813,('PDT000202189.2':0.00708156,('PDT000175881.2':0.00449352,('PDT000538502.1':0.00360888,('PDT000205195.2':0.0021132,'PDT000425501.1':0.00197902)'':0.00188357)'':0.00181204)'':0.00155637)'':0.00105424)'':0.00162111)'':0.00416085)'':0.001635)'':0.000814265)'':-0.00139829)'':0.00351497)'':0.00407032)'':-0.000954348,('PDT000615936.1':0.0307439,(('PDT000433608.1':0.0225993,('PDT000713882.1':0.0148403,'PDT000714086.1':0.015848)'':0.00509548)'':0.00175012,(('PDT000521134.1':0.0214321,('PDT000520744.1':0.0140515,'PDT000520781.1':0.0119109)'':0.00525854)'':0.00370185,(('PDT000713808.1':0.0226136,('PDT000202759.2':0.0166119,('PDT000574923.1':0.0117477,('PDT000615804.1':0.0109967,'PDT000520891.1':0.0110738)'':0.002117)'':0.00627068)'':0.00204509)'':-0.00299243,('PDT000485988.1':0.0215434,('PDT000543110.1':0.0151773,('PDT000067783.1':0.0145696,(('PDT000663844.1':0.00376885,('PDT000615938.1':0.000731783,'PDT000632283.1':0.000867257)'':0.00182427)'':0.00920901,('PDT000426801.1':0.0107571,('PDT000704911.1':0.00995206,('PDT000704921.1':0.00419358,'PDT000508885.1':0.00498806)'':0.00312312)'':0.00147158)'':0.000514769)'':0.00343221)'':0.00438714)'':0.00173056)'':-0.00204605)'':0.00300251)'':0.00288084)'':0.00569963)'':-0.0077126)'':0.00903608)'':-0.005391)'':0.00785242)'':0.000987628)'':-0.00466219)'':0.000429202)'':0.00453219)'':0.00499231)'':-0.00506084)'':0.0157422)'':-0.0121532)'':-0.00389519,(((('PDT000205166.2':0.0104342,'PDT000636222.1':0.0124332)'':0.0203593,('PDT000426997.1':0.0243679,'PDT000690262.1':0.0256565)'':0.00528019)'':0.0062347,(('PDT000219964.1':0.0422976,('PDT000713230.1':0.0340919,('PDT000253677.1':0.0300737,('PDT000415824.1':0.0271989,('PDT000714103.1':0.0243543,(('PDT000143072.2':0.0210569,'PDT000652361.1':0.0196693)'':0.00242501,('PDT000205172.2':0.0221584,('PDT000652328.1':0.015573,(('PDT000661528.1':0.0118974,('PDT000581457.1':0.0106152,'PDT000661575.1':0.0115608)'':0.00261231)'':0.00101898,('PDT000426008.1':0.0178628,('PDT000694483.1':0.0160044,('PDT000538395.1':0.00995958,('PDT000713597.1':0.00950484,('PDT000136036.2':0.00393839,('PDT000135965.2':0.00221683,'PDT000640360.1':0.00297953)'':0.00160197)'':0.00441576)'':0.00156564)'':0.00229145)'':0.00210083)'':-0.00121977)'':0.00276762)'':0.00181415)'':-0.000292241)'':0.00696738)'':0.00120872)'':0.00235555)'':0.00103576)'':0.00167945)'':-0.00288536,(('PDT000221715.3':0.0315034,('PDT000276537.2':0.022371,'PDT000581710.1':0.0226202)'':0.00511255)'':0.0054995,((('PDT000248457.1':0.00250064,'PDT000252014.1':0.00218904)'':0.0223132,(('PDT000423945.1':0.0129601,'PDT000622231.1':0.0168772)'':0.00259507,('PDT000673611.1':0.00569794,'PDT000248464.1':0.00715576)'':0.0118736)'':0.00510141)'':0.00296321,(('PDT000426665.1':0.0235635,('PDT000407995.1':0.0215737,('PDT000175960.2':0.0176773,'PDT000202049.2':0.0211625)'':0.00318142)'':0.00281788)'':0.00411602,(('PDT000427490.1':0.0194533,'PDT000705090.1':0.0160661)'':0.00710839,((('PDT000423884.1':0.00987657,('PDT000423865.1':0.0035318,'PDT000423919.1':0.00324719)'':0.00785784)'':0.00182972,('PDT000034438.1':0.0135203,('PDT000285161.1':0.00190333,'PDT000285162.1':0.00189249)'':0.00943388)'':0.000865629)'':0.00231854,('PDT000622592.1':0.0116947,('PDT000143975.3':0.0137484,('PDT000175922.2':0.0103811,('PDT000693775.1':0.00906441,('PDT000175900.2':0.00719551,('PDT000210709.1':0.00481226,('PDT000616053.1':0.00364753,('PDT000176005.2':0.00312187,'PDT000136017.2':0.00296231)'':0.000447146)'':0.00211498)'':0.00150101)'':0.000935359)'':0.00137897)'':0.000501822)'':0.00195077)'':0.0012362)'':0.0102602)'':-0.00157925)'':0.00193786)'':0.00746677)'':-0.00352811)'':0.00437773)'':0.00150938,((('PDT000408028.1':0.028674,('PDT000125967.1':0.015265,('PDT000616135.1':0.0122236,'PDT000520960.1':0.012915)'':0.00128736)'':0.0113112)'':0.00764544,('PDT000425797.1':0.0506558,((('PDT000508883.1':0.0253401,'PDT000615957.1':0.0249584)'':0.00506362,('PDT000202783.2':0.0331882,('PDT000425606.1':0.0143667,'PDT000441355.1':0.0219317)'':0.0111646)'':0.00138029)'':0.000643405,(('PDT000042735.1':0.025969,('PDT000636137.1':0.0252918,('PDT000543102.1':0.0190416,('PDT000615875.1':0.0125595,'PDT000145878.3':0.0147267)'':0.00459513)'':0.00178971)'':0.0015604)'':0.00732651,(('PDT000069665.1':0.0268062,('PDT000618894.1':0.0254378,('PDT000205138.2':0.0184059,'PDT000069636.1':0.0190521)'':0.00562673)'':0.00477524)'':0.000475446,(('PDT000433789.1':0.0276836,'PDT000622521.1':0.0246755)'':0.00160574,(('PDT000508888.1':0.0266287,('PDT000694401.1':0.0156275,('PDT000176010.2':0.0159567,'PDT000125971.1':0.0161225)'':0.00368087)'':0.00499072)'':-0.000364769,('PDT000042630.1':0.0244,(('PDT000622433.1':0.017728,'PDT000587990.1':0.016619)'':0.00525445,('PDT000647662.1':0.0252284,('PDT000694456.1':0.0255167,(('PDT000704914.1':0.0159718,'PDT000705256.1':0.014802)'':0.00202138,(('PDT000103036.1':0.0182507,'PDT000347030.1':0.0127701)'':0.00090015,(('PDT000543104.1':0.0107253,'PDT000346984.1':0.0117299)'':0.00493117,('PDT000136141.2':0.0150075,('PDT000205134.2':0.0120098,('PDT000704945.1':0.0106469,('PDT000448231.1':0.00682237,('PDT000175916.2':0.00769917,'PDT000175970.2':0.00789633)'':0.000862732)'':0.00247989)'':0.00105115)'':0.00294738)'':0.000211065)'':0.000192059)'':0.000773648)'':0.00474405)'':0.00191534)'':-0.00282049)'':0.00419324)'':0.000462714)'':0.00346028)'':-0.000777835)'':4.20969e-05)'':0.00260431)'':0.0127863)'':-0.00837491)'':-0.000588636,(('PDT000143932.3':0.0337591,(('PDT000095190.3':0.0209345,'PDT000576263.1':0.0282572)'':0.00311368,('PDT000414008.1':0.0301437,(('PDT000485981.1':0.0107572,('PDT000175996.2':0.00549143,'PDT000248449.1':0.00496677)'':0.00927523)'':0.00398855,(('PDT000616009.1':0.0138376,('PDT000426290.1':0.00777158,'PDT000615819.1':0.00765742)'':0.00546598)'':0.00262388,('PDT000033989.1':0.00365106,('PDT000032374.1':0.00148563,'PDT000034406.1':0.00171022)'':0.00133527)'':0.0169325)'':0.000785578)'':0.00470208)'':0.00142204)'':0.0103191)'':0.0021674,(('PDT000672239.1':0.0290274,('PDT000536581.1':0.0209098,('PDT000536574.1':0.0133519,'PDT000536590.1':0.0192286)'':0.00531606)'':0.00496507)'':0.0117951,(('PDT000713368.1':0.0329166,('PDT000508910.1':0.0286399,('PDT000346995.1':0.0272507,'PDT000482609.1':0.0248806)'':0.0043728)'':0.00127014)'':0.00292293,(('PDT000510085.1':0.0357287,('PDT000125974.1':0.026828,(('PDT000661310.1':0.0222641,('PDT000705076.1':0.0183263,('PDT000102176.1':0.0140749,('PDT000668804.1':0.00974795,'PDT000688009.1':0.00811455)'':0.00621901)'':0.0050031)'':0.00152676)'':0.000820905,(('PDT000427581.1':0.0109664,'PDT000368939.1':0.0133609)'':0.00375534,('PDT000705165.1':0.0184648,('PDT000425676.1':0.0156711,('PDT000426397.1':0.00877924,'PDT000615850.1':0.00919136)'':0.00362299)'':0.00263102)'':8.11372e-05)'':0.00466376)'':0.00214437)'':0.00828078)'':-0.00823686,(('PDT000125952.1':0.0403838,('PDT000545928.1':0.0260207,('PDT000615925.1':0.0213392,('PDT000030562.1':0.00828954,('PDT000408000.1':0.0044091,'PDT000408012.1':0.00556099)'':0.00202902)'':0.00876351)'':0.0055888)'':0.00986564)'':-0.00480722,(('PDT000176223.2':0.0208006,'PDT000622506.1':0.0197426)'':0.00132716,(('PDT000103018.1':0.0154598,'PDT000129569.1':0.0139462)'':0.0092039,('PDT000713827.1':0.0215454,('PDT000019249.2':0.0201125,(('PDT000713367.1':0.0172059,('PDT000247208.2':0.0117026,'PDT000705251.1':0.0108985)'':0.00425528)'':0.000153199,(('PDT000103067.1':0.0102319,'PDT000019261.2':0.0115245)'':0.00461205,(('PDT000019254.2':0.0115017,(('PDT000427192.1':0.00721544,'PDT000136167.2':0.00573016)'':0.00036105,('PDT000176020.2':0.00563636,'PDT000107523.1':0.00520724)'':0.00166015)'':0.00510759)'':0.00170676,(('PDT000175829.2':0.0169145,'PDT000427514.1':0.0133434)'':0.000879979,('PDT000175800.2':0.0168899,('PDT000115270.2':0.0129502,(('PDT000175893.2':0.00771283,'PDT000175949.2':0.00689657)'':0.00229983,('PDT000107515.1':0.0117302,'PDT000131235.1':0.00925436)'':0.000409499)'':0.00216844)'':0.00129415)'':0.000274785)'':0.00140534)'':0.00137467)'':-3.01003e-05)'':0.00359945)'':0.000350348)'':0.000365054)'':0.001383)'':0.00198473)'':-0.000983289)'':0.00646615)'':0.00526612)'':-0.00389679)'':0.00166761)'':5.83434e-05)'':0.000245064)'':0.00485107)'':-0.00101071)'':0.00413947)'':0.00198403)'':-0.000541903)'':-0.00108301,(('PDT000443874.1':0.0402178,'PDT000616021.1':0.0406159)'':0.0495977,(((('PDT000534864.1':0.0413854,'PDT000534930.1':0.0404679)'':0.00526579,(('PDT000720443.1':0.0376555,('PDT000535078.1':0.0279753,('PDT000721995.1':0.0230157,('PDT000618974.1':0.020217,('PDT000535043.1':0.018648,('PDT000688118.1':0.0200146,('PDT000535070.1':0.0119096,'PDT000636626.1':0.0103391)'':0.00524317)'':0.00119844)'':0.00301593)'':0.00464748)'':0.00423376)'':0.00745046)'':-0.00445737,(('PDT000536670.1':0.0275495,('PDT000543148.1':0.00962724,'PDT000485923.1':0.0115546)'':0.0148856)'':0.00123009,((('PDT000181661.2':0.0192512,'PDT000543114.1':0.0177049)'':0.00719288,('PDT000485947.1':0.0255808,('PDT000202190.2':0.0171545,'PDT000534908.1':0.0152945)'':0.0097077)'':0.00169954)'':0.000242401,(('PDT000444997.1':0.0149017,('PDT000181716.2':0.0094092,'PDT000636153.1':0.0109115)'':0.00225085)'':0.0128383,(('PDT000720322.1':0.0212228,('PDT000534865.1':0.0133089,'PDT000616165.1':0.0143642)'':0.00479498)'':0.00615971,('PDT000626765.1':0.0240832,('PDT000535256.1':0.0247435,(('PDT000535093.1':0.0133886,'PDT000720449.1':0.016514)'':0.00274475,('PDT000485985.1':0.0123315,('PDT000535383.1':0.0136,('PDT000346730.1':0.0061389,'PDT000346999.1':0.007302)'':0.0076937)'':0.000441235)'':0.00661886)'':0.00260953)'':0.000938961)'':-0.000724758)'':0.00225662)'':0.00253417)'':0.0046709)'':-2.86272e-05)'':0.00669966)'':0.00761932,((('PDT000538415.1':0.0345123,('PDT000445116.1':0.0259344,('PDT000618942.1':0.0226276,'PDT000297991.1':0.0242044)'':0.0037118)'':0.00927109)'':0.00978712,((('PDT000615962.1':0.0184763,('PDT000673452.1':0.00757567,'PDT000543107.1':0.00667303)'':0.0108431)'':0.0121294,('PDT000661688.1':0.0330872,(('PDT000713779.1':0.0239396,('PDT000205162.2':0.019419,'PDT000426124.1':0.0227383)'':0.00493485)'':-0.000796635,('PDT000137823.2':0.0176193,('PDT000394412.1':0.013527,('PDT000636193.1':0.0069707,('PDT000425604.1':0.00378704,'PDT000127576.1':0.00557551)'':0.00460398)'':0.004526)'':0.00506444)'':0.00926649)'':0.00545253)'':-0.00258538)'':0.00165265,(('PDT000336956.1':0.0210977,'PDT000577552.1':0.0251257)'':0.0181094,(('PDT000536633.1':0.0229953,('PDT000176041.2':0.00823123,('PDT000713240.1':0.00235997,('PDT000426472.1':0.00235925,'PDT000107544.1':0.00223845)'':0.00033291)'':0.00749208)'':0.010875)'':0.00322166,('PDT000632290.1':0.0211127,('PDT000433479.1':0.0164124,('PDT000581464.1':0.0135585,('PDT000616052.1':0.0092071,('PDT000616005.1':0.00631312,'PDT000007696.3':0.00796408)'':0.0021786)'':0.00389321)'':0.0024082)'':0.00588423)'':0.000413204)'':0.0102201)'':-0.00291117)'':0.0133427)'':-0.00514232,(((('PDT000030563.1':0.0278027,('PDT000640356.1':0.0109228,'PDT000640361.1':0.0128134)'':0.0189426)'':0.00307631,('PDT000661580.1':0.0311365,('PDT000636164.1':0.0239263,('PDT000661343.1':0.0216014,('PDT000425570.1':0.0146169,('PDT000107934.2':0.0050177,'PDT000508891.1':0.0070738)'':0.00948414)'':0.00319665)'':0.00163128)'':0.0056818)'':0.00439139)'':0.00627485,('PDT000433313.1':0.0559225,(('PDT000181843.2':0.0357217,('PDT000067786.1':0.019881,('PDT000433610.1':0.0172442,('PDT000158299.1':0.0161366,('PDT000543143.1':0.0118686,'PDT000069654.1':0.0115487)'':0.00411418)'':0.00248542)'':0.00355989)'':0.0106176)'':0.00202274,('PDT000593799.1':0.0301286,(('PDT000095191.3':0.0131446,'PDT000616121.1':0.0141236)'':0.00944664,('PDT000622597.1':0.0256412,('PDT000434475.1':0.0118974,'PDT000407992.1':0.0087273)'':0.0121782)'':0.00390664)'':0.0098823)'':0.00329086)'':0.0162325)'':-0.012866)'':0.000933,((('PDT000668910.1':0.0305544,('PDT000636252.1':0.0195424,('PDT000202634.2':0.0106248,'PDT000616054.1':0.0118238)'':0.010221)'':0.00524601)'':0.0107461,(('PDT000616136.1':0.0325626,('PDT000616041.1':0.0237066,'PDT000469695.1':0.0224714)'':0.00846934)'':0.00516456,('PDT000351863.1':0.0350218,('PDT000581604.1':0.0313135,(('PDT000636226.1':0.0201864,'PDT000661775.1':0.0197822)'':0.00367178,(('PDT000423913.1':0.00383343,'PDT000713769.1':0.00335076)'':0.012025,(('PDT000538937.1':0.0118452,('PDT000636191.1':0.00422401,'PDT000661476.1':0.00567005)'':0.00580026)'':0.0036473,('PDT000433777.1':0.01273,('PDT000488881.1':0.0119777,('PDT000632356.1':0.00575914,'PDT000636170.1':0.00540176)'':0.00514194)'':0.00233119)'':0.000858094)'':0.00340731)'':0.00261656)'':0.0100171)'':0.00459642)'':0.00102424)'':0.00311397)'':0.000807842,(('PDT000368928.1':0.0212029,('PDT000189426.1':0.0234096,('PDT000636218.1':0.0207377,('PDT000636262.1':0.0153428,'PDT000636621.1':0.0132399)'':0.00378966)'':0.00272616)'':0.0031137)'':0.00962758,((('PDT000433493.1':0.0202735,'PDT000433664.1':0.0214425)'':0.00665158,(('PDT000647672.1':0.00972931,'PDT000485984.1':0.00942699)'':0.00364609,(('PDT000538524.1':0.0121047,'PDT000485936.1':0.0104097)'':0.00134559,(('PDT000622486.1':0.00997765,('PDT000616093.1':0.00702435,'PDT000082673.4':0.00758075)'':0.00119865)'':0.00293446,('PDT000543085.1':0.00843182,('PDT000144512.2':0.012302,('PDT000673427.1':0.00385921,('PDT000616115.1':0.00442301,'PDT000408021.1':0.0038596)'':0.000736887)'':0.00428469)'':0.0017783)'':0.000448596)'':0.00212728)'':0.00135312)'':0.0123371)'':0.00719971,('PDT000716090.1':0.0390446,(('PDT000123134.2':0.0546349,('PDT000618717.1':0.0313415,('PDT000627032.1':0.0257793,('PDT000176004.2':0.0164671,('PDT000673597.1':0.0170051,('PDT000433818.1':0.00540773,'PDT000661750.1':0.00648277)'':0.0101322)'':0.000943464)'':0.00648748)'':0.00452374)'':0.0114452)'':-0.00691652,(('PDT000716948.1':0.0280124,('PDT000615993.1':0.0143718,'PDT000508903.1':0.017899)'':0.00999445)'':0.00475328,('PDT000661657.1':0.0390499,(('PDT000713366.1':0.0144496,'PDT000484654.1':0.0169089)'':0.00608891,('PDT000433307.1':0.0140199,('PDT000618987.1':0.0146923,('PDT000618852.1':0.0112055,('PDT000636234.1':0.0102295,('PDT000538526.1':0.00613904,'PDT000285157.1':0.00790616)'':0.00362158)'':0.000744474)'':0.0012773)'':0.00129099)'':0.00470058)'':0.0116904)'':-0.00546909)'':0.0066529)'':0.00489134)'':-0.00145902)'':0.00244036)'':0.000723792)'':0.000325083)'':0.000425287)'':0.00689181)'':0.00037617,(((('PDT000433764.1':0.0308325,('PDT000438025.1':0.0238714,'PDT000618754.1':0.0231874)'':0.00546481)'':0.0090068,('PDT000426968.1':0.028621,(('PDT000661568.1':0.0123726,'PDT000661691.1':0.0119429)'':0.0167085,('PDT000543216.1':0.0203227,('PDT000202707.2':0.0203831,'PDT000136010.2':0.0181127)'':0.00218605)'':0.00628797)'':0.00426341)'':0.00526406)'':0.010391,(('PDT000144555.3':0.0487636,('PDT000536642.1':0.0385656,'PDT000346751.1':0.0354651)'':0.0114977)'':-0.00134235,('PDT000103022.1':0.0438679,('PDT000685874.1':0.0477554,('PDT000518644.1':0.0363915,('PDT000536605.1':0.0419521,('PDT000176213.2':0.0369529,('PDT000427579.1':0.034026,('PDT000132123.2':0.0303544,('PDT000131870.2':0.0293945,((('PDT000226141.1':0.0217339,'PDT000346994.1':0.0183801)'':0.00455803,('PDT000713405.1':0.0259675,('PDT000136005.2':0.0192332,('PDT000426429.1':0.00607825,('PDT000482850.1':0.00205307,'PDT000482856.1':0.00224135)'':0.00332234)'':0.00910072)'':0.00732925)'':0.000965587)'':0.00170947,('PDT000408019.1':0.0323438,('PDT000346748.1':0.0341769,(('PDT000673423.1':0.0246069,'PDT000197015.1':0.022834)'':0.00267341,('PDT000485928.1':0.029516,((('PDT000205198.2':0.0147903,'PDT000069626.1':0.0144241)'':0.00457522,('PDT000135955.2':0.0207605,'PDT000125957.1':0.018861)'':0.000640432)'':0.00151509,('PDT000205214.2':0.0231765,(('PDT000325759.1':0.0121565,'PDT000346711.1':0.010519)'':0.00492526,('PDT000616185.1':0.0139258,'PDT000352399.1':0.0135435)'':0.00221804)'':0.00493801)'':-0.00121269)'':0.0059106)'':-0.0024967)'':0.00515683)'':0.0010862)'':-0.00411604)'':0.00269998)'':0.00149321)'':0.00120239)'':0.00186732)'':0.00421494)'':0.000538377)'':0.00109263)'':0.000809398)'':0.0016315)'':0.00104889)'':1.58094e-06,(('PDT000685879.1':0.0713367,('PDT000622551.1':0.0528271,((('PDT000535062.1':0.0271892,('PDT000152112.2':0.0174527,'PDT000433663.1':0.0227443)'':0.00911062)'':0.00531481,('PDT000314336.2':0.0361853,('PDT000346728.1':0.0306029,('PDT000243748.2':0.0289315,('PDT000543147.1':0.0260418,('PDT000346975.1':0.0263452,('PDT000202150.2':0.0166505,'PDT000368913.1':0.0204525)'':0.00545932)'':0.00251217)'':0.000462216)'':0.0031811)'':0.00301507)'':-0.00353016)'':0.00467923,(('PDT000186889.2':0.0330508,('PDT000713964.1':0.0290267,'PDT000368943.1':0.0302268)'':0.00496275)'':0.00969515,('PDT000716968.1':0.0600021,(('PDT000346694.1':0.0283216,('PDT000590079.1':0.0224021,'PDT000460582.1':0.0233663)'':0.00810555)'':0.00642485,('PDT000673450.1':0.041239,(('PDT000136994.2':0.0297081,('PDT000690531.1':0.020219,'PDT000705341.1':0.0178378)'':0.0120731)'':0.00472346,(('PDT000103099.1':0.0319616,('PDT000704984.1':0.0335155,('PDT000175959.2':0.0250485,('PDT000202180.2':0.0131741,'PDT000135968.2':0.0136595)'':0.0102964)'':0.00425922)'':0.0023505)'':-0.00108296,(('PDT000042614.1':0.0304415,'PDT000347004.1':0.0292716)'':0.00166376,('PDT000125979.1':0.033424,('PDT000279210.1':0.0368773,((('PDT000433289.1':0.023299,'PDT000346951.1':0.0227196)'':0.00171424,('PDT000522776.1':0.024459,('PDT000129632.1':0.0236581,'PDT000325638.1':0.0184837)'':0.00570849)'':0.00152921)'':0.000229656,(('PDT000704923.1':0.0264419,'PDT000346710.1':0.0292282)'':0.00220462,('PDT000716952.1':0.0256536,(('PDT000694377.1':0.0218301,('PDT000448229.1':0.0143723,('PDT000425595.1':0.00885753,'PDT000538521.1':0.00900667)'':0.00529584)'':0.00615777)'':-0.000169514,(('PDT000176038.2':0.0137062,('PDT000176018.2':0.0124747,'PDT000176216.2':0.0119325)'':0.00244321)'':0.00483103,('PDT000347008.1':0.018416,('PDT000135958.2':0.0137678,'PDT000615868.1':0.0170166)'':0.0021875)'':0.000299147)'':0.00310042)'':0.00427891)'':0.0006746)'':-0.000886399)'':0.00590706)'':0.000100587)'':-0.000194846)'':-0.000698891)'':0.00338297)'':0.00512138)'':-0.00339869)'':0.0121631)'':-0.00861704)'':-0.00357659)'':0.0103141)'':0.00533361)'':-0.0119699,((('PDT000368940.1':0.0871041,(('PDT000202201.2':0.0351224,('PDT000538480.1':0.0214205,'PDT000538495.1':0.0192299)'':0.00900729)'':0.0001182,('PDT000347016.1':0.0285852,('PDT000433291.1':0.0248883,(('PDT000323679.1':0.0208493,'PDT000378720.1':0.0214414)'':0.00298121,('PDT000618949.1':0.0163125,'PDT000626763.1':0.0159646)'':0.00626051)'':0.00232145)'':0.00264522)'':0.0036984)'':0.0252106)'':-0.0189945,('PDT000251011.2':0.0553535,(('PDT000223028.2':0.023058,('PDT000433814.1':0.0226592,'PDT000347053.1':0.0229645)'':0.00207043)'':0.00849591,(('PDT000668787.1':0.0418883,'PDT000668809.1':0.0362986)'':0.00297427,('PDT000132219.2':0.0442118,((('PDT000343391.1':0.0335171,('PDT000616092.1':0.0262541,('PDT000433449.1':0.0219032,('PDT000618965.1':0.0246565,(('PDT000423902.1':0.0157422,'PDT000720279.1':0.014411)'':0.00165762,('PDT000535409.1':0.0139186,'PDT000661549.1':0.0138095)'':0.00211378)'':0.00463692)'':0.000536634)'':0.00236333)'':0.00654993)'':-0.00286671,(('PDT000543226.1':0.0244176,('PDT000042627.1':0.0153874,('PDT000042487.1':0.0104235,('PDT000548992.1':0.00525344,'PDT000107522.1':0.00450752)'':0.00392057)'':0.00328546)'':0.00803295)'':0.000389438,(('PDT000499080.1':0.0183024,('PDT000433702.1':0.0122016,'PDT000652364.1':0.0127984)'':0.00512184)'':0.00527221,('PDT000618943.1':0.0234935,('PDT000346757.1':0.0186304,(('PDT000426542.1':0.0137903,'PDT000618724.1':0.0114306)'':0.00351827,('PDT000652369.1':0.00963833,'PDT000347039.1':0.0116087)'':0.00389176)'':0.00271342)'':0.00265648)'':0.00102405)'':0.000548917)'':0.00208086)'':-0.000463704,('PDT000368935.1':0.0577198,((('PDT000543116.1':0.0208866,'PDT000485921.1':0.0217483)'':0.00343933,('PDT000543127.1':0.0224254,('PDT000082671.4':0.0188722,('PDT000543092.1':0.022293,('PDT000485967.1':0.0199144,('PDT000102013.2':0.0146046,'PDT000647667.1':0.0138846)'':0.00586934)'':0.00140766)'':0.00167981)'':0.00338554)'':-0.000289513)'':0.00173406,(('PDT000346992.1':0.0259763,('PDT000545930.1':0.0290345,(('PDT000705164.1':0.0138421,('PDT000673433.1':0.0151438,('PDT000508896.1':0.0126794,'PDT000346828.1':0.011578)'':0.00175032)'':0.000457113)'':0.00110914,('PDT000433696.1':0.0140781,('PDT000427618.1':0.0141265,('PDT000204787.1':0.0129104,('PDT000598456.1':0.00748805,'PDT000543089.1':0.00625015)'':0.00306367)'':0.00247642)'':0.00214297)'':-0.000400134)'':0.0067)'':0.00433379)'':-0.00281042,(('PDT000347071.1':0.029939,(('PDT000502096.1':0.0199212,('PDT000346716.1':0.0135113,'PDT000347068.1':0.014868)'':0.00619954)'':-0.000583942,(('PDT000202047.2':0.0141958,'PDT000347025.1':0.0120415)'':0.00382092,('PDT000535068.1':0.013354,('PDT000346903.1':0.0152867,('PDT000522784.1':0.0139918,('PDT000347026.1':0.0107552,'PDT000347051.1':0.012934)'':0.00123291)'':0.00129613)'':0.00102424)'':0.00297323)'':0.00199187)'':0.00717777)'':-0.00424437,(('PDT000202651.2':0.0257104,('PDT000433626.1':0.0169742,'PDT000368915.1':0.0166384)'':0.00744918)'':0.00743678,('PDT000581622.1':0.044702,('PDT000279198.1':0.036223,(('PDT000433794.1':0.0204159,'PDT000581962.1':0.0244369)'':0.00730658,((('PDT000538530.1':0.0163717,'PDT000485955.1':0.0150693)'':0.00354446,('PDT000368934.1':0.0289375,(('PDT000201875.2':0.0149987,'PDT000713563.1':0.0149698)'':0.00310913,(('PDT000538460.1':0.0133727,('PDT000433688.1':0.0143482,('PDT000362441.1':0.0122243,('PDT000149565.2':0.00987454,'PDT000202195.2':0.0100344)'':0.00120615)'':0.00360262)'':0.000339909)'':0.0011964,(('PDT000581467.1':0.0167328,('PDT000433660.1':0.0118446,'PDT000485977.1':0.00951658)'':0.00439618)'':-0.000207664,('PDT000485963.1':0.011709,('PDT000437891.1':0.0127651,('PDT000626762.1':0.0116742,('PDT000433440.1':0.00924556,'PDT000636263.1':0.0100253)'':0.00107489)'':0.00104098)'':0.00100649)'':0.00201962)'':0.00109624)'':0.00195169)'':0.00465965)'':-0.00287173)'':0.000751234,(('PDT000538555.1':0.0239118,'PDT000347074.1':0.0238471)'':0.00321721,((('PDT000618761.1':0.0188108,('PDT000346857.1':0.0097689,'PDT000346946.1':0.0095533)'':0.00775268)'':0.00134258,('PDT000346991.1':0.0187086,('PDT000636178.1':0.00706241,('PDT000616103.1':0.00503721,'PDT000522773.1':0.00560959)'':0.00177269)'':0.00821505)'':0.00549731)'':0.00130382,((('PDT000654370.1':0.0225862,'PDT000103049.1':0.0219411)'':0.00151042,('PDT000543113.1':0.021869,(('PDT000713820.1':0.0103862,('PDT000581703.1':0.00938712,('PDT000095270.3':0.0105951,'PDT000647663.1':0.00916073)'':0.00190718)'':0.000842431)'':0.00144705,('PDT000346822.1':0.0121226,('PDT000324625.1':0.0133996,('PDT000485975.1':0.0126587,('PDT000535280.1':0.0095414,('PDT000618937.1':0.00778409,('PDT000582015.1':0.0067832,'PDT000616107.1':0.0050647)'':0.00281566)'':0.000740094)'':0.00282688)'':0.00148142)'':0.000437399)'':0.000711397)'':0.00413562)'':0.00329757)'':-0.00309555,(('PDT000705260.1':0.0243032,(('PDT000520840.1':0.0151252,'PDT000347022.1':0.0162234)'':0.00108751,(('PDT000621432.1':0.0054651,('PDT000673171.1':0.0026363,'PDT000673173.1':0.00235372)'':0.00207522)'':0.0066802,('PDT000502191.1':0.0126109,('PDT000181659.2':0.0103817,('PDT000618975.1':0.00851425,('PDT000647660.1':0.00680919,'PDT000346803.1':0.00716011)'':0.0016583)'':0.00125491)'':0.00370548)'':-0.00131389)'':0.00332541)'':0.00920398)'':-0.00615331,('PDT000673444.1':0.029483,(('PDT000538461.1':0.017069,('PDT000626810.1':0.0188366,'PDT000347054.1':0.0175413)'':0.000931793)'':0.000378205,('PDT000683479.1':0.0275027,((('PDT000538387.1':0.0112485,'PDT000618939.1':0.0132096)'':0.003054,('PDT000651332.1':0.0188381,('PDT000522774.1':0.0140223,(('PDT000321332.1':0.00830049,('PDT000101138.2':0.00560081,'PDT000426116.1':0.00745349)'':0.00236836)'':0.000230634,('PDT000426814.1':0.00821954,('PDT000618850.1':0.00896731,'PDT000694466.1':0.00783019)'':0.000110009)'':0.00234203)'':0.00405902)'':0.00269289)'':-0.00156115)'':-0.000531328,('PDT000368936.1':0.0202288,(('PDT000433431.1':0.0137617,'PDT000346986.1':0.0158964)'':0.000282748,('PDT000362535.1':0.0166024,('PDT000538528.1':0.0145509,(('PDT000426167.1':0.0109987,'PDT000346744.1':0.00983675)'':0.00184263,(('PDT000425768.1':0.00937174,'PDT000538482.1':0.00818776)'':0.002864,('PDT000426339.1':0.0118384,('PDT000522777.1':0.00828002,'PDT000346876.1':0.00853588)'':0.00182303)'':2.10296e-05)'':0.000734129)'':0.0013259)'':0.00295215)'':-0.00107065)'':0.00233503)'':-0.00178116)'':0.00707641)'':-0.0039004)'':0.00814439)'':-0.00725972)'':0.000591038)'':0.00241675)'':0.00438281)'':-0.00640016)'':0.00582592)'':0.00431805)'':0.0044665)'':-0.00900316)'':-0.00345639)'':0.000885466)'':0.00258433)'':0.0149478)'':-0.0165532)'':0.0160182)'':-0.00551231)'':-0.00280813)'':0.0115057)'':-0.0107321)'':0.000852061,((('PDT000558496.1':0.0387348,('PDT000131919.2':0.0311041,'PDT000469349.1':0.0289252)'':0.00877503)'':0.00426392,('PDT000296233.2':0.0437951,('PDT000538498.1':0.047053,('PDT000502153.1':0.0239698,('PDT000346793.1':0.0251154,('PDT000125955.1':0.0203652,'PDT000501921.1':0.0184181)'':0.00499029)'':0.00443236)'':0.00948413)'':0.00275318)'':-0.000225557)'':0.00211688,((('PDT000521098.1':0.0561436,('PDT000347007.1':0.0479394,('PDT000294786.1':0.0381776,('PDT000720480.1':0.0399782,('PDT000485972.1':0.0326126,('PDT000279212.1':0.0351831,('PDT000405275.1':0.0291631,(('PDT000705248.1':0.00775634,'PDT000705417.1':0.00704886)'':0.0196158,(('PDT000205211.2':0.0285023,('PDT000175952.2':0.0139847,('PDT000042628.1':0.0128395,'PDT000019280.2':0.0132449)'':0.00201249)'':0.00592133)'':-0.00248766,('PDT000346895.1':0.0177813,('PDT000520664.1':0.0142926,('PDT000346795.1':0.0138442,('PDT000520926.1':0.0107476,'PDT000352414.1':0.00956554)'':0.00222098)'':0.00177663)'':0.00185945)'':0.00140182)'':0.00435364)'':0.0036077)'':0.00367611)'':0.000611409)'':0.00358744)'':0.00259943)'':0.0041951)'':0.00932511)'':-0.0148114,(('PDT000651858.1':0.0385135,('PDT000042588.1':0.0342927,(('PDT000346993.1':0.0222817,('PDT000181723.2':0.0174049,'PDT000101139.2':0.0191553)'':0.00287278)'':0.00657304,('PDT000583840.1':0.0271778,('PDT000581554.1':0.0294425,(('PDT000423888.1':0.0219677,'PDT000705337.1':0.0171765)'':0.00577034,(('PDT000145827.3':0.0178164,('PDT000175851.2':0.0125958,'PDT000543318.1':0.0141617)'':0.00301988)'':0.00298938,('PDT000705186.1':0.0291557,('PDT000202746.2':0.0182346,('PDT000107556.1':0.0198199,('PDT000705342.1':0.00837815,('PDT000427719.1':0.00414411,'PDT000705089.1':0.00521285)'':0.00488457)'':0.00458938)'':0.000973839)'':0.00516585)'':-0.000581749)'':0.0015635)'':0.00334513)'':0.00201505)'':-0.00053866)'':0.00459793)'':0.00242507)'':-0.00417729,((('PDT000705414.1':0.0332771,'PDT000107780.1':0.0310885)'':0.00280322,(('PDT000205213.2':0.0300689,'PDT000127573.1':0.0365913)'':0.00113127,('PDT000673574.1':0.0262204,('PDT000176025.2':0.0237289,'PDT000714016.1':0.0228055)'':0.00165371)'':0.00525119)'':0.00161115)'':0.00364111,(('PDT000661720.1':0.0341778,('PDT000018573.1':0.0286891,(('PDT000145883.3':0.0248244,'PDT000129584.1':0.0235532)'':0.000105035,('PDT000042678.1':0.00778242,'PDT000042679.1':0.00720108)'':0.0165458)'':0.00380904)'':0.00413717)'':0.00302439,(('PDT000042456.1':0.0472835,(('PDT000705244.1':0.0319025,('PDT000704980.1':0.0230256,'PDT000103101.1':0.0229312)'':0.0077944)'':-0.00119208,('PDT000141672.2':0.0281183,(('PDT000407972.1':0.0210113,('PDT000042481.1':0.0193652,'PDT000103079.1':0.0173284)'':0.00242237)'':0.00291051,('PDT000448225.1':0.0223762,('PDT000705004.1':0.0223751,('PDT000705381.1':0.0152894,'PDT000352419.1':0.0162222)'':0.00315392)'':0.00224047)'':0.00155074)'':0.00369677)'':0.000135594)'':0.00980302)'':-0.00867271,('PDT000148015.2':0.0580029,(('PDT000125944.1':0.0305542,('PDT000067784.1':0.0269346,('PDT000443913.1':0.0177254,'PDT000713668.1':0.0196789)'':0.00743522)'':0.00166987)'':0.00795891,((('PDT000705110.1':0.0202715,'PDT000705353.1':0.0191635)'':0.0043169,(('PDT000143078.2':0.0235218,'PDT000438278.1':0.0221953)'':0.00170123,('PDT000485979.1':0.0282982,('PDT000704917.1':0.0226305,('PDT000705066.1':0.0215309,('PDT000187347.5':0.0180992,(('PDT000705014.1':0.00951078,'PDT000428846.1':0.0120834)'':0.000864738,('PDT000520978.1':0.00664292,'PDT000521027.1':0.00571628)'':0.00491514)'':0.00404094)'':0.00413123)'':0.000343369)'':0.00446998)'':-0.00085348)'':0.000339311)'':0.000589115,(('PDT000226144.1':0.0407459,('PDT000227581.2':0.0272733,('PDT000175937.2':0.0179612,'PDT000202733.2':0.0232656)'':0.00996154)'':0.0077691)'':-0.00364414,('PDT000426769.1':0.0392794,('PDT000705365.1':0.0268695,(('PDT000705071.1':0.0278534,'PDT000368938.1':0.0241623)'':0.00248118,(('PDT000069640.1':0.0316775,(('PDT000520649.1':0.0204784,'PDT000216784.1':0.015257)'':0.00312049,('PDT000125972.1':0.0229024,('PDT000042511.1':0.0164583,('PDT000205158.2':0.0138706,'PDT000136033.2':0.0105124)'':0.00381831)'':0.00582486)'':0.00160989)'':0.00392802)'':-0.00252482,('PDT000685890.1':0.0331741,('PDT000050249.1':0.0356398,(('PDT000042493.1':0.0222636,'PDT000069699.1':0.0267752)'':0.00161929,('PDT000411333.1':0.0320297,('PDT000103023.1':0.029454,(('PDT000582308.1':0.0234706,('PDT000521080.1':0.0178245,'PDT000346717.1':0.0161757)'':0.00275911)'':0.0017452,('PDT000103102.1':0.0252872,('PDT000711050.1':0.0254306,('PDT000675372.1':0.024471,('PDT000615974.1':0.0207356,('PDT000704987.1':0.0223986,('PDT000427235.1':0.0164998,(('PDT000261982.1':0.0184419,('PDT000661453.1':0.0142424,(('PDT000175983.2':0.0135594,('PDT000704929.1':0.00862281,'PDT000705003.1':0.00853309)'':0.00279919)'':0.00034354,('PDT000030558.1':0.0121466,('PDT000103033.1':0.0138,('PDT000705080.1':0.00933076,('PDT000427043.1':0.00704239,('PDT000042607.1':0.00654273,('PDT000713209.1':0.00272156,'PDT000030549.1':0.00296309)'':0.002708)'':0.00174139)'':0.000746839)'':0.00208648)'':0.00153103)'':-0.000888624)'':0.0028153)'':0.00238812)'':-0.0033826,(('PDT000346722.1':0.0150578,(('PDT000136152.2':0.0116478,'PDT000448227.1':0.0118816)'':0.000968167,('PDT000716811.1':0.0106225,('PDT000705015.1':0.00734691,('PDT000103094.1':0.00696395,('PDT000520865.1':0.00754363,('PDT000202744.2':0.0054377,'PDT000426719.1':0.0049219)'':0.00179182)'':0.000180755)'':0.000696968)'':0.00214963)'':0.00120092)'':0.0030601)'':-0.00134261,('PDT000705079.1':0.0191814,('PDT000125928.1':0.0183213,(('PDT000622415.1':0.0145971,('PDT000505151.1':0.00835281,'PDT000713713.1':0.0100662)'':0.00320726)'':0.0003724,('PDT000042670.1':0.0112717,('PDT000520684.1':0.0103594,(('PDT000704941.1':0.00594254,'PDT000129571.1':0.00894406)'':0.00140637,('PDT000713549.1':0.0087347,('PDT000426570.1':0.00519158,'PDT000720316.1':0.00606902)'':0.0035698)'':0.000108097)'':0.00150686)'':0.000689904)'':0.00133511)'':0.00345972)'':0.000824968)'':-0.00260973)'':0.000446741)'':0.00369914)'':0.000860341)'':0.000377528)'':0.00251735)'':0.00146111)'':0.00147302)'':-0.00567141)'':0.0061336)'':0.000379636)'':-0.00381232)'':0.00435012)'':0.000515452)'':-0.00550947)'':0.00449476)'':0.00322922)'':0.00268357)'':-0.00299375)'':-0.00417103)'':0.00644058)'':0.0122901)'':-0.0162836)'':0.00324099)'':0.000831862)'':-0.000239755)'':0.00186086)'':-0.00174131,((('PDT000124735.1':0.042226,(('PDT000147565.2':0.0285927,'PDT000347033.1':0.0273085)'':0.00111013,('PDT000522775.1':0.0266062,('PDT000346767.1':0.0245846,('PDT000588038.1':0.0227845,('PDT000346990.1':0.0217198,('PDT000201896.2':0.016584,'PDT000618745.1':0.0166016)'':0.00260332)'':0.00107454)'':0.00253329)'':0.00436943)'':4.27584e-05)'':0.00668768)'':-0.00147316,('PDT000042533.1':0.0709446,(('PDT000538522.1':0.0276411,('PDT000252484.1':0.0249307,(('PDT000535218.1':0.0214114,'PDT000042618.1':0.0234145)'':0.00547378,('PDT000502138.1':0.0225564,'PDT000587976.1':0.0223248)'':0.0026019)'':0.000979414)'':0.000366903)'':0.00247515,(('PDT000191160.1':0.0250854,'PDT000673628.1':0.0264832)'':0.00270933,('PDT000654371.1':0.0309286,(('PDT000426989.1':0.0256241,'PDT000042461.1':0.0259504)'':0.000821299,('PDT000347028.1':0.0238251,('PDT000616170.1':0.0166261,'PDT000502018.1':0.0207708)'':0.00589894)'':0.00358967)'':0.00152373)'':0.000533131)'':0.000636327)'':0.01845)'':-0.0147456)'':0.00435768,(('PDT000317708.1':0.0358298,(('PDT000538451.1':0.0266021,'PDT000538484.1':0.0234225)'':0.00543479,(('PDT000363576.1':0.0250198,('PDT000011751.2':0.0202194,('PDT000176031.2':0.0198137,'PDT000616037.1':0.0190365)'':0.00121379)'':0.0039679)'':0.0013946,(('PDT000426181.1':0.0194914,'PDT000414012.1':0.0240943)'':0.00343207,((('PDT000205174.2':0.00223559,'PDT000520741.1':0.0025534)'':0.0193808,('PDT000131237.1':0.0137493,('PDT000175967.2':0.0161036,'PDT000534945.1':0.015057)'':0.00167216)'':0.00345108)'':0.000876077,('PDT000017787.2':0.0271356,(('PDT000713249.1':0.0106706,'PDT000017691.2':0.00876331)'':0.0032069,('PDT000129583.1':0.0142579,('PDT000625081.1':0.0012502,'PDT000134249.2':0.00104911)'':0.0112689)'':0.00183541)'':0.00644124)'':0.00203191)'':0.00090758)'':0.00209594)'':0.00509783)'':0.00208911)'':0.00482937,((('PDT000661689.1':0.0307801,('PDT000175845.2':0.027446,('PDT000205161.2':0.0247403,'PDT000520707.1':0.0204809)'':0.00229697)'':0.00689401)'':0.00209214,(('PDT000443884.1':0.0296317,'PDT000622583.1':0.0300889)'':0.0052008,(('PDT000618989.1':0.0355698,'PDT000202658.2':0.0280362)'':0.000777024,(('PDT000210720.1':0.0241698,('PDT000124871.2':0.011186,'PDT000143094.3':0.0166528)'':0.00771319)'':0.00366491,(('PDT000317703.1':0.0175036,'PDT000508904.1':0.0145925)'':0.0067874,('PDT000661749.1':0.0226774,(('PDT000636381.1':0.0127642,'PDT000362454.1':0.0124559)'':0.00276876,('PDT000668905.1':0.0130178,('PDT000248458.1':0.00931786,('PDT000136162.2':0.00731572,('PDT000423898.1':0.00480028,'PDT000444018.1':0.00446267)'':0.00183811)'':0.000744282)'':0.00370236)'':0.00281812)'':0.0071963)'':0.00142193)'':7.55874e-05)'':0.00441709)'':0.00516414)'':-0.000764279)'':0.0019086,(((('PDT000279202.1':0.0381819,'PDT000279219.1':0.0378567)'':0.0070054,('PDT000191159.1':0.0316396,(('PDT000136064.2':0.0228007,'PDT000279197.1':0.0205457)'':0.00353498,('PDT000279204.1':0.0182641,'PDT000279208.1':0.0204264)'':0.00786109)'':0.00811775)'':0.00585262)'':0.00188422,('PDT000032148.1':0.0619066,('PDT000042469.1':0.0639679,(('PDT000202596.2':0.0297338,'PDT000485976.1':0.0258653)'':0.00479524,(('PDT000069631.1':0.0228443,'PDT000019248.2':0.0225081)'':0.00725379,('PDT000103058.1':0.0290363,('PDT000651375.1':0.0288924,(('PDT000508895.1':0.0363127,(('PDT000175895.2':0.0159546,('PDT000103093.1':0.0166508,'PDT000448207.1':0.0162913)'':0.00121891)'':0.00223363,(('PDT000176229.2':0.00961246,'PDT000448223.1':0.00975774)'':0.00507899,('PDT000145856.3':0.015328,'PDT000704912.1':0.0109616)'':0.00330824)'':0.00126802)'':0.0114306)'':-0.0062627,(('PDT000705016.1':0.0222164,('PDT000705024.1':0.016437,'PDT000191157.1':0.0170743)'':0.00304965)'':0.00236783,(('PDT000042553.1':0.0230395,'PDT000103015.1':0.022942)'':0.00444005,(('PDT000201770.1':0.0193552,('PDT000202193.2':0.0230419,('PDT000459439.1':0.0210978,('PDT000347006.1':0.019674,('PDT000030561.1':0.0160314,('PDT000705252.1':0.0122728,('PDT000621426.1':0.0153535,'PDT000069655.1':0.0116253)'':0.00115158)'':0.00236761)'':0.00144048)'':0.00149596)'':0.000941316)'':0.00260926)'':-0.00222827,(('PDT000427456.1':0.013781,'PDT000705011.1':0.016585)'':0.00484337,('PDT000673418.1':0.0237989,('PDT000150951.2':0.0209229,((('PDT000704922.1':0.00661074,'PDT000705257.1':0.0105584)'':0.00551364,('PDT000425632.1':0.0116666,'PDT000175981.2':0.0123747)'':0.00197884)'':0.00348983,('PDT000279206.1':0.0213829,(('PDT000521044.1':0.0122893,('PDT000205229.2':0.00777028,'PDT000705085.1':0.00853192)'':0.0044129)'':0.000627745,('PDT000640372.1':0.0128556,('PDT000103235.1':0.0106171,('PDT000103077.1':0.00874189,'PDT000129580.1':0.00755381)'':0.00124518)'':0.00277917)'':-0.000164853)'':0.00585642)'':-0.00271753)'':0.00329752)'':0.00204489)'':-0.00223736)'':0.000377137)'':0.00408508)'':-0.000779949)'':-0.00123981)'':0.00491711)'':0.00115668)'':-0.0012593)'':0.00382852)'':0.0105842)'':0.0111444)'':-0.0184011)'':-0.00383838,(((('PDT000430634.1':0.0263416,'PDT000685891.1':0.0286837)'':0.00289525,('PDT000325107.1':0.0272855,('PDT000322071.1':0.020639,'PDT000233056.1':0.0214051)'':0.00575211)'':0.00503585)'':0.00396056,('PDT000364529.1':0.0356512,('PDT000347063.1':0.0318659,('PDT000538413.1':0.0298439,('PDT000616048.1':0.0227919,('PDT000202135.2':0.0122775,'PDT000720329.1':0.0147842)'':0.0105782)'':0.00387015)'':0.00292173)'':0.00141882)'':0.00544044)'':0.00106407,(('PDT000252013.1':0.0464105,('PDT000425698.1':0.0343149,(('PDT000427498.1':0.0305841,'PDT000042612.1':0.0285374)'':0.00274638,(('PDT000175928.2':0.0249225,'PDT000622881.1':0.0227524)'':0.00464899,('PDT000069702.1':0.0231514,('PDT000535056.1':0.0168545,'PDT000346918.1':0.0198138)'':0.0065973)'':0.00175791)'':0.00333592)'':0.00361927)'':0.00488774)'':-0.00285524,(('PDT000448233.1':0.0263194,('PDT000191148.1':0.0303883,('PDT000651369.1':0.017869,'PDT000414010.1':0.0204747)'':0.00550796)'':0.00475883)'':0.00733969,('PDT000210713.1':0.0499323,('PDT000217954.2':0.0374369,(('PDT000421154.1':0.0347777,'PDT000522779.1':0.0299635)'':0.00328857,((('PDT000205180.2':0.0168375,'PDT000713291.1':0.0158377)'':0.00705036,('PDT000581463.1':0.0152791,'PDT000030557.1':0.0147012)'':0.00486339)'':0.00447541,(('PDT000390622.1':0.0258748,'PDT000705423.1':0.0233584)'':0.00601151,('PDT000622477.1':0.0338003,((('PDT000144553.3':0.0196813,'PDT000651358.1':0.0203737)'':0.00693222,('PDT000569669.1':0.0270661,(('PDT000125926.1':0.0142845,('PDT000125980.1':0.00923016,'PDT000019260.2':0.00940094)'':0.00474744)'':0.00399869,('PDT000668782.1':0.0234281,(('PDT000673146.1':0.0131847,'PDT000107530.1':0.0104185)'':0.00243923,('PDT000346867.1':0.012877,('PDT000582016.1':0.0127581,'PDT000661319.1':0.0140782)'':0.000247734)'':0.000690189)'':0.00696217)'':-0.0027067)'':0.00497118)'':-0.000219829)'':-0.000120233,('PDT000636174.1':0.0269077,((('PDT000694449.1':0.0159569,'PDT000457711.1':0.0189949)'':0.00832659,('PDT000145844.3':0.0291249,('PDT000084305.3':0.0241177,('PDT000042726.1':0.0122602,('PDT000633299.1':0.00890247,'PDT000502042.1':0.00688073)'':0.00656057)'':0.00433678)'':0.00696786)'':-0.001024)'':-7.3395e-05,(('PDT000490273.1':0.0286573,('PDT000434798.1':0.0223458,('PDT000570883.1':0.0132683,'PDT000346831.1':0.0117762)'':0.00929419)'':0.00185222)'':2.78112e-05,(('PDT000205209.2':0.028568,'PDT000404821.1':0.0274728)'':0.00399969,((('PDT000619867.1':0.0236186,'PDT000651347.1':0.0275478)'':0.00413135,('PDT000626685.1':0.0289493,(('PDT000704942.1':0.0209624,('PDT000705006.1':0.0130966,'PDT000030560.1':0.0110201)'':0.00584133)'':0.00356888,('PDT000677121.1':0.0270107,((('PDT000694467.1':0.0185218,'PDT000705253.1':0.0200043)'':0.00238116,(('PDT000144527.2':0.0127227,'PDT000713802.1':0.0154796)'':0.00847359,('PDT000216788.1':0.026263,('PDT000705348.1':0.0199262,('PDT000226139.1':0.0192362,('PDT000346766.1':0.0193736,(('PDT000705258.1':0.0102165,('PDT000136065.2':0.00798309,'PDT000705067.1':0.00770401)'':0.00220233)'':0.00185725,('PDT000042547.1':0.0165134,('PDT000521068.1':0.0135282,('PDT000042523.1':0.00854082,('PDT000616164.1':0.00631645,'PDT000415818.1':0.00724695)'':0.00503173)'':0.00066259)'':0.0028545)'':-0.000851184)'':0.00291494)'':0.00110663)'':0.00220571)'':0.000693179)'':-0.000614269)'':0.00153452)'':-0.00235835,((('PDT000107537.1':0.0164062,'PDT000522778.1':0.013579)'':0.00219504,('PDT000534977.1':0.018154,('PDT000346709.1':0.0158934,('PDT000433629.1':0.0121337,'PDT000673561.1':0.0113038)'':0.00110042)'':0.00407473)'':0.000360562)'':0.00186304,('PDT000502216.1':0.0272616,('PDT000176014.2':0.0192953,(('PDT000134068.2':0.0136377,'PDT000581602.1':0.0147557)'':0.00167788,(('PDT000302790.2':0.0104533,'PDT000352406.1':0.00975073)'':0.00248877,('PDT000125983.1':0.0152032,('PDT000502108.1':0.0103443,('PDT000627291.1':0.0127279,('PDT000212485.2':0.00467823,('PDT000705099.1':0.00372229,'PDT000448230.1':0.00375006)'':0.00119357)'':0.00652419)'':0.000491253)'':0.0014275)'':0.000361535)'':0.00294672)'':0.00129434)'':0.00518175)'':-0.00408071)'':0.00043985)'':0.00546468)'':-0.00291891)'':0.00272616)'':-8.62572e-07)'':-0.00325739,('PDT000414183.1':0.045667,('PDT000346988.1':0.0375195,('PDT000705096.1':0.0351201,('PDT000536994.1':0.0323538,((('PDT000407973.1':0.0204936,('PDT000694399.1':0.0107756,'PDT000713302.1':0.00987783)'':0.00724219)'':0.0043737,('PDT000448232.1':0.024478,('PDT000125959.1':0.0186063,('PDT000626706.1':0.0142104,(('PDT000132072.2':0.0117746,'PDT000485964.1':0.0117711)'':0.000363444,('PDT000042655.1':0.00778113,'PDT000110715.1':0.00680287)'':0.00442411)'':0.00280181)'':0.00321637)'':0.00126637)'':0.00122031)'':-0.000231398,(('PDT000572841.1':0.00748428,('PDT000574940.1':0.0032991,'PDT000574941.1':0.00359263)'':0.00477941)'':0.0177238,(('PDT000651377.1':0.0228453,'PDT000380899.1':0.0219542)'':0.00290877,(('PDT000616152.1':0.0208065,'PDT000346864.1':0.0213019)'':0.00190871,('PDT000636247.1':0.0256333,(('PDT000625965.1':0.0242018,'PDT000502175.1':0.0241012)'':0.000656394,(('PDT000713818.1':0.0213825,('PDT000720283.1':0.0201192,(('PDT000621428.1':0.0125233,'PDT000346836.1':0.00901517)'':0.00278969,(('PDT000346989.1':0.00629929,'PDT000407987.1':0.00663091)'':0.00719661,('PDT000286105.1':0.0137119,('PDT000347067.1':0.010269,('PDT000135986.2':0.00884777,'PDT000581606.1':0.00857383)'':0.00272449)'':0.0006195)'':0.00154609)'':0.000366578)'':0.00418045)'':0.00282604)'':-0.00303973,('PDT000633642.1':0.0230589,('PDT000175991.2':0.0196652,((('PDT000705077.1':0.0148551,('PDT000704909.1':0.00642127,'PDT000705379.1':0.00665783)'':0.00624884)'':0.00631356,(('PDT000352392.1':0.010671,'PDT000501875.1':0.0121986)'':0.00607615,(('PDT000673459.1':0.0085238,'PDT000705169.1':0.0078794)'':0.00248372,(('PDT000713614.1':0.0085283,('PDT000482577.1':0.00409341,'PDT000705097.1':0.00488415)'':0.00281762)'':0.000685566,('PDT000175931.2':0.00858912,('PDT000352385.1':0.00489155,'PDT000415826.1':0.00597695)'':0.00154363)'':0.00192472)'':0.00148573)'':0.00661639)'':0.000380242)'':-0.000466668,(('PDT000705084.1':0.0151539,('PDT000103045.1':0.0155658,'PDT000103075.1':0.017784)'':0.000562004)'':0.00270155,('PDT000485934.1':0.0231289,('PDT000626690.1':0.0189688,('PDT000107532.1':0.0165287,(('PDT000713787.1':0.0117052,'PDT000103050.1':0.0142303)'':0.00165968,('PDT000042625.1':0.0136585,(('PDT000135945.2':0.00792842,'PDT000535289.1':0.00795408)'':0.00474508,('PDT000125947.1':0.0124591,('PDT000705354.1':0.0119243,('PDT000501918.1':0.00958944,('PDT000346796.1':0.0090456,(('PDT000705064.1':0.00497631,('PDT000705349.1':0.00359209,'PDT000346762.1':0.00409945)'':0.00089654)'':0.00119258,('PDT000463601.1':0.00586775,('PDT000705350.1':0.00433954,('PDT000135937.2':0.00373025,'PDT000705343.1':0.004249)'':0.000645186)'':0.00079092)'':0.000732483)'':0.00193319)'':0.000354047)'':0.0021552)'':0.000493892)'':-0.000812448)'':0.00193911)'':-0.000387258)'':0.00315757)'':0.000886187)'':0.00643451)'':-0.0059379)'':-0.000676871)'':0.00364324)'':0.00116834)'':-0.00369275)'':0.00319698)'':0.00328274)'':-0.00265454)'':0.000759246)'':-0.000752561)'':-0.00160647)'':0.00769621)'':0.0024123)'':0.00018487)'':0.00524597)'':-0.0143071)'':0.00469693)'':-0.00146813)'':-0.000425745)'':0.00306218)'':-0.0030761)'':0.00700924)'':-0.00385293)'':-0.00130443)'':0.00406074)'':0.00442207)'':0.0110923)'':-0.0154056)'':0.00194661)'':0.000271666)'':-0.00323222)'':0.00308788)'':0.00155582)'':5.1613e-05)'':-0.00251705)'':0.00872184)'':-0.00702213)'':0.00267328)'':0.00182445)'':0.00363716)'':0.0404282)'':-0.0403912)'':0.029058)'':-0.00517777)'':-0.0169364)'':0.145196)'':-0.116567)'':0.0138408)'':0.0153042)'':0.064661)'':0.0449295,(((((('PDT000248686.1':0.165193,('PDT000248683.1':0.0109608,'PDT000248702.1':0.00980796)'':0.162734)'':0.0180149,(('PDT000469852.1':0.044108,'PDT000553143.1':0.0492482)'':0.0215795,(('PDT000132121.2':0.0505189,'PDT000233301.2':0.0623921)'':0.00149787,(('PDT000281627.3':0.0246081,'PDT000132434.3':0.024548)'':0.00843302,(('PDT000316473.2':0.0238914,'PDT000063271.3':0.0251493)'':0.00355323,('PDT000671881.1':0.0295998,('PDT000417257.2':0.0239299,('PDT000399361.2':0.0166679,('PDT000492053.2':0.0195527,('PDT000050574.2':0.0156042,('PDT000063273.3':0.0149699,('PDT000213907.3':0.0136684,('PDT000630810.2':0.0140201,('PDT000235718.3':0.00877788,('PDT000146520.3':0.00648358,'PDT000372148.2':0.00655382)'':0.00204502)'':0.00258366)'':0.00136309)'':0.00230732)'':0.00120891)'':0.000558724)'':0.00154371)'':0.001117)'':0.00790981)'':-0.00305756)'':0.00236784)'':0.0267736)'':0.00852415)'':0.119672)'':0.00993425,(('PDT000009324.2':0.0651316,((('PDT000009325.2':0.011802,'PDT000249744.1':0.0131214)'':0.0199782,('PDT000009322.2':0.0259254,'PDT000009326.2':0.0216466)'':0.0072471)'':0.0027271,('PDT000009329.2':0.0283931,(('PDT000009328.2':0.0168481,('PDT000008808.2':0.0120074,'PDT000009321.2':0.0164281)'':0.00543991)'':0.00684411,('PDT000006505.2':0.0196565,('PDT000009327.2':0.017186,'PDT000357687.1':0.0158939)'':0.00735901)'':0.00563832)'':0.00233489)'':0.00678865)'':0.0223836)'':0.0369475,((('PDT000372025.2':0.0578896,'PDT000248717.1':0.0517614)'':0.0302733,(('PDT000081274.1':0.0403675,'PDT000005204.2':0.0437976)'':0.00919043,(('PDT000443055.1':0.0202486,'PDT000610895.1':0.0200811)'':0.010428,('PDT000081321.1':0.0139059,'PDT000081389.1':0.0176825)'':0.011704)'':0.0224135)'':0.0250445)'':0.00268948,('PDT000358753.1':0.0870195,(('PDT000526192.1':0.04692,('PDT000038929.1':0.00618047,'PDT000248710.1':0.00437413)'':0.0383372)'':0.0134643,('PDT000081320.1':0.0727908,('PDT000030354.1':0.0424445,(('PDT000081272.1':0.0232004,'PDT000081390.1':0.0154868)'':0.0176982,('PDT000198906.1':0.0245039,('PDT000081261.1':0.0166948,'PDT000081287.1':0.0176417)'':0.00827577)'':0.0172536)'':0.00332494)'':0.0186991)'':-0.00082143)'':0.0367231)'':-0.00898608)'':0.0189231)'':0.0947013)'':0.00178251,(('PDT000587963.1':0.20203,(('PDT000250012.1':0.0310125,('PDT000553196.1':0.0225518,'PDT000248691.1':0.0229294)'':0.00459383)'':0.149411,(('PDT000553213.1':0.0371682,'PDT000346755.1':0.0363328)'':0.0191928,('PDT000015425.3':0.0386453,(('PDT000015426.3':0.0132389,'PDT000008732.2':0.0166462)'':0.00914525,('PDT000008482.2':0.0281786,(('PDT000132056.2':0.0245359,'PDT000010341.3':0.0261467)'':0.000586294,('PDT000717614.1':0.025012,('PDT000306476.2':0.0152607,('PDT000325686.1':0.00885512,('PDT000634640.1':0.00622664,('PDT000157910.2':0.00447423,('PDT000148291.2':0.00505585,'PDT000283043.2':0.00450876)'':0.00093389)'':0.00135262)'':0.00123915)'':0.00457179)'':0.00476003)'':0.00692867)'':0.00251171)'':-0.000376119)'':0.0117453)'':0.0209173)'':0.119913)'':0.0179647)'':-0.0158583,((('PDT000251372.1':0.10151,('PDT000646110.1':0.0730062,'PDT000081312.1':0.0661858)'':0.0358674)'':0.0879731,((('PDT000105660.3':0.0659537,'PDT000081326.1':0.0694673)'':0.0948246,((('PDT000157087.2':0.03408,'PDT000129855.2':0.0326778)'':0.0487536,('PDT000325114.1':0.0459465,'PDT000248697.1':0.040453)'':0.0449135)'':0.0505251,('PDT000377122.1':0.123733,('PDT000203854.2':0.0391462,('PDT000016399.2':0.0324219,('PDT000132178.2':0.0106904,'PDT000391619.1':0.0124504)'':0.0175764)'':0.0057408)'':0.0846621)'':0.013405)'':0.0197716)'':0.0089003,('PDT000635979.1':0.166145,(('PDT000718973.1':0.0881818,('PDT000213256.2':0.00962013,'PDT000248696.1':0.0119089)'':0.0719582)'':0.0609862,((('PDT000238203.2':0.00828767,'PDT000594845.1':0.00814003)'':0.0437216,('PDT000081309.1':0.0295765,'PDT000083464.1':0.0260339)'':0.0352509)'':0.0111417,(('PDT000252999.1':0.0235861,('PDT000502995.1':0.0155328,('PDT000253137.2':0.00374644,'PDT000502994.1':0.00303931)'':0.00922143)'':0.0104897)'':0.0318706,((('PDT000280665.2':0.046542,'PDT000500617.1':0.0442449)'':0.0077313,('PDT000081265.1':0.0563342,('PDT000132086.2':0.0340044,'PDT000081301.1':0.0318284)'':0.00892107)'':0.00546671)'':0.00132333,(('PDT000081268.1':0.0254547,('PDT000081269.1':0.0131118,'PDT000081292.1':0.0082621)'':0.0121591)'':0.0204211,('PDT000198907.1':0.0482926,('PDT000248615.1':0.0392817,('PDT000081279.1':0.032257,('PDT000030353.1':0.005357,'PDT000081300.1':0.0052876)'':0.0309326)'':0.00410423)'':0.00377693)'':-0.000511951)'':0.00728993)'':0.00186545)'':0.0230957)'':0.0767571)'':0.0173029)'':0.00615999)'':0.0136637)'':-0.00495581,((('PDT000253420.2':0.168964,(('PDT000081295.1':0.0211798,'PDT000081324.1':0.0246217)'':0.00456374,('PDT000253453.2':0.028395,('PDT000253471.2':0.0220088,'PDT000248682.1':0.0227747)'':0.00343345)'':0.00299107)'':0.149464)'':-0.00553925,(('PDT000040688.2':0.0492535,('PDT000105646.3':0.027332,'PDT000622801.1':0.0333749)'':0.0191442)'':0.0432421,(('PDT000610897.1':0.061847,('PDT000190562.2':0.0497687,'PDT000081322.1':0.0511283)'':0.0111205)'':0.00350963,('PDT000357650.1':0.0539229,('PDT000248656.1':0.0256162,'PDT000357646.1':0.0285537)'':0.0285216)'':0.0141598)'':0.0356884)'':0.0778702)'':0.00339769,(('PDT000248687.1':0.143129,('PDT000248732.1':0.0690863,('PDT000132071.2':0.0446666,('PDT000132050.2':0.0309649,'PDT000392505.1':0.0279219)'':0.0197679)'':0.0186902)'':0.0643103)'':0.0349767,((('PDT000081314.1':0.0758663,('PDT000081282.1':0.0312228,'PDT000198903.1':0.0320202)'':0.0345347)'':0.00464488,('PDT000619097.1':0.0740474,(('PDT000367765.1':0.0551831,'PDT000625465.1':0.0515849)'':0.0051279,(('PDT000480347.2':0.0142847,'PDT000677896.1':0.014718)'':0.0320391,('PDT000016388.2':0.0347632,(('PDT000129811.2':0.0301964,'PDT000675063.1':0.0344495)'':0.00238602,('PDT000553157.1':0.0234736,('PDT000553132.1':0.0166993,'PDT000553158.1':0.0188956)'':0.00127493)'':0.0126647)'':0.00198495)'':0.00955796)'':0.0101928)'':0.00501112)'':0.0026218)'':0.042827,(('PDT000198908.1':0.0899323,('PDT000083525.1':0.0479252,('PDT000081297.1':0.0378881,('PDT000081315.1':0.0210213,('PDT000248608.1':0.0155378,('PDT000081270.1':0.0121296,'PDT000081291.1':0.011318)'':0.00379513)'':0.0089614)'':0.00319827)'':0.021258)'':0.0093536)'':0.0209088,(('PDT000161967.2':0.0276674,'PDT000615194.1':0.0313055)'':0.0238194,(('PDT000619770.1':0.0486984,('PDT000275056.2':0.0292514,'PDT000114763.2':0.0286111)'':0.0102095)'':0.000949291,('PDT000120910.2':0.0399934,(('PDT000553221.1':0.016148,('PDT000553216.1':0.0141289,'PDT000553218.1':0.0150117)'':0.00254311)'':0.00886755,('PDT000132276.2':0.0334531,('PDT000553214.1':0.0206537,'PDT000553217.1':0.0210007)'':0.00940853)'':-0.000854368)'':0.0185393)'':-0.00117269)'':0.0108595)'':0.0172276)'':0.030441)'':0.0577537)'':0.00657204)'':0.00134716)'':0.000658694)'':0.015228)'':0.0149593,(((('PDT000619831.1':0.0216997,'PDT000346533.1':0.0273746)'':0.144137,(('PDT000720387.1':0.0660583,(('PDT000625502.1':0.0337355,'PDT000688673.1':0.0323314)'':0.0282214,('PDT000083466.2':0.0494839,'PDT000248679.1':0.0548411)'':0.00956943)'':0.00169559)'':0.0119218,((('PDT000570964.1':0.0670804,'PDT000610893.1':0.0639186)'':0.00263307,('PDT000081316.1':0.0491993,'PDT000198904.1':0.0414635)'':0.029244)'':0.000768094,('PDT000687850.1':0.097917,(('PDT000129871.2':0.039985,('PDT000610881.1':0.0182141,'PDT000463731.1':0.0224468)'':0.0291397)'':0.00021333,('PDT000625369.1':0.05057,(('PDT000161548.2':0.0230645,'PDT000601931.1':0.0205209)'':0.00928952,('PDT000376164.1':0.0225009,'PDT000016360.2':0.0170611)'':0.0104417)'':0.0181263)'':-0.00148346)'':0.0237871)'':-0.00461965)'':0.00664762)'':0.089771)'':0.0124292,(('PDT000547722.1':0.0291931,('PDT000129821.2':0.0299643,('PDT000387118.1':0.0158333,('PDT000213258.2':0.00928848,'PDT000008733.2':0.00874742)'':0.00528517)'':0.00899683)'':0.00257955)'':0.108108,((('PDT000425602.1':0.0126523,'PDT000467238.1':0.0156505)'':0.0192304,('PDT000160454.2':0.0154455,('PDT000320824.1':0.0161708,('PDT000562135.1':0.0139722,('PDT000319318.1':0.00679562,('PDT000156900.2':0.00431618,'PDT000335803.1':0.00436906)'':0.00308176)'':0.00306658)'':0.00319048)'':0.00154895)'':0.012933)'':0.0552329,(('PDT000468261.1':0.0176988,'PDT000523353.1':0.0180507)'':0.00104023,('PDT000010345.3':0.0212443,(('PDT000016398.2':0.0112762,('PDT000332817.1':0.00700724,'PDT000083471.1':0.00640296)'':0.00270708)'':0.00269292,('PDT000150177.1':0.0129572,(('PDT000353578.1':0.00679932,('PDT000158739.2':0.0064841,'PDT000466906.1':0.0063517)'':0.000649826)'':0.000329607,('PDT000083581.2':0.00720457,('PDT000032394.1':0.00647105,('PDT000150176.1':0.00585698,('PDT000524018.1':0.0031585,'PDT000083596.1':0.00383311)'':0.00131208)'':0.000715729)'':0.000894209)'':0.000441904)'':0.00461685)'':0.000533034)'':0.00852359)'':-0.00305079)'':0.0665946)'':0.0611558)'':0.0356299)'':0.00580043,('PDT000126260.2':0.217338,(('PDT000470820.1':0.10489,('PDT000687920.1':0.0301772,'PDT000719038.1':0.0307286)'':0.0639035)'':0.0926815,(((('PDT000494266.1':0.019387,'PDT000553191.1':0.0233092)'':0.114353,('PDT000689085.1':0.114324,('PDT000409044.1':0.0246786,'PDT000251675.1':0.0289402)'':0.0855077)'':0.0200003)'':0.0156314,(('PDT000293433.1':0.0119061,'PDT000293437.1':0.00855303)'':0.0969867,('PDT000587566.1':0.0459519,('PDT000016362.2':0.02217,'PDT000011802.2':0.0165866)'':0.0230542)'':0.0586054)'':0.0436072)'':0.0176009,(((('PDT000132205.2':0.0105527,'PDT000132316.2':0.00713137)'':0.0362351,(('PDT000213196.2':0.0338961,'PDT000103580.2':0.0326152)'':0.0161625,(('PDT000107950.3':0.0289436,'PDT000168469.1':0.0230212)'':0.0166225,('PDT000367254.1':0.0360178,(('PDT000637604.1':0.0206705,('PDT000690909.1':0.0137899,'PDT000016385.2':0.0147336)'':0.00282275)'':0.00191659,('PDT000610891.1':0.0219591,('PDT000321839.1':0.0203005,('PDT000365931.1':0.0136145,('PDT000132042.2':0.0107962,('PDT000553159.1':0.00990845,('PDT000534067.1':0.00970018,('PDT000120874.2':0.0074698,'PDT000120908.2':0.0080132)'':0.00109392)'':0.000851513)'':0.000241875)'':0.00459805)'':0.00438068)'':0.000472923)'':-0.000699608)'':0.00897978)'':0.00759257)'':0.003002)'':0.00677967)'':0.0483925,((('PDT000470564.1':0.0554967,'PDT000553154.1':0.0513113)'':0.00366907,(('PDT000017847.2':0.0120085,('PDT000040705.3':0.0131336,('PDT000168468.1':0.00477616,'PDT000208737.1':0.00458826)'':0.00680955)'':0.002988)'':0.00631768,('PDT000107952.3':0.0267301,('PDT000147688.3':0.0212926,('PDT000593049.1':0.0188208,('PDT000148111.2':0.0134606,('PDT000132077.2':0.0104464,('PDT000191698.3':0.011066,('PDT000300556.1':0.00696944,('PDT000160011.3':0.00548409,'PDT000161609.3':0.00526431)'':0.00137856)'':0.00191266)'':0.00263033)'':0.00104433)'':0.0045301)'':0.00126898)'':0.00352076)'':0.00190036)'':0.0373076)'':0.0108337,((('PDT000238183.2':0.0174924,'PDT000353616.1':0.0204536)'':0.00717399,('PDT000306097.1':0.0245845,(('PDT000040700.2':0.0152587,'PDT000061336.2':0.0194608)'':0.00303135,('PDT000132224.2':0.016455,'PDT000716351.1':0.0156411)'':0.00314828)'':0.00286301)'':0.00195813)'':0.0188813,(('PDT000553155.1':0.0320179,'PDT000016376.2':0.0371675)'':0.00940206,('PDT000317167.1':0.0187423,('PDT000610879.1':0.017818,('PDT000619626.1':0.0159679,('PDT000572832.1':0.00974092,('PDT000384336.1':0.00755457,('PDT000306093.1':0.00639307,('PDT000040696.2':0.00538712,('PDT000718884.1':0.00242793,'PDT000008637.2':0.00425336)'':0.00165819)'':0.00130183)'':0.00153262)'':0.00261628)'':0.00401402)'':0.00222329)'':0.00137663)'':0.0223169)'':0.00176566)'':0.0200209)'':0.0310599)'':0.0325851,((('PDT000157840.2':0.00789439,('PDT000131981.2':0.00715382,'PDT000553164.1':0.00701928)'':0.000933309)'':0.103429,((('PDT000161553.2':0.0219868,'PDT000502282.1':0.0247321)'':0.0210142,('PDT000293434.1':0.0365164,('PDT000553195.1':0.0347957,'PDT000392510.1':0.030643)'':0.0053799)'':0.0106291)'':0.000198558,(('PDT000293440.1':0.0275887,('PDT000016369.2':0.0207452,('PDT000129866.2':0.00708017,'PDT000132191.2':0.00733483)'':0.0139226)'':0.00878874)'':0.0102926,(('PDT000414381.1':0.0198436,('PDT000421018.1':0.0120586,('PDT000213257.2':0.00391107,'PDT000484106.1':0.002874)'':0.00896124)'':0.00549674)'':0.0112596,('PDT000103607.2':0.0412028,('PDT000481467.1':0.035352,('PDT000553197.1':0.0237606,('PDT000359621.1':0.0219432,(('PDT000667874.2':0.00940383,'PDT000379482.1':0.00888277)'':0.00504852,('PDT000553198.1':0.0134942,('PDT000495428.1':0.0109791,('PDT000132000.2':0.00392146,('PDT000619720.1':0.00262672,'PDT000251860.1':0.00336669)'':0.000761566)'':0.00955267)'':0.00207937)'':0.00030711)'':0.00339553)'':0.00522758)'':0.00674672)'':0.00316079)'':-0.0041565)'':0.0093763)'':0.00741883)'':0.0654357)'':-0.0434122,(('PDT000253431.2':0.0469656,('PDT000168470.1':0.0422916,('PDT000060941.2':0.0196116,('PDT000635558.2':0.0166097,('PDT000711044.1':0.00860951,'PDT000168486.1':0.00972269)'':0.00354151)'':0.00673288)'':0.0203276)'':0.00855703)'':0.00868898,(('PDT000034422.1':0.0243892,('PDT000082652.1':0.0138694,'PDT000083557.1':0.0166944)'':0.0147243)'':0.0360409,(('PDT000249008.3':0.0146076,'PDT000397313.2':0.01481)'':0.00766836,((('PDT000564269.2':0.0194278,('PDT000213255.3':0.010718,'PDT000123295.1':0.00743301)'':0.0143843)'':-0.00178669,('PDT000120860.3':0.0185284,('PDT000249006.3':0.016473,(('PDT000397314.2':0.00817105,'PDT000108923.3':0.00752695)'':0.000287576,(('PDT000123269.1':0.00737851,'PDT000123345.1':0.00616599)'':0.000582534,('PDT000115039.3':0.00618794,('PDT000107943.3':0.00475032,'PDT000107947.3':0.00491103)'':0.0015586)'':0.000643988)'':0.00112484)'':0.00668933)'':0.00125276)'':0.00305009)'':-0.00293828,(('PDT000146006.3':0.0175393,('PDT000208709.1':0.0119891,('PDT000249003.3':0.005804,('PDT000249012.3':0.00285565,'PDT000495427.1':0.00344309)'':0.00277478)'':0.00753036)'':0.00297744)'':-0.00205519,(('PDT000553199.1':0.0133635,'PDT000553200.1':0.0123106)'':0.00610249,('PDT000321316.1':0.0268011,('PDT000132379.3':0.0156206,('PDT000120880.3':0.0125264,('PDT000108910.3':0.0119128,('PDT000651762.2':0.00961556,('PDT000123286.1':0.0104238,('PDT000161590.3':0.00716282,('PDT000108889.3':0.00706528,('PDT000108898.3':0.00438818,(('PDT000249014.3':0.00413292,('PDT000123310.1':0.00293879,('PDT000161913.3':0.00222223,'PDT000249013.3':0.00207047)'':0.000711962)'':0.000495627)'':0.000291437,(('PDT000108871.3':0.0026198,'PDT000108912.3':0.00276566)'':0.000535394,('PDT000108901.3':0.00231731,'PDT000108922.3':0.00197625)'':0.000959761)'':0.000500168)'':0.0020252)'':0.00104034)'':0.000416766)'':0.000488587)'':0.000452643)'':0.000847394)'':0.0017733)'':0.00201585)'':0.00798749)'':-0.003255)'':-0.00154123)'':0.000458485)'':0.00310059)'':0.0426567)'':-0.00914203)'':0.0224535)'':0.0570787)'':0.0399743)'':0.0135637)'':0.0221693)'':-0.0211522)'':0.0191043)'':0.0126735,('PDT000502133.1':0.233485,((((('PDT000501850.1':0.156742,('PDT000139234.2':0.101045,'PDT000470821.1':0.100898)'':0.0561182)'':0.026744,('PDT000718986.1':0.114145,('PDT000719207.1':0.0633927,('PDT000477461.1':0.0334836,('PDT000631944.1':0.0290047,('PDT000250865.2':0.0245059,'PDT000636023.1':0.029231)'':0.00313201)'':0.00349833)'':0.0314119)'':0.0499892)'':0.0704322)'':0.0108049,((('PDT000215086.3':0.0257392,('PDT000285807.3':0.0228045,('PDT000176790.3':0.00654029,('PDT000213213.3':0.00264433,'PDT000098342.3':0.00375055)'':0.00503742)'':0.0100797)'':0.00270784)'':0.132336,(('PDT000083593.1':0.0608842,'PDT000251988.1':0.0719768)'':0.0824408,((('PDT000629323.1':0.0268105,'PDT000480306.1':0.0276742)'':0.0396768,('PDT000635563.1':0.0607237,('PDT000553150.1':0.054145,'PDT000251398.1':0.043679)'':0.00956326)'':0.00923557)'':0.0656469,(('PDT000641374.1':0.073486,('PDT000485915.1':0.0406087,('PDT000610889.1':0.0450389,('PDT000502123.1':0.0299927,('PDT000713408.1':0.0285395,('PDT000104683.2':0.0205784,'PDT000661555.1':0.021174)'':0.00462096)'':0.00892026)'':0.00861695)'':0.0025252)'':0.021707)'':0.0109758,('PDT000132143.2':0.0951369,('PDT000183022.3':0.0790629,('PDT000502189.1':0.0502916,('PDT000081313.1':0.0570212,('PDT000385041.1':0.0430814,('PDT000610882.1':0.0186349,'PDT000083505.1':0.0228914)'':0.0186084)'':0.00282844)'':0.00237283)'':0.019506)'':0.00685268)'':0.00518677)'':0.0488493)'':0.014532)'':0.00736272)'':-0.0108135,((('PDT000293430.1':0.0409205,('PDT000293428.1':0.020551,'PDT000293500.1':0.0220418)'':0.0171756)'':0.0897833,((('PDT000553202.1':0.0432116,('PDT000120940.2':0.0233636,'PDT000405303.1':0.0195642)'':0.0238238)'':0.0249408,('PDT000132243.2':0.0614537,('PDT000625519.1':0.0568987,('PDT000324640.1':0.0371508,'PDT000625385.1':0.0418093)'':0.0099488)'':0.00788087)'':0.0101672)'':0.034595,('PDT000395412.1':0.12128,(('PDT000251568.1':0.0743972,('PDT000267830.1':0.0212633,'PDT000267831.1':0.027309)'':0.0456986)'':0.0148756,('PDT000719012.1':0.0790856,('PDT000148191.3':0.0779593,('PDT000131921.3':0.0678077,('PDT000083588.1':0.057167,'PDT000251562.1':0.06197)'':0.00476275)'':0.00725331)'':0.0110228)'':-0.00558387)'':0.0192454)'':0.00939288)'':0.0147588)'':-0.00740734,((('PDT000610894.1':0.0319638,('PDT000083485.1':0.0253085,('PDT000083465.1':0.00237244,'PDT000083502.1':0.00331845)'':0.0152208)'':0.0126005)'':0.0254607,('PDT000083598.1':0.0469168,('PDT000081262.1':0.0387244,('PDT000083548.1':0.0299742,(('PDT000524019.1':0.013495,('PDT000083506.1':0.00373628,'PDT000083551.1':0.0045546)'':0.00576105)'':0.00357164,('PDT000524020.1':0.00922033,('PDT000083437.2':0.0109293,('PDT000082649.1':0.00527282,'PDT000083565.1':0.00639918)'':0.00340868)'':0.00124129)'':0.00241018)'':0.0206785)'':0.00325161)'':0.0117829)'':0.00555763)'':0.0520812,(((('PDT000115163.1':0.032533,'PDT000115165.1':0.0324525)'':0.0182938,('PDT000248617.1':0.0100547,'PDT000248716.1':0.0069232)'':0.0379556)'':0.0164794,(('PDT000081267.1':0.0329998,('PDT000081286.1':0.0252445,'PDT000198902.1':0.0244462)'':0.0122869)'':0.0158729,('PDT000248695.1':0.0497412,('PDT000610898.1':0.0340396,('PDT000016377.2':0.0307988,(('PDT000081298.1':0.00155167,'PDT000081323.1':0.00174536)'':0.0243107,('PDT000081294.1':0.0246883,('PDT000619801.1':0.0129442,'PDT000016375.2':0.013762)'':0.00620866)'':0.00905629)'':0.00862344)'':0.00149536)'':0.00992901)'':0.00336353)'':0.0143755)'':0.0333948,((('PDT000228376.1':0.111162,('PDT000290382.2':0.0540879,('PDT000016357.2':0.0165081,'PDT000168473.1':0.0214653)'':0.0236418)'':0.0420359)'':-0.0124962,('PDT000668935.1':0.0496044,('PDT000136039.2':0.0397031,('PDT000061129.3':0.0262222,(('PDT000168482.1':0.0132523,('PDT000123261.1':0.0115129,'PDT000123275.1':0.0148133)'':0.00220906)'':0.00376091,('PDT000638583.1':0.0214837,('PDT000641264.1':0.00743977,'PDT000016396.2':0.00954663)'':0.00813607)'':0.00037862)'':0.0071438)'':0.00921588)'':0.0179315)'':0.0246615)'':0.00824482,(('PDT000687072.1':0.0824188,(('PDT000131901.2':0.0391501,'PDT000710989.1':0.0469387)'':0.014728,(('PDT000148510.2':0.0293845,'PDT000668477.2':0.0242678)'':0.0179904,('PDT000713852.1':0.0452754,('PDT000346499.1':0.0275215,('PDT000625379.1':0.020497,'PDT000291254.1':0.0217301)'':0.0103744)'':0.00349038)'':0.0115363)'':0.010855)'':0.0098176)'':0.00985732,('PDT000625424.1':0.121348,('PDT000572836.1':0.104649,('PDT000251731.1':0.122056,('PDT000249749.1':0.100327,((('PDT000103583.2':0.0556451,'PDT000120934.2':0.0543269)'':0.00942534,('PDT000553140.1':0.0702679,('PDT000104576.2':0.0596709,('PDT000405298.1':0.0306194,'PDT000687848.1':0.0281069)'':0.0222785)'':0.00855136)'':0.00188088)'':0.00518298,(('PDT000155540.2':0.0696972,'PDT000470108.1':0.0708038)'':0.00013195,('PDT000538593.1':0.0695928,(('PDT000251495.1':0.0734295,(('PDT000161662.2':0.055072,'PDT000293439.1':0.068228)'':0.00435809,(('PDT000426380.1':0.0570624,('PDT000332107.1':0.00870237,'PDT000600082.1':0.0111163)'':0.0441488)'':0.0103268,(('PDT000520771.1':0.0436317,('PDT000615193.1':0.0175447,'PDT000619946.1':0.0203959)'':0.0254562)'':0.0102796,('PDT000161664.2':0.0388265,('PDT000230076.2':0.0449184,('PDT000253430.2':0.0286637,'PDT000083591.1':0.0303027)'':0.0123664)'':0.00111501)'':0.00758714)'':0.00544797)'':0.00414447)'':0.00379796)'':-0.000751703,(('PDT000042576.1':0.0188039,'PDT000016386.2':0.0213931)'':0.0531769,(('PDT000083587.1':0.0539221,(('PDT000293429.1':0.00774615,'PDT000293435.1':0.00992915)'':0.0361189,('PDT000103602.2':0.00447713,('PDT000322764.1':0.00421198,'PDT000625508.1':0.00415539)'':0.00101411)'':0.0314646)'':0.0128289)'':0.00253544,(('PDT000422532.1':0.0476557,('PDT000502114.1':0.0292952,('PDT000129835.2':0.013049,'PDT000359706.1':0.012112)'':0.0129592)'':0.0147096)'':5.89244e-05,(('PDT000161654.2':0.0211468,'PDT000132144.2':0.0237628)'':0.0127748,('PDT000252632.1':0.03217,('PDT000625478.1':0.0138721,'PDT000678623.1':0.0160953)'':0.0166593)'':0.00277444)'':0.00727445)'':0.0112933)'':0.0172746)'':-0.0111114)'':0.00295743)'':0.00091328)'':-0.000805438)'':0.0220915)'':0.0196741)'':0.00643069)'':0.004117)'':-0.0430123)'':0.00108419)'':0.011317)'':0.00404956)'':0.0175079)'':0.0236857)'':0.0464659)'':0.00503386,(((('PDT000293541.1':0.13822,('PDT000687490.1':0.0514909,('PDT000249972.1':0.0441598,('PDT000714112.1':0.033093,('PDT000497391.1':0.0152433,'PDT000249971.1':0.0141714)'':0.0149611)'':0.0119067)'':0.00877419)'':0.0709947)'':0.0632263,('PDT000228241.1':0.183447,(('PDT000293516.1':0.123192,('PDT000395404.1':0.0503856,'PDT000395108.1':0.0635014)'':0.0600167)'':0.0139511,((('PDT000293461.1':0.0370173,'PDT000293532.1':0.0329749)'':0.0619599,('PDT000293482.1':0.0317512,('PDT000293505.1':0.0143954,('PDT000293455.1':0.010757,'PDT000293487.1':0.0138755)'':0.0030826)'':0.0141425)'':0.0676971)'':0.014947,((('PDT000077080.2':0.0471784,'PDT000395444.1':0.0500863)'':0.0130008,('PDT000505386.1':0.0474185,'PDT000645105.1':0.0589855)'':0.0125059)'':0.03549,(('PDT000274081.2':0.0940919,('PDT000556971.1':0.0417425,'PDT000251429.1':0.0380821)'':0.0504383)'':-0.00669623,((('PDT000566133.1':0.0471053,'PDT000394836.1':0.041062)'':0.0322459,(('PDT000497354.1':0.0293561,'PDT000143979.3':0.0319101)'':0.0220659,('PDT000051188.1':0.0605185,('PDT000652756.1':0.0448418,('PDT000047181.2':0.0248746,'PDT000421822.1':0.0254355)'':0.0141074)'':0.0118383)'':0.000729729)'':0.00797535)'':0.00242485,(('PDT000321779.1':0.0569482,'PDT000140487.3':0.0647328)'':0.0142158,(('PDT000168322.3':0.0484642,'PDT000228165.1':0.0519458)'':0.0210516,('PDT000251500.1':0.0704954,(('PDT000017922.2':0.0388582,'PDT000424760.1':0.0378207)'':0.00834739,('PDT000222759.3':0.0529566,('PDT000250213.1':0.0233736,('PDT000673495.1':0.020754,'PDT000252720.1':0.0179397)'':0.00558837)'':0.0191018)'':0.00228261)'':0.0191748)'':0.000750182)'':0.00269793)'':-0.00292122)'':0.0121975)'':0.010216)'':0.00174916)'':0.036069)'':0.0545608)'':0.00464916)'':-0.00558301,((('PDT000610885.1':0.077651,(('PDT000637238.2':0.0636547,'PDT000252550.1':0.0678973)'':0.00211663,('PDT000430737.1':0.0594651,'PDT000017902.2':0.0620719)'':0.0134999)'':0.0116067)'':0.0403642,(('PDT000006902.2':0.0756411,('PDT000306671.2':0.0515776,'PDT000141573.2':0.0625374)'':0.0148499)'':0.027097,(('PDT000497319.1':0.0282131,'PDT000550326.1':0.028029)'':0.0493515,(('PDT000231465.2':0.0419663,('PDT000395263.1':0.040847,('PDT000034469.2':0.0312456,'PDT000395296.1':0.0294641)'':0.00978196)'':0.00490268)'':0.0241572,(('PDT000008473.2':0.0398735,('PDT000427503.1':0.017552,'PDT000713357.1':0.0217615)'':0.0138856)'':0.0144535,('PDT000158852.2':0.0499104,('PDT000395361.1':0.046692,('PDT000713269.1':0.0499479,('PDT000202015.2':0.0337398,('PDT000228323.1':0.0213622,('PDT000250651.2':0.0136032,'PDT000011818.2':0.0119895)'':0.0071766)'':0.0108439)'':0.0090769)'':0.00312761)'':0.00361032)'':0.00478522)'':0.00556281)'':0.00896239)'':0.025488)'':0.0282796)'':0.0522199,(('PDT000228194.1':0.133291,('PDT000050579.1':0.131231,('PDT000327941.1':0.010971,'PDT000326077.2':0.0143026)'':0.0943108)'':0.0195789)'':0.0428503,(('PDT000353666.1':0.0176462,'PDT000578969.1':0.0168537)'':0.0971285,((('PDT000360419.2':0.0348199,'PDT000387727.2':0.0382236)'':0.0515807,(('PDT000719134.1':0.0516994,'PDT000251506.1':0.0470613)'':0.0325411,('PDT000499487.1':0.0603122,('PDT000405225.1':0.050162,'PDT000717299.1':0.053805)'':0.0113288)'':0.0147595)'':0.0119525)'':-0.00148075,(('PDT000128664.3':0.114044,('PDT000687891.1':0.0759236,('PDT000160210.3':0.0579124,('PDT000161870.3':0.0601386,'PDT000637611.2':0.0545914)'':0.00177559)'':0.0229836)'':0.023175)'':-0.00168208,(('PDT000042626.1':0.0412387,('PDT000051399.2':0.0231472,'PDT000502744.1':0.0234327)'':0.0153202)'':0.0207506,(('PDT000293535.1':0.0635155,('PDT000394921.1':0.0694372,((('PDT000545605.1':0.0378489,'PDT000566141.1':0.0398838)'':0.00303173,('PDT000645067.1':0.0376682,'PDT000714096.1':0.0379295)'':0.00364082)'':0.00457624,('PDT000014163.2':0.0513667,('PDT000362524.1':0.0530322,('PDT000502285.1':0.0343005,('PDT000352376.1':0.0268399,(('PDT000430782.1':0.00547022,'PDT000023240.1':0.00611238)'':0.0125638,('PDT000140922.1':0.0133326,'PDT000024925.1':0.00941048)'':0.00880937)'':0.00734499)'':0.0108532)'':0.00659195)'':0.00366874)'':-0.0047171)'':0.0109975)'':0.00608164)'':-0.00442532,(('PDT000485958.1':0.0465222,('PDT000557113.1':0.0357525,('PDT000101127.2':0.0224376,'PDT000396612.1':0.0187944)'':0.0104383)'':0.00717689)'':0.0110399,(('PDT000107244.2':0.0492437,'PDT000397988.1':0.0512403)'':0.024349,('PDT000501871.1':0.0460251,('PDT000622532.1':0.0334221,(('PDT000384914.1':0.0290882,'PDT000362507.1':0.0331626)'':0.00305889,('PDT000713210.1':0.0385338,('PDT000233096.1':0.0389981,('PDT000395485.1':0.026234,('PDT000042589.1':0.0210986,('PDT000042463.1':0.0096421,'PDT000210716.1':0.0098725)'':0.00973385)'':0.00379253)'':0.00504509)'':0.00267505)'':-0.000543812)'':0.000565605)'':0.00475944)'':0.0187415)'':-0.00496573)'':0.00137984)'':0.00572792)'':0.0197708)'':-0.000975901)'':0.0347824)'':0.0479046)'':0.00363435)'':0.004773)'':0.00213358,(((('PDT000535243.1':0.0775307,'PDT000251570.1':0.0861573)'':0.0305861,('PDT000682356.1':0.115633,((('PDT000293481.1':0.0386005,'PDT000293533.1':0.0359355)'':0.0112531,('PDT000293551.1':0.0484605,('PDT000293465.1':0.043523,'PDT000293484.1':0.0376212)'':0.00516445)'':0.00112078)'':0.0523662,(('PDT000477483.1':0.0694553,'PDT000662173.1':0.0837017)'':0.0165574,(('PDT000042515.1':0.0600649,'PDT000668939.1':0.0563411)'':0.0137623,('PDT000137867.2':0.0589135,('PDT000675269.1':0.0550966,('PDT000377996.1':0.0242545,'PDT000017775.2':0.0193826)'':0.0273945)'':0.0118443)'':0.0114915)'':0.0155842)'':0.00228597)'':0.0210432)'':-0.00768526)'':0.0755661,(('PDT000132194.2':0.145098,('PDT000293453.1':0.0392746,'PDT000293471.1':0.0394952)'':0.126565)'':0.0322597,(('PDT000293456.1':0.176734,('PDT000304212.1':0.146454,('PDT000721345.1':0.125083,('PDT000140534.2':0.0859713,'PDT000517649.1':0.0840247)'':0.0396765)'':0.0257945)'':0.0165545)'':0.00175632,((('PDT000668483.2':0.0271831,'PDT000682557.1':0.033411)'':0.160757,('PDT000293494.1':0.129383,('PDT000637161.1':0.0316679,('PDT000415611.1':0.0113263,('PDT000147978.3':0.0128035,'PDT000556591.2':0.0125767)'':0.00127674)'':0.0186888)'':0.0827823)'':0.0543062)'':0.00335417,(('PDT000716935.1':0.0780331,('PDT000136147.2':0.0556328,('PDT000502075.1':0.047615,'PDT000661554.1':0.0491808)'':0.0105748)'':0.0252769)'':0.0837098,(('PDT000557250.1':0.13135,('PDT000015295.2':0.127411,(('PDT000251428.1':0.10627,('PDT000157308.1':0.0550341,('PDT000318573.1':0.0378378,'PDT000683167.1':0.0315207)'':0.0164655)'':0.0522182)'':0.0158657,(('PDT000619782.1':0.0821822,'PDT000251425.1':0.0815068)'':0.00714689,('PDT000566216.1':0.0466181,('PDT000395203.1':0.0266362,('PDT000397684.1':0.0138181,'PDT000397828.1':0.0139321)'':0.0114232)'':0.0147631)'':0.0525744)'':0.03151)'':0.00333783)'':0.00574441)'':0.00755284,((('PDT000619557.1':0.0218466,'PDT000619558.1':0.015545)'':0.0601979,('PDT000397687.1':0.0405865,('PDT000550334.1':0.0212613,('PDT000394862.1':0.0158073,'PDT000654618.1':0.0138113)'':0.00291572)'':0.022967)'':0.0303549)'':0.0229055,(('PDT000252708.1':0.0390296,('PDT000683615.1':0.023226,'PDT000683618.1':0.0252132)'':0.0210244)'':0.0367975,(('PDT000488268.1':0.0905253,(('PDT000641734.1':0.0551155,'PDT000293451.1':0.0652915)'':0.00711866,('PDT000251861.1':0.0534312,('PDT000641733.1':0.0472228,'PDT000721988.1':0.0398014)'':0.0119906)'':0.0187857)'':0.0180243)'':-0.0118916,(('PDT000641721.1':0.0368168,'PDT000720481.1':0.0424259)'':0.0324861,('PDT000205423.1':0.0669646,(('PDT000405235.1':0.0526685,'PDT000293454.1':0.0555015)'':0.00186702,('PDT000251430.1':0.0340208,('PDT000414248.1':0.0217357,'PDT000425992.1':0.0241431)'':0.01716)'':0.0184415)'':0.00654623)'':0.00432575)'':0.0104297)'':0.00703118)'':0.00453759)'':0.0518145)'':0.0310597)'':0.00742993)'':0.00169263)'':0.00545694)'':-0.0030765)'':0.00122771,(((('PDT000293450.1':0.0287739,'PDT000293548.1':0.0357549)'':0.105601,(('PDT000040449.3':0.050512,'PDT000293534.1':0.058679)'':0.0427931,('PDT000636815.1':0.0491072,('PDT000522649.1':0.0274655,'PDT000252624.1':0.0275213)'':0.018653)'':0.0475519)'':0.032882)'':0.00971536,(('PDT000579453.1':0.127614,('PDT000683613.1':0.0566546,'PDT000395418.1':0.0618094)'':0.0699938)'':-0.000580304,(('PDT000415812.1':0.0420666,('PDT000309386.2':0.0217671,'PDT000631629.1':0.0210142)'':0.0369478)'':0.0204695,('PDT000644885.1':0.043819,('PDT000285380.2':0.0208303,('PDT000285368.2':0.0101225,'PDT000285374.2':0.00948731)'':0.0122483)'':0.0260002)'':0.0270501)'':0.0615896)'':0.0109092)'':0.0123339,(((('PDT000719201.1':0.0684709,'PDT000352550.1':0.0763041)'':0.00972709,('PDT000205443.1':0.0644717,('PDT000293512.1':0.0673156,('PDT000716909.1':0.0639566,('PDT000691095.1':0.0470716,('PDT000719125.1':0.0493478,('PDT000354922.1':0.0448001,('PDT000415754.1':0.0236564,'PDT000430827.1':0.0293099)'':0.00706888)'':0.00838511)'':0.0039259)'':0.00505548)'':0.00477878)'':0.00270521)'':0.012398)'':0.0289439,(('PDT000641732.1':0.0688379,('PDT000502435.1':0.0611431,('PDT000687417.1':0.0319582,'PDT000477882.1':0.0318613)'':0.0210732)'':0.00688683)'':0.0315391,((('PDT000426310.1':0.0345776,'PDT000588028.1':0.0303142)'':0.033423,('PDT000661468.1':0.0514769,'PDT000097045.1':0.0581691)'':0.0115564)'':0.0298942,('PDT000252456.1':0.0813639,(('PDT000418050.1':0.0285135,'PDT000713474.1':0.0271556)'':0.00228701,('PDT000427064.1':0.0368213,(('PDT000619669.1':0.0224153,'PDT000619970.1':0.0219855)'':0.00495182,('PDT000574930.1':0.0153944,('PDT000435637.1':0.0149222,('PDT000136170.2':0.0118818,'PDT000354155.1':0.009175)'':0.00130501)'':0.0053667)'':0.00565083)'':0.00626641)'':-0.00253018)'':0.0559514)'':0.0090897)'':0.00467421)'':0.00674672)'':0.00539231,((('PDT000485255.1':0.0948194,'PDT000637698.1':0.108361)'':0.0164988,('PDT000713285.1':0.0923005,('PDT000484122.1':0.0647771,'PDT000566028.1':0.0605759)'':0.031998)'':0.0152182)'':0.0054146,((('PDT000252722.1':0.0574432,('PDT000427500.1':0.04321,'PDT000250027.1':0.0506157)'':0.0101529)'':0.0438945,((('PDT000713484.1':0.0408376,'PDT000661770.1':0.0428802)'':0.0280187,('PDT000604385.1':0.0374138,('PDT000302787.2':0.0273588,'PDT000252567.1':0.0299804)'':0.00948298)'':0.0367956)'':0.0174286,(('PDT000293543.1':0.0311464,('PDT000293478.1':0.0133106,('PDT000293463.1':0.0057587,'PDT000293508.1':0.0064012)'':0.00722879)'':0.013432)'':0.0443293,('PDT000687843.1':0.0864834,('PDT000683170.1':0.0331581,('PDT000544775.1':0.027739,('PDT000538918.2':0.0205226,('PDT000250631.2':0.0141626,'PDT000469354.1':0.0180637)'':0.00334669)'':0.00587412)'':0.00406469)'':0.0437749)'':-0.00638757)'':0.0203533)'':0.0103929)'':0.00554196,((('PDT000395105.1':0.016479,'PDT000622372.1':0.0149408)'':0.0618045,('PDT000158710.3':0.0655139,('PDT000060231.2':0.0445817,(('PDT000639593.1':0.0214623,'PDT000653457.1':0.0213263)'':0.0137746,('PDT000645040.1':0.0282569,'PDT000645104.1':0.0223434)'':0.00693408)'':0.0113627)'':0.0198732)'':-0.00275188)'':0.0345267,('PDT000610890.1':0.111541,('PDT000249779.1':0.0942027,((('PDT000296183.2':0.0267858,'PDT000639532.1':0.0221267)'':0.0086956,('PDT000549128.1':0.0127946,'PDT000042663.1':0.0182132)'':0.016719)'':0.0368516,('PDT000293519.1':0.0709872,(('PDT000619784.1':0.0240923,'PDT000362499.1':0.0245297)'':0.0227427,('PDT000556904.1':0.0354099,('PDT000306708.2':0.00984533,('PDT000645111.1':0.00400789,'PDT000687918.1':0.00477718)'':0.00529234)'':0.0229803)'':0.00732051)'':0.0148104)'':0.00913326)'':0.0109347)'':0.0208701)'':-0.00796719)'':0.00723251)'':0.0142875)'':-0.00959859)'':0.0451583)'':0.0276716)'':0.00111754)'':0.0115105)'':-9.62852e-05,((((('PDT000343836.1':0.0334233,'PDT000132355.2':0.0303256)'':0.170391,(('PDT000132230.2':0.0451073,'PDT000213203.2':0.0463273)'':0.0858921,(('PDT000156925.2':0.0389318,'PDT000575657.1':0.0365873)'':0.0703771,('PDT000343890.1':0.101935,'PDT000378642.1':0.108249)'':0.00285007)'':0.0192767)'':0.06765)'':-0.00436864,(('PDT000365290.1':0.107183,('PDT000042446.1':0.0649604,'PDT000276849.1':0.0744246)'':0.0474428)'':0.0260065,(('PDT000328046.1':0.0793335,('PDT000535535.1':0.0247962,'PDT000362503.1':0.0249108)'':0.060756)'':0.020445,('PDT000097373.2':0.0976487,('PDT000132152.2':0.0841989,((('PDT000005197.2':0.0436027,('PDT000320908.1':0.0266899,'PDT000480398.1':0.031294)'':0.0238268)'':0.00345002,('PDT000357579.1':0.0512179,('PDT000718447.1':0.0438322,('PDT000714157.1':0.0259187,'PDT000251975.1':0.0258256)'':0.0154704)'':0.00560725)'':0.00279322)'':0.00856437,(('PDT000047742.3':0.0160268,'PDT000357658.1':0.026296)'':0.0448631,((('PDT000535420.1':0.0401033,'PDT000373317.1':0.040277)'':0.00677538,('PDT000137280.2':0.0469514,('PDT000502087.1':0.0389509,'PDT000015308.2':0.0433294)'':0.00219747)'':0.00369567)'':0.00169344,(('PDT000371947.1':0.0348228,'PDT000092534.1':0.0315729)'':0.00329866,('PDT000025982.1':0.00405828,('PDT000032211.1':0.00285652,'PDT000025976.1':0.00352434)'':0.000430856)'':0.0364214)'':0.00603901)'':0.0141306)'':-0.00168321)'':0.0218258)'':0.0128775)'':0.0114904)'':0.0514179)'':0.0283328)'':0.00121435,((('PDT000688004.1':0.08411,('PDT000168449.1':0.0416695,'PDT000688001.1':0.0673855)'':0.0425965)'':0.0124979,(('PDT000287876.2':0.0641233,('PDT000443037.1':0.0271814,('PDT000438177.1':0.0138234,'PDT000340393.1':0.0124794)'':0.0139804)'':0.0355294)'':0.0185082,('PDT000702551.1':0.0863741,('PDT000720391.1':0.0636433,(('PDT000548627.2':0.0140959,'PDT000132357.3':0.0149354)'':0.0246349,('PDT000251564.1':0.0323098,('PDT000161410.2':0.0138601,'PDT000438215.1':0.0169061)'':0.0192469)'':0.00274266)'':0.0241431)'':0.0230143)'':-0.00387477)'':0.0174835)'':0.00297213,((('PDT000386064.2':0.0159983,('PDT000239337.3':0.0140558,'PDT000132390.3':0.0162833)'':0.00293525)'':0.0105384,(('PDT000548563.2':0.0196099,('PDT000324029.2':0.0185531,'PDT000470330.2':0.017249)'':0.00401149)'':0.00226371,('PDT000050585.1':0.0315876,('PDT000629890.2':0.031809,('PDT000155823.3':0.0213252,('PDT000107671.2':0.018392,('PDT000322759.2':0.0178561,(('PDT000325130.2':0.0145945,'PDT000485315.2':0.0108402)'':0.00170002,('PDT000548304.2':0.0105806,(('PDT000431651.2':0.00877548,'PDT000139196.3':0.00745472)'':0.000624438,('PDT000466936.2':0.00655573,('PDT000160213.3':0.00599596,'PDT000324643.2':0.00654634)'':0.00102902)'':0.00131732)'':0.00335423)'':0.000844528)'':0.00311081)'':0.00218134)'':0.00221943)'':0.0047615)'':0.00307469)'':-0.00248692)'':0.00139465)'':0.00961461,(('PDT000538221.2':0.029519,('PDT000581127.2':0.0153572,(('PDT000649133.2':0.0116847,'PDT000038871.1':0.0121556)'':0.00241228,('PDT000328034.2':0.00632283,('PDT000131877.3':0.00625731,('PDT000487875.2':0.00520819,'PDT000660056.2':0.00564081)'':0.00138704)'':0.000124438)'':0.00727276)'':0.00302391)'':0.00431165)'':0.0101968,((('PDT000562133.2':0.023672,('PDT000212461.3':0.0120447,'PDT000327262.2':0.0123914)'':0.00748091)'':0.00119733,('PDT000583565.2':0.012813,('PDT000239338.3':0.0106394,'PDT000325794.2':0.00868523)'':0.00335623)'':0.00683251)'':0.00347628,((('PDT000342305.2':0.0186207,'PDT000621345.2':0.0173534)'':0.00569517,('PDT000667893.2':0.0238889,('PDT000314187.3':0.0119542,('PDT000123309.1':0.00880887,('PDT000378043.2':0.00809763,'PDT000060878.3':0.00778487)'':0.000366034)'':0.00468601)'':0.00938438)'':0.00207484)'':-0.000305728,(('PDT000197048.3':0.0257511,('PDT000190758.3':0.0209475,('PDT000272387.3':0.0182863,'PDT000313053.3':0.0174806)'':0.00382457)'':0.0077648)'':0.00661219,(('PDT000487472.2':0.0311566,('PDT000314222.3':0.0160516,((('PDT000377075.2':0.00707746,'PDT000415214.2':0.00596774)'':0.00719117,('PDT000155733.3':0.0165913,('PDT000341776.2':0.0116244,('PDT000603074.2':0.00807362,('PDT000548547.2':0.00403307,('PDT000115423.3':0.00208125,'PDT000132414.3':0.00271204)'':0.00275099)'':0.00186057)'':0.00246227)'':0.00539241)'':-0.00185148)'':0.00141402,(('PDT000038908.1':0.0148645,('PDT000272394.3':0.0111914,('PDT000390808.2':0.00885736,('PDT000503001.2':0.00340187,('PDT000272397.3':0.00209822,('PDT000160239.3':0.00202173,'PDT000272391.3':0.00187243)'':0.000200225)'':0.00164456)'':0.0046363)'':0.00148034)'':0.00104512)'':0.00114378,(('PDT000377024.2':0.0109464,'PDT000325088.2':0.0103344)'':0.000565733,('PDT000160208.3':0.00796449,(('PDT000107658.2':0.00736865,('PDT000156909.3':0.00492567,('PDT000469845.2':0.00320988,'PDT000492090.2':0.0038809)'':0.00163926)'':0.00243939)'':0.000817483,('PDT000629896.2':0.0114346,('PDT000317140.2':0.00746771,('PDT000501008.2':0.00446272,('PDT000333962.2':0.00395781,'PDT000548303.2':0.00411233)'':0.000711097)'':0.00205517)'':0.0018843)'':-0.00107261)'':0.00153374)'':0.000109614)'':0.00287817)'':0.00179866)'':0.00284719)'':0.013958)'':-0.0134846,(('PDT000259763.3':0.0174949,('PDT000484022.2':0.0143889,('PDT000365924.2':0.0103319,'PDT000548540.2':0.0097038)'':0.00402026)'':0.0017265)'':0.000916031,((('PDT000389764.2':0.0121084,('PDT000503531.2':0.00684462,'PDT000102807.3':0.00719778)'':0.00268417)'':0.00380702,('PDT000272395.3':0.0209127,(('PDT000520315.2':0.00657341,'PDT000677277.1':0.00797709)'':0.00558378,(('PDT000625593.2':0.0105262,'PDT000114655.3':0.00847497)'':0.0020575,(('PDT000272389.3':0.0103265,('PDT000587199.2':0.00818512,('PDT000368381.2':0.00712622,'PDT000679756.1':0.00671978)'':0.00123213)'':0.00208889)'':-0.000766878,(('PDT000161920.3':0.00618008,'PDT000398520.2':0.00705772)'':0.00144142,('PDT000107666.2':0.00989119,('PDT000548608.2':0.00453087,('PDT000239335.3':0.00403718,'PDT000548234.2':0.00453795)'':0.000548899)'':0.00363141)'':-0.000219665)'':0.00112761)'':0.00228918)'':0.00101134)'':0.00571144)'':-0.00464876)'':0.000813822,(('PDT000480979.2':0.0208706,'PDT000038864.1':0.0227786)'':0.00200038,((('PDT000189640.3':0.0137147,'PDT000213946.3':0.0118295)'':0.00253928,('PDT000250859.3':0.0135023,'PDT000672766.1':0.0136662)'':0.00105399)'':0.000890856,(('PDT000239329.3':0.0128886,('PDT000711008.1':0.00689691,'PDT000721003.1':0.00732909)'':0.00462475)'':0.000791707,('PDT000351902.2':0.0120421,('PDT000272392.3':0.0119396,(('PDT000505041.2':0.00898891,('PDT000671534.1':0.00790081,('PDT000192806.3':0.00534665,'PDT000706319.1':0.00630205)'':0.00162219)'':0.00146066)'':0.000713138,('PDT000322962.1':0.00995556,('PDT000155369.3':0.00688445,('PDT000160277.3':0.00607652,('PDT000316981.2':0.0039222,'PDT000085628.3':0.00415196)'':0.00266785)'':0.000782021)'':0.00251757)'':-0.00031786)'':0.00228107)'':0.00222755)'':-0.00159868)'':0.00225848)'':0.00552765)'':-0.00532173)'':0.00317675)'':-0.000746507)'':0.0113676)'':-0.00377574)'':0.00241477)'':0.00399493)'':0.00784994)'':0.0685311)'':0.082433)'':0.0236069,((((('PDT000409803.1':0.0715401,('PDT000443775.1':0.0555544,('PDT000019971.2':0.041979,('PDT000713845.1':0.0253289,('PDT000281734.4':0.0265109,'PDT000443223.1':0.0264337)'':0.00357417)'':0.0116524)'':0.00489372)'':0.0170152)'':0.00865098,('PDT000250229.1':0.0818776,('PDT000136129.2':0.0597092,(('PDT000550085.1':0.053115,'PDT000104794.2':0.04954)'':0.00337728,('PDT000502048.1':0.0401778,('PDT000502006.1':0.0456131,('PDT000675569.1':0.0373777,('PDT000661328.1':0.0268172,('PDT000325688.1':0.0158997,'PDT000713404.1':0.0206675)'':0.00998319)'':0.00478788)'':0.00157404)'':0.00262459)'':0.00654338)'':0.0128573)'':0.00847985)'':0.00709858)'':0.0242226,((('PDT000520862.1':0.0527745,('PDT000713970.1':0.0371156,'PDT000640375.1':0.0325843)'':0.0147106)'':-0.000163196,(('PDT000444167.1':0.0218238,'PDT000451102.1':0.0186358)'':0.0123158,('PDT000433649.1':0.0386023,('PDT000157487.2':0.0238921,'PDT000694397.1':0.0210627)'':0.00884106)'':0.000302355)'':0.0183004)'':0.00151376,(('PDT000610892.1':0.0239792,'PDT000478191.1':0.0242337)'':0.0208449,(('PDT000448176.1':0.0398087,('PDT000425645.1':0.0289289,('PDT000426049.1':0.0228581,('PDT000136067.2':0.0196392,'PDT000636192.1':0.0199657)'':0.00562585)'':0.0062775)'':0.00963202)'':0.00114938,('PDT000111118.1':0.0438957,('PDT000459484.1':0.0376443,('PDT000042672.1':0.0358231,(('PDT000427577.1':0.0285569,('PDT000636338.1':0.0244406,('PDT000443166.1':0.0115695,'PDT000713566.1':0.0131903)'':0.0089366)'':0.00768218)'':-0.000718196,(('PDT000716962.1':0.0227454,('PDT000537959.1':0.0114165,'PDT000485953.1':0.0112927)'':0.0103487)'':0.00352607,('PDT000652379.1':0.023275,('PDT000485951.1':0.0202345,('PDT000186891.3':0.0200671,('PDT000438030.1':0.0114926,'PDT000409997.1':0.0142862)'':0.00509657)'':0.0035762)'':0.00208143)'':0.0021123)'':0.0018022)'':0.00822335)'':0.00440915)'':0.00425236)'':-0.00738593)'':0.00667405)'':0.00855927)'':0.0467807)'':0.0719229,(((('PDT000713515.1':0.0705265,'PDT000543333.1':0.0666125)'':0.0461788,(('PDT000251576.1':0.0750621,('PDT000042584.1':0.0482786,'PDT000435641.1':0.0587574)'':0.0116564)'':0.0239964,('PDT000661507.1':0.0872569,(('PDT000718998.1':0.0863617,('PDT000132560.1':0.013854,('PDT000619646.1':0.0107338,'PDT000718451.1':0.0109386)'':0.00211541)'':0.0600969)'':0.00346999,('PDT000252573.1':0.0682076,('PDT000042497.1':0.0395719,('PDT000230493.1':0.035785,('PDT000252678.1':0.0184935,'PDT000279196.1':0.0184232)'':0.0126412)'':0.00738331)'':0.0292981)'':0.00352783)'':0.0057586)'':0.0112602)'':0.0262178)'':-0.0188686,(('PDT000713548.1':0.0252739,'PDT000661486.1':0.0216814)'':0.0272119,(('PDT000427742.1':0.0370857,'PDT000713969.1':0.0349846)'':0.00314803,((('PDT000017673.2':0.0157253,('PDT000713687.1':0.00476261,'PDT000251713.1':0.00501978)'':0.00720999)'':0.0164089,(('PDT000062951.2':0.0119001,'PDT000357695.1':0.0197723)'':0.0109062,('PDT000694537.1':0.0134054,'PDT000017831.2':0.0146523)'':0.00457279)'':0.00887841)'':0.00301966,(('PDT000325139.1':0.0203691,'PDT000270463.1':0.0225021)'':0.00798497,(('PDT000679870.1':0.0139677,'PDT000210711.1':0.0113149)'':0.0108163,('PDT000426547.1':0.0112492,('PDT000382804.1':0.00371123,'PDT000661561.1':0.00416986)'':0.00664965)'':0.013906)'':0.0021945)'':0.00425914)'':0.00543891)'':0.0143571)'':0.0497803)'':0.0303165,((('PDT000661305.1':0.0567007,(('PDT000535476.1':0.0290361,'PDT000632561.1':0.0322182)'':0.0073265,('PDT000502604.1':0.0476732,(('PDT000427497.1':0.0202432,'PDT000520964.1':0.0200042)'':0.00589608,('PDT000233332.1':0.0300834,(('PDT000347020.1':0.0208091,('PDT000502009.1':0.0147902,('PDT000136026.2':0.0082719,'PDT000346704.1':0.0065865)'':0.00491188)'':0.00675136)'':0.00106202,(('PDT000346957.1':0.0151085,('PDT000175975.2':0.0139941,'PDT000132049.2':0.0104132)'':0.00368649)'':0.00254154,(('PDT000460008.1':0.012826,('PDT000619089.1':0.00351956,'PDT000661765.1':0.00355568)'':0.0100512)'':0.00340614,(('PDT000501922.1':0.00947982,'PDT000661511.1':0.0130526)'':0.00126444,('PDT000188333.1':0.0104385,'PDT000415792.1':0.00849896)'':0.00315496)'':0.00477832)'':0.0034725)'':0.000225917)'':0.00372439)'':0.000500895)'':0.0132717)'':-0.00527512)'':0.0111621)'':-0.00738339,((('PDT000202699.2':0.0427128,('PDT000485971.1':0.0268801,('PDT000536638.1':0.0195609,('PDT000535096.1':0.0109612,'PDT000636152.1':0.0139095)'':0.00773967)'':0.00505593)'':0.00853627)'':-0.00225539,(('PDT000433697.1':0.0240709,'PDT000542614.1':0.0220799)'':0.00807882,('PDT000189670.3':0.0279969,('PDT000433426.1':0.0179548,'PDT000407981.1':0.0189543)'':0.0150414)'':1.50226e-05)'':0.00320947)'':0.0042383,('PDT000661583.1':0.0922608,('PDT000523266.1':0.055298,((('PDT000517625.1':0.017616,('PDT000618740.1':0.0154761,'PDT000359257.2':0.018843)'':0.00540173)'':0.0176603,(('PDT000433677.1':0.0323423,('PDT000433436.1':0.0234626,('PDT000417163.1':0.0186241,('PDT000521123.1':0.0103521,'PDT000226325.1':0.0109667)'':0.00588326)'':0.0110264)'':0.00237817)'':-0.00103693,('PDT000132176.2':0.0321619,('PDT000445077.1':0.0324055,(('PDT000425868.1':0.0264653,'PDT000633759.1':0.0259147)'':0.00195594,(('PDT000155832.2':0.0157707,'PDT000616125.1':0.0149181)'':0.00786537,(('PDT000546927.1':0.0163521,('PDT000618719.1':0.0137971,'PDT000716917.1':0.0128854)'':0.00508453)'':0.00161314,('PDT000362522.1':0.0342041,('PDT000673419.1':0.0200723,('PDT000636371.1':0.0182456,(('PDT000720296.1':0.0111705,('PDT000397898.1':0.0118265,('PDT000397985.1':0.00890247,'PDT000352412.1':0.00899193)'':0.00136235)'':0.000668921)'':0.000868315,(('PDT000202750.2':0.00584184,'PDT000347023.1':0.00650996)'':0.00418199,('PDT000536639.1':0.0103858,('PDT000124857.2':0.00862536,('PDT000535415.1':0.00521955,'PDT000673547.1':0.00522825)'':0.00316799)'':0.00174991)'':3.07186e-05)'':0.0017913)'':0.00346934)'':0.00264082)'':0.00816206)'':-0.00649443)'':0.00321021)'':0.00171679)'':0.00480393)'':0.00267901)'':-0.00570265)'':0.0105604)'':-0.00617168,(('PDT000716934.1':0.0618421,(('PDT000538600.1':0.0251909,'PDT000632129.1':0.0242715)'':0.00422576,('PDT000347027.1':0.0225143,('PDT000535091.1':0.0233017,('PDT000618791.1':0.0119515,'PDT000346731.1':0.0137557)'':0.00768112)'':0.00188522)'':0.00373413)'':0.0169446)'':-0.0103272,(('PDT000067797.1':0.0254847,('PDT000626708.1':0.0172327,'PDT000483524.1':0.0186121)'':0.004558)'':0.00920873,(('PDT000347976.1':0.042356,(('PDT000654360.1':0.0270772,('PDT000713631.1':0.0126908,'PDT000661574.1':0.0170693)'':0.00526949)'':-0.000269668,('PDT000485925.1':0.0197741,('PDT000247184.2':0.0207986,(('PDT000496663.1':0.0131093,'PDT000535095.1':0.0123848)'':0.00217818,('PDT000710125.1':0.00925414,'PDT000346733.1':0.010959)'':0.00357284)'':0.00412369)'':0.000675495)'':0.00252247)'':0.014948)'':-0.012401,(('PDT000534891.1':0.0293496,('PDT000200507.2':0.0267097,'PDT000661127.1':0.0219427)'':0.0051442)'':0.00277816,(('PDT000391855.1':0.0315356,('PDT000347005.1':0.0263728,(('PDT000632292.1':0.0143511,('PDT000632505.1':0.0103653,'PDT000632763.1':0.00889032)'':0.00343505)'':0.00214132,('PDT000411322.1':0.0165195,('PDT000346768.1':0.0157178,('PDT000388466.1':0.0139803,('PDT000633786.1':0.0112395,('PDT000126251.2':0.00414254,('PDT000603049.1':0.00504584,'PDT000635913.1':0.00441914)'':0.000646033)'':0.00543176)'':0.00140602)'':0.000811483)'':0.00189761)'':-0.00016584)'':0.00678622)'':0.00152031)'':-0.00341196,(('PDT000433751.1':0.0190333,'PDT000030555.1':0.0188343)'':0.0109359,('PDT000640366.1':0.0359691,(('PDT000720770.1':0.0216187,'PDT000485952.1':0.0164756)'':0.00806245,('PDT000535411.1':0.0281364,(('PDT000618829.1':0.0192408,('PDT000387738.1':0.0105309,'PDT000407601.1':0.00964601)'':0.00872621)'':-0.00137682,('PDT000125956.1':0.0353193,(('PDT000616101.1':0.0141492,'PDT000457705.1':0.015834)'':0.000650507,('PDT000535418.1':0.0235056,('PDT000158856.2':0.0139606,('PDT000713476.1':0.0144401,(('PDT000042629.1':0.0110685,('PDT000143964.3':0.00999471,('PDT000640371.1':0.00559909,'PDT000415790.1':0.00535161)'':0.00216169)'':0.00139915)'':-1.52178e-05,(('PDT000618969.1':0.00728764,('PDT000673618.1':0.00451887,'PDT000694497.1':0.00425306)'':0.00309895)'':-9.86981e-05,('PDT000620985.1':0.00840641,('PDT000328043.1':0.00863766,('PDT000502056.1':0.00556891,('PDT000673425.1':0.00442534,('PDT000713658.1':0.00275018,('PDT000713314.1':0.00183236,('PDT000427011.1':0.0010821,'PDT000714028.1':0.00121583)'':0.000766759)'':0.000655903)'':0.00172245)'':0.0007857)'':0.0017806)'':0.00113059)'':-0.000671516)'':0.000502904)'':0.00364948)'':0.00445647)'':0.000521474)'':-0.00162127)'':0.00683295)'':-0.00603929)'':0.0120435)'':-0.00420071)'':0.00576619)'':-0.00327234)'':-0.00577218)'':0.00626048)'':-0.00113046)'':0.00323005)'':0.000706403)'':-0.0017461)'':0.0194434)'':0.0233435)'':-0.036349)'':0.00205138)'':0.044377,(('PDT000161000.2':0.0167099,'PDT000357715.1':0.0175414)'':0.142076,((('PDT000354206.1':0.0492676,((('PDT000521042.1':0.0160515,'PDT000521106.1':0.0208749)'':0.00988962,('PDT000426969.1':0.0358871,('PDT000520910.1':0.0276287,('PDT000713624.1':0.0250178,'PDT000652348.1':0.0235393)'':0.00139815)'':0.00627802)'':0.00221137)'':0.00439447,('PDT000160741.2':0.0444233,(('PDT000713311.1':0.0245987,'PDT000713733.1':0.0195553)'':0.0138334,('PDT000438212.1':0.0337351,(('PDT000017703.2':0.017288,'PDT000017723.2':0.0143076)'':0.0121645,('PDT000716881.1':0.0145054,('PDT000713376.1':0.0127255,('PDT000017804.2':0.00562649,'PDT000017834.2':0.00542161)'':0.00822253)'':0.00292933)'':0.0114591)'':0.00413879)'':0.00175509)'':0.0129606)'':-0.0112112)'':0.0197685)'':0.0114173,(('PDT000444095.1':0.0468222,('PDT000538457.1':0.0350781,('PDT000636179.1':0.0341914,('PDT000136014.2':0.0271119,(('PDT000501867.1':0.0149831,('PDT000444110.1':0.018211,'PDT000249801.1':0.0140725)'':0.00272379)'':0.00180011,('PDT000713992.1':0.0105417,('PDT000574936.1':0.010264,('PDT000694376.1':0.00545027,'PDT000661669.1':0.00500683)'':0.00268223)'':0.00223731)'':0.0128402)'':0.00568284)'':0.00185637)'':0.00500713)'':0.00688845)'':0.00397488,('PDT000444116.1':0.0424449,('PDT000427527.1':0.0351316,('PDT000132174.2':0.0234699,('PDT000158726.3':0.0220593,(('PDT000713832.1':0.00879828,'PDT000362515.1':0.00945752)'':0.00622761,('PDT000713929.1':0.014843,(('PDT000535258.1':0.00640503,'PDT000713727.1':0.00554477)'':0.00278439,('PDT000362439.1':0.00808073,('PDT000380166.1':0.00571733,('PDT000136046.2':0.00310044,'PDT000535046.1':0.00289117)'':0.00224112)'':0.00200871)'':0.000133665)'':0.00637203)'':-0.000913225)'':0.00628419)'':0.00442708)'':0.00514301)'':0.00838056)'':0.00486664)'':0.0213384)'':0.0141405,(((('PDT000502176.1':0.0351412,('PDT000502134.1':0.0264503,'PDT000502140.1':0.0278072)'':0.00628458)'':0.0103196,(('PDT000188185.2':0.0286844,'PDT000502003.1':0.0319783)'':0.00718581,(('PDT000188137.2':0.0176566,'PDT000501864.1':0.0228369)'':0.0110515,('PDT000501964.1':0.0335464,('PDT000347046.1':0.027361,('PDT000502183.1':0.0278138,'PDT000502186.1':0.0260391)'':0.00262157)'':0.00300331)'':0.000572368)'':0.00686721)'':0.00205589)'':0.00906858,(('PDT000502036.1':0.0393457,('PDT000188068.2':0.0294665,'PDT000125987.1':0.0335019)'':0.00675456)'':0.00705027,(('PDT000521079.1':0.0413489,(('PDT000069669.1':0.0205511,'PDT000011756.2':0.0254509)'':0.00265949,(('PDT000716223.1':0.00833546,'PDT000661746.1':0.00814914)'':0.0118214,('PDT000714115.1':0.0156024,'PDT000713672.1':0.0185381)'':0.00294609)'':0.00527936)'':0.0101549)'':-0.00143128,(('PDT000017651.2':0.0333023,('PDT000501980.1':0.0200011,(('PDT000397664.1':0.0171589,'PDT000249785.1':0.0143013)'':0.00372473,('PDT000249760.1':0.0190024,'PDT000017840.2':0.0160888)'':0.00135717)'':0.00274648)'':0.00926197)'':0.00449394,(('PDT000135928.2':0.0191951,'PDT000502112.1':0.020892)'':0.00746086,(('PDT000120864.2':0.026277,'PDT000136038.2':0.0254723)'':0.00256045,('PDT000018737.1':0.0253042,('PDT000161905.2':0.0247049,'PDT000713445.1':0.0202887)'':0.00551173)'':2.93431e-05)'':0.00461972)'':0.00467607)'':0.00290921)'':0.00328037)'':0.00933716)'':0.0164479,(((('PDT000397631.1':0.0286655,'PDT000251464.1':0.0283883)'':0.00272376,('PDT000115427.3':0.0360812,'PDT000362518.1':0.0319254)'':0.00452424)'':0.0373699,(('PDT000443920.1':0.0529787,('PDT000661671.1':0.0262243,('PDT000713278.1':0.0184168,('PDT000248501.1':0.0103127,('PDT000188936.1':0.00681148,('PDT000636332.1':0.00350324,'PDT000713947.1':0.00438414)'':0.00301578)'':0.00452584)'':0.00559086)'':0.00652862)'':0.028071)'':-0.0108097,('PDT000444148.1':0.0414563,('PDT000598715.1':0.0401555,('PDT000156001.2':0.0263123,('PDT000714075.1':0.0139411,('PDT000694472.1':0.00812955,('PDT000713596.1':0.00499762,('PDT000661800.1':0.0048248,('PDT000132382.2':0.00409973,'PDT000661401.1':0.00448739)'':0.000845474)'':0.000196389)'':0.00388703)'':0.00502725)'':0.00988385)'':0.0109292)'':0.00242958)'':0.00211064)'':0.0317579)'':-0.00247823,(('PDT000136106.2':0.0392604,'PDT000346708.1':0.0371258)'':0.0212437,('PDT000716872.1':0.0761897,(('PDT000397600.1':0.0284145,(('PDT000132126.2':0.0202363,'PDT000397928.1':0.0253393)'':0.00240246,('PDT000131861.2':0.0167694,('PDT000209173.2':0.0168569,'PDT000111119.1':0.0128932)'':0.0065232)'':0.00122356)'':0.00797932)'':-0.000518571,(('PDT000661551.1':0.0385327,('PDT000397696.1':0.0285324,'PDT000661541.1':0.0227471)'':0.00770415)'':-0.000462563,(('PDT000042524.1':0.029826,('PDT000322955.1':0.0233129,(('PDT000673426.1':0.0134475,'PDT000042449.1':0.0145751)'':0.00314369,('PDT000175899.2':0.0229033,('PDT000397931.1':0.0152327,('PDT000690196.1':0.0158034,(('PDT000673524.1':0.00710308,'PDT000017795.2':0.00690032)'':0.00132747,('PDT000713880.1':0.0038893,'PDT000714094.1':0.00437846)'':0.00287745)'':0.00609198)'':0.00271034)'':0.00209947)'':-0.000556343)'':0.00811987)'':0.00144985)'':-0.0034665,((('PDT000295840.2':0.0188409,'PDT000042734.1':0.0192384)'':0.00248447,('PDT000713523.1':0.0226524,('PDT000135975.2':0.0174258,('PDT000398011.1':0.0160781,('PDT000665177.1':0.0130662,('PDT000661559.1':0.00907903,('PDT000661384.1':0.00368213,('PDT000713237.1':0.00242223,'PDT000415805.1':0.00286825)'':0.000810803)'':0.0036455)'':0.00659971)'':0.00387643)'':0.00326845)'':0.00225242)'':0.00049314)'':0.000218605,(('PDT000042491.1':0.0317269,('PDT000672267.1':0.0267689,('PDT000683482.1':0.0210755,('PDT000636350.1':0.0114222,('PDT000397601.1':0.00980433,('PDT000017871.2':0.00320317,('PDT000385480.1':0.00185712,'PDT000714152.1':0.00173641)'':0.000890891)'':0.00534618)'':0.00100288)'':0.00716059)'':0.00807743)'':0.00215483)'':-0.00553412,(('PDT000427082.1':0.0176041,('PDT000362437.1':0.0210879,'PDT000362497.1':0.020309)'':0.00121034)'':0.00619701,(('PDT000713906.1':0.0167977,'PDT000015306.2':0.02006)'':0.00918601,(('PDT000426977.1':0.018891,'PDT000427536.1':0.0162575)'':0.000753903,(('PDT000132085.2':0.0162168,'PDT000661598.1':0.0155965)'':0.00193094,(('PDT000238189.2':0.00976794,'PDT000397748.1':0.0105835)'':0.00442074,(('PDT000155291.3':0.00878564,'PDT000673525.1':0.00617996)'':0.00403483,(('PDT000156975.2':0.00714966,'PDT000519354.1':0.00927644)'':0.00512091,('PDT000415839.1':0.0142627,(('PDT000674749.1':0.00472309,('PDT000574934.1':0.00360911,('PDT000713329.1':0.00245822,'PDT000380164.1':0.00243005)'':0.000479979)'':0.00108908)'':0.00389455,('PDT000479297.1':0.00881405,('PDT000713986.1':0.00778058,('PDT000713309.1':0.00645048,('PDT000574929.1':0.00472855,'PDT000011733.2':0.00454918)'':0.00111056)'':0.000311679)'':0.00234459)'':-5.33944e-06)'':0.00415628)'':-0.00203884)'':0.00343101)'':0.000908335)'':0.00187481)'':0.00210936)'':0.00176739)'':-0.000415353)'':-0.000360666)'':0.00366253)'':0.000936122)'':0.00729887)'':0.000181312)'':0.025587)'':0.003904)'':0.00821677)'':0.00188618)'':0.0046718)'':0.0784741)'':-0.0660758)'':0.040211)'':0.0425396)'':0.016176,((((('PDT000619767.1':0.0518352,('PDT000694364.1':0.0405932,('PDT000443870.1':0.0297851,'PDT000017809.2':0.0279633)'':0.0133876)'':0.00853196)'':0.00847417,((('PDT000713412.1':0.0457216,('PDT000017761.2':0.0212011,'PDT000017766.2':0.0214812)'':0.0228145)'':0.00567074,(('PDT000714006.1':0.0210156,'PDT000716878.1':0.0211826)'':0.0161779,('PDT000387604.2':0.0444889,('PDT000492094.2':0.0276764,'PDT000102987.2':0.0309806)'':0.0115716)'':0.00171861)'':0.00352881)'':3.8158e-05,('PDT000188075.2':0.0430393,(('PDT000702607.1':0.0493394,('PDT000566854.1':0.0360655,('PDT000157701.2':0.0263845,'PDT000661289.1':0.0282053)'':0.0102115)'':0.00647543)'':-0.00031979,((('PDT000562901.1':0.0258589,'PDT000713648.1':0.0214271)'':0.00666617,('PDT000710531.1':0.0237351,'PDT000267828.1':0.0228884)'':0.00405693)'':0.00858378,('PDT000714005.1':0.0436294,(('PDT000252652.1':0.024393,('PDT000317139.1':0.0188056,'PDT000714140.1':0.0209964)'':0.00535085)'':0.00133056,(('PDT000694536.1':0.0236225,('PDT000600794.1':0.0180457,'PDT000713437.1':0.014954)'':0.00922073)'':0.00124878,('PDT000102990.1':0.0264882,('PDT000465790.1':0.0127354,('PDT000588040.1':0.0130148,('PDT000661493.1':0.00450154,'PDT000661560.1':0.00515981)'':0.00442207)'':0.00324058)'':0.00991663)'':-0.00104732)'':0.00470951)'':0.0116356)'':-0.00390431)'':0.00534619)'':0.00457427)'':-0.00681176)'':0.0125638)'':0.0224086,((('PDT000502239.1':0.0460184,('PDT000372689.1':0.0390677,('PDT000588004.1':0.0432909,('PDT000132245.2':0.0268915,('PDT000713371.1':0.0198858,'PDT000713965.1':0.0237427)'':0.00863415)'':0.00862069)'':0.00427251)'':0.00399982)'':-0.00341902,(('PDT000017682.2':0.0498533,('PDT000131867.2':0.0292633,'PDT000713534.1':0.0352528)'':0.00685528)'':0.0033841,(('PDT000324692.1':0.0436769,'PDT000713683.1':0.0490065)'':0.00144521,(('PDT000251985.1':0.0474075,('PDT000249793.1':0.02813,('PDT000249766.1':0.0285847,'PDT000448213.1':0.0271073)'':0.00156604)'':0.0102896)'':-0.000424371,(('PDT000713308.1':0.0271514,('PDT000427688.1':0.024521,'PDT000131910.2':0.0270667)'':0.00458995)'':0.00305605,('PDT000346724.1':0.0311155,('PDT000661469.1':0.0306724,(('PDT000713829.1':0.0261256,'PDT000588000.1':0.0228488)'':0.00330584,('PDT000716029.1':0.0220584,'PDT000661706.1':0.0232228)'':0.00362676)'':0.00528185)'':0.00317145)'':0.00120816)'':0.00469681)'':0.00419525)'':0.00111893)'':-3.62023e-05)'':0.0225188,('PDT000639588.1':0.0912474,((('PDT000397870.1':0.0261887,('PDT000344809.1':0.0174661,'PDT000616946.1':0.0157716)'':0.00601519)'':0.0256938,(('PDT000111114.1':0.024997,'PDT000017748.2':0.0232452)'':0.0211207,(('PDT000316552.1':0.0330986,'PDT000713248.1':0.0282928)'':0.000718433,('PDT000610878.1':0.0239025,('PDT000249947.1':0.0148626,'PDT000017776.2':0.0108116)'':0.0139264)'':0.00480751)'':0.0208191)'':0.00640137)'':0.00373066,(('PDT000132363.2':0.0356411,('PDT000713456.1':0.026907,('PDT000482860.1':0.0225273,('PDT000714033.1':0.0101074,'PDT000346726.1':0.0108023)'':0.0102855)'':0.00534829)'':0.00698019)'':0.0210872,((('PDT000148023.2':0.0210655,'PDT000713282.1':0.0242862)'':0.00181415,('PDT000248516.1':0.0170884,'PDT000448210.1':0.0171535)'':0.0121943)'':0.0146142,(('PDT000161495.2':0.0112338,'PDT000638840.1':0.0106582)'':0.0090251,('PDT000042599.1':0.0120076,('PDT000713886.1':0.0137739,('PDT000572838.1':0.00989944,'PDT000103055.1':0.00897226)'':0.00404878)'':0.00211694)'':0.00324524)'':0.0205189)'':0.0141349)'':0.0058183)'':0.0362254)'':-0.0298124)'':0.0119599)'':0.094063,(((('PDT000368004.1':0.0227383,'PDT000042489.1':0.0278797)'':0.12002,('PDT000019564.1':0.0637756,('PDT000674895.1':0.0707779,('PDT000497217.1':0.0592055,('PDT000600659.1':0.0572039,'PDT000619676.1':0.0549371)'':0.00457653)'':0.00514928)'':0.023427)'':0.0678407)'':0.0131243,((((('PDT000557228.1':0.0404257,('PDT000502362.1':0.033315,'PDT000251907.1':0.0282885)'':0.0124243)'':0.00477678,('PDT000311248.2':0.048521,(('PDT000109486.2':0.0273145,'PDT000502343.1':0.0278606)'':0.00299689,('PDT000583461.1':0.0280731,('PDT000557168.1':0.0155649,('PDT000396832.1':0.0105019,('PDT000109420.2':0.0105519,'PDT000109452.2':0.00849669)'':0.00260252)'':0.00258985)'':0.00739262)'':0.00392662)'':0.0110498)'':0.00774159)'':0.00875623,('PDT000339371.1':0.0527292,(('PDT000424629.1':0.0279268,'PDT000555595.1':0.0299285)'':0.00346916,('PDT000443377.1':0.0288095,((('PDT000557062.1':0.0199024,'PDT000395330.1':0.0198191)'':0.00226996,('PDT000663996.1':0.0171757,('PDT000258368.2':0.016389,'PDT000311268.2':0.011599)'':0.00729187)'':4.90542e-05)'':0.00254557,('PDT000311256.2':0.0179592,('PDT000095697.2':0.019626,('PDT000504748.1':0.0113443,('PDT000311249.2':0.00939323,'PDT000504745.1':0.00725347)'':0.0073663)'':0.00447173)'':0.00139353)'':0.000338379)'':0.00347406)'':0.00379525)'':0.0182684)'':0.00712756)'':0.0346862,(('PDT000616858.2':0.0383471,('PDT000490928.2':0.0259533,(('PDT000131760.2':0.0200024,'PDT000111120.1':0.0199074)'':0.00160369,('PDT000599854.1':0.0119569,('PDT000576795.2':0.0136144,('PDT000399937.2':0.0101859,'PDT000119996.3':0.00873267)'':0.00190777)'':0.00106517)'':0.00607655)'':0.00537086)'':0.00729408)'':-0.00527725,('PDT000443267.2':0.0459694,('PDT000602880.2':0.0330761,('PDT000386998.2':0.027952,('PDT000721701.1':0.0335827,('PDT000497776.1':0.0136236,(('PDT000719884.1':0.00939474,('PDT000281754.3':0.0104077,('PDT000139886.3':0.00711277,('PDT000662471.1':0.00580064,'PDT000098747.3':0.00418737)'':0.00112083)'':0.00360635)'':0.0020292)'':9.74124e-05,(('PDT000502954.2':0.0067929,('PDT000671979.1':0.00104492,'PDT000502393.1':0.00135388)'':0.00425235)'':0.00217241,('PDT000497928.1':0.00998479,('PDT000201200.3':0.00335794,('PDT000271230.3':0.0022706,'PDT000499244.2':0.00251886)'':0.000730355)'':0.00507404)'':-0.00114198)'':0.00397329)'':0.00115853)'':0.0133546)'':0.00278074)'':0.0013405)'':0.0069019)'':-0.00949742)'':0.0588499)'':0.0218691,((('PDT000571138.1':0.0503785,('PDT000078530.1':0.0632419,(('PDT000262727.1':0.0466759,('PDT000425687.1':0.0427701,'PDT000714102.1':0.0322521)'':0.00763136)'':0.00406494,(('PDT000713692.1':0.0217866,('PDT000661769.1':0.0105248,'PDT000497263.1':0.0132871)'':0.00851297)'':0.0221621,('PDT000017665.2':0.0342661,('PDT000017721.2':0.0370718,('PDT000010595.2':0.023436,('PDT000426974.1':0.0132566,'PDT000427530.1':0.0130072)'':0.00937272)'':0.00365021)'':0.00603401)'':0.00906781)'':0.00297322)'':0.00139038)'':0.00285333)'':0.00325217,((('PDT000534968.1':0.0382246,('PDT000534910.1':0.0259716,('PDT000216020.1':0.0107232,'PDT000304209.1':0.0160367)'':0.0127342)'':0.0109739)'':0.00269054,(('PDT000205424.1':0.0250474,('PDT000713844.1':0.0280372,('PDT000520988.1':0.0192382,'PDT000205419.1':0.0167863)'':0.0062515)'':0.00378712)'':0.00328079,('PDT000616108.1':0.0423427,('PDT000720380.1':0.0281001,('PDT000102982.1':0.0309648,('PDT000262728.1':0.0206227,('PDT000397603.1':0.0119997,'PDT000193800.1':0.0130575)'':0.0058623)'':0.00677797)'':0.00272098)'':0.0029202)'':0.00103478)'':0.00810082)'':0.00989928,((('PDT000694419.1':0.0373214,(('PDT000135926.2':0.0239868,'PDT000019142.1':0.0202443)'':0.0118064,('PDT000719016.1':0.0357757,('PDT000674922.1':0.031394,('PDT000425635.1':0.0235278,('PDT000719078.1':0.00870983,'PDT000251728.1':0.0105632)'':0.0145534)'':0.00395102)'':0.00669333)'':-0.000918829)'':0.00529572)'':0.00225328,(('PDT000283905.1':0.0183305,('PDT000283900.1':0.016195,'PDT000283904.1':0.0115301)'':0.00435027)'':0.0219632,('PDT000181715.2':0.0534217,('PDT000714131.1':0.0378807,('PDT000320707.1':0.027929,('PDT000205447.1':0.0297065,('PDT000535014.1':0.0205182,('PDT000535263.1':0.0144423,('PDT000713634.1':0.0102843,'PDT000357736.1':0.0104108)'':0.00521134)'':0.00260108)'':0.00819153)'':0.000398096)'':0.00653723)'':0.0050499)'':0.000488737)'':-0.000351929)'':0.00577548,(('PDT000042664.1':0.0405937,(('PDT000426336.1':0.0233479,'PDT000675120.1':0.02807)'':0.00276035,('PDT000318970.1':0.0222682,'PDT000319013.1':0.0210602)'':0.004136)'':0.0114997)'':0.00364933,(('PDT000136142.2':0.0407653,'PDT000651342.1':0.0411453)'':0.00175201,('PDT000397850.1':0.0420966,((('PDT000661477.1':0.0263906,'PDT000007694.2':0.0236633)'':0.00980452,('PDT000132048.2':0.0335552,'PDT000161622.2':0.0305076)'':0.00232163)'':0.00343519,((('PDT000136012.2':0.0258302,'PDT000278766.2':0.0262887)'':0.00972944,('PDT000694455.1':0.0226621,'PDT000017689.2':0.0248028)'':0.00885494)'':0.00152781,(('PDT000231582.2':0.0314764,'PDT000713455.1':0.0295693)'':0.00230893,(('PDT000066959.1':0.019453,'PDT000278765.2':0.0175584)'':0.00816126,('PDT000346735.1':0.0280378,(('PDT000713258.1':0.00986653,'PDT000713670.1':0.0112532)'':0.0045448,('PDT000713414.1':0.0157357,'PDT000066958.1':0.00931421)'':0.00428025)'':0.0101908)'':0.00187377)'':0.00216166)'':0.0037276)'':0.00179151)'':0.00571571)'':0.00116575)'':-0.0023866)'':0.00190955)'':0.0037804)'':0.00286419)'':0.00522769,((('PDT000661666.1':0.0375391,('PDT000251891.1':0.0295529,('PDT000535059.1':0.0128755,'PDT000714598.1':0.0104382)'':0.0159838)'':0.0040258)'':0.0133557,(('PDT000637580.1':0.0322568,('PDT000047634.1':0.02995,('PDT000162089.1':0.00854742,'PDT000162102.1':0.00824178)'':0.0130791)'':0.00797171)'':0.00429293,(('PDT000398006.1':0.0240973,('PDT000106257.2':0.0211449,'PDT000111110.1':0.0195334)'':0.00157936)'':0.00348151,('PDT000385031.1':0.0212091,('PDT000251720.1':0.0198079,('PDT000188169.2':0.013546,('PDT000352418.1':0.00966696,('PDT000543322.1':0.00856619,('PDT000042453.1':0.00850515,'PDT000193785.1':0.00888015)'':0.00125476)'':0.00207887)'':0.00122631)'':0.00590542)'':0.00289908)'':0.000838683)'':0.0086528)'':0.00935399)'':-0.0061554,((('PDT000618446.1':0.0235155,(('PDT000502120.1':0.0106304,'PDT000502237.1':0.0116424)'':0.00222787,('PDT000502137.1':0.0144226,('PDT000346732.1':0.0132072,('PDT000346815.1':0.00729632,'PDT000362442.1':0.00750598)'':0.00520677)'':0.0017941)'':-0.000291236)'':0.00999385)'':0.00952254,('PDT000626802.1':0.0456949,(('PDT000042704.1':0.021768,('PDT000134598.2':0.0176759,'PDT000193787.1':0.0160031)'':0.00582234)'':0.00437363,(('PDT000069658.1':0.0221391,('PDT000093016.2':0.0150173,('PDT000102976.1':0.0116504,('PDT000618453.1':0.0127031,('PDT000415788.1':0.00836062,('PDT000103004.1':0.00747381,'PDT000205428.1':0.00753319)'':0.00139783)'':0.00145526)'':0.000993649)'':0.00375371)'':0.00384183)'':2.80484e-05,('PDT000042659.1':0.0240479,('PDT000545921.1':0.018771,('PDT000017915.2':0.0177959,('PDT000543109.1':0.0136611,('PDT000251732.1':0.0143822,('PDT000201781.1':0.00980117,('PDT000572842.1':0.00959232,('PDT000115031.2':0.00835907,'PDT000067800.1':0.00691653)'':0.00105438)'':0.000838608)'':0.00200234)'':0.00238585)'':0.00350627)'':0.00184144)'':0.000963571)'':-0.00073782)'':0.00354282)'':0.0104913)'':-0.00180229)'':0.00263757,(('PDT000543270.1':0.0434082,('PDT000042652.1':0.0285921,(('PDT000397602.1':0.0207288,'PDT000397868.1':0.0222002)'':0.00324118,(('PDT000397695.1':0.0250174,('PDT000081040.1':0.0112889,('PDT000086433.1':0.00878153,'PDT000011754.2':0.00867187)'':0.000586586)'':0.010725)'':-0.000645255,('PDT000713621.1':0.0185055,(('PDT000042610.1':0.0061954,'PDT000216010.1':0.0073572)'':0.00566275,('PDT000233060.1':0.0121759,('PDT000619637.1':0.0073054,'PDT000346847.1':0.0074239)'':0.00371631)'':0.000704084)'':0.00363128)'':0.0019194)'':0.00282962)'':0.00597096)'':0.00378703)'':-0.00341767,('PDT000630465.1':0.0437538,('PDT000421171.1':0.0504995,(((('PDT000502170.1':0.0206787,'PDT000007689.2':0.019911)'':0.0018842,('PDT000042738.1':0.0181149,('PDT000267867.2':0.0112511,'PDT000128425.2':0.0162198)'':0.00219319)'':0.00590446)'':0.00131697,(('PDT000153288.2':0.0154823,'PDT000042680.1':0.0123312)'':0.00878399,('PDT000713422.1':0.0216805,('PDT000582889.1':0.0131149,('PDT000017886.2':0.0104964,('PDT000162101.1':0.00805952,'PDT000249757.1':0.00772528)'':0.00305397)'':0.00595535)'':0.0060281)'':-0.00111409)'':0.00420303)'':0.000496457,('PDT000502118.1':0.027489,('PDT000501849.1':0.0254322,('PDT000346712.1':0.0214656,('PDT000082908.1':0.0167398,(('PDT000188107.2':0.013358,('PDT000188073.2':0.00789905,'PDT000346807.1':0.00907025)'':0.00406227)'':0.00241976,(('PDT000588059.1':0.0105781,'PDT000501858.1':0.00916856)'':0.00313959,('PDT000502178.1':0.0118997,('PDT000501899.1':0.00819886,('PDT000502129.1':0.0049722,'PDT000501908.1':0.0053062)'':0.00106004)'':0.00342947)'':0.00146817)'':0.00194355)'':0.00355328)'':0.000788746)'':0.000743712)'':0.00533802)'':0.0026479)'':0.0130874)'':0.00349923)'':-0.0149587)'':0.00538195)'':0.00410965)'':0.0200543)'':0.0453049)'':0.0593713)'':-0.0625533,((((('PDT000713835.1':0.0333974,('PDT000176225.2':0.0263077,'PDT000443862.1':0.0177938)'':0.0124573)'':0.0144246,(('PDT000380133.1':0.032808,('PDT000713480.1':0.0322913,'PDT000249948.1':0.0282195)'':0.00607396)'':0.00969418,(('PDT000534950.1':0.0124002,('PDT000281731.2':0.0104919,'PDT000380148.1':0.0107762)'':0.00327086)'':0.00436878,(('PDT000181826.2':0.0106735,'PDT000535252.1':0.00939596)'':0.00624885,('PDT000534997.1':0.0126926,'PDT000346818.1':0.0134021)'':0.00401047)'':0.0022531)'':0.028753)'':0.00203917)'':0.0099572,('PDT000380160.1':0.040764,('PDT000714122.1':0.0291152,('PDT000427723.1':0.0391595,('PDT000535090.1':0.0261683,(('PDT000571142.1':0.0161126,'PDT000362456.1':0.0181192)'':0.00068956,(('PDT000619595.1':0.013005,'PDT000380136.1':0.0129806)'':0.00893432,(('PDT000438211.1':0.0127355,'PDT000042542.1':0.00996013)'':0.00486649,('PDT000011835.2':0.0174339,('PDT000325719.1':0.0149306,('PDT000042445.1':0.0120408,('PDT000042541.1':0.0100404,(('PDT000316940.1':0.00748286,'PDT000661719.1':0.00844704)'':0.00178717,('PDT000378715.1':0.00948178,('PDT000380149.1':0.00885792,(('PDT000713573.1':0.00448067,'PDT000713578.1':0.00468998)'':0.00138456,('PDT000380139.1':0.00395201,'PDT000017778.2':0.00413182)'':0.00105319)'':0.00181911)'':0.00146069)'':0.000242598)'':0.00100224)'':0.00105756)'':0.00163985)'':0.00116316)'':-0.000241899)'':0.00245931)'':0.00107265)'':0.004045)'':0.00492931)'':0.00204921)'':0.00229644)'':0.0231045)'':0.0312871,((('PDT000713370.1':0.0096534,'PDT000017901.2':0.0110184)'':0.0310874,((('PDT000346720.1':0.0150924,('PDT000502205.1':0.0105847,'PDT000501917.1':0.011856)'':0.00500405)'':0.00615958,('PDT000502209.1':0.0184974,('PDT000346772.1':0.0152543,('PDT000501861.1':0.0111618,('PDT000346727.1':0.0078141,'PDT000501912.1':0.0070502)'':0.0054732)'':0.00369538)'':0.00574533)'':-0.00225398)'':0.014882,('PDT000346706.1':0.0444561,(('PDT000188172.2':0.0264763,'PDT000502002.1':0.0293244)'':0.00330128,('PDT000502231.1':0.0366174,('PDT000502233.1':0.0236226,(('PDT000188074.2':0.007776,'PDT000346718.1':0.0067499)'':0.0057107,('PDT000501855.1':0.0173066,(('PDT000188069.2':0.00857188,'PDT000502162.1':0.0113623)'':0.000729193,('PDT000346763.1':0.00813417,('PDT000502023.1':0.0142627,('PDT000346784.1':0.00892739,(('PDT000346888.1':0.00587496,'PDT000347059.1':0.00637604)'':0.00227651,('PDT000502046.1':0.00900492,('PDT000502146.1':0.00665322,'PDT000502598.1':0.00630148)'':0.000901977)'':0.000801234)'':0.000664394)'':0.0017661)'':0.000451959)'':-0.000519685)'':0.00307705)'':-0.00124949)'':0.0116157)'':0.00679747)'':-0.00445238)'':0.00739217)'':-0.0010895)'':0.00631099)'':0.0255138,('PDT000042595.1':0.183798,((((('PDT000714163.1':0.0152648,('PDT000162086.1':0.00974013,'PDT000249792.1':0.00785607)'':0.00743437)'':0.00218599,('PDT000587616.1':0.016516,('PDT000356232.1':0.0159954,('PDT000470052.1':0.0111363,('PDT000713454.1':0.00890094,'PDT000713645.1':0.0107108)'':0.00249267)'':0.00324143)'':0.00310653)'':0.000230743)'':0.00110901,('PDT000017632.2':0.0189627,('PDT000718837.1':0.00899115,('PDT000713438.1':0.0105267,(('PDT000008471.2':0.00406011,'PDT000248498.1':0.00301795)'':0.00252095,('PDT000694423.1':0.00855722,('PDT000103234.1':0.00493264,('PDT000557299.1':0.00365831,('PDT000713360.1':0.00178839,'PDT000011792.2':0.00170742)'':0.00123375)'':0.00116737)'':0.00213296)'':-0.000374979)'':0.00297439)'':0.0019137)'':0.00361374)'':0.00200009)'':0.0162895,(('PDT000042450.1':0.0372319,('PDT000502013.1':0.0405889,((('PDT000501866.1':0.0212201,('PDT000160689.1':0.0127581,'PDT000346800.1':0.0136383)'':0.00494668)'':0.00428845,('PDT000713652.1':0.0232131,(('PDT000205429.1':0.0192396,'PDT000239341.1':0.0187077)'':0.00242287,('PDT000134067.2':0.0155501,('PDT000135978.2':0.0140225,'PDT000502025.1':0.0151008)'':0.00117213)'':0.00540249)'':0.00256683)'':0.00124778)'':0.00517018,(('PDT000502076.1':0.0238354,'PDT000502224.1':0.018995)'':0.0122387,(('PDT000502601.1':0.0289421,'PDT000250136.1':0.0284373)'':0.00102969,(('PDT000317146.1':0.0209307,('PDT000103013.1':0.0230603,('PDT000415780.1':0.0167763,('PDT000325513.1':0.0118305,'PDT000492225.1':0.0100941)'':0.00461329)'':0.00393585)'':0.00157078)'':-0.00145185,('PDT000069642.1':0.0200152,('PDT000318560.1':0.0140161,('PDT000162088.1':0.0179525,('PDT000193797.1':0.0147736,('PDT000707603.1':0.0119908,('PDT000042488.1':0.00795431,('PDT000510010.1':0.00717981,('PDT000713430.1':0.00643701,('PDT000694513.1':0.00353324,'PDT000713974.1':0.00314605)'':0.00218534)'':0.00169983)'':0.00035412)'':0.00446667)'':0.000715589)'':0.00189723)'':0.00185075)'':0.0035603)'':0.00114934)'':0.00657904)'':0.00425818)'':-0.00267433)'':0.00662099)'':0.00044636)'':-0.00739374,((('PDT000502083.1':0.022919,'PDT000502144.1':0.0241835)'':0.00128034,('PDT000685906.1':0.030247,('PDT000216009.1':0.0206788,('PDT000426820.1':0.0176734,('PDT000397805.1':0.0167262,('PDT000415784.1':0.0116996,('PDT000162099.1':0.0114826,('PDT000397744.1':0.00746727,('PDT000714007.1':0.00298875,'PDT000162098.1':0.00269817)'':0.00265317)'':0.00409226)'':0.00146382)'':0.00154148)'':0.00224819)'':0.00664129)'':0.00185277)'':-0.000684012)'':-0.0015251,(('PDT000248469.1':0.0313341,('PDT000008584.2':0.0120698,'PDT000103091.1':0.0119048)'':0.0148829)'':0.00247023,('PDT000502603.1':0.027981,('PDT000501851.1':0.0348397,(('PDT000162096.1':0.0239028,'PDT000346985.1':0.0223405)'':0.00160541,(('PDT000661525.1':0.0137705,'PDT000661697.1':0.0132646)'':0.0112001,(('PDT000188161.2':0.0169899,'PDT000433331.1':0.0169334)'':0.00135574,(('PDT000502078.1':0.0195394,('PDT000193808.1':0.0135317,('PDT000188072.2':0.0129287,('PDT000316115.1':0.0122838,'PDT000103028.1':0.0111179)'':0.000977248)'':0.00255563)'':0.00288522)'':-0.00162856,(('PDT000139200.2':0.0158935,('PDT000234051.2':0.0113586,'PDT000279213.1':0.0133523)'':0.00473438)'':-9.00416e-06,(('PDT000690291.1':0.0109949,('PDT000673493.1':0.00892307,('PDT000069627.1':0.00416856,'PDT000448218.1':0.00611804)'':0.00465728)'':0.00457386)'':-0.000530397,('PDT000372688.1':0.0131604,(('PDT000455373.1':0.01244,'PDT000499223.1':0.0103279)'':0.00129626,('PDT000581713.1':0.016196,(('PDT000193802.1':0.00780184,('PDT000193807.1':0.00729937,('PDT000042527.1':0.00470399,('PDT000249805.1':0.00154875,'PDT000017774.2':0.00184684)'':0.00207871)'':0.00101033)'':0.00143989)'':0.000463178,('PDT000193789.1':0.00872319,('PDT000604964.1':0.0091933,(('PDT000160688.1':0.00630588,'PDT000193809.1':0.00632092)'':0.00022767,('PDT000334868.1':0.0063426,('PDT000332150.1':0.00353702,('PDT000069621.1':0.00383677,(('PDT000017659.2':0.00180692,'PDT000017792.2':0.00158799)'':0.00083335,('PDT000102873.2':0.00203739,('PDT000249961.1':0.00150867,('PDT000193796.1':0.00149279,'PDT000251992.1':0.00200511)'':0.000288127)'':0.000394463)'':0.00038148)'':0.00119231)'':0.00037729)'':0.000348624)'':0.000713862)'':0.00259923)'':0.00124759)'':-0.00287416)'':0.00755561)'':-0.00256603)'':0.0041714)'':-0.000552762)'':0.00258324)'':-0.00123983)'':0.00382635)'':0.00118235)'':0.00190369)'':0.00440141)'':0.000948455)'':-0.00251312)'':-0.00250354)'':0.00276727)'':0.00697642)'':-0.000249995,((('PDT000362464.1':0.0359824,('PDT000557112.1':0.0276458,'PDT000714059.1':0.0235507)'':0.00981593)'':-0.0013644,((('PDT000668385.1':0.0131865,'PDT000618444.1':0.0120419)'':0.0059388,('PDT000714550.1':0.0185229,(('PDT000136060.2':0.00047208,'PDT000713525.1':0.00052752)'':0.0153672,('PDT000017729.2':0.0112284,'PDT000448204.1':0.0122332)'':0.00273213)'':0.00399928)'':0.000885944)'':0.00348996,(('PDT000714011.1':0.0175908,'PDT000011807.2':0.0160938)'':0.00890991,((('PDT000251516.1':0.00490144,('PDT000252575.1':0.00253103,'PDT000011797.2':0.0023587)'':0.00276279)'':0.0023679,('PDT000713289.1':0.0107066,(('PDT000011799.2':0.00376618,'PDT000017793.2':0.00501012)'':0.00159343,('PDT000661304.1':0.00558997,'PDT000017685.2':0.00437913)'':0.000526127)'':0.00319496)'':0.000389061)'':0.00504446,('PDT000125986.1':0.023537,(('PDT000017655.2':0.00887239,('PDT000042460.1':0.00588453,('PDT000249839.1':0.00339649,'PDT000475562.1':0.00489439)'':0.00215028)'':0.00190642)'':0.000868714,('PDT000096925.2':0.0131665,('PDT000252704.1':0.00778989,('PDT000017670.2':0.00779,('PDT000397751.1':0.00395372,('PDT000103097.1':0.00278114,('PDT000694412.1':0.00304222,'PDT000360986.1':0.0035387)'':0.000215822)'':0.000224722)'':0.0037735)'':0.00190787)'':0.00297371)'':-0.00281189)'':0.00965696)'':-0.00576984)'':0.0092341)'':-0.00329359)'':0.0104281)'':0.00336723,((('PDT000162084.1':0.0293595,(('PDT000397599.1':0.0227893,'PDT000270468.1':0.0152498)'':0.00180447,('PDT000042451.1':0.0177135,('PDT000397896.1':0.0150099,'PDT000042580.1':0.0159404)'':0.00471527)'':0.00208201)'':0.00809308)'':-0.00023323,((('PDT000188087.2':0.012487,('PDT000397999.1':0.0120935,'PDT000713753.1':0.0123813)'':0.00165769)'':0.00636796,('PDT000397752.1':0.00774674,('PDT000042583.1':0.00414249,'PDT000042619.1':0.00364738)'':0.00335392)'':0.0154814)'':0.00290373,('PDT000249783.1':0.0469712,(('PDT000042656.1':0.00878506,('PDT000317709.1':0.00467161,'PDT000251944.1':0.00468628)'':0.0040635)'':0.00875792,('PDT000616067.1':0.0253048,('PDT000517623.1':0.0145664,('PDT000713792.1':0.0151545,('PDT000661510.1':0.0116722,('PDT000694393.1':0.00484257,('PDT000130572.2':0.00394691,'PDT000714164.1':0.0039224)'':0.000831796)'':0.00526971)'':0.00492615)'':0.000122701)'':0.00694094)'':-0.00207618)'':0.0122242)'':-0.00566209)'':0.00421531)'':0.00126056,((('PDT000347049.1':0.0439936,('PDT000346777.1':0.0203497,('PDT000188111.2':0.0177511,'PDT000502010.1':0.0194532)'':0.00338729)'':0.0188097)'':-0.000201652,((('PDT000145371.2':0.0235598,'PDT000586211.1':0.0220862)'':0.00398061,(('PDT000627029.1':0.0086581,'PDT000668423.2':0.0097407)'':0.00710307,('PDT000093014.2':0.0201871,('PDT000313237.2':0.0139903,'PDT000342371.1':0.0136143)'':0.00346591)'':0.000255825)'':0.0112096)'':-0.00194846,((('PDT000037313.1':0.0108146,('PDT000694422.1':0.00152808,'PDT000011806.2':0.0016684)'':0.00972155)'':0.00483034,('PDT000502230.1':0.0143135,('PDT000188342.1':0.00239757,'PDT000188371.1':0.00209345)'':0.0130981)'':0.00456935)'':0.00491806,('PDT000502229.1':0.0360861,(('PDT000017714.2':0.0149149,(('PDT000137878.2':0.00774805,'PDT000086323.1':0.00775425)'':0.00322007,('PDT000714042.1':0.00956926,'PDT000011757.2':0.0106889)'':0.00217563)'':0.00335189)'':0.0014578,(('PDT000008700.2':0.0181001,('PDT000553703.1':0.0113609,'PDT000713768.1':0.0130511)'':0.00340413)'':0.00121064,('PDT000714003.1':0.0238021,('PDT000188070.2':0.0204296,('PDT000714141.1':0.0223129,('PDT000225337.2':0.019199,('PDT000161887.2':0.0176819,(('PDT000136044.2':0.0122655,'PDT000397678.1':0.0140685)'':0.000300904,('PDT000661359.1':0.0180428,(('PDT000636330.1':0.00716121,'PDT000713937.1':0.00674839)'':0.00377991,(('PDT000136040.2':0.00881529,(('PDT000136114.2':0.00649781,'PDT000362452.1':0.00574709)'':0.0010108,(('PDT000173936.2':0.00520208,'PDT000397675.1':0.00534732)'':0.000931859,('PDT000635725.1':0.00476609,'PDT000713374.1':0.0041011)'':0.0015932)'':0.000382813)'':0.00141771)'':-0.000539163,(('PDT000661446.1':0.0085801,'PDT000380147.1':0.0084162)'':0.000611192,('PDT000714058.1':0.00788576,('PDT000248494.1':0.0058812,('PDT000714142.1':0.00490493,('PDT000713362.1':0.00334064,('PDT000713533.1':0.0027791,'PDT000713729.1':0.00250768)'':0.00114805)'':0.000619049)'':0.00143437)'':0.000142081)'':0.00135259)'':-0.000326288)'':0.00304858)'':0.0045786)'':-0.00318892)'':0.0027907)'':0.00153469)'':0.00125973)'':0.000393411)'':0.00485232)'':-0.00521885)'':-0.000838764)'':0.013985)'':-0.00944249)'':0.00403134)'':0.0101507)'':-0.00860123,(('PDT000426778.1':0.0197338,'PDT000103236.1':0.0225378)'':0.0036146,(('PDT000397691.1':0.0263617,('PDT000580714.1':0.0177313,('PDT000397809.1':0.012099,('PDT000042485.1':0.00728757,('PDT000162090.1':0.00726035,'PDT000193804.1':0.00796505)'':0.00143263)'':0.00673627)'':0.00532473)'':0.00185388)'':0.00103585,('PDT000499268.1':0.0260502,((('PDT000587141.1':0.0194239,'PDT000604208.1':0.0160844)'':0.0088867,('PDT000502115.1':0.0275528,(('PDT000502607.1':0.0176139,'PDT000501853.1':0.0198541)'':0.00318973,('PDT000448205.1':0.0242877,(('PDT000042643.1':0.0138831,('PDT000248510.1':0.0115691,('PDT000082907.1':0.00763848,('PDT000713485.1':0.0067212,('PDT000397699.1':0.0044386,'PDT000248512.1':0.00353827)'':0.00213577)'':0.00149656)'':0.00196994)'':0.00469849)'':-0.00347299,('PDT000501878.1':0.0102462,(('PDT000713759.1':0.005235,'PDT000713972.1':0.00405571)'':0.00414276,('PDT000501898.1':0.00772956,('PDT000501963.1':0.00665399,'PDT000502204.1':0.00563131)'':0.000910143)'':0.000782888)'':0.00100515)'':0.00148078)'':0.00981066)'':-0.00237146)'':0.00608033)'':-0.000676823)'':-0.00457896,(('PDT000321352.1':0.025108,('PDT000042465.1':0.0215874,('PDT000502726.2':0.0168056,('PDT000145414.2':0.0121158,'PDT000713544.1':0.0119127)'':0.00399459)'':0.00350017)'':0.00432896)'':-0.00350982,(('PDT000385054.1':0.0235216,('PDT000398001.1':0.0111867,('PDT000610246.1':0.0097599,'PDT000030551.1':0.0082071)'':0.00156349)'':0.00866125)'':-0.0018994,('PDT000380132.1':0.0414947,(('PDT000397853.1':0.0211272,'PDT000415817.1':0.0193908)'':0.0014133,(('PDT000588025.1':0.00867618,'PDT000661704.1':0.0127152)'':0.00720723,('PDT000042616.1':0.0209689,(('PDT000433457.1':0.0193118,('PDT000677289.1':0.0170114,('PDT000193805.1':0.00644792,(('PDT000042457.1':0.00248309,'PDT000042540.1':0.00270601)'':0.00205402,('PDT000162095.1':0.00327454,'PDT000162097.1':0.00422246)'':0.00119501)'':0.00147644)'':0.00480928)'':0.00245495)'':-0.00132219,('PDT000005609.3':0.0195441,(('PDT000131881.2':0.0134554,'PDT000427248.1':0.0129672)'':0.000978989,('PDT000713684.1':0.0212136,('PDT000346688.1':0.0137427,(('PDT000397674.1':0.00902629,'PDT000715947.1':0.00719581)'':0.00291255,('PDT000162085.1':0.0123866,(('PDT000148021.2':0.00206933,'PDT000713699.1':0.00182716)'':0.00693322,('PDT000042472.1':0.0112909,('PDT000694492.1':0.0138916,('PDT000673471.1':0.0070931,('PDT000319047.1':0.00710745,('PDT000135990.2':0.00643153,('PDT000397607.1':0.00765599,('PDT000137875.2':0.00718508,('PDT000661604.1':0.00611728,('PDT000249791.1':0.00340592,(('PDT000572843.1':0.0022283,('PDT000234050.2':0.00148141,'PDT000484077.1':0.0016158)'':0.000665148)'':0.000313678,(('PDT000713781.1':0.000670861,'PDT000162094.1':0.000828839)'':0.00147178,('PDT000427546.1':0.00166887,('PDT000427459.1':0.00232056,('PDT000137883.2':0.0020474,('PDT000713771.1':0.00107898,('PDT000136123.2':0.00108581,('PDT000713707.1':0.000948699,('PDT000713836.1':0.000713275,'PDT000716995.1':0.000486005)'':0.000350061)'':9.63007e-05)'':0.000412767)'':0.000217212)'':7.69198e-05)'':0.000103128)'':0.000176401)'':0.000266079)'':0.00199308)'':0.000351159)'':0.000362096)'':0.000304543)'':0.000218234)'':0.00107569)'':0.00025184)'':0.00265178)'':0.000545991)'':-0.00230033)'':0.00314564)'':-0.00200714)'':0.00384232)'':0.00109109)'':-0.00242047)'':0.00590868)'':-0.00583432)'':0.00634789)'':-0.0019203)'':0.000917941)'':0.00929932)'':-0.0088499)'':0.00308664)'':-9.00966e-05)'':0.00650596)'':-0.00477325)'':0.00385429)'':-0.00234859)'':0.00684687)'':0.00152193)'':-0.000219875)'':0.129774)'':-0.100148)'':0.0259616)'':-0.00545565,((((('PDT000661323.1':0.0272091,'PDT000661416.1':0.0268661)'':0.00897027,('PDT000399799.2':0.0125808,'PDT000500834.1':0.0228879)'':0.0167903)'':0.00371114,(('PDT000017917.2':0.0252193,('PDT000136127.2':0.0284527,'PDT000520893.1':0.0221402)'':0.00492506)'':0.00842627,('PDT000713999.1':0.0561975,(('PDT000713755.1':0.0268162,'PDT000017757.2':0.0352391)'':0.0113456,('PDT000167698.2':0.0517269,('PDT000283899.1':0.0408969,('PDT000430300.1':0.0321885,(('PDT000157124.2':0.0232775,('PDT000661766.1':0.0167739,('PDT000426722.1':0.018154,('PDT000017798.2':0.0143309,('PDT000011712.2':0.010712,('PDT000136159.2':0.00964942,('PDT000713416.1':0.0062794,'PDT000713809.1':0.0074492)'':0.00224223)'':0.00138267)'':0.00321012)'':0.00248392)'':0.00188657)'':0.00196925)'':0.00368437,('PDT000326175.1':0.0342373,('PDT000017773.2':0.0262301,(('PDT000017852.2':0.0169168,('PDT000713243.1':0.0103224,'PDT000300594.1':0.00988363)'':0.00741668)'':0.000167049,('PDT000713493.1':0.0138356,('PDT000103062.1':0.00869105,('PDT000672268.1':0.00652544,('PDT000005196.3':0.00191515,'PDT000019970.2':0.00177968)'':0.00314835)'':0.00245226)'':0.00506524)'':0.00143018)'':0.00478434)'':0.00521453)'':-0.00422121)'':0.00583932)'':0.00773229)'':0.00129721)'':-0.000735232)'':0.00754154)'':-0.00736063)'':0.00217473)'':0.0265812,((('PDT000645065.1':0.0577039,'PDT000252540.1':0.0690191)'':0.00761326,(('PDT000661820.1':0.0542203,('PDT000160589.2':0.0212833,'PDT000661417.1':0.0225512)'':0.0149981)'':0.00706051,('PDT000389330.1':0.0356901,('PDT000625634.1':0.0278698,('PDT000017657.2':0.0263027,('PDT000713904.1':0.0134052,'PDT000714170.1':0.0106648)'':0.0125583)'':0.00633019)'':0.00285114)'':0.00850831)'':0.0171433)'':-0.0014464,((('PDT000570882.1':0.0333683,'PDT000019608.1':0.0324582)'':0.0112724,('PDT000397743.1':0.064777,(('PDT000661668.1':0.0215864,'PDT000008710.2':0.0292661)'':0.0131314,(('PDT000397825.1':0.0318188,('PDT000718873.1':0.0187029,('PDT000395060.1':0.0133851,'PDT000448191.1':0.0119289)'':0.00511127)'':0.0101994)'':0.00499041,('PDT000683480.1':0.0447217,('PDT000287997.1':0.0401698,('PDT000661462.1':0.0381085,((('PDT000082905.1':0.0286736,'PDT000125937.1':0.0237438)'':0.00196192,('PDT000616062.1':0.0186773,('PDT000534946.1':0.0169328,('PDT000380154.1':0.0122213,(('PDT000535229.1':0.0037337,('PDT000534956.1':0.00335838,'PDT000534999.1':0.0029234)'':0.00156804)'':0.000797129,('PDT000535300.1':0.00491945,('PDT000535086.1':0.00417238,('PDT000535237.1':0.000935091,'PDT000535248.1':0.000963389)'':0.00230941)'':0.00137298)'':0.000976818)'':0.00421762)'':0.00284685)'':0.00415034)'':0.0076644)'':-0.0012011,('PDT000283898.1':0.0420604,('PDT000017908.2':0.0234337,((('PDT000557072.1':0.0161068,'PDT000713343.1':0.0144276)'':0.00470479,('PDT000719241.1':0.0150619,'PDT000380157.1':0.0155463)'':0.00360121)'':0.00156394,(('PDT000380141.1':0.0166726,'PDT000380150.1':0.0181847)'':0.00390217,('PDT000718821.1':0.0215147,('PDT000380161.1':0.020127,('PDT000713800.1':0.00895033,('PDT000193792.1':0.0058219,('PDT000557165.1':0.00360781,'PDT000017785.2':0.00288051)'':0.00135147)'':0.00363965)'':0.00629417)'':0.00432123)'':-0.00203215)'':0.00249448)'':0.0030316)'':0.00843722)'':-0.00825469)'':0.00608833)'':0.00240556)'':0.000854461)'':-0.00431466)'':0.00859101)'':0.0123975)'':-0.00783881)'':-0.00155831,(('PDT000421864.1':0.0462164,'PDT000143595.1':0.0466332)'':0.0239396,((('PDT000652358.1':0.0322952,'PDT000661364.1':0.0343135)'':0.0063664,('PDT000694388.1':0.0416257,('PDT000380135.1':0.0373722,(('PDT000661810.1':0.0252152,('PDT000051477.1':0.00389841,'PDT000283903.1':0.00289208)'':0.0221454)'':0.00529673,(('PDT000168460.1':0.0281612,'PDT000011342.2':0.0256012)'':0.00504291,(('PDT000380152.1':0.0261364,('PDT000380131.1':0.0178444,'PDT000017783.2':0.0181738)'':0.00583545)'':-0.0015132,(('PDT000714168.1':0.0115891,('PDT000427197.1':0.0109155,'PDT000466050.1':0.00936277)'':0.00203785)'':0.000777459,('PDT000380155.1':0.0148767,('PDT000694482.1':0.00783074,('PDT000155404.2':0.00495479,'PDT000713457.1':0.00560931)'':0.00188851)'':0.00312809)'':0.00130006)'':0.00738353)'':0.00805072)'':0.00112906)'':0.00271223)'':0.00580631)'':-0.00313987)'':0.00238132,(('PDT000161916.2':0.0150789,'PDT000661802.1':0.016372)'':0.0489847,(('PDT000694380.1':0.0280259,('PDT000619554.1':0.0189857,'PDT000694415.1':0.0197252)'':0.0085517)'':0.0125616,(('PDT000433753.1':0.0370848,('PDT000713246.1':0.0196331,'PDT000380126.1':0.020363)'':0.00782263)'':0.0153665,(('PDT000380128.1':0.0240234,'PDT000380159.1':0.0285979)'':0.0201315,(('PDT000360997.1':0.038824,('PDT000346713.1':0.0261215,'PDT000380138.1':0.0284983)'':0.00793001)'':-3.576e-05,(('PDT000017867.2':0.0438358,('PDT000162087.1':0.0339591,((('PDT000713236.1':0.0166024,'PDT000380158.1':0.0181725)'':0.000721962,('PDT000618454.1':0.0142783,'PDT000017868.2':0.0147932)'':0.00275344)'':0.00855992,((('PDT000427770.1':0.00215108,'PDT000661650.1':0.00264125)'':0.008982,('PDT000713927.1':0.0129906,'PDT000380137.1':0.012465)'':0.000658084)'':0.00955659,('PDT000017903.2':0.0185175,('PDT000694535.1':0.0103026,(('PDT000283902.1':0.00169314,'PDT000497240.1':0.00210231)'':0.00387246,('PDT000283907.1':0.00372921,('PDT000283901.1':0.00239663,'PDT000017636.2':0.00239379)'':0.000562879)'':0.00169213)'':0.00339999)'':0.00854124)'':0.0034308)'':0.00204489)'':0.00539447)'':0.00970389)'':-0.00756901,(('PDT000636363.1':0.0347948,('PDT000661332.1':0.0244777,(('PDT000136155.2':0.0147964,'PDT000380140.1':0.0170796)'':0.0045135,('PDT000380151.1':0.0150819,('PDT000380142.1':0.0135885,('PDT000380134.1':0.00958197,('PDT000713319.1':0.00662081,'PDT000011759.2':0.00760379)'':0.00139268)'':0.00293116)'':0.00164505)'':0.00286142)'':0.00396706)'':0.0121829)'':-0.011471,('PDT000661597.1':0.0285374,('PDT000521005.1':0.0191595,(('PDT000380127.1':0.0145865,('PDT000161877.2':0.00903855,('PDT000380129.1':0.00481888,'PDT000380146.1':0.00605502)'':0.00307345)'':0.00250053)'':0.0045411,('PDT000713617.1':0.00713255,(('PDT000426992.1':0.00430357,'PDT000636232.1':0.00546907)'':0.000752718,('PDT000015298.2':0.00380742,('PDT000380153.1':0.00380396,('PDT000380143.1':0.00315431,'PDT000380156.1':0.00313249)'':0.000685875)'':0.000610465)'':0.000817495)'':0.00467721)'':0.00647738)'':0.00101278)'':0.00763174)'':-0.00400121)'':0.0104311)'':0.0038842)'':0.00423616)'':-0.000289698)'':0.00194353)'':0.0136499)'':-0.0181522)'':0.026393)'':-0.0225981)'':0.0158524)'':0.00517622)'':0.0133463,((('PDT000017634.2':0.0496917,((('PDT000167635.2':0.0248193,'PDT000466984.1':0.019744)'':0.0138155,('PDT000557284.1':0.0322305,'PDT000719266.1':0.0353751)'':0.00446649)'':0.00314717,('PDT000167662.2':0.0412817,('PDT000713887.1':0.0305008,('PDT000502180.1':0.0246742,('PDT000155057.1':0.0144295,('PDT000721347.1':0.00762569,'PDT000248766.1':0.00701721)'':0.00609142)'':0.00894199)'':0.0118307)'':0.00398357)'':-0.00208447)'':0.00876664)'':0.0209181,(('PDT000396776.1':0.0393422,'PDT000251454.1':0.0442551)'':0.0290067,(('PDT000668945.1':0.0372965,('PDT000510057.1':0.0243059,'PDT000719259.1':0.0292321)'':0.0127233)'':0.0115033,(('PDT000167701.2':0.0325082,('PDT000133945.1':0.0210005,('PDT000167707.2':0.0171793,'PDT000455225.1':0.017317)'':0.00518259)'':0.00923485)'':0.00979044,(('PDT000251581.1':0.0330997,(('PDT000397705.1':0.0218886,'PDT000397710.1':0.01596)'':0.00494923,('PDT000250066.1':0.0198434,('PDT000309222.2':0.0122518,'PDT000046846.1':0.00922146)'':0.00708331)'':0.00414667)'':0.0114405)'':-0.00502909,('PDT000167627.2':0.0281572,('PDT000661751.1':0.0317413,('PDT000143592.1':0.0242214,('PDT000713316.1':0.0170897,('PDT000145732.1':0.0118789,('PDT000557056.1':0.00274557,'PDT000713244.1':0.00264258)'':0.00527999)'':0.00346942)'':0.00493706)'':0.00729695)'':0.00310176)'':-0.000684451)'':0.00605523)'':0.0133626)'':0.00993092)'':0.0050967)'':0.0303776,((('PDT000538604.1':0.0865356,('PDT000592989.1':0.057821,('PDT000273840.2':0.041242,(('PDT000017645.2':0.0312495,('PDT000249780.1':0.0249812,(('PDT000137879.2':0.0167485,'PDT000661760.1':0.0142761)'':0.00385645,('PDT000535113.1':0.00394727,'PDT000535170.1':0.00492965)'':0.0161557)'':0.00159059)'':0.00655069)'':0.000736535,(('PDT000535181.1':0.0172376,'PDT000535276.1':0.0184379)'':0.00223295,('PDT000627366.1':0.0191985,('PDT000535000.1':0.0147635,('PDT000534921.1':0.00372678,'PDT000534994.1':0.00256944)'':0.00670297)'':0.00700684)'':0.00305827)'':0.00899979)'':0.00416605)'':0.0127373)'':0.0250125)'':-0.00556668,(('PDT000291269.1':0.0675731,('PDT000004786.2':0.0132652,'PDT000283906.1':0.0150693)'':0.0537751)'':0.0167788,((('PDT000557006.1':0.0613964,('PDT000273990.1':0.0442504,('PDT000501883.1':0.0326483,('PDT000042545.1':0.031813,('PDT000535107.1':0.0208946,('PDT000535106.1':0.0152587,('PDT000587996.1':0.00660417,'PDT000346747.1':0.00783803)'':0.00959862)'':0.00578876)'':0.0034928)'':0.00414141)'':0.0120606)'':0.0197603)'':-0.00825899,('PDT000534979.1':0.0505328,(('PDT000694368.1':0.036945,'PDT000713854.1':0.0393042)'':0.00542673,(('PDT000673451.1':0.0307914,'PDT000042640.1':0.0246002)'':0.00596464,(('PDT000135970.2':0.0240626,'PDT000011836.2':0.0183322)'':0.00870589,('PDT000720463.1':0.0323817,('PDT000535118.1':0.0197946,'PDT000714100.1':0.0182772)'':0.00714766)'':0.000650475)'':0.00238927)'':0.00923463)'':0.0112957)'':0.00158599)'':0.00361522,((('PDT000354271.1':0.0111627,('PDT000354254.1':0.00823233,'PDT000354270.1':0.00593507)'':0.00286444)'':0.0569699,(('PDT000352455.1':0.0395672,('PDT000661380.1':0.0306792,('PDT000587999.1':0.00857021,'PDT000352464.1':0.00795049)'':0.0217161)'':0.0071719)'':-3.0059e-05,('PDT000626688.1':0.0391777,('PDT000651997.1':0.0334075,('PDT000188339.1':0.0289725,('PDT000713575.1':0.0152395,('PDT000535120.1':0.0151827,('PDT000535128.1':0.0140165,('PDT000535239.1':0.00920745,'PDT000713953.1':0.00798085)'':0.00557348)'':0.000693765)'':0.00155857)'':0.00966316)'':0.00180464)'':0.00824613)'':-0.00445455)'':0.0303192)'':-0.0131323,((('PDT000193793.1':0.0511315,'PDT000311278.1':0.0528465)'':0.0035033,('PDT000248842.1':0.0549733,(('PDT000661499.1':0.0252749,'PDT000008583.2':0.030821)'':0.00414129,('PDT000193788.1':0.027373,('PDT000107553.1':0.025114,('PDT000534854.1':0.0132195,('PDT000535201.1':0.0155525,('PDT000534893.1':0.0130476,('PDT000535049.1':0.00832217,('PDT000535006.1':0.00522283,('PDT000027909.2':0.00363741,'PDT000535110.1':0.00424525)'':0.00133882)'':0.00171528)'':0.00356275)'':0.00213908)'':0.000982731)'':0.00968032)'':0.000287985)'':0.00541053)'':0.027413)'':-0.00662853)'':-0.00802588,((('PDT000619091.1':0.0229633,'PDT000082906.1':0.0232462)'':0.00468611,('PDT000433781.1':0.0286367,('PDT000535109.1':0.0248537,('PDT000535349.1':0.0177571,'PDT000535511.1':0.0140408)'':0.0054841)'':0.00539642)'':0.000595666)'':0.00630737,(('PDT000097044.1':0.0332138,('PDT000034490.2':0.0214778,('PDT000397862.1':0.0195589,('PDT000017763.2':0.0140487,('PDT000319884.1':0.0142074,('PDT000713691.1':0.0099307,('PDT000501880.1':0.0108245,('PDT000205219.2':0.00682857,'PDT000193806.1':0.00659223)'':0.00432558)'':0.000421042)'':0.00277217)'':0.0016626)'':0.00130389)'':0.000943292)'':0.00951404)'':-0.0042639,(('PDT000661509.1':0.0214928,'PDT000661630.1':0.0247568)'':0.00429652,('PDT000433797.1':0.0189889,('PDT000587994.1':0.0158794,('PDT000135953.2':0.0130937,('PDT000661677.1':0.00841586,('PDT000136117.2':0.00595015,('PDT000587984.1':0.0036903,'PDT000188338.1':0.00459561)'':0.00189646)'':0.00258035)'':0.00439568)'':0.00189559)'':0.00319307)'':0.00628618)'':-7.12779e-05)'':0.00794987)'':0.0136361)'':0.00355536)'':0.00578226)'':0.0171437)'':-0.00606498)'':0.00703727,(((('PDT000636317.1':0.0302048,'PDT000661776.1':0.0318743)'':0.0122152,('PDT000713968.1':0.0426747,('PDT000589831.1':0.0358143,('PDT000636147.1':0.0210498,'PDT000661780.1':0.017788)'':0.00719518)'':0.00785433)'':0.00563982)'':0.00976296,(('PDT000501996.1':0.0480052,'PDT000346750.1':0.0403265)'':0.0171668,((('PDT000372698.1':0.0341856,('PDT000184495.2':0.0273032,'PDT000193794.1':0.0248545)'':0.0120468)'':0.00529045,('PDT000433716.1':0.0444761,('PDT000162100.1':0.0347484,(('PDT000264725.2':0.0245146,('PDT000158907.2':0.0209171,'PDT000017722.2':0.022101)'':0.00215715)'':0.00256923,('PDT000019230.2':0.0229266,(('PDT000042514.1':0.00948369,'PDT000017811.2':0.00846911)'':0.00942561,('PDT000162093.1':0.0160239,'PDT000017829.2':0.0180771)'':0.00141049)'':0.00321094)'':0.0056433)'':0.00291945)'':0.00842881)'':-0.00187795)'':0.00187308,(('PDT000423886.1':0.038151,'PDT000423889.1':0.0460017)'':0.0204831,(('PDT000587995.1':0.0324218,('PDT000162091.1':0.0328429,('PDT000397814.1':0.0325157,('PDT000716913.1':0.0271556,('PDT000618443.1':0.0198295,('PDT000162083.1':0.0193322,'PDT000193803.1':0.0163574)'':0.00360838)'':0.00546917)'':0.00280413)'':0.0011683)'':0.00141246)'':0.000961741,((('PDT000397698.1':0.0194373,('PDT000158892.2':0.0180129,'PDT000069620.1':0.016187)'':0.0021614)'':0.010956,('PDT000347044.1':0.0349988,('PDT000673567.1':0.0221491,'PDT000673579.1':0.020482)'':0.00731982)'':0.000361143)'':0.00275857,('PDT000019263.2':0.0327632,('PDT000042476.1':0.0254639,('PDT000184435.2':0.0267008,(('PDT000193790.1':0.0171963,'PDT000346707.1':0.0162488)'':0.00587535,('PDT000155870.2':0.0191612,('PDT000713831.1':0.0146843,('PDT000042531.1':0.0114577,'PDT000042635.1':0.00818933)'':0.00654575)'':0.00316991)'':0.000771275)'':0.00460422)'':0.00287813)'':0.00225121)'':0.00167584)'':0.00260859)'':0.0137938)'':-0.00987364)'':0.0144273)'':-0.0035504)'':0.0139067,(('PDT000720476.1':0.0717456,('PDT000664954.1':0.056746,('PDT000625967.1':0.0528015,('PDT000248459.1':0.0393768,'PDT000274333.1':0.0483425)'':0.00892265)'':0.00416624)'':0.0159774)'':0.0037701,(('PDT000218309.1':0.0442574,('PDT000251999.1':0.0464301,('PDT000352434.1':0.0363372,'PDT000354157.1':0.0360803)'':0.00845071)'':0.00728508)'':0.0106751,((('PDT000535129.1':0.0246871,'PDT000535275.1':0.0242797)'':0.033834,((('PDT000466085.1':0.0457343,'PDT000627363.1':0.046079)'':0.00567871,('PDT000668944.1':0.00864196,'PDT000283908.1':0.00711394)'':0.0450461)'':0.00216223,('PDT000719264.1':0.0444556,('PDT000433635.1':0.0426014,('PDT000501888.1':0.0357086,('PDT000535254.1':0.0214193,('PDT000535031.1':0.00999571,'PDT000535321.1':0.00947969)'':0.00501282)'':0.0144574)'':0.00360214)'':0.0048493)'':0.00974545)'':0.00607899)'':0.00156528,('PDT000543272.1':0.073821,(('PDT000661362.1':0.0640107,('PDT000019234.2':0.0595086,('PDT000017676.2':0.0536603,(('PDT000160997.2':0.0252181,('PDT000047201.3':0.00238658,'PDT000424757.1':0.00241054)'':0.0227645)'':0.0145944,('PDT000017843.2':0.0355251,('PDT000713797.1':0.0303092,('PDT000262730.1':0.0284702,'PDT000017802.2':0.0287705)'':0.00364726)'':0.00307648)'':0.00639557)'':0.00412243)'':0.00369987)'':0.00295843)'':-0.00102825,((('PDT000714014.1':0.0329877,'PDT000201773.1':0.0400322)'':0.0105081,('PDT000468862.1':0.0500757,('PDT000713428.1':0.0389405,('PDT000135984.2':0.0381801,('PDT000136023.2':0.0250552,('PDT000425748.1':0.0247488,('PDT000714080.1':0.0259934,('PDT000694520.1':0.00576827,'PDT000694531.1':0.00449993)'':0.0131165)'':0.00700621)'':0.00113216)'':0.00797478)'':0.00588561)'':0.00392098)'':-0.00234679)'':0.00949772,(('PDT000713746.1':0.0512097,(('PDT000019236.2':0.0344626,'PDT000162081.1':0.0342723)'':0.000956414,('PDT000619570.1':0.0213498,'PDT000501894.1':0.0239388)'':0.0111293)'':0.0139307)'':0.000798698,((('PDT000160447.2':0.0408339,'PDT000636277.1':0.0367489)'':0.00159785,('PDT000132096.2':0.0398241,'PDT000125964.2':0.0349404)'':0.00281445)'':0.00157983,('PDT000015288.2':0.0490543,('PDT000042592.1':0.0307962,(('PDT000193791.1':0.021447,('PDT000378756.1':0.0213735,('PDT000587977.1':0.0117806,'PDT000015318.2':0.0119395)'':0.00443325)'':0.00241489)'':0.00104599,('PDT000719909.1':0.0275202,(('PDT000188071.2':0.0140441,'PDT000632131.1':0.0178221)'':0.00415386,('PDT000713810.1':0.0167036,('PDT000713388.1':0.0116987,'PDT000713743.1':0.0132243)'':0.00228686)'':0.0047841)'':0.00759576)'':-0.00318548)'':0.00613301)'':0.0153537)'':-0.00437918)'':0.00686122)'':0.00181475)'':0.00445405)'':0.0129992)'':-0.0106292)'':0.00547343)'':0.00175652)'':-0.00208525)'':0.00740997)'':0.0201898)'':-0.0105196)'':0.00312083)'':0.0131935)'':0.0743246)'':0.00279174,(((((('PDT000557255.1':0.101453,('PDT000487868.2':0.0856099,(('PDT000637304.1':0.0438913,('PDT000307542.2':0.0301494,'PDT000233068.1':0.0296104)'':0.0105208)'':0.0253169,('PDT000359700.1':0.0537191,('PDT000535382.1':0.0397837,'PDT000205422.1':0.0389732)'':0.0127068)'':0.00349613)'':0.0134515)'':0.00507998)'':0.0516847,(('PDT000252673.1':0.112151,('PDT000502223.1':0.0369168,'PDT000502570.1':0.0322638)'':0.0632296)'':0.018525,(('PDT000252002.1':0.0761348,('PDT000391634.1':0.0406633,'PDT000716957.1':0.0458018)'':0.0268412)'':0.0175373,('PDT000161630.3':0.101871,('PDT000397959.1':0.0597595,'PDT000252642.1':0.0493085)'':0.0367268)'':0.00787245)'':0.0227219)'':0.0174696)'':0.0088917,((('PDT000713869.1':0.0676329,'PDT000250193.1':0.0620781)'':0.040711,(('PDT000250197.1':0.0834933,('PDT000505152.1':0.078659,'PDT000248790.1':0.080346)'':0.00926317)'':0.00589827,(('PDT000502037.1':0.0226556,'PDT000502077.1':0.0208354)'':0.0594982,(('PDT000155786.2':0.0757644,'PDT000394882.1':0.0822636)'':0.00537186,(('PDT000694369.1':0.0451592,'PDT000395244.1':0.0528439)'':0.0230687,('PDT000719198.1':0.0708099,('PDT000249929.1':0.0441775,('PDT000502164.1':0.0271156,'PDT000340390.1':0.0283802)'':0.016543)'':0.014454)'':0.00594877)'':0.0107947)'':0.00547557)'':0.00720364)'':0.003563)'':0.00353228,('PDT000687691.1':0.10875,((('PDT000502211.1':0.0764805,'PDT000572835.1':0.0657475)'':0.000696976,(('PDT000714090.1':0.0462308,('PDT000638764.1':0.0347155,('PDT000469855.1':0.0320585,'PDT000566224.1':0.0343618)'':0.00456979)'':0.012318)'':0.000385027,(('PDT000027918.2':0.03493,'PDT000566148.1':0.040777)'':0.00700869,('PDT000550401.1':0.0339748,('PDT000034461.2':0.0253728,'PDT000025507.2':0.0261093)'':0.00662757)'':0.00734491)'':0.00847599)'':0.018803)'':-0.00753409,(('PDT000095746.2':0.0534992,(('PDT000566022.1':0.0268437,'PDT000395380.1':0.0190087)'':0.0113815,('PDT000644898.1':0.012136,('PDT000565911.1':0.00182971,'PDT000639632.1':0.00156825)'':0.0102039)'':0.0152489)'':0.02408)'':0.0118817,(('PDT000644958.1':0.059832,(('PDT000285432.2':0.00416248,'PDT000550359.1':0.00552202)'':0.047374,((('PDT000394936.1':0.0139792,'PDT000395384.1':0.0144142)'':0.00562652,('PDT000397615.1':0.00914506,'PDT000398013.1':0.00811184)'':0.0101976)'':0.00741164,(('PDT000550400.1':0.0133384,'PDT000627589.1':0.0162134)'':0.00400028,('PDT000566713.1':0.0156999,'PDT000639562.1':0.0169753)'':0.00136537)'':0.00780876)'':0.0284876)'':0.0096258)'':0.00373294,((('PDT000394798.1':0.0304287,'PDT000397922.1':0.0335744)'':0.0187649,('PDT000566188.1':0.0220762,('PDT000397994.1':0.00694211,'PDT000566105.1':0.00681389)'':0.0119996)'':0.029513)'':0.00679538,(('PDT000645527.2':0.0505459,('PDT000128785.2':0.0393971,'PDT000399691.1':0.0450305)'':0.00890182)'':0.0038159,(('PDT000523874.1':0.0264685,'PDT000639601.1':0.0262554)'':0.00959071,('PDT000397709.1':0.0409113,('PDT000167633.2':0.0167669,'PDT000497375.1':0.0244692)'':0.0177162)'':0.00171148)'':0.0129723)'':0.00131299)'':0.00421857)'':0.00196501)'':0.000388063)'':0.0288715)'':0.0126731)'':0.0427397)'':0.0259808,(('PDT000426737.1':0.0334263,('PDT000362514.1':0.0244096,('PDT000443245.1':0.0230041,(('PDT000425471.1':0.0209985,'PDT000425724.1':0.0194344)'':0.00359073,(('PDT000570877.1':0.00166147,('PDT000555544.1':0.000656962,'PDT000570878.1':0.000941918)'':0.00063728)'':0.0196524,('PDT000191874.2':0.0224635,('PDT000702621.1':0.0192995,('PDT000639563.1':0.0106633,(('PDT000610884.1':0.0099676,'PDT000015336.2':0.0079482)'':0.00268567,('PDT000306641.2':0.00929057,('PDT000228047.1':0.0062907,('PDT000389660.1':0.00360677,'PDT000011750.2':0.00328014)'':0.00241375)'':0.00341569)'':-0.000172026)'':0.00279313)'':0.00204734)'':0.00269305)'':-0.00159394)'':0.00731041)'':0.00124985)'':0.0022099)'':0.00151974)'':0.140602,(('PDT000617223.1':0.163495,('PDT000251859.1':0.143586,('PDT000250177.1':0.0541324,('PDT000396607.1':0.0383903,'PDT000357723.1':0.0485832)'':0.00840184)'':0.0880469)'':0.0242562)'':0.00786326,((('PDT000293522.1':0.0540973,('PDT000293442.1':0.0295485,'PDT000293529.1':0.0387072)'':0.0218814)'':0.108418,(('PDT000357734.1':0.0363526,'PDT000017649.2':0.037394)'':0.102908,('PDT000309218.2':0.0921767,('PDT000661667.1':0.0809898,('PDT000556948.1':0.0769619,'PDT000228353.1':0.0672361)'':0.0117537)'':0.00461559)'':0.0588533)'':0.00755038)'':0.0153469,(((('PDT000009541.2':0.0487205,('PDT000425607.1':0.0343344,'PDT000017780.2':0.0359766)'':0.0143111)'':0.00940904,('PDT000069677.1':0.0400523,('PDT000714106.1':0.0273749,('PDT000129902.2':0.0222893,'PDT000713324.1':0.022671)'':0.0036609)'':0.00995155)'':0.0116602)'':0.0270813,(('PDT000520753.1':0.0683986,('PDT000019252.2':0.0608742,('PDT000027913.2':0.0163315,'PDT000395236.1':0.0146985)'':0.0299773)'':0.00847719)'':0.0120499,((('PDT000688694.1':0.0491394,'PDT000201772.1':0.0519466)'':0.009007,(('PDT000714234.1':0.0436245,('PDT000565931.1':0.0359519,('PDT000535492.1':0.0087546,'PDT000042473.1':0.0091875)'':0.0193698)'':0.00512274)'':0.0026691,(('PDT000427681.1':0.019455,'PDT000713744.1':0.0173633)'':0.0340524,(('PDT000188077.2':0.0296824,('PDT000360100.1':0.0286848,'PDT000673412.1':0.0322587)'':0.00289334)'':0.00309019,('PDT000468022.1':0.0297463,('PDT000060311.2':0.00708417,'PDT000688654.1':0.00975183)'':0.0183171)'':0.00895389)'':0.00744475)'':0.00065001)'':0.00681167)'':0.00607566,((('PDT000636132.1':0.0437253,'PDT000233054.1':0.0546226)'':0.00591475,(('PDT000332753.2':0.0299964,('PDT000713653.1':0.0167478,('PDT000438230.1':0.0158742,'PDT000713418.1':0.0128273)'':0.0026763)'':0.0121058)'':0.00334747,('PDT000298094.1':0.0383437,('PDT000617957.1':0.0271372,('PDT000717212.1':0.0259578,(('PDT000661410.1':0.0151698,'PDT000661830.1':0.0134866)'':0.00721158,('PDT000661399.1':0.0251454,'PDT000011787.2':0.0207659)'':0.00272572)'':0.0012239)'':0.00409842)'':0.0075268)'':-0.0021192)'':0.0168509)'':-0.00774938,(('PDT000385055.1':0.0432239,('PDT000544629.2':0.0262547,'PDT000710126.1':0.0233394)'':0.0112664)'':-0.000390639,(('PDT000200650.1':0.0107146,('PDT000200653.1':0.00236116,'PDT000200660.1':0.00242878)'':0.00500387)'':0.022088,('PDT000008475.2':0.0434152,(('PDT000706136.1':0.015189,'PDT000042566.1':0.0138911)'':0.0113999,('PDT000042703.1':0.024566,(('PDT000051241.3':0.025945,('PDT000281625.2':0.0175228,('PDT000714051.1':0.0172284,'PDT000706524.1':0.0152276)'':0.00228827)'':0.00655572)'':-0.000270784,(('PDT000502071.1':0.0168425,'PDT000502119.1':0.0185244)'':0.00697996,((('PDT000351555.1':0.0127481,('PDT000544811.1':0.00872304,'PDT000017838.2':0.00862946)'':0.00315524)'':0.00202953,('PDT000713238.1':0.0082516,('PDT000042549.1':0.00767529,'PDT000346780.1':0.00731971)'':0.00126275)'':0.00452218)'':0.00237466,('PDT000213958.2':0.017769,('PDT000373145.1':0.0152256,('PDT000069644.1':0.0129899,('PDT000451146.1':0.0105239,('PDT000694405.1':0.00722885,('PDT000137872.2':0.00336309,'PDT000233039.1':0.0030184)'':0.00294496)'':0.00232892)'':0.00279953)'':0.000872123)'':0.0021469)'':-0.00164174)'':0.00750607)'':-0.00034179)'':0.00236565)'':7.57996e-05)'':0.00683878)'':-0.00478918)'':0.00941187)'':0.0039281)'':0.0125634)'':0.00665451)'':0.0179747)'':0.0666712,(('PDT000535200.1':0.141877,('PDT000427016.1':0.117097,'PDT000069959.1':0.125644)'':0.0209791)'':0.0103748,(('PDT000639579.1':0.150546,('PDT000372146.1':0.0887092,('PDT000354943.1':0.0268366,('PDT000641343.1':0.0254786,'PDT000669209.1':0.0187639)'':0.010583)'':0.0540441)'':0.0563025)'':-0.0135341,(((('PDT000024559.2':0.0484977,'PDT000502085.1':0.0477795)'':0.0189202,('PDT000652924.1':0.0621964,('PDT000051494.2':0.0424168,'PDT000713529.1':0.0470574)'':0.010364)'':0.0051206)'':0.00804285,(('PDT000250232.1':0.0601689,(('PDT000639564.1':0.0367608,'PDT000502333.1':0.0345919)'':0.0120081,('PDT000619783.1':0.0445011,('PDT000706210.1':0.0367076,('PDT000712478.1':0.030542,('PDT000550327.1':0.0212997,'PDT000619632.1':0.0202608)'':0.00781049)'':0.00417215)'':0.0100481)'':-0.00148944)'':0.0134388)'':0.000347553,('PDT000125925.1':0.0536623,(('PDT000137811.2':0.0486754,('PDT000132555.1':0.023513,'PDT000251740.1':0.0204733)'':0.0173195)'':0.00124202,(('PDT000720248.1':0.0370957,'PDT000017639.2':0.0362899)'':0.00282613,('PDT000251389.1':0.0408758,('PDT000251858.1':0.0271869,('PDT000687468.1':0.0221881,('PDT000413095.1':0.0194425,'PDT000556894.1':0.018312)'':0.00281745)'':0.00510207)'':0.0055952)'':0.00556625)'':0.0021182)'':0.00871264)'':0.00549904)'':0.00759208)'':0.0450148,((('PDT000145882.3':0.029256,'PDT000063447.1':0.0379163)'':0.00713558,('PDT000502079.1':0.0395308,('PDT000672724.1':0.0327544,('PDT000233106.1':0.0242433,('PDT000501987.1':0.0124995,'PDT000502093.1':0.00846704)'':0.0125315)'':0.00636976)'':0.00505393)'':0.00786794)'':0.0230649,(('PDT000233088.1':0.0728862,(('PDT000312659.2':0.0467019,'PDT000394984.1':0.048038)'':0.00271639,('PDT000719001.1':0.0362838,('PDT000566151.1':0.0212701,'PDT000394959.1':0.0190983)'':0.0189582)'':0.00890472)'':0.0133097)'':-0.0048412,('PDT000521121.1':0.0963646,('PDT000188366.1':0.0726456,(('PDT000346498.1':0.031278,('PDT000095678.2':0.028602,('PDT000714547.1':0.0136017,'PDT000069622.1':0.0138539)'':0.0116332)'':0.00347305)'':0.0026139,('PDT000714130.1':0.0388799,(('PDT000007742.2':0.0195557,'PDT000352378.1':0.017289)'':0.00901399,('PDT000673458.1':0.02906,('PDT000082077.2':0.0240593,('PDT000252579.1':0.021709,('PDT000042651.1':0.0200395,(('PDT000008023.2':0.0142652,('PDT000252701.1':0.0123545,'PDT000346749.1':0.0107491)'':0.00141702)'':0.0019615,(('PDT000430692.1':0.0121,('PDT000407929.1':0.00282053,'PDT000687845.1':0.00217398)'':0.00962146)'':0.000274536,('PDT000510899.1':0.00715458,('PDT000305086.2':0.00127556,'PDT000228233.1':0.00122269)'':0.0065535)'':0.00440186)'':0.00205202)'':0.00390912)'':0.0023685)'':0.00146925)'':0.00324817)'':-0.00130216)'':0.00727287)'':-0.00505034)'':0.0439133)'':0.0055139)'':-0.0291313)'':0.00627985)'':0.050216)'':0.0175625)'':0.0230737)'':-0.00380656)'':0.0152704)'':0.0042818)'':-0.00326844)'':9.82013e-05)'':0.00105438,(((('PDT000136119.2':0.075769,'PDT000550344.1':0.083966)'':0.0271109,(('PDT000690063.1':0.0351503,'PDT000719188.1':0.0388735)'':0.0176314,(('PDT000241782.1':0.0269105,('PDT000713724.1':0.0215281,'PDT000713983.1':0.0222496)'':0.00907244)'':0.0073479,(('PDT000651914.1':0.0251859,('PDT000358618.1':0.0242124,('PDT000385061.1':0.0121602,'PDT000668402.1':0.0148021)'':0.00544377)'':0.000962821)'':0.00703797,('PDT000042598.1':0.0275678,('PDT000324623.1':0.0225383,('PDT000557258.1':0.0129773,('PDT000252648.1':0.0120132,'PDT000015307.2':0.0123162)'':0.00381964)'':0.00490965)'':0.00713052)'':0.00301462)'':0.00511998)'':0.0219878)'':0.0445588)'':0.0643764,((('PDT000714118.1':0.14566,('PDT000482195.1':0.0741789,('PDT000251744.1':0.0759065,('PDT000415659.1':0.037054,'PDT000543151.1':0.0398989)'':0.0258861)'':0.0172683)'':0.0717303)'':0.00805441,(('PDT000526045.2':0.0964727,'PDT000251496.1':0.0996733)'':0.0521856,('PDT000081731.1':0.142528,('PDT000502045.1':0.109769,('PDT000251483.1':0.072693,(('PDT000310642.2':0.0264679,'PDT000719265.1':0.0287342)'':0.00913302,('PDT000631949.1':0.0192713,'PDT000252443.1':0.0174731)'':0.0116183)'':0.0242199)'':0.0572191)'':0.0287904)'':0.00137692)'':0.00722732)'':0.00743925,(('PDT000610239.1':0.142278,(('PDT000486956.1':0.0494426,'PDT000719092.1':0.0424609)'':0.0232795,(('PDT000270089.2':0.0451501,('PDT000687373.1':0.028202,('PDT000520801.1':0.0241871,('PDT000521084.1':0.0139469,('PDT000520774.1':0.00423596,'PDT000521051.1':0.0050464)'':0.00971592)'':0.00636666)'':0.00511543)'':0.00953343)'':-0.000198286,('PDT000415768.1':0.0633381,(('PDT000500555.1':0.00967132,'PDT000536925.1':0.00607048)'':0.0259764,('PDT000619639.1':0.0414004,(('PDT000694444.1':0.0128433,('PDT000710119.1':0.00665962,'PDT000521096.1':0.00572428)'':0.00480642)'':0.00489123,('PDT000716944.1':0.0295168,('PDT000397588.1':0.0151022,('PDT000610880.1':0.0102228,('PDT000623224.1':0.010202,('PDT000319033.1':0.00653426,('PDT000362525.1':0.00477653,('PDT000235804.2':0.00156206,'PDT000397740.1':0.001733)'':0.00231349)'':0.00205192)'':0.0013774)'':0.000883075)'':0.00387836)'':0.00824879)'':-0.00534186)'':0.0133559)'':-0.00436371)'':0.0322302)'':-0.0226314)'':0.0336662)'':0.0703277)'':0.0151997,(((('PDT000285385.2':0.00770314,'PDT000285625.2':0.00615526)'':0.067625,('PDT000251549.1':0.0410045,('PDT000251465.1':0.0295842,'PDT000251479.1':0.0286844)'':0.0115558)'':0.0216939)'':0.0353642,('PDT000346782.1':0.0335835,('PDT000346808.1':0.0253163,('PDT000310635.2':0.0194618,('PDT000430797.1':0.00437288,('PDT000354942.1':0.00421489,'PDT000430809.1':0.00366541)'':0.00106126)'':0.0100214)'':0.00954338)'':0.00316295)'':0.0722853)'':0.0023027,((('PDT000042668.1':0.0348326,'PDT000300586.1':0.031218)'':0.0298324,('PDT000359704.1':0.0222972,('PDT000636810.1':0.0205295,'PDT000359696.1':0.021092)'':0.0048989)'':0.0366699)'':0.0243579,(('PDT000675150.1':0.0376859,('PDT000408992.1':0.0251949,'PDT000111113.1':0.0236285)'':0.00900438)'':0.00184637,('PDT000390633.1':0.0382536,((('PDT000405307.1':0.0151701,'PDT000668320.1':0.0142069)'':0.00240568,('PDT000477474.1':0.0148253,('PDT000251837.1':0.0169245,('PDT000360077.1':0.00733826,('PDT000234057.2':0.00519487,'PDT000688712.1':0.0046706)'':0.0038397)'':0.00463044)'':0.00311917)'':-0.000410216)'':0.00908461,('PDT000598394.1':0.0284946,(('PDT000430635.1':0.0184948,('PDT000391653.1':0.0165306,'PDT000487568.1':0.0172425)'':0.00194974)'':0.00130867,(('PDT000484087.1':0.0157389,('PDT000549079.1':0.0147293,'PDT000568657.1':0.0143778)'':0.00227958)'':0.00210237,('PDT000321760.1':0.0214662,('PDT000499488.1':0.0146977,('PDT000522646.1':0.010326,('PDT000619447.1':0.0059171,('PDT000271711.2':0.00303145,'PDT000252436.1':0.00285149)'':0.00264513)'':0.00229427)'':0.0069085)'':0.00304588)'':-0.000803134)'':0.00497416)'':0.00838116)'':-0.00660897)'':0.0182098)'':-0.0113823)'':0.0642387)'':0.00961265)'':0.0602145)'':0.00644351)'':0.00372514)'':-8.68548e-05,(('PDT000718900.1':0.0351747,('PDT000251702.1':0.0382377,('PDT000604332.1':0.0196118,'PDT000228345.1':0.0237077)'':0.0132412)'':0.00410714)'':0.143066,((('PDT000706396.1':0.0497075,'PDT000228216.1':0.0395471)'':0.0553304,('PDT000354877.1':0.085791,('PDT000619552.1':0.0625384,('PDT000042519.1':0.0422824,('PDT000251900.1':0.0405314,('PDT000042471.1':0.035164,('PDT000625971.1':0.0325298,('PDT000395411.1':0.0289008,('PDT000085636.2':0.00983331,('PDT000042480.1':0.00156818,'PDT000359675.1':0.00133037)'':0.00938857)'':0.00964304)'':0.0090485)'':0.00431817)'':0.00598481)'':0.00263139)'':0.0115795)'':0.0144853)'':0.013596)'':0.0457671,((('PDT000167669.2':0.0247225,('PDT000415776.1':0.0150353,('PDT000427686.1':0.0100087,'PDT000622868.1':0.00930974)'':0.00795251)'':0.00445909)'':0.0453913,((('PDT000634742.1':0.0777821,('PDT000627293.1':0.0390099,'PDT000223303.1':0.0292532)'':0.0333203)'':0.00374259,(('PDT000081777.1':0.0519037,('PDT000557347.1':0.0504993,'PDT000352450.1':0.0474502)'':0.00718301)'':0.00364058,(('PDT000636356.1':0.06107,('PDT000502200.1':0.0512135,('PDT000494527.1':0.043957,('PDT000661613.1':0.028463,('PDT000310617.2':0.0252218,'PDT000581454.1':0.023661)'':0.00329153)'':0.010013)'':0.007537)'':0.00611292)'':-0.00619199,(('PDT000618334.1':0.030885,('PDT000633071.1':0.0236264,'PDT000011831.2':0.0206364)'':0.00476123)'':0.0156661,('PDT000354240.1':0.052279,(('PDT000277080.3':0.0397195,'PDT000618335.1':0.0376179)'':0.00477166,('PDT000332749.2':0.0428469,(('PDT000661705.1':0.0205879,('PDT000713875.1':0.0181166,'PDT000714015.1':0.0184221)'':0.00465817)'':0.00631656,(('PDT000205669.1':0.025188,('PDT000425542.1':0.0192224,'PDT000011771.2':0.0204312)'':0.0026014)'':0.00377038,('PDT000651938.1':0.0347055,('PDT000521033.1':0.0224595,('PDT000343489.1':0.0179848,('PDT000017895.2':0.0157339,'PDT000501891.1':0.0142163)'':0.00352365)'':0.00202023)'':0.00867788)'':-0.00151727)'':0.00244724)'':0.00630483)'':-2.50372e-05)'':0.0109534)'':-0.0055166)'':0.00225652)'':0.00724127)'':0.0129547)'':-0.0128169,(('PDT000127710.3':0.0455365,(('PDT000661768.1':0.0233247,'PDT000661816.1':0.0315874)'':0.0124526,('PDT000661724.1':0.0277724,('PDT000623222.1':0.0207402,'PDT000011716.2':0.0209511)'':0.00326909)'':0.0126603)'':0.00913701)'':-0.000363028,('PDT000463604.1':0.0494848,((('PDT000661472.1':0.0255501,('PDT000131893.2':0.0146582,'PDT000038899.1':0.0182673)'':0.00627502)'':0.00657122,(('PDT000618830.1':0.0249416,'PDT000410906.1':0.0252778)'':0.00175303,('PDT000687405.1':0.0205815,('PDT000718446.1':0.0180231,'PDT000463608.1':0.0191417)'':0.00321093)'':0.00692288)'':0.00230984)'':0.0046366,('PDT000011762.2':0.0471837,('PDT000426560.1':0.0449509,(('PDT000623228.1':0.0274382,'PDT000362481.1':0.0249661)'':0.01343,('PDT000362511.1':0.0368767,(('PDT000427730.1':0.031701,'PDT000520769.1':0.026918)'':0.00466238,(('PDT000521009.1':0.0252268,('PDT000221730.2':0.0210764,'PDT000132401.2':0.0201151)'':0.00526111)'':0.0051987,('PDT000717311.1':0.0384942,('PDT000520869.1':0.0356946,('PDT000427677.1':0.0289052,((('PDT000427573.1':0.0114093,'PDT000661538.1':0.00910814)'':0.00817549,('PDT000661423.1':0.0194593,('PDT000427564.1':0.00857433,('PDT000713856.1':0.00531192,'PDT000661753.1':0.00475088)'':0.00195002)'':0.00875502)'':0.00370375)'':0.00274899,('PDT000713513.1':0.026598,(('PDT000136138.2':0.0176535,'PDT000694413.1':0.0149111)'':0.00581831,('PDT000360431.1':0.0179024,(('PDT000448201.1':0.0133264,('PDT000427552.1':0.00960964,'PDT000014100.3':0.00933686)'':0.00283356)'':0.00354225,(('PDT000568697.1':0.00867651,'PDT000587563.1':0.00820199)'':0.00449236,('PDT000661623.1':0.0139019,'PDT000011769.2':0.0112302)'':0.00230731)'':0.00263086)'':0.00182064)'':0.00140441)'':0.00399685)'':-0.00147823)'':0.00283207)'':0.00691408)'':0.00300076)'':-0.00700442)'':0.00499798)'':0.00793557)'':-0.00705966)'':0.00722421)'':0.000925641)'':-0.00786954)'':0.0104872)'':0.00121177)'':0.00545544)'':0.00726311)'':0.0196432,((('PDT000317770.2':0.0878724,((('PDT000589036.1':0.0385327,'PDT000288000.1':0.04086)'':0.0417797,('PDT000662174.1':0.0528269,('PDT000167673.2':0.0388029,'PDT000574932.1':0.0361874)'':0.0122079)'':0.0189143)'':0.00178237,('PDT000589849.1':0.0630065,(('PDT000365284.1':0.0401324,'PDT000357597.1':0.0362946)'':0.0108841,('PDT000065068.1':0.0459502,'PDT000011717.2':0.0463117)'':0.0072998)'':0.00401529)'':0.0167146)'':0.00778832)'':-0.0108471,(('PDT000706407.1':0.064827,'PDT000618336.1':0.059019)'':0.00544622,(('PDT000426299.1':0.0484358,('PDT000317181.2':0.0479778,(('PDT000618333.1':0.0265436,'PDT000620530.1':0.0314132)'':0.00351207,('PDT000137840.2':0.0309704,('PDT000661375.1':0.0278863,('PDT000426578.1':0.0201956,('PDT000427242.1':0.0184732,'PDT000619649.1':0.0225338)'':0.00125979)'':0.00495423)'':0.00395844)'':0.00267051)'':0.00494665)'':0.00536353)'':0.00795015,(('PDT000427232.1':0.0417096,('PDT000713410.1':0.0308442,'PDT000011790.2':0.0285754)'':0.00885511)'':0.00143986,('PDT000576621.1':0.0496712,('PDT000668937.1':0.0434028,('PDT000354939.1':0.0344248,('PDT000011742.2':0.0245175,('PDT000645879.1':0.0178058,('PDT000047635.1':0.014423,'PDT000017661.2':0.0150816)'':0.00396754)'':0.00280791)'':0.012862)'':0.00492263)'':0.00963843)'':-0.0037086)'':0.000664716)'':0.0147217)'':0.0073781)'':0.00377211,((('PDT000497422.1':0.0317934,('PDT000395331.1':0.0283579,'PDT000395147.1':0.0238418)'':0.00729721)'':0.0379225,((('PDT000409805.1':0.0370576,('PDT000710121.1':0.0180343,('PDT000669478.1':0.0186511,('PDT000687353.1':0.0124434,('PDT000266011.2':0.00457152,'PDT000405274.1':0.00570378)'':0.00626395)'':0.00658727)'':0.00151337)'':0.022748)'':0.0150994,('PDT000042699.1':0.0445633,('PDT000683159.1':0.0326442,(('PDT000549086.1':0.020284,('PDT000604399.1':0.0124886,'PDT000718860.1':0.013497)'':0.00619937)'':0.00435453,('PDT000397851.1':0.0272896,('PDT000011819.2':0.0179412,('PDT000415607.1':0.0165785,'PDT000719209.1':0.0165903)'':0.000680799)'':0.00547189)'':0.00151712)'':0.00598182)'':0.00518492)'':0.0203708)'':0.00809862,(('PDT000581596.1':0.0802212,('PDT000293525.1':0.0732937,('PDT000390641.1':0.0517525,('PDT000415750.1':0.0446972,('PDT000288457.2':0.0139616,'PDT000706401.1':0.0121012)'':0.0214698)'':0.0120665)'':0.0203399)'':0.00770048)'':-0.015795,(('PDT000390642.1':0.0415832,'PDT000320215.1':0.0368882)'':0.0118257,(('PDT000010934.3':0.0474285,('PDT000390644.1':0.0156984,('PDT000598375.1':0.00183563,'PDT000707604.1':0.00276253)'':0.0130788)'':0.0303849)'':-0.0100426,((('PDT000713390.1':0.0143883,'PDT000302393.1':0.0100774)'':0.00368724,('PDT000433242.1':0.019888,('PDT000372162.1':0.0078433,'PDT000302396.1':0.0109492)'':0.00410777)'':0.00488579)'':0.00153997,('PDT000250628.2':0.0154475,('PDT000719239.1':0.0223477,('PDT000310625.2':0.0174978,('PDT000604372.1':0.00762479,('PDT000517636.1':0.0070352,'PDT000719180.1':0.0063923)'':0.00125136)'':0.00544432)'':0.00298298)'':0.00197615)'':0.000381907)'':0.0171502)'':0.0171369)'':0.00686569)'':0.00181515)'':0.0167617)'':-0.0109551,(('PDT000538573.1':0.0733103,(('PDT000535017.1':0.0104231,'PDT000535152.1':0.0127379)'':0.0209214,('PDT000536577.1':0.0342631,('PDT000559201.1':0.0288179,'PDT000347032.1':0.0244752)'':0.00678048)'':0.00205223)'':0.0261983)'':0.00647819,('PDT000719146.1':0.0975046,(('PDT000433319.1':0.0575321,('PDT000618958.1':0.0611098,(('PDT000137845.2':0.045285,'PDT000433653.1':0.0409437)'':0.000188262,(('PDT000131839.2':0.0268711,('PDT000324038.1':0.0152398,'PDT000689821.1':0.0141981)'':0.0105965)'':0.0103093,('PDT000719175.1':0.0352884,('PDT000136135.2':0.0354272,(('PDT000397854.1':0.025513,'PDT000251380.1':0.0234298)'':0.00403608,(('PDT000136128.2':0.0241373,'PDT000520855.1':0.0259164)'':0.00362807,('PDT000661700.1':0.0229387,('PDT000535098.1':0.0110541,'PDT000535460.1':0.0115442)'':0.0138096)'':0.00566103)'':0.00133822)'':0.00373627)'':0.00269484)'':0.00015351)'':0.0026718)'':0.0139177)'':0.00274964)'':-0.0128568,('PDT000188112.2':0.0722924,('PDT000583247.2':0.0531903,((('PDT000140924.1':0.00054938,'PDT000140925.1':0.00065002)'':0.0178763,('PDT000140923.1':0.013544,(('PDT000307276.1':0.00132364,('PDT000695893.1':0.00120514,'PDT000695895.1':0.00109233)'':7.69859e-05)'':0.0113205,('PDT000368910.1':0.00633009,('PDT000368911.1':0.00345502,('PDT000485990.1':0.00308012,('PDT000346783.1':0.00278303,('PDT000006357.2':0.00204971,'PDT000134153.1':0.002444)'':0.000407471)'':0.000371796)'':0.00103736)'':0.00427556)'':0.000335775)'':0.00731972)'':-0.000173294)'':0.0226875,(('PDT000718453.1':0.0335806,('PDT000538921.2':0.0309239,'PDT000086321.1':0.0308297)'':0.00407333)'':0.0028471,((('PDT000366293.1':0.0281169,'PDT000397923.1':0.0287514)'':0.0027147,(('PDT000042512.1':0.0277085,('PDT000636229.1':0.0178844,'PDT000718902.1':0.0203997)'':0.00716363)'':0.00068605,('PDT000249915.1':0.0315876,('PDT000213910.3':0.023487,('PDT000228360.1':0.0202223,('PDT000203367.3':0.0144494,('PDT000152935.3':0.0154972,('PDT000397930.1':0.0107212,('PDT000385045.1':0.00701198,'PDT000252444.1':0.00811542)'':0.00222963)'':0.00355705)'':0.000762011)'':0.00192709)'':0.0037562)'':0.00557741)'':-0.000973416)'':0.00323398)'':-0.00113467,(('PDT000538914.2':0.0217235,'PDT000156072.1':0.02149)'':0.00961605,(('PDT000093005.2':0.0240539,'PDT000713214.1':0.026636)'':0.00281118,('PDT000672621.1':0.0329444,(('PDT000719280.1':0.0208976,'PDT000249759.1':0.0208386)'':0.00123331,('PDT000598276.1':0.0256404,('PDT000252630.1':0.0200518,('PDT000396800.1':0.014649,('PDT000246493.2':0.0184107,('PDT000557357.1':0.0203247,('PDT000351550.1':0.0157464,('PDT000661702.1':0.0132918,('PDT000398005.1':0.00925577,('PDT000397590.1':0.0088124,('PDT000011828.2':0.00676182,('PDT000148506.2':0.00299071,'PDT000309320.2':0.00240551)'':0.00250872)'':0.00209497)'':0.00107948)'':0.00156715)'':0.00208512)'':0.00202155)'':0.000963084)'':0.000499883)'':0.00067392)'':0.00448223)'':0.000389477)'':0.00755121)'':-0.00405127)'':0.00206961)'':-0.00293117)'':0.00913885)'':0.000653108)'':0.0115493)'':0.0252821)'':-0.0316512)'':0.0454566)'':-0.0236447)'':0.00216752)'':0.00735623)'':0.00974988)'':0.0510339)'':0.0395734)'':-0.00569496)'':0.0032)'':0.00429258,((('PDT000687960.1':0.156073,((('PDT000661513.1':0.0638263,('PDT000713570.1':0.0495112,('PDT000502124.1':0.0374647,(('PDT000132159.2':0.02333,'PDT000340387.1':0.0242424)'':0.00393805,('PDT000373316.1':0.0324348,('PDT000276390.2':0.0123628,('PDT000233105.1':0.0109974,'PDT000346700.1':0.0116714)'':0.00487909)'':0.00937035)'':0.00143426)'':0.00853166)'':0.00298965)'':0.0179073)'':-0.00488448,(('PDT000385027.1':0.0464017,('PDT000103011.1':0.0449695,('PDT000426294.1':0.00783284,'PDT000618749.1':0.00997636)'':0.0271116)'':0.0102745)'':-0.00234117,('PDT000575919.2':0.0446656,('PDT000103074.1':0.0449189,(('PDT000485911.1':0.0222098,('PDT000480953.1':0.0197368,('PDT000233081.1':0.00919316,'PDT000279209.1':0.0105469)'':0.00850317)'':0.00359716)'':0.00241479,('PDT000520705.1':0.0448599,(('PDT000238222.2':0.0306683,('PDT000042536.1':0.0182856,(('PDT000694512.1':0.0184631,'PDT000103078.1':0.0183702)'':0.00228406,('PDT000157495.2':0.0212325,(('PDT000132111.2':0.0101597,'PDT000581460.1':0.00909019)'':0.00202115,('PDT000397750.1':0.0087432,'PDT000572837.1':0.007678)'':0.00386128)'':0.00430538)'':0.000398963)'':0.00449348)'':0.00557175)'':-0.00410664,(('PDT000713817.1':0.0206527,('PDT000627623.1':0.0158634,'PDT000042477.1':0.014971)'':0.00404145)'':0.00631227,('PDT000042572.1':0.0216669,('PDT000136006.2':0.0160437,('PDT000398004.1':0.0177157,('PDT000427070.1':0.0198416,('PDT000132246.2':0.0124722,('PDT000299818.2':0.0143568,('PDT000135996.2':0.0118948,('PDT000156504.2':0.00914938,'PDT000158759.2':0.00871132)'':0.00155717)'':0.00260129)'':0.000656154)'':0.000933899)'':0.000902364)'':0.00178528)'':0.00431789)'':-0.000149202)'':-0.000779255)'':0.0113591)'':-0.00704225)'':0.00842562)'':0.00629474)'':0.00339702)'':0.0141254)'':0.037503,(('PDT000372149.1':0.106061,'PDT000583892.1':0.103125)'':0.0128277,((('PDT000661586.1':0.0623363,'PDT000345832.1':0.0506747)'':0.00416877,('PDT000636347.1':0.0538806,('PDT000017743.2':0.0411296,('PDT000566032.1':0.0274162,'PDT000017669.2':0.0321345)'':0.0100045)'':0.0126264)'':0.006262)'':0.0334049,('PDT000547573.1':0.0860489,(('PDT000293499.1':0.10148,(('PDT000557312.1':0.0318672,('PDT000669512.1':0.0152932,'PDT000710113.1':0.0125479)'':0.0143776)'':0.00781921,('PDT000687350.1':0.0307398,('PDT000719195.1':0.0202447,('PDT000710115.1':0.0131938,('PDT000356907.1':0.0109121,'PDT000710128.1':0.00992738)'':0.00326395)'':0.00588109)'':0.0118809)'':0.00635767)'':0.061064)'':-0.0144142,(((('PDT000463625.1':0.0189303,'PDT000463671.1':0.0189059)'':0.0367853,('PDT000637778.1':0.0367152,('PDT000586443.1':0.0304568,('PDT000536965.1':0.0304768,('PDT000132171.2':0.0195672,'PDT000714076.1':0.0188716)'':0.0112607)'':0.00431456)'':0.00559229)'':0.0146147)'':0.00982811,((('PDT000409047.1':0.0241243,'PDT000713584.1':0.031178)'':0.00418086,('PDT000687416.1':0.0290117,('PDT000683160.1':0.0356531,('PDT000568628.1':0.0233373,('PDT000360084.1':0.00702585,'PDT000683171.1':0.00632475)'':0.0104103)'':0.00618878)'':0.0043568)'':0.00155683)'':0.00509633,((('PDT000498728.1':0.0267338,('PDT000131990.2':0.0176907,('PDT000668379.1':0.0163439,('PDT000409788.1':0.00770412,'PDT000716805.1':0.0101798)'':0.00537573)'':0.00131708)'':0.0113963)'':0.000973323,('PDT000469360.1':0.0233587,(('PDT000250044.1':0.0197668,('PDT000487592.1':0.0119323,'PDT000668376.1':0.0121927)'':0.00503422)'':0.0019238,('PDT000549078.1':0.0224112,('PDT000469353.1':0.00924717,('PDT000481201.1':0.0107954,'PDT000673701.1':0.0102895)'':0.000987681)'':0.00791292)'':-0.000926926)'':0.00516815)'':0.00122626)'':0.00311408,(('PDT000486438.1':0.0257383,('PDT000717163.1':0.0204335,('PDT000379266.1':0.0147838,'PDT000719037.1':0.0159733)'':0.00453292)'':0.00732264)'':-0.000976352,((('PDT000583908.1':0.0192634,'PDT000604316.1':0.0155684)'':0.00278381,('PDT000372169.1':0.0169999,('PDT000395604.1':0.0150236,('PDT000673695.1':0.00682236,'PDT000687394.1':0.00690624)'':0.00439203)'':0.00663656)'':-0.000840385)'':0.00659313,('PDT000619608.1':0.0292624,('PDT000610235.1':0.0261384,('PDT000661821.1':0.0269017,('PDT000619658.1':0.0204079,('PDT000409024.1':0.0147224,('PDT000623226.1':0.0153218,('PDT000683174.1':0.0104487,('PDT000395598.1':0.00303112,'PDT000499472.1':0.00286063)'':0.00715487)'':0.00223408)'':0.00245677)'':0.00168964)'':0.00277027)'':0.00335093)'':0.0032619)'':-0.00351998)'':0.00304291)'':0.00268402)'':0.00645527)'':0.028468)'':-0.017451,((((('PDT000477469.1':0.0132961,'PDT000652772.1':0.0175422)'':0.00195399,('PDT000619446.1':0.00777352,('PDT000678637.1':0.00251139,'PDT000683163.1':0.00238371)'':0.00506293)'':0.00668498)'':0.0150859,(('PDT000415604.1':0.0205349,('PDT000610245.1':0.00885274,'PDT000678633.1':0.00949676)'':0.00909444)'':0.00381847,('PDT000430755.1':0.00996062,(('PDT000382802.1':0.00105247,'PDT000555705.1':0.00124661)'':0.00284631,('PDT000393981.1':0.00357007,'PDT000544816.1':0.00380633)'':0.00183127)'':0.00691552)'':0.00526901)'':0.0113852)'':0.0031216,('PDT000713663.1':0.0389156,(('PDT000717296.1':0.0168123,('PDT000706402.1':0.00914496,'PDT000719930.1':0.0142013)'':0.00680892)'':0.00372509,(('PDT000409479.1':0.0176623,('PDT000362327.1':0.0148857,'PDT000687396.1':0.0135475)'':0.00299866)'':0.00130578,(('PDT000568674.1':0.0161931,('PDT000250534.2':0.00464854,'PDT000499441.1':0.00423458)'':0.00906154)'':0.0032008,('PDT000460005.1':0.0148765,(('PDT000623229.1':0.0139482,('PDT000668486.2':0.00692934,('PDT000678631.1':0.00649356,('PDT000707322.1':0.00291795,'PDT000710130.1':0.00277011)'':0.00175591)'':0.00291791)'':0.00515221)'':-0.000217876,('PDT000678607.1':0.0133907,(('PDT000505140.1':0.00919846,'PDT000622873.1':0.00916744)'':0.00378409,('PDT000430684.1':0.0122988,('PDT000604338.1':0.00770012,('PDT000360069.1':0.0122687,('PDT000631555.1':0.00662387,('PDT000477470.1':0.00523062,('PDT000348073.1':0.00293651,'PDT000522640.1':0.00345774)'':0.0030432)'':0.00152311)'':0.000760436)'':0.000678813)'':0.00180258)'':-0.000444118)'':0.00242233)'':-4.42383e-05)'':0.00431459)'':-0.000403216)'':0.000881883)'':0.000858068)'':0.0283838)'':-0.0137141)'':0.00336194,(('PDT000687474.1':0.0331395,'PDT000588060.1':0.0288986)'':0.0180634,(('PDT000042593.1':0.0575303,((('PDT000719902.1':0.0265633,('PDT000412130.1':0.0172745,'PDT000691091.1':0.0153782)'':0.00697234)'':0.00469922,(('PDT000499513.1':0.0166172,('PDT000409781.1':0.0134983,'PDT000598374.1':0.0136708)'':0.00344485)'':0.00885848,('PDT000487571.1':0.0260544,('PDT000477467.1':0.0174974,('PDT000469355.1':0.00885996,('PDT000499496.1':0.00463301,'PDT000583910.1':0.00534603)'':0.00127057)'':0.0120983)'':0.00543657)'':0.00135379)'':0.0024496)'':0.000624492,(('PDT000372788.1':0.0214195,('PDT000716840.1':0.0103622,'PDT000352470.1':0.00938571)'':0.013671)'':0.00623149,(('PDT000626917.1':0.0153077,('PDT000296180.2':0.0113551,('PDT000309193.2':0.00723842,'PDT000683169.1':0.00993578)'':0.00195014)'':0.00236313)'':0.00708315,(('PDT000391656.1':0.01335,'PDT000717293.1':0.0155982)'':0.00641125,(('PDT000522670.1':0.00904192,'PDT000522761.1':0.00750838)'':0.00838308,('PDT000668353.1':0.015731,('PDT000408980.1':0.0126863,'PDT000523264.1':0.012049)'':0.00259498)'':0.00485272)'':0.00389072)'':-0.000277364)'':0.00561635)'':0.0017866)'':0.0315431)'':-0.0281311,(('PDT000522672.1':0.0540688,('PDT000687471.1':0.0323309,('PDT000372175.1':0.0208651,('PDT000290369.2':0.025426,'PDT000679661.1':0.024662)'':0.00301633)'':0.00434536)'':0.0131418)'':-0.00538526,('PDT000500727.1':0.0450787,(('PDT000136054.2':0.0136072,('PDT000568703.1':0.00757184,'PDT000637148.1':0.00510536)'':0.00577351)'':0.0260814,((('PDT000458787.1':0.0194068,'PDT000719925.1':0.0181251)'':0.00277602,('PDT000673700.1':0.0215689,('PDT000716778.1':0.0261065,('PDT000583880.1':0.012249,'PDT000654225.1':0.0135114)'':0.00784917)'':0.0030984)'':0.000814966)'':0.00443599,(('PDT000491859.1':0.0322331,'PDT000568663.1':0.0303522)'':0.00154464,(('PDT000469365.1':0.0284071,('PDT000678609.1':0.0248325,('PDT000190724.2':0.0132122,'PDT000368001.1':0.0184769)'':0.00413438)'':0.00369332)'':0.00369602,((('PDT000353559.1':0.0160918,'PDT000498710.1':0.0150773)'':0.00242982,('PDT000405293.1':0.0122918,('PDT000719009.1':0.0148015,('PDT000668330.1':0.00837759,('PDT000109372.2':0.00463256,'PDT000668399.1':0.00665954)'':0.00390191)'':0.00266424)'':0.00235673)'':0.00239791)'':0.00176951,(('PDT000477479.1':0.0168408,'PDT000604351.1':0.0148911)'':0.00528971,(('PDT000622880.1':0.0163727,'PDT000710118.1':0.0171447)'':0.00479694,('PDT000683173.1':0.0162608,(('PDT000287705.2':0.00440464,'PDT000017850.2':0.00588196)'':0.00931901,('PDT000687467.1':0.0145127,('PDT000505148.1':0.0116177,'PDT000679662.1':0.0110735)'':0.00329316)'':0.000106985)'':0.00404031)'':0.00241495)'':0.00168693)'':-0.00172917)'':0.00838833)'':0.00619466)'':-0.00540417)'':0.00695779)'':0.00960606)'':-0.00594795)'':-0.00550923)'':0.0282884)'':-0.0233978)'':0.00790314)'':0.0395816)'':0.0112263)'':0.000831434)'':0.012725)'':-0.00540318)'':0.0614966)'':0.00277739,((('PDT000717024.1':0.12956,('PDT000619664.1':0.0363565,('PDT000713217.1':0.0203533,'PDT000719210.1':0.0173638)'':0.0157806)'':0.0934315)'':0.0260811,(('PDT000534971.1':0.0386151,('PDT000535212.1':0.0346795,('PDT000535346.1':0.0328065,('PDT000535036.1':0.0220823,('PDT000535071.1':0.0139819,'PDT000535528.1':0.0132574)'':0.00907524)'':0.00709713)'':0.00439705)'':0.004297)'':0.0100568,(('PDT000498721.1':0.0559342,('PDT000250034.1':0.0427911,('PDT000351547.1':0.0316472,'PDT000719219.1':0.0316058)'':0.00890279)'':0.0108548)'':-0.00251954,(('PDT000312281.2':0.0368898,'PDT000716837.1':0.0359323)'':0.014564,('PDT000252512.1':0.0460902,('PDT000251833.1':0.0481906,(('PDT000618456.1':0.0289081,('PDT000216004.1':0.0147337,'PDT000251878.1':0.0129229)'':0.0177544)'':0.000714532,('PDT000675274.1':0.0384806,('PDT000718874.1':0.0227176,('PDT000536936.1':0.0171867,('PDT000719081.1':0.00540926,('PDT000228253.1':0.00387631,('PDT000249763.1':0.00141535,'PDT000249912.1':0.00158046)'':0.00111693)'':0.00379869)'':0.0108585)'':0.00617593)'':0.00632391)'':-0.00206501)'':0.00582419)'':0.00607742)'':0.00429055)'':0.000141895)'':0.00231703)'':0.104333)'':-0.00121726,(((('PDT000293518.1':0.0062189,'PDT000293521.1':0.0061243)'':0.0760096,(('PDT000293443.1':0.0105025,'PDT000293464.1':0.00932812)'':0.0308771,('PDT000293538.1':0.0316395,('PDT000293460.1':0.00869102,'PDT000293520.1':0.00997088)'':0.0221339)'':0.0138444)'':0.0371201)'':0.0648287,(('PDT000556933.1':0.0892303,('PDT000252589.1':0.0155199,('PDT000719112.1':0.00193597,'PDT000719237.1':0.00165828)'':0.0108027)'':0.0891792)'':0.0152255,(('PDT000155328.2':0.0652893,'PDT000682546.1':0.0742147)'':0.0457957,(('PDT000619576.1':0.0901709,'PDT000713295.1':0.0796191)'':0.00446735,('PDT000412172.1':0.0685338,('PDT000252558.1':0.0529988,('PDT000604355.1':0.026685,'PDT000042535.1':0.0308077)'':0.0142514)'':0.0167937)'':0.0202586)'':0.0166503)'':0.0196759)'':0.014794)'':-0.00106421,((('PDT000415785.1':0.0799425,(('PDT000604215.1':0.0544526,('PDT000309191.2':0.0220316,'PDT000557286.1':0.0267585)'':0.0296651)'':0.000992732,(('PDT000710129.1':0.0425812,'PDT000719033.1':0.0498965)'':0.0073118,('PDT000397897.1':0.0505069,('PDT000522470.1':0.0419033,('PDT000683168.1':0.0261069,('PDT000241779.1':0.022114,'PDT000276843.1':0.0240127)'':0.0017288)'':0.0158633)'':0.00596354)'':0.00146905)'':0.00399068)'':0.0203195)'':-0.0107433,('PDT000372171.1':0.0838398,((('PDT000336242.1':0.026715,'PDT000233067.1':0.0304781)'':0.00870622,('PDT000415769.1':0.0288414,'PDT000557304.1':0.0248224)'':0.0172732)'':0.00363402,(('PDT000718859.1':0.0235101,'PDT000069681.1':0.0207062)'':0.0175419,(('PDT000017835.2':0.0247462,'PDT000017893.2':0.0221594)'':0.00881042,('PDT000661503.1':0.0377233,('PDT000042482.1':0.0389584,('PDT000713940.1':0.0172561,('PDT000017827.2':0.00498654,('PDT000270460.1':0.00114523,'PDT000017873.2':0.0011527)'':0.00250622)'':0.0157267)'':0.0100189)'':0.00561016)'':-0.00390642)'':0.00566242)'':0.0036504)'':0.0304445)'':-0.0139983)'':0.0138673,((('PDT000716895.1':0.0396776,('PDT000004711.2':0.0297453,(('PDT000360080.1':0.0202744,'PDT000619629.1':0.0253025)'':0.00367415,('PDT000556989.1':0.0273353,(('PDT000469348.1':0.0113635,'PDT000668364.1':0.0104662)'':0.0107649,('PDT000362328.1':0.0215989,('PDT000109487.2':0.0156343,'PDT000469361.1':0.0178248)'':0.0035126)'':0.00123364)'':0.00355272)'':0.00101257)'':0.006346)'':0.00813625)'':0.0249653,(((('PDT000487584.1':0.0226752,'PDT000717318.1':0.0272851)'':0.00923215,('PDT000631591.1':0.0336291,('PDT000582972.1':0.0182144,'PDT000678611.1':0.0199262)'':0.0159923)'':0.000929296)'':0.00816513,(('PDT000309246.2':0.0531416,('PDT000544815.1':0.0279254,('PDT000542485.1':0.0234096,('PDT000234084.2':0.0117035,'PDT000719929.1':0.014304)'':0.00706598)'':0.00385767)'':0.0172674)'':-0.00334667,(('PDT000188171.2':0.0336392,('PDT000481210.1':0.0236396,'PDT000710120.1':0.0229925)'':0.0068702)'':-0.000949279,(('PDT000652003.1':0.0135694,('PDT000637132.1':0.0141591,'PDT000651919.1':0.013503)'':0.00387299)'':0.00768333,(('PDT000583857.1':0.0156201,'PDT000719278.1':0.0116436)'':0.00249652,('PDT000042606.1':0.0185455,(('PDT000568681.1':0.00836117,'PDT000713229.1':0.00882023)'':0.00320575,('PDT000499325.1':0.0116285,('PDT000309198.2':0.00542196,'PDT000719919.1':0.00736394)'':0.00349773)'':0.00181757)'':0.00262444)'':0.000465654)'':0.00475029)'':0.00708655)'':0.006953)'':0.00612993)'':0.0137659,(('PDT000427052.1':0.0812769,(('PDT000397741.1':0.0343542,'PDT000713558.1':0.0300856)'':0.0198777,('PDT000430735.1':0.0428562,(('PDT000618712.1':0.0426602,('PDT000438118.1':0.0274657,'PDT000636168.1':0.0237433)'':0.0103966)'':-0.00149883,('PDT000694499.1':0.0296173,('PDT000423846.1':0.0224052,('PDT000372772.1':0.0184878,'PDT000584490.1':0.0199926)'':0.00364624)'':0.00606595)'':0.00808739)'':0.00642737)'':0.00244821)'':0.0165395)'':-0.00681924,('PDT000210718.1':0.0894927,((('PDT000674905.1':0.0218549,'PDT000102984.1':0.0220817)'':0.0201197,('PDT000685877.1':0.0566948,(('PDT000510067.1':0.016485,'PDT000017699.2':0.0171052)'':0.00784692,('PDT000716045.1':0.015423,'PDT000661763.1':0.0170805)'':0.00962101)'':0.0186027)'':-0.000887888)'':0.00524014,('PDT000017897.2':0.0604088,('PDT000685878.1':0.0789937,(('PDT000604344.1':0.0255609,'PDT000713554.1':0.0267675)'':0.0128934,(('PDT000321561.1':0.0102724,'PDT000522494.1':0.00787498)'':0.0305746,(('PDT000019131.2':0.0340744,('PDT000719088.1':0.03405,((('PDT000132030.2':0.0126249,'PDT000713704.1':0.0132039)'':0.0116823,('PDT000661351.1':0.0249608,('PDT000267885.2':0.0136854,'PDT000713542.1':0.0161446)'':0.00521128)'':0.000804797)'':0.00407253,('PDT000618452.1':0.0237984,(('PDT000250676.2':0.014622,'PDT000713336.1':0.0148652)'':0.00598862,('PDT000104640.2':0.0137732,'PDT000535519.1':0.0148775)'':0.00661243)'':0.00728048)'':0.00194826)'':0.00269897)'':0.0016738)'':0.000520258,(('PDT000625870.1':0.0246438,'PDT000252520.1':0.0209757)'':0.00621612,(('PDT000397873.1':0.0269161,'PDT000008474.2':0.030866)'':0.00282406,('PDT000233064.1':0.02989,(('PDT000137877.2':0.0243891,('PDT000069657.1':0.0169048,'PDT000276844.1':0.012621)'':0.00867759)'':0.0028186,('PDT000448211.1':0.019572,('PDT000197856.2':0.0252661,(('PDT000107790.2':0.0118082,('PDT000042466.1':0.00368843,'PDT000042617.1':0.00408979)'':0.00668681)'':0.00182748,('PDT000716989.1':0.0153229,('PDT000103054.1':0.010427,('PDT000619685.1':0.00922566,('PDT000005932.2':0.00901676,'PDT000252001.1':0.00883144)'':0.000924336)'':0.00210711)'':0.00461189)'':-0.00206265)'':0.0069155)'':0.00364028)'':-0.00186513)'':0.00645045)'':-0.000542924)'':0.0018496)'':0.00497084)'':0.00337226)'':0.00231384)'':0.0161144)'':0.00193772)'':-0.014464)'':0.0294866)'':-0.020324)'':0.000865965)'':0.0118245)'':-0.00982348,(('PDT000042558.1':0.0699035,('PDT000188170.2':0.0496127,'PDT000188181.2':0.0538953)'':0.012208)'':-0.000757962,(('PDT000421762.1':0.0476923,'PDT000588036.1':0.0520026)'':0.0150411,((('PDT000325716.1':0.0476123,('PDT000415777.1':0.0423584,('PDT000720318.1':0.0379399,('PDT000423963.1':0.0379681,('PDT000534788.1':0.0292337,('PDT000017653.2':0.022448,(('PDT000565117.1':0.00895338,'PDT000622370.1':0.0110127)'':0.00364246,('PDT000042642.1':0.0114945,'PDT000011783.2':0.00963022)'':0.00213942)'':0.00472964)'':0.00520427)'':0.00380728)'':0.0067318)'':0.00172788)'':0.00607402)'':0.00473911,((('PDT000661467.1':0.0397126,('PDT000438219.1':0.0246068,'PDT000031685.1':0.0261756)'':0.0100959)'':0.00809345,('PDT000685883.1':0.0751753,('PDT000716262.1':0.0506674,(('PDT000630458.1':0.0231742,'PDT000017820.2':0.024398)'':0.00518603,(('PDT000713260.1':0.0268183,(('PDT000132089.2':0.0195178,'PDT000132415.2':0.0165449)'':0.00388273,('PDT000382824.1':0.0150457,'PDT000233103.1':0.0151497)'':0.00509609)'':0.00325975)'':-0.000912325,(('PDT000137881.2':0.0095566,'PDT000661329.1':0.0084212)'':0.0028896,('PDT000288455.2':0.00662856,('PDT000713495.1':0.00421625,'PDT000713572.1':0.00525345)'':0.00475304)'':0.0050678)'':0.00822903)'':0.00453243)'':0.0192358)'':0.00527081)'':-0.00996301)'':-0.00260499,(('PDT000685872.1':0.0648003,(('PDT000477471.1':0.0394004,('PDT000678715.1':0.0339748,'PDT000719026.1':0.0365498)'':0.00890016)'':0.00429468,('PDT000136151.2':0.0391333,(('PDT000346715.1':0.0346471,(('PDT000042613.1':0.0224134,'PDT000618460.1':0.0218984)'':0.00530354,('PDT000042644.1':0.0213052,'PDT000661345.1':0.0243461)'':0.00397809)'':0.00472399)'':-0.000183013,(('PDT000672073.1':0.0249062,'PDT000346697.1':0.0252113)'':0.00470539,(('PDT000713436.1':0.0202628,'PDT000713888.1':0.0145215)'':0.0127469,('PDT000713369.1':0.0240147,('PDT000405304.1':0.0277406,'PDT000582882.1':0.0225807)'':0.00334941)'':0.00305756)'':0.00307813)'':0.000931098)'':0.00561096)'':0.00419518)'':0.00837744)'':-0.00942966,(('PDT000427088.1':0.0393114,('PDT000535265.1':0.0214857,('PDT000132215.2':0.013459,('PDT000438117.1':0.00833564,'PDT000500565.1':0.00937056)'':0.00479384)'':0.00959062)'':0.0153521)'':0.00494891,((('PDT000588018.1':0.0271243,'PDT000042657.1':0.0312791)'':0.00396282,('PDT000125942.1':0.025001,('PDT000279227.1':0.0252115,('PDT000158725.2':0.0228103,'PDT000233055.1':0.0199574)'':0.00354702)'':0.00525066)'':0.00741917)'':0.00649321,(('PDT000535288.1':0.034389,'PDT000388471.1':0.0307627)'':0.0140883,(('PDT000583868.1':0.0312831,('PDT000717012.1':0.0241391,('PDT000158923.2':0.0187716,'PDT000398007.1':0.0193311)'':0.00427816)'':0.00656488)'':0.00615221,(('PDT000305091.2':0.0247605,('PDT000522466.2':0.0256698,('PDT000622942.1':0.0147409,'PDT000125932.1':0.0156905)'':0.0102794)'':0.00126498)'':0.00616662,(('PDT000687399.1':0.0345767,('PDT000590090.1':0.0189185,('PDT000042645.1':0.0154985,('PDT000694549.1':0.00904855,'PDT000042443.1':0.0114063)'':0.00254664)'':0.00265714)'':0.0132125)'':0.000923399,('PDT000427081.1':0.0430948,(('PDT000385081.1':0.0238257,'PDT000719202.1':0.0204168)'':0.00509602,('PDT000135721.1':0.0324979,(('PDT000477473.1':0.0176226,'PDT000251945.1':0.0161819)'':0.0115597,(('PDT000299751.2':0.0155574,'PDT000427493.1':0.0159883)'':0.00508854,('PDT000069632.1':0.0190392,('PDT000287715.2':0.0210366,('PDT000188108.2':0.0157323,('PDT000131112.2':0.0117721,'PDT000372005.1':0.0118842)'':0.0032966)'':0.00273641)'':0.000639523)'':0.00149703)'':0.00233575)'':0.00231018)'':-0.00102949)'':0.00891895)'':-0.00534866)'':0.00147304)'':0.00396025)'':0.00755996)'':-0.00202983)'':0.000678412)'':-0.000282874)'':0.00218356)'':0.00537735)'':-0.00205213,((('PDT000714107.1':0.0201092,'PDT000252703.1':0.0212309)'':0.0151474,('PDT000492261.1':0.0408289,('PDT000252626.1':0.0475368,(('PDT000181739.2':0.0242869,'PDT000619778.1':0.0206414)'':0.00458445,(('PDT000137704.1':0.0186615,'PDT000483525.1':0.0222391)'':0.00707768,('PDT000276840.1':0.0269241,('PDT000485819.2':0.0210625,('PDT000718836.1':0.0191918,('PDT000019282.2':0.00792865,'PDT000477880.1':0.00775375)'':0.011726)'':0.0024715)'':0.00276938)'':0.000165889)'':0.00487152)'':0.0098515)'':0.00311757)'':0.000649531)'':0.00368807,((('PDT000520708.1':0.0472202,('PDT000398002.1':0.0218839,'PDT000535510.1':0.0241113)'':0.0193854)'':0.00165376,(('PDT000694468.1':0.0368937,('PDT000360066.1':0.0354818,('PDT000397738.1':0.0262923,'PDT000425804.1':0.0292633)'':0.00480976)'':0.00252749)'':0.00586328,(('PDT000438081.1':0.0263551,'PDT000713642.1':0.0323992)'':0.0109725,(('PDT000588001.1':0.0326607,'PDT000042552.1':0.0307653)'':0.00195624,(('PDT000627216.1':0.0279904,'PDT000252497.1':0.0303323)'':0.00337581,('PDT000535215.1':0.0243329,'PDT000042496.1':0.0275124)'':0.00854679)'':0.00178711)'':0.00275809)'':0.00595948)'':0.000654821)'':0.00061179,(((('PDT000673416.1':0.0334942,'PDT000485966.1':0.0260844)'':0.012437,('PDT000032149.1':0.0370951,'PDT000102993.1':0.0314941)'':0.0200872)'':0.00439747,(('PDT000482225.1':0.0593875,'PDT000622958.1':0.0521065)'':0.000697952,(('PDT000556972.1':0.00667166,'PDT000415793.1':0.00609894)'':0.0256784,('PDT000557094.1':0.0486901,('PDT000135745.2':0.036387,(('PDT000719181.1':0.0268751,('PDT000408997.1':0.0114469,'PDT000604278.1':0.0108612)'':0.015319)'':0.00444384,('PDT000042452.1':0.0322124,('PDT000685897.1':0.0346485,('PDT000719289.1':0.0266955,('PDT000714020.1':0.0285266,('PDT000672274.1':0.0237414,('PDT000719178.1':0.0210997,('PDT000444975.1':0.0240427,('PDT000281487.1':0.0174688,(('PDT000718974.1':0.0136617,'PDT000069617.1':0.0129904)'':0.0032289,('PDT000215994.1':0.0121621,('PDT000249762.1':0.0123636,('PDT000719069.1':0.00937351,('PDT000713822.1':0.00640454,'PDT000714089.1':0.00693146)'':0.00123254)'':0.00305466)'':0.00124113)'':0.00101983)'':0.00187273)'':0.00224743)'':0.00113703)'':0.00128216)'':0.00198178)'':0.00250131)'':0.00209295)'':0.00541146)'':-0.00356251)'':0.00552377)'':0.00342331)'':-0.00354051)'':0.0106799)'':0.000788265)'':-0.00747429,(('PDT000132017.2':0.0446313,('PDT000319878.1':0.0300874,'PDT000672265.1':0.030063)'':0.0109158)'':0.00635659,(('PDT000713814.1':0.0165616,'PDT000543292.1':0.01854)'':0.0313014,(('PDT000426706.1':0.0470907,(('PDT000713259.1':0.0314883,('PDT000713643.1':0.0305193,'PDT000188373.1':0.0272859)'':0.00434345)'':0.00840221,(('PDT000147992.2':0.0282332,'PDT000713565.1':0.0226777)'':0.00527606,('PDT000555619.1':0.0274434,'PDT000006423.2':0.0258638)'':0.00656941)'':0.00539377)'':0.00472154)'':-0.00283409,(('PDT000557095.1':0.0426356,('PDT000352485.1':0.0298682,('PDT000069667.1':0.0206699,'PDT000216013.1':0.0205308)'':0.00771682)'':0.0119565)'':0.00235142,('PDT000521099.1':0.0645962,(('PDT000397620.1':0.0401877,'PDT000694548.1':0.0347712)'':0.00439039,('PDT000460590.1':0.0487673,('PDT000661487.1':0.0411412,(('PDT000042586.1':0.0301101,('PDT000713954.1':0.0305717,('PDT000619863.1':0.0133792,'PDT000713353.1':0.0130774)'':0.0111179)'':0.00297777)'':0.000910252,(('PDT000713526.1':0.0208182,'PDT000661807.1':0.0204423)'':0.00980057,('PDT000713936.1':0.0303394,(('PDT000136034.2':0.02414,'PDT000714072.1':0.0233573)'':0.0050146,('PDT000042562.1':0.03268,('PDT000673612.1':0.0338104,('PDT000683175.1':0.0295405,((('PDT000675513.1':0.0249057,'PDT000017810.2':0.0282306)'':0.00032123,('PDT000714133.1':0.0147191,('PDT000017707.2':0.0127208,('PDT000588045.1':0.00576206,'PDT000042513.1':0.00559814)'':0.00700068)'':0.00192199)'':0.00806897)'':0.000851406,(('PDT000719235.1':0.0263689,'PDT000017674.2':0.0212454)'':0.00142786,('PDT000290005.1':0.0306543,(('PDT000011734.2':0.0187782,'PDT000017900.2':0.0215531)'':0.00124932,('PDT000713920.1':0.0274196,('PDT000517646.1':0.0188219,(('PDT000596023.1':0.00500872,'PDT000636204.1':0.00616448)'':0.0133604,(('PDT000017767.2':0.020084,('PDT000131865.2':0.0105152,('PDT000017837.2':0.0107761,('PDT000713288.1':0.0072204,'PDT000017816.2':0.0063119)'':0.00162643)'':0.00116046)'':0.0070692)'':0.00149411,('PDT000188139.2':0.0169701,('PDT000713528.1':0.0161871,(('PDT000713763.1':0.0103902,'PDT000714048.1':0.00989216)'':0.0043288,('PDT000216002.1':0.00994044,('PDT000707296.1':0.010809,('PDT000673474.1':0.00848807,('PDT000415811.1':0.0068328,'PDT000019132.2':0.0061955)'':0.00104748)'':0.00179917)'':0.00247594)'':0.0029073)'':0.000887965)'':0.00219544)'':-0.000211994)'':0.00117329)'':0.00111582)'':0.00294072)'':-0.00104674)'':0.00471677)'':-0.0034013)'':-0.00113201)'':0.00373527)'':0.00126841)'':0.000973505)'':-0.00149375)'':0.00207886)'':-0.000142875)'':0.00279558)'':0.00899611)'':0.00190852)'':-0.00455372)'':0.0111992)'':-0.0108315)'':-0.00116258)'':0.0108399)'':-0.00433581)'':-0.00532196)'':0.00232198)'':0.00348535)'':-0.000603427)'':0.021451)'':-0.00107888)'':-0.0102806)'':0.0185002)'':0.0591335)'':0.018915)'':0.0162733)'':0.00132128,((('PDT000425522.1':0.148569,((('PDT000694465.1':0.0338688,'PDT000661335.1':0.0336541)'':0.0166542,('PDT000443050.1':0.0456499,('PDT000442384.1':0.035416,('PDT000443080.1':0.00701294,'PDT000443087.1':0.00813256)'':0.0243558)'':0.00918186)'':0.0159316)'':0.0108699,('PDT000713385.1':0.0546226,(('PDT000307419.2':0.0404242,'PDT000248827.1':0.0382916)'':0.00201224,(('PDT000042546.1':0.0330871,'PDT000249755.1':0.0349507)'':0.00252767,('PDT000120858.2':0.0186508,('PDT000415825.1':0.0160189,('PDT000190973.2':0.0113774,'PDT000103061.1':0.0084954)'':0.00503491)'':0.00467374)'':0.0173589)'':0.0062918)'':0.0126775)'':0.00893898)'':0.0768571)'':-0.0259986,((('PDT000600633.1':0.0822975,(('PDT000604851.1':0.0390359,'PDT000395032.1':0.0342295)'':0.0325851,(('PDT000718879.1':0.00279555,'PDT000228214.1':0.00299403)'':0.0533197,('PDT000354222.1':0.0309775,('PDT000482846.1':0.00117856,'PDT000482847.1':0.0012188)'':0.0340489)'':0.0189453)'':0.00621259)'':0.00552703)'':0.0326036,((('PDT000444064.1':0.060204,'PDT000444190.1':0.065269)'':0.00342017,('PDT000251478.1':0.0682726,('PDT000443067.1':0.03986,('PDT000213270.2':0.0313604,'PDT000443989.1':0.0312855)'':0.00880629)'':0.0277762)'':0.00024063)'':0.0106109,(('PDT000713678.1':0.034882,'PDT000249949.1':0.0361589)'':0.0331867,('PDT000034431.1':0.0146741,(('PDT000033991.1':0.00290244,'PDT000034433.1':0.00268248)'':0.00485047,('PDT000034372.1':0.00692202,('PDT000034364.1':0.00530012,'PDT000034365.1':0.0046531)'':0.00282822)'':0.000206231)'':0.00988975)'':0.0494581)'':0.00864674)'':0.0255461)'':0.00339356,(((('PDT000636331.1':0.0502081,'PDT000017779.2':0.0510659)'':0.0024858,((('PDT000423851.1':0.0270604,'PDT000636287.1':0.0282409)'':0.00530491,('PDT000553673.1':0.0126142,('PDT000443812.1':0.00567203,'PDT000444041.1':0.00411622)'':0.00741448)'':0.0192667)'':0.00752601,('PDT000636251.1':0.044391,(('PDT000555596.1':0.02754,'PDT000345826.1':0.0273016)'':0.0111642,('PDT000520735.1':0.0369537,('PDT000713987.1':0.0312102,('PDT000535112.1':0.029302,('PDT000160692.2':0.0169031,('PDT000713840.1':0.00785386,'PDT000380167.1':0.0100405)'':0.00757078)'':0.00895924)'':0.00297084)'':0.00201971)'':0.0026222)'':0.00973843)'':-0.00621592)'':0.0139472)'':0.0118552,((('PDT000354163.1':0.0398586,('PDT000494536.1':0.0167158,'PDT000494550.1':0.012693)'':0.0296938)'':0.0170575,(('PDT000714068.1':0.051009,('PDT000362417.1':0.0243591,(('PDT000014097.3':0.0232524,('PDT000372787.1':0.0182834,'PDT000498777.1':0.0155643)'':0.00550452)'':0.000773471,('PDT000421823.1':0.0221453,('PDT000500566.1':0.0161215,'PDT000536979.1':0.0201916)'':0.00210485)'':0.00263636)'':0.00314067)'':0.0262549)'':0.00254825,(('PDT000557307.1':0.0332695,('PDT000279223.1':0.0168811,('PDT000557295.1':0.0147159,'PDT000719161.1':0.0142367)'':0.0051203)'':0.00949081)'':0.0215813,(('PDT000382806.1':0.0360735,'PDT000718868.1':0.0391152)'':0.0068406,(('PDT000279225.1':0.049379,('PDT000683481.1':0.0338164,'PDT000362494.1':0.0263332)'':0.0163239)'':-0.000335347,(('PDT000236848.1':0.0328512,'PDT000346740.1':0.0303098)'':0.00448542,('PDT000010610.2':0.0244169,('PDT000213233.2':0.014324,'PDT000279228.1':0.0164145)'':0.0128482)'':0.0105302)'':0.0023903)'':0.00758511)'':0.000115002)'':0.0021889)'':0.0094469)'':-0.00260787,(('PDT000664804.1':0.071981,'PDT000103100.1':0.050038)'':0.0164029,(('PDT000015106.2':0.0504466,('PDT000015105.2':0.0454802,('PDT000015108.2':0.0305432,(('PDT000015099.2':0.0105844,'PDT000015111.2':0.0114525)'':0.00440548,('PDT000015104.2':0.0260164,(('PDT000015107.2':0.0109967,('PDT000015100.2':0.00575092,'PDT000015101.2':0.00558898)'':0.00374147)'':0.00213088,('PDT000015102.2':0.013262,('PDT000015109.2':0.00718143,'PDT000015110.2':0.00654447)'':0.00393315)'':0.000825153)'':0.00769527)'':-0.000888219)'':0.00874616)'':0.00627622)'':0.00539029)'':0.00103395,(('PDT000327448.1':0.0245133,(('PDT000713773.1':0.00124859,'PDT000713946.1':0.00134933)'':0.0112649,('PDT000415838.1':0.00586749,'PDT000316459.1':0.00442941)'':0.00909174)'':0.0114697)'':0.00567155,(('PDT000535061.1':0.0339444,'PDT000616181.1':0.039953)'':0.000312258,(('PDT000175884.2':0.0242577,('PDT000135948.2':0.0191852,('PDT000694427.1':0.0179887,('PDT000713275.1':0.00736177,'PDT000714073.1':0.00936963)'':0.00873384)'':0.00160465)'':0.00507067)'':0.00274873,('PDT000713354.1':0.0280313,('PDT000397697.1':0.0260763,('PDT000103080.1':0.0138078,('PDT000713696.1':0.00504328,('PDT000713382.1':0.00164662,'PDT000713876.1':0.00224715)'':0.00301331)'':0.00619914)'':0.0120961)'':0.00383842)'':-0.0042809)'':0.0101273)'':0.00104524)'':0.00897496)'':0.0243263)'':-0.0113803)'':0.00795834)'':0.0279255,((((('PDT000017647.2':0.0223298,('PDT000438167.1':0.00426257,'PDT000444139.1':0.00353042)'':0.0211279)'':0.0302127,('PDT000443868.1':0.0444928,(('PDT000400916.1':0.0310091,('PDT000713413.1':0.0161003,'PDT000042639.1':0.0130734)'':0.0121879)'':-0.000894602,('PDT000535282.1':0.0219021,('PDT000661759.1':0.020803,('PDT000535021.1':0.0171559,('PDT000615992.1':0.0135507,('PDT000661437.1':0.00717381,'PDT000661715.1':0.00794909)'':0.00578863)'':0.00275659)'':0.00203899)'':0.00763339)'':0.00166417)'':0.0194108)'':0.00490794)'':0.00272439,(('PDT000716877.1':0.0372739,('PDT000236852.1':0.0308167,('PDT000714049.1':0.0248299,(('PDT000213200.2':0.00957438,'PDT000713407.1':0.010941)'':0.000607735,('PDT000661435.1':0.00929293,'PDT000157305.1':0.00737377)'':0.00113359)'':0.0127044)'':0.00954498)'':0.00150388)'':-0.00448306,(('PDT000397745.1':0.012579,'PDT000397754.1':0.0118892)'':0.01699,('PDT000095293.1':0.0278605,(('PDT000433222.1':0.0128157,'PDT000713424.1':0.0143076)'':0.00384466,('PDT000535026.1':0.0195971,(('PDT000713501.1':0.0136118,'PDT000157310.1':0.014923)'':0.000396184,('PDT000661512.1':0.00966154,('PDT000249758.1':0.00767319,'PDT000017730.2':0.00598581)'':0.00207396)'':0.00281953)'':0.00434737)'':-0.00181537)'':0.011674)'':-0.00194673)'':0.00763153)'':0.0180706)'':0.014446,((('PDT000713517.1':0.0189436,('PDT000694507.1':0.0191266,('PDT000042611.1':0.0111163,('PDT000136131.2':0.00892398,'PDT000694500.1':0.00853292)'':0.00158296)'':0.00563592)'':0.00586952)'':0.029784,(('PDT000097040.1':0.0320714,'PDT000428850.1':0.0302408)'':0.0319744,((('PDT000423863.1':0.0142213,'PDT000433456.1':0.0138059)'':0.0121619,('PDT000588061.1':0.00956761,('PDT000135944.2':0.00686176,'PDT000661515.1':0.00666524)'':0.00173264)'':0.00965521)'':0.02707,(('PDT000322812.1':0.0277685,'PDT000713350.1':0.0250762)'':0.00792313,('PDT000713788.1':0.02967,('PDT000103002.1':0.0297585,(('PDT000251969.1':0.0223762,'PDT000346734.1':0.0212433)'':0.00524078,('PDT000251982.1':0.0145853,('PDT000713654.1':0.0128231,'PDT000032191.1':0.0127594)'':0.00283935)'':0.0106252)'':0.00534025)'':0.0033947)'':0.00393695)'':0.00913869)'':0.00724726)'':-0.000489856)'':0.010623,(('PDT000433462.1':0.0282004,('PDT000555614.1':0.0223027,'PDT000588023.1':0.0248095)'':0.00225065)'':0.0274356,((('PDT000661371.1':0.0493922,('PDT000443054.1':0.0341556,'PDT000713979.1':0.0330558)'':0.0105619)'':-0.00346821,('PDT000423914.1':0.0432444,(('PDT000694539.1':0.0252649,'PDT000713351.1':0.0229906)'':0.00156086,('PDT000520714.1':0.0243464,(('PDT000714088.1':0.01512,('PDT000015309.2':0.00556254,'PDT000017687.2':0.00518586)'':0.00985035)'':0.000296987,('PDT000519442.1':0.0147119,('PDT000160342.2':0.017005,('PDT000636374.1':0.015441,('PDT000156541.2':0.00921328,('PDT000713816.1':0.00443321,('PDT000155730.2':0.00424966,('PDT000161919.2':0.00320495,'PDT000688572.1':0.00357606)'':0.000832697)'':0.000828097)'':0.00191286)'':0.004752)'':0.00467899)'':0.00135713)'':-0.00135452)'':0.00705432)'':3.91598e-05)'':0.0133937)'':0.0035166)'':0.00172523,((('PDT000520688.1':0.038309,'PDT000520834.1':0.0317503)'':0.00638739,(('PDT000713536.1':0.0244144,('PDT000176032.2':0.0261152,'PDT000720324.1':0.0275472)'':0.00182163)'':0.0121339,(('PDT000713833.1':0.0135777,'PDT000661470.1':0.0150759)'':0.0153957,('PDT000694416.1':0.0322762,('PDT000713355.1':0.0311496,('PDT000636224.1':0.0215691,('PDT000135980.2':0.0196006,('PDT000661366.1':0.0247551,('PDT000661465.1':0.012805,('PDT000132383.2':0.0106895,('PDT000425794.1':0.00631821,('PDT000713207.1':0.00426544,('PDT000175968.2':0.00188667,'PDT000017735.2':0.00190878)'':0.00220885)'':0.0008302)'':0.00228901)'':0.00227959)'':0.00587085)'':0.00250735)'':0.00117818)'':0.00672952)'':0.00247793)'':0.00170462)'':0.00164145)'':0.00265291)'':0.00228934,(('PDT000426642.1':0.0470275,(('PDT000534925.1':0.0250159,('PDT000443038.1':0.0125113,'PDT000444087.1':0.0146094)'':0.0113335)'':0.011066,(('PDT000535042.1':0.0166905,'PDT000535345.1':0.0176531)'':0.0133495,('PDT000373311.1':0.0145387,('PDT000433682.1':0.00836013,('PDT000433602.1':0.00300073,'PDT000619078.1':0.00278308)'':0.00444472)'':0.00402131)'':0.0184636)'':0.0034294)'':0.00598588)'':0.00297504,(('PDT000249796.1':0.0265951,('PDT000017635.2':0.0276197,('PDT000694387.1':0.019342,'PDT000249770.1':0.018077)'':0.00681426)'':0.00507852)'':0.00133496,('PDT000535041.1':0.0360807,(('PDT000661701.1':0.0162654,('PDT000659726.1':0.0082271,'PDT000713828.1':0.0082788)'':0.00615908)'':0.00518738,('PDT000135957.2':0.0220884,('PDT000535027.1':0.0185066,('PDT000714134.1':0.00956571,'PDT000252574.1':0.00922159)'':0.00624939)'':0.00306888)'':-0.000631005)'':0.00917255)'':0.00817539)'':0.00611102)'':0.00117778)'':0.00118067)'':0.0150651)'':0.00460523)'':0.0061885)'':0.00838111,(((('PDT000557097.1':0.0377772,'PDT000604334.1':0.0319971)'':0.0248371,(('PDT000390623.1':0.0417784,'PDT000623230.1':0.043799)'':0.00395522,('PDT000557310.1':0.0516923,(('PDT000556947.1':0.0368448,'PDT000208670.1':0.031357)'':0.00343387,('PDT000405287.1':0.0330567,(('PDT000604292.1':0.0292328,('PDT000499429.1':0.0177408,('PDT000481206.1':0.0129634,('PDT000470000.1':0.00595414,'PDT000492224.1':0.00832586)'':0.00454186)'':0.00458657)'':0.00706106)'':-0.000214628,(('PDT000408993.1':0.0199311,'PDT000430693.1':0.0188438)'':0.00428433,('PDT000498773.1':0.0208008,('PDT000718901.1':0.0237658,('PDT000669514.1':0.0138759,('PDT000619617.1':0.0104278,('PDT000257686.2':0.00345013,'PDT000258460.2':0.00413849)'':0.00599522)'':0.00516369)'':0.00462889)'':0.000496391)'':0.00128448)'':0.00173551)'':0.00818311)'':0.000290903)'':0.0084602)'':0.00611901)'':0.00992151)'':0.00973666,(((('PDT000557099.1':0.0111709,'PDT000557144.1':0.0110887)'':0.0302681,('PDT000391637.1':0.0295878,(('PDT000719225.1':0.0129884,('PDT000637569.1':0.0112454,'PDT000719915.1':0.00950312)'':0.00745949)'':0.00420503,('PDT000719920.1':0.0201495,('PDT000251838.1':0.0212785,'PDT000346810.1':0.0199748)'':0.000512268)'':0.00116836)'':0.00850051)'':0.00419867)'':0.00117859,(('PDT000502193.1':0.0338698,('PDT000707320.1':0.0235339,('PDT000135949.2':0.0175637,'PDT000522683.1':0.0230117)'':0.0030103)'':0.00411284)'':0.00221402,('PDT000623225.1':0.0306028,('PDT000132556.1':0.0304309,(('PDT000499494.1':0.0116423,'PDT000707295.1':0.0157085)'':0.00635229,('PDT000687463.1':0.0229243,('PDT000171788.1':0.0132483,'PDT000017921.2':0.0146342)'':0.00554666)'':0.000265256)'':0.00544142)'':0.00228884)'':-0.000722587)'':0.00461151)'':0.0138889,((('PDT000382841.1':0.03459,'PDT000631681.1':0.0333929)'':0.00776575,('PDT000359707.1':0.0448755,('PDT000143980.3':0.0335219,'PDT000717020.1':0.0348583)'':0.00669804)'':0.00218811)'':0.000285677,(('PDT000425689.1':0.0614417,(('PDT000713644.1':0.0246338,('PDT000135960.2':0.0152272,'PDT000017861.2':0.0172442)'':0.00575789)'':0.00510741,('PDT000251857.1':0.0341357,(('PDT000619577.1':0.00203464,'PDT000017758.2':0.00195857)'':0.0168202,('PDT000026733.1':0.0136522,'PDT000623223.1':0.0144821)'':0.00471628)'':0.0172761)'':-0.00611564)'':0.0193167)'':-0.0105382,('PDT000289900.1':0.0520602,(('PDT000501854.1':0.0238547,('PDT000535427.1':0.0197799,('PDT000427069.1':0.0196515,'PDT000017880.2':0.0188365)'':0.000948648)'':0.00304034)'':0.00212319,('PDT000017706.2':0.0194615,('PDT000250119.1':0.022419,('PDT000719892.1':0.0205401,('PDT000382810.1':0.0127683,'PDT000017806.2':0.0146602)'':0.00501461)'':0.00360517)'':0.00179051)'':0.000760113)'':0.0237462)'':-0.0141519)'':0.00633884)'':-0.000707033)'':0.0192585)'':0.00087364,((('PDT000372006.1':0.0331417,'PDT000252635.1':0.0321497)'':0.0646553,('PDT000252641.1':0.0786403,(('PDT000557096.1':0.0506851,('PDT000265738.2':0.036239,('PDT000354159.1':0.0259471,'PDT000357654.1':0.0328247)'':0.00734037)'':0.00953512)'':0.00285137,('PDT000694511.1':0.0415131,('PDT000619633.1':0.0442729,('PDT000694533.1':0.0218415,('PDT000346842.1':0.0160854,'PDT000501882.1':0.0148902)'':0.00704528)'':0.0166105)'':0.00596802)'':0.00408413)'':0.0317882)'':0.0092314)'':-0.0149823,(((('PDT000651915.1':0.039524,('PDT000409032.1':0.031827,'PDT000017863.2':0.0325107)'':0.00599988)'':0.00909818,('PDT000652006.1':0.0482504,(('PDT000651912.1':0.0309816,'PDT000252548.1':0.0329391)'':0.000789731,('PDT000672695.1':0.0297776,(('PDT000296184.2':0.0190083,'PDT000668377.1':0.0219931)'':0.00664041,('PDT000430766.1':0.0213717,'PDT000719192.1':0.0204549)'':0.00359834)'':0.00240865)'':0.00368879)'':0.0122885)'':0.00121294)'':0.0134428,((('PDT000687403.1':0.032615,('PDT000583869.1':0.014876,('PDT000409802.1':0.00831411,'PDT000249888.1':0.00681479)'':0.00709502)'':0.0142394)'':0.0121668,(('PDT000719027.1':0.0295848,'PDT000086325.1':0.0280586)'':0.0140274,('PDT000719203.1':0.0352762,(('PDT000188109.2':0.0213603,'PDT000502145.1':0.0214996)'':0.00465599,('PDT000502102.1':0.0182406,('PDT000069711.1':0.0107076,('PDT000265862.2':0.00934984,'PDT000069649.1':0.00778846)'':0.00203641)'':0.00688648)'':0.00848997)'':0.00716778)'':0.00477462)'':-0.0020472)'':0.011444,((('PDT000427056.1':0.0331856,'PDT000619620.1':0.0370957)'':0.00836554,(('PDT000017920.2':0.0283581,'PDT000019133.2':0.0252431)'':0.0161665,('PDT000290405.2':0.0358484,('PDT000557527.1':0.0289765,('PDT000619599.1':0.00754735,'PDT000661633.1':0.00808235)'':0.0187714)'':0.00721453)'':0.00131239)'':0.00159666)'':0.00584698,(('PDT000713231.1':0.0103698,'PDT000661729.1':0.00992316)'':0.0337589,(('PDT000645010.1':0.022132,('PDT000286933.2':0.0185098,'PDT000639543.1':0.0204577)'':0.00507767)'':0.0216434,('PDT000717313.1':0.0450498,((('PDT000719215.1':0.0348959,('PDT000719079.1':0.0303689,'PDT000719224.1':0.0297478)'':0.00372966)'':0.000814712,('PDT000687479.1':0.0285097,('PDT000460006.1':0.0261348,('PDT000252710.1':0.0202617,('PDT000583905.1':0.0160487,'PDT000718996.1':0.0162859)'':0.00692393)'':0.00336)'':0.00143855)'':0.00462361)'':4.5998e-05,('PDT000252729.1':0.0373979,(('PDT000719217.1':0.029367,'PDT000673699.1':0.029199)'':0.00132711,('PDT000668332.1':0.0297122,('PDT000547559.1':0.0238198,('PDT000351548.1':0.0279417,('PDT000568642.1':0.0151449,('PDT000109488.2':0.00942627,'PDT000042508.1':0.00903683)'':0.00460309)'':0.00848055)'':0.00139129)'':0.00331819)'':0.00509733)'':0.00450745)'':-0.00466405)'':0.00711475)'':0.00334747)'':0.00335155)'':0.00173528)'':0.00425058)'':0.00394703)'':0.00333521,(((('PDT000132559.1':0.0455892,('PDT000713218.1':0.0226273,'PDT000713268.1':0.0257835)'':0.0115994)'':0.00921916,(('PDT000673523.1':0.047221,('PDT000673235.1':0.0354029,('PDT000426579.1':0.0314155,'PDT000619568.1':0.0241567)'':0.0108116)'':0.00685997)'':-0.00285104,('PDT000435644.1':0.0396692,('PDT000694523.1':0.0308474,('PDT000318971.1':0.0235916,('PDT000426770.1':0.0328202,('PDT000427018.1':0.0163646,('PDT000426137.1':0.0147286,('PDT000713257.1':0.00888311,'PDT000011728.2':0.00866769)'':0.00300698)'':0.00417521)'':0.00694185)'':0.00257335)'':0.00156501)'':0.0106806)'':-0.00102757)'':0.00613972)'':0.00405128,(('PDT000042486.1':0.0715406,('PDT000501925.1':0.0358017,('PDT000415606.1':0.020823,'PDT000719030.1':0.027651)'':0.0124818)'':0.0164413)'':0.00424062,((('PDT000500561.1':0.0404762,('PDT000716871.1':0.0222038,('PDT000574927.1':0.0153998,'PDT000346701.1':0.0156225)'':0.00500291)'':0.0159334)'':0.00311173,('PDT000714034.1':0.0461469,('PDT000622955.1':0.0383067,('PDT000498726.1':0.0273905,('PDT000487589.1':0.021705,('PDT000427827.1':0.0119432,'PDT000619650.1':0.0096115)'':0.00922881)'':0.00364404)'':0.011266)'':0.00401888)'':0.000491659)'':0.00524244,(('PDT000618737.1':0.0412437,'PDT000011729.2':0.0431637)'':0.00970013,(('PDT000713667.1':0.0256857,'PDT000346719.1':0.0300559)'':0.0164917,('PDT000661757.1':0.0393786,('PDT000061123.2':0.0376161,(('PDT000661643.1':0.0265902,('PDT000135943.2':0.0194372,'PDT000397860.1':0.0202726)'':0.00468627)'':0.00260583,('PDT000270464.1':0.0268518,('PDT000603058.1':0.0228162,('PDT000213247.2':0.0104839,'PDT000157306.1':0.014237)'':0.010574)'':0.00367112)'':0.0012745)'':0.00609817)'':0.00496909)'':0.00183604)'':0.00348183)'':-0.00120569)'':0.00807718)'':-0.00431796)'':0.00568727,(('PDT000214896.1':0.0245934,(('PDT000360068.1':0.0206743,'PDT000556946.1':0.0158758)'':0.000333495,('PDT000651987.1':0.0181857,('PDT000448217.1':0.014342,(('PDT000362513.1':0.00776791,('PDT000385062.1':0.00380164,'PDT000557143.1':0.00357623)'':0.00137076)'':0.00429939,('PDT000103001.1':0.0157636,('PDT000042615.1':0.0111242,('PDT000713641.1':0.00526006,('PDT000382828.1':0.00270097,'PDT000536939.1':0.00219609)'':0.00350902)'':0.00372713)'':0.00103025)'':-0.00108409)'':0.00331893)'':0.000897435)'':0.00241852)'':0.00341757)'':0.0104997,((('PDT000455343.1':0.0263392,'PDT000718963.1':0.0278961)'':0.00681644,(('PDT000716808.1':0.019514,'PDT000661505.1':0.0188457)'':0.00342642,('PDT000604269.1':0.0174754,('PDT000360076.1':0.0103013,'PDT000683166.1':0.0130101)'':0.0087154)'':0.00165403)'':0.00850835)'':0.0123563,(('PDT000397737.1':0.0143956,'PDT000233061.1':0.0122007)'':0.02552,(('PDT000391629.1':0.0178798,'PDT000673580.1':0.0182078)'':0.0157326,(('PDT000719260.1':0.0342252,(('PDT000344858.1':0.0130079,'PDT000713331.1':0.0112098)'':0.0124754,('PDT000346738.1':0.0267894,('PDT000713995.1':0.0185238,(('PDT000188088.2':0.0119811,'PDT000408017.1':0.012238)'':0.000789968,('PDT000572839.1':0.00727587,('PDT000156725.2':0.00672767,('PDT000042492.1':0.00291248,'PDT000125977.1':0.00288404)'':0.00195341)'':0.0044627)'':0.00323972)'':0.00454892)'':0.00544182)'':-0.000761972)'':0.00713347)'':-0.00695931,('PDT000448214.1':0.0341835,('PDT000685888.1':0.0289574,('PDT000042573.1':0.0247274,(('PDT000167950.2':0.0100862,'PDT000103084.1':0.0106351)'':0.00723081,('PDT000188076.2':0.0156964,('PDT000520973.1':0.0187942,(('PDT000382822.1':0.00783471,'PDT000716927.1':0.00838899)'':0.00269028,(('PDT000083197.2':0.00347415,'PDT000408016.1':0.00290862)'':0.00315371,('PDT000477478.1':0.0087189,(('PDT000713527.1':0.00208906,'PDT000103069.1':0.00230303)'':0.00130978,('PDT000713479.1':0.00249251,('PDT000409798.1':0.00207782,('PDT000688006.1':0.000975655,'PDT000233098.1':0.000922635)'':0.00141985)'':0.0004617)'':0.00132874)'':0.00233995)'':-0.000305051)'':0.0020866)'':0.00450044)'':0.00283033)'':-0.00159931)'':0.00616646)'':0.00182049)'':0.00135493)'':3.66365e-05)'':0.0108707)'':0.00270689)'':-0.00111478)'':-0.00207375)'':0.0185122)'':0.00260614)'':0.0156067)'':-0.00980345)'':0.0129824)'':0.00895645)'':0.0213135)'':0.00892875)'':0.0503313,((('PDT000425919.1':0.0347909,'PDT000325634.1':0.0351188)'':0.127477,((('PDT000293491.1':0.015041,'PDT000293544.1':0.0122118)'':0.082357,('PDT000205432.1':0.0827441,('PDT000687921.1':0.0931179,('PDT000719039.1':0.0754861,(('PDT000548634.2':0.0357624,('PDT000604371.1':0.0142764,'PDT000042556.1':0.0141316)'':0.0279877)'':0.0261356,(('PDT000305446.2':0.0593579,'PDT000235713.3':0.0550471)'':0.00708723,((('PDT000661791.1':0.0364812,('PDT000494515.1':0.0101722,'PDT000494525.1':0.00850732)'':0.0321538)'':0.00774924,((('PDT000372784.1':0.0128106,'PDT000713967.1':0.0185827)'':0.0028749,('PDT000343407.1':0.00759848,('PDT000252477.1':0.00374084,'PDT000343401.1':0.00354604)'':0.00567298)'':0.00836591)'':0.0170989,('PDT000718819.1':0.0162809,('PDT000719173.1':0.0117082,('PDT000718892.1':0.0111311,('PDT000467010.1':0.00072016,'PDT000251509.1':0.00097933)'':0.00908377)'':0.00229681)'':0.000910846)'':0.0209097)'':0.0119881)'':-0.00151118,('PDT000112495.3':0.0618057,(('PDT000522630.1':0.0458184,('PDT000610886.1':0.0346242,('PDT000252636.1':0.00137903,'PDT000477878.1':0.00141901)'':0.039148)'':0.0060951)'':0.000444733,('PDT000131903.2':0.0495848,(('PDT000430806.1':0.0144364,('PDT000346785.1':0.00130408,'PDT000346787.1':0.00149396)'':0.0163938)'':0.0150688,('PDT000719211.1':0.0271013,('PDT000427571.1':0.0239424,('PDT000661703.1':0.016614,('PDT000575788.1':0.0126129,'PDT000210717.1':0.0117628)'':0.00540741)'':0.00472131)'':0.00894169)'':-0.00255785)'':0.0138131)'':0.00172226)'':0.00900677)'':-0.00307318)'':0.0107659)'':0.00146236)'':0.00671718)'':0.00936135)'':0.00187091)'':0.00449134)'':-0.00372192,(('PDT000619944.1':0.0544766,('PDT000042622.1':0.032851,('PDT000710131.1':0.0257677,((('PDT000397577.1':0.0187591,'PDT000713951.1':0.0178598)'':0.00291371,('PDT000397847.1':0.0117609,'PDT000302400.1':0.0118408)'':0.00725746)'':0.004461,('PDT000520752.1':0.0245569,('PDT000356182.1':0.0224414,('PDT000618449.1':0.015547,'PDT000718454.1':0.0177764)'':0.00127287)'':0.0043723)'':0.00181201)'':0.00213555)'':0.00427674)'':0.0276587)'':0.00179136,(('PDT000521071.1':0.0601089,('PDT000426297.1':0.0505161,('PDT000713574.1':0.0294211,('PDT000426731.1':0.0241151,'PDT000427574.1':0.0258311)'':0.00436484)'':0.012921)'':0.00852957)'':-0.00398451,((('PDT000619628.1':0.0280534,('PDT000017815.2':0.0280286,('PDT000017720.2':0.0182218,'PDT000017877.2':0.017847)'':0.00674545)'':0.00493624)'':0.00982413,('PDT000468260.1':0.0414158,(('PDT000132251.2':0.0294598,('PDT000713717.1':0.0276913,'PDT000717159.1':0.02439)'':0.00358276)'':0.00793802,(('PDT000713604.1':0.0353875,'PDT000713647.1':0.0291885)'':0.00547878,('PDT000713971.1':0.0394396,('PDT000668327.1':0.0305569,('PDT000147775.2':0.0248797,('PDT000619645.1':0.0148093,'PDT000038893.1':0.0137678)'':0.0117302)'':0.00303221)'':0.00671669)'':0.00028171)'':0.00304461)'':0.0045881)'':-0.00117935)'':0.000186507,(('PDT000357012.1':0.0469659,'PDT000123339.1':0.0354596)'':0.0082133,('PDT000619684.1':0.0532746,('PDT000426809.1':0.0433387,((('PDT000672262.1':0.0293601,'PDT000427529.1':0.0321012)'':0.00412199,('PDT000714121.1':0.0308359,('PDT000156806.2':0.024474,('PDT000716178.1':0.0190886,'PDT000716838.1':0.0219224)'':0.00684109)'':0.00114721)'':0.0129514)'':0.00324855,((('PDT000136153.2':0.0287555,'PDT000713325.1':0.028736)'':0.0101857,('PDT000299940.3':0.0311769,('PDT000604217.1':0.0246039,'PDT000713706.1':0.0305638)'':0.00483546)'':0.00349006)'':0.00269436,('PDT000042470.1':0.0404853,('PDT000713254.1':0.0365844,('PDT000714153.1':0.0224201,('PDT000720326.1':0.0174332,('PDT000526158.1':0.0163534,'PDT000716994.1':0.0139496)'':0.00401072)'':0.00569929)'':0.00886344)'':0.00615081)'':-0.000687652)'':0.0018047)'':0.00241713)'':0.0052966)'':-0.000363522)'':-0.00521198)'':0.00373454)'':0.013427)'':0.0203522)'':0.0823963)'':0.00535163,((('PDT000275041.2':0.130111,('PDT000481207.1':0.0635486,(('PDT000455345.1':0.0230902,'PDT000532769.1':0.0238831)'':0.00644625,('PDT000694445.1':0.0279481,('PDT000216014.1':0.00190642,'PDT000216017.1':0.00169178)'':0.0231259)'':0.00390377)'':0.0312215)'':0.0577759)'':0.0441778,(((((('PDT000719218.1':0.0245132,'PDT000502597.1':0.025638)'':0.00773127,('PDT000017869.2':0.0295796,('PDT000251998.1':0.0264277,('PDT000137843.2':0.0259483,'PDT000672261.1':0.0257303)'':0.000757436)'':0.00275178)'':0.00535374)'':0.000364396,('PDT000354160.1':0.0399012,(('PDT000713599.1':0.0223006,'PDT000713627.1':0.0238004)'':0.00553671,(('PDT000120871.2':0.0136808,'PDT000137874.2':0.0141548)'':0.00403194,('PDT000713807.1':0.0158227,('PDT000713399.1':0.0147302,('PDT000619567.1':0.0123491,'PDT000713567.1':0.0116794)'':0.000585552)'':0.00160967)'':0.0037276)'':0.0105693)'':0.012524)'':-0.00907635)'':0.0010788,(('PDT000713380.1':0.0324469,('PDT000137847.2':0.0199628,('PDT000011739.2':0.0172091,('PDT000400921.1':0.0122499,('PDT000017746.2':0.00700904,'PDT000400917.1':0.00834336)'':0.0033043)'':0.00664702)'':0.00610708)'':0.0116719)'':-0.00354601,(('PDT000672263.1':0.015125,'PDT000713239.1':0.015172)'':0.0127623,(('PDT000713742.1':0.0194264,'PDT000007695.2':0.0189821)'':0.000599627,(('PDT000275038.4':0.011877,('PDT000262732.1':0.00701771,'PDT000011722.2':0.00718429)'':0.00547575)'':0.0060022,('PDT000421869.1':0.0180713,('PDT000354210.1':0.0150199,('PDT000017878.2':0.0116669,('PDT000017855.2':0.0120812,('PDT000017896.2':0.00786031,('PDT000713650.1':0.00559287,'PDT000017892.2':0.00668013)'':0.00151384)'':0.00248192)'':0.00107564)'':0.00081674)'':0.00504945)'':-0.00104971)'':0.00254898)'':0.00513573)'':0.00356709)'':0.00859095)'':0.0195985,((('PDT000396815.1':0.0585273,'PDT000716158.1':0.0553497)'':0.000887731,(('PDT000673531.1':0.024659,'PDT000354186.1':0.026952)'':0.0146236,('PDT000382805.1':0.0422708,(('PDT000557080.1':0.0306144,'PDT000714081.1':0.0284147)'':0.00333117,(('PDT000409787.1':0.0286189,'PDT000042543.1':0.0263251)'':0.00327024,(('PDT000583980.1':0.0141151,'PDT000252694.1':0.0101977)'':0.0160705,(('PDT000557234.1':0.0142875,'PDT000687475.1':0.0142161)'':0.0052416,('PDT000305455.2':0.0171344,('PDT000300832.2':0.0123038,'PDT000354161.1':0.0119656)'':0.00553836)'':0.00217555)'':0.00333501)'':0.0076286)'':0.00227023)'':0.0119749)'':-0.00594455)'':0.0134573)'':-0.00900234,((('PDT000259470.2':0.0355562,('PDT000042597.1':0.0318703,('PDT000455386.1':0.0251256,'PDT000500568.1':0.0248695)'':0.00677799)'':0.00430531)'':-0.000415862,(('PDT000502159.1':0.0264153,'PDT000502171.1':0.020703)'':0.0123114,('PDT000610247.1':0.026952,(('PDT000372790.1':0.0166637,'PDT000517620.1':0.0169257)'':0.00346667,('PDT000042662.1':0.0204356,('PDT000522650.1':0.00936661,('PDT000271751.2':0.0109543,'PDT000360070.1':0.00831851)'':0.00184309)'':0.00698214)'':0.00157203)'':0.00462553)'':0.0114791)'':-0.0019787)'':0.00444489,(('PDT000687352.1':0.0436469,(('PDT000385048.1':0.0219835,'PDT000713394.1':0.0224873)'':0.00647518,('PDT000287706.2':0.0263381,('PDT000517634.1':0.0231445,('PDT000132261.2':0.0155493,(('PDT000713379.1':0.0146258,'PDT000042464.1':0.0126066)'':0.00245473,('PDT000382835.1':0.0157122,('PDT000360073.1':0.0129718,('PDT000710117.1':0.00402489,'PDT000718946.1':0.00434414)'':0.00682711)'':0.00181829)'':0.00127989)'':0.0010897)'':0.00486992)'':0.00437848)'':0.00331497)'':0.0133404)'':-0.00494679,(('PDT000713745.1':0.0291981,(('PDT000425870.1':0.0126047,'PDT000713824.1':0.0107941)'':0.00426567,('PDT000713342.1':0.012826,'PDT000661306.1':0.0117625)'':0.00503598)'':0.00912241)'':0.000457168,(('PDT000717385.1':0.0266247,('PDT000212505.2':0.019438,'PDT000415610.1':0.0239326)'':0.00316513)'':0.00151616,(('PDT000619657.1':0.0319221,(('PDT000555702.1':0.0141353,'PDT000713326.1':0.0138015)'':0.00467551,('PDT000687480.1':0.0196023,'PDT000713850.1':0.0192183)'':0.00169172)'':0.00628295)'':-0.0013771,(('PDT000430716.1':0.0174069,'PDT000604265.1':0.0176999)'':0.00443028,(('PDT000668317.1':0.0264652,('PDT000661596.1':0.0144575,'PDT000011727.2':0.013518)'':0.008413)'':-0.000266829,(('PDT000713907.1':0.00348487,'PDT000623220.1':0.00349211)'':0.0151369,('PDT000604304.1':0.0220765,('PDT000619636.1':0.0166651,('PDT000093006.2':0.0164825,('PDT000672271.1':0.0156084,(('PDT000619652.1':0.00734442,('PDT000713860.1':0.0042376,'PDT000714018.1':0.00412893)'':0.00388172)'':0.00110327,('PDT000502545.1':0.00470214,('PDT000668348.1':0.00374442,'PDT000713386.1':0.00362977)'':0.00126748)'':0.00240598)'':0.00380298)'':0.00115051)'':0.00213417)'':0.0039026)'':-0.00193316)'':0.00515952)'':0.00270881)'':-0.000585007)'':0.00138612)'':0.00382131)'':0.00681021)'':0.0028616)'':0.007111)'':0.0136813)'':0.0112622,((('PDT000502105.1':0.0438753,('PDT000293515.1':0.03413,(('PDT000103082.1':0.019174,'PDT000205434.1':0.0173094)'':0.00831731,('PDT000372054.1':0.0207723,'PDT000486141.1':0.0234694)'':0.00432984)'':0.00569179)'':0.00398135)'':0.0211039,('PDT000155758.2':0.0584834,('PDT000520776.1':0.0370641,('PDT000661331.1':0.0347365,('PDT000694515.1':0.0269431,('PDT000427519.1':0.024232,('PDT000270461.1':0.0234808,('PDT000543320.1':0.0128515,'PDT000011726.2':0.0123829)'':0.00667887)'':0.00213378)'':0.00277193)'':0.00676887)'':0.00435672)'':0.0192932)'':0.00650058)'':0.0238929,((((('PDT000397700.1':0.0373486,'PDT000687414.1':0.0356829)'':0.0114542,(('PDT000415668.1':0.0274194,'PDT000568675.1':0.0257565)'':0.000538197,('PDT000710123.1':0.0197426,'PDT000017826.2':0.0156284)'':0.00473448)'':0.0210047)'':0.0142428,((('PDT000093004.2':0.0275863,'PDT000412505.1':0.0268667)'':0.0142261,('PDT000718960.1':0.0283537,'PDT000719208.1':0.0309915)'':0.0121563)'':0.0095674,(('PDT000637674.1':0.0276782,'PDT000252586.1':0.0260873)'':0.0138401,('PDT000266048.2':0.0440524,('PDT000200968.2':0.0205156,'PDT000598384.1':0.0197706)'':0.0215496)'':0.00368961)'':0.00954598)'':0.0076604)'':0.00173563,(((('PDT000321798.1':0.0198502,'PDT000583907.1':0.020876)'':0.0160148,('PDT000415663.1':0.0276268,'PDT000678648.1':0.0248757)'':0.00310966)'':0.00757773,(('PDT000265870.2':0.021794,('PDT000651855.1':0.0197614,('PDT000713959.1':0.00961901,'PDT000623232.1':0.0101094)'':0.00804928)'':0.00392541)'':0.00523705,('PDT000197973.2':0.0215656,('PDT000491862.1':0.010485,('PDT000397734.1':0.0053541,'PDT000713409.1':0.0060016)'':0.00592199)'':0.00736358)'':0.00703358)'':0.0069017)'':0.0102671,(('PDT000718905.1':0.0440458,('PDT000382816.1':0.0203437,('PDT000427032.1':0.0216988,('PDT000042462.1':0.0183149,('PDT000361013.1':0.0164757,'PDT000477884.1':0.0177014)'':0.00239414)'':0.00319872)'':0.00488419)'':0.0139476)'':0.000848604,(('PDT000360082.1':0.0383426,('PDT000619099.1':0.0249647,'PDT000636328.1':0.0208592)'':0.0162253)'':0.00366564,(('PDT000713213.1':0.0251436,'PDT000713783.1':0.0259777)'':0.0119363,(('PDT000160999.2':0.0178877,'PDT000357631.1':0.0223953)'':0.014243,(('PDT000281740.2':0.0267211,('PDT000502181.1':0.0214921,'PDT000357596.1':0.0206463)'':0.00663118)'':0.00142432,('PDT000619631.1':0.0332274,('PDT000017662.2':0.0256492,('PDT000713811.1':0.023435,('PDT000318965.1':0.010112,('PDT000427708.1':0.010106,'PDT000017851.2':0.00813202)'':0.000359516)'':0.0120371)'':0.00461439)'':0.00276939)'':0.00104805)'':0.0034572)'':0.00285527)'':0.00179527)'':-0.000740072)'':0.00400669)'':0.0165682)'':0.0117259,((('PDT000287697.2':0.0255455,'PDT000115432.2':0.0224286)'':0.029632,('PDT000661614.1':0.0557464,('PDT000713322.1':0.0304927,('PDT000372755.1':0.0300577,(('PDT000136140.2':0.0135031,'PDT000568624.1':0.014884)'':0.00367837,('PDT000574928.1':0.0174075,'PDT000252547.1':0.0148283)'':0.000912604)'':0.0121484)'':0.000517165)'':0.0163477)'':0.00682217)'':0.0231213,(('PDT000372595.1':0.0442059,('PDT000372168.1':0.0255008,'PDT000687472.1':0.0261291)'':0.0237266)'':0.04293,((((('PDT000714126.1':0.00652943,'PDT000019967.2':0.00729727)'':0.0367963,('PDT000685899.1':0.0267697,('PDT000602248.1':0.0124635,('PDT000685893.1':0.0124063,('PDT000397872.1':0.0106889,'PDT000572831.1':0.0109857)'':0.00102014)'':0.00460823)'':0.0073772)'':0.0224734)'':-0.00504485,(('PDT000661481.1':0.0309674,('PDT000015122.2':0.0202904,('PDT000673573.1':0.00857978,'PDT000042475.1':0.00609672)'':0.00877922)'':0.0081715)'':0.00722743,(('PDT000015117.2':0.01055,'PDT000015121.2':0.0135296)'':0.000594508,('PDT000015118.2':0.00992638,('PDT000015119.2':0.00953454,'PDT000015120.2':0.0102115)'':0.00153667)'':0.0018696)'':0.0158697)'':0.011369)'':0.0146208,(('PDT000501852.1':0.0415878,('PDT000502600.1':0.0229842,'PDT000501910.1':0.0225148)'':0.0139079)'':0.0131814,('PDT000351545.1':0.0658823,('PDT000414677.1':0.0441659,('PDT000006426.2':0.0409593,(('PDT000538922.2':0.0266726,'PDT000415791.1':0.0344792)'':0.00434503,('PDT000425672.1':0.0399768,(('PDT000397572.1':0.0148694,'PDT000042506.1':0.0162632)'':0.010523,('PDT000713837.1':0.0221738,('PDT000385051.1':0.0189773,('PDT000397927.1':0.0190338,('PDT000710116.1':0.0114722,'PDT000125948.1':0.0117444)'':0.00468114)'':0.00320798)'':0.0044019)'':0.000783823)'':0.0113311)'':-0.00396693)'':0.0073283)'':0.00119404)'':0.0118882)'':-0.0051074)'':0.00527879)'':0.00752831,(('PDT000251862.1':0.0852371,(('PDT000312640.2':0.0503692,'PDT000412165.1':0.0498508)'':0.00238124,(('PDT000412162.2':0.0247899,'PDT000469352.1':0.0252647)'':0.00659858,('PDT000258396.2':0.0330775,('PDT000486449.1':0.0218986,'PDT000669510.1':0.0211406)'':0.014202)'':0.00117539)'':0.0166287)'':0.0301569)'':-0.00769124,((('PDT000397797.1':0.0778061,('PDT000399646.1':0.0462257,(('PDT000011758.2':0.0288067,('PDT000167667.2':0.0211348,'PDT000661602.1':0.0194578)'':0.00531912)'':0.00385756,(('PDT000394937.1':0.0267808,('PDT000167677.2':0.0195957,'PDT000619605.1':0.0167787)'':0.00501751)'':0.00137326,('PDT000167660.2':0.021518,('PDT000125730.2':0.0109594,'PDT000600645.1':0.0142115)'':0.00693388)'':0.00384339)'':0.00809943)'':0.0123256)'':0.0341373)'':-0.0163727,(('PDT000346801.1':0.0241419,('PDT000502098.1':0.0121701,'PDT000327858.1':0.0116488)'':0.0125002)'':0.0318299,((('PDT000328102.2':0.0373813,'PDT000713748.1':0.0354149)'':0.0076258,('PDT000017786.2':0.0322946,('PDT000034441.1':0.00788566,'PDT000024460.2':0.00926634)'':0.0259805)'':0.00617223)'':0.00353775,(('PDT000618770.1':0.0497186,('PDT000607894.1':0.044597,('PDT000713261.1':0.0229564,'PDT000017883.2':0.0268052)'':0.0108222)'':0.00433725)'':0.00234159,(('PDT000017789.2':0.0198254,('PDT000299794.2':0.0179459,'PDT000713799.1':0.0148719)'':0.00279957)'':0.0223205,('PDT000619654.1':0.0429934,(('PDT000557260.1':0.0105818,'PDT000102992.1':0.00991531)'':0.0218695,('PDT000017906.2':0.0406332,('PDT000618442.1':0.0325743,((('PDT000719214.1':0.0215367,'PDT000017914.2':0.0223849)'':0.000914908,(('PDT000017907.2':0.0101533,'PDT000019135.2':0.0108368)'':0.0104893,('PDT000233090.1':0.0124888,('PDT000557340.1':0.009056,'PDT000588057.1':0.0077683)'':0.00328981)'':0.00786998)'':0.00240272)'':0.000979086,('PDT000017817.2':0.0247801,(('PDT000427181.1':0.00837635,'PDT000018289.2':0.00741465)'':0.00951366,('PDT000017833.2':0.0158269,(('PDT000673463.1':0.011278,'PDT000017814.2':0.012437)'':0.00133686,('PDT000619648.1':0.011219,('PDT000343860.1':0.0118311,'PDT000017923.2':0.00945809)'':0.00122246)'':0.00266785)'':0.00247397)'':0.00143269)'':0.00502028)'':-0.00115594)'':0.00679197)'':0.00191663)'':-0.00283254)'':0.00808013)'':0.00263759)'':0.00382252)'':0.00417814)'':0.00802352)'':0.00741601)'':-0.000496054,((('PDT000673440.1':0.0789444,(('PDT000362438.1':0.0557678,('PDT000719261.1':0.0361555,('PDT000713675.1':0.0187915,'PDT000362527.1':0.0225382)'':0.0160495)'':0.0100722)'':0.00441564,('PDT000626795.1':0.0587508,('PDT000654615.1':0.0547241,(('PDT000156832.2':0.0253649,('PDT000522762.1':0.021063,'PDT000719182.1':0.0197593)'':0.0063274)'':0.0108109,('PDT000678639.1':0.0381585,('PDT000293493.1':0.0382569,('PDT000625972.1':0.0271349,'PDT000011761.2':0.027665)'':0.00478596)'':0.00309909)'':0.00108872)'':0.00864108)'':0.011585)'':-0.00876862)'':0.0317218)'':-0.0225109,(('PDT000535202.1':0.0450404,('PDT000713849.1':0.0462548,('PDT000502142.1':0.0424303,'PDT000042505.1':0.0423642)'':0.00291077)'':0.00290561)'':0.0214196,((('PDT000233041.1':0.03632,('PDT000397813.1':0.0305872,'PDT000034437.1':0.0270505)'':0.00331587)'':0.00318362,(('PDT000382817.1':0.0110191,'PDT000433330.1':0.00914065)'':0.0208659,('PDT000017633.2':0.0308057,'PDT000424783.1':0.031022)'':0.00445396)'':0.00258645)'':0.00627403,(('PDT000661456.1':0.017632,'PDT000661537.1':0.0150069)'':0.0223932,('PDT000694452.1':0.0434715,('PDT000717006.1':0.0366098,('PDT000713638.1':0.0340038,(('PDT000602252.1':0.0137572,'PDT000713693.1':0.0174183)'':0.0142092,('PDT000427227.1':0.0241165,('PDT000427524.1':0.0283225,(('PDT000636282.1':0.0182473,'PDT000661777.1':0.0197149)'':0.00442855,('PDT000017910.2':0.0266234,('PDT000427225.1':0.0221091,('PDT000361005.1':0.0205439,('PDT000017859.2':0.015111,('PDT000019968.2':0.0103657,('PDT000319045.1':0.0063158,'PDT000017641.2':0.0068473)'':0.00421774)'':0.0034266)'':0.0032595)'':0.0022181)'':0.00430137)'':-0.00207286)'':0.00417315)'':0.000307287)'':-8.3283e-05)'':0.00265827)'':0.00219936)'':0.00546588)'':-0.00442726)'':0.00892669)'':0.0202459)'':-0.00328479)'':-0.00122827,((('PDT000360078.1':0.0326082,('PDT000249958.1':0.0217025,('PDT000714555.1':0.0131557,'PDT000501887.1':0.0141251)'':0.00760607)'':0.00684423)'':0.00619894,(('PDT000535053.1':0.030104,('PDT000042539.1':0.0281265,('PDT000042529.1':0.0223337,('PDT000042530.1':0.0185557,('PDT000397712.1':0.0140044,'PDT000397753.1':0.0149025)'':0.00626498)'':0.00291512)'':0.00170038)'':0.00356236)'':0.00149128,('PDT000535266.1':0.0170831,('PDT000534917.1':0.0176392,(('PDT000535125.1':0.0104862,('PDT000501923.1':0.00689239,'PDT000141112.1':0.00714861)'':0.00103955)'':0.00103437,('PDT000661599.1':0.00973344,('PDT000535015.1':0.00614861,('PDT000535050.1':0.00427863,('PDT000535010.1':0.00346929,'PDT000535274.1':0.00262827)'':0.00139325)'':0.00100203)'':0.00377546)'':0.00197873)'':0.00304621)'':0.00161638)'':0.0168477)'':0.00841776)'':0.0160804,('PDT000623219.1':0.105024,((('PDT000017684.2':0.00729062,'PDT000017759.2':0.00769738)'':0.0216036,('PDT000477468.1':0.036938,('PDT000604276.1':0.0306786,('PDT000228274.1':0.0295883,('PDT000093010.2':0.020926,(('PDT000619779.1':0.0162098,'PDT000672273.1':0.0171927)'':0.00578653,('PDT000397983.1':0.023711,(('PDT000358611.1':0.00734993,'PDT000716806.1':0.00843797)'':0.0122324,((('PDT000382856.1':0.00801151,'PDT000719888.1':0.00918359)'':0.00438489,('PDT000583871.1':0.0138404,('PDT000372781.1':0.0121609,'PDT000481214.1':0.0110442)'':0.00270124)'':0.000607327)'':0.00277686,('PDT000251534.1':0.0226317,('PDT000017839.2':0.0169637,(('PDT000719184.1':0.00992387,'PDT000252494.1':0.0100364)'':0.00489072,('PDT000678619.1':0.0143424,('PDT000619651.1':0.0108758,('PDT000557361.1':0.00546372,'PDT000669503.1':0.00558218)'':0.00509437)'':0.00249476)'':0.00247623)'':0.00249867)'':0.00269584)'':-0.00354905)'':0.0031363)'':0.00227836)'':-0.000809033)'':0.00331183)'':0.0030153)'':0.00147914)'':0.00136038)'':0.000775035)'':0.0251951,((('PDT000252538.1':0.0454924,('PDT000469983.1':0.0285671,'PDT000566047.1':0.0381522)'':0.0125759)'':0.00604465,('PDT000661801.1':0.0444084,('PDT000382809.1':0.0372242,('PDT000201776.1':0.018889,('PDT000103059.2':0.0211354,('PDT000672264.1':0.0172705,('PDT000713661.1':0.0138921,('PDT000332760.1':0.0132874,('PDT000589037.1':0.0106667,('PDT000176669.2':0.00557272,('PDT000251957.1':0.00237694,'PDT000252003.1':0.00211767)'':0.00383638)'':0.00333621)'':0.00472275)'':0.00167276)'':0.003385)'':0.00283857)'':0.0023298)'':0.0111404)'':0.00541158)'':0.0102052)'':0.00367101,((('PDT000685901.1':0.0535919,('PDT000343820.1':0.0436109,('PDT000409464.1':0.0292764,('PDT000610302.1':0.00708129,'PDT000228035.1':0.00796001)'':0.0194901)'':0.00974335)'':0.0173384)'':-0.00639367,(('PDT000694519.1':0.0360055,('PDT000481213.1':0.0229129,'PDT000017849.2':0.0240637)'':0.00763127)'':0.00682001,('PDT000042671.1':0.0359319,(('PDT000307433.2':0.0212644,('PDT000713884.1':0.0131399,'PDT000647170.1':0.0125594)'':0.00780939)'':0.00475002,('PDT000713467.1':0.027972,('PDT000160767.2':0.0231509,('PDT000135959.2':0.0212221,'PDT000011753.2':0.0246621)'':0.00176227)'':0.00161859)'':0.00140247)'':0.00896871)'':0.000306313)'':0.0127957)'':-0.00410794,('PDT000661829.1':0.0396212,(('PDT000427526.1':0.0305388,'PDT000046473.1':0.026427)'':0.00484562,('PDT000619612.1':0.0536446,((('PDT000409797.1':0.0202879,'PDT000685876.1':0.0283019)'':0.0104915,(('PDT000382826.1':0.0122909,'PDT000588048.1':0.0175361)'':0.0172873,('PDT000017805.2':0.0181407,('PDT000362462.1':0.0145017,('PDT000647168.1':0.00984518,'PDT000647171.1':0.00848702)'':0.00716455)'':0.000983662)'':0.0106577)'':0.00249249)'':0.0010016,(('PDT000019231.2':0.0331451,('PDT000557142.1':0.0219349,('PDT000661522.1':0.0115949,'PDT000661629.1':0.0120966)'':0.011726)'':0.00606261)'':0.00414053,(('PDT000694391.1':0.0257593,('PDT000136113.2':0.0218369,'PDT000136169.2':0.0206954)'':0.00287193)'':0.000487827,(('PDT000427622.1':0.0175179,('PDT000427531.1':0.00717537,'PDT000713620.1':0.00625883)'':0.00527509)'':0.00965741,('PDT000320883.1':0.0164017,('PDT000397852.1':0.0130361,'PDT000415795.1':0.0106554)'':0.0063089)'':0.00384029)'':0.00207664)'':0.00607102)'':0.00143477)'':0.00863044)'':-0.00334984)'':0.00683142)'':0.000132826)'':0.00398771)'':0.00261629)'':0.03212)'':-0.0314505)'':0.00627586)'':0.00314626)'':0.00978157)'':-0.00699565)'':0.0158332)'':-0.00831993)'':-0.00317219)'':0.0204602)'':-0.0203071)'':0.0910393)'':-0.0838308,(((((('PDT000484655.1':0.0518688,('PDT000661595.1':0.0385656,'PDT000262735.1':0.0279883)'':0.019805)'':0.000625266,('PDT000205445.1':0.0509725,(('PDT000713588.1':0.0317642,'PDT000718444.1':0.0269498)'':0.009903,('PDT000066280.2':0.0359643,(('PDT000136001.2':0.0182775,'PDT000714091.1':0.0184532)'':0.00725345,('PDT000034435.1':0.0245334,'PDT000400925.1':0.0240635)'':0.00309025)'':0.0088179)'':-0.000589605)'':0.0112821)'':0.000516299)'':0.0017012,(('PDT000661491.1':0.0488805,('PDT000713429.1':0.0289109,('PDT000127705.2':0.026045,('PDT000011798.2':0.0218372,'PDT000017680.2':0.0230195)'':0.00452357)'':0.00517792)'':0.0106728)'':0.000431135,(('PDT000714174.1':0.0300068,('PDT000251978.1':0.0172857,'PDT000017854.2':0.0136771)'':0.0126158)'':0.0026036,('PDT000714009.1':0.0231891,('PDT000714083.1':0.0100679,('PDT000672272.1':0.00656964,'PDT000017740.2':0.00664446)'':0.0038466)'':0.010049)'':0.00409432)'':0.0212816)'':0.00261347)'':0.00297241,((('PDT000135985.2':0.0291494,('PDT000124694.1':0.0168833,'PDT000346853.1':0.0182097)'':0.0151194)'':0.0219789,(('PDT000132327.2':0.036378,'PDT000694390.1':0.0310804)'':0.0053624,('PDT000188334.1':0.0302353,('PDT000713686.1':0.0141007,'PDT000017712.2':0.0168097)'':0.00919168)'':0.0109852)'':0.0158398)'':0.00342669,((('PDT000631761.1':0.049125,('PDT000536663.1':0.0270567,('PDT000631763.1':0.00407501,'PDT000631764.1':0.00450782)'':0.0192664)'':0.00737307)'':0.00535949,(('PDT000714162.1':0.00822767,('PDT000136021.2':0.00611576,('PDT000661588.1':0.00444487,'PDT000661709.1':0.00592813)'':0.00133019)'':0.00314925)'':0.0116663,(('PDT000093012.2':0.0225693,'PDT000502072.1':0.0221688)'':0.0080063,('PDT000502082.1':0.0286069,(('PDT000346791.1':0.0126691,'PDT000501865.1':0.0153238)'':0.00432586,('PDT000042565.1':0.0196706,(('PDT000502099.1':0.0132938,'PDT000346692.1':0.0120653)'':0.00251943,('PDT000017872.2':0.0122153,('PDT000694473.1':0.0121745,('PDT000713460.1':0.0130104,('PDT000694373.1':0.0096206,('PDT000719221.1':0.00507015,('PDT000135932.2':0.0042181,'PDT000713815.1':0.0048583)'':0.000599251)'':0.00268175)'':0.001435)'':0.0016418)'':0.00178197)'':0.000744885)'':0.00350225)'':-0.000997354)'':0.00607612)'':0.0010482)'':-0.00111726)'':0.0084512)'':-0.00146758,(('PDT000682315.1':0.0410674,(('PDT000188162.2':0.0136605,('PDT000501962.1':0.0112787,'PDT000501886.1':0.0118295)'':0.00500345)'':0.000338067,('PDT000713984.1':0.0156301,('PDT000385035.1':0.00984018,('PDT000132173.2':0.00402759,('PDT000618461.1':0.00406948,('PDT000673605.1':0.00304347,'PDT000714099.1':0.00214771)'':0.00121544)'':0.00115512)'':0.00409674)'':0.00558339)'':-0.000603903)'':0.0125548)'':-0.00555268,(('PDT000502605.1':0.0210184,('PDT000502092.1':0.0138189,('PDT000502235.1':0.0122897,('PDT000713889.1':0.00600015,'PDT000502070.1':0.00807175)'':0.00640531)'':0.000558143)'':0.00443934)'':0.000858058,(('PDT000400919.1':0.00581772,'PDT000470451.1':0.00467078)'':0.0138109,('PDT000042520.1':0.0196089,('PDT000713756.1':0.0135847,('PDT000298096.1':0.0150641,('PDT000327101.1':0.0099187,('PDT000042504.1':0.00704944,('PDT000397736.1':0.00469798,'PDT000630440.1':0.00448641)'':0.00224986)'':0.00188268)'':0.00163587)'':0.000705149)'':0.00588127)'':0.000598225)'':-0.000319348)'':0.00487375)'':0.0102555)'':0.0208435)'':-0.00154294)'':0.00545616,(('PDT000713576.1':0.0526967,'PDT000201778.1':0.0445777)'':0.0244327,(((('PDT000427540.1':0.0238684,('PDT000713427.1':0.00589309,'PDT000714054.1':0.00685721)'':0.0165085)'':0.0146191,('PDT000136049.2':0.0414169,('PDT000713607.1':0.0281451,('PDT000427282.1':0.0237047,('PDT000716833.1':0.0117899,('PDT000714021.1':0.0119834,'PDT000011770.2':0.00904933)'':0.0030393)'':0.00606779)'':0.00839187)'':0.009817)'':-0.00297175)'':0.0076092,((('PDT000713539.1':0.0233836,('PDT000188361.1':0.00254833,'PDT000188363.1':0.0020457)'':0.0214197)'':0.00868735,('PDT000213254.2':0.0277284,('PDT000125981.1':0.0206203,('PDT000713858.1':0.0196066,'PDT000249790.1':0.0182558)'':0.00183038)'':0.00752598)'':0.00324237)'':0.00928865,('PDT000017734.2':0.040237,(((('PDT000397592.1':0.0183085,'PDT000125930.1':0.0164947)'':0.00648177,('PDT000248509.1':0.0138686,'PDT000354204.1':0.013747)'':0.00676925)'':0.00456328,('PDT000347058.1':0.0305635,('PDT000102974.1':0.021876,(('PDT000661342.1':0.0167128,('PDT000382834.1':0.00721,'PDT000102979.1':0.0058625)'':0.00519763)'':0.00287386,('PDT000042474.1':0.0123647,('PDT000102757.1':0.0112961,('PDT000248496.1':0.00815125,'PDT000415821.1':0.00726705)'':0.00283491)'':0.00160773)'':0.00146862)'':0.00407434)'':0.0115959)'':-0.00176362)'':-0.000672468,(('PDT000102998.1':0.0302162,'PDT000249756.1':0.0267268)'':0.00269172,(('PDT000713524.1':0.0300383,'PDT000713555.1':0.0263816)'':0.00063623,(('PDT000125975.1':0.0189891,('PDT000103076.1':0.0129001,'PDT000249788.1':0.0114587)'':0.00501587)'':0.00382602,('PDT000417910.1':0.0232666,('PDT000136041.2':0.0147866,('PDT000132187.2':0.0150332,('PDT000136116.2':0.0136149,('PDT000713902.1':0.00496114,('PDT000713922.1':0.00163531,'PDT000017642.2':0.00116329)'':0.00301287)'':0.00421312)'':0.001833)'':0.00168768)'':0.0117509)'':-0.00326903)'':0.00448059)'':0.00123099)'':-0.000463966)'':0.0112279)'':-0.00180971)'':0.00298684)'':0.00192983,((('PDT000131871.2':0.0116095,'PDT000367732.1':0.0133334)'':0.0338207,('PDT000488017.1':0.032255,'PDT000713847.1':0.0288795)'':0.0112167)'':0.00210094,((('PDT000661739.1':0.0300398,('PDT000636329.1':0.0222132,('PDT000636171.1':0.0184578,('PDT000694383.1':0.0174579,('PDT000661414.1':0.0146043,('PDT000636321.1':0.0103924,'PDT000661694.1':0.0115496)'':0.00176306)'':0.00268604)'':0.00297905)'':0.00107051)'':0.00357838)'':-0.00254289,('PDT000661754.1':0.0276257,(('PDT000661378.1':0.0138452,'PDT000661831.1':0.0143754)'':0.00207416,('PDT000661779.1':0.0152043,(('PDT000713628.1':0.0107301,'PDT000713873.1':0.00838394)'':0.00170895,('PDT000713640.1':0.00765263,('PDT000362460.1':0.00881588,('PDT000017646.2':0.00478543,('PDT000714038.1':0.00260753,'PDT000270462.1':0.00218241)'':0.00218032)'':0.00228674)'':0.00226608)'':0.00131368)'':0.00426642)'':0.000998068)'':0.0083366)'':-0.00312916)'':0.0111122,(('PDT000136004.2':0.0473538,'PDT000427722.1':0.0423113)'':0.000125851,('PDT000249787.1':0.0380038,(('PDT000132095.2':0.0286359,'PDT000470448.1':0.0257365)'':0.00783149,(('PDT000713623.1':0.0270161,'PDT000019899.3':0.036658)'':0.00259229,(('PDT000661552.1':0.027425,'PDT000248505.1':0.0258051)'':0.00697202,((('PDT000713625.1':0.0171662,'PDT000019245.2':0.0157402)'':0.00384569,('PDT000661818.1':0.0202699,('PDT000713361.1':0.0186448,'PDT000015317.2':0.013046)'':0.00394634)'':0.000260427)'':0.00207849,('PDT000510056.1':0.0247411,(('PDT000713491.1':0.011904,'PDT000713765.1':0.0103403)'':0.012461,(('PDT000137862.2':0.015607,'PDT000248511.1':0.0156276)'':0.00223099,(('PDT000713463.1':0.00994157,('PDT000571178.1':0.00688216,'PDT000661793.1':0.0102835)'':0.00247458)'':0.00655413,('PDT000713739.1':0.0182763,('PDT000332121.1':0.0179227,('PDT000427693.1':0.0130986,('PDT000694421.1':0.0103481,('PDT000249781.1':0.00557744,'PDT000356854.1':0.0041991)'':0.00357446)'':0.0011837)'':0.0041088)'':0.00284014)'':-0.00142106)'':0.00197539)'':0.00178014)'':0.00526555)'':-0.00552859)'':0.0082806)'':0.00142288)'':0.0023604)'':0.00105017)'':0.0051851)'':-0.00757147)'':0.00968295)'':0.00312943)'':0.0208363)'':-0.00785342)'':0.00977914,(((('PDT000587982.1':0.035407,('PDT000011829.2':0.0161023,'PDT000017660.2':0.0198583)'':0.0186036)'':0.00302579,('PDT000044124.1':0.0842737,((('PDT000102975.1':0.0196992,'PDT000319783.1':0.0231831)'':0.00295323,('PDT000538913.2':0.0151991,('PDT000042479.1':0.0144334,'PDT000661833.1':0.0163481)'':0.00346876)'':0.00499076)'':0.00316335,(('PDT000136125.2':0.0271867,('PDT000136030.2':0.0206442,('PDT000248517.1':0.0103422,('PDT000103021.1':0.00970629,'PDT000249784.1':0.010765)'':0.00122701)'':0.00625748)'':0.00322855)'':0.00290279,(('PDT000346934.1':0.0190869,('PDT000713335.1':0.0136866,'PDT000713868.1':0.0134609)'':0.00601269)'':0.00420651,(('PDT000346781.1':0.01229,('PDT000713317.1':0.00850906,'PDT000587997.1':0.0101992)'':0.00378702)'':0.00409239,('PDT000017709.2':0.0142724,('PDT000125965.1':0.0123212,'PDT000215991.1':0.011712)'':0.00264759)'':0.00357349)'':0.00687844)'':0.00134075)'':0.00273723)'':0.028472)'':-0.0167547)'':0.000913346,((('PDT000535179.1':0.0338948,('PDT000534955.1':0.0183847,('PDT000534961.1':0.0128119,'PDT000535164.1':0.0148917)'':0.00370871)'':0.0191559)'':0.00814464,(('PDT000534960.1':0.0221483,('PDT000534985.1':0.0132822,'PDT000588021.1':0.0166744)'':0.00882425)'':0.00690614,(('PDT000535308.1':0.0159631,('PDT000534879.1':0.00798446,'PDT000535007.1':0.00832424)'':0.00588607)'':0.0028976,('PDT000618751.1':0.0187963,('PDT000534923.1':0.00835466,'PDT000534959.1':0.00804204)'':0.0100914)'':0.00109408)'':0.0120979)'':0.0143634)'':0.00281523,((('PDT000460594.1':0.0206885,('PDT000460588.1':0.0104613,('PDT000460589.1':0.00988006,('PDT000460591.1':0.00645948,'PDT000460593.1':0.00590962)'':0.00334324)'':0.000873805)'':0.00761895)'':0.0270109,('PDT000415656.1':0.0605306,(('PDT000205421.1':0.0299904,('PDT000713448.1':0.0202129,('PDT000249772.1':0.0171107,'PDT000017663.2':0.0173381)'':0.00316404)'':0.00797104)'':-0.00183638,('PDT000354208.1':0.0324424,('PDT000017716.2':0.0199232,(('PDT000694529.1':0.0149878,'PDT000574933.1':0.0173817)'':0.00309643,('PDT000694478.1':0.0158827,('PDT000713551.1':0.0125251,('PDT000137871.2':0.0113961,('PDT000427576.1':0.00335919,'PDT000713487.1':0.00372099)'':0.00589629)'':0.000688449)'':0.00371295)'':0.00259323)'':0.00525603)'':0.00113702)'':0.000981572)'':0.0269947)'':-0.0114008)'':-0.00379764,(('PDT000534939.1':0.0259809,'PDT000535072.1':0.027449)'':0.0192179,(('PDT000427515.1':0.068131,('PDT000661497.1':0.0397775,('PDT000685885.1':0.0533825,('PDT000719024.1':0.0256011,(('PDT000017704.2':0.0203121,('PDT000042739.1':0.0117024,('PDT000421865.1':0.00955731,'PDT000427806.1':0.00890419)'':0.00348865)'':0.00329892)'':0.00481323,('PDT000397692.1':0.0204592,('PDT000535374.1':0.0182546,('PDT000397589.1':0.0108519,'PDT000427479.1':0.0150962)'':0.00578975)'':0.00294489)'':0.000386593)'':0.00264083)'':0.0126564)'':0.00131123)'':0.00917742)'':-0.0124779,(((('PDT000042544.1':0.0217201,'PDT000661501.1':0.0268862)'':0.00112573,('PDT000131880.2':0.0189556,'PDT000326088.1':0.0208356)'':0.00162787)'':0.00679281,(('PDT000636141.1':0.0269656,'PDT000714010.1':0.0259739)'':0.00154072,('PDT000713862.1':0.0178562,('PDT000661377.1':0.0173683,('PDT000067788.1':0.0166341,('PDT000069672.1':0.0154112,('PDT000713530.1':0.0146303,('PDT000067789.1':0.0129778,('PDT000713826.1':0.00939116,'PDT000588032.1':0.0115227)'':0.00142116)'':0.0029661)'':0.00207144)'':0.000991889)'':0.000774757)'':0.00111467)'':0.00688497)'':0.00282387)'':0.000683101,(('PDT000175836.2':0.0245726,'PDT000717002.1':0.0240825)'':0.0163491,('PDT000427053.1':0.035034,(('PDT000427238.1':0.0132577,'PDT000713411.1':0.0111519)'':0.0221253,('PDT000017797.2':0.0328375,(('PDT000587988.1':0.0196945,'PDT000661646.1':0.0142106)'':0.0128655,('PDT000433748.1':0.0325934,('PDT000433798.1':0.0313423,(('PDT000427544.1':0.0186524,('PDT000713425.1':0.0186423,'PDT000216024.1':0.0162174)'':0.00181293)'':0.00594487,(('PDT000103086.1':0.0148269,('PDT000042581.1':0.00685927,'PDT000042587.1':0.00562573)'':0.00874369)'':0.00547877,('PDT000382852.1':0.0237818,('PDT000042494.1':0.030886,('PDT000661707.1':0.0195221,('PDT000042740.1':0.0133062,('PDT000683478.1':0.018537,(('PDT000713926.1':0.0152396,'PDT000716876.1':0.014791)'':0.000508501,(('PDT000017713.2':0.00936513,('PDT000714116.1':0.00761463,('PDT000694496.1':0.00823055,'PDT000111111.1':0.00875055)'':0.00117362)'':0.000648015)'':0.0025936,('PDT000713492.1':0.0151194,(('PDT000713531.1':0.00678392,'PDT000661498.1':0.00777238)'':0.00491495,(('PDT000661692.1':0.00805102,'PDT000017881.2':0.00715618)'':0.00244194,(('PDT000673585.1':0.00858511,('PDT000661695.1':0.00684658,'PDT000103046.1':0.00726172)'':0.000963736)'':-5.17329e-05,('PDT000156937.2':0.00836312,('PDT000086320.1':0.00728195,('PDT000137863.2':0.00512348,('PDT000248508.1':0.00182968,'PDT000248514.1':0.00216712)'':0.00224832)'':0.00196751)'':0.00208614)'':-0.000571625)'':0.00113353)'':0.000787333)'':0.00241095)'':-0.00178395)'':0.00168424)'':0.00268142)'':0.000137858)'':0.0024134)'':0.0051481)'':0.00177946)'':-0.00521719)'':0.00169277)'':0.00686937)'':0.000895995)'':-0.00133491)'':0.00424735)'':-0.000615164)'':0.00659143)'':-0.00176214)'':-0.00855391)'':0.00402383)'':0.0165343)'':-0.0100676)'':0.00534281)'':-0.00104047)'':0.0186336,((('PDT000661577.1':0.0957405,(('PDT000688650.1':0.0405137,('PDT000661660.1':0.0365993,('PDT000439508.1':0.0244209,('PDT000713778.1':0.0195871,('PDT000006422.2':0.0139616,('PDT000136056.2':0.00474347,'PDT000713912.1':0.00471868)'':0.0104382)'':0.00617208)'':0.00224137)'':0.00945046)'':0.00236071)'':-0.000245913,('PDT000713396.1':0.0488833,('PDT000100155.2':0.0376743,((('PDT000661651.1':0.00764818,'PDT000007693.2':0.00611742)'':0.00668351,('PDT000694508.1':0.020008,('PDT000361019.1':0.0155073,('PDT000427027.1':0.0123697,('PDT000017911.2':0.00818322,('PDT000661376.1':0.00652705,('PDT000270158.1':0.00180063,'PDT000017756.2':0.00209236)'':0.00241035)'':0.00214463)'':0.00217819)'':0.00462591)'':0.0019983)'':0.000174819)'':9.62414e-07,('PDT000636155.1':0.0263358,('PDT000636175.1':0.0200724,('PDT000618954.1':0.0139343,(('PDT000616034.1':0.0100693,('PDT000673487.1':0.00798748,('PDT000673607.1':0.00548317,'PDT000720395.1':0.00509143)'':0.00198042)'':0.00117673)'':0.00259241,('PDT000574924.1':0.00605621,('PDT000694494.1':0.00687167,('PDT000346756.1':0.00446814,'PDT000415835.1':0.00361731)'':0.00196601)'':0.00161684)'':0.00184377)'':0.0048258)'':0.000553794)'':0.00495151)'':-0.00239127)'':0.0189718)'':0.00489971)'':-0.00430023)'':0.0409446)'':-0.033022,((('PDT000661755.1':0.0655555,('PDT000137866.2':0.0291734,'PDT000004467.2':0.0257436)'':0.0222883)'':0.00257267,((('PDT000135941.2':0.00989335,'PDT000713328.1':0.0127073)'':0.0135801,('PDT000470447.1':0.0138569,('PDT000131841.2':0.00982342,'PDT000017652.2':0.0109087)'':0.00196191)'':0.00812225)'':0.00702906,(('PDT000443079.1':0.0174901,'PDT000661676.1':0.0208006)'':0.0030455,(('PDT000713777.1':0.0143901,'PDT000389326.1':0.0164582)'':0.00248495,('PDT000033982.1':0.010139,('PDT000033987.1':0.00619736,('PDT000034388.1':0.0073306,('PDT000033986.1':0.00362839,('PDT000034386.1':0.00460496,'PDT000034393.1':0.00366197)'':0.00055676)'':0.000856258)'':0.00135269)'':0.00277166)'':0.0123507)'':0.00219124)'':0.00931388)'':0.0190591)'':-0.00643927,('PDT000683133.1':0.0901381,((('PDT000536593.1':0.0340775,(('PDT000433642.1':0.0232338,'PDT000616078.1':0.0222475)'':0.00870618,('PDT000661789.1':0.0216545,('PDT000713226.1':0.0204697,('PDT000132034.2':0.0253161,('PDT000534852.1':0.0133754,'PDT000661400.1':0.0120148)'':0.00890742)'':0.00325185)'':0.00135712)'':0.000810381)'':0.00586078)'':0.001381,('PDT000426714.1':0.0512858,('PDT000107490.2':0.0186652,('PDT000588055.1':0.0191885,(('PDT000534931.1':0.00982715,'PDT000042521.1':0.00894705)'':0.0043066,('PDT000233076.1':0.0118197,('PDT000713747.1':0.00669631,('PDT000713557.1':0.00412579,('PDT000574925.1':0.00227745,'PDT000250015.1':0.00231658)'':0.00115212)'':0.00139844)'':0.00783666)'':-0.00029329)'':0.00478975)'':0.00156979)'':0.029649)'':-0.0154066)'':0.00243278,(('PDT000589822.1':0.052015,(('PDT000069641.1':0.0246617,('PDT000276846.1':0.0187063,('PDT000415786.1':0.00940443,('PDT000017725.2':0.00505534,'PDT000017857.2':0.00520256)'':0.00347052)'':0.0118159)'':0.0038207)'':0.00182732,('PDT000427558.1':0.0358708,(('PDT000238387.2':0.00893035,'PDT000131938.2':0.00773965)'':0.00299184,('PDT000661718.1':0.0134203,('PDT000132083.2':0.00779304,(('PDT000694545.1':0.00220863,'PDT000714025.1':0.001884)'':0.0008288,('PDT000713666.1':0.00177959,'PDT000714065.1':0.00181502)'':0.0010165)'':0.00472985)'':0.00639274)'':-0.00136641)'':0.015218)'':0.000331431)'':0.0140099)'':-0.00985312,(('PDT000713855.1':0.037753,('PDT000713736.1':0.0180938,('PDT000136031.2':0.0114924,'PDT000427528.1':0.0101196)'':0.00887281)'':0.0198142)'':-0.008897,('PDT000647172.1':0.0453557,('PDT000661711.1':0.0345584,(('PDT000017697.2':0.0221725,('PDT000661733.1':0.0166623,('PDT000714078.1':0.018534,('PDT000713550.1':0.0085951,('PDT000357011.1':0.00641412,('PDT000136121.2':0.00421556,'PDT000571155.1':0.00566172)'':0.00327324)'':0.0020066)'':0.00565063)'':0.00259666)'':0.00172985)'':-0.00076563,((('PDT000132116.2':0.0097394,'PDT000661806.1':0.0099817)'':0.00307022,('PDT000583901.1':0.0111224,('PDT000661792.1':0.00542087,('PDT000131996.2':0.0041812,'PDT000136104.2':0.00500227)'':0.000843494)'':0.00757208)'':0.0010294)'':0.000307586,('PDT000661406.1':0.0227808,('PDT000623231.1':0.0168857,('PDT000713571.1':0.0164946,('PDT000019251.2':0.0123397,('PDT000213253.2':0.0150106,('PDT000713444.1':0.00940554,(('PDT000433231.1':0.00865777,'PDT000017794.2':0.00800393)'':0.00386669,(('PDT000630058.1':0.00457935,'PDT000661473.1':0.00499862)'':0.0032807,(('PDT000535030.1':0.00471691,'PDT000018255.1':0.0046503)'':0.00260334,('PDT000716887.1':0.00791212,(('PDT000042585.1':0.00495483,'PDT000017698.2':0.00530717)'':0.00100794,('PDT000587978.1':0.00716354,('PDT000448226.1':0.0056149,('PDT000716890.1':0.00465785,'PDT000661745.1':0.00551165)'':0.000650602)'':0.00115913)'':1.57648e-05)'':0.00205754)'':-0.000695669)'':0.00101713)'':0.000416007)'':0.0012345)'':0.000691531)'':0.000610643)'':0.00377269)'':0.00130505)'':0.00448265)'':-0.00746277)'':0.0059139)'':0.0150545)'':0.0144412)'':-0.0167238)'':0.000880824)'':0.00381265)'':0.060439)'':-0.052392)'':0.00192651)'':0.00550873,((('PDT000661670.1':0.0515791,('PDT000713720.1':0.0336571,'PDT000362516.1':0.0380626)'':0.01284)'':-0.000812555,((('PDT000604314.1':0.0134737,'PDT000008472.2':0.0159468)'':0.00355285,('PDT000714061.1':0.0051987,'PDT000661439.1':0.0055636)'':0.0136409)'':0.0211296,(('PDT000427488.2':0.0552113,('PDT000694381.1':0.0281936,(('PDT000332839.1':0.0256799,(('PDT000694546.1':0.0100751,'PDT000715963.1':0.00906409)'':0.00128172,('PDT000661315.1':0.0112988,'PDT000661790.1':0.00861163)'':0.00223295)'':0.0131415)'':-0.0032049,('PDT000713988.1':0.0234056,(('PDT000131906.2':0.011116,'PDT000714139.1':0.0103755)'':0.00476108,('PDT000713290.1':0.00840933,'PDT000714040.1':0.0100945)'':0.00490742)'':0.00581718)'':-0.000367253)'':0.00835998)'':0.0200352)'':-0.0162562,('PDT000685904.1':0.048738,(('PDT000694389.1':0.020766,'PDT000103063.1':0.0182414)'':0.00355596,('PDT000685887.1':0.0399731,('PDT000499510.1':0.0168882,('PDT000107914.1':0.0216339,('PDT000489642.1':0.0180585,(('PDT000397888.1':0.011273,('PDT000096851.2':0.00834505,'PDT000103029.1':0.00787065)'':0.00206497)'':0.000549834,('PDT000360063.1':0.013873,('PDT000673558.1':0.00971723,('PDT000627371.1':0.00648881,('PDT000067781.1':0.00209386,'PDT000233087.1':0.0023985)'':0.00318361)'':0.00360044)'':0.00256938)'':-0.00144482)'':0.00443435)'':0.00187171)'':0.000977133)'':0.00862917)'':-0.00340815)'':0.0138228)'':-0.00588707)'':0.00667214)'':0.00511039)'':0.0059546,((('PDT000661425.1':0.0369293,('PDT000443900.1':0.00842438,('PDT000443702.1':0.00266895,'PDT000443866.1':0.00172841)'':0.00659109)'':0.0250486)'':0.029158,('PDT000535121.1':0.0451439,('PDT000713709.1':0.0370105,('PDT000534922.1':0.0219704,'PDT000535082.1':0.020627)'':0.0160537)'':0.0158319)'':0.0104765)'':0.00368981,(('PDT000713220.1':0.0299532,('PDT000120942.2':0.0133957,'PDT000716874.1':0.0133226)'':0.0183484)'':0.0214742,(('PDT000713883.1':0.0321194,((('PDT000161547.2':0.0178817,'PDT000362472.1':0.0154055)'':0.00447517,('PDT000636310.1':0.0164391,('PDT000397900.1':0.00868494,'PDT000623221.1':0.00997136)'':0.00399463)'':0.00771722)'':0.001077,(('PDT000444143.1':0.0174089,'PDT000714149.1':0.0119119)'':0.00559602,(('PDT000360998.1':0.0132327,'PDT000362449.1':0.0136023)'':0.00456086,('PDT000675039.1':0.0241888,('PDT000363566.1':0.0175333,('PDT000017675.2':0.00979679,('PDT000713866.1':0.0102584,('PDT000661732.1':0.00708092,'PDT000362520.1':0.00843528)'':0.00338817)'':0.000698738)'':0.00444686)'':0.00310517)'':0.000746364)'':0.00094174)'':0.00427801)'':0.0142458)'':0.00341153,((('PDT000017656.2':0.0205799,('PDT000397992.1':0.0201587,'PDT000362517.1':0.0222434)'':0.0050487)'':0.00857946,((('PDT000376181.2':0.0208472,'PDT000017874.2':0.0201921)'':0.0013876,('PDT000587981.1':0.0127058,'PDT000588037.1':0.00927659)'':0.0100512)'':0.00423488,(('PDT000716221.1':0.0184414,'PDT000441842.1':0.0219187)'':0.0018958,(('PDT000713273.1':0.013491,('PDT000325181.1':0.0116414,'PDT000017751.2':0.0127101)'':0.00283217)'':-9.92915e-05,(('PDT000661774.1':0.00788928,'PDT000362526.1':0.00944312)'':0.00470098,('PDT000017686.2':0.0126573,('PDT000425912.1':0.0117321,('PDT000714070.1':0.00742423,('PDT000132370.2':0.00672887,'PDT000011731.2':0.00609033)'':0.00143332)'':0.00168752)'':0.00341906)'':-0.000563939)'':0.0030999)'':0.00412571)'':0.00657693)'':0.00665074)'':0.00586937,(('PDT000587983.1':0.0201115,('PDT000400924.1':0.0104344,'PDT000391863.2':0.0107301)'':0.00908481)'':0.037193,((('PDT000713853.1':0.0341422,('PDT000017671.2':0.0201089,'PDT000017700.2':0.021705)'':0.0117896)'':-0.000609215,(('PDT000017978.2':0.0154129,('PDT000713216.1':0.00218389,'PDT000017695.2':0.00231027)'':0.0120303)'':0.00633536,(('PDT000233100.1':0.0115214,('PDT000397866.1':0.00669893,'PDT000714160.1':0.00733367)'':0.00612231)'':0.00363747,(('PDT000249836.1':0.0173967,'PDT000249837.1':0.0144354)'':0.002283,('PDT000156528.2':0.0115286,('PDT000132151.2':0.00892395,('PDT000713774.1':0.00388239,('PDT000713796.1':0.00157372,'PDT000714178.1':0.00162276)'':0.00210039)'':0.00317523)'':0.00449513)'':0.00297141)'':0.00156258)'':0.00389755)'':0.0121889)'':-0.00454032,(('PDT000661361.1':0.0378388,('PDT000399918.1':0.0201661,('PDT000199115.2':0.0114823,('PDT000469502.1':0.01109,'PDT000318562.1':0.00826357)'':0.00155987)'':0.0042841)'':0.0145251)'':0.00151079,(('PDT000587979.1':0.0457738,(('PDT000587980.1':0.014742,'PDT000400923.1':0.0163078)'':0.00249674,('PDT000588058.1':0.014852,('PDT000400918.1':0.00947283,'PDT000470450.1':0.0108662)'':0.0034621)'':0.00284816)'':0.0178467)'':-0.000791176,((('PDT000427460.1':0.0129795,('PDT000132078.2':0.00683378,('PDT000318815.1':0.00494069,'PDT000713898.1':0.00554041)'':0.00297637)'':0.00407815)'':0.00212047,(('PDT000427720.1':0.00242412,'PDT000011833.2':0.00256939)'':0.00841401,('PDT000588044.1':0.0118052,('PDT000135972.2':0.00362008,'PDT000588016.1':0.00385823)'':0.00653503)'':0.00163137)'':0.00386957)'':0.00634012,((('PDT000443026.1':0.0095482,'PDT000443081.1':0.0138191)'':0.0225342,('PDT000444145.1':0.0284345,'PDT000444150.1':0.0266284)'':0.00608807)'':0.00324522,(('PDT000714166.1':0.0253695,('PDT000713780.1':0.0145264,('PDT000426617.1':0.0111911,'PDT000661804.1':0.0131895)'':0.00322294)'':0.00503021)'':0.00225118,(('PDT000426325.1':0.0205867,('PDT000713758.1':0.0140776,('PDT000694504.1':0.00935888,'PDT000426369.1':0.00966042)'':0.00337309)'':0.00499919)'':0.00165349,(('PDT000713294.1':0.0137188,'PDT000248460.1':0.0145006)'':0.0175378,('PDT000228316.1':0.0324931,(('PDT000173733.2':0.0105444,('PDT000694458.1':0.0103923,('PDT000661504.1':0.00876775,('PDT000427024.1':0.00481572,'PDT000716965.1':0.0045459)'':0.00182224)'':0.00203409)'':0.00151153)'':0.00129606,('PDT000685905.1':0.0275551,('PDT000661420.1':0.018897,('PDT000661815.1':0.022114,(('PDT000135952.2':0.00575219,'PDT000713697.1':0.00571571)'':0.00875974,(('PDT000713731.1':0.00491136,('PDT000158669.2':0.00505626,('PDT000713458.1':0.00423453,'PDT000713905.1':0.00463001)'':0.000519145)'':0.000490827)'':0.00314287,('PDT000713532.1':0.0111729,('PDT000400920.1':0.0100319,(('PDT000694428.1':0.00297595,'PDT000716879.1':0.00310823)'':0.00266718,('PDT000694440.1':0.00819887,('PDT000713719.1':0.00588067,('PDT000425634.1':0.0092173,('PDT000687363.1':0.00534628,('PDT000713830.1':0.00849951,('PDT000250860.2':0.00558741,('PDT000713590.1':0.00715717,('PDT000017750.2':0.0054477,('PDT000588027.1':0.00412618,('PDT000135999.2':0.00545996,('PDT000017677.2':0.00425559,('PDT000713442.1':0.00282238,('PDT000576232.1':0.00287748,('PDT000713522.1':0.00337849,('PDT000155470.2':0.00296225,('PDT000132222.2':0.00322548,(('PDT000713541.1':0.00160601,'PDT000713711.1':0.00149089)'':0.000722454,(('PDT000714039.1':0.0016,'PDT000017638.2':0.00159553)'':0.000313897,(('PDT000673603.1':0.00153324,('PDT000714066.1':0.00133127,('PDT000713665.1':0.00131454,'PDT000716999.1':0.00128234)'':6.77215e-05)'':0.000507125)'':9.54897e-05,('PDT000713737.1':0.00166826,('PDT000713636.1':0.000964348,('PDT000713942.1':0.00107583,'PDT000714023.1':0.000922774)'':0.000234872)'':0.000816583)'':0.000155648)'':0.000184387)'':0.000242618)'':0.000630477)'':0.000207957)'':0.000197188)'':0.000126475)'':0.000195425)'':0.000305264)'':0.00026487)'':0.000971479)'':9.75267e-05)'':0.000131078)'':0.000717348)'':5.15226e-05)'':0.00012692)'':0.000941221)'':0.000198514)'':0.000213083)'':-0.00112192)'':0.0025016)'':0.000273462)'':-0.0005628)'':0.00336852)'':0.00686686)'':0.000644172)'':0.00336821)'':-0.0115554)'':0.0150038)'':0.001196)'':-0.0071044)'':0.00212987)'':0.010539)'':-0.00875906)'':0.00990481)'':0.0024284)'':-0.00719723)'':0.028279)'':-0.0189996)'':0.00366142)'':0.0145464)'':0.00697993)'':-0.00985407)'':-0.00370726)'':0.00939586)'':0.0121764)'':0.00521439)'':0.0902994)'':-0.00365744)'':0.00473093)'':0.00627964)'':0.00353107)'':0.0129582)'':0.0106217)'':-0.00596872)'':0.0469732)'':-0.018625)'':-0.0056469)'':0.0775927)'':0.0190078)'':0.0109829,(((((('PDT000661099.1':0.137767,('PDT000316946.2':0.0963655,'PDT000652433.1':0.105865)'':0.031631)'':0.00444339,('PDT000104650.2':0.11633,('PDT000293452.1':0.122765,(('PDT000444955.1':0.104916,'PDT000687924.1':0.0968736)'':0.00278437,('PDT000251971.1':0.0990149,('PDT000557230.1':0.105725,('PDT000557170.1':0.0624912,('PDT000415748.1':0.0574839,'PDT000721984.1':0.0500151)'':0.00685383)'':0.0226258)'':0.00548661)'':0.0155413)'':0.00806239)'':0.00585242)'':0.00661658)'':0.156923,((('PDT000190754.2':0.106909,(('PDT000320839.1':0.0132272,'PDT000479197.1':0.0145863)'':0.0542213,(('PDT000132087.2':0.0356024,'PDT000251973.1':0.0307227)'':0.0166149,('PDT000649805.1':0.0193237,'PDT000082648.1':0.0192172)'':0.033158)'':0.0182476)'':0.0247628)'':0.138494,(('PDT000718926.1':0.234557,('PDT000671630.1':0.0676791,('PDT000209873.3':0.0332381,'PDT000484192.2':0.0447775)'':0.0316741)'':0.144761)'':0.0118146,((('PDT000011813.2':0.191392,'PDT000652454.1':0.197357)'':0.00283612,('PDT000218476.5':0.0609316,'PDT000654308.1':0.0642914)'':0.133516)'':0.0230051,('PDT000251851.1':0.198699,(('PDT000557050.1':0.106069,'PDT000661262.1':0.117643)'':0.0614409,('PDT000557344.1':0.0647974,('PDT000417500.1':0.0455645,'PDT000689845.1':0.0445742)'':0.0167143)'':0.109149)'':0.0286832)'':0.00896719)'':0.00662346)'':0.00602873)'':0.0342256,(('PDT000034489.2':0.0587538,'PDT000249880.1':0.0620492)'':0.197419,(('PDT000366728.1':0.160092,(('PDT000042557.1':0.0592509,'PDT000249967.1':0.0840001)'':0.0821192,('PDT000160444.3':0.0821813,('PDT000707300.1':0.0506652,'PDT000502051.1':0.0381802)'':0.0402165)'':0.0647247)'':0.0119268)'':0.0225421,('PDT000443049.1':0.176346,((('PDT000188028.3':0.0266781,'PDT000716950.1':0.027743)'':0.012745,('PDT000003669.2':0.046941,('PDT000257697.2':0.0316798,'PDT000312642.2':0.0344158)'':0.00530817)'':0.00274265)'':0.0379597,(('PDT000395242.1':0.0891005,('PDT000566691.1':0.0191244,'PDT000568760.1':0.0242895)'':0.064235)'':-0.00353981,(('PDT000249942.1':0.0548145,'PDT000252658.1':0.0483785)'':0.00867669,('PDT000034466.3':0.0617594,(('PDT000399707.1':0.0476989,'PDT000639630.1':0.0511161)'':0.0025339,('PDT000399703.1':0.0450213,('PDT000395407.1':0.0250768,('PDT000394917.1':0.0247795,'PDT000653455.1':0.0265753)'':0.00548077)'':0.0160036)'':0.00804629)'':0.00517427)'':0.00661714)'':0.0179049)'':0.018736)'':0.0989955)'':0.00590793)'':0.057384)'':0.0138853)'':0.0127191)'':-0.00741567,(((('PDT000258375.2':0.198978,'PDT000714672.1':0.208149)'':0.0205656,('PDT000005860.2':0.21229,('PDT000645516.2':0.221475,('PDT000228243.1':0.115385,('PDT000405245.1':0.0149477,'PDT000405267.1':0.0198461)'':0.0940761)'':0.0923906)'':0.005531)'':0.0126722)'':0.000438826,(('PDT000423860.1':0.169514,('PDT000240669.2':0.126919,'PDT000482206.1':0.132634)'':0.0397077)'':0.0609844,(('PDT000713928.1':0.230211,(('PDT000619619.1':0.0751638,'PDT000675168.1':0.0683742)'':0.131666,('PDT000359622.1':0.194299,('PDT000557017.1':0.145402,'PDT000415827.1':0.146086)'':0.0455345)'':0.0062115)'':0.0102116)'':0.000659946,((('PDT000008840.2':0.104708,('PDT000204997.2':0.0334722,'PDT000619586.1':0.0371138)'':0.0725741)'':0.0979383,('PDT000502173.1':0.0660627,('PDT000132225.2':0.0353358,('PDT000640006.1':0.031497,'PDT000716973.1':0.0325969)'':0.00446613)'':0.0369523)'':0.13637)'':0.00527807,((('PDT000510050.1':0.051692,'PDT000661494.1':0.051177)'':0.026634,('PDT000533183.1':0.0981412,('PDT000359653.1':0.043816,('PDT000687473.1':0.0285802,('PDT000231459.2':0.0173007,'PDT000719094.1':0.0145483)'':0.0088934)'':0.0130254)'':0.0327675)'':0.000679688)'':0.0739845,(('PDT000535433.1':0.0796889,'PDT000488270.1':0.0679051)'':0.0663264,(('PDT000372151.1':0.0572607,'PDT000011812.2':0.0528073)'':0.0124731,('PDT000281744.2':0.0692836,('PDT000438128.1':0.0324161,'PDT000571165.1':0.0379362)'':0.0281687)'':0.0167474)'':0.0642943)'':0.0169751)'':0.0581197)'':0.00687264)'':0.0049149)'':-0.000366064)'':0.00217313,((((('PDT000719283.1':0.0894674,('PDT000631583.1':0.0669447,('PDT000399652.1':0.062615,'PDT000644847.1':0.060019)'':0.00665178)'':0.00749422)'':0.00848748,(('PDT000352438.1':0.00106768,'PDT000354290.1':0.000830989)'':0.00978641,('PDT000355776.1':0.00727959,('PDT000354258.1':0.00880939,('PDT000352459.1':0.00209754,'PDT000354248.1':0.00269575)'':0.00288331)'':0.00143311)'':0.00631479)'':0.0723784)'':0.00892455,(('PDT000520733.1':0.0487361,'PDT000352555.1':0.0507602)'':0.0217024,(('PDT000157309.1':0.0332794,'PDT000352508.1':0.0309588)'':0.0345148,(('PDT000710412.1':0.0266,'PDT000240306.1':0.0354525)'':0.0315643,('PDT000372008.1':0.0811154,(('PDT000250122.1':0.026587,'PDT000251523.1':0.0311355)'':0.018061,('PDT000469960.1':0.0547745,(('PDT000510042.1':0.0253289,'PDT000008716.2':0.0251347)'':0.00141198,('PDT000556959.1':0.0277513,'PDT000557033.1':0.0237063)'':0.00222545)'':0.0182915)'':0.00177247)'':0.0208139)'':-0.00275181)'':0.00547143)'':0.00311002)'':0.0148668)'':0.00430105,(('PDT000003811.2':0.0817439,'PDT000439497.1':0.0831551)'':0.0166526,((('PDT000003614.2':0.0858665,('PDT000353612.1':0.0416891,'PDT000716958.1':0.0451759)'':0.027702)'':-0.0013481,('PDT000252603.1':0.053974,('PDT000249969.1':0.0436587,('PDT000069666.1':0.0459366,('PDT000716827.1':0.0364913,'PDT000476467.1':0.0350964)'':0.00873383)'':0.00403842)'':0.0067725)'':0.0201738)'':0.00352238,('PDT000426062.1':0.101061,('PDT000008737.3':0.0931555,('PDT000502891.1':0.0853908,('PDT000228065.1':0.0766332,('PDT000290366.2':0.0896457,('PDT000644944.1':0.0807106,('PDT000477456.1':0.0682188,(('PDT000252618.1':0.0750114,('PDT000424627.1':0.0661013,('PDT000287698.2':0.0594178,(('PDT000426030.1':0.0386859,('PDT000634324.1':0.0318564,'PDT000718863.1':0.0345628)'':0.00207199)'':0.00779552,('PDT000594981.1':0.0485606,('PDT000488318.1':0.0323576,('PDT000109464.2':0.0293574,'PDT000557209.1':0.0251631)'':0.0063033)'':0.00789917)'':0.00400107)'':0.00712958)'':0.00585164)'':0.0045903)'':-0.00450763,('PDT000346071.1':0.0725487,('PDT000473269.2':0.0773272,(('PDT000433463.1':0.0434396,'PDT000535485.1':0.0421434)'':0.0157403,('PDT000645092.1':0.0591432,(('PDT000396813.1':0.0144796,'PDT000251455.1':0.0125581)'':0.0325769,(('PDT000426550.1':0.0255848,'PDT000520250.1':0.0251717)'':0.00625074,('PDT000668391.1':0.0335285,('PDT000305447.2':0.0287607,'PDT000104646.2':0.0274497)'':0.00427358)'':0.00144085)'':0.00805922)'':0.00775607)'':-0.000297268)'':0.0115623)'':0.0038129)'':-0.0121757)'':0.0122909)'':0.001731)'':0.00228825)'':0.00054252)'':0.00397457)'':0.0109113)'':0.00900289)'':-0.0206867)'':0.0142005)'':0.00024596)'':0.115101,(((('PDT000485424.1':0.069185,'PDT000594462.1':0.070939)'':0.0105772,(('PDT000397563.1':0.0562911,'PDT000425803.1':0.0494739)'':0.0103107,(('PDT000631948.1':0.0525998,'PDT000161884.3':0.0521872)'':0.000672459,('PDT000649148.2':0.047323,'PDT000252577.1':0.0339073)'':0.011035)'':0.0063386)'':0.0106353)'':0.106103,((('PDT000362872.1':0.045056,'PDT000500530.1':0.0400215)'':0.100316,(('PDT000397634.1':0.10216,'PDT000608596.1':0.0878139)'':0.00903058,(('PDT000427749.1':0.0576029,'PDT000583391.1':0.0583621)'':0.0138192,('PDT000233300.1':0.0713035,('PDT000318977.1':0.070276,(('PDT000426521.1':0.0456314,'PDT000362490.1':0.0427712)'':0.00319289,('PDT000181766.2':0.0204884,('PDT000093011.2':0.0101772,'PDT000538578.1':0.0105097)'':0.0055386)'':0.0251555)'':0.0107369)'':0.00379716)'':0.00620683)'':0.0345653)'':0.0278659)'':0.0383266,(('PDT000350389.1':0.160301,('PDT000502054.1':0.0442949,('PDT000673472.1':0.024495,'PDT000042490.1':0.023124)'':0.0144192)'':0.119916)'':0.00929427,(('PDT000336927.1':0.152292,(('PDT000427073.1':0.0430131,('PDT000218365.1':0.0116026,'PDT000354207.1':0.00927239)'':0.0389259)'':0.0530398,(('PDT000248685.1':0.0957115,'PDT000357635.1':0.0851345)'':0.0100964,('PDT000151503.3':0.0169496,('PDT000545118.2':0.013568,'PDT000673128.1':0.0142481)'':0.00685761)'':0.0778568)'':0.00984978)'':0.0322714)'':0.0360014,((('PDT000535483.1':0.0405501,'PDT000346619.1':0.0504915)'':0.0645933,('PDT000042621.1':0.0706964,('PDT000535084.1':0.0562109,'PDT000463725.1':0.0539321)'':0.0124701)'':0.0412741)'':0.0308057,(('PDT000434499.1':0.0151157,'PDT000482201.1':0.013325)'':0.0980679,(('PDT000426333.1':0.0775775,('PDT000486469.2':0.0510383,('PDT000535432.1':0.0338583,'PDT000538562.1':0.0394117)'':0.0148275)'':0.0264817)'':0.0210184,((('PDT000604843.1':0.0443484,'PDT000543154.1':0.0474455)'':0.0050331,('PDT000642324.1':0.0409886,'PDT000042674.1':0.0450869)'':0.00352338)'':0.00858882,('PDT000687461.1':0.0642272,('PDT000425765.1':0.0491599,('PDT000502154.1':0.0472471,(('PDT000042654.1':0.0348547,'PDT000720479.1':0.0369203)'':0.00713706,(('PDT000669523.1':0.0351879,'PDT000502063.1':0.0317659)'':0.00369388,(('PDT000502103.1':0.0270181,'PDT000415779.1':0.0206292)'':0.00451392,(('PDT000649103.1':0.0202741,'PDT000395107.1':0.018176)'':0.00197888,('PDT000713387.1':0.0242142,(('PDT000360467.1':0.011051,'PDT000395089.1':0.0128106)'':0.00517064,(('PDT000015985.3':0.00135762,'PDT000015986.3':0.00193711)'':0.000124737,('PDT000015978.2':0.00153154,'PDT000015984.2':0.00146486)'':0.000524488)'':0.0147586)'':0.00489766)'':-4.06181e-05)'':0.00502091)'':0.00677142)'':0.00443159)'':0.00717489)'':0.000486504)'':0.0156878)'':-0.00630151)'':0.0396144)'':0.0213525)'':0.0118783)'':0.0324624)'':0.00599965)'':-0.00478104)'':0.0116963)'':0.00320222,((('PDT000312284.2':0.0461516,(('PDT000644876.1':0.0225168,'PDT000645051.1':0.0244981)'':0.0168472,('PDT000313032.2':0.0410025,('PDT000395096.1':0.0291601,('PDT000051500.2':0.0168311,'PDT000241770.1':0.0167154)'':0.0100907)'':0.0139227)'':0.000421589)'':0.00611868)'':0.0143023,(('PDT000538619.1':0.0576441,'PDT000716923.1':0.0638629)'':0.00519061,(('PDT000538622.1':0.050127,('PDT000414861.1':0.0462158,('PDT000417150.1':0.0455084,('PDT000589794.1':0.0316926,'PDT000637159.1':0.0299312)'':0.00605798)'':0.00395504)'':0.00269832)'':0.00954688,(('PDT000533064.1':0.0255002,('PDT000362444.1':0.0130911,('PDT000543283.1':0.00583182,'PDT000395442.1':0.00641068)'':0.00486142)'':0.0192038)'':0.0100739,('PDT000618463.1':0.0602271,('PDT000588033.1':0.0337031,('PDT000495010.1':0.0277062,('PDT000549077.1':0.0159822,('PDT000671624.1':0.0150111,'PDT000694384.1':0.0152352)'':0.0058916)'':0.00660311)'':0.00467741)'':0.0112635)'':-0.0058914)'':0.0093686)'':0.00690923)'':0.00349728)'':0.0426894,((('PDT000537825.1':0.0514152,('PDT000033994.1':0.0120355,'PDT000034426.1':0.0122848)'':0.0391886)'':0.0406949,(('PDT000208706.1':0.0530276,('PDT000534780.1':0.043183,'PDT000548297.2':0.0439655)'':0.00489319)'':0.0280368,(('PDT000537882.1':0.0196035,'PDT000651088.1':0.0185622)'':0.0455602,('PDT000426113.1':0.0512072,('PDT000432555.1':0.0457732,('PDT000535228.1':0.0356926,'PDT000652347.1':0.0353082)'':0.0074204)'':0.0042553)'':0.010935)'':0.024359)'':0.00474445)'':0.0149149,(('PDT000379265.1':0.0742115,('PDT000291208.2':0.036212,'PDT000480959.1':0.0314242)'':0.0264239)'':0.0325868,(('PDT000445008.1':0.0805266,('PDT000535208.1':0.0356541,('PDT000236046.1':0.0138294,'PDT000285332.1':0.0136817)'':0.0265596)'':0.0355499)'':0.0139369,((('PDT000107259.2':0.0630901,'PDT000520784.1':0.0610749)'':0.0206086,('PDT000188136.2':0.0663521,('PDT000602884.2':0.070865,'PDT000714552.1':0.055294)'':0.0024819)'':0.0118825)'':0.00981095,((('PDT000397676.1':0.0644092,('PDT000397595.1':0.0667158,('PDT000669116.1':0.0650645,('PDT000397567.1':0.0621134,'PDT000625875.1':0.0588276)'':0.00532404)'':0.00421)'':0.00259692)'':0.00685765,('PDT000266331.2':0.0891962,(('PDT000422146.1':0.0729318,'PDT000252685.1':0.0687492)'':0.0154274,('PDT000014101.3':0.0635872,('PDT000241781.1':0.050726,'PDT000250147.1':0.050528)'':0.0094598)'':0.0164398)'':0.00141921)'':-0.00180293)'':0.000766234,('PDT000656992.1':0.11491,('PDT000534937.1':0.101673,(('PDT000129839.2':0.0868281,'PDT000357728.1':0.0980459)'':0.00214843,(('PDT000252557.1':0.0619204,('PDT000250120.1':0.0467701,'PDT000250191.1':0.0393784)'':0.0212434)'':0.0178066,(('PDT000251593.1':0.0401034,('PDT000466041.1':0.0158021,'PDT000252615.1':0.0172753)'':0.0129604)'':0.0491141,(('PDT000443163.1':0.084718,('PDT000425769.1':0.0615455,'PDT000654577.1':0.0531865)'':0.011888)'':0.015422,(('PDT000269983.3':0.0593979,'PDT000661581.1':0.0500191)'':0.0193319,('PDT000600985.1':0.108434,(('PDT000498782.1':0.0613968,('PDT000392129.1':0.0469537,('PDT000316124.1':0.0365561,'PDT000576200.1':0.0433501)'':0.00979304)'':0.0107362)'':0.00715311,('PDT000661089.1':0.0851283,((('PDT000669186.1':0.0624473,('PDT000227278.3':0.0331238,'PDT000114758.3':0.0369424)'':0.0247921)'':0.00757506,('PDT000258440.2':0.0769041,((('PDT000228230.1':0.0499405,'PDT000571137.1':0.0599955)'':0.000736383,('PDT000568719.1':0.0231089,'PDT000011814.2':0.0227248)'':0.0277216)'':0.00590778,((('PDT000425639.1':0.0115489,'PDT000713419.1':0.00840923)'':0.0110961,('PDT000022625.2':0.0178346,'PDT000011274.4':0.0184605)'':0.00465454)'':0.0263791,('PDT000548451.2':0.0600413,('PDT000084252.3':0.0510998,(('PDT000502019.1':0.0333993,'PDT000502058.1':0.0337859)'':0.0197624,('PDT000228315.1':0.0546234,('PDT000249773.1':0.0530507,(('PDT000135792.2':0.0318569,'PDT000713235.1':0.0327749)'':0.0045615,('PDT000469953.1':0.0345909,'PDT000415807.1':0.0317644)'':0.00396783)'':0.00637201)'':0.00154946)'':0.00168721)'':0.00206303)'':0.0068575)'':-0.00520295)'':0.00585725)'':0.012905)'':-0.00854455)'':-0.00144871,(('PDT000562123.1':0.0641868,('PDT000502029.1':0.0576226,('PDT000502157.1':0.0183755,'PDT000502168.1':0.0195905)'':0.0267689)'':0.0115004)'':0.00500803,(('PDT000618728.1':0.0594686,('PDT000716896.1':0.019368,'PDT000588056.1':0.0204515)'':0.0337191)'':0.00551384,('PDT000136708.3':0.0807174,(('PDT000720289.1':0.0562637,('PDT000627298.1':0.0457494,('PDT000426092.1':0.0212209,'PDT000502091.1':0.013977)'':0.0183922)'':0.012447)'':0.00362279,('PDT000652573.1':0.0687613,(('PDT000015114.2':0.0221367,'PDT000015115.2':0.0226557)'':0.0235091,((('PDT000444968.1':0.0378553,'PDT000675355.1':0.0385276)'':0.00382497,('PDT000135789.2':0.0323096,('PDT000691093.1':0.030034,('PDT000718896.1':0.0125369,'PDT000252433.1':0.012886)'':0.0125675)'':0.00620958)'':0.0032994)'':0.00267642,(('PDT000687459.1':0.0387833,'PDT000502163.1':0.0328533)'':0.00258712,('PDT000485909.1':0.0365136,(('PDT000373250.1':0.0308873,'PDT000249868.1':0.0258087)'':0.00315817,('PDT000631947.1':0.0332465,'PDT000716873.1':0.0266455)'':0.00307238)'':0.00420876)'':0.00314316)'':7.72817e-05)'':0.00622071)'':0.0142151)'':-0.00989751)'':0.0137036)'':-0.008816)'':0.00349572)'':-0.000732302)'':0.0198416)'':-0.0153973)'':0.0171598)'':-0.00669786)'':-0.000485063)'':0.00076259)'':0.00146658)'':0.00703324)'':0.00875384)'':0.0189541)'':-0.0352087)'':0.00790381)'':0.00428081)'':0.00571311)'':-0.00180148)'':0.0122874)'':0.0755081)'':0.0297396)'':0.0145799)'':0.0258217)'':-0.00890017,(((((('PDT000539906.1':0.121097,'PDT000640342.1':0.129348)'':0.00579249,('PDT000359665.1':0.0572,('PDT000566130.1':0.0486847,('PDT000470097.1':0.0355613,'PDT000248784.1':0.032746)'':0.00684944)'':0.0207483)'':0.0616823)'':0.0371417,((('PDT000469877.2':0.0748246,'PDT000502126.1':0.0667284)'':0.00652723,('PDT000438107.1':0.0560548,('PDT000442397.1':0.0420082,'PDT000443058.1':0.045278)'':0.0244796)'':0.0175002)'':0.0457696,(('PDT000024554.2':0.101609,('PDT000538611.1':0.0857114,('PDT000556928.1':0.0811805,('PDT000587123.1':0.0613767,'PDT000177181.2':0.0534193)'':0.028896)'':0.00993948)'':0.00506473)'':0.00416072,(('PDT000168824.1':0.0638391,'PDT000168864.1':0.0667859)'':0.0185197,('PDT000168740.1':0.0646947,('PDT000417169.1':0.055609,'PDT000708232.1':0.058125)'':0.0135328)'':0.00713267)'':0.0383801)'':0.031859)'':0.041571)'':0.03992,((('PDT000319812.1':0.0764726,'PDT000205112.2':0.0664514)'':0.0323425,('PDT000619634.1':0.0856781,(('PDT000131833.2':0.0524518,('PDT000160297.2':0.0462214,'PDT000632485.1':0.0524697)'':0.00454435)'':0.0116404,(('PDT000466908.1':0.0176057,'PDT000660047.1':0.0208673)'':0.0254287,('PDT000438092.1':0.0339281,('PDT000205094.2':0.0239091,('PDT000155383.2':0.0210865,'PDT000443893.1':0.018835)'':0.00393205)'':0.0130981)'':0.00841346)'':0.0177019)'':0.0336612)'':0.00058822)'':0.0743335,(('PDT000405277.1':0.123063,('PDT000231468.2':0.0968161,(('PDT000310630.2':0.0316561,'PDT000359650.1':0.0278561)'':0.0535577,('PDT000395062.1':0.0633296,'PDT000714063.1':0.0564224)'':0.0202962)'':0.0135564)'':0.0447344)'':0.00809595,(('PDT000394897.1':0.0976999,(('PDT000095825.2':0.040789,'PDT000631997.1':0.0422873)'':0.01893,('PDT000652867.1':0.0572895,('PDT000215567.2':0.0605984,'PDT000471041.1':0.0503216)'':0.00671049)'':0.0174609)'':0.0248222)'':0.0100716,(('PDT000683489.1':0.0938344,('PDT000395029.1':0.0870334,'PDT000424631.1':0.0755626)'':0.0138141)'':0.00262164,(('PDT000562853.1':0.0530232,'PDT000433468.1':0.0504388)'':0.0271197,('PDT000644861.1':0.075309,('PDT000620190.1':0.0671438,('PDT000425753.1':0.0450082,'PDT000631996.1':0.0610348)'':0.0102212)'':0.0110436)'':0.0124026)'':0.0119824)'':0.00128795)'':0.0435719)'':0.0265922)'':0.0426757)'':0.0178475,(((('PDT000502533.1':0.0837381,(('PDT000709068.1':0.0792645,'PDT000588031.1':0.0767595)'':0.00539663,(('PDT000425662.1':0.0522108,'PDT000228217.1':0.0529922)'':0.000760521,('PDT000426580.1':0.0504143,('PDT000411284.1':0.0455842,('PDT000426305.1':0.0438832,('PDT000188347.1':0.0323304,'PDT000362521.1':0.0345445)'':0.00625036)'':0.0029061)'':0.00577095)'':0.00555583)'':0.0208617)'':0.0068905)'':0.0869815,(((('PDT000274723.2':0.0884155,'PDT000251959.1':0.0894095)'':0.00335058,('PDT000557149.1':0.0206283,'PDT000249769.1':0.0217388)'':0.0686344)'':0.0197284,(('PDT000334214.1':0.0438385,'PDT000417103.1':0.0512343)'':0.037005,('PDT000205072.2':0.0727816,('PDT000205091.2':0.0539798,('PDT000615937.1':0.0445443,('PDT000720278.1':0.0334374,('PDT000132340.2':0.0328608,('PDT000426075.1':0.0104591,'PDT000426728.1':0.014526)'':0.0131315)'':0.0059999)'':0.0139057)'':0.0093239)'':0.0231443)'':0.00421712)'':0.0207632)'':0.0213899,((('PDT000124703.1':0.0935313,'PDT000008445.2':0.0925177)'':0.0073601,('PDT000050567.1':0.0997109,('PDT000205435.1':0.0769853,('PDT000426452.1':0.0700863,(('PDT000466971.1':0.0525743,('PDT000235942.2':0.0187959,'PDT000543133.1':0.0202697)'':0.0186993)'':0.0179597,('PDT000397560.1':0.0722419,('PDT000304206.1':0.0703303,('PDT000354288.1':0.0592051,('PDT000228105.1':0.0568984,(('PDT000354893.1':0.0296425,'PDT000557308.1':0.0321831)'':0.0142982,('PDT000003674.2':0.0401235,('PDT000047636.1':0.0211152,('PDT000485237.1':0.0158833,'PDT000538915.2':0.0144825)'':0.00753887)'':0.011101)'':0.00439317)'':0.00852149)'':0.00233032)'':0.00212752)'':0.00744193)'':-0.00202799)'':0.00701839)'':0.00375386)'':0.0244563)'':-0.00515709)'':-0.00765163,((('PDT000471039.1':0.0270191,('PDT000204884.2':0.0234007,'PDT000281239.2':0.021221)'':0.0107525)'':0.0190592,(('PDT000609595.1':0.0333585,('PDT000365308.1':0.0222956,'PDT000394981.1':0.0222209)'':0.0154679)'':0.0130579,('PDT000196588.2':0.0492291,('PDT000395034.1':0.0431863,'PDT000395291.1':0.0351069)'':0.0108566)'':0.000102923)'':0.00265785)'':0.00863638,((('PDT000565843.1':0.054257,'PDT000200194.4':0.055068)'':0.0101847,(('PDT000395468.1':0.0489552,'PDT000395366.1':0.0494274)'':0.00112541,('PDT000034479.3':0.0377769,'PDT000644973.1':0.0369624)'':0.0108969)'':0.00841863)'':-0.00132868,('PDT000060533.2':0.0617475,(('PDT000566173.1':0.0353156,'PDT000594457.1':0.037867)'':0.00136225,('PDT000397667.1':0.0370606,('PDT000395015.1':0.0279359,('PDT000027888.3':0.0151412,'PDT000395113.1':0.01432)'':0.0146009)'':0.00776902)'':0.000767098)'':0.0187957)'':-0.00161964)'':0.00153892)'':0.0357144)'':0.0433236)'':0.0508349)'':0.013705,((('PDT000129854.2':0.129809,('PDT000470047.1':0.0608248,'PDT000252578.1':0.0604392)'':0.0699593)'':0.0538651,((('PDT000631692.1':0.102163,('PDT000625453.1':0.0710111,('PDT000034427.1':0.012579,'PDT000034430.1':0.0128103)'':0.0542458)'':0.0401049)'':-0.00396144,('PDT000017865.2':0.0785007,('PDT000487591.1':0.0685245,('PDT000719029.1':0.0576964,(('PDT000307416.2':0.0532393,'PDT000556317.1':0.0474087)'':0.00545765,('PDT000425834.1':0.0495659,'PDT000557075.2':0.0538361)'':0.0048751)'':0.00374619)'':0.005646)'':0.0032759)'':0.037349)'':0.0452067,(('PDT000347076.1':0.139712,('PDT000482182.1':0.120455,('PDT000568661.1':0.0872027,('PDT000636824.1':0.0722267,(('PDT000485293.1':0.049367,'PDT000641726.1':0.0433264)'':0.00127297,('PDT000557110.1':0.0454666,'PDT000714587.1':0.045988)'':0.00348198)'':0.0160349)'':0.0276137)'':0.010559)'':0.0276938)'':-0.0236157,((('PDT000721990.1':0.0631689,('PDT000434752.1':0.0448993,('PDT000618722.1':0.0314981,'PDT000714236.1':0.0313235)'':0.00816623)'':0.0107423)'':0.0172415,(('PDT000171784.1':0.0324458,('PDT000107793.2':0.0235403,'PDT000583832.1':0.023686)'':0.00447572)'':0.0337283,('PDT000438321.1':0.0892809,(('PDT000251515.1':0.0703778,('PDT000687854.1':0.0495602,'PDT000706411.1':0.0517998)'':0.00936269)'':0.00185433,(('PDT000006903.2':0.0337105,('PDT000347043.1':0.0224071,('PDT000107798.2':0.0143508,'PDT000600787.1':0.0183478)'':0.00333364)'':0.0157936)'':0.00286989,('PDT000645588.2':0.051702,('PDT000502015.1':0.0255589,('PDT000713461.1':0.0197217,'PDT000448206.1':0.0166056)'':0.00567622)'':0.0146942)'':-0.00237478)'':0.0221657)'':0.0185338)'':-0.0132333)'':0.00386509)'':0.00628572,((('PDT000027890.2':0.0374457,'PDT000639581.1':0.0345342)'':0.0169585,('PDT000034473.2':0.0530894,('PDT000645126.1':0.0433575,('PDT000203069.2':0.0389104,'PDT000095731.2':0.0403673)'':0.00342056)'':0.00863913)'':0.000458719)'':0.0351345,('PDT000336254.1':0.0818146,(('PDT000435636.1':0.0968806,('PDT000617728.1':0.0784236,('PDT000713938.1':0.0730536,(('PDT000145164.2':0.0372107,'PDT000131127.2':0.0399283)'':0.0153291,('PDT000687911.1':0.0500312,(('PDT000654368.1':0.0392873,'PDT000081746.1':0.0331118)'':0.00970394,(('PDT000201339.2':0.0290963,'PDT000646845.1':0.0335571)'':0.0118903,(('PDT000426207.1':0.0225256,'PDT000619596.1':0.0210762)'':0.0113408,('PDT000111116.1':0.0258246,'PDT000251691.1':0.0286941)'':0.00376499)'':0.00643073)'':0.00291988)'':0.00519448)'':-0.00249758)'':0.0129161)'':0.00616994)'':0.00837768)'':-0.01757,(('PDT000373158.2':0.0365301,('PDT000321156.2':0.0237353,'PDT000129896.3':0.0235488)'':0.0119461)'':0.00865167,('PDT000201194.3':0.0330561,(('PDT000253172.3':0.0189952,'PDT000333177.2':0.019421)'':0.00102916,('PDT000115013.3':0.017516,('PDT000201217.3':0.011776,(('PDT000655310.2':0.0116779,('PDT000289062.3':0.00529929,'PDT000115040.3':0.00595681)'':0.00268418)'':0.00235589,('PDT000107033.3':0.00922779,('PDT000367747.2':0.00388166,'PDT000131026.3':0.00359144)'':0.00486691)'':0.002544)'':0.00125188)'':0.00326682)'':0.00183273)'':0.0104737)'':0.0164531)'':0.0116414)'':0.0210779)'':-0.0005833)'':0.000652708)'':0.0264825)'':0.0428852)'':0.0308557)'':0.00455744,((('PDT000319005.1':0.134605,('PDT000062934.2':0.0441245,(('PDT000367786.1':0.0210404,'PDT000410710.1':0.0176041)'':0.0033575,(('PDT000007741.2':0.0170271,('PDT000501177.1':0.00971956,'PDT000625391.1':0.0123903)'':0.00511661)'':0.00189067,(('PDT000320962.1':0.00646947,'PDT000205074.2':0.00559193)'':0.00367697,('PDT000534914.1':0.00950202,'PDT000537852.1':0.012718)'':0.00116283)'':0.00676809)'':0.00412451)'':0.0249547)'':0.0790786)'':0.0607792,((('PDT000645660.2':0.12331,('PDT000114638.3':0.00650222,'PDT000115023.3':0.00526718)'':0.0987161)'':0.0560963,(('PDT000163361.3':0.102168,('PDT000389680.1':0.0260874,'PDT000526185.1':0.0291677)'':0.0832373)'':0.00964639,(('PDT000425557.1':0.0897859,('PDT000062952.2':0.0148991,'PDT000357657.1':0.0224767)'':0.0735917)'':0.0143571,('PDT000354936.1':0.106831,(('PDT000373247.1':0.0382408,'PDT000683616.1':0.0315493)'':0.0429062,('PDT000206512.2':0.0172424,('PDT000279619.2':0.0102988,'PDT000042602.1':0.0106955)'':0.00509325)'':0.0510758)'':0.0296461)'':0.00384363)'':0.00740014)'':0.0489304)'':-0.00942939,((('PDT000280806.1':0.068535,('PDT000713587.1':0.0573646,('PDT000212791.1':0.0551466,('PDT000652973.1':0.0334284,'PDT000653010.1':0.0280813)'':0.0262748)'':0.00498545)'':0.0191441)'':0.00368111,('PDT000171434.1':0.0577421,('PDT000517929.1':0.0540549,('PDT000212792.1':0.0458331,('PDT000397578.1':0.0327396,('PDT000644907.1':0.0168935,'PDT000103072.1':0.0175485)'':0.0153011)'':0.00694387)'':0.00736131)'':0.00759229)'':0.0129797)'':0.0954115,(((('PDT000319234.1':0.048616,'PDT000132233.2':0.0433778)'':0.0640301,('PDT000707861.1':0.115039,'PDT000719254.1':0.117017)'':0.0035008)'':0.00617523,(('PDT000357192.1':0.105732,'PDT000248612.1':0.100568)'':0.0188073,(('PDT000682542.1':0.00834711,'PDT000687919.1':0.00679229)'':0.113249,('PDT000274716.2':0.0991641,('PDT000228205.1':0.0949623,('PDT000720188.1':0.0865759,('PDT000354169.1':0.0686178,('PDT000619776.1':0.0593604,'PDT000252454.1':0.0583696)'':0.0060822)'':0.0220231)'':0.00571214)'':0.00616928)'':6.17255e-05)'':0.0235922)'':-0.00153618)'':0.00634691,(('PDT000155990.2':0.0387025,'PDT000520671.1':0.043696)'':0.0362682,(('PDT000625423.1':0.0301422,('PDT000533047.1':0.0232963,'PDT000687064.1':0.0296268)'':0.00382292)'':0.0271873,(('PDT000535380.1':0.0410553,('PDT000538641.1':0.00916025,'PDT000188360.1':0.00839925)'':0.0256622)'':0.0202906,('PDT000161659.2':0.0564654,('PDT000325082.1':0.0473913,('PDT000434563.1':0.0505216,('PDT000131934.2':0.0265396,(('PDT000155260.2':0.0112477,'PDT000357196.1':0.0150655)'':0.00252009,('PDT000157289.2':0.0108861,'PDT000534996.1':0.00852465)'':0.00613866)'':0.0109589)'':0.0226325)'':0.00531723)'':0.00293907)'':0.00023661)'':0.00175126)'':0.0192961)'':0.0430258)'':0.0269252)'':0.00629076)'':0.030615)'':-0.0182601,((((('PDT000433658.1':0.0211861,'PDT000631696.1':0.0212233)'':0.0185718,('PDT000248688.1':0.0284567,('PDT000580394.1':0.0244932,('PDT000600958.1':0.00998547,'PDT000649262.1':0.00991943)'':0.0172428)'':0.0044923)'':0.00678186)'':0.0136462,('PDT000674810.1':0.0498696,('PDT000403979.1':0.0411967,('PDT000652372.1':0.0266654,('PDT000708934.1':0.0166155,(('PDT000535387.1':0.00955386,'PDT000632938.1':0.0113167)'':0.00334949,('PDT000317241.1':0.0135014,('PDT000443352.1':0.00967743,'PDT000362528.1':0.00935127)'':0.00312781)'':0.000648608)'':0.0050343)'':0.00234879)'':0.00780051)'':0.0143551)'':0.00505079)'':0.0269166,(('PDT000252645.1':0.0419437,('PDT000132032.2':0.0266063,'PDT000707845.1':0.0271918)'':0.0194228)'':0.0263575,((('PDT000326092.1':0.0265343,'PDT000397747.1':0.0278464)'':0.00869322,('PDT000333267.1':0.0353961,('PDT000213272.2':0.0278293,('PDT000228052.1':0.018268,('PDT000360391.1':0.0098908,'PDT000651649.1':0.0073902)'':0.0108655)'':0.00812232)'':0.00493038)'':0.00496583)'':0.0207066,('PDT000248811.1':0.0341547,('PDT000251960.1':0.0297466,('PDT000397680.1':0.0258542,('PDT000400779.1':0.0158824,('PDT000517788.1':0.0118223,(('PDT000451172.1':0.00323335,'PDT000650492.1':0.00225896)'':0.00803374,('PDT000486037.1':0.00933572,'PDT000580396.1':0.00759828)'':0.00384308)'':0.00327107)'':0.00345946)'':0.00622895)'':0.0118176)'':0.00366903)'':0.023036)'':0.0131722)'':0.00185612)'':0.0321038,(((('PDT000557273.1':0.0588647,'PDT000520799.1':0.0645793)'':0.0180303,('PDT000045622.1':0.0656391,('PDT000718983.1':0.0330726,'PDT000346205.1':0.034571)'':0.0238131)'':0.0137121)'':0.0168627,(('PDT000047195.3':0.0155471,'PDT000129860.2':0.0182554)'':0.0543722,(('PDT000380283.1':0.0459895,'PDT000557229.1':0.0412013)'':0.0252447,('PDT000415669.1':0.0506987,('PDT000510069.1':0.0598758,('PDT000556895.1':0.0462102,('PDT000557161.1':0.0477428,'PDT000228369.1':0.040678)'':0.00930124)'':0.00325434)'':0.0014575)'':0.01239)'':0.00565602)'':0.0254745)'':0.0174321,(('PDT000721937.1':0.122719,(('PDT000228176.1':0.0719881,('PDT000032364.1':0.00410122,('PDT000032365.1':0.0032761,'PDT000026012.1':0.00290597)'':0.000684491)'':0.0577918)'':0.0139595,(('PDT000421753.1':0.0350731,'PDT000484710.1':0.0258266)'':0.0491873,('PDT000346630.1':0.044628,('PDT000323028.1':0.0333109,('PDT000537842.1':0.0297532,'PDT000634050.1':0.0317567)'':0.00399648)'':0.00697492)'':0.0337384)'':0.0036958)'':0.0178914)'':-0.00680536,((('PDT000535203.1':0.0189287,'PDT000535207.1':0.0224499)'':0.022037,('PDT000536646.1':0.034003,('PDT000042648.1':0.0291282,('PDT000661436.1':0.0348072,(('PDT000176029.2':0.0317024,('PDT000173880.2':0.0268154,'PDT000125939.1':0.0255906)'':0.000987142)'':0.00319833,(('PDT000161688.2':0.0215172,'PDT000616184.1':0.022413)'':0.00834032,('PDT000175951.2':0.0178506,'PDT000176212.2':0.0232549)'':0.00647663)'':0.000357772)'':0.00294481)'':0.00240261)'':0.00188529)'':0.00570617)'':0.0114065,('PDT000424774.1':0.0797455,(('PDT000417134.1':0.0634785,('PDT000425484.1':0.030791,('PDT000397980.1':0.024659,'PDT000426639.1':0.0271595)'':0.012555)'':0.023581)'':0.00189239,(('PDT000218104.2':0.058846,'PDT000520921.1':0.058119)'':0.0115462,('PDT000501901.1':0.0605502,('PDT000200651.1':0.0771157,('PDT000188353.1':0.0648864,('PDT000042722.1':0.0613449,(('PDT000618726.1':0.0286439,('PDT000535537.1':0.0153762,'PDT000618944.1':0.0173913)'':0.0110712)'':0.0162064,(('PDT000443988.1':0.0258484,'PDT000444308.1':0.0329288)'':0.0137872,(('PDT000433326.1':0.0354783,'PDT000191146.1':0.0380511)'':0.0181561,('PDT000718449.1':0.0399437,('PDT000044122.1':0.0412926,((('PDT000017666.2':0.0256883,('PDT000248495.1':0.0052458,'PDT000248502.1':0.0061043)'':0.0113142)'':0.00393067,('PDT000251995.1':0.021907,('PDT000017844.2':0.0156574,('PDT000325632.1':0.0166886,('PDT000135939.2':0.00939858,'PDT000557348.1':0.00769032)'':0.00528428)'':0.00191486)'':0.003338)'':4.67471e-05)'':0.00737756,(('PDT000042567.1':0.0316763,'PDT000042725.1':0.0231387)'':0.0113343,(('PDT000202132.2':0.030292,('PDT000218041.2':0.0314236,('PDT000616160.1':0.0185774,'PDT000103047.1':0.0223312)'':0.00582726)'':0.00195463)'':0.00127938,((('PDT000103040.1':0.0212944,'PDT000019233.2':0.0216782)'':0.00442704,(('PDT000135966.2':0.00727243,'PDT000713441.1':0.00617107)'':0.00783488,('PDT000319035.1':0.011437,('PDT000175865.2':0.011478,'PDT000103065.1':0.0126637)'':0.000516562)'':0.00134322)'':0.00723743)'':0.0059635,(('PDT000673455.1':0.031009,'PDT000103030.1':0.0313932)'':0.00338636,((('PDT000175883.2':0.01817,'PDT000175875.2':0.01959)'':0.00163904,('PDT000425510.1':0.0142023,('PDT000175897.2':0.0110053,'PDT000425621.1':0.0102354)'':0.00325367)'':0.00552482)'':0.00656338,(('PDT000444111.1':0.0152203,'PDT000588014.1':0.0166231)'':0.0118997,(('PDT000042730.1':0.0217901,('PDT000175795.2':0.0188606,'PDT000176217.2':0.0204036)'':0.000753175)'':0.00572786,(('PDT000426343.1':0.0256333,'PDT000482616.1':0.0242587)'':0.00491917,('PDT000426037.1':0.0397319,(('PDT000521140.1':0.0260077,('PDT000673448.1':0.0216713,('PDT000103007.1':0.0159469,('PDT000103043.1':0.00821278,'PDT000352425.1':0.00797072)'':0.00306911)'':0.00478585)'':0.00237302)'':-0.00130585,('PDT000011827.2':0.0221407,('PDT000017764.2':0.0213574,(('PDT000042698.1':0.0134063,('PDT000103010.1':0.00858097,'PDT000103032.1':0.0122523)'':0.00292129)'':0.000888539,('PDT000380163.1':0.0184671,(('PDT000103060.1':0.00955233,'PDT000011795.2':0.00743897)'':0.0013069,('PDT000661529.1':0.0155239,(('PDT000397574.1':0.00890973,'PDT000485949.1':0.00885177)'':0.0012728,('PDT000017667.2':0.0077889,('PDT000176023.2':0.00713578,('PDT000202009.2':0.00696251,('PDT000582022.1':0.00405538,('PDT000136055.2':0.00357444,'PDT000501895.1':0.0030091)'':0.00113155)'':0.000582913)'':0.001099)'':0.000445834)'':0.00234083)'':0.00311906)'':-0.00134467)'':0.00503464)'':-0.00345054)'':0.006317)'':0.00237711)'':-0.00409492)'':0.0101897)'':-0.00450162)'':-0.00111156)'':0.00408598)'':-0.00369225)'':0.00782047)'':-0.00705243)'':0.00395152)'':0.00834706)'':-0.00624913)'':0.00788716)'':0.00507483)'':-0.000915265)'':0.00121521)'':-0.00120698)'':0.0142295)'':0.00340699)'':0.00184596)'':0.0029247)'':-0.0140296)'':-0.00230042)'':0.0355966)'':-0.0316699)'':0.034448)'':0.0208501)'':-0.0106975)'':0.0624775)'':0.0211436)'':0.00412496)'':0.0126501,(((('PDT000136079.2':0.0984923,(('PDT000395699.1':0.045529,'PDT000625444.1':0.0388283)'':0.00253434,('PDT000551134.1':0.0420742,('PDT000649256.1':0.027563,'PDT000675334.1':0.0338191)'':0.0102797)'':0.00433429)'':0.0492428)'':-0.00775176,((('PDT000538589.1':0.0330941,'PDT000636318.1':0.0381188)'':0.0416409,('PDT000682785.1':0.0661753,('PDT000426822.1':0.0533303,('PDT000435631.1':0.0379342,(('PDT000636195.1':0.0280688,'PDT000038895.1':0.0277093)'':0.00804949,('PDT000610654.1':0.0415576,('PDT000610697.1':0.0377557,('PDT000535060.1':0.0256007,'PDT000535392.1':0.0247131)'':0.00721634)'':0.00320372)'':0.00141262)'':0.00313951)'':0.0120962)'':0.00405346)'':0.0158022)'':0.011184,((('PDT000693776.1':0.0534088,('PDT000426595.1':0.0333469,'PDT000011810.2':0.030885)'':0.0151357)'':0.00574432,('PDT000538624.1':0.0436942,('PDT000589805.1':0.0347259,'PDT000520858.1':0.0262606)'':0.00887418)'':0.0106704)'':0.00951841,('PDT000384993.1':0.0225284,(('PDT000384988.1':0.00673795,('PDT000384982.1':0.00616583,'PDT000384995.1':0.00596137)'':0.000730746)'':0.00341769,(('PDT000384986.1':0.00909895,'PDT000384999.1':0.0120341)'':0.00297079,('PDT000127572.1':0.0102405,('PDT000384989.1':0.00861723,('PDT000384985.1':0.00629584,('PDT000384983.1':0.00638594,('PDT000384997.1':0.00638943,('PDT000384990.1':0.00473013,'PDT000384998.1':0.00532067)'':0.000764817)'':0.00025527)'':0.000321779)'':0.00116519)'':0.00050234)'':0.00389059)'':0.0006646)'':0.0142918)'':0.0390654)'':0.0209926)'':0.00479526)'':0.0275419,(('PDT000251968.1':0.115126,(('PDT000710428.1':0.0753031,('PDT000193070.2':0.0320416,'PDT000251881.1':0.0302455)'':0.0426449)'':0.0287529,('PDT000362467.1':0.0935553,('PDT000423295.2':0.066019,(('PDT000713851.1':0.027256,'PDT000488880.1':0.036513)'':0.00750809,('PDT000362483.1':0.0263713,('PDT000661654.1':0.019083,'PDT000362450.1':0.0161073)'':0.00416368)'':0.00987223)'':0.0144132)'':0.0349054)'':0.0213618)'':0.0112698)'':0.00361462,('PDT000502518.1':0.151792,(((('PDT000535524.1':0.0580734,'PDT000662561.1':0.0566356)'':0.0281289,('PDT000566457.1':0.0479687,'PDT000248791.1':0.0368881)'':0.0439447)'':0.00170959,(('PDT000248689.1':0.0518308,('PDT000625317.1':0.0356071,('PDT000205114.2':0.0213099,'PDT000388611.1':0.0182029)'':0.0109219)'':0.0208974)'':0.000444265,(('PDT000205093.2':0.0367001,('PDT000625455.1':0.0203743,'PDT000633682.1':0.0279647)'':0.0143646)'':0.0148755,(('PDT000501181.1':0.0409752,('PDT000253141.2':0.0230774,'PDT000388609.1':0.023716)'':0.0152161)'':-0.00189544,('PDT000651795.1':0.0313374,(('PDT000510678.1':0.0156242,'PDT000649524.1':0.0160809)'':0.0114029,(('PDT000452369.1':0.0229982,'PDT000625460.1':0.0258637)'':0.00369931,(('PDT000365937.1':0.0159787,'PDT000611930.1':0.0164479)'':0.00432419,('PDT000649272.1':0.0187794,('PDT000205057.2':0.0164665,(('PDT000205109.2':0.00791525,'PDT000625395.1':0.00636475)'':0.00481775,('PDT000551011.1':0.0108878,('PDT000333294.1':0.0057605,'PDT000483984.1':0.0055828)'':0.00166704)'':0.00354305)'':0.00307956)'':0.00315743)'':-0.000845072)'':0.00591874)'':0.00584508)'':0.000417247)'':0.0061488)'':0.0166891)'':0.00642662)'':0.0327174)'':0.0124177,(('PDT000032367.1':0.0839926,('PDT000238185.2':0.0950076,('PDT000535249.1':0.0463956,('PDT000535073.1':0.0344594,('PDT000248730.1':0.0364785,('PDT000534987.1':0.0288391,('PDT000651158.1':0.0341707,'PDT000661397.1':0.0295802)'':0.00110787)'':0.00205702)'':0.00176467)'':0.00689775)'':0.045402)'':0.00357369)'':0.00761686,(((('PDT000397619.1':0.0129333,'PDT000249950.1':0.013625)'':0.0363626,('PDT000556937.1':0.0342035,'PDT000557108.1':0.0358713)'':0.0142581)'':0.0343865,('PDT000104651.2':0.080832,('PDT000251938.1':0.0619917,(('PDT000535536.1':0.0324044,'PDT000719034.1':0.0301867)'':0.0312292,('PDT000671612.1':0.063298,('PDT000069653.1':0.043499,('PDT000691299.1':0.0346145,('PDT000664769.1':0.012851,'PDT000664817.1':0.00832154)'':0.0272389)'':0.0123171)'':0.00861175)'':0.00376764)'':0.00693804)'':0.0119786)'':0.00763157)'':0.00306236,(('PDT000323895.1':0.100404,(('PDT000156936.2':0.0639825,'PDT000097365.2':0.0668135)'':0.00706468,('PDT000537893.1':0.0256354,'PDT000285320.1':0.0249826)'':0.0624948)'':0.011882)'':-0.000973309,(('PDT000314220.3':0.0540907,('PDT000406989.1':0.0452867,('PDT000641698.2':0.0378862,'PDT000708529.1':0.0278261)'':0.00621212)'':0.0112535)'':0.0205195,(('PDT000334434.2':0.0166274,'PDT000115016.3':0.0178656)'':0.0482155,('PDT000671357.1':0.034623,(('PDT000400855.2':0.00993226,'PDT000569332.2':0.00958624)'':0.00939622,('PDT000690497.1':0.0194453,'PDT000115048.3':0.0217236)'':0.00339663)'':0.0074456)'':0.035046)'':0.00903958)'':0.0112845)'':0.00334287)'':0.0111781)'':0.00246648)'':0.0487186)'':-0.0335978)'':-4.40598e-05)'':0.0421814,((('PDT000394818.1':0.0825427,('PDT000588030.1':0.068828,(('PDT000026890.2':0.0480987,'PDT000519812.1':0.0416013)'':0.016628,(('PDT000687913.1':0.0514958,('PDT000032718.1':0.0136087,'PDT000233092.1':0.013429)'':0.0352647)'':0.00213856,(('PDT000443799.1':0.0351965,'PDT000522469.1':0.0310563)'':0.00496364,('PDT000645131.1':0.0346192,'PDT000125985.1':0.0371201)'':0.00497376)'':0.0110282)'':0.0027323)'':0.00840566)'':0.00723274)'':0.0455973,(('PDT000299906.2':0.155382,(('PDT000128802.2':0.0529043,'PDT000652914.1':0.0582707)'':0.0585976,('PDT000285623.2':0.0964461,('PDT000687496.1':0.0847115,('PDT000396809.1':0.0640504,('PDT000396788.1':0.0516894,('PDT000557107.1':0.0389175,('PDT000303070.2':0.0312996,('PDT000501900.1':0.0273543,('PDT000557283.1':0.0202912,'PDT000251790.1':0.0171731)'':0.0074451)'':0.00363387)'':0.0031013)'':0.0121372)'':0.00966992)'':0.0243593)'':0.00500377)'':0.015298)'':0.0499775)'':-0.035587,(('PDT000249774.1':0.0964962,(('PDT000318715.1':0.0427358,('PDT000123313.1':0.0462234,('PDT000687074.1':0.034326,'PDT000008763.2':0.0309299)'':0.00541704)'':0.00588321)'':0.00784149,(('PDT000487595.2':0.0363449,'PDT000115038.3':0.0355726)'':0.0142339,('PDT000483995.2':0.0325482,(('PDT000143249.3':0.0175533,'PDT000375757.2':0.0142572)'':0.0108273,(('PDT000189535.3':0.0136186,'PDT000208183.3':0.0149274)'':0.00827486,('PDT000201315.3':0.0176767,'PDT000542703.2':0.0179357)'':0.00575399)'':0.00646795)'':0.00853317)'':0.00543529)'':0.00993057)'':0.0308907)'':0.0189202,(((('PDT000557256.1':0.0722411,('PDT000714145.1':0.0536752,('PDT000249962.1':0.0344091,'PDT000251553.1':0.0268618)'':0.0196699)'':0.00732897)'':-0.00137219,('PDT000551985.2':0.05488,('PDT000329821.1':0.0612171,(('PDT000556913.1':0.0486983,'PDT000557119.1':0.04236)'':0.00362389,('PDT000445053.1':0.0467563,(('PDT000115134.3':0.032651,'PDT000216005.1':0.0284579)'':0.00932678,('PDT000713716.1':0.0441137,('PDT000661618.1':0.0174182,'PDT000661837.1':0.0147183)'':0.0213416)'':0.000807)'':0.00579683)'':0.00110654)'':0.00782256)'':0.00325803)'':-0.000736829)'':0.0109285,(('PDT000426891.2':0.062141,('PDT000143254.3':0.0577812,('PDT000394952.1':0.0412815,('PDT000569427.2':0.0205007,('PDT000203168.3':0.0129086,'PDT000134057.3':0.00815984)'':0.0134626)'':0.0172914)'':0.00805532)'':0.0120111)'':0.00129345,(('PDT000427751.1':0.06461,'PDT000249747.1':0.062067)'':0.0097395,('PDT000661117.1':0.0540381,(('PDT000659010.1':0.0387566,('PDT000106260.2':0.0370924,'PDT000690472.1':0.039452)'':0.00371634)'':0.00475554,(('PDT000451177.1':0.0221251,'PDT000452712.1':0.024307)'':0.00986425,('PDT000505356.1':0.0185155,('PDT000496762.1':0.0111157,'PDT000496775.1':0.0144666)'':0.00575064)'':0.0135018)'':0.0141114)'':0.00533132)'':0.012804)'':-0.00215138)'':0.00515518)'':0.0193902,(('PDT000415666.1':0.0769533,('PDT000640341.1':0.0567477,('PDT000719042.1':0.03499,'PDT000399412.1':0.0273561)'':0.0244192)'':0.0186277)'':0.0231129,((('PDT000311251.2':0.0784648,(('PDT000428367.1':0.0783271,'PDT000685898.1':0.0705569)'':0.00236052,(('PDT000251913.1':0.0466443,('PDT000202778.2':0.0242152,'PDT000627364.1':0.0180689)'':0.0337511)'':0.00174207,('PDT000141127.2':0.0267078,('PDT000362193.2':0.00205329,'PDT000575835.2':0.00253799)'':0.0234474)'':0.0299716)'':0.0137062)'':0.00964742)'':-0.00583724,('PDT000502675.1':0.074564,(('PDT000661516.1':0.0586736,('PDT000339358.1':0.0341536,'PDT000541633.1':0.0336922)'':0.0292795)'':0.00832999,(('PDT000251912.1':0.0625597,('PDT000205005.2':0.0425489,'PDT000249841.1':0.0384039)'':0.0148674)'':0.00151299,(('PDT000624700.1':0.0360858,'PDT000502084.1':0.0334445)'':0.0148969,('PDT000713365.1':0.0475786,('PDT000502222.1':0.0248032,'PDT000501860.1':0.0214243)'':0.0207085)'':0.00548345)'':0.00662657)'':0.00715257)'':0.00591866)'':0.001585)'':0.0119702,((('PDT000652243.1':0.0379799,'PDT000661526.1':0.0369214)'':0.0200344,('PDT000033988.1':0.00194116,('PDT000029847.1':0.00227975,'PDT000033985.1':0.00161402)'':0.000454861)'':0.0577569)'':0.0347778,(('PDT000355761.1':0.0769892,(('PDT000249752.1':0.0613051,('PDT000109477.2':0.0524529,('PDT000412149.1':0.0461101,'PDT000011796.2':0.0428345)'':0.00563551)'':0.00736702)'':0.00104867,(('PDT000307434.2':0.0487936,'PDT000557192.1':0.0528504)'':0.00610775,('PDT000042591.1':0.0615998,('PDT000713344.1':0.0436792,('PDT000644837.1':0.0218981,'PDT000252541.1':0.0199775)'':0.0247974)'':0.00927966)'':-0.00104374)'':0.00372869)'':0.010168)'':-0.00602737,(('PDT000092461.3':0.037089,('PDT000247213.3':0.0303283,'PDT000334398.2':0.0316444)'':0.00392708)'':0.0016836,('PDT000532414.2':0.0349256,('PDT000311086.3':0.0313148,('PDT000525107.2':0.0218812,('PDT000257226.3':0.0198004,('PDT000429541.2':0.0227167,(('PDT000151499.3':0.00759671,('PDT000407579.2':0.00758528,'PDT000719885.1':0.00904982)'':0.000772188)'':0.00775608,('PDT000342249.2':0.0150172,('PDT000421022.2':0.00748273,('PDT000334181.2':0.00184454,'PDT000468291.2':0.0014515)'':0.00463255)'':0.00635792)'':0.00308678)'':0.0028651)'':0.000914045)'':0.000918123)'':0.00466616)'':0.00483758)'':0.00911419)'':0.027815)'':0.028367)'':-0.00588391)'':0.00903115)'':-0.00716947)'':0.00924291)'':0.00558129)'':0.00892154)'':-0.00178877,((('PDT000667703.2':0.251783,(((('PDT000362489.1':0.0248755,('PDT000444109.1':0.0176217,'PDT000661303.1':0.0213381)'':0.00414714)'':0.00632532,('PDT000672258.1':0.0335243,('PDT000661743.1':0.0239554,(('PDT000713839.1':0.0174301,'PDT000661542.1':0.0146528)'':0.00165263,('PDT000661566.1':0.00188346,'PDT000362510.1':0.00181247)'':0.0122622)'':0.00368209)'':0.0116878)'':-0.00588661)'':0.0130781,((('PDT000661532.1':0.0350358,'PDT000661592.1':0.0322654)'':0.000714771,('PDT000636244.1':0.0242918,('PDT000661632.1':0.0174828,'PDT000362492.1':0.0177998)'':0.00530695)'':0.00704328)'':0.00244427,(('PDT000661381.1':0.0243981,('PDT000433695.1':0.024912,'PDT000571139.1':0.0284122)'':0.00272812)'':0.00406394,('PDT000661787.1':0.0303041,('PDT000636280.1':0.0244838,(('PDT000451159.1':0.0130215,'PDT000535450.1':0.0110893)'':0.00308535,('PDT000661367.1':0.0157531,('PDT000535327.1':0.00746822,'PDT000588047.1':0.00885348)'':0.00418339)'':0.000619705)'':0.00696815)'':0.00749073)'':-0.00181819)'':0.0015201)'':0.0123062)'':0.0268517,((('PDT000426829.1':0.0164483,'PDT000636268.1':0.0231586)'':0.0603947,(('PDT000536600.1':0.0542543,'PDT000167657.2':0.0562517)'':0.00328824,('PDT000150267.2':0.0474071,'PDT000626570.1':0.0475762)'':0.0140139)'':0.022976)'':-0.00147242,((('PDT000167656.2':0.0150864,'PDT000713934.1':0.0126281)'':0.0447704,('PDT000132142.2':0.0323995,('PDT000128663.2':0.0276774,('PDT000716300.1':0.0211305,('PDT000316165.1':0.0180557,'PDT000672275.1':0.024844)'':0.00561569)'':0.00722842)'':0.00527337)'':0.0231737)'':0.0117654,(('PDT000565526.1':0.0677608,('PDT000132213.2':0.0540763,(('PDT000714047.1':0.0306624,'PDT000329825.1':0.032571)'':0.00479795,('PDT000607501.1':0.0242512,'PDT000346629.1':0.0260074)'':0.00690045)'':0.0134495)'':0.00668814)'':0.0109643,((('PDT000414386.1':0.0443686,'PDT000709921.1':0.0347752)'':0.0120735,('PDT000425981.1':0.0247633,'PDT000571170.1':0.0215102)'':0.017869)'':0.0114886,((('PDT000433625.1':0.0347171,'PDT000663798.1':0.0370602)'':0.0110445,('PDT000716967.1':0.0303055,('PDT000176698.2':0.01349,'PDT000669467.1':0.0144612)'':0.0147647)'':0.0144199)'':0.00733762,(('PDT000425503.1':0.0540968,'PDT000654691.1':0.0475312)'':0.017632,(('PDT000176758.3':0.0462142,('PDT000227137.2':0.0376447,'PDT000640348.1':0.0305593)'':0.0144067)'':0.000996976,(('PDT000433739.1':0.0464641,'PDT000694476.1':0.0492105)'':0.000841476,(('PDT000688495.1':0.049254,'PDT000346089.1':0.0462618)'':0.00755558,('PDT000321335.1':0.0524734,(('PDT000150956.2':0.0384615,'PDT000502148.1':0.0410774)'':0.00436132,('PDT000362505.1':0.0506832,(('PDT000015293.2':0.0351252,('PDT000426696.1':0.0292867,('PDT000425988.1':0.0173363,'PDT000426065.1':0.0175749)'':0.00574215)'':0.00687367)'':0.00100389,('PDT000279199.1':0.0443345,(('PDT000618727.1':0.019577,'PDT000011743.2':0.0252213)'':0.00739864,('PDT000535034.1':0.0266742,('PDT000319008.1':0.0272422,('PDT000444595.1':0.0246692,(('PDT000652366.1':0.00326508,'PDT000720190.1':0.00331387)'':0.0145173,('PDT000162962.3':0.0121779,('PDT000296182.2':0.00909295,'PDT000248489.1':0.00823255)'':0.00192619)'':0.00412299)'':0.00692049)'':0.00153831)'':0.00183828)'':0.000392234)'':0.00521918)'':-0.00293507)'':0.00866215)'':-0.00312693)'':0.0100466)'':-0.00213179)'':0.00212663)'':0.000126496)'':0.00622525)'':0.000234193)'':0.00159068)'':0.00358973)'':0.00306928)'':0.00614902)'':-0.00587545)'':0.140424)'':-0.125821,(((('PDT000457360.1':0.0317077,'PDT000675300.1':0.0329333)'':0.00723202,('PDT000661460.1':0.0346674,('PDT000566589.1':0.0306609,'PDT000092540.1':0.0281338)'':0.00707491)'':0.00228864)'':0.0481968,(('PDT000077082.2':0.0733557,('PDT000077103.2':0.0606071,('PDT000399660.1':0.0205748,'PDT000395269.1':0.0174525)'':0.0293788)'':0.0109766)'':0.00859079,(('PDT000636296.1':0.0503956,'PDT000661445.1':0.0517154)'':0.020543,(('PDT000519325.1':0.0469039,'PDT000661430.1':0.0507247)'':0.0173916,(('PDT000251666.1':0.0671717,('PDT000175838.2':0.0662848,('PDT000713613.1':0.0536599,('PDT000691364.1':0.0592394,('PDT000661422.1':0.0534593,('PDT000136164.2':0.0445836,(('PDT000188179.2':0.0320749,'PDT000249908.1':0.0312335)'':0.00578268,('PDT000395204.1':0.0382509,('PDT000497248.1':0.0325172,('PDT000425675.1':0.0339431,('PDT000235604.2':0.0305235,('PDT000426304.1':0.0264576,('PDT000400016.1':0.0145808,'PDT000011820.2':0.0154172)'':0.0110432)'':0.00108637)'':0.00364515)'':0.00234517)'':0.00216099)'':-0.00170985)'':0.00450152)'':0.0043075)'':0.00506059)'':0.00150244)'':0.00436888)'':0.00599988)'':-0.010589,((('PDT000672257.1':0.0516736,('PDT000661413.1':0.0446313,('PDT000500755.1':0.0354173,'PDT000661409.1':0.0367656)'':0.00782143)'':0.00436354)'':0.00134634,('PDT000661786.1':0.0409792,('PDT000425670.1':0.0412714,('PDT000520882.1':0.0319362,('PDT000325619.1':0.0221572,'PDT000439399.1':0.0203396)'':0.0103197)'':0.00409276)'':0.00256582)'':0.00969904)'':0.00205705,(('PDT000425976.1':0.0333375,'PDT000426126.1':0.042779)'':0.026588,(('PDT000395083.1':0.0410559,'PDT000691361.1':0.0447973)'':0.00193546,(('PDT000425667.1':0.0364256,'PDT000426068.1':0.0348022)'':0.0147325,('PDT000425915.1':0.0505748,('PDT000384979.1':0.0493191,('PDT000537539.1':0.0491471,(('PDT000135982.2':0.034821,('PDT000150898.1':0.0302458,'PDT000259193.1':0.0278325)'':0.00453752)'':0.00480656,('PDT000463721.1':0.0494083,(('PDT000125951.1':0.0253823,'PDT000257776.1':0.0248179)'':0.00290175,('PDT000137849.2':0.0346149,('PDT000713247.1':0.025725,('PDT000714031.1':0.020457,('PDT000132396.2':0.0172827,'PDT000069960.1':0.0161262)'':0.00401842)'':0.00150072)'':0.00490932)'':-7.9081e-05)'':0.0112595)'':-0.00622267)'':0.00700275)'':0.00319533)'':0.00145183)'':-0.00382385)'':0.00268091)'':0.0120602)'':-0.0087216)'':0.000226056)'':0.0109964)'':0.00674431)'':-0.0037426)'':0.0179173)'':-0.00705906,(('PDT000672543.1':0.05855,('PDT000034440.1':0.0353257,(('PDT000468241.1':0.0213916,'PDT000600199.1':0.0176169)'':0.00259134,(('PDT000549215.1':0.0144056,'PDT000625400.1':0.0145122)'':0.00556397,('PDT000649265.1':0.0211328,('PDT000465802.1':0.0189666,('PDT000205095.2':0.0153246,('PDT000408626.1':0.0108311,('PDT000635711.1':0.00329823,('PDT000480379.1':0.00263782,'PDT000485850.1':0.00265424)'':0.000438916)'':0.00548957)'':0.00291217)'':0.00250337)'':0.00220585)'':-0.000982036)'':0.00237943)'':0.0171673)'':0.0176478)'':0.0135909,((('PDT000411099.1':0.0684906,('PDT000426526.1':0.0341058,'PDT000535351.1':0.0357066)'':0.0205787)'':0.00644622,(('PDT000178583.2':0.0459627,'PDT000204889.2':0.0410483)'':0.0177664,('PDT000645616.2':0.0588523,'PDT000362509.1':0.0567117)'':0.00692884)'':0.00504955)'':0.0123944,((('PDT000674900.1':0.0413178,('PDT000661631.1':0.0372157,('PDT000644983.1':0.0203582,('PDT000328126.1':0.00494933,'PDT000661819.1':0.00581727)'':0.0131581)'':0.00859905)'':0.0105519)'':-0.000543934,('PDT000425760.1':0.0471332,('PDT000543311.1':0.0452104,('PDT000445044.1':0.0328436,('PDT000661356.1':0.0237875,('PDT000716918.1':0.0198494,'PDT000661402.1':0.0180742)'':0.00219275)'':0.00735486)'':0.00474715)'':0.0034346)'':0.00107423)'':0.0211241,(((('PDT000566042.1':0.0295887,'PDT000566160.1':0.035701)'':0.0252401,('PDT000644948.1':0.0539509,('PDT000149563.2':0.0493593,('PDT000325023.1':0.0419913,('PDT000435593.1':0.0305324,'PDT000616149.1':0.0388971)'':0.00626139)'':0.00285659)'':0.005266)'':0.00258128)'':0.00629336,('PDT000645039.1':0.0542352,(('PDT000342660.1':0.0382392,('PDT000230579.2':0.0295948,'PDT000594978.1':0.0295716)'':0.00646302)'':0.00439888,('PDT000480903.1':0.035085,('PDT000428353.1':0.0245508,'PDT000566714.1':0.0213108)'':0.0117534)'':0.00313364)'':0.00689501)'':0.0133055)'':0.00558125,(('PDT000543277.1':0.0390693,('PDT000714004.1':0.0231771,('PDT000367746.1':0.0222995,'PDT000210707.1':0.0213213)'':0.0051084)'':0.00611288)'':0.0302658,(('PDT000581605.1':0.0624291,('PDT000337147.1':0.0498558,('PDT000176393.1':0.0427133,('PDT000190133.2':0.0244122,('PDT000673620.1':0.0156956,'PDT000706413.1':0.0170263)'':0.00689603)'':0.00957169)'':0.00914949)'':0.00375942)'':-0.00193693,(('PDT000188066.2':0.0408624,'PDT000616151.1':0.0450834)'':0.0161363,(('PDT000306637.2':0.0530924,'PDT000311270.2':0.0480786)'':0.00711931,((('PDT000167715.2':0.0169426,'PDT000673604.1':0.0212026)'':0.0120923,('PDT000497324.1':0.0213812,'PDT000361015.1':0.0229934)'':0.00789001)'':0.0127755,(('PDT000317618.1':0.0346652,('PDT000312803.2':0.0112426,'PDT000435583.2':0.0133899)'':0.0216568)'':0.00390915,('PDT000519822.1':0.050037,(('PDT000077060.3':0.0435377,'PDT000694525.1':0.0397714)'':0.00450029,('PDT000236349.5':0.0401083,(('PDT000167948.2':0.0237757,'PDT000673610.1':0.0190056)'':0.00949184,(('PDT000161879.2':0.0158041,'PDT000136051.2':0.016368)'':0.012661,(('PDT000713356.1':0.0238765,'PDT000713651.1':0.0198469)'':0.00318743,('PDT000120876.2':0.0221438,('PDT000713440.1':0.00747939,'PDT000571147.1':0.00673531)'':0.0168351)'':0.00395017)'':0.000909428)'':0.000940307)'':0.00545725)'':0.00362311)'':0.0045849)'':-0.00425029)'':0.00156983)'':0.00892254)'':0.00535732)'':-0.00327899)'':0.0132909)'':-0.00321073)'':0.00257334)'':0.00343951)'':0.00588469)'':0.00508419)'':0.00297013)'':0.0150739,(((('PDT000106118.1':0.0546482,('PDT000713339.1':0.0286063,('PDT000042742.1':0.0304161,'PDT000661296.1':0.0277654)'':0.00220091)'':0.0108506)'':0.007607,(('PDT000319773.1':0.0385776,'PDT000448185.1':0.0365624)'':0.013995,('PDT000135998.2':0.0450391,(('PDT000661412.1':0.0282051,('PDT000162961.3':0.0171661,'PDT000604967.1':0.0188058)'':0.0103073)'':0.00549646,(('PDT000669579.1':0.0290568,'PDT000713975.1':0.029881)'':0.00374889,('PDT000694367.1':0.0267247,'PDT000328138.1':0.0237219)'':0.00669256)'':0.00418495)'':0.00677386)'':0.00183971)'':0.00787531)'':0.00362397,(('PDT000632132.1':0.070254,('PDT000188147.2':0.0398891,('PDT000618843.1':0.0388911,('PDT000543095.1':0.0257834,('PDT000538598.1':0.0269433,('PDT000103000.1':0.0223682,('PDT000538563.1':0.0203925,'PDT000714155.1':0.025546)'':0.00111932)'':0.00397777)'':0.00121986)'':0.00527931)'':0.00592456)'':0.027325)'':-0.0187726,(('PDT000502190.1':0.0431642,(('PDT000520987.1':0.0328912,'PDT000583369.1':0.0277475)'':0.00106744,('PDT000426139.1':0.0140237,'PDT000626806.1':0.0182184)'':0.0143789)'':0.0105818)'':1.54869e-05,(('PDT000502177.1':0.0341587,('PDT000716897.1':0.0364147,('PDT000397874.1':0.0305582,('PDT000015343.2':0.027135,('PDT000426559.1':0.0241824,('PDT000625968.1':0.0188211,'PDT000626687.1':0.0182087)'':0.00796958)'':0.00167067)'':0.000960272)'':0.0049894)'':0.00279651)'':-0.00106954,('PDT000713848.1':0.0359272,('PDT000535493.1':0.040516,(('PDT000616081.1':0.0288875,('PDT000714159.1':0.0233442,'PDT000543316.1':0.0196602)'':0.00373304)'':0.000419735,(('PDT000435624.1':0.0200154,('PDT000233107.1':0.016967,'PDT000360995.1':0.0169897)'':0.00553672)'':0.000895752,('PDT000188332.1':0.0188487,('PDT000534899.1':0.0158668,('PDT000501983.1':0.0147897,'PDT000714549.1':0.0145205)'':0.00304832)'':0.00139068)'':0.00510965)'':0.00348778)'':0.0075971)'':0.00172863)'':-0.0026814)'':0.00434815)'':0.0112587)'':0.00577958)'':0.0234638,((('PDT000631882.1':0.0782002,('PDT000631876.1':0.0754164,('PDT000631760.1':0.0417398,('PDT000631878.1':0.0344955,('PDT000631881.1':0.0320713,('PDT000631907.1':0.0343299,('PDT000631762.1':0.0244847,('PDT000631880.1':0.0195193,(('PDT000631911.1':0.00896982,('PDT000631901.1':0.00625777,('PDT000631908.1':0.00817327,('PDT000631905.1':0.0051432,'PDT000631906.1':0.0053244)'':0.00154688)'':0.00070439)'':0.00134121)'':0.000762249,('PDT000631877.1':0.0112231,('PDT000631879.1':0.00734278,('PDT000631902.1':0.00436025,('PDT000631884.1':0.00327266,('PDT000631873.1':0.00273532,'PDT000631875.1':0.00285071)'':0.000417147)'':0.00115335)'':0.0012836)'':0.00284898)'':-0.000948319)'':0.00516755)'':0.00346514)'':0.00450787)'':0.000851269)'':0.000608991)'':0.00254446)'':0.0186875)'':0.00127307)'':0.0131763,(('PDT000433603.1':0.0467874,'PDT000618890.1':0.0521169)'':0.0320934,(((('PDT000526183.1':0.0463069,('PDT000713333.1':0.045124,('PDT000661712.1':0.0330477,(('PDT000661393.1':0.0262796,'PDT000347091.1':0.0222422)'':0.00410025,('PDT000668324.1':0.0184777,('PDT000713735.1':0.00882501,'PDT000717023.1':0.0109269)'':0.00424757)'':0.00532214)'':0.0111609)'':0.00759782)'':0.00260555)'':0.00059463,(('PDT000063283.2':0.0512009,('PDT000397902.1':0.0382359,'PDT000694406.1':0.0325925)'':0.0124045)'':0.0030194,(('PDT000707683.1':0.0351424,'PDT000661742.1':0.0301613)'':0.0132575,('PDT000407991.1':0.0405894,('PDT000581876.1':0.0399097,('PDT000492231.1':0.0340261,('PDT000397865.1':0.0359789,(('PDT000397608.1':0.02402,'PDT000433228.1':0.0236317)'':0.00126272,('PDT000086326.1':0.021439,('PDT000687844.1':0.0179124,'PDT000714158.1':0.0193657)'':0.00358013)'':0.00401347)'':0.00733346)'':0.00142265)'':0.00312063)'':0.00301169)'':0.00561483)'':0.00399568)'':-0.00152827)'':0.00671716,(('PDT000189441.1':0.0821783,('PDT000535347.1':0.00950196,('PDT000243749.2':0.00798233,'PDT000433684.1':0.00867777)'':0.000294541)'':0.0495027)'':-0.00142974,((('PDT000017724.2':0.0303457,('PDT000661699.1':0.0279212,('PDT000015337.2':0.0110723,('PDT000661556.1':0.0131631,'PDT000661679.1':0.0102433)'':0.00147713)'':0.00761598)'':0.00564508)'':0.00474273,(('PDT000616066.1':0.0294293,('PDT000132064.2':0.0248649,'PDT000714129.1':0.0239682)'':0.000574203)'':8.92079e-05,('PDT000583368.1':0.0165592,('PDT000636253.1':0.0110191,'PDT000636314.1':0.0114628)'':0.00394854)'':0.0102118)'':0.0120504)'':0.0115763,(('PDT000713346.1':0.0454102,('PDT000137860.2':0.0176433,'PDT000694409.1':0.0251598)'':0.0206857)'':0.0063762,(('PDT000713506.1':0.0381262,('PDT000672236.1':0.0433046,(('PDT000443898.1':0.0129247,'PDT000713469.1':0.0182068)'':0.0128223,('PDT000006416.2':0.029271,('PDT000102989.1':0.023199,('PDT000356251.1':0.0126455,'PDT000125936.1':0.0152064)'':0.00917775)'':0.00359868)'':0.000230831)'':0.00555971)'':0.00190055)'':0.00118417,(('PDT000017772.2':0.0388383,('PDT000714085.1':0.024645,'PDT000571162.1':0.020353)'':0.0150426)'':0.00112429,('PDT000323666.1':0.0394671,('PDT000672276.1':0.0383403,('PDT000137854.2':0.0309829,('PDT000017803.2':0.0251973,'PDT000415778.1':0.0264445)'':0.00236077)'':0.00305253)'':0.00295919)'':0.00304828)'':0.00184728)'':0.00951542)'':-0.0023559)'':0.0140769)'':-0.00775526)'':0.00454138,((('PDT000361001.1':0.0405902,('PDT000694398.1':0.0246512,'PDT000270465.1':0.0211315)'':0.0101746)'':0.0145888,('PDT000521281.1':0.0408427,('PDT000362463.1':0.0372202,('PDT000578967.1':0.0205686,('PDT000661341.1':0.0212336,('PDT000661523.1':0.016875,('PDT000713312.1':0.0107456,('PDT000564710.1':0.00396319,'PDT000714124.1':0.00450126)'':0.00300446)'':0.00548769)'':0.0017046)'':0.0049316)'':0.0067165)'':0.00368183)'':0.0146063)'':0.00336508,((('PDT000520892.1':0.042058,('PDT000425850.1':0.0313705,'PDT000661736.1':0.0319898)'':0.0131602)'':0.000274256,('PDT000661454.1':0.0439511,(('PDT000136146.2':0.0367833,'PDT000017708.2':0.0335368)'':0.00271897,(('PDT000713503.1':0.0295822,('PDT000017732.2':0.0208799,('PDT000132076.2':0.0215806,'PDT000713505.1':0.0236816)'':0.00261357)'':0.00461122)'':0.00397398,('PDT000011766.2':0.0272999,('PDT000132060.2':0.0257477,('PDT000135995.2':0.0224527,('PDT000135983.2':0.0175156,('PDT000713252.1':0.00922668,'PDT000713990.1':0.0106461)'':0.00624414)'':0.00302063)'':0.00499908)'':0.0035868)'':0.000557264)'':0.0013632)'':0.00879989)'':0.00111126)'':0.00822929,(('PDT000616186.1':0.0381312,('PDT000017726.2':0.0281958,('PDT000017728.2':0.0258897,('PDT000318985.1':0.0169762,'PDT000017648.2':0.0177269)'':0.00392108)'':0.000658102)'':0.0161008)'':0.00938723,(('PDT000694438.1':0.0177623,('PDT000713287.1':0.0178693,'PDT000713903.1':0.0214221)'':0.00437572)'':0.0237961,('PDT000502066.1':0.0365964,('PDT000501856.1':0.0456652,('PDT000346760.1':0.0380469,('PDT000661459.1':0.0425725,(('PDT000548295.2':0.023982,('PDT000520886.1':0.0188175,('PDT000714098.1':0.0183942,('PDT000694411.1':0.013574,'PDT000714161.1':0.0150342)'':0.00299319)'':0.00183989)'':0.00135952)'':0.00605467,('PDT000535213.1':0.0284495,(('PDT000540590.1':0.0375406,('PDT000589966.1':0.0268323,('PDT000397694.1':0.0182726,('PDT000346699.1':0.017023,(('PDT000433458.1':0.00966861,'PDT000501916.1':0.0113348)'':0.00189937,(('PDT000535020.1':0.0108441,'PDT000501890.1':0.0122267)'':0.000856667,('PDT000535001.1':0.00861294,('PDT000633016.1':0.00710699,'PDT000651230.1':0.00634731)'':0.00188736)'':0.00318276)'':0.00136588)'':0.00377895)'':0.00465714)'':0.00521523)'':0.00482911)'':-0.00910914,(('PDT000427049.1':0.0146956,'PDT000535157.1':0.0178658)'':0.00572608,(('PDT000502043.1':0.0144828,('PDT000534900.1':0.008798,'PDT000535238.1':0.0129693)'':0.00575255)'':0.00620716,('PDT000640364.1':0.0255496,('PDT000397735.1':0.0193462,('PDT000534934.1':0.0117265,('PDT000390155.1':0.0118769,('PDT000535052.1':0.0118581,('PDT000175863.2':0.0105531,('PDT000362458.1':0.0067192,('PDT000535163.1':0.00541732,'PDT000535283.1':0.00493608)'':0.0020273)'':0.00205897)'':0.000833065)'':0.00128904)'':0.00101969)'':0.00442113)'':0.00260707)'':0.000448046)'':0.00259619)'':-0.000395813)'':0.00823645)'':-0.00548755)'':0.0137369)'':0.000324413)'':0.00116667)'':0.00113382)'':0.00701846)'':0.00486096)'':-0.00205169)'':0.0031306)'':0.00199011)'':0.0178089)'':-0.0141364)'':0.0100643,(((('PDT000150873.1':0.0544259,'PDT000384971.1':0.0540361)'':0.0230841,('PDT000352515.1':0.0543589,'PDT000017694.2':0.0592341)'':0.0113564)'':0.00581592,(('PDT000414201.1':0.0695299,(('PDT000714069.1':0.0171729,'PDT000017692.2':0.019289)'':0.00905034,('PDT000425656.1':0.0342136,('PDT000661520.1':0.0137922,('PDT000713932.1':0.00517882,'PDT000713976.1':0.00650248)'':0.00737078)'':0.0120576)'':0.00423429)'':0.0266236)'':-0.00705738,(('PDT000536596.1':0.0358448,'PDT000521120.1':0.0341416)'':0.0135004,(('PDT000132059.2':0.00772397,'PDT000625509.1':0.00660153)'':0.0247428,('PDT000713449.1':0.027162,('PDT000390977.1':0.0235737,('PDT000702587.1':0.0172396,('PDT000502498.1':0.0177662,('PDT000011816.2':0.00894255,'PDT000017819.2':0.0101929)'':0.00572315)'':0.00237528)'':0.00204545)'':0.00230294)'':0.00412117)'':0.0179454)'':-0.000507021)'':0.0257648)'':0.010489,(((('PDT000535147.1':0.0634354,(('PDT000132341.2':0.0224226,'PDT000347001.1':0.0239448)'':0.00799609,('PDT000543275.1':0.0315711,('PDT000618959.1':0.0112859,'PDT000352487.1':0.00885696)'':0.022595)'':0.00395673)'':0.0269593)'':0.00273188,(('PDT000157173.2':0.0380084,'PDT000536635.1':0.0414241)'':0.00239429,('PDT000015340.2':0.0367404,('PDT000652337.1':0.025656,('PDT000274282.2':0.0161464,('PDT000571146.1':0.00982667,'PDT000718814.1':0.0108011)'':0.00672758)'':0.0027603)'':0.009721)'':0.00893779)'':0.0223088)'':0.0115096,((('PDT000128850.2':0.0968668,('PDT000417124.1':0.0534562,'PDT000238072.2':0.0539708)'':0.0302707)'':0.0116513,(('PDT000137827.2':0.0664058,'PDT000426776.1':0.0649292)'':0.00597393,(('PDT000589768.1':0.0404851,'PDT000675261.1':0.0357559)'':0.0278511,(('PDT000188149.2':0.067355,('PDT000136057.2':0.0605391,('PDT000102758.1':0.0480735,('PDT000534928.1':0.0192465,'PDT000537808.1':0.0207181)'':0.0252823)'':0.00804068)'':0.00285309)'':-0.00112024,(('PDT000423869.1':0.0519999,'PDT000555616.1':0.0508771)'':0.005165,(('PDT000656969.1':0.0563926,'PDT000133944.1':0.0547394)'':0.00556429,((('PDT000520739.1':0.0424667,'PDT000521000.1':0.0441672)'':0.00139344,('PDT000710124.1':0.0383672,('PDT000173724.2':0.0287272,('PDT000538642.1':0.0164609,'PDT000252514.1':0.0128369)'':0.0138412)'':0.00962971)'':0.00659362)'':-0.000417969,('PDT000205418.1':0.0424419,(('PDT000262729.1':0.0284924,('PDT000570880.1':0.0124235,'PDT000042665.1':0.00854504)'':0.0182731)'':0.00464332,('PDT000535437.1':0.0385961,('PDT000361008.1':0.0373816,('PDT000661154.1':0.0270521,('PDT000534904.1':0.023164,'PDT000019259.2':0.0224066)'':0.000661796)'':0.00952902)'':0.00410337)'':0.00161082)'':0.00211474)'':3.31119e-05)'':0.00886179)'':0.00446917)'':-0.000306608)'':0.0073221)'':0.00771393)'':0.00644121)'':-0.00845097,((('PDT000520881.1':0.054108,(('PDT000415799.1':0.0334346,('PDT000713212.1':0.0178454,'PDT000015321.2':0.0214208)'':0.0136194)'':0.0117868,(('PDT000486285.1':0.0347538,('PDT000658614.1':0.0290593,'PDT000661383.1':0.0351421)'':0.00484403)'':0.00152376,(('PDT000571149.1':0.0226061,'PDT000173908.1':0.0222066)'':0.00925521,('PDT000445013.1':0.0434736,('PDT000425955.1':0.0333948,('PDT000433434.1':0.0256433,('PDT000175963.2':0.0217273,'PDT000543315.1':0.017535)'':0.0029216)'':0.0106148)'':0.00501504)'':-0.000599131)'':0.00109922)'':0.00586337)'':0.0188639)'':0.00251581,(('PDT000633967.1':0.0596246,(('PDT000346870.1':0.0287373,('PDT000535076.1':0.0319109,'PDT000713556.1':0.0254403)'':0.0020587)'':0.00346716,('PDT000502220.1':0.0269896,('PDT000543223.1':0.0229877,('PDT000434536.1':0.0148902,'PDT000415830.1':0.0119675)'':0.010212)'':0.0017547)'':0.00880648)'':0.0216479)'':-0.00342517,('PDT000426971.1':0.0752254,('PDT000433303.1':0.058879,(('PDT000636240.1':0.0358334,('PDT000571189.1':0.0296924,('PDT000132164.2':0.0163647,('PDT000415787.1':0.0137085,('PDT000673420.1':0.00716914,('PDT000136143.2':0.00654198,'PDT000571185.1':0.00804632)'':0.00143351)'':0.00476917)'':0.00237427)'':0.00891433)'':0.00584087)'':-0.00316276,('PDT000443039.1':0.0409798,(('PDT000713892.1':0.0235038,('PDT000425464.1':0.0173455,('PDT000015979.3':0.00304874,(('PDT000015980.2':0.00230829,'PDT000015981.2':0.00218228)'':0.000933351,('PDT000015987.2':0.00297568,'PDT000016000.2':0.00311093)'':0.000412922)'':0.000422414)'':0.011555)'':0.00661529)'':-0.00473219,(('PDT000713948.1':0.0181025,'PDT000713750.1':0.0154652)'':0.00209389,('PDT000188368.1':0.00473624,('PDT000188375.1':0.00638796,('PDT000188351.1':0.00265124,('PDT000188344.1':0.00274636,'PDT000188348.1':0.00264825)'':0.000141109)'':0.00141322)'':0.0019137)'':0.0112442)'':0.00223255)'':0.0210715)'':-0.0126841)'':0.0206606)'':0.0192875)'':-0.0143693)'':0.00626404)'':0.000206156,((('PDT000426645.1':0.038587,'PDT000441823.1':0.0426059)'':0.00801753,('PDT000392139.1':0.0383897,('PDT000582121.1':0.0241084,'PDT000720320.1':0.0200556)'':0.00916518)'':0.00939843)'':0.01727,(('PDT000426123.1':0.0746573,'PDT000426337.1':0.0831677)'':0.00974455,((('PDT000397929.1':0.0193425,('PDT000362326.1':0.0121765,('PDT000433229.1':0.00502426,'PDT000415810.1':0.00365578)'':0.00455909)'':0.0089851)'':0.0241301,('PDT000252793.2':0.0439609,(('PDT000487588.1':0.0297557,'PDT000694433.1':0.0298192)'':0.0111409,(('PDT000582014.1':0.0145114,'PDT000713981.1':0.0147023)'':0.0162939,('PDT000674856.1':0.030324,(('PDT000694457.1':0.0225659,'PDT000661664.1':0.0209722)'':0.00639255,(('PDT000694404.1':0.0159628,'PDT000713478.1':0.0195958)'':0.0037879,('PDT000433225.1':0.0193465,('PDT000713633.1':0.019658,('PDT000571141.1':0.00934127,'PDT000571153.1':0.0106846)'':0.00454706)'':0.00405461)'':0.0022592)'':0.00257719)'':0.00237649)'':0.00520792)'':0.00585273)'':0.000857927)'':0.00334801)'':-0.00151849,('PDT000415833.1':0.0546647,('PDT000415816.1':0.0636287,(('PDT000571171.1':0.0350166,('PDT000427075.1':0.019257,('PDT000427533.1':0.0153435,'PDT000713762.1':0.0111341)'':0.00211783)'':0.0139966)'':0.00530829,('PDT000120938.2':0.0403494,(('PDT000175979.2':0.0329866,('PDT000704944.1':0.0213937,'PDT000030566.1':0.0206619)'':0.0120469)'':0.0012428,('PDT000310648.2':0.0439293,('PDT000426399.1':0.0375835,(('PDT000705416.1':0.0197645,('PDT000225513.2':0.0227233,'PDT000713341.1':0.0188648)'':0.00216083)'':0.00908351,('PDT000352410.1':0.0373351,('PDT000042620.1':0.0327694,(('PDT000397756.1':0.0222019,('PDT000019250.2':0.019636,('PDT000705176.1':0.0171526,'PDT000407969.1':0.0159462)'':0.00109976)'':0.000610659)'':0.00456236,(('PDT000705420.1':0.0215736,'PDT000103031.1':0.0178058)'':0.00541721,('PDT000648058.1':0.0207653,('PDT000704943.1':0.0191577,'PDT000069706.1':0.0207711)'':0.00235439)'':0.000354724)'':0.000760244)'':0.00524199)'':0.00282388)'':-0.00117503)'':0.00167515)'':0.00384278)'':-0.00398184)'':0.0106952)'':-0.00741798)'':0.00920117)'':0.00796067)'':-0.0128335)'':0.0313995)'':-0.00838069)'':-0.00350136)'':0.00819786)'':0.00723007)'':-0.00802723,((('PDT000427509.1':0.0759596,('PDT000713296.1':0.0406632,(('PDT000132218.2':0.0324248,'PDT000694395.1':0.0365908)'':0.0049239,('PDT000289029.2':0.0391829,(('PDT000069634.1':0.0284222,('PDT000135991.2':0.0223261,('PDT000535426.1':0.0151918,'PDT000694396.1':0.0157259)'':0.00433267)'':0.00458158)'':0.00246829,(('PDT000566860.1':0.0285278,'PDT000714114.1':0.028198)'':0.00200936,('PDT000107915.1':0.028527,('PDT000397810.1':0.0216255,'PDT000397932.1':0.0198166)'':0.00808578)'':0.00163883)'':0.00162116)'':0.00580699)'':-0.000422209)'':0.0113945)'':0.0044433)'':-0.0131512,(('PDT000019321.2':0.0475861,('PDT000397981.1':0.0344023,'PDT000714041.1':0.0295426)'':0.00735931)'':-0.00161478,(('PDT000661489.1':0.0444982,('PDT000397655.1':0.0316338,('PDT000588024.1':0.0250468,('PDT000651857.1':0.0191773,'PDT000201775.1':0.0213338)'':0.000793506)'':0.0101242)'':0.00392744)'':-0.00486338,((('PDT000443226.1':0.0153542,'PDT000694443.1':0.0174311)'':0.0108037,('PDT000713764.1':0.0183596,('PDT000135963.2':0.0186928,'PDT000233099.1':0.0196819)'':0.00285976)'':0.00252261)'':0.00126776,(('PDT000425812.1':0.0318329,'PDT000409993.1':0.0295067)'':0.000938295,((('PDT000713785.1':0.0226647,'PDT000713794.1':0.0158159)'':0.00468935,(('PDT000673530.1':0.0152605,'PDT000115032.2':0.0194521)'':0.00257929,('PDT000618839.1':0.0196318,('PDT000640359.1':0.0120491,('PDT000713722.1':0.00932683,('PDT000017640.2':0.0034694,'PDT000460548.1':0.00350966)'':0.00390974)'':0.00193224)'':0.0082711)'':-0.00143253)'':0.00329493)'':-0.000331258,(('PDT000125958.1':0.0211103,('PDT000713977.1':0.0142771,('PDT000136158.2':0.0139326,'PDT000714067.1':0.0108214)'':0.00177907)'':0.00387345)'':0.00181887,(('PDT000018735.1':0.0158891,('PDT000397899.1':0.0162965,'PDT000019253.2':0.0156499)'':0.00262867)'':0.00410894,('PDT000633700.1':0.0187991,('PDT000324103.1':0.0122868,('PDT000713867.1':0.00601214,'PDT000015312.2':0.00574666)'':0.00494235)'':0.00470397)'':0.00197041)'':0.0024841)'':0.00127341)'':0.0061704)'':-0.00353551)'':0.00434371)'':0.00502445)'':-0.000273956)'':0.0177602,(('PDT000216026.1':0.0616858,('PDT000566097.1':0.0600643,('PDT000652935.1':0.0601273,('PDT000426261.1':0.0556002,('PDT000652955.1':0.0603395,('PDT000024324.3':0.0406737,('PDT000568775.1':0.0338243,'PDT000354255.1':0.0328553)'':0.00437851)'':0.0100664)'':0.00283352)'':0.00263466)'':0.00760608)'':0.00976729)'':0.026268,('PDT000522684.1':0.0607256,((('PDT000661353.1':0.0420671,(('PDT000713630.1':0.0351987,(('PDT000130850.2':0.0107213,'PDT000661457.1':0.0118888)'':0.0167378,('PDT000713507.1':0.0230885,'PDT000535502.1':0.0250782)'':0.00379247)'':0.00226315)'':0.000244968,('PDT000694420.1':0.0339658,('PDT000713364.1':0.0217676,('PDT000661634.1':0.0161363,('PDT000137852.2':0.0152171,('PDT000714097.1':0.00416506,'PDT000640350.1':0.00473316)'':0.00767602)'':0.00367117)'':0.00430051)'':0.00779428)'':-0.000475665)'':0.0124429)'':-0.0105188,(('PDT000661771.1':0.0305922,('PDT000713349.1':0.0174805,'PDT000042525.1':0.0170396)'':0.00718477)'':0.00363845,('PDT000675323.1':0.035386,(('PDT000233094.1':0.0242951,'PDT000125976.2':0.0263066)'':0.002495,(('PDT000714036.1':0.0208362,('PDT000713705.1':0.0209712,('PDT000136048.2':0.0146573,'PDT000443865.1':0.0139339)'':0.00534202)'':0.00404007)'':0.00105785,('PDT000713234.1':0.032803,('PDT000713535.1':0.0335718,(('PDT000358734.1':0.00455447,('PDT000358735.1':0.00266473,'PDT000358736.1':0.0023228)'':0.00233249)'':0.013351,('PDT000694442.1':0.0151321,('PDT000042478.1':0.01558,('PDT000011768.2':0.0118737,('PDT000136161.2':0.0135702,'PDT000444151.1':0.011541)'':0.00139329)'':0.00340176)'':0.000947549)'':-0.000681538)'':0.00879991)'':0.00263035)'':-0.0040136)'':0.00588719)'':0.00263126)'':-0.00168621)'':0.00130593)'':0.0162908,((('PDT000673560.1':0.0275728,'PDT000673608.1':0.0262696)'':0.00972067,('PDT000125940.1':0.044325,('PDT000713521.1':0.0267991,('PDT000397605.1':0.0243822,('PDT000661449.1':0.0142407,'PDT000125960.1':0.0166607)'':0.00552006)'':0.00558312)'':0.00585001)'':0.00206192)'':0.0173216,((('PDT000236854.1':0.0607257,('PDT000069673.1':0.0561174,(('PDT000136029.2':0.030837,('PDT000427559.1':0.0170845,'PDT000009632.2':0.0199781)'':0.007956)'':0.00696215,(('PDT000019606.1':0.0366501,('PDT000131842.2':0.0317184,('PDT000017705.2':0.0288941,'PDT000017818.2':0.0271643)'':0.00818314)'':0.000944592)'':0.00267743,(('PDT000137846.2':0.0228927,'PDT000714150.1':0.0210849)'':0.00924326,('PDT000713598.1':0.026602,'PDT000017822.2':0.0316609)'':0.00548446)'':0.00343012)'':0.00188319)'':0.0170554)'':0.00412404)'':-0.0155141,(('PDT000136072.2':0.0529255,('PDT000444108.1':0.033851,('PDT000132166.2':0.022078,'PDT000443847.1':0.0222338)'':0.0125854)'':0.00978159)'':-0.00147279,(('PDT000636373.1':0.0415954,('PDT000501924.1':0.0354441,'PDT000042458.1':0.0353161)'':0.00536736)'':0.000933703,('PDT000718300.1':0.044951,(('PDT000005861.2':0.0249432,'PDT000017765.2':0.0268687)'':0.00213662,(('PDT000538920.2':0.018606,'PDT000005862.2':0.0157647)'':0.00594088,('PDT000017753.2':0.0281963,(('PDT000538924.2':0.0214107,('PDT000017790.2':0.0191717,('PDT000005198.3':0.0176819,'PDT000017760.2':0.0163799)'':0.00140693)'':0.00332132)'':0.000162133,('PDT000397606.1':0.025753,('PDT000713270.1':0.0217895,('PDT000640358.1':0.0154158,('PDT000442387.1':0.0153675,'PDT000442396.1':0.0145378)'':0.00273629)'':0.00154447)'':0.00416276)'':-0.00150999)'':0.00361419)'':0.000472289)'':0.00172558)'':0.012716)'':0.000667757)'':0.000486112)'':-0.0010808)'':0.00207437,('PDT000661482.1':0.0716083,((('PDT000538152.1':0.0414889,'PDT000675200.1':0.0370791)'':0.00463073,(('PDT000685884.1':0.0477636,('PDT000042634.1':0.0262833,('PDT000042684.1':0.0206465,('PDT000202763.2':0.0216514,'PDT000042528.1':0.0180516)'':0.000986094)'':0.00340751)'':0.00975928)'':-0.00229545,(('PDT000188110.2':0.0143034,'PDT000409477.1':0.0156946)'':0.0102637,('PDT000415782.1':0.0293688,('PDT000673226.1':0.023383,('PDT000175819.2':0.02124,('PDT000103037.1':0.0174068,('PDT000668383.1':0.013005,'PDT000672266.1':0.0146248)'':0.00275343)'':0.00228807)'':0.00277679)'':0.00137938)'':-1.06302e-05)'':0.0042078)'':0.00785138)'':0.00458698,(('PDT000444113.1':0.038783,(('PDT000713299.1':0.0172295,('PDT000443036.1':0.00694433,'PDT000443064.1':0.0102248)'':0.00617646)'':0.0148284,('PDT000444002.1':0.0202324,('PDT000444037.1':0.0120512,'PDT000444591.1':0.0135924)'':0.0147025)'':0.00857281)'':0.00636806)'':0.00657831,(('PDT000534881.1':0.0179389,'PDT000534954.1':0.0157645)'':0.0249808,('PDT000694516.1':0.0455834,('PDT000425720.1':0.0412091,(('PDT000713471.1':0.0362376,'PDT000017788.2':0.031461)'':0.00471302,(('PDT000673445.1':0.0362721,'PDT000713890.1':0.0390009)'':0.00170326,(('PDT000714055.1':0.0294482,('PDT000588017.1':0.027363,('PDT000136137.2':0.0192397,'PDT000694517.1':0.0211673)'':0.00494981)'':0.00543022)'':-0.000483623,(('PDT000172051.3':0.0271326,'PDT000427047.1':0.0206666)'':0.00213156,(('PDT000017681.2':0.015778,'PDT000017796.2':0.0154473)'':0.00900879,('PDT000018736.1':0.0277059,('PDT000714148.1':0.0259756,('PDT000135942.2':0.0211556,('PDT000017654.2':0.0143756,('PDT000017742.2':0.0128858,('PDT000011720.2':0.010699,('PDT000135927.2':0.00761926,'PDT000103003.1':0.00797004)'':0.0020647)'':0.00374872)'':0.00159484)'':0.00143649)'':0.00413152)'':0.00181078)'':-0.00135343)'':0.000186195)'':0.00477206)'':0.00530485)'':0.00115225)'':0.00487892)'':0.000663236)'':0.000119335)'':-0.000953102)'':0.00287511)'':0.0151797)'':-0.0154966)'':0.00844402)'':-0.00506383)'':0.00840631)'':0.0251429)'':-0.025984)'':0.0143224)'':0.0176336)'':-0.0145474)'':0.00693383)'':0.0198496)'':0.0129912)'':0.0418916)'':0.044236)'':0.0297133)'':0.00526743,(((('PDT000136918.1':0.00587747,'PDT000251681.1':0.00920293)'':0.282429,((('PDT000518121.1':0.0896621,('PDT000584543.1':0.0665078,('PDT000345970.1':0.0459967,'PDT000369030.1':0.0554033)'':0.0204177)'':0.0229131)'':0.0119482,(('PDT000583477.1':0.0919456,('PDT000671285.1':0.0701996,('PDT000132552.1':0.0439876,('PDT000287385.1':0.0038229,('PDT000292883.1':0.00321756,'PDT000358767.1':0.00327724)'':0.00126742)'':0.0429506)'':0.0267241)'':0.0114186)'':-0.00914097,(('PDT000672723.1':0.0671636,'PDT000689794.1':0.0603624)'':0.0152923,(('PDT000630269.1':0.0615956,('PDT000638927.2':0.0489623,('PDT000205003.3':0.0504687,'PDT000668366.2':0.0529203)'':0.0129687)'':0.00486749)'':0.00470385,('PDT000671287.1':0.0584587,('PDT000722592.1':0.0481243,(('PDT000689860.1':0.0281783,'PDT000251745.1':0.0321632)'':0.00134282,(('PDT000380307.2':0.0139447,'PDT000228382.1':0.0221398)'':0.0140683,('PDT000702614.1':0.018137,'PDT000722595.1':0.0178583)'':0.0130138)'':0.00313483)'':0.0149683)'':0.0115268)'':-0.000206153)'':0.0128378)'':0.00397838)'':0.0209644)'':0.143001,((((('PDT000341218.1':0.0506316,('PDT000109444.2':0.0475923,'PDT000488295.1':0.0444889)'':0.00477008)'':0.153263,('PDT000425806.1':0.098736,('PDT000249746.1':0.0898061,'PDT000251447.1':0.0799739)'':0.0308995)'':0.0921314)'':0.0126425,(('PDT000502923.1':0.187101,('PDT000675648.1':0.17536,'PDT000359654.1':0.167461)'':0.0199614)'':0.0139703,('PDT000234197.2':0.192134,('PDT000124971.3':0.0329582,('PDT000190726.3':0.0173198,'PDT000230035.3':0.0169383)'':0.0100402)'':0.162731)'':0.00800416)'':0.0122537)'':0.000879728,((('PDT000541642.1':0.0793419,'PDT000488302.1':0.0851211)'':0.118645,('PDT000505171.2':0.0361614,'PDT000242512.1':0.0359563)'':0.163046)'':0.0157381,(('PDT000645610.2':0.215088,('PDT000403366.1':0.205289,('PDT000251804.1':0.196077,('PDT000158843.2':0.147266,('PDT000184583.1':0.079527,'PDT000184587.1':0.046582)'':0.119882)'':0.0341531)'':0.00772316)'':0.00186725)'':0.0077863,(('PDT000140465.3':0.0825148,('PDT000619564.1':0.0238018,'PDT000661637.1':0.0306522)'':0.0502497)'':0.13228,('PDT000713734.1':0.197243,(('PDT000149352.2':0.137658,('PDT000604238.1':0.076928,'PDT000205448.1':0.080577)'':0.0636765)'':0.0362251,(('PDT000345956.1':0.0770916,('PDT000305464.2':0.0402701,'PDT000389655.1':0.0447105)'':0.0227656)'':0.0117666,('PDT000111709.2':0.0685804,('PDT000274051.3':0.0355998,('PDT000465860.2':0.0208422,('PDT000351841.2':0.0189222,('PDT000390228.2':0.00596727,'PDT000101027.3':0.00489363)'':0.00988551)'':0.00551959)'':0.0111965)'':0.0377972)'':0.00564988)'':0.106627)'':0.0234564)'':-0.00160736)'':0.00447997)'':0.00763724)'':-0.000423784)'':0.000308377,(((('PDT000622924.1':0.0488831,'PDT000641346.1':0.0486428)'':0.156359,(('PDT000011760.2':0.148337,('PDT000691086.1':0.0382866,'PDT000719004.1':0.0318803)'':0.10329)'':0.0528908,('PDT000006905.2':0.208663,(('PDT000414673.2':0.0216151,'PDT000574556.2':0.0223496)'':0.00576906,('PDT000550041.2':0.0246022,('PDT000549250.2':0.0233956,('PDT000084254.3':0.0212007,('PDT000151498.3':0.0171186,'PDT000212470.3':0.0144886)'':0.00247442)'':0.00204929)'':0.00371136)'':0.00272913)'':0.168891)'':0.000644774)'':0.00520538)'':0.0138774,((('PDT000228258.1':0.132417,(('PDT000323555.1':0.042137,'PDT000405212.1':0.041148)'':0.0246825,('PDT000566025.1':0.0172175,'PDT000248518.1':0.0177682)'':0.0460431)'':0.052033)'':0.0469163,(('PDT000517617.1':0.0972996,'PDT000046099.1':0.0988914)'':0.0577436,('PDT000652920.1':0.162447,('PDT000109065.2':0.140435,'PDT000572857.1':0.137883)'':0.0117755)'':0.00254709)'':0.00832205)'':0.00397529,(('PDT000577101.1':0.085707,'PDT000201777.1':0.097667)'':0.0212184,((('PDT000584548.1':0.0485126,'PDT000265969.2':0.0533344)'':0.00685839,('PDT000640120.1':0.0649235,('PDT000346102.1':0.0454215,('PDT000619630.1':0.0219276,'PDT000228279.1':0.0266944)'':0.0114037)'':0.0203468)'':0.00417369)'':0.0145575,((('PDT000312791.2':0.03279,'PDT000566115.1':0.0423003)'':0.0118417,('PDT000228173.1':0.0472935,'PDT000346494.1':0.0357676)'':0.00312722)'':0.0196807,('PDT000394916.1':0.0596235,('PDT000502896.1':0.0650426,('PDT000619780.1':0.0553871,'PDT000502679.1':0.0593199)'':0.00306089)'':0.00398603)'':0.0066827)'':0.011649)'':0.039474)'':0.0522258)'':0.0417947)'':0.000782611,(('PDT000084344.1':0.0985522,(('PDT000251989.1':0.0351734,('PDT000040391.3':0.0230906,'PDT000069713.1':0.0217862)'':0.0102702)'':0.0268051,('PDT000248798.1':0.0731921,(('PDT000161969.2':0.0325575,'PDT000635772.1':0.037309)'':0.0196158,('PDT000268688.2':0.0467637,('PDT000190750.2':0.0404057,('PDT000150236.2':0.0219589,'PDT000719025.1':0.0255524)'':0.0123968)'':0.00953301)'':0.00575016)'':0.00600954)'':0.00523938)'':0.0228196)'':0.129686,(('PDT000641348.1':0.209421,('PDT000098345.3':0.0661504,('PDT000362564.2':0.0410831,('PDT000404018.2':0.0305518,('PDT000480950.2':0.0349853,('PDT000710775.1':0.0292441,('PDT000069910.3':0.0185286,'PDT000300544.1':0.022308)'':0.00612077)'':0.0019615)'':0.00343903)'':0.00549497)'':0.0246463)'':0.13998)'':0.0127424,(('PDT000289984.1':0.255422,(('PDT000499463.1':0.0270626,'PDT000678614.1':0.0292576)'':0.173662,(('PDT000216205.2':0.103902,('PDT000103034.1':0.0977569,'PDT000252440.1':0.0855591)'':0.011234)'':0.0946704,(('PDT000293483.1':0.0570299,('PDT000691088.1':0.0419842,'PDT000248814.1':0.0355978)'':0.0111466)'':0.151474,(('PDT000497379.1':0.0646257,'PDT000142537.3':0.0614973)'':0.127725,('PDT000279468.2':0.186552,('PDT000435640.1':0.187707,('PDT000719227.1':0.121643,(('PDT000494558.1':0.00304969,'PDT000566700.1':0.00303267)'':0.0937333,('PDT000121050.2':0.0251802,('PDT000719281.1':0.019542,'PDT000250170.1':0.0179752)'':0.00676043)'':0.058551)'':0.0389981)'':0.0584767)'':0.00749589)'':0.00261255)'':0.00104423)'':0.00622453)'':0.00848114)'':0.0224802)'':-0.0304907,((('PDT000653000.1':0.0657962,'PDT000395151.1':0.0635238)'':0.0556593,('PDT000713877.1':0.0656362,('PDT000631561.1':0.0327269,('PDT000394847.1':0.022207,'PDT000588648.1':0.0194431)'':0.00701335)'':0.0427516)'':0.0514086)'':0.0743222,(('PDT000205446.1':0.0788097,('PDT000208167.2':0.0605791,'PDT000617731.1':0.0616799)'':0.0168573)'':0.104877,(('PDT000465408.1':0.038908,'PDT000482861.1':0.0478142)'':0.135451,(('PDT000517648.1':0.0553376,('PDT000502104.1':0.0367505,('PDT000582974.1':0.0139127,'PDT000249887.1':0.0163843)'':0.019915)'':0.0169184)'':0.100014,(('PDT000323505.1':0.060238,('PDT000417504.1':0.0608242,('PDT000204897.2':0.0560578,('PDT000294553.2':0.0417317,'PDT000565924.1':0.0376903)'':0.0112289)'':0.0125202)'':0.00319939)'':0.0211282,('PDT000425583.1':0.0314554,(('PDT000132035.2':0.0115798,'PDT000619716.1':0.00995651)'':0.00806595,('PDT000105071.2':0.00962679,('PDT000557305.1':0.011051,'PDT000016366.2':0.00797292)'':0.00168341)'':0.00441938)'':0.0101099)'':0.0454553)'':0.0841637)'':0.015757)'':0.00469611)'':0.0165086)'':0.00615673)'':0.00669031)'':0.00989086)'':-0.00347862)'':0.00140552)'':0.00389747)'':0.0447481)'':-0.0394684,(((('PDT000019607.1':0.0767564,('PDT000252602.1':0.0521719,'PDT000359684.1':0.0433045)'':0.0161589)'':0.0150435,((('PDT000274715.2':0.0702139,'PDT000547574.1':0.0708311)'':0.0026385,('PDT000713891.1':0.0228012,'PDT000248447.1':0.0230641)'':0.0507658)'':0.00669348,(('PDT000006424.2':0.0440601,('PDT000400014.1':0.0356404,'PDT000019258.2':0.0354063)'':0.00864469)'':0.0297176,(('PDT000434498.1':0.0497363,('PDT000691210.1':0.0342986,'PDT000717243.1':0.0343564)'':0.0176939)'':0.0139688,((('PDT000583049.1':0.0652091,'PDT000448212.1':0.0724539)'':0.00189215,('PDT000198421.3':0.0562025,('PDT000502203.1':0.0546873,('PDT000002973.3':0.0477412,('PDT000502158.1':0.0281113,('PDT000713486.1':0.0186219,'PDT000009629.2':0.013414)'':0.01602)'':0.0103116)'':0.00391199)'':0.00560613)'':0.00898491)'':-0.00355362,(('PDT000143055.2':0.0461239,'PDT000505146.1':0.0461045)'':0.0169005,(('PDT000718919.1':0.0496333,'PDT000354249.1':0.0574577)'':0.00616471,('PDT000409800.1':0.049147,(('PDT000400440.1':0.025686,'PDT000691208.1':0.0290654)'':0.00585245,('PDT000409033.1':0.0336696,('PDT000103083.1':0.0313638,('PDT000141596.2':0.0205496,('PDT000372164.1':0.0132857,'PDT000359170.2':0.0154642)'':0.00235386)'':0.0142413)'':0.00456368)'':0.00217068)'':0.00734164)'':0.0140553)'':0.00240746)'':0.00343581)'':0.00398785)'':0.00514621)'':0.00277969)'':0.00581623)'':0.123844,((('PDT000136066.2':0.152488,(('PDT000238530.1':0.0877935,('PDT000032202.1':0.0646557,'PDT000409986.1':0.0631443)'':0.012268)'':0.0400142,('PDT000228234.1':0.0722299,('PDT000718970.1':0.0706945,('PDT000672661.1':0.062086,(('PDT000557138.1':0.0268515,'PDT000248768.1':0.0314009)'':0.0206801,('PDT000557343.1':0.0269294,'PDT000719062.1':0.0245455)'':0.0266315)'':0.00668417)'':0.00615948)'':0.0163181)'':0.042779)'':0.0381497)'':0.0221993,(('PDT000618841.1':0.0682882,'PDT000502065.1':0.0553558)'':0.0136624,(('PDT000557015.1':0.0853363,('PDT000566110.1':0.075971,'PDT000482864.1':0.080239)'':0.0110852)'':-0.0002625,('PDT000583457.1':0.0786646,(('PDT000565894.1':0.0386718,'PDT000395266.1':0.036037)'':0.0250664,(('PDT000713439.1':0.0520084,('PDT000713511.1':0.022523,'PDT000251494.1':0.020759)'':0.015265)'':0.00792456,('PDT000339373.1':0.0471265,('PDT000604850.1':0.0408205,('PDT000556882.1':0.02868,'PDT000691309.1':0.0285018)'':0.0149971)'':0.00259613)'':0.00427332)'':0.00735413)'':0.00700247)'':0.0112864)'':0.00278576)'':0.0994853)'':0.0211501,((('PDT000394688.2':0.0201289,'PDT000498153.1':0.0216073)'':0.0243436,('PDT000619603.1':0.0470437,('PDT000426504.1':0.0503641,('PDT000123324.1':0.0324474,('PDT000123323.1':0.030091,(('PDT000212362.3':0.0106505,'PDT000498019.1':0.0118925)'':0.00433265,('PDT000497913.1':0.0173008,('PDT000140507.3':0.0152695,('PDT000497848.1':0.0151483,'PDT000497866.1':0.0149039)'':0.00048648)'':0.00387777)'':0.000423102)'':0.00985923)'':0.0026297)'':0.017088)'':0.00390966)'':-0.0049038)'':0.0197279,(('PDT000124589.1':0.0350689,'PDT000357676.1':0.0283176)'':0.0106092,((('PDT000324116.2':0.0209674,('PDT000326266.2':0.0128782,'PDT000599128.2':0.0147483)'':0.00452911)'':0.00445433,(('PDT000213873.3':0.0171907,'PDT000318570.2':0.0205991)'':0.00179642,('PDT000251956.1':0.0189914,('PDT000544478.2':0.0154487,'PDT000707719.1':0.0114736)'':0.0071385)'':0.00149172)'':0.00374635)'':0.00437735,(('PDT000498025.1':0.0240082,('PDT000498020.1':0.0219665,('PDT000201167.3':0.0198428,'PDT000134044.3':0.0184861)'':0.00383592)'':0.00102023)'':0.00307849,('PDT000498070.1':0.0314003,(('PDT000400849.2':0.0171518,'PDT000502258.1':0.0174329)'':0.00574968,(('PDT000176785.3':0.0130105,('PDT000548604.2':0.0122386,('PDT000443345.2':0.00860581,'PDT000566788.2':0.0110157)'':0.00391741)'':0.00162942)'':0.00125525,('PDT000296053.3':0.0186529,('PDT000443254.2':0.0086999,('PDT000142851.3':0.00982322,('PDT000491914.2':0.00784552,('PDT000147509.3':0.00339344,'PDT000689211.1':0.00298678)'':0.00234087)'':0.00211082)'':0.00216421)'':0.00717575)'':-0.00156088)'':0.00637306)'':0.00576379)'':-0.00423065)'':0.00221048)'':0.0156597)'':0.0195051)'':0.126648)'':0.00672147)'':0.00379417,((('PDT000565528.1':0.124363,('PDT000095702.2':0.0598377,('PDT000557126.1':0.0336904,'PDT000714669.1':0.0286198)'':0.0262842)'':0.0282258)'':0.114477,((('PDT000196621.2':0.0538679,'PDT000140569.3':0.0495051)'':0.110327,('PDT000619594.1':0.0843359,'PDT000060917.3':0.0890491)'':0.0670398)'':0.0328302,('PDT000652568.1':0.0839759,('PDT000129903.2':0.0759385,('PDT000717613.1':0.0543092,'PDT000248819.1':0.0412341)'':0.0271589)'':0.0211813)'':0.100688)'':0.00516793)'':0.00328821,(('PDT000024457.1':0.192222,(('PDT000425796.1':0.0538542,'PDT000501977.1':0.045297)'':0.110353,(('PDT000252446.1':0.114719,('PDT000427224.1':0.0323381,('PDT000426513.1':0.0246551,'PDT000501979.1':0.0236706)'':0.00533298)'':0.0800915)'':0.0160112,(('PDT000095691.2':0.0140572,'PDT000228222.1':0.0122508)'':0.0328999,('PDT000652961.1':0.0564224,(('PDT000713911.1':0.0481501,('PDT000469388.1':0.026765,'PDT000714551.1':0.0261487)'':0.0121248)'':0.00448692,(('PDT000150245.2':0.0396079,'PDT000017824.2':0.0394004)'':0.00161592,('PDT000641696.1':0.050079,('PDT000460014.1':0.0384204,('PDT000242953.1':0.0345064,(('PDT000622934.1':0.0208559,'PDT000488298.1':0.0160638)'':0.00625894,('PDT000011784.2':0.0242852,('PDT000253184.2':0.0136571,'PDT000102977.1':0.0181109)'':0.00564937)'':0.000257466)'':0.00763373)'':0.00262889)'':0.00561646)'':-0.00218904)'':-1.53861e-06)'':0.0099162)'':-0.00274709)'':0.0831562)'':0.0285922)'':0.0314999)'':-0.00298788,(((('PDT000426104.1':0.0749643,('PDT000188359.1':0.035531,('PDT000354252.1':0.0118392,'PDT000354276.1':0.0100252)'':0.0160626)'':0.0468532)'':0.00456733,('PDT000251828.1':0.0644385,(('PDT000354950.1':0.0351731,'PDT000616087.1':0.0455183)'':0.0142287,('PDT000394898.1':0.0494849,'PDT000719005.1':0.0444281)'':0.00686361)'':0.00587227)'':0.00778277)'':0.00422207,(('PDT000397661.1':0.0567709,('PDT000397920.1':0.0306746,('PDT000285614.2':0.0275461,'PDT000568767.1':0.0206838)'':0.00984024)'':0.0235326)'':0.0139046,((('PDT000052520.2':0.0248429,'PDT000540521.1':0.0297757)'':0.0392108,('PDT000644866.1':0.0551267,('PDT000397621.1':0.0395325,'PDT000479856.1':0.0381892)'':0.00560182)'':0.014541)'':0.00745212,(('PDT000290522.2':0.0667213,'PDT000645009.1':0.0604687)'':0.00366661,('PDT000397827.1':0.0651663,('PDT000190553.2':0.0320817,('PDT000305495.2':0.0233653,'PDT000304213.1':0.0222004)'':0.00331466)'':0.0344982)'':-0.000246484)'':0.00877611)'':0.00392193)'':0.00948303)'':0.0241654,(((('PDT000343493.1':0.0567429,('PDT000694386.1':0.0306571,'PDT000713579.1':0.0290095)'':0.0221128)'':0.0166254,(('PDT000131799.2':0.0226613,'PDT000661678.1':0.0238754)'':0.0467509,(('PDT000570879.1':0.0504129,('PDT000320264.1':0.0411958,'PDT000661723.1':0.0450689)'':0.00944492)'':0.00679115,('PDT000716780.1':0.0593874,('PDT000672253.1':0.0290066,'PDT000520986.1':0.031828)'':0.0214983)'':0.00985745)'':0.0111445)'':0.0027439)'':0.0152811,(('PDT000713315.1':0.0425969,'PDT000150857.1':0.0575211)'':0.0281148,('PDT000716976.1':0.100507,(('PDT000017637.2':0.0603111,('PDT000557029.1':0.0438302,'PDT000713585.1':0.043466)'':0.0119668)'':-0.00274206,(('PDT000309315.2':0.0567264,'PDT000459677.1':0.0493166)'':0.00370422,(('PDT000661297.1':0.0404141,'PDT000661324.1':0.0339789)'':0.00636278,('PDT000354891.1':0.033679,('PDT000380284.1':0.0305282,'PDT000228183.1':0.0331617)'':0.00364137)'':0.00583613)'':0.00556825)'':0.00938293)'':0.0228919)'':-0.00573204)'':0.0133833)'':0.0027203,(('PDT000428899.1':0.0516768,('PDT000443035.1':0.0420653,'PDT000228152.1':0.0361432)'':0.0143256)'':0.0719681,('PDT000584736.1':0.107081,(('PDT000311253.2':0.0689826,('PDT000254351.2':0.0566114,('PDT000415658.1':0.0290354,'PDT000357590.1':0.0283593)'':0.0167248)'':0.0124705)'':0.0109067,(('PDT000502107.1':0.0347266,('PDT000664930.1':0.0155916,'PDT000664931.1':0.0277311)'':0.0197408)'':0.032577,(('PDT000051848.2':0.0691063,'PDT000655773.1':0.0526827)'':0.0146897,('PDT000304210.1':0.0729624,((('PDT000397693.1':0.0142542,'PDT000636814.1':0.00996591)'':0.0309891,('PDT000524071.1':0.0458875,(('PDT000382844.1':0.0100951,'PDT000228240.1':0.0103719)'':0.0101124,('PDT000714013.1':0.0178716,'PDT000395426.1':0.0161844)'':0.00409079)'':0.0179387)'':0.000921705)'':0.0121313,(('PDT000203692.1':0.0508319,('PDT000372797.1':0.0298802,'PDT000580422.1':0.029315)'':0.0129126)'':-0.000774058,('PDT000135993.2':0.037488,('PDT000636138.1':0.0286426,('PDT000583263.1':0.0229182,('PDT000425984.1':0.0218264,('PDT000713372.1':0.0146134,'PDT000011801.2':0.0129223)'':0.00262542)'':0.000864284)'':0.00801062)'':0.00320402)'':0.0121845)'':0.0112717)'':0.0119183)'':-0.00118975)'':8.81527e-05)'':0.00350809)'':0.02825)'':0.00976609)'':-0.0187098)'':0.00684516)'':0.0886343)'':0.0102546)'':0.00662625)'':0.0159903)'':0.00089782,((((('PDT000363552.1':0.0428038,('PDT000397591.1':0.0235851,'PDT000490358.1':0.0212766)'':0.0227729)'':0.0237704,(('PDT000426437.1':0.00862456,'PDT000426813.1':0.0114193)'':0.0232146,('PDT000249935.1':0.0119923,'PDT000251545.1':0.0116594)'':0.0217163)'':0.0433103)'':0.0267427,(('PDT000357682.1':0.0787077,('PDT000357589.1':0.0272654,('PDT000161033.2':0.026447,'PDT000358755.1':0.0276897)'':0.00486543)'':0.040835)'':0.0287305,(('PDT000354167.1':0.0196688,'PDT000357628.1':0.0224148)'':0.0588043,(('PDT000346754.1':0.0437659,('PDT000319212.1':0.037905,('PDT000714151.1':0.0216308,'PDT000011718.2':0.0243836)'':0.0139423)'':0.00900309)'':0.0183825,('PDT000357581.1':0.0753667,(((('PDT000136037.2':0.0278776,'PDT000019134.2':0.0303596)'':0.015793,('PDT000354165.1':0.0317446,('PDT000160818.2':0.0222411,'PDT000160820.2':0.0205007)'':0.0171534)'':0.00531835)'':0.00584666,(('PDT000716889.1':0.0287931,('PDT000694484.1':0.022028,'PDT000017650.2':0.0277395)'':0.00209306)'':0.00721807,('PDT000713805.1':0.0339248,('PDT000713601.1':0.0293558,('PDT000713685.1':0.0164029,'PDT000248830.1':0.0140469)'':0.00917559)'':0.00729961)'':0.00264744)'':0.0152595)'':0.00388208,(('PDT000714045.1':0.0392044,'PDT000716885.1':0.0445519)'':0.00948417,('PDT000249786.1':0.0410961,(('PDT000443069.1':0.020679,'PDT000443075.1':0.0254445)'':0.0160972,(('PDT000153290.2':0.0270088,'PDT000661836.1':0.0244438)'':0.00481583,(('PDT000157245.2':0.0267185,('PDT000248619.1':0.0245616,('PDT000557257.1':0.0149619,'PDT000557259.1':0.0176586)'':0.00481101)'':0.00388311)'':-0.00126174,('PDT000190541.2':0.0332961,('PDT000019454.2':0.025507,('PDT000405279.1':0.0168433,'PDT000068048.1':0.0153671)'':0.00934951)'':0.00385572)'':-0.000930482)'':0.000691137)'':0.00754912)'':0.00712858)'':0.00412467)'':0.00470013)'':0.012492)'':-0.00329139)'':0.0109186)'':0.0295189)'':-0.00404556)'':0.019538,(((('PDT000426301.1':0.0453057,('PDT000635916.1':0.0327213,'PDT000644525.1':0.0359803)'':0.0105815)'':0.00574844,('PDT000668920.1':0.0589034,(('PDT000618794.1':0.0291165,'PDT000210712.1':0.03443)'':0.0112223,((('PDT000030559.1':0.0212535,'PDT000407982.1':0.0225843)'':0.0119508,('PDT000673422.1':0.0323481,('PDT000434490.1':0.0214609,('PDT000622542.1':0.012966,('PDT000222833.2':0.00874605,'PDT000444871.1':0.00775005)'':0.00589846)'':0.00746908)'':0.00123172)'':0.00304957)'':0.00414512,('PDT000427066.1':0.0484791,('PDT000125984.1':0.0394151,('PDT000691310.1':0.0258622,('PDT000427012.1':0.019336,('PDT000433213.1':0.0215689,('PDT000433747.1':0.0126628,('PDT000534797.1':0.00595561,'PDT000206604.1':0.00549859)'':0.00318641)'':0.00823297)'':0.00121978)'':0.00244227)'':0.00831375)'':0.0179242)'':-0.013378)'':0.00926274)'':0.0119118)'':-0.00353411)'':0.0113128,(((('PDT000633689.1':0.0339019,('PDT000431876.1':0.0288228,'PDT000215972.3':0.0255047)'':0.0122517)'':0.00636857,('PDT000365437.1':0.0665571,(('PDT000536636.1':0.0329688,('PDT000067793.1':0.0248294,('PDT000433706.1':0.0177303,('PDT000423960.1':0.0192644,'PDT000425491.1':0.0165591)'':0.00371241)'':0.00606398)'':0.00542361)'':-0.00175162,('PDT000427244.1':0.0406001,(('PDT000675641.1':0.0390018,('PDT000181712.2':0.0131531,'PDT000427063.1':0.0116337)'':0.0143515)'':-0.000780579,(('PDT000423882.1':0.0242341,'PDT000427025.1':0.0246609)'':0.00610814,(('PDT000535298.1':0.0131937,'PDT000618729.1':0.0139722)'':0.0113953,('PDT000632345.1':0.0152151,('PDT000389328.1':0.00894186,('PDT000632859.1':0.00866889,('PDT000181676.2':0.00483779,('PDT000181705.2':0.0029363,'PDT000427039.1':0.00304635)'':0.00025143)'':0.00329114)'':0.00324193)'':0.00458665)'':0.00738513)'':0.00459386)'':0.00168741)'':0.00888882)'':-0.00686982)'':0.0172823)'':-0.00840776)'':0.00803732,((('PDT000581465.1':0.0289878,'PDT000632927.1':0.0388239)'':0.0016036,('PDT000668924.1':0.0042827,'PDT000668925.1':0.00418175)'':0.0345271)'':0.00825846,((('PDT000425867.1':0.0332893,('PDT000661428.1':0.0277361,('PDT000633540.1':0.0145082,'PDT000633544.1':0.0188814)'':0.0106072)'':0.00337929)'':0.00387034,('PDT000633025.1':0.0397142,('PDT000226142.1':0.0296104,(('PDT000708223.1':0.0245056,'PDT000408018.1':0.0209713)'':0.000938705,('PDT000069683.1':0.0134963,('PDT000720291.1':0.00887008,'PDT000321337.1':0.0112594)'':0.00846634)'':0.0023843)'':0.00475209)'':0.00544503)'':-0.0018673)'':0.00207347,('PDT000619102.1':0.0604219,('PDT000142661.1':0.0344089,('PDT000535083.1':0.0309195,(('PDT000661326.1':0.0177701,('PDT000668897.1':0.00159699,'PDT000668899.1':0.00149929)'':0.0152476)'':0.00224733,('PDT000181686.2':0.0211349,('PDT000020626.1':0.0162527,(('PDT000534940.1':0.0166336,'PDT000535260.1':0.012202)'':0.00289514,('PDT000534989.1':0.0137016,'PDT000616014.1':0.0129518)'':0.00200633)'':0.00151706)'':0.00304752)'':0.00112211)'':0.006307)'':0.00469692)'':0.0162203)'':-0.0134774)'':0.00992414)'':0.00549319)'':0.00478693,(((('PDT000427023.1':0.0297567,'PDT000297987.1':0.0301014)'':0.016622,('PDT000426882.1':0.0365142,('PDT000431982.1':0.0131105,('PDT000675026.1':0.00430207,'PDT000212817.1':0.00389465)'':0.0123864)'':0.0180493)'':0.0148916)'':0.0075203,(('PDT000674975.1':0.0354283,('PDT000019467.2':0.0257469,'PDT000308023.1':0.0289658)'':0.00681294)'':0.00937589,('PDT000535188.1':0.0477632,('PDT000590071.1':0.0277727,('PDT000427209.1':0.0215459,'PDT000694481.1':0.0196225)'':0.00453414)'':0.018497)'':-0.00173633)'':0.0134817)'':0.000586936,(('PDT000264904.2':0.0448221,('PDT000535151.1':0.0443356,('PDT000433740.1':0.0252936,'PDT000417610.1':0.0236513)'':0.0111879)'':0.0103778)'':0.00566091,(((('PDT000136058.2':0.0324325,'PDT000620538.2':0.0376523)'':0.00371562,('PDT000427700.1':0.0352771,('PDT000267155.1':0.0306929,('PDT000427591.1':0.0242968,('PDT000423858.1':0.0195991,'PDT000425457.1':0.0204989)'':0.00438844)'':0.00107035)'':0.00868842)'':-0.000611511)'':0.00610908,(('PDT000520692.1':0.0469022,'PDT000583360.1':0.0363178)'':0.00970746,(('PDT000675403.1':0.0257775,('PDT000431890.1':0.0260224,'PDT000432055.1':0.0203346)'':0.00687363)'':0.00515993,('PDT000675122.1':0.0207627,('PDT000654377.1':0.0186441,'PDT000485906.1':0.015046)'':0.00426396)'':0.0081132)'':0.014455)'':-0.000749648)'':0.00779703,(('PDT000427071.1':0.0629723,('PDT000538462.1':0.05228,('PDT000433699.1':0.0393214,(('PDT000427484.1':0.0200589,'PDT000625966.1':0.0195839)'':0.0149215,('PDT000345827.1':0.0432886,('PDT000427584.1':0.0261621,('PDT000364526.1':0.0274699,(('PDT000425763.1':0.0132017,'PDT000668900.1':0.0156216)'':0.0057579,('PDT000425569.1':0.0099653,'PDT000615943.1':0.0094445)'':0.00874355)'':0.00302067)'':0.00280763)'':0.0105229)'':-0.00222743)'':0.00669054)'':0.00413096)'':0.00725573)'':-0.012517,('PDT000124685.1':0.0539895,(('PDT000124661.1':0.0267071,('PDT000202099.2':0.0233005,'PDT000534966.1':0.024705)'':0.00273587)'':0.00583619,('PDT000713509.1':0.0299533,(('PDT000425722.1':0.0273115,'PDT000714167.1':0.0212236)'':0.00226439,('PDT000414011.1':0.0248881,(('PDT000534981.1':0.018538,'PDT000616155.1':0.0224017)'':0.00217992,('PDT000176011.2':0.0179015,('PDT000616171.1':0.0137229,'PDT000705170.1':0.0123208)'':0.00162798)'':0.00531932)'':0.0012203)'':0.00392607)'':0.00431681)'':-0.00203237)'':0.0219843)'':-0.0108545)'':0.00629341)'':0.00311097)'':0.00308595)'':-0.00107045)'':0.00781103)'':0.0151,((((('PDT000535653.1':0.027579,('PDT000675114.1':0.0149695,'PDT000253681.1':0.0122871)'':0.0153549)'':0.0245906,('PDT000632343.1':0.0652178,(('PDT000324624.1':0.0417041,('PDT000719019.1':0.0315204,('PDT000042686.1':0.0102591,'PDT000227982.1':0.00842507)'':0.0168648)'':0.0107807)'':0.00028554,(('PDT000538172.1':0.0381237,'PDT000206602.1':0.0311446)'':0.00593383,(('PDT000633077.1':0.0250143,'PDT000520969.1':0.019358)'':0.00781927,('PDT000432054.1':0.0322311,(('PDT000181804.2':0.0172075,'PDT000633149.1':0.0168028)'':0.00264481,('PDT000206607.1':0.0156912,'PDT000212813.1':0.0164207)'':0.00498532)'':0.0086728)'':-0.000211546)'':0.00920664)'':0.00146296)'':0.0137956)'':-0.00156489)'':0.00768635,((('PDT000647665.1':0.0343657,('PDT000626700.1':0.0234099,('PDT000267147.1':0.0204927,'PDT000482857.1':0.0185567)'':0.00653302)'':0.00272434)'':0.00815823,(('PDT000625970.1':0.0394947,'PDT000408006.1':0.0389785)'':0.00245261,('PDT000616154.1':0.0385415,('PDT000720303.1':0.022444,'PDT000408031.1':0.0259146)'':0.0137302)'':0.00119664)'':0.000812837)'':0.0110533,((('PDT000426292.1':0.0395556,'PDT000641950.1':0.0341573)'':0.012613,(('PDT000348065.1':0.0397712,'PDT000521135.1':0.0409825)'':0.00452003,('PDT000455751.1':0.0461253,(('PDT000175947.2':0.0236977,'PDT000501870.1':0.0238278)'':0.00891805,('PDT000615867.1':0.0221351,'PDT000102994.1':0.0263474)'':0.00908025)'':0.00870904)'':-0.00101542)'':0.00163752)'':-0.000596746,('PDT000545929.1':0.0637067,(('PDT000626799.1':0.0365275,('PDT000705426.1':0.0283261,('PDT000616180.1':0.0189018,'PDT000618847.1':0.0160535)'':0.00676669)'':0.00855268)'':-0.00149297,(('PDT000175853.2':0.0277563,'PDT000226143.1':0.0292057)'':0.00408008,('PDT000067796.1':0.0338224,(('PDT000103027.1':0.0209926,'PDT000125927.1':0.0177266)'':0.00336001,('PDT000103092.1':0.00987079,'PDT000125938.1':0.00688401)'':0.0182196)'':0.00593478)'':0.00112885)'':0.00676565)'':0.0141142)'':-0.00491214)'':0.00491767)'':0.00823589)'':0.00909268,(((('PDT000426664.1':0.0171017,'PDT000412506.1':0.0175952)'':0.0327488,('PDT000434542.1':0.0309964,'PDT000720311.1':0.0313821)'':0.0246048)'':0.00734485,((('PDT000632363.1':0.0370045,('PDT000218046.2':0.0256777,'PDT000589914.1':0.0285374)'':0.00670375)'':0.00576129,(('PDT000136101.2':0.0228077,'PDT000538139.1':0.0200085)'':0.0125926,('PDT000535187.1':0.0252043,'PDT000094680.1':0.0249178)'':0.012281)'':0.0086403)'':0.00419572,((('PDT000675306.1':0.0151014,'PDT000675397.1':0.0138083)'':0.0134211,(('PDT000298003.1':0.0127685,'PDT000298065.1':0.0143656)'':0.00953604,('PDT000297990.1':0.0208253,'PDT000716955.1':0.0221633)'':0.00217538)'':0.00420425)'':0.0138975,(('PDT000423972.1':0.0396966,'PDT000427597.1':0.0514331)'':0.00957923,('PDT000209465.1':0.0420755,('PDT000589915.1':0.037116,(('PDT000431894.1':0.0190197,'PDT000431897.1':0.015535)'':0.0123319,('PDT000520685.1':0.0308108,(('PDT000705109.1':0.0192533,'PDT000428847.1':0.0199335)'':0.000933439,('PDT000661741.1':0.0210903,('PDT000226149.1':0.0125851,('PDT000615831.1':0.00822218,'PDT000292892.1':0.00720532)'':0.00351204)'':0.00775017)'':0.000139917)'':0.00588566)'':0.000310776)'':0.0075588)'':0.00251117)'':0.0013019)'':0.00257682)'':0.00535006)'':0.0138515)'':0.00677506,((('PDT000018575.2':0.0556362,('PDT000668931.1':0.0226703,'PDT000363569.1':0.0156462)'':0.036323)'':-0.00156461,((('PDT000556902.1':0.0369015,'PDT000173909.1':0.0262211)'':0.00846571,('PDT000501859.1':0.030918,('PDT000408008.1':0.0188529,'PDT000495414.1':0.0176413)'':0.0095587)'':0.00445764)'':0.00649228,(('PDT000372034.1':0.0355744,'PDT000616147.1':0.0373017)'':0.00881461,('PDT000042550.1':0.0397918,('PDT000581806.1':0.0315351,'PDT000250215.1':0.0333608)'':0.00472137)'':0.00363117)'':0.000589304)'':0.0060762)'':0.00198061,((('PDT000720448.1':0.0418654,'PDT000441354.1':0.0432963)'':0.00810753,('PDT000313489.2':0.0368809,(('PDT000520676.1':0.029083,'PDT000520687.1':0.0284017)'':0.000980953,('PDT000535285.1':0.0218535,('PDT000520698.1':0.0108293,('PDT000520727.1':0.00354627,'PDT000520823.1':0.00414143)'':0.00896363)'':0.00768573)'':0.00292026)'':0.0029839)'':0.0142801)'':0.0057202,('PDT000292879.1':0.0547335,(('PDT000523069.1':0.0401123,('PDT000181844.2':0.0267122,'PDT000346764.1':0.03075)'':0.00519241)'':0.00888183,('PDT000391857.1':0.0387943,(('PDT000534782.1':0.0272232,'PDT000495411.1':0.0242388)'':0.00828199,('PDT000145742.1':0.0273309,('PDT000136088.2':0.0186287,('PDT000589761.1':0.0168146,'PDT000248451.1':0.016808)'':0.00444374)'':0.00549505)'':0.00312456)'':0.00448809)'':0.0100091)'':0.0111889)'':-0.00920774)'':0.00530806)'':0.0141751)'':0.00227194)'':0.00578788,(('PDT000240650.2':0.0527821,('PDT000568670.1':0.00664225,'PDT000571148.1':0.00610425)'':0.0390995)'':0.0370242,(('PDT000433309.1':0.106693,('PDT000173723.3':0.0728025,((('PDT000534860.1':0.0375526,('PDT000426204.1':0.0354355,('PDT000673238.1':0.0102423,'PDT000673247.1':0.00975572)'':0.0233448)'':0.00861653)'':0.00563613,(('PDT000141690.2':0.042963,'PDT000426107.1':0.0404104)'':0.0103894,('PDT000427223.1':0.0609673,(('PDT000425577.1':0.0473845,'PDT000520808.1':0.0392365)'':0.00555502,(('PDT000687360.1':0.0359283,('PDT000618893.1':0.020079,'PDT000632286.1':0.0168517)'':0.0171062)'':0.00347899,(('PDT000649096.1':0.0294297,('PDT000572852.1':0.00831531,'PDT000572855.1':0.00809609)'':0.0237295)'':0.00543573,(('PDT000444990.1':0.0295442,'PDT000536595.1':0.0251641)'':0.00690002,(('PDT000535054.1':0.0253384,'PDT000714052.1':0.0237282)'':0.00612623,(('PDT000136103.2':0.0336599,'PDT000615960.1':0.0372421)'':0.00105283,('PDT000536619.1':0.0202968,('PDT000425567.1':0.00266837,'PDT000713914.1':0.00232414)'':0.0184061)'':0.00580503)'':0.00298805)'':0.00330956)'':0.00223951)'':0.00416989)'':0.00318075)'':0.0102789)'':-0.00578617)'':-0.00186776)'':-5.08157e-05,(('PDT000538520.1':0.0548834,(('PDT000347036.1':0.0131075,('PDT000622471.1':0.00191578,'PDT000622560.1':0.00177942)'':0.0156974)'':0.0113468,('PDT000701138.1':0.00889865,('PDT000701139.1':0.000484058,'PDT000701140.1':0.000815812)'':0.00718357)'':0.013152)'':0.0224297)'':-0.00414736,(('PDT000427454.1':0.0362687,'PDT000615861.1':0.028301)'':0.00748245,(('PDT000713896.1':0.0228076,'PDT000036733.1':0.0276816)'':0.016333,(('PDT000426133.1':0.0287295,'PDT000705081.1':0.0267553)'':0.00987352,(('PDT000716911.1':0.0317166,'PDT000583398.1':0.0321215)'':0.00182522,((('PDT000535512.1':0.0175733,'PDT000581707.1':0.0191049)'':0.00679919,('PDT000538599.1':0.0179267,'PDT000661386.1':0.0174628)'':0.00617708)'':0.00382219,('PDT000674842.1':0.0295803,('PDT000622453.1':0.0178855,('PDT000616050.1':0.0211877,('PDT000668906.1':0.00899127,('PDT000615812.1':0.00945058,('PDT000622513.1':0.00271222,'PDT000673609.1':0.00317777)'':0.00449858)'':0.00235279)'':0.00629637)'':0.00245041)'':0.00223019)'':0.00241257)'':0.00654559)'':0.00291656)'':0.00234027)'':0.00141423)'':0.00441099)'':0.00407194)'':0.0195935)'':0.0194822)'':-0.0240023,(('PDT000397658.1':0.073556,('PDT000443756.1':0.00314531,'PDT000443880.1':0.00343954)'':0.0546695)'':0.0165413,(((('PDT000306712.2':0.0468322,('PDT000226140.1':0.0219857,'PDT000321347.1':0.0211041)'':0.0262341)'':0.0101024,('PDT000252004.1':0.0478489,('PDT000719230.1':0.0377344,'PDT000719232.1':0.0443119)'':0.00770799)'':0.0146669)'':0.00327025,('PDT000425683.1':0.0563138,(('PDT000188938.1':0.0049047,('PDT000188340.1':0.00285343,'PDT000188354.1':0.00283406)'':0.00245716)'':0.0376913,(('PDT000520976.1':0.0312429,('PDT000581963.1':0.0260418,('PDT000503833.1':0.00429006,'PDT000503834.1':0.00578874)'':0.0209164)'':0.00525993)'':-0.000189558,('PDT000157065.2':0.0247664,(('PDT000427583.1':0.0200483,('PDT000427067.1':0.0157495,'PDT000636323.1':0.0179895)'':0.00415662)'':0.00200531,('PDT000534027.1':0.0245326,(('PDT000017679.2':0.00592256,'PDT000017823.2':0.00672814)'':0.00748309,('PDT000521116.1':0.0148384,'PDT000520860.1':0.0161609)'':0.00255319)'':0.00489915)'':0.000374591)'':0.00182915)'':0.00650241)'':0.00569739)'':0.0183329)'':0.00578339)'':0.000381352,((('PDT000015103.2':0.0673584,(('PDT000441351.1':0.0392407,('PDT000175820.2':0.0226733,'PDT000705107.1':0.0263628)'':0.014052)'':0.0111262,('PDT000294374.1':0.053036,(('PDT000615830.1':0.0219539,'PDT000520842.1':0.0206684)'':0.0142701,('PDT000432047.1':0.0336635,'PDT000432057.1':0.0321005)'':0.010106)'':0.00785002)'':0.00299514)'':0.0212905)'':-0.0121473,(('PDT000136086.2':0.0251063,('PDT000691332.1':0.0242995,'PDT000691336.1':0.0235521)'':0.00857492)'':0.0314434,('PDT000615802.1':0.0520584,(('PDT000520812.1':0.0348496,('PDT000176214.2':0.0234647,'PDT000425516.1':0.0286371)'':0.00696907)'':0.00452738,('PDT000520706.1':0.0222372,(('PDT000615815.1':0.0105873,('PDT000427617.1':0.0117088,'PDT000615893.1':0.012077)'':0.00105757)'':0.00496705,('PDT000441347.1':0.0191448,('PDT000042575.1':0.0124971,'PDT000428845.1':0.0127354)'':0.00434763)'':0.00222393)'':0.0038318)'':0.0164328)'':0.0110198)'':0.00849275)'':-0.000561407)'':0.000676058,((('PDT000719231.1':0.0292393,('PDT000382838.1':0.0271098,'PDT000144502.5':0.0219868)'':0.00441296)'':0.019485,('PDT000322914.1':0.030288,('PDT000534905.1':0.0240024,'PDT000535455.1':0.0292928)'':0.00620199)'':0.017029)'':0.00430783,((('PDT000102017.2':0.0375113,'PDT000616024.1':0.0372335)'':0.00353402,(('PDT000673606.1':0.0312042,('PDT000673234.1':0.0231095,'PDT000713682.1':0.0245795)'':0.00225867)'':0.00764958,('PDT000427585.1':0.0230046,('PDT000426273.1':0.027924,(('PDT000538431.1':0.00531783,'PDT000616056.1':0.00775737)'':0.00303225,('PDT000615913.1':0.00865152,'PDT000308024.1':0.00773378)'':0.000659677)'':0.0103565)'':0.00442648)'':0.0109809)'':0.00786767)'':0.00923757,(('PDT000636227.1':0.0524548,('PDT000520815.1':0.0328184,('PDT000538500.1':0.036565,'PDT000380902.1':0.0285582)'':0.00132335)'':0.0103128)'':-0.00185155,('PDT000675483.1':0.0442045,('PDT000534991.1':0.0341868,('PDT000433638.1':0.0282323,(('PDT000202022.2':0.0207257,'PDT000391852.1':0.0231245)'':0.000230386,(('PDT000626694.1':0.013356,'PDT000636307.1':0.0105389)'':0.00384921,('PDT000636291.1':0.0110883,('PDT000535278.1':0.0106839,('PDT000535301.1':0.00652484,'PDT000535325.1':0.00573106)'':0.00146107)'':0.00256734)'':0.00579892)'':0.00533849)'':0.00665032)'':0.00145521)'':0.00428947)'':0.0024676)'':0.013804)'':0.00297986)'':0.00709683)'':0.00497021)'':0.00946011)'':-0.0124064)'':0.0300645)'':-0.0197082)'':0.00293664)'':0.0407822)'':0.0933614,((('PDT000357592.1':0.125067,('PDT000345829.1':0.0256049,'PDT000357719.1':0.0368219)'':0.088392)'':0.121449,('PDT000556996.1':0.130185,('PDT000322093.1':0.0355171,('PDT000135981.2':0.0155691,'PDT000672280.1':0.0174316)'':0.0111878)'':0.108345)'':0.0892374)'':0.00773117,((('PDT000063360.2':0.0634515,'PDT000129838.2':0.0594465)'':0.140481,((('PDT000128842.2':0.104633,'PDT000565967.1':0.0951501)'':0.0408107,('PDT000425733.1':0.0664441,'PDT000520711.1':0.0783529)'':0.0606126)'':0.00708498,(('PDT000304284.1':0.108871,('PDT000140513.3':0.0668942,('PDT000619563.2':0.060267,('PDT000702628.1':0.0346276,('PDT000250121.1':0.0310691,('PDT000250202.1':0.0224469,'PDT000250239.1':0.0240395)'':0.0075172)'':0.00772235)'':0.0117181)'':0.00541424)'':0.0581068)'':-0.0134323,(('PDT000469962.1':0.0586122,(('PDT000518115.1':0.0531515,'PDT000645876.1':0.0569165)'':0.00892308,('PDT000630271.1':0.0447349,'PDT000653442.1':0.0521733)'':0.00886007)'':0.000239425)'':0.0207795,(('PDT000242876.1':0.0719014,('PDT000667861.2':0.0571492,'PDT000388473.1':0.0572368)'':0.0118851)'':0.0129624,(('PDT000155879.3':0.0605838,('PDT000155970.3':0.0397805,'PDT000619565.2':0.0400615)'':0.0163577)'':0.018644,(('PDT000242976.1':0.0543445,'PDT000357716.1':0.0446635)'':0.00824092,('PDT000124648.1':0.0668559,('PDT000124673.1':0.033276,'PDT000208697.1':0.0348465)'':0.0217169)'':0.00398738)'':0.00631406)'':0.00518102)'':0.0057946)'':0.0248125)'':0.0384535)'':0.0671769)'':0.0227089,((('PDT000129875.2':0.134272,(('PDT000149340.2':0.105659,'PDT000443381.1':0.104293)'':0.010198,('PDT000459525.1':0.0647493,('PDT000129856.2':0.0224235,'PDT000509154.1':0.0247528)'':0.031917)'':0.0476971)'':0.0355148)'':-0.0128647,(('PDT000557019.1':0.118902,'PDT000653001.1':0.114649)'':0.00397277,((('PDT000719272.1':0.0667614,('PDT000652876.1':0.0363302,'PDT000653026.1':0.0365513)'':0.0392213)'':0.0156362,('PDT000396812.1':0.0767307,(('PDT000051478.1':0.0641544,('PDT000497216.1':0.0107069,('PDT000505165.1':0.00906176,'PDT000497229.1':0.00989234)'':0.00211122)'':0.0547034)'':-0.00100484,(('PDT000687687.1':0.037268,('PDT000550388.1':0.0188373,'PDT000660634.1':0.0202468)'':0.0183172)'':0.0168006,(('PDT000424632.1':0.0465917,'PDT000630068.1':0.0395036)'':0.00521587,(('PDT000397942.1':0.0153654,'PDT000568755.1':0.0193672)'':0.0255139,('PDT000395136.1':0.0232487,('PDT000621299.1':0.0183897,'PDT000702841.1':0.0199888)'':0.00125337)'':0.0198463)'':0.0062101)'':0.0059738)'':0.00742722)'':0.0259567)'':0.00902485)'':0.0147112,(('PDT000285376.2':0.114051,(('PDT000228099.1':0.0399672,'PDT000228119.1':0.052034)'':0.0252298,('PDT000395207.1':0.0687007,'PDT000395494.1':0.0650403)'':0.0030678)'':0.0321059)'':0.00848449,((('PDT000168822.1':0.0861609,'PDT000286099.1':0.0823591)'':0.0157009,(('PDT000395087.1':0.0320066,'PDT000395474.1':0.0279895)'':0.0553717,(('PDT000396609.1':0.0587557,('PDT000128819.2':0.0443705,'PDT000253657.1':0.0366424)'':0.0215974)'':0.0202007,('PDT000298088.1':0.0938585,('PDT000128858.2':0.0683748,('PDT000645513.2':0.0552422,'PDT000298032.1':0.0574808)'':0.0217317)'':0.0114385)'':0.000524749)'':0.00781722)'':0.0125146)'':0.00339335,(('PDT000688687.1':0.0848159,('PDT000286098.1':0.0666128,(('PDT000395022.1':0.0241996,'PDT000395033.1':0.0292027)'':0.0234192,('PDT000019615.1':0.0449092,(('PDT000644915.1':0.0110526,'PDT000395173.1':0.0134708)'':0.0289145,('PDT000128808.2':0.0384574,('PDT000543298.1':0.037769,'PDT000652342.1':0.0303816)'':0.00353195)'':0.00335901)'':0.00919611)'':0.00118096)'':0.0158994)'':0.0324357)'':-0.0173216,('PDT000557338.1':0.0724069,(('PDT000318194.1':0.0450171,('PDT000352059.1':0.0338377,'PDT000644686.1':0.0343287)'':0.00405819)'':0.00557134,('PDT000034516.2':0.0505656,('PDT000394858.1':0.0264262,(('PDT000200198.2':0.0203762,'PDT000200199.2':0.0194546)'':0.00397655,('PDT000594315.1':0.0205286,'PDT000318197.1':0.0241381)'':0.0029902)'':0.00252057)'':0.0183576)'':0.000566837)'':0.0280892)'':0.00668216)'':0.0219221)'':0.00906554)'':-0.00149247)'':0.0146733)'':0.0294373)'':0.0527118,((('PDT000550373.1':0.10701,('PDT000656716.1':0.074855,('PDT000088572.2':0.0636415,'PDT000689893.1':0.0602625)'':0.00517252)'':0.0232905)'':0.0298887,((('PDT000252653.1':0.0692288,('PDT000014099.3':0.0199248,'PDT000251489.1':0.0140983)'':0.0510096)'':0.0126293,('PDT000415808.1':0.0801946,(('PDT000083958.2':0.0537497,'PDT000372154.1':0.0570673)'':0.00119883,('PDT000445038.1':0.0487917,('PDT000713520.1':0.0271354,'PDT000011805.2':0.0304818)'':0.0109527)'':0.0126688)'':0.0195145)'':0.00434228)'':-0.00449137,(('PDT000661370.1':0.0403179,('PDT000136098.2':0.0355859,('PDT000719036.1':0.0249291,('PDT000626701.1':0.0268619,'PDT000637158.1':0.0266817)'':0.00427406)'':0.000877658)'':0.00309655)'':0.0116801,(('PDT000427593.1':0.0425927,'PDT000675552.1':0.0549517)'':0.00456442,(('PDT000545920.1':0.0254405,'PDT000241783.1':0.0277802)'':0.00689185,('PDT000626804.1':0.0341612,('PDT000619096.1':0.0192535,('PDT000234066.2':0.0195454,('PDT000619080.1':0.0151216,('PDT000618731.1':0.012976,('PDT000034465.2':0.00519513,'PDT000460547.1':0.00767917)'':0.00349915)'':0.00339981)'':0.00295092)'':0.00124175)'':0.0105829)'':0.00184848)'':0.0113447)'':0.000971528)'':0.0298838)'':0.0490675)'':0.0112759,(((('PDT000394877.1':0.0569626,('PDT000566200.1':0.0519638,('PDT000200184.2':0.0497683,'PDT000235916.5':0.0480578)'':0.00723711)'':0.00315664)'':0.0688862,(('PDT000645514.2':0.112507,('PDT000556958.1':0.0817865,('PDT000520829.1':0.0654345,('PDT000178585.2':0.0462923,'PDT000397956.1':0.04795)'':0.0232479)'':0.0102617)'':0.0183756)'':-0.000521823,(('PDT000652351.1':0.0484955,('PDT000427605.1':0.0506942,'PDT000653021.1':0.0478796)'':0.00683606)'':0.0337016,(('PDT000630466.1':0.0571704,('PDT000517923.1':0.0584277,'PDT000395323.1':0.0530243)'':0.00719913)'':0.0285652,('PDT000702839.1':0.0913746,('PDT000395277.1':0.0832082,(('PDT000395102.1':0.05258,('PDT000565914.1':0.0412189,('PDT000394872.1':0.0271703,'PDT000568758.1':0.030872)'':0.00930945)'':0.0119674)'':0.0234198,(('PDT000337006.1':0.0528261,'PDT000610394.1':0.0563499)'':0.00293007,('PDT000639536.1':0.0470514,('PDT000399708.1':0.0341123,'PDT000505388.1':0.0411146)'':0.00456599)'':0.00913139)'':0.0182985)'':0.00829258)'':0.00815665)'':-0.00219214)'':0.00237151)'':0.0171596)'':0.010043)'':0.021901,((('PDT000292811.2':0.0694251,('PDT000332080.1':0.0514752,('PDT000645054.1':0.0391299,('PDT000565876.1':0.0321598,'PDT000395408.1':0.0242612)'':0.0136007)'':0.00668943)'':0.0186817)'':0.0416968,((('PDT000285621.2':0.0335235,('PDT000332084.1':0.0218244,('PDT000496897.1':0.0199844,'PDT000521632.1':0.0184809)'':0.00566113)'':0.0102324)'':0.0517341,('PDT000425811.1':0.0778301,('PDT000565920.1':0.0828035,('PDT000398016.1':0.0662572,'PDT000720330.1':0.0537858)'':0.0114675)'':0.00723973)'':0.0056268)'':0.0123344,(('PDT000565954.1':0.0981345,(('PDT000497329.1':0.0434453,'PDT000594466.1':0.0484063)'':0.0182792,('PDT000394915.1':0.0626906,('PDT000620188.1':0.0323336,('PDT000717287.1':0.0340028,'PDT000502199.1':0.027459)'':0.00407847)'':0.0154153)'':0.00216927)'':0.0247404)'':0.00459724,((('PDT000200182.2':0.0239921,'PDT000717626.1':0.027189)'':0.0551045,('PDT000205420.1':0.0872333,('PDT000128794.2':0.0728466,(('PDT000109430.2':0.0288781,('PDT000372182.1':0.0245457,'PDT000557218.1':0.0235511)'':0.00302581)'':0.0148273,('PDT000502053.1':0.03824,('PDT000675001.1':0.0346626,('PDT000188101.2':0.0112791,'PDT000103089.1':0.00972556)'':0.0168277)'':0.0056404)'':0.00144445)'':0.026255)'':0.00159592)'':-0.000971977)'':0.00393553,(('PDT000042559.1':0.0618947,'PDT000240257.1':0.0646253)'':0.00795837,(('PDT000218037.2':0.0543183,'PDT000150858.1':0.0598337)'':0.00554867,(('PDT000652932.1':0.0644632,('PDT000642302.1':0.0279229,('PDT000150256.2':0.0113088,'PDT000318999.1':0.0123968)'':0.0115307)'':0.0233685)'':0.00511674,('PDT000433631.1':0.0601718,('PDT000535105.1':0.043934,(('PDT000129810.2':0.0319538,'PDT000694448.1':0.0264807)'':0.011521,(('PDT000141215.2':0.0245761,'PDT000716839.1':0.0289093)'':0.00799628,('PDT000652884.1':0.0401268,('PDT000718849.1':0.0187724,'PDT000661620.1':0.0178923)'':0.0133082)'':0.000900948)'':0.00323833)'':0.00763694)'':0.0158956)'':-0.011589)'':0.0124567)'':0.00646469)'':-0.000508013)'':0.0130291)'':0.00409395)'':0.0132121)'':0.00362008,((('PDT000136144.2':0.105442,('PDT000352552.1':0.0924395,('PDT000344863.1':0.0304619,'PDT000174354.1':0.032263)'':0.0611225)'':0.0100844)'':0.00472274,(('PDT000616804.1':0.035294,('PDT000534798.1':0.0302959,'PDT000297985.1':0.0349088)'':0.00345222)'':0.0257459,(('PDT000175817.2':0.0344235,'PDT000363575.1':0.0337138)'':0.0173282,(('PDT000615951.1':0.0154183,'PDT000720282.1':0.0166203)'':0.0210826,('PDT000618967.1':0.0445112,('PDT000438270.1':0.0278813,'PDT000616061.1':0.0264835)'':0.0113377)'':0.00384604)'':0.0110253)'':0.00863097)'':0.0447526)'':0.00865125,(((('PDT000319042.1':0.0685516,('PDT000481042.1':0.0767865,('PDT000557038.1':0.0527718,'PDT000502047.1':0.0579582)'':0.013895)'':0.00308723)'':0.00599875,(('PDT000228301.1':0.0454905,'PDT000248775.1':0.0442457)'':0.0147298,('PDT000352456.1':0.0469503,('PDT000226498.2':0.032712,'PDT000708560.1':0.0350568)'':0.00921281)'':0.0138317)'':0.0191509)'':0.00280583,('PDT000479553.1':0.103816,((('PDT000290373.2':0.0381039,'PDT000582191.1':0.0501112)'':0.0134819,('PDT000652331.1':0.0645292,('PDT000652893.1':0.0592922,'PDT000435638.1':0.0554608)'':0.0055568)'':0.000321165)'':0.00793405,(('PDT000627604.1':0.0551167,'PDT000672199.1':0.0516363)'':0.0190572,(('PDT000425839.1':0.0727018,'PDT000109466.2':0.0664232)'':0.0039108,('PDT000645702.2':0.0731297,(('PDT000502704.1':0.024195,'PDT000360996.1':0.029394)'':0.0244498,('PDT000502702.1':0.0474273,'PDT000426750.1':0.049393)'':0.00503674)'':0.0096995)'':0.0051367)'':0.0025836)'':-0.000398202)'':0.0168861)'':0.00221631)'':0.0114691,((('PDT000414680.1':0.0384234,(('PDT000426605.1':0.0277877,'PDT000287622.1':0.0308011)'':0.00193375,('PDT000675303.1':0.0238039,'PDT000168703.1':0.0269236)'':0.00330125)'':0.00505503)'':0.0286255,('PDT000181780.2':0.0964958,(('PDT000427727.1':0.0665047,('PDT000618781.1':0.0583642,'PDT000448170.1':0.0543988)'':0.0143528)'':0.00758629,(('PDT000664652.2':0.043316,'PDT000191149.1':0.0488736)'':0.0150506,('PDT000364520.2':0.0233061,('PDT000443871.1':0.00456936,'PDT000535377.1':0.00351689)'':0.0160493)'':0.033081)'':0.0112487)'':0.00670747)'':-0.0133951)'':0.0191897,(('PDT000359123.2':0.0787681,('PDT000535024.1':0.0556872,('PDT000651218.1':0.0332911,'PDT000664686.2':0.0303135)'':0.019308)'':0.0210117)'':-0.00200959,(('PDT000203691.1':0.0420949,('PDT000438032.1':0.016882,'PDT000720308.1':0.0166056)'':0.022831)'':0.0167185,(('PDT000534779.1':0.0443271,'PDT000534784.1':0.0380042)'':0.0103572,('PDT000502244.1':0.0511786,('PDT000661619.1':0.0452113,(('PDT000720277.1':0.016517,('PDT000445084.1':0.0074725,'PDT000538553.1':0.008016)'':0.00705299)'':0.0165361,(('PDT000425519.1':0.0291334,'PDT000652524.1':0.0305522)'':0.00603256,('PDT000215975.1':0.0230825,('PDT000426797.1':0.0125755,'PDT000713391.1':0.0167372)'':0.00531402)'':0.0142833)'':0.00120165)'':0.00928336)'':0.00464618)'':-0.00353829)'':0.0080814)'':0.0125041)'':0.0139728)'':0.00975903)'':0.0135389)'':-0.0024513)'':0.00724105)'':0.00585434,((('PDT000565890.1':0.102789,(((('PDT000025243.2':0.0331034,'PDT000395250.1':0.0282684)'':0.00684532,('PDT000490610.1':0.0400437,'PDT000519819.1':0.0377902)'':0.00262563)'':0.0348255,('PDT000346009.1':0.0936708,(('PDT000329023.1':0.019626,'PDT000329044.1':0.0209305)'':0.0504725,('PDT000239190.2':0.0235663,'PDT000651553.1':0.0204632)'':0.045373)'':0.0126982)'':-0.000569084)'':0.00366023,(('PDT000355846.1':0.0667002,('PDT000352049.1':0.0542127,('PDT000395275.1':0.0396866,'PDT000394806.1':0.0395598)'':0.0122816)'':0.0119201)'':0.00472386,(('PDT000519816.1':0.0219311,'PDT000643444.1':0.0269314)'':0.0478024,(('PDT000147458.3':0.0274551,'PDT000196614.2':0.0317695)'':0.0338292,(('PDT000504743.1':0.0618806,('PDT000482810.1':0.0553699,('PDT000568707.1':0.0362849,'PDT000241780.1':0.0355511)'':0.0193385)'':0.00407611)'':-0.000976856,(('PDT000501970.1':0.0336548,'PDT000485914.1':0.0339402)'':0.0139759,('PDT000397624.1':0.0451221,('PDT000175802.2':0.0239016,('PDT000717164.1':0.0114195,'PDT000017853.2':0.0132096)'':0.0187323)'':0.00890182)'':0.00874004)'':0.00303042)'':0.0103912)'':0.00260714)'':0.00129802)'':0.00484972)'':0.0210553)'':-0.00172706,(((('PDT000632718.1':0.0208479,'PDT000587986.1':0.0269782)'':0.0256447,('PDT000226134.1':0.043844,(('PDT000501868.1':0.0301608,('PDT000426372.1':0.0167858,('PDT000233272.1':0.000622151,'PDT000233273.1':0.000877099)'':0.0204481)'':0.0114158)'':-0.000244567,(('PDT000181708.2':0.0105988,'PDT000535150.1':0.0131162)'':0.00976444,('PDT000668938.1':0.0185532,('PDT000426156.1':0.00925334,('PDT000181735.2':0.00785945,('PDT000181697.2':0.0047594,'PDT000633657.1':0.00500545)'':0.00389628)'':0.00214634)'':0.00398249)'':0.00165548)'':0.00807748)'':0.00909197)'':0.015413)'':0.00800976,(('PDT000675520.1':0.0541848,('PDT000431871.1':0.0366885,'PDT000411324.1':0.0385865)'':0.0113432)'':0.00632608,(('PDT000664689.2':0.0405278,('PDT000534913.1':0.0361563,'PDT000535344.1':0.0368109)'':0.00349515)'':0.0062894,('PDT000636294.1':0.0487512,(('PDT000618947.1':0.0249549,('PDT000434537.1':0.0224542,('PDT000535177.1':0.0208362,('PDT000534964.1':0.0161109,'PDT000691202.1':0.0138921)'':0.00280945)'':0.0022472)'':0.00428716)'':0.00503053,('PDT000226150.1':0.0462359,(('PDT000202100.2':0.0247632,('PDT000535182.1':0.0175187,'PDT000535205.1':0.0218373)'':0.00587786)'':0.00536188,('PDT000485922.1':0.0275364,('PDT000704931.1':0.0168216,('PDT000618897.1':0.0158033,('PDT000615847.1':0.0176385,'PDT000626811.1':0.0148143)'':0.00331788)'':0.00153938)'':0.00421397)'':0.0032331)'':0.0114969)'':-0.0110843)'':0.0139615)'':-0.000378021)'':0.00902741)'':0.00476082)'':0.012642,(('PDT000443094.1':0.0833026,('PDT000427048.1':0.067874,('PDT000427586.1':0.0374564,('PDT000426295.1':0.0178308,'PDT000495412.1':0.0155256)'':0.0145228)'':0.0334892)'':0.0221816)'':-0.00479141,(((('PDT000705412.1':0.0396276,'PDT000502226.1':0.0363856)'':0.00552244,('PDT000346837.1':0.0449504,('PDT000346776.1':0.0351066,('PDT000433672.1':0.02599,'PDT000633443.1':0.0235529)'':0.00964856)'':0.00674209)'':0.00805135)'':0.00700377,(('PDT000347040.1':0.0698202,('PDT000347061.1':0.0491206,(('PDT000502055.1':0.0197345,'PDT000346908.1':0.0168182)'':0.00263336,('PDT000502044.1':0.012732,('PDT000626793.1':0.0112043,'PDT000704920.1':0.00923016)'':0.00550515)'':0.00521844)'':0.0188872)'':0.0209465)'':-0.00941074,((('PDT000145820.3':0.0304752,('PDT000636208.1':0.0258411,'PDT000588006.1':0.0202083)'':0.00405514)'':0.019596,('PDT000705345.1':0.0262698,('PDT000705091.1':0.0189461,'PDT000705092.1':0.0164687)'':0.00630036)'':0.0128677)'':0.00206539,('PDT000626758.1':0.0546783,('PDT000618881.1':0.0484222,('PDT000534963.1':0.0322364,('PDT000248479.1':0.030379,('PDT000673545.1':0.0228716,('PDT000132311.2':0.0122969,'PDT000352391.1':0.0111615)'':0.00694661)'':0.00568264)'':0.00576345)'':0.00248026)'':0.0066)'':0.00116307)'':0.00689686)'':0.00607351)'':0.00379188,(((('PDT000535343.1':0.0359437,'PDT000616173.1':0.0366842)'':0.0152445,('PDT000588042.1':0.0372932,'PDT000042647.1':0.0333431)'':0.010648)'':0.00217884,('PDT000616168.1':0.0381229,('PDT000668796.1':0.0371504,('PDT000346698.1':0.031582,('PDT000426125.1':0.0242361,'PDT000632130.1':0.0258472)'':0.00524647)'':0.00368635)'':0.00760348)'':0.00789139)'':0.0055799,(('PDT000581956.1':0.0463455,'PDT000248482.1':0.0493697)'':0.00404198,('PDT000718034.1':0.0451468,(('PDT000618976.1':0.0327028,('PDT000268749.2':0.0200483,'PDT000535384.1':0.0228795)'':0.00815101)'':1.61369e-05,('PDT000423874.1':0.0458928,('PDT000616148.1':0.0408725,(('PDT000543144.1':0.0187427,'PDT000448180.1':0.0204768)'':0.0108132,(('PDT000444156.1':0.0246699,'PDT000030571.1':0.0250468)'':0.00419518,(('PDT000226502.2':0.0173253,'PDT000463732.1':0.0206368)'':0.00560699,('PDT000445113.1':0.0282674,('PDT000181685.2':0.0214904,('PDT000309256.2':0.0177852,('PDT000535448.1':0.011657,('PDT000535456.1':0.010475,'PDT000716978.1':0.0077757)'':0.00236028)'':0.00668116)'':0.00192041)'':0.00326231)'':-0.000590094)'':0.00186906)'':0.00160533)'':0.00887792)'':0.00136974)'':-0.0028132)'':0.00744223)'':0.00249007)'':0.00604779)'':0.00522617)'':0.0156325)'':-0.00443496)'':0.0257617)'':0.0110769,(((('PDT000615857.1':0.0840594,(('PDT000161668.2':0.0243398,'PDT000362448.1':0.0177831)'':0.048709,('PDT000315609.1':0.0756062,('PDT000587138.1':0.0660188,('PDT000129827.2':0.0628901,(('PDT000129841.2':0.0297329,'PDT000618906.1':0.0376817)'':0.0127295,(('PDT000131927.2':0.024093,'PDT000437902.1':0.0206025)'':0.0140598,('PDT000691301.1':0.0255818,('PDT000205073.2':0.0114929,'PDT000325805.1':0.0152163)'':0.0189815)'':0.00775162)'':0.00804647)'':0.0123831)'':0.0086034)'':0.00459647)'':-0.000393564)'':0.0170903)'':-0.00880694,(('PDT000160469.2':0.05166,('PDT000132047.2':0.0405812,'PDT000636228.1':0.039957)'':0.00782673)'':0.0302638,((('PDT000132256.2':0.0558051,'PDT000422213.1':0.0546929)'':0.00186658,('PDT000421796.1':0.0487044,'PDT000438013.1':0.0536416)'':0.00624642)'':0.00981928,(('PDT000542149.1':0.0379376,('PDT000647673.1':0.0318304,('PDT000156948.2':0.0264322,('PDT000520747.1':0.0202995,'PDT000253670.1':0.017171)'':0.00615814)'':0.0013056)'':0.00948689)'':0.0330904,(('PDT000136007.2':0.0604266,'PDT000136016.2':0.0625384)'':0.00262979,(('PDT000625516.1':0.0378915,'PDT000637071.1':0.0495245)'':0.00839238,('PDT000625852.1':0.0517648,('PDT000615952.1':0.0287651,'PDT000248453.1':0.0225347)'':0.0218986)'':0.000181407)'':0.0177436)'':0.000800353)'':0.0040496)'':0.0151972)'':0.00227587)'':0.00123351,(('PDT000018407.2':0.150951,((('PDT000425798.1':0.0527555,'PDT000441820.1':0.0630825)'':0.00627916,('PDT000175942.2':0.0463657,'PDT000427601.1':0.0415689)'':0.00865554)'':0.00891642,('PDT000636346.1':0.0549259,('PDT000668922.1':0.059395,('PDT000720290.1':0.032354,('PDT000438132.1':0.01318,'PDT000636299.1':0.0138737)'':0.0178989)'':0.0109155)'':0.00777153)'':0.0135479)'':0.0424893)'':-0.0227738,((('PDT000248486.1':0.0796564,(('PDT000323281.1':0.016653,'PDT000625431.1':0.0141178)'':0.0352316,('PDT000222838.2':0.047083,'PDT000622522.1':0.0460321)'':0.00730406)'':0.0292345)'':0.0005483,((('PDT000421751.1':0.00128502,'PDT000421794.1':0.00171318)'':0.0569874,('PDT000520716.1':0.0327747,'PDT000520816.1':0.035603)'':0.0360144)'':0.00249561,('PDT000429710.1':0.0754125,(('PDT000426518.1':0.0162524,'PDT000618769.1':0.0184691)'':0.00995717,('PDT000535115.1':0.029735,('PDT000425752.1':0.0198871,('PDT000175901.2':0.00912256,'PDT000202154.2':0.00773864)'':0.0109206)'':0.00583181)'':0.00472564)'':0.0435078)'':-0.00281394)'':0.0047423)'':0.00189697,(('PDT000289951.1':0.079998,('PDT000395657.1':0.0599783,('PDT000661357.1':0.0544857,('PDT000711409.1':0.0405664,('PDT000618380.1':0.0329016,('PDT000636230.1':0.0230542,'PDT000448189.1':0.0268423)'':0.00908314)'':0.00592183)'':0.00862943)'':0.00905328)'':0.00833602)'':0.00727912,((('PDT000443083.1':0.0802834,('PDT000425671.1':0.0524387,('PDT000426825.1':0.0557034,('PDT000426083.1':0.0216267,'PDT000520890.1':0.0291101)'':0.0375832)'':0.00579166)'':0.0130276)'':-0.00379472,(('PDT000020192.1':0.0327847,('PDT000496773.1':0.0291808,('PDT000205099.2':0.0209545,'PDT000445088.1':0.014114)'':0.0063066)'':0.00840802)'':0.020647,('PDT000487931.1':0.0568779,('PDT000675502.1':0.0570221,('PDT000426790.1':0.0564949,('PDT000426185.1':0.0288808,('PDT000675103.1':0.0151679,'PDT000675416.1':0.0101561)'':0.0205983)'':0.0163575)'':0.00750398)'':0.00332812)'':-0.00354648)'':0.0132452)'':-0.00119111,((('PDT000509695.1':0.0548558,('PDT000175997.2':0.0338441,'PDT000615859.1':0.0342689)'':0.0216502)'':0.00304729,('PDT000429770.1':0.0520175,('PDT000713963.1':0.0496281,('PDT000325199.1':0.0437342,'PDT000610595.1':0.0420513)'':0.00214532)'':0.0117941)'':0.00146954)'':0.00843456,((('PDT000032345.1':0.00440266,'PDT000032346.1':0.00466109)'':0.0493319,('PDT000673537.1':0.059427,('PDT000636258.1':0.0395815,('PDT000033984.1':0.0258317,'PDT000034370.1':0.0159949)'':0.0168514)'':0.0154729)'':0.000503343)'':-0.000735119,('PDT000445105.1':0.0571751,(('PDT000433795.1':0.0363408,'PDT000520826.1':0.026858)'':0.0151477,('PDT000716963.1':0.0534399,(('PDT000520765.1':0.0544959,('PDT000390804.1':0.0385438,('PDT000318822.1':0.0305895,'PDT000636269.1':0.0296281)'':0.00745921)'':0.00784609)'':0.000553415,(('PDT000713459.1':0.0347056,'PDT000521107.1':0.0391069)'':0.0133238,('PDT000716830.1':0.0541226,('PDT000652377.1':0.0463949,(('PDT000483986.1':0.0337505,('PDT000150959.2':0.0305558,'PDT000678428.1':0.0263476)'':0.00813617)'':0.00201461,('PDT000655226.1':0.0291103,('PDT000426183.1':0.0260212,('PDT000626699.1':0.0104515,'PDT000636196.1':0.0122916)'':0.0153051)'':0.00223591)'':0.00700221)'':0.00457098)'':0.00842406)'':-0.00376665)'':-0.00155638)'':0.0074785)'':-0.00468215)'':0.0111238)'':-0.00605684)'':0.0122121)'':2.97859e-05)'':0.010844)'':0.00366894)'':0.00325747)'':0.00146845)'':0.0126619,(((('PDT000537887.1':0.0802922,('PDT000501915.1':0.035842,('PDT000520889.1':0.0340677,'PDT000228355.1':0.0381627)'':0.00416074)'':0.0251778)'':0.0180656,((('PDT000636345.1':0.0336824,('PDT000713715.1':0.0265997,'PDT000397859.1':0.0235524)'':0.00370233)'':0.00252281,(('PDT000423911.1':0.0266861,'PDT000294789.1':0.0191909)'':0.00646294,('PDT000107792.2':0.0128668,'PDT000102999.1':0.0114291)'':0.0136224)'':0.00146717)'':0.0168754,(('PDT000492385.1':0.0420306,(('PDT000636377.1':0.0273066,'PDT000543126.1':0.026223)'':0.00257216,('PDT000538605.1':0.0280014,('PDT000543153.1':0.022782,'PDT000485962.1':0.0190532)'':0.00465702)'':0.00563512)'':0.0129347)'':0.000862566,(('PDT000535336.1':0.0408198,'PDT000204786.1':0.0416047)'':0.00371422,(('PDT000542616.1':0.012072,'PDT000485924.1':0.0106259)'':0.0173843,('PDT000485926.1':0.0264187,('PDT000538621.1':0.023885,('PDT000543150.1':0.0194674,('PDT000720450.1':0.0169725,('PDT000650950.1':0.0129264,('PDT000616141.1':0.0104586,'PDT000716925.1':0.00900708)'':0.00319812)'':0.00295269)'':0.00196464)'':0.00363948)'':0.000250388)'':0.00140335)'':0.00783401)'':0.00147898)'':0.0129765)'':0.0328265)'':0.00634656,((('PDT000534957.1':0.044439,('PDT000534873.1':0.0410606,('PDT000535159.1':0.0215573,'PDT000537848.1':0.0227979)'':0.0164948)'':0.00453818)'':0.0275727,('PDT000107548.1':0.0642775,('PDT000425782.1':0.0667797,(('PDT000397575.1':0.0355061,'PDT000626693.1':0.0464773)'':0.00427177,(('PDT000443796.1':0.0430115,'PDT000478260.1':0.0409698)'':0.00241347,(('PDT000618772.1':0.0394334,'PDT000143926.3':0.0374595)'':0.00197915,(('PDT000685894.1':0.044785,('PDT000443370.1':0.0244339,'PDT000042605.1':0.0239643)'':0.0148811)'':0.000890322,(('PDT000188051.2':0.0377016,('PDT000673446.1':0.0364293,('PDT000636185.1':0.0244377,(('PDT000175930.2':0.0184362,'PDT000544886.1':0.0160772)'':0.00147215,('PDT000664687.2':0.0188511,('PDT000534886.1':0.013372,('PDT000136120.2':0.0152677,'PDT000664651.1':0.0146316)'':0.00166052)'':0.00174962)'':0.000682652)'':0.00857247)'':0.00406051)'':0.00825183)'':-0.000109369,(('PDT000069685.1':0.0323632,'PDT000227981.1':0.0341483)'':0.00273501,('PDT000175870.2':0.0334866,(('PDT000425711.1':0.0180989,'PDT000443897.1':0.0189127)'':0.0110636,('PDT000713913.1':0.0277499,('PDT000444942.1':0.0245744,('PDT000209178.2':0.0237314,('PDT000389327.1':0.0192162,('PDT000011748.2':0.0130533,('PDT000181695.2':0.0128394,'PDT000373305.1':0.0100864)'':0.00302391)'':0.00362144)'':0.00260795)'':0.00386823)'':0.00333835)'':-0.000106369)'':0.00549343)'':0.00402701)'':-0.0022712)'':0.00377068)'':0.00441441)'':0.00331759)'':0.00115186)'':0.0122907)'':0.00465773)'':0.00629655)'':0.00225196,((('PDT000433793.1':0.0393863,('PDT000253663.1':0.036578,('PDT000426094.1':0.0291,'PDT000615836.1':0.0228826)'':0.00866376)'':0.00718656)'':0.0119937,('PDT000019903.2':0.0452029,('PDT000652374.1':0.0526886,('PDT000184585.1':0.0531154,('PDT000618908.1':0.0210495,'PDT000231014.1':0.0204825)'':0.0144368)'':0.005923)'':0.00557842)'':0.00361776)'':0.0196777,('PDT000425532.1':0.0994209,('PDT000156799.2':0.096842,(('PDT000425905.1':0.0741053,('PDT000082675.4':0.0605724,('PDT000520695.1':0.0566389,('PDT000538916.2':0.0432313,('PDT000017664.2':0.0429313,('PDT000102981.1':0.0346932,('PDT000536650.1':0.0225214,('PDT000632880.1':0.0224665,('PDT000137851.2':0.0188114,'PDT000017828.2':0.0166147)'':0.00309206)'':0.00536036)'':0.0073968)'':0.00712899)'':0.00226051)'':0.00904886)'':0.00630724)'':0.00291364)'':-0.00727063,(('PDT000175965.2':0.0506008,(('PDT000201396.2':0.0142557,'PDT000543307.1':0.0135276)'':0.0205697,('PDT000502606.1':0.0204509,('PDT000717156.1':0.0192063,('PDT000714132.1':0.00547971,'PDT000346702.1':0.00528259)'':0.0134426)'':0.007705)'':0.00545385)'':0.015443)'':0.00236279,(('PDT000716922.1':0.0566117,'PDT000228221.1':0.0408413)'':0.00611862,((('PDT000598667.1':0.0305252,'PDT000482852.1':0.0310701)'':0.00534556,('PDT000582017.1':0.0255393,'PDT000346771.1':0.0201445)'':0.0171437)'':8.47318e-05,(('PDT000669008.1':0.0360637,('PDT000426564.1':0.0235726,'PDT000716945.1':0.0280973)'':0.00876937)'':-0.000469814,(('PDT000632224.1':0.0166785,'PDT000408011.1':0.0156564)'':0.0111617,('PDT000713305.1':0.015795,'PDT000520879.1':0.01455)'':0.00822825)'':0.0139855)'':0.00370134)'':0.0136907)'':-0.00214374)'':0.0015159)'':0.0342551)'':0.00308848)'':-0.0211949)'':0.000669741)'':0.0163582)'':-0.00199804,(((('PDT000651027.1':0.0176515,'PDT000651076.1':0.0117398)'':0.0557602,('PDT000347002.1':0.0496915,('PDT000494518.1':0.0469856,'PDT000716919.1':0.0477302)'':0.000845128)'':0.022556)'':0.00659831,(('PDT000535307.1':0.0613122,('PDT000589874.1':0.0415696,('PDT000445075.1':0.0417165,('PDT000618844.1':0.0327982,('PDT000535085.1':0.0281465,('PDT000202182.2':0.0205888,('PDT000426021.1':0.00672889,'PDT000485969.1':0.00734861)'':0.0130664)'':0.00712009)'':0.00207988)'':0.00923094)'':0.00348681)'':0.00773008)'':-0.00828481,(('PDT000675139.1':0.0344288,('PDT000421746.1':0.0278763,'PDT000633193.1':0.0264864)'':0.00411706)'':0.00277234,('PDT000426312.1':0.0325388,('PDT000433683.1':0.0416598,('PDT000364528.1':0.033727,(('PDT000716908.1':0.0247219,(('PDT000652345.1':0.00955335,'PDT000652378.1':0.00911045)'':0.0060585,(('PDT000523068.1':0.00394575,'PDT000664604.2':0.00284339)'':0.0125655,('PDT000652389.1':0.011265,'PDT000485978.1':0.0124406)'':0.00319307)'':0.00136215)'':0.0089708)'':-0.0047031,('PDT000184582.1':0.0248155,('PDT000280805.1':0.026296,(('PDT000485927.1':0.0175948,('PDT000626792.1':0.00877505,'PDT000636349.1':0.00757105)'':0.00963572)'':0.000599334,((('PDT000582309.1':0.00852137,'PDT000647657.1':0.00794523)'':0.00443846,('PDT000616088.1':0.00772695,('PDT000202169.2':0.00502728,'PDT000581601.1':0.00714832)'':0.00120525)'':0.00601099)'':8.68589e-05,(('PDT000191592.1':0.0129693,'PDT000231012.1':0.00953269)'':0.00451503,('PDT000583394.1':0.0127634,('PDT000425688.1':0.00999493,('PDT000202127.2':0.011104,(('PDT000583381.1':0.00332638,'PDT000257778.1':0.00296167)'':0.00288213,('PDT000712181.1':0.00764766,('PDT000426268.1':0.00313257,'PDT000582304.1':0.00553104)'':0.00265679)'':9.53458e-05)'':0.00148982)'':0.00159152)'':0.0032558)'':-0.000361607)'':0.00138081)'':0.00139701)'':0.00525973)'':0.00428672)'':-0.0042766)'':0.00833135)'':0.000574869)'':0.00263699)'':-0.00360612)'':0.0134774)'':0.0322339)'':0.00249244,((('PDT000443165.1':0.0541154,('PDT000661822.1':0.0520135,('PDT000652343.1':0.0200618,'PDT000248485.1':0.0216664)'':0.0232805)'':0.00756301)'':0.0206984,('PDT000427554.1':0.126979,('PDT000661407.1':0.0633893,('PDT000499939.2':0.0735182,('PDT000498197.1':0.0231904,('PDT000011764.2':0.0141051,('PDT000425455.1':0.0118306,'PDT000428838.1':0.0151428)'':0.00160474)'':0.00447023)'':0.0331858)'':0.00501324)'':0.0242004)'':-0.0148952)'':0.0141226,(('PDT000309673.2':0.0727278,'PDT000581709.1':0.0634462)'':0.0177319,(('PDT000717046.1':0.0711349,('PDT000534776.1':0.0764706,('PDT000535305.1':0.0783432,('PDT000157490.2':0.0543763,(('PDT000226326.1':0.0343187,('PDT000538571.1':0.0287722,('PDT000537850.1':0.0273346,'PDT000583389.1':0.0277935)'':0.00255335)'':0.00430154)'':0.00617518,(('PDT000535405.1':0.0279893,('PDT000534970.1':0.0212715,'PDT000616179.1':0.0176152)'':0.0093824)'':0.00830956,(('PDT000426121.1':0.0207836,'PDT000583365.1':0.0228606)'':0.00797053,('PDT000098200.1':0.0309806,('PDT000581466.1':0.0242069,('PDT000425536.1':0.0131164,'PDT000632228.1':0.016032)'':0.00874929)'':0.002681)'':0.00351675)'':0.00295091)'':0.00602628)'':0.016502)'':0.00538006)'':0.00405193)'':0.00889704)'':0.00392579,((('PDT000675075.1':0.0431535,('PDT000674926.1':0.0170368,'PDT000714265.1':0.021896)'':0.0153992)'':0.0217633,('PDT000675559.1':0.0705908,(('PDT000441818.1':0.0355081,'PDT000716920.1':0.0324399)'':0.00375247,('PDT000136154.2':0.0405156,('PDT000593090.1':0.0483024,('PDT000441837.1':0.0363045,('PDT000674854.1':0.0238089,'PDT000675383.1':0.0351835)'':0.0099503)'':0.00528868)'':0.00239765)'':-0.000417589)'':0.0177253)'':-0.00585014)'':0.0131483,(('PDT000675157.1':0.0668378,('PDT000433469.1':0.0591278,('PDT000716906.1':0.045638,('PDT000675126.1':0.038316,('PDT000411321.1':0.0304973,('PDT000618940.1':0.0236893,(('PDT000674951.1':0.0217855,'PDT000520828.1':0.0180603)'':0.00184443,('PDT000097042.1':0.0288009,('PDT000675193.1':0.0168816,(('PDT000622454.1':0.00537684,'PDT000510084.1':0.00749616)'':0.00308227,('PDT000157223.2':0.0104654,('PDT000445974.1':0.00898768,'PDT000674897.1':0.00924822)'':0.00159049)'':0.000132734)'':0.00763226)'':0.00299716)'':0.00268012)'':0.00211961)'':0.00291932)'':0.00119233)'':0.0139032)'':0.00620992)'':0.0051275)'':-0.000596659,((('PDT000537806.1':0.0250886,'PDT000708696.1':0.0227311)'':0.0115482,('PDT000425998.1':0.0306251,('PDT000537883.1':0.0410252,('PDT000434513.1':0.0184151,('PDT000537817.1':0.0153057,('PDT000537815.1':0.0132805,'PDT000538561.1':0.0135731)'':0.00037838)'':0.00542046)'':0.0109508)'':0.00426589)'':0.0017485)'':0.0208939,(('PDT000537816.1':0.0539365,'PDT000501905.1':0.0514965)'':0.0126649,(('PDT000632497.1':0.0181874,'PDT000632888.1':0.0261328)'':0.0243749,('PDT000426661.1':0.0480758,(('PDT000202091.2':0.04822,'PDT000097037.1':0.0455718)'':0.00378328,(('PDT000346788.1':0.0356518,('PDT000173740.2':0.024429,(('PDT000583364.1':0.0114312,'PDT000583396.1':0.0111901)'':0.0063814,('PDT000538576.1':0.0231483,('PDT000536665.1':0.0131527,'PDT000520852.1':0.0167618)'':0.00474643)'':0.00130647)'':0.00284804)'':0.00702014)'':-0.000731047,(('PDT000616068.1':0.0190574,'PDT000618826.1':0.0220021)'':0.0107828,(('PDT000619095.1':0.0123707,'PDT000627025.1':0.0165041)'':0.0116377,(('PDT000176036.2':0.016004,'PDT000538614.1':0.01634)'':0.00408151,('PDT000482851.1':0.024617,('PDT000188192.2':0.0120328,'PDT000426785.1':0.014919)'':0.00623957)'':0.000645313)'':0.00481668)'':0.00815858)'':-0.00281235)'':0.00899112)'':0.00364336)'':0.00249443)'':0.0130495)'':-0.0125709)'':0.00660329)'':0.0161384)'':0.00541032)'':0.00789912)'':-0.00142943)'':-0.000127246)'':0.000284445)'':0.0114557)'':0.0123987)'':0.0162959)'':0.0156445)'':0.0462652)'':0.0413225)'':0.00411197)'':-0.019211)'':0.0145005)'':0.00663152)'':0.0104105)'':0.0349295,((((('PDT000203707.1':0.196002,('PDT000607583.1':0.132132,('PDT000565529.1':0.165025,('PDT000426562.1':0.114445,(('PDT000557014.1':0.09101,('PDT000252509.1':0.075608,('PDT000372312.1':0.0658545,('PDT000548138.1':0.0497585,'PDT000714242.1':0.0413886)'':0.0164395)'':0.0228455)'':0.00494799)'':0.00881219,(('PDT000208426.2':0.0718425,('PDT000626014.1':0.0268641,'PDT000633112.1':0.0346744)'':0.0278862)'':0.0160153,('PDT000713843.1':0.0699175,('PDT000656748.1':0.0783914,('PDT000289916.1':0.0439373,('PDT000389643.1':0.0452835,('PDT000524504.1':0.0339054,'PDT000718334.1':0.0334722)'':0.00508174)'':0.00203356)'':0.0240779)'':0.00895252)'':-0.00183303)'':0.0128333)'':0.0140952)'':0.0154035)'':0.00157831)'':0.0572888)'':0.00865456,(('PDT000566709.1':0.0733543,'PDT000251765.1':0.0769377)'':0.100375,('PDT000502201.1':0.172344,(('PDT000703268.1':0.0844844,('PDT000196663.2':0.0510718,('PDT000141278.2':0.0448507,('PDT000290429.2':0.0331874,('PDT000109427.2':0.0236025,'PDT000466979.1':0.0260771)'':0.0105165)'':0.00400765)'':0.00922232)'':0.0101189)'':0.037629,('PDT000494195.1':0.0941031,('PDT000568765.1':0.0961297,('PDT000637180.1':0.0899971,('PDT000359649.1':0.0677905,(('PDT000687835.1':0.0577449,('PDT000365303.1':0.00819479,'PDT000712996.1':0.00689201)'':0.0367108)'':0.0178649,('PDT000027889.2':0.0601564,('PDT000185860.1':0.0561468,('PDT000354894.1':0.0277178,'PDT000598395.1':0.0339937)'':0.0124849)'':0.00550671)'':0.00140153)'':0.0024821)'':0.0107359)'':0.00848702)'':0.0126009)'':0.0070049)'':0.0604777)'':0.0181077)'':0.00629322)'':0.0253642,(((('PDT000662618.1':0.0365969,'PDT000250132.1':0.0319527)'':0.166349,('PDT000252614.1':0.11472,('PDT000101289.1':0.0870297,'PDT000101302.1':0.0894083)'':0.0280674)'':0.0908061)'':0.0127059,(('PDT000111706.2':0.0811555,'PDT000248718.1':0.0897655)'':0.00383715,('PDT000610769.1':0.033271,(('PDT000097453.2':0.03166,('PDT000443098.1':0.0199938,'PDT000578968.1':0.0215045)'':0.00627587)'':0.000530591,('PDT000357711.1':0.02016,('PDT000097452.2':0.0119067,'PDT000132231.2':0.0153994)'':0.00542114)'':0.00490887)'':0.00696841)'':0.0517453)'':0.141401)'':0.0126274,(((('PDT000548605.2':0.137165,('PDT000706408.1':0.0836479,('PDT000466091.1':0.0578503,'PDT000664967.1':0.0655457)'':0.0385326)'':0.0475334)'':0.00960196,(('PDT000325694.1':0.142372,'PDT000672206.1':0.135619)'':0.000461189,(('PDT000656719.1':0.1166,'PDT000491116.1':0.111623)'':0.0167527,('PDT000617804.1':0.0675494,('PDT000196608.2':0.0260738,'PDT000196665.2':0.0283513)'':0.0378191)'':0.0752358)'':0.00908765)'':0.0013891)'':0.00142581,(('PDT000359332.1':0.035966,'PDT000660649.1':0.0426507)'':0.0689691,('PDT000509493.1':0.0867572,(('PDT000557007.1':0.0496467,('PDT000157195.2':0.0544446,'PDT000355103.1':0.0378872)'':0.0129727)'':0.0115373,(('PDT000557087.1':0.0490913,'PDT000228161.1':0.047285)'':0.00804474,('PDT000305517.2':0.0552457,('PDT000499662.1':0.0345693,'PDT000509496.1':0.0354182)'':0.0199666)'':0.00712407)'':0.013002)'':0.0268005)'':0.00776847)'':0.0489102)'':0.0146165,(('PDT000063291.3':0.120487,('PDT000205449.1':0.107668,(('PDT000445007.1':0.0733829,(('PDT000710127.1':0.0526816,'PDT000042676.1':0.0483254)'':0.0230837,('PDT000011825.2':0.0546776,('PDT000203703.1':0.0535652,('PDT000675365.1':0.043973,'PDT000607483.1':0.0443442)'':0.00214978)'':0.00987718)'':0.0073783)'':0.000741503)'':0.00444805,('PDT000247371.2':0.0890912,('PDT000026015.1':0.0903231,(('PDT000411013.1':0.0445122,'PDT000718886.1':0.0480679)'':0.0155902,('PDT000240652.3':0.0487255,(('PDT000417906.2':0.0358828,'PDT000137554.3':0.0355458)'':0.00592653,('PDT000114754.3':0.0324471,'PDT000300539.1':0.0387165)'':0.00166729)'':0.0148768)'':0.0107624)'':0.0127496)'':0.00163698)'':-0.00587298)'':0.00949042)'':0.0233087)'':0.00860411,(('PDT000399567.1':0.0955725,('PDT000240305.1':0.0749818,('PDT000326061.1':0.067661,'PDT000069933.3':0.064858)'':0.0146932)'':0.0155584)'':-0.0034593,((('PDT000015853.3':0.0407732,'PDT000261044.1':0.0355184)'':0.00359137,(('PDT000327976.1':0.0223799,'PDT000261039.1':0.0249771)'':0.000438077,('PDT000213246.2':0.0107317,('PDT000157827.2':0.00788847,('PDT000548989.1':0.00633051,'PDT000625430.1':0.00921039)'':0.00359143)'':0.00614189)'':0.00373356)'':0.0122925)'':0.0167197,((('PDT000611512.1':0.0236741,('PDT000323330.1':0.00740656,'PDT000129831.2':0.00563084)'':0.0120437)'':0.0104279,('PDT000625496.1':0.0196168,('PDT000327900.1':0.0119866,('PDT000625518.1':0.00645985,('PDT000555118.1':0.00428528,'PDT000396206.1':0.00280196)'':0.00670973)'':0.00230252)'':0.00379057)'':0.0136529)'':0.0125405,(('PDT000336794.1':0.0254292,(('PDT000618543.1':0.00722361,'PDT000635986.1':0.00426949)'':0.00332922,('PDT000651399.1':0.0139742,('PDT000319823.1':0.0054879,('PDT000254356.2':0.00229333,'PDT000317717.1':0.00229795)'':0.00129281)'':0.00478171)'':6.4987e-05)'':0.010452)'':-9.0402e-05,(('PDT000313354.2':0.0209114,'PDT000261042.1':0.02506)'':0.00120591,(('PDT000120861.2':0.0219726,'PDT000399943.1':0.0189723)'':0.00252615,('PDT000309604.2':0.0149615,(('PDT000261045.1':0.00339548,('PDT000483975.1':0.00230685,'PDT000486023.1':0.00399)'':0.0026897)'':0.000645771,('PDT000662585.1':0.00909112,('PDT000486029.1':0.00494402,('PDT000625517.1':0.00208265,('PDT000150709.2':0.00136013,('PDT000386187.1':0.000581269,('PDT000579615.1':0.000420805,'PDT000667974.2':0.000378875)'':0.000118321)'':0.000697964)'':0.00119317)'':0.0031796)'':0.00226294)'':-0.000759724)'':0.00516795)'':0.00456798)'':0.00293654)'':0.00124906)'':0.0206977)'':0.00363932)'':0.0501148)'':0.0233133)'':0.0444687)'':0.0715084)'':0.00531129)'':-0.00807362,((('PDT000290524.2':0.0270953,('PDT000645107.1':0.0249091,'PDT000346600.1':0.0222727)'':0.00386815)'':0.166972,(((('PDT000242980.2':0.0708104,('PDT000497307.1':0.0454413,'PDT000627342.1':0.0418908)'':0.0212286)'':0.0160531,('PDT000619960.1':0.0602251,('PDT000242874.1':0.0269947,'PDT000242904.1':0.0280019)'':0.0496641)'':0.0195733)'':0.0275601,(('PDT000228198.1':0.0415133,'PDT000248595.1':0.051295)'':0.0437884,('PDT000518126.1':0.0818082,('PDT000556919.1':0.0470161,('PDT000718829.1':0.0216376,'PDT000719151.1':0.0228836)'':0.0155292)'':0.0251785)'':0.00940368)'':0.0308788)'':0.0066244,(('PDT000155888.2':0.029113,'PDT000272561.2':0.0261811)'':0.107899,('PDT000690316.1':0.102897,(('PDT000307439.2':0.0629813,('PDT000228068.1':0.0498615,'PDT000250155.1':0.0562295)'':0.00860269)'':0.0203643,('PDT000519804.1':0.0547554,(('PDT000425481.1':0.0519017,'PDT000083498.1':0.0557353)'':0.0011503,('PDT000290528.2':0.060535,('PDT000619575.1':0.0335275,'PDT000251387.1':0.0350761)'':0.0108809)'':0.00926241)'':0.00113937)'':0.016599)'':0.0283755)'':0.0193634)'':-0.00284591)'':0.0838873)'':0.040996,((((('PDT000571194.1':0.142732,(('PDT000671950.1':0.0215912,'PDT000679872.1':0.0236811)'':0.0134745,('PDT000689926.1':0.0318468,('PDT000692384.1':0.028675,('PDT000390955.1':0.0259435,'PDT000120869.2':0.0182265)'':0.00418513)'':0.00680439)'':0.00103)'':0.138037)'':0.0743096,((('PDT000160853.4':0.007641,'PDT000378634.2':0.0084365)'':0.0180627,('PDT000384310.2':0.00749728,'PDT000403492.2':0.00904972)'':0.0130331)'':0.0227281,((('PDT000006481.2':0.0160561,('PDT000039728.1':0.00523216,'PDT000039729.1':0.0040465)'':0.0125906)'':0.00864707,('PDT000190868.3':0.0227812,('PDT000380947.1':0.0127709,('PDT000289971.1':0.00696796,('PDT000380954.1':0.00372903,'PDT000380958.1':0.00486238)'':0.00228096)'':0.0106034)'':0.00878357)'':0.00324242)'':0.0122303,((('PDT000384314.2':0.00580747,'PDT000403476.2':0.00455213)'':0.0134551,('PDT000377041.2':0.0232297,('PDT000378520.2':0.00752972,'PDT000379449.2':0.0103168)'':0.0127258)'':0.00125708)'':0.00874966,(('PDT000289972.1':0.00117562,'PDT000289973.1':0.00142204)'':0.0233439,('PDT000261048.1':0.0366573,(('PDT000289976.1':0.0154622,('PDT000292859.1':0.0158841,('PDT000289974.1':0.00943282,'PDT000292855.1':0.0121786)'':0.0039799)'':0.0051627)'':0.00664766,('PDT000048760.1':0.0373995,(('PDT000004650.2':0.00816048,'PDT000354227.1':0.00822972)'':0.0083699,('PDT000289975.1':0.0133775,('PDT000289969.1':0.00169024,'PDT000485338.1':0.00240812)'':0.010479)'':0.00163408)'':0.0119297)'':-0.00497257)'':0.00793784)'':-0.0027373)'':0.00570081)'':0.00348568)'':0.0104957)'':0.228208)'':0.0081085,('PDT000619618.1':0.259038,(('PDT000346616.1':0.230733,((('PDT000565912.1':0.0510834,'PDT000687689.1':0.0696616)'':0.0155276,('PDT000367278.1':0.0512521,('PDT000231471.2':0.0478872,'PDT000375003.1':0.0435403)'':0.00508566)'':0.0309929)'':0.0381213,('PDT000379418.1':0.150109,(('PDT000656951.1':0.0791328,('PDT000329051.1':0.0681463,'PDT000426457.1':0.0778917)'':0.0022787)'':0.0312446,('PDT000289888.1':0.0870135,('PDT000332054.1':0.0877655,('PDT000299891.2':0.0612053,('PDT000258737.2':0.0175856,'PDT000045603.1':0.0174917)'':0.0466866)'':0.0192966)'':0.00410421)'':0.0176054)'':0.0226336)'':-0.00786608)'':0.110818)'':-0.00852161,(((('PDT000557668.2':0.0559433,'PDT000251777.1':0.0527277)'':0.151008,(('PDT000044121.1':0.0754257,'PDT000346615.1':0.0924923)'':0.0345955,('PDT000228101.1':0.107847,('PDT000228108.1':0.0616208,'PDT000238519.1':0.0857252)'':0.0166674)'':0.0160075)'':0.10069)'':0.00753166,((('PDT000619574.1':0.165102,('PDT000109328.2':0.117397,('PDT000249876.1':0.113618,('PDT000332757.1':0.09636,'PDT000257774.1':0.091277)'':0.0203178)'':0.00953282)'':0.0246427)'':-0.0107664,(('PDT000653420.1':0.0718754,'PDT000050625.1':0.0735256)'':0.0518737,((('PDT000202038.2':0.0830285,('PDT000549002.1':0.086806,('PDT000357624.1':0.0599721,'PDT000360993.1':0.0596579)'':0.014584)'':0.00576651)'':0.00559581,('PDT000346607.1':0.0461517,('PDT000662559.1':0.027286,('PDT000034404.1':0.00528629,'PDT000025993.1':0.00585471)'':0.0165521)'':0.015133)'':0.0526868)'':0.00898313,((('PDT000502740.1':0.0591648,'PDT000541648.1':0.0610022)'':0.00211462,('PDT000480346.1':0.0432548,'PDT000644676.1':0.0434387)'':0.0117834)'':0.040273,(('PDT000276020.1':0.0848438,('PDT000598441.1':0.040746,('PDT000279507.3':0.0306736,'PDT000497936.1':0.027246)'':0.0172643)'':0.0263165)'':0.0029249,('PDT000648041.1':0.0911631,('PDT000645529.2':0.0788111,('PDT000346593.1':0.0560878,('PDT000485405.1':0.0390083,('PDT000485430.1':0.036756,'PDT000719117.1':0.0392838)'':0.000472217)'':0.00885836)'':0.0128413)'':0.006129)'':-0.00293154)'':0.0217006)'':0.00105068)'':0.0339961)'':0.0140085)'':0.0556947,(('PDT000541627.1':0.078203,('PDT000228162.1':0.0881417,('PDT000231469.2':0.0421528,'PDT000386757.1':0.0386962)'':0.0361388)'':0.00587854)'':0.112592,(('PDT000709969.1':0.183939,('PDT000687962.1':0.168601,('PDT000145173.2':0.0467211,'PDT000502538.1':0.052839)'':0.134091)'':0.014563)'':0.0091287,((('PDT000228144.1':0.0575447,('PDT000294673.2':0.0587552,('PDT000248466.1':0.0443686,('PDT000242917.1':0.0440572,'PDT000242999.1':0.0337983)'':0.0113141)'':0.00890773)'':0.00905543)'':0.02483,('PDT000266404.2':0.0521367,('PDT000417383.1':0.0304891,('PDT000258730.2':0.023797,('PDT000644916.1':0.0194186,('PDT000191802.2':0.0203689,'PDT000408753.1':0.0183946)'':0.00106408)'':0.00625133)'':0.00495333)'':0.0127199)'':0.042136)'':0.00408493,('PDT000333202.1':0.100528,(('PDT000420679.1':0.0249882,'PDT000230854.1':0.0330072)'':0.0389178,('PDT000363572.1':0.0663398,('PDT000109304.2':0.0588762,('PDT000573620.1':0.0496852,(('PDT000645706.2':0.0386417,'PDT000369035.1':0.0437249)'':0.00177796,('PDT000479091.1':0.0330716,('PDT000355637.1':0.0202533,'PDT000252534.1':0.0213929)'':0.00615208)'':0.0155225)'':0.00711867)'':0.00521853)'':0.00263635)'':0.00248496)'':0.0175225)'':0.00784185)'':0.107711)'':0.0027552)'':-0.0071797)'':0.0103763)'':0.00794298,(((('PDT000241668.2':0.0780617,'PDT000081727.1':0.0906663)'':0.090752,('PDT000264798.2':0.0235323,'PDT000655598.1':0.0217037)'':0.15525)'':0.0382947,(('PDT000161408.2':0.0331717,'PDT000345946.1':0.0445367)'':0.0617885,(('PDT000147823.2':0.0569335,'PDT000577095.1':0.0471185)'':0.0270802,('PDT000240692.2':0.0724491,('PDT000346495.1':0.0571129,'PDT000411328.1':0.0632081)'':0.00933541)'':0.00908784)'':0.0178866)'':0.115322)'':0.00261248,((('PDT000357709.1':0.207885,('PDT000290389.2':0.0337912,'PDT000035572.1':0.0402033)'':0.170754)'':-0.00362083,((('PDT000497676.2':0.0714965,('PDT000352545.1':0.0367325,('PDT000500492.1':0.0215236,('PDT000518004.1':0.0187947,('PDT000342663.1':0.01033,'PDT000499663.1':0.0113876)'':0.00767151)'':0.00676398)'':0.0187135)'':0.0333826)'':0.113695,(('PDT000652425.1':0.0676198,('PDT000175382.2':0.0547376,('PDT000145180.2':0.0504084,'PDT000405231.1':0.0465922)'':0.00493361)'':0.0204627)'':0.0206224,('PDT000620271.1':0.0464557,('PDT000325533.1':0.0361585,('PDT000106250.2':0.0327978,'PDT000501829.1':0.0341976)'':0.0023591)'':0.00787994)'':0.0490404)'':0.0871602)'':0.0176221,(('PDT000395399.1':0.0732138,('PDT000497369.1':0.0335836,('PDT000600796.1':0.0260478,'PDT000639535.1':0.0256091)'':0.0138391)'':0.0370106)'':0.131901,('PDT000102995.1':0.19247,('PDT000435642.1':0.200283,('PDT000342280.2':0.0481787,(('PDT000574391.2':0.0294017,('PDT000177081.3':0.0207121,'PDT000351840.2':0.0190112)'':0.00689003)'':0.0033032,(('PDT000491879.2':0.025731,('PDT000152932.3':0.00926336,('PDT000702731.1':0.00782841,('PDT000551010.2':0.00752875,('PDT000537061.2':0.00835827,('PDT000547408.2':0.00224316,'PDT000618309.2':0.00294646)'':0.00320762)'':0.00102714)'':0.00104214)'':0.00139877)'':0.011923)'':-0.00310304,('PDT000721066.1':0.0261389,(('PDT000084226.3':0.0186209,('PDT000660098.2':0.0143835,('PDT000521269.2':0.00881968,'PDT000551400.2':0.00769942)'':0.00387367)'':0.0040428)'':0.000146753,('PDT000102824.3':0.0191124,('PDT000098735.3':0.0153852,('PDT000569694.2':0.0126025,('PDT000142616.3':0.00874115,'PDT000137674.3':0.0105945)'':0.00180562)'':0.00234716)'':0.00344576)'':-0.000454066)'':0.00647843)'':-0.00320827)'':0.00896799)'':0.0068567)'':0.162482)'':0.00400387)'':-0.00875016)'':0.000373016)'':0.00947314)'':-0.00290626,((('PDT000251742.1':0.0318396,('PDT000714582.1':0.0221124,'PDT000250168.1':0.0185622)'':0.0142493)'':0.0169039,('PDT000178566.2':0.0471089,('PDT000274857.2':0.0494176,(('PDT000468334.1':0.033621,'PDT000571464.1':0.0384405)'':0.00326532,('PDT000396835.1':0.0269553,'PDT000045590.1':0.0267673)'':0.00981218)'':0.00518655)'':0.00729057)'':0.00211071)'':0.149931,(('PDT000718846.1':0.168183,('PDT000129904.2':0.0725422,(('PDT000293480.1':0.00927903,'PDT000293524.1':0.0115502)'':0.0200741,('PDT000293485.1':0.0149495,'PDT000293497.1':0.0154492)'':0.0216764)'':0.0588453)'':0.0981999)'':0.0307331,(('PDT000145184.2':0.0291216,'PDT000258401.2':0.0328055)'':0.205039,(('PDT000619600.1':0.171658,'PDT000716384.1':0.185761)'':0.0200524,(((('PDT000333673.1':0.066088,'PDT000250061.1':0.089219)'':0.0849025,('PDT000228352.1':0.0571122,('PDT000619610.1':0.0495499,'PDT000250194.1':0.0412017)'':0.00874403)'':0.0978312)'':0.0365448,('PDT000714681.1':0.160029,('PDT000716392.1':0.15159,(('PDT000494430.1':0.0594426,'PDT000659700.1':0.0467924)'':0.00685672,('PDT000637083.1':0.0458527,('PDT000656993.1':0.0404438,('PDT000365289.1':0.0312918,'PDT000645006.1':0.0390833)'':0.00852003)'':0.0050652)'':0.0067312)'':0.104686)'':0.00933348)'':0.0228696)'':0.00120566,('PDT000641720.1':0.193136,(('PDT000129907.2':0.052331,('PDT000357619.1':0.0248199,'PDT000357702.1':0.0282389)'':0.0174023)'':0.049713,('PDT000652420.1':0.0687837,(('PDT000228045.1':0.0781432,('PDT000339369.1':0.0414086,'PDT000137870.2':0.0438926)'':0.0148467)'':0.00129201,('PDT000354923.1':0.0635634,(('PDT000139203.2':0.0302901,'PDT000652309.1':0.0344141)'':0.00793447,('PDT000619560.1':0.0482528,('PDT000288551.2':0.0397306,('PDT000500529.1':0.0215695,'PDT000501929.1':0.0205265)'':0.0150316)'':0.00379341)'':-0.000838064)'':0.0114364)'':0.00588273)'':0.011725)'':0.0249406)'':0.0872609)'':0.00140653)'':0.0069542)'':0.00572096)'':-0.0016623)'':0.000241281)'':0.00181209)'':0.0085186)'':-0.00148034)'':0.00473825)'':0.038246)'':-0.0225558)'':-0.00775623,(((((('PDT000298064.1':0.116551,('PDT000417140.1':0.0849475,'PDT000621001.1':0.0712935)'':0.0224032)'':0.0452335,(('PDT000215764.2':0.0852355,'PDT000709968.1':0.0709145)'':0.0158899,('PDT000672192.1':0.0864523,('PDT000128817.2':0.107913,(('PDT000644684.1':0.0494829,('PDT000397562.1':0.0226483,'PDT000397819.1':0.022157)'':0.0213222)'':0.00302171,('PDT000425729.1':0.0682749,('PDT000250030.1':0.0398656,('PDT000573636.1':0.0381075,('PDT000497321.1':0.0261924,('PDT000545604.1':0.018656,'PDT000565952.1':0.0225811)'':0.00131127)'':0.0130427)'':0.00342849)'':0.0146022)'':-0.00526551)'':0.0272361)'':0.00364337)'':-0.00140497)'':0.0425389)'':-0.00346376,(('PDT000228138.1':0.0892521,'PDT000252734.1':0.0950129)'':0.0694755,(('PDT000584531.1':0.0724855,('PDT000355628.1':0.0454621,('PDT000150227.2':0.0364447,'PDT000639986.1':0.0441596)'':0.0232989)'':0.0119332)'':0.0197941,('PDT000718878.1':0.0785677,(('PDT000622874.1':0.0501079,'PDT000251473.1':0.0648151)'':0.0139671,('PDT000088261.2':0.0731335,(('PDT000502250.1':0.0483826,'PDT000383094.1':0.0473158)'':0.00379123,('PDT000645476.2':0.0599468,('PDT000654214.1':0.0467996,('PDT000228280.1':0.0240108,'PDT000252700.1':0.0289003)'':0.0157657)'':0.00787078)'':6.94344e-05)'':0.00900536)'':0.00582136)'':0.00723309)'':0.000402985)'':0.065943)'':-0.00984475)'':0.00790958,((('PDT000411309.1':0.0906226,('PDT000433670.1':0.0525316,'PDT000653016.1':0.0503854)'':0.0352144)'':0.0139498,((('PDT000196632.2':0.0294254,'PDT000200129.2':0.0426289)'':0.0220214,('PDT000249822.1':0.0401837,('PDT000285619.2':0.038881,'PDT000375012.1':0.0321434)'':0.00578736)'':0.00727987)'':0.0225832,(('PDT000557172.1':0.0398736,('PDT000581030.1':0.0386975,'PDT000017841.2':0.0328355)'':0.00749502)'':0.000637155,('PDT000519810.1':0.0405145,('PDT000270501.1':0.0348247,('PDT000485570.1':0.0243085,('PDT000221666.2':0.0186542,'PDT000639558.1':0.0204283)'':0.00858963)'':0.00979646)'':0.006393)'':0.00306449)'':0.0339414)'':0.0246511)'':-0.00399825,((('PDT000414212.1':0.116624,('PDT000298020.1':0.0929058,('PDT000285388.2':0.0790856,('PDT000239221.2':0.0533621,'PDT000225231.1':0.0600779)'':0.0258699)'':0.00883476)'':0.0282634)'':-0.0172365,('PDT000656978.1':0.10645,(('PDT000314214.2':0.0882976,('PDT000645518.2':0.0693541,'PDT000346451.1':0.0514219)'':0.0234274)'':0.00241795,('PDT000336557.1':0.0786826,('PDT000672219.1':0.0682071,(('PDT000645500.2':0.0496769,('PDT000136715.1':0.0189567,'PDT000346614.1':0.0198114)'':0.0280972)'':0.0161823,('PDT000468838.1':0.0299273,('PDT000675440.1':0.0296034,('PDT000584537.1':0.0184807,'PDT000319027.1':0.0155119)'':0.00373222)'':0.0036961)'':0.0156919)'':0.00243041)'':0.0231244)'':0.00532313)'':0.00835959)'':0.000365684)'':-0.00182255,(('PDT000661181.1':0.0967483,(('PDT000468913.1':0.0913698,('PDT000417130.1':0.0673699,'PDT000287995.1':0.0566351)'':0.0219072)'':0.00457097,(('PDT000048248.1':0.0376809,'PDT000289879.1':0.0526518)'':0.0274015,('PDT000298044.1':0.0695483,'PDT000459423.1':0.0830967)'':0.00920143)'':0.0031036)'':0.0111231)'':0.000278827,('PDT000346613.1':0.0757046,(('PDT000124663.1':0.0704424,('PDT000417088.1':0.0771784,('PDT000202199.2':0.0512865,'PDT000448187.1':0.0701185)'':0.00980764)'':0.00874409)'':0.00261665,(('PDT000396741.1':0.0284379,'PDT000396744.1':0.0254391)'':0.017176,('PDT000632688.1':0.0463689,('PDT000289894.1':0.0326211,'PDT000346594.1':0.0399926)'':0.0122857)'':0.00700181)'':0.0270268)'':0.00113244)'':0.00599595)'':0.00715104)'':0.0122302)'':0.0288556)'':0.0620761,(('PDT000293477.1':0.254219,(('PDT000652575.1':0.226552,((('PDT000293436.1':0.0217906,'PDT000293501.1':0.0287334)'':0.158175,('PDT000406997.1':0.0951016,('PDT000326059.1':0.0621641,('PDT000379420.1':0.0395955,'PDT000345962.1':0.0366329)'':0.0208042)'':0.0364881)'':0.0988898)'':0.00124928,('PDT000714655.1':0.176028,('PDT000652506.1':0.159239,('PDT000671252.1':0.12596,('PDT000252453.1':0.0850855,('PDT000107084.2':0.0144999,'PDT000135791.2':0.0105573)'':0.0651859)'':0.056408)'':0.0291389)'':0.0113183)'':0.00457731)'':0.0216901)'':-0.000705189,(('PDT000337222.2':0.0315569,'PDT000359101.2':0.0373307)'':0.164056,('PDT000667757.2':0.206599,(('PDT000439455.1':0.149957,'PDT000701136.1':0.159731)'':0.0227096,((('PDT000687973.1':0.118123,'PDT000716407.1':0.14148)'':0.0371699,('PDT000652240.1':0.157295,'PDT000709941.1':0.164462)'':0.0108674)'':0.00410106,('PDT000701137.1':0.140144,(('PDT000252709.1':0.127764,('PDT000652252.1':0.0894809,('PDT000150859.1':0.0997769,'PDT000233073.1':0.0838041)'':0.00101112)'':0.0277657)'':0.0265676,(('PDT000129914.3':0.107391,'PDT000604977.1':0.0902838)'':0.0311172,('PDT000251739.1':0.0539779,('PDT000503657.1':0.0725172,'PDT000568676.2':0.0671088)'':0.0142971)'':0.061435)'':0.0229764)'':0.00453227)'':0.0104193)'':0.0125821)'':0.00378735)'':0.0125997)'':0.0134711)'':0.0415895)'':-0.0400886,(('PDT000547547.1':0.0659434,'PDT000628614.1':0.0485066)'':0.169905,((('PDT000661531.1':0.200925,(('PDT000253473.3':0.00827855,'PDT000619452.2':0.00829545)'':0.0295222,('PDT000406855.2':0.0324338,('PDT000161985.3':0.0262998,('PDT000253449.3':0.0223124,('PDT000253465.3':0.0105931,'PDT000123321.1':0.00971198)'':0.00759373)'':0.00123969)'':0.00453499)'':0.00314231)'':0.167692)'':-0.00645993,(('PDT000036085.1':0.211443,('PDT000652888.1':0.111113,'PDT000652980.1':0.0939664)'':0.104581)'':0.00558881,((('PDT000535522.1':0.186119,'PDT000626809.1':0.17592)'':0.0134487,(('PDT000713619.1':0.0536951,'PDT000619681.1':0.0649669)'':0.0100497,(('PDT000354920.1':0.0706838,'PDT000250022.1':0.0705192)'':0.00245813,('PDT000654341.1':0.0598994,('PDT000343731.1':0.0203916,'PDT000688595.1':0.0338998)'':0.0184662)'':0.0185753)'':0.00370906)'':0.105893)'':0.00143058,((('PDT000150233.2':0.0731599,'PDT000691363.1':0.0745601)'':0.0878959,(('PDT000709930.1':0.0465227,'PDT000345969.1':0.0473088)'':0.0613933,('PDT000081745.1':0.0596745,('PDT000317571.2':0.0574536,'PDT000134253.1':0.0531934)'':0.0146975)'':0.0344063)'':0.0509644)'':0.0181921,((('PDT000408650.1':0.0443716,('PDT000019203.2':0.0337929,('PDT000161552.2':0.0114143,('PDT000466035.1':0.00857277,'PDT000553149.1':0.00538673)'':0.00793018)'':0.0232695)'':0.00480176)'':0.10161,('PDT000706520.1':0.0451511,(('PDT000497362.1':0.012002,'PDT000677958.1':0.0160428)'':0.010343,(('PDT000626528.1':0.0186354,'PDT000714686.1':0.0176478)'':0.00164543,('PDT000389641.1':0.0152763,'PDT000395387.1':0.0204309)'':0.00705512)'':0.0015362)'':0.0103563)'':0.112667)'':0.0257535,((('PDT000661129.1':0.11377,'PDT000008771.2':0.0976271)'':0.0770257,(('PDT000208732.1':0.172846,'PDT000395218.1':0.143224)'':0.00669917,('PDT000354168.1':0.157211,('PDT000095748.2':0.0542895,('PDT000652234.1':0.0232907,'PDT000652237.1':0.0231298)'':0.0273523)'':0.0904396)'':0.00968018)'':0.00659021)'':0.00308126,(('PDT000293498.1':0.159748,('PDT000177798.1':0.101516,('PDT000652419.1':0.0702335,('PDT000405220.1':0.0264821,'PDT000487580.1':0.0229528)'':0.038459)'':0.0132888)'':0.0581365)'':0.0264911,(('PDT000619573.1':0.158763,('PDT000395042.1':0.0831772,'PDT000639519.1':0.0699808)'':0.0566165)'':0.0294042,(('PDT000502338.1':0.175964,('PDT000717209.1':0.141996,'PDT000502028.1':0.140231)'':0.0177707)'':0.00107181,(('PDT000444813.1':0.0904562,('PDT000382558.1':0.0769344,('PDT000163302.3':0.0715632,('PDT000553332.2':0.0557178,('PDT000574389.2':0.0193678,'PDT000677008.1':0.0275815)'':0.0216783)'':0.0204432)'':0.00951817)'':0.00747443)'':0.0647014,(('PDT000050610.1':0.0157588,'PDT000498093.1':0.0161341)'':0.124688,(('PDT000313333.2':0.0245857,('PDT000128818.2':0.0258867,'PDT000128810.2':0.0256527)'':0.00941641)'':0.0272,('PDT000251737.1':0.0300908,('PDT000645108.1':0.018527,'PDT000395368.1':0.0161693)'':0.00833788)'':0.0276888)'':0.0754708)'':0.0201269)'':0.0214216)'':0.000892814)'':0.00146653)'':0.001316)'':0.00138512)'':0.00344127)'':0.0039192)'':0.00675647)'':-0.0018136)'':0.000334534,(('PDT000604370.1':0.0995219,('PDT000310563.2':0.0892667,'PDT000251383.1':0.0727793)'':0.0268106)'':0.0822561,((('PDT000627369.1':0.0418582,'PDT000017904.2':0.0357929)'':0.0483494,('PDT000557271.1':0.105935,(('PDT000661714.1':0.0269393,'PDT000011730.2':0.0261848)'':0.0452047,(('PDT000718448.1':0.0522042,('PDT000252572.1':0.0528311,('PDT000385540.1':0.0505972,'PDT000548630.2':0.0489744)'':0.00618063)'':0.0045102)'':0.00151519,('PDT000362436.1':0.0496117,('PDT000042577.1':0.0342252,('PDT000584664.1':0.029923,'PDT000571192.1':0.0297176)'':0.00642368)'':0.0111962)'':0.0142477)'':0.00704201)'':0.0328764)'':0.00351574)'':0.0864976,(('PDT000218941.1':0.138004,('PDT000619969.1':0.0441713,('PDT000689886.1':0.052561,('PDT000645157.1':0.0121547,'PDT000668360.2':0.0151802)'':0.0389778)'':0.00691726)'':0.0788999)'':0.05828,(((('PDT000364408.1':0.0487017,'PDT000619621.1':0.0410717)'':0.110672,(('PDT000262160.2':0.0530815,'PDT000717305.1':0.0512495)'':0.0963436,('PDT000426142.1':0.0631521,('PDT000719064.1':0.0404057,('PDT000027607.1':0.0447098,('PDT000109325.2':0.024137,'PDT000251788.1':0.0208334)'':0.0131601)'':0.0046548)'':0.0152743)'':0.0859582)'':0.00983985)'':0.00382361,('PDT000669208.1':0.155927,(('PDT000573610.1':0.0645959,'PDT000654700.1':0.0834131)'':0.0490457,(('PDT000510901.1':0.0695688,('PDT000653023.1':0.0582589,('PDT000502699.1':0.0340347,('PDT000393061.1':0.0221433,'PDT000412018.1':0.0182449)'':0.0130953)'':0.0185186)'':0.00817266)'':0.00948198,('PDT000176699.3':0.0868028,('PDT000123260.1':0.062121,('PDT000496159.2':0.0462806,(('PDT000389390.2':0.0295307,'PDT000107692.2':0.030378)'':0.00360359,('PDT000524121.2':0.0144449,'PDT000625614.2':0.0126248)'':0.0169988)'':0.0158593)'':0.00926979)'':0.0241909)'':0.0154371)'':0.0508247)'':0.0202746)'':0.0142271)'':0.0113125,('PDT000324960.1':0.178023,(('PDT000150888.1':0.152487,('PDT000061198.3':0.0137298,'PDT000502897.1':0.0179799)'':0.138326)'':0.0327939,(('PDT000719023.1':0.169058,(('PDT000417978.1':0.0712871,'PDT000501944.1':0.0627389)'':0.0558216,('PDT000719200.1':0.108334,'PDT000304214.1':0.116365)'':0.0229516)'':0.0411531)'':0.00956385,((('PDT000502499.1':0.146099,'PDT000005859.2':0.148396)'':0.021276,('PDT000293506.1':0.191751,('PDT000713803.1':0.157331,('PDT000150308.2':0.1143,('PDT000359702.1':0.0409708,('PDT000639509.1':0.0439703,'PDT000720178.1':0.0476291)'':0.0079123)'':0.0659285)'':0.0463993)'':0.00374504)'':-0.00067027)'':0.0010191,('PDT000550032.2':0.177534,(('PDT000395044.1':0.169372,'PDT000395188.1':0.153656)'':0.0108849,(('PDT000377015.1':0.168986,('PDT000228322.1':0.163576,('PDT000252699.1':0.148393,('PDT000307440.2':0.0366297,'PDT000251831.1':0.0292192)'':0.114231)'':0.00806996)'':0.00850856)'':-0.00523177,(('PDT000463700.1':0.0700398,(('PDT000386737.1':0.0356497,'PDT000637303.1':0.0500898)'':0.00804985,('PDT000714701.1':0.0522484,'PDT000345934.1':0.044632)'':0.00177217)'':0.00883594)'':0.0155511,(('PDT000605057.2':0.0358227,('PDT000537183.2':0.024871,(('PDT000351891.2':0.0126969,'PDT000360416.2':0.0111503)'':0.0118429,('PDT000563752.2':0.0169011,('PDT000390237.2':0.0123308,'PDT000651811.2':0.0130619)'':0.00567189)'':0.00239059)'':0.00462076)'':0.00795538)'':-0.00422345,(('PDT000150684.3':0.0207161,('PDT000510670.2':0.0190936,'PDT000137151.3':0.0137177)'':0.00441086)'':0.00652239,(('PDT000240622.3':0.0168689,('PDT000144032.3':0.00756096,'PDT000051834.3':0.00653894)'':0.00413316)'':0.00715023,('PDT000600195.2':0.0207622,('PDT000491896.2':0.0184556,('PDT000465580.2':0.0131255,('PDT000144008.3':0.0151222,'PDT000625231.2':0.0122981)'':0.00198178)'':0.000570902)'':0.0030317)'':0.00249013)'':0.00567475)'':0.00155722)'':0.0391026)'':0.0979807)'':0.00927355)'':0.00565373)'':-0.0108867)'':0.00523916)'':0.0022403)'':0.0166756)'':-0.0174006)'':0.0118006)'':-0.00773072)'':0.00677222)'':-0.0038935)'':0.0155446)'':0.0051361)'':0.000742823)'':0.00613522,(((('PDT000250791.3':0.116985,'PDT000395694.2':0.125554)'':0.080955,(('PDT000502052.1':0.178133,('PDT000607568.1':0.110818,'PDT000248774.1':0.126456)'':0.0572975)'':0.00572227,(('PDT000216182.2':0.0792389,'PDT000652437.1':0.0681301)'':0.101935,(('PDT000228346.1':0.174168,'PDT000008699.2':0.177728)'':0.00339281,('PDT000372170.1':0.180426,('PDT000443344.2':0.0474497,'PDT000240258.1':0.0499985)'':0.109307)'':0.0137941)'':0.0028673)'':0.00807492)'':0.00558031)'':0.00306058,(((('PDT000394815.1':0.0959812,('PDT000128857.2':0.0871103,'PDT000197018.1':0.0794627)'':0.0120708)'':0.0118947,('PDT000426024.1':0.115873,('PDT000252713.1':0.108948,(('PDT000619562.1':0.0725138,'PDT000243001.1':0.0710492)'':0.0179789,(('PDT000203075.4':0.0567062,('PDT000203113.2':0.0237823,'PDT000598054.1':0.0277056)'':0.0217976)'':0.0176917,('PDT000395139.1':0.0739298,('PDT000292819.2':0.0408178,'PDT000411076.1':0.0464908)'':0.0230909)'':-0.000161612)'':0.0375202)'':0.00199415)'':0.0104386)'':-0.00364315)'':0.0490956,(('PDT000355748.1':0.0112756,'PDT000251437.1':0.0107861)'':0.125836,(('PDT000360976.1':0.0965398,(('PDT000636238.1':0.0829802,'PDT000251723.1':0.0838238)'':0.00592278,('PDT000219012.1':0.0736753,'PDT000221497.2':0.0771897)'':0.00540372)'':0.00603746)'':0.0240235,(('PDT000206125.2':0.101762,'PDT000553433.1':0.0953224)'':0.0104128,(('PDT000495423.1':0.110288,('PDT000409821.1':0.0706892,('PDT000685895.1':0.0548388,('PDT000501934.1':0.0470383,'PDT000495417.1':0.0482274)'':0.00305384)'':0.0099601)'':0.0103937)'':0.00743931,(('PDT000017912.2':0.0833776,('PDT000129912.2':0.0321353,'PDT000619671.1':0.0274094)'':0.0337026)'':0.00790224,('PDT000132381.2':0.0822006,('PDT000509484.1':0.0785423,(('PDT000311267.2':0.0755345,'PDT000319040.1':0.0712765)'':0.00600177,('PDT000577060.1':0.074034,('PDT000429815.1':0.0598693,('PDT000556914.1':0.0450516,('PDT000228087.1':0.0338905,'PDT000252544.1':0.0394428)'':0.00553068)'':0.0109833)'':0.00528116)'':0.0027628)'':0.00739776)'':0.0048107)'':-0.00337256)'':0.00456227)'':0.00935239)'':0.00913606)'':0.0419636)'':0.00569661)'':0.0342129,((('PDT000719291.1':0.146897,('PDT000632596.1':0.0786475,'PDT000463672.1':0.0882385)'':0.0410123)'':0.040702,('PDT000480341.1':0.135981,(('PDT000468875.1':0.0609497,'PDT000468878.1':0.0551793)'':0.0298033,(('PDT000168826.1':0.0638717,'PDT000346617.1':0.0636343)'':0.0193073,('PDT000228063.1':0.062382,(('PDT000168833.1':0.0574493,'PDT000396758.1':0.0575057)'':0.0112677,('PDT000291255.1':0.0670137,'PDT000468850.1':0.0503763)'':0.00682903)'':0.00753459)'':0.00363876)'':0.00563885)'':0.0646449)'':0.0282643)'':0.00450138,((('PDT000275035.4':0.114963,(('PDT000397649.1':0.0518435,('PDT000004354.2':0.0282194,'PDT000395206.1':0.0243555)'':0.0371106)'':0.0172014,('PDT000088298.2':0.0633828,(('PDT000557189.1':0.0444429,'PDT000360985.1':0.0377133)'':0.00753932,('PDT000425478.1':0.0527694,'PDT000425487.1':0.0555826)'':0.00398066)'':0.00171688)'':0.0142832)'':0.0468581)'':-0.0180987,((('PDT000687445.1':0.0734634,'PDT000394930.1':0.0761156)'':0.0169892,('PDT000459426.1':0.0767084,('PDT000346612.1':0.0763418,('PDT000095783.2':0.0311287,'PDT000397954.1':0.026972)'':0.0381559)'':0.0124629)'':0.00985535)'':0.00138064,(('PDT000128833.2':0.0441759,'PDT000128841.2':0.0346924)'':0.0242466,('PDT000639523.1':0.0606836,('PDT000346983.1':0.0637382,('PDT000095695.2':0.0544034,('PDT000190741.2':0.0474916,('PDT000565946.1':0.0119478,'PDT000627590.1':0.0127704)'':0.0205331)'':0.0084498)'':0.00597975)'':0.00156384)'':-0.000620472)'':0.031842)'':0.0119257)'':0.0319778,((('PDT000319276.2':0.0260801,'PDT000327955.2':0.0212398)'':0.0306084,('PDT000679865.1':0.0450517,(('PDT000481472.2':0.00762863,'PDT000115482.3':0.00838487)'':0.0337305,(('PDT000326099.2':0.0262345,('PDT000689098.1':0.0250932,'PDT000130439.3':0.0284293)'':0.00152465)'':0.00669848,(('PDT000391620.2':0.0215728,'PDT000115405.3':0.0250532)'':0.00442293,(('PDT000441298.2':0.0141364,'PDT000601942.2':0.018144)'':0.000424956,('PDT000038913.1':0.0106658,('PDT000500632.2':0.0084766,'PDT000578367.2':0.007035)'':0.00504537)'':0.0046208)'':0.0092198)'':0.00611525)'':0.00778142)'':0.0089685)'':-0.00425692)'':-0.00673016,(('PDT000317215.2':0.0352329,(('PDT000148027.3':0.0200989,'PDT000193096.3':0.0190281)'':0.00594656,('PDT000278339.3':0.0281781,('PDT000317230.2':0.0169085,'PDT000132328.3':0.0169564)'':0.0077219)'':1.32367e-05)'':0.00984203)'':0.000417255,('PDT000426939.2':0.0372678,(('PDT000320910.2':0.0273988,('PDT000503123.2':0.0312748,('PDT000325687.2':0.0198247,'PDT000651390.2':0.0206161)'':0.00698937)'':0.000252482)'':0.00317638,(('PDT000332152.2':0.0318749,('PDT000549585.2':0.0185482,'PDT000565130.2':0.0159448)'':0.00843569)'':0.00214447,('PDT000377133.2':0.0319258,(('PDT000316108.2':0.0173984,('PDT000544249.2':0.0119229,'PDT000132404.3':0.0115053)'':0.00494849)'':0.00565257,(('PDT000377984.2':0.0221828,'PDT000444904.2':0.0231516)'':0.000628947,(('PDT000389699.2':0.0172645,('PDT000641414.2':0.0148523,'PDT000132375.3':0.011998)'':0.00295688)'':0.000570472,('PDT000115449.3':0.0197086,('PDT000324035.2':0.012139,('PDT000415211.2':0.0154405,('PDT000669595.1':0.0131282,('PDT000332141.2':0.00769598,'PDT000060842.3':0.00889942)'':0.00250528)'':0.00237812)'':0.000366239)'':0.00411567)'':-0.00103972)'':0.00177119)'':0.00393345)'':0.010891)'':-0.00526139)'':-0.000172718)'':0.0050395)'':-0.00302663)'':0.00904243)'':0.113085)'':0.0373597)'':0.0147399)'':0.00582822)'':0.0084442,((('PDT000286093.1':0.117414,('PDT000410426.1':0.103648,('PDT000652895.1':0.0380929,('PDT000653017.1':0.028839,'PDT000346126.1':0.0352347)'':0.011522)'':0.0620212)'':0.0237839)'':-0.000134518,(('PDT000275055.4':0.0762843,'PDT000672149.1':0.0903667)'':0.0417168,(('PDT000468829.1':0.0909649,('PDT000285328.1':0.0339157,'PDT000448193.1':0.0460061)'':0.0523952)'':0.0122211,('PDT000372697.1':0.0852604,('PDT000721940.1':0.082432,(('PDT000505385.1':0.0758398,('PDT000645145.1':0.0496897,'PDT000250246.1':0.0455713)'':0.0299387)'':-0.00168758,('PDT000468931.1':0.0740753,(('PDT000251395.1':0.063156,('PDT000652891.1':0.0502922,'PDT000358758.1':0.0456129)'':0.00976342)'':0.00124829,('PDT000128834.2':0.0512097,('PDT000289857.2':0.0570375,'PDT000652836.1':0.0524025)'':0.0034703)'':0.00652362)'':0.0106987)'':-0.000224665)'':0.00183377)'':0.00781004)'':0.017352)'':0.0143934)'':0.00887153)'':0.026275,(((('PDT000645492.2':0.0683568,'PDT000661434.1':0.0674512)'':0.00620654,('PDT000645494.2':0.0679775,('PDT000645508.2':0.0651016,'PDT000645510.2':0.0535364)'':0.0123695)'':0.0159481)'':0.0532026,(('PDT000426281.1':0.0990982,(('PDT000228104.1':0.02856,'PDT000228109.1':0.0284212)'':0.0302175,('PDT000128866.2':0.0653659,('PDT000652978.1':0.0557972,('PDT000652945.1':0.04395,'PDT000346136.1':0.0423988)'':0.0101124)'':0.0092948)'':0.00549172)'':0.0182347)'':0.0116931,(('PDT000397703.1':0.0930495,('PDT000251715.1':0.0960975,('PDT000399683.1':0.0985675,'PDT000157307.1':0.0866175)'':0.0078525)'':0.00319096)'':0.00019367,(('PDT000363561.1':0.0706157,('PDT000583953.1':0.042094,('PDT000583440.1':0.036127,'PDT000482834.1':0.0328318)'':0.013416)'':0.0127287)'':0.00774426,((('PDT000004351.2':0.0352031,('PDT000024561.2':0.00704456,'PDT000644867.1':0.00737184)'':0.0357009)'':0.0317678,(('PDT000006901.2':0.0421883,'PDT000352454.1':0.0482326)'':0.00296657,('PDT000588019.1':0.0219514,('PDT000354241.1':0.00146471,'PDT000354262.1':0.00193325)'':0.0252749)'':0.0276543)'':0.0189691)'':0.00465612,(('PDT000433806.1':0.0982173,'PDT000414191.1':0.0877127)'':0.00820728,(((('PDT000636211.1':0.0255999,'PDT000652861.1':0.0321151)'':0.0437833,('PDT000718302.1':0.0642059,'PDT000346083.1':0.0529731)'':0.00864268)'':0.00097426,(('PDT000397582.1':0.0549758,'PDT000399710.1':0.0543162)'':0.0102905,('PDT000289925.1':0.0488317,'PDT000353626.1':0.0472965)'':0.0187107)'':0.00438586)'':0.00447701,((('PDT000718918.1':0.0218328,'PDT000248506.1':0.0201965)'':0.0451012,('PDT000565988.1':0.0682965,('PDT000018572.1':0.0657704,('PDT000652838.1':0.0460451,('PDT000521117.1':0.0201209,'PDT000081163.1':0.0224407)'':0.0258383)'':0.0186605)'':0.00509818)'':0.00235846)'':0.000738368,(('PDT000566106.1':0.0692785,(('PDT000424622.1':0.033676,'PDT000524497.1':0.0409362)'':0.016274,('PDT000397844.1':0.0537053,('PDT000027902.2':0.0369303,('PDT000196664.2':0.0451179,'PDT000645553.2':0.0402996)'':0.00659516)'':0.014198)'':0.00118186)'':0.0123287)'':1.436e-05,('PDT000394980.1':0.0748283,(('PDT000136136.2':0.0657576,('PDT000309357.2':0.0523379,('PDT000706047.1':0.00943819,'PDT000395356.1':0.00976391)'':0.0416805)'':0.0127704)'':-0.000283785,('PDT000025245.2':0.0568932,('PDT000425759.1':0.0489873,('PDT000719128.1':0.0461802,('PDT000306649.2':0.0340337,'PDT000396823.1':0.0341415)'':0.00423918)'':0.0085658)'':0.00472074)'':0.00197301)'':0.00662456)'':-0.000958142)'':0.00448218)'':0.00330589)'':0.0140835)'':-0.00822207)'':0.00875892)'':0.01638)'':-0.00194775)'':0.0275102)'':-0.00275904,(('PDT000463669.1':0.100471,('PDT000251403.1':0.0711045,('PDT000556903.1':0.0661205,'PDT000251484.1':0.0575235)'':0.015017)'':0.021057)'':0.0734044,(((('PDT000290431.2':0.0695762,'PDT000251459.1':0.0602608)'':0.0551571,(('PDT000167699.2':0.0691033,'PDT000664971.1':0.0725967)'':0.0142244,(('PDT000357591.1':0.0680572,('PDT000167680.2':0.0457423,'PDT000352452.1':0.0527249)'':0.0138952)'':0.00994766,('PDT000251472.1':0.0619647,('PDT000357700.1':0.0401787,('PDT000249911.1':0.027259,'PDT000251419.1':0.0221639)'':0.00884815)'':0.0278647)'':0.0149178)'':0.00780194)'':0.0257022)'':0.00385,('PDT000228282.1':0.149746,(('PDT000132067.3':0.0506615,('PDT000047683.4':0.0399947,('PDT000332701.2':0.0332699,'PDT000384340.2':0.0342911)'':0.00985756)'':0.00874348)'':-0.000877261,('PDT000636800.2':0.0308261,(('PDT000559706.2':0.0181496,'PDT000600049.2':0.0211956)'':0.0109001,('PDT000616670.2':0.0234266,('PDT000098336.3':0.0285754,('PDT000168309.3':0.0195682,'PDT000334481.2':0.0208766)'':0.00694359)'':0.00297047)'':0.00288076)'':0.000337549)'':0.0177155)'':0.0803289)'':-0.00603732)'':0.00683836,((('PDT000494253.1':0.0457953,('PDT000425928.1':0.0450465,('PDT000354170.1':0.0310343,'PDT000424759.1':0.0298081)'':0.00499319)'':0.0121548)'':0.0807799,(('PDT000251726.1':0.0760375,(('PDT000396780.1':0.0519563,('PDT000228132.1':0.028849,'PDT000249878.1':0.0250032)'':0.0209895)'':0.00125552,('PDT000556889.1':0.0650693,('PDT000557000.1':0.00955165,'PDT000249970.1':0.00796075)'':0.0359825)'':-0.000506864)'':0.0280208)'':0.0202387,('PDT000619747.2':0.107133,(('PDT000354268.1':0.014267,('PDT000354280.1':0.00707387,(('PDT000354234.1':0.00394167,'PDT000354256.1':0.00393942)'':0.00169989,('PDT000354253.1':0.00335784,'PDT000354265.1':0.00442737)'':0.00243141)'':0.00243453)'':0.0112282)'':0.0556684,(('PDT000718852.1':0.0516963,('PDT000397639.1':0.0465023,('PDT000147827.2':0.0316868,'PDT000501989.1':0.0342483)'':0.00336838)'':0.00890731)'':0.00610195,(('PDT000183008.3':0.0422083,'PDT000652849.1':0.040876)'':0.00680163,('PDT000394824.1':0.0485304,'PDT000645675.2':0.0396593)'':0.00225232)'':0.0112838)'':0.012587)'':0.0380048)'':-0.0101125)'':0.0296848)'':-0.00546152,(('PDT000714111.1':0.118277,('PDT000459682.1':0.101055,('PDT000228232.1':0.0531371,('PDT000309208.2':0.0469627,('PDT000322484.1':0.027317,'PDT000482339.1':0.0333871)'':0.0112876)'':0.00936847)'':0.052733)'':0.0148026)'':-0.00377719,((('PDT000047750.3':0.088068,'PDT000424765.1':0.089286)'':0.00820316,(('PDT000357663.1':0.0846639,('PDT000390643.1':0.0512027,'PDT000663927.1':0.0657423)'':0.0214071)'':0.00537012,('PDT000352436.1':0.0917532,('PDT000661817.1':0.0624556,('PDT000128826.2':0.0633073,'PDT000467006.1':0.0589347)'':0.00502645)'':0.0303023)'':0.000964879)'':0.00510105)'':0.00345618,(('PDT000162046.3':0.0964658,(('PDT000316868.1':0.0403307,'PDT000183402.2':0.0417847)'':0.0357149,('PDT000398049.1':0.0642378,'PDT000345927.1':0.0887452)'':0.0134239)'':0.0110737)'':0.00936188,(('PDT000652855.1':0.0355761,'PDT000253656.1':0.0314128)'':0.0707928,('PDT000482830.1':0.117983,('PDT000426586.1':0.0930402,('PDT000394884.1':0.0505272,('PDT000394962.1':0.03388,('PDT000095755.2':0.026336,'PDT000565870.1':0.0223517)'':0.0102655)'':0.0222496)'':0.0445301)'':0.0144006)'':-0.00652261)'':0.0091087)'':0.000856831)'':0.00840128)'':0.00362859)'':0.00600796)'':0.0455879)'':-0.0483966)'':0.00970955)'':0.0718293)'':0.0105952)'':0.00625742)'':0.00229915,((('PDT000345987.1':0.224422,('PDT000228046.1':0.0860234,('PDT000303275.2':0.0431091,'PDT000714645.1':0.0408618)'':0.0444732)'':0.12979)'':0.000372664,(('PDT000190728.1':0.107341,('PDT000253447.2':0.0923534,(('PDT000095778.2':0.0255346,('PDT000252612.1':0.0249851,('PDT000221929.3':0.00687146,'PDT000469889.2':0.00517794)'':0.0183863)'':0.00215054)'':0.0463605,(('PDT000468197.2':0.0265645,'PDT000457518.2':0.0258858)'':0.00548719,('PDT000619585.2':0.0183238,('PDT000353317.1':0.017005,'PDT000502290.1':0.0130665)'':0.0024654)'':0.0133917)'':0.0410208)'':0.0178329)'':0.00945723)'':-0.00252584,((('PDT000242872.1':0.0814291,'PDT000345975.1':0.0785079)'':0.00244095,(('PDT000129812.2':0.039031,'PDT000266790.3':0.0365297)'':0.0251968,('PDT000479625.1':0.046441,('PDT000088279.2':0.0163922,'PDT000619964.1':0.0161458)'':0.0318224)'':0.0187892)'':0.013544)'':-0.00392553,(((('PDT000702550.1':0.0256473,('PDT000027925.1':0.0204253,('PDT000565121.2':0.0134602,('PDT000158015.3':0.00615097,'PDT000395668.2':0.00607443)'':0.00496228)'':0.00675829)'':0.00332471)'':0.0228197,(('PDT000052320.1':0.031976,'PDT000252588.1':0.0258728)'':0.00353572,('PDT000498133.1':0.0242438,('PDT000038875.1':0.0164058,'PDT000050573.2':0.0197924)'':0.00395082)'':0.00791666)'':0.0185524)'':0.0136592,(('PDT000238495.2':0.0397948,('PDT000523426.2':0.0384683,('PDT000679774.1':0.0295979,'PDT000108921.3':0.03145)'':0.00456081)'':0.0033464)'':0.000451967,(('PDT000081308.1':0.0264247,('PDT000375959.1':0.0217265,('PDT000227354.3':0.0198198,('PDT000150972.3':0.0147657,'PDT000367202.2':0.0142112)'':0.00498911)'':0.00487957)'':0.00344227)'':0.00331402,('PDT000081305.1':0.0492589,('PDT000328008.1':0.03703,(('PDT000156578.2':0.0212965,('PDT000494132.1':0.00671532,'PDT000638582.1':0.00802128)'':0.0145194)'':-0.00022152,('PDT000086166.3':0.0308413,(('PDT000158034.2':0.0136816,'PDT000238572.2':0.0133962)'':0.00114106,(('PDT000671739.1':0.013698,'PDT000102781.3':0.0113442)'':0.00102968,(('PDT000325514.1':0.00944435,'PDT000598424.2':0.0126167)'':0.000786459,('PDT000384020.1':0.00619559,('PDT000372765.2':0.00505658,'PDT000132458.3':0.00529782)'':0.00222261)'':0.00453162)'':0.00311157)'':0.00224221)'':0.00636544)'':-0.000738555)'':0.00925241)'':0.00124738)'':-0.00605039)'':0.00913911)'':0.0221016)'':0.00776807,((((('PDT000086312.3':0.0254832,(('PDT000192005.3':0.00726285,'PDT000683595.1':0.00965085)'':0.00789578,('PDT000309901.3':0.016223,('PDT000189674.3':0.00735529,'PDT000270040.3':0.00676561)'':0.00688735)'':0.00365193)'':0.00492964)'':0.00460137,((('PDT000110469.3':0.03265,'PDT000718652.1':0.0305907)'':0.00556909,(('PDT000108870.3':0.0136962,'PDT000108883.3':0.0195316)'':0.00961828,(('PDT000160146.3':0.0173777,'PDT000325191.2':0.021751)'':0.00250916,('PDT000161979.3':0.0204652,('PDT000123251.1':0.0126939,('PDT000586492.2':0.0115101,('PDT000161581.3':0.00797745,'PDT000238499.3':0.00762735)'':0.00321044)'':0.00211216)'':0.00333284)'':0.00165685)'':0.00312056)'':0.00480096)'':-0.00268194,('PDT000155142.3':0.0429197,('PDT000238505.3':0.0366136,('PDT000377055.2':0.025547,('PDT000238507.3':0.0219287,('PDT000238582.3':0.0156064,('PDT000470334.2':0.0153923,('PDT000238571.3':0.0131556,('PDT000410711.2':0.00728091,('PDT000592996.1':0.00777521,('PDT000238506.3':0.00745898,('PDT000238511.3':0.00802184,('PDT000238508.3':0.00565426,(('PDT000156718.3':0.00500999,'PDT000238513.3':0.00475291)'':0.000390037,('PDT000238588.3':0.00459274,('PDT000485735.2':0.00435241,'PDT000132258.3':0.00490777)'':0.000186699)'':0.00066805)'':0.000156834)'':0.00125985)'':0.000352914)'':0.000133304)'':0.0013106)'':0.00256135)'':0.000297376)'':0.00514833)'':0.000777081)'':0.00111639)'':0.00467569)'':0.00296202)'':-0.0017652)'':0.00187494)'':0.0077513,(('PDT000213957.3':0.0173673,'PDT000421977.2':0.0150064)'':0.0220182,(((('PDT000050184.3':0.0218652,'PDT000127746.3':0.0205876)'':0.00831723,('PDT000300581.1':0.0200718,(('PDT000107493.3':0.0184331,'PDT000593004.1':0.0213886)'':0.00125731,('PDT000105076.3':0.0148613,('PDT000342239.2':0.0129164,('PDT000065203.2':0.0120852,'PDT000359279.2':0.0142228)'':0.00103774)'':0.00219987)'':0.00367094)'':0.0036148)'':0.0028422)'':0.00455023,(('PDT000593063.1':0.0310527,'PDT000593068.1':0.0246821)'':0.0129164,('PDT000593040.1':0.0355157,(('PDT000593024.1':0.0315335,'PDT000593031.1':0.0316592)'':0.00268199,(('PDT000342324.2':0.0240728,'PDT000107200.3':0.0178748)'':0.00369033,('PDT000054736.3':0.0254079,('PDT000593010.1':0.0280285,('PDT000137338.3':0.0159908,(('PDT000208175.3':0.00531239,('PDT000271172.3':0.00239974,'PDT000273491.3':0.00269059)'':0.00165915)'':0.00760062,(('PDT000593013.1':0.00214779,'PDT000593025.1':0.00244394)'':0.00936706,('PDT000085634.3':0.0102065,('PDT000605054.1':0.0127028,('PDT000593008.1':0.00647085,('PDT000593014.1':0.00630761,'PDT000593027.1':0.00762269)'':0.00182965)'':0.00187604)'':0.00127695)'':-0.000440099)'':0.00111625)'':0.00442689)'':0.00561072)'':0.0011899)'':-0.00285907)'':0.00573421)'':0.00638161)'':0.00369976)'':-0.00677228)'':0.00258365,(('PDT000593075.1':0.0282855,(('PDT000264702.3':0.0104311,'PDT000641980.2':0.00973978)'':0.00709881,('PDT000120830.3':0.0166657,('PDT000410934.2':0.0103401,('PDT000193133.3':0.00741795,'PDT000409224.2':0.00757855)'':0.00324347)'':0.00359865)'':0.000136999)'':0.0118208)'':0.00266246,((('PDT000399889.2':0.0106667,('PDT000593001.1':0.00199957,'PDT000593002.1':0.00239428)'':0.00791389)'':0.00894927,('PDT000603498.2':0.0129193,('PDT000587167.2':0.0113508,'PDT000593082.1':0.0102934)'':0.00260039)'':0.00401355)'':0.00605841,(('PDT000247461.3':0.0462484,('PDT000211175.3':0.0170781,'PDT000225470.3':0.0200509)'':0.0138506)'':0.000250238,(('PDT000372712.2':0.0215305,'PDT000593000.1':0.0210143)'':0.00188123,(('PDT000593007.1':0.0306337,('PDT000086174.3':0.0139278,('PDT000201176.3':0.0138276,('PDT000498193.1':0.00984493,('PDT000315863.2':0.00497539,'PDT000710906.1':0.00410192)'':0.00645462)'':0.00261663)'':0.000821296)'':0.00871994)'':-0.00476787,('PDT000584179.2':0.0227354,(('PDT000224224.3':0.011523,('PDT000621248.2':0.0110701,('PDT000421063.2':0.00924693,('PDT000191697.3':0.00669516,'PDT000113707.3':0.00865114)'':0.00224817)'':0.00142749)'':0.00314779)'':0.00102643,('PDT000120020.3':0.0247463,('PDT000628001.2':0.0110722,('PDT000624854.2':0.0163365,('PDT000360485.2':0.00799505,('PDT000280711.3':0.00575685,('PDT000162988.4':0.00361277,'PDT000593019.1':0.00377393)'':0.00189595)'':0.00282676)'':0.00371774)'':0.00154034)'':0.00303305)'':-0.00228228)'':0.0058225)'':-0.00336191)'':0.00411512)'':0.00569258)'':-0.00248463)'':0.00582113)'':0.00139223)'':0.00502544)'':-0.00252477)'':0.0023203,(((('PDT000213920.3':0.0325777,'PDT000608817.2':0.0375754)'':0.00317547,('PDT000604315.2':0.0320543,('PDT000630477.2':0.0260223,'PDT000636720.2':0.0304111)'':0.00652362)'':0.000731279)'':0.00726591,(('PDT000593043.1':0.025621,'PDT000593065.1':0.0296848)'':0.0151196,(('PDT000020516.1':0.0235958,('PDT000314326.3':0.0165484,('PDT000191673.3':0.0192472,(('PDT000191696.3':0.0107822,'PDT000313393.3':0.0123788)'':0.00452658,('PDT000214055.3':0.00970098,'PDT000132575.3':0.0104659)'':0.00627247)'':0.00384775)'':0.00237296)'':0.00433447)'':-0.000364461,(('PDT000465868.2':0.011765,('PDT000293109.3':0.00929623,'PDT000102244.3':0.00816577)'':0.00254454)'':0.00856809,('PDT000351911.2':0.0232528,('PDT000173957.3':0.0221641,('PDT000190110.3':0.0195044,('PDT000209866.3':0.0123129,'PDT000414380.2':0.0132241)'':0.00558374)'':0.0018452)'':0.00146469)'':0.00128818)'':0.00690147)'':0.0140462)'':-0.00069762)'':0.00495918,(('PDT000480966.2':0.0515695,(('PDT000238501.3':0.0382516,(('PDT000238581.3':0.022971,'PDT000500636.2':0.0184859)'':0.00880563,('PDT000107664.2':0.0237133,'PDT000251652.1':0.0211931)'':0.0046924)'':0.00707621)'':0.00328609,(('PDT000617951.1':0.0351649,'PDT000101070.3':0.0377861)'':0.0021938,(('PDT000050451.3':0.0265202,('PDT000316787.1':0.0256649,('PDT000230062.3':0.0229988,'PDT000569666.2':0.0215308)'':0.00107355)'':0.00405315)'':0.00112244,('PDT000648857.2':0.0341794,('PDT000341784.2':0.0213394,('PDT000629888.2':0.0143831,('PDT000432448.2':0.00709228,'PDT000137697.3':0.00753172)'':0.00467623)'':0.00892533)'':0.0106794)'':-0.000371473)'':0.00630041)'':0.00140311)'':0.00980673)'':-0.00766327,(((('PDT000593041.1':0.0361611,'PDT000593045.1':0.0337838)'':0.00971774,(('PDT000574073.2':0.017391,'PDT000593059.1':0.014666)'':0.00514305,(('PDT000593072.1':0.010352,'PDT000593076.1':0.00994897)'':0.00634537,('PDT000107680.2':0.0210375,('PDT000050459.3':0.0113826,('PDT000215107.2':0.0061971,('PDT000191631.3':0.00730158,'PDT000593012.1':0.00853822)'':0.000267152)'':0.00259878)'':0.00788587)'':-0.00119127)'':0.0024447)'':0.0154135)'':-0.0100152,(('PDT000490351.2':0.0158052,'PDT000549286.2':0.0165803)'':0.00901885,(('PDT000247198.3':0.0220863,'PDT000547694.1':0.0215372)'':0.000432236,(('PDT000334266.2':0.0221261,('PDT000083201.3':0.0119307,'PDT000139208.3':0.0124933)'':0.00563607)'':0.00476794,('PDT000492425.1':0.0368285,('PDT000324589.2':0.0170142,('PDT000492432.1':0.0175037,(('PDT000492424.1':0.0146218,('PDT000492423.1':0.0119301,'PDT000492452.1':0.0142376)'':0.000468787)'':0.00130012,('PDT000136743.3':0.0130883,(('PDT000351912.2':0.0104088,'PDT000357216.2':0.0094207)'':0.000518387,('PDT000372748.2':0.0107447,('PDT000571014.2':0.00774596,('PDT000555991.2':0.00504709,('PDT000678432.1':0.00375914,('PDT000541603.2':0.00323106,'PDT000595796.2':0.00295533)'':0.00141887)'':0.00128402)'':0.00274314)'':0.000534047)'':0.000138457)'':0.0030158)'':0.000231951)'':0.0013227)'':0.00140971)'':0.00816903)'':-0.00497685)'':0.00279799)'':0.00446431)'':0.000574703)'':0.00612488,(((('PDT000324573.2':0.015899,'PDT000360424.2':0.0157859)'':0.00580593,('PDT000137649.3':0.0128264,('PDT000083182.3':0.0120802,('PDT000130565.3':0.00751028,('PDT000243714.3':0.00589508,'PDT000125011.3':0.00527472)'':0.00252257)'':0.00551752)'':0.00139103)'':0.00496408)'':0.00326077,(('PDT000350546.2':0.0263165,'PDT000110464.3':0.0227597)'':0.00446042,('PDT000540172.2':0.0205589,((('PDT000540199.2':0.0157047,'PDT000619163.2':0.0127168)'':0.00232792,('PDT000601928.2':0.00786734,'PDT000615213.2':0.00696446)'':0.00577988)'':0.0049166,(('PDT000365919.2':0.00629448,'PDT000573315.2':0.00553922)'':0.00465627,('PDT000381315.2':0.0141437,('PDT000554132.2':0.00676348,('PDT000386152.2':0.00330977,'PDT000273066.5':0.00397493)'':0.00163767)'':0.00500906)'':-0.000634343)'':0.00655036)'':0.00268948)'':0.00831617)'':-0.00577705)'':0.00809503,(('PDT000251842.1':0.0335509,(('PDT000593056.1':0.0181428,'PDT000497298.2':0.0192487)'':0.0068333,('PDT000248593.1':0.0262706,'PDT000252714.1':0.0233957)'':0.00133687)'':0.00471781)'':0.002021,(('PDT000052316.1':0.0285327,('PDT000213865.3':0.0257194,'PDT000502380.1':0.0258904)'':0.00200076)'':0.00664708,('PDT000492239.2':0.0314676,(('PDT000588392.2':0.0303208,'PDT000124993.3':0.0217216)'':0.00290558,(('PDT000505014.2':0.0168576,'PDT000580805.2':0.0177445)'':0.00650464,(('PDT000146056.2':0.0200243,'PDT000573086.2':0.0208121)'':0.00586156,(('PDT000357270.2':0.0157182,'PDT000540118.2':0.0157104)'':0.00173138,('PDT000040715.3':0.0264778,(('PDT000131032.3':0.0170376,('PDT000212531.3':0.0121374,'PDT000602886.2':0.0143981)'':0.00342591)'':0.00123762,('PDT000251655.1':0.0197957,('PDT000557004.1':0.013364,(('PDT000184448.3':0.0110382,'PDT000344820.2':0.0133038)'':0.00205384,('PDT000233647.3':0.0132925,('PDT000226820.3':0.0130794,('PDT000486015.2':0.00847803,('PDT000145405.3':0.00787442,'PDT000173742.3':0.00674368)'':0.00216937)'':0.000878794)'':0.00311791)'':0.00014188)'':0.00166074)'':0.000707732)'':-6.67511e-05)'':0.00686843)'':-0.00285708)'':0.00329273)'':0.00142987)'':0.00209841)'':0.00667015)'':-0.00297134)'':-0.000601475)'':0.00035437)'':0.000522331)'':0.0047793)'':0.000769257)'':0.000976969)'':0.000529209,(('PDT000238502.3':0.0495852,(('PDT000510824.2':0.03692,('PDT000215092.3':0.0280211,'PDT000150146.3':0.0354367)'':0.00548037)'':0.0028193,('PDT000498195.1':0.0321308,(('PDT000334329.2':0.0233435,'PDT000593039.1':0.0241406)'':0.00517776,('PDT000558470.2':0.0192522,'PDT000575019.2':0.0159286)'':0.0102831)'':0.00555845)'':0.00231344)'':0.00753726)'':0.00940638,(('PDT000275751.3':0.0494946,(('PDT000466056.2':0.0134119,'PDT000125556.3':0.0119808)'':0.00440734,('PDT000491536.2':0.0193059,'PDT000687877.1':0.0126893)'':0.00194201)'':0.0230519)'':0.00716964,(('PDT000285778.3':0.0362587,('PDT000190540.2':0.0352318,(('PDT000313023.2':0.0215815,'PDT000399927.2':0.0247676)'':0.00421083,('PDT000131933.2':0.0264915,(('PDT000281524.3':0.0189035,('PDT000221934.2':0.00613807,'PDT000588384.1':0.00494313)'':0.00988236)'':0.000127085,('PDT000336957.2':0.0258195,('PDT000360428.2':0.0185567,(('PDT000242803.3':0.0105564,(('PDT000250775.3':0.00357485,'PDT000553342.2':0.00362155)'':0.00262847,('PDT000537060.2':0.00393852,'PDT000134040.3':0.00463063)'':0.0027869)'':0.0032581)'':0.00446504,('PDT000679121.1':0.0146984,('PDT000195759.3':0.0102362,('PDT000124921.3':0.00769385,('PDT000386169.2':0.00622794,'PDT000718271.1':0.00865716)'':0.000441301)'':0.00546924)'':0.00198336)'':0.000432379)'':0.00198987)'':0.00418419)'':-0.000814103)'':0.00543232)'':0.000924159)'':0.0043223)'':0.000355748)'':0.00353367,(((('PDT000238509.3':0.0325497,('PDT000377074.2':0.0213129,'PDT000060804.3':0.0229618)'':0.00682931)'':0.00253366,('PDT000132394.3':0.0217183,('PDT000161577.3':0.013495,('PDT000108924.3':0.015794,('PDT000238494.3':0.00960099,'PDT000238565.3':0.0125089)'':0.0032014)'':0.00115521)'':0.00177673)'':0.0117701)'':-0.000196798,(('PDT000356221.2':0.0273611,('PDT000238491.3':0.0216594,('PDT000127712.3':0.00498784,('PDT000238560.3':0.00488683,'PDT000238579.3':0.00419048)'':0.000745657)'':0.0107624)'':0.00781975)'':0.000216887,(('PDT000060866.3':0.0279121,(('PDT000123257.1':0.0206039,'PDT000248613.1':0.0162565)'':0.00256069,('PDT000238503.3':0.0181407,('PDT000238498.3':0.0131232,'PDT000238580.3':0.0135198)'':0.00222306)'':0.00492164)'':0.003359)'':-0.00160837,('PDT000016353.2':0.0204824,(('PDT000576244.2':0.00940751,'PDT000691359.1':0.00988249)'':0.00880586,('PDT000320295.2':0.0145858,(('PDT000399719.2':0.009903,'PDT000030575.1':0.0101845)'':0.000879302,('PDT000107662.2':0.00777167,'PDT000038903.1':0.00694733)'':0.0038698)'':0.00302254)'':0.00145405)'':0.00416168)'':0.000280996)'':0.00232806)'':0.00303141)'':0.0058906,((('PDT000051171.1':0.0155351,'PDT000345833.1':0.0150923)'':0.0370982,(('PDT000593042.1':0.0229771,('PDT000593050.1':0.0208339,'PDT000593055.1':0.0208246)'':0.00225093)'':0.0017945,(('PDT000524330.2':0.0289681,('PDT000509772.2':0.0159142,'PDT000137683.3':0.01283)'':0.0102042)'':-0.000212393,(('PDT000548453.2':0.0228656,('PDT000593089.1':0.0182172,('PDT000592990.1':0.00759674,'PDT000592995.1':0.00632256)'':0.0110871)'':0.00398113)'':0.00301168,(('PDT000492427.1':0.0165761,(('PDT000492428.1':0.0162617,'PDT000492453.1':0.016209)'':9.74395e-05,('PDT000492429.1':0.0117261,'PDT000492454.1':0.0119349)'':0.00242904)'':0.00113731)'':0.00813047,(('PDT000147489.3':0.0265413,('PDT000592992.1':0.0133514,'PDT000593087.1':0.0119449)'':0.00754824)'':0.00256636,(('PDT000691304.1':0.019375,('PDT000296222.3':0.010076,(('PDT000222575.3':0.00302662,'PDT000264787.3':0.00305513)'':0.00190783,('PDT000593026.1':0.00467602,'PDT000691354.1':0.0047899)'':0.000938537)'':0.00584699)'':0.00828897)'':-0.00293155,(('PDT000557535.1':0.0147468,'PDT000592994.1':0.0168695)'':0.0052885,('PDT000303559.3':0.0171112,(('PDT000307870.3':0.0110606,'PDT000085620.3':0.0115787)'':0.00242508,('PDT000063162.3':0.0213972,(('PDT000150158.3':0.0121645,'PDT000105117.3':0.0118711)'':0.00115922,(('PDT000177794.1':0.00936207,'PDT000177797.1':0.0111472)'':0.00679843,((('PDT000707500.1':0.0076699,'PDT000107459.3':0.0082537)'':0.00152443,('PDT000605053.1':0.00738819,'PDT000248635.1':0.00701531)'':0.00216882)'':0.00113285,('PDT000492455.1':0.0137812,('PDT000086986.1':0.0115518,('PDT000576167.2':0.0123502,('PDT000195765.3':0.0086549,(('PDT000593018.1':0.00544268,'PDT000593020.1':0.00394981)'':0.00334984,(('PDT000050453.3':0.00581281,'PDT000592991.1':0.00655259)'':0.000119723,('PDT000110630.3':0.00571998,('PDT000190367.3':0.00318405,('PDT000044730.4':0.0036279,('PDT000051215.3':0.00290077,'PDT000063178.3':0.00278388)'':0.00055647)'':0.000193316)'':0.00120246)'':0.000874939)'':0.000659774)'':0.00169492)'':0.0016347)'':0.0015133)'':0.000542927)'':-0.00106984)'':0.00278586)'':0.000926879)'':0.0043463)'':-0.00295343)'':0.0040062)'':-0.00296551)'':-0.000898756)'':0.00347554)'':-0.000272817)'':0.00158204)'':0.00222855)'':0.00118711)'':0.022564)'':-0.0176468,((('PDT000372732.2':0.0336501,(('PDT000238574.3':0.0174343,'PDT000050455.3':0.0221677)'':0.00169464,('PDT000556574.2':0.0248477,(('PDT000313039.3':0.0110726,'PDT000532405.2':0.0113849)'':0.000355664,(('PDT000162274.3':0.00729355,'PDT000133836.3':0.00720065)'':0.000779164,('PDT000230797.3':0.00656179,('PDT000227385.3':0.00626459,'PDT000407583.2':0.00537371)'':0.000538306)'':0.00174733)'':0.00326415)'':0.0123693)'':-0.00340503)'':0.00623867)'':-0.00411551,(('PDT000329288.2':0.0251459,('PDT000146012.3':0.0154002,('PDT000263976.3':0.0117796,'PDT000624082.2':0.0093179)'':0.00469206)'':0.00909893)'':-0.00039769,('PDT000485698.2':0.0285,(('PDT000239621.3':0.0205339,('PDT000644240.2':0.0168403,('PDT000517782.2':0.015242,('PDT000579385.2':0.0119294,('PDT000150152.3':0.00950356,('PDT000547969.2':0.00473615,('PDT000092285.3':0.00574353,('PDT000289016.3':0.00324009,'PDT000410909.2':0.00354295)'':0.000940221)'':0.00107059)'':0.00190037)'':0.00318187)'':0.00300536)'':0.000814932)'':0.00157225)'':-0.00360033,('PDT000252510.1':0.0209145,(('PDT000389811.2':0.00936078,'PDT000569050.2':0.0106273)'':0.00116443,(('PDT000230823.3':0.00508468,'PDT000565076.2':0.00468114)'':0.00610739,('PDT000492434.1':0.0116175,(('PDT000492390.1':0.00300393,'PDT000492431.1':0.00278334)'':0.00780951,('PDT000579386.2':0.00449728,('PDT000136762.3':0.00447807,('PDT000221627.3':0.00331277,'PDT000351921.2':0.00407025)'':0.00135542)'':0.000644727)'':0.0031861)'':0.000468976)'':0.000821112)'':0.00107968)'':0.00363654)'':-0.00237282)'':0.00995112)'':-0.00300298)'':-0.000419346)'':0.00711051,(('PDT000399913.2':0.0282025,(('PDT000492421.1':0.02793,('PDT000152936.3':0.0201584,'PDT000126261.3':0.0218675)'':0.00295471)'':-0.000160305,('PDT000101111.3':0.0142804,('PDT000124769.3':0.0114884,'PDT000492388.1':0.0112884)'':0.00516241)'':0.00492819)'':0.0028221)'':0.0176316,((('PDT000355488.2':0.0327809,'PDT000110482.3':0.0342935)'':0.00227542,('PDT000492426.1':0.0395039,('PDT000502591.1':0.0196726,('PDT000114759.2':0.0165782,('PDT000336553.1':0.00947164,'PDT000072787.2':0.00870836)'':0.00741067)'':0.00411973)'':0.00931837)'':0.00146741)'':-0.000266967,((('PDT000132306.3':0.0227392,'PDT000592998.1':0.0195738)'':0.00569258,('PDT000266783.2':0.0208804,('PDT000592999.1':0.00959846,'PDT000593005.1':0.0131851)'':0.00883234)'':0.000311169)'':0.012468,((('PDT000149197.3':0.0278423,'PDT000137585.3':0.0282334)'':0.000856089,('PDT000553156.1':0.0332468,(('PDT000679759.1':0.00952113,('PDT000394294.2':0.00605048,'PDT000592997.1':0.00652562)'':0.00211702)'':0.018455,(('PDT000115015.2':0.0191614,('PDT000367773.1':0.0160987,'PDT000410903.1':0.0141235)'':0.00210568)'':0.00213532,(('PDT000592993.1':0.0105108,'PDT000593022.1':0.0103844)'':0.00379167,('PDT000081132.3':0.0120824,(('PDT000173958.3':0.00787691,'PDT000492389.1':0.00753369)'':0.00368795,('PDT000510817.2':0.0102393,('PDT000296178.3':0.010093,('PDT000065211.3':0.00566922,('PDT000417230.2':0.00561106,('PDT000127768.3':0.00429656,('PDT000102245.3':0.00352698,('PDT000149190.3':0.00323837,('PDT000176597.3':0.00240135,'PDT000532333.2':0.00228833)'':0.000551077)'':0.000837373)'':0.00131163)'':0.000832888)'':0.000489111)'':0.00259898)'':0.00221561)'':0.000133708)'':0.00188335)'':0.000320711)'':0.00607937)'':0.00429972)'':0.00654175)'':-0.00410499)'':-0.000793836,(('PDT000110446.3':0.0481304,(('PDT000593062.1':0.0284666,('PDT000050461.3':0.0192542,'PDT000085599.3':0.0184649)'':0.00928535)'':0.00261112,('PDT000137699.3':0.0241452,('PDT000598827.1':0.0272781,('PDT000564139.2':0.0220458,(('PDT000050457.3':0.0189377,'PDT000065196.3':0.01739)'':0.00390543,('PDT000158022.3':0.0192535,('PDT000388607.2':0.0175477,(('PDT000264699.3':0.0128892,'PDT000598420.2':0.015004)'':0.00288878,('PDT000388897.2':0.0107787,'PDT000050183.3':0.0110035)'':0.0062009)'':0.00160269)'':0.0025119)'':-0.000220867)'':0.00237097)'':0.00145404)'':0.00275367)'':-0.0003433)'':0.0176747)'':-0.0139827,(('PDT000040434.4':0.0242621,('PDT000115043.3':0.0234627,('PDT000313061.3':0.0181734,(('PDT000072444.3':0.00727855,('PDT000492398.1':0.00197392,'PDT000492406.1':0.00192063)'':0.00667297)'':0.00375851,(('PDT000221962.3':0.00948749,'PDT000237103.3':0.0112817)'':0.00182259,('PDT000336941.2':0.00665497,'PDT000492387.1':0.00776143)'':0.00490629)'':0.00154023)'':0.00486538)'':0.000748402)'':0.0011579)'':0.000459088,((('PDT000274048.3':0.0251736,'PDT000524425.2':0.0320903)'':0.00696564,('PDT000593058.1':0.0349968,('PDT000593064.1':0.0276598,(('PDT000593011.1':0.0187578,'PDT000605052.1':0.0144829)'':0.00607103,(('PDT000300593.1':0.0198484,('PDT000235270.3':0.0161134,('PDT000065364.1':0.0111783,('PDT000102906.3':0.0116377,'PDT000125563.3':0.0096917)'':0.00155266)'':0.00250283)'':0.00313169)'':5.30499e-05,('PDT000040699.3':0.0205064,(('PDT000235353.3':0.0031331,'PDT000084097.3':0.00334746)'':0.0147171,((('PDT000148548.3':0.0103845,'PDT000557765.2':0.00910828)'':0.00198823,('PDT000593069.1':0.0114493,(('PDT000107833.3':0.00238854,'PDT000125477.3':0.00220503)'':0.00554451,('PDT000077510.1':0.00774989,('PDT000636620.2':0.00489744,('PDT000252770.3':0.00420586,(('PDT000083156.3':0.00416314,'PDT000367055.2':0.00421007)'':0.000203853,('PDT000285824.3':0.00304371,'PDT000677626.1':0.00413404)'':0.000934297)'':0.000654185)'':0.000834782)'':0.00159366)'':0.000673348)'':0.00526732)'':0.000303228)'':-0.00100271,('PDT000498018.1':0.0145285,(('PDT000639476.2':0.0154602,(('PDT000242763.3':0.00515149,'PDT000096817.3':0.00509831)'':0.0040337,('PDT000492157.2':0.00898632,('PDT000156867.3':0.00730955,'PDT000383099.1':0.00701455)'':0.00107463)'':0.00134557)'':0.00256108)'':-0.000627676,(('PDT000248573.1':0.00541835,'PDT000248669.1':0.00395073)'':0.00563838,('PDT000163781.3':0.00911826,('PDT000492187.2':0.00598431,('PDT000050189.3':0.00277984,('PDT000146505.3':0.00248551,'PDT000683660.1':0.00210577)'':0.000708613)'':0.00263225)'':0.00257034)'':0.00288503)'':0.000567775)'':0.00362511)'':-0.00251564)'':0.00262102)'':0.00381241)'':-0.00328467)'':0.00375844)'':0.00601764)'':0.0103188)'':-0.00667676)'':-0.00963732,((('PDT000426893.2':0.0199427,'PDT000459063.2':0.0197911)'':0.00165753,(('PDT000565070.2':0.0223422,('PDT000375726.2':0.0144444,'PDT000593030.1':0.014947)'':0.00346887)'':0.00191957,(('PDT000412105.2':0.0122498,'PDT000718786.1':0.0121718)'':0.00681129,(('PDT000521254.2':0.0090449,('PDT000492049.2':0.00701791,'PDT000620962.2':0.00809899)'':0.0019514)'':0.00078882,('PDT000421042.2':0.0107074,('PDT000342383.2':0.010432,('PDT000124998.3':0.0121957,('PDT000465852.2':0.00721897,(('PDT000546163.2':0.00588443,('PDT000465766.2':0.00498835,('PDT000312071.3':0.00408452,'PDT000389726.2':0.00399125)'':0.00102595)'':0.00105687)'':0.000984108,('PDT000468337.2':0.00536087,('PDT000247193.3':0.00594366,('PDT000520373.2':0.00410823,'PDT000125999.3':0.00416035)'':0.0020073)'':0.000495584)'':0.000536785)'':0.00210582)'':0.000810186)'':0.00164251)'':0.000532494)'':-0.00100592)'':0.00540598)'':0.000146542)'':0.00602006)'':-0.00387126,(('PDT000267884.3':0.0212951,('PDT000399322.2':0.0120996,('PDT000390716.2':0.00939391,('PDT000130983.3':0.0074991,('PDT000593028.1':0.00500979,'PDT000492386.1':0.00573431)'':0.0031303)'':0.00303149)'':0.000938888)'':0.00612672)'':-0.00241304,((('PDT000081130.3':0.0104458,'PDT000107492.3':0.00952044)'':0.00150582,('PDT000219676.3':0.0104818,(('PDT000145353.3':0.0062859,'PDT000083960.3':0.0079486)'':0.00341799,(('PDT000340521.2':0.00385634,'PDT000127714.3':0.00371715)'':0.00393298,(('PDT000250786.3':0.00685427,'PDT000429522.2':0.00538213)'':0.00222701,('PDT000145998.3':0.00577378,'PDT000289026.3':0.00477662)'':0.00202694)'':9.82024e-05)'':0.000873122)'':0.0018947)'':0.00267755)'':-0.0015356,('PDT000598301.2':0.0219318,('PDT000598711.1':0.0173376,('PDT000492411.1':0.0113805,('PDT000072727.3':0.0107507,('PDT000499828.2':0.00720786,('PDT000492420.1':0.00968118,('PDT000492456.1':0.00985738,('PDT000492422.1':0.00910166,('PDT000492419.1':0.0058799,('PDT000492430.1':0.00597517,('PDT000077529.1':0.00412732,('PDT000492408.1':0.00316615,('PDT000492409.1':0.00320126,('PDT000492451.1':0.00274059,('PDT000492400.1':0.00285505,('PDT000492407.1':0.00285955,(('PDT000492418.1':0.00251352,('PDT000492433.1':0.00218105,('PDT000492404.1':0.00154419,'PDT000492442.1':0.00185106)'':0.000313157)'':0.000407249)'':-3.83089e-05,('PDT000492450.1':0.00250591,('PDT000492441.1':0.00222861,(('PDT000492444.1':0.00196056,('PDT000492448.1':0.001727,('PDT000492391.1':0.00162648,'PDT000492403.1':0.00166891)'':7.01475e-05)'':0.000345914)'':7.86085e-07,('PDT000492399.1':0.00185027,('PDT000492447.1':0.00145659,('PDT000492410.1':0.00161233,('PDT000492405.1':0.00131863,('PDT000492401.1':0.00135638,'PDT000492402.1':0.00114112)'':7.94305e-05)'':0.000191473)'':0.000149416)'':0.000481452)'':-0.000242574)'':0.000384389)'':0.000105482)'':-0.00016615)'':0.000464441)'':0.000135456)'':0.000234316)'':0.000350514)'':0.000126126)'':0.000382544)'':0.00105761)'':0.000333256)'':0.0017905)'':0.000559846)'':0.000159141)'':0.000966477)'':0.000101214)'':0.00169612)'':0.000566606)'':0.0128076)'':-0.0120004)'':0.00272776)'':0.00199974)'':0.00147007)'':0.00261643)'':0.00362836)'':0.000243319)'':0.00534075)'':-0.000426666)'':0.00397947)'':-0.00859772)'':0.00313034)'':0.002308)'':0.00181838)'':0.0132156)'':-0.00537707)'':-0.00520864)'':0.0282797)'':0.00630626)'':0.0212065)'':0.11493)'':0.00316711,((('PDT000502484.1':0.198807,('PDT000205454.1':0.185026,('PDT000129878.2':0.0749705,'PDT000357603.1':0.0683835)'':0.109011)'':0.0142017)'':-0.00038181,(('PDT000095753.2':0.0608689,('PDT000502391.1':0.0642219,('PDT000441065.2':0.0475385,('PDT000375773.2':0.0126214,'PDT000115044.3':0.00904835)'':0.0239881)'':0.0201743)'':0.00598649)'':0.00965271,('PDT000502575.1':0.0640192,('PDT000243026.1':0.0376327,('PDT000347481.1':0.0282022,('PDT000502262.1':0.0176344,('PDT000347419.1':0.0155221,'PDT000359066.3':0.0142457)'':0.00526314)'':0.00569815)'':0.00301285)'':0.0370112)'':0.00820879)'':0.128318)'':0.0309926,((('PDT000254357.2':0.0160465,'PDT000176399.2':0.0154317)'':0.0451973,(('PDT000319294.1':0.0155369,'PDT000339395.1':0.0213313)'':0.0342032,('PDT000625389.1':0.0233887,('PDT000147799.2':0.0148074,'PDT000333321.1':0.0150625)'':0.00875546)'':0.0311885)'':0.00653045)'':0.187428,(((('PDT000281741.2':0.0287793,('PDT000083510.1':0.0186855,'PDT000016348.2':0.0142739)'':0.0139685)'':0.0198502,(('PDT000009346.2':0.0346409,'PDT000354175.1':0.0392178)'':0.018104,(('PDT000016352.2':0.0267243,('PDT000278346.2':0.0249433,'PDT000083552.1':0.0186207)'':0.00493554)'':0.00747068,('PDT000131828.2':0.0251995,('PDT000083508.1':0.0206686,('PDT000083594.1':0.0179412,('PDT000443044.1':0.0132466,'PDT000082635.1':0.0136628)'':0.00343216)'':0.00317798)'':0.00977017)'':-0.00098983)'':0.0197522)'':0.00180048)'':0.0324885,('PDT000232035.3':0.0917953,((('PDT000622965.2':0.0332549,'PDT000008686.2':0.0281624)'':0.00442717,('PDT000248493.1':0.032036,('PDT000553204.1':0.0264224,'PDT000009343.2':0.0262964)'':0.00509332)'':0.0118122)'':0.0134842,(('PDT000265127.1':0.0524149,('PDT000156033.2':0.0173437,('PDT000651388.1':0.00741865,'PDT000687091.1':0.00738955)'':0.00976887)'':0.0326368)'':0.0059722,(('PDT000619803.1':0.0112898,'PDT000619808.1':0.01189)'':0.033552,(('PDT000357680.1':0.0358204,('PDT000548628.2':0.0246192,('PDT000009345.2':0.0140123,('PDT000007739.2':0.0218836,('PDT000190719.3':0.0114966,('PDT000016347.2':0.0103116,('PDT000009344.2':0.00987394,('PDT000719071.1':0.00655574,('PDT000253203.3':0.00573385,'PDT000270315.1':0.00531205)'':0.000598862)'':0.00179223)'':0.0019776)'':0.0035576)'':0.00173128)'':0.00138579)'':0.00362583)'':0.0139571)'':-0.00170296,((('PDT000365734.2':0.0335223,'PDT000559692.2':0.0214074)'':0.00554963,('PDT000253460.2':0.0315875,(('PDT000162753.1':0.0187733,('PDT000558682.2':0.0098127,('PDT000161901.3':0.00719735,'PDT000390576.2':0.00791195)'':0.00150965)'':0.0130233)'':5.90658e-05,('PDT000548625.2':0.0231518,('PDT000498131.1':0.0210268,(('PDT000468062.1':0.011366,('PDT000553206.1':0.00629696,'PDT000553208.1':0.00694214)'':0.00521616)'':0.00216185,('PDT000147608.2':0.0140398,('PDT000595044.2':0.0159321,('PDT000253518.3':0.0107702,('PDT000130970.1':0.0111642,('PDT000232034.3':0.00879012,('PDT000146559.3':0.00827235,'PDT000063255.3':0.00676005)'':0.00143133)'':0.000778163)'':0.00123733)'':0.00245708)'':0.0005606)'':0.000764052)'':0.00288116)'':0.00142253)'':0.00219836)'':0.0107097)'':-0.0034159)'':6.63897e-05,('PDT000593741.2':0.0319565,(('PDT000227502.3':0.0227139,'PDT000137553.3':0.0219176)'':0.000926705,(('PDT000355525.2':0.0217941,('PDT000005323.2':0.0118475,('PDT000063256.2':0.007075,'PDT000101107.3':0.0070642)'':0.00295723)'':0.00606696)'':-0.00176982,(('PDT000016350.2':0.0168583,('PDT000208727.1':0.00993516,('PDT000627365.1':0.0129721,('PDT000719287.1':0.00514749,('PDT000242777.3':0.00170174,'PDT000084191.3':0.00169351)'':0.00421009)'':0.00375478)'':0.000848591)'':0.00464658)'':-0.00259644,('PDT000063254.3':0.0164257,('PDT000296025.3':0.00842043,('PDT000173898.3':0.00327079,('PDT000121083.3':0.00191522,('PDT000362591.2':0.00159855,'PDT000562834.2':0.00129855)'':3.20902e-05)'':0.00121961)'':0.00714271)'':0.00271766)'':0.00146703)'':0.00393555)'':0.00450243)'':0.00435066)'':0.00323084)'':0.00696581)'':0.00516265)'':0.0116463)'':0.00158962)'':0.0374852)'':-0.0129021)'':0.0125814,(('PDT000191515.2':0.0477284,('PDT000008685.3':0.0252826,(('PDT000399403.1':0.0264867,('PDT000063299.2':0.017525,('PDT000416745.1':0.00855374,'PDT000480964.2':0.00990196)'':0.00752102)'':0.00699715)'':-0.00226694,(('PDT000283076.3':0.0119899,'PDT000009335.2':0.0140962)'':0.00517438,('PDT000242217.3':0.0236759,('PDT000208729.1':0.0236128,(('PDT000242236.3':0.0173796,('PDT000242192.3':0.00737226,('PDT000242188.3':0.00592962,'PDT000242191.3':0.00532988)'':0.00135094)'':0.00424114)'':0.003083,('PDT000083421.1':0.017006,('PDT000083408.1':0.0105886,('PDT000083422.1':0.0119565,('PDT000009743.2':0.00931404,('PDT000083418.1':0.0104765,(('PDT000110502.3':0.00525683,'PDT000051552.1':0.00590747)'':0.00241525,('PDT000127955.3':0.00584263,'PDT000083420.1':0.00462907)'':0.00101217)'':0.000999818)'':0.0011654)'':0.00225097)'':0.000639318)'':0.00242296)'':-2.81187e-05)'':0.00751642)'':0.00106554)'':-0.00256533)'':0.000760837)'':0.00862704)'':0.0121044)'':0.0161658,((('PDT000156784.2':0.0398291,('PDT000008683.2':0.0310164,(('PDT000282993.3':0.0259957,('PDT000010857.3':0.0181641,('PDT000424800.1':0.0132242,'PDT000625570.2':0.0119268)'':0.007267)'':0.00208165)'':-0.00254318,(('PDT000264790.3':0.0121656,'PDT000623894.2':0.0130694)'':0.00447688,(('PDT000594854.2':0.00693206,('PDT000375662.2':0.00483174,'PDT000010319.3':0.00493019)'':0.00130633)'':0.00239136,('PDT000120804.3':0.00741584,('PDT000668165.2':0.010944,(('PDT000672562.1':0.00561667,'PDT000112442.3':0.00433556)'':0.00128014,('PDT000625229.2':0.0104198,('PDT000678084.1':0.00540689,('PDT000562863.2':0.0053185,('PDT000645910.2':0.00455872,('PDT000047664.3':0.00319446,'PDT000329310.2':0.00269729)'':0.00101561)'':0.000846755)'':0.000380848)'':0.00176653)'':-0.000664379)'':0.00153914)'':0.00090417)'':-0.000517097)'':0.00430939)'':0.00604954)'':0.0171928)'':0.00112193)'':0.00473686,((('PDT000316598.1':0.0352652,(('PDT000314216.2':0.00874805,'PDT000356185.1':0.00899165)'':0.00446336,('PDT000110434.2':0.0132677,('PDT000378034.1':0.00921962,('PDT000441297.1':0.00878682,('PDT000156660.2':0.00517556,'PDT000601098.1':0.00528364)'':0.00270408)'':0.000501049)'':0.00365711)'':7.41886e-05)'':0.013092)'':0.00351295,(('PDT000344347.2':0.0147677,'PDT000421274.2':0.0168497)'':0.00597686,('PDT000158744.3':0.0196561,(('PDT000329866.2':0.00926848,('PDT000212523.3':0.00486686,('PDT000063305.3':0.0033914,'PDT000063314.3':0.00368807)'':0.00191245)'':0.00416357)'':0.00089681,('PDT000146280.3':0.0125346,('PDT000390224.2':0.012086,('PDT000471146.2':0.00959526,('PDT000392122.2':0.0062011,'PDT000487967.2':0.0060243)'':0.00276539)'':0.000274255)'':0.00187943)'':0.000237573)'':0.00572358)'':0.000258409)'':0.0117725)'':0.00286878,((('PDT000353615.2':0.0319966,('PDT000327229.2':0.0306301,'PDT000612024.2':0.0199675)'':0.00513524)'':0.0017716,('PDT000061171.3':0.0315782,('PDT000323647.2':0.0252071,('PDT000389362.2':0.0230131,('PDT000320884.2':0.0188529,'PDT000061008.3':0.0165061)'':0.00351533)'':0.0052437)'':0.00374838)'':0.00328755)'':0.00089283,(('PDT000668494.1':0.03229,('PDT000333340.2':0.0265992,('PDT000378578.2':0.028346,(('PDT000127956.2':0.0137243,('PDT000061180.3':0.0127139,'PDT000127958.2':0.0105188)'':0.00214297)'':0.0043275,(('PDT000158703.3':0.0138281,'PDT000333947.2':0.0149183)'':0.00219882,('PDT000160431.3':0.0156077,'PDT000582680.2':0.0150599)'':0.00243725)'':0.00184942)'':0.00500877)'':0.00153648)'':0.00117539)'':-0.00207958,((('PDT000549000.1':0.0156935,('PDT000127957.1':0.0116345,'PDT000061120.3':0.0124273)'':0.0027672)'':0.00388662,('PDT000376246.1':0.0189241,('PDT000242107.2':0.0117708,'PDT000617959.1':0.0182571)'':0.00488752)'':0.000183547)'':0.00597608,(('PDT000452774.2':0.0226321,'PDT000583952.2':0.0228135)'':0.00161077,(('PDT000325740.1':0.0216721,('PDT000559289.2':0.0148932,('PDT000546955.2':0.00955382,('PDT000378722.2':0.0128867,('PDT000326109.2':0.00732097,'PDT000582580.2':0.00992903)'':0.00073869)'':0.00236548)'':0.0023256)'':0.00519473)'':-0.00138401,((('PDT000241747.3':0.00886095,'PDT000519356.2':0.011252)'':0.000881189,('PDT000389360.2':0.0119569,('PDT000325153.2':0.00935021,('PDT000565116.2':0.00955653,('PDT000378602.2':0.00413328,'PDT000061122.3':0.00425665)'':0.00305956)'':0.00208893)'':0.00360481)'':-0.000202961)'':0.00166996,('PDT000155652.3':0.0239848,(('PDT000548999.2':0.0144302,'PDT000562125.2':0.0142652)'':0.00605555,(('PDT000242138.2':0.00946897,'PDT000378645.2':0.00928293)'':0.00315702,(('PDT000562127.2':0.00981759,('PDT000242355.3':0.00759913,'PDT000376177.2':0.00848347)'':0.00166106)'':1.83331e-05,('PDT000323309.2':0.00952546,('PDT000155905.3':0.00834218,('PDT000333945.2':0.00580541,'PDT000481583.2':0.00593589)'':0.000726867)'':0.00106333)'':0.000920003)'':0.00378987)'':0.00178459)'':0.00714303)'':-0.0057202)'':0.00126652)'':0.00838943)'':-0.00151633)'':0.0014068)'':0.00391354)'':0.00531932)'':0.011969)'':0.00858878,(((('PDT000491488.2':0.0217743,('PDT000412842.2':0.00710744,'PDT000418053.2':0.00759406)'':0.00917587)'':0.0156906,(('PDT000145346.3':0.0282352,'PDT000242361.3':0.0284347)'':0.00280622,((('PDT000254237.3':0.016947,'PDT000584113.2':0.0185426)'':0.0104286,(('PDT000375789.2':0.0185014,'PDT000060783.3':0.0151262)'':0.0023052,(('PDT000691229.1':0.0146564,'PDT000356188.1':0.0137993)'':0.00317743,('PDT000441299.2':0.0213613,('PDT000601753.2':0.0160457,('PDT000317784.2':0.00877708,'PDT000332826.2':0.0106404)'':0.00798828)'':0.000518109)'':0.00111044)'':0.00248653)'':0.00460888)'':0.000755769,(('PDT000061286.3':0.0244937,('PDT000642702.2':0.0194292,(('PDT000446022.2':0.0111624,'PDT000481460.2':0.010903)'':0.00226748,(('PDT000158715.3':0.00359445,'PDT000472557.2':0.00378268)'':0.00632182,('PDT000486269.2':0.00747866,('PDT000376198.2':0.00535349,'PDT000377104.2':0.00529221)'':0.00264694)'':0.00215562)'':0.00143487)'':0.00541787)'':0.00806157)'':-0.00650566,(('PDT000334428.2':0.0136262,'PDT000601089.2':0.0158639)'':0.0105766,(('PDT000161545.2':0.0190582,(('PDT000242373.3':0.0135489,'PDT000061132.3':0.0140781)'':0.00101881,('PDT000156922.3':0.0167877,(('PDT000241921.3':0.0124575,'PDT000409982.2':0.0113685)'':0.000946725,('PDT000327969.2':0.0115886,'PDT000128058.2':0.0101053)'':0.00193543)'':0.00189794)'':8.00288e-05)'':0.00380722)'':-0.00168082,(('PDT000443378.2':0.00782334,('PDT000343950.2':0.00629703,('PDT000242653.3':0.0076999,('PDT000110477.3':0.0041559,('PDT000233976.3':0.00206578,'PDT000284046.3':0.00232763)'':0.00241676)'':0.00139092)'':0.000556296)'':0.00214158)'':0.00127013,('PDT000333276.2':0.0184885,('PDT000434234.2':0.0114884,(('PDT000157869.3':0.00502151,'PDT000239040.3':0.00593689)'':0.00147667,('PDT000400782.2':0.00658417,('PDT000128059.2':0.00569048,('PDT000356186.1':0.00279052,'PDT000356187.1':0.00349064)'':0.00143954)'':0.00226686)'':2.53375e-05)'':0.00252808)'':0.00450971)'':-0.00131417)'':0.00243381)'':0.00516828)'':-0.00295125)'':0.00407659)'':0.00289893)'':0.0109173)'':0.00389076,((('PDT000061339.3':0.0239339,(('PDT000320875.2':0.0247352,'PDT000377016.2':0.0211736)'':0.00199772,('PDT000321612.2':0.0171655,('PDT000155982.3':0.022131,'PDT000242352.3':0.0195275)'':0.00108619)'':0.00268258)'':0.00346821)'':0.0117953,('PDT000239625.3':0.0392406,((('PDT000157859.3':0.0317288,'PDT000651761.2':0.0348733)'':0.00742519,((('PDT000348119.3':0.0190688,'PDT000642319.2':0.0201101)'':0.00657507,(('PDT000158880.3':0.0249037,'PDT000366987.2':0.0230221)'':0.00309652,('PDT000155806.3':0.0233236,('PDT000328061.2':0.0203849,('PDT000326135.2':0.0141415,'PDT000327354.2':0.0137705)'':0.00426028)'':0.00450733)'':0.000966918)'':0.00201878)'':0.00100813,('PDT000242257.3':0.0302507,((('PDT000151304.3':0.00692791,'PDT000448190.1':0.00830199)'':0.014966,(('PDT000583330.2':0.0138857,'PDT000637617.2':0.0115001)'':0.00793871,(('PDT000127959.2':0.0114814,('PDT000155512.3':0.0043833,'PDT000157943.3':0.0060957)'':0.00629446)'':0.00515572,('PDT000604999.2':0.0211001,('PDT000160536.3':0.0123551,'PDT000322735.2':0.0131314)'':0.0059705)'':0.000869448)'':0.0014568)'':0.00407248)'':-0.00067935,('PDT000061005.3':0.028883,(('PDT000415292.2':0.0201707,('PDT000502877.1':0.0145036,'PDT000147645.3':0.0169479)'':0.00365151)'':0.0014965,(('PDT000326072.2':0.00985634,'PDT000377151.2':0.0104908)'':0.00485025,(('PDT000327255.2':0.00769582,'PDT000343870.2':0.0103231)'':0.00208524,('PDT000414309.2':0.0139479,('PDT000242115.3':0.00646586,'PDT000446024.2':0.00656634)'':0.00476385)'':0.00250604)'':0.00405551)'':0.00292609)'':0.00565849)'':-0.00078855)'':0.00535241)'':-0.000736663)'':0.00959424)'':-0.00934746,('PDT000158754.3':0.037664,((('PDT000318820.2':0.0288683,('PDT000683386.1':0.0238508,('PDT000496793.2':0.00935061,('PDT000242068.3':0.00930665,'PDT000320189.2':0.00827905)'':0.00133779)'':0.00809397)'':0.00392737)'':0.00320221,(('PDT000156976.3':0.0262919,('PDT000232032.3':0.0153618,'PDT000702546.1':0.0161436)'':0.00637831)'':0.000952971,('PDT000359199.2':0.0240674,('PDT000576261.2':0.0242815,('PDT000060979.3':0.0168987,('PDT000439362.2':0.0112751,('PDT000060828.3':0.00887058,'PDT000061173.3':0.00695662)'':0.00304444)'':0.00442007)'':0.00558665)'':0.0046048)'':-0.000979821)'':0.00222316)'':0.000904209,((('PDT000155287.3':0.0177354,('PDT000649401.2':0.0147469,'PDT000668501.1':0.0137681)'':0.00262527)'':0.00874188,(('PDT000158929.3':0.0118409,'PDT000328014.2':0.0133991)'':0.00306046,('PDT000260763.3':0.0209112,('PDT000157082.3':0.0171703,(('PDT000327966.2':0.00931658,'PDT000157025.3':0.0115914)'':0.000374863,('PDT000061116.3':0.00777606,('PDT000409896.2':0.00693803,'PDT000061030.3':0.00737747)'':0.000180491)'':0.00190862)'':0.00418033)'':0.00157199)'':0.00128063)'':0.00318725)'':0.00129416,(('PDT000148080.3':0.014973,('PDT000319782.2':0.0106205,('PDT000242462.3':0.00604713,'PDT000383184.2':0.00599507)'':0.00205184)'':0.00421085)'':0.0074996,(('PDT000232039.3':0.0181851,'PDT000065119.3':0.0157333)'':0.00269637,('PDT000540019.2':0.0197998,('PDT000324172.2':0.0160551,('PDT000061048.3':0.0152564,('PDT000155350.3':0.0143409,('PDT000065573.3':0.0111804,('PDT000157091.3':0.00594169,('PDT000324201.2':0.00305891,'PDT000061057.3':0.00273414)'':0.00285404)'':0.00431153)'':0.00273826)'':0.00127246)'':0.00312485)'':0.00239298)'':0.000734982)'':-0.000125842)'':0.00295327)'':0.00377814)'':0.00711996)'':-0.00463475)'':0.00706891)'':0.000935398)'':-0.000726134,((((('PDT000158033.3':0.0175828,('PDT000234181.3':0.0107395,'PDT000672922.1':0.0124956)'':0.00689086)'':0.000326343,('PDT000160548.3':0.014751,('PDT000157758.3':0.0128733,'PDT000667890.2':0.00961199)'':0.00302308)'':0.00259657)'':0.00154429,(('PDT000061089.3':0.0339119,('PDT000387120.2':0.0189403,(('PDT000479103.2':0.0122302,'PDT000668496.1':0.0111663)'':0.00368955,('PDT000063302.3':0.00913705,(('PDT000110490.3':0.00754006,('PDT000668490.1':0.00558226,'PDT000110498.3':0.00457004)'':0.00198689)'':0.000761906,('PDT000586369.2':0.0100499,(('PDT000241982.3':0.00426487,'PDT000242144.3':0.00440047)'':0.00146998,('PDT000324673.2':0.00527092,'PDT000394267.2':0.00557698)'':0.000606572)'':0.00199608)'':-0.000482633)'':0.00241425)'':0.00236017)'':0.00670837)'':0.00344332)'':-0.00464868,('PDT000060915.3':0.0278248,(('PDT000241714.3':0.0170226,('PDT000241769.3':0.0155845,'PDT000241691.3':0.0117243)'':0.00285454)'':-0.000307152,('PDT000710988.1':0.0225335,('PDT000518574.2':0.0155019,('PDT000490370.2':0.00886229,('PDT000241965.3':0.0070983,(('PDT000157876.3':0.00394847,'PDT000060816.3':0.00412569)'':0.00222736,('PDT000394252.2':0.00597825,('PDT000409952.2':0.00455856,'PDT000628276.2':0.00440826)'':0.00118369)'':0.00133481)'':0.000735433)'':0.00144757)'':0.00243217)'':0.0048287)'':-0.00246621)'':0.0126771)'':-0.0107763)'':0.0041117)'':0.0109211,((('PDT000017945.3':0.0289823,'PDT000017947.3':0.0313314)'':0.00416154,((('PDT000158842.3':0.0193029,(('PDT000158005.3':0.0131164,'PDT000486041.2':0.0154494)'':0.00379067,('PDT000317219.2':0.0176626,'PDT000327953.2':0.0131918)'':0.00420838)'':0.000939904)'':0.00211402,(('PDT000628111.2':0.0198345,('PDT000010542.3':0.0152408,('PDT000151312.3':0.013288,'PDT000656401.2':0.0138993)'':0.00168423)'':0.00166973)'':0.00335926,('PDT000282974.3':0.0206216,('PDT000547349.2':0.0155315,('PDT000490282.2':0.0118172,('PDT000429510.1':0.0118172,('PDT000151299.3':0.0110363,'PDT000258728.3':0.0110473)'':0.000676249)'':0.00036475)'':0.00194307)'':0.00848587)'':-0.00215889)'':0.00211068)'':0.00116929,('PDT000160453.3':0.0369369,(('PDT000288422.3':0.0213997,('PDT000378616.2':0.0159983,('PDT000517890.2':0.010785,'PDT000061059.3':0.0100958)'':0.00306203)'':0.00696059)'':0.000745661,(('PDT000157244.3':0.0205612,('PDT000319901.1':0.0170145,('PDT000577035.2':0.0177416,('PDT000241910.3':0.0146607,'PDT000378712.2':0.0147743)'':0.000919131)'':0.00288704)'':0.00194034)'':0.000151561,(('PDT000333258.2':0.0174764,'PDT000634655.2':0.0172438)'':0.00548261,(('PDT000342679.2':0.0147305,('PDT000155662.3':0.0109874,'PDT000711428.1':0.0107905)'':0.00282918)'':0.00170388,(('PDT000127963.2':0.0111547,'PDT000061457.3':0.0121103)'':0.000423743,('PDT000491540.2':0.0111462,'PDT000127966.2':0.00878242)'':0.00234808)'':0.00242463)'':0.00378672)'':0.000473309)'':0.00166934)'':0.00777322)'':-0.00547106)'':0.0088845)'':-0.00409908,(('PDT000322051.2':0.0267303,('PDT000324056.2':0.0233556,'PDT000498676.2':0.0236952)'':0.00145982)'':0.0155658,((('PDT000465897.2':0.0333664,('PDT000421963.2':0.0299696,('PDT000588683.1':0.0330464,'PDT000544626.2':0.0324333)'':0.0021797)'':0.00321836)'':0.000701064,(('PDT000588685.2':0.0160708,'PDT000667894.2':0.0183338)'':0.00603863,(('PDT000148024.3':0.0175398,('PDT000600045.2':0.0142413,('PDT000160416.3':0.0148995,'PDT000319835.2':0.0145935)'':0.000743957)'':0.000880968)'':0.00429694,(('PDT000309611.3':0.0180804,'PDT000377934.2':0.0152628)'':0.00208687,('PDT000316522.2':0.0134271,'PDT000061068.3':0.0137253)'':0.00341563)'':0.0033975)'':0.000939217)'':0.00842649)'':-0.000171724,((('PDT000160394.3':0.0251582,'PDT000253479.3':0.0217168)'':0.00781713,(('PDT000157781.3':0.0233669,'PDT000548268.2':0.0229894)'':0.0034767,(('PDT000394732.2':0.0224227,('PDT000708831.1':0.00907287,('PDT000252745.3':0.00711958,'PDT000385888.2':0.00623902)'':0.00363643)'':0.00905135)'':-0.00252259,('PDT000559251.2':0.0196205,(('PDT000259341.2':0.0124991,('PDT000258476.3':0.0131823,'PDT000495032.2':0.0096987)'':0.00296976)'':0.00026797,(('PDT000244060.3':0.0100438,'PDT000110478.3':0.00873227)'':0.00313117,(('PDT000147767.3':0.0117649,('PDT000112840.3':0.0122492,('PDT000063298.3':0.00966676,('PDT000328137.2':0.00800192,('PDT000671974.1':0.005677,(('PDT000146282.3':0.00335078,'PDT000389777.2':0.00472016)'':0.00121963,('PDT000627227.2':0.00367338,'PDT000119931.3':0.00310831)'':0.000858936)'':0.0013925)'':0.00208357)'':0.001735)'':0.00144144)'':0.000474901)'':0.000229956,('PDT000532182.2':0.01675,(('PDT000309676.3':0.00653881,'PDT000429484.2':0.00658769)'':0.000549746,('PDT000604956.1':0.0101705,('PDT000468177.2':0.00931075,('PDT000429479.2':0.00666867,(('PDT000649703.2':0.00479725,('PDT000499154.2':0.00474097,('PDT000637098.2':0.00264928,'PDT000682960.1':0.00294233)'':0.00223661)'':0.000826554)'':3.79907e-05,('PDT000627088.2':0.00538727,('PDT000296208.3':0.00446629,('PDT000499255.2':0.00401038,('PDT000137984.3':0.00358745,('PDT000191643.3':0.00326205,('PDT000275608.3':0.00269813,'PDT000389712.2':0.00229289)'':0.00112205)'':0.000255684)'':0.000519439)'':0.000388686)'':0.000407013)'':0.000153598)'':0.00200919)'':0.000902216)'':0.0010451)'':-0.00105235)'':0.00432285)'':-0.00248885)'':0.00318888)'':0.000520259)'':0.00468035)'':-0.00436316)'':0.00711263)'':0.00258214)'':-0.00187493,((('PDT000267691.2':0.0283078,(('PDT000320898.2':0.0207565,('PDT000629675.2':0.0198369,('PDT000389724.2':0.0156541,('PDT000190539.3':0.0130729,'PDT000333937.2':0.010637)'':0.00133282)'':0.00203885)'':0.00085572)'':0.00229685,(('PDT000245505.3':0.0163008,('PDT000155176.3':0.0133607,(('PDT000155485.3':0.00931771,'PDT000377945.2':0.00921729)'':0.00103476,('PDT000158702.3':0.00700811,'PDT000242064.3':0.00760279)'':0.00335279)'':0.00164877)'':0.00315075)'':0.00159699,('PDT000316482.2':0.0160921,(('PDT000160357.3':0.007225,'PDT000378615.2':0.0064286)'':0.00267631,('PDT000148088.3':0.0101906,('PDT000155444.3':0.00541497,'PDT000158860.3':0.00533023)'':0.00405267)'':0.00126333)'':0.00608905)'':-0.000873607)'':0.0018016)'':0.00906705)'':-0.00480798,((('PDT000112571.3':0.0218154,('PDT000625633.2':0.0182896,('PDT000009338.2':0.017134,(('PDT000108553.2':0.0160575,('PDT000622610.1':0.0112767,('PDT000353617.1':0.00794966,'PDT000378708.1':0.00892714)'':0.00195711)'':0.00378699)'':-0.00134025,(('PDT000444741.2':0.0108547,'PDT000582576.2':0.00999726)'':0.000474973,('PDT000065689.3':0.00845564,'PDT000084340.1':0.00850196)'':0.0013381)'':0.00311372)'':0.0043527)'':0.00159388)'':0.000630911)'':-0.00112633,(('PDT000242429.3':0.0177812,'PDT000505170.2':0.0166675)'':0.00383775,(('PDT000158911.3':0.0104487,('PDT000628735.2':0.00868761,('PDT000060821.2':0.00476325,'PDT000398514.1':0.00509044)'':0.00210714)'':0.00394647)'':-0.000333358,('PDT000712689.1':0.0138469,('PDT000065641.3':0.00910715,(('PDT000155398.3':0.00455659,'PDT000160457.3':0.00361725)'':0.00157585,('PDT000377935.2':0.00291345,'PDT000687078.1':0.00327788)'':0.00206055)'':0.00339371)'':0.00366447)'':-0.000452318)'':0.00311205)'':0.00393629)'':0.00115291,(('PDT000555937.2':0.0256142,'PDT000640720.2':0.0295178)'':0.00840377,((('PDT000324704.2':0.0101992,'PDT000395691.2':0.0114708)'':0.00395737,('PDT000519377.2':0.0186467,('PDT000061204.3':0.0103669,('PDT000061104.3':0.00845542,('PDT000378542.2':0.00558036,'PDT000380331.2':0.00516374)'':0.00225383)'':0.00078099)'':0.00479966)'':-0.000837254)'':0.00638557,(('PDT000156815.3':0.0166181,'PDT000390954.2':0.0145389)'':0.00270305,(('PDT000332838.2':0.0161373,'PDT000061121.3':0.0136264)'':0.00465358,(('PDT000244070.3':0.016314,(('PDT000147750.3':0.0115489,'PDT000242379.3':0.00990523)'':0.00195582,('PDT000155166.3':0.00657642,'PDT000242394.3':0.00685508)'':0.00297636)'':0.00358192)'':-0.000897537,(('PDT000110489.3':0.0135842,('PDT000241914.3':0.0104939,('PDT000155759.3':0.00759968,('PDT000242074.3':0.00454212,'PDT000573933.2':0.00521495)'':0.00453249)'':0.00179553)'':0.00159036)'':0.00145008,(('PDT000523791.2':0.0120536,('PDT000316462.2':0.00805562,('PDT000325080.2':0.0063779,'PDT000332742.2':0.0061757)'':0.00175948)'':0.00206006)'':0.000817322,(('PDT000333288.2':0.007414,'PDT000127965.2':0.0075084)'':0.00211684,('PDT000322936.2':0.00883309,('PDT000242301.3':0.0076692,('PDT000158887.3':0.00510249,('PDT000160543.3':0.00418992,'PDT000611946.2':0.00448492)'':0.000814302)'':0.00189429)'':0.00111371)'':0.00100356)'':0.000622032)'':0.0025067)'':0.0011546)'':0.00362037)'':0.000795733)'':0.00124706)'':0.00763164)'':-0.00841839)'':0.000714984)'':0.000621691,(('PDT000342674.2':0.0284897,('PDT000378056.2':0.0166445,'PDT000576849.2':0.0177735)'':0.00819811)'':0.00863198,(('PDT000325738.2':0.0156539,('PDT000326345.2':0.0196757,'PDT000421056.2':0.0174778)'':0.00440573)'':0.0104564,(((('PDT000127960.2':0.00942027,'PDT000083413.1':0.00999433)'':0.00688531,('PDT000518573.2':0.014437,('PDT000320172.2':0.0120358,('PDT000325789.2':0.0107218,'PDT000602558.2':0.0101776)'':0.00171965)'':0.00218664)'':0.00172651)'':0.00128087,(('PDT000526042.2':0.014308,('PDT000155235.3':0.0132895,('PDT000155505.3':0.00594397,'PDT000061203.3':0.00631073)'':0.00461776)'':0.00200938)'':0.00244688,('PDT000342272.2':0.0184089,('PDT000377197.2':0.0137123,('PDT000479288.2':0.00882754,'PDT000523435.2':0.00852326)'':0.00198748)'':0.00353874)'':0.00218815)'':0.00419958)'':0.00221679,('PDT000160351.3':0.032838,(('PDT000061025.3':0.0232444,('PDT000157733.3':0.0110123,('PDT000241943.3':0.0125008,'PDT000242146.3':0.0135707)'':0.000930244)'':0.00826101)'':0.00420831,((('PDT000317701.2':0.0159594,('PDT000270063.3':0.0127798,('PDT000241963.3':0.0124567,('PDT000147706.3':0.00880747,'PDT000127961.2':0.00826623)'':0.00249479)'':0.0026855)'':0.00183585)'':-0.000425468,('PDT000222593.3':0.0128238,('PDT000127962.2':0.0152237,('PDT000157767.3':0.0104269,('PDT000157829.3':0.00644066,'PDT000060824.3':0.00570594)'':0.0025089)'':0.00166119)'':0.00194337)'':0.00082337)'':0.0035734,(('PDT000017944.3':0.0170597,('PDT000017936.3':0.0107884,'PDT000017948.3':0.0131981)'':0.00365227)'':0.00401657,((('PDT000110510.2':0.0147995,'PDT000604958.1':0.013577)'':0.000777575,('PDT000377035.2':0.0170028,(('PDT000110520.2':0.00917775,'PDT000155389.2':0.0123585)'':0.00226202,(('PDT000626680.2':0.00885965,('PDT000324694.2':0.0084462,'PDT000378689.2':0.0069047)'':0.000651198)'':0.00240731,('PDT000490371.2':0.0131844,(('PDT000318847.2':0.00561669,'PDT000342266.2':0.00745981)'':0.000253303,('PDT000120851.3':0.00419692,('PDT000365938.2':0.00440363,'PDT000063311.3':0.00377347)'':0.000388445)'':0.00191835)'':0.0046934)'':-0.00129729)'':0.00251473)'':0.00234261)'':0.000520242)'':0.000247387,((('PDT000578775.2':0.00797637,('PDT000485864.2':0.00688364,'PDT000083406.1':0.00852696)'':0.00183408)'':0.00438586,('PDT000108919.2':0.0128325,('PDT000061015.3':0.0119758,('PDT000649406.2':0.00869665,('PDT000157178.3':0.00303502,'PDT000278464.3':0.00305341)'':0.00497669)'':0.00296283)'':0.00236838)'':-0.00135828)'':0.000695213,('PDT000239061.3':0.0208898,(('PDT000242461.3':0.0181258,('PDT000486533.2':0.013753,('PDT000520388.2':0.0068431,('PDT000542734.2':0.00405023,'PDT000600084.2':0.00402232)'':0.00192272)'':0.00656873)'':0.00165314)'':-0.00039658,(('PDT000322922.2':0.0173481,(('PDT000389358.2':0.0111917,('PDT000398045.2':0.00597704,'PDT000476695.2':0.00695316)'':0.0037485)'':5.48575e-05,('PDT000641387.2':0.0103581,('PDT000239080.3':0.0073575,('PDT000324715.2':0.00525548,'PDT000416742.2':0.00532122)'':0.00216695)'':0.00341672)'':-0.000663361)'':0.00426654)'':-0.000839381,(('PDT000160519.2':0.0102882,(('PDT000239066.2':0.00764664,'PDT000244068.2':0.0101714)'':0.00168401,('PDT000358751.1':0.00755566,'PDT000395700.1':0.00739054)'':0.00149564)'':0.000311732)'':0.00134434,(('PDT000061107.3':0.0157272,(('PDT000241926.3':0.00812725,('PDT000155498.3':0.0071699,('PDT000245461.3':0.00360762,('PDT000160517.3':0.00267574,'PDT000324053.2':0.00360918)'':0.000530754)'':0.00228602)'':0.00174117)'':-0.000289609,('PDT000060830.3':0.0082361,(('PDT000136967.3':0.00677635,'PDT000242271.3':0.00743135)'':0.000195799,('PDT000168700.1':0.00496675,('PDT000158916.3':0.00420712,('PDT000156047.3':0.00309356,('PDT000061376.3':0.00273553,'PDT000065659.3':0.00255495)'':0.000494199)'':0.00123742)'':0.000905332)'':0.00111583)'':0.00312447)'':-0.00182151)'':0.00382375)'':-0.00280854,('PDT000039652.1':0.0183549,(('PDT000324718.2':0.00664504,'PDT000326277.2':0.00880856)'':0.00150787,(('PDT000576257.2':0.00811427,'PDT000595108.1':0.00957163)'':0.000160274,(('PDT000378027.2':0.00753097,(('PDT000337249.2':0.00453098,'PDT000600051.2':0.00482971)'':0.000599537,('PDT000155330.3':0.00375812,('PDT000160480.3':0.00290379,'PDT000241908.3':0.00327767)'':0.000378609)'':0.00110315)'':0.00158135)'':-0.000750803,('PDT000061188.3':0.00568756,(('PDT000084341.1':0.00593403,('PDT000127964.2':0.000810019,'PDT000083410.1':0.00108846)'':0.00332167)'':0.000484003,('PDT000322052.2':0.00369959,('PDT000061190.3':0.00324021,'PDT000065559.3':0.00304283)'':0.000586563)'':0.000838516)'':0.00133927)'':-0.000389715)'':0.00159626)'':0.00109437)'':0.0039322)'':-0.00378743)'':0.00327512)'':0.000497606)'':0.00160313)'':0.00432364)'':-0.00524727)'':0.000925936)'':0.00598346)'':-0.00522555)'':0.00754794)'':0.00351886)'':-0.00682916)'':0.0052144)'':0.0019389)'':-0.0080358)'':0.00155686)'':0.00460147)'':0.00851153)'':-0.00993766)'':0.00197089)'':-0.00159086,(((('PDT000151359.3':0.0244928,'PDT000112549.3':0.0221719)'':0.00481668,(('PDT000136714.3':0.0292437,('PDT000157531.3':0.0157741,'PDT000385948.2':0.0122327)'':0.00788874)'':0.00783227,((('PDT000216227.3':0.0144321,'PDT000314482.4':0.0111197)'':0.0108228,('PDT000123496.3':0.0142423,('PDT000069978.3':0.0140661,'PDT000115251.3':0.0122133)'':0.00563804)'':0.00264673)'':0.000384079,('PDT000252185.1':0.0192869,('PDT000421017.2':0.0125391,('PDT000357212.2':0.0161512,('PDT000611519.2':0.0127247,('PDT000114513.3':0.00713331,('PDT000279451.3':0.00591983,'PDT000617923.2':0.00542567)'':0.00250239)'':0.00788684)'':0.000880443)'':0.00112434)'':0.00634927)'':-0.0008912)'':0.00534533)'':0.00409294)'':0.000128764,(('PDT000480415.2':0.0200852,('PDT000609734.2':0.0202244,(('PDT000275638.3':0.0176532,'PDT000247421.3':0.016497)'':0.00132608,('PDT000499229.2':0.0172416,('PDT000366102.2':0.0222933,('PDT000226902.3':0.0117358,('PDT000619329.2':0.0122431,('PDT000242645.3':0.00854628,('PDT000386109.2':0.00443044,'PDT000389806.2':0.00453728)'':0.00344376)'':0.00239281)'':0.00183926)'':0.00398262)'':0.00136439)'':0.00168338)'':0.00287792)'':0.00148193)'':-0.000421121,((('PDT000336975.2':0.0194591,'PDT000404067.2':0.0172627)'':0.00333774,(('PDT000293315.3':0.00852795,'PDT000386111.2':0.0127996)'':0.00277703,('PDT000559172.2':0.025529,('PDT000114555.3':0.0178787,((('PDT000250821.3':0.00671676,'PDT000312072.3':0.00493774)'':0.000775497,('PDT000650495.2':0.00752787,('PDT000355546.2':0.00492869,('PDT000629906.2':0.00424741,('PDT000525269.2':0.00327142,'PDT000627636.2':0.00361069)'':0.00163263)'':0.000831391)'':0.0005639)'':-3.35931e-05)'':0.00265118,('PDT000610912.2':0.00991504,('PDT000569501.2':0.00990538,(('PDT000359145.2':0.00670489,('PDT000312555.3':0.00577342,('PDT000143260.3':0.00420075,('PDT000227566.3':0.00369057,('PDT000547650.2':0.00310882,('PDT000385931.2':0.00223569,'PDT000421151.2':0.00235604)'':0.000679212)'':0.000465915)'':0.000531827)'':0.00141048)'':0.00135373)'':-4.11857e-05,(('PDT000313403.3':0.00543099,'PDT000409240.2':0.00541051)'':0.00119165,('PDT000222860.3':0.00764747,('PDT000386195.2':0.00573899,('PDT000252829.3':0.00359504,('PDT000216589.3':0.00382676,('PDT000718787.1':0.00330413,('PDT000355454.2':0.00285209,('PDT000400826.2':0.0027746,('PDT000233655.3':0.00299098,('PDT000357203.2':0.00215343,('PDT000281069.3':0.00229292,('PDT000238336.3':0.00160896,('PDT000351839.2':0.00162274,'PDT000125998.3':0.00157247)'':0.0001878)'':0.000372289)'':0.000242028)'':0.000137853)'':0.000449889)'':0.000139806)'':0.000903063)'':0.000219511)'':0.000437891)'':0.00242443)'':0.0012226)'':-0.00123154)'':-0.000671083)'':0.005917)'':0.000124821)'':-0.00358844)'':0.0100761)'':0.00110511)'':-0.00607282)'':0.00605401)'':-0.0060925,(('PDT000359808.1':0.0275187,('PDT000017940.3':0.017128,('PDT000139221.3':0.014217,('PDT000309581.3':0.0116306,'PDT000502942.2':0.0104355)'':0.00385423)'':0.00333291)'':0.00507766)'':-0.000250635,(('PDT000599500.1':0.0200839,'PDT000083415.1':0.0198298)'':0.0135977,((('PDT000231306.3':0.0116058,'PDT000496788.2':0.00918824)'':0.00453247,('PDT000285840.3':0.0166498,('PDT000399872.2':0.00849119,('PDT000221937.3':0.0086357,'PDT000532323.2':0.0088263)'':0.00203671)'':0.00559614)'':0.000406214)'':0.00151256,('PDT000009724.3':0.0224407,(('PDT000342391.2':0.00994038,('PDT000359174.2':0.0112651,('PDT000630446.2':0.00653362,('PDT000485348.2':0.0046503,('PDT000539862.2':0.00440369,'PDT000126229.3':0.00446262)'':0.000939975)'':0.00281296)'':0.00166592)'':0.000816345)'':-0.000852905,(('PDT000247151.3':0.0115027,('PDT000181347.3':0.0109291,('PDT000287871.3':0.00682895,'PDT000388914.2':0.00758605)'':0.00350998)'':0.00227325)'':0.000439982,(('PDT000232223.3':0.0107308,'PDT000375996.2':0.0106113)'':0.000210375,(('PDT000399254.2':0.00867289,'PDT000083326.3':0.0101181)'':0.00129945,(('PDT000273838.3':0.00487904,'PDT000072785.3':0.00518976)'':0.00327867,('PDT000389733.2':0.0141161,('PDT000452716.2':0.0131507,(('PDT000392389.2':0.00528309,('PDT000273513.3':0.00469525,('PDT000394752.2':0.00459663,('PDT000383992.2':0.00340772,'PDT000111543.3':0.00376931)'':0.00048319)'':0.000257011)'':0.000350626)'':0.00198298,('PDT000273517.3':0.00974377,('PDT000421036.2':0.00875114,('PDT000193136.3':0.0140597,(('PDT000344356.2':0.00698466,'PDT000574232.2':0.00744034)'':0.000727759,(('PDT000270090.3':0.00756162,'PDT000669289.1':0.00590878)'':0.00139421,(('PDT000190146.3':0.00576656,'PDT000625142.2':0.00577344)'':0.00092159,('PDT000281559.3':0.00766674,('PDT000702470.1':0.00947149,('PDT000603488.2':0.00606191,(('PDT000289027.3':0.00265281,'PDT000580802.2':0.0026345)'':0.00131173,('PDT000604423.2':0.0051837,(('PDT000190667.3':0.0046799,'PDT000598510.2':0.00350455)'':0.000601608,('PDT000376002.2':0.00408004,(('PDT000481400.2':0.00306855,('PDT000357158.2':0.00241257,'PDT000641977.2':0.00197821)'':0.000721484)'':-6.42552e-05,(('PDT000193132.3':0.00202383,'PDT000386112.2':0.0017701)'':0.00121723,(('PDT000399904.2':0.00224105,('PDT000557837.2':0.00205678,'PDT000646706.2':0.00173677)'':0.000755327)'':-9.45506e-05,('PDT000499115.1':0.00234653,('PDT000384111.2':0.00234754,('PDT000162966.4':0.00182153,('PDT000344353.2':0.00179202,'PDT000386170.2':0.00140351)'':0.000176557)'':0.000287182)'':8.18074e-05)'':0.000101577)'':0.00053088)'':0.000111106)'':0.00117539)'':-0.000219079)'':0.00120211)'':-0.000792394)'':0.00134137)'':0.000922189)'':0.00069725)'':-0.00114751)'':0.000386201)'':0.000830433)'':0.00229065)'':0.000328784)'':0.00014318)'':-0.00310615)'':0.00405916)'':0.00090418)'':-0.00271056)'':0.000374588)'':0.00214569)'':-0.000939317)'':-0.000611233)'':0.0104065)'':-0.0062462)'':0.0138982)'':-0.00516227)'':-0.00793911)'':0.00489972)'':0.00550593)'':0.000432856,(('PDT000502969.2':0.0292289,(('PDT000359849.1':0.0131359,('PDT000060982.3':0.0144932,'PDT000006236.2':0.0142902)'':0.000816325)'':0.0059556,((('PDT000151358.3':0.0074007,('PDT000151318.4':0.00697463,'PDT000051186.1':0.00694047)'':0.0024962)'':0.00443019,('PDT000557537.1':0.0177831,('PDT000193813.1':0.00944552,('PDT000006096.2':0.0112251,(('PDT000151311.3':0.0081061,('PDT000151286.5':0.00715104,'PDT000151356.3':0.00736906)'':0.00130825)'':0.00056236,('PDT000359860.1':0.00772576,('PDT000359848.1':0.0035657,('PDT000359833.1':0.00316066,('PDT000359815.1':0.00268332,'PDT000359840.1':0.00280242)'':0.000724996)'':0.000176938)'':0.00202176)'':0.00314519)'':0.00071837)'':0.00155572)'':0.00656296)'':-0.00480143)'':0.000531077,(('PDT000151301.3':0.0128002,'PDT000216168.3':0.0114495)'':0.00551673,(('PDT000009741.3':0.0114011,'PDT000359712.1':0.00953139)'':0.00355393,('PDT000151324.3':0.0205881,('PDT000009722.3':0.0147441,((('PDT000151357.3':0.0106693,'PDT000132563.2':0.0102591)'':0.00270718,('PDT000359811.1':0.00393543,'PDT000359858.1':0.00363882)'':0.00670342)'':0.000232627,(('PDT000191516.3':0.00899563,'PDT000112574.3':0.00838447)'':0.00403469,('PDT000081601.3':0.0114989,('PDT000112578.3':0.00822022,'PDT000141598.1':0.00819438)'':0.00233275)'':7.9842e-05)'':0.00165986)'':0.00288168)'':0.000900306)'':-0.00128271)'':0.00131915)'':0.000458683)'':0.00160383)'':0.0121568)'':0.000706055,((('PDT000326079.1':0.0222017,'PDT000160508.3':0.0278378)'':0.00566272,('PDT000155673.3':0.0151543,('PDT000421031.2':0.0100274,('PDT000649121.1':0.00753157,('PDT000189524.3':0.00823075,('PDT000480431.2':0.00546196,('PDT000155861.3':0.00455161,'PDT000313019.3':0.00452389)'':0.00144409)'':0.000802701)'':0.0014215)'':0.000710039)'':0.00134265)'':0.0101409)'':0.00711683,(('PDT000312525.3':0.0299261,('PDT000381318.2':0.0186214,'PDT000587497.2':0.0192924)'':0.00694436)'':0.00266377,((('PDT000690140.1':0.0205354,('PDT000264769.3':0.0129193,'PDT000421749.2':0.0136907)'':0.00219659)'':0.00225103,(('PDT000061144.3':0.0165193,('PDT000281557.3':0.00849406,('PDT000627615.1':0.00820155,'PDT000480439.2':0.00711575)'':0.00116719)'':0.00631167)'':-0.00039941,('PDT000112784.3':0.0138323,('PDT000216217.3':0.00971665,('PDT000356206.2':0.00358418,'PDT000357119.2':0.00630593)'':0.0056218)'':0.00147911)'':0.00406638)'':0.00178477)'':0.00504713,((('PDT000112639.8':0.0182047,'PDT000009729.3':0.018184)'':0.0133668,(((('PDT000010530.3':0.0119547,'PDT000010525.3':0.00959451)'':0.00715567,(('PDT000010325.3':0.00944691,('PDT000151302.3':0.00582074,'PDT000114451.3':0.00454296)'':0.00219089)'':0.00364167,('PDT000501757.2':0.00803193,('PDT000112631.8':0.00785977,('PDT000368339.2':0.00581032,'PDT000010329.3':0.00594738)'':0.00068758)'':0.00143812)'':0.00290631)'':0.00491139)'':0.000908117,('PDT000156664.3':0.033758,('PDT000495511.2':0.0227175,(('PDT000452719.2':0.00561906,('PDT000600782.2':0.00484712,'PDT000628110.2':0.00442044)'':0.00280201)'':0.00267838,('PDT000480901.2':0.00906662,('PDT000148036.3':0.00775573,('PDT000302815.3':0.00761798,'PDT000491479.2':0.00924322)'':0.000475519)'':0.00150696)'':0.000453612)'':0.0087619)'':0.00395163)'':-0.0031182)'':0.000557474,(('PDT000323935.2':0.0132296,'PDT000608810.2':0.0151638)'':0.000704841,((('PDT000618289.2':0.0132145,('PDT000373143.2':0.013036,'PDT000538657.2':0.00926588)'':0.000787089)'':0.00212014,(('PDT000266337.3':0.0103472,('PDT000338809.2':0.00397808,'PDT000539896.2':0.00399402)'':0.00419408)'':0.000785437,('PDT000386180.2':0.00836016,('PDT000623996.2':0.00533277,'PDT000644507.2':0.00613393)'':0.00340904)'':0.0016882)'':0.00334528)'':0.000523898,('PDT000151353.3':0.0116744,(('PDT000107856.3':0.0114781,('PDT000151323.5':0.0106169,('PDT000250226.1':0.00833528,('PDT000151350.3':0.00625488,('PDT000151321.3':0.00488149,'PDT000092282.3':0.00448199)'':0.000766026)'':0.00196388)'':0.00273903)'':0.00230478)'':-0.000679761,(('PDT000162343.3':0.00785684,'PDT000252809.3':0.00814876)'':0.0038632,(('PDT000502350.1':0.0103016,('PDT000359870.1':0.00852356,('PDT000112773.3':0.00551303,'PDT000119940.3':0.00562677)'':0.00367669)'':0.000826819)'':0.000276051,('PDT000497932.1':0.00985104,('PDT000177796.1':0.0112197,(('PDT000163288.4':0.0067476,'PDT000101847.3':0.0070736)'':0.000398047,('PDT000216156.3':0.00477464,('PDT000215962.1':0.00326634,'PDT000499122.1':0.00332312)'':0.00205838)'':0.0015029)'':0.00110759)'':0.00237426)'':-0.000930593)'':0.000820972)'':-0.000191649)'':0.00270126)'':-0.000250579)'':0.00228861)'':0.00159486)'':0.00842316)'':-0.00650193,((('PDT000064955.1':0.0220065,('PDT000282899.3':0.0144582,'PDT000622608.1':0.0152233)'':0.00690258)'':0.00036144,('PDT000160387.2':0.028738,((('PDT000155390.2':0.00753622,'PDT000156577.3':0.00805308)'':0.0115856,('PDT000060874.3':0.0187857,(('PDT000157068.3':0.00831572,'PDT000320816.2':0.00869918)'':0.00363384,('PDT000060781.3':0.0117958,'PDT000712681.1':0.009939)'':0.000928957)'':0.00360814)'':0.00205966)'':-0.000542557,((('PDT000191506.2':0.00940344,'PDT000351801.1':0.00897156)'':0.00175689,('PDT000573571.1':0.0118923,('PDT000622609.1':0.00788628,('PDT000622606.1':0.00287487,'PDT000457712.1':0.00461988)'':0.00317169)'':0.00348682)'':0.00021862)'':0.00262086,('PDT000009339.2':0.0178547,(('PDT000468287.2':0.0153756,(('PDT000241917.3':0.00958427,'PDT000060809.3':0.00923883)'':0.00169707,(('PDT000587574.2':0.00961754,('PDT000298391.3':0.00460006,('PDT000184473.3':0.00334438,'PDT000619294.2':0.0029487)'':0.00222032)'':0.00322577)'':0.000911553,('PDT000355457.2':0.0058681,('PDT000072778.3':0.00427201,('PDT000517895.2':0.00211351,'PDT000119939.3':0.00197994)'':0.00219808)'':0.00137808)'':0.00307244)'':0.0022367)'':0.00284208)'':-0.00248258,(('PDT000252757.3':0.0125867,(('PDT000538246.2':0.00511525,'PDT000141049.3':0.00435917)'':0.00262033,('PDT000184504.3':0.00357739,('PDT000360450.2':0.00281532,('PDT000625573.2':0.00182964,'PDT000092279.3':0.00166477)'':0.00137905)'':0.00101892)'':0.00331099)'':0.00642948)'':-0.00239275,('PDT000480367.2':0.0129161,((('PDT000112552.3':0.00511562,'PDT000134575.3':0.00493418)'':0.00167696,('PDT000151307.3':0.0073067,('PDT000384101.2':0.00283798,'PDT000112778.3':0.0027514)'':0.00245251)'':0.00051678)'':0.00211201,('PDT000010535.3':0.00931909,('PDT000359845.1':0.00493112,('PDT000250220.1':0.00371335,('PDT000112551.3':0.00286539,'PDT000119944.3':0.00312024)'':0.000376875)'':0.00167615)'':0.00154226)'':9.55052e-05)'':0.00654778)'':-0.00322841)'':0.00125266)'':0.0048695)'':-0.00308262)'':0.00296208)'':0.00893124)'':-0.00220222)'':-0.00703125,((('PDT000321913.2':0.0151919,('PDT000712483.1':0.0106227,('PDT000199191.3':0.0111748,('PDT000268816.3':0.00608705,'PDT000281082.3':0.00555355)'':0.00349455)'':0.00230686)'':0.00158181)'':0.000227108,('PDT000366062.2':0.0287568,(('PDT000598423.2':0.00983236,('PDT000069973.3':0.00846239,'PDT000618233.2':0.00675541)'':0.000573488)'':0.00650907,('PDT000157828.3':0.0194154,(('PDT000235345.3':0.0246132,'PDT000641592.2':0.0152274)'':0.000143751,('PDT000240666.3':0.0153965,(('PDT000499215.2':0.00843097,('PDT000540018.2':0.00817208,'PDT000097334.3':0.00881922)'':0.00011573)'':0.00164939,('PDT000577057.2':0.0113528,('PDT000640708.2':0.01744,(('PDT000325452.2':0.00599882,'PDT000127407.3':0.00695328)'':0.00371043,('PDT000009740.2':0.0147946,(('PDT000112582.3':0.00640019,'PDT000721979.1':0.00651711)'':0.00111187,((('PDT000480885.2':0.00445116,'PDT000482578.2':0.00432514)'':0.00369629,('PDT000558515.2':0.00533723,('PDT000359812.1':0.00531811,('PDT000524431.2':0.00497719,('PDT000270000.3':0.00322067,'PDT000417231.2':0.00296264)'':0.00189894)'':0.000580697)'':0.000567801)'':0.000770499)'':-0.000278315,('PDT000232776.3':0.0110075,('PDT000303461.3':0.0107395,('PDT000429051.2':0.00571058,('PDT000617892.2':0.00566295,(('PDT000662028.1':0.00438988,('PDT000509943.2':0.00475869,'PDT000682825.1':0.00440374)'':0.000486292)'':0.000358856,('PDT000171545.3':0.00703788,('PDT000189525.3':0.00389815,(('PDT000429543.2':0.00245691,'PDT000568644.2':0.00253211)'':0.000986375,('PDT000571466.2':0.00366584,('PDT000343453.2':0.00257129,('PDT000197032.3':0.00346572,('PDT000325461.2':0.00231627,'PDT000624157.2':0.00247558)'':0.000324191)'':0.000279386)'':0.000414709)'':0.000111322)'':0.000624112)'':0.000414162)'':-9.44468e-05)'':0.000555082)'':0.000973157)'':0.00145904)'':0.00122373)'':-0.00201795)'':0.00197611)'':0.00209341)'':-0.00216606)'':0.00403593)'':0.000401685)'':-0.00237293)'':0.00420472)'':-0.000202304)'':0.00310805)'':-0.00413916)'':0.00681332)'':-0.00429131)'':0.00133459,((('PDT000421225.2':0.0316364,'PDT000625569.2':0.0288545)'':0.0022909,(('PDT000148181.3':0.0158816,('PDT000623179.2':0.0143818,'PDT000656195.2':0.0101709)'':0.00402683)'':0.000302496,('PDT000693865.1':0.0157907,('PDT000112436.3':0.0102725,('PDT000225519.3':0.00779779,'PDT000537363.2':0.00768451)'':0.00199566)'':0.00476753)'':0.00117378)'':0.00900899)'':-0.00341669,(('PDT000324599.2':0.0182883,('PDT000344774.2':0.0184712,'PDT000709801.1':0.015981)'':0.0027784)'':0.00211106,('PDT000112588.3':0.0471063,((('PDT000112568.3':0.00659894,('PDT000238330.3':0.00417627,('PDT000217930.3':0.00344078,'PDT000668372.2':0.00284602)'':0.00180349)'':0.00153632)'':0.00193538,(('PDT000303080.3':0.00637791,'PDT000503117.2':0.00946349)'':0.00127255,('PDT000359242.2':0.00634832,('PDT000625572.2':0.00630793,('PDT000359827.1':0.0048266,('PDT000313031.3':0.00399543,(('PDT000340514.2':0.00231595,'PDT000654598.2':0.00247734)'':8.69865e-05,('PDT000270670.3':0.00214187,('PDT000226784.3':0.00218559,('PDT000342353.2':0.00155029,'PDT000471121.2':0.00184428)'':0.000408743)'':0.000184574)'':0.000208044)'':0.00219465)'':0.000340938)'':0.00211419)'':0.00042269)'':-2.26185e-05)'':-2.85837e-05)'':0.00152711,(('PDT000302827.3':0.0108465,('PDT000151300.3':0.00793738,('PDT000247445.3':0.00549635,('PDT000191519.3':0.00506488,'PDT000607757.2':0.00538702)'':0.000780884)'':0.00181462)'':0.00563325)'':-0.00114431,(('PDT000366341.2':0.00899435,('PDT000189619.3':0.00902599,('PDT000279490.3':0.0059688,('PDT000388904.2':0.00467031,('PDT000020179.1':0.00563341,(('PDT000151309.3':0.00288092,'PDT000151320.4':0.00290693)'':0.00083815,('PDT000162309.3':0.00327304,('PDT000230507.3':0.00217717,'PDT000490406.2':0.00201528)'':0.00131702)'':0.00014603)'':0.000854936)'':0.000751966)'':0.000408945)'':0.000792806)'':0.00154895)'':4.29948e-05,('PDT000404050.2':0.00889426,(('PDT000112585.3':0.00510148,('PDT000303509.3':0.00199021,'PDT000344788.2':0.00250574)'':0.00275984)'':0.00232419,(('PDT000239244.3':0.0049819,'PDT000119937.3':0.00498521)'':0.00156384,(('PDT000215961.1':0.00691329,('PDT000250092.1':0.00607563,('PDT000359800.1':0.00282418,'PDT000359869.1':0.00286047)'':0.00206159)'':0.00135935)'':-0.000420915,('PDT000107867.3':0.0082461,(('PDT000622604.1':0.00467906,'PDT000010327.3':0.00467976)'':0.000976141,('PDT000044149.5':0.0046495,('PDT000151310.3':0.00380047,('PDT000151322.3':0.00223537,'PDT000112587.3':0.00235866)'':0.0017794)'':0.00106444)'':-0.000111444)'':0.00112788)'':-0.000226683)'':0.00125088)'':0.00125188)'':0.00237872)'':-0.00189621)'':0.0011294)'':-0.000352122)'':0.0158917)'':-0.00748555)'':-0.000820939)'':-0.00267478)'':0.00308923)'':0.000339747)'':0.0046361)'':0.00943931)'':-0.0031365)'':-0.00368816)'':-0.0030825)'':0.00288961)'':0.0052085)'':0.00925679)'':-0.00315489,((((('PDT000359719.1':0.0207498,'PDT000359725.1':0.022536)'':0.01361,('PDT000359740.1':0.0273286,('PDT000359741.1':0.0227836,('PDT000359729.1':0.0190553,('PDT000359718.1':0.018287,(('PDT000151340.3':0.012922,'PDT000386056.2':0.0134484)'':0.00232589,('PDT000359723.1':0.0104919,('PDT000359721.1':0.00688214,'PDT000359724.1':0.00695836)'':0.00360782)'':0.00462397)'':0.0032926)'':0.00154461)'':0.00219003)'':0.00249603)'':0.0112396)'':-0.00932953,(('PDT000009746.2':0.0299407,('PDT000093300.3':0.02784,('PDT000594290.2':0.0258611,('PDT000010165.2':0.0295817,('PDT000065603.3':0.0256638,('PDT000323674.2':0.0199808,('PDT000421159.2':0.0222014,(('PDT000112557.3':0.0156977,'PDT000598675.1':0.0163378)'':0.00218999,(('PDT000124878.3':0.0209234,'PDT000359727.1':0.014276)'':0.000512361,('PDT000282953.3':0.0139999,('PDT000234162.3':0.0201534,('PDT000366999.2':0.0162238,(('PDT000191433.3':0.0106621,'PDT000010398.3':0.0115315)'':0.00252475,(('PDT000124797.3':0.00926951,('PDT000201671.3':0.00848683,'PDT000549848.2':0.00847077)'':0.00121674)'':0.00326387,('PDT000459080.2':0.0109361,(('PDT000470557.2':0.00579474,'PDT000107865.3':0.00776186)'':0.00239276,('PDT000227536.3':0.0073839,('PDT000227368.3':0.00768507,'PDT000307855.3':0.00730403)'':0.000461603)'':0.00194619)'':0.00289721)'':-0.000771784)'':0.00157582)'':0.00162659)'':0.00125727)'':0.000780217)'':-0.00024495)'':0.0003918)'':0.00435282)'':0.000637041)'':0.00125908)'':0.000554897)'':0.00093834)'':0.00195066)'':0.000418743)'':-0.00259764,((('PDT000359775.1':0.0171561,('PDT000151343.3':0.0120852,'PDT000598659.1':0.0128008)'':0.0115563)'':0.0064819,(('PDT000010847.3':0.0154976,'PDT000010860.2':0.0177003)'':0.00390972,(('PDT000086992.1':0.0182069,('PDT000112451.3':0.0162822,('PDT000252978.3':0.00641129,('PDT000359839.1':0.00569047,('PDT000010532.3':0.00473039,(('PDT000017925.3':0.00453135,'PDT000359786.1':0.00424583)'':0.000498709,('PDT000112780.3':0.00370154,('PDT000338864.2':0.00223769,'PDT000107859.3':0.00235175)'':0.00148794)'':0.00138406)'':0.000168375)'':0.00119042)'':0.000415984)'':0.00433082)'':0.00817713)'':-0.0079851,(('PDT000359803.1':0.0170753,('PDT000185678.3':0.00987039,'PDT000387721.2':0.0124311)'':0.00435437)'':0.00191334,('PDT000017963.3':0.0150174,('PDT000151373.3':0.00942805,('PDT000063322.3':0.00688004,('PDT000062973.3':0.0074721,(('PDT000252751.3':0.00486605,('PDT000671867.1':0.00489832,('PDT000107845.3':0.00319744,'PDT000107866.3':0.00379277)'':0.00110096)'':0.000174394)'':3.48416e-05,('PDT000156500.3':0.00473013,('PDT000151375.3':0.00351056,('PDT000171574.3':0.00360222,('PDT000112455.3':0.00275546,('PDT000191680.3':0.00276473,'PDT000107857.3':0.00272703)'':0.000235774)'':0.00083424)'':0.000483371)'':0.000576791)'':-5.50023e-05)'':0.00298377)'':0.000371677)'':0.00201618)'':0.00582878)'':-0.000494613)'':-0.00221958)'':0.010326)'':-0.000362904)'':-0.00192922,(((('PDT000627376.1':0.00761957,'PDT000136926.2':0.0102482)'':0.00632955,('PDT000359759.1':0.0149414,('PDT000626419.2':0.0107286,('PDT000691368.1':0.00713135,('PDT000065690.3':0.00706152,'PDT000010538.3':0.00629178)'':0.000583652)'':0.00235091)'':0.0013255)'':0.000432945)'':0.00282259,(('PDT000548523.2':0.0134723,('PDT000359867.1':0.0135195,('PDT000191437.3':0.00545993,(('PDT000154400.1':0.00247308,'PDT000359767.1':0.0028137)'':0.000904875,('PDT000359850.1':0.00250378,'PDT000359864.1':0.00258553)'':0.000748917)'':0.00172441)'':0.00460062)'':0.00659113)'':-0.00011502,(('PDT000039518.4':0.0160899,'PDT000136927.2':0.0161713)'':0.00211263,(('PDT000282954.3':0.0174106,('PDT000282989.3':0.0123288,(('PDT000155812.3':0.00962697,'PDT000065633.3':0.00920543)'':0.000770599,('PDT000282976.3':0.00823525,'PDT000412266.2':0.00776875)'':0.0018291)'':0.00318178)'':0.00369249)'':-0.00349615,(('PDT000010539.3':0.014522,'PDT000010864.3':0.0190086)'':0.00398296,((('PDT000326103.2':0.0110552,('PDT000314839.3':0.00616387,'PDT000557363.2':0.00686703)'':0.00702473)'':-0.000320267,(('PDT000412239.2':0.00457371,('PDT000112445.3':0.00337575,'PDT000112560.3':0.0032091)'':0.00144534)'':0.00406823,(('PDT000359787.1':0.005547,'PDT000359799.1':0.006795)'':0.0013498,('PDT000359841.1':0.00409459,'PDT000359847.1':0.0046686)'':0.00365798)'':0.00131305)'':0.00369641)'':-6.97189e-05,(('PDT000441066.2':0.00689798,('PDT000127880.3':0.0028436,'PDT000127920.3':0.00364472)'':0.00382531)'':0.01043,(('PDT000010330.3':0.0117447,('PDT000151370.3':0.00685071,'PDT000634350.2':0.00706169)'':0.00343248)'':0.00473765,(('PDT000281614.3':0.0123245,('PDT000359743.1':0.00814345,('PDT000649665.2':0.0103126,('PDT000359761.1':0.0081658,('PDT000250093.1':0.00613119,(('PDT000387601.2':0.00216329,('PDT000429509.2':0.00217657,'PDT000112575.3':0.00141876)'':0.000232153)'':0.000890694,('PDT000359765.1':0.00154659,('PDT000151647.1':0.00080877,'PDT000499524.1':0.00098979)'':0.000950464)'':0.00123251)'':0.00243837)'':0.000843373)'':0.00101405)'':0.000583616)'':0.00223377)'':-0.00186846,(('PDT000009330.2':0.00891931,('PDT000009333.2':0.00624019,('PDT000009331.2':0.0042154,'PDT000009332.2':0.0049516)'':0.00216526)'':0.00347772)'':0.000727333,((('PDT000233839.1':0.00605789,('PDT000375661.2':0.00446078,('PDT000319657.2':0.00251867,'PDT000690144.1':0.00307071)'':0.00111986)'':0.00253792)'':0.000712409,('PDT000025726.2':0.00588745,('PDT000359816.1':0.00560255,('PDT000359746.1':0.00300995,'PDT000359818.1':0.00337855)'':0.00195221)'':0.00132676)'':6.91195e-05)'':0.000755594,(('PDT000151303.3':0.0108788,'PDT000065600.3':0.0120801)'':0.000609744,('PDT000359739.1':0.0141769,('PDT000151338.3':0.0127904,('PDT000359749.1':0.0128825,(('PDT000249799.1':0.0123244,'PDT000250097.1':0.00842369)'':0.000647109,((('PDT000359844.1':0.00420815,'PDT000359863.1':0.00377588)'':0.0030292,(('PDT000595105.1':0.00873242,('PDT000249964.1':0.00436639,'PDT000497293.2':0.00390054)'':0.00140606)'':0.00107997,('PDT000004437.2':0.00922607,('PDT000010313.3':0.0061724,(('PDT000191513.3':0.00289425,'PDT000010397.3':0.00278983)'':0.000827386,('PDT000107678.2':0.00322614,'PDT000035833.1':0.00256055)'':0.00117288)'':0.00157852)'':0.00170387)'':-0.000544709)'':0.000288721)'':0.000604134,(('PDT000216277.3':0.00668907,'PDT000421055.2':0.00722463)'':0.00232713,('PDT000226742.3':0.0100862,(('PDT000222015.3':0.00666594,'PDT000295806.3':0.00747186)'':0.000971746,('PDT000359822.1':0.0123133,(('PDT000242108.3':0.00637039,('PDT000109777.3':0.0053983,'PDT000110492.3':0.0048688)'':0.000737662)'':0.000676548,((('PDT000359720.1':0.00323379,'PDT000359726.1':0.0034515)'':0.00233577,(('PDT000359856.1':0.00568106,('PDT000359846.1':0.00401666,('PDT000359819.1':0.00317796,('PDT000359825.1':0.0016078,'PDT000359854.1':0.00168824)'':0.00110964)'':0.000393955)'':0.00136408)'':-3.72668e-07,('PDT000359861.1':0.00807455,(('PDT000627618.2':0.00326127,'PDT000351834.2':0.00252658)'':0.00052226,('PDT000359868.1':0.00724969,('PDT000251377.1':0.00387569,('PDT000359758.1':0.00246574,(('PDT000359754.1':0.00124289,'PDT000359774.1':0.00125411)'':0.000241014,('PDT000270314.1':0.00148582,('PDT000253652.1':0.0015476,('PDT000499521.1':0.000726442,('PDT000499140.1':0.000660529,('PDT000499525.1':0.000534111,('PDT000510082.1':0.000301374,('PDT000499129.1':0.000159812,'PDT000499523.1':0.000240108)'':9.84664e-05)'':6.61973e-05)'':0.000206026)'':0.000329583)'':0.000203177)'':0.000332259)'':0.000261701)'':0.00056452)'':0.00111685)'':0.000929805)'':-0.000738227)'':0.00158465)'':-0.000493654)'':0.00163023)'':-0.000839746,('PDT000498984.2':0.0126756,(('PDT000151325.3':0.00375121,'PDT000242648.3':0.00412202)'':0.00209981,('PDT000151289.3':0.00616441,('PDT000320032.2':0.0074157,(('PDT000216243.3':0.0042773,'PDT000352095.2':0.00418884)'':0.000734531,('PDT000107854.3':0.00732345,(('PDT000359127.2':0.00710602,(('PDT000191524.3':0.00431004,'PDT000564143.2':0.00365888)'':0.000613169,('PDT000239291.3':0.00345267,('PDT000359857.1':0.00270893,('PDT000359766.1':0.00192172,'PDT000471411.1':0.00247432)'':0.000382424)'':0.000878409)'':0.00111135)'':0.000858704)'':3.93284e-05,('PDT000376000.2':0.00664115,(('PDT000412241.2':0.00292271,'PDT000494839.2':0.00325998)'':0.000501928,('PDT000151292.3':0.00380998,('PDT000359148.2':0.00397796,('PDT000279488.3':0.00417368,('PDT000151365.3':0.00300085,('PDT000216358.3':0.00232536,(('PDT000204917.3':0.00264253,('PDT000434257.2':0.00176921,'PDT000126277.3':0.00172485)'':0.000355448)'':0.000258034,('PDT000197029.3':0.00269423,('PDT000252749.3':0.00222626,('PDT000163043.4':0.00157672,'PDT000112558.3':0.00171834)'':0.000767464)'':0.000307518)'':-4.42571e-05)'':0.000177031)'':0.0004383)'':0.000238969)'':0.000235103)'':0.000352909)'':-0.000317993)'':0.00159435)'':-0.000637524)'':0.00196668)'':-0.000999301)'':0.00136036)'':0.000192204)'':-0.000787597)'':0.00164428)'':-0.00275729)'':0.00108764)'':0.0040298)'':-0.00259737)'':0.00145069)'':-0.00123329)'':-0.000806825)'':0.00192097)'':0.00318428)'':0.000383855)'':0.00145441)'':-0.00293569)'':-0.00314361)'':0.00170973)'':-0.00131717)'':0.00270954)'':0.00331295)'':-0.00422549)'':0.00834272)'':-0.00563697)'':0.0055601)'':-0.00417996)'':0.00165922)'':-0.00209443,((('PDT000298381.3':0.019947,('PDT000497943.1':0.0120686,('PDT000234903.3':0.010647,('PDT000359851.1':0.0097838,(('PDT000367200.2':0.00634432,'PDT000410882.2':0.00598168)'':0.00165741,('PDT000250094.1':0.00702828,'PDT000359842.1':0.00659412)'':0.000684188)'':0.00214683)'':0.00105366)'':0.00153094)'':0.00552625)'':-0.00278177,('PDT000386818.2':0.0230715,(('PDT000212356.3':0.0170229,'PDT000634349.2':0.015635)'':0.00463642,('PDT000568597.2':0.022676,(('PDT000145057.3':0.014773,'PDT000221613.3':0.0151116)'':0.00188136,(('PDT000389787.2':0.0131477,'PDT000638761.2':0.0110227)'':0.00457065,(('PDT000497880.1':0.0124275,'PDT000497923.1':0.0127577)'':0.00200091,('PDT000086141.3':0.0146087,('PDT000638772.2':0.0180759,(('PDT000367212.2':0.0118505,'PDT000108917.3':0.0101864)'':0.000830468,('PDT000238400.3':0.010944,(('PDT000222062.3':0.00623868,'PDT000407584.2':0.00678452)'':0.00257384,('PDT000359788.1':0.00950305,('PDT000616956.2':0.00985528,('PDT000599463.2':0.00836705,('PDT000242770.3':0.00494302,'PDT000385893.2':0.00531798)'':0.0017303)'':0.000955906)'':0.000969287)'':8.53692e-05)'':0.00126417)'':0.000350651)'':0.00381329)'':0.000804529)'':-0.000902234)'':0.000662165)'':0.000366371)'':0.00223117)'':-0.00149942)'':0.00536157)'':-0.00488973)'':0.00202994,(('PDT000253500.3':0.0189607,('PDT000497873.1':0.0189634,(('PDT000407606.2':0.00644651,'PDT000582863.2':0.00815569)'':0.00187578,((('PDT000360415.2':0.00384406,'PDT000628581.2':0.00372943)'':0.00165424,('PDT000443280.2':0.00471459,'PDT000617962.2':0.00475039)'':0.000777782)'':0.00177223,('PDT000227330.3':0.00247654,('PDT000071654.3':0.00181604,('PDT000242782.3':0.00127021,'PDT000315877.2':0.00152783)'':0.000579881)'':0.000391748)'':0.00357253)'':0.00242364)'':0.00472855)'':0.00416272)'':0.00233494,(('PDT000135347.3':0.0368318,(('PDT000524341.2':0.0134508,('PDT000226458.3':0.011257,('PDT000627799.2':0.0107222,'PDT000086338.3':0.00876485)'':0.00129335)'':0.000943487)'':-0.000621545,('PDT000471178.2':0.0120473,('PDT000707945.1':0.00853905,('PDT000458992.2':0.00522011,('PDT000213922.3':0.00572476,'PDT000532039.2':0.00452404)'':0.00145334)'':0.00180512)'':0.00131077)'':0.00176887)'':0.0104038)'':-0.00214704,(('PDT000499135.1':0.0222142,('PDT000359832.1':0.0189428,('PDT000151491.3':0.0162511,('PDT000253061.3':0.00846816,'PDT000618281.2':0.00888954)'':0.00682651)'':0.00362639)'':0.00381344)'':0.0017066,(('PDT000110514.3':0.0192052,'PDT000110527.3':0.020301)'':0.0106007,((('PDT000497879.1':0.0117213,('PDT000359737.1':0.00466432,'PDT000359738.1':0.00529485)'':0.0056799)'':0.00335668,('PDT000625632.2':0.0140813,(('PDT000086980.1':0.00937873,'PDT000215960.1':0.00928877)'':0.00186816,('PDT000356176.2':0.00821331,('PDT000279548.3':0.00697468,('PDT000579138.2':0.00491699,'PDT000627640.2':0.00523731)'':0.00207647)'':0.00212451)'':0.0016452)'':0.00515266)'':-0.00167232)'':0.00068993,((('PDT000025334.1':0.0159462,('PDT000010852.3':0.0101339,'PDT000017966.3':0.0118518)'':0.00522999)'':0.000349338,(('PDT000311059.3':0.00876111,'PDT000386102.2':0.0101256)'':0.00132504,('PDT000086991.1':0.0140468,(('PDT000497878.1':0.00662055,'PDT000497918.1':0.00758425)'':0.0022109,('PDT000383976.2':0.0126757,('PDT000107849.3':0.0103289,('PDT000384107.2':0.00767006,('PDT000623896.2':0.0101593,(('PDT000151382.3':0.00359414,'PDT000082283.3':0.00438671)'':0.00145384,('PDT000499218.2':0.00663297,('PDT000662056.1':0.00446438,('PDT000359116.2':0.00497482,(('PDT000317405.2':0.00317911,('PDT000376028.2':0.00212642,('PDT000344776.2':0.00165529,'PDT000404071.2':0.00174131)'':0.000766371)'':0.00098339)'':0.000525494,(('PDT000334243.2':0.00310445,'PDT000443415.2':0.00317984)'':0.000787496,('PDT000639468.2':0.00346365,('PDT000191649.3':0.0031087,('PDT000279482.3':0.00264598,'PDT000471123.2':0.00324166)'':0.000379516)'':0.000366788)'':-3.13568e-05)'':0.000317803)'':0.000583634)'':0.000803219)'':0.00166961)'':-0.000953808)'':0.00137918)'':0.0003662)'':0.000596068)'':0.00371083)'':-0.00262707)'':0.00377218)'':-0.00260802)'':0.00608798)'':-0.0051605,(('PDT000010326.3':0.0206917,('PDT000264713.3':0.0143864,'PDT000359866.1':0.0145062)'':0.00222466)'':0.00243159,(('PDT000204918.3':0.0137318,('PDT000107853.3':0.0134399,('PDT000151381.3':0.00763884,('PDT000412252.2':0.00515393,'PDT000250225.1':0.00601257)'':0.00273231)'':0.00227209)'':0.00166527)'':0.00493083,(('PDT000314331.3':0.0231231,(('PDT000044914.3':0.01016,'PDT000624086.2':0.0102096)'':0.0033536,('PDT000248857.1':0.015645,('PDT000191512.3':0.0107984,(('PDT000359770.1':0.00354858,'PDT000359776.1':0.00313338)'':0.00421432,('PDT000112420.3':0.00974513,('PDT000260793.3':0.00660213,('PDT000222596.3':0.00647146,('PDT000081710.3':0.00561693,('PDT000254248.3':0.00516199,('PDT000553323.2':0.00573668,('PDT000248860.1':0.00456128,'PDT000359763.1':0.0037073)'':0.000924156)'':0.000540508)'':0.000988599)'':0.000603479)'':0.000337841)'':0.000243248)'':-0.000427089)'':0.00242686)'':0.0034952)'':-0.000844282)'':0.00536632)'':-0.00664706,(('PDT000498674.2':0.0116786,('PDT000673663.1':0.0112263,('PDT000638945.2':0.00943617,('PDT000351807.2':0.00593477,('PDT000384113.2':0.00439189,('PDT000009731.3':0.00395079,('PDT000252759.3':0.00350224,('PDT000484042.2':0.00319375,('PDT000142588.3':0.00229862,'PDT000190092.3':0.00219285)'':0.00119326)'':0.000253315)'':0.000299895)'':0.000502438)'':0.00178765)'':0.00206496)'':0.00140173)'':0.00156668)'':0.001956,('PDT000063301.2':0.0231532,(('PDT000060954.3':0.0182107,('PDT000394054.2':0.0129807,('PDT000010537.3':0.0109953,('PDT000123491.3':0.0106543,('PDT000314487.3':0.00771173,('PDT000603490.2':0.0103081,(('PDT000321151.2':0.00490806,'PDT000359810.1':0.00504516)'':0.0012015,('PDT000201201.3':0.00584473,('PDT000076983.3':0.00403558,('PDT000293126.3':0.00391156,('PDT000407642.2':0.00280659,'PDT000537418.2':0.00297952)'':0.00157245)'':0.000667336)'':0.00049868)'':0.000440293)'':0.00123315)'':0.00139722)'':0.00037269)'':0.00102815)'':0.000804088)'':0.00455376)'':-0.00541841,(('PDT000250004.1':0.00864681,(('PDT000566612.2':0.00511293,'PDT000609736.2':0.00375249)'':0.00288961,('PDT000365694.2':0.00691767,('PDT000551256.2':0.00403147,('PDT000208185.3':0.00383256,'PDT000265134.3':0.00324268)'':0.000156178)'':0.00408434)'':-0.000480526)'':0.00327302)'':0.00102405,((('PDT000359764.1':0.00913616,('PDT000359790.1':0.00723953,(('PDT000359755.1':0.00309081,'PDT000359829.1':0.00348945)'':0.000868712,('PDT000359828.1':0.00401752,'PDT000359830.1':0.00395378)'':0.00042093)'':0.00278706)'':0.000503498)'':0.000120891,('PDT000604960.1':0.0114444,(('PDT000359871.1':0.0074284,'PDT000499134.1':0.007699)'':0.00151011,('PDT000010316.3':0.011085,('PDT000005522.2':0.0107195,(('PDT000010331.3':0.00463212,'PDT000249807.1':0.0043356)'':0.000773433,('PDT000435653.1':0.00587244,('PDT000598660.1':0.00518751,('PDT000359768.1':0.00384747,(('PDT000253650.1':0.00207614,'PDT000157303.1':0.00191667)'':0.000299712,('PDT000123374.1':0.00214221,'PDT000406653.1':0.00195369)'':0.00044944)'':0.00125841)'':0.000869946)'':0.000662872)'':0.000533952)'':0.00257879)'':0.000930911)'':-0.000984398)'':0.00240267)'':-0.00234418)'':0.000700934,(('PDT000253501.3':0.00913543,('PDT000151327.3':0.00863383,'PDT000468225.2':0.00933317)'':0.00118072)'':-8.35851e-05,('PDT000216373.3':0.0155439,('PDT000044151.3':0.0128439,(('PDT000412260.2':0.0075815,('PDT000161778.3':0.00650351,'PDT000191438.3':0.00632849)'':0.0028636)'':0.00141012,(('PDT000227530.3':0.0059604,'PDT000388888.2':0.0060722)'':0.00563757,('PDT000672460.1':0.0145545,('PDT000366254.2':0.011287,(('PDT000289822.3':0.00905227,'PDT000082653.3':0.00709443)'':0.000893185,('PDT000151376.3':0.00858061,(('PDT000141646.3':0.0090258,('PDT000124817.3':0.00685234,('PDT000625565.2':0.00743943,('PDT000191436.3':0.00599528,('PDT000407644.2':0.0050236,('PDT000384163.2':0.00451508,('PDT000583943.2':0.00476737,(('PDT000551255.2':0.00385618,('PDT000225309.3':0.00383605,'PDT000625600.2':0.00364077)'':0.000227193)'':0.000305184,(('PDT000313004.3':0.00285717,'PDT000107868.3':0.00302753)'':0.000852874,(('PDT000321921.2':0.00128477,'PDT000107956.3':0.00151159)'':0.00207328,('PDT000137550.3':0.00281344,('PDT000171517.3':0.00232431,'PDT000269984.3':0.00266621)'':0.000681463)'':0.000220576)'':0.000277694)'':0.000205057)'':0.000673534)'':0.000304782)'':0.000193386)'':0.000490932)'':0.000479675)'':0.000140078)'':0.00190963)'':-0.00290697,((('PDT000239288.3':0.00478774,('PDT000231575.3':0.00350023,'PDT000366966.2':0.00298356)'':0.00128006)'':0.000333206,('PDT000357131.2':0.0049933,('PDT000281608.3':0.00284873,'PDT000593615.2':0.00273953)'':0.0019679)'':0.000137509)'':0.000221108,('PDT000163279.4':0.00608011,('PDT000107852.3':0.00401576,('PDT000122016.6':0.00354053,('PDT000076965.3':0.00415299,('PDT000575867.2':0.00315965,('PDT000564133.2':0.00335846,('PDT000293196.3':0.00252361,('PDT000521322.2':0.00259793,('PDT000690532.1':0.00216374,('PDT000576984.2':0.00186993,'PDT000579448.2':0.00182342)'':0.000430375)'':0.00031812)'':0.000221861)'':0.000268482)'':0.000244242)'':0.000533197)'':0.000276994)'':0.000220383)'':0.000785276)'':-0.000589272)'':0.000398287)'':0.00278613)'':-0.000562402)'':0.0018085)'':0.00310152)'':-0.0023466)'':-0.000688125)'':0.00198686)'':0.00265332)'':-0.00348609)'':-0.00209968)'':0.00261654)'':-0.000668741)'':0.00915706)'':-0.00814427)'':-0.000117375)'':0.00360472)'':0.00234406)'':-0.00528716)'':0.00120944)'':0.0118803)'':-0.00623307)'':-0.00300399)'':0.00208472)'':-0.00103157)'':-0.0009956)'':0.00257051)'':0.00299394)'':0.00843965)'':0.00331501,((((('PDT000692391.1':0.0329835,('PDT000649298.2':0.0137049,'PDT000205673.1':0.0145141)'':0.00835933)'':0.00733474,('PDT000472544.2':0.0342362,('PDT000156938.3':0.0220583,(('PDT000128002.2':0.00904408,'PDT000692403.1':0.0116607)'':0.00256277,('PDT000155451.3':0.0108248,('PDT000188056.3':0.00632748,'PDT000061042.3':0.00700192)'':0.00459287)'':0.00226816)'':0.00306363)'':0.0106523)'':0.00532176)'':-0.00135554,((('PDT000459076.2':0.0231682,('PDT000579363.2':0.00699385,'PDT000672676.1':0.00756535)'':0.0106866)'':-0.000737654,(('PDT000443269.2':0.0240315,('PDT000336980.2':0.0133601,('PDT000283084.3':0.00973609,'PDT000407521.2':0.00938611)'':0.00650593)'':0.0026638)'':0.000970242,((('PDT000147655.3':0.00737332,'PDT000304184.1':0.00772398)'':0.00246545,('PDT000155152.2':0.00771854,('PDT000355516.2':0.00617552,'PDT000389361.2':0.00954428)'':0.00261111)'':0.00147863)'':0.00375392,('PDT000488884.1':0.0245728,('PDT000240583.3':0.0128347,(('PDT000592439.2':0.00987259,'PDT000623904.2':0.0121979)'':0.00401956,('PDT000060996.3':0.0126578,('PDT000281597.5':0.00972989,('PDT000157230.3':0.00327475,'PDT000157679.3':0.00291287)'':0.0046232)'':0.00142718)'':0.00140776)'':0.00164071)'':0.00637411)'':-0.00461095)'':0.00256451)'':0.00213726)'':0.00220425,((('PDT000189527.2':0.0142079,('PDT000287887.3':0.00845777,'PDT000320266.2':0.00924133)'':0.00591176)'':0.00148151,('PDT000548273.2':0.0153785,('PDT000317172.2':0.0100343,('PDT000326146.2':0.00399489,'PDT000378628.2':0.00388541)'':0.00482517)'':0.00529522)'':0.00112659)'':0.00238054,(('PDT000155311.3':0.022746,('PDT000160594.3':0.0150758,'PDT000112444.3':0.0131002)'':0.00353647)'':0.00324146,(('PDT000486268.2':0.0166891,(('PDT000548320.2':0.00956658,'PDT000548341.2':0.00803482)'':0.00292145,('PDT000711055.1':0.0150039,('PDT000548163.2':0.00767954,('PDT000548324.2':0.00725211,('PDT000517222.2':0.00480697,'PDT000630113.2':0.00497152)'':0.0026563)'':0.000698826)'':0.00212866)'':0.000946248)'':0.00235912)'':0.00204759,('PDT000692394.1':0.0355561,('PDT000334344.2':0.027797,('PDT000160421.3':0.0250319,('PDT000467014.2':0.0155747,(('PDT000133954.1':0.00723404,'PDT000249775.1':0.00837546)'':0.000474331,('PDT000130627.2':0.00660397,(('PDT000107650.2':0.0029246,'PDT000692395.1':0.00436301)'':0.00259166,('PDT000160420.3':0.00544863,(('PDT000060989.3':0.0031998,'PDT000061007.3':0.00318551)'':0.000911835,('PDT000160574.3':0.00512011,('PDT000324113.2':0.00248976,'PDT000325797.2':0.00249877)'':0.00164988)'':2.5482e-05)'':0.00148718)'':-0.000127446)'':0.00170499)'':0.0010071)'':0.003592)'':0.00418311)'':0.00358038)'':0.00346511)'':-0.00892795)'':0.00206226)'':-0.00127837)'':0.00483666)'':0.0065321)'':0.00139669,((('PDT000060840.3':0.0226944,('PDT000208720.1':0.000899074,'PDT000252190.1':0.000899306)'':0.024836)'':0.00125807,('PDT000157002.3':0.019231,(('PDT000148094.3':0.012136,'PDT000241759.3':0.0116782)'':0.00579587,(('PDT000155826.3':0.00874437,'PDT000061365.3':0.00953133)'':0.000713977,('PDT000060944.3':0.0062237,'PDT000061065.3':0.0059253)'':0.0038701)'':0.00590628)'':0.00313274)'':0.00400959)'':0.0164433,(('PDT000010849.3':0.0307642,(('PDT000016394.2':0.0189245,('PDT000548203.2':0.0140979,('PDT000250091.1':0.0131706,('PDT000242109.3':0.0044828,('PDT000367224.2':0.00423788,'PDT000107679.2':0.00344522)'':0.000904198)'':0.00819324)'':0.00339584)'':0.00285236)'':0.0025858,('PDT000449050.2':0.0246006,('PDT000199117.2':0.0145379,(('PDT000360461.2':0.00472725,'PDT000589647.2':0.00514314)'':0.00119841,('PDT000471116.2':0.00687952,'PDT000657009.2':0.00539108)'':0.000933718)'':0.00536089)'':0.0131387)'':-0.000710767)'':0.0055997)'':-0.00370629,((('PDT000557532.1':0.0153369,'PDT000009748.2':0.0140428)'':0.00859757,(('PDT000366176.2':0.0169075,('PDT000009721.3':0.0170448,('PDT000112573.3':0.0124354,('PDT000112589.3':0.0110289,'PDT000010328.3':0.00939951)'':0.00231149)'':0.00186076)'':0.00313482)'':0.000280311,('PDT000319679.2':0.0160026,('PDT000566459.2':0.00850336,('PDT000558460.2':0.00970556,('PDT000340816.2':0.00624351,('PDT000189571.3':0.00438377,'PDT000293117.3':0.00408658)'':0.00151171)'':0.00116178)'':0.000786619)'':0.00899081)'':0.000686273)'':0.00561862)'':0.00277917,(('PDT000227505.3':0.0394511,('PDT000465770.2':0.0180423,('PDT000569437.2':0.0117606,('PDT000429563.2':0.00981493,('PDT000679466.1':0.00831624,('PDT000192810.3':0.00918581,('PDT000312999.3':0.00716021,('PDT000315833.2':0.00431936,('PDT000443371.2':0.00331855,('PDT000214019.3':0.00348939,'PDT000356223.2':0.00338997)'':0.000772273)'':0.000138066)'':0.00150846)'':0.00106816)'':0.000531262)'':0.00114073)'':0.00460072)'':0.00130633)'':0.0117894)'':-0.00701319,((('PDT000112550.3':0.032107,('PDT000063303.3':0.021649,('PDT000503667.2':0.0147542,('PDT000112636.8':0.00898308,'PDT000112581.3':0.0109055)'':0.00170975)'':0.00761303)'':0.00480344)'':0.000886239,('PDT000112781.3':0.050212,(('PDT000151369.3':0.0177927,'PDT000283351.3':0.0170967)'':0.00423684,('PDT000041405.3':0.010368,(('PDT000242006.3':0.00718147,('PDT000110459.3':0.00419256,'PDT000250001.1':0.00459251)'':0.00244665)'':0.000873657,('PDT000366212.2':0.0129229,('PDT000112779.3':0.0139684,('PDT000009755.2':0.0100541,(('PDT000112640.10':0.00466001,'PDT000052198.3':0.00432292)'':0.00286111,('PDT000429550.2':0.0113032,('PDT000367030.2':0.00557271,(('PDT000285877.3':0.00379821,'PDT000387610.2':0.00397304)'':0.000751212,('PDT000112583.3':0.00524105,(('PDT000119943.3':0.00326852,('PDT000112580.3':0.00316735,('PDT000112592.3':0.00245617,'PDT000119941.3':0.00223726)'':0.000520134)'':0.000447852)'':0.00048327,('PDT000568693.2':0.00473242,('PDT000571678.2':0.00357576,('PDT000106361.3':0.0033846,('PDT000556898.2':0.00316113,('PDT000542130.2':0.00282471,'PDT000625597.2':0.00256236)'':0.000530446)'':0.00062553)'':0.000505131)'':0.000401746)'':-0.000266417)'':0.00173189)'':-0.000839864)'':0.00189091)'':0.000360184)'':-0.000494653)'':0.00259094)'':0.000481291)'':0.000278107)'':-0.00218023)'':0.00340607)'':0.00568965)'':0.0123977)'':-0.00984905)'':-0.00116778,(('PDT000157011.2':0.017903,'PDT000332737.2':0.01432)'':0.0126364,('PDT000157730.3':0.0454127,(('PDT000499046.2':0.0166977,('PDT000147692.3':0.0118885,'PDT000625560.2':0.0112959)'':0.00650259)'':-0.000446686,(('PDT000250784.3':0.0138989,'PDT000559297.2':0.0170676)'':0.00601709,('PDT000205672.1':0.0200698,(('PDT000282958.3':0.0118983,'PDT000671981.1':0.0107637)'':0.00308195,('PDT000522452.2':0.0146134,('PDT000603503.2':0.0133117,(('PDT000147696.3':0.0114387,'PDT000114757.3':0.0112638)'':0.00271754,(('PDT000151296.3':0.00613707,'PDT000063300.3':0.00572023)'':0.00647128,('PDT000010344.3':0.0114889,(('PDT000390249.2':0.00963401,'PDT000063326.3':0.0102808)'':0.000606379,('PDT000151345.3':0.0115691,('PDT000394749.2':0.0100129,(('PDT000218945.1':0.00616737,('PDT000287892.3':0.00433466,'PDT000386828.2':0.00492829)'':0.00128397)'':0.00129457,(('PDT000112632.8':0.00901592,('PDT000217119.3':0.00757563,('PDT000576168.2':0.00546398,('PDT000191523.3':0.00489684,('PDT000575656.2':0.00462894,'PDT000112419.3':0.00593406)'':0.000179057)'':0.000512971)'':0.000821103)'':0.000839018)'':-0.000509009,(('PDT000412263.2':0.00479154,('PDT000008635.2':0.00349951,'PDT000362541.2':0.00327812)'':0.00229744)'':4.52365e-05,('PDT000115246.3':0.0047098,('PDT000584145.2':0.00272055,'PDT000119938.3':0.00247063)'':0.00155901)'':0.000755324)'':0.00147277)'':0.000658014)'':0.00249308)'':0.00145317)'':-0.00176712)'':0.00360182)'':-0.0020574)'':0.000215454)'':0.00296671)'':0.000155485)'':-0.00188051)'':0.00643378)'':-0.00218993)'':0.00117858)'':0.0110439)'':0.00115991)'':-0.0134696)'':0.00433025)'':0.000372253)'':0.000535448)'':0.00922974)'':-0.00284665)'':0.00201178,(((('PDT000156593.3':0.0317357,'PDT000061040.3':0.0370694)'':0.00208449,('PDT000065645.3':0.0221855,'PDT000128004.2':0.0264187)'':0.00729198)'':0.00277741,(('PDT000017946.3':0.0297482,(('PDT000151295.3':0.0165835,'PDT000333310.2':0.0194437)'':0.00379741,(('PDT000065555.2':0.0131816,'PDT000242453.3':0.0179449)'':0.00267486,('PDT000128006.2':0.0118165,('PDT000110495.3':0.00849705,'PDT000128005.2':0.0115229)'':0.00140701)'':0.00442155)'':0.00391527)'':0.00607037)'':0.000515341,(('PDT000158662.3':0.0247448,'PDT000061314.3':0.0280059)'':0.0158083,(('PDT000155783.3':0.0219673,('PDT000366965.2':0.0157932,'PDT000485306.2':0.0247179)'':0.00407075)'':0.00465012,(('PDT000357649.1':0.0257757,('PDT000377939.2':0.0220041,('PDT000325127.2':0.0199726,('PDT000304187.1':0.0134166,('PDT000155183.3':0.0121449,'PDT000241749.3':0.0127587)'':0.000966725)'':0.00323469)'':0.000984903)'':0.00406041)'':0.00267046,('PDT000128003.2':0.0300901,(('PDT000327191.2':0.0243174,('PDT000158930.3':0.00494395,('PDT000564285.2':0.00532492,'PDT000582673.2':0.00454547)'':0.000634551)'':0.0122322)'':0.00675448,((('PDT000326184.2':0.0100402,('PDT000327345.2':0.00771001,('PDT000155376.3':0.00556547,'PDT000157691.3':0.00577333)'':0.000939539)'':0.00172193)'':0.00621914,(('PDT000242365.3':0.0107641,('PDT000385511.2':0.00643577,'PDT000600093.2':0.00581643)'':0.00235487)'':0.00282706,('PDT000377008.2':0.0108873,('PDT000519382.2':0.0090658,('PDT000061162.3':0.00836376,('PDT000319257.2':0.0067336,'PDT000342681.2':0.0074981)'':0.00105119)'':0.000642392)'':0.00218101)'':0.00098231)'':0.00143473)'':0.00630341,(('PDT000017937.3':0.0242119,('PDT000588638.1':0.0157561,('PDT000148078.3':0.0132945,(('PDT000320895.2':0.00421003,'PDT000377145.2':0.00455491)'':0.00122475,('PDT000324146.2':0.00344046,('PDT000241729.3':0.00258147,'PDT000398393.2':0.00310375)'':0.000449579)'':0.00303317)'':0.00433972)'':0.00923501)'':0.00660961)'':-0.010229,(('PDT000483959.2':0.0215727,'PDT000065662.3':0.0202371)'':0.00624108,(('PDT000160303.3':0.0200623,('PDT000155661.3':0.0148261,('PDT000157166.3':0.0059287,'PDT000160423.3':0.0066086)'':0.00590164)'':0.0040771)'':0.00210176,(('PDT000083411.1':0.0092634,('PDT000157096.3':0.00697948,'PDT000168702.1':0.00635122)'':0.00134035)'':0.00466237,('PDT000409966.2':0.0247499,(('PDT000108557.3':0.0170219,('PDT000536995.2':0.00847278,'PDT000618351.2':0.00772032)'':0.00775278)'':-0.000613088,(('PDT000550098.2':0.0098169,'PDT000603041.2':0.0112639)'':0.00544924,('PDT000065621.3':0.0190715,(('PDT000242079.3':0.0126419,'PDT000061358.3':0.0108653)'':0.00156054,('PDT000625078.2':0.0145189,(('PDT000338972.2':0.00992831,'PDT000603046.2':0.00765039)'':0.00151749,(('PDT000465207.2':0.00748228,'PDT000061363.3':0.00796672)'':0.00233415,('PDT000532324.2':0.0105736,(('PDT000322087.2':0.00512426,'PDT000539357.2':0.00571934)'':0.00230476,('PDT000065654.2':0.00642051,(('PDT000060988.3':0.00439111,('PDT000317412.2':0.00281386,('PDT000128008.2':0.00156623,'PDT000128009.2':0.00143077)'':0.00157792)'':0.00129906)'':0.00143869,(('PDT000242353.3':0.00474023,'PDT000325154.2':0.00461766)'':0.000558765,('PDT000242099.3':0.0053191,('PDT000603032.2':0.00561686,('PDT000332870.2':0.00409296,('PDT000326256.2':0.00345234,('PDT000655140.2':0.00330936,'PDT000671996.2':0.00337527)'':0.000436644)'':0.000473465)'':0.000607843)'':0.000451984)'':-0.000160459)'':0.000724486)'':0.00173412)'':-0.000622177)'':0.0019362)'':-0.000780213)'':0.000450616)'':0.00192378)'':-3.9327e-05)'':0.00498009)'':-0.00271639)'':0.000376279)'':0.00543284)'':-0.00363777)'':-0.000189707)'':0.00844078)'':-0.00631959)'':-0.000291702)'':0.00300241)'':0.00901872)'':-0.00840694)'':0.00471114)'':0.00469187)'':-0.002145)'':0.00555745)'':-0.000187107,((('PDT000627629.1':0.026245,('PDT000329841.1':0.0135817,'PDT000671302.1':0.0161419)'':0.00938906)'':0.00745645,((('PDT000061464.3':0.0239476,('PDT000136939.2':0.0166738,'PDT000136928.2':0.0133461)'':0.00809238)'':0.00225135,(('PDT000009336.2':0.0324057,(('PDT000334317.2':0.0167825,'PDT000072715.3':0.012781)'':0.00415743,('PDT000267236.3':0.0193561,('PDT000414375.2':0.0203406,((('PDT000306766.3':0.0068642,'PDT000406956.2':0.0064864)'':0.00626558,('PDT000557801.2':0.00712764,('PDT000344332.2':0.00536691,('PDT000347378.2':0.00569472,'PDT000459477.2':0.00516938)'':0.000768333)'':0.000740134)'':0.00385451)'':-0.000490155,('PDT000334284.1':0.00891108,(('PDT000396576.2':0.00790845,'PDT000407524.2':0.00837435)'':0.000737512,('PDT000362542.2':0.00703343,('PDT000334315.2':0.00529925,('PDT000230561.3':0.00358182,'PDT000324595.2':0.00290132)'':0.00196111)'':0.0014221)'':0.000542742)'':0.00149622)'':0.00123528)'':0.00523299)'':0.000857266)'':-0.00111887)'':0.0102745)'':-0.00411119,(('PDT000110511.3':0.0206953,('PDT000469844.2':0.0158008,('PDT000678014.1':0.0129022,'PDT000136934.2':0.0119117)'':0.00272415)'':0.00538878)'':-0.00167265,(('PDT000259504.3':0.0143351,('PDT000258497.3':0.00891999,'PDT000678425.1':0.010051)'':0.00565759)'':0.0019105,((('PDT000247061.3':0.00937648,('PDT000644445.2':0.00497322,'PDT000083409.1':0.00498099)'':0.00470226)'':0.000803281,('PDT000009732.2':0.0107811,('PDT000065684.3':0.00979879,'PDT000580739.2':0.0117546)'':0.00107882)'':-0.000113217)'':0.00328768,(('PDT000517601.2':0.0130261,'PDT000503530.3':0.0178289)'':0.00140062,('PDT000128012.2':0.0162655,('PDT000087345.3':0.0113595,('PDT000384166.2':0.00911155,'PDT000221676.2':0.00888225)'':0.0034556)'':0.00187559)'':0.000631999)'':0.000681707)'':0.00330227)'':0.00277822)'':0.00359826)'':0.000487844)'':0.00153673,((('PDT000061095.3':0.0212557,('PDT000242416.3':0.0167594,'PDT000316570.2':0.0220669)'':0.00104546)'':0.00605148,('PDT000377072.2':0.0223038,('PDT000635551.2':0.0223717,(('PDT000155322.3':0.00778875,'PDT000235265.3':0.00792325)'':0.00433115,('PDT000060880.3':0.00969893,'PDT000128013.2':0.0112773)'':0.00142528)'':0.00586893)'':0.00279002)'':-0.00023047)'':0.00389544,(('PDT000242283.3':0.028416,('PDT000155238.3':0.0249164,('PDT000156642.3':0.0149498,'PDT000061174.3':0.0177836)'':0.00784347)'':0.00428907)'':-0.00084194,(('PDT000242321.3':0.0251985,(('PDT000061098.3':0.00692556,'PDT000128018.2':0.00630824)'':0.00725489,('PDT000242164.3':0.014288,('PDT000157673.3':0.00790618,('PDT000242038.3':0.00529988,'PDT000323714.2':0.00514682)'':0.00291377)'':0.00272492)'':0.00345101)'':0.00813655)'':-0.00051425,('PDT000496148.1':0.0314019,(('PDT000060886.3':0.0238227,(('PDT000242025.3':0.00940676,('PDT000245464.3':0.00701849,'PDT000321562.2':0.00729701)'':0.000779187)'':0.00654869,('PDT000332854.2':0.0149772,(('PDT000242397.3':0.00729031,'PDT000577018.2':0.00922709)'':0.00577585,('PDT000316966.2':0.0105977,'PDT000060967.3':0.00999435)'':0.00219963)'':0.00118219)'':0.000493634)'':0.00434747)'':-0.00393872,((('PDT000242395.3':0.0161823,'PDT000598485.2':0.0166159)'':0.00352628,('PDT000245473.3':0.0104052,('PDT000241679.3':0.00855209,'PDT000319837.2':0.00717561)'':0.000634651)'':0.0078865)'':0.000380992,(('PDT000245493.3':0.0206423,('PDT000128016.2':0.00587042,('PDT000605005.2':0.00402063,('PDT000242045.3':0.00301683,('PDT000241934.3':0.00273363,'PDT000136979.3':0.00305018)'':0.00027509)'':0.00103709)'':0.00114615)'':0.0116139)'':-0.00325465,((('PDT000060999.3':0.0108641,('PDT000060925.3':0.00771495,'PDT000061078.3':0.00786505)'':0.00335989)'':-0.000263389,(('PDT000556753.2':0.00771022,'PDT000128017.2':0.00749848)'':0.00174663,('PDT000600101.2':0.0075739,('PDT000242134.3':0.00615753,'PDT000332724.2':0.00481507)'':0.00175095)'':0.000915282)'':0.00266508)'':0.0020636,(('PDT000242316.3':0.0102663,('PDT000157932.3':0.00917173,'PDT000241978.3':0.00888737)'':0.00120836)'':-4.91443e-05,((('PDT000061170.3':0.00733019,'PDT000065661.3':0.00639571)'':0.00326109,(('PDT000061086.3':0.00977495,'PDT000168695.1':0.00778455)'':0.00073404,(('PDT000242096.3':0.00656322,'PDT000322066.2':0.00559788)'':0.000903485,('PDT000241987.3':0.00448491,'PDT000353574.2':0.00438582)'':0.00140547)'':0.000893798)'':0.00125893)'':-0.000495151,(('PDT000242026.3':0.0105712,'PDT000242327.3':0.00788081)'':0.00105182,(('PDT000383924.2':0.00750441,('PDT000434231.2':0.00672325,('PDT000241937.3':0.00510761,'PDT000242423.3':0.00544909)'':0.00136665)'':0.0012162)'':-0.000213873,('PDT000322753.2':0.00851209,('PDT000241991.3':0.00642576,('PDT000239042.3':0.00595679,('PDT000711098.1':0.00509148,('PDT000242014.3':0.00446143,'PDT000245454.3':0.00430089)'':0.000779633)'':0.00073752)'':0.000483357)'':0.000946064)'':0.000251443)'':0.0014715)'':-0.000459786)'':0.00285767)'':-0.000491605)'':0.00177871)'':0.00145026)'':-0.000488496)'':0.0104553)'':-0.00551809)'':0.00389564)'':0.00136399)'':0.00074232)'':0.00605013)'':-0.00621363,(('PDT000010162.3':0.0435223,(('PDT000484000.2':0.0181504,('PDT000692210.1':0.0165524,('PDT000189663.3':0.0156692,'PDT000334485.2':0.012959)'':0.00539684)'':0.00431223)'':0.00430403,('PDT000264910.3':0.031457,('PDT000623176.1':0.0213317,('PDT000642682.2':0.0243784,(('PDT000387706.2':0.0113292,'PDT000060971.3':0.0112317)'':0.00670197,(('PDT000189693.3':0.0161575,'PDT000549587.2':0.0195147)'':0.00130745,('PDT000234147.3':0.0190504,(('PDT000719905.1':0.0104484,('PDT000259484.3':0.0117006,'PDT000481021.2':0.00955275)'':0.000823592)'':0.000225812,('PDT000127937.3':0.0133321,('PDT000241742.3':0.00936743,('PDT000386004.2':0.0105857,('PDT000534058.2':0.00695733,('PDT000322435.2':0.00449615,'PDT000112453.5':0.00437104)'':0.00120302)'':0.00263343)'':0.00102089)'':0.00234231)'':8.69652e-05)'':0.00285668)'':0.00109578)'':0.00157929)'':0.00475869)'':0.00108304)'':0.00239285)'':-0.00258706)'':0.0175331)'':-0.0139739,('PDT000275628.3':0.0495574,(('PDT000324055.2':0.0334234,('PDT000147632.3':0.0340365,('PDT000108555.2':0.0219593,('PDT000155867.3':0.0166861,'PDT000242442.3':0.0102627)'':0.0158327)'':0.00291125)'':0.0027268)'':-0.00210019,((('PDT000237155.3':0.0136425,'PDT000708943.1':0.0120011)'':0.0162234,('PDT000064676.3':0.0186573,('PDT000375953.2':0.0165697,'PDT000065601.3':0.0145317)'':0.00345408)'':0.00789849)'':0.00172413,((('PDT000241950.3':0.0199808,'PDT000409954.2':0.0181088)'':0.00535928,(('PDT000065543.3':0.0131596,'PDT000128014.2':0.0148602)'':0.00305456,('PDT000065605.3':0.0184521,('PDT000320821.2':0.0165288,('PDT000065675.3':0.015369,('PDT000060977.3':0.0116292,('PDT000321651.2':0.00869966,('PDT000317212.2':0.00475578,'PDT000061469.3':0.00400829)'':0.0048471)'':0.00239221)'':0.00110713)'':0.0020706)'':0.00229431)'':-0.00154972)'':0.00331088)'':-0.000614989,((('PDT000239072.3':0.0240266,'PDT000061362.3':0.017603)'':0.00786436,(('PDT000010337.3':0.0202312,('PDT000278463.3':0.0117731,'PDT000061461.3':0.0104183)'':0.00916376)'':6.2698e-05,(('PDT000242127.3':0.0165164,'PDT000060911.3':0.0147305)'':0.00145599,('PDT000155241.3':0.014902,('PDT000061379.3':0.0124521,'PDT000128019.2':0.0129909)'':0.000867029)'':0.0015191)'':0.00799885)'':0.0031163)'':-0.00211827,(('PDT000061400.3':0.0168972,('PDT000651798.2':0.0152735,('PDT000157185.3':0.0100694,'PDT000320228.2':0.0127053)'':0.00567383)'':0.00486284)'':0.0023765,(((('PDT000128052.2':0.012353,('PDT000128029.2':0.00455223,('PDT000574920.1':0.0060397,('PDT000128045.2':0.00114579,'PDT000128048.2':0.00125277)'':0.00192759)'':0.000804373)'':0.0032765)'':0.00127503,('PDT000061472.3':0.0106924,('PDT000314184.3':0.00938473,('PDT000318659.2':0.00685082,('PDT000061459.3':0.0049414,('PDT000160476.3':0.00392118,'PDT000061295.3':0.00404933)'':0.00133074)'':0.000948978)'':0.00224478)'':0.00127768)'':-0.00164017)'':0.000341883,('PDT000578932.2':0.0181703,('PDT000328070.2':0.0134315,('PDT000065598.3':0.00840694,('PDT000316200.2':0.0107756,('PDT000328015.2':0.00814895,(('PDT000061160.3':0.00634512,'PDT000061320.3':0.00770148)'':0.000699776,('PDT000110524.3':0.0106585,((('PDT000061422.3':0.00558569,'PDT000061481.3':0.00426996)'':0.000665313,('PDT000061420.3':0.00448917,('PDT000060980.3':0.0041306,('PDT000061374.3':0.00328729,'PDT000061415.3':0.00349914)'':0.000855475)'':0.000864915)'':0.00030436)'':0.000129084,(('PDT000128030.2':0.00348243,('PDT000128043.2':0.00331681,('PDT000128036.2':0.00108384,('PDT000128035.2':0.000648428,'PDT000128049.2':0.000750732)'':0.00041424)'':0.00117378)'':6.92565e-05)'':0.00281172,('PDT000160427.3':0.00695512,('PDT000065644.3':0.00439883,('PDT000155381.3':0.00466903,(('PDT000241684.3':0.00376437,'PDT000215135.3':0.00352834)'':3.83781e-05,('PDT000242118.3':0.00307587,('PDT000157904.3':0.00261654,'PDT000242015.3':0.00257308)'':0.000317148)'':0.000450386)'':0.000717193)'':0.000492094)'':0.000702692)'':-0.000502757)'':0.000428681)'':0.0023812)'':-0.000322251)'':0.000853558)'':0.000248408)'':0.00151311)'':0.000860549)'':0.000612384)'':-0.00250631)'':0.00207169,(((('PDT000486908.2':0.0132685,('PDT000324698.2':0.00962691,('PDT000332833.2':0.00694131,('PDT000160347.3':0.00616539,'PDT000061145.3':0.00648151)'':0.000898694)'':0.000575724)'':0.00405699)'':0.000280547,(('PDT000316158.2':0.00779873,'PDT000519339.2':0.00680637)'':0.00172997,('PDT000060787.3':0.0111531,('PDT000156538.3':0.00806398,('PDT000160316.3':0.00530236,'PDT000061063.3':0.00516934)'':0.00106532)'':0.00254473)'':-5.13106e-05)'':0.00122284)'':0.00125526,(('PDT000061465.3':0.0192409,('PDT000270973.3':0.00912494,'PDT000061183.3':0.010039)'':0.00531299)'':-2.42253e-05,('PDT000061082.3':0.0121989,('PDT000315548.2':0.00914521,(('PDT000128034.2':0.00714932,('PDT000328155.2':0.00608231,'PDT000645861.2':0.00547949)'':0.00195268)'':0.000663349,('PDT000157719.3':0.00746034,('PDT000409885.2':0.00452909,'PDT000065687.3':0.00424634)'':0.0030521)'':0.000996602)'':0.00112976)'':0.00139392)'':0.0026803)'':-0.000193621)'':0.00127061,('PDT000555982.2':0.0196742,(('PDT000061046.3':0.0112818,'PDT000128024.2':0.0106276)'':0.00140174,(('PDT000065679.3':0.00877923,('PDT000065579.3':0.00757675,('PDT000061404.3':0.00811326,'PDT000065670.3':0.00679874)'':0.000480848)'':0.00175979)'':0.000626391,((('PDT000575705.1':0.00678955,'PDT000595106.1':0.00944715)'':0.000157333,(('PDT000128055.2':0.00790926,'PDT000128056.2':0.00536954)'':0.00043467,('PDT000013410.2':0.00665174,('PDT000040714.3':0.00545788,('PDT000128033.2':0.00513974,('PDT000128039.2':0.00299889,('PDT000128026.2':0.00210153,'PDT000128027.2':0.00159551)'':0.00108837)'':0.00149827)'':0.000982244)'':0.000304368)'':0.000442341)'':0.00125961)'':0.00116637,('PDT000060916.3':0.0150979,(('PDT000065688.3':0.0102384,('PDT000389372.2':0.00763833,'PDT000532051.2':0.00794327)'':0.00188937)'':0.000490667,('PDT000061197.3':0.0100429,('PDT000065631.3':0.0103948,(('PDT000060945.3':0.00544867,'PDT000061125.3':0.00549773)'':0.00197831,(('PDT000242258.3':0.00626087,('PDT000640722.2':0.00609057,('PDT000390580.2':0.00549206,('PDT000242357.3':0.00511827,'PDT000573815.2':0.00483297)'':0.000983618)'':0.00077838)'':0.00040312)'':0.000422385,(('PDT000156655.3':0.00534734,('PDT000157196.3':0.00347393,'PDT000160314.3':0.00360554)'':0.00151303)'':0.00213571,('PDT000598452.2':0.00766309,(('PDT000242071.3':0.0042024,'PDT000328022.2':0.00406618)'':0.00122972,(('PDT000245457.3':0.00509132,'PDT000061156.3':0.0048619)'':0.000803878,(('PDT000155593.3':0.00514382,'PDT000242012.3':0.00572138)'':0.000219444,('PDT000317193.2':0.00480766,'PDT000065551.3':0.0044516)'':0.000768861)'':0.000561544)'':0.00044618)'':0.0010007)'':-0.000711502)'':0.000135392)'':0.000826916)'':0.00222007)'':0.000185815)'':-0.00101707)'':0.00394352)'':-0.00424538)'':0.00127635)'':0.00141194)'':0.0107446)'':-0.0094429)'':0.000389177)'':0.00440472)'':0.0039976)'':0.000387157)'':0.00567291)'':0.00263892)'':0.0143443)'':-0.0153544)'':-0.00174333)'':0.00599092)'':0.00086075)'':0.000419094)'':0.00060656,((((('PDT000378609.2':0.0204643,'PDT000492515.2':0.0191026)'':0.00386877,('PDT000235763.3':0.0296983,('PDT000242286.3':0.0163284,('PDT000158888.3':0.0185278,'PDT000327912.2':0.0141208)'':0.00506423)'':0.00485769)'':0.00132135)'':0.00895426,('PDT000548346.2':0.0637489,(('PDT000552701.2':0.0234758,('PDT000332904.2':0.0170744,'PDT000061290.3':0.0151773)'':0.0033681)'':0.00194898,(('PDT000061341.3':0.0270279,'PDT000564280.2':0.0242819)'':0.000800558,(('PDT000320891.2':0.0246985,'PDT000161506.3':0.0233437)'':5.53021e-05,(('PDT000156573.3':0.0237358,('PDT000157895.3':0.0152003,'PDT000242167.3':0.0175396)'':0.00256329)'':0.000920845,((('PDT000328105.2':0.00819021,'PDT000332909.2':0.00851289)'':0.00439203,('PDT000241689.3':0.00893358,'PDT000452770.2':0.00824962)'':0.00576362)'':0.000459647,('PDT000359118.2':0.0120657,('PDT000110522.3':0.00776028,('PDT000242160.3':0.0079856,'PDT000127982.2':0.0080104)'':0.00127017)'':0.00593917)'':0.00192428)'':0.00330856)'':0.00300991)'':0.00136769)'':0.000490222)'':0.0301949)'':-0.0222642)'':0.0120979,((('PDT000324677.2':0.040047,('PDT000376233.2':0.0301543,('PDT000321625.2':0.0226474,(('PDT000158885.3':0.013276,'PDT000342683.2':0.0141727)'':0.00159098,('PDT000110503.3':0.0107738,('PDT000061210.3':0.00647439,'PDT000548340.2':0.00707551)'':0.00455574)'':0.00364231)'':0.00476722)'':0.00585407)'':0.00289976)'':-0.00440711,((('PDT000132546.1':0.0161342,(('PDT000231314.3':0.00868055,'PDT000242326.3':0.00884405)'':0.0019469,('PDT000538884.2':0.0141917,('PDT000410868.2':0.00865495,('PDT000573295.2':0.00912776,('PDT000378694.2':0.00746875,('PDT000545349.2':0.00410515,'PDT000646243.2':0.00346759)'':0.00359758)'':0.00155549)'':0.00114505)'':0.0022977)'':-0.000248256)'':0.00573132)'':-0.000979054,(('PDT000242312.3':0.0131188,'PDT000379562.2':0.0135149)'':0.00624964,(('PDT000242262.3':0.0213144,('PDT000538644.2':0.0106387,('PDT000157766.3':0.00723029,'PDT000414769.2':0.00738781)'':0.000976952)'':0.00599524)'':-0.00225764,('PDT000155935.3':0.0236967,(('PDT000323291.2':0.0150033,(('PDT000410721.2':0.00631749,('PDT000637607.2':0.00591275,('PDT000155500.3':0.00512808,'PDT000600123.2':0.00493172)'':0.000565203)'':0.000517462)'':0.00141008,('PDT000239346.1':0.0088426,('PDT000061100.3':0.0048158,('PDT000539315.2':0.00375551,'PDT000576044.2':0.00442323)'':0.000963805)'':0.00387761)'':-0.000682761)'':0.00344192)'':-0.0018832,(('PDT000323318.2':0.00973827,'PDT000341786.2':0.0126757)'':0.00344265,('PDT000548327.2':0.0118652,(('PDT000212318.3':0.00581497,(('PDT000706153.1':0.00551638,('PDT000420990.2':0.00387238,'PDT000595817.2':0.0036996)'':0.000773341)'':0.000289128,('PDT000168701.1':0.00394172,('PDT000127973.2':0.00343954,'PDT000127975.2':0.00333945)'':0.00124204)'':0.00079166)'':0.00063923)'':0.0010182,('PDT000320268.2':0.0109452,('PDT000326236.2':0.00783098,(('PDT000242415.3':0.005574,('PDT000523354.2':0.00347503,'PDT000625224.2':0.00291283)'':0.00256112)'':0.000106579,('PDT000160294.3':0.005122,('PDT000061360.3':0.00488356,('PDT000377076.2':0.00401285,('PDT000147720.3':0.00366689,('PDT000241755.3':0.00349938,('PDT000242459.3':0.00234262,'PDT000326162.2':0.00244923)'':0.000784577)'':0.000488086)'':0.000314204)'':0.000938594)'':0.000370486)'':0.000455895)'':0.00149625)'':0.00236797)'':-0.00276612)'':0.00438365)'':-0.00110255)'':-0.00147336)'':0.00951635)'':-0.00505128)'':0.00185172)'':-0.000691476)'':0.00324071,(('PDT000379495.2':0.0229419,(('PDT000320166.2':0.0133896,'PDT000009340.2':0.0135384)'':0.000526442,('PDT000112771.3':0.00837462,'PDT000120805.3':0.00937808)'':0.00499776)'':0.00587546)'':-0.00107046,((('PDT000061061.3':0.0133807,'PDT000065630.3':0.0157934)'':0.00302381,('PDT000311716.3':0.0150363,('PDT000524329.2':0.00958346,'PDT000620659.2':0.00989384)'':0.00714821)'':0.000612286)'':0.00531863,('PDT000155452.3':0.0445905,('PDT000039650.1':0.0278239,(('PDT000127977.2':0.0163332,(('PDT000242011.3':0.0123267,'PDT000127978.2':0.0114075)'':0.00240599,('PDT000061324.3':0.0114744,('PDT000107661.2':0.00830486,('PDT000065592.3':0.00840303,(('PDT000281599.3':0.00363803,'PDT000587578.2':0.0030486)'':0.00260597,('PDT000127976.2':0.00353004,'PDT000127980.2':0.0034539)'':0.00300275)'':0.0020106)'':0.00111122)'':0.0028064)'':0.000868977)'':0.00305915)'':-0.00170994,((('PDT000061316.3':0.007063,('PDT000061148.3':0.00539137,'PDT000065553.3':0.00594623)'':0.000386745)'':0.00597889,('PDT000084342.1':0.00870306,('PDT000060860.3':0.00970045,('PDT000061050.3':0.00647629,'PDT000720398.1':0.00586451)'':0.0012263)'':0.00174306)'':0.00186928)'':0.00151636,('PDT000564261.2':0.0136081,('PDT000038882.1':0.0129645,('PDT000682398.1':0.0124984,('PDT000378524.2':0.0108306,(('PDT000315556.2':0.011565,'PDT000157249.3':0.00715822)'':0.00123928,(('PDT000377982.2':0.00634598,('PDT000404820.2':0.00573688,'PDT000409953.2':0.00660022)'':0.000606118)'':0.000595508,(('PDT000332747.2':0.00657015,'PDT000060883.3':0.00668475)'':0.00141886,(('PDT000060934.3':0.0050502,'PDT000060938.3':0.00480251)'':0.000846074,(('PDT000065578.3':0.00463647,('PDT000060847.3':0.00368062,'PDT000061060.3':0.00429466)'':0.000885297)'':0.000824787,('PDT000624241.2':0.00598278,('PDT000323329.2':0.00474597,('PDT000160378.3':0.00339195,'PDT000241748.3':0.00398371)'':0.000829126)'':0.00105999)'':-6.39058e-05)'':0.000596226)'':0.00115363)'':2.94948e-05)'':0.00172158)'':0.00125199)'':0.00181817)'':0.00064711)'':0.00313951)'':-0.00403806)'':0.000758267)'':0.00816566)'':0.00459968)'':-0.00794174)'':0.00330857)'':-0.00459045)'':0.00554514)'':-0.00141723,((('PDT000245480.3':0.0310075,('PDT000241762.3':0.0195015,'PDT000084343.1':0.0194071)'':0.00739273)'':-0.0011821,('PDT000157845.3':0.0321722,(('PDT000155621.3':0.0146358,'PDT000414310.2':0.018216)'':0.000879368,(('PDT000155489.3':0.0122148,'PDT000588692.2':0.0122863)'':0.0029724,('PDT000155601.3':0.0159578,('PDT000389359.2':0.0134425,('PDT000600105.2':0.0138696,('PDT000127981.2':0.00718305,'PDT000227980.1':0.00751115)'':0.00545221)'':0.000586983)'':0.00113363)'':0.00108068)'':0.00170675)'':0.00568173)'':-8.07929e-05)'':0.000463456,((('PDT000241973.3':0.0292973,('PDT000324033.2':0.0145516,('PDT000570781.2':0.0127805,(('PDT000157852.3':0.00750031,'PDT000241904.3':0.00798509)'':0.00128009,('PDT000158672.3':0.00871188,('PDT000327230.2':0.00398918,'PDT000327403.2':0.00438904)'':0.00467886)'':9.59875e-05)'':0.00125286)'':0.00241563)'':0.00511108)'':0.0041634,(('PDT000549484.2':0.0141655,'PDT000110509.3':0.0124244)'':0.00289782,('PDT000151492.3':0.0134641,('PDT000242389.3':0.0171353,('PDT000061308.3':0.0175814,('PDT000569204.2':0.0104815,('PDT000565139.2':0.0114522,('PDT000386969.2':0.0099175,('PDT000147691.3':0.00921363,('PDT000242376.3':0.00883346,('PDT000245498.3':0.0105763,('PDT000241713.3':0.00751997,('PDT000242306.3':0.00667499,(('PDT000160370.3':0.00440739,'PDT000242067.3':0.00487497)'':0.000850158,('PDT000060837.3':0.00423735,('PDT000065624.3':0.00358441,'PDT000136943.3':0.00339465)'':0.000937569)'':0.00101738)'':0.000746479)'':0.000530086)'':0.000879885)'':0.000754197)'':0.00051181)'':0.00121199)'':0.000976639)'':0.000370009)'':0.00123324)'':0.000363551)'':0.00137263)'':-0.00114467)'':0.00368915)'':-0.00197412,((('PDT000155468.3':0.0166251,('PDT000110521.3':0.010303,('PDT000539359.2':0.00640264,('PDT000242204.3':0.00592668,'PDT000711414.1':0.00651612)'':0.000352114)'':0.00236652)'':0.00420597)'':-0.00204821,(('PDT000065594.3':0.0137275,('PDT000155230.3':0.00878709,(('PDT000539361.2':0.00412836,'PDT000649400.2':0.00345118)'':0.00197288,(('PDT000320260.2':0.00405738,'PDT000582678.2':0.00480981)'':0.000416925,('PDT000155359.3':0.00488097,('PDT000378557.2':0.00419641,('PDT000327145.2':0.00291209,'PDT000327338.2':0.00297085)'':0.000979111)'':0.000628465)'':8.19077e-05)'':0.000263091)'':0.00344069)'':0.00343254)'':-0.00316618,(('PDT000341780.2':0.00699042,('PDT000245481.3':0.00671044,('PDT000395666.2':0.00527923,('PDT000155562.3':0.00463301,('PDT000242072.3':0.00387748,('PDT000333935.2':0.00426992,('PDT000160495.3':0.00392778,('PDT000157497.3':0.00315941,'PDT000061389.3':0.00322081)'':0.000458561)'':0.000797827)'':0.000295728)'':0.00086117)'':0.000210232)'':0.000692686)'':0.000271661)'':0.00096647,('PDT000011386.3':0.0144537,('PDT000316201.2':0.0147192,('PDT000061350.3':0.0154233,('PDT000322948.2':0.0118174,((('PDT000316902.2':0.00354027,'PDT000061473.3':0.00354061)'':0.00135101,('PDT000158649.3':0.00395988,('PDT000327077.2':0.00360715,'PDT000383907.2':0.003374)'':0.0010245)'':0.000828491)'':0.00205417,(('PDT000334874.2':0.00609044,'PDT000065606.3':0.00475106)'':0.00173834,('PDT000155208.3':0.00893918,('PDT000155442.3':0.0122532,('PDT000160158.3':0.00899688,((('PDT000242019.3':0.00552842,('PDT000127985.2':0.00270636,('PDT000127984.2':0.00151156,'PDT000168698.1':0.00198739)'':0.000784135)'':0.00238221)'':0.000345293,('PDT000157759.3':0.00593975,(('PDT000383908.2':0.00448251,'PDT000637618.2':0.00409776)'':0.000951728,('PDT000242069.3':0.00478445,('PDT000136981.3':0.00517363,(('PDT000061425.3':0.00463276,'PDT000061407.3':0.00384182)'':0.000235321,(('PDT000155227.3':0.00251336,'PDT000241895.3':0.00257544)'':0.000437682,('PDT000241896.3':0.00260361,'PDT000065626.3':0.00219064)'':0.000457138)'':0.000881325)'':0.00114467)'':0.000258466)'':-0.000289566)'':0.000862373)'':-0.000307907)'':0.000196422,(('PDT000242169.3':0.00539415,('PDT000601760.2':0.0045245,('PDT000618551.2':0.00329919,('PDT000500401.2':0.00308354,'PDT000624285.2':0.00300246)'':0.000590843)'':0.000692548)'':0.000690391)'':3.00786e-05,('PDT000065548.3':0.00571981,(('PDT000322931.2':0.004309,('PDT000061038.3':0.00319738,'PDT000061381.3':0.00349058)'':0.000781712)'':0.000710011,('PDT000598451.2':0.00564723,('PDT000244066.3':0.00467707,('PDT000158676.3':0.00426444,(('PDT000242057.3':0.00299577,('PDT000148083.3':0.00297656,('PDT000241962.3':0.00246783,'PDT000326110.2':0.00232116)'':0.000518052)'':0.000321569)'':-8.98543e-05,('PDT000160470.3':0.00356946,('PDT000157108.3':0.00253316,('PDT000155149.3':0.00190685,'PDT000157676.3':0.00198653)'':0.000559131)'':0.000329386)'':9.41486e-05)'':0.000741573)'':0.000514441)'':0.00139887)'':-0.00112993)'':0.0015186)'':-0.00133749)'':0.000420612)'':0.00169226)'':0.00137306)'':0.000113083)'':-0.00113601)'':-0.000839801)'':0.00362884)'':0.00138905)'':0.00043839)'':7.14059e-05)'':-0.00445264)'':0.000617427)'':0.00386617)'':-0.00171018,(('PDT000244743.3':0.0198577,(('PDT000316565.2':0.00736652,'PDT000378754.2':0.00801958)'':0.00375975,('PDT000061471.3':0.010602,('PDT000157144.3':0.00940704,('PDT000334455.2':0.00674746,'PDT000060990.3':0.00669344)'':0.00237401)'':0.00204198)'':-0.000185051)'':0.00512924)'':0.00132305,((('PDT000379464.2':0.0156355,'PDT000601101.2':0.0166736)'':0.000924916,('PDT000546419.2':0.0175614,(('PDT000320270.2':0.0126167,'PDT000564289.2':0.0146245)'':0.000586605,('PDT000327925.2':0.0151566,(('PDT000110497.3':0.00959278,(('PDT000320196.2':0.0061254,'PDT000158760.3':0.0060018)'':0.000943597,('PDT000157886.3':0.00372249,'PDT000548440.2':0.00365611)'':0.00267208)'':0.00281093)'':-0.00171003,((('PDT000333298.2':0.00480202,'PDT000518575.2':0.00546508)'':0.00183965,('PDT000381418.2':0.00726202,('PDT000324711.2':0.00561933,('PDT000381419.2':0.00462115,'PDT000394251.2':0.00484006)'':0.00104281)'':0.000394293)'':0.00107712)'':9.50268e-05,('PDT000327928.2':0.00702177,('PDT000147675.3':0.00619515,('PDT000156522.3':0.00546089,('PDT000176705.3':0.00437441,('PDT000319920.2':0.00463157,('PDT000616659.2':0.00253914,'PDT000617925.2':0.00254966)'':0.00193717)'':0.00042746)'':0.000404436)'':0.00130266)'':0.000365467)'':0.000433571)'':0.00112458)'':0.00327919)'':-0.000135034)'':0.00270734)'':0.000980191)'':-0.00418926,((('PDT000409913.2':0.00766968,('PDT000564276.2':0.00711952,'PDT000600035.2':0.00719458)'':0.00144872)'':0.00369912,('PDT000242451.3':0.0108893,((('PDT000378574.2':0.00464469,'PDT000557367.2':0.00571801)'':0.00148486,('PDT000242335.3':0.00505465,('PDT000332901.2':0.00312154,'PDT000378726.2':0.00306053)'':0.00240042)'':0.000484123)'':0.000489298,(('PDT000245508.3':0.00840834,(('PDT000155133.3':0.0022312,'PDT000241721.3':0.00245848)'':0.00135191,('PDT000556148.2':0.00255542,'PDT000601759.2':0.00303061)'':0.00124526)'':0.00376732)'':-0.000996514,(('PDT000333318.2':0.00426362,'PDT000383909.2':0.00480464)'':0.000744011,('PDT000386971.2':0.00446617,('PDT000702556.1':0.00359841,'PDT000710994.1':0.00308088)'':0.00151291)'':0.000236657)'':0.00126966)'':0.000900877)'':0.00377698)'':-0.00160033)'':0.000533633,((('PDT000319784.2':0.0101058,'PDT000136987.3':0.00963042)'':0.00118741,('PDT000242439.3':0.0131483,('PDT000368388.2':0.0116599,('PDT000562129.2':0.00914576,('PDT000618341.2':0.00773052,(('PDT000245507.3':0.00375826,'PDT000612022.2':0.00362181)'':0.00217187,('PDT000341794.2':0.0061467,('PDT000377970.2':0.00567951,'PDT000317217.2':0.00469349)'':0.000511605)'':0.000495245)'':0.00143333)'':0.00139829)'':0.00126116)'':0.000602311)'':-0.00036925)'':0.000234716,((('PDT000668488.1':0.0144358,('PDT000242314.3':0.00497972,'PDT000352688.2':0.00450321)'':0.00818336)'':0.000142292,('PDT000161535.3':0.00814047,('PDT000157115.3':0.00901203,('PDT000065562.3':0.00685137,('PDT000158873.3':0.00398029,('PDT000341803.2':0.00419444,'PDT000532076.2':0.00347794)'':0.000803854)'':0.00220157)'':0.000705477)'':0.000503515)'':0.00272053)'':-0.0010255,(('PDT000327296.2':0.00683288,'PDT000011111.2':0.00694512)'':0.00244118,(('PDT000061009.3':0.00921943,(('PDT000250609.3':0.00616258,'PDT000157225.3':0.00471242)'':0.00068653,('PDT000378655.2':0.00523179,'PDT000317177.2':0.00522221)'':0.000865032)'':0.00305634)'':-0.000179916,('PDT000523355.2':0.00946782,(('PDT000378533.2':0.00587428,('PDT000110515.3':0.0044261,('PDT000155834.3':0.00333883,'PDT000718565.1':0.0038425)'':0.00046212)'':0.00136081)'':0.00157214,(('PDT000327306.2':0.00904399,('PDT000317152.2':0.00754292,('PDT000242003.3':0.00625814,(('PDT000332708.2':0.00382688,'PDT000542735.2':0.00524952)'':0.000815372,('PDT000409962.2':0.00555367,(('PDT000577027.2':0.00385652,'PDT000577033.2':0.00441041)'':0.000511914,(('PDT000377171.2':0.00445534,('PDT000621328.2':0.00373946,('PDT000382866.2':0.00307976,'PDT000607591.2':0.00370125)'':0.000551847)'':0.000141537)'':0.000331918,('PDT000433922.2':0.00403647,('PDT000664000.1':0.00447529,('PDT000388952.2':0.00324006,('PDT000601924.2':0.00384963,('PDT000416746.2':0.00240989,'PDT000546418.2':0.00258013)'':0.00083349)'':0.000168915)'':0.000443339)'':7.59618e-05)'':0.000198718)'':0.00045426)'':0.000950516)'':-0.000436409)'':0.00158176)'':0.00051608)'':0.000271085)'':-0.000670535,('PDT000083414.1':0.0123319,(('PDT000060889.3':0.00695193,('PDT000155930.3':0.00390443,'PDT000242291.3':0.0044646)'':0.00192435)'':0.00163056,('PDT000157868.3':0.0115683,((('PDT000510764.2':0.00412387,('PDT000239300.3':0.00263343,'PDT000362555.2':0.00335459)'':0.00095315)'':0.000547739,(('PDT000107684.2':0.00307016,'PDT000127983.2':0.00332089)'':0.00182476,(('PDT000548728.2':0.00243412,'PDT000587568.2':0.00186287)'':0.00144923,('PDT000355502.2':0.00360427,('PDT000280704.3':0.00253316,'PDT000344780.2':0.00235608)'':0.00118319)'':0.000244184)'':0.001091)'':0.000135592)'':0.000403003,('PDT000241971.3':0.00697677,('PDT000158903.3':0.00834086,(('PDT000156768.3':0.00546735,('PDT000155395.3':0.00423955,(('PDT000158784.3':0.00240549,('PDT000316887.2':0.00163512,'PDT000409944.2':0.00165994)'':0.000788509)'':0.000455162,('PDT000061166.3':0.00239827,('PDT000586042.2':0.00256065,'PDT000603602.2':0.00202971)'':0.000593061)'':0.000269262)'':0.00086265)'':0.000691714)'':-0.000180491,(('PDT000060912.3':0.00489339,('PDT000061036.3':0.00346749,'PDT000065615.3':0.0033115)'':0.000975456)'':0.000142634,(('PDT000156544.3':0.00359552,'PDT000242114.3':0.00309378)'':0.00116679,('PDT000322978.2':0.00459368,(('PDT000155698.3':0.00417409,'PDT000065561.3':0.0039041)'':0.00025091,('PDT000157001.3':0.00323421,('PDT000155182.3':0.00391559,('PDT000155988.3':0.00318527,'PDT000157848.3':0.00259796)'':0.000375541)'':0.000332088)'':0.000386495)'':0.000188734)'':7.76378e-05)'':0.000187405)'':-1.12373e-05)'':0.00160884)'':0.00112537)'':-0.00177651)'':0.00246758)'':-0.00100041)'':0.00280533)'':-0.00303985)'':0.00104991)'':0.00169335)'':-0.00143308)'':0.00331447)'':-0.00172964)'':0.000522281)'':-5.36117e-06)'':0.000257882)'':0.00308376)'':-0.00267739)'':0.00508994)'':0.00379109)'':-0.000653756)'':0.0110232)'':0.00103743,((('PDT000311676.3':0.0175839,'PDT000595866.2':0.0152144)'':0.018415,((('PDT000452666.2':0.0214199,'PDT000119932.3':0.0155368)'':0.00798519,('PDT000386063.2':0.0273917,('PDT000388603.2':0.0159606,('PDT000317266.2':0.0169368,('PDT000242454.3':0.0154673,('PDT000242097.3':0.0129201,('PDT000627641.2':0.012825,('PDT000216133.3':0.00248686,'PDT000465579.2':0.0038037)'':0.00871071)'':0.000522607)'':0.00108382)'':0.000759768)'':0.00145191)'':0.00543002)'':0.00136023)'':-0.00107517,(('PDT000017973.3':0.0129819,'PDT000010859.2':0.0138584)'':0.00843747,((('PDT000495509.2':0.0109077,('PDT000377221.2':0.00817106,'PDT000669479.1':0.00682104)'':0.00486827)'':0.00254116,(('PDT000069930.3':0.0161372,('PDT000509949.2':0.00778401,('PDT000373142.2':0.00660641,('PDT000101835.3':0.00601016,'PDT000119942.3':0.00622744)'':0.00045749)'':0.00103789)'':0.00521444)'':-0.00207109,('PDT000244749.3':0.0170218,(('PDT000421983.2':0.00847448,'PDT000520362.2':0.00850162)'':0.00358279,('PDT000126006.3':0.0101914,(('PDT000359125.2':0.00659101,'PDT000009754.3':0.00603579)'':0.000329103,('PDT000539015.2':0.00636345,('PDT000352685.2':0.00269907,'PDT000393029.2':0.00259194)'':0.0028173)'':0.00110615)'':0.00263088)'':0.000683218)'':0.00420812)'':-0.00278743)'':0.00318105)'':0.000131522,(('PDT000534057.1':0.0151302,('PDT000156649.3':0.0113761,'PDT000710245.1':0.0144962)'':0.00405456)'':0.00052721,('PDT000252187.1':0.0248617,(('PDT000076993.2':0.0135189,'PDT000065691.3':0.016712)'':0.00129668,(('PDT000333960.2':0.00989409,'PDT000110517.3':0.00900571)'':0.00558272,(('PDT000151290.3':0.0097204,'PDT000260753.3':0.0106837)'':0.00110711,('PDT000523887.2':0.0157898,('PDT000304191.1':0.00987903,('PDT000409925.2':0.00970429,('PDT000504866.2':0.00879661,('PDT000242406.3':0.00645226,(('PDT000480932.2':0.0034682,'PDT000623019.2':0.00411285)'':0.00116847,(('PDT000334412.2':0.00302588,('PDT000407590.2':0.00105851,'PDT000141152.3':0.00123942)'':0.00226613)'':0.00264761,('PDT000593813.2':0.00756182,('PDT000122019.4':0.00428889,(('PDT000151648.1':0.000351477,('PDT000151645.1':0.000267951,'PDT000151649.1':0.000431979)'':4.83631e-05)'':0.0029219,('PDT000122021.4':0.00343667,('PDT000216376.3':0.00188291,'PDT000114448.3':0.0021119)'':0.0011465)'':-0.000153152)'':0.00170257)'':0.000331121)'':-0.000241416)'':0.000262948)'':0.00190894)'':0.00092916)'':0.000537856)'':0.000938518)'':0.00290584)'':-2.20575e-06)'':0.000608486)'':0.00124876)'':0.00478387)'':-0.00413804)'':-0.0014563)'':0.00564586)'':-0.00226695)'':0.0130463)'':-0.000955014,(((((('PDT000548252.2':0.00467181,('PDT000548221.2':0.004693,'PDT000548619.2':0.00525725)'':0.00110391)'':0.00815567,('PDT000548249.2':0.0111726,('PDT000548160.2':0.0100029,'PDT000548238.2':0.00768737)'':0.00254133)'':0.000758531)'':0.00161922,(('PDT000548177.2':0.0134623,'PDT000010524.3':0.0165387)'':0.00187172,(('PDT000548176.2':0.00946472,'PDT000548191.2':0.00858548)'':0.00209717,(('PDT000548329.2':0.0126452,('PDT000548241.2':0.0069397,('PDT000548201.2':0.00550063,'PDT000634645.2':0.00544787)'':0.00261)'':0.00210195)'':0.000850448,(('PDT000548573.2':0.00788944,('PDT000548232.2':0.00783286,'PDT000548301.2':0.00603384)'':0.00177346)'':0.00340789,('PDT000548448.2':0.00677592,('PDT000548147.2':0.00721539,('PDT000548482.2':0.0056989,('PDT000321957.2':0.00399694,('PDT000548347.2':0.00335199,'PDT000548592.2':0.0041308)'':0.00048371)'':0.00218205)'':0.00043136)'':0.00147577)'':0.00155999)'':0.00118866)'':9.62832e-06)'':0.00387068)'':-0.00163791)'':0.00460222,(('PDT000548355.2':0.0147554,('PDT000548231.2':0.0184741,'PDT000548509.2':0.013321)'':0.00556133)'':0.00627363,(('PDT000548319.2':0.0246798,('PDT000604201.2':0.0177466,('PDT000449053.2':0.00931104,'PDT000548271.2':0.010882)'':0.00533036)'':0.00479578)'':-0.00101613,(('PDT000548209.2':0.0169557,('PDT000548445.2':0.00987414,('PDT000548580.2':0.00631603,('PDT000548291.2':0.00395065,'PDT000548569.2':0.00362209)'':0.00326055)'':0.00409764)'':0.00348109)'':0.00179703,(('PDT000548198.2':0.0154416,('PDT000548156.2':0.0125683,'PDT000548187.2':0.0111232)'':0.00685583)'':2.67255e-05,(('PDT000548243.2':0.0210543,('PDT000204924.3':0.00928686,'PDT000593756.2':0.00867304)'':0.00962284)'':0.00142447,('PDT000201236.3':0.0209521,(('PDT000548506.2':0.00736715,'PDT000548522.2':0.00727425)'':0.00177045,('PDT000672408.1':0.012996,('PDT000197063.3':0.00777504,('PDT000191518.3':0.00841521,('PDT000250079.1':0.0121768,('PDT000497988.1':0.00660345,(('PDT000407566.2':0.0042696,'PDT000107841.3':0.0037025)'':0.00131247,('PDT000357151.2':0.005223,('PDT000081607.3':0.00527337,(('PDT000599872.2':0.00347381,('PDT000205740.3':0.00235436,('PDT000235341.3':0.00188111,'PDT000399941.2':0.00161504)'':0.000538401)'':0.000571482)'':0.000559181,('PDT000627802.2':0.00795937,('PDT000627632.2':0.00745735,('PDT000355530.2':0.00482639,(('PDT000284934.3':0.0028685,('PDT000602557.2':0.00168303,('PDT000268677.3':0.00137709,'PDT000112556.3':0.00151886)'':0.00021448)'':0.00128079)'':-0.000390533,('PDT000691314.1':0.00265733,('PDT000264766.3':0.00260004,('PDT000668726.1':0.00170133,('PDT000627807.2':0.00178103,('PDT000392383.2':0.00103855,'PDT000606268.2':0.00105935)'':0.000515138)'':0.000123363)'':0.00054537)'':0.0002435)'':0.000169056)'':0.00144084)'':0.000763982)'':0.000167847)'':-0.00139318)'':0.00185501)'':9.94169e-05)'':-0.000451874)'':0.00180843)'':0.00118997)'':0.000402104)'':0.000548669)'':0.00220877)'':-0.00198083)'':0.00524163)'':0.00145687)'':0.000515932)'':-9.01667e-05)'':0.00234292)'':0.00510712)'':-0.00456175)'':0.00598142,(('PDT000242425.3':0.0401262,((('PDT000595107.1':0.0260021,('PDT000061115.2':0.0221865,('PDT000158872.2':0.0180342,('PDT000061333.2':0.0108605,('PDT000242008.2':0.00771651,'PDT000242215.2':0.00713599)'':0.00290021)'':0.00539862)'':0.003569)'':0.00333184)'':-0.0021385,(('PDT000242130.2':0.0147243,('PDT000242095.3':0.0139245,'PDT000110170.3':0.0117267)'':0.0044097)'':0.00189954,('PDT000110519.2':0.015923,(('PDT000157760.2':0.00820548,'PDT000242055.2':0.00790432)'':0.00837911,(('PDT000158849.2':0.0150664,('PDT000242042.2':0.0118909,'PDT000060833.2':0.00927576)'':0.0038262)'':0.000250295,('PDT000160434.2':0.00848465,(('PDT000160540.3':0.00642545,'PDT000586371.2':0.00798235)'':0.00216677,(('PDT000293911.2':0.00603737,'PDT000390828.1':0.00520973)'':0.00119568,('PDT000390829.1':0.00535319,('PDT000242030.2':0.00541575,('PDT000157035.2':0.00333065,'PDT000242170.2':0.00375235)'':0.00135358)'':0.000700035)'':0.000881696)'':0.00172974)'':0.00197207)'':0.000587923)'':0.00188291)'':0.00522312)'':-0.0032859)'':0.00213851)'':0.000317101,(('PDT000065569.3':0.0191852,('PDT000604957.1':0.012435,('PDT000586374.2':0.0149789,('PDT000136978.3':0.00766837,('PDT000065614.3':0.00571996,('PDT000377215.2':0.00522468,'PDT000110480.3':0.00513592)'':0.000838637)'':0.00244309)'':0.00189202)'':0.00360083)'':0.000935121)'':-0.00148146,(('PDT000110152.3':0.017213,('PDT000245455.3':0.0139035,'PDT000604971.1':0.0129873)'':0.00121601)'':0.000682818,('PDT000285170.1':0.0134131,((('PDT000327089.2':0.0068329,('PDT000293420.3':0.00554232,'PDT000060870.3':0.00602298)'':0.000586897)'':0.000914922,(('PDT000158035.3':0.00450852,('PDT000156983.3':0.0039227,'PDT000381416.2':0.00385319)'':0.0010621)'':0.00189132,(('PDT000155855.3':0.00518148,'PDT000469330.2':0.00576382)'':0.000697084,('PDT000294290.3':0.00606933,('PDT000060924.3':0.00284454,'PDT000061337.3':0.00284579)'':0.00244781)'':0.000879848)'':0.000640149)'':0.00104377)'':0.00173523,((('PDT000293410.3':0.00602434,'PDT000405453.2':0.00661496)'':0.00171563,('PDT000333303.2':0.00684426,'PDT000293907.3':0.00699074)'':0.00106602)'':6.5855e-05,(('PDT000083419.1':0.00367391,'PDT000016367.2':0.00410121)'':0.00246202,('PDT000110193.3':0.00815418,(('PDT000570439.2':0.00476864,'PDT000110169.3':0.00420355)'':0.000569468,('PDT000488011.2':0.00308435,('PDT000127968.2':0.00185258,'PDT000604962.1':0.00144445)'':0.00181291)'':0.00224807)'':0.00273973)'':-0.000628781)'':0.000858295)'':0.000311742)'':0.00666564)'':-0.000855963)'':0.0011)'':0.00266334)'':0.014628)'':-0.00945731,(((('PDT000548254.2':0.0198028,('PDT000548148.2':0.0160575,'PDT000548426.2':0.0151648)'':0.00254579)'':0.00448032,('PDT000127970.2':0.0265044,('PDT000548247.2':0.0153281,'PDT000548328.2':0.015662)'':0.00613559)'':-0.000323628)'':0.00249377,((('PDT000548460.2':0.0151001,'PDT000548520.2':0.013539)'':0.00706311,(('PDT000357648.1':0.029053,('PDT000065655.3':0.0209638,('PDT000060928.3':0.00791813,'PDT000110163.3':0.00802607)'':0.00766003)'':0.00477511)'':-0.00166449,(('PDT000548167.2':0.0156916,'PDT000127971.2':0.0186241)'':0.0037279,(('PDT000548349.2':0.00876772,('PDT000629718.1':0.00688042,('PDT000324608.2':0.00653374,'PDT000548230.2':0.00560796)'':0.00108433)'':0.00287856)'':8.31533e-05,('PDT000548358.2':0.0131578,(('PDT000671021.1':0.00675101,('PDT000548170.2':0.00292372,'PDT000548599.2':0.00306012)'':0.00340827)'':0.00025949,(('PDT000060896.3':0.00478241,'PDT000570808.2':0.00447777)'':0.00148095,(('PDT000549832.2':0.00529202,('PDT000340499.2':0.00445548,'PDT000548595.2':0.0052184)'':0.000791392)'':0.000195671,('PDT000548438.2':0.00468913,('PDT000156727.3':0.0036416,'PDT000548146.2':0.00344706)'':0.00162375)'':0.000600647)'':0.000945133)'':0.000454019)'':0.00669111)'':-0.00181628)'':0.00403961)'':0.00293901)'':0.00615745)'':-0.00443451,(('PDT000548157.2':0.0139587,'PDT000548462.2':0.01492)'':0.0020519,(('PDT000548612.2':0.0154976,('PDT000548570.2':0.0143928,(('PDT000542736.2':0.00812501,'PDT000548473.2':0.00860309)'':0.00398235,('PDT000548175.2':0.0113349,('PDT000347570.2':0.00910241,'PDT000548242.2':0.00869969)'':0.00129752)'':0.00144259)'':0.00229093)'':0.00133627)'':-0.00113619,(('PDT000548508.2':0.010378,('PDT000548615.2':0.00998944,('PDT000687693.1':0.0067964,('PDT000548155.2':0.00356133,'PDT000717362.1':0.00382169)'':0.00333299)'':0.00293072)'':0.00101272)'':0.00541111,(('PDT000548195.2':0.0193636,('PDT000548290.2':0.0113108,'PDT000548464.2':0.0107509)'':0.004339)'':-8.17359e-05,(('PDT000548390.2':0.0142862,'PDT000548417.2':0.010053)'':0.00204074,((('PDT000548217.2':0.0068813,('PDT000548257.2':0.00649775,('PDT000548193.2':0.00375382,'PDT000548466.2':0.00382647)'':0.0018542)'':0.000226315)'':0.00330693,('PDT000548222.2':0.0104,('PDT000548215.2':0.00789931,('PDT000548205.2':0.00534193,'PDT000548389.2':0.00481947)'':0.00242234)'':0.00217339)'':0.000466273)'':0.00144356,(('PDT000548465.2':0.0103077,('PDT000548154.2':0.00964884,'PDT000548538.2':0.00804676)'':0.00199888)'':0.00188697,(('PDT000548338.2':0.00626229,'PDT000548611.2':0.00579311)'':0.00528512,('PDT000548404.2':0.0126329,('PDT000548594.2':0.00778647,('PDT000548344.2':0.00666142,('PDT000620655.2':0.0066515,'PDT000168697.1':0.006979)'':0.000108977)'':0.00117107)'':0.00201155)'':-0.000155994)'':0.00189978)'':-3.14554e-05)'':0.00260688)'':0.000491901)'':-0.000474232)'':0.000288145)'':0.00400604)'':-0.000743513)'':0.00301266)'':0.000285239,(('PDT000548356.2':0.0279976,(('PDT000338969.2':0.0236684,'PDT000548207.2':0.0225489)'':0.000670486,(('PDT000683083.1':0.017481,('PDT000548282.2':0.0123295,'PDT000548507.2':0.0142183)'':0.00452806)'':0.00489729,('PDT000548598.2':0.0261313,(('PDT000155586.3':0.0162163,('PDT000548172.2':0.0106051,'PDT000548342.2':0.0131453)'':0.00574593)'':0.00262201,('PDT000495588.2':0.0254758,('PDT000293908.3':0.0187032,('PDT000112438.3':0.0164147,('PDT000548437.2':0.0118511,'PDT000548525.2':0.0118357)'':0.00242497)'':0.00287118)'':0.00394328)'':-0.000932305)'':0.00375318)'':5.25152e-05)'':0.00132359)'':0.00420238)'':-0.000777417,((('PDT000548581.1':0.019693,'PDT000548190.2':0.0198027)'':0.000736205,('PDT000548250.2':0.0172245,('PDT000294299.3':0.0199848,(('PDT000548245.2':0.00750137,'PDT000548577.2':0.00727053)'':0.00467898,('PDT000548192.2':0.0137555,('PDT000548206.2':0.00776515,'PDT000548326.2':0.00898125)'':0.00280608)'':0.00223599)'':0.00360279)'':0.00323025)'':-0.000245401)'':0.000501127,((('PDT000548391.2':0.0337735,('PDT000548237.2':0.0194727,('PDT000548325.2':0.0159581,(('PDT000232033.3':0.00769332,'PDT000548272.2':0.00801718)'':0.00747737,('PDT000395664.2':0.0146071,('PDT000548564.2':0.00994974,('PDT000325717.2':0.00905053,'PDT000496338.2':0.00897017)'':0.00240721)'':0.00302848)'':-2.32125e-05)'':0.00381905)'':0.00255624)'':0.00233414)'':-0.00499421,('PDT000548575.2':0.010994,('PDT000548568.2':0.0098326,(('PDT000242275.3':0.00537186,'PDT000548394.2':0.00527804)'':0.00551195,(('PDT000113667.3':0.00470961,'PDT000368903.1':0.00494982)'':0.00428032,('PDT000503393.2':0.00444467,('PDT000293415.3':0.00384904,'PDT000578917.2':0.00402969)'':0.000342158)'':0.00398257)'':0.00104522)'':0.00128935)'':0.00208611)'':0.0052258)'':0.00192939,(('PDT000010850.3':0.0412672,(('PDT000342319.2':0.0144368,('PDT000061014.3':0.0127791,('PDT000504051.2':0.00518567,'PDT000548229.2':0.00576943)'':0.00369953)'':0.00647409)'':0.00210234,('PDT000548197.2':0.0206055,('PDT000293421.3':0.0182736,('PDT000312505.3':0.0121881,('PDT000548343.2':0.0106667,('PDT000710999.1':0.0093993,('PDT000548200.2':0.00832083,('PDT000127969.2':0.00813917,('PDT000061346.3':0.00624125,('PDT000025378.2':0.00558798,('PDT000009750.2':0.00526606,('PDT000151326.3':0.00362102,('PDT000283077.3':0.00258215,('PDT000061012.3':0.00277109,'PDT000112547.3':0.00241801)'':0.000609865)'':0.000955351)'':0.000584371)'':0.000860652)'':0.000296972)'':0.00257053)'':0.000691523)'':0.00146163)'':0.000742974)'':0.00129725)'':0.00190749)'':0.00881601)'':-0.00352158)'':0.0206229)'':-0.0202155,('PDT000548159.2':0.025723,((('PDT000548357.2':0.0107765,('PDT000548597.2':0.00624666,('PDT000548270.2':0.00444496,'PDT000548572.2':0.00533548)'':0.00286577)'':0.00451372)'':0.000988428,('PDT000384155.2':0.0117895,('PDT000499241.2':0.0168608,('PDT000548149.2':0.0146315,('PDT000548288.2':0.00929904,('PDT000376144.2':0.00706243,'PDT000548220.2':0.00589477)'':0.00233526)'':0.00124863)'':0.00200662)'':0.000439792)'':-0.000645156)'':0.00342826,(('PDT000548529.2':0.0123295,('PDT000145362.3':0.00801212,'PDT000548542.2':0.00738628)'':0.00373547)'':0.0028664,(('PDT000548623.2':0.00982339,('PDT000548246.2':0.00975968,('PDT000548418.2':0.00944783,'PDT000548423.2':0.0121121)'':0.000680524)'':0.00203557)'':0.0024961,(('PDT000548202.2':0.0109679,'PDT000548199.2':0.0106634)'':0.00131085,('PDT000548551.2':0.0102449,('PDT000548350.2':0.00958848,('PDT000554240.2':0.00771173,('PDT000548213.2':0.00703149,'PDT000548477.2':0.00721291)'':0.000520923)'':0.0021578)'':0.000858079)'':0.00335843)'':0.00194486)'':0.000268803)'':5.76092e-05)'':0.0112763)'':-0.0080439)'':0.00137937)'':0.00201734)'':0.0028879)'':-0.00222208)'':0.00146111)'':0.00227385)'':0.00162561,((('PDT000232038.2':0.0323062,(('PDT000622607.1':0.00723413,'PDT000123373.1':0.00860097)'':0.00426985,('PDT000582579.1':0.0103044,('PDT000151339.3':0.0102077,('PDT000136925.2':0.00543243,'PDT000136933.2':0.00571297)'':0.00500851)'':0.00248067)'':0.000613563)'':0.0104662)'':0.000519381,((('PDT000319690.2':0.0277848,('PDT000060887.3':0.0227038,('PDT000017125.1':0.0280802,('PDT000110472.3':0.0200319,('PDT000717521.1':0.0181258,('PDT000302813.3':0.0210351,('PDT000267233.3':0.016784,('PDT000228229.1':0.0144219,('PDT000221998.3':0.0147357,('PDT000403967.2':0.0125371,('PDT000195764.3':0.0121663,('PDT000625188.2':0.0114267,('PDT000510660.2':0.00778464,('PDT000235343.3':0.00675537,'PDT000400787.2':0.00715973)'':0.00100221)'':0.00331292)'':0.000257769)'':0.00111758)'':0.00177433)'':0.00123249)'':0.000985419)'':0.000833487)'':0.000685572)'':0.000828499)'':0.00121686)'':0.000273121)'':0.00264338)'':-0.00147279,((('PDT000390212.2':0.0157964,'PDT000347090.1':0.0146143)'':0.00431703,('PDT000010931.2':0.0160495,('PDT000253489.3':0.013879,('PDT000065682.3':0.0126749,(('PDT000464159.2':0.00681069,'PDT000136922.2':0.00821721)'':0.00405532,('PDT000155212.3':0.00953667,('PDT000388873.2':0.00912149,('PDT000627613.2':0.00653742,('PDT000222571.3':0.00594571,'PDT000324614.2':0.00648349)'':0.000367182)'':0.00194805)'':0.00158925)'':0.000414738)'':0.00147283)'':0.00323799)'':0.00333298)'':0.0032875)'':-0.00335764,('PDT000414367.2':0.0204802,('PDT000252811.3':0.0313259,(('PDT000411981.2':0.0165505,('PDT000375686.2':0.0156669,'PDT000010362.3':0.0142045)'':0.00151979)'':0.000342228,('PDT000595103.1':0.0165783,(('PDT000258481.3':0.012917,'PDT000258500.3':0.0104382)'':0.00293592,(('PDT000563710.2':0.0106188,('PDT000399880.2':0.00831957,'PDT000107481.3':0.00865483)'':0.00237489)'':-1.61497e-05,(('PDT000171549.3':0.00585242,'PDT000640710.2':0.00647848)'':0.00504009,(('PDT000065593.3':0.00730891,('PDT000136931.2':0.00501475,('PDT000136936.2':0.00529305,'PDT000136930.2':0.00477275)'':0.00075285)'':0.00255124)'':0.00145031,('PDT000273849.3':0.018032,(('PDT000338713.2':0.00802918,'PDT000062289.1':0.00908522)'':0.00163358,('PDT000563113.2':0.0131655,('PDT000366187.2':0.0119659,(('PDT000355550.2':0.00673711,'PDT000381304.2':0.00620339)'':0.00174623,(('PDT000366184.2':0.00498867,('PDT000107869.3':0.00405766,('PDT000257242.3':0.00352226,'PDT000124895.3':0.00454949)'':0.000921086)'':0.000642477)'':0.00117532,(('PDT000583334.2':0.00672273,('PDT000123487.3':0.00579473,('PDT000351780.2':0.00453447,('PDT000562546.2':0.0039734,'PDT000565111.2':0.00389591)'':0.000299212)'':0.000633385)'':0.00151412)'':-0.000194041,(('PDT000107858.3':0.00727446,'PDT000120802.3':0.00588084)'':0.000869056,('PDT000259528.3':0.0098991,('PDT000627801.2':0.0106162,(('PDT000295811.3':0.00368767,('PDT000351554.2':0.00297027,('PDT000356241.2':0.00226249,('PDT000156857.3':0.00221788,('PDT000181343.3':0.00133782,'PDT000616411.2':0.00165858)'':0.000774488)'':0.00017492)'':0.00115435)'':0.00128021)'':-0.000329977,('PDT000258488.3':0.00689942,('PDT000266787.3':0.00795772,('PDT000060806.3':0.00510317,('PDT000258492.3':0.00547168,(('PDT000708537.1':0.00337334,('PDT000389775.2':0.00239707,('PDT000140521.3':0.00232739,('PDT000351830.2':0.00181084,'PDT000520256.2':0.00198309)'':0.00036624)'':0.000174734)'':0.00171534)'':-0.000407268,('PDT000321942.2':0.00743381,('PDT000499646.2':0.00438154,(('PDT000532412.2':0.0025496,'PDT000126372.4':0.00243893)'':0.000872185,(('PDT000010361.3':0.00145085,'PDT000435655.1':0.00164729)'':0.00163354,('PDT000156858.3':0.00332222,('PDT000151520.3':0.00315543,('PDT000549337.2':0.00241621,(('PDT000357276.2':0.00231491,'PDT000510681.2':0.00227591)'':0.000121936,(('PDT000244741.3':0.00227947,('PDT000106366.3':0.00139587,'PDT000107957.3':0.00130197)'':0.000912422)'':0.000144682,('PDT000532760.2':0.00191922,('PDT000639763.2':0.00189716,('PDT000188061.3':0.00153193,'PDT000366298.2':0.00146358)'':0.000399918)'':0.000178248)'':0.000109216)'':9.83317e-05)'':0.000234519)'':0.000715381)'':0.00024865)'':-0.000178012)'':0.000211354)'':0.000794755)'':0.000908659)'':-0.00138653)'':0.00146205)'':7.30467e-05)'':0.000592874)'':0.000660802)'':-0.00212174)'':0.00352311)'':0.000856236)'':-0.00171703)'':-0.00111521)'':0.000528084)'':0.00162602)'':0.00150792)'':0.000117081)'':-0.000193524)'':0.00463583)'':-0.00564087)'':0.00247115)'':0.000270573)'':0.00115487)'':0.00486548)'':-0.00394264)'':0.00479178)'':0.00282459)'':-0.00600187)'':0.00287697)'':0.00386351,((('PDT000504803.2':0.0153468,('PDT000259784.3':0.00982437,'PDT000322994.2':0.00704363)'':0.00498123)'':0.00453665,(('PDT000060872.3':0.0143605,('PDT000065552.3':0.00724418,'PDT000065612.3':0.00737832)'':0.00488966)'':0.00499737,(('PDT000061161.3':0.0156356,(('PDT000065667.3':0.00799317,'PDT000242102.3':0.00768923)'':0.00107586,('PDT000157879.3':0.00644764,'PDT000128000.2':0.00659756)'':0.00234272)'':0.00477578)'':-0.00207818,(('PDT000239077.3':0.011625,'PDT000065647.3':0.0104433)'':0.000526279,(('PDT000160327.3':0.00652863,'PDT000065616.3':0.00571747)'':0.00154136,('PDT000065681.3':0.00720161,('PDT000061399.3':0.00495422,'PDT000061484.3':0.00520308)'':0.00235339)'':0.00148934)'':0.00252027)'':0.00231066)'':0.00232242)'':0.00341381)'':0.00474489,(('PDT000548591.2':0.0272431,('PDT000017931.3':0.0221977,('PDT000151334.3':0.0213613,('PDT000163567.3':0.0207657,('PDT000009747.3':0.0163472,('PDT000373186.2':0.0161695,('PDT000355490.2':0.0144485,('PDT000198180.3':0.0118855,('PDT000625564.2':0.00927397,'PDT000248551.1':0.00917113)'':0.00118243)'':0.00251653)'':0.00199366)'':0.00053684)'':0.00230767)'':0.00353497)'':0.00250078)'':0.00127431)'':-0.000884104,((('PDT000258484.3':0.0134473,'PDT000136932.2':0.0146354)'':0.000916995,(('PDT000241763.3':0.00763075,'PDT000136937.2':0.00914015)'':0.00861407,('PDT000160354.3':0.0116256,('PDT000295780.3':0.0111087,('PDT000258491.3':0.010686,('PDT000645943.2':0.00897055,('PDT000258482.3':0.00862032,('PDT000258501.3':0.00633653,('PDT000212464.3':0.00586081,'PDT000547721.2':0.00597409)'':0.000721422)'':0.00107086)'':0.000499985)'':0.00107338)'':0.00234486)'':0.00127315)'':-0.000289312)'':0.000802347)'':0.00461421,((('PDT000110457.3':0.0236162,('PDT000065566.3':0.0104455,'PDT000110456.3':0.0113929)'':0.00710542)'':0.000773343,(('PDT000325727.2':0.0183323,'PDT000342315.2':0.0226765)'':0.00131036,(('PDT000086988.1':0.013799,('PDT000133829.3':0.0119648,('PDT000409948.2':0.0113438,'PDT000359798.1':0.00983379)'':0.00149419)'':0.00101462)'':0.00102459,('PDT000435650.1':0.0111087,('PDT000497877.1':0.0126379,('PDT000235302.3':0.0129705,('PDT000008636.2':0.010083,('PDT000385877.2':0.00768407,'PDT000443404.2':0.00960013)'':0.00271239)'':0.000972996)'':0.000668723)'':0.00053385)'':0.00100696)'':0.00550474)'':2.72806e-07)'':0.00123137,((('PDT000548188.2':0.0127227,'PDT000595104.1':0.0132577)'':0.00369801,(('PDT000258490.3':0.0147105,'PDT000267235.3':0.0183719)'':0.000564737,('PDT000009739.3':0.0131141,(('PDT000359241.2':0.00801853,'PDT000112548.3':0.00738747)'':0.00456787,('PDT000396577.2':0.0117393,('PDT000065674.3':0.0104374,(('PDT000208731.1':0.00829044,('PDT000061462.3':0.0052651,('PDT000061483.3':0.00269698,'PDT000065683.3':0.00299051)'':0.00220048)'':0.00273688)'':0.0018008,('PDT000598661.1':0.0126154,('PDT000157252.3':0.00903733,(('PDT000541978.1':0.00568549,'PDT000359794.1':0.00461241)'':0.00355495,('PDT000106293.3':0.0122945,('PDT000625575.2':0.0104794,('PDT000663861.1':0.00520074,(('PDT000190328.3':0.00239991,'PDT000471176.2':0.00238955)'':0.000914637,('PDT000283217.3':0.00395008,'PDT000128743.3':0.00333172)'':8.37002e-05)'':0.00195686)'':0.00211994)'':0.00162026)'':-0.000144447)'':0.00124567)'':0.00093094)'':-0.000457152)'':0.000580017)'':0.00100704)'':-0.000662926)'':0.00244453)'':0.00188702)'':0.00260051)'':-0.00356951,(('PDT000010540.3':0.0287147,('PDT000010855.3':0.0133649,('PDT000017934.3':0.00803948,('PDT000017932.3':0.00669441,'PDT000017961.3':0.00747439)'':0.00341047)'':0.00442572)'':0.00957131)'':-0.00415755,(('PDT000110194.2':0.0166108,('PDT000110173.3':0.0110679,('PDT000110155.3':0.00683508,'PDT000110174.3':0.00670132)'':0.00414567)'':0.0103349)'':-0.00305533,(('PDT000468270.2':0.0206484,('PDT000470590.2':0.0179057,(('PDT000151355.3':0.0060291,'PDT000213947.3':0.0061975)'':0.000782039,('PDT000710896.1':0.00887523,(('PDT000340801.2':0.00649502,'PDT000631509.2':0.00605478)'':0.000797688,('PDT000010213.3':0.00541067,('PDT000123484.3':0.00528485,('PDT000360446.2':0.00350712,('PDT000214004.3':0.00205086,'PDT000399916.2':0.00174269)'':0.00197177)'':0.00138143)'':0.00105286)'':0.000469099)'':0.00194117)'':-0.000993783)'':0.00860338)'':0.00603513)'':-0.0086404,('PDT000108872.3':0.0205489,('PDT000155682.3':0.0155376,('PDT000341793.2':0.0124557,(('PDT000297754.3':0.0112146,(('PDT000627650.2':0.00451616,('PDT000303556.3':0.00367581,('PDT000242786.3':0.00264899,'PDT000604959.1':0.00314811)'':0.00121606)'':0.000690691)'':0.00055372,(('PDT000110168.3':0.00336065,'PDT000110184.3':0.00302658)'':0.000673548,('PDT000110178.3':0.00318663,('PDT000110165.3':0.00284433,'PDT000110179.3':0.00324107)'':0.000103873)'':0.00041678)'':0.00157797)'':0.00462231)'':-0.000137556,((('PDT000061094.3':0.00736079,('PDT000267348.4':0.0044134,'PDT000061043.3':0.00454628)'':0.00487322)'':0.00279072,(('PDT000110526.3':0.00766542,('PDT000065556.3':0.00668736,('PDT000242444.3':0.00478565,(('PDT000241958.3':0.00314494,'PDT000060797.3':0.00343991)'':0.000517221,('PDT000242000.3':0.00457823,'PDT000523351.2':0.00409834)'':7.49215e-05)'':0.00119809)'':0.00188397)'':0.00149324)'':-0.000710335,('PDT000219962.1':0.00861963,('PDT000009744.3':0.00587335,('PDT000267690.3':0.00484078,('PDT000110512.3':0.00364679,('PDT000534762.2':0.00334119,('PDT000282962.3':0.00246894,'PDT000112635.8':0.00202163)'':0.000350173)'':0.000891955)'':0.00105822)'':0.000950656)'':0.00112448)'':0.000562554)'':0.00274256)'':-0.00205476,(('PDT000051407.3':0.00944758,('PDT000421049.2':0.00738587,('PDT000289663.3':0.00538903,('PDT000356200.2':0.00449471,'PDT000112569.3':0.0044739)'':0.00107081)'':0.00139224)'':0.00233786)'':-0.000252109,('PDT000366153.2':0.0106123,('PDT000407577.2':0.0154145,('PDT000151336.3':0.00877203,('PDT000309681.3':0.0109569,(('PDT000250089.1':0.00973769,('PDT000359797.1':0.00363416,('PDT000359865.1':0.00474969,('PDT000359836.1':0.0026506,('PDT000359801.1':0.00215459,'PDT000359859.1':0.00244081)'':0.000490487)'':0.0010436)'':0.000422514)'':0.00333462)'':-0.00141447,(('PDT000156962.3':0.00635689,'PDT000333627.2':0.00665981)'':0.00127121,('PDT000151333.3':0.00667273,(('PDT000010321.3':0.0045181,'PDT000017949.3':0.00365737)'':0.00142154,(('PDT000017950.3':0.00543365,'PDT000009756.2':0.00591645)'':0.000367532,('PDT000264736.3':0.0048439,('PDT000044148.3':0.00454597,(('PDT000270312.1':0.00153803,('PDT000510083.1':0.00107075,'PDT000168696.1':0.00112705)'':0.000759402)'':0.00210059,('PDT000366963.2':0.00318321,(('PDT000193135.3':0.00269248,'PDT000563128.2':0.00279271)'':0.000814604,(('PDT000151291.3':0.00340418,('PDT000076987.3':0.00321686,'PDT000137596.3':0.0028655)'':0.000485708)'':-5.31736e-05,('PDT000434259.2':0.00243016,('PDT000257238.3':0.00270935,('PDT000303460.3':0.00237768,('PDT000112777.3':0.0021261,'PDT000114449.3':0.00216489)'':0.00011638)'':0.000408994)'':7.64603e-05)'':0.000400454)'':0.000372553)'':0.000459919)'':-0.000349842)'':0.00142986)'':0.000495351)'':0.000438892)'':4.70267e-05)'':0.00161409)'':0.000147632)'':-0.00116323)'':0.00292964)'':0.000678683)'':0.000224046)'':0.00013223)'':-0.00240552)'':0.000301905)'':0.000153853)'':0.00604006)'':0.0004793)'':0.0012361)'':-0.00497189)'':0.00381567)'':0.004289)'':-0.00517805)'':0.002551)'':0.00115591)'':0.00280401)'':-0.00125217)'':0.00147435)'':0.00435592)'':-0.00259384,('PDT000407490.1':0.136808,(('PDT000548289.2':0.0183604,('PDT000532045.2':0.0147506,('PDT000061134.3':0.00574386,'PDT000641395.2':0.00720044)'':0.00769344)'':0.00851341)'':0.0274471,(('PDT000678121.1':0.0328123,(('PDT000488333.2':0.022541,('PDT000548168.2':0.011125,'PDT000127972.2':0.00998081)'':0.00904545)'':0.00502961,('PDT000258502.3':0.0183692,((('PDT000158742.3':0.0133549,'PDT000342308.2':0.0110812)'':0.00661581,(('PDT000390265.2':0.011207,'PDT000390289.2':0.0114349)'':0.00115696,('PDT000171519.3':0.0121523,('PDT000227324.3':0.00188417,'PDT000273845.3':0.00161024)'':0.00640062)'':0.00506145)'':0.00172643)'':0.000202431,('PDT000218988.1':0.0215933,('PDT000110475.3':0.0175133,(('PDT000127997.2':0.0112878,('PDT000190543.3':0.00800539,'PDT000400029.2':0.00829351)'':0.00180066)'':0.00219958,(('PDT000471141.2':0.0122376,('PDT000147972.3':0.00781009,'PDT000072782.3':0.00807081)'':0.00435431)'':0.0023925,('PDT000274724.3':0.014199,('PDT000399258.2':0.0143294,('PDT000252746.3':0.0119115,('PDT000136924.2':0.0113644,(('PDT000162310.3':0.00778085,('PDT000580809.2':0.00764363,'PDT000598669.1':0.00666767)'':0.000852246)'':0.00136386,(('PDT000523324.2':0.00658284,'PDT000627628.2':0.00544736)'':0.00385285,('PDT000598493.2':0.00727229,('PDT000480988.2':0.00582174,('PDT000225453.3':0.00323192,'PDT000389783.2':0.00324928)'':0.00104675)'':0.00234153)'':0.0014022)'':0.000144361)'':0.00175918)'':0.00049269)'':0.000886838)'':0.00144889)'':-0.00109365)'':0.00129946)'':0.00414125)'':0.00108616)'':-0.00423124)'':0.00412541)'':0.0080547)'':0.00648469)'':-0.0106242,(('PDT000503110.2':0.0265645,((('PDT000231270.3':0.00948322,'PDT000616796.2':0.00924078)'':0.00677912,(('PDT000176921.3':0.0153522,'PDT000683657.1':0.0140132)'':0.00175733,('PDT000136923.2':0.00357384,'PDT000136938.2':0.00320447)'':0.0143218)'':0.00457658)'':-0.000670137,('PDT000010846.3':0.0230954,('PDT000564996.2':0.0208792,(('PDT000286186.3':0.0135277,('PDT000009337.2':0.0116404,('PDT000471165.2':0.0140597,'PDT000010211.3':0.00958186)'':0.000651797)'':0.00369485)'':-0.000952839,(('PDT000107842.3':0.00714602,('PDT000471169.2':0.00749855,'PDT000499116.1':0.00696095)'':0.000546932)'':0.00337087,('PDT000258504.3':0.0139126,(('PDT000107960.3':0.0068198,('PDT000683647.1':0.00584047,'PDT000106368.3':0.00629403)'':0.00120145)'':0.00100581,('PDT000712381.1':0.0101176,('PDT000399255.2':0.00611032,'PDT000068783.4':0.00573758)'':0.00189906)'':0.000963027)'':0.00356438)'':-0.000779581)'':0.00236145)'':0.00638906)'':0.00456855)'':-0.00607319)'':0.00801903)'':-0.00592667,((('PDT000252181.1':0.00312698,('PDT000252182.1':0.00273393,'PDT000252184.1':0.00285155)'':0.000361734)'':0.0273138,('PDT000010320.3':0.0304916,('PDT000405472.2':0.0391433,('PDT000009733.3':0.0182225,('PDT000110513.3':0.0168226,('PDT000136935.2':0.0118971,('PDT000677071.1':0.0098662,(('PDT000540046.2':0.00628448,('PDT000184464.3':0.00567296,('PDT000342346.2':0.00599356,('PDT000631307.2':0.00388945,'PDT000112429.3':0.00428765)'':0.00126007)'':0.000308804)'':0.00126141)'':0.000239302,('PDT000480437.2':0.0113901,('PDT000548503.2':0.00894411,('PDT000233963.3':0.0100803,('PDT000503041.2':0.00711312,('PDT000136929.2':0.00692225,('PDT000041436.3':0.00515018,('PDT000127996.2':0.00502262,(('PDT000566461.2':0.00378014,('PDT000283353.3':0.00457194,('PDT000213921.3':0.00443363,('PDT000112570.3':0.00220989,('PDT000646100.2':0.00181085,('PDT000359213.2':0.00176743,'PDT000583332.2':0.00192666)'':0.00028677)'':0.000591446)'':0.000790005)'':0.0002294)'':0.000396266)'':-0.000445074,('PDT000265414.3':0.00281803,('PDT000216253.3':0.00336214,(('PDT000112446.3':0.0007835,'PDT000124821.3':0.00111593)'':0.000897825,('PDT000250777.3':0.00163012,('PDT000683642.1':0.00131112,('PDT000284932.3':0.00105515,'PDT000683652.1':0.000943254)'':0.000286556)'':0.000470751)'':-5.92745e-05)'':0.00168062)'':0.00022248)'':4.1274e-05)'':0.000714054)'':0.000525683)'':0.000698893)'':0.000567759)'':0.00161461)'':0.000403515)'':0.00179805)'':-0.0036445)'':0.00533426)'':0.00076058)'':0.00214942)'':0.00300762)'':0.0049264)'':0.00595737)'':-0.00581058)'':-0.00930912,('PDT000359734.1':0.0768505,('PDT000468181.2':0.0335219,(('PDT000569571.2':0.0235827,('PDT000298393.3':0.014247,('PDT000342328.2':0.00607519,'PDT000607743.2':0.00733631)'':0.00743154)'':0.00695239)'':-0.000733769,(('PDT000216743.1':0.0280195,(('PDT000257186.3':0.00703247,('PDT000307494.3':0.00525957,('PDT000625117.2':0.00458834,'PDT000010209.3':0.00329275)'':0.00141672)'':0.00153103)'':0.00497774,(('PDT000233644.3':0.0182591,('PDT000259813.3':0.0101753,('PDT000110530.3':0.0105145,(('PDT000162302.3':0.00468435,'PDT000227388.3':0.00537245)'':0.000737956,('PDT000232219.3':0.00501158,'PDT000572760.2':0.00574432)'':0.000702219)'':0.00298081)'':0.00120536)'':0.00471745)'':-0.00372708,(('PDT000574231.2':0.0123271,'PDT000638756.2':0.0109864)'':0.00480271,('PDT000559234.2':0.0163626,('PDT000242117.3':0.0131248,(('PDT000360427.2':0.00729616,'PDT000383999.2':0.00618904)'':0.00244243,(('PDT000557848.2':0.00960937,'PDT000574153.2':0.00847133)'':0.00167817,(('PDT000604729.2':0.00597469,'PDT000645173.2':0.00587551)'':0.00261391,('PDT000076989.3':0.00797934,('PDT000366337.2':0.00935925,('PDT000289041.3':0.00987998,((('PDT000318452.2':0.00476719,'PDT000084140.3':0.00498794)'':0.000840347,('PDT000035527.1':0.00329294,'PDT000005061.2':0.00289345)'':0.00386267)'':0.000978386,('PDT000260732.3':0.0123234,(('PDT000387618.2':0.00488461,('PDT000234153.3':0.00340294,'PDT000557852.2':0.0025809)'':0.00128865)'':0.00202431,(('PDT000231295.3':0.00447151,('PDT000409247.2':0.00379288,('PDT000384170.2':0.00426501,'PDT000568950.2':0.00410903)'':0.000551827)'':0.000308404)'':0.00115263,('PDT000319279.2':0.00638122,('PDT000303781.3':0.00498352,(('PDT000176834.3':0.00438032,'PDT000692191.1':0.00408835)'':0.000786212,(('PDT000212472.3':0.00160577,'PDT000547693.2':0.00168929)'':0.00198422,('PDT000636823.2':0.00259751,('PDT000702892.1':0.00246363,('PDT000683477.1':0.00310559,('PDT000106365.3':0.00134528,'PDT000136710.3':0.00135121)'':0.00108721)'':0.000265566)'':0.000304481)'':0.000703079)'':0.000730771)'':0.00143319)'':0.000293405)'':-0.0008694)'':0.000326535)'':0.00234063)'':-0.000933754)'':0.000960141)'':0.000436278)'':0.000712615)'':-0.000596759)'':0.000580603)'':0.000224678)'':0.0018065)'':0.000432335)'':0.00413756)'':-0.00380095)'':0.00134529)'':0.00846214)'':-0.00791741,(((('PDT000017115.1':0.00540128,'PDT000017134.1':0.00554402)'':0.00535517,('PDT000017123.1':0.00775169,'PDT000017126.1':0.00726871)'':0.0054896)'':0.00556518,('PDT000010845.3':0.0158044,(('PDT000399259.2':0.0102468,'PDT000017111.1':0.0108632)'':0.00419641,(('PDT000017132.2':0.00554304,('PDT000017119.1':0.00689735,('PDT000017120.1':0.00399878,(('PDT000017113.1':0.0033327,'PDT000017127.1':0.00354529)'':5.37206e-05,('PDT000017108.1':0.00312464,('PDT000017109.1':0.00286638,('PDT000017122.1':0.00236383,'PDT000017135.1':0.00282372)'':0.000224151)'':0.000359393)'':0.000337266)'':0.000152125)'':0.00108128)'':0.000808696)'':0.000504394,('PDT000017128.1':0.0105343,('PDT000017133.1':0.0109003,('PDT000017131.1':0.00896168,('PDT000017129.1':0.00544513,('PDT000017130.1':0.00485124,('PDT000017112.1':0.00408305,(('PDT000017114.1':0.00364444,'PDT000017118.1':0.00323355)'':0.000519447,('PDT000017116.1':0.00316387,'PDT000017117.1':0.00361376)'':0.000375468)'':0.000853053)'':0.000284678)'':0.000244579)'':0.00115311)'':0.000684198)'':0.00126072)'':-0.00179753)'':0.00463605)'':0.00637922)'':-0.00622071)'':0.00153414,(('PDT000242247.3':0.018083,('PDT000325462.2':0.0158356,('PDT000645174.2':0.0088671,('PDT000247379.3':0.00850383,'PDT000298520.3':0.00835787)'':0.0013828)'':0.00246467)'':0.00143043)'':-0.000192178,(('PDT000359730.1':0.00847042,('PDT000359732.1':0.00869272,'PDT000359735.1':0.00846318)'':0.000255425)'':0.00822182,(('PDT000275624.3':0.0108867,('PDT000157532.3':0.0075948,('PDT000173619.3':0.00688553,('PDT000383963.2':0.00680913,'PDT000060815.3':0.00691267)'':0.000361919)'':0.00104358)'':0.00451255)'':0.000751186,(('PDT000188064.3':0.0107294,'PDT000480913.2':0.00924659)'':0.00910044,(('PDT000327919.2':0.0106907,('PDT000587575.2':0.00965597,('PDT000595046.2':0.00571484,'PDT000617232.2':0.00641966)'':0.00388948)'':0.002085)'':0.00107287,('PDT000662362.1':0.0195036,((('PDT000270097.3':0.00469486,('PDT000547723.2':0.00393086,'PDT000638774.2':0.00433937)'':0.00170108)'':0.00366897,('PDT000092465.3':0.0103871,('PDT000458996.2':0.00709183,('PDT000222093.3':0.00695699,('PDT000321933.2':0.00573262,('PDT000226427.3':0.00631824,('PDT000145347.3':0.00625077,'PDT000238322.3':0.00412123)'':0.00112657)'':0.000586919)'':0.000581221)'':0.000584283)'':0.0023994)'':-0.00112575)'':0.000477183,('PDT000025364.1':0.0174031,('PDT000147762.3':0.0175099,(('PDT000342320.2':0.014304,('PDT000499537.2':0.0180566,(('PDT000636618.2':0.00759692,'PDT000119934.3':0.00701978)'':0.000860329,('PDT000010901.2':0.0118493,('PDT000010902.2':0.0132792,('PDT000260734.3':0.0141884,('PDT000394090.2':0.00695782,(('PDT000009734.3':0.00359545,'PDT000017926.3':0.00368344)'':0.00207967,('PDT000198182.3':0.00794033,(('PDT000146058.3':0.00579184,'PDT000250874.3':0.00524966)'':0.000493284,('PDT000237158.3':0.00548591,('PDT000221621.3':0.0063668,(('PDT000061292.3':0.00575818,('PDT000197028.3':0.00474871,('PDT000263912.5':0.00332444,('PDT000258477.3':0.00185263,'PDT000107863.3':0.00224)'':0.000959518)'':0.000741107)'':0.00104363)'':-0.000753014,('PDT000333618.2':0.00309677,('PDT000573246.2':0.002758,('PDT000324582.2':0.00180801,('PDT000495510.2':0.0015248,'PDT000715510.1':0.00137172)'':8.84809e-05)'':0.000967999)'':0.00042776)'':0.00113849)'':0.00132562)'':0.000160756)'':-0.000169272)'':0.000839016)'':-0.000491785)'':0.00143284)'':0.00247313)'':0.000534155)'':0.000982047)'':-0.00242897)'':0.005824)'':0.000416668)'':-0.00607561,(('PDT000162757.1':0.0125158,('PDT000430816.2':0.00990586,'PDT000569570.2':0.0106604)'':0.00144977)'':0.000635672,(('PDT000209470.1':0.00606028,('PDT000509774.2':0.00319211,'PDT000435649.1':0.00390151)'':0.00238601)'':0.00324696,(('PDT000359728.1':0.00769732,('PDT000359731.1':0.00551115,'PDT000359733.1':0.00572815)'':0.00222658)'':0.00178003,(('PDT000409205.2':0.0103198,'PDT000628116.2':0.0130442)'':0.00443001,(('PDT000009726.3':0.00586636,('PDT000234222.3':0.0050373,('PDT000598481.2':0.00359348,('PDT000386177.2':0.00225229,'PDT000587629.2':0.00274172)'':0.000591374)'':0.00222471)'':0.000928796)'':0.00166541,('PDT000532225.2':0.0153536,('PDT000283225.3':0.012697,(('PDT000096630.3':0.00890955,('PDT000556168.2':0.00793654,('PDT000263913.3':0.00406152,'PDT000564145.2':0.00361775)'':0.00406147)'':0.0013384)'':0.00103111,(('PDT000366210.2':0.00736823,'PDT000429532.2':0.00695447)'':0.000948368,('PDT000106363.3':0.0200313,('PDT000679706.1':0.0148236,('PDT000061191.3':0.0117781,('PDT000199189.3':0.00926844,('PDT000110455.3':0.00919481,('PDT000039651.1':0.010922,(('PDT000162756.1':0.00633679,('PDT000162755.1':0.00613215,('PDT000162762.1':0.00695726,('PDT000162754.1':0.00435515,('PDT000162761.1':0.00450201,('PDT000046096.1':0.00246465,('PDT000106401.2':0.000673699,('PDT000435652.1':0.000476846,('PDT000629255.2':0.000512636,'PDT000435654.1':0.000486964)'':0.000123014)'':0.000184766)'':0.00136337)'':0.00135176)'':0.000676677)'':0.00120759)'':0.000246924)'':0.00086017)'':-0.000444935,(('PDT000102886.3':0.00554783,('PDT000039522.3':0.00228811,'PDT000551636.2':0.0026026)'':0.00231039)'':0.00254315,(('PDT000263900.3':0.00533854,'PDT000480345.2':0.00581236)'':0.0014288,('PDT000692197.1':0.00736028,((('PDT000471120.2':0.00119469,'PDT000627631.2':0.00110301)'':0.00284653,('PDT000559155.2':0.00317893,'PDT000076950.3':0.00270929)'':0.00118091)'':0.00168374,('PDT000157499.3':0.00987178,('PDT000271307.3':0.00596838,('PDT000642459.2':0.00512873,('PDT000136921.2':0.00687418,(('PDT000409223.2':0.00376323,'PDT000504035.2':0.00361611)'':0.00189168,(('PDT000237105.3':0.00448417,('PDT000569702.2':0.00298354,('PDT000197033.3':0.00201345,'PDT000265142.3':0.00188032)'':0.000411795)'':0.0011206)'':0.000278135,('PDT000285878.3':0.00509333,('PDT000295797.3':0.0079553,(('PDT000407554.2':0.00534203,('PDT000101913.3':0.00384742,(('PDT000344351.2':0.00290969,'PDT000421960.2':0.00247899)'':0.000662647,('PDT000351798.2':0.00328145,('PDT000209192.3':0.00300575,'PDT000313247.3':0.0025881)'':0.000605115)'':0.000228475)'':0.000749892)'':0.00021051)'':-0.000272879,('PDT000063174.3':0.00523563,('PDT000503810.2':0.00481749,(('PDT000367784.2':0.00414882,('PDT000359225.2':0.003515,('PDT000041402.3':0.00219554,'PDT000259789.3':0.00209588)'':0.00117519)'':0.000745634)'':-0.000479371,(('PDT000273059.3':0.00397574,'PDT000410932.2':0.00399715)'':0.00109917,('PDT000484012.2':0.00465682,('PDT000190140.3':0.00490806,(('PDT000385878.2':0.00245167,('PDT000257969.3':0.00216796,('PDT000230067.3':0.00164721,('PDT000232773.3':0.00142527,'PDT000471136.2':0.00176962)'':0.000350488)'':0.000310894)'':0.000452475)'':0.000220192,('PDT000551259.2':0.00592286,('PDT000130425.3':0.00376248,((('PDT000553347.2':0.00205272,('PDT000471113.2':0.00142509,'PDT000107848.3':0.00157102)'':0.000641874)'':0.000143133,('PDT000321908.2':0.00208388,('PDT000387704.2':0.00197817,('PDT000171571.3':0.001748,'PDT000384115.2':0.00164792)'':0.000117607)'':0.000438184)'':2.79034e-06)'':0.000471287,(('PDT000624215.2':0.00337756,('PDT000163285.4':0.00217265,'PDT000478266.2':0.00182016)'':0.00081637)'':9.18023e-05,('PDT000336987.2':0.00263069,(('PDT000202812.3':0.00197395,('PDT000621257.2':0.0017826,('PDT000242654.3':0.00142405,('PDT000510651.2':0.00108305,'PDT000520028.2':0.00131455)'':0.00017393)'':0.000290531)'':0.000514577)'':-0.000229049,(('PDT000273386.3':0.00105941,'PDT000351822.2':0.00113927)'':0.000437222,('PDT000336933.2':0.00167988,('PDT000281763.3':0.00147685,('PDT000312074.3':0.00119066,'PDT000683658.1':0.00170615)'':0.000121507)'':4.33234e-05)'':2.76069e-05)'':0.000283814)'':0.00122326)'':-0.000664578)'':2.59017e-07)'':0.00117562)'':0.00021449)'':-0.00105998)'':0.00166232)'':0.000266045)'':-0.00076929)'':-7.18566e-05)'':0.00121941)'':0.000220665)'':-0.00164611)'':0.00174603)'':0.000207802)'':-0.00146666)'':0.00093587)'':0.0011905)'':9.42619e-05)'':0.000145863)'':0.000311675)'':-0.00183494)'':0.00303099)'':-0.00171367)'':3.9408e-05)'':-0.000607769)'':0.00254212)'':0.0003342)'':0.000713194)'':0.000338543)'':0.000841975)'':0.000637666)'':-0.00245803)'':-0.000373436)'':0.00391116)'':0.000470285)'':-0.00579952)'':0.00396334)'':-0.00087161)'':0.000853649)'':0.00231776)'':-0.00470723)'':0.00521979)'':0.00506649)'':-0.00950248)'':0.0109122)'':-0.00894578)'':0.00375498)'':-0.00361954)'':0.00657452)'':-0.00276021)'':-0.00322311)'':-0.000503196)'':0.00581912)'':0.0195215)'':0.0252108)'':-0.0479455)'':0.0024785)'':0.00226541)'':0.0247813)'':0.0881148)'':-0.109052)'':0.00492732)'':0.00678179)'':-0.000689079)'':0.000558669)'':0.00375845)'':0.0176278)'':0.00774287)'':0.0330526)'':0.124184)'':0.00791627)'':-0.0141564)'':0.0154477)'':0.0104095)'':0.019189)'':0.0121198,(((('PDT000243049.1':0.168598,('PDT000346100.1':0.145537,('PDT000661334.1':0.125619,('PDT000537536.1':0.0797373,('PDT000589380.1':0.0600442,'PDT000642304.1':0.0684688)'':0.00527718)'':0.0540967)'':0.020294)'':0.018129)'':-0.00217111,('PDT000491100.1':0.166593,(('PDT000621062.1':0.0854656,'PDT000275561.1':0.0699504)'':0.0739604,('PDT000551133.1':0.15842,(('PDT000537526.1':0.0875598,'PDT000463706.1':0.0817602)'':0.0409906,('PDT000048137.1':0.082995,('PDT000412044.1':0.0703197,(('PDT000372318.1':0.0206144,'PDT000589381.1':0.0220201)'':0.0209913,('PDT000243035.1':0.0520994,('PDT000045600.1':0.0378683,('PDT000617794.1':0.0324583,'PDT000710417.1':0.0286966)'':0.00834998)'':0.0101302)'':0.00191296)'':0.0141706)'':0.00263161)'':0.0508534)'':0.0200967)'':0.0182644)'':0.00741279)'':-0.00161824)'':0.0146376,((('PDT000471008.1':0.0391572,'PDT000463636.1':0.0535471)'':0.11865,(('PDT000557016.1':0.0791709,'PDT000421033.2':0.0920751)'':0.00397675,('PDT000346571.1':0.0720359,'PDT000463677.1':0.0693331)'':0.0347987)'':0.0668401)'':-0.000566654,(((('PDT000291162.2':0.0801286,'PDT000396797.1':0.0662404)'':0.0279721,('PDT000627350.1':0.0746226,'PDT000250165.1':0.0630084)'':0.0380934)'':0.0210438,(('PDT000535364.1':0.0523314,('PDT000651114.1':0.023016,'PDT000497204.1':0.03301)'':0.0197145)'':0.0691373,((('PDT000202173.2':0.0618867,'PDT000364523.1':0.0616063)'':0.0207914,('PDT000095674.2':0.054182,'PDT000509500.1':0.074245)'':0.0207153)'':0.0169643,(('PDT000397716.1':0.0653312,('PDT000537857.1':0.0705823,('PDT000204895.2':0.0364567,'PDT000423338.1':0.0338052)'':0.0169562)'':0.0140539)'':0.0179674,('PDT000621012.1':0.0941578,('PDT000491110.1':0.0692531,('PDT000109481.2':0.0358189,('PDT000228371.1':0.0264049,'PDT000249900.1':0.0256302)'':0.0127458)'':0.0340324)'':0.0177153)'':-0.00221554)'':0.00929008)'':0.00867127)'':0.0141921)'':0.0246979,(('PDT000294640.2':0.15902,'PDT000228168.1':0.150207)'':0.00100626,(('PDT000203408.2':0.0611636,('PDT000468322.1':0.0600196,('PDT000395231.1':0.042629,('PDT000627428.1':0.0297256,('PDT000149349.2':0.0286672,'PDT000566058.1':0.0287131)'':0.00272724)'':0.0109816)'':0.00792983)'':0.00671856)'':0.055275,(('PDT000444816.1':0.0461066,('PDT000557076.1':0.0445879,(('PDT000158921.2':0.0276767,('PDT000114756.2':0.0215489,'PDT000691348.1':0.0210731)'':0.00910895)'':0.00685854,(('PDT000162925.4':0.00794405,'PDT000010346.3':0.00892935)'':0.0176494,('PDT000557081.1':0.019342,'PDT000691338.1':0.0231903)'':0.0103191)'':0.0091377)'':0.0021934)'':0.0109107)'':0.0566421,(('PDT000426803.1':0.12579,(('PDT000325577.1':0.0755,'PDT000720200.1':0.078303)'':0.00587123,('PDT000396833.1':0.0655778,('PDT000129813.3':0.0526498,('PDT000400011.1':0.0243735,'PDT000491104.1':0.0254695)'':0.0188925)'':0.025648)'':0.00084053)'':0.0419054)'':-0.0316428,(('PDT000239348.1':0.0722155,(('PDT000690186.1':0.0625076,'PDT000502283.1':0.0507074)'':0.00115283,('PDT000501645.2':0.0411446,('PDT000162889.4':0.0154936,'PDT000045679.1':0.0144955)'':0.029726)'':0.00783466)'':0.0152122)'':-0.0104353,('PDT000487976.2':0.0586763,('PDT000583400.1':0.0457047,('PDT000050577.1':0.0345898,(('PDT000101050.3':0.0180113,'PDT000045595.1':0.0172076)'':0.0141463,('PDT000396806.2':0.0250674,('PDT000690259.1':0.0121461,('PDT000630451.2':0.0142557,'PDT000639746.2':0.00934284)'':0.00316849)'':0.00791064)'':0.00614187)'':0.0065922)'':0.00238355)'':0.00985999)'':-5.28053e-05)'':0.0468424)'':0.00945878)'':0.00317285)'':0.0353723)'':0.00956704)'':0.0147491)'':0.0161009)'':0.0755907,(((('PDT000104575.2':0.194415,(((('PDT000380912.1':0.0384018,(('PDT000319799.1':0.0190607,'PDT000465227.1':0.0167246)'':0.00786099,('PDT000289989.1':0.0217602,('PDT000601103.1':0.027712,(('PDT000191775.3':0.0197664,('PDT000190769.2':0.0114893,'PDT000380905.1':0.00904086)'':0.00504417)'':0.00307106,('PDT000500781.1':0.0136128,(('PDT000343895.1':0.00671302,'PDT000646135.1':0.00689718)'':0.00121995,('PDT000350564.1':0.00545495,'PDT000410855.1':0.00648655)'':0.00214557)'':0.0066728)'':0.00214851)'':0.00525331)'':0.00153302)'':-0.00228487)'':0.0147567)'':0.035258,((('PDT000289982.1':0.0189468,('PDT000191762.2':0.00805391,'PDT000465213.1':0.00947769)'':0.0104496)'':-0.00157584,(('PDT000190756.2':0.012032,'PDT000191774.2':0.00762283)'':0.00666193,(('PDT000319788.1':0.00761937,'PDT000190776.2':0.00825523)'':0.00176956,('PDT000414563.1':0.0075881,('PDT000503128.1':0.00626325,'PDT000627180.1':0.00754565)'':0.000883498)'':0.00273542)'':0.00773231)'':0.00466275)'':0.0466476,(('PDT000292888.1':0.0479479,('PDT000097357.2':0.0106082,'PDT000292890.1':0.0142807)'':0.0266062)'':0.0224272,((('PDT000278507.2':0.0234902,'PDT000289990.1':0.0290952)'':0.00927155,('PDT000190780.2':0.0407021,('PDT000190937.2':0.0278737,'PDT000380907.1':0.0234084)'':0.00882457)'':0.00512448)'':0.0152948,((('PDT000289958.1':0.0221568,('PDT000679705.1':0.00985945,'PDT000289986.1':0.00824635)'':0.00602111)'':0.011497,('PDT000173325.1':0.0270253,(('PDT000318716.1':0.00826013,'PDT000158761.2':0.00757177)'':0.0116742,('PDT000191759.2':0.0129201,'PDT000097369.2':0.0120774)'':0.00288362)'':0.0112732)'':0.00412535)'':0.00942703,((('PDT000009767.2':0.0253783,'PDT000289979.1':0.0218072)'':0.0124304,(('PDT000097368.2':0.0215692,('PDT000278480.2':0.0221694,'PDT000289985.1':0.0242825)'':0.00213433)'':0.00912193,(('PDT000176395.2':0.0268599,('PDT000190760.2':0.0144995,'PDT000641417.1':0.0123254)'':0.0134004)'':0.000251536,(('PDT000190770.2':0.0084159,'PDT000191773.2':0.008345)'':0.0181355,(('PDT000702568.1':0.0118327,('PDT000191758.2':0.00981943,'PDT000415202.1':0.011299)'':0.00100648)'':0.00417026,('PDT000601085.1':0.0123622,('PDT000191765.2':0.0129501,('PDT000191763.2':0.00659499,('PDT000628412.1':0.00374011,'PDT000380915.1':0.00324662)'':0.00407229)'':0.00277735)'':0.00165468)'':0.00372278)'':0.0123357)'':-0.00030669)'':0.00672987)'':0.00634539)'':-0.00642486,(('PDT000176401.2':0.0327632,(('PDT000325141.1':0.0118769,'PDT000540398.1':0.0128389)'':0.00865432,('PDT000518577.1':0.0234586,('PDT000619117.1':0.0128547,'PDT000148573.1':0.0133405)'':0.00712035)'':0.00091655)'':0.0101024)'':-0.00220208,(('PDT000610557.1':0.0161784,('PDT000149826.1':0.0100955,'PDT000150556.1':0.0102131)'':0.00718441)'':0.0013196,('PDT000417571.1':0.0305245,(('PDT000190944.2':0.0173815,('PDT000398523.1':0.0075175,'PDT000349623.1':0.0086724)'':0.00637837)'':0.00027885,(('PDT000190497.2':0.0099908,'PDT000206890.2':0.008419)'':0.00247972,(('PDT000645792.1':0.00801479,'PDT000714270.1':0.00879441)'':0.00416341,('PDT000149824.1':0.0101571,('PDT000574301.1':0.00678577,('PDT000190777.2':0.00343525,'PDT000603846.1':0.00413749)'':0.00126266)'':0.00313202)'':0.00286341)'':0.00329121)'':0.0012787)'':0.00675503)'':-0.00393494)'':0.0124501)'':0.00612786)'':0.00781939)'':0.0146846)'':0.00754858)'':-0.00181003)'':0.00113668)'':0.0460735,(('PDT000344016.1':0.0395029,'PDT000289987.1':0.0490378)'':0.081303,(((('PDT000289981.1':0.0082425,'PDT000354225.1':0.0080629)'':0.021065,('PDT000380922.1':0.0268754,('PDT000173312.1':0.0274238,('PDT000380918.1':0.0111423,('PDT000190784.2':0.00978111,('PDT000380946.1':0.0067307,'PDT000380968.1':0.0089223)'':0.00311719)'':0.00138795)'':0.0102589)'':0.00491242)'':0.00603075)'':0.000728482,(('PDT000481988.1':0.0173585,'PDT000509788.1':0.0178764)'':0.0103321,(('PDT000047001.1':0.0158108,'PDT000395684.1':0.0194926)'':0.00379851,(('PDT000389687.1':0.00497352,'PDT000424706.1':0.00538088)'':0.0109637,('PDT000316513.1':0.00975065,('PDT000190990.2':0.0070227,'PDT000333934.1':0.0080641)'':0.00598065)'':6.74501e-05)'':0.00532766)'':0.00383804)'':0.0121053)'':0.0159215,(('PDT000097356.2':0.0418077,'PDT000289996.1':0.0380104)'':0.0239577,((('PDT000190935.2':0.0146447,'PDT000380925.1':0.0180582)'':0.0119745,('PDT000289954.1':0.0205882,('PDT000191738.2':0.00970381,'PDT000316939.1':0.0127324)'':0.0140553)'':0.00617886)'':0.010522,((('PDT000005018.2':0.0314123,('PDT000289995.1':0.016797,('PDT000271654.2':0.0094936,'PDT000097358.2':0.0123556)'':0.00565377)'':0.016428)'':0.00253114,((('PDT000485339.1':0.031463,('PDT000457979.1':0.0238044,('PDT000148037.2':0.0138717,'PDT000190773.2':0.0140022)'':0.00486541)'':0.0105756)'':-0.00135732,(('PDT000520396.1':0.0222191,('PDT000494135.1':0.013816,('PDT000047002.1':0.00754382,'PDT000259499.1':0.0112081)'':0.00431994)'':0.00814565)'':0.00353594,('PDT000377956.1':0.0270164,('PDT000610753.1':0.0101059,(('PDT000156913.2':0.00610239,'PDT000325203.1':0.00682391)'':0.00216557,('PDT000259493.1':0.00339499,'PDT000259496.1':0.00427968)'':0.00616699)'':0.00474753)'':0.010085)'':0.000611299)'':0.00460521)'':0.00196528,(('PDT000289963.1':0.0139697,'PDT000289997.1':0.0191527)'':0.0106897,((('PDT000173313.1':0.0123518,('PDT000147716.2':0.00775869,'PDT000191736.2':0.00928341)'':0.00515024)'':0.000926564,(('PDT000479293.1':0.00807135,'PDT000559303.1':0.00912715)'':0.00492897,('PDT000544262.1':0.00992933,'PDT000599127.1':0.0114505)'':0.00357928)'':0.00112177)'':0.0097491,(('PDT000190934.2':0.010901,'PDT000456853.1':0.0124267)'':0.0079156,(('PDT000191735.3':0.0137345,'PDT000292853.1':0.0190831)'':0.00579382,(('PDT000322769.1':0.00860177,'PDT000047003.1':0.0106828)'':0.00104414,('PDT000155538.2':0.00490612,'PDT000191731.2':0.00476294)'':0.00375433)'':0.00773254)'':0.00232332)'':0.00534127)'':0.00707314)'':-0.00267527)'':0.00435322)'':-0.00257282,((('PDT000328051.1':0.0263451,('PDT000191740.2':0.0195632,'PDT000207483.2':0.0262398)'':0.00845396)'':0.00276759,('PDT000311051.2':0.0245725,('PDT000667876.2':0.0270202,(('PDT000494131.1':0.0228043,('PDT000692380.1':0.0154566,'PDT000702610.1':0.0146454)'':0.00303596)'':-0.000409228,('PDT000156506.2':0.0253684,('PDT000147564.2':0.00842975,('PDT000046999.1':0.00385936,'PDT000145734.1':0.00500606)'':0.00450034)'':0.00574237)'':0.00361293)'':0.00178866)'':0.00553867)'':-0.000286626)'':0.00876627,(('PDT000191746.2':0.0139994,'PDT000577551.1':0.0139358)'':0.0233459,(('PDT000155600.2':0.0203914,'PDT000332180.1':0.0249783)'':0.00703383,(('PDT000388961.1':0.0266453,('PDT000325772.1':0.00920825,(('PDT000523433.1':0.0070073,'PDT000389677.1':0.0065642)'':0.0032512,('PDT000155461.2':0.00907831,('PDT000160545.2':0.00668098,('PDT000317209.1':0.00387359,'PDT000160481.2':0.0047024)'':0.00208742)'':0.000850748)'':0.00263338)'':0.0026672)'':0.00701821)'':0.00372815,('PDT000380930.1':0.0298319,(('PDT000046990.1':0.0145878,('PDT000173314.1':0.0158108,('PDT000380917.1':0.00802799,('PDT000173310.1':0.00799214,'PDT000173318.1':0.00484876)'':0.00374756)'':0.00433788)'':0.000903186)'':-0.000242434,(('PDT000674779.1':0.00923857,('PDT000624277.1':0.00598957,('PDT000517598.1':0.00420397,'PDT000702562.1':0.00406791)'':0.000883037)'':0.00385605)'':0.00410923,('PDT000190995.2':0.0214301,('PDT000221696.2':0.00769148,('PDT000387109.1':0.00728596,('PDT000190759.2':0.00653228,('PDT000316950.1':0.00788149,('PDT000323033.1':0.00412387,('PDT000157711.2':0.00336041,('PDT000221698.2':0.00256812,'PDT000221700.2':0.0026215)'':0.000674666)'':0.000487363)'':0.00212203)'':0.000755875)'':0.000242093)'':0.00228943)'':0.00602064)'':-0.00307133)'':0.00243656)'':0.0169187)'':-0.0122635)'':0.0084456)'':0.00696198)'':-0.00680263)'':0.00482399)'':0.0112848)'':0.0146107)'':-0.00768982)'':0.0837728)'':-0.0234698)'':0.00415036,((('PDT000289960.1':0.0188678,('PDT000191747.2':0.0166345,('PDT000190763.2':0.00419257,'PDT000271656.2':0.00368537)'':0.00708285)'':0.0100052)'':0.00834002,(('PDT000485301.1':0.021372,'PDT000173326.1':0.0171727)'':0.00423113,('PDT000173319.1':0.0273475,(('PDT000520381.1':0.0140468,'PDT000641420.1':0.0129082)'':0.00168215,('PDT000316186.1':0.00793976,('PDT000591771.1':0.0068073,('PDT000155618.2':0.0056671,('PDT000147602.2':0.00491794,('PDT000155443.2':0.00594748,('PDT000206889.2':0.00449077,('PDT000191733.2':0.00244799,'PDT000191748.2':0.0030372)'':0.00147789)'':0.0014852)'':0.000398027)'':0.000756737)'':0.000369283)'':0.00158251)'':0.00442817)'':0.00981411)'':-0.00419832)'':0.0098232)'':0.0446696,((((('PDT000004651.2':0.0268076,('PDT000067313.1':0.0148181,'PDT000008791.2':0.020949)'':0.00486908)'':0.00350557,(('PDT000147623.2':0.0125435,'PDT000292884.1':0.0154735)'':0.00821957,('PDT000190764.2':0.0214953,'PDT000260557.2':0.0133642)'':0.00961516)'':0.00976492)'':0.00230463,((('PDT000481589.1':0.0068983,'PDT000610546.1':0.006479)'':0.0112335,('PDT000147574.2':0.0241112,(('PDT000160307.2':0.00715109,'PDT000191741.2':0.00552231)'':0.0057214,('PDT000602986.1':0.0105281,('PDT000316865.1':0.00625941,('PDT000271655.2':0.00307751,'PDT000420987.1':0.0033129)'':0.00399813)'':0.00250052)'':0.00245688)'':0.0048386)'':-0.000164997)'':0.000945499,('PDT000475741.1':0.0219941,(('PDT000148235.2':0.0137271,('PDT000145810.1':0.00832111,('PDT000318598.1':0.00658138,('PDT000409964.1':0.00352014,'PDT000519406.1':0.00416142)'':0.00346419)'':0.000568245)'':0.0024546)'':0.00212156,('PDT000315523.1':0.0155843,('PDT000046998.1':0.00646365,('PDT000148183.2':0.00268894,('PDT000654630.1':0.00260139,('PDT000157062.2':0.0019971,'PDT000202825.2':0.00209676)'':0.000594575)'':0.000473224)'':0.00275714)'':0.00594081)'':0.0033193)'':0.00380933)'':0.000897933)'':0.0167534)'':0.00778899,(('PDT000009763.2':0.0601693,('PDT000380920.1':0.0353924,(('PDT000317199.1':0.00715888,'PDT000148115.2':0.00835892)'':0.0202721,(('PDT000420983.1':0.0212375,'PDT000047000.1':0.0171519)'':0.00546831,('PDT000692246.1':0.0178805,('PDT000190785.2':0.0219869,('PDT000191752.2':0.0172443,('PDT000557967.1':0.0118279,('PDT000190767.2':0.00987983,('PDT000602664.1':0.00698842,('PDT000191739.2':0.00319515,'PDT000191754.2':0.00348681)'':0.0032522)'':0.00182471)'':0.00309647)'':0.000789123)'':0.00273823)'':0.00640588)'':0.000707764)'':0.00274707)'':0.0121069)'':0.0287826)'':-0.0309754,(('PDT000289952.1':0.00940888,'PDT000380932.1':0.00845002)'':0.0356457,((('PDT000046994.1':0.0233597,('PDT000046996.1':0.0112797,('PDT000380933.1':0.00541067,'PDT000380934.1':0.00586003)'':0.00534065)'':0.015102)'':-0.00682895,(('PDT000457970.1':0.0222813,('PDT000327329.1':0.0176487,(('PDT000409970.1':0.00856641,'PDT000161620.2':0.00725919)'':0.00024279,('PDT000173324.1':0.0097493,('PDT000431613.1':0.0073741,('PDT000316476.1':0.00217051,'PDT000155627.2':0.00212091)'':0.00324684)'':0.00124237)'':0.000731322)'':0.00579506)'':0.00293914)'':0.000179572,('PDT000319352.1':0.0428721,('PDT000316189.1':0.0258394,('PDT000414569.1':0.0172872,(('PDT000160428.2':0.00976441,'PDT000442719.1':0.0105769)'':0.00122633,('PDT000320813.1':0.0131457,('PDT000384312.1':0.0109425,('PDT000155316.2':0.00572634,('PDT000190762.2':0.00599533,('PDT000315500.1':0.00185034,('PDT000316575.1':0.00187105,'PDT000155402.2':0.00222322)'':0.000344507)'':0.00278327)'':0.0011863)'':0.00176303)'':0.00302344)'':-0.000229498)'':0.00403566)'':0.00258721)'':0.00926786)'':-0.0109122)'':0.00294505)'':0.00480808,((('PDT000157146.2':0.0207415,'PDT000271637.2':0.0250421)'':0.00499213,(('PDT000340890.1':0.016242,('PDT000258414.2':0.0130035,('PDT000190779.2':0.00707044,'PDT000191749.2':0.00656956)'':0.0058335)'':0.00305456)'':0.00285548,(('PDT000191756.2':0.0117335,'PDT000173316.1':0.0137398)'':0.00986973,('PDT000050495.1':0.0119304,('PDT000190744.2':0.0123404,('PDT000156029.2':0.00651491,('PDT000047005.1':0.00651417,('PDT000271649.2':0.00334674,'PDT000280863.2':0.00374758)'':0.00213013)'':0.00157761)'':0.00152857)'':0.00217199)'':0.00438196)'':0.0010217)'':0.00512712)'':-0.0024248,(('PDT000619457.1':0.0222766,('PDT000323276.1':0.0235108,('PDT000322798.1':0.00988626,'PDT000717637.1':0.0111038)'':0.00736848)'':0.00262085)'':0.00191131,('PDT000420991.1':0.030535,('PDT000376172.1':0.029701,(('PDT000333973.1':0.0107552,'PDT000304591.1':0.0119001)'':0.00312467,(('PDT000191751.2':0.00661808,('PDT000322765.1':0.00355627,'PDT000322930.1':0.00283478)'':0.00182884)'':0.00361698,('PDT000173315.1':0.0126898,(('PDT000155499.2':0.00449291,'PDT000191755.2':0.00427028)'':0.00240109,('PDT000149823.1':0.00895871,'PDT000145736.1':0.00850849)'':0.000606142)'':0.00337393)'':-0.00163604)'':0.00184573)'':0.0103129)'':0.00299643)'':-0.00507233)'':0.000143449)'':0.00252095)'':0.019803)'':-0.0112393)'':0.0129974)'':0.00628954,('PDT000156503.2':0.0739596,(('PDT000176409.2':0.046638,('PDT000321619.1':0.0298472,'PDT000467387.1':0.0316808)'':0.0169486)'':0.000806893,(('PDT000161005.2':0.0107922,'PDT000161043.2':0.00933668)'':0.0314449,(('PDT000271636.2':0.0553659,(('PDT000271645.2':0.0238269,'PDT000271642.2':0.0173334)'':0.0236078,('PDT000403485.1':0.0457015,(('PDT000379553.1':0.0327789,(('PDT000271634.2':0.00265044,'PDT000271653.2':0.00194084)'':0.0153917,('PDT000271640.2':0.00859115,'PDT000271647.2':0.00902955)'':0.00516566)'':0.0121429)'':-0.003407,(('PDT000254354.2':0.00977854,'PDT000380914.1':0.0108761)'':0.0092901,('PDT000190931.2':0.0257938,('PDT000271633.2':0.0228959,('PDT000634371.1':0.0194346,('PDT000160373.2':0.0121863,('PDT000321169.1':0.00669136,'PDT000156038.2':0.00755164)'':0.00255009)'':0.00522433)'':0.00162572)'':0.00374884)'':-0.000768891)'':0.00481756)'':0.00857388)'':0.000205108)'':0.00645409)'':-0.00528132,(('PDT000077175.2':0.0609747,(('PDT000155455.2':0.0103826,('PDT000157920.2':0.00619858,('PDT000377036.1':0.00396338,'PDT000600098.1':0.00331043)'':0.00303236)'':0.00528007)'':0.0167799,(('PDT000176405.2':0.0272902,('PDT000610675.1':0.0226687,('PDT000319274.1':0.0197372,('PDT000320211.1':0.0169288,(('PDT000418523.1':0.0145846,'PDT000637699.1':0.0108988)'':0.00165133,(('PDT000158713.2':0.00414079,'PDT000327346.1':0.00383528)'':0.00318026,('PDT000610681.1':0.0057948,'PDT000610851.1':0.0086546)'':0.00244088)'':0.00406404)'':0.00423572)'':0.000768431)'':0.00134576)'':0.00715897)'':-0.000409814,(('PDT000610585.1':0.030379,('PDT000669590.1':0.0253403,('PDT000316862.1':0.0166868,'PDT000271629.2':0.0130669)'':0.0051548)'':0.00810437)'':0.000550204,('PDT000316504.1':0.0356287,((('PDT000225469.2':0.0162994,'PDT000271630.2':0.0167381)'':0.00835784,('PDT000289962.1':0.0183782,(('PDT000491607.1':0.0187612,('PDT000173327.1':0.00966675,('PDT000156629.2':0.00962006,('PDT000610768.1':0.00688771,'PDT000610808.1':0.00599679)'':0.00387154)'':0.0020389)'':0.00453044)'':-0.000496109,('PDT000500615.1':0.0200711,('PDT000546427.1':0.0130496,(('PDT000271639.2':0.00155468,'PDT000271644.2':0.00154099)'':0.00368559,('PDT000271641.2':0.00543995,'PDT000271652.2':0.00600855)'':0.00185387)'':0.0053039)'':0.0039383)'':-0.00220949)'':0.0046195)'':-0.000309573)'':-0.00127228,((('PDT000147600.2':0.0121891,('PDT000316516.1':0.00772186,'PDT000332835.1':0.0110616)'':0.00221058)'':0.0032824,(('PDT000645763.1':0.00599071,'PDT000711052.1':0.00675699)'':0.00675747,('PDT000148113.2':0.0151982,('PDT000442724.1':0.012591,('PDT000333985.1':0.00792509,('PDT000442725.1':0.00901452,'PDT000467768.1':0.00698788)'':0.00142871)'':0.00249228)'':0.00164656)'':-0.000755956)'':0.00273404)'':0.00283226,(('PDT000519342.1':0.0154161,('PDT000457503.1':0.017879,('PDT000466210.1':0.00805863,'PDT000546441.1':0.00885507)'':0.00595843)'':0.00173161)'':-0.000986103,('PDT000610532.1':0.0148674,('PDT000481293.1':0.0107973,('PDT000332786.1':0.00610484,('PDT000156902.2':0.00350415,('PDT000327957.1':0.003559,('PDT000160374.2':0.00308568,'PDT000160488.2':0.0028049)'':0.000231119)'':0.000205318)'':0.00210278)'':0.00193749)'':0.00215648)'':0.00218936)'':0.00375589)'':0.00307806)'':0.0175397)'':-0.0119778)'':-0.000656794)'':0.00315338)'':0.0204277)'':-0.0203143,(('PDT000105087.2':0.0377237,('PDT000484859.1':0.0251859,('PDT000156791.2':0.011565,('PDT000155929.2':0.00674147,'PDT000323696.1':0.00707013)'':0.00617934)'':0.0104598)'':0.0100555)'':-0.00199203,(('PDT000322803.1':0.0247445,('PDT000341797.1':0.0186735,'PDT000377990.1':0.0198145)'':0.00453848)'':0.00289149,(('PDT000711061.1':0.0279048,('PDT000353560.1':0.0151373,('PDT000318612.1':0.0099316,('PDT000320922.1':0.00938258,'PDT000325701.1':0.00890402)'':0.0011177)'':0.00415024)'':0.00658551)'':0.00157002,(('PDT000316151.1':0.021281,'PDT000157192.2':0.0222995)'':0.00250351,((('PDT000191790.2':0.0135592,'PDT000387027.1':0.0161487)'':0.00180262,('PDT000320261.1':0.0145011,(('PDT000155493.2':0.0111306,('PDT000148258.2':0.00558756,'PDT000332852.1':0.00568204)'':0.00468468)'':0.00062417,('PDT000156640.2':0.0116823,('PDT000589280.1':0.00805631,('PDT000396208.1':0.00449944,'PDT000398034.1':0.00547164)'':0.0058968)'':0.00242042)'':0.00074309)'':0.00221609)'':0.00160779)'':0.00153767,(('PDT000491621.1':0.01373,'PDT000520404.1':0.0152784)'':0.0140953,(('PDT000160504.2':0.0145527,'PDT000484860.1':0.0134007)'':0.00860628,((('PDT000542760.1':0.013297,'PDT000635727.1':0.0120094)'':0.00336768,(('PDT000160419.2':0.009504,'PDT000380913.1':0.008714)'':0.00513799,('PDT000467689.1':0.0155311,('PDT000484861.1':0.00833534,('PDT000333293.1':0.00716637,('PDT000325795.1':0.00465873,'PDT000333966.1':0.00590117)'':0.00211603)'':0.00187905)'':0.00310217)'':-0.000864438)'':0.00165356)'':0.000806504,(('PDT000620627.1':0.0137109,(('PDT000191788.2':0.0118825,'PDT000488335.1':0.00973797)'':0.000737028,('PDT000161619.2':0.00549575,('PDT000148126.2':0.00164218,'PDT000155651.2':0.00155335)'':0.00526934)'':0.00218927)'':0.00655019)'':-0.00414753,('PDT000231017.1':0.0163372,('PDT000221718.2':0.0139051,('PDT000557369.1':0.0118431,('PDT000327235.1':0.0116894,('PDT000379500.1':0.00871131,('PDT000155941.2':0.00825415,('PDT000190933.2':0.00511281,('PDT000317191.1':0.00768121,('PDT000467297.1':0.00508389,(('PDT000327201.1':0.00277306,'PDT000328020.1':0.00241604)'':0.000754128,('PDT000155407.2':0.00348035,('PDT000155575.2':0.00220853,'PDT000319840.1':0.00228159)'':0.000311283)'':0.000404541)'':0.00157074)'':0.000648856)'':0.000670229)'':0.000299799)'':0.000916295)'':0.000877849)'':0.00169695)'':0.00180571)'':0.00512047)'':-0.00451767)'':0.00275731)'':0.0043576)'':0.00194878)'':-0.00224686)'':0.00212645)'':0.00055893)'':0.00167029)'':0.00150494)'':0.00278717)'':0.00895036)'':0.00800705)'':0.00249607)'':0.0215487)'':-0.0164212)'':0.0196058)'':0.0332604)'':0.127059)'':-0.0119078,((((('PDT000080585.1':0.051114,('PDT000080595.1':0.0514025,'PDT000080663.1':0.0393313)'':0.00673979)'':0.000919611,('PDT000080791.1':0.0426712,('PDT000080587.1':0.0262754,'PDT000080631.1':0.029401)'':0.0157609)'':0.00132282)'':0.0218169,(('PDT000161070.2':0.0582647,('PDT000595819.1':0.0263009,'PDT000080993.1':0.0289784)'':0.0279777)'':0.00176499,(('PDT000467789.1':0.0354064,'PDT000080917.1':0.0397627)'':0.00371272,(('PDT000324653.1':0.0213632,('PDT000610755.1':0.0173336,'PDT000346313.1':0.0151642)'':0.00541495)'':0.0164956,('PDT000139833.1':0.022768,('PDT000080921.1':0.0142217,'PDT000080956.1':0.0176323)'':0.00945303)'':0.00963423)'':0.00571207)'':0.0173892)'':0.00709165)'':0.00520757,(((('PDT000080882.1':0.0547881,('PDT000080968.1':0.0323658,(('PDT000260637.2':0.0160483,('PDT000260628.2':0.0153805,'PDT000260644.2':0.0156388)'':0.0040305)'':0.00927851,('PDT000080930.1':0.0268047,(('PDT000161008.2':0.0164057,'PDT000161078.2':0.0176436)'':0.00184785,('PDT000161019.2':0.0123865,'PDT000161022.2':0.0130767)'':0.0069252)'':0.00398283)'':0.00185811)'':0.00578953)'':0.0200323)'':-0.0159277,(('PDT000260639.2':0.0198615,'PDT000598662.1':0.0207506)'':0.00326665,(('PDT000380959.1':0.0196423,('PDT000380949.1':0.0189554,'PDT000380960.1':0.015069)'':0.00632353)'':0.00607647,('PDT000260280.2':0.0264894,(('PDT000253349.2':0.0131452,'PDT000260246.2':0.0146134)'':0.00078276,('PDT000388475.1':0.017155,('PDT000260263.2':0.0128627,'PDT000260625.2':0.00997736)'':0.00301858)'':0.000343329)'':0.00807575)'':0.00156327)'':0.00391558)'':0.00843999)'':0.000859976,((('PDT000080890.1':0.0524343,('PDT000161028.2':0.020804,'PDT000161050.2':0.017945)'':0.0131373)'':-0.000637362,(('PDT000078626.1':0.018745,'PDT000080845.1':0.0264283)'':0.00231002,('PDT000526196.1':0.0178283,('PDT000009765.2':0.00502233,'PDT000009764.2':0.00701867)'':0.0152224)'':0.00473302)'':0.00481671)'':0.00159981,((('PDT000078590.1':0.032921,('PDT000260624.2':0.0289217,'PDT000260631.2':0.0348977)'':0.00319484)'':0.00105346,(('PDT000253351.2':0.0170719,'PDT000380973.1':0.012424)'':0.0145252,('PDT000260293.2':0.026806,('PDT000080961.1':0.017724,'PDT000357562.1':0.0176916)'':0.00465848)'':0.00580712)'':0.00491604)'':0.00134459,('PDT000080852.1':0.0427655,(('PDT000346312.1':0.0232329,('PDT000157172.2':0.0152329,'PDT000467378.1':0.0172481)'':0.00289065)'':-7.82027e-05,('PDT000357268.1':0.016775,('PDT000245850.2':0.0191397,('PDT000237808.2':0.0130066,('PDT000237811.2':0.00258827,'PDT000258391.2':0.00339617)'':0.00563199)'':0.00756541)'':0.00148012)'':0.00177581)'':0.0175955)'':-0.00477135)'':0.00362736)'':0.000896373)'':0.0101303,((('PDT000130933.2':0.0540829,('PDT000598666.1':0.0448856,('PDT000423296.1':0.0250852,('PDT000318872.1':0.0206539,'PDT000080828.1':0.0193304)'':0.0043803)'':0.0133209)'':0.00214324)'':-0.0011023,(('PDT000649379.1':0.0667739,(('PDT000327178.1':0.0270925,'PDT000422215.1':0.0316161)'':0.00031013,('PDT000316515.1':0.0248952,(('PDT000519323.1':0.0201742,'PDT000491616.1':0.0167641)'':0.00073276,('PDT000318610.1':0.013165,('PDT000357297.1':0.0113121,('PDT000646320.1':0.00713519,'PDT000646385.1':0.00628161)'':0.00218047)'':0.00268955)'':0.00534366)'':0.00271514)'':0.00702721)'':0.0161965)'':-0.0104431,('PDT000186091.1':0.0510838,(('PDT000080786.1':0.0355268,('PDT000078663.1':0.0103234,'PDT000080753.1':0.0163433)'':0.0152958)'':0.00596575,((('PDT000311681.2':0.0176744,('PDT000130762.2':0.0151477,'PDT000540388.1':0.0143605)'':0.0071439)'':0.00898331,('PDT000628756.1':0.0274325,('PDT000185968.1':0.0211724,('PDT000185901.1':0.018055,'PDT000186000.1':0.0162713)'':0.00101883)'':0.0123134)'':0.00385928)'':5.58584e-05,(('PDT000185942.1':0.03724,('PDT000548860.1':0.0148193,('PDT000185962.1':0.00691716,'PDT000185981.1':0.0102762)'':0.00521552)'':0.0188047)'':-0.000616939,('PDT000080738.1':0.0516277,(('PDT000318581.1':0.0286542,('PDT000008483.2':0.0142968,'PDT000185876.1':0.0168108)'':0.00929025)'':0.00030426,(('PDT000708800.1':0.0127934,'PDT000711112.1':0.0138838)'':0.00967636,((('PDT000327232.1':0.0138631,'PDT000155190.2':0.0131719)'':0.00213137,('PDT000185889.1':0.0135614,('PDT000080947.1':0.0127471,('PDT000155250.2':0.0120257,('PDT000185976.1':0.00838179,'PDT000186044.1':0.00928571)'':0.00268629)'':0.000432403)'':0.00127128)'':0.00219065)'':0.00262254,('PDT000689847.1':0.0199392,(('PDT000321565.1':0.0160046,('PDT000389381.1':0.0133212,'PDT000621314.1':0.0115371)'':0.00354758)'':0.00027535,(('PDT000628751.1':0.0105874,'PDT000080942.1':0.00986535)'':0.000633034,('PDT000485317.1':0.00663635,('PDT000160493.2':0.0031221,'PDT000130755.2':0.0027626)'':0.00280625)'':0.00486992)'':0.00340824)'':0.00504954)'':-0.00283939)'':0.00447193)'':-0.00111464)'':0.00961583)'':-0.00460067)'':-0.000186026)'':0.00589077)'':0.0118354)'':-0.0106351)'':0.00849755)'':0.00449537,(('PDT000689081.1':0.0322815,(('PDT000185908.1':0.0307692,('PDT000185921.1':0.0329933,('PDT000186071.1':0.0251738,'PDT000346300.1':0.0220572)'':0.00499256)'':0.00588968)'':0.000653829,('PDT000185878.1':0.0288524,('PDT000185925.1':0.0238836,(('PDT000185935.1':0.00918781,'PDT000186088.1':0.00821819)'':0.00167979,('PDT000186160.1':0.0116677,('PDT000185972.1':0.00626406,'PDT000186145.1':0.00859584)'':0.00389468)'':0.00021095)'':0.00720525)'':0.00995851)'':0.0036753)'':0.00885494)'':0.00627825,((('PDT000176410.2':0.0487352,('PDT000139831.1':0.0427804,('PDT000295732.2':0.0329562,('PDT000080865.1':0.0237306,('PDT000467379.1':0.0202756,'PDT000467740.1':0.014866)'':0.00609913)'':0.00586693)'':0.0061577)'':0.0062487)'':-0.00232033,((('PDT000590349.1':0.0190607,'PDT000717394.1':0.0147304)'':0.011328,(('PDT000467731.1':0.022106,'PDT000467678.1':0.0234644)'':0.00108248,('PDT000623865.1':0.00998227,('PDT000523258.1':0.0115287,'PDT000646373.1':0.0126904)'':0.000947825)'':0.00593848)'':0.00296552)'':0.00533554,('PDT000161029.2':0.0243859,((('PDT000260252.2':0.0202381,'PDT000467571.1':0.0197894)'':0.00368123,('PDT000357561.1':0.0171196,('PDT000467757.1':0.0076908,('PDT000646395.1':0.0101528,('PDT000470594.1':0.00495791,'PDT000628608.1':0.00570899)'':0.0028707)'':0.00147907)'':0.00307974)'':0.00675941)'':0.00209534,(('PDT000467790.1':0.016578,('PDT000467700.1':0.0149762,('PDT000467645.1':0.0133842,('PDT000467606.1':0.00556881,'PDT000467805.1':0.00567499)'':0.00556766)'':0.00160371)'':0.00124327)'':0.00799245,('PDT000491630.1':0.029347,('PDT000467288.1':0.0248276,(('PDT000605097.1':0.0150282,'PDT000081000.1':0.0121988)'':0.00451979,('PDT000721038.1':0.019138,('PDT000321616.1':0.0138983,(('PDT000346331.1':0.0100145,('PDT000610737.1':0.00285512,'PDT000610789.1':0.0028301)'':0.00555907)'':0.00269815,(('PDT000467565.1':0.00846628,'PDT000646388.1':0.00591792)'':0.00426721,('PDT000160367.2':0.0126849,('PDT000467652.1':0.00550821,('PDT000467376.1':0.00337445,'PDT000646631.1':0.00410535)'':0.00115899)'':0.00301548)'':-9.65503e-05)'':0.00143511)'':0.00300795)'':0.00264194)'':-0.000559338)'':0.00479468)'':0.00268422)'':-0.00299653)'':0.000603007)'':0.00223785)'':0.00287929)'':0.00776796)'':-0.00669875,((('PDT000575973.1':0.0201204,'PDT000186101.1':0.0215259)'':0.00173109,('PDT000469930.1':0.0220307,('PDT000634560.1':0.0110599,((('PDT000558530.1':0.00401146,'PDT000637350.1':0.00316486)'':0.0038585,('PDT000186104.1':0.009684,('PDT000189715.2':0.00474546,'PDT000186153.1':0.00572944)'':0.001549)'':0.00212731)'':0.00206514,('PDT000185982.1':0.00509019,('PDT000186163.1':0.00398831,('PDT000186025.1':0.0025017,('PDT000185939.1':0.00173546,'PDT000186050.1':0.00165979)'':0.00178712)'':0.00102381)'':0.000926604)'':0.00365971)'':0.00619597)'':0.00588654)'':-0.00283773)'':0.0113577,((('PDT000260292.2':0.0321289,'PDT000305183.2':0.0290856)'':0.00137959,(('PDT000186043.1':0.0170591,'PDT000186090.1':0.0180407)'':0.00838289,(('PDT000260632.2':0.0197952,'PDT000610795.1':0.0168695)'':0.00722867,(('PDT000186108.1':0.0177373,('PDT000646383.1':0.0149093,'PDT000185917.1':0.0105388)'':0.00408751)'':0.00446696,('PDT000260275.2':0.0253022,('PDT000185986.1':0.0163539,('PDT000185903.1':0.0173807,(('PDT000186166.1':0.0128535,('PDT000185870.1':0.00594796,'PDT000186151.1':0.00748484)'':0.00676214)'':0.000242572,('PDT000186032.1':0.010467,('PDT000185996.1':0.0086087,'PDT000346414.1':0.0078743)'':0.0013223)'':0.00119629)'':0.00176147)'':0.00202189)'':0.00244234)'':0.000873557)'':0.00293507)'':0.00086694)'':0.00415665)'':-0.00177215,('PDT000078648.1':0.0534331,('PDT000080732.1':0.0737749,('PDT000078557.1':0.0466306,('PDT000260266.2':0.0246179,(('PDT000185963.1':0.0263427,('PDT000185933.1':0.0148196,('PDT000185896.1':0.00867922,'PDT000187867.1':0.00824628)'':0.00467722)'':0.00540259)'':0.00387334,(('PDT000080682.1':0.010609,('PDT000080664.1':0.00573424,'PDT000080840.1':0.00691136)'':0.00202004)'':0.0146809,(('PDT000260630.2':0.0141651,'PDT000186048.1':0.0166223)'':0.0068582,((('PDT000185949.1':0.0066108,'PDT000185985.1':0.0115656)'':0.0128661,('PDT000078681.1':0.0203633,('PDT000380963.1':0.0106481,('PDT000185877.1':0.00362227,'PDT000186066.1':0.0056703)'':0.00782935)'':0.00465836)'':2.96091e-05)'':0.00158231,(('PDT000080632.1':0.0208002,('PDT000080974.1':0.0103451,'PDT000185953.1':0.0146252)'':0.00449311)'':0.000784851,(('PDT000157232.2':0.0127133,'PDT000467363.1':0.0111434)'':0.00366447,(('PDT000186136.1':0.00990836,('PDT000186142.1':0.00750973,'PDT000187866.1':0.00467317)'':0.00310454)'':0.00442178,('PDT000186095.1':0.0222894,('PDT000485313.1':0.0122574,('PDT000186100.1':0.0126792,('PDT000080824.1':0.00702417,('PDT000185941.1':0.0082478,('PDT000080652.1':0.00600042,('PDT000080964.1':0.00676874,('PDT000185881.1':0.00215622,'PDT000186040.1':0.00123801)'':0.00356779)'':0.000662757)'':0.00070359)'':0.000638028)'':0.00286221)'':0.000898274)'':0.00221591)'':-0.00105451)'':0.00285145)'':-0.000615386)'':0.000596321)'':0.00147525)'':0.0033093)'':0.000297458)'':0.00602726)'':0.00280894)'':0.00903201)'':0.000588536)'':-0.0173377)'':0.00300049)'':0.00234179)'':0.0143498)'':-0.00466554)'':0.00699463)'':0.0195301)'':-0.010529,('PDT000160823.2':0.187212,(((('PDT000155658.2':0.032265,('PDT000186144.1':0.0194821,('PDT000186134.1':0.0153748,('PDT000185964.1':0.0115659,('PDT000185869.1':0.0106261,'PDT000186036.1':0.00805153)'':0.000692376)'':0.00347802)'':0.00276994)'':0.0253727)'':0.00268366,((('PDT000080646.1':0.0288499,('PDT000687069.1':0.0245035,'PDT000078690.1':0.0246431)'':0.00618185)'':0.003194,('PDT000080909.1':0.0270631,('PDT000078664.1':0.0214889,('PDT000078646.1':0.00777006,'PDT000080618.1':0.0110959)'':0.0105988)'':0.00927413)'':0.00250249)'':0.00484274,(('PDT000148180.2':0.021677,('PDT000646097.1':0.0171523,'PDT000716354.1':0.0135668)'':0.00497873)'':0.00723124,(((('PDT000343938.1':0.0137226,'PDT000682796.1':0.0181428)'':0.000669085,('PDT000571613.1':0.0123226,'PDT000583570.1':0.0106272)'':0.00529999)'':0.00295776,(('PDT000322060.1':0.0175321,'PDT000417280.1':0.0164466)'':0.00205265,('PDT000557364.1':0.0203349,('PDT000664563.2':0.0141935,('PDT000346402.1':0.0102592,('PDT000328121.1':0.00366932,'PDT000635723.1':0.00402222)'':0.00734097)'':0.0028933)'':0.00527837)'':-0.000903093)'':0.00377833)'':0.0020193,('PDT000080834.1':0.0205163,((('PDT000148008.2':0.0127331,'PDT000161927.2':0.0115219)'':0.00194714,('PDT000080614.1':0.00683677,'PDT000080737.1':0.00570683)'':0.00681966)'':0.00200015,('PDT000080923.1':0.0110003,(('PDT000080888.1':0.00704944,'PDT000081013.1':0.00539336)'':0.00418595,('PDT000080972.1':0.0142528,('PDT000080609.1':0.00123293,'PDT000080957.1':0.00116443)'':0.00577638)'':0.001247)'':0.00581222)'':0.00143381)'':0.00769247)'':-0.00280052)'':0.00380338)'':0.00600232)'':0.0127345)'':-0.00881917,(('PDT000080963.1':0.0748684,(('PDT000523423.1':0.0196445,'PDT000674759.1':0.0260122)'':0.0144055,(('PDT000379496.1':0.0289797,'PDT000415193.1':0.0308327)'':0.00483747,('PDT000377157.1':0.0342961,(('PDT000556430.1':0.0228044,('PDT000720397.1':0.0237885,('PDT000324645.1':0.013339,'PDT000479294.1':0.016038)'':0.00469893)'':0.001836)'':0.00333552,(('PDT000369007.1':0.0219232,('PDT000130830.2':0.0196822,('PDT000160359.2':0.0208824,(('PDT000346390.1':0.013655,('PDT000080757.1':0.00809487,'PDT000102759.1':0.00593503)'':0.00489283)'':0.0031718,('PDT000547713.1':0.0164954,('PDT000130804.2':0.0151588,('PDT000355591.1':0.0112189,'PDT000130712.2':0.0103695)'':0.00263247)'':0.00204383)'':-0.000529785)'':0.00227171)'':0.00108484)'':0.00109614)'':-0.00134569,(('PDT000326100.1':0.0167838,('PDT000548985.1':0.0159893,('PDT000334878.1':0.0113501,('PDT000398048.1':0.00988601,'PDT000689101.1':0.00868949)'':0.00322886)'':0.00280956)'':0.00264815)'':-0.000409564,(('PDT000394089.1':0.015118,'PDT000645767.1':0.0154239)'':0.00172628,('PDT000324062.1':0.0160269,('PDT000339386.1':0.0108028,('PDT000157712.2':0.00485285,'PDT000338962.1':0.00541025)'':0.00384188)'':0.00641142)'':0.000718862)'':0.000914375)'':0.000779592)'':0.00354718)'':0.00706737)'':0.00011676)'':0.00656673)'':0.0177301)'':-0.0237106,((('PDT000316579.1':0.0246988,'PDT000467243.1':0.025438)'':0.00116492,('PDT000380948.1':0.0164422,('PDT000345841.1':0.0192318,('PDT000185884.1':0.0155188,'PDT000185991.1':0.00914202)'':0.00472737)'':0.00310334)'':0.0058157)'':0.0054866,('PDT000078740.1':0.0713025,((('PDT000388950.1':0.0189459,(('PDT000157900.2':0.0079972,'PDT000565120.1':0.0093076)'':0.00133685,('PDT000130941.2':0.00834651,'PDT000611907.1':0.0102032)'':0.00118578)'':0.00411964)'':0.00619499,(('PDT000078595.1':0.0165815,'PDT000080823.1':0.0237795)'':0.00128029,((('PDT000080674.1':0.00591859,('PDT000078562.1':0.00682129,'PDT000078585.1':0.00424221)'':0.00190996)'':0.00434048,('PDT000185937.1':0.0116875,('PDT000078624.1':0.00633868,'PDT000080626.1':0.00634352)'':0.00421329)'':0.000410121)'':0.00182791,(('PDT000324044.1':0.00805144,'PDT000324700.1':0.00912656)'':0.00173839,((('PDT000377108.1':0.00593663,'PDT000689096.1':0.00589477)'':0.00141252,('PDT000635718.1':0.00636062,('PDT000481586.1':0.00496805,'PDT000718563.1':0.00587665)'':0.00230628)'':0.000617748)'':0.0012675,(('PDT000311117.2':0.012158,'PDT000610859.1':0.0140377)'':0.00285377,('PDT000332111.1':0.0122139,('PDT000315570.1':0.00929936,(('PDT000459879.1':0.00383201,'PDT000497796.1':0.00413929)'':0.00214726,('PDT000156766.2':0.00754547,('PDT000130843.2':0.00703134,('PDT000130909.2':0.00543209,('PDT000130765.2':0.00425739,('PDT000324169.1':0.00448798,('PDT000320913.1':0.00432405,('PDT000317693.1':0.00323795,('PDT000147702.2':0.0028629,('PDT000327302.1':0.00242512,('PDT000321578.1':0.00179619,'PDT000324077.1':0.00179985)'':0.000766083)'':0.000274859)'':0.00058221)'':0.000735496)'':0.000292284)'':0.00015616)'':0.000964898)'':0.000190015)'':0.00123144)'':-0.000996357)'':0.00282787)'':0.00219539)'':0.000687502)'':-0.00149809)'':0.00545123)'':-0.00180476)'':0.0084208)'':-0.00427833)'':-0.000907861,(('PDT000155611.2':0.0280543,('PDT000317000.1':0.0135141,'PDT000130813.2':0.0166836)'':0.0101042)'':0.00620174,('PDT000080670.1':0.0653809,('PDT000080778.1':0.0488187,(('PDT000210704.1':0.0271454,(('PDT000327908.1':0.0141181,('PDT000319280.1':0.0108852,'PDT000598415.1':0.0133458)'':0.00281273)'':0.00401558,('PDT000080667.1':0.0120275,('PDT000078680.1':0.0116337,'PDT000080743.1':0.011132)'':0.00516023)'':0.00111136)'':0.00492126)'':-0.000473033,(('PDT000610843.1':0.0162871,('PDT000323614.1':0.00763458,'PDT000532513.1':0.00748682)'':0.00559868)'':0.00200028,((('PDT000433885.1':0.0134713,('PDT000315544.1':0.0127485,('PDT000324683.1':0.00407335,'PDT000327105.1':0.00271647)'':0.00523482)'':0.00316026)'':-0.000106969,('PDT000464786.1':0.0167961,('PDT000602561.1':0.00839061,('PDT000320273.1':0.00794923,('PDT000157948.2':0.00648482,('PDT000315549.1':0.00591452,'PDT000326113.1':0.00984298)'':0.000270428)'':0.00111941)'':0.00132008)'':0.00348714)'':-0.00184468)'':0.00565292,(('PDT000485299.1':0.0126819,'PDT000640365.1':0.0159465)'':0.00724519,((('PDT000080958.1':0.0208074,('PDT000078658.1':0.0107439,('PDT000078636.1':0.00949808,('PDT000078629.1':0.00720518,('PDT000078697.1':0.0101419,'PDT000080837.1':0.00583247)'':0.00131092)'':0.00156688)'':0.000503417)'':0.00471683)'':-0.00232527,('PDT000080696.1':0.01538,(('PDT000078536.1':0.00356226,'PDT000080598.1':0.00510826)'':0.00284716,(('PDT000005348.2':0.00489962,'PDT000248487.1':0.00575028)'':0.00117131,('PDT000380938.1':0.00725985,('PDT000078644.1':0.00708988,('PDT000080901.1':0.00524718,'PDT000368999.1':0.00502922)'':0.000607166)'':0.00178668)'':0.00019696)'':0.00159978)'':0.00370251)'':0.000623765)'':0.000908561,('PDT000399876.1':0.0203457,('PDT000325151.1':0.0188195,(('PDT000176408.2':0.0163998,('PDT000646096.1':0.0125197,(('PDT000328113.1':0.00550585,'PDT000334894.1':0.00455895)'':0.00298546,(('PDT000398402.1':0.00712765,'PDT000417567.1':0.00817135)'':0.00101976,('PDT000482555.1':0.0048006,'PDT000677484.1':0.0042749)'':0.00423394)'':0.000775743)'':0.00562098)'':0.00125717)'':-0.00380959,(('PDT000601937.1':0.0132738,('PDT000155613.2':0.0091082,('PDT000147812.2':0.00447713,'PDT000444742.1':0.00529064)'':0.00347237)'':0.0022547)'':0.0013663,(('PDT000664576.2':0.010809,('PDT000319871.1':0.00529753,'PDT000327155.1':0.00535237)'':0.00412194)'':0.00108941,('PDT000130897.2':0.012803,('PDT000130683.2':0.013135,('PDT000388921.1':0.00854932,(('PDT000157267.2':0.00514669,('PDT000320928.1':0.00305907,('PDT000319898.1':0.00306685,'PDT000600115.1':0.00401403)'':3.53157e-05)'':0.00164691)'':0.00104084,('PDT000327993.1':0.00825708,('PDT000322767.1':0.00682313,('PDT000324200.1':0.00660215,('PDT000646366.1':0.00498116,(('PDT000130748.2':0.00378762,'PDT000130861.2':0.00369069)'':0.000263464,(('PDT000130882.2':0.00391777,'PDT000160598.2':0.00396489)'':0.000187635,(('PDT000320855.1':0.00244764,'PDT000130917.2':0.00323815)'':0.000538417,('PDT000160422.2':0.00211913,'PDT000333311.1':0.00207332)'':0.0012023)'':0.000828722)'':0.000681101)'':0.000925769)'':0.000497114)'':0.000195976)'':0.000417853)'':-0.000265423)'':0.00190419)'':0.000645004)'':0.00229957)'':-0.000944185)'':-0.000104642)'':0.000577637)'':0.00742118)'':0.00441364)'':-0.00951863)'':0.00825853)'':-0.0073605)'':0.00366514)'':-0.000679242)'':0.0218892)'':0.000858255)'':-0.0175677)'':-0.00545525)'':0.0283258)'':-0.0212521)'':0.000801989)'':0.00529418)'':0.010622,((('PDT000155579.2':0.0267431,'PDT000080717.1':0.0312523)'':0.032076,('PDT000260255.2':0.0420085,'PDT000260279.2':0.0455848)'':0.00790211)'':0.00436783,(((('PDT000080683.1':0.0369986,'PDT000080889.1':0.0376254)'':0.0289464,((('PDT000080910.2':0.0127369,('PDT000078608.1':0.00582207,'PDT000080945.1':0.00493493)'':0.00599497)'':0.00230318,(('PDT000080819.2':0.0105013,'PDT000080593.1':0.00990691)'':0.00109468,('PDT000345843.1':0.014268,('PDT000078651.1':0.00513609,('PDT000080606.1':0.00357086,'PDT000080616.1':0.00530164)'':0.000827683)'':0.0054394)'':0.000712681)'':0.00697362)'':0.0115322,(('PDT000080836.1':0.0390187,('PDT000078587.1':0.0268058,'PDT000080638.1':0.0303622)'':0.00360006)'':0.0108191,('PDT000080700.1':0.0575824,('PDT000078627.1':0.0474865,('PDT000081001.1':0.0462282,('PDT000078750.1':0.0462049,('PDT000080701.1':0.0412452,('PDT000080767.1':0.0331271,'PDT000080992.1':0.0319914)'':0.00275259)'':0.00224202)'':0.000884071)'':0.00159871)'':0.00829419)'':-0.00322324)'':0.00342094)'':0.0122687)'':-0.00210082,(('PDT000470261.1':0.034667,('PDT000470575.1':0.0264574,'PDT000585945.1':0.0209074)'':0.0062547)'':0.00129478,((('PDT000080675.1':0.0130343,'PDT000080706.1':0.0127568)'':0.0125908,('PDT000634496.1':0.0161268,('PDT000625580.1':0.0132413,('PDT000190489.2':0.0110918,'PDT000677100.1':0.0107764)'':0.00169825)'':0.00169946)'':0.0123068)'':-0.000622177,(('PDT000201094.2':0.0162399,(('PDT000521295.1':0.0148821,'PDT000625549.1':0.0126925)'':0.00261705,('PDT000213223.2':0.00377647,('PDT000190466.2':0.00183631,'PDT000258458.2':0.00126028)'':0.00121508)'':0.0144423)'':0.0013062)'':0.00291852,(('PDT000627736.1':0.0342566,('PDT000222050.2':0.0222009,('PDT000471119.1':0.018879,('PDT000289108.2':0.0139761,(('PDT000112864.4':0.00572177,'PDT000627746.1':0.00684553)'':0.00283426,('PDT000112869.2':0.00728673,'PDT000634510.1':0.00760457)'':0.00170872)'':0.00204225)'':0.0058453)'':0.00153352)'':0.00543175)'':-0.00148474,(('PDT000625541.1':0.00801388,'PDT000627699.1':0.00817762)'':0.0198573,('PDT000231285.2':0.0154703,(('PDT000173978.2':0.00906626,'PDT000627692.1':0.00969304)'':0.00395768,('PDT000634436.1':0.0126875,('PDT000100154.2':0.00826528,('PDT000627755.1':0.00798879,('PDT000470253.1':0.00846006,('PDT000112856.2':0.00693079,'PDT000634440.1':0.00639591)'':0.000628987)'':0.000321462)'':0.00248021)'':0.000955371)'':3.42906e-05)'':0.00599057)'':0.00416487)'':-0.00271123)'':0.00366714)'':0.00173742)'':0.0041657)'':0.00791003)'':0.00653851,(((('PDT000664573.2':0.0334727,(('PDT000080624.1':0.0146371,'PDT000369015.1':0.0178864)'':0.00530728,('PDT000078683.1':0.0260515,('PDT000080856.1':0.0218566,('PDT000081006.1':0.0142088,('PDT000078543.1':0.00696106,'PDT000080773.1':0.00579044)'':0.0069806)'':0.00332031)'':0.0045237)'':0.00264816)'':0.00842672)'':-0.00308832,(('PDT000078551.1':0.0159689,'PDT000080997.1':0.0168441)'':0.0058601,(('PDT000078679.1':0.0268292,('PDT000080623.1':0.00517934,('PDT000078649.1':0.00439306,'PDT000081010.1':0.00536887)'':0.00225195)'':0.0100227)'':0.00496481,(('PDT000080768.1':0.0156854,'PDT000080783.1':0.0106226)'':0.0078335,('PDT000080727.1':0.0116577,('PDT000080833.1':0.0104067,'PDT000080981.1':0.00911759)'':0.00323571)'':0.0069648)'':0.00369997)'':0.0038623)'':0.00499612)'':0.00803064,(('PDT000080719.1':0.0300527,'PDT000080885.1':0.0368316)'':0.00422722,(('PDT000185989.1':0.0377519,('PDT000627817.1':0.0227658,('PDT000439269.1':0.0229391,('PDT000318611.1':0.0165074,'PDT000326197.1':0.0140616)'':0.00546111)'':0.00275908)'':0.00527462)'':0.000254241,((('PDT000467519.1':0.0160178,('PDT000288442.2':0.00665052,'PDT000467512.1':0.00687778)'':0.00630479)'':0.00715491,('PDT000672484.1':0.0226634,(('PDT000323004.1':0.0161114,'PDT000503839.1':0.013248)'':0.00185427,('PDT000467533.1':0.0128312,('PDT000470599.1':0.0061818,'PDT000346318.1':0.0071808)'':0.00623833)'':0.00173145)'':0.00582026)'':-0.000285993)'':0.000142148,('PDT000080913.1':0.0459185,(('PDT000304595.1':0.0180421,'PDT000304596.1':0.0233243)'':0.0111817,((('PDT000304592.1':0.0208637,'PDT000346338.1':0.0215225)'':0.00725905,('PDT000345840.1':0.0205847,('PDT000259497.1':0.0146838,('PDT000195693.2':0.0150097,(('PDT000040357.2':0.00449317,'PDT000439271.1':0.00427614)'':0.00441358,('PDT000346302.1':0.00746531,('PDT000610584.1':0.00513322,'PDT000227978.1':0.0042396)'':0.00230758)'':0.00220149)'':0.00216857)'':0.00213946)'':0.00285581)'':0.00259213)'':-0.000640409,(('PDT000186150.1':0.0199387,'PDT000106400.2':0.0168675)'':0.00383553,(('PDT000467826.1':0.0205665,('PDT000467646.1':0.0091058,'PDT000486071.1':0.0099399)'':0.0052647)'':0.00355022,('PDT000185900.1':0.0226658,('PDT000610788.1':0.0144044,(('PDT000610767.1':0.0112498,'PDT000346362.1':0.00882372)'':0.00536218,('PDT000610812.1':0.00982494,'PDT000346366.1':0.00959256)'':0.00472602)'':0.0010531)'':0.00418049)'':-0.00111686)'':0.00149832)'':-0.000632784)'':0.00513367)'':0.00629006)'':-0.00983339)'':0.00371645)'':0.0100849)'':-0.00432795)'':0.000962475,(((('PDT000260257.2':0.0176327,('PDT000260276.2':0.0139618,'PDT000260289.2':0.0110856)'':0.00252628)'':0.00574646,('PDT000338209.1':0.00724643,('PDT000139836.1':0.00555464,'PDT000139872.1':0.0102296)'':0.00156452)'':0.0117331)'':0.00605155,(('PDT000181335.2':0.027457,'PDT000112852.2':0.0237022)'':0.00884675,(('PDT000260642.2':0.0230127,'PDT000467732.1':0.0226656)'':0.00503605,((('PDT000186078.1':0.0205717,('PDT000345871.1':0.0171632,('PDT000321849.1':0.0135042,'PDT000368989.1':0.0136999)'':0.00223345)'':0.00292692)'':0.00106466,(('PDT000080745.1':0.0147776,'PDT000080829.1':0.0149567)'':0.00552522,('PDT000080733.1':0.0159917,('PDT000078655.1':0.0152237,'PDT000080951.1':0.0142606)'':0.00172043)'':0.00239611)'':0.000958877)'':0.00096247,('PDT000487634.1':0.0320531,((('PDT000346358.1':0.0113726,'PDT000346436.1':0.00679467)'':0.00865296,(('PDT000634540.1':0.00373994,'PDT000640611.1':0.00313668)'':0.0135449,('PDT000346340.1':0.0151692,('PDT000139824.1':0.0130373,'PDT000231028.1':0.0130471)'':0.00111848)'':0.0020877)'':0.00161053)'':0.00332338,(('PDT000216664.2':0.015592,('PDT000326233.1':0.0101093,'PDT000332123.1':0.010592)'':0.00473376)'':0.00226731,('PDT000518100.1':0.017947,(('PDT000646364.1':0.0102084,'PDT000646367.1':0.00957677)'':0.00304608,('PDT000318691.1':0.0141934,('PDT000260290.2':0.00473495,'PDT000260325.2':0.00502698)'':0.00678708)'':0.000648226)'':0.00316499)'':-0.000124983)'':0.000909168)'':0.00753485)'':-0.00509487)'':0.00447422)'':0.00191682)'':-0.0005481)'':0.00283906,(('PDT000678005.1':0.0418477,('PDT000227977.1':0.0273257,('PDT000629719.1':0.0301702,(('PDT000213985.2':0.0245777,'PDT000346374.1':0.027323)'':0.00405615,('PDT000649806.1':0.0250624,('PDT000634545.1':0.0133745,(('PDT000221709.2':0.00935672,'PDT000470315.1':0.0123157)'':0.00238993,('PDT000346371.1':0.0112925,('PDT000346428.1':0.010814,('PDT000346401.1':0.00751605,'PDT000346408.1':0.00520385)'':0.00175347)'':0.00317459)'':0.00123096)'':0.00206546)'':0.00667584)'':0.000362461)'':0.00257706)'':0.00256733)'':0.00532035)'':0.000924695,(('PDT000260288.2':0.0334137,('PDT000280835.2':0.0346172,('PDT000260627.2':0.00791408,('PDT000260250.2':0.0114479,'PDT000260307.2':0.00866164)'':0.00177272)'':0.0126646)'':0.00523705)'':0.00732593,((('PDT000634429.1':0.0190484,('PDT000186096.1':0.00492309,('PDT000185928.1':0.00306348,'PDT000186009.1':0.00342808)'':0.00223693)'':0.0100981)'':0.00558772,('PDT000380972.1':0.0230258,('PDT000591875.1':0.0155196,('PDT000589276.1':0.0147514,('PDT000080883.1':0.0153685,('PDT000610564.1':0.0120287,('PDT000326246.1':0.00792601,('PDT000186093.1':0.0113624,('PDT000186007.1':0.00489855,('PDT000186059.1':0.00412727,'PDT000346356.1':0.00554275)'':0.000180662)'':0.00205983)'':0.00102017)'':0.00241057)'':0.00147888)'':0.00201237)'':0.00325765)'':0.00236174)'':-0.00253694)'':0.0108746,((('PDT000326202.1':0.00806258,'PDT000610858.1':0.00768072)'':0.0175431,('PDT000009766.2':0.0315614,('PDT000139865.1':0.0182162,('PDT000078751.1':0.00869685,'PDT000080604.1':0.0150013)'':0.0055514)'':0.00789433)'':0.00106039)'':0.000130222,((('PDT000333650.1':0.0221126,('PDT000376045.1':0.0121785,'PDT000479302.1':0.0143359)'':0.00407823)'':0.000717222,('PDT000580042.1':0.019068,('PDT000345845.1':0.0172259,('PDT000345881.1':0.0131474,('PDT000610731.1':0.0128303,('PDT000346391.1':0.0099927,('PDT000346350.1':0.00482333,'PDT000346352.1':0.00523047)'':0.0034964)'':0.00214744)'':0.000309354)'':0.00480159)'':0.00106208)'':0.00058621)'':0.00400617,(('PDT000078538.1':0.0333357,('PDT000080899.1':0.0335159,('PDT000080785.1':0.0218456,('PDT000078591.1':0.00826043,'PDT000345865.1':0.00998087)'':0.0112034)'':0.00355058)'':0.00173627)'':-0.000491907,(('PDT000646606.1':0.0265103,('PDT000155919.2':0.0164171,'PDT000541976.1':0.0175999)'':0.00286091)'':0.0024861,(('PDT000243638.2':0.0226117,('PDT000346387.1':0.0120031,'PDT000413590.1':0.00960671)'':0.0099002)'':0.00680214,('PDT000155922.2':0.0479685,((('PDT000185894.1':0.0125456,'PDT000185974.1':0.0120478)'':0.00419925,('PDT000078546.1':0.0233402,(('PDT000320220.1':0.0120161,('PDT000481127.1':0.00778259,(('PDT000161933.2':0.00393786,'PDT000646585.1':0.00465097)'':0.00217874,('PDT000215095.2':0.00472401,'PDT000295746.2':0.0049335)'':0.000791913)'':0.0024378)'':0.000886602)'':0.000544971,('PDT000046989.1':0.010183,('PDT000380943.1':0.00855379,('PDT000186156.1':0.00486593,('PDT000610647.1':0.00472138,('PDT000047008.1':0.00423137,('PDT000610805.1':0.00311052,'PDT000346292.1':0.00386924)'':0.000107847)'':0.00157418)'':0.000676125)'':0.00302251)'':0.00261622)'':-0.00133401)'':0.00864396)'':-0.00122741)'':-0.00122686,('PDT000078559.1':0.0213363,(('PDT000185966.1':0.0314554,(('PDT000209464.1':0.0182233,('PDT000078581.1':0.0112574,('PDT000080603.1':0.00946714,'PDT000346380.1':0.0104813)'':0.00208328)'':0.00391245)'':-0.000697662,((('PDT000157090.2':0.00720773,('PDT000244722.2':0.00234461,'PDT000302833.2':0.00255196)'':0.00480123)'':0.00337049,('PDT000467654.1':0.0169242,('PDT000646463.1':0.00947555,(('PDT000646541.1':0.00880282,('PDT000520382.1':0.00643822,('PDT000467515.1':0.00410442,('PDT000646361.1':0.00343484,'PDT000646474.1':0.0030483)'':0.000674173)'':0.00243654)'':0.00155699)'':-0.000769774,('PDT000409973.1':0.00880957,(('PDT000157818.2':0.00419793,('PDT000227495.2':0.00259094,'PDT000480481.1':0.00300067)'':0.00132667)'':9.49284e-05,('PDT000155362.2':0.00389761,('PDT000722635.1':0.00394829,('PDT000212364.2':0.00247339,'PDT000467453.1':0.00251812)'':0.0012408)'':0.000868118)'':-0.000137358)'':0.00323913)'':-0.000139658)'':0.00254554)'':0.00196204)'':-0.00268111)'':0.000827044,((('PDT000593743.1':0.00614527,(('PDT000323683.1':0.00340091,'PDT000130698.2':0.00358512)'':0.00178477,('PDT000213212.2':0.00314514,'PDT000517566.1':0.00423273)'':0.00158424)'':0.00102343)'':0.000918564,('PDT000080864.1':0.013597,('PDT000610752.1':0.00747854,('PDT000080592.1':0.00723699,('PDT000078688.1':0.00411138,('PDT000368997.1':0.0023179,('PDT000369001.1':0.00262288,('PDT000346299.1':0.00158793,'PDT000346316.1':0.00160728)'':0.000468367)'':0.000252587)'':0.00144558)'':0.00205407)'':0.00225755)'':0.00206989)'':-0.00230377)'':0.00381469,('PDT000610679.1':0.0292045,('PDT000380961.1':0.0152035,(('PDT000078615.1':0.0100377,'PDT000080722.1':0.00984102)'':0.00133034,('PDT000185897.1':0.0081342,('PDT000346474.1':0.01063,('PDT000346304.1':0.00711257,(('PDT000047010.1':0.00442705,('PDT000345894.1':0.00466273,('PDT000139847.1':0.00255836,'PDT000345850.1':0.00292957)'':0.00111007)'':0.000378959)'':0.000204922,('PDT000610735.1':0.00560242,('PDT000080746.1':0.00414387,('PDT000345891.1':0.00390519,('PDT000346296.1':0.00324376,('PDT000346330.1':0.00212435,'PDT000346393.1':0.00337235)'':0.000846708)'':0.000732363)'':0.000964907)'':0.000709397)'':-0.000322794)'':0.00255596)'':0.000652484)'':0.000785571)'':0.00189175)'':0.00350816)'':0.00276618)'':-0.00680679)'':0.000441995)'':0.00345321)'':0.00773492)'':-0.00927493,('PDT000080861.1':0.0342684,(('PDT000078568.1':0.0179462,(('PDT000080711.1':0.0146617,('PDT000080691.1':0.00549465,('PDT000259494.1':0.00515645,('PDT000080721.1':0.00243406,'PDT000080741.1':0.00326081)'':0.00141077)'':0.0015714)'':0.00334731)'':-0.00144973,('PDT000346322.1':0.0149319,(('PDT000346294.1':0.00462344,('PDT000346309.1':0.00205499,'PDT000346483.1':0.00233535)'':0.00105469)'':0.00183513,('PDT000369014.1':0.00843396,('PDT000346479.1':0.00370084,('PDT000345846.1':0.00224868,'PDT000346348.1':0.00244147)'':0.000781837)'':0.00227018)'':-0.000470374)'':0.00420158)'':-0.00168401)'':0.00493703)'':-0.00364444,('PDT000646114.1':0.0159936,(('PDT000216707.1':0.0110784,'PDT000216708.1':0.00933381)'':0.00530275,(('PDT000610531.1':0.011273,('PDT000467673.1':0.008232,'PDT000646429.1':0.0086398)'':0.00173878)'':0.00116833,((('PDT000345869.1':0.00623406,('PDT000346361.1':0.00220754,'PDT000346473.1':0.0016909)'':0.00222042)'':0.00121701,('PDT000610542.1':0.00425748,(('PDT000345893.1':0.00275475,('PDT000346323.1':0.00134395,'PDT000346431.1':0.00125319)'':0.000836288)'':0.000998219,('PDT000346290.1':0.00207665,('PDT000346434.1':0.00166989,'PDT000346476.1':0.00112759)'':0.00091582)'':0.00101867)'':0.00126775)'':0.000511059)'':0.00329847,(('PDT000646393.1':0.00916673,('PDT000611838.1':0.00424031,('PDT000467567.1':0.00285895,('PDT000467358.1':0.00227321,('PDT000201139.2':0.00204589,('PDT000054734.2':0.00203,'PDT000467582.1':0.00206182)'':0.000648951)'':0.000204966)'':0.000619039)'':0.000240305)'':0.00258196)'':-0.000412559,('PDT000333263.1':0.0101441,('PDT000486468.1':0.0101488,('PDT000160429.2':0.00915678,('PDT000362592.1':0.0078187,('PDT000467502.1':0.00789392,('PDT000664575.2':0.00460318,('PDT000467507.1':0.00404674,(('PDT000233630.2':0.00301893,('PDT000201152.2':0.00243096,'PDT000203886.2':0.00225966)'':0.00116599)'':0.000891241,('PDT000130919.2':0.00468043,('PDT000155134.2':0.00351013,(('PDT000155445.2':0.00230416,'PDT000333972.1':0.00298526)'':0.000243833,('PDT000467737.1':0.00246133,'PDT000646394.1':0.00242791)'':0.000302647)'':0.000107002)'':0.00101882)'':7.23284e-05)'':0.000467058)'':0.000557439)'':0.000630219)'':0.00074174)'':0.000815007)'':0.00148431)'':0.000611753)'':-0.00293917)'':0.00144492)'':0.00113312)'':0.00206883)'':0.00284373)'':-0.00457473)'':0.00963821)'':-0.00689949)'':0.00706315)'':-0.00583489)'':0.0148958)'':-0.00974852)'':0.00158465)'':-0.000316896)'':-0.00293761)'':0.00942727)'':-0.00253475)'':0.00901577)'':-0.00814177)'':-0.000343999)'':0.002376)'':0.00678412)'':0.0101141)'':-0.0109808)'':0.148762)'':-0.127448)'':0.137692)'':0.0124124,(((('PDT000289961.1':0.0387798,(('PDT000190976.2':0.0323499,'PDT000190993.2':0.0194785)'':0.00546299,('PDT000651786.1':0.00601607,('PDT000147644.2':0.00466262,'PDT000147687.2':0.004992)'':0.00283737)'':0.0188645)'':0.0199587)'':0.0333457,(('PDT000191742.2':0.0744004,'PDT000292889.1':0.0745916)'':0.0153025,('PDT000289983.1':0.0794505,('PDT000101337.2':0.0344377,('PDT000191750.2':0.0130374,('PDT000332800.1':0.0134746,('PDT000420994.1':0.00454862,('PDT000156756.2':0.00203175,'PDT000629841.1':0.00206252)'':0.00193253)'':0.00877127)'':0.00324072)'':0.0154555)'':0.0394272)'':0.00770269)'':-0.00114103)'':0.0348584,((('PDT000155533.2':0.0229446,('PDT000008632.2':0.0154708,'PDT000289999.1':0.0170993)'':0.00762088)'':0.0736456,((('PDT000147636.2':0.0104801,'PDT000155779.2':0.014893)'':0.0172281,('PDT000289988.1':0.0219709,'PDT000380928.1':0.0179271)'':0.00872577)'':0.00724929,('PDT000702590.1':0.0160772,('PDT000147735.2':0.012175,(('PDT000147680.2':0.00316789,'PDT000147573.2':0.00291568)'':0.00151655,('PDT000191743.2':0.00161433,'PDT000191745.2':0.00207976)'':0.00255564)'':0.00451696)'':0.00630665)'':0.0189968)'':0.0690192)'':-0.000941865,((('PDT000009762.2':0.0353203,'PDT000292885.1':0.034116)'':0.00209284,('PDT000191737.2':0.00951527,('PDT000147619.2':0.00615076,'PDT000417565.1':0.00588544)'':0.00591863)'':0.0234613)'':0.0433813,((('PDT000280901.2':0.0323624,('PDT000191768.2':0.0154143,'PDT000308401.2':0.0123138)'':0.0201711)'':0.000346802,('PDT000289947.1':0.0336602,('PDT000097455.2':0.0351263,((('PDT000158771.2':0.0212543,'PDT000678047.1':0.0267811)'':0.000437867,('PDT000160464.2':0.0225534,('PDT000158729.2':0.0138101,'PDT000191767.2':0.0183164)'':0.00612553)'':0.00217372)'':0.00356102,(('PDT000280845.2':0.0215775,'PDT000687068.1':0.0239799)'':0.00739136,('PDT000380950.1':0.0204097,('PDT000451089.1':0.0151689,('PDT000160299.2':0.00989743,'PDT000191760.2':0.00961717)'':0.00582266)'':0.00263172)'':0.00468507)'':0.00224837)'':0.00805673)'':0.00392368)'':-0.00634793)'':0.0106226,(((('PDT000380937.1':0.0237089,'PDT000380942.1':0.0150885)'':0.00272501,('PDT000160563.2':0.00840139,('PDT000158732.2':0.00803718,'PDT000158816.2':0.00542112)'':0.00180536)'':0.0152245)'':0.000471045,(('PDT000160291.2':0.0299747,('PDT000158730.2':0.0097265,('PDT000158757.2':0.00583345,('PDT000267336.3':0.00394644,'PDT000267355.3':0.00632476)'':0.00132015)'':0.00706225)'':0.0113369)'':-0.00119371,(('PDT000046991.1':0.0224133,('PDT000190774.2':0.0183504,('PDT000278474.2':0.00830239,('PDT000318526.1':0.00510823,'PDT000323338.1':0.00455409)'':0.00232205)'':0.00684153)'':0.00813996)'':0.00170307,(('PDT000278489.2':0.00744407,('PDT000191761.2':0.00503842,'PDT000191770.2':0.00653838)'':0.00273218)'':0.00728046,('PDT000380926.1':0.0134377,('PDT000155655.2':0.00314402,('PDT000155401.2':0.00243458,'PDT000160177.2':0.00255594)'':0.00104327)'':0.0112122)'':-0.000164253)'':0.00845423)'':0.000325758)'':0.00199522)'':0.00939456,((('PDT000005942.2':0.0281237,('PDT000190771.2':0.0127565,('PDT000651410.1':0.0118969,('PDT000319852.1':0.00898847,'PDT000158791.2':0.00719983)'':0.00304986)'':0.00206384)'':0.00843902)'':-0.00295606,('PDT000161661.2':0.0153658,(('PDT000156062.2':0.0085428,'PDT000161676.2':0.0079418)'':0.00440456,('PDT000158758.2':0.00978316,'PDT000674801.1':0.0114112)'':0.000461737)'':0.00849446)'':-0.00034332)'':0.0177744,(('PDT000008788.2':0.0179571,'PDT000455747.1':0.0178347)'':0.0129267,(('PDT000158825.2':0.0172954,(('PDT000191771.2':0.00718118,'PDT000278478.2':0.00954192)'':0.00134716,('PDT000155476.2':0.00662255,'PDT000635875.1':0.00720275)'':0.00486407)'':0.0105255)'':-0.0032351,(('PDT000160332.2':0.00557032,'PDT000716348.1':0.00657988)'':0.00558316,('PDT000157963.2':0.0118493,('PDT000711408.1':0.00767766,('PDT000278486.2':0.0103019,('PDT000317769.1':0.00301004,'PDT000191769.2':0.00347634)'':0.00278651)'':0.00171846)'':0.00322127)'':-0.00019337)'':0.0089574)'':0.0152554)'':-0.000658396)'':0.00157998)'':0.0104011)'':0.0429082)'':0.0193202)'':0.0197627)'':0.11594,(('PDT000589426.1':0.0576792,('PDT000494538.1':0.00137317,'PDT000583856.1':0.00112408)'':0.0537406)'':0.146449,(((('PDT000161073.2':0.0237674,'PDT000377974.1':0.0290813)'':0.0364602,(('PDT000260336.2':0.0517026,('PDT000160751.2':0.0302552,'PDT000161011.2':0.0296396)'':0.0131116)'':0.0125725,(('PDT000467712.1':0.0199597,'PDT000484195.1':0.0236343)'':0.0396132,(('PDT000008835.2':0.033963,('PDT000260365.2':0.0404954,'PDT000260658.2':0.0400185)'':0.00347886)'':0.00805391,(('PDT000158706.2':0.0204556,('PDT000160810.2':0.0102924,'PDT000161030.2':0.0147648)'':0.00583883)'':0.0229045,(('PDT000261038.1':0.0251924,('PDT000357558.1':0.0115708,'PDT000357560.1':0.0155821)'':0.0115602)'':0.00373192,(('PDT000320825.1':0.0194379,('PDT000126288.2':0.00803918,'PDT000465930.1':0.00649972)'':0.012899)'':0.00569215,(('PDT000332705.1':0.0207651,'PDT000325118.1':0.0140149)'':0.00272447,('PDT000376179.1':0.0176367,'PDT000464596.1':0.0185375)'':0.00380071)'':0.0057338)'':0.00852157)'':0.00231688)'':0.0192574)'':0.00165397)'':0.000462535)'':0.0086005)'':0.0363941,((('PDT000191797.2':0.024115,'PDT000121021.2':0.0240553)'':0.00252087,(('PDT000467406.1':0.0162977,'PDT000647849.1':0.0114166)'':0.00507875,('PDT000191782.2':0.0182289,(('PDT000155631.2':0.0106988,'PDT000467716.1':0.0139386)'':0.00164484,('PDT000442723.1':0.00602701,('PDT000327100.1':0.00512793,'PDT000646138.1':0.00463497)'':0.00103789)'':0.00384023)'':0.00759497)'':0.00276646)'':0.00497261)'':0.021974,(('PDT000467697.1':0.0283136,'PDT000711121.1':0.029018)'':0.0313137,(('PDT000467781.1':0.0356945,('PDT000260693.2':0.0186186,('PDT000569760.1':0.00349961,'PDT000672493.1':0.00328411)'':0.0179713)'':0.00907928)'':0.00184088,('PDT000260633.2':0.0464638,('PDT000047011.1':0.0372451,(('PDT000155151.2':0.0121175,('PDT000316129.1':0.00515583,'PDT000637716.1':0.00558937)'':0.00416294)'':0.0165727,(('PDT000260361.2':0.0161089,'PDT000121013.2':0.0176334)'':0.00984693,((('PDT000191778.2':0.00981223,'PDT000467318.1':0.00805207)'':0.0134172,('PDT000344059.1':0.00881583,('PDT000320871.1':0.00325828,'PDT000191777.2':0.00302476)'':0.0044299)'':0.0163986)'':0.000169484,('PDT000486467.1':0.0297803,(('PDT000350569.1':0.0121926,'PDT000656197.1':0.010668)'':0.00422848,('PDT000372704.1':0.00878441,('PDT000651775.1':0.00796272,('PDT000173616.2':0.00466665,'PDT000692350.1':0.00660965)'':0.00284738)'':0.00320843)'':0.00625592)'':0.00746158)'':-0.00196752)'':0.000709004)'':0.00117474)'':0.0096597)'':0.00157201)'':-0.000800725)'':0.0231266)'':-0.00885497)'':0.0544652)'':0.00284848,(((('PDT000191796.2':0.0294544,('PDT000260376.2':0.0162983,('PDT000190437.2':0.0108488,('PDT000260368.2':0.00403645,'PDT000260408.2':0.00482986)'':0.00613995)'':0.00207671)'':0.0105674)'':0.00273483,(('PDT000008799.2':0.0177823,'PDT000289950.1':0.0244236)'':0.00403537,('PDT000327234.1':0.0242932,((('PDT000600090.1':0.0133132,('PDT000316850.1':0.00608288,'PDT000390834.1':0.00578032)'':0.00553988)'':-3.53158e-05,('PDT000428762.1':0.00473204,('PDT000322028.1':0.00336867,('PDT000325707.1':0.00194254,'PDT000326132.1':0.00175155)'':0.00132307)'':0.00129711)'':0.00874529)'':0.00509653,('PDT000319853.1':0.0189125,('PDT000669589.1':0.0166097,(('PDT000377023.1':0.00724923,('PDT000158871.2':0.0064006,'PDT000191791.2':0.0054496)'':0.00197452)'':0.00260986,('PDT000467317.1':0.00476977,('PDT000610683.1':0.00151692,'PDT000610775.1':0.00128112)'':0.00432081)'':0.0053199)'':0.00398041)'':0.00206092)'':-0.00199891)'':0.00433927)'':0.0111771)'':-0.00215021)'':0.0731907,((((('PDT000316882.1':0.00892541,'PDT000148077.2':0.00748269)'':0.0157882,('PDT000287841.2':0.021297,'PDT000342234.1':0.0201492)'':0.00366945)'':0.00266163,(('PDT000637496.1':0.0146534,('PDT000352714.1':0.0123232,('PDT000163233.3':0.00909986,'PDT000634357.1':0.00972504)'':0.00294848)'':0.0022803)'':0.0107406,('PDT000563604.1':0.0289206,('PDT000470226.1':0.0176184,('PDT000588381.1':0.0075975,('PDT000484023.1':0.00683027,'PDT000640313.1':0.00701573)'':0.00154245)'':0.00375234)'':0.00573613)'':0.00354929)'':0.0035969)'':0.00273655,(('PDT000394276.1':0.00808218,('PDT000467423.1':0.00646003,('PDT000610863.1':0.00626402,(('PDT000324170.1':0.00298559,'PDT000157842.2':0.00460378)'':0.000964526,('PDT000467492.1':0.00297612,('PDT000467335.1':0.00200377,'PDT000467397.1':0.00209091)'':0.00061786)'':0.00154309)'':0.000550867)'':0.000907864)'':0.000649341)'':0.0114691,(('PDT000008798.2':0.00730094,'PDT000463645.1':0.00498436)'':0.0172092,(('PDT000320845.1':0.0141441,('PDT000324685.1':0.0159511,('PDT000467495.1':0.0146741,('PDT000610825.1':0.00856562,('PDT000404828.1':0.00346117,'PDT000424705.1':0.00460736)'':0.00449886)'':0.00428236)'':0.00219522)'':0.00174561)'':0.00222915,(('PDT000467630.1':0.00766076,('PDT000332187.1':0.00792893,'PDT000467761.1':0.00757807)'':0.00129869)'':0.00549075,(('PDT000417263.1':0.0109668,'PDT000499264.1':0.0129966)'':0.00167891,(('PDT000601920.1':0.0187144,'PDT000640718.1':0.0188877)'':0.000292963,(('PDT000702549.1':0.0159869,(('PDT000552700.1':0.00785668,'PDT000583553.1':0.00597692)'':0.00202392,('PDT000720394.1':0.0074976,('PDT000588080.1':0.00470128,('PDT000656078.1':0.00427205,'PDT000658735.1':0.00399983)'':0.000970966)'':0.00252417)'':0.00215549)'':0.00324977)'':-0.00128375,(('PDT000682909.1':0.00862593,('PDT000315542.1':0.00360013,'PDT000317018.1':0.00388117)'':0.00469707)'':0.00179706,(('PDT000157752.2':0.00683211,'PDT000415293.1':0.00817339)'':0.00271166,(('PDT000472552.1':0.00570493,('PDT000378701.1':0.00408254,('PDT000377182.1':0.00359403,'PDT000414554.1':0.0039772)'':0.000656696)'':0.00110535)'':0.00153925,(('PDT000322068.1':0.00710535,('PDT000318849.1':0.00387951,('PDT000324134.1':0.00312888,'PDT000148134.2':0.00285377)'':0.00130748)'':0.00149198)'':0.000277444,(('PDT000610580.1':0.00622294,('PDT000212354.2':0.00283604,('PDT000201321.2':0.00193879,'PDT000627804.1':0.00215507)'':0.000259861)'':0.00360252)'':-0.000368767,(('PDT000155253.2':0.00313358,'PDT000325782.1':0.00394801)'':0.00286116,('PDT000319284.1':0.00433366,(('PDT000155656.2':0.00448502,('PDT000203330.2':0.00339829,('PDT000155614.2':0.00228438,('PDT000315572.1':0.00226632,'PDT000467774.1':0.00223008)'':0.000458439)'':0.00115009)'':0.000528263)'':-8.1416e-05,('PDT000155254.2':0.00493425,('PDT000120981.2':0.00370382,('PDT000120976.2':0.00392968,('PDT000156829.2':0.00299147,'PDT000566774.1':0.00358748)'':0.000412157)'':0.000930427)'':0.000144168)'':-0.00035025)'':0.000648806)'':-8.93099e-05)'':0.0011388)'':0.000736067)'':0.000388778)'':0.00172316)'':0.00105161)'':0.00112396)'':0.00414503)'':0.00082696)'':0.000579161)'':-0.000972246)'':0.0144569)'':-0.0118802)'':0.00722007)'':0.027069,(((('PDT000157027.2':0.00673042,'PDT000651392.1':0.00751258)'':0.0115979,('PDT000316185.1':0.00981819,('PDT000349631.1':0.00845506,'PDT000710977.1':0.00751874)'':0.00200376)'':0.00615262)'':0.0108084,((('PDT000290464.2':0.028272,('PDT000190461.2':0.0128862,'PDT000207485.2':0.0172571)'':0.00660714)'':0.00535078,('PDT000161064.2':0.0146852,('PDT000161049.2':0.02085,('PDT000260422.2':0.0118746,'PDT000260670.2':0.0100817)'':0.00739956)'':0.00144439)'':0.00593007)'':0.00162741,(('PDT000448156.1':0.0261199,('PDT000260682.2':0.0159771,('PDT000260652.2':0.00648858,('PDT000260362.2':0.002366,'PDT000260372.2':0.00202873)'':0.0030507)'':0.00355967)'':0.0133715)'':0.00428501,(('PDT000260296.2':0.0339586,('PDT000628372.1':0.010842,('PDT000634372.1':0.0137471,('PDT000190450.2':0.0139427,('PDT000190972.2':0.00342235,'PDT000634375.1':0.00376256)'':0.0050666)'':0.00222862)'':0.00225281)'':0.0133379)'':0.000984034,(('PDT000260397.2':0.0269021,'PDT000260424.2':0.0255792)'':0.00521478,('PDT000161069.2':0.0218826,((('PDT000467333.1':0.0110464,'PDT000519391.1':0.0140777)'':0.00278061,('PDT000022686.1':0.013927,('PDT000320178.1':0.00890755,'PDT000260666.2':0.00984985)'':0.0037972)'':0.00173764)'':0.00114543,(('PDT000201198.2':0.0167489,('PDT000264677.2':0.0115186,'PDT000693813.1':0.0113465)'':0.00456181)'':-1.06102e-05,('PDT000671992.1':0.00830928,(('PDT000191793.2':0.00798796,('PDT000587202.1':0.00499405,'PDT000602988.1':0.00337665)'':0.00316449)'':0.000829467,('PDT000315533.1':0.00601205,('PDT000324184.1':0.0038592,('PDT000148102.2':0.00265215,'PDT000332814.1':0.00223612)'':0.00122573)'':0.0014042)'':0.0019618)'':0.00102501)'':0.00554073)'':0.00346196)'':0.00727909)'':-0.000332208)'':-0.00208439)'':0.00196069)'':0.00161935)'':0.00203378)'':0.0025843,((('PDT000190994.2':0.0115239,'PDT000260386.2':0.0120218)'':0.0331493,('PDT000176400.2':0.0275224,('PDT000671631.1':0.0299118,(('PDT000316558.1':0.00880595,'PDT000635770.1':0.0107843)'':0.00364366,('PDT000333252.1':0.00718936,'PDT000519314.1':0.00850544)'':0.00690624)'':0.00615881)'':0.00828717)'':0.0162382)'':-0.000524095,((('PDT000368970.1':0.042368,(('PDT000327138.1':0.0155203,('PDT000293092.2':0.0134958,('PDT000599824.1':0.0105584,('PDT000158893.2':0.00644865,'PDT000718694.1':0.00689665)'':0.00455641)'':0.00192569)'':0.00112119)'':0.00162611,('PDT000276429.2':0.00989293,(('PDT000711062.1':0.00733391,'PDT000380970.1':0.00814289)'':0.00109527,('PDT000120986.2':0.00604259,('PDT000575164.1':0.00744185,('PDT000322998.1':0.00535593,'PDT000467709.1':0.00490607)'':0.00156015)'':0.000935795)'':0.000872075)'':0.00038781)'':0.00494633)'':0.0310301)'':-0.0225384,(('PDT000467574.1':0.0192343,('PDT000443293.1':0.00508538,'PDT000501726.1':0.0041748)'':0.0160824)'':0.0154447,(('PDT000351927.1':0.0164143,('PDT000468164.1':0.0126317,('PDT000500791.1':0.00821851,('PDT000289078.2':0.00655616,('PDT000293100.2':0.00293839,'PDT000575163.1':0.00245083)'':0.00411243)'':0.00165018)'':0.00244759)'':0.00600589)'':-0.000543332,('PDT000344831.1':0.0422346,('PDT000588977.1':0.0205852,('PDT000311101.2':0.0164442,('PDT000486138.1':0.00961356,(('PDT000579637.1':0.00309029,'PDT000722048.1':0.00299814)'':0.000987126,('PDT000273859.2':0.00508562,('PDT000394761.1':0.00336694,'PDT000525764.1':0.00411137)'':0.000642639)'':0.000418794)'':0.00395529)'':0.00377014)'':0.00159611)'':0.0107795)'':-0.00940801)'':0.00904448)'':-0.00114397)'':0.0019349,(('PDT000378042.1':0.0297244,('PDT000319310.1':0.0277718,('PDT000157125.2':0.0208677,('PDT000332771.1':0.0133322,('PDT000304590.1':0.0110649,('PDT000157202.2':0.00470681,'PDT000380941.1':0.0048769)'':0.00234109)'':0.00372508)'':0.00562262)'':0.00311546)'':0.00145646)'':0.000314866,(('PDT000190999.2':0.0139353,('PDT000120982.2':0.00835654,('PDT000556551.1':0.00565303,'PDT000072669.1':0.00738827)'':0.00377116)'':0.00332585)'':0.0073189,(('PDT000048222.1':0.0184396,('PDT000442726.1':0.00796859,('PDT000320827.1':0.00854799,'PDT000328106.1':0.00677691)'':0.000877164)'':0.00857169)'':0.000371554,(('PDT000410728.1':0.019115,'PDT000489901.1':0.0219257)'':0.00308312,('PDT000432282.1':0.0308135,(('PDT000322773.1':0.0104794,('PDT000549595.1':0.00699928,('PDT000317203.1':0.00403101,'PDT000710982.1':0.0049376)'':0.00214487)'':0.00327714)'':0.00356578,('PDT000433988.1':0.0228328,('PDT000378560.1':0.0126367,(('PDT000329344.1':0.0051118,('PDT000130863.2':0.00440748,'PDT000485316.1':0.00395905)'':0.000470155)'':0.00277069,(('PDT000598411.1':0.00997854,('PDT000496169.1':0.0069692,('PDT000720392.1':0.00540969,('PDT000546416.1':0.00378937,'PDT000656081.1':0.00398265)'':0.00121113)'':0.000766542)'':0.00270578)'':-0.000632344,('PDT000044015.2':0.00827615,('PDT000442727.1':0.0077559,('PDT000319912.1':0.00820466,('PDT000156814.2':0.00505044,(('PDT000444312.1':0.0029935,('PDT000318864.1':0.00270948,'PDT000353611.1':0.00247911)'':0.000695713)'':0.000623808,('PDT000315601.1':0.00306536,('PDT000148018.2':0.00214697,('PDT000155872.2':0.00166321,'PDT000328007.1':0.0019314)'':0.000446726)'':0.000712153)'':0.000850677)'':0.00110664)'':0.00126368)'':0.000331599)'':0.00188905)'':-0.00112661)'':0.00156865)'':0.00235566)'':0.00389307)'':-0.00404812)'':0.00927278)'':-0.00143466)'':-0.000739958)'':0.00207145)'':0.00276328)'':0.00140443)'':0.0142583)'':-0.00866112)'':0.0173268)'':0.0468035)'':-0.00170728,((('PDT000289959.1':0.0397868,('PDT000047007.1':0.0295611,('PDT000147567.2':0.0117702,'PDT000190940.2':0.0123238)'':0.0140824)'':0.0125058)'':-0.00739398,('PDT000173322.1':0.0308093,(('PDT000683251.1':0.0124225,('PDT000155257.2':0.00506858,'PDT000157785.2':0.00488266)'':0.00740236)'':-0.000331929,('PDT000161509.2':0.00462281,('PDT000498961.1':0.0035545,('PDT000643676.1':0.00297984,'PDT000645784.1':0.00240669)'':0.00102895)'':0.000789095)'':0.00537617)'':0.0198166)'':0.00581673)'':0.0454825,((((('PDT000004724.2':0.0179046,('PDT000158691.2':0.00819633,'PDT000161074.2':0.00983517)'':0.00727301)'':0.00614884,(('PDT000161020.2':0.0166005,'PDT000289998.1':0.0233325)'':0.00370928,('PDT000357559.1':0.0064983,'PDT000494194.1':0.0093557)'':0.0139982)'':0.00604221)'':0.033232,((('PDT000260338.2':0.029525,('PDT000380919.1':0.0300052,('PDT000260668.2':0.0135149,('PDT000260410.2':0.00904836,'PDT000380953.1':0.00910284)'':0.00491881)'':0.00618622)'':0.00305236)'':0.0128363,(('PDT000380962.1':0.0414212,('PDT000357556.1':0.0355012,('PDT000380956.1':0.0242073,('PDT000161061.2':0.0191673,'PDT000378591.1':0.0149191)'':0.00390956)'':0.00716787)'':0.00439493)'':0.000492763,(('PDT000161081.2':0.0235769,('PDT000161037.2':0.0152176,'PDT000485340.1':0.0261088)'':0.00976478)'':0.0010261,(('PDT000191784.2':0.0217336,'PDT000467392.1':0.0219973)'':0.00553871,(('PDT000161056.2':0.0164246,'PDT000022688.1':0.0160358)'':0.00563475,('PDT000380952.1':0.0166887,('PDT000191785.2':0.0132948,('PDT000009027.2':0.0112846,'PDT000289993.1':0.0107339)'':0.00381958)'':0.00167065)'':0.00578152)'':0.00340664)'':0.0041152)'':0.00531528)'':0.00285013)'':0.00125964,(((('PDT000222025.2':0.0241947,('PDT000579624.1':0.0242328,('PDT000321174.1':0.0100925,'PDT000576050.1':0.0127159)'':0.0117822)'':0.000861628)'':0.00143267,(('PDT000190448.2':0.0171834,'PDT000260379.2':0.0171802)'':0.00479483,('PDT000260324.2':0.00925039,'PDT000120998.2':0.00978591)'':0.00736265)'':0.00363684)'':0.0117908,((('PDT000226808.2':0.027162,'PDT000455653.1':0.0258103)'':0.00913478,(('PDT000627788.1':0.0192851,('PDT000321982.1':0.00304579,('PDT000394764.1':0.00244743,'PDT000526238.1':0.00234299)'':0.000351082)'':0.0116389)'':0.00603315,('PDT000351917.1':0.0228121,('PDT000443424.1':0.0156151,('PDT000599400.1':0.0128761,(('PDT000351928.1':0.00666113,'PDT000717391.1':0.00695717)'':0.00351162,(('PDT000190932.2':0.00496426,'PDT000569408.1':0.00550954)'':0.00072196,('PDT000295758.2':0.00423212,('PDT000585268.1':0.00344707,'PDT000656194.1':0.00324089)'':0.00154758)'':0.00149431)'':0.00114708)'':0.00356002)'':0.00163169)'':0.00520752)'':-0.00121633)'':0.0128173)'':-0.00230782,(('PDT000260683.2':0.0321466,(('PDT000289965.1':0.0138774,('PDT000253215.2':0.00785414,'PDT000260390.2':0.00532346)'':0.00563584)'':0.00364688,('PDT000260654.2':0.0156725,('PDT000260692.2':0.0103812,('PDT000260351.2':0.00692052,'PDT000260672.2':0.00651498)'':0.00268984)'':0.00317013)'':0.00317085)'':0.0117537)'':-0.00247332,(('PDT000316472.1':0.0132084,'PDT000491537.1':0.0155244)'':0.0115624,('PDT000260391.2':0.0131233,('PDT000260392.2':0.0137605,(('PDT000260641.2':0.00462184,('PDT000260380.2':0.000892646,'PDT000260405.2':0.00130647)'':0.00265786)'':0.00184271,('PDT000260394.2':0.00280475,('PDT000260383.2':0.00266656,'PDT000260425.2':0.00222708)'':0.000683861)'':0.002073)'':0.00618056)'':0.00133128)'':0.00891779)'':-0.00115813)'':0.00527924)'':0.0045223)'':0.00145989,(((('PDT000378921.1':0.015853,'PDT000494191.1':0.0135443)'':0.00888343,(('PDT000610541.1':0.00475518,'PDT000610839.1':0.00701192)'':0.00235185,('PDT000449078.1':0.00448726,('PDT000600059.1':0.00327608,'PDT000630864.1':0.00290599)'':0.00155344)'':0.0034429)'':0.0111166)'':0.00857946,('PDT000289955.1':0.0240173,(('PDT000248488.2':0.0125145,('PDT000626590.1':0.00603325,'PDT000667993.1':0.00642565)'':0.00419068)'':0.00140073,('PDT000679869.1':0.00979095,('PDT000455705.1':0.0119775,('PDT000157159.2':0.00339965,'PDT000328134.1':0.00377738)'':0.00454379)'':0.00225645)'':0.00453649)'':0.0143828)'':-0.00183351)'':0.00575469,((('PDT000191781.2':0.0133523,'PDT000470244.1':0.0119566)'':0.0115003,(('PDT000022687.1':0.0206083,('PDT000260360.2':0.0115421,('PDT000260353.2':0.00924016,'PDT000260356.2':0.00748954)'':0.00478305)'':0.00894643)'':-0.00177664,(('PDT000260794.2':0.00523977,'PDT000598529.1':0.00591003)'':0.0132392,('PDT000312568.2':0.0146664,('PDT000190457.2':0.00926446,('PDT000421244.1':0.00642547,('PDT000568921.1':0.00272805,'PDT000630833.1':0.00286691)'':0.0027155)'':0.00316723)'':0.00435287)'':0.000965699)'':0.00902708)'':0.00665351)'':0.00215211,(('PDT000470232.1':0.0228156,('PDT000610748.1':0.0124613,('PDT000121008.2':0.00687946,('PDT000260370.2':0.00526551,('PDT000260638.2':0.00457559,'PDT000634354.1':0.00569561)'':0.000601086)'':0.00112501)'':0.00218766)'':0.00946312)'':0.00453153,((('PDT000227504.2':0.0190865,('PDT000322455.1':0.00692595,'PDT000524340.1':0.00718095)'':0.0161212)'':0.00482125,('PDT000120959.2':0.019958,(('PDT000253195.2':0.00946341,'PDT000356279.1':0.00944399)'':0.00360097,('PDT000501171.1':0.00453539,('PDT000190436.2':0.00202253,'PDT000191780.2':0.00197228)'':0.00208348)'':0.0081269)'':0.0109251)'':6.1116e-05)'':0.00340178,(('PDT000305210.2':0.0165107,('PDT000022699.1':0.0123886,('PDT000158900.2':0.00686805,'PDT000121017.2':0.00685915)'':0.00319833)'':0.00459349)'':0.00164007,('PDT000311095.2':0.0174108,(('PDT000176402.2':0.0183987,('PDT000598417.1':0.00952468,('PDT000547557.1':0.00642297,('PDT000264882.2':0.0048151,'PDT000334584.1':0.00485107)'':0.0030279)'':0.00308351)'':0.0055581)'':-0.000411127,(('PDT000415207.1':0.010997,'PDT000432276.1':0.011665)'':0.00781031,(('PDT000155539.2':0.0242061,(('PDT000121023.2':0.0101657,('PDT000148158.2':0.00585469,('PDT000327417.1':0.00176431,('PDT000323665.1':0.00156934,'PDT000155854.2':0.00162619)'':0.000632757)'':0.00380849)'':0.00800438)'':-0.0038566,('PDT000366222.1':0.00923542,('PDT000408653.1':0.00671315,('PDT000156906.2':0.00581531,('PDT000376128.1':0.00428046,'PDT000413144.1':0.00419412)'':0.000758356)'':0.000357717)'':0.0011469)'':0.00248651)'':0.00723962)'':-0.00480787,('PDT000610632.1':0.0207772,(('PDT000307925.2':0.00877374,('PDT000338801.1':0.00554308,('PDT000388887.1':0.0040016,'PDT000643064.1':0.00347448)'':0.00142142)'':0.00273819)'':0.0039342,('PDT000573142.1':0.0116249,(('PDT000360488.1':0.00663069,'PDT000562684.1':0.00511061)'':0.000594033,('PDT000640265.1':0.00511773,('PDT000622990.1':0.0062497,(('PDT000579399.1':0.00325229,'PDT000388912.1':0.00322827)'':0.000292571,('PDT000277072.2':0.00417098,('PDT000663846.1':0.00337563,('PDT000690254.1':0.00265672,(('PDT000318664.1':0.00271973,'PDT000148282.2':0.00236806)'':0.000303011,(('PDT000532939.1':0.00163731,'PDT000690289.1':0.0018578)'':0.00101268,('PDT000235314.2':0.00210975,('PDT000465926.1':0.00145397,'PDT000649111.1':0.00154273)'':0.000684288)'':0.000155395)'':0.000455585)'':0.000279675)'':0.0007866)'':0.000245246)'':-0.000271816)'':0.00129518)'':0.00021892)'':0.000539749)'':0.00659709)'':-0.0019721)'':0.00761044)'':-0.00546278)'':0.00357793)'':-4.44384e-05)'':0.00514001)'':-0.00201724)'':0.00517132)'':0.0054776)'':-0.000440626)'':0.00238239)'':0.00281413)'':0.00854054)'':0.0191181)'':-0.012816,(((('PDT000455244.1':0.0180178,'PDT000495508.1':0.0214455)'':0.013876,(('PDT000650692.1':0.0236723,('PDT000198056.2':0.00364247,'PDT000706356.1':0.00393707)'':0.014687)'':0.00744745,('PDT000470237.1':0.00843479,('PDT000631738.1':0.00298645,('PDT000264885.2':0.00277,'PDT000717381.1':0.00211876)'':0.000606482)'':0.00418691)'':0.0202004)'':0.00949754)'':0.00783127,(('PDT000289991.1':0.0295746,(('PDT000161042.2':0.017869,'PDT000161048.2':0.0191388)'':0.0124676,('PDT000357557.1':0.0225209,('PDT000380967.1':0.0102106,'PDT000022690.1':0.0115804)'':0.0104368)'':0.00339764)'':0.00387446)'':0.00328769,((('PDT000532752.1':0.0266888,'PDT000634366.1':0.0322443)'':0.00559161,('PDT000008804.2':0.0368353,('PDT000124786.2':0.0174226,('PDT000619108.1':0.0114232,'PDT000380929.1':0.0138845)'':0.00204152)'':0.0116205)'':0.00107657)'':-0.000218957,(((('PDT000467763.1':0.0159854,('PDT000467579.1':0.0160468,'PDT000216704.1':0.0133064)'':0.00479591)'':0.00495763,('PDT000484853.1':0.018561,('PDT000679781.1':0.00942718,'PDT000333950.1':0.00950432)'':0.00655099)'':0.00247138)'':0.0041836,(('PDT000148576.1':0.016226,('PDT000121034.2':0.00705487,('PDT000320280.1':0.00928968,'PDT000467407.1':0.00647092)'':0.00166748)'':0.00773349)'':-2.26904e-05,(('PDT000203329.2':0.0107917,'PDT000332146.1':0.011083)'':0.00475377,('PDT000120979.2':0.00749892,('PDT000619106.1':0.00864194,('PDT000634376.1':0.00661849,('PDT000649709.1':0.00485944,('PDT000321585.1':0.00231189,'PDT000157741.2':0.00308002)'':0.00131001)'':0.000751289)'':0.00454002)'':0.000662508)'':0.00526866)'':0.00124748)'':0.00605265)'':0.0025774,(('PDT000190930.2':0.0297531,(('PDT000190453.2':0.00739875,'PDT000190964.2':0.00573165)'':0.010679,('PDT000590307.1':0.018687,('PDT000215090.2':0.0145173,'PDT000259870.2':0.0105869)'':0.00353086)'':0.00198339)'':0.0102168)'':-0.00411483,(('PDT000509701.1':0.031108,(('PDT000271610.2':0.0171328,'PDT000289966.1':0.0221914)'':0.00235126,('PDT000289970.1':0.0158947,('PDT000647859.1':0.0153675,('PDT000679858.1':0.0111045,('PDT000344828.1':0.0112449,('PDT000191799.2':0.00509703,('PDT000163247.3':0.00679596,'PDT000191757.2':0.00427194)'':0.0012207)'':0.00318006)'':0.0010931)'':0.00170163)'':0.0021123)'':0.00187271)'':0.0119889)'':-0.0102468,(('PDT000190939.2':0.0164708,('PDT000206883.2':0.00633254,'PDT000467758.1':0.00521666)'':0.0150038)'':0.0051771,(('PDT000121006.2':0.021541,'PDT000467715.1':0.0174136)'':0.00269329,(('PDT000377995.1':0.0197533,(('PDT000627791.1':0.0135032,'PDT000627795.1':0.0134537)'':0.000474958,('PDT000467410.1':0.0107235,('PDT000121027.2':0.0106829,(('PDT000190958.2':0.00730123,'PDT000121041.2':0.00720157)'':0.00102575,(('PDT000610836.1':0.00433046,'PDT000046997.1':0.00455)'':0.00415224,('PDT000120969.2':0.00850824,('PDT000148157.2':0.00582955,('PDT000120989.2':0.00373109,('PDT000121000.2':0.00206061,'PDT000121045.2':0.00203202)'':0.00175073)'':0.00209909)'':0.00048291)'':0.000450631)'':0.0016936)'':0.00145461)'':0.00063609)'':0.00122909)'':0.00582265)'':-0.00512103,('PDT000120970.2':0.0210249,('PDT000105070.2':0.015028,(('PDT000406899.1':0.00874551,('PDT000610846.1':0.00769258,'PDT000610865.1':0.00858692)'':0.00173894)'':0.00089256,('PDT000243634.2':0.0107254,('PDT000714181.1':0.0102148,(('PDT000545376.1':0.00535736,('PDT000634353.1':0.00466782,('PDT000190444.2':0.00455967,'PDT000481080.1':0.00461922)'':0.00101186)'':0.000570235)'':0.00125131,('PDT000551440.1':0.00784593,('PDT000332891.1':0.00656513,('PDT000467766.1':0.00424691,'PDT000634359.1':0.00451978)'':0.00149302)'':0.00189901)'':8.36066e-05)'':0.00166753)'':0.0016522)'':-0.000467247)'':0.00299636)'':0.00405541)'':-0.00435201)'':0.00574521)'':-0.00025503)'':-0.000451302)'':0.00442859)'':0.00020236)'':0.00546583)'':0.00366742)'':0.00297568)'':-0.00273963,(((('PDT000627808.1':0.0162689,(('PDT000297024.2':0.0134211,'PDT000312554.2':0.0125169)'':0.00228662,('PDT000499243.1':0.00870728,('PDT000235262.2':0.00578167,'PDT000564096.1':0.00678443)'':0.00198627)'':0.00459346)'':0.00537842)'':0.000320818,('PDT000407748.1':0.0172346,('PDT000392382.1':0.0146964,('PDT000317598.1':0.00876471,('PDT000569903.1':0.0070094,('PDT000191776.2':0.00544691,('PDT000190978.2':0.00396583,'PDT000548732.1':0.00300976)'':0.00114196)'':0.00328556)'':0.00135957)'':0.0030481)'':0.00150868)'':0.0100231)'':0.00837436,(('PDT000317407.1':0.0164186,'PDT000287840.2':0.0126655)'':0.0116922,('PDT000190454.2':0.0204003,(('PDT000203869.2':0.0169466,('PDT000565129.1':0.0122048,('PDT000692374.1':0.0137849,('PDT000318658.1':0.0145174,(('PDT000414581.1':0.00412306,'PDT000519361.1':0.00396885)'':0.00311757,('PDT000484868.1':0.00649332,('PDT000324122.1':0.00612294,'PDT000467556.1':0.00511526)'':0.00146008)'':0.000865385)'':0.00261593)'':0.00146499)'':0.000383006)'':0.00345568)'':-0.000927361,('PDT000156743.2':0.0250283,(('PDT000687621.1':0.0114859,'PDT000317205.1':0.0126821)'':0.000562547,((('PDT000610537.1':0.000920521,'PDT000610601.1':0.00127904)'':0.00345322,('PDT000610704.1':0.00168375,'PDT000368950.1':0.00151146)'':0.00281298)'':0.00439706,('PDT000576188.1':0.01181,(('PDT000327313.1':0.00465756,'PDT000383910.1':0.00411175)'':0.000964911,('PDT000414579.1':0.00428261,('PDT000394274.1':0.00384459,'PDT000598478.1':0.00353034)'':0.00128826)'':0.000127096)'':0.00435536)'':-0.00136864)'':0.00240662)'':0.00875108)'':-0.00572645)'':0.00456361)'':0.00254166)'':0.0065975)'':-0.00229122,((((('PDT000319867.1':0.0128239,('PDT000603027.1':0.00828484,('PDT000468171.1':0.00702163,'PDT000505395.1':0.00811177)'':0.00122891)'':0.00104543)'':0.00286483,('PDT000253183.2':0.0192888,(('PDT000325166.1':0.00755389,'PDT000325725.1':0.00707881)'':0.00386795,('PDT000327139.1':0.00986598,('PDT000588087.1':0.00865946,('PDT000158019.2':0.00749697,'PDT000316893.1':0.00591323)'':0.00308644)'':0.00243034)'':0.00138127)'':0.00338672)'':-0.00173505)'':0.00531452,('PDT000625872.1':0.0264114,('PDT000121009.2':0.0162137,('PDT000486912.1':0.0143853,('PDT000160403.2':0.0205598,('PDT000378755.1':0.0147841,(('PDT000692377.1':0.00859064,('PDT000409959.1':0.00720399,'PDT000467237.1':0.00877141)'':0.000756306)'':0.000972322,('PDT000467628.1':0.00904707,(('PDT000325146.1':0.0070067,'PDT000157066.2':0.0068214)'':0.000823671,('PDT000231024.1':0.00836957,('PDT000156992.2':0.00516508,('PDT000467696.1':0.00538696,('PDT000156069.2':0.00382837,'PDT000333260.1':0.00315139)'':0.000792633)'':0.00180166)'':0.000752366)'':0.000495322)'':0.00236405)'':6.7867e-05)'':0.00374629)'':0.000774905)'':0.000531473)'':0.00635207)'':0.00221207)'':-0.00488825)'':0.00133723,(('PDT000155560.2':0.0204832,(('PDT000160525.2':0.0127918,('PDT000155878.2':0.00541726,'PDT000160229.2':0.00532794)'':0.00531265)'':0.00228952,('PDT000444743.1':0.014036,('PDT000121024.2':0.0132632,(('PDT000540383.1':0.00654303,'PDT000716342.1':0.00780387)'':0.000460885,(('PDT000316589.1':0.00533705,'PDT000157952.2':0.00382356)'':0.0026628,('PDT000157509.2':0.00649914,'PDT000610572.1':0.00697656)'':0.000350674)'':0.00181991)'':0.00304445)'':0.000734777)'':0.00454433)'':0.00218583)'':-0.00155567,('PDT000155795.2':0.0270651,((('PDT000315554.1':0.00733206,'PDT000413146.1':0.00747904)'':0.00376927,('PDT000459882.1':0.00651704,'PDT000612042.1':0.00532966)'':0.00560556)'':0.00143093,('PDT000261049.1':0.018796,('PDT000556796.1':0.0103193,(('PDT000481990.1':0.00503057,('PDT000412492.1':0.00391805,'PDT000669606.1':0.00574042)'':0.000747837)'':0.000342594,(('PDT000191794.2':0.00420142,'PDT000398046.1':0.00398058)'':0.000676484,('PDT000539786.1':0.00514764,('PDT000324696.1':0.00352651,('PDT000317789.1':0.00338275,('PDT000158876.2':0.00298502,'PDT000326222.1':0.00280109)'':0.000308066)'':0.000223909)'':0.0010024)'':0.000119201)'':0.000767477)'':0.00270991)'':0.00710081)'':-0.00385527)'':0.00895966)'':-0.00317384)'':0.00211836)'':0.00512398,(('PDT000575977.1':0.0349844,('PDT000155210.2':0.0244666,'PDT000556139.1':0.0195723)'':0.0196047)'':0.00533514,((((('PDT000318893.1':0.00417026,'PDT000156006.2':0.00469693)'':0.00196262,('PDT000503838.1':0.00478286,'PDT000532066.1':0.00318844)'':0.00284716)'':0.00484843,(('PDT000156811.2':0.00602743,'PDT000610635.1':0.00714357)'':0.000607099,('PDT000332739.1':0.00738787,('PDT000317196.1':0.00490218,'PDT000155901.2':0.00466243)'':0.00139823)'':0.000268324)'':0.00313527)'':0.00315077,((('PDT000691459.1':0.0115329,('PDT000327144.1':0.0074544,'PDT000476911.1':0.0066792)'':0.0054279)'':0.000105636,(('PDT000548994.1':0.010072,('PDT000378684.1':0.00824108,'PDT000382867.1':0.00686522)'':0.00203409)'':0.000982303,('PDT000322846.1':0.0134592,('PDT000158949.2':0.00716336,'PDT000177057.2':0.00615144)'':0.00467033)'':-3.38078e-05)'':0.00274002)'':-0.000456094,('PDT000602994.1':0.0217777,('PDT000647858.1':0.0161684,('PDT000124782.2':0.00947613,('PDT000191962.2':0.00805275,('PDT000190459.2':0.0130885,(('PDT000496621.1':0.00651082,'PDT000637389.1':0.00563698)'':0.00340401,('PDT000201168.2':0.00847161,('PDT000254363.2':0.00274467,('PDT000190996.2':0.00183576,'PDT000191795.2':0.0020584)'':0.000645059)'':0.00279261)'':0.000406252)'':0.00289286)'':0.000545414)'':0.00131609)'':0.00261299)'':0.00129693)'':-0.00156765)'':0.00022031)'':0.0030869,(('PDT000491525.1':0.020487,('PDT000168323.2':0.0208041,'PDT000467557.1':0.0196248)'':0.00121076)'':0.00501574,('PDT000319324.1':0.0249675,(((('PDT000624017.1':0.00718379,'PDT000631344.1':0.00836801)'':0.00553235,('PDT000524135.1':0.0116606,'PDT000372706.1':0.0105616)'':0.00119387)'':0.0103545,((('PDT000610860.1':0.0143332,('PDT000465638.1':0.00217582,'PDT000604558.1':0.00291654)'':0.0109316)'':0.000386584,(('PDT000239584.2':0.00671315,'PDT000360470.1':0.00660425)'':0.000794519,('PDT000471100.1':0.0048525,'PDT000481097.1':0.005898)'':0.00271596)'':0.00563107)'':0.00355983,(('PDT000441300.1':0.00852065,('PDT000320235.1':0.00640684,('PDT000157206.2':0.00671512,('PDT000317210.1':0.00458778,'PDT000157046.2':0.00478691)'':0.00103234)'':0.000246836)'':0.00341722)'':0.00312392,('PDT000667995.1':0.0118642,('PDT000581047.1':0.0100685,(('PDT000720113.1':0.00612703,('PDT000668341.2':0.00372202,'PDT000722637.1':0.00415121)'':0.00221507)'':0.00253439,('PDT000522491.1':0.00976419,('PDT000595874.1':0.00747666,('PDT000201320.2':0.00413179,'PDT000625328.1':0.00424476)'':0.00116601)'':0.00345964)'':-0.000593297)'':0.00276294)'':0.000417666)'':0.00176504)'':0.00220534)'':0.00327617)'':-0.00159657,(('PDT000467580.1':0.0237761,('PDT000333306.1':0.024501,('PDT000323032.1':0.0133731,'PDT000327210.1':0.0132606)'':0.0075923)'':0.00516055)'':-0.00209262,(((('PDT000191695.2':0.010276,'PDT000355560.1':0.0109731)'':0.000445902,(('PDT000586509.1':0.00689886,('PDT000645799.1':0.00617728,('PDT000415206.1':0.00512803,'PDT000519362.1':0.00651377)'':0.00208387)'':7.41352e-05)'':0.00206872,('PDT000190929.2':0.0110538,(('PDT000191783.2':0.00710324,'PDT000720970.1':0.00571856)'':0.00122616,('PDT000259849.2':0.00454066,'PDT000683201.1':0.00432123)'':0.00151544)'':0.00308348)'':-3.80359e-06)'':0.00199904)'':0.000699896,(('PDT000311100.2':0.0144561,'PDT000641701.1':0.0125629)'':0.000796322,('PDT000671181.1':0.0169246,('PDT000657021.1':0.00852526,((('PDT000191779.2':0.00843862,'PDT000524129.1':0.00803618)'':0.000171811,('PDT000158870.2':0.00421101,('PDT000195700.2':0.0027976,'PDT000205678.2':0.00399154)'':0.00200472)'':0.00186271)'':0.00297961,('PDT000718775.1':0.00803768,(('PDT000678341.1':0.00713088,'PDT000636639.1':0.00691152)'':0.000819748,(('PDT000353163.1':0.00332562,'PDT000719867.1':0.00523926)'':0.00286328,('PDT000264884.2':0.00517477,'PDT000634369.1':0.00550063)'':0.00138592)'':0.00107247)'':0.00115286)'':-0.000812204)'':0.00110825)'':0.00524423)'':0.000225683)'':-0.00255767)'':0.000100204,(('PDT000328139.1':0.025833,'PDT000413591.1':0.0206627)'':0.00344982,('PDT000046995.1':0.0208549,(('PDT000519312.1':0.0175558,(('PDT000416440.1':0.00846653,'PDT000484200.1':0.00932667)'':0.00300937,(('PDT000321632.1':0.00953635,('PDT000120977.2':0.00707471,('PDT000382874.1':0.00721688,('PDT000190982.2':0.00566653,('PDT000205676.2':0.00606619,('PDT000121033.2':0.00367565,'PDT000332728.1':0.00399979)'':0.00134244)'':0.00109223)'':0.00112581)'':0.001503)'':0.00122442)'':-0.000414853,(('PDT000156588.2':0.00540293,'PDT000451220.1':0.00504487)'':0.00249233,('PDT000497545.1':0.0101768,('PDT000688107.1':0.00572435,('PDT000318859.1':0.00412305,('PDT000259835.2':0.00222308,'PDT000322796.1':0.00297121)'':0.00194835)'':0.00164198)'':0.00187503)'':0.00071909)'':0.000471084)'':0.00331247)'':0.00809966)'':-0.0086426,(('PDT000414577.1':0.0132172,'PDT000526240.1':0.0127838)'':0.00883678,(('PDT000532059.1':0.00954089,('PDT000519370.1':0.00882727,(('PDT000465216.1':0.0069258,'PDT000640724.1':0.0075785)'':0.000191025,('PDT000637680.1':0.00582778,('PDT000625867.1':0.00464924,'PDT000643673.1':0.00452873)'':0.000762034)'':0.00139985)'':0.00158166)'':0.00261101)'':-0.00153746,(('PDT000610597.1':0.00845856,('PDT000380940.1':0.00702543,('PDT000610691.1':0.0076777,('PDT000610569.1':0.0035907,'PDT000610688.1':0.00548842)'':0.00419474)'':0.000515902)'':0.0015283)'':0.000742185,(('PDT000155766.2':0.0114631,'PDT000378546.1':0.0116268)'':4.2092e-05,('PDT000641396.1':0.00951396,('PDT000316948.1':0.00682269,((('PDT000418049.1':0.00449693,'PDT000598444.1':0.00456501)'':0.000783696,('PDT000612041.1':0.00488631,'PDT000669584.1':0.00417563)'':0.00148918)'':0.000486976,('PDT000324179.1':0.00779663,((('PDT000355582.1':0.00564658,'PDT000360469.1':0.00412898)'':0.000523417,('PDT000467783.1':0.0036518,('PDT000134596.2':0.00329443,'PDT000544304.1':0.00298923)'':0.000533862)'':0.000420973)'':0.0014663,('PDT000320851.1':0.00830401,(('PDT000377028.1':0.0064083,('PDT000320209.1':0.00431124,('PDT000316463.1':0.00230564,'PDT000326171.1':0.00228472)'':0.00146212)'':0.00176756)'':-0.00038055,(('PDT000157253.2':0.00396083,('PDT000155506.2':0.00225003,'PDT000156609.2':0.00263921)'':0.00142341)'':0.000809143,('PDT000321646.1':0.00626033,('PDT000322810.1':0.00417555,('PDT000316571.1':0.00419,('PDT000403487.1':0.00383226,(('PDT000328156.1':0.00339976,'PDT000467493.1':0.00298237)'':0.000681245,('PDT000328024.1':0.00313046,('PDT000322969.1':0.00344685,('PDT000155696.2':0.00228562,('PDT000323360.1':0.00183757,'PDT000155979.2':0.00175596)'':0.000607116)'':0.000901175)'':0.000289772)'':0.000314712)'':0.000671263)'':0.000386102)'':0.000148875)'':0.000352438)'':-0.000111909)'':0.000207511)'':0.000837096)'':-0.000471812)'':0.00144947)'':-0.000773722)'':0.00120966)'':0.00343198)'':-0.00107074)'':-0.000425805)'':0.000485833)'':0.00597329)'':-0.00392995)'':0.00979689)'':-0.00332017)'':-0.00540413)'':0.0096719)'':-0.0050985)'':0.00440762)'':0.00116283)'':-0.00632193)'':0.0171051)'':-0.0117699)'':0.00426781)'':0.0121323)'':0.0144656)'':0.00198299,(((('PDT000161047.2':0.0184368,'PDT000260340.2':0.0224642)'':0.00993049,('PDT000022689.1':0.0274923,('PDT000051550.1':0.0166007,('PDT000369021.1':0.0220924,(('PDT000161009.2':0.0111332,('PDT000160765.2':0.00592375,'PDT000161035.2':0.00669925)'':0.00358518)'':0.00210558,(('PDT000585920.1':0.00376447,'PDT000008805.2':0.00668743)'':0.00728973,('PDT000225353.1':0.0117552,'PDT000004465.2':0.0108912)'':0.00050794)'':0.00175645)'':0.00390658)'':0.00181173)'':0.00502059)'':-0.000667544)'':-0.00182635,((('PDT000345898.1':0.0107756,'PDT000345906.1':0.00661744)'':0.00856711,('PDT000642470.1':0.0124431,('PDT000455637.1':0.0142654,('PDT000345897.1':0.00847378,('PDT000642465.1':0.00970722,'PDT000145808.1':0.00969958)'':0.00111927)'':0.00253901)'':0.00217376)'':0.00608241)'':0.00121934,(('PDT000111925.1':0.0191734,'PDT000230873.1':0.0194207)'':0.00525778,(('PDT000455639.1':0.0160942,('PDT000345895.1':0.0122713,'PDT000455645.1':0.0110378)'':0.00291597)'':0.000818017,(('PDT000037692.1':0.00541977,'PDT000037693.1':0.00493983)'':0.00717811,('PDT000132504.1':0.0159709,('PDT000132505.1':0.0141153,('PDT000216705.2':0.0104263,('PDT000132497.1':0.0142985,('PDT000132495.1':0.00456649,('PDT000132498.1':0.00385837,'PDT000132507.1':0.00262348)'':0.00140306)'':0.00487834)'':0.00175217)'':0.00222172)'':0.000802381)'':-0.000448429)'':0.000613208)'':0.0052419)'':-0.00214213)'':0.006965)'':0.00671327,((('PDT000161603.2':0.0115219,'PDT000191792.2':0.012097)'':0.0111733,(('PDT000157680.2':0.014536,('PDT000158695.2':0.00897512,(('PDT000610533.1':0.00563402,'PDT000610625.1':0.00634918)'':0.00364695,(('PDT000523447.1':0.00294687,'PDT000598458.1':0.00273892)'':0.00291474,('PDT000316871.1':0.00658378,('PDT000157160.2':0.00413672,'PDT000160156.2':0.00294487)'':0.00213862)'':0.00108215)'':0.00162531)'':0.0027667)'':0.00418067)'':0.00142337,(('PDT000155488.2':0.0148484,'PDT000547687.1':0.0146741)'':0.00491572,(('PDT000155450.2':0.0107936,('PDT000383917.1':0.00400539,'PDT000576247.1':0.00416601)'':0.00481444)'':0.00828884,(('PDT000160175.2':0.0107041,'PDT000120964.2':0.0103904)'':0.0013435,('PDT000318576.1':0.0109591,('PDT000316184.1':0.0130411,(('PDT000191798.2':0.00667392,'PDT000260403.2':0.00556608)'':0.00298823,('PDT000658750.1':0.00810871,('PDT000610636.1':0.00878364,('PDT000158906.2':0.00425535,'PDT000205682.2':0.0051156)'':0.00193029)'':0.0018024)'':-0.000283445)'':0.0016247)'':0.00244813)'':0.000496554)'':0.00144068)'':0.00200872)'':-0.0016029)'':0.0147204)'':0.00590269,(((('PDT000283228.2':0.020775,'PDT000302832.2':0.0200413)'':0.00819237,('PDT000260669.2':0.0302548,('PDT000260416.2':0.0227294,('PDT000260378.2':0.0122582,('PDT000191786.2':0.00767024,'PDT000260387.2':0.00683836)'':0.00569625)'':0.00411383)'':0.00517032)'':0.00457531)'':0.00110619,(('PDT000624165.1':0.0308127,(('PDT000260312.2':0.00523443,'PDT000260316.2':0.00502347)'':0.0054676,('PDT000066961.1':0.00944487,('PDT000322976.1':0.00719255,('PDT000260318.2':0.00452293,'PDT000260663.2':0.00326928)'':0.003294)'':0.00300301)'':0.0040813)'':0.0135778)'':-0.007523,(('PDT000378692.1':0.0119769,('PDT000634360.1':0.0147191,('PDT000206515.2':0.00726494,'PDT000634378.1':0.00846746)'':0.00175483)'':0.00412699)'':0.00815473,(('PDT000372122.1':0.0101903,'PDT000619223.1':0.00871326)'':0.0103161,('PDT000292861.1':0.0258494,(('PDT000548639.1':0.00691034,('PDT000489286.1':0.00576541,(('PDT000320724.1':0.00440821,'PDT000594683.1':0.00515069)'':0.00037858,('PDT000366231.1':0.00533717,'PDT000581142.1':0.00552153)'':0.000633995)'':0.00105266)'':0.000342382)'':0.00150669,(('PDT000351930.1':0.00988056,'PDT000690242.1':0.00956694)'':0.00410994,('PDT000480464.1':0.0158243,(('PDT000494858.1':0.00714277,'PDT000705817.1':0.00810383)'':0.00142828,(('PDT000242626.2':0.00802433,('PDT000148281.2':0.00702815,'PDT000357298.1':0.00699895)'':0.00170862)'':0.000877683,(('PDT000634374.1':0.00427432,'PDT000610629.1':0.00560789)'':0.00261633,('PDT000651743.1':0.0112344,('PDT000250868.2':0.0126641,('PDT000616930.1':0.00796603,((('PDT000650912.1':0.00413627,'PDT000722023.1':0.00435439)'':0.00150705,('PDT000551003.1':0.00464713,('PDT000417904.1':0.00389427,('PDT000455292.1':0.00361366,'PDT000569369.1':0.00416844)'':0.000784139)'':0.000233797)'':0.000407255)'':0.000537475,((('PDT000321963.1':0.00363964,'PDT000494966.1':0.00483071)'':0.0008945,('PDT000190755.2':0.00485401,('PDT000533253.1':0.00381853,'PDT000653451.1':0.00375723)'':0.000432623)'':0.000918771)'':0.0005159,('PDT000378702.1':0.00546235,('PDT000628291.1':0.00461239,('PDT000562689.1':0.00474105,('PDT000311113.2':0.00500798,('PDT000624853.1':0.00329831,('PDT000570309.1':0.00268185,('PDT000687056.1':0.00283242,('PDT000173601.2':0.0027135,'PDT000480467.1':0.00187686)'':0.000958081)'':0.000421356)'':0.000284395)'':0.000194057)'':0.000836022)'':0.000157245)'':0.00112675)'':-0.000975634)'':0.000643515)'':0.00120852)'':0.00186931)'':0.000445722)'':-0.00137557)'':0.000200026)'':0.000850396)'':0.00228137)'':0.000536723)'':-0.00292679)'':0.0106098)'':0.00582278)'':-0.00403091)'':-0.000521489)'':0.0047847)'':0.00216346,(((('PDT000285343.1':0.0139013,'PDT000285344.1':0.01026)'':0.0275009,(('PDT000394234.1':0.0122323,('PDT000414247.1':0.00554157,('PDT000325737.1':0.00413202,('PDT000455660.1':0.00521064,('PDT000320858.1':0.00355465,('PDT000593746.1':0.00287795,'PDT000688099.1':0.00221136)'':0.00103068)'':0.000451426)'':0.000382504)'':0.000957766)'':0.00417072)'':0.00478521,(('PDT000627790.1':0.0163133,('PDT000634368.1':0.0150187,('PDT000611847.1':0.0114793,'PDT000346266.1':0.0114761)'':0.00284555)'':0.000411464)'':0.0005339,(('PDT000353567.1':0.00986534,((('PDT000624232.1':0.00533327,('PDT000519313.1':0.00518967,'PDT000649139.1':0.00487513)'':0.00048298)'':0.00189566,('PDT000327225.1':0.00945349,('PDT000467632.1':0.00494265,('PDT000318684.1':0.00342813,'PDT000333982.1':0.00275949)'':0.00107664)'':0.00292139)'':-0.00031754)'':0.00225896,('PDT000147780.2':0.0117549,('PDT000326128.1':0.0121444,('PDT000148161.2':0.0109764,(('PDT000635987.1':0.00398382,'PDT000687075.1':0.00468843)'':0.00175186,('PDT000158894.2':0.00546301,(('PDT000327218.1':0.00441356,('PDT000318579.1':0.00230431,'PDT000156981.2':0.00258835)'':0.000964176)'':0.000679283,('PDT000326123.1':0.00466737,('PDT000205390.2':0.00336932,('PDT000156940.2':0.00480852,'PDT000121042.2':0.00346006)'':2.64935e-05)'':0.00150444)'':-0.000104165)'':0.00198288)'':-0.000411521)'':0.00256388)'':0.000763002)'':0.00100872)'':-0.000867556)'':0.00154342)'':-0.000478093,(('PDT000628363.1':0.012762,'PDT000610745.1':0.0116185)'':0.00468242,('PDT000559232.1':0.0130482,('PDT000173323.1':0.022163,('PDT000346270.1':0.0120758,(('PDT000369019.1':0.00458734,'PDT000369022.1':0.00556796)'':0.00468218,('PDT000455629.1':0.0137836,(('PDT000455690.1':0.00267251,'PDT000455701.1':0.00251608)'':0.00274621,('PDT000346263.1':0.00710854,(('PDT000455691.1':0.00362625,'PDT000346283.1':0.00365192)'':0.0014381,(('PDT000455659.1':0.00393101,(('PDT000368956.1':0.00292739,'PDT000368958.1':0.0024656)'':0.000504892,('PDT000455698.1':0.00398139,('PDT000610582.1':0.00150483,'PDT000455683.1':0.00139112)'':0.000907871)'':0.000317691)'':0.00144615)'':-0.000466803,('PDT000455696.1':0.00313905,('PDT000455651.1':0.00204258,('PDT000455687.1':0.00189941,('PDT000346265.1':0.00137406,('PDT000455655.1':0.000814443,'PDT000455670.1':0.000784597)'':0.000323881)'':0.000422376)'':0.000906387)'':0.00115683)'':-0.000204873)'':0.000859696)'':0.002908)'':-0.001403)'':0.00293047)'':-0.0015039)'':0.00232191)'':0.000837773)'':0.00282871)'':0.000736475)'':-0.00258121)'':0.00293464)'':-0.000212396)'':0.0137369)'':-0.00755969,((((('PDT000600096.1':0.00321479,'PDT000682794.1':0.00346584)'':0.00266062,('PDT000317224.1':0.00507743,('PDT000190377.2':0.00476947,('PDT000324159.1':0.00319184,('PDT000315574.1':0.00231851,('PDT000157005.2':0.00294884,'PDT000327385.1':0.00204716)'':7.55805e-05)'':0.00121284)'':0.000850661)'':0.0011957)'':0.000942509)'':0.00971517,(('PDT000398055.1':0.00945507,('PDT000121014.2':0.00489181,'PDT000721760.1':0.00639799)'':0.00366258)'':0.00414525,('PDT000494136.1':0.0171143,('PDT000321622.1':0.00694598,('PDT000470257.1':0.00566822,('PDT000161585.2':0.00283681,'PDT000322965.1':0.00325588)'':0.00199916)'':0.00404191)'':0.00939946)'':-0.00284058)'':0.00101951)'':0.00321218,(('PDT000317227.1':0.0149652,('PDT000316511.1':0.00776824,'PDT000409976.1':0.00818386)'':0.00409421)'':0.00124122,((('PDT000464599.1':0.00473579,'PDT000610715.1':0.00535811)'':0.00116919,('PDT000350565.1':0.00598838,('PDT000157727.2':0.00137319,'PDT000328044.1':0.00122421)'':0.00328492)'':0.00029218)'':0.00430025,(('PDT000635980.1':0.014205,('PDT000627050.1':0.00463214,('PDT000410929.1':0.00299032,'PDT000607828.1':0.0026983)'':0.00273659)'':0.0091639)'':-0.00122132,('PDT000667979.1':0.0173128,(('PDT000611944.1':0.00827228,('PDT000600100.1':0.00486462,('PDT000601079.1':0.00371331,('PDT000328005.1':0.00337022,'PDT000413910.1':0.00382187)'':0.000420755)'':0.000485534)'':0.00227619)'':0.00072811,(('PDT000610656.1':0.00566242,'PDT000368955.1':0.00548848)'':0.00226444,(('PDT000148196.2':0.00517162,('PDT000147768.2':0.00277745,'PDT000317297.1':0.0024101)'':0.00198963)'':0.00113348,('PDT000481090.1':0.00756155,('PDT000314317.2':0.00432543,('PDT000156628.2':0.00161117,('PDT000316206.1':0.00119164,'PDT000155300.2':0.00120596)'':0.000484853)'':0.00192199)'':0.00327048)'':-0.000899193)'':0.00111572)'':0.000729462)'':0.0057535)'':-0.00272612)'':0.00069942)'':0.00133847)'':0.00465707)'':0.00423107,(('PDT000260301.2':0.0193524,('PDT000525300.1':0.034346,('PDT000598663.1':0.0217134,('PDT000607928.1':0.0113926,('PDT000351931.1':0.00626845,('PDT000312552.2':0.00526644,('PDT000481083.1':0.00547,'PDT000525268.1':0.0039997)'':0.00129394)'':0.00189994)'':0.00667348)'':0.00383344)'':0.00465254)'':0.00580898)'':0.00208889,('PDT000289992.1':0.0408864,(('PDT000161004.2':0.0265272,(('PDT000161021.2':0.00970601,'PDT000161034.2':0.00577789)'':0.0114075,('PDT000161041.2':0.0179997,('PDT000160814.2':0.0144642,('PDT000161016.2':0.0103556,'PDT000022685.1':0.0112115)'':0.00367896)'':0.00211556)'':0.00233506)'':0.00119539)'':-0.000549893,(('PDT000651406.1':0.022999,('PDT000157076.2':0.00980873,'PDT000328075.1':0.0115355)'':0.0070937)'':0.00337504,('PDT000289994.1':0.0199381,(('PDT000260662.2':0.0141277,'PDT000007276.2':0.01465)'':0.000170058,(('PDT000467780.1':0.00834997,'PDT000601102.1':0.00877643)'':0.0100097,('PDT000720690.1':0.0205321,((('PDT000320879.1':0.00346034,'PDT000157816.2':0.00282144)'':0.00404628,(('PDT000455743.1':0.00488685,'PDT000455745.1':0.0046749)'':0.000377099,('PDT000455741.1':0.00441448,('PDT000610664.1':0.00342439,('PDT000455744.1':0.00320499,('PDT000380936.1':0.00187415,'PDT000380939.1':0.00282162)'':0.000385335)'':0.00106403)'':0.000705689)'':0.00064016)'':0.00265942)'':0.0015149,((('PDT000342684.1':0.0082097,'PDT000379536.1':0.0059281)'':0.00140689,('PDT000626576.1':0.0065056,('PDT000390819.1':0.00387702,'PDT000649807.1':0.00399229)'':0.00136169)'':0.000873025)'':0.00158766,('PDT000319267.1':0.0115915,(('PDT000638592.1':0.0067702,('PDT000716370.1':0.00721989,('PDT000549589.1':0.00535698,(('PDT000412495.1':0.00283594,'PDT000600110.1':0.00285041)'':0.00114918,('PDT000489902.1':0.00383808,('PDT000395701.1':0.00257324,'PDT000643667.1':0.00301447)'':0.00134456)'':0.000194291)'':0.000997544)'':0.000689879)'':0.00173585)'':-3.92841e-05,('PDT000148091.2':0.0144454,((('PDT000260381.2':0.00388383,'PDT000705818.1':0.00507942)'':0.00190457,('PDT000470231.1':0.005086,'PDT000628863.1':0.00397323)'':0.00156164)'':0.000239329,(('PDT000147802.2':0.00405714,'PDT000350562.1':0.00462463)'':0.00153692,(('PDT000121010.2':0.00396459,'PDT000499261.1':0.00490702)'':0.00160637,('PDT000315512.1':0.00524413,('PDT000327431.1':0.00299916,('PDT000156757.2':0.00276517,'PDT000677123.1':0.00322166)'':0.000147061)'':0.0017595)'':0.000730178)'':0.000677408)'':0.000290625)'':0.0037601)'':-0.00281488)'':0.00296166)'':-0.00222273)'':0.00040656)'':0.00606288)'':-0.00232167)'':0.00102761)'':0.00494956)'':-0.00121301)'':0.00546904)'':0.0105056)'':-0.00975952)'':-0.0046524)'':0.000158712)'':0.00114287,((('PDT000155323.2':0.0280505,(('PDT000260349.2':0.0147494,'PDT000260389.2':0.0150202)'':0.00542977,(('PDT000260348.2':0.010374,'PDT000260404.2':0.00985411)'':0.00367091,('PDT000710980.1':0.0116424,('PDT000345835.1':0.00783059,'PDT000455667.1':0.00826321)'':0.00458367)'':4.94709e-05)'':0.00654509)'':0.00463189)'':-0.000986386,((('PDT000551380.1':0.018638,'PDT000455679.1':0.0155112)'':0.00877796,((('PDT000293880.2':0.0130416,'PDT000465914.1':0.0113752)'':0.00127073,('PDT000570795.1':0.0121907,'PDT000708484.1':0.0116259)'':0.00199942)'':0.00540877,('PDT000551622.1':0.0225515,('PDT000387671.1':0.0229377,('PDT000356301.1':0.0209102,(('PDT000668275.2':0.00486878,('PDT000564058.1':0.00312768,'PDT000616904.1':0.00296075)'':0.00219212)'':0.00617996,('PDT000627793.1':0.0113784,('PDT000295764.2':0.00931026,('PDT000627794.1':0.00259131,'PDT000703132.1':0.0024027)'':0.00555309)'':0.00264281)'':0.00029685)'':0.00353964)'':0.00415684)'':0.00154375)'':-0.000575199)'':0.00543036)'':-0.00164988,(('PDT000022709.1':0.0243117,('PDT000132509.1':0.0217383,('PDT000022673.1':0.00263695,('PDT000022667.1':0.00126779,'PDT000022677.1':0.00132883)'':0.000361389)'':0.0151769)'':0.0034378)'':0.00276383,(('PDT000644093.1':0.022647,('PDT000644107.1':0.0271464,(('PDT000644090.1':0.0148836,'PDT000644152.1':0.0211077)'':0.00149255,(('PDT000644087.1':0.00869303,'PDT000352557.1':0.00861007)'':0.00367097,('PDT000644089.1':0.0112216,('PDT000644064.1':0.0106161,'PDT000644103.1':0.00905599)'':0.00153362)'':0.0028241)'':0.00537731)'':0.00410895)'':0.00188474)'':-0.00247627,('PDT000617875.2':0.0374457,('PDT000679775.1':0.0197989,(('PDT000121038.2':0.012841,('PDT000562998.1':0.0118764,('PDT000672494.1':0.0110426,('PDT000320206.1':0.00811072,'PDT000148001.2':0.00835918)'':0.000923424)'':0.00114357)'':0.00205785)'':0.00135717,('PDT000455627.1':0.0174054,('PDT000259487.1':0.0121569,('PDT000148048.2':0.0105976,('PDT000670818.1':0.00963955,('PDT000610548.1':0.00645339,'PDT000345901.1':0.00716901)'':0.0043071)'':0.000816396)'':0.0012122)'':0.00192582)'':-0.000441618)'':0.00300853)'':0.00953712)'':-0.00777476)'':0.002293)'':0.00085203)'':0.00217844)'':0.00293425,(((('PDT000621343.1':0.010858,('PDT000629720.1':0.00752762,'PDT000470455.1':0.0122329)'':0.00459007)'':0.00866511,(('PDT000644070.1':0.0196994,('PDT000628595.1':0.0106863,'PDT000721695.1':0.0098087)'':0.00941431)'':0.00332669,((('PDT000247469.2':0.00964916,'PDT000598664.1':0.0104518)'':0.00284129,('PDT000321974.1':0.0137371,('PDT000413912.1':0.00953202,('PDT000599178.1':0.00495723,('PDT000378047.1':0.00422988,('PDT000544345.1':0.00285415,'PDT000634370.1':0.00233599)'':0.00186421)'':0.00172376)'':0.00610077)'':0.00163117)'':0.000583886)'':0.000977781,(('PDT000484031.1':0.00893253,('PDT000412863.1':0.00508161,'PDT000689145.1':0.00685869)'':0.00278957)'':0.00441306,('PDT000197053.2':0.0141467,('PDT000522473.1':0.00986533,('PDT000646334.1':0.00991816,('PDT000523225.1':0.00721486,(('PDT000311687.2':0.00274984,'PDT000509670.1':0.00223968)'':0.00370935,('PDT000532784.1':0.00468087,('PDT000426912.1':0.00341656,'PDT000714606.1':0.00306852)'':0.000604479)'':0.000587537)'':0.00152449)'':0.000927663)'':0.000865217)'':0.00260308)'':-0.00050961)'':0.0021702)'':0.00533651)'':0.00340634)'':-0.00120597,((('PDT000481079.1':0.01449,'PDT000484864.1':0.0153401)'':0.00312591,('PDT000465633.1':0.0143832,('PDT000292863.1':0.0167599,('PDT000455677.1':0.0127296,('PDT000190451.2':0.0074099,('PDT000455666.1':0.0070109,('PDT000345836.1':0.00316806,'PDT000455631.1':0.00451887)'':0.00169574)'':0.00238426)'':0.00114626)'':0.00362009)'':0.00263524)'':0.00392002)'':0.00236856,(('PDT000644127.1':0.0187477,('PDT000610819.1':0.0169871,'PDT000112850.1':0.0139938)'':0.00682217)'':0.000288875,((('PDT000327906.1':0.00321274,'PDT000332130.1':0.00267137)'':0.0109052,(('PDT000327886.1':0.00775843,'PDT000155801.2':0.00813507)'':0.00227219,('PDT000624244.1':0.00856906,('PDT000448154.1':0.00615251,('PDT000148101.2':0.00366847,'PDT000484869.1':0.00360679)'':0.00209781)'':0.00245241)'':0.0011517)'':0.00590056)'':-0.000308683,(('PDT000040354.2':0.0156168,('PDT000212785.1':0.0111905,('PDT000644074.1':0.0042004,'PDT000644112.1':0.00496751)'':0.00643795)'':0.00284831)'':0.00120144,('PDT000644083.1':0.0206877,('PDT000022694.1':0.0150557,('PDT000679704.1':0.00987323,('PDT000644091.1':0.00874075,('PDT000644080.1':0.0048329,'PDT000644119.1':0.0053356)'':0.00314685)'':0.00187006)'':0.00381469)'':0.00133965)'':-0.000441859)'':0.0018139)'':0.00727016)'':-0.00297248)'':0.000131351)'':0.00202599,(((('PDT000292851.1':0.01387,('PDT000292872.1':0.00877332,'PDT000292873.1':0.0145126)'':0.00228582)'':0.00929296,(('PDT000066073.1':0.0135605,('PDT000066067.1':0.0107275,'PDT000066071.1':0.0099341)'':0.00459579)'':0.00579277,('PDT000066077.1':0.0167219,('PDT000066075.1':0.00800385,('PDT000066068.1':0.00845358,'PDT000066076.1':0.00832392)'':0.00142665)'':0.00818263)'':-0.000684553)'':0.00228665)'':0.00128491,('PDT000292874.1':0.0328707,('PDT000292868.2':0.0198405,('PDT000066074.1':0.0141644,('PDT000292870.1':0.0130044,('PDT000524413.2':0.0155699,('PDT000318191.1':0.0113416,('PDT000040366.7':0.0114177,('PDT000443299.2':0.00700394,('PDT000711422.1':0.00633606,(('PDT000066072.1':0.00354305,'PDT000072668.1':0.00403951)'':0.00243223,('PDT000649509.2':0.00490864,('PDT000362747.2':0.00239199,('PDT000687780.1':0.0020297,('PDT000687781.1':0.00145021,('PDT000190449.3':0.00149125,'PDT000342352.2':0.0014047)'':0.000148304)'':0.000917733)'':0.000415306)'':0.00267383)'':-0.000149134)'':0.00145369)'':0.000210936)'':0.00251164)'':0.00101091)'':0.00208217)'':0.000586946)'':0.00262748)'':0.000673282)'':0.00333455)'':-3.5135e-05)'':0.00720685,((('PDT000132503.1':0.0296853,('PDT000132488.1':0.0296935,('PDT000644142.1':0.0254775,('PDT000644144.1':0.0190532,'PDT000644151.1':0.0166011)'':0.0057308)'':0.000838267)'':0.0111114)'':-0.00429422,('PDT000644115.1':0.030928,((('PDT000467581.1':0.00843711,'PDT000644078.1':0.0103315)'':0.00889602,('PDT000369024.1':0.0234257,('PDT000644108.1':0.0190822,'PDT000494192.1':0.0154859)'':0.00325043)'':0.0014295)'':0.00163296,(('PDT000626976.1':0.023971,('PDT000585272.1':0.0160197,('PDT000191694.2':0.0131608,'PDT000500637.1':0.0124169)'':0.00303487)'':0.00179803)'':-0.000623655,('PDT000644371.1':0.0198074,(('PDT000467554.1':0.0131492,'PDT000368952.1':0.0140381)'':0.00360764,('PDT000667889.2':0.0131376,('PDT000120994.2':0.0109749,('PDT000155657.2':0.00972264,'PDT000378529.1':0.00902926)'':0.00226826)'':0.00154383)'':0.00300363)'':0.00237636)'':-0.000799505)'':0.00184875)'':0.00616519)'':0.0026974)'':-0.00170586,(('PDT000644150.1':0.0456179,('PDT000644139.1':0.0361277,'PDT000644153.1':0.0409296)'':0.00559444)'':0.0076306,((('PDT000644095.1':0.036289,(('PDT000644121.1':0.0125735,'PDT000644140.1':0.0111536)'':0.00580559,('PDT000022668.1':0.0137181,'PDT000022676.1':0.0136372)'':0.00336866)'':0.00773724)'':-0.00240285,(('PDT000022721.1':0.0205298,('PDT000022710.1':0.0155395,(('PDT000022701.1':0.0124866,('PDT000022711.1':0.0111529,'PDT000022712.1':0.0103216)'':0.00405668)'':0.000438674,('PDT000022674.1':0.0150666,('PDT000022719.1':0.0105351,('PDT000022718.1':0.0105481,('PDT000022716.1':0.00484042,('PDT000022684.1':0.00437538,'PDT000022714.1':0.00410089)'':0.00113164)'':0.00369329)'':0.000563264)'':0.00138431)'':2.43917e-05)'':0.00237869)'':0.00288232)'':0.00116241,(('PDT000132496.1':0.0222948,('PDT000644066.1':0.0152688,('PDT000216698.1':0.00912868,('PDT000216697.1':0.00739018,'PDT000216709.1':0.00621732)'':0.00143617)'':0.00691799)'':0.00406442)'':0.000751778,('PDT000132502.1':0.0244776,('PDT000156907.2':0.0130902,(('PDT000600039.1':0.00989977,('PDT000315551.1':0.00680865,'PDT000467577.1':0.00956355)'':0.00285168)'':-0.000130729,('PDT000644137.1':0.00564792,('PDT000007549.2':0.00535527,('PDT000644134.1':0.00452105,('PDT000644063.1':0.00252991,'PDT000644128.1':0.00295966)'':0.000854858)'':0.000624751)'':0.000842565)'':0.00534981)'':0.00283777)'':0.00659096)'':-0.00463019)'':0.0025619)'':0.00236337)'':0.000647962,((('PDT000635726.1':0.0165917,('PDT000155957.2':0.00418262,'PDT000120971.2':0.00369532)'':0.0142555)'':0.00118323,(('PDT000121012.2':0.0150823,'PDT000467485.1':0.0197159)'':0.00487258,(('PDT000191787.2':0.0164764,(('PDT000046992.1':0.00866109,('PDT000292852.1':0.00859579,(('PDT000455658.1':0.00222586,'PDT000455664.1':0.00216799)'':7.71397e-05,('PDT000346260.1':0.00143712,('PDT000346259.1':0.000590185,('PDT000346261.1':0.000332001,'PDT000346264.1':0.000367859)'':0.00040928)'':0.000225618)'':0.000864009)'':0.00419257)'':0.00322213)'':-0.0015504,(('PDT000190967.2':0.00562311,'PDT000191789.2':0.00513069)'':0.00150684,('PDT000327323.1':0.00980072,('PDT000617877.1':0.00564609,(('PDT000386119.1':0.00418633,'PDT000392378.1':0.00457686)'':0.000231536,('PDT000222074.2':0.00515511,('PDT000467412.1':0.00380093,'PDT000603003.1':0.00327642)'':0.000716616)'':0.000265569)'':0.00193762)'':0.00154877)'':-0.000850493)'':0.000101638)'':0.00868973)'':-0.00648046,('PDT000176406.2':0.0222976,(('PDT000607598.1':0.00501232,'PDT000669588.1':0.00514608)'':0.00262001,('PDT000121022.2':0.0120436,('PDT000332142.1':0.00615758,('PDT000464597.1':0.00630513,(('PDT000467417.1':0.00303805,'PDT000467488.1':0.0026466)'':0.00176931,(('PDT000322777.1':0.00420478,'PDT000148033.2':0.00556981)'':0.000783724,(('PDT000327312.1':0.00422185,'PDT000121040.2':0.00384588)'':0.000382441,(('PDT000157019.2':0.00248868,('PDT000155563.2':0.00244762,'PDT000160497.2':0.00224253)'':0.000403647)'':0.000204183,('PDT000157994.2':0.00230485,('PDT000157729.2':0.00220392,('PDT000322107.1':0.0014162,'PDT000155247.2':0.00177996)'':0.000390006)'':0.000225317)'':0.00140622)'':0.000864219)'':0.000509112)'':0.000716305)'':0.00115389)'':0.00105774)'':0.000828259)'':-0.000409481)'':0.0122316)'':-0.0115814)'':0.00809725)'':-0.000638329)'':0.000824244,((('PDT000467762.1':0.0253576,'PDT000634812.1':0.0168034)'':0.0170083,(('PDT000260346.2':0.0213029,('PDT000318854.1':0.0137113,'PDT000520401.1':0.0130462)'':0.00774702)'':-0.000307008,('PDT000293099.2':0.0205373,(('PDT000222754.2':0.0219769,(('PDT000190552.2':0.0102747,'PDT000309646.2':0.0109555)'':0.001253,(('PDT000596026.1':0.0125616,('PDT000147830.2':0.0122339,'PDT000157987.2':0.0116957)'':0.00107246)'':0.000143086,('PDT000596030.1':0.0150359,(('PDT000321604.1':0.010181,('PDT000155771.2':0.00478283,'PDT000158705.2':0.00397862)'':0.0030472)'':0.000247668,('PDT000157873.2':0.0109197,(('PDT000316604.1':0.00616211,'PDT000320177.1':0.00478429)'':0.00240424,(('PDT000155346.2':0.00711462,'PDT000156545.2':0.00505128)'':0.00105831,('PDT000161570.2':0.0042391,(('PDT000315501.1':0.00359508,'PDT000155862.2':0.00329321)'':0.000717727,('PDT000155769.2':0.00362488,'PDT000155986.2':0.00485731)'':0.00016918)'':0.00138486)'':8.91667e-05)'':0.00195373)'':0.00163592)'':-0.00068682)'':0.00399692)'':-0.0012344)'':0.00234841)'':0.00424203)'':-0.00647104,('PDT000672487.1':0.0294006,((('PDT000398036.1':0.00764222,'PDT000692386.1':0.00905928)'':0.0159065,(('PDT000641412.1':0.0188315,'PDT000683252.1':0.0161073)'':0.00482731,(('PDT000496091.1':0.0113501,('PDT000667885.2':0.00759628,'PDT000683254.1':0.00730532)'':0.0022051)'':0.00392033,(('PDT000500640.1':0.012205,(('PDT000472548.1':0.00562721,('PDT000498962.1':0.00504847,'PDT000394286.1':0.0048062)'':0.000651904)'':0.000836308,('PDT000710976.1':0.00782379,('PDT000601094.1':0.0065756,(('PDT000466213.1':0.00433893,'PDT000677480.1':0.00403261)'':0.000692033,('PDT000578746.1':0.0044071,('PDT000711110.1':0.00438346,'PDT000588088.1':0.00388512)'':0.00017221)'':0.00085129)'':0.000235082)'':0.00181428)'':-0.000183811)'':0.00355313)'':-0.00207814,('PDT000417563.1':0.0211863,(('PDT000610295.1':0.0101137,'PDT000678041.1':0.00862329)'':0.00389674,('PDT000537442.1':0.00980684,('PDT000422734.1':0.0103332,(('PDT000691458.1':0.00699986,('PDT000491606.1':0.00462931,'PDT000689097.1':0.00463272)'':0.00137122)'':9.29177e-05,((('PDT000341782.1':0.00562633,(('PDT000377981.1':0.00431741,'PDT000466237.1':0.00406499)'':0.000449354,('PDT000377174.1':0.00421855,'PDT000651773.1':0.00345459)'':0.000493333)'':0.00149914)'':-0.000985436,(('PDT000479190.1':0.00359091,'PDT000660036.1':0.00368363)'':0.000654233,('PDT000599126.1':0.00379195,('PDT000394262.1':0.00317248,'PDT000417566.1':0.00281136)'':0.000593493)'':0.000512411)'':0.000680297)'':0.000922981,(('PDT000382889.1':0.00403781,'PDT000524125.1':0.00522883)'':0.0018237,('PDT000586502.1':0.00818024,(('PDT000611957.1':0.00473,('PDT000641408.1':0.00463207,('PDT000648347.1':0.00345029,('PDT000484211.1':0.00325317,'PDT000721087.1':0.00283465)'':0.000534269)'':0.000600749)'':0.000420239)'':0.000483191,('PDT000475743.1':0.00530173,('PDT000679862.1':0.00703128,('PDT000383185.1':0.00407679,('PDT000482556.1':0.00396795,('PDT000711413.1':0.00330813,(('PDT000517240.1':0.00238609,'PDT000711116.1':0.00220427)'':0.000261674,('PDT000476885.1':0.00285213,('PDT000486913.1':0.00222825,'PDT000640692.1':0.00206274)'':0.000541273)'':0.000154682)'':0.000552052)'':0.000461209)'':0.00011014)'':0.00126148)'':0.000747897)'':-0.00125709)'':0.00247169)'':-0.000737851)'':-0.000296344)'':0.000801291)'':0.00261976)'':0.000499496)'':-0.000334893)'':0.0063625)'':-0.0079008)'':0.00262456)'':0.0046749)'':0.00156841)'':-0.006995,(('PDT000627199.1':0.0126303,(('PDT000317719.1':0.00562865,('PDT000498949.1':0.00424485,'PDT000540381.1':0.00393307)'':0.0014523)'':0.000840382,(('PDT000452812.1':0.00551045,'PDT000656083.1':0.00643935)'':0.000749161,('PDT000500619.1':0.00509284,'PDT000689100.1':0.00546386)'':0.00112846)'':0.00049203)'':0.00299903)'':-0.000416315,(('PDT000467785.1':0.012205,('PDT000467479.1':0.00261242,'PDT000467576.1':0.00277626)'':0.0069218)'':0.0120061,(('PDT000702608.1':0.00943757,('PDT000649128.1':0.00810586,('PDT000156826.2':0.00516985,'PDT000157217.2':0.00419643)'':0.00396625)'':0.000791119)'':0.00203594,((('PDT000155296.2':0.00396787,('PDT000156666.2':0.00258296,'PDT000157143.2':0.00260407)'':0.00162378)'':0.00212777,('PDT000467759.1':0.00898666,('PDT000611848.1':0.00448079,('PDT000148223.2':0.00467066,'PDT000155943.2':0.00421424)'':0.000792909)'':0.00126618)'':-0.000403924)'':0.000583903,(('PDT000327126.1':0.0129481,('PDT000562694.1':0.00918641,('PDT000124775.2':0.00991062,('PDT000148266.2':0.0115272,('PDT000155597.2':0.00791196,('PDT000470223.1':0.00600308,((('PDT000325182.1':0.00288787,'PDT000678228.1':0.00290865)'':0.000442179,('PDT000157047.2':0.0034441,('PDT000317208.1':0.00216191,('PDT000323722.1':0.0018712,'PDT000629860.1':0.00222102)'':0.000432858)'':0.000894401)'':6.96672e-05)'':0.000599721,('PDT000160165.2':0.00469129,('PDT000156673.2':0.00401211,('PDT000157109.2':0.00250807,('PDT000467408.1':0.00261446,((('PDT000319849.1':0.002105,'PDT000205392.2':0.00198804)'':0.000141406,('PDT000319268.1':0.00216856,'PDT000120958.2':0.00242272)'':0.000358142)'':7.11109e-05,('PDT000327203.1':0.00229075,('PDT000322758.1':0.00223077,('PDT000148133.2':0.0019076,('PDT000160183.2':0.00157632,('PDT000156531.2':0.00184278,('PDT000148138.2':0.00138358,'PDT000157926.2':0.00141278)'':0.000253087)'':0.000137324)'':0.000376001)'':0.000151411)'':0.000228462)'':-2.97992e-05)'':0.000385563)'':0.000135963)'':0.000572127)'':0.000315695)'':-0.000462965)'':0.00161903)'':0.00125432)'':0.000666413)'':0.00116189)'':0.000970909)'':0.000872762)'':-0.00446159,('PDT000195695.2':0.00902671,(('PDT000316181.1':0.00485366,'PDT000599602.1':0.00470143)'':0.000850847,(('PDT000148175.2':0.00418913,'PDT000156706.2':0.00457756)'':0.00010224,('PDT000319308.1':0.00567561,(('PDT000318653.1':0.00448436,('PDT000315530.1':0.00223657,'PDT000327901.1':0.00225714)'':0.0015878)'':4.30123e-05,(('PDT000322942.1':0.00363265,('PDT000156039.2':0.00371005,('PDT000155292.2':0.00349126,('PDT000322029.1':0.00267106,'PDT000160176.2':0.00241724)'':0.000498394)'':0.000242059)'':0.00021442)'':0.000773519,('PDT000467776.1':0.00543449,(('PDT000465932.1':0.00230859,'PDT000634362.1':0.00248135)'':0.00156347,('PDT000161492.2':0.00334683,('PDT000326275.1':0.0032758,('PDT000160344.2':0.00380937,(('PDT000148162.2':0.00235804,'PDT000155790.2':0.00233117)'':0.00103802,(('PDT000316538.1':0.00207326,'PDT000320814.1':0.00182129)'':0.000623303,('PDT000467772.1':0.00244932,(('PDT000317759.1':0.00179287,('PDT000148248.2':0.00194151,'PDT000155793.2':0.0015529)'':0.000305438)'':0.000329293,('PDT000322776.1':0.00247645,('PDT000319856.1':0.00167992,'PDT000203332.2':0.00161645)'':0.000517478)'':0.000149047)'':0.000332304)'':-2.69724e-05)'':4.49767e-05)'':0.000125536)'':0.000670612)'':0.000520695)'':-0.000193859)'':0.000394457)'':-0.000491578)'':0.000374731)'':0.0010154)'':-0.000457609)'':0.00107457)'':0.00249262)'':-0.00307629)'':0.00188017)'':0.00190941)'':0.00394579)'':-0.0038622)'':0.000382781)'':0.019426)'':-0.0182578)'':0.0102769)'':-0.00486069)'':0.00685214)'':-0.0104318,('PDT000644145.1':0.0592998,((('PDT000455684.1':0.0113449,('PDT000345902.1':0.00371846,('PDT000455643.1':0.00210092,'PDT000455644.1':0.00169415)'':0.00256562)'':0.0059807)'':0.00555792,('PDT000132499.1':0.0191923,(('PDT000598665.1':0.0129635,'PDT000644069.1':0.0116063)'':0.00330399,('PDT000176403.2':0.0102856,(('PDT000455738.1':0.00445715,'PDT000368969.1':0.00579775)'':0.000838466,('PDT000345903.1':0.00487686,('PDT000455647.1':0.00456376,('PDT000345899.1':0.00474625,'PDT000455646.1':0.00412094)'':0.000663943)'':0.000689629)'':0.000505711)'':0.00601999)'':0.00109579)'':0.00351079)'':0.000617783)'':0.000177203,((('PDT000022707.1':0.019549,('PDT000022670.1':0.0114988,('PDT000644096.1':0.0088841,('PDT000022671.1':0.00430108,('PDT000022663.1':0.00185318,'PDT000022665.1':0.00253804)'':0.00147812)'':0.00412919)'':0.00299071)'':0.002215)'':0.000564562,(('PDT000022703.1':0.0114286,('PDT000022696.1':0.00836132,'PDT000022704.1':0.0110754)'':0.00107026)'':0.00259949,(('PDT000022675.1':0.00683259,'PDT000022678.1':0.00838981)'':0.00164366,(('PDT000022669.1':0.00713438,'PDT000022717.1':0.00736562)'':0.00155596,('PDT000022672.1':0.00819375,'PDT000022695.1':0.00739405)'':0.00101187)'':0.00195625)'':0.00244299)'':0.000727268)'':0.0042231,((('PDT000470241.1':0.0119708,('PDT000366312.1':0.0106715,'PDT000674798.1':0.00955263)'':0.000897857)'':0.00349171,(((('PDT000316550.1':0.00239136,'PDT000323319.1':0.00289912)'':0.00447,('PDT000328039.1':0.00700414,('PDT000394273.1':0.00395089,('PDT000469497.1':0.00209569,'PDT000623043.1':0.00229772)'':0.00123104)'':0.00228373)'':0.0022193)'':0.00568646,(('PDT000708802.1':0.00397472,('PDT000590342.1':0.00308765,('PDT000646321.1':0.00198007,('PDT000307924.2':0.00209513,'PDT000611864.1':0.00189808)'':0.000216828)'':0.000465147)'':0.00115062)'':0.00432572,('PDT000634356.1':0.0212318,('PDT000205684.2':0.0133289,('PDT000156572.2':0.0110906,('PDT000158026.2':0.00529937,(('PDT000319915.1':0.00650346,('PDT000467688.1':0.00393698,('PDT000324672.1':0.00360512,('PDT000319227.1':0.00257565,('PDT000315578.1':0.00269884,'PDT000326098.1':0.00238895)'':0.000217383)'':0.00133347)'':0.000304533)'':0.00164754)'':-0.000791579,(('PDT000333942.1':0.00364358,'PDT000332115.1':0.00413076)'':0.000694754,(('PDT000136960.2':0.00446292,'PDT000629784.1':0.00471414)'':9.30159e-05,('PDT000155942.2':0.00338161,(('PDT000316855.1':0.00211339,'PDT000158922.2':0.00217931)'':0.000496704,('PDT000268746.2':0.00136526,'PDT000270098.2':0.00113174)'':0.00144971)'':0.00102624)'':0.000891495)'':0.00029146)'':0.00071232)'':0.000113809)'':0.00225518)'':0.00174005)'':0.0038093)'':-0.00391408)'':0.00119081)'':-0.000551022,(('PDT000455634.1':0.0164517,('PDT000333304.1':0.00925075,'PDT000355585.1':0.00810865)'':0.0058628)'':0.000715121,('PDT000692208.1':0.0150308,('PDT000465619.1':0.0124607,('PDT000635849.1':0.0123493,(('PDT000390533.1':0.00694649,('PDT000240657.2':0.00567878,('PDT000469402.1':0.00538681,(('PDT000595706.1':0.00333766,'PDT000600473.1':0.00473409)'':0.00101224,('PDT000280738.2':0.00413727,('PDT000521236.1':0.00324592,('PDT000289074.2':0.00230248,'PDT000624018.1':0.00238954)'':0.00173746)'':0.000140093)'':0.000649037)'':0.000306045)'':0.000433972)'':0.00299136)'':-0.00170032,('PDT000547041.1':0.0100472,('PDT000667983.1':0.00840241,('PDT000486134.1':0.00905682,('PDT000457389.1':0.00740555,(('PDT000142857.2':0.00632379,('PDT000143050.2':0.0038279,'PDT000432451.1':0.00374635)'':0.00172234)'':0.000121907,((('PDT000226917.2':0.00280807,'PDT000242630.2':0.00287828)'':0.00228628,('PDT000702891.1':0.00498416,('PDT000558441.1':0.00588038,(('PDT000465634.1':0.00340495,'PDT000602989.1':0.00268227)'':0.000579913,(('PDT000465636.1':0.00231191,'PDT000665195.1':0.0035769)'':0.00110021,('PDT000590299.1':0.00261982,('PDT000465936.1':0.00182471,'PDT000628382.1':0.00227038)'':0.000572232)'':0.000437215)'':0.000277627)'':0.00074122)'':0.000603803)'':-0.000133038)'':0.000221201,((('PDT000548718.1':0.00324164,'PDT000611852.1':0.00404743)'':6.87356e-05,('PDT000581289.1':0.00184631,'PDT000687623.1':0.00264471)'':0.0016241)'':0.000321707,('PDT000553354.1':0.0034551,(('PDT000260791.2':0.00259403,'PDT000337034.1':0.00210174)'':0.000453078,(('PDT000142855.2':0.00213904,'PDT000647203.1':0.00185577)'':0.000396769,('PDT000553356.1':0.00230526,'PDT000653426.1':0.00248707)'':0.000100481)'':0.000466296)'':0.00132737)'':-0.000525655)'':0.000593491)'':0.000961396)'':0.00060566)'':0.00210424)'':0.000208073)'':0.000826904)'':-0.00290264)'':0.00401425)'':0.00105773)'':0.00325314)'':-0.00167617)'':-0.00244177)'':0.00287734)'':-0.000964864,((('PDT000455635.1':0.0155771,('PDT000455632.1':0.00943537,'PDT000369018.1':0.0104893)'':0.00717558)'':0.000701564,(('PDT000455636.1':0.0154182,'PDT000369020.1':0.0166388)'':0.000792591,('PDT000634813.1':0.0120355,('PDT000190439.2':0.0111906,('PDT000259489.1':0.00782966,'PDT000455638.1':0.00613544)'':0.00496332)'':0.00128652)'':0.00412923)'':0.00373937)'':0.00326349,((('PDT000230874.1':0.0201324,('PDT000230875.1':0.0248886,(('PDT000644130.1':0.00185862,'PDT000644138.1':0.00203437)'':0.00472555,('PDT000644067.1':0.00538598,('PDT000644082.1':0.00343178,'PDT000644104.1':0.00364346)'':0.00150422)'':0.00205021)'':0.00912089)'':0.00329889)'':-0.00388609,(('PDT000022691.1':0.00585605,'PDT000022715.1':0.00380819)'':0.00502599,('PDT000022705.1':0.0100437,('PDT000022692.1':0.00908122,('PDT000022700.1':0.00748566,('PDT000022682.1':0.0036195,'PDT000022693.1':0.00356326)'':0.00147221)'':0.000992236)'':0.000754421)'':0.00214185)'':0.00544202)'':0.00113372,(('PDT000627786.1':0.0145777,('PDT000377086.1':0.0144872,(('PDT000386138.1':0.00696456,'PDT000481300.1':0.00900764)'':0.00222826,(('PDT000332189.1':0.00480329,('PDT000157263.2':0.00469722,('PDT000317250.1':0.00434941,'PDT000155670.2':0.00491723)'':0.000970849)'':0.000587875)'':0.00072205,('PDT000510625.1':0.00644857,('PDT000351939.1':0.00621418,('PDT000306818.2':0.00589902,('PDT000421995.1':0.00422533,'PDT000627812.1':0.00404572)'':0.000864204)'':0.00100906)'':0.000614339)'':0.000209388)'':0.00236471)'':0.00387156)'':0.000562595)'':-0.00164798,(('PDT000604979.1':0.036485,('PDT000455693.1':0.0122408,('PDT000644113.1':0.0137217,('PDT000644084.1':0.00964093,'PDT000230872.1':0.0105992)'':0.00326513)'':0.000741744)'':0.0103347)'':-0.00173351,(('PDT000132500.1':0.0260091,'PDT000132506.1':0.0209042)'':0.003513,('PDT000111926.1':0.0453074,(('PDT000292862.1':0.0201548,'PDT000292864.1':0.0260097)'':0.00380562,(('PDT000271212.2':0.00594561,(('PDT000173615.2':0.00495203,'PDT000190464.2':0.00441239)'':0.00156231,('PDT000231276.2':0.00455636,('PDT000466072.1':0.0029018,'PDT000643888.1':0.00308682)'':0.000818799)'':0.00207285)'':0.000373477)'':0.00215726,('PDT000132489.1':0.0248469,('PDT000132493.1':0.0219843,(('PDT000046988.1':0.00784973,'PDT000046993.1':0.00824567)'':0.00208504,('PDT000455654.1':0.0198748,(('PDT000132486.1':0.0106092,('PDT000132501.1':0.0054763,('PDT000132490.1':0.00400322,'PDT000132508.1':0.00327495)'':0.00174561)'':0.00268635)'':0.00020739,((('PDT000644079.1':0.00928481,('PDT000644081.1':0.00951552,'PDT000455682.1':0.00896438)'':0.00017459)'':0.00110519,(('PDT000644076.1':0.00827467,('PDT000644073.1':0.00819515,('PDT000644110.1':0.00717297,(('PDT000644100.1':0.00395176,('PDT000644071.1':0.00159408,'PDT000644117.1':0.00130273)'':0.00123355)'':0.000866668,('PDT000644094.1':0.00279997,(('PDT000644065.1':0.00266774,'PDT000644077.1':0.00361969)'':0.00017233,('PDT000644072.1':0.00294633,('PDT000644101.1':0.00317199,('PDT000644098.1':0.0025743,('PDT000644097.1':0.00191825,'PDT000644118.1':0.00197513)'':0.000820738)'':0.000177368)'':0.000184559)'':0.000157187)'':0.000402777)'':-9.52851e-05)'':0.00154068)'':0.00103261)'':0.000543913)'':-0.000906481,('PDT000259490.1':0.0151571,('PDT000455676.1':0.00992127,('PDT000406589.1':0.0112991,((('PDT000231021.1':0.00292863,'PDT000231029.1':0.0022584)'':0.000753075,('PDT000455640.1':0.00444359,('PDT000346465.1':0.00140213,('PDT000455680.1':0.00131984,('PDT000455700.1':0.00129782,'PDT000368951.1':0.00139867)'':7.83421e-05)'':0.000270863)'':0.00125175)'':-0.000269791)'':0.000453156,('PDT000292860.1':0.0122633,('PDT000259488.1':0.00408806,('PDT000231027.1':0.00307692,(('PDT000455706.1':0.00245293,('PDT000455665.1':0.00177801,('PDT000455688.1':0.000708376,('PDT000455697.1':0.000746327,('PDT000455663.1':0.000365753,'PDT000455671.1':0.000434007)'':0.000352818)'':0.000106617)'':0.000374441)'':0.00043156)'':-2.27812e-05,('PDT000346262.1':0.00145197,('PDT000368959.1':0.00173997,('PDT000346460.1':0.00241658,('PDT000346269.1':0.00193588,('PDT000346459.1':0.000667764,('PDT000346275.1':0.000804894,'PDT000346279.1':0.000893746)'':3.18312e-05)'':0.000449612)'':0.000151002)'':0.000127987)'':4.07624e-05)'':-5.92276e-05)'':0.00155481)'':0.000806645)'':0.00295343)'':-0.00354196)'':0.00491366)'':0.000310278)'':0.000491182)'':-0.0035516)'':0.00240669)'':-0.00230107,(('PDT000455678.1':0.00790176,(('PDT000610832.1':0.00468917,'PDT000610587.1':0.00536663)'':0.000499182,('PDT000610566.1':0.00690459,((('PDT000455630.1':0.00277489,'PDT000455740.1':0.00232002)'':0.000822379,('PDT000455662.1':0.00222886,'PDT000368963.1':0.00176395)'':0.00143971)'':0.000458448,('PDT000368966.1':0.00406667,(('PDT000346271.1':0.00245142,'PDT000346273.1':0.00224107)'':0.000515647,(('PDT000455661.1':0.00150274,'PDT000455669.1':0.00259112)'':0.000910095,('PDT000455699.1':0.00243583,'PDT000346471.1':0.0019567)'':0.000435795)'':0.000303489)'':0.00138615)'':-0.000516084)'':0.00205897)'':-0.000643356)'':0.00215857)'':-0.00193595,(('PDT000644102.1':0.0068237,'PDT000346258.1':0.0079188)'':0.00223652,(('PDT000346464.1':0.00504694,'PDT000368947.1':0.00813066)'':0.00391436,('PDT000455739.1':0.00906831,(('PDT000610740.1':0.00649263,('PDT000644131.1':0.00290888,('PDT000644116.1':0.00170693,'PDT000644126.1':0.00178783)'':0.00138057)'':0.00210954)'':0.000322774,(('PDT000231019.1':0.00712686,(('PDT000455672.1':0.00298461,'PDT000346458.1':0.00280208)'':0.000822658,(('PDT000644129.1':0.00226109,'PDT000644132.1':0.00252933)'':0.000802907,(('PDT000610833.1':0.00190708,'PDT000455702.1':0.00198786)'':0.000936743,('PDT000610791.1':0.00277442,('PDT000455675.1':0.002567,('PDT000610567.1':0.00256659,'PDT000346285.1':0.00192891)'':0.000424651)'':0.000341159)'':0.0002725)'':0.000204716)'':0.000508739)'':0.00382398)'':-0.00277749,(('PDT000455673.1':0.00420679,'PDT000368972.1':0.00326556)'':0.00076886,('PDT000369023.1':0.00582561,('PDT000455656.1':0.00517501,(('PDT000346462.1':0.00309028,('PDT000346277.1':0.00174224,'PDT000455736.1':0.00235244)'':0.000500723)'':0.000285986,(('PDT000455657.1':0.00257175,'PDT000346267.1':0.0016228)'':0.000585872,(('PDT000455641.1':0.00162364,'PDT000346469.1':0.00147295)'':0.00105517,('PDT000455681.1':0.00251345,(('PDT000368960.1':0.00176687,('PDT000455692.1':0.00183284,'PDT000455735.1':0.00146222)'':3.12747e-05)'':0.000107205,('PDT000346280.1':0.00122619,('PDT000346257.1':0.000944286,('PDT000346274.1':0.000729222,'PDT000368949.1':0.000770028)'':0.000254469)'':0.000353338)'':0.000195282)'':0.000695714)'':0.000142751)'':0.000134811)'':0.000237709)'':0.00142929)'':0.00224513)'':-0.00118519)'':-0.000699266)'':0.000525691)'':0.00455343)'':-0.00222893)'':0.000666539)'':-0.00217063)'':0.000215725)'':0.00137031)'':0.00676981)'':-0.00368697)'':0.00630004)'':0.00426336)'':-0.0116443)'':0.00805694)'':0.0107203)'':-0.00689377)'':-0.00432137)'':-0.00454234)'':0.00316284)'':0.00306635)'':-0.00573864)'':0.00354299)'':0.00385051)'':0.0182537)'':-0.0244115)'':0.00282211)'':0.00233908)'':0.0271075)'':-0.0227329)'':-0.000626283)'':-0.0015748)'':0.00285569)'':-0.00104567)'':0.00617555)'':0.00800495)'':0.000564598)'':0.0166622)'':0.026455)'':0.00926668)'':0.0220815)'':0.138867)'':-0.0131665)'':0.013153)'':0.0195995,(((((('PDT000129874.2':0.0808166,('PDT000178601.2':0.0774236,('PDT000468326.1':0.0463793,('PDT000496977.1':0.0430429,('PDT000290523.2':0.0419007,'PDT000479627.1':0.0355243)'':0.00702381)'':0.00100936)'':0.0375191)'':0.00238578)'':0.0645792,('PDT000249927.1':0.181794,(('PDT000394812.1':0.0513704,('PDT000495020.1':0.020343,'PDT000640126.1':0.0200373)'':0.0196758)'':0.0726155,(('PDT000556967.1':0.0614716,'PDT000468906.1':0.0775934)'':0.0350813,('PDT000357625.1':0.0959156,('PDT000522715.1':0.0469354,('PDT000645505.2':0.0320189,'PDT000242994.1':0.0364207)'':0.0205731)'':0.030316)'':0.00733222)'':0.0142269)'':0.0390886)'':-0.00100563)'':0.0120358,('PDT000357724.1':0.139151,('PDT000252549.1':0.0941468,('PDT000252531.1':0.137316,(('PDT000713215.1':0.0321236,(('PDT000649263.1':0.0180333,('PDT000428851.1':0.0143872,'PDT000429673.1':0.00905891)'':0.00567085)'':0.00453806,('PDT000428893.1':0.0165169,('PDT000428863.1':0.00861265,('PDT000428853.1':0.00784393,'PDT000428862.1':0.00695837)'':0.00542685)'':0.0081588)'':0.0116456)'':0.0255064)'':-0.00520457,(((('PDT000221021.1':0.0228066,('PDT000151014.2':0.00846394,'PDT000629342.1':0.0134659)'':0.012685)'':0.0104049,('PDT000250137.1':0.0362431,(('PDT000251922.1':0.0178715,('PDT000428897.1':0.0237214,('PDT000010919.2':0.0167331,'PDT000429763.1':0.0198739)'':0.00519888)'':0.00315312)'':0.00129608,('PDT000221025.1':0.0180695,('PDT000221019.1':0.00914608,('PDT000221020.1':0.000916048,'PDT000221024.1':0.00108195)'':0.0107453)'':0.0108913)'':0.00708476)'':0.00673114)'':0.00292924)'':-0.00212485,(('PDT000429707.1':0.0139429,('PDT000429679.1':0.00478564,'PDT000429690.1':0.00448285)'':0.00585795)'':0.00935597,('PDT000429755.1':0.013324,('PDT000429793.1':0.0124004,('PDT000429752.1':0.00591361,('PDT000429748.1':0.00319438,('PDT000429776.1':0.00277719,(('PDT000429780.1':0.00192735,'PDT000429781.1':0.00226762)'':0.00018434,('PDT000429744.1':0.00218611,'PDT000429784.1':0.0018063)'':0.00061209)'':0.000266179)'':0.000429337)'':0.00394704)'':0.00183019)'':0.00251354)'':0.0135706)'':0.0075487)'':0.00454517,(('PDT000015715.3':0.0325254,('PDT000015701.2':0.0244073,(('PDT000015794.3':0.0159228,'PDT000015828.3':0.015358)'':0.000466394,('PDT000015820.3':0.0132341,('PDT000015824.3':0.0118666,(('PDT000015799.3':0.00886122,'PDT000015810.3':0.00868788)'':0.000933986,('PDT000014139.3':0.00639868,'PDT000015823.3':0.0102033)'':0.00255424)'':0.0010756)'':0.00119498)'':0.000787694)'':0.00999225)'':0.0108186)'':0.00861771,(('PDT000428855.1':0.0233537,('PDT000377026.1':0.0184497,(('PDT000429702.1':0.0115821,('PDT000014153.3':0.0082507,'PDT000429724.1':0.0062261)'':0.0032929)'':0.0027784,('PDT000428885.1':0.0135014,('PDT000428870.1':0.00473754,('PDT000428871.1':0.00378866,('PDT000428865.1':0.00343908,'PDT000428907.1':0.00353859)'':0.000700597)'':0.000370336)'':0.00950625)'':0.00259365)'':0.00507568)'':0.00144773)'':0.0038838,('PDT000428894.1':0.0154317,((('PDT000034369.1':0.0106174,('PDT000034389.1':0.00720575,'PDT000034392.1':0.010265)'':0.00541331)'':0.00482416,('PDT000429756.1':0.0195862,('PDT000340392.1':0.0076042,'PDT000429674.1':0.0044922)'':0.00582361)'':0.0024048)'':0.000583333,(('PDT000303749.2':0.0190069,('PDT000316562.1':0.0201927,('PDT000132150.2':0.00495272,'PDT000429669.1':0.00682848)'':0.00685855)'':0.00746515)'':-0.00490637,(('PDT000428898.1':0.0212348,('PDT000429723.1':0.00680906,('PDT000428914.1':0.00794961,(('PDT000429729.1':0.00368366,'PDT000429788.1':0.00339369)'':0.000235056,('PDT000429741.1':0.00222058,'PDT000429783.1':0.00226954)'':0.00130529)'':0.00253672)'':0.00323895)'':0.00903974)'':-0.00187679,(('PDT000327297.1':0.0111781,('PDT000429772.1':0.00892958,('PDT000429699.1':0.010553,('PDT000428908.1':0.00398075,'PDT000429764.1':0.00488732)'':0.00259602)'':0.00201656)'':0.00284388)'':9.64861e-06,(('PDT000392121.1':0.00472331,'PDT000422545.1':0.00434857)'':0.00225761,('PDT000496779.1':0.00749566,('PDT000662571.1':0.0107945,('PDT000682959.1':0.00473374,('PDT000429798.1':0.00283589,('PDT000429746.1':0.00238436,'PDT000429751.1':0.0023072)'':0.000354444)'':0.00162256)'':0.00268534)'':0.00154286)'':0.000226961)'':0.00216555)'':0.00243489)'':0.00275381)'':0.00164224)'':0.00314236)'':0.00174104)'':0.0121181)'':0.00323214)'':0.00629259)'':0.068806)'':0.00741514)'':0.0108504)'':0.0425664)'':0.0196512,(((('PDT000417980.1':0.134497,(('PDT000274102.2':0.0506612,'PDT000411305.1':0.0534218)'':0.069151,('PDT000235762.3':0.104223,'PDT000668927.1':0.115104)'':0.00457202)'':0.00607429)'':0.0139867,(('PDT000549604.1':0.0968471,(('PDT000399688.1':0.0597795,'PDT000639507.1':0.0652445)'':0.00126793,('PDT000137876.2':0.0446519,'PDT000714127.1':0.0493136)'':0.0139806)'':0.0134028)'':0.00127563,(('PDT000060395.2':0.053845,('PDT000092537.1':0.0279851,'PDT000395488.1':0.0308557)'':0.0147791)'':0.0262313,('PDT000399693.1':0.0606234,('PDT000026908.2':0.0598611,('PDT000394808.1':0.0479051,'PDT000645110.1':0.048746)'':0.00462489)'':0.0212716)'':0.0127145)'':0.00712599)'':0.0696949)'':0.0554182,(('PDT000359872.1':0.177053,(((('PDT000175868.2':0.0421878,('PDT000372696.1':0.0208431,'PDT000713767.1':0.0245575)'':0.0200644)'':0.0486097,(('PDT000426036.1':0.0662796,'PDT000444946.1':0.0553404)'':0.0165972,('PDT000716245.1':0.0498316,('PDT000501972.1':0.0496604,('PDT000399995.1':0.0297386,'PDT000483545.1':0.0271926)'':0.00358166)'':0.00451532)'':0.0161584)'':0.0264002)'':0.00219421,(('PDT000716304.1':0.0929467,'PDT000652395.1':0.102679)'':0.00681094,('PDT000652392.1':0.101244,('PDT000652399.1':0.0813996,(('PDT000426050.1':0.0351273,'PDT000426391.1':0.0305165)'':0.0310904,('PDT000526187.1':0.0543988,('PDT000652332.1':0.0572968,'PDT000652393.1':0.0642982)'':0.00129871)'':0.00821061)'':0.0195891)'':0.00506158)'':0.00665699)'':0.0111746)'':0.00109008,(('PDT000006417.2':0.0538512,'PDT000448203.1':0.0680278)'':0.046957,((('PDT000497357.1':0.0917742,('PDT000716151.1':0.0661725,('PDT000161628.2':0.0737963,('PDT000399994.1':0.0620219,'PDT000011746.2':0.0485941)'':0.0154042)'':0.00787664)'':0.00197152)'':0.0115284,('PDT000716077.1':0.0677941,('PDT000717587.1':0.0865974,('PDT000716018.1':0.0751038,('PDT000627599.1':0.0616113,(('PDT000394905.1':0.0613673,'PDT000627598.1':0.0596467)'':0.00365123,('PDT000399678.1':0.0429296,('PDT000060649.2':0.0589936,'PDT000622373.1':0.0559984)'':0.00415988)'':0.00918898)'':0.00420015)'':0.00580274)'':0.00715572)'':0.00237523)'':0.00383799)'':0.00975012,('PDT000715901.1':0.089128,(('PDT000716237.1':0.0503679,'PDT000222309.1':0.0548851)'':0.0307065,(('PDT000347045.1':0.0633457,('PDT000426018.1':0.0456995,'PDT000652401.1':0.0496072)'':0.0156779)'':0.00881851,('PDT000649092.1':0.0849889,('PDT000328144.1':0.0529258,('PDT000682736.1':0.0397825,('PDT000387345.1':0.0394935,('PDT000387340.1':0.0313195,('PDT000387347.1':0.0331449,(('PDT000387341.1':0.0246075,'PDT000387343.1':0.0199797)'':0.00524398,('PDT000407967.1':0.0155532,('PDT000387344.1':0.0210406,('PDT000273989.1':0.0170134,('PDT000387337.1':0.0100711,'PDT000387338.1':0.0102219)'':0.00489626)'':0.00155523)'':0.00275729)'':0.00193634)'':0.00981752)'':0.00111985)'':0.00833865)'':0.0021025)'':0.015238)'':0.0199279)'':-0.00682526)'':0.00987489)'':0.0121654)'':0.0101451)'':0.0122933)'':-0.0114786)'':0.0912817)'':-0.0556185,(('PDT000161981.2':0.154572,(('PDT000129868.2':0.123843,('PDT000661762.1':0.0811386,('PDT000571156.1':0.0496974,'PDT000571158.1':0.0478264)'':0.0328045)'':0.0288373)'':0.0207086,(('PDT000644883.1':0.124003,(('PDT000309964.3':0.0865082,'PDT000644989.1':0.0663818)'':0.0154002,('PDT000397817.1':0.0604572,('PDT000639537.1':0.0652996,'PDT000644873.1':0.0437874)'':0.0155948)'':0.0159117)'':0.0524138)'':-0.00557098,((('PDT000622369.1':0.0659723,'PDT000710820.1':0.0795597)'':0.0205358,('PDT000399624.1':0.0591573,(('PDT000034475.2':0.0544541,'PDT000395480.1':0.0566569)'':0.00400058,('PDT000712477.1':0.0513449,('PDT000395259.1':0.0461251,('PDT000034510.2':0.0151763,'PDT000600660.1':0.0186902)'':0.03061)'':0.00653892)'':0.000799135)'':0.0154722)'':0.00925668)'':0.0412959,(('PDT000104793.2':0.114669,'PDT000086968.1':0.112522)'':0.00213233,(('PDT000397719.1':0.0993582,('PDT000566062.1':0.0693551,'PDT000395119.1':0.0693499)'':0.0248168)'':0.00949789,(('PDT000713611.1':0.104523,('PDT000250054.1':0.0744253,('PDT000385030.1':0.0517575,'PDT000319002.1':0.0460279)'':0.0177005)'':0.0280506)'':0.00654346,(('PDT000652383.1':0.0785358,(('PDT000357583.1':0.0449721,'PDT000395264.1':0.0523935)'':0.0122181,('PDT000566069.1':0.0295489,('PDT000687814.1':0.0333321,'PDT000395498.1':0.0265476)'':0.00576599)'':0.0284344)'':0.0238451)'':0.0137593,(('PDT000716234.1':0.107801,('PDT000712755.1':0.102262,('PDT000645084.1':0.0753621,(('PDT000107236.2':0.101371,('PDT000399666.1':0.0242653,'PDT000250047.1':0.0254417)'':0.0400461)'':0.00296343,('PDT000312632.2':0.072907,('PDT000399715.1':0.0370335,('PDT000051512.2':0.045065,('PDT000397659.1':0.0431136,'PDT000604853.1':0.0471156)'':0.00789813)'':0.00516886)'':0.0238799)'':-0.000173251)'':0.0109217)'':0.00589214)'':0.00375282)'':0.00379564,((('PDT000062930.2':0.050465,'PDT000357712.1':0.058313)'':0.0284956,('PDT000566096.1':0.0709489,('PDT000034460.2':0.0668606,'PDT000258340.2':0.0748424)'':0.000994616)'':0.0179891)'':0.00896877,(('PDT000565945.1':0.0748711,'PDT000620193.1':0.0743229)'':0.0115293,('PDT000394982.1':0.0751729,('PDT000669189.1':0.0397153,('PDT000566132.1':0.0472314,'PDT000669500.1':0.0480567)'':0.00292382)'':0.0392527)'':0.00979228)'':0.000606227)'':0.00421353)'':0.00128493)'':0.00829074)'':0.00261193)'':0.0122964)'':0.000781283)'':0.00809413)'':0.0147332)'':0.0250388)'':-0.0468996,((('PDT000395235.1':0.0601899,(('PDT000497347.1':0.0547639,'PDT000571136.1':0.042465)'':0.0029083,(('PDT000316872.1':0.0330611,'PDT000397770.1':0.035191)'':0.00620496,('PDT000332178.1':0.0316058,'PDT000564734.1':0.0217697)'':0.0108087)'':0.00857743)'':0.00336567)'':0.0451229,(('PDT000324075.1':0.0957255,('PDT000252594.1':0.0474055,('PDT000253202.2':0.0394156,'PDT000497388.1':0.0389856)'':0.00932687)'':0.0446507)'':-0.00215942,('PDT000132020.2':0.0855816,('PDT000592077.1':0.0416084,('PDT000395130.1':0.0531389,('PDT000702845.1':0.039952,('PDT000389631.1':0.0371083,'PDT000571183.1':0.0256879)'':0.00970201)'':0.00597107)'':0.00228552)'':0.0347331)'':-0.00811894)'':0.0328381)'':0.017471,((('PDT000291135.2':0.134777,'PDT000565989.1':0.130027)'':0.000531628,('PDT000283182.1':0.1379,('PDT000280684.2':0.089098,(('PDT000550366.1':0.073578,('PDT000120813.2':0.0524829,'PDT000387346.1':0.0569621)'':0.00863401)'':0.0136033,(('PDT000044715.2':0.035787,'PDT000425832.1':0.0309119)'':0.0286562,('PDT000111117.1':0.0469795,'PDT000015320.2':0.050159)'':0.00630987)'':0.0157831)'':0.0147374)'':0.00810793)'':0.0217269)'':0.0010363,(('PDT000631532.1':0.123955,('PDT000318973.1':0.115942,(('PDT000318962.1':0.0935518,'PDT000107242.2':0.0838412)'':0.00520837,('PDT000343867.1':0.101236,'PDT000395228.1':0.0913283)'':0.00849388)'':0.0248001)'':0.00592356)'':-0.00196561,((('PDT000129908.2':0.083522,'PDT000557104.1':0.087549)'':0.0299191,(('PDT000625080.1':0.100183,(('PDT000015330.2':0.0759103,('PDT000535652.1':0.058282,('PDT000066401.2':0.0495432,('PDT000501975.1':0.0348454,'PDT000572854.1':0.0434247)'':0.0151959)'':0.0084093)'':0.018297)'':-0.00994577,('PDT000106252.2':0.0781229,('PDT000107239.2':0.0486536,('PDT000644991.1':0.0259994,('PDT000104652.2':0.01557,'PDT000397912.1':0.0185995)'':0.00766927)'':0.0400319)'':0.0167066)'':0.00209858)'':0.0262943)'':0.00691591,('PDT000395464.1':0.109116,(('PDT000685903.1':0.0956779,('PDT000713966.1':0.0570989,'PDT000319046.1':0.0623031)'':0.0287111)'':0.0104891,((('PDT000319795.1':0.0182172,'PDT000111356.1':0.0233716)'':0.0605523,('PDT000399986.1':0.0727845,'PDT000572848.1':0.0741235)'':0.00743356)'':0.00720084,('PDT000713908.1':0.0459441,('PDT000713874.1':0.0367305,('PDT000069163.2':0.0265584,'PDT000565970.1':0.0256469)'':0.0113108)'':0.00461648)'':0.0419637)'':0.0138973)'':0.0169864)'':-0.0132383)'':0.0187197)'':-0.0134765,(('PDT000714244.1':0.0839202,('PDT000708229.1':0.0400547,('PDT000565903.1':0.00922839,'PDT000627811.1':0.0145075)'':0.0251796)'':0.0135569)'':0.0654901,((('PDT000104641.2':0.0742717,('PDT000407965.1':0.0403542,('PDT000034496.2':0.0124116,'PDT000621298.1':0.0174465)'':0.0181409)'':0.0288923)'':0.0338511,('PDT000288721.1':0.108402,('PDT000627353.1':0.0474357,('PDT000167951.2':0.0365427,'PDT000566084.1':0.0410864)'':0.0187228)'':0.0350832)'':0.000868662)'':0.00899791,('PDT000426636.1':0.112236,('PDT000713872.1':0.0757482,(('PDT000399626.1':0.0610522,('PDT000066398.2':0.0671671,('PDT000395254.1':0.0750864,('PDT000497322.1':0.0454105,'PDT000566066.1':0.0430851)'':0.0145268)'':0.0128467)'':0.00139505)'':0.00673954,(('PDT000395431.1':0.0418167,('PDT000395225.1':0.0417831,('PDT000395398.1':0.0238596,'PDT000604840.1':0.0273091)'':0.0130866)'':0.0164861)'':0.0302078,('PDT000034519.2':0.0870428,('PDT000644901.1':0.0804971,('PDT000662179.1':0.0461879,('PDT000395043.1':0.024691,'PDT000557060.1':0.0297154)'':0.0355774)'':0.0149149)'':0.00423371)'':0.00425877)'':0.00488348)'':0.00797943)'':0.00957096)'':0.0028799)'':0.0176929)'':-0.00503391)'':0.00664629)'':0.0047131)'':0.00370249)'':-0.00281351)'':0.0119607)'':0.0717719)'':-0.00617486,((('PDT000047728.3':0.0104653,'PDT000357697.1':0.0173125)'':0.0487797,('PDT000346723.1':0.0535573,('PDT000323515.1':0.0539472,'PDT000384285.1':0.0594248)'':0.00361271)'':0.0110989)'':0.148328,(((('PDT000645715.2':0.0994069,'PDT000252643.1':0.101847)'':0.0352049,('PDT000398003.1':0.110742,(('PDT000520954.1':0.101551,('PDT000358754.1':0.0940091,('PDT000566036.1':0.0552218,('PDT000397831.1':0.0276187,'PDT000566196.1':0.0263879)'':0.0345194)'':0.0241855)'':0.00306572)'':0.00187415,(('PDT000566221.1':0.0766852,'PDT000644893.1':0.0925858)'':0.0180896,('PDT000425649.1':0.0968809,(('PDT000645495.2':0.06254,'PDT000011745.2':0.053254)'':0.0261996,('PDT000550346.1':0.035439,('PDT000394855.1':0.0224309,'PDT000565955.1':0.0230861)'':0.011156)'':0.0560324)'':0.00647685)'':0.00484789)'':0.00144638)'':0.0200573)'':0.0130722)'':0.0523131,(('PDT000652910.1':0.0198178,'PDT000652971.1':0.0204256)'':0.172731,(((('PDT000228134.1':0.0616595,'PDT000228184.1':0.0582845)'':0.0119686,('PDT000468905.1':0.0832133,('PDT000645506.2':0.0631503,(('PDT000652845.1':0.0360388,'PDT000346054.1':0.0352657)'':0.0186184,('PDT000652928.1':0.0209199,'PDT000652952.1':0.0163404)'':0.0380573)'':0.00491385)'':0.0209899)'':-0.00537832)'':0.0741821,(('PDT000357656.1':0.122066,('PDT000228088.1':0.0320958,'PDT000228125.1':0.0325623)'':0.077617)'':0.0115537,(('PDT000060714.2':0.02419,'PDT000420671.1':0.0328966)'':0.0419945,('PDT000592041.1':0.106807,(('PDT000342664.1':0.0569095,'PDT000357572.1':0.0486035)'':0.00969198,('PDT000566330.1':0.0572118,('PDT000197951.2':0.0577142,('PDT000502136.1':0.0406016,'PDT000034383.1':0.0401103)'':0.00978934)'':0.00484758)'':0.00499327)'':0.0149881)'':0.00266966)'':0.0444861)'':0.00993691)'':0.000732908,((('PDT000063327.2':0.0288088,('PDT000578740.1':0.031502,'PDT000010343.3':0.0228884)'':0.0114295)'':-3.66238e-05,('PDT000557197.1':0.0306127,('PDT000500523.1':0.0296276,('PDT000394987.1':0.0246338,('PDT000579964.1':0.0146047,('PDT000368382.1':0.00890485,'PDT000395174.1':0.00953475)'':0.00386373)'':0.00541021)'':0.00956957)'':0.00542723)'':0.000210114)'':0.109335,(('PDT000672706.1':0.0688049,('PDT000264628.3':0.0543115,'PDT000242511.1':0.0651305)'':0.0220011)'':0.0713477,(('PDT000172090.3':0.048889,'PDT000502612.1':0.0435018)'':0.0280214,((('PDT000161627.3':0.0115056,'PDT000129832.3':0.00752775)'':0.0370015,('PDT000343944.2':0.0379179,'PDT000379478.2':0.0341989)'':0.0161802)'':0.00972199,('PDT000140452.3':0.0639768,('PDT000487930.1':0.0503984,('PDT000265149.5':0.0322036,'PDT000110499.3':0.0330875)'':0.00530215)'':0.0103512)'':0.000710659)'':0.0115402)'':0.078156)'':8.7711e-05)'':0.00591638)'':0.0493473)'':-0.027074)'':0.011951,(((('PDT000354940.1':0.0419023,'PDT000252445.1':0.0293923)'':0.0662007,(('PDT000161984.3':0.0563979,('PDT000311077.3':0.0583729,('PDT000501605.2':0.0323385,(('PDT000303480.3':0.0150639,'PDT000354917.2':0.0107726)'':0.00444651,('PDT000273349.3':0.0109841,('PDT000347484.2':0.00313474,('PDT000253036.3':0.00161214,'PDT000253145.3':0.00288157)'':0.000854088)'':0.00428324)'':0.00802455)'':0.0258016)'':0.0132229)'':0.00457291)'':0.0230984,((('PDT000146109.3':0.0279696,'PDT000337140.2':0.0232299)'':0.0413002,('PDT000553425.2':0.0130626,('PDT000252962.3':0.0105859,'PDT000501575.2':0.0113791)'':0.00577602)'':0.0552135)'':0.00127898,(('PDT000240620.3':0.0372483,'PDT000126422.3':0.0396822)'':0.010678,('PDT000145382.3':0.0380266,('PDT000497862.1':0.0439078,('PDT000251886.1':0.028997,'PDT000497749.1':0.0340066)'':0.010105)'':0.00504828)'':0.0117577)'':0.00947037)'':0.0132687)'':0.0430862)'':-0.0123981,((('PDT000710998.1':0.0440832,('PDT000655232.2':0.0202688,'PDT000717616.1':0.0200864)'':0.019544)'':0.0226156,('PDT000303504.3':0.0420234,('PDT000487928.1':0.0506009,('PDT000501577.2':0.0419756,(('PDT000147672.3':0.0165818,'PDT000444920.2':0.0129929)'':0.00907468,(('PDT000592582.1':0.0197768,'PDT000147998.3':0.0207174)'':0.00326787,('PDT000096906.3':0.0122927,'PDT000112501.3':0.012895)'':0.0128352)'':0.0048286)'':0.0120587)'':0.00827708)'':0.00959708)'':0.0189578)'':0.0194224,((('PDT000718951.1':0.0317338,'PDT000045611.1':0.026)'':0.0285797,(('PDT000502373.1':0.0311995,'PDT000502386.1':0.0287772)'':0.0175549,('PDT000502462.1':0.0571902,('PDT000249955.1':0.0253819,(('PDT000344880.2':0.0123474,'PDT000410733.2':0.0159376)'':0.0110579,('PDT000502298.1':0.0319626,('PDT000498162.1':0.00322479,'PDT000498167.1':0.00365389)'':0.0216298)'':0.00151564)'':0.00927188)'':0.0151375)'':-0.00509368)'':0.0128651)'':0.0231606,((('PDT000557026.2':0.0172042,'PDT000497750.1':0.0227879)'':0.0246123,(('PDT000045587.1':0.0232337,'PDT000077507.1':0.0221051)'':0.017705,(('PDT000149879.3':0.010982,'PDT000400848.2':0.0121102)'':0.00445058,('PDT000500763.2':0.00985326,'PDT000136920.1':0.0115944)'':0.00757092)'':0.0130242)'':0.0102593)'':0.012106,(('PDT000213245.3':0.026678,'PDT000271438.3':0.0279881)'':0.0269557,((('PDT000672392.1':0.021618,'PDT000048145.1':0.0225184)'':0.0161328,('PDT000657100.2':0.0183247,('PDT000163002.4':0.00566568,'PDT000472759.2':0.00892852)'':0.00884826)'':0.0156042)'':0.0125868,(('PDT000334506.2':0.0322519,('PDT000351811.2':0.0287416,'PDT000502347.1':0.0372011)'':0.00392961)'':0.0155319,('PDT000152687.3':0.0313543,('PDT000627235.2':0.0214569,('PDT000084088.3':0.0161259,'PDT000502360.1':0.0186224)'':0.00103109)'':0.00964817)'':0.0113159)'':0.00716127)'':0.00663641)'':0.00126289)'':0.0270334)'':0.00386133)'':0.0163607)'':0.0650392,((('PDT000426115.1':0.158291,(('PDT000566090.1':0.094957,'PDT000329819.1':0.067045)'':0.00854183,('PDT000409453.1':0.0576089,('PDT000249886.1':0.0472304,('PDT000712928.1':0.0405252,'PDT000042666.1':0.0369172)'':0.0114926)'':0.0108568)'':0.0117224)'':0.0594604)'':-0.00867555,(('PDT000249965.1':0.111883,('PDT000652911.1':0.0581331,('PDT000346007.1':0.0573667,'PDT000346055.1':0.0545723)'':0.00813535)'':0.0440335)'':0.0133996,('PDT000274854.2':0.0961588,(('PDT000619309.1':0.0504279,'PDT000714573.1':0.0509911)'':0.0128744,('PDT000522471.1':0.0849269,('PDT000644854.1':0.0661441,('PDT000538607.1':0.0647772,'PDT000651047.1':0.0604948)'':0.00590235)'':0.00210845)'':0.000556739)'':0.0212275)'':0.0114158)'':0.0386914)'':0.0175905,((('PDT000238205.2':0.100145,(('PDT000501976.1':0.0825851,'PDT000041343.1':0.0709529)'':0.00979275,(('PDT000502049.1':0.075363,'PDT000502068.1':0.084905)'':0.00388659,('PDT000716058.1':0.026847,('PDT000715917.1':0.029351,('PDT000715958.1':0.00659394,'PDT000716067.1':0.00831656)'':0.0198095)'':0.00681143)'':0.0596415)'':0.0117142)'':0.0115828)'':0.0136945,(('PDT000325798.1':0.0580567,('PDT000132104.2':0.061047,('PDT000238188.2':0.0371098,'PDT000015334.2':0.0379844)'':0.0197189)'':0.00682927)'':-0.00220295,('PDT000494551.1':0.0394432,('PDT000502225.1':0.036918,('PDT000644544.1':0.036591,('PDT000145340.2':0.0349641,('PDT000565897.1':0.0256474,'PDT000501985.1':0.0243477)'':0.00787586)'':0.000461385)'':0.00452646)'':0.0125576)'':0.0121555)'':0.0650643)'':0.0444653,(('PDT000129884.2':0.0903161,'PDT000569428.1':0.0798179)'':0.0855535,((('PDT000716007.1':0.147429,('PDT000543337.1':0.116163,('PDT000672853.1':0.0736196,('PDT000426165.1':0.0809879,'PDT000713552.1':0.0861101)'':0.00391844)'':0.00728314)'':0.0377488)'':0.0089156,(('PDT000015702.3':0.0179594,'PDT000014155.2':0.0180649)'':0.010235,('PDT000015706.2':0.0150038,('PDT000014128.3':0.0212423,('PDT000651383.1':0.00836718,('PDT000015805.3':0.011639,('PDT000015789.3':0.0101465,('PDT000015813.3':0.0114348,('PDT000015790.3':0.00927166,('PDT000015802.3':0.00912328,('PDT000015779.3':0.00797031,('PDT000015795.3':0.00828094,('PDT000015791.3':0.00812258,'PDT000014145.3':0.00410402)'':0.000446755)'':0.0003161)'':0.000419655)'':0.00030911)'':0.000447045)'':0.000233982)'':0.000281221)'':0.00324787)'':0.00310244)'':0.000833303)'':0.0100601)'':0.0957031)'':0.0280895,('PDT000485907.1':0.16501,(((('PDT000385038.1':0.100915,'PDT000017830.2':0.0954632)'':0.0133263,('PDT000205416.1':0.106926,('PDT000661087.1':0.0776615,('PDT000231031.1':0.0628949,'PDT000435626.1':0.0604101)'':0.00658996)'':0.0258055)'':0.0176012)'':0.0137855,(('PDT000107235.2':0.0654161,'PDT000251704.1':0.0511629)'':0.0734584,(('PDT000318993.1':0.110706,('PDT000565895.1':0.0974099,'PDT000652394.1':0.120529)'':0.0108295)'':0.000548336,(('PDT000081760.1':0.0558826,'PDT000212818.1':0.0767414)'':0.020386,('PDT000107789.2':0.0877701,('PDT000101298.1':0.0539524,'PDT000352473.1':0.0619766)'':0.00875341)'':0.0111745)'':0.0148327)'':0.017323)'':0.000936811)'':0.0152319,((('PDT000253450.2':0.0609161,'PDT000713795.1':0.0539219)'':0.0217673,('PDT000104940.2':0.0960479,('PDT000719101.1':0.125844,('PDT000652398.1':0.124454,(('PDT000135940.2':0.0647181,('PDT000106249.2':0.0586288,'PDT000652390.1':0.0600052)'':0.0184874)'':0.01587,('PDT000101303.1':0.0852655,('PDT000395075.1':0.065337,('PDT000691291.1':0.0669293,('PDT000566194.1':0.0519003,'PDT000719130.1':0.0586307)'':0.00956621)'':0.00352876)'':0.00800472)'':0.00191518)'':0.00589115)'':0.00540314)'':0.000863781)'':0.00186837)'':0.00176502,((('PDT000536584.1':0.0687588,('PDT000600744.1':0.0611726,(('PDT000566184.1':0.0213287,'PDT000395355.1':0.0256492)'':0.00522807,('PDT000644970.1':0.0280275,('PDT000630061.1':0.0246308,('PDT000566027.1':0.0254936,('PDT000343899.1':0.00777187,'PDT000343803.1':0.00588853)'':0.0121521)'':0.00469391)'':0.0023957)'':0.000251321)'':0.0153701)'':0.0219465)'':0.00186759,(('PDT000652391.1':0.0734747,('PDT000425766.1':0.0542078,('PDT000372700.1':0.0380302,'PDT000675087.1':0.0503827)'':0.0171347)'':0.00391327)'':0.014246,(('PDT000502959.1':0.0228169,('PDT000565959.1':0.0108668,'PDT000644872.1':0.00887915)'':0.0081245)'':0.0314482,(('PDT000083148.2':0.0352692,('PDT000385063.1':0.0318564,'PDT000251705.1':0.0268808)'':0.0012038)'':0.0184769,('PDT000650945.1':0.0584149,('PDT000617378.1':0.0481054,('PDT000501998.1':0.0370927,('PDT000400008.1':0.0428683,('PDT000631663.1':0.027221,('PDT000336791.1':0.0308254,('PDT000583873.1':0.0250375,'PDT000448208.1':0.0283089)'':0.0042957)'':0.00148086)'':0.00384615)'':0.00601424)'':0.0138862)'':0.00409237)'':-0.00252484)'':0.00558162)'':0.00340945)'':0.00200837)'':0.00364773,(('PDT000397553.1':0.0688207,'PDT000652397.1':0.0811793)'':0.0305298,(('PDT000394947.1':0.0812279,'PDT000318960.1':0.0840941)'':0.00698433,(('PDT000600639.1':0.0690809,('PDT000397711.1':0.0371207,(('PDT000497403.1':0.0202803,'PDT000565992.1':0.0277095)'':0.000840598,('PDT000566210.1':0.0150531,('PDT000395158.1':0.0133118,'PDT000395178.1':0.00907176)'':0.00418256)'':0.00725063)'':0.00830239)'':0.0146697)'':0.0124643,('PDT000395489.1':0.0709708,(('PDT000645090.1':0.0576843,('PDT000522409.1':0.033661,'PDT000395317.1':0.0448106)'':0.00734648)'':0.0155364,(('PDT000034521.2':0.0377195,('PDT000252640.1':0.0332409,('PDT000238196.2':0.035912,('PDT000252537.1':0.0339087,('PDT000639636.1':0.0231929,'PDT000228336.1':0.033316)'':0.00230111)'':0.00245869)'':0.00136368)'':0.00242452)'':0.0176253,('PDT000502121.1':0.048442,('PDT000441343.1':0.0510795,(('PDT000136166.2':0.0238118,'PDT000600646.1':0.0208749)'':0.0102024,('PDT000587230.1':0.0323401,('PDT000565966.1':0.0230393,('PDT000395099.1':0.0132298,'PDT000399672.1':0.0111001)'':0.0116971)'':0.00119801)'':0.00658054)'':0.00763014)'':0.00234492)'':0.00533872)'':0.0161605)'':0.0132262)'':-0.0139365)'':0.0191332)'':0.0145367)'':-0.0277633)'':0.0232761)'':0.0480666)'':0.0324333)'':-0.02264)'':0.0273443)'':-0.0258842)'':0.00477775)'':0.0114621)'':0.0151272)'':0.031595)'':-0.0270255)'':0.0070008)'':0.0152308,(((('PDT000571831.1':0.0330417,('PDT000501961.1':0.0288129,('PDT000353310.1':0.012029,('PDT000065382.1':0.0089963,'PDT000065383.1':0.0080621)'':0.00601154)'':0.00694823)'':0.0113837)'':0.132616,(('PDT000357585.1':0.035635,('PDT000535180.1':0.0318376,('PDT000443028.1':0.0382992,'PDT000357588.1':0.0290517)'':0.00579805)'':0.00258113)'':0.0601884,('PDT000705005.1':0.0979598,('PDT000354184.1':0.0780505,('PDT000535240.1':0.00951128,('PDT000535356.1':0.00249655,'PDT000535496.1':0.00329418)'':0.0032403)'':0.0693533)'':0.0157437)'':0.00411111)'':0.0667898)'':0.0261384,((('PDT000645697.2':0.0707044,('PDT000540520.1':0.0146502,'PDT000550386.1':0.0159671)'':0.0649919)'':-0.000918511,('PDT000395229.1':0.0701565,('PDT000565853.1':0.0461774,'PDT000251587.1':0.0501656)'':0.0392045)'':0.0161174)'':0.0364087,(((('PDT000399694.1':0.065194,('PDT000397645.1':0.0359585,'PDT000565995.1':0.041606)'':0.0171693)'':0.0287323,('PDT000155981.2':0.0752387,((('PDT000394878.1':0.028466,'PDT000394968.1':0.0302201)'':0.00955592,('PDT000395369.1':0.0420003,'PDT000394943.1':0.0380806)'':0.0203973)'':0.0278356,('PDT000395170.1':0.043259,('PDT000397890.1':0.0263393,('PDT000557176.1':0.0155327,'PDT000252619.1':0.0156187)'':0.0297711)'':0.0156941)'':0.0280485)'':0.00620972)'':0.00371634)'':0.012148,(('PDT000395262.1':0.0589424,('PDT000565974.1':0.0560382,('PDT000395491.1':0.0341133,('PDT000395085.1':0.0283052,'PDT000600974.1':0.0283208)'':0.011503)'':0.0117767)'':0.00939341)'':0.0359111,(('PDT000644880.1':0.0817137,('PDT000645068.1':0.0961752,'PDT000250207.1':0.0786268)'':0.00368584)'':0.00979123,('PDT000637360.1':0.0925586,('PDT000395430.1':0.082171,(('PDT000395100.1':0.0722381,'PDT000639587.1':0.0627029)'':0.00998988,('PDT000395449.1':0.0733257,(('PDT000397713.1':0.0304941,'PDT000662181.1':0.0292262)'':0.00766451,('PDT000565889.1':0.0341737,'PDT000566123.1':0.0400447)'':0.00757971)'':0.0209075)'':-0.00131515)'':0.0153429)'':0.006917)'':0.00308468)'':0.000923782)'':0.00531953)'':0.0132721,((('PDT000252522.1':0.106688,'PDT000357729.1':0.107656)'':0.0164825,(('PDT000129897.2':0.0665385,('PDT000714169.1':0.0558903,'PDT000092535.1':0.0608617)'':0.012562)'':0.000677973,(('PDT000426256.1':0.0653143,'PDT000200051.1':0.0693177)'':0.00896973,('PDT000251946.1':0.0561725,('PDT000589375.1':0.0531455,'PDT000691297.1':0.0494895)'':0.00376805)'':0.0168079)'':0.00251158)'':0.0219731)'':-0.00234288,('PDT000004348.2':0.135232,((('PDT000497386.1':0.0941487,('PDT000550365.1':0.0241313,'PDT000395149.1':0.0332699)'':0.0565192)'':0.000932439,('PDT000397666.1':0.0915224,('PDT000644939.1':0.0759798,('PDT000394931.1':0.0577749,('PDT000397586.1':0.0406832,'PDT000395066.1':0.0396137)'':0.0224981)'':0.00430713)'':0.00222029)'':0.0181468)'':0.00870141,((('PDT000426048.1':0.0733602,'PDT000691294.1':0.0916988)'':0.0170495,('PDT000566198.1':0.0842593,('PDT000205337.1':0.0817868,('PDT000250697.2':0.0914461,('PDT000571168.1':0.0656414,('PDT000661347.1':0.0507865,('PDT000278764.1':0.0551687,('PDT000396819.1':0.0356758,'PDT000520160.1':0.0367066)'':0.010932)'':0.00292374)'':0.0127166)'':0.00446132)'':0.00293048)'':0.0085482)'':-0.00075955)'':0.00200795,(('PDT000589750.1':0.0613357,'PDT000588041.1':0.0645223)'':0.0310228,(('PDT000718949.1':0.0537299,'PDT000358761.1':0.0569801)'':0.00799209,('PDT000661395.1':0.0906535,(('PDT000316947.1':0.049557,'PDT000395186.1':0.055102)'':0.00186566,('PDT000394840.1':0.0362325,('PDT000566000.1':0.0349315,'PDT000205417.1':0.0376249)'':0.0080379)'':0.00821167)'':0.0191291)'':-0.00147558)'':0.0194247)'':0.00317585)'':0.00154033)'':0.0298916)'':-0.0280622)'':0.00610283)'':0.00646304)'':0.0727412)'':0.0122188,(((((('PDT000098158.2':0.0243648,'PDT000357725.1':0.0255912)'':0.0473352,(('PDT000008276.2':0.044294,'PDT000358773.1':0.0437129)'':0.00683349,(('PDT000357685.1':0.031668,'PDT000098157.2':0.0322443)'':0.00113549,('PDT000132200.2':0.039474,('PDT000208678.1':0.0318023,('PDT000062972.2':0.0287131,('PDT000097449.2':0.0237903,('PDT000316518.1':0.019976,('PDT000132107.2':0.00439877,'PDT000610576.1':0.00557828)'':0.00946633)'':0.00429248)'':0.00620498)'':0.00120581)'':0.00193161)'':0.00380328)'':0.0129514)'':0.0188963)'':0.00213335,(((('PDT000132310.2':0.0472899,'PDT000610828.1':0.0366316)'':0.00205252,('PDT000320226.1':0.0342509,('PDT000131998.2':0.0142225,'PDT000484723.1':0.0137481)'':0.0200815)'':0.0101761)'':0.00356437,(('PDT000161487.2':0.0249188,('PDT000322047.1':0.00558601,'PDT000610698.1':0.00587729)'':0.016182)'':0.0050228,(('PDT000651395.1':0.0347783,(('PDT000610686.1':0.0149001,'PDT000610856.1':0.0175816)'':0.00231973,('PDT000157186.2':0.0125419,('PDT000702588.1':0.0137062,('PDT000132038.2':0.00854088,'PDT000132080.2':0.00794862)'':0.00519697)'':0.00270745)'':7.03264e-05)'':0.0116786)'':-0.00675987,('PDT000651779.1':0.0305906,(('PDT000687096.1':0.014693,('PDT000323014.1':0.01217,'PDT000557793.1':0.00802102)'':0.00201036)'':0.00437996,(('PDT000394254.1':0.0178428,'PDT000442722.1':0.0154116)'':0.00494512,('PDT000600075.1':0.0119271,(('PDT000324045.1':0.0143592,'PDT000333265.1':0.0133197)'':0.00246421,('PDT000610590.1':0.0171229,('PDT000481470.1':0.00978806,('PDT000350575.1':0.00475569,'PDT000679864.1':0.00561731)'':0.00465394)'':0.00384208)'':-0.000138149)'':0.0010947)'':0.00462189)'':0.000241303)'':0.00860416)'':-0.00324889)'':0.00129802)'':0.0172041)'':0.00438233,(('PDT000155558.2':0.0254004,('PDT000316204.1':0.0207484,('PDT000155840.2':0.0134568,('PDT000316878.1':0.00994919,'PDT000618534.1':0.00979871)'':0.00609137)'':0.00576769)'':0.00275401)'':0.0359101,(('PDT000173330.1':0.0473316,('PDT000249844.1':0.0395883,'PDT000358759.1':0.0356983)'':0.00512755)'':0.0184044,(('PDT000132364.2':0.0424349,(('PDT000213275.2':0.0239488,('PDT000314194.2':0.0233697,'PDT000610754.1':0.0252189)'':0.00273367)'':0.00523834,('PDT000610657.1':0.0301424,('PDT000610568.1':0.00383765,'PDT000610615.1':0.00473919)'':0.0259139)'':0.00253042)'':0.00492918)'':-0.00114506,((('PDT000155324.2':0.0209561,'PDT000161521.2':0.0276564)'':0.00277899,('PDT000519372.1':0.0143518,'PDT000610677.1':0.0154708)'':0.01187)'':0.00492901,(('PDT000712688.1':0.016471,('PDT000326186.1':0.00922752,'PDT000484209.1':0.00843468)'':0.00616266)'':0.00712099,('PDT000380338.1':0.0254912,('PDT000161565.2':0.0103559,('PDT000191764.2':0.00801022,('PDT000318578.1':0.00654119,('PDT000610571.1':0.00789557,'PDT000610674.1':0.00749673)'':0.00128036)'':0.00239544)'':0.0026786)'':0.006445)'':0.00668086)'':0.00339328)'':0.00558569)'':0.0174885)'':0.00176137)'':-0.00730034)'':0.020905)'':0.11024,(((('PDT000129851.2':0.118856,'PDT000240293.1':0.10938)'':0.00344945,(('PDT000228033.1':0.0546406,'PDT000345917.1':0.0491214)'':0.0273399,('PDT000441069.2':0.0731508,('PDT000523903.1':0.048581,'PDT000488241.1':0.0396087)'':0.0353658)'':0.0192183)'':0.0360146)'':0.0356877,(('PDT000252519.1':0.0949592,(('PDT000394970.1':0.0742242,('PDT000645127.1':0.0468151,('PDT000713786.1':0.030817,'PDT000661404.1':0.0332122)'':0.00982958)'':0.0221523)'':0.00432637,(('PDT000251892.1':0.0578392,('PDT000601914.1':0.0283738,('PDT000004350.2':0.0306433,'PDT000027896.2':0.0298449)'':0.00330348)'':0.0218191)'':0.0168687,(('PDT000497408.1':0.0358671,'PDT000566701.1':0.0445582)'':0.00230471,('PDT000098358.2':0.0415408,('PDT000204995.2':0.0238319,'PDT000318968.1':0.0219479)'':0.00630433)'':0.00492666)'':0.0335687)'':0.00432968)'':0.0215163)'':-0.00190055,(('PDT000639514.1':0.0371195,'PDT000394983.1':0.0308379)'':0.0658959,((('PDT000104985.2':0.0579833,('PDT000425817.1':0.0542729,'PDT000426553.1':0.0580621)'':0.00562721)'':0.00419861,('PDT000521062.1':0.0783845,('PDT000086972.1':0.0495668,('PDT000042608.1':0.04436,'PDT000357609.1':0.0347219)'':0.00932408)'':0.0111355)'':0.0122968)'':0.00966446,((('PDT000397665.1':0.0487605,('PDT000518584.1':0.0370278,'PDT000716068.1':0.0358314)'':0.0122494)'':0.00467581,(('PDT000425463.1':0.0573488,'PDT000716961.1':0.0606762)'':0.0133778,('PDT000397644.1':0.0427118,('PDT000011713.2':0.0471317,(('PDT000397685.1':0.0260067,('PDT000550338.1':0.0125451,'PDT000600643.1':0.0180438)'':0.0054553)'':-7.42261e-05,(('PDT000566087.1':0.0262833,'PDT000644957.1':0.0226903)'':0.00140457,('PDT000394997.1':0.0170145,'PDT000395397.1':0.0216362)'':0.00578938)'':0.00162736)'':0.0154463)'':0.00378302)'':0.0104861)'':0.00104628)'':0.0230226,(('PDT000426031.1':0.0706476,('PDT000019128.2':0.0511333,('PDT000132186.2':0.0409332,'PDT000251805.1':0.0412055)'':0.00687367)'':0.00457923)'':0.0227581,(('PDT000581173.1':0.0688386,'PDT000661823.1':0.0757644)'':0.00950334,('PDT000502095.1':0.0810788,(('PDT000714172.1':0.0489399,'PDT000435628.1':0.0388222)'':0.0117835,('PDT000566100.1':0.0558322,('PDT000042641.1':0.0549396,('PDT000572853.1':0.0604313,('PDT000535457.1':0.047598,('PDT000129843.2':0.0242437,'PDT000129916.2':0.0327695)'':0.00872031)'':0.00530497)'':0.00186944)'':0.0127149)'':0.00859772)'':0.010203)'':-0.00093236)'':-0.00163059)'':0.00367859)'':0.00672651)'':0.0106441)'':-0.00695829)'':0.079115)'':0.0117287,(((('PDT000032327.1':0.127631,(('PDT000047206.3':0.0163439,'PDT000602890.1':0.0296849)'':0.0749558,((('PDT000397822.1':0.0856971,'PDT000411293.1':0.0881359)'':0.00169278,('PDT000661216.1':0.083155,('PDT000129825.2':0.0748008,'PDT000621065.1':0.0862522)'':0.00420654)'':0.00483087)'':0.00224797,('PDT000251771.1':0.0846925,('PDT000352478.1':0.0782851,('PDT000092536.1':0.0573691,('PDT000109457.2':0.0436466,'PDT000712988.1':0.0511944)'':0.00271037)'':0.0144903)'':0.00660879)'':0.00931453)'':0.00652868)'':0.0260584)'':0.02222,(('PDT000709933.1':0.145439,('PDT000291137.2':0.0896843,(('PDT000088571.2':0.065447,'PDT000424768.1':0.061161)'':0.00330236,('PDT000140206.2':0.0531046,'PDT000425545.1':0.0754914)'':0.0183644)'':0.0152403)'':0.0347954)'':0.00846211,(('PDT000644681.1':0.0913725,'PDT000352495.1':0.101418)'':0.0553537,(('PDT000346672.1':0.0745877,'PDT000354197.1':0.0572863)'':0.0605978,(('PDT000243052.1':0.0531046,('PDT000526205.1':0.0521573,'PDT000249873.1':0.0518267)'':0.00212943)'':0.0692465,(('PDT000337001.1':0.0704074,'PDT000346669.1':0.0816856)'':0.0132334,('PDT000359336.1':0.0650544,('PDT000417376.1':0.032506,('PDT000526129.1':0.0231216,'PDT000463711.1':0.0327509)'':0.0125565)'':0.0341565)'':0.00611875)'':0.0426539)'':0.00579676)'':0.0125842)'':-0.00389552)'':0.00857675)'':0.00919204,((('PDT000414194.1':0.105369,('PDT000395115.1':0.0743356,('PDT000592120.1':0.0683682,'PDT000411299.1':0.0808238)'':0.0121129)'':0.00446526)'':0.0569485,(('PDT000150229.2':0.101528,('PDT000557235.1':0.0796367,'PDT000200955.3':0.100184)'':0.0113965)'':0.0380685,('PDT000397950.1':0.144873,(('PDT000551880.1':0.0896532,'PDT000243030.1':0.0969638)'':0.0190706,('PDT000226488.2':0.120141,(('PDT000047737.3':0.0990027,'PDT000661496.1':0.0965043)'':0.0121482,(('PDT000426752.1':0.0476708,'PDT000571134.1':0.0430384)'':0.0232559,('PDT000524054.1':0.0723503,(('PDT000109421.2':0.0532975,'PDT000111321.1':0.0569485)'':0.00469294,('PDT000417940.1':0.0365888,('PDT000111368.1':0.0369702,('PDT000218119.2':0.0419152,('PDT000150875.1':0.0373081,('PDT000273394.2':0.0310753,(('PDT000399992.1':0.0251858,'PDT000384482.1':0.0205749)'':0.00408687,('PDT000111340.1':0.0245332,('PDT000517277.1':0.0216594,('PDT000343866.1':0.0128327,'PDT000400007.1':0.014095)'':0.00663795)'':0.00566504)'':2.93957e-05)'':0.00364486)'':0.00482184)'':0.00247893)'':0.00373754)'':0.00215718)'':0.0105443)'':0.00889878)'':-0.0022757)'':0.025803)'':0.0227696)'':-0.001139)'':0.0316358)'':-0.00426289)'':-0.00189598)'':0.00339473,((('PDT000174371.1':0.104091,('PDT000711071.1':0.081041,('PDT000285338.1':0.0577173,'PDT000373314.1':0.0429357)'':0.0411785)'':0.0172796)'':-0.00196033,(('PDT000025994.1':0.0647078,('PDT000025995.1':0.00223849,'PDT000026019.1':0.00185414)'':0.0358674)'':0.0394638,(('PDT000155738.2':0.0599473,'PDT000321336.1':0.0550837)'':0.00346456,(('PDT000253448.2':0.016879,'PDT000611960.1':0.0169693)'':0.00278041,('PDT000456849.1':0.0120648,('PDT000649108.1':0.00337675,'PDT000668154.2':0.00280532)'':0.00654763)'':0.00580531)'':0.049199)'':0.0151988)'':0.0194081)'':0.0574066,((('PDT000003953.2':0.11666,'PDT000394809.1':0.13193)'':0.0117426,('PDT000346941.1':0.124272,('PDT000346937.1':0.076251,(('PDT000095699.2':0.0545819,'PDT000394869.1':0.0585021)'':0.0261463,('PDT000645543.2':0.0539939,('PDT000411300.1':0.0314934,'PDT000411313.1':0.0289568)'':0.0220645)'':0.0272528)'':0.00638037)'':0.0250918)'':0.00185892)'':0.00988367,((('PDT000131995.2':0.0855464,('PDT000377933.1':0.0964861,('PDT000205425.1':0.086731,'PDT000435625.1':0.074361)'':0.0143924)'':0.00247768)'':0.0338187,(('PDT000501990.1':0.0647242,('PDT000069119.2':0.0759818,'PDT000568769.1':0.0688802)'':0.00970127)'':0.000378855,('PDT000565861.1':0.0810903,(('PDT000426805.1':0.0654592,'PDT000069712.1':0.0433558)'':0.0146461,('PDT000497309.1':0.0215115,'PDT000042537.1':0.0225449)'':0.0488922)'':0.0153499)'':0.005316)'':0.0275071)'':0.0218262,(('PDT000645060.1':0.112037,('PDT000034500.3':0.0616274,'PDT000395095.1':0.0745126)'':0.029737)'':0.00801286,(('PDT000150299.2':0.0662566,('PDT000141382.2':0.0590607,'PDT000251962.1':0.0477543)'':0.00445894)'':0.0168888,(('PDT000004345.2':0.108666,('PDT000444918.2':0.102832,('PDT000650286.1':0.0629247,'PDT000357608.1':0.0583013)'':0.0254969)'':0.00761229)'':-0.00105236,('PDT000660651.1':0.0617018,(('PDT000343948.1':0.0692629,('PDT000129917.2':0.0584584,'PDT000319016.1':0.0697416)'':0.00311557)'':0.00248125,(('PDT000306526.2':0.0521448,'PDT000397818.1':0.0497152)'':0.00933009,(('PDT000385047.1':0.051295,'PDT000399668.1':0.062383)'':0.00256363,('PDT000129887.2':0.0457426,('PDT000354938.1':0.0456802,'PDT000400018.1':0.0403755)'':0.0102252)'':0.00953164)'':0.00421613)'':0.0134753)'':0.00284521)'':0.00351523)'':0.00856203)'':0.00450318)'':0.0366349)'':0.00490077)'':0.0075381)'':0.0187789)'':0.00889549)'':0.00578851,(((('PDT000083585.1':0.0359303,('PDT000332770.1':0.0387408,'PDT000651401.1':0.038502)'':0.0023302)'':0.0112863,(('PDT000161409.2':0.0184425,'PDT000083504.1':0.0239505)'':0.00533132,('PDT000316157.1':0.0313584,('PDT000316137.1':0.0192106,'PDT000324081.1':0.0151869)'':0.00792298)'':0.000897307)'':0.0243514)'':0.0945999,(('PDT000652353.1':0.159476,'PDT000652396.1':0.163372)'':0.0217499,('PDT000150244.2':0.103021,(('PDT000494413.1':0.0797204,('PDT000399966.2':0.042621,'PDT000242493.1':0.036652)'':0.0432311)'':0.0172833,(('PDT000103008.1':0.0664569,'PDT000248783.1':0.0686731)'':0.0167578,('PDT000441068.2':0.0976284,('PDT000346679.1':0.0717289,'PDT000468891.1':0.0722291)'':0.0218026)'':0.000878197)'':0.00340289)'':0.0071688)'':0.0439031)'':-0.00902527)'':0.030559,((('PDT000414193.1':0.0908862,('PDT000621034.1':0.0674541,('PDT000346575.1':0.057079,(('PDT000417179.1':0.0481865,'PDT000594464.2':0.0503474)'':0.00216511,('PDT000535016.1':0.0275458,'PDT000463632.1':0.0256436)'':0.0151034)'':0.00563105)'':0.013617)'':0.00604173)'':0.0368559,(('PDT000652255.1':0.111317,('PDT000709912.1':0.0519568,('PDT000437961.1':0.0139279,'PDT000407985.1':0.0112705)'':0.025182)'':0.057812)'':0.00607332,((('PDT000706523.1':0.103478,(('PDT000718952.1':0.0461252,('PDT000271545.2':0.0275624,'PDT000394948.1':0.0358671)'':0.0121808)'':0.0142747,('PDT000218932.1':0.0641931,('PDT000362889.1':0.0546619,('PDT000604327.1':0.0426528,'PDT000501930.1':0.0454922)'':0.00865255)'':0.00237322)'':0.000434854)'':0.0334357)'':-0.0191532,(('PDT000264632.2':0.0616779,'PDT000111372.1':0.0578441)'':0.034205,('PDT000713320.1':0.0936532,('PDT000340385.1':0.102541,('PDT000395232.1':0.0838494,('PDT000589754.1':0.0691497,(('PDT000376086.1':0.0406562,('PDT000688645.1':0.0115755,(('PDT000524047.1':0.00670961,'PDT000524069.1':0.00903529)'':0.0028742,('PDT000524085.1':0.00959435,'PDT000524093.1':0.010132)'':0.00150417)'':0.00463227)'':0.0269352)'':0.00314884,('PDT000719148.1':0.0503636,('PDT000242934.1':0.053208,(('PDT000537952.1':0.037372,('PDT000372330.1':0.0249235,'PDT000342661.1':0.0252861)'':0.0057978)'':0.000196942,('PDT000471044.1':0.0457635,('PDT000145167.2':0.0206977,('PDT000631556.1':0.0160741,('PDT000093013.2':0.0162911,'PDT000564916.1':0.012703)'':0.00194185)'':0.00142543)'':0.0152246)'':0.00237377)'':0.0109853)'':0.00275029)'':-0.00748185)'':0.0187484)'':0.00692915)'':0.00629925)'':0.00187957)'':0.00382459)'':-0.0079879)'':0.00641102,(('PDT000660090.2':0.0660259,('PDT000132558.1':0.0454747,('PDT000675220.1':0.0420918,'PDT000482845.1':0.0455037)'':0.01129)'':0.0106247)'':0.000130903,((('PDT000431883.1':0.0248311,'PDT000674942.1':0.0360831)'':0.0307774,('PDT000168771.1':0.059733,(('PDT000534787.1':0.0237925,'PDT000226137.1':0.0302245)'':0.0142325,('PDT000484656.1':0.051736,('PDT000398350.1':0.0289423,('PDT000675032.1':0.0177377,'PDT000177286.1':0.0224404)'':0.0149702)'':0.00917438)'':0.00123032)'':0.00645822)'':0.00819521)'':0.0076815,((('PDT000433650.1':0.0492037,'PDT000674935.1':0.0606133)'':0.00966802,('PDT000202646.2':0.0500969,('PDT000202029.2':0.04206,('PDT000426532.1':0.0351546,'PDT000520944.1':0.0396753)'':0.00345404)'':0.00772831)'':0.00846429)'':-0.00118989,('PDT000364513.1':0.0614211,((('PDT000491867.1':0.0193349,'PDT000689967.1':0.0258863)'':0.0244814,('PDT000202736.2':0.0393455,('PDT000143947.3':0.0285636,'PDT000289927.1':0.0337347)'':0.00713345)'':0.0059908)'':0.00522154,((('PDT000375422.1':0.056383,('PDT000297984.1':0.028943,'PDT000495425.1':0.032801)'':0.00765748)'':0.00685881,(('PDT000443030.1':0.0239505,'PDT000534799.1':0.0290017)'':0.00690983,('PDT000446002.1':0.0257265,'PDT000674911.1':0.02029)'':0.00295574)'':0.00994758)'':0.0101763,(('PDT000425546.1':0.0474787,'PDT000673532.1':0.0475682)'':0.0101559,(('PDT000202171.2':0.0240199,'PDT000425705.1':0.0209161)'':0.0110348,(('PDT000425654.1':0.0151663,'PDT000425697.1':0.0197462)'':0.0170153,('PDT000466084.1':0.0198247,'PDT000632999.1':0.0236062)'':0.00458345)'':0.00912935)'':0.0101873)'':0.00200558)'':0.00394262)'':0.0107951)'':-0.0092109)'':0.00678226)'':0.0137736)'':0.0174845)'':0.014128)'':0.00833768)'':0.0432778,(((('PDT000542497.1':0.10866,('PDT000426141.1':0.0953131,('PDT000668914.1':0.0887207,('PDT000336607.1':0.0415081,('PDT000233289.1':0.0419839,'PDT000346667.1':0.0456395)'':0.0101006)'':0.0260137)'':0.017284)'':0.00848822)'':0.013995,(('PDT000346445.1':0.0787817,('PDT000395486.1':0.0573193,'PDT000395322.1':0.0674287)'':0.00847132)'':0.00853208,('PDT000228012.1':0.0755864,('PDT000228084.1':0.0511074,('PDT000557047.1':0.0475792,'PDT000228136.1':0.0615778)'':0.00287293)'':0.0146021)'':0.00392595)'':0.0529111)'':0.0236757,(('PDT000095773.2':0.121734,(('PDT000644922.1':0.0639257,'PDT000171789.1':0.0413273)'':0.058183,('PDT000716167.1':0.125712,('PDT000399608.1':0.0892799,('PDT000107237.2':0.0768234,'PDT000397610.1':0.0828636)'':0.0107386)'':0.00966494)'':0.0197469)'':0.0169556)'':0.016584,('PDT000251772.1':0.134141,(('PDT000097371.3':0.0474911,'PDT000497795.1':0.0494838)'':0.0892149,('PDT000014132.2':0.128604,('PDT000249826.1':0.0984787,(('PDT000124662.1':0.0713233,'PDT000373312.1':0.0551227)'':0.0227526,(('PDT000395070.1':0.047068,'PDT000571181.1':0.0403753)'':0.0473318,(('PDT000290427.2':0.0383319,('PDT000252459.1':0.0288403,('PDT000251547.1':0.00499671,'PDT000251548.1':0.00387136)'':0.0163981)'':0.0101969)'':0.0420179,('PDT000109305.2':0.0840537,(('PDT000252611.1':0.0484056,('PDT000109379.2':0.0281336,'PDT000252595.1':0.0356376)'':0.0245859)'':0.0096826,('PDT000557122.1':0.0631361,('PDT000197013.1':0.0491898,('PDT000220295.1':0.040922,'PDT000251418.1':0.0363991)'':0.00699862)'':0.00745187)'':0.0043241)'':0.0117654)'':-0.00575022)'':0.00582187)'':0.0037879)'':0.0110159)'':0.0173433)'':-0.00161486)'':0.0144763)'':0.014241)'':0.000281876)'':0.00831835,(((('PDT000488299.1':0.0680768,('PDT000251877.1':0.0566123,'PDT000252425.1':0.0577167)'':0.0120497)'':0.00754786,('PDT000463610.1':0.0929585,(('PDT000003797.3':0.0692963,'PDT000714687.1':0.0522227)'':0.0101427,(('PDT000425941.1':0.0616105,'PDT000129697.1':0.0493945)'':0.00948027,('PDT000710442.1':0.0672771,('PDT000109409.2':0.0449244,'PDT000652571.1':0.0438816)'':0.0104949)'':0.0082677)'':0.00757393)'':0.00441115)'':0.0045219)'':0.065361,(('PDT000274334.1':0.0506278,('PDT000306552.2':0.0548669,'PDT000672175.1':0.0463711)'':0.00167401)'':0.0912338,(('PDT000626531.1':0.0206504,'PDT000651545.1':0.015792)'':0.0719206,('PDT000688714.1':0.108165,(('PDT000394891.1':0.032106,'PDT000642301.1':0.0428177)'':0.025114,('PDT000129844.3':0.0638418,('PDT000251921.1':0.0383702,'PDT000357662.1':0.0469214)'':0.00856387)'':0.00222167)'':0.0197204)'':0.000218194)'':0.0512703)'':0.00960487)'':0.00136831,(('PDT000003787.2':0.110004,('PDT000109490.2':0.0838715,('PDT000502697.1':0.0742987,('PDT000703274.1':0.0507792,'PDT000252535.1':0.0609108)'':0.0054353)'':0.0212974)'':0.00872091)'':0.0474427,(((('PDT000661360.1':0.0977744,('PDT000571186.1':0.0789013,'PDT000354191.1':0.0763117)'':0.0195666)'':0.0175681,(('PDT000667754.2':0.115809,('PDT000285776.2':0.107823,('PDT000372156.1':0.0726784,('PDT000011774.2':0.0598546,('PDT000228320.1':0.0390253,'PDT000252561.1':0.0370806)'':0.00568283)'':0.0228162)'':0.0140008)'':0.026947)'':-0.00989547,(('PDT000160930.2':0.066217,('PDT000517538.1':0.0539771,'PDT000011737.2':0.0496479)'':0.00727946)'':0.0254093,('PDT000017879.2':0.0905674,('PDT000343612.1':0.0637319,'PDT000488267.1':0.0717001)'':0.0144651)'':0.0191412)'':0.00560018)'':0.0122242)'':0.000590864,('PDT000648080.1':0.205172,(('PDT000631848.1':0.136758,'PDT000395216.1':0.0726372)'':0.0288318,(('PDT000191867.2':0.0824117,('PDT000407066.1':0.0617843,'PDT000520703.1':0.0718837)'':0.0369188)'':0.00881638,('PDT000150847.1':0.101422,(('PDT000479550.1':0.0695066,'PDT000395495.1':0.0701204)'':0.0348736,('PDT000228089.1':0.107443,(('PDT000111344.1':0.0536489,('PDT000109471.2':0.047923,'PDT000566181.1':0.0446287)'':0.0170298)'':0.00558505,('PDT000174338.1':0.0424982,('PDT000174328.1':0.0400278,'PDT000174329.1':0.0303411)'':0.00464095)'':0.0405761)'':0.0125848)'':0.00616832)'':0.00175066)'':-0.00140018)'':0.0112332)'':0.0276404)'':-0.0199623)'':0.00107368,(('PDT000174375.1':0.114574,(('PDT000156567.2':0.0787405,(('PDT000397804.1':0.0474258,'PDT000588049.1':0.0519833)'':0.00428628,('PDT000151502.3':0.0487179,('PDT000311145.2':0.0373886,'PDT000136052.2':0.044246)'':0.0109049)'':0.00177566)'':0.00784474)'':0.0143745,(('PDT000718894.1':0.0976411,(('PDT000626935.1':0.06061,('PDT000427804.1':0.0434035,'PDT000111338.1':0.0349438)'':0.0178644)'':0.00293224,(('PDT000654250.1':0.0559329,('PDT000720226.1':0.0251444,'PDT000252188.1':0.0280639)'':0.0278426)'':0.00127802,('PDT000537534.1':0.063459,('PDT000479636.1':0.0345133,('PDT000720233.1':0.0294098,('PDT000362885.1':0.0296619,('PDT000471021.1':0.0290332,'PDT000471023.1':0.0180221)'':0.00540142)'':0.00307448)'':0.0116691)'':0.023672)'':0.00125519)'':0.00733109)'':0.0211313)'':-0.0103087,((('PDT000495050.1':0.0538946,'PDT000395445.1':0.0549744)'':0.00806809,('PDT000517930.1':0.0394156,'PDT000019616.1':0.0421481)'':0.0158007)'':0.0205535,('PDT000174307.1':0.0836578,(('PDT000494425.1':0.0636551,'PDT000713423.1':0.0505319)'':0.0167964,(('PDT000017888.2':0.0532832,('PDT000251990.1':0.053338,('PDT000645140.1':0.0287891,('PDT000277741.2':0.00159909,'PDT000653456.1':0.00139821)'':0.0285329)'':0.0218547)'':0.00213871)'':0.00515633,(('PDT000088269.2':0.0360483,('PDT000496891.1':0.024698,'PDT000124608.1':0.0214513)'':0.0104312)'':0.0198348,('PDT000400000.1':0.0392228,('PDT000015123.2':0.021496,('PDT000015112.2':0.0135944,'PDT000015113.2':0.010533)'':0.00766333)'':0.0174176)'':0.0118315)'':0.00230277)'':0.0110754)'':0.00540951)'':-0.00635583)'':0.00205228)'':0.00310678)'':0.0282981)'':-0.0265893,(('PDT000709952.1':0.0630717,(('PDT000252159.1':0.0474877,('PDT000482373.1':0.033705,'PDT000563582.1':0.0401603)'':0.0135567)'':0.0103192,(('PDT000497387.1':0.0452168,'PDT000242489.1':0.0351798)'':0.0107692,('PDT000395315.1':0.055775,('PDT000639560.1':0.0594325,('PDT000397879.1':0.0413037,('PDT000591885.1':0.0298113,('PDT000178595.2':0.0182391,'PDT000271528.2':0.0214293)'':0.0155789)'':0.00576777)'':0.011946)'':0.00455798)'':-0.0012028)'':0.00159529)'':0.00837981)'':0.018176,((('PDT000191152.1':0.0741652,('PDT000395077.1':0.0712856,'PDT000397547.1':0.0693884)'':0.0050858)'':0.0021521,((('PDT000537211.1':0.0510819,'PDT000687935.1':0.0461055)'':0.0101605,('PDT000713676.1':0.0539979,('PDT000661322.1':0.0466482,'PDT000111366.1':0.0488889)'':0.0103851)'':0.00221689)'':0.00137167,((('PDT000333778.1':0.0420756,'PDT000600523.1':0.0375594)'':0.0129952,(('PDT000566219.1':0.0320974,'PDT000081063.1':0.0313576)'':0.020663,('PDT000652327.1':0.0525184,('PDT000361002.1':0.0384129,('PDT000497370.1':0.0111041,'PDT000645034.1':0.0131677)'':0.0225397)'':0.00646388)'':0.000125259)'':0.00364642)'':0.00444614,('PDT000639594.1':0.0614522,('PDT000500483.1':0.0500427,(('PDT000397794.1':0.0365851,('PDT000066402.2':0.0378802,'PDT000069114.2':0.0397193)'':0.00677584)'':0.00283755,(('PDT000052519.2':0.0281842,'PDT000003839.2':0.0235702)'':0.00125024,(('PDT000060188.2':0.0139621,'PDT000399677.1':0.0182384)'':0.00066364,('PDT000003872.2':0.0103154,'PDT000395008.1':0.0101333)'':0.010003)'':0.00931665)'':0.00925699)'':0.0125766)'':0.00781754)'':0.00257346)'':0.0057018)'':0.00177363)'':0.0092286,((('PDT000482533.1':0.0690267,('PDT000343936.1':0.0407194,'PDT000716943.1':0.0479482)'':0.0141845)'':0.0105471,(('PDT000470049.1':0.0337292,'PDT000636364.1':0.0363357)'':0.006255,(('PDT000003868.2':0.0334367,'PDT000399616.1':0.0409048)'':0.00660863,('PDT000241771.1':0.0395647,('PDT000051501.2':0.0265539,('PDT000397918.1':0.0209011,'PDT000566155.1':0.0160366)'':0.00905742)'':0.00798387)'':0.00180836)'':0.00815139)'':0.0312492)'':0.000350101,(((('PDT000225860.2':0.0204466,'PDT000422111.1':0.0229975)'':0.00183721,('PDT000306437.2':0.0162347,'PDT000509509.1':0.0132618)'':0.00572042)'':0.0416066,('PDT000713274.1':0.0404542,('PDT000360987.1':0.0407015,('PDT000459680.1':0.0454099,'PDT000642577.1':0.0359809)'':0.00527958)'':0.00235019)'':0.0172824)'':0.00213824,('PDT000709932.1':0.118432,(('PDT000432553.1':0.0713882,'PDT000534806.1':0.0596068)'':0.0171371,(('PDT000640117.1':0.0759477,('PDT000502540.1':0.0743899,('PDT000363556.1':0.0580385,('PDT000272565.2':0.0572898,('PDT000288425.2':0.044912,'PDT000566012.1':0.0434445)'':0.00800941)'':0.000934828)'':0.00619185)'':0.00357685)'':-0.00360016,((('PDT000645613.2':0.0560359,'PDT000024454.1':0.0467701)'':0.00605708,('PDT000003812.2':0.0736646,('PDT000720250.1':0.038218,('PDT000720195.1':0.0313995,'PDT000218925.1':0.0266013)'':0.00987281)'':0.0151346)'':0.00534636)'':0.000986745,('PDT000675260.1':0.0535321,('PDT000321348.1':0.0658977,('PDT000111381.1':0.0409165,('PDT000277854.1':0.0678977,('PDT000688005.1':0.0363292,('PDT000632985.1':0.00329544,'PDT000583390.1':0.0024901)'':0.0385543)'':0.00502684)'':0.0038007)'':0.00461391)'':0.00194111)'':0.00629273)'':0.00207122)'':0.00899634)'':0.0135045)'':-0.0144429)'':0.00320803)'':0.000796058)'':0.00694575)'':-0.00079414)'':0.0386484)'':0.0274106)'':0.00613986)'':-0.00120003)'':0.00349639)'':0.0120859)'':-0.00517863)'':0.00366403)'':0.00485965)'':0.000875981,(('PDT000083566.1':0.131587,('PDT000444610.1':0.100962,('PDT000556871.1':0.0622007,'PDT000345918.1':0.0799143)'':0.0383371)'':0.0341181)'':0.0761809,((((('PDT000702836.1':0.1616,('PDT000394902.1':0.11863,'PDT000395497.1':0.116913)'':0.0125047)'':-0.000209858,((('PDT000620738.1':0.10037,'PDT000212790.1':0.116283)'':0.00996646,('PDT000228120.1':0.081587,'PDT000354187.1':0.087233)'':0.0176588)'':0.0275157,((('PDT000250204.1':0.0453523,'PDT000252650.1':0.0495271)'':0.0490734,('PDT000252661.1':0.0406857,'PDT000463621.1':0.047561)'':0.0390988)'':0.0218826,('PDT000109413.2':0.128583,((('PDT000502496.1':0.0816046,('PDT000095805.2':0.0553776,'PDT000267174.1':0.0576214)'':0.0115574)'':0.00575365,('PDT000691344.1':0.0644188,('PDT000706095.1':0.0565153,('PDT000719233.1':0.0299823,'PDT000017905.2':0.0324692)'':0.0151608)'':0.00497103)'':0.0188826)'':0.0258657,('PDT000488233.1':0.0891472,(('PDT000242950.1':0.0482955,'PDT000251701.1':0.0611455)'':0.0201289,('PDT000491123.1':0.049942,('PDT000717056.1':0.0626647,('PDT000109454.2':0.04653,('PDT000178579.2':0.0312534,('PDT000045584.1':0.0300084,('PDT000316827.1':0.0154042,'PDT000471013.1':0.0197637)'':0.00770304)'':0.00291241)'':0.0170663)'':0.013651)'':0.00638793)'':0.0177106)'':0.00747977)'':0.0166084)'':0.014725)'':-0.00893594)'':0.0162642)'':0.00792223)'':0.0145907,(('PDT000251693.1':0.133399,(('PDT000150867.1':0.0867137,('PDT000384980.1':0.0548616,(('PDT000396846.1':0.0495928,('PDT000333764.1':0.0252084,'PDT000251793.1':0.0216727)'':0.0300965)'':0.00186246,('PDT000396825.1':0.044399,('PDT000309333.2':0.0268931,'PDT000329274.1':0.0345968)'':0.00891886)'':0.0166957)'':0.0206697)'':0.00265074)'':0.0278939,(('PDT000469956.1':0.08317,('PDT000714243.1':0.0267514,('PDT000220290.1':0.0198899,('PDT000313313.2':0.00956146,'PDT000479794.1':0.0187187)'':0.00937384)'':0.00191836)'':0.0619557)'':0.0142613,(('PDT000174301.1':0.033891,'PDT000174381.1':0.0402407)'':0.0527058,(('PDT000718922.1':0.00320617,'PDT000718923.1':0.00416876)'':0.091481,('PDT000109374.2':0.0763019,('PDT000309337.2':0.0591902,'PDT000581035.1':0.0559058)'':0.0125651)'':0.00607664)'':0.00343419)'':0.0148278)'':0.00648519)'':0.0176537)'':-0.0144881,(('PDT000319868.1':0.0665646,'PDT000111349.1':0.0677634)'':0.0561928,(('PDT000557187.1':0.110743,'PDT000568778.1':0.10342)'':0.0253509,('PDT000621059.1':0.140206,(('PDT000252005.1':0.103215,((('PDT000357623.1':0.0682153,'PDT000483527.1':0.0814877)'':0.0282058,('PDT000719006.1':0.0810929,'PDT000129895.2':0.10234)'':0.00301797)'':0.0090885,(('PDT000417501.1':0.0604208,'PDT000435627.1':0.0655902)'':0.0280484,('PDT000625976.1':0.0888152,'PDT000661217.1':0.0918618)'':0.0144623)'':0.00485757)'':0.0094201)'':-0.00814388,(('PDT000271787.1':0.0272543,'PDT000491125.1':0.0233313)'':0.094961,(('PDT000060734.2':0.0977715,('PDT000346526.1':0.0521397,('PDT000313330.2':0.0408116,('PDT000394888.1':0.021505,'PDT000584790.1':0.0211857)'':0.0131258)'':0.029705)'':0.0207754)'':0.004507,((('PDT000688631.1':0.0325355,'PDT000346227.1':0.0418398)'':0.017571,('PDT000459675.1':0.07408,('PDT000354910.1':0.038529,'PDT000668912.1':0.0361325)'':0.0247137)'':0.00642534)'':0.0163302,(('PDT000661225.1':0.040111,('PDT000395381.1':0.0404106,(('PDT000288578.2':0.0211954,'PDT000672205.1':0.0166483)'':0.00458124,('PDT000619644.1':0.0248238,('PDT000196590.2':0.0160479,'PDT000424756.1':0.0111771)'':0.0051681)'':0.00312931)'':0.00837865)'':0.00318471)'':0.0185178,('PDT000491105.1':0.0734168,('PDT000109453.2':0.0753396,(('PDT000111328.1':0.0611842,'PDT000251711.1':0.0602848)'':0.00767389,(('PDT000556995.1':0.0466797,'PDT000252651.1':0.0440035)'':0.00602156,(('PDT000710445.1':0.0508844,'PDT000249862.1':0.0439523)'':0.00143029,('PDT000425807.1':0.050036,('PDT000352513.1':0.0523254,('PDT000710440.1':0.0369452,('PDT000517270.1':0.0287822,'PDT000250112.1':0.0386863)'':0.00316125)'':0.00542185)'':0.00340986)'':0.00241767)'':0.000977075)'':0.0144209)'':0.00543897)'':0.00469743)'':-0.0140761)'':0.00772038)'':0.00366133)'':0.0409093)'':-0.0135074)'':0.0214811)'':-0.0126665)'':0.00703625)'':-0.0196212)'':0.0402348)'':0.0064113,(('PDT000421003.2':0.136442,(('PDT000714652.1':0.0473553,'PDT000571174.1':0.0367605)'':0.00875209,(('PDT000616361.1':0.0384084,'PDT000127574.1':0.0415685)'':0.00438662,('PDT000538141.1':0.0381235,('PDT000228009.1':0.0270565,'PDT000249803.1':0.0334187)'':0.00993626)'':0.00798376)'':0.00507543)'':0.0551011)'':0.0411592,((('PDT000661994.1':0.0976796,'PDT000228272.1':0.0941724)'':0.0123521,('PDT000129877.2':0.126112,('PDT000346595.1':0.118286,('PDT000644902.1':0.0928102,('PDT000129836.2':0.0732271,('PDT000645070.1':0.0659021,'PDT000713543.1':0.0685699)'':0.0236064)'':0.0113135)'':0.0118932)'':0.00909859)'':0.0107197)'':0.0171044,(('PDT000157687.2':0.0363557,('PDT000251709.1':0.0164092,'PDT000252599.1':0.0211551)'':0.0206649)'':0.0787072,(('PDT000651087.1':0.092948,('PDT000294725.2':0.0636608,'PDT000395400.1':0.0386492)'':0.042327)'':0.016328,((('PDT000132057.2':0.0783524,'PDT000534062.1':0.0736956)'':0.00655995,(('PDT000526140.1':0.0682763,'PDT000171827.1':0.0696227)'':0.00627166,('PDT000584650.1':0.0970056,('PDT000467239.1':0.0774654,('PDT000032198.1':0.0586426,('PDT000344854.1':0.0417714,'PDT000661215.1':0.0482778)'':0.0112723)'':0.0108208)'':0.00528932)'':0.00777132)'':0.00686076)'':0.014808,((('PDT000238191.2':0.112458,'PDT000346610.1':0.104257)'':0.0173142,('PDT000661226.1':0.0947221,(('PDT000430828.1':0.09418,('PDT000649798.1':0.0815546,('PDT000427195.1':0.0734089,'PDT000340386.1':0.0626541)'':0.0023799)'':0.00327687)'':0.0111256,('PDT000346935.1':0.0853954,(('PDT000104647.2':0.0764685,'PDT000535655.1':0.0771215)'':0.00987137,('PDT000718989.1':0.0665609,'PDT000631766.1':0.0555471)'':0.00638738)'':0.0112337)'':-0.0026513)'':0.00990557)'':0.00778777)'':0.0040917,((('PDT000343877.1':0.0982106,('PDT000387336.1':0.0821497,('PDT000329856.1':0.0766595,'PDT000557249.1':0.0618675)'':0.00625183)'':0.0343579)'':0.00880312,('PDT000251751.1':0.0929391,('PDT000649088.1':0.0716643,('PDT000713303.1':0.0498303,('PDT000718988.1':0.0514167,'PDT000357637.1':0.0559933)'':0.00305165)'':0.0246668)'':0.0206093)'':0.00272817)'':0.00491673,(('PDT000127577.1':0.118657,(('PDT000652280.1':0.0856672,('PDT000588659.1':0.0669203,'PDT000463657.1':0.0554537)'':0.0146693)'':0.00852405,(('PDT000032351.1':0.00171944,('PDT000032357.1':0.00187457,('PDT000032361.1':0.000774422,'PDT000034378.1':0.000924218)'':0.000421813)'':0.000236044)'':0.0852645,(('PDT000101300.1':0.0664441,('PDT000443939.1':0.0259645,'PDT000571184.1':0.0386375)'':0.0170604)'':0.0218192,('PDT000521482.2':0.0673856,('PDT000101299.1':0.0585777,('PDT000032210.1':0.0019607,'PDT000032213.1':0.00233115)'':0.0573053)'':0.0159339)'':0.00683825)'':0.00445086)'':0.00225484)'':0.0247269)'':-0.0175818,(('PDT000032335.1':0.110769,('PDT000415837.1':0.109986,(('PDT000025152.2':0.0912983,('PDT000343602.1':0.0729002,'PDT000425877.1':0.0765578)'':0.0120137)'':0.00117391,(('PDT000427045.1':0.0789069,('PDT000120936.2':0.0625683,'PDT000415798.1':0.0706117)'':0.0166111)'':0.00272097,(('PDT000233323.1':0.083318,'PDT000358760.1':0.061534)'':0.00939805,('PDT000520701.1':0.0649444,('PDT000373308.1':0.0700313,('PDT000566225.1':0.0636526,('PDT000309572.2':0.0565207,'PDT000385042.1':0.0553253)'':0.00551787)'':0.00390693)'':0.00759978)'':0.000923066)'':0.00128185)'':0.0055409)'':0.0213573)'':0.0105242)'':-0.0192845,('PDT000428842.1':0.102646,(('PDT000346673.1':0.0866942,(('PDT000675482.1':0.0639334,'PDT000318982.1':0.0647516)'':0.0056183,('PDT000042518.1':0.0467752,'PDT000251694.1':0.040223)'':0.0271166)'':0.0141756)'':0.0055169,(('PDT000310613.2':0.127565,('PDT000345828.1':0.0843586,('PDT000228225.1':0.0775877,'PDT000228312.1':0.0646483)'':0.00822142)'':0.0183207)'':-0.00513705,(('PDT000188177.2':0.115181,('PDT000019322.2':0.0849049,('PDT000267335.3':0.0663426,'PDT000713639.1':0.0684114)'':0.0127881)'':0.0133248)'':0.00288861,(('PDT000346668.1':0.0959414,('PDT000708202.1':0.0659155,'PDT000346819.1':0.0651225)'':0.0135041)'':0.0155752,('PDT000015325.2':0.091682,(('PDT000346790.1':0.0759608,('PDT000032147.1':0.0740476,('PDT000343606.1':0.049073,('PDT000566235.1':0.0204923,'PDT000252525.1':0.0156392)'':0.0195582)'':0.0225504)'':0.0151911)'':0.00543144,('PDT000385029.1':0.0689032,(('PDT000644890.1':0.063945,'PDT000718981.1':0.059416)'':0.00697638,(('PDT000129901.2':0.0568533,'PDT000252543.1':0.0542047)'':0.00273684,('PDT000106255.2':0.0429577,'PDT000399990.1':0.049266)'':0.00694133)'':0.00164073)'':0.00432365)'':0.00390484)'':0.00523031)'':0.00308008)'':0.00446936)'':0.00302868)'':0.000175559)'':0.0148081)'':-0.0107906)'':0.00596391)'':0.00271493)'':0.00230957)'':0.00443937)'':0.0115706)'':0.00984544)'':0.0147838)'':0.02561)'':-0.0102128)'':0.00893891,((((('PDT000200114.2':0.0233653,'PDT000190718.3':0.0335227)'':0.0586951,('PDT000146686.3':0.0872579,('PDT000040382.4':0.05912,('PDT000502529.1':0.027339,('PDT000045619.1':0.0323414,'PDT000048133.1':0.0377211)'':0.00485169)'':0.0223868)'':0.0184376)'':0.0127332)'':0.0683474,((('PDT000496905.1':0.104408,'PDT000644936.1':0.0969688)'':0.00943507,('PDT000633390.1':0.08958,('PDT000707863.1':0.0794851,'PDT000230853.1':0.0869339)'':0.025255)'':0.00631559)'':0.0227041,(('PDT000491119.1':0.105303,('PDT000283582.1':0.0974252,'PDT000354185.1':0.0724078)'':0.0236858)'':0.00203097,('PDT000652877.1':0.131077,(('PDT000397643.1':0.0767366,'PDT000557356.1':0.0734824)'':0.00422669,('PDT000565971.1':0.0590722,('PDT000556964.1':0.0592896,'PDT000482814.1':0.0527424)'':0.00862276)'':0.0179927)'':0.0174549)'':0.017205)'':0.00578488)'':0.0227999)'':0.00534815,((('PDT000661147.1':0.149619,(('PDT000132130.2':0.0717485,'PDT000228245.1':0.0750975)'':0.0364979,('PDT000395103.1':0.0635584,'PDT000592114.1':0.0659596)'':0.0251046)'':0.0421497)'':0.00881085,((('PDT000249798.1':0.0757704,('PDT000691298.1':0.0880518,('PDT000557747.1':0.0466165,'PDT000627593.1':0.0631515)'':0.0356642)'':0.00310887)'':-0.00148765,('PDT000294409.1':0.0843258,('PDT000092359.1':0.00430381,('PDT000092338.1':0.00266783,'PDT000092358.1':0.00301625)'':0.000483171)'':0.0766462)'':0.0047676)'':0.0505485,('PDT000292891.1':0.144188,((('PDT000502194.1':0.100003,'PDT000250101.1':0.104485)'':0.00901415,('PDT000239091.1':0.0860124,('PDT000098364.2':0.0523095,'PDT000015333.2':0.0550195)'':0.0294766)'':0.0311651)'':0.0264854,('PDT000103025.1':0.116771,(('PDT000501928.1':0.0819215,'PDT000233843.2':0.0898045)'':0.00886656,('PDT000346558.1':0.119951,('PDT000588839.1':0.0822433,('PDT000466990.1':0.0835413,'PDT000433329.1':0.0984867)'':0.00667773)'':0.00769096)'':-0.000170785)'':0.0170477)'':-0.00289305)'':0.025678)'':-0.00469062)'':0.00517796)'':-0.00570269,((('PDT000620997.1':0.168237,('PDT000181819.2':0.152787,((('PDT000708205.1':0.0740869,'PDT000233322.1':0.0669511)'':0.0210248,('PDT000129817.2':0.0834073,('PDT000571135.1':0.039308,'PDT000607556.1':0.04745)'':0.0321177)'':0.0109891)'':0.00255665,((('PDT000310437.2':0.0472641,'PDT000617732.1':0.0414548)'':0.0101458,('PDT000604445.1':0.0539726,'PDT000237131.1':0.0578204)'':0.004004)'':0.0224127,('PDT000563564.1':0.073704,('PDT000346786.1':0.0747023,('PDT000191805.2':0.0556903,'PDT000242884.1':0.0522817)'':0.0136247)'':0.00780277)'':0.004965)'':0.011217)'':0.0483009)'':0.00667654)'':-0.0237737,((('PDT000132339.2':0.0825876,'PDT000233327.1':0.0992654)'':0.0217973,('PDT000626970.1':0.10633,'PDT000672167.1':0.0882761)'':0.0211444)'':0.00332643,('PDT000557680.2':0.122667,((('PDT000652510.1':0.0836255,('PDT000517920.1':0.0522223,'PDT000587989.1':0.0579517)'':0.011174)'':0.00865055,(('PDT000517596.1':0.0541596,'PDT000604342.1':0.0540694)'':0.0110963,('PDT000651123.1':0.0648238,'PDT000720228.1':0.0690092)'':0.00476868)'':0.00354121)'':0.0195118,(('PDT000577026.1':0.104339,(('PDT000181782.2':0.0376551,'PDT000345948.1':0.0428812)'':0.0554422,('PDT000644665.1':0.0599948,'PDT000565527.1':0.0887822)'':0.0156972)'':0.00725744)'':0.00141018,('PDT000548980.1':0.0872071,(('PDT000226475.2':0.0336528,'PDT000477065.1':0.0277843)'':0.0183324,('PDT000496904.1':0.0567232,'PDT000677988.1':0.0483098)'':0.00146259)'':0.0266972)'':0.0194383)'':0.00278944)'':0.0340336)'':-0.0156636)'':0.0101304)'':0.00478841,(('PDT000483528.1':0.139887,('PDT000426078.1':0.135258,('PDT000333936.2':0.0918938,('PDT000719073.1':0.0847034,('PDT000174313.1':0.0136726,'PDT000174372.1':0.0280688)'':0.0728134)'':0.00711129)'':0.0230272)'':0.010796)'':0.00835316,(('PDT000357639.1':0.105655,('PDT000251933.1':0.102554,('PDT000422537.1':0.0647884,('PDT000111702.2':0.0408551,('PDT000662613.1':0.0198485,'PDT000250148.1':0.0144089)'':0.0294607)'':0.0172537)'':0.0414788)'':0.00621478)'':0.00521395,(('PDT000251926.1':0.138704,(('PDT000501845.1':0.0662654,('PDT000277717.2':0.0848976,('PDT000661825.1':0.0455906,'PDT000346505.1':0.0465775)'':0.0124333)'':0.0142758)'':0.0128934,(('PDT000584552.2':0.102455,('PDT000353638.1':0.0385204,('PDT000485393.1':0.0267834,('PDT000150272.2':0.0379098,'PDT000384280.1':0.0386802)'':0.00296601)'':0.00884847)'':0.0572456)'':0.00546626,((('PDT000689839.1':0.0777034,'PDT000710419.1':0.0611646)'':0.00903157,(('PDT000109470.2':0.0421035,'PDT000111335.1':0.0331035)'':0.00744101,('PDT000386741.1':0.0118368,'PDT000417417.1':0.0183665)'':0.0307376)'':0.0325989)'':0.00153932,('PDT000651586.1':0.0657862,(('PDT000409819.1':0.0459981,('PDT000306433.2':0.0341844,'PDT000095762.2':0.0322291)'':0.0153569)'':0.00105361,(('PDT000264662.2':0.0174036,'PDT000403071.1':0.0176112)'':0.00873189,('PDT000660646.1':0.0273198,'PDT000660668.1':0.0207912)'':0.00432549)'':0.0185404)'':0.0066376)'':0.0314351)'':0.00584299)'':0.00757812)'':0.0349474)'':-0.0200602,((('PDT000611952.2':0.0238941,'PDT000132262.3':0.0165478)'':0.0175804,('PDT000532170.2':0.042381,('PDT000410717.2':0.0285184,('PDT000407654.2':0.0259741,(('PDT000598520.2':0.0187967,'PDT000587161.2':0.0173597)'':0.00322918,('PDT000326140.2':0.0194392,(('PDT000216613.3':0.00527349,'PDT000663794.1':0.00801191)'':0.00334986,('PDT000624200.2':0.0061545,'PDT000678122.1':0.0065831)'':0.00098301)'':0.00960567)'':-0.00163237)'':0.00929573)'':0.00367425)'':0.00570498)'':-0.00251637)'':0.0640264,(('PDT000479555.1':0.100375,('PDT000174373.1':0.058617,'PDT000357693.1':0.042852)'':0.0533629)'':0.00664395,(('PDT000492384.1':0.0875765,('PDT000517938.1':0.0371165,('PDT000109462.2':0.0285713,'PDT000353633.1':0.0307961)'':0.004968)'':0.0268568)'':0.0338647,(('PDT000346940.1':0.172722,((('PDT000661463.1':0.0485072,('PDT000714154.1':0.0416328,'PDT000018256.1':0.0375229)'':0.0175365)'':0.0127537,(('PDT000463646.1':0.0206423,'PDT000463653.1':0.0142252)'':0.00448855,('PDT000463684.1':0.0155303,'PDT000463693.1':0.0231142)'':0.0106703)'':0.0485497)'':0.0177223,(('PDT000251931.1':0.0680934,('PDT000714703.1':0.0642461,('PDT000424782.1':0.0638329,('PDT000571145.1':0.0467624,'PDT000357601.1':0.0560096)'':0.00886859)'':0.0066469)'':0.00123614)'':0.00132633,(('PDT000544963.1':0.060095,'PDT000644961.1':0.06014)'':0.0141732,(('PDT000619572.1':0.0566516,('PDT000651571.1':0.0378518,'PDT000267182.1':0.04832)'':0.00766704)'':0.018315,('PDT000216001.1':0.0622258,('PDT000709935.1':0.0498233,'PDT000017801.2':0.0565717)'':0.00670123)'':0.00721621)'':0.00213544)'':0.00754214)'':0.00854857)'':0.0392679)'':-0.0356274,((('PDT000242862.1':0.00483368,'PDT000242869.1':0.00472427)'':0.075937,('PDT000346069.1':0.0843949,('PDT000425442.1':0.0639277,'PDT000425716.1':0.0706423)'':0.0166341)'':0.00121067)'':0.00275752,((('PDT000157928.2':0.0487893,'PDT000034439.1':0.0396277)'':0.00170167,('PDT000675038.1':0.0387826,('PDT000397720.1':0.0260074,('PDT000242985.1':0.016178,'PDT000252623.1':0.0168204)'':0.0074267)'':0.00754)'':0.00939727)'':0.0315961,(('PDT000251964.1':0.0892982,(('PDT000147460.3':0.0773264,'PDT000571143.1':0.0783156)'':0.00553879,('PDT000517546.1':0.0467937,'PDT000591949.1':0.0429983)'':0.0351422)'':0.00642577)'':-0.0034178,((('PDT000537541.1':0.0711838,'PDT000393133.1':0.0679822)'':0.0108976,('PDT000521015.1':0.0870105,(('PDT000564915.1':0.0486641,'PDT000352493.1':0.0427981)'':0.00715074,('PDT000129906.2':0.0592393,('PDT000196612.2':0.041093,'PDT000502742.1':0.0438945)'':0.00804994)'':0.00339962)'':0.0134957)'':0.000337237)'':0.000613195,(('PDT000654340.1':0.0846229,('PDT000019455.2':0.0626039,('PDT000006506.2':0.0377161,'PDT000395276.1':0.0497594)'':0.010974)'':0.013297)'':0.000951559,('PDT000445108.1':0.0628661,('PDT000518065.1':0.0593139,('PDT000502921.1':0.0389486,'PDT000652465.1':0.0401216)'':0.010676)'':0.00648953)'':0.00456475)'':0.00655828)'':0.00469532)'':0.00248478)'':0.00279933)'':0.00295253)'':0.0135244)'':0.00604993)'':-0.0122378)'':0.00931837)'':0.00302636)'':0.0262432)'':-0.0133029)'':0.00529055)'':0.0150107)'':0.00473272,((('PDT000046597.1':0.0796423,(('PDT000425507.1':0.0403529,'PDT000425562.1':0.0414993)'':0.0178988,('PDT000219685.1':0.0609819,('PDT000547561.1':0.0374005,('PDT000157498.2':0.0404713,'PDT000535198.1':0.0401739)'':0.00508501)'':0.00569608)'':0.00357798)'':0.0120989)'':0.0617609,((('PDT000619611.1':0.0206776,'PDT000619624.1':0.0191572)'':0.0623995,(('PDT000426593.1':0.0317607,'PDT000636301.1':0.0331026)'':0.0454996,(('PDT000706405.1':0.0532581,'PDT000019185.2':0.0441275)'':0.00967649,('PDT000318601.1':0.062027,('PDT000015812.3':0.0546735,'PDT000482866.1':0.0559795)'':0.00448805)'':0.00397361)'':0.0110957)'':0.00570962)'':0.0432566,(('PDT000271356.2':0.0554397,('PDT000675644.1':0.039652,'PDT000016981.1':0.0371658)'':0.00663444)'':0.0849298,((('PDT000352528.1':0.087074,('PDT000444886.1':0.0631803,('PDT000176016.2':0.0236844,'PDT000448224.1':0.024028)'':0.0255395)'':0.0325108)'':0.0220998,(('PDT000129925.1':0.0567642,('PDT000520761.1':0.0373514,'PDT000042741.1':0.0313754)'':0.0198544)'':0.00556569,(('PDT000175873.2':0.0609544,('PDT000425486.1':0.0204238,'PDT000619074.1':0.0242349)'':0.0279182)'':0.00180702,('PDT000212800.1':0.031024,('PDT000217984.2':0.0256885,'PDT000521138.1':0.0175717)'':0.0029478)'':0.0311766)'':0.00300911)'':0.0506276)'':0.000608522,((('PDT000618713.1':0.0832032,(('PDT000615931.1':0.0523221,('PDT000720263.1':0.032273,'PDT000429810.1':0.0268033)'':0.00860665)'':0.0163699,('PDT000709916.1':0.0630245,('PDT000518002.1':0.0535057,('PDT000161669.2':0.0446543,'PDT000675560.1':0.0411626)'':0.00951071)'':0.00938516)'':-0.0047565)'':0.0232693)'':0.000606545,((('PDT000709906.1':0.0562874,'PDT000709962.1':0.0612426)'':0.0169709,('PDT000298082.1':0.0728672,('PDT000471047.1':0.0375593,'PDT000345994.1':0.0396384)'':0.0369949)'':0.00406899)'':0.00768695,(('PDT000645645.2':0.0636347,'PDT000171807.1':0.0523483)'':0.0172124,(('PDT000181742.2':0.0945588,'PDT000225241.1':0.0952302)'':0.000895118,('PDT000081769.1':0.0699485,('PDT000181797.2':0.0742855,('PDT000081753.1':0.0511123,'PDT000483537.1':0.0514547)'':0.01156)'':0.00445601)'':0.00559812)'':0.00397694)'':0.00115304)'':0.00179686)'':0.00445458,(('PDT000583370.1':0.0350667,'PDT000346821.1':0.0462222)'':0.0809965,(('PDT000067314.1':0.0859505,(('PDT000417109.1':0.0963051,'PDT000319004.1':0.0771769)'':0.00511694,('PDT000659698.1':0.0501137,'PDT000180920.1':0.0563483)'':0.0191171)'':0.00381897)'':0.0127903,('PDT000425939.1':0.119043,(('PDT000283181.1':0.0493874,('PDT000240705.1':0.0578036,'PDT000353628.1':0.0509124)'':0.0039076)'':0.0491886,(('PDT000661185.1':0.109218,('PDT000432557.1':0.0950492,(('PDT000425647.1':0.0413586,'PDT000290008.1':0.0520496)'':0.0139682,('PDT000534868.1':0.0532509,'PDT000535123.1':0.0613781)'':0.00726092)'':0.0154015)'':0.00552785)'':-0.000814404,(('PDT000534890.1':0.0764249,'PDT000395122.1':0.0725921)'':0.00344358,('PDT000713223.1':0.0785234,(('PDT000709955.1':0.0525656,('PDT000188144.3':0.0595363,'PDT000241784.1':0.0679497)'':0.00200089)'':0.005422,(('PDT000425655.1':0.058297,('PDT000279220.1':0.0427995,'PDT000395279.1':0.0387055)'':0.0134793)'':0.00144062,(('PDT000632819.1':0.0473013,('PDT000627346.1':0.0394031,'PDT000327855.1':0.0419453)'':0.00486646)'':0.00794412,(('PDT000534938.1':0.0396069,'PDT000621073.1':0.058794)'':0.0156431,('PDT000524051.1':0.00342503,'PDT000524061.1':0.00415224)'':0.0475508)'':0.00333501)'':0.00454448)'':0.00283235)'':0.0137429)'':0.00271441)'':-0.0012261)'':0.0165579)'':0.0120304)'':-0.00899706)'':0.0321025)'':-0.0340125)'':0.0145577)'':0.0273894)'':0.000350746)'':0.0128925)'':-0.0120141,((('PDT000109315.2':0.0517603,'PDT000399615.1':0.0503157)'':0.0518843,(('PDT000226162.2':0.0362807,'PDT000485567.1':0.0375215)'':0.0132199,(('PDT000691367.1':0.0335393,'PDT000004910.2':0.0268992)'':0.00727934,('PDT000644931.1':0.0395538,'PDT000645137.1':0.0338255)'':0.00232736)'':0.00417683)'':0.0385754)'':0.0398411,(((('PDT000661158.1':0.137036,('PDT000181846.2':0.106328,('PDT000411308.1':0.0818931,('PDT000518046.1':0.0546996,('PDT000621020.1':0.0506515,'PDT000124659.1':0.043396)'':0.00805664)'':0.00737376)'':0.0139959)'':0.0426607)'':-0.0215703,(('PDT000463628.1':0.0892573,('PDT000120883.2':0.074698,'PDT000205437.1':0.06512)'':0.0116342)'':0.00548398,(('PDT000523060.1':0.0357914,'PDT000645106.1':0.0418258)'':0.0492963,(('PDT000616159.1':0.0780327,'PDT000042707.1':0.0762613)'':0.0163555,('PDT000032203.1':0.0846192,'PDT000249930.1':0.0772218)'':0.0108752)'':0.00373618)'':0.00167093)'':0.00696703)'':0.0221559,(('PDT000129899.2':0.0912957,('PDT000712984.1':0.0984957,('PDT000343939.1':0.0752972,('PDT000129880.2':0.0562249,('PDT000086971.1':0.0413238,('PDT000645014.1':0.0201773,'PDT000682517.1':0.0188183)'':0.01045)'':0.0290822)'':0.0190441)'':0.00692399)'':0.00741175)'':0.0271857,('PDT000649800.1':0.162752,((('PDT000309190.2':0.0995107,('PDT000250073.1':0.0617954,'PDT000252571.1':0.0700266)'':0.0215048)'':0.00479662,(('PDT000455206.1':0.0812993,'PDT000504754.1':0.0828547)'':0.00977827,(('PDT000427691.1':0.0390384,'PDT000652339.1':0.0397343)'':0.0422994,('PDT000060122.2':0.0819472,('PDT000208411.2':0.0557976,'PDT000242997.1':0.0617544)'':0.0256818)'':0.00441184)'':0.00240088)'':0.00307033)'':0.0116215,(('PDT000332122.1':0.0853832,(('PDT000109478.2':0.0401448,'PDT000427702.1':0.049127)'':0.00537388,('PDT000032390.1':0.00402043,('PDT000032386.1':0.00305277,'PDT000032389.1':0.00283663)'':0.000411737)'':0.0457836)'':0.0263821)'':0.00890193,(('PDT000443032.1':0.0706819,'PDT000445079.1':0.0675991)'':0.0223264,('PDT000242914.1':0.100083,(('PDT000092542.1':0.0527696,'PDT000251580.1':0.0424455)'':0.0039542,('PDT000485248.1':0.0340998,('PDT000566048.1':0.0271929,'PDT000251753.1':0.0218923)'':0.00883248)'':0.0101534)'':0.020302)'':0.0072006)'':0.0027658)'':0.0121528)'':0.034287)'':-0.0194144)'':-0.00317389)'':0.00545909,(('PDT000181671.3':0.126591,('PDT000321142.2':0.0867565,(('PDT000637310.1':0.0666167,'PDT000226472.3':0.0652643)'':0.00940914,('PDT000719113.1':0.0533309,('PDT000638922.1':0.0446405,'PDT000689789.1':0.047552)'':0.00847873)'':0.0214439)'':0.0093847)'':0.0248751)'':0.0440576,(((('PDT000601030.1':0.0560289,'PDT000714230.1':0.0601801)'':0.0450205,('PDT000437966.1':0.0817049,('PDT000713466.1':0.0389899,('PDT000275526.2':0.0320113,'PDT000376087.1':0.0289715)'':0.0156534)'':0.0380193)'':0.0134304)'':0.00725693,(('PDT000601910.1':0.0829688,'PDT000687910.1':0.0541862)'':0.0346209,(('PDT000661675.1':0.0462617,('PDT000206107.2':0.0397634,'PDT000497376.1':0.0266671)'':0.0256789)'':0.0283408,(('PDT000380291.1':0.0684568,(('PDT000095683.2':0.0541759,'PDT000548944.1':0.0539941)'':0.0024032,('PDT000397779.1':0.0187527,'PDT000397915.1':0.020519)'':0.042284)'':0.0239472)'':0.0111523,('PDT000635553.2':0.101821,(('PDT000127571.1':0.0997268,('PDT000667710.2':0.0507594,'PDT000248762.1':0.0373666)'':0.0312127)'':4.84794e-06,('PDT000568774.1':0.0915249,('PDT000355060.1':0.0693265,('PDT000258742.2':0.0596592,('PDT000145159.2':0.049503,'PDT000306431.2':0.0414672)'':0.0134432)'':0.00798421)'':0.0134691)'':-0.00146178)'':0.0158063)'':-0.00627821)'':0.00213384)'':0.0135178)'':0.00317908)'':0.00285247,((('PDT000181656.2':0.0430795,('PDT000626695.1':0.0506915,'PDT000633512.1':0.0478625)'':0.0023331)'':0.0505495,(('PDT000583422.1':0.0581071,'PDT000354213.1':0.0546389)'':0.00919405,('PDT000675307.1':0.0530354,('PDT000411021.1':0.0391464,('PDT000484736.1':0.0337275,('PDT000535451.1':0.0182431,'PDT000535508.1':0.0171124)'':0.0165897)'':0.00229795)'':0.00753425)'':0.0263121)'':0.0154056)'':0.0132164,((('PDT000174363.1':0.080965,('PDT000141216.2':0.0725274,'PDT000174333.1':0.0729676)'':0.012378)'':0.0259066,(('PDT000394835.1':0.0580218,'PDT000565837.1':0.0666102)'':0.0220346,('PDT000466988.1':0.0594751,('PDT000714660.1':0.0352943,('PDT000326050.1':0.0448713,'PDT000114753.2':0.0315831)'':0.00844399)'':0.018931)'':0.0264138)'':0.0115679)'':0.00746052,((('PDT000034424.1':0.0875963,'PDT000251951.1':0.0946827)'':0.013618,('PDT000217973.2':0.0572711,('PDT000566108.1':0.0609077,('PDT000397769.1':0.0168423,'PDT000399617.1':0.0193617)'':0.0432563)'':0.00730384)'':0.0337329)'':-0.000653117,(('PDT000714688.1':0.0374282,('PDT000722594.1':0.0310757,'PDT000251413.1':0.0306335)'':0.00399142)'':0.056717,(('PDT000471031.1':0.0874692,(('PDT000081750.1':0.0474077,('PDT000208321.2':0.0284717,'PDT000619625.1':0.0394991)'':0.0113019)'':0.0221512,('PDT000573834.2':0.0756864,('PDT000346559.1':0.0364976,('PDT000582303.1':0.023752,'PDT000045628.1':0.0189369)'':0.0141162)'':0.0350886)'':-0.00304109)'':0.0139278)'':-0.00432596,('PDT000444005.1':0.0829853,((('PDT000435585.1':0.042669,'PDT000414195.1':0.0402236)'':0.00304623,('PDT000720281.1':0.0399893,'PDT000364518.1':0.0398715)'':0.0100741)'':0.0169059,('PDT000632334.1':0.0526122,('PDT000178604.2':0.0426574,('PDT000124672.1':0.0338338,('PDT000285409.2':0.018047,'PDT000661078.1':0.0198258)'':0.0168578)'':0.00910324)'':0.0103808)'':0.0137181)'':0.017649)'':-0.000413945)'':0.0106449)'':0.00558345)'':0.00471571)'':0.000628878)'':0.00228204)'':0.0200671)'':0.00135994)'':0.0118242)'':-0.0110439)'':0.0270668)'':0.0064356)'':0.039858)'':-0.0251254)'':0.0132588,(((((('PDT000652949.1':0.142891,('PDT000687454.1':0.0704054,('PDT000329058.1':0.0271534,'PDT000212795.1':0.0307275)'':0.0587972)'':0.0653538)'':0.0379285,(('PDT000651043.1':0.0502046,('PDT000027742.2':0.0325391,'PDT000248455.1':0.0331791)'':0.0293623)'':0.0582159,(('PDT000467015.1':0.0855781,('PDT000354233.1':0.0659146,'PDT000354274.1':0.0947034)'':0.000158927)'':0.0173021,('PDT000251687.1':0.0872623,(('PDT000128805.2':0.0632537,('PDT000672147.1':0.0592504,'PDT000346018.1':0.0648526)'':0.000663788)'':0.0127131,(('PDT000652826.1':0.0679197,('PDT000645551.2':0.0451576,'PDT000346042.1':0.047819)'':0.0112395)'':0.00950272,('PDT000652950.1':0.0684765,('PDT000652986.1':0.0368664,'PDT000652993.1':0.0345276)'':0.031335)'':0.00416212)'':0.00843233)'':0.0171633)'':0.00594597)'':0.0228856)'':0.0435241)'':0.00144261,((('PDT000712623.1':0.066743,('PDT000088288.2':0.0509388,'PDT000563558.1':0.0508922)'':0.0165735)'':0.0770235,('PDT000661094.1':0.101599,('PDT000346601.1':0.0743645,('PDT000215761.2':0.0429336,'PDT000425547.1':0.0414546)'':0.0141479)'':0.0400403)'':0.0414337)'':0.00137874,(('PDT000642305.1':0.0363227,('PDT000393095.1':0.0311161,'PDT000396515.1':0.0280121)'':0.0095466)'':0.103435,(('PDT000425857.1':0.0832262,('PDT000346817.1':0.0721174,('PDT000150946.2':0.0575113,'PDT000717187.1':0.0642647)'':0.00805708)'':0.010597)'':0.00511022,('PDT000228114.1':0.0831799,(('PDT000395382.1':0.0797709,('PDT000539294.1':0.0374614,('PDT000382209.1':0.0220357,('PDT000150237.2':0.0248212,'PDT000325579.1':0.0235706)'':0.000658422)'':0.00879581)'':0.0242663)'':0.0030458,(('PDT000188121.2':0.033193,'PDT000403080.1':0.0303791)'':0.0266924,('PDT000357732.1':0.03767,(('PDT000495042.1':0.0454891,'PDT000249881.1':0.0390846)'':0.00334567,('PDT000069146.2':0.0284438,('PDT000566176.1':0.0312669,'PDT000644953.1':0.0243813)'':0.00500291)'':0.00984178)'':0.0134487)'':0.000941087)'':0.00539669)'':0.0218963)'':0.00235709)'':0.0647088)'':-0.00137739)'':0.0157578)'':0.00096984,((('PDT000444193.1':0.114135,('PDT000524026.1':0.0352478,'PDT000083484.1':0.0495046)'':0.0651554)'':0.0562668,('PDT000630427.1':0.125127,(('PDT000346680.1':0.0998407,('PDT000372309.1':0.103481,'PDT000242888.1':0.102123)'':0.00309825)'':0.0075676,(('PDT000426317.1':0.111801,('PDT000294655.2':0.0317806,'PDT000541623.1':0.0436027)'':0.0584216)'':0.0102589,('PDT000346605.1':0.0897496,('PDT000240707.1':0.0686726,'PDT000346671.1':0.0419204)'':0.0391114)'':0.0258618)'':0.0062757)'':0.0215399)'':0.0299815)'':0.0119399,(((('PDT000077061.2':0.0529621,'PDT000395048.1':0.0538319)'':0.0919812,('PDT000661101.1':0.140512,((('PDT000632008.1':0.063191,('PDT000132221.2':0.0656168,'PDT000395146.1':0.0551142)'':0.011844)'':0.0104362,(('PDT000258736.2':0.0594593,'PDT000467011.1':0.0603067)'':0.00593026,('PDT000346936.1':0.0772845,(('PDT000403171.1':0.0213397,'PDT000645135.1':0.0198872)'':0.0203236,(('PDT000503830.1':0.024891,'PDT000346938.1':0.0239181)'':0.0201522,('PDT000346544.1':0.0365371,'PDT000346563.1':0.024341)'':0.0207871)'':0.00521182)'':0.0248608)'':-0.000736728)'':0.00679258)'':0.00920656,(('PDT000661192.1':0.0824408,'PDT000661200.1':0.0901532)'':0.0228114,(('PDT000128852.2':0.0835625,('PDT000661317.1':0.0660591,('PDT000497342.1':0.0497954,'PDT000550342.1':0.0543536)'':0.0142414)'':0.00785051)'':-0.00129885,('PDT000221033.1':0.0691337,('PDT000221039.1':0.0615392,('PDT000394409.1':0.0525255,('PDT000346568.1':0.0461489,('PDT000221046.1':0.0304591,('PDT000221041.1':0.0155139,'PDT000221062.1':0.0123742)'':0.0164327)'':0.0137314)'':0.004454)'':0.00675047)'':0.00340237)'':0.00371686)'':0.0208912)'':-0.00838833)'':0.043444)'':-0.00160307)'':-0.0307195,(('PDT000289890.1':0.0752307,'PDT000435647.1':0.0837653)'':0.0295787,(('PDT000334833.1':0.0848133,('PDT000426265.1':0.0907285,('PDT000669011.1':0.0763284,('PDT000412508.1':0.052489,'PDT000448194.1':0.048587)'':0.0223856)'':0.00411327)'':0.00387225)'':0.00835144,('PDT000237135.1':0.104488,(('PDT000535290.1':0.0621344,'PDT000620185.1':0.0677946)'':0.0131612,(('PDT000395123.1':0.0620061,'PDT000412509.1':0.0625799)'':0.00662595,((('PDT000346812.1':0.0269626,'PDT000346823.1':0.0290811)'':0.0369686,('PDT000519809.1':0.0655687,('PDT000188376.1':0.0474597,'PDT000205444.1':0.037446)'':0.0178384)'':0.00577131)'':0.00202178,(('PDT000616406.1':0.0811442,('PDT000289912.1':0.0631917,('PDT000206256.3':0.0301131,'PDT000483207.1':0.0278452)'':0.0155197)'':0.01599)'':1.51262e-05,(('PDT000206129.2':0.0589542,'PDT000502539.1':0.0515268)'':0.0162784,('PDT000346816.1':0.0640707,('PDT000411269.1':0.0689317,(('PDT000277719.2':0.0409329,'PDT000221013.1':0.0345002)'':0.0121159,('PDT000346789.1':0.051149,'PDT000346809.1':0.040891)'':0.00322036)'':0.00639814)'':0.00514513)'':-6.24302e-05)'':0.00331096)'':0.00124949)'':0.00455452)'':0.000863958)'':0.0187216)'':-0.012958)'':0.0131464)'':0.00409245)'':0.0640024,(((('PDT000346016.1':0.0860401,('PDT000251445.1':0.0565649,('PDT000719057.1':0.033742,'PDT000228187.1':0.0368131)'':0.021253)'':0.0314685)'':0.0605664,('PDT000398043.1':0.121675,((('PDT000284893.2':0.0773929,'PDT000346442.1':0.0705891)'':0.0113526,('PDT000235919.5':0.0845674,('PDT000203406.2':0.0581227,'PDT000204890.2':0.0482253)'':0.0326996)'':0.00622437)'':0.0108131,('PDT000627351.1':0.0810789,(('PDT000110390.2':0.00192437,'PDT000357692.1':0.00287371)'':0.0545569,('PDT000250059.1':0.0258447,'PDT000304311.1':0.0241895)'':0.0273355)'':0.0276734)'':-0.00586657)'':0.0296246)'':0.0176986)'':0.0046287,((('PDT000213199.2':0.0235283,('PDT000422538.1':0.0128643,'PDT000428909.1':0.0165855)'':0.0154137)'':0.00924664,('PDT000015720.2':0.0465332,('PDT000615218.1':0.0228024,'PDT000250160.1':0.0191132)'':0.0131871)'':0.00483149)'':0.0346921,('PDT000129891.2':0.0359397,('PDT000015775.3':0.0367233,('PDT000015707.3':0.0196977,('PDT000015733.3':0.0164381,('PDT000015736.3':0.0108417,('PDT000015732.3':0.0139172,('PDT000014105.2':0.00971558,(('PDT000015741.2':0.00813018,'PDT000015743.2':0.00951452)'':0.000765574,('PDT000015744.3':0.0100826,('PDT000015739.3':0.00838733,('PDT000015869.3':0.00394554,'PDT000015731.3':0.00620776)'':0.00123502)'':0.000861103)'':1.01214e-05)'':0.00156318)'':0.00268432)'':0.00101273)'':0.0017252)'':0.00638792)'':0.0129843)'':0.00248498)'':0.0372727)'':0.0742825)'':0.0223975,((('PDT000479795.1':0.0748641,'PDT000566685.1':0.0868909)'':0.00588696,('PDT000656971.1':0.0724894,('PDT000644971.1':0.040369,'PDT000346127.1':0.0309484)'':0.0323369)'':0.00154049)'':0.0909463,((('PDT000218344.1':0.0965519,'PDT000228375.1':0.10085)'':0.0257621,('PDT000396732.1':0.125498,(('PDT000537889.1':0.0811906,('PDT000485436.1':0.0721557,'PDT000544450.1':0.0702573)'':0.0132719)'':0.0352248,('PDT000706399.1':0.0936152,('PDT000538620.1':0.0938002,('PDT000307438.2':0.0849409,(('PDT000360094.1':0.0399231,'PDT000403162.1':0.0419793)'':0.027922,('PDT000346449.1':0.0778505,('PDT000333678.1':0.062054,'PDT000129098.2':0.061546)'':0.00630152)'':0.00152042)'':0.00706385)'':0.00991841)'':0.00207861)'':0.0318337)'':0.00504533)'':0.00125429)'':-0.00214965,((('PDT000088286.2':0.0244256,('PDT000088297.2':0.0163972,'PDT000389639.1':0.0173007)'':0.0111695)'':0.0829971,('PDT000556935.1':0.0993364,('PDT000346450.1':0.0742694,('PDT000181567.3':0.04189,'PDT000360417.2':0.0370319)'':0.0256131)'':0.0149767)'':0.0279826)'':0.00573405,('PDT000285325.1':0.104816,(('PDT000399587.1':0.04432,('PDT000441057.2':0.0170753,'PDT000488259.1':0.0201189)'':0.019959)'':-2.52568e-05,('PDT000435582.2':0.0350407,('PDT000518550.1':0.0321995,('PDT000303101.3':0.0234182,'PDT000671432.1':0.0199361)'':0.00762007)'':0.00385857)'':0.0165085)'':0.0558584)'':0.026574)'':0.0124581)'':0.0369844)'':0.00370404)'':0.00191693)'':0.00694894)'':-0.00700595)'':0.00665865,(((('PDT000689791.1':0.0736377,('PDT000389644.1':0.058309,'PDT000389653.1':0.063626)'':0.00698781)'':0.0572349,(('PDT000288647.2':0.068113,'PDT000565848.1':0.07724)'':0.0396258,('PDT000466962.1':0.08532,'PDT000627431.1':0.083521)'':0.0245432)'':0.0133518)'':0.0027548,('PDT000429809.1':0.123875,('PDT000298041.1':0.142539,('PDT000168870.1':0.113609,(('PDT000298012.1':0.0766557,'PDT000396763.1':0.0934153)'':0.0132345,('PDT000433427.1':0.0896713,('PDT000298040.1':0.105548,'PDT000346814.1':0.0841254)'':0.00518821)'':0.0139332)'':0.0103121)'':0.00564301)'':0.00539496)'':0.0264872)'':0.0064215,(((('PDT000238197.2':0.116255,('PDT000346820.1':0.0913187,(('PDT000346797.1':0.0597745,('PDT000435623.1':0.0575087,('PDT000465795.1':0.053579,'PDT000618978.1':0.07201)'':0.00303727)'':0.000426781)'':0.0113001,('PDT000101295.1':0.0686513,('PDT000538610.1':0.0558193,('PDT000618736.1':0.0571191,('PDT000291148.2':0.0461775,'PDT000632909.1':0.0478168)'':0.00684479)'':0.00639942)'':0.00903462)'':0.00675865)'':0.0234041)'':0.0224599)'':0.00297896,(('PDT000252493.1':0.0815244,('PDT000008593.2':0.0649952,'PDT000485989.1':0.0570118)'':0.0165016)'':0.0168828,(('PDT000555607.1':0.0206597,'PDT000294388.1':0.0292714)'':0.0656681,('PDT000656958.1':0.0772533,(('PDT000661394.1':0.0826158,'PDT000346076.1':0.0778622)'':0.00383157,(('PDT000259562.2':0.0614335,'PDT000524495.1':0.0447395)'':0.0270961,(('PDT000397905.1':0.0435842,'PDT000566006.1':0.0527385)'':0.00921259,('PDT000643448.1':0.0472341,'PDT000213686.1':0.0545119)'':0.00367091)'':0.00941581)'':0.00997939)'':0.00851078)'':0.0046387)'':0.0158669)'':0.0160726)'':0.00281711,((('PDT000280838.2':0.110037,'PDT000017808.2':0.107957)'':0.00594786,((('PDT000346932.1':0.0824679,('PDT000145185.2':0.0528043,'PDT000668934.1':0.0505237)'':0.0136296)'':0.00348463,('PDT000294636.2':0.0477705,('PDT000271389.2':0.0480191,('PDT000644899.1':0.0306666,('PDT000394871.1':0.0215119,('PDT000425953.1':0.0275819,'PDT000713450.1':0.0208514)'':0.00238808)'':0.00873018)'':0.00894374)'':0.00327706)'':0.0329304)'':0.00952879,(('PDT000135976.2':0.0781698,('PDT000357615.1':0.0636613,'PDT000357691.1':0.0579657)'':0.00189671)'':0.0101074,('PDT000167714.2':0.077286,(('PDT000661517.1':0.0417028,'PDT000661605.1':0.0440335)'':0.021818,(('PDT000128875.2':0.0437768,'PDT000520788.1':0.0500657)'':0.0143361,('PDT000181808.2':0.0747629,(('PDT000311247.2':0.0423859,('PDT000235745.2':0.0279897,'PDT000408770.1':0.0283208)'':0.0107058)'':0.018816,('PDT000661773.1':0.0601474,('PDT000521094.1':0.0471618,('PDT000161642.2':0.0309137,('PDT000584538.1':0.0381552,('PDT000652341.1':0.0344717,('PDT000281479.2':0.035398,'PDT000294700.2':0.0306692)'':0.00581931)'':0.000463871)'':0.000577394)'':0.00260633)'':0.0138729)'':-0.00119841)'':0.00827706)'':-0.00186733)'':0.00398825)'':0.00283826)'':0.0217845)'':0.0026929)'':0.0206234)'':-0.00883947,((('PDT000397848.1':0.0667563,('PDT000250161.1':0.0553205,('PDT000397858.1':0.0293896,'PDT000466960.1':0.0380041)'':0.0189538)'':0.0108604)'':0.0254126,(('PDT000305506.2':0.0561171,'PDT000396604.1':0.0712409)'':0.0106743,('PDT000005943.2':0.0896649,('PDT000238201.2':0.0660917,('PDT000619971.1':0.0425192,('PDT000412037.1':0.0375191,'PDT000517936.1':0.0337782)'':0.0074582)'':0.0277706)'':0.00850393)'':-0.00170672)'':0.00945619)'':-0.000268882,(('PDT000188935.1':0.0668984,'PDT000205433.1':0.0589076)'':0.0597077,(('PDT000555543.1':0.0753548,('PDT000460401.1':0.0738827,('PDT000249869.1':0.0470256,'PDT000251776.1':0.0529934)'':0.0115908)'':0.000679728)'':0.0233255,('PDT000109378.2':0.0757801,(('PDT000497234.1':0.0490182,('PDT000252521.1':0.0333689,'PDT000019235.2':0.0388151)'':0.00592582)'':0.0119745,(('PDT000060282.2':0.0516142,'PDT000143981.3':0.0536138)'':0.00913683,('PDT000586241.1':0.0676283,('PDT000017813.2':0.0477178,(('PDT000619680.1':0.0390581,('PDT000397638.1':0.0331847,'PDT000482854.1':0.0406552)'':0.00242539)'':0.0029447,('PDT000661662.1':0.0540127,(('PDT000497332.1':0.0252414,('PDT000394889.1':0.0214651,'PDT000397970.1':0.0170536)'':0.00357892)'':0.00803355,('PDT000494561.1':0.0337831,('PDT000397559.1':0.0306167,'PDT000422115.1':0.0236583)'':0.00276451)'':0.0021196)'':0.01421)'':-0.00586897)'':0.0124639)'':0.00240502)'':-0.0035974)'':0.000668716)'':0.0276859)'':0.000244451)'':0.0151245)'':-0.00729991)'':0.00859561)'':0.0185259)'':0.00272916,(((('PDT000482808.1':0.0721682,'PDT000482815.1':0.0826808)'':0.022762,(('PDT000235935.5':0.051726,'PDT000252715.1':0.045271)'':0.0473532,(('PDT000426561.1':0.083011,'PDT000652875.1':0.07318)'':0.00924192,('PDT000482813.1':0.0824051,('PDT000298011.1':0.051673,'PDT000394978.1':0.044825)'':0.0197894)'':0.00172711)'':0.00400208)'':0.00168294)'':0.0431732,(('PDT000652827.1':0.0901169,'PDT000656942.1':0.0795261)'':0.0707952,(('PDT000306636.2':0.112344,(('PDT000661385.1':0.0639848,('PDT000230514.2':0.0206324,'PDT000644929.1':0.0230979)'':0.0331376)'':0.0477295,(('PDT000397581.1':0.0604213,'PDT000645005.1':0.0640697)'':0.0127294,('PDT000559249.1':0.0315637,'PDT000639521.1':0.035435)'':0.0457337)'':0.025581)'':0.00741818)'':0.0037569,('PDT000652869.1':0.119226,(('PDT000426302.1':0.0715631,('PDT000652975.1':0.0550013,'PDT000019612.1':0.0614427)'':0.0277164)'':0.016753,(('PDT000394813.1':0.0571519,'PDT000228073.1':0.0635261)'':0.0209159,('PDT000653018.1':0.0552306,('PDT000415661.1':0.0401558,'PDT000497378.1':0.0411567)'':0.0210307)'':0.0201624)'':0.0337694)'':0.0267805)'':-0.00315057)'':0.0229119)'':-0.00863724)'':0.0102336,(('PDT000651908.1':0.140997,(('PDT000550347.1':0.0858571,'PDT000482825.1':0.0923729)'':0.00867669,('PDT000521052.1':0.0478936,'PDT000571140.1':0.0327595)'':0.0481935)'':0.0379709)'':-0.00990391,(('PDT000006418.2':0.0891829,('PDT000718830.1':0.0756973,(('PDT000304949.2':0.0488335,'PDT000027745.2':0.050096)'':0.0249673,(('PDT000171431.1':0.0570909,('PDT000468923.1':0.0303095,'PDT000485944.1':0.0282385)'':0.0114473)'':0.0238596,('PDT000520925.1':0.0687914,('PDT000188374.1':0.0430028,('PDT000534992.1':0.0189253,'PDT000468886.1':0.0216272)'':0.0290162)'':0.00676839)'':0.00658351)'':0.00507267)'':0.00865378)'':0.0214785)'':0.0112099,((('PDT000417153.1':0.101442,('PDT000656948.1':0.0427106,'PDT000645523.2':0.0331137)'':0.0594317)'':0.0164118,('PDT000668387.2':0.0969646,('PDT000668334.2':0.0645337,('PDT000645710.2':0.0466148,('PDT000242864.1':0.0403919,'PDT000242940.1':0.0417077)'':0.00705849)'':0.0174559)'':0.0452985)'':0.00730477)'':0.0134485,((('PDT000638924.1':0.135871,'PDT000171799.1':0.0979202)'':0.00783201,(('PDT000353630.1':0.0800312,('PDT000308360.1':0.0442791,'PDT000308361.1':0.0446738)'':0.0437243)'':0.0112428,(('PDT000517948.1':0.0838641,('PDT000399664.1':0.0608904,'PDT000346666.1':0.0547136)'':0.0233139)'':0.00628549,('PDT000250254.1':0.0922161,('PDT000502214.1':0.0718657,('PDT000565993.1':0.0591659,'PDT000346670.1':0.0507121)'':0.0168113)'':0.00477511)'':-0.000640506)'':0.0113132)'':0.0222758)'':-0.0127818,((('PDT000651227.1':0.0440316,(('PDT000149343.2':0.0126236,'PDT000286924.2':0.0111388)'':0.00743531,('PDT000095703.2':0.0265729,('PDT000425933.1':0.015888,('PDT000420669.1':0.00968426,('PDT000403153.1':0.00524512,'PDT000500481.1':0.00541328)'':0.00240009)'':0.00463252)'':0.00615456)'':0.000302837)'':0.00956905)'':0.0561452,(('PDT000565135.2':0.0936183,('PDT000311240.2':0.0186868,'PDT000394890.1':0.0263484)'':0.0581971)'':0.00999863,(('PDT000535372.1':0.078531,('PDT000354880.1':0.064353,('PDT000426067.1':0.0686777,('PDT000127570.1':0.0560453,'PDT000253669.1':0.0444687)'':0.0107133)'':0.00562027)'':0.0159024)'':0.00910231,('PDT000362446.1':0.0833859,('PDT000618845.2':0.0781085,('PDT000535452.1':0.0398238,('PDT000471017.1':0.0280684,'PDT000534935.1':0.0334551)'':0.0138759)'':0.0179861)'':0.00812733)'':0.0105658)'':0.00498343)'':-9.89352e-05)'':0.0102189,(('PDT000668405.2':0.0486546,('PDT000637317.1':0.0293375,'PDT000637334.1':0.0262181)'':0.0201103)'':0.0567165,((('PDT000599885.1':0.0894446,('PDT000633678.1':0.0824739,'PDT000346826.1':0.0895701)'':0.00995489)'':0.0167267,(('PDT000354278.1':0.0847135,('PDT000176021.2':0.0732768,('PDT000534898.1':0.0660661,'PDT000538581.1':0.0674479)'':0.00781466)'':0.00638377)'':0.00529992,(('PDT000212407.2':0.0593841,('PDT000429703.1':0.040738,('PDT000032360.1':0.0107447,'PDT000034403.1':0.0118743)'':0.03382)'':0.0118672)'':0.0167549,('PDT000625403.1':0.0734228,(('PDT000326153.1':0.0385941,'PDT000480397.1':0.0303007)'':0.00597828,('PDT000557291.1':0.0264432,'PDT000502219.1':0.0273764)'':0.00706944)'':0.0246759)'':-0.0033818)'':0.0261508)'':0.0129361)'':-0.00653995,(('PDT000251914.1':0.052506,'PDT000357640.1':0.056217)'':0.054954,((('PDT000313234.2':0.0502974,'PDT000360395.1':0.0495123)'':0.0398113,('PDT000143596.1':0.0553692,('PDT000652991.1':0.0397977,'PDT000645590.2':0.036023)'':0.0219955)'':0.0379284)'':0.00549037,(('PDT000510185.1':0.0870576,'PDT000124619.1':0.0908984)'':0.010978,((('PDT000720458.1':0.0163984,'PDT000720471.1':0.0167774)'':0.0752255,(('PDT000129911.2':0.0393535,'PDT000536588.1':0.0504464)'':0.0296266,('PDT000689927.1':0.0675679,'PDT000718818.1':0.0726891)'':0.00719492)'':0.00622574)'':0.00223533,('PDT000634092.1':0.0731062,('PDT000397826.1':0.0686245,('PDT000083974.2':0.0501941,('PDT000137158.1':0.0416712,('PDT000096253.1':0.031501,(('PDT000277853.1':0.011758,'PDT000480308.1':0.0136272)'':0.001438,('PDT000343334.1':0.0084702,('PDT000424789.1':0.00354677,('PDT000004919.2':0.00315969,('PDT000294398.1':0.00327288,('PDT000286102.1':0.00163534,('PDT000286100.1':0.00120008,('PDT000286101.1':0.000904809,('PDT000286103.1':0.000748663,('PDT000286104.1':0.000468166,'PDT000286107.1':0.000531434)'':0.000250777)'':0.000110679)'':0.000201183)'':8.36933e-05)'':0.00157116)'':0.00126077)'':0.00200105)'':0.00189548)'':0.00039755)'':0.0216568)'':0.0118143)'':0.00255514)'':0.00945976)'':0.00476071)'':0.0181835)'':0.00787668)'':-0.000515845)'':0.0175008)'':-0.015182)'':0.0034031)'':-0.00305244)'':0.00564959)'':0.00350844)'':0.000132701)'':0.00809762)'':0.0049818)'':-0.00282789)'':0.00789212)'':0.0455187)'':0.0123477,((((((('PDT000283896.1':0.0899445,('PDT000095777.2':0.059165,'PDT000300815.4':0.051046)'':0.0315595)'':0.064693,(('PDT000186200.2':0.0911439,'PDT000242960.1':0.0842751)'':0.0260326,(('PDT000397656.1':0.0697545,'PDT000687817.1':0.0811485)'':0.0170186,(('PDT000355052.1':0.0802209,'PDT000602923.1':0.0898551)'':0.00507823,('PDT000228102.1':0.0698845,('PDT000345968.1':0.0689076,(('PDT000228100.1':0.0393286,'PDT000228286.1':0.0480463)'':0.0089926,('PDT000395194.1':0.0461118,('PDT000237137.1':0.0255699,'PDT000346642.1':0.0158685)'':0.0266944)'':0.00683034)'':0.00805475)'':0.00767991)'':0.00111621)'':0.00884981)'':0.00315533)'':0.0465637)'':0.00719395,(('PDT000251752.1':0.125942,('PDT000326145.1':0.0559393,(('PDT000378601.1':0.0322946,'PDT000161538.3':0.0293833)'':0.017955,('PDT000320939.1':0.0536914,('PDT000136766.2':0.0404166,('PDT000034387.1':0.014562,('PDT000032208.1':0.00505668,'PDT000032400.1':0.0038167)'':0.0118167)'':0.0233954)'':0.00916971)'':-0.0029129)'':0.0153609)'':0.0609257)'':-0.00739861,(('PDT000524115.1':0.0753579,('PDT000306642.2':0.0894119,'PDT000651543.1':0.0786761)'':0.00788756)'':0.0108186,('PDT000547552.1':0.105345,('PDT000242896.1':0.0733683,(('PDT000621055.1':0.0725752,'PDT000250185.1':0.0624678)'':0.00416935,('PDT000556998.1':0.0662943,('PDT000191851.2':0.0544658,'PDT000251391.1':0.0525852)'':0.0107012)'':0.000417597)'':0.00234147)'':0.024349)'':-0.00648524)'':0.0158793)'':0.0357794)'':0.00398981,((('PDT000289910.1':0.118418,'PDT000346573.1':0.135191)'':0.0151988,('PDT000534056.1':0.0835811,('PDT000168853.1':0.084953,('PDT000616497.1':0.0564154,'PDT000620996.1':0.0642856)'':0.0151135)'':0.00703195)'':0.0428212)'':0.032647,(((('PDT000425787.1':0.0572791,'PDT000485253.1':0.0516549)'':0.0639951,('PDT000109329.2':0.108681,(('PDT000200980.2':0.0566006,'PDT000620989.1':0.0873434)'':0.0370144,('PDT000488321.1':0.0638767,('PDT000592138.1':0.0451106,'PDT000600993.1':0.0447388)'':0.0173146)'':0.0211626)'':0.015845)'':-0.00240795)'':0.0058747,('PDT000663936.1':0.138413,(('PDT000558478.2':0.0530433,'PDT000346096.1':0.0505657)'':0.0223182,(('PDT000689876.1':0.0282681,'PDT000718327.1':0.0239926)'':0.00937944,('PDT000227991.1':0.0468667,('PDT000467021.1':0.0361765,'PDT000718858.1':0.0323605)'':0.0107958)'':0.0041432)'':0.0228752)'':0.0595818)'':-0.00699713)'':0.0112068,('PDT000242881.1':0.126527,(('PDT000474438.1':0.0929538,('PDT000621027.1':0.0520041,'PDT000168787.1':0.0579609)'':0.0521657)'':0.00583016,('PDT000621017.1':0.117329,('PDT000032187.1':0.0960218,(('PDT000577103.1':0.0711216,('PDT000642581.1':0.0703308,'PDT000718864.1':0.0588482)'':0.00923589)'':0.00479495,('PDT000167706.2':0.0904088,(('PDT000337003.1':0.0747052,'PDT000660633.1':0.0713668)'':0.00135859,(('PDT000228123.1':0.0661104,('PDT000333781.1':0.0533452,'PDT000367714.1':0.0524988)'':0.00216464)'':0.00478128,('PDT000562857.1':0.0671978,('PDT000399561.1':0.0621682,('PDT000719165.1':0.0492496,('PDT000333777.1':0.0301736,'PDT000386718.1':0.0283744)'':0.0252737)'':0.00759349)'':0.00624248)'':0.000202213)'':0.00427503)'':0.00801888)'':-0.00370984)'':0.0182162)'':0.00391564)'':-0.00470205)'':0.022818)'':0.0174483)'':0.0144986)'':0.00165009)'':0.00506698,(((('PDT000417180.1':0.0861947,'PDT000566020.1':0.0652363)'':0.00291115,('PDT000128828.2':0.0562562,('PDT000345932.1':0.0530941,('PDT000566014.1':0.0488011,'PDT000566154.1':0.0309462)'':0.00886842)'':0.00622265)'':0.0231269)'':0.0663046,(((('PDT000502569.1':0.0438564,('PDT000150894.1':0.03754,('PDT000107691.1':0.0273364,'PDT000502561.1':0.0309569)'':0.00657006)'':0.00497177)'':0.0167986,('PDT000346082.1':0.0268088,('PDT000145163.2':0.0262213,('PDT000553193.1':0.0191951,'PDT000150887.1':0.0196188)'':0.011289)'':0.00416176)'':0.0293669)'':0.0371915,((('PDT000276022.1':0.0747881,'PDT000346654.1':0.0814879)'':0.00599354,('PDT000115465.3':0.0642612,('PDT000262771.1':0.0484061,('PDT000565893.1':0.0234959,'PDT000565981.1':0.0320542)'':0.0274053)'':0.00989586)'':0.0127386)'':0.00539211,(('PDT000253144.2':0.0659311,('PDT000083520.1':0.0605336,('PDT000083474.1':0.042612,('PDT000082640.1':0.00992597,'PDT000083579.1':0.00768593)'':0.0336496)'':0.0226266)'':0.00968967)'':0.0142264,(('PDT000426611.1':0.0354407,('PDT000619752.1':0.0240458,'PDT000719234.1':0.0151562)'':0.0167269)'':0.00371638,('PDT000083448.1':0.0205007,('PDT000083451.1':0.0106686,('PDT000082645.1':0.00810344,('PDT000083574.1':0.00414845,('PDT000083475.1':0.00130489,'PDT000083582.1':0.00139322)'':0.00203635)'':0.00367924)'':0.00169297)'':0.00680658)'':0.0221789)'':0.0450667)'':0.000194548)'':0.0133972)'':0.0286326,((('PDT000242897.1':0.0514087,'PDT000242948.1':0.0385513)'':0.0560382,(('PDT000553142.1':0.0636854,'PDT000318158.2':0.0620056)'':0.0276516,(('PDT000425723.1':0.0877997,('PDT000350425.1':0.0363009,'PDT000095806.2':0.0323783)'':0.0374912)'':-0.000516654,('PDT000276016.1':0.0822841,(('PDT000248610.1':0.0495814,'PDT000392513.1':0.0540726)'':0.0125482,('PDT000548144.1':0.0446056,('PDT000317616.1':0.0395795,'PDT000088265.2':0.0318838)'':0.00515433)'':0.0135839)'':0.00901731)'':0.00624149)'':0.00841823)'':0.00186721)'':0.00788517,((('PDT000318171.1':0.0385923,('PDT000619717.2':0.0352547,'PDT000251868.1':0.0362848)'':0.0110294)'':0.00677929,(('PDT000040401.3':0.0407963,('PDT000443423.2':0.0461122,'PDT000085587.3':0.0308489)'':0.00444359)'':0.0149777,('PDT000047674.4':0.0466438,(('PDT000318155.1':0.0318809,'PDT000318169.1':0.0321175)'':0.00575386,('PDT000318172.2':0.0418365,('PDT000318168.1':0.0221314,'PDT000318170.2':0.02034)'':0.0110108)'':0.00125642)'':0.0109251)'':0.00245297)'':0.00582472)'':0.0147365,((('PDT000377961.2':0.0590141,('PDT000318163.2':0.0333569,('PDT000304227.1':0.0280291,('PDT000711035.1':0.0252202,('PDT000661608.1':0.0172127,'PDT000318186.1':0.01019)'':0.00977299)'':0.00709578)'':0.00366634)'':0.0200385)'':-0.01179,('PDT000038887.1':0.0342335,('PDT000304235.1':0.0323903,(('PDT000318184.1':0.0221058,'PDT000318188.2':0.021595)'':0.00746142,(('PDT000342302.2':0.0189535,('PDT000304229.1':0.0162379,'PDT000304230.1':0.015963)'':0.00337542)'':0.00459271,(('PDT000304237.1':0.0254658,'PDT000318182.1':0.0233291)'':0.00368933,('PDT000304233.1':0.0216915,(('PDT000063167.3':0.00906829,'PDT000318160.2':0.0113908)'':0.0071111,('PDT000318183.1':0.0132098,('PDT000160202.3':0.0131509,'PDT000160251.3':0.0101049)'':0.00138858)'':0.00187022)'':0.00873117)'':2.71476e-05)'':0.000116262)'':0.00276132)'':0.00369953)'':0.00769795)'':0.0011612)'':0.0132589,(('PDT000131887.3':0.0845627,('PDT000590353.2':0.0626337,('PDT000602887.1':0.0466657,('PDT000319830.2':0.0297907,'PDT000130793.3':0.0343595)'':0.00894587)'':0.0160018)'':0.0245204)'':-0.0170426,((('PDT000357236.2':0.0167215,'PDT000649131.2':0.0131382)'':0.0192059,('PDT000457521.2':0.01997,'PDT000562142.2':0.0260047)'':0.0138243)'':0.016031,(('PDT000318179.2':0.0476632,('PDT000318173.2':0.0295038,('PDT000318176.1':0.0310639,'PDT000318177.1':0.0293649)'':0.00404841)'':0.00461146)'':0.00343446,(('PDT000411999.2':0.0569687,('PDT000711118.1':0.0268489,'PDT000318159.1':0.0242997)'':0.0132016)'':0.00015307,('PDT000377213.2':0.0456368,('PDT000466234.2':0.0402173,('PDT000602888.1':0.0418823,('PDT000421092.2':0.0338693,(('PDT000063168.3':0.040101,('PDT000146552.3':0.0156679,'PDT000385958.2':0.0220064)'':0.0117332)'':0.00201877,(('PDT000304226.1':0.0237173,'PDT000304228.1':0.0278665)'':0.00153632,(('PDT000318174.1':0.0210919,'PDT000318175.1':0.0306492)'':0.000588898,('PDT000304239.1':0.0277555,('PDT000132122.2':0.0253174,(('PDT000161542.3':0.0148003,'PDT000407919.2':0.0198399)'':0.00189507,('PDT000304224.1':0.0151369,('PDT000300573.1':0.0146429,('PDT000063166.3':0.0121491,'PDT000318161.2':0.0100599)'':0.00394307)'':0.00174554)'':0.00412891)'':0.00255933)'':0.00233354)'':0.000588466)'':0.00184611)'':0.00529518)'':0.00154771)'':0.00395909)'':0.00215864)'':0.00806397)'':-0.00696443)'':0.00225966)'':0.00703314)'':0.0122353)'':-0.00817416)'':0.0194078)'':0.0314208)'':0.0242795)'':0.0179656)'':0.00696253,(((('PDT000014154.3':0.0332859,('PDT000250704.1':0.019602,('PDT000015800.3':0.0150496,'PDT000015858.3':0.0102656)'':0.0194386)'':0.0101633)'':0.0749332,((('PDT000173711.2':0.107858,'PDT000032339.1':0.0982989)'':0.0105068,('PDT000275048.2':0.101342,('PDT000645488.2':0.0842531,('PDT000626954.1':0.0749631,'PDT000045612.1':0.0696369)'':0.0073309)'':0.00787265)'':0.0152811)'':-0.00125674,(('PDT000270467.1':0.0873418,(('PDT000174339.1':0.0709529,('PDT000032192.1':0.0338287,'PDT000050593.1':0.0305549)'':0.0434208)'':0.00937439,(('PDT000174347.1':0.0720307,'PDT000174350.1':0.0408233)'':0.043699,('PDT000174352.1':0.0144275,('PDT000174340.1':0.00323309,'PDT000174353.1':0.00345354)'':0.00350762)'':0.0726202)'':0.00292178)'':0.00744951)'':-0.000450964,('PDT000610899.1':0.0929413,('PDT000221047.1':0.0833457,('PDT000174349.1':0.0847072,('PDT000174342.1':0.0825508,('PDT000221063.1':0.0698145,(('PDT000221050.1':0.0129844,'PDT000221064.1':0.0161332)'':0.0486935,(('PDT000221067.1':0.0291979,'PDT000250146.1':0.027312)'':0.0156542,('PDT000221038.1':0.0208307,('PDT000662611.1':0.0243239,('PDT000174327.1':0.0317754,'PDT000174348.1':0.0214224)'':0.0033707)'':0.0011372)'':0.0260689)'':0.0186075)'':0.00294835)'':0.00564534)'':0.00405795)'':0.0026339)'':0.0161034)'':0.00437081)'':0.00918301)'':0.00381842)'':0.00728011,(('PDT000602891.1':0.0518368,(('PDT000250064.1':0.0211719,('PDT000722600.1':0.0315332,('PDT000380271.2':0.0144072,'PDT000718324.1':0.0179662)'':0.00864031)'':0.00513021)'':-0.00173586,(('PDT000556953.1':0.0219626,'PDT000689801.1':0.0219264)'':0.00614067,('PDT000557023.1':0.00901929,('PDT000556992.1':0.00769442,'PDT000717609.1':0.00927658)'':0.00675356)'':0.0127395)'':0.00283543)'':0.025973)'':0.0141629,((('PDT000584530.1':0.0791478,('PDT000124667.1':0.0564169,'PDT000388474.1':0.0467011)'':0.0302977)'':0.0011454,((('PDT000124652.1':0.0338429,'PDT000124682.1':0.029906)'':0.0238443,('PDT000124614.1':0.0497693,('PDT000246506.2':0.0118336,'PDT000557065.1':0.01095)'':0.027613)'':0.00119513)'':0.00815674,(('PDT000668393.2':0.0493742,'PDT000242899.1':0.0526548)'':0.00578453,(('PDT000651910.1':0.0562579,'PDT000672678.1':0.0401853)'':0.0020536,('PDT000010547.2':0.0222745,('PDT000372030.2':0.0161288,'PDT000645158.1':0.0195112)'':0.00542161)'':0.0231122)'':0.0096314)'':0.00232788)'':0.0147759)'':-0.00337312,(('PDT000714646.1':0.0616601,('PDT000031872.1':0.0268003,'PDT000124646.1':0.0339629)'':0.0310638)'':0.00379732,('PDT000556941.1':0.044406,(('PDT000668336.2':0.0338025,'PDT000672647.1':0.027712)'':0.00729443,(('PDT000672628.1':0.043905,('PDT000640017.1':0.0191544,'PDT000653440.1':0.0252044)'':0.020233)'':0.00133632,('PDT000651903.1':0.0288825,(('PDT000655792.1':0.012199,'PDT000655942.1':0.0116766)'':0.00741249,('PDT000556974.1':0.0153606,('PDT000640043.1':0.0176893,'PDT000651909.1':0.0101134)'':0.00497427)'':0.000460967)'':0.0024301)'':0.0117387)'':0.00939241)'':0.0135354)'':0.00683106)'':0.00393362)'':0.0169051)'':0.061156)'':0.015613,((((('PDT000482817.1':0.0618541,('PDT000285327.1':0.0492537,'PDT000285340.1':0.0764853)'':0.0212807)'':0.0283458,(('PDT000150181.1':0.0795753,'PDT000176456.1':0.0818187)'':0.0143417,('PDT000576620.1':0.0732905,('PDT000251755.1':0.0726706,(('PDT000652916.1':0.0242878,'PDT000652954.1':0.0244354)'':0.0467591,(('PDT000290365.2':0.0479231,'PDT000228116.1':0.0462618)'':0.0159835,(('PDT000228081.1':0.0373921,'PDT000228110.1':0.0295657)'':0.0256826,('PDT000228127.1':0.0516503,('PDT000228113.1':0.0351583,'PDT000248807.1':0.0415119)'':0.00755601)'':0.00818524)'':0.00602364)'':0.00426024)'':0.00925191)'':0.000951512)'':0.00313154)'':0.00931084)'':-0.00288776,(('PDT000318165.1':0.0878607,('PDT000386732.1':0.0746731,'PDT000360980.1':0.0814869)'':0.00824781)'':0.00373714,(('PDT000283366.2':0.0711377,('PDT000315526.1':0.0631046,('PDT000377078.1':0.047997,('PDT000155607.2':0.0251434,'PDT000150180.1':0.0312154)'':0.024774)'':0.00682714)'':0.0168016)'':0.00543591,('PDT000346022.1':0.0938018,('PDT000215762.2':0.0924485,(('PDT000652889.1':0.0563298,'PDT000318166.1':0.0568542)'':0.00677076,('PDT000213712.1':0.0389041,('PDT000281469.2':0.0233892,('PDT000318164.1':0.0201822,('PDT000281079.2':0.0160324,'PDT000471038.1':0.0124088)'':0.00560584)'':0.00423562)'':0.00817819)'':0.0179442)'':0.0263302)'':0.00740913)'':-0.00474326)'':0.00572687)'':-0.000377205)'':0.0115238,(('PDT000191845.2':0.0887404,(('PDT000240685.2':0.0482309,'PDT000275540.2':0.046152)'':0.0282645,(('PDT000288645.2':0.0360031,'PDT000417399.1':0.0401103)'':0.0311333,('PDT000325580.1':0.0636252,('PDT000150238.2':0.0589372,('PDT000659702.1':0.0559327,(('PDT000239225.2':0.0470301,('PDT000231467.2':0.0453673,'PDT000407231.1':0.0482603)'':0.00374597)'':0.0007582,(('PDT000306427.2':0.0424602,('PDT000541630.1':0.0292988,'PDT000646915.1':0.0296506)'':0.00850937)'':0.00840276,('PDT000490608.1':0.0436645,('PDT000288617.2':0.0305243,'PDT000428394.1':0.0300465)'':0.00939585)'':0.00331497)'':0.00214577)'':0.0100667)'':0.00124455)'':0.0126483)'':0.00400308)'':0.00581239)'':0.00751818)'':0.00507371,(('PDT000626530.1':0.0534468,('PDT000485564.1':0.0475881,'PDT000563562.1':0.0464992)'':0.00559906)'':0.0120587,(('PDT000323512.1':0.048923,('PDT000095689.2':0.0348634,'PDT000600988.1':0.0425891)'':0.00659702)'':0.0177414,((('PDT000303124.2':0.0522674,'PDT000485550.1':0.043999)'':0.00812593,('PDT000271395.2':0.0494164,('PDT000482515.1':0.0461815,'PDT000288609.2':0.0483164)'':0.00101331)'':0.00702279)'':0.00279747,(('PDT000386749.1':0.0556437,('PDT000382195.1':0.0413852,'PDT000500495.1':0.0416837)'':0.0106547)'':0.00754026,((('PDT000258724.2':0.0487993,'PDT000290513.2':0.0492586)'':0.015892,('PDT000274090.2':0.0474311,'PDT000631992.1':0.037974)'':0.0175155)'':0.0057165,(('PDT000490606.1':0.0352634,'PDT000390717.1':0.0409123)'':0.0162189,('PDT000337000.1':0.0572462,('PDT000259571.2':0.0301409,'PDT000651585.1':0.0270076)'':0.01453)'':0.0102211)'':0.004114)'':0.00288432)'':5.7499e-05)'':0.00459049)'':0.000495112)'':0.0235349)'':0.0178563)'':0.0387611,(('PDT000124691.1':0.161151,(('PDT000397786.1':0.11301,'PDT000252467.1':0.0918465)'':0.0171158,('PDT000111374.1':0.0889908,('PDT000252634.1':0.0682608,('PDT000130430.2':0.0342205,'PDT000251852.1':0.0344603)'':0.0214573)'':0.0130802)'':0.0285566)'':0.0242324)'':-0.00376083,((('PDT000421760.1':0.0789006,'PDT000586231.1':0.101015)'':0.0416138,('PDT000253443.2':0.127205,(('PDT000386719.1':0.0611283,('PDT000535516.1':0.0505245,'PDT000559255.1':0.0394419)'':0.00923146)'':0.0509937,(('PDT000688610.1':0.0439233,('PDT000708217.1':0.050643,'PDT000346674.1':0.049081)'':0.0072762)'':0.0611548,(('PDT000316507.1':0.117995,('PDT000346094.1':0.115285,('PDT000538424.1':0.0708504,('PDT000417158.1':0.062215,('PDT000362848.1':0.0251542,'PDT000346645.1':0.0233771)'':0.0268354)'':0.00855093)'':0.0384088)'':0.00347075)'':0.00125663,('PDT000386717.1':0.0885042,(('PDT000250156.1':0.0479188,'PDT000346657.1':0.0496817)'':0.0251532,(('PDT000559722.2':0.0809576,'PDT000242951.1':0.0992884)'':0.00446768,('PDT000203398.2':0.0742362,('PDT000417974.1':0.0464446,'PDT000651233.1':0.0594214)'':0.0187403)'':0.00676704)'':0.00474287)'':0.0083549)'':0.0136635)'':0.00546344)'':0.0034858)'':0.0023647)'':0.00119415)'':-0.00174062,(('PDT000619764.1':0.0595438,('PDT000131925.2':0.0575502,(('PDT000082646.1':0.0290289,('PDT000253424.2':0.0225154,'PDT000083546.1':0.0207284)'':0.0124656)'':0.00733848,('PDT000161890.2':0.0324738,('PDT000524022.1':0.0287169,('PDT000248699.1':0.0285682,('PDT000032373.1':0.0216519,('PDT000083477.1':0.0127743,'PDT000083528.1':0.0136091)'':0.00682722)'':0.00219304)'':0.0017793)'':0.00404807)'':0.00381373)'':0.0235477)'':0.000929567)'':0.0593015,((('PDT000394961.1':0.119288,('PDT000336999.1':0.103695,'PDT000646902.1':0.112056)'':0.0133139)'':0.00577961,('PDT000132223.2':0.0933486,('PDT000557002.1':0.0329327,'PDT000228112.1':0.0434039)'':0.0646046)'':0.0243116)'':0.00444731,(('PDT000251542.1':0.0937291,('PDT000663912.1':0.0108188,('PDT000663913.1':0.00503376,'PDT000663930.1':0.00803874)'':0.00256288)'':0.0839495)'':0.0185258,((('PDT000251533.1':0.109222,('PDT000587142.1':0.0781773,'PDT000361012.1':0.0621087)'':0.0301793)'':-0.00200211,(('PDT000275523.2':0.103575,'PDT000455220.1':0.094905)'':0.0020681,(('PDT000346546.1':0.0764934,'PDT000346662.1':0.0593636)'':0.0432626,((('PDT000417138.1':0.0497358,'PDT000233304.1':0.0466107)'':0.0302204,('PDT000233271.2':0.0565956,'PDT000298049.1':0.0598064)'':0.0382044)'':0.00579114,('PDT000667725.2':0.0792883,('PDT000710705.1':0.0723873,('PDT000292867.1':0.0623964,('PDT000346665.1':0.0677698,('PDT000525368.1':0.0591727,('PDT000129630.1':0.0645543,('PDT000518560.1':0.0439079,'PDT000482824.1':0.0406898)'':0.00870688)'':0.00429807)'':0.00468557)'':0.00172423)'':0.00832887)'':0.00374978)'':0.000624589)'':0.0114914)'':0.00735791)'':0.00604954)'':0.00058633,(('PDT000129840.2':0.139624,(('PDT000559684.1':0.0497493,'PDT000689879.1':0.0327219)'':0.0548713,('PDT000373307.1':0.111644,('PDT000571520.1':0.0714682,('PDT000672185.1':0.0374625,'PDT000355788.1':0.0466496)'':0.0274828)'':0.0329107)'':0.00368497)'':0.0121309)'':-0.00626948,(('PDT000289907.1':0.0983496,'PDT000346565.1':0.105586)'':0.017461,((('PDT000380289.1':0.0401207,'PDT000463691.1':0.0426439)'':0.0420749,('PDT000535530.1':0.0850374,('PDT000463642.1':0.0811824,('PDT000212782.1':0.0837825,('PDT000346664.1':0.0734275,('PDT000101296.1':0.0684711,('PDT000136027.2':0.0439006,'PDT000318167.1':0.05008)'':0.00763311)'':0.00478984)'':0.0115543)'':0.00140985)'':0.00943773)'':0.00184728)'':0.00560472,((('PDT000620999.1':0.0390578,'PDT000621002.1':0.0439858)'':0.0646296,('PDT000129698.1':0.0791321,'PDT000346641.1':0.0762009)'':0.0161958)'':0.0242056,((('PDT000382811.1':0.0407405,'PDT000251994.1':0.0314366)'':0.037836,('PDT000129837.2':0.0836348,('PDT000156605.2':0.0634085,'PDT000346637.1':0.0614015)'':0.0156957)'':0.00315417)'':0.00656845,(('PDT000583479.1':0.0941023,'PDT000667742.2':0.0866697)'':0.0128239,((('PDT000120846.3':0.0647067,'PDT000357671.1':0.0599883)'':0.0177279,('PDT000020079.1':0.0716307,('PDT000444960.1':0.043341,'PDT000252010.1':0.0458686)'':0.010904)'':0.0233532)'':0.00680893,((('PDT000416503.1':0.0943681,('PDT000589781.1':0.0585872,'PDT000289918.1':0.0636058)'':0.0231269)'':0.00500167,(('PDT000181845.2':0.0359088,'PDT000607505.1':0.0442596)'':0.0430778,('PDT000397878.1':0.0756698,('PDT000289885.1':0.0869783,('PDT000345993.1':0.0704849,('PDT000372341.1':0.0478924,('PDT000228020.1':0.0439103,'PDT000424791.1':0.0369579)'':0.0027771)'':0.0157012)'':0.00659743)'':0.00431571)'':0.00520522)'':0.00242405)'':-0.00047913,((('PDT000633903.1':0.0787499,'PDT000425784.1':0.0701251)'':0.00864118,('PDT000356785.1':0.0319396,'PDT000346633.1':0.0324859)'':0.0385711)'':0.000731939,('PDT000395471.1':0.0747748,('PDT000346643.1':0.0759567,('PDT000710704.1':0.0568054,('PDT000346655.1':0.0614856,('PDT000034373.1':0.00825129,('PDT000034432.1':0.00614754,('PDT000034434.1':0.00557633,(('PDT000034376.1':0.00487982,'PDT000034416.1':0.0037907)'':0.000283171,('PDT000034394.1':0.00299361,'PDT000034413.1':0.00269047)'':0.000859087)'':0.00153134)'':0.000948837)'':0.000734554)'':0.0426777)'':0.0103759)'':0.00113348)'':0.0033875)'':0.00641091)'':0.00924714)'':0.00325001)'':0.00561389)'':-0.00346054)'':0.0107811)'':-0.00141107)'':0.0120319)'':-0.00952463)'':0.00279625)'':0.00551087)'':0.00133527)'':0.0019819)'':-0.00152094)'':0.0154073)'':-0.00594365)'':0.00608423)'':0.00167163)'':0.0172585)'':0.0181031,(((('PDT000616440.1':0.157584,(('PDT000496906.1':0.104247,('PDT000242492.1':0.0753138,('PDT000259575.2':0.0383738,'PDT000275534.2':0.0437266)'':0.044507)'':0.0189536)'':0.0281516,(('PDT000131025.3':0.0662267,('PDT000484196.2':0.0294452,'PDT000345923.1':0.0223382)'':0.0276711)'':0.013319,(('PDT000483976.2':0.0283104,'PDT000544352.2':0.0328782)'':0.0227891,('PDT000669816.1':0.0404466,('PDT000332110.2':0.0474307,'PDT000553540.2':0.0366886)'':0.00706624)'':0.010449)'':0.00856303)'':0.062036)'':0.0348017)'':0.0172723,((('PDT000667965.1':0.1374,('PDT000203146.2':0.110159,'PDT000501844.1':0.10322)'':0.0234091)'':0.0378483,(('PDT000196618.2':0.0879849,('PDT000181848.2':0.115122,'PDT000562856.1':0.104632)'':0.00385661)'':0.0439099,(('PDT000539051.1':0.0324364,'PDT000645102.1':0.030173)'':0.0504541,('PDT000621067.1':0.0974064,(('PDT000564192.1':0.0536864,'PDT000396762.1':0.0513466)'':0.00825302,('PDT000645129.1':0.0406333,('PDT000648088.1':0.0512812,('PDT000651124.1':0.0236764,'PDT000712627.1':0.024139)'':0.00961419)'':0.0148457)'':0.0226566)'':0.00660872)'':0.00912465)'':0.0398326)'':0.0281065)'':0.00111443,((('PDT000557044.2':0.14947,'PDT000691347.1':0.126285)'':0.0150618,(('PDT000200999.2':0.129793,'PDT000203390.2':0.124489)'':0.023707,('PDT000081747.1':0.0771605,('PDT000343879.1':0.051203,('PDT000332730.1':0.0588296,'PDT000415802.1':0.0510334)'':0.015651)'':0.0109118)'':0.0809711)'':0.0127609)'':0.00489262,((('PDT000308022.1':0.0451331,('PDT000622843.1':0.0352266,('PDT000632809.1':0.028953,'PDT000036732.1':0.0288755)'':0.0083022)'':0.00904685)'':0.089267,('PDT000496987.1':0.0951896,('PDT000429779.1':0.0362348,('PDT000428888.1':0.0261319,'PDT000448175.1':0.0310864)'':0.0120418)'':0.0495738)'':0.0295076)'':0.0116288,(('PDT000346531.1':0.120702,('PDT000285326.1':0.112963,('PDT000242908.1':0.0322942,'PDT000242922.1':0.0302058)'':0.0784301)'':0.0213795)'':-0.00479266,('PDT000661076.1':0.102921,(('PDT000235614.2':0.0758297,'PDT000714682.1':0.0718083)'':0.011191,('PDT000233294.2':0.0895628,('PDT000206242.3':0.0599488,('PDT000003672.2':0.0546018,'PDT000095812.2':0.0453789)'':0.0033603)'':0.0206555)'':0.00926499)'':0.0193963)'':0.00972913)'':0.00787194)'':0.0401223)'':0.000566036)'':0.00649933)'':0.00150989,((((('PDT000203392.2':0.135457,'PDT000621071.1':0.162557)'':0.0381142,('PDT000497404.1':0.107781,('PDT000003780.2':0.0673626,('PDT000237134.1':0.0497443,'PDT000396738.1':0.0528267)'':0.0143234)'':0.0458114)'':0.0415124)'':0.00602525,(('PDT000168795.1':0.0818263,'PDT000289902.1':0.0842857)'':0.0303012,('PDT000417396.1':0.0976872,(('PDT000128876.2':0.0806043,('PDT000203391.2':0.0702734,'PDT000307427.2':0.0566346)'':0.00835716)'':0.0144712,('PDT000497242.1':0.0871087,('PDT000584733.1':0.0520385,'PDT000395451.1':0.0576235)'':0.0276133)'':0.00086035)'':0.00683348)'':0.00818734)'':0.0678546)'':0.0033573,((('PDT000216116.2':0.144115,'PDT000616408.1':0.150882)'':0.0032875,('PDT000620995.1':0.164891,(('PDT000203393.2':0.140884,('PDT000621014.1':0.138373,('PDT000417191.1':0.111922,'PDT000168794.1':0.123204)'':0.0352809)'':0.00620867)'':-0.00333341,((('PDT000206245.3':0.0762048,'PDT000621018.1':0.0885262)'':0.00407641,('PDT000626013.1':0.082553,('PDT000047204.3':0.0636737,('PDT000466973.1':0.0500882,'PDT000251934.1':0.0615668)'':0.00653176)'':0.0120615)'':0.00177423)'':0.0376066,(('PDT000291283.1':0.111997,('PDT000294386.1':0.0707445,('PDT000417137.1':0.0725975,'PDT000346663.1':0.0676955)'':0.00744501)'':0.0161354)'':0.0199892,(('PDT000003960.2':0.127681,((('PDT000625083.1':0.0614891,'PDT000346652.1':0.0417489)'':0.0523588,('PDT000645045.1':0.0851046,('PDT000346647.1':0.0751732,('PDT000652967.1':0.0592496,'PDT000228016.1':0.0618254)'':0.0134938)'':0.0136664)'':0.00856863)'':0.00558301,(('PDT000180921.1':0.100008,'PDT000252429.1':0.0954864)'':0.0044352,('PDT000667708.2':0.0884205,(('PDT000215773.2':0.0674484,'PDT000690663.1':0.0769266)'':0.0130337,('PDT000583482.1':0.0813524,'PDT000396750.1':0.0763856)'':0.00736703)'':0.00578109)'':0.00422902)'':0.00461443)'':0.0166897)'':-0.00319245,(('PDT000346554.1':0.0884721,('PDT000289892.1':0.0887084,('PDT000712619.1':0.0810792,'PDT000409991.1':0.0794208)'':0.0112456)'':0.0105586)'':0.0148185,(('PDT000626947.1':0.0977977,('PDT000290384.2':0.108853,'PDT000250219.1':0.0743823)'':0.00974881)'':0.0114957,('PDT000425956.1':0.107285,(('PDT000631554.1':0.0815827,('PDT000167687.2':0.0604098,'PDT000625999.1':0.0647912)'':0.0185273)'':0.0163462,('PDT000206120.2':0.0872771,('PDT000651174.1':0.0695437,('PDT000027910.2':0.0643508,'PDT000251458.1':0.0605902)'':0.0146158)'':0.00498751)'':0.018199)'':0.000670182)'':0.00478741)'':0.00279395)'':-0.000237924)'':0.00796948)'':0.00306997)'':0.0121558)'':0.0274642)'':-0.0045398)'':0.00133224,(('PDT000242877.1':0.0782015,'PDT000242982.1':0.0688715)'':0.0689172,((('PDT000620993.1':0.0702787,'PDT000168810.1':0.0730303)'':0.0250405,('PDT000621038.1':0.0729862,('PDT000168820.1':0.0614203,'PDT000289915.1':0.0552497)'':0.0101813)'':0.00877426)'':0.0500844,(('PDT000206127.2':0.1207,('PDT000621053.1':0.0665946,'PDT000168769.1':0.0818074)'':0.0509747)'':0.0050048,((('PDT000537944.1':0.0933089,'PDT000285335.1':0.0873751)'':0.0194244,((('PDT000652528.1':0.0602712,'PDT000429814.1':0.0571928)'':0.00198008,(('PDT000421761.1':0.0436869,'PDT000625990.1':0.0489485)'':0.00777166,('PDT000425790.1':0.0494814,'PDT000346569.1':0.0535226)'':0.00656999)'':0.0144026)'':0.00687598,(('PDT000634648.1':0.0678192,'PDT000250706.1':0.0674598)'':0.0172019,((('PDT000034385.1':0.00973731,'PDT000034402.1':0.0155263)'':0.0502975,('PDT000285329.1':0.00266186,'PDT000285330.1':0.00272521)'':0.0635792)'':0.00910421,('PDT000347092.1':0.0460415,(('PDT000149880.2':0.0287028,'PDT000334279.1':0.0305723)'':0.0223089,('PDT000709931.1':0.0382767,('PDT000083135.2':0.0217959,'PDT000046340.1':0.0284063)'':0.0145564)'':0.011789)'':0.00454207)'':0.0144303)'':0.0104927)'':-0.00101556)'':0.0237797)'':0.00529975,(('PDT000248681.1':0.0475042,('PDT000248724.1':0.0385437,'PDT000389329.1':0.0389876)'':0.00675139)'':0.0628115,('PDT000010153.2':0.118844,('PDT000251558.1':0.117428,(('PDT000457699.1':0.114499,('PDT000661138.1':0.0912233,'PDT000429778.1':0.0989537)'':0.00377046)'':0.00310779,(('PDT000652887.1':0.10295,'PDT000661280.1':0.0911279)'':0.0194524,(('PDT000388614.1':0.0275323,'PDT000662580.1':0.0273291)'':0.0693567,(('PDT000687089.1':0.0619066,'PDT000250145.1':0.0613094)'':0.0253628,('PDT000544351.1':0.111044,('PDT000221028.1':0.0884401,('PDT000425822.1':0.0788873,(('PDT000032384.1':0.0635547,'PDT000463639.1':0.0646653)'':6.37304e-05,('PDT000136156.2':0.0603619,('PDT000626966.1':0.0655729,'PDT000373313.1':0.0536551)'':0.00468007)'':0.00292091)'':0.00628001)'':0.00773813)'':0.00451057)'':0.001469)'':0.00309375)'':0.00379839)'':-0.000317928)'':0.0144458)'':0.00224028)'':-0.0118271)'':-0.00227566)'':0.00888298)'':0.00763476)'':0.0172772)'':0.0163463)'':0.0141975)'':-0.00230793,(((('PDT000712618.1':0.13246,'PDT000111354.1':0.0974239)'':0.0179244,(('PDT000264295.1':0.0934915,('PDT000466997.1':0.0735309,('PDT000663946.1':0.0635345,'PDT000463655.1':0.0707075)'':0.0133596)'':0.00429684)'':0.0255481,(('PDT000062957.2':0.0906497,'PDT000496761.1':0.100214)'':0.0253618,(('PDT000357595.1':0.094674,'PDT000384978.1':0.106344)'':0.00790185,('PDT000271522.2':0.096575,('PDT000032354.1':0.0827573,('PDT000485446.1':0.0667134,'PDT000251789.1':0.0740996)'':0.00902016)'':0.0081481)'':0.00186791)'':0.0123004)'':0.00274466)'':0.010833)'':0.0141902,(('PDT000174325.1':0.0994985,('PDT000466082.1':0.0850217,('PDT000438165.1':0.0589973,('PDT000235747.2':0.0668433,'PDT000351580.1':0.0600207)'':0.00376821)'':0.00311673)'':0.0361419)'':0.0331089,(('PDT000683485.1':0.113474,('PDT000468873.1':0.104022,('PDT000289873.1':0.0625647,'PDT000353629.1':0.0572233)'':0.0498959)'':0.00841124)'':0.0270208,(('PDT000396749.1':0.0797536,'PDT000468836.1':0.0897774)'':0.0523906,('PDT000621056.1':0.142372,('PDT000084292.3':0.112983,('PDT000346015.1':0.102755,('PDT000251537.1':0.0832476,('PDT000339384.1':0.0685452,'PDT000661131.1':0.0701518)'':0.00823495)'':0.00859614)'':0.00442372)'':0.0111168)'':0.00256549)'':0.00198032)'':0.0111708)'':0.00714054)'':0.0186166,((('PDT000395039.1':0.17811,(('PDT000537819.1':0.144482,('PDT000362471.1':0.131199,('PDT000034484.3':0.134786,('PDT000237138.1':0.0645596,'PDT000411298.1':0.0732644)'':0.0520496)'':0.00726241)'':0.00215336)'':0.00822209,(('PDT000346631.1':0.111519,'PDT000346653.1':0.135451)'':0.00508718,(('PDT000171806.1':0.0631879,'PDT000346661.1':0.0760101)'':0.0310338,('PDT000252666.1':0.072558,'PDT000336606.1':0.075133)'':0.0168917)'':0.00865682)'':0.0299023)'':0.0186234)'':-0.00060143,((((('PDT000368351.1':0.0463063,'PDT000325639.1':0.0348709)'':0.0428316,('PDT000171815.1':0.0555513,'PDT000346773.1':0.0379732)'':0.0411866)'':0.00933678,(('PDT000584657.1':0.0971156,'PDT000610218.1':0.0753234)'':0.0139201,('PDT000319014.1':0.0648314,('PDT000626018.1':0.0631703,'PDT000336605.1':0.0590847)'':0.00652058)'':0.0192227)'':0.00684465)'':0.0125403,((('PDT000557196.1':0.0548177,('PDT000536981.1':0.0232551,'PDT000557151.1':0.0138781)'':0.0469252)'':0.0211595,('PDT000639506.1':0.0464888,('PDT000557003.1':0.0363389,('PDT000333779.1':0.026626,('PDT000524496.1':0.0175972,'PDT000644964.1':0.0196515)'':0.0129471)'':0.0154083)'':0.00234203)'':0.0349241)'':0.0096208,(('PDT000586233.1':0.0269309,'PDT000346592.1':0.0284513)'':0.0640411,(('PDT000242911.1':0.00991047,'PDT000242912.1':0.00999253)'':0.0848492,('PDT000242871.1':0.0861097,('PDT000541631.1':0.0698489,('PDT000181747.2':0.0612829,('PDT000566697.1':0.039465,'PDT000644672.1':0.0442443)'':0.020003)'':0.00869372)'':0.0084598)'':0.00430029)'':0.0116193)'':-0.000701819)'':0.00874953)'':0.00494867,((('PDT000346746.1':0.0632175,('PDT000677015.1':0.0394099,('PDT000114751.2':0.0235433,'PDT000129918.2':0.0357078)'':0.00541435)'':0.0329469)'':0.032379,(('PDT000395011.1':0.0539307,('PDT000565948.1':0.0218999,'PDT000395446.1':0.0230835)'':0.0423781)'':0.037148,(('PDT000003803.2':0.0696332,'PDT000403072.1':0.0833108)'':0.00711074,('PDT000661106.1':0.0615243,('PDT000181751.2':0.0686573,('PDT000569552.1':0.048068,'PDT000395364.1':0.0247263)'':0.0255961)'':0.00628228)'':0.00670465)'':0.00835856)'':0.0182234)'':0.00477988,('PDT000626894.1':0.126403,(('PDT000311250.2':0.0561371,'PDT000411296.1':0.0643489)'':0.0336286,(('PDT000228111.1':0.0907797,('PDT000535475.1':0.0691528,'PDT000411290.1':0.0724902)'':0.0278088)'':0.00271677,(('PDT000652550.1':0.0856719,('PDT000104791.2':0.04087,'PDT000590159.1':0.0365413)'':0.0504135)'':0.000459995,(('PDT000346511.1':0.0625536,'PDT000463705.1':0.0731454)'':0.0107169,('PDT000357565.1':0.0866557,('PDT000233324.1':0.0701283,('PDT000160446.2':0.0416201,'PDT000161518.2':0.0210173)'':0.036736)'':0.0069387)'':0.00473762)'':0.0041904)'':0.00268778)'':0.00751071)'':0.0135376)'':-0.0106632)'':0.00400067)'':0.0360254)'':0.00494002,(((('PDT000652556.1':0.11347,('PDT000672701.1':0.123621,'PDT000709965.1':0.10149)'':0.0170685)'':0.0324366,(('PDT000141407.2':0.0813079,('PDT000651070.1':0.0664913,('PDT000233288.1':0.0511774,('PDT000425786.1':0.0362632,'PDT000426681.1':0.0362216)'':0.0107818)'':0.00803642)'':0.00994047)'':0.0178923,(('PDT000175889.2':0.0290298,'PDT000713298.1':0.0398678)'':0.00885788,('PDT000251935.1':0.0282452,('PDT000325758.1':0.0226223,('PDT000132403.2':0.0159314,'PDT000442721.1':0.0157959)'':0.00804756)'':0.0076723)'':0.0100263)'':0.0563476)'':0.0331105)'':0.0127474,((('PDT000161001.2':0.0739284,(('PDT000316478.1':0.0372064,'PDT000366125.1':0.0407006)'':0.00342128,(('PDT000379489.1':0.0126848,'PDT000496080.1':0.0130961)'':0.00480736,('PDT000253451.2':0.0166253,('PDT000253464.2':0.0132254,'PDT000377216.1':0.00797531)'':0.00997204)'':0.00279623)'':0.0200978)'':0.0400059)'':-0.0013706,(('PDT000253468.2':0.0159651,('PDT000651770.1':0.0212351,('PDT000651387.1':0.0163451,('PDT000316520.1':0.00788118,'PDT000253444.2':0.00740872)'':0.00662026)'':0.0026287)'':0.00137588)'':0.0453908,('PDT000399956.1':0.0506183,(('PDT000280766.2':0.0297652,'PDT000486043.1':0.0362488)'':0.0109363,(('PDT000161696.2':0.0242905,('PDT000253428.2':0.013122,'PDT000637707.1':0.0131392)'':0.0102191)'':0.0056674,('PDT000253454.2':0.0309735,('PDT000155844.2':0.0064755,('PDT000412844.1':0.00415246,'PDT000487473.1':0.003826)'':0.000680122)'':0.017356)'':0.00518875)'':0.00881315)'':0.00408488)'':0.0219638)'':0.00618808)'':0.0727771,(('PDT000157216.2':0.131945,('PDT000025980.1':0.00446879,'PDT000025984.1':0.00320206)'':0.122981)'':0.00583784,(('PDT000545913.1':0.109169,(('PDT000062946.2':0.106675,'PDT000238521.1':0.111115)'':0.0111806,('PDT000646123.1':0.0471085,'PDT000358764.1':0.0328644)'':0.0824967)'':0.00969338)'':-0.00518977,((('PDT000536975.1':0.0614169,'PDT000251487.1':0.0536631)'':0.0196759,('PDT000345914.1':0.0664765,('PDT000048136.1':0.0576937,'PDT000250243.1':0.0582363)'':0.00765398)'':0.0140298)'':0.00820551,(('PDT000672705.1':0.0765342,(('PDT000043872.1':0.0670116,'PDT000123300.1':0.0555944)'':0.0250252,(('PDT000303146.3':0.0507932,'PDT000242500.1':0.039268)'':0.0134873,('PDT000533132.2':0.0524711,'PDT000029748.3':0.0615549)'':0.00506759)'':0.0129783)'':0.0106217)'':-0.00721042,('PDT000344018.2':0.0773667,(('PDT000238432.3':0.0449815,('PDT000140532.3':0.0465911,'PDT000252428.1':0.0312153)'':0.00718155)'':0.00441586,('PDT000140447.3':0.0715067,(('PDT000137590.3':0.0323607,('PDT000242603.3':0.0198876,'PDT000242501.1':0.0277881)'':0.00358423)'':0.0213739,(('PDT000545119.2':0.0432436,'PDT000639576.1':0.0368812)'':0.00330332,('PDT000312542.3':0.0464619,('PDT000487929.1':0.0387688,('PDT000351832.2':0.0237522,'PDT000242498.1':0.0345885)'':0.00531424)'':0.00544918)'':0.000150433)'':0.00204677)'':0.0136373)'':-0.00393397)'':0.0133358)'':-0.00581978)'':0.0238987)'':0.0320698)'':0.0133336)'':0.00152673)'':0.00502754)'':0.00197388,((('PDT000221060.1':0.102038,('PDT000566009.1':0.0939827,(('PDT000305151.2':0.0573362,'PDT000250205.1':0.0556878)'':0.00523061,('PDT000397880.1':0.0578985,'PDT000568759.1':0.0464155)'':0.0108665)'':0.0232785)'':0.0075623)'':0.00785587,('PDT000201012.2':0.125145,('PDT000504740.1':0.129314,('PDT000460576.1':0.0999444,('PDT000218440.2':0.0894175,('PDT000537940.1':0.0771762,('PDT000203399.2':0.0677383,'PDT000354259.1':0.0649297)'':0.0225093)'':0.00468386)'':0.00226916)'':0.0106248)'':0.0106045)'':-0.0118366)'':0.031122,(('PDT000439628.1':0.16979,(('PDT000181760.2':0.127597,'PDT000220632.1':0.142368)'':0.0205301,('PDT000217106.2':0.122361,(('PDT000167661.2':0.0936295,('PDT000632982.1':0.0837829,'PDT000273985.1':0.0689871)'':0.021075)'':0.013882,('PDT000414209.1':0.0754831,('PDT000168846.1':0.0904625,('PDT000168812.1':0.0737221,('PDT000289897.1':0.0724703,'PDT000291282.1':0.0737237)'':0.00637343)'':0.00343986)'':0.00737041)'':0.0238003)'':0.0236316)'':0.00843018)'':0.0175184)'':-0.0158917,((('PDT000266405.2':0.107453,(('PDT000124669.1':0.0783556,'PDT000363562.1':0.0703584)'':0.00927881,('PDT000425483.1':0.0990717,('PDT000625086.1':0.0889512,('PDT000661179.1':0.0594943,'PDT000168862.1':0.0599897)'':0.0197668)'':0.00434169)'':0.021794)'':0.0216499)'':0.0135756,('PDT000621031.1':0.150628,(('PDT000616166.1':0.0820712,('PDT000463704.1':0.0762249,('PDT000661169.1':0.0692466,('PDT000713749.1':0.0610993,('PDT000535395.1':0.0558083,'PDT000571187.1':0.0545677)'':0.00596873)'':0.00517152)'':0.00630663)'':0.00738396)'':0.0202689,(('PDT000372339.1':0.0868602,'PDT000548113.1':0.0846158)'':0.0300122,(('PDT000396603.2':0.103126,'PDT000463624.1':0.10125)'':0.00389515,('PDT000621054.1':0.118156,('PDT000621028.1':0.104227,(('PDT000249943.1':0.0771743,('PDT000719056.1':0.0676274,'PDT000131428.1':0.0686546)'':0.0050312)'':0.00195488,('PDT000353639.1':0.0770091,('PDT000249936.1':0.0528232,'PDT000251779.1':0.0536998)'':0.0149439)'':0.0100387)'':0.0159567)'':0.00848704)'':8.4021e-05)'':0.00120175)'':0.00314042)'':0.0136295)'':-0.00284951)'':0.0186859,(((('PDT000174351.1':0.0985073,('PDT000501687.2':0.0262237,'PDT000523970.2':0.01876)'':0.105178)'':0.00729167,('PDT000252006.1':0.0908057,('PDT000161708.2':0.0552422,'PDT000463690.1':0.0634438)'':0.0276488)'':0.0220943)'':0.00468503,(('PDT000250029.1':0.113507,(('PDT000566698.1':0.0765709,'PDT000011815.2':0.0677311)'':0.0313226,('PDT000167678.2':0.104187,('PDT000563566.1':0.0775954,'PDT000652578.1':0.0635356)'':0.0246223)'':0.00474129)'':0.00752254)'':0.000834588,('PDT000718961.1':0.114498,('PDT000667711.2':0.0833197,(('PDT000310439.2':0.0750816,'PDT000484652.1':0.0643834)'':0.00779607,('PDT000414203.1':0.0745143,('PDT000346200.1':0.0574996,('PDT000683607.1':0.0334817,'PDT000384974.1':0.0361182)'':0.008654)'':0.0155465)'':0.0250913)'':0.00671732)'':0.00847301)'':0.0187189)'':0.00369756)'':0.00294218,(('PDT000594470.1':0.147776,(('PDT000534932.1':0.0636138,'PDT000210703.1':0.0744362)'':0.0428976,((('PDT000332095.1':0.0406441,'PDT000228147.1':0.0352625)'':0.0433511,('PDT000250159.1':0.0801334,('PDT000188377.1':0.0668657,'PDT000257159.1':0.0699323)'':0.0165406)'':0.00338672)'':0.0158218,('PDT000142807.2':0.0998187,(('PDT000689852.1':0.0270194,'PDT000691270.1':0.0385416)'':0.0242615,('PDT000619670.1':0.0566612,('PDT000518123.1':0.0385613,'PDT000124658.1':0.0420527)'':0.0171438)'':0.00598404)'':0.0380427)'':0.00459051)'':0.0116139)'':0.00896674)'':-0.00968389,((('PDT000171439.1':0.13204,('PDT000537838.1':0.109684,'PDT000168863.1':0.11635)'':0.00986006)'':7.10559e-05,(('PDT000232209.2':0.0894032,'PDT000612017.1':0.107143)'':0.0180668,(('PDT000672148.1':0.0824789,'PDT000017846.2':0.0626511)'':0.0201662,('PDT000411301.1':0.0062946,'PDT000411312.1':0.0039818)'':0.0751041)'':0.00759699)'':0.0181747)'':0.00142222,(('PDT000262162.2':0.0600593,'PDT000252473.1':0.0615057)'':0.0463029,(('PDT000292795.2':0.110326,('PDT000240290.1':0.096483,('PDT000350437.1':0.087657,('PDT000299961.4':0.0668942,'PDT000568772.1':0.0719658)'':0.014891)'':0.00718628)'':0.00831776)'':0.00152512,('PDT000390720.1':0.0686648,('PDT000468881.1':0.081615,('PDT000656959.1':0.0704417,('PDT000124714.1':0.0492969,'PDT000124715.1':0.0513121)'':0.0156643)'':0.00735789)'':0.00304301)'':0.0262517)'':0.00845124)'':0.00817985)'':0.00200948)'':0.0079905)'':0.00581428)'':0.000385548)'':0.00225543)'':0.00874417)'':0.0123969)'':0.00607191)'':0.00540516)'':0.00783368)'':-0.00515397,(((((('PDT000161610.3':0.0386734,'PDT000316840.2':0.0408886)'':0.012283,('PDT000254258.3':0.0434439,'PDT000491918.2':0.0501833)'':0.00491386)'':0.0296399,('PDT000588079.2':0.0770145,('PDT000253469.3':0.0478802,('PDT000161618.3':0.0410311,'PDT000520402.2':0.043346)'':0.00258341)'':0.0292031)'':0.00851059)'':0.072763,(('PDT000346093.1':0.110873,(('PDT000290371.4':0.0677414,('PDT000307130.2':0.060481,'PDT000251541.1':0.058419)'':0.0060291)'':0.0139258,(('PDT000417178.1':0.0846109,'PDT000502241.1':0.0828551)'':0.00213308,('PDT000409458.1':0.0750658,('PDT000675346.1':0.0662002,'PDT000669035.1':0.0627838)'':0.00788872)'':0.00798768)'':0.002668)'':0.00448482)'':0.00532112,(('PDT000417133.1':0.0726738,'PDT000290006.1':0.0803122)'':0.0142999,(('PDT000417185.1':0.04118,'PDT000346676.1':0.049465)'':0.0477315,(('PDT000417175.1':0.0894493,'PDT000621037.1':0.0729537)'':0.00371891,('PDT000433743.1':0.0440767,('PDT000417135.1':0.0295492,'PDT000219687.1':0.0311776)'':0.0168783)'':0.0174594)'':0.0104534)'':0.014718)'':0.00415422)'':0.0336307)'':0.0276332,(((('PDT000424779.1':0.0772815,('PDT000111708.2':0.0317204,'PDT000600950.1':0.0422222)'':0.0324692)'':0.00590611,('PDT000047724.3':0.0657426,('PDT000354864.1':0.0407387,('PDT000721346.1':0.015232,'PDT000252008.1':0.0170994)'':0.0131127)'':0.0371)'':0.0218059)'':0.0484248,((('PDT000188154.2':0.0611552,'PDT000426551.1':0.0754818)'':0.026323,('PDT000319001.1':0.102761,('PDT000661085.1':0.0581926,('PDT000422735.1':0.0221145,'PDT000577097.1':0.0208867)'':0.0319733)'':0.0351238)'':0.00508768)'':0.0318365,(('PDT000032352.1':0.00163751,'PDT000034401.1':0.00215604)'':0.120515,('PDT000235732.2':0.10722,(('PDT000095719.2':0.0467064,'PDT000146458.3':0.0705646)'':0.0293441,('PDT000709959.1':0.0787882,('PDT000544962.1':0.0441025,'PDT000644884.1':0.045418)'':0.0181871)'':0.0169826)'':0.0183944)'':0.000138443)'':0.0190316)'':0.00542824)'':0.0174737,((('PDT000233293.1':0.0824083,('PDT000475455.1':0.0519924,'PDT000535518.1':0.0707156)'':0.00988174)'':0.00868865,('PDT000463716.1':0.0678622,('PDT000571502.1':0.0634818,'PDT000251823.1':0.0642542)'':0.0055993)'':0.00767116)'':0.0493504,(((('PDT000181741.2':0.106675,'PDT000220631.1':0.0967692)'':0.0140017,('PDT000584529.1':0.100432,('PDT000537866.1':0.0877826,('PDT000242910.1':0.0713107,'PDT000463714.1':0.0625093)'':0.0075879)'':0.0133741)'':0.011061)'':0.0182266,(('PDT000393066.1':0.0338131,'PDT000548072.1':0.033668)'':0.0810912,('PDT000124657.1':0.0935391,(('PDT000648057.1':0.0759701,'PDT000708224.1':0.0679779)'':0.0155244,(('PDT000355694.1':0.0656954,'PDT000364396.1':0.0641576)'':0.017263,('PDT000708207.1':0.0755445,('PDT000069148.2':0.0631936,('PDT000519820.1':0.056757,(('PDT000380294.1':0.0370774,'PDT000557164.1':0.0358624)'':0.0205074,('PDT000642599.1':0.042472,'PDT000250216.1':0.0513568)'':0.00145068)'':0.00328431)'':0.00679496)'':0.00573645)'':0.0114666)'':0.00199376)'':0.00693994)'':0.0119928)'':0.0032406)'':0.000217604,((('PDT000716937.1':0.0736697,('PDT000621026.1':0.0758732,'PDT000710823.1':0.0765948)'':0.00547028)'':0.0561733,('PDT000081765.1':0.120484,(('PDT000285942.2':0.0851835,'PDT000250184.1':0.0650145)'':0.0124636,('PDT000652428.1':0.0887632,('PDT000405306.1':0.0698242,(('PDT000409864.1':0.0489346,'PDT000228130.1':0.0512384)'':0.00749488,('PDT000251850.1':0.0618711,('PDT000467020.1':0.0216814,'PDT000252605.1':0.0260412)'':0.0303541)'':0.00506398)'':0.00715869)'':0.00796856)'':0.000700372)'':0.013517)'':0.018299)'':-0.00629778,(((('PDT000317738.1':0.0865317,('PDT000711027.1':0.0454337,'PDT000014143.3':0.0607713)'':0.0492133)'':0.0253982,(('PDT000336241.1':0.076488,'PDT000250196.1':0.08413)'':0.00496065,(('PDT000539276.1':0.028046,'PDT000557788.1':0.0321868)'':0.0551154,('PDT000620268.1':0.0697941,'PDT000640115.1':0.0722999)'':0.00771393)'':0.0022891)'':0.0243398)'':-0.00418602,(('PDT000661119.1':0.107773,('PDT000389656.1':0.0799239,'PDT000358756.1':0.0830561)'':0.0115535)'':-0.00219088,('PDT000181757.2':0.116605,(('PDT000651133.1':0.0874931,('PDT000651073.1':0.0512791,'PDT000651080.1':0.0566509)'':0.0321254)'':0.0120565,((('PDT000298081.1':0.0591597,'PDT000414185.1':0.0747593)'':0.0447645,('PDT000228036.1':0.0794286,(('PDT000517280.1':0.0697466,'PDT000537860.1':0.0847424)'':0.00683809,('PDT000619967.1':0.0603901,('PDT000346644.1':0.0288126,'PDT000346651.1':0.0248675)'':0.0370179)'':0.00624044)'':0.00628057)'':0.0114504)'':-0.00371531,(('PDT000650991.1':0.0680244,('PDT000637070.1':0.0427357,'PDT000661118.1':0.0578633)'':0.0181406)'':0.0172433,(('PDT000604333.1':0.0529987,'PDT000228060.1':0.0607273)'':0.0215983,('PDT000017807.2':0.0614659,(('PDT000241673.2':0.021918,'PDT000417416.1':0.0241554)'':0.044943,('PDT000557288.1':0.050361,('PDT000389652.1':0.0516717,'PDT000587792.1':0.0507753)'':0.00561254)'':0.0116017)'':0.00101436)'':0.010453)'':0.00227015)'':0.00459168)'':0.0115368)'':0.0133008)'':-0.0112257)'':0.00370695)'':0.00279344,('PDT000168852.1':0.159587,(('PDT000124688.1':0.0655601,('PDT000503831.1':0.0469378,'PDT000124638.1':0.0409936)'':0.0139997)'':0.0869116,((('PDT000611992.1':0.0710218,'PDT000124594.1':0.0699672)'':0.0124941,(('PDT000181653.2':0.0568165,'PDT000394892.1':0.0500815)'':0.021844,('PDT000124653.1':0.0613542,(('PDT000397953.1':0.0402669,'PDT000719160.1':0.0359894)'':0.00558077,('PDT000228076.1':0.0511032,'PDT000345925.1':0.0544658)'':0.00102971)'':0.0103032)'':0.00758773)'':0.00499682)'':0.0196013,((('PDT000372320.1':0.0933633,'PDT000566059.1':0.0978287)'':0.0159809,(('PDT000216230.2':0.0847626,'PDT000497203.1':0.0813794)'':0.0101511,('PDT000181754.2':0.0819449,'PDT000240292.1':0.0880821)'':0.00479988)'':0.014606)'':0.00414678,('PDT000253437.2':0.12246,(('PDT000298054.1':0.073465,('PDT000319041.1':0.0650744,('PDT000687842.1':0.026241,'PDT000718332.1':0.0311842)'':0.0204075)'':0.0131701)'':0.0123303,(('PDT000088280.2':0.0787944,('PDT000566319.1':0.0670863,'PDT000566325.1':0.0573347)'':0.0103201)'':0.011298,('PDT000621021.1':0.0944399,('PDT000242945.1':0.091792,('PDT000346142.1':0.0863377,('PDT000150307.2':0.0788614,('PDT000088287.2':0.070967,('PDT000557282.1':0.0680173,((('PDT000389648.1':0.053419,'PDT000242905.1':0.059095)'':0.00410861,('PDT000242981.1':0.0383616,'PDT000250199.1':0.0356546)'':0.0186218)'':0.00304513,('PDT000239098.1':0.0622964,(('PDT000081771.1':0.0481518,'PDT000250247.1':0.048392)'':0.00477372,('PDT000346603.1':0.053086,('PDT000228055.1':0.0257602,'PDT000345984.1':0.0350393)'':0.0160164)'':0.00710524)'':0.00610777)'':-0.00212937)'':0.0096929)'':0.0031264)'':0.00109167)'':0.00557902)'':0.000720753)'':0.00362903)'':-0.00172197)'':0.00351115)'':0.0188404)'':-0.00982258)'':0.000273661)'':0.019402)'':0.027138)'':-0.0394374)'':0.00736546)'':0.00300405)'':0.0187265)'':0.0203134)'':0.00155084)'':0.00792221,(((('PDT000674944.1':0.0298129,('PDT000342263.1':0.0142961,'PDT000619084.1':0.0173256)'':0.00785346)'':0.112216,('PDT000463695.1':0.0989937,('PDT000081741.1':0.0996268,('PDT000437885.1':0.0688109,'PDT000652404.1':0.0650231)'':0.0368787)'':0.00632261)'':0.0170641)'':0.0440413,(('PDT000137332.2':0.119479,('PDT000649093.1':0.116992,('PDT000174303.1':0.0172864,'PDT000174310.1':0.010856)'':0.100299)'':0.00601024)'':0.00481436,(('PDT000322021.1':0.0887642,'PDT000155813.2':0.0902718)'':0.031688,('PDT000160772.2':0.115213,('PDT000537872.1':0.0960454,(('PDT000032218.1':0.00472877,('PDT000032190.1':0.00189891,'PDT000032402.1':0.00259166)'':0.00150178)'':0.0573713,('PDT000032329.1':0.00375033,('PDT000032341.1':0.00156184,'PDT000025987.1':0.00173322)'':0.000934077)'':0.0562498)'':0.0319315)'':0.00741765)'':0.0126706)'':0.00470256)'':0.0529842)'':0.00349064,(((('PDT000535012.1':0.131509,'PDT000661162.1':0.131342)'':0.00684061,('PDT000672230.1':0.121381,('PDT000496890.1':0.106341,('PDT000225863.2':0.0591934,'PDT000417938.1':0.0523126)'':0.0380149)'':0.0253738)'':0.0154799)'':0.0209833,((('PDT000200209.2':0.0531561,'PDT000206110.2':0.0575129)'':0.106172,('PDT000252705.1':0.118913,('PDT000249797.1':0.052509,('PDT000181795.2':0.0336975,'PDT000660643.1':0.0384825)'':0.0231835)'':0.0646885)'':0.0188264)'':-0.000733958,(('PDT000095794.2':0.0566344,'PDT000429525.1':0.0453416)'':0.0455029,(('PDT000129809.2':0.0752332,'PDT000015041.2':0.0708958)'':0.0195522,(('PDT000148124.2':0.0208616,'PDT000345208.1':0.0350372)'':0.0294162,('PDT000009353.2':0.0713063,('PDT000205113.2':0.0224028,'PDT000557245.1':0.0242456)'':0.027986)'':0.00211436)'':0.0235574)'':0.00562152)'':0.0441539)'':0.0159993)'':0.00448895,(('PDT000081277.1':0.0808388,'PDT000248727.1':0.0745572)'':0.0877823,(((('PDT000155860.2':0.0584101,('PDT000384032.1':0.0383879,('PDT000360328.1':0.0227531,'PDT000469876.1':0.0218522)'':0.0188707)'':0.0123497)'':0.0741397,(('PDT000718854.1':0.0449599,('PDT000553187.1':0.031833,('PDT000395000.1':0.0224967,'PDT000620255.1':0.0202888)'':0.0121306)'':0.0147189)'':0.0434727,(('PDT000626585.1':0.0963433,('PDT000131987.2':0.0362794,('PDT000625469.1':0.0334618,('PDT000157488.2':0.0167869,('PDT000132004.2':0.0130366,'PDT000718992.1':0.0105324)'':0.00194883)'':0.0146897)'':0.00846224)'':0.0469615)'':-0.00379785,(('PDT000081310.1':0.117046,('PDT000553171.1':0.0428553,'PDT000661100.1':0.0542691)'':0.0234233)'':0.00323813,('PDT000248726.1':0.0674814,('PDT000625479.1':0.0275965,(('PDT000156917.2':0.0141743,'PDT000132211.2':0.0124156)'':0.0128066,('PDT000553183.1':0.0198152,'PDT000016382.2':0.025011)'':0.00269622)'':0.00329576)'':0.0302907)'':0.0118554)'':0.00563648)'':0.00992093)'':0.0308956)'':-0.0165876,((('PDT000088264.2':0.0762376,(('PDT000322920.2':0.052258,('PDT000322802.1':0.0345416,'PDT000326237.1':0.033283)'':0.0164756)'':0.000653531,('PDT000299797.3':0.0346477,('PDT000580737.2':0.0146337,'PDT000139173.3':0.0169089)'':0.0162389)'':0.020873)'':0.0208127)'':0.0023492,('PDT000243703.2':0.0613847,('PDT000251829.1':0.055064,('PDT000276021.1':0.0584976,('PDT000060865.2':0.0419695,('PDT000471034.1':0.0464534,(('PDT000306638.2':0.0345973,'PDT000120909.2':0.0332124)'':0.00280926,(('PDT000149353.2':0.0234938,'PDT000199185.2':0.0206443)'':0.00826879,(('PDT000390817.1':0.0189449,'PDT000201852.2':0.018533)'':0.00652147,('PDT000199180.2':0.0294676,'PDT000199183.2':0.0266438)'':0.00178768)'':0.00414744)'':0.00569681)'':0.00242131)'':0.00322051)'':0.00833017)'':0.00523224)'':0.00371572)'':0.0163045)'':0.0124005,((('PDT000502366.1':0.042519,'PDT000502379.1':0.059873)'':0.0100504,(('PDT000016384.2':0.0352255,('PDT000199188.2':0.0241992,'PDT000625402.1':0.0248051)'':0.0130413)'':-0.000385334,('PDT000719263.1':0.0331376,('PDT000158024.2':0.0161681,('PDT000293418.2':0.0162531,('PDT000459499.2':0.0166497,'PDT000501955.1':0.014751)'':0.00466701)'':0.000449986)'':0.0108261)'':0.0069721)'':0.0128315)'':0.00172839,(('PDT000094201.2':0.0432266,'PDT000502359.1':0.046131)'':0.00748597,(('PDT000048140.1':0.0358217,(('PDT000602243.1':0.016219,'PDT000502355.1':0.0230395)'':0.00608656,('PDT000654698.1':0.0225817,'PDT000251916.1':0.0201299)'':0.00285664)'':0.00520538)'':-0.000845443,((('PDT000501590.2':0.0118236,'PDT000252966.3':0.0150934)'':0.00093196,('PDT000252963.3':0.0123303,'PDT000501703.2':0.0160637)'':0.00537967)'':0.0106387,('PDT000598390.2':0.0188568,('PDT000502371.1':0.015071,('PDT000151068.3':0.00922314,('PDT000253055.3':0.00812549,('PDT000162965.4':0.00403314,'PDT000289017.3':0.00414478)'':0.0085271)'':0.00159976)'':0.00169288)'':0.00240541)'':0.00527272)'':0.00733953)'':0.0142109)'':0.00785026)'':0.036377)'':0.0164797)'':0.0251219,((('PDT000548977.1':0.085232,'PDT000228166.1':0.093296)'':0.0551535,(('PDT000720194.1':0.117673,('PDT000095801.2':0.0650839,'PDT000346635.1':0.0795741)'':0.0424848)'':0.0168326,('PDT000346632.1':0.136995,('PDT000251394.1':0.114487,('PDT000181675.2':0.0804733,'PDT000233299.2':0.0846937)'':0.0368215)'':0.0215011)'':-0.00209812)'':0.00877803)'':0.00579078,(('PDT000534926.1':0.127081,('PDT000621043.1':0.129671,(('PDT000621000.1':0.0903278,('PDT000242907.1':0.0335846,'PDT000242936.1':0.0291827)'':0.053231)'':0.0208376,('PDT000642464.1':0.0857348,('PDT000399798.1':0.0921515,'PDT000412511.1':0.0700535)'':0.00454717)'':0.0075061)'':0.0389129)'':0.000710508)'':0.00392828,((('PDT000131999.2':0.121679,('PDT000429802.1':0.0952267,('PDT000060981.2':0.0341444,'PDT000641386.1':0.0370499)'':0.0522616)'':0.0253195)'':-0.000192743,('PDT000024638.1':0.0896971,('PDT000651776.1':0.0379496,('PDT000034420.1':0.0267438,('PDT000633640.1':0.0132887,'PDT000689225.1':0.0115677)'':0.0158733)'':0.0114343)'':0.0447671)'':0.0301947)'':0.0199748,((('PDT000346659.1':0.0599845,'PDT000459433.1':0.0654665)'':0.0590389,('PDT000305503.2':0.0943138,('PDT000621041.1':0.0853407,('PDT000417493.1':0.0717269,'PDT000571465.1':0.0548341)'':0.00843934)'':0.0117234)'':0.0167369)'':0.0188005,((('PDT000181834.2':0.122062,'PDT000395121.1':0.121237)'':0.0059392,('PDT000181832.2':0.118342,('PDT000233326.1':0.113704,('PDT000651025.1':0.0942793,'PDT000219686.1':0.0888537)'':0.00869542)'':0.00540963)'':0.0172509)'':0.00497269,((('PDT000200978.2':0.0899105,'PDT000426718.1':0.0869685)'':0.0330296,(('PDT000290010.1':0.129664,('PDT000346634.1':0.0984077,('PDT000435646.1':0.0944072,('PDT000417122.1':0.0883655,'PDT000233270.1':0.104927)'':0.00941684)'':0.0102703)'':0.00965743)'':0.00446507,(('PDT000417154.1':0.101683,('PDT000417094.1':0.104224,'PDT000168735.1':0.0961989)'':0.0147626)'':0.0149683,('PDT000621006.1':0.0862599,(('PDT000661178.1':0.056935,('PDT000621033.1':0.0415878,'PDT000708199.1':0.0478226)'':0.0084723)'':0.0291462,('PDT000417118.1':0.0833512,('PDT000417145.1':0.0599161,'PDT000414207.1':0.0565159)'':0.0129808)'':0.00159564)'':0.00918392)'':0.0285042)'':0.000598128)'':0.00476897)'':0.0003041,(('PDT000451152.2':0.0383346,'PDT000496981.2':0.037148)'':0.0996201,((('PDT000714648.1':0.0943794,('PDT000398141.1':0.0347514,'PDT000542482.1':0.0442525)'':0.0638481)'':0.0229774,((('PDT000633351.1':0.0349733,'PDT000346648.1':0.0363572)'':0.0172905,('PDT000632651.1':0.0556032,('PDT000346646.1':0.0374398,'PDT000346677.1':0.0397717)'':0.015455)'':0.00560051)'':0.0514425,(('PDT000018993.1':0.0856655,('PDT000149341.2':0.0482011,'PDT000088301.2':0.0591579)'':0.0257345)'':0.0196963,((('PDT000466995.1':0.0703035,'PDT000467012.1':0.0617315)'':0.0266461,('PDT000524063.1':0.0871056,('PDT000266390.2':0.0764845,'PDT000626915.1':0.0586405)'':0.0139899)'':0.00148925)'':0.00953843,((('PDT000667779.2':0.0796835,'PDT000621060.1':0.0792745)'':0.00983931,('PDT000620991.1':0.101189,('PDT000621039.1':0.0594982,'PDT000236047.1':0.0784198)'':0.0296741)'':0.000925662)'':0.00320865,('PDT000124624.1':0.11679,(('PDT000101291.1':0.0767753,('PDT000710703.1':0.0680599,('PDT000534978.1':0.0484313,'PDT000346649.1':0.0621087)'':0.0256491)'':0.00145003)'':0.0086283,('PDT000346547.1':0.101671,(('PDT000718893.1':0.0532182,'PDT000228171.1':0.0569428)'':0.0187349,('PDT000289906.1':0.0880951,('PDT000637325.1':0.0792767,((('PDT000346493.1':0.0573691,'PDT000346557.1':0.0766949)'':0.00706668,('PDT000221427.2':0.0610739,(('PDT000332725.1':0.0392586,'PDT000714685.1':0.0450192)'':0.00541736,('PDT000541634.1':0.0309147,'PDT000346636.1':0.048217)'':0.0156008)'':0.00676144)'':0.00193193)'':0.00270739,(('PDT000339381.1':0.0463358,'PDT000355845.1':0.041264)'':0.0261996,(('PDT000706395.1':0.0440471,'PDT000228299.1':0.0438787)'':0.0128145,('PDT000346530.1':0.0522189,('PDT000467008.1':0.0448032,'PDT000319025.1':0.037666)'':0.00626803)'':0.00153104)'':0.00980223)'':0.000525498)'':0.00706777)'':0.00450021)'':-0.00126836)'':0.00697048)'':-0.0034917)'':0.0120154)'':-0.00764759)'':0.00477557)'':0.00920791)'':-0.00338334)'':0.0158961)'':-0.0130733,((('PDT000395045.1':0.0223446,'PDT000566168.1':0.0250455)'':0.0787944,('PDT000027621.2':0.0708064,('PDT000518552.1':0.0646625,'PDT000550349.1':0.0501415)'':0.0216541)'':0.0253452)'':0.0122613,((('PDT000346060.1':0.101002,('PDT000291286.1':0.0794203,'PDT000346650.1':0.0830177)'':0.0146354)'':0.0215013,('PDT000168838.1':0.118469,(('PDT000626944.1':0.0784314,'PDT000252432.1':0.0716756)'':0.00957515,(('PDT000200984.2':0.0505652,'PDT000095729.2':0.0501148)'':0.0382566,(('PDT000535219.1':0.0719329,'PDT000081737.1':0.0714211)'':0.0143517,('PDT000200988.2':0.0774853,('PDT000201011.2':0.0376802,('PDT000498783.1':0.0204888,'PDT000688002.1':0.0269373)'':0.00887315)'':0.0367545)'':0.00960654)'':0.00823061)'':0.00203132)'':0.0138676)'':4.87446e-05)'':-0.0065083,(((('PDT000237132.1':0.0818415,('PDT000346678.1':0.0673052,'PDT000414192.1':0.0724818)'':0.014926)'':0.0111373,('PDT000411304.1':0.058028,('PDT000710702.1':0.0239958,('PDT000535011.1':0.027326,'PDT000675341.1':0.036116)'':0.00757682)'':0.00788319)'':0.0504283)'':0.00420926,(('PDT000032401.1':0.0772748,('PDT000032215.1':0.00490231,('PDT000032197.1':0.00396442,'PDT000032216.1':0.00311858)'':0.000372284)'':0.0682155)'':0.0125172,('PDT000346638.1':0.0590235,(('PDT000145161.2':0.0334224,'PDT000471006.1':0.0410272)'':0.00853728,('PDT000274855.2':0.0573553,('PDT000346560.1':0.0426594,'PDT000395390.1':0.042127)'':0.00754855)'':0.00136559)'':0.00497335)'':0.0188516)'':0.0167934)'':0.00140196,(('PDT000537531.1':0.117768,('PDT000331967.1':0.0768018,('PDT000423929.1':0.0877098,'PDT000626006.1':0.0608662)'':0.0136207)'':0.0238529)'':-0.00464073,((('PDT000225244.1':0.0492247,'PDT000346639.1':0.0371358)'':0.0488435,('PDT000233303.2':0.0987565,('PDT000652946.1':0.0961498,('PDT000409987.1':0.0791896,('PDT000178598.2':0.0604736,('PDT000346656.1':0.0410454,'PDT000346660.1':0.0393092)'':0.0208756)'':0.0146689)'':0.00364455)'':0.002254)'':-0.00116039)'':0.0188932,(('PDT000181744.2':0.106416,('PDT000346640.1':0.0747882,('PDT000181668.2':0.058453,(('PDT000225239.1':0.0341783,'PDT000250057.1':0.0320809)'':0.0177198,('PDT000346551.1':0.0239416,'PDT000346587.1':0.019447)'':0.0169135)'':0.0107647)'':0.0275079)'':0.0319187)'':-0.00497399,((('PDT000248764.1':0.0481948,'PDT000463719.1':0.0498821)'':0.0187724,('PDT000354196.1':0.0509141,'PDT000482867.1':0.0676839)'':0.0041726)'':0.0288103,(('PDT000550377.1':0.0827924,('PDT000652519.1':0.00652705,'PDT000652565.1':0.00720975)'':0.0722802)'':0.0110725,(('PDT000411295.1':0.0794794,('PDT000710706.1':0.0665585,'PDT000346675.1':0.0751695)'':0.0133721)'':-0.000943317,(('PDT000562851.1':0.080566,'PDT000346658.1':0.078733)'':0.0136807,('PDT000621035.1':0.0602767,'PDT000227987.1':0.0809303)'':0.0332445)'':0.00301642)'':0.00254795)'':0.00406373)'':0.00230886)'':0.000926255)'':0.00312606)'':-0.00253108)'':0.002783)'':0.00661039)'':-0.00457432)'':0.016153)'':-0.010911)'':0.00831027)'':0.00112954)'':0.00215121)'':0.00469772)'':0.0205612)'':-0.0185353)'':0.0143181)'':0.015388)'':0.000670243)'':0.00218631)'':0.00464464)'':0.00356139)'':0.0010676,((((((('PDT000250025.1':0.13305,('PDT000129100.2':0.108999,('PDT000167700.2':0.0780748,('PDT000307417.2':0.0636425,'PDT000319049.1':0.0456365)'':0.0197527)'':0.010683)'':0.0232397)'':0.0069287,(('PDT000360982.1':0.0925048,('PDT000652886.1':0.0403929,'PDT000144344.1':0.053548)'':0.0318473)'':0.0595876,(('PDT000557128.1':0.115726,('PDT000550398.1':0.0614896,('PDT000586227.1':0.0389104,'PDT000298083.1':0.0406307)'':0.0254373)'':0.0425624)'':-0.00339996,(('PDT000243982.2':0.0169645,'PDT000651794.1':0.0186935)'':0.0251924,('PDT000523262.1':0.0357826,('PDT000586490.1':0.0196604,'PDT000042366.1':0.0224283)'':0.00236156)'':0.00689053)'':0.0458519)'':0.0224298)'':0.0047365)'':0.0330048,(('PDT000538549.1':0.0819204,(('PDT000346611.1':0.0345619,('PDT000535461.1':0.0367438,'PDT000661441.1':0.0283757)'':0.00366582)'':0.0426782,(('PDT000661783.1':0.0589347,'PDT000015332.2':0.0561293)'':0.00885008,('PDT000501872.1':0.057941,('PDT000610694.1':0.0126669,'PDT000661218.1':0.0183162)'':0.0370029)'':0.00805777)'':0.0151613)'':0.00389225)'':0.042479,((('PDT000443078.1':0.108465,(('PDT000213269.2':0.078376,'PDT000251750.1':0.068058)'':0.0180612,('PDT000213274.2':0.0587393,'PDT000017870.2':0.0703207)'':0.0179453)'':0.0153479)'':0.0100783,(('PDT000251942.1':0.0735245,('PDT000130901.2':0.0680905,'PDT000425686.1':0.0767645)'':0.0151355)'':0.0282292,('PDT000636316.1':0.0821357,('PDT000132405.2':0.0565734,'PDT000716979.1':0.0450386)'':0.0378558)'':0.0189284)'':0.0138248)'':0.00889145,((('PDT000500701.1':0.0573891,'PDT000346830.1':0.0527189)'':0.0455996,(('PDT000305440.2':0.0909495,('PDT000425529.1':0.0482321,'PDT000713757.1':0.0483937)'':0.0427076)'':-0.0018664,(('PDT000425993.1':0.0735721,'PDT000496896.1':0.0946729)'':0.0178686,('PDT000661557.1':0.0504062,('PDT000240664.2':0.0556159,'PDT000650534.1':0.0540211)'':0.00625127)'':0.0209856)'':0.0138162)'':0.00477322)'':0.00719821,(('PDT000253662.1':0.0605879,('PDT000667722.2':0.0329016,'PDT000720317.1':0.0324943)'':0.0253752)'':0.0238472,('PDT000174298.1':0.0855607,('PDT000213264.2':0.0665479,(((('PDT000422536.1':0.0190693,'PDT000429774.1':0.0230349)'':0.00755257,('PDT000429662.1':0.0301293,('PDT000428858.1':0.0387374,('PDT000428913.1':0.0167003,('PDT000428856.1':0.00387698,'PDT000428906.1':0.00339973)'':0.021233)'':0.0079523)'':0.00236874)'':0.00726384)'':0.020817,('PDT000251976.1':0.0529231,('PDT000316542.1':0.0474025,('PDT000402759.1':0.036505,('PDT000662574.1':0.024653,('PDT000261028.1':0.0254052,('PDT000174302.1':0.0223888,('PDT000602735.1':0.0122705,'PDT000662582.1':0.0182532)'':0.00432372)'':0.00574355)'':0.00253033)'':0.00294889)'':0.0114362)'':0.00852921)'':6.00514e-05)'':0.00353504,((('PDT000261034.1':0.0438957,('PDT000213248.2':0.0278946,'PDT000261035.1':0.0260935)'':0.0157044)'':0.00254463,(('PDT000213204.2':0.00839552,'PDT000662560.1':0.0100899)'':0.0215134,('PDT000015748.3':0.0187092,('PDT000151013.2':0.0137764,'PDT000014475.2':0.0161009)'':0.00193644)'':0.0234283)'':0.0131368)'':0.00245093,(('PDT000618552.1':0.0343931,('PDT000662577.1':0.0232356,('PDT000625498.1':0.0307467,('PDT000541577.1':0.0209408,('PDT000469859.1':0.00655836,'PDT000599890.1':0.00672044)'':0.0123297)'':0.00273977)'':0.00163672)'':0.00776281)'':0.00566844,('PDT000334526.1':0.044954,(('PDT000625421.1':0.0200636,'PDT000640373.1':0.0146373)'':0.00519199,('PDT000599882.1':0.0249147,('PDT000336805.1':0.0195138,('PDT000156030.2':0.00521457,'PDT000399951.1':0.00804173)'':0.0184538)'':0.00191339)'':0.00508543)'':0.0136627)'':-0.00317753)'':0.0139439)'':0.00642103)'':0.00834206)'':0.0165644)'':0.00865847)'':0.0222441)'':0.00539738)'':0.00872856)'':0.0145912)'':0.00280192,((('PDT000200990.2':0.128938,('PDT000417119.1':0.121189,('PDT000291285.1':0.0992964,'PDT000346869.1':0.0949526)'':0.0222107)'':0.00977638)'':0.0152801,(('PDT000468894.1':0.0792941,('PDT000524503.1':0.0696003,'PDT000380901.1':0.0800827)'':0.00370588)'':0.0525712,('PDT000329064.1':0.139734,('PDT000346561.1':0.108562,(('PDT000181793.2':0.101364,'PDT000171425.1':0.106297)'':0.013506,('PDT000648089.1':0.101409,('PDT000373302.1':0.100832,(('PDT000309382.2':0.0716802,'PDT000557739.1':0.0579348)'':0.00846109,('PDT000420662.1':0.054547,'PDT000289896.1':0.064426)'':0.0223399)'':0.0188104)'':0.0110465)'':-0.00285492)'':0.023179)'':0.0305241)'':-0.0205424)'':0.0108094)'':0.00171838,(('PDT000329816.1':0.152724,(('PDT000551917.1':0.0831266,('PDT000271524.2':0.0368475,'PDT000550394.1':0.0369838)'':0.0629742)'':0.0271093,(('PDT000095820.2':0.0751613,('PDT000395393.1':0.0452168,'PDT000395217.1':0.04438)'':0.0257893)'':0.00156103,('PDT000500522.1':0.0533196,('PDT000394965.1':0.0357932,('PDT000095772.2':0.0271558,'PDT000565960.1':0.024543)'':0.0132073)'':0.0145408)'':0.021175)'':0.050607)'':0.0271854)'':0.0139154,((('PDT000556999.1':0.1119,('PDT000200985.2':0.114439,('PDT000217107.2':0.0798473,'PDT000432474.1':0.0693567)'':0.0308965)'':0.00872042)'':0.00526176,('PDT000625978.1':0.100193,('PDT000495098.1':0.079352,('PDT000095745.2':0.0587399,('PDT000323520.1':0.0686484,'PDT000275551.1':0.0615696)'':0.00156764)'':0.017343)'':0.0132596)'':0.0219838)'':0.000997351,('PDT000042551.1':0.122279,(((('PDT000243986.2':0.0275552,'PDT000663938.1':0.0234762)'':0.020065,('PDT000243984.2':0.0467525,('PDT000063355.2':0.00372444,'PDT000189443.1':0.00395942)'':0.0414953)'':0.00419449)'':0.0282016,('PDT000661098.1':0.0936768,((('PDT000324658.1':0.0115638,('PDT000415198.1':0.0105782,('PDT000010916.3':0.0090954,('PDT000243978.2':0.00602252,'PDT000015760.2':0.00977798)'':0.00291505)'':0.00220128)'':0.00200308)'':0.0169395,('PDT000261029.1':0.0459907,('PDT000020376.1':0.025886,(('PDT000048016.1':0.00402611,'PDT000325291.1':0.00384634)'':0.000979129,('PDT000244034.2':0.00212101,'PDT000399905.1':0.00217041)'':0.00310236)'':0.0127067)'':0.00791622)'':-0.000107422)'':0.0247761,(('PDT000322827.1':0.0588144,('PDT000316937.1':0.0452098,('PDT000015755.3':0.0208995,('PDT000015749.3':0.00913155,'PDT000015742.3':0.0101106)'':0.0109477)'':0.0230909)'':0.0134047)'':0.0123271,('PDT000261046.1':0.0451629,(('PDT000250138.1':0.03431,('PDT000120849.2':0.0292354,('PDT000244026.2':0.0320876,('PDT000662562.1':0.0197614,'PDT000662564.1':0.0280931)'':0.00921975)'':0.00158346)'':0.0059844)'':-0.00124914,(('PDT000323339.1':0.0293294,'PDT000635777.1':0.0331338)'':0.00123825,('PDT000345905.1':0.0284899,(('PDT000244023.2':0.0206497,'PDT000261036.1':0.0142386)'':0.00667785,(('PDT000390171.1':0.00849907,'PDT000168489.1':0.0111418)'':0.0162201,('PDT000236904.1':0.019769,('PDT000244027.2':0.00450958,'PDT000663922.1':0.00544067)'':0.011896)'':0.00143361)'':0.0025569)'':0.00407315)'':0.00366119)'':0.00264199)'':0.0137302)'':0.0101467)'':0.000135039)'':0.0496567)'':-0.0314561)'':0.00176134,((('PDT000243977.2':0.0289494,'PDT000300831.2':0.0314165)'':0.00422784,('PDT000243981.2':0.027871,('PDT000327320.1':0.0292453,('PDT000156912.2':0.0282323,('PDT000333266.1':0.0213646,('PDT000244024.2':0.0233284,(('PDT000309649.2':0.0094933,'PDT000386171.1':0.0079826)'':0.00424889,('PDT000662569.1':0.0147825,('PDT000243990.2':0.0103929,('PDT000151385.1':0.00791205,('PDT000244038.2':0.00782689,('PDT000319895.1':0.00799198,('PDT000244032.2':0.00706529,'PDT000120862.2':0.00457421)'':0.000962918)'':0.000420107)'':0.000805943)'':0.00290866)'':0.00147382)'':0.000289873)'':0.00256317)'':0.00874531)'':0.0035972)'':0.00508822)'':0.00135535)'':0.00186747)'':0.0151219,((('PDT000014142.2':0.0185277,('PDT000389393.1':0.0117688,'PDT000716341.1':0.0120737)'':0.0080566)'':0.0258725,('PDT000261043.1':0.0431202,(('PDT000034419.1':0.0241566,'PDT000174320.1':0.0196798)'':0.0143129,('PDT000244030.2':0.0354408,('PDT000323346.1':0.026509,(('PDT000015730.3':0.0144924,'PDT000014106.3':0.013521)'':0.00109945,('PDT000015872.2':0.0135827,('PDT000015745.3':0.0138512,'PDT000015724.2':0.0147457)'':0.00209248)'':0.00069956)'':0.0163923)'':0.00891369)'':0.000370262)'':0.0112658)'':-0.00171831)'':-0.000994103,(('PDT000250124.1':0.0281241,'PDT000250135.1':0.0304586)'':0.00946725,('PDT000244029.2':0.0550392,(('PDT000015738.3':0.0361348,('PDT000243989.2':0.0307072,('PDT000244039.2':0.0233636,('PDT000625473.1':0.0195965,'PDT000662566.1':0.0211857)'':0.00351995)'':0.00453167)'':0.00380941)'':4.41224e-05,('PDT000651393.1':0.0205265,(('PDT000668183.2':0.0199956,('PDT000316505.1':0.0122613,'PDT000318688.1':0.0129488)'':0.00521313)'':-0.000365062,('PDT000377963.1':0.0155385,('PDT000551619.1':0.00661204,'PDT000718629.1':0.00573856)'':0.0111371)'':0.00590887)'':0.00455379)'':0.00952447)'':0.0122927)'':-0.00170792)'':0.00119598)'':0.00889051)'':0.0299373)'':0.0488281)'':0.00399856)'':0.0117805)'':0.00446607)'':0.00444767)'':0.00253042,(((('PDT000604155.1':0.012594,'PDT000632758.1':0.0161479)'':0.0917951,(('PDT000625121.1':0.0457884,('PDT000131829.2':0.0268401,'PDT000622842.1':0.037765)'':0.0286751)'':0.032512,('PDT000131843.2':0.0563659,('PDT000414383.1':0.0522374,('PDT000629339.1':0.0228343,('PDT000157241.2':0.018077,('PDT000295877.2':0.0160987,('PDT000319875.1':0.0058262,'PDT000132350.2':0.0062292)'':0.00956662)'':0.0041053)'':0.00823195)'':0.0075658)'':0.0270548)'':0.0157714)'':0.0418034)'':0.0221475,((('PDT000242979.1':0.0793861,('PDT000430840.1':0.0852705,'PDT000557327.1':0.0921055)'':0.00346795)'':0.0295406,(('PDT000537868.1':0.0630644,'PDT000645050.1':0.0658786)'':0.0387182,(('PDT000651044.1':0.0783871,'PDT000651128.1':0.0675609)'':0.00340758,('PDT000653445.1':0.0477045,('PDT000323540.1':0.0316631,'PDT000566143.1':0.0329967)'':0.00413021)'':0.0246153)'':0.0442272)'':0.00866342)'':0.00159275,(('PDT000497223.1':0.0811591,('PDT000167664.2':0.0627285,'PDT000628171.1':0.0729545)'':0.00825786)'':0.031922,((('PDT000168868.1':0.0745651,'PDT000289881.1':0.0643509)'':0.0473517,(('PDT000616407.1':0.0823219,'PDT000294376.1':0.0806881)'':0.0049016,('PDT000346841.1':0.0887206,('PDT000168807.1':0.08693,('PDT000248836.1':0.0536564,'PDT000313447.1':0.0649506)'':0.0185495)'':0.0058464)'':0.0124249)'':0.0201039)'':0.00568685,(('PDT000616414.1':0.0826512,'PDT000289884.1':0.103115)'':0.0143219,(('PDT000397917.1':0.0363814,'PDT000494258.1':0.0422443)'':0.0604329,('PDT000188367.1':0.0829138,('PDT000425969.1':0.0894061,('PDT000034520.2':0.0615073,('PDT000108222.2':0.0362457,'PDT000346880.1':0.0469908)'':0.0173885)'':0.0145547)'':0.006286)'':0.00907994)'':0.00885684)'':0.00537477)'':0.0113884)'':-0.00216433)'':0.0135327)'':0.00312081,(((('PDT000438120.1':0.0255837,'PDT000443070.1':0.0316855)'':0.0287872,('PDT000661564.1':0.0277181,('PDT000426095.1':0.0184568,'PDT000426096.1':0.0172575)'':0.00600814)'':0.0345367)'':0.0741576,((('PDT000306432.2':0.0582799,'PDT000720478.1':0.0681071)'':0.0109958,(('PDT000572629.1':0.0546585,'PDT000710825.1':0.0658265)'':0.0131851,('PDT000661153.1':0.0624401,('PDT000534941.1':0.0369081,'PDT000346845.1':0.0461089)'':0.00789294)'':0.0129019)'':0.0088723)'':0.0438546,((('PDT000168804.1':0.125409,'PDT000236052.1':0.126218)'':0.0131557,(('PDT000661079.1':0.101521,'PDT000289882.1':0.110831)'':0.00687538,('PDT000645608.2':0.116454,('PDT000661180.1':0.104444,(('PDT000537834.1':0.0863407,'PDT000346850.1':0.0844603)'':0.00294959,('PDT000181469.4':0.0874604,('PDT000621042.1':0.0839063,'PDT000463629.1':0.0819197)'':0.00311361)'':0.00388687)'':0.00939234)'':0.00457502)'':0.0110783)'':0.00604288)'':0.00656692,(('PDT000252604.1':0.136954,(('PDT000196629.2':0.107445,('PDT000298018.1':0.0686654,('PDT000128792.2':0.0175874,'PDT000128832.2':0.020946)'':0.0587324)'':0.0223247)'':0.00591838,(('PDT000535142.1':0.0785234,('PDT000535067.1':0.051403,'PDT000535236.1':0.0309056)'':0.0403963)'':0.0177497,('PDT000095711.2':0.0902248,('PDT000397877.1':0.0832959,('PDT000303023.2':0.0642859,'PDT000646924.1':0.0888371)'':0.00491911)'':0.00347115)'':0.00463138)'':0.00717579)'':0.0220437)'':-0.0169733,(('PDT000168789.1':0.104751,'PDT000270575.1':0.0868328)'':0.0705964,(((('PDT000425517.1':0.0983302,('PDT000369034.1':0.0918261,('PDT000714093.1':0.0842881,'PDT000661625.1':0.0785649)'':0.00395291)'':0.0127458)'':0.00624136,('PDT000171828.1':0.113019,('PDT000571133.1':0.103914,('PDT000352549.1':0.0898778,(('PDT000346849.1':0.0594318,('PDT000167653.2':0.0463521,'PDT000451101.1':0.0368594)'':0.0127149)'':0.02163,('PDT000285362.2':0.0628975,('PDT000203149.2':0.0591635,'PDT000468834.1':0.0534515)'':0.019603)'':0.00943964)'':0.00931745)'':0.00210267)'':0.0130154)'':-0.00994378)'':0.00101906,(('PDT000168819.1':0.0858972,('PDT000662399.1':0.0362197,'PDT000661518.1':0.0301158)'':0.0290566)'':0.0357772,(('PDT000645112.1':0.0768825,('PDT000024325.2':0.060643,'PDT000556984.1':0.067027)'':0.00821653)'':0.0134525,(('PDT000652445.1':0.0827771,'PDT000411310.1':0.0792669)'':0.00586773,(('PDT000251670.1':0.0570518,'PDT000463649.1':0.0684192)'':0.00599336,(('PDT000216273.2':0.057396,'PDT000217118.2':0.063377)'':0.0139969,('PDT000251673.1':0.0823232,'PDT000285337.1':0.0674438)'':0.00248159)'':0.00705839)'':0.0038598)'':0.00492627)'':0.00540291)'':0.00267936)'':0.00409369,(('PDT000168809.1':0.0793923,('PDT000621036.1':0.0232227,'PDT000459421.1':0.0381495)'':0.0318386)'':0.0357656,('PDT000621030.1':0.10141,(('PDT000124641.1':0.0956235,('PDT000346839.1':0.0992846,('PDT000242490.1':0.090797,('PDT000632507.1':0.0725439,(('PDT000146003.2':0.0424314,'PDT000621051.1':0.0506211)'':0.0130795,('PDT000150253.2':0.0462685,'PDT000248492.1':0.050025)'':0.00644172)'':0.0172505)'':0.00568297)'':0.00671926)'':0.00220535)'':-0.00499249,((('PDT000621058.1':0.0687163,'PDT000124613.1':0.0667977)'':0.0220073,(('PDT000298025.1':0.0663529,('PDT000621040.1':0.061912,'PDT000346862.1':0.065211)'':0.0105941)'':0.000568204,('PDT000537847.1':0.0680253,('PDT000181800.2':0.0609012,'PDT000417147.1':0.0623198)'':0.00590773)'':0.0095654)'':0.00672855)'':0.0052461,(('PDT000720247.1':0.0826237,'PDT000168815.1':0.0876573)'':0.00501958,('PDT000491106.1':0.0847098,('PDT000505380.1':0.0706891,(('PDT000372026.1':0.0590994,'PDT000538595.1':0.0537576)'':0.00653112,('PDT000720259.1':0.0495018,'PDT000435621.1':0.0499616)'':0.0124229)'':0.00521656)'':0.00181119)'':0.013157)'':0.000712514)'':0.00275859)'':0.015897)'':-0.005689)'':0.00985911)'':0.0314033)'':-0.0256081)'':0.00406227)'':0.00810325)'':0.00791582)'':-0.00549631,(((('PDT000592115.1':0.120039,'PDT000346040.1':0.113701)'':0.0226507,(('PDT000233320.1':0.110122,('PDT000399342.1':0.0879592,('PDT000233286.1':0.0924656,('PDT000294377.1':0.0337628,'PDT000346604.1':0.0304439)'':0.0469241)'':0.0169647)'':0.0137315)'':0.0039421,('PDT000242913.1':0.133077,((('PDT000718934.1':0.0681893,('PDT000111361.1':0.0379607,('PDT000425846.1':0.0222362,'PDT000661388.1':0.0254155)'':0.00828797)'':0.0411659)'':-0.00169823,(('PDT000172358.2':0.078755,'PDT000520775.1':0.070149)'':0.00910221,('PDT000566053.1':0.0540372,('PDT000718954.1':0.0471304,('PDT000129845.2':0.0307218,'PDT000690064.1':0.0404814)'':0.00868072)'':0.00645136)'':0.00888607)'':0.0014334)'':0.0241129,(('PDT000621070.1':0.121217,('PDT000497430.1':0.075186,('PDT000386739.1':0.0411696,'PDT000468920.1':0.0451875)'':0.021388)'':0.0370168)'':-0.000199439,('PDT000661268.1':0.101643,(('PDT000565900.1':0.0701678,'PDT000573625.1':0.0800142)'':0.0053925,(('PDT000242964.1':0.0905354,(('PDT000305442.2':0.0545342,'PDT000051493.2':0.0509848)'':0.0119091,('PDT000233045.1':0.0458191,('PDT000251432.1':0.0215137,'PDT000491095.1':0.0181)'':0.0269042)'':0.00883786)'':0.00959824)'':0.00468594,(('PDT000661288.1':0.0828379,('PDT000663985.1':0.0563863,('PDT000651100.1':0.0235898,'PDT000212802.1':0.0268148)'':0.0306489)'':0.0224588)'':-0.00288761,('PDT000497313.1':0.0555004,(('PDT000095821.2':0.0393669,'PDT000471003.1':0.0358749)'':0.0116935,('PDT000218935.1':0.0408363,'PDT000463660.1':0.042321)'':0.00524275)'':0.014074)'':0.00752034)'':0.00193097)'':0.0120415)'':0.0126269)'':-0.00349223)'':0.000189481)'':0.0224401)'':-0.0134901)'':0.0193421)'':-0.0119008,(('PDT000651231.1':0.0720373,'PDT000369026.1':0.0706717)'':0.0719695,(('PDT000502234.1':0.112906,(('PDT000616496.1':0.0767048,'PDT000252716.1':0.0648392)'':0.0304053,((('PDT000675469.1':0.0683881,'PDT000354189.1':0.0488389)'':0.0333512,('PDT000714692.1':0.064082,('PDT000191850.2':0.0489361,'PDT000131766.2':0.0593879)'':0.00576301)'':0.0299506)'':0.0109222,(('PDT000309323.2':0.0694596,'PDT000171427.1':0.0852304)'':0.0195678,('PDT000494196.1':0.0798378,('PDT000537813.1':0.0867578,('PDT000357283.1':0.0466891,'PDT000538617.1':0.0437324)'':0.0180369)'':0.00944609)'':0.00623502)'':0.0118025)'':0.00677779)'':0.0104918)'':-0.00261152,(('PDT000287993.1':0.123766,('PDT000436729.1':0.0997637,('PDT000661097.1':0.0813757,('PDT000621063.1':0.049296,'PDT000623164.1':0.0449368)'':0.0358834)'':0.0141245)'':0.0226423)'':-0.00846864,(('PDT000312790.2':0.0603907,'PDT000051388.2':0.0453613)'':0.0453482,(('PDT000488291.1':0.0629202,('PDT000145186.2':0.0339154,'PDT000425881.1':0.0427808)'':0.0137017)'':0.0281112,(('PDT000221066.1':0.0929195,'PDT000261030.1':0.0891825)'':0.00156363,(('PDT000045606.1':0.0683917,('PDT000124706.1':0.0525592,'PDT000395006.1':0.0536778)'':0.0091763)'':0.00742657,('PDT000662570.1':0.0866072,('PDT000174383.1':0.0329138,('PDT000662555.1':0.0248815,('PDT000015778.3':0.0136924,('PDT000015787.3':0.0141614,(('PDT000015837.2':0.0116845,('PDT000015825.3':0.0114155,'PDT000015842.3':0.00968202)'':0.000568792)'':0.000178557,('PDT000015829.3':0.01105,('PDT000015845.3':0.0111868,('PDT000015819.3':0.0109931,('PDT000015843.3':0.0107868,(('PDT000014141.3':0.00599148,'PDT000015841.3':0.00627542)'':0.00109128,(('PDT000015815.3':0.00862297,'PDT000015807.3':0.00883393)'':0.000305606,('PDT000015840.3':0.00960192,'PDT000015798.3':0.00973568)'':0.000474144)'':0.00015102)'':0.000334842)'':0.000229386)'':5.62765e-05)'':0.000373365)'':-0.00029535)'':0.00162084)'':0.000259103)'':0.0139618)'':0.00720631)'':0.0312168)'':0.0121789)'':0.00698627)'':0.00512157)'':0.00358859)'':0.00596655)'':0.0158969)'':0.0143242)'':-0.0139786)'':0.00190949,(('PDT000181759.2':0.130536,('PDT000443061.1':0.132167,(('PDT000712624.1':0.14879,('PDT000168836.1':0.119896,('PDT000181749.2':0.119993,('PDT000565978.1':0.0809223,(('PDT000252739.1':0.0717003,('PDT000661150.1':0.0535151,('PDT000632804.1':0.0338754,'PDT000373301.1':0.0272226)'':0.0171842)'':0.0314161)'':0.00838108,(('PDT000181786.2':0.046355,'PDT000651551.1':0.057495)'':0.0226225,(('PDT000556864.1':0.0557361,'PDT000252551.1':0.0484029)'':0.00442398,('PDT000124670.1':0.0557432,'PDT000346958.1':0.0616428)'':0.00584927)'':0.0108408)'':0.00320693)'':0.0122806)'':0.0117475)'':0.010634)'':0.00399667)'':-0.00610354,('PDT000252725.1':0.137336,(('PDT000132342.2':0.0843941,'PDT000346854.1':0.0761229)'':0.0278967,((('PDT000294680.2':0.0697922,('PDT000095763.2':0.0743166,'PDT000346835.1':0.0675114)'':0.00549777)'':0.016378,('PDT000346945.1':0.079953,('PDT000618733.1':0.0869458,('PDT000494437.1':0.0480276,'PDT000251939.1':0.0565964)'':0.0236932)'':0.00643292)'':0.00549643)'':0.00929867,((('PDT000290531.2':0.0669724,'PDT000384905.1':0.0681706)'':0.0109336,('PDT000617799.1':0.0672208,'PDT000584742.1':0.0628532)'':0.00688988)'':0.00794847,('PDT000301016.2':0.0982649,(('PDT000551888.1':0.0716913,'PDT000661674.1':0.0709917)'':0.00497128,('PDT000689819.1':0.0194274,'PDT000250176.1':0.0170263)'':0.0583341)'':0.0131454)'':-0.00118835)'':0.00503035)'':0.00783837)'':0.0191922)'':-0.0202632)'':0.0136312)'':0.00941178)'':-0.00640046,(('PDT000346074.1':0.14601,((('PDT000258726.2':0.0835281,('PDT000631542.1':0.0760835,'PDT000639817.1':0.0588125)'':0.0100589)'':0.00222699,('PDT000354188.1':0.0501616,('PDT000346943.1':0.0506412,'PDT000373310.1':0.0403119)'':0.0112999)'':0.0115253)'':0.0311139,('PDT000264660.2':0.141426,(('PDT000178563.2':0.0899959,('PDT000497340.1':0.0539845,'PDT000632005.1':0.0441149)'':0.0387239)'':0.00829906,('PDT000651077.1':0.0733277,('PDT000656703.1':0.0646158,'PDT000353624.1':0.0690382)'':0.00608377)'':0.0192898)'':0.0324858)'':-0.0186037)'':0.0370202)'':-0.0121475,('PDT000639591.1':0.16965,(('PDT000409996.1':0.104291,('PDT000034470.2':0.116728,'PDT000645011.1':0.104006)'':0.00450507)'':0.0190635,(('PDT000497326.1':0.130008,('PDT000281474.2':0.105033,'PDT000095768.2':0.109611)'':0.0141118)'':0.00371016,((('PDT000280705.2':0.120595,(('PDT000129898.2':0.0940429,('PDT000124717.1':0.0294894,'PDT000354215.1':0.024173)'':0.0594579)'':0.00845098,('PDT000668480.2':0.0922226,('PDT000289924.1':0.0684441,('PDT000242973.1':0.0673218,('PDT000032195.1':0.0459123,'PDT000637077.1':0.0465855)'':0.0176108)'':0.00419112)'':0.0155883)'':0.0179401)'':0.007547)'':0.00219007,('PDT000435620.1':0.131158,((('PDT000321634.1':0.0924987,'PDT000188365.1':0.0824053)'':0.0070194,('PDT000216314.2':0.0909261,'PDT000191358.3':0.0753659)'':0.0180406)'':0.00798598,(('PDT000403583.1':0.0649979,('PDT000397768.1':0.0483502,'PDT000008589.2':0.0428806)'':0.0160867)'':0.0458299,('PDT000544820.1':0.0838969,('PDT000501058.1':0.0661598,'PDT000537953.1':0.0700392)'':0.0234651)'':0.00932845)'':0.00484426)'':0.0120449)'':-0.00570544)'':0.00198101,((('PDT000346874.1':0.0772277,('PDT000292814.2':0.0312293,('PDT000477098.1':0.0268815,'PDT000610362.1':0.0244558)'':0.0151491)'':0.0406427)'':0.0386764,('PDT000621047.1':0.132315,('PDT000463673.1':0.0841512,('PDT000395012.1':0.0858789,('PDT000557352.1':0.0430277,'PDT000403084.1':0.0473932)'':0.0218027)'':0.0240115)'':0.0269556)'':-0.00988413)'':0.0100135,((('PDT000412043.1':0.09339,('PDT000212789.1':0.0956109,('PDT000687688.1':0.0689014,'PDT000468839.1':0.0753106)'':0.0284821)'':0.00526896)'':0.00859366,(('PDT000346859.1':0.0815922,('PDT000228359.1':0.0382982,'PDT000298091.1':0.0381263)'':0.0372036)'':0.0104072,('PDT000675421.1':0.10043,(('PDT000362836.1':0.0737242,'PDT000652176.1':0.0768448)'':0.00607771,('PDT000652832.1':0.0900482,('PDT000426535.1':0.0672654,'PDT000429812.1':0.0693066)'':0.00921728)'':0.010328)'':0.00273483)'':0.00617141)'':0.00418005)'':0.0133073,((('PDT000394941.1':0.0843596,'PDT000616405.1':0.0847674)'':0.0231139,(('PDT000003949.2':0.0817516,'PDT000289914.1':0.0804434)'':0.0167819,('PDT000285384.2':0.10852,('PDT000565944.1':0.106837,('PDT000621032.1':0.0930178,'PDT000395261.1':0.0705602)'':0.0139103)'':0.00614101)'':-3.01419e-05)'':0.00449294)'':0.00275893,(('PDT000251461.1':0.123797,('PDT000535092.1':0.106634,'PDT000063651.1':0.0997992)'':0.00501298)'':0.00597688,('PDT000241674.2':0.116804,(('PDT000558433.1':0.0654619,('PDT000346884.1':0.0576606,('PDT000298060.1':0.0580303,('PDT000557783.2':0.0285331,'PDT000626011.1':0.0260372)'':0.0164076)'':0.00607691)'':0.0160556)'':0.0228756,(('PDT000639572.1':0.0641905,'PDT000463718.1':0.0662735)'':0.0225444,(('PDT000399698.1':0.0318771,'PDT000395339.1':0.0313613)'':0.0437447,('PDT000181750.2':0.0661136,('PDT000047676.3':0.0315302,'PDT000025734.2':0.0215994)'':0.0295681)'':0.0154962)'':0.0133869)'':0.00131282)'':0.011655)'':0.00112272)'':-0.00225177)'':0.0011115)'':0.00366204)'':0.000266704)'':0.00490671)'':0.00192368)'':0.0103543)'':-0.0111883)'':0.000401019)'':0.000235529)'':0.00457172)'':0.00401049)'':0.0100659)'':0.0161205,((('PDT000645071.1':0.153795,('PDT000298048.1':0.0983062,('PDT000428382.1':0.089758,('PDT000149337.2':0.0855789,('PDT000491107.1':0.0481821,('PDT000526201.1':0.0448253,'PDT000111343.1':0.0375356)'':0.00712722)'':0.0176133)'':0.00462724)'':0.00356237)'':0.0460065)'':-0.0243919,(('PDT000228141.1':0.0699923,(('PDT000174321.1':0.0178732,'PDT000428860.1':0.0140871)'':0.04433,(('PDT000663934.1':0.0248972,'PDT000174366.1':0.0308887)'':0.0129224,(('PDT000174361.1':0.0119572,'PDT000174385.1':0.0125643)'':0.0271661,('PDT000429680.1':0.034309,('PDT000174382.1':0.00336603,'PDT000422542.1':0.0032208)'':0.0252325)'':0.00534114)'':0.00895395)'':0.0101403)'':0.0166722)'':0.00851922,((('PDT000645502.2':0.0608228,'PDT000708233.1':0.0580002)'':0.00946811,('PDT000652894.1':0.0596035,('PDT000468899.1':0.0582719,('PDT000645603.2':0.0474884,('PDT000652941.1':0.0445751,'PDT000645546.2':0.038026)'':0.00717548)'':0.0137036)'':0.00350522)'':0.00152527)'':0.00655215,('PDT000352051.1':0.0488308,('PDT000538785.1':0.0413369,(('PDT000266407.2':0.0203453,'PDT000548126.1':0.0196586)'':0.00612714,('PDT000430839.1':0.0271706,('PDT000424621.1':0.0114037,('PDT000274860.2':0.00862968,'PDT000499665.1':0.00648722)'':0.00821719)'':0.0113811)'':0.00154072)'':0.0109741)'':0.00697879)'':0.0173775)'':0.0239135)'':0.0383008)'':0.0399948,((('PDT000063346.3':0.159228,('PDT000541641.1':0.133279,('PDT000277742.2':0.0993607,'PDT000448184.1':0.101658)'':0.0237483)'':0.00687012)'':0.00601969,(((('PDT000394907.1':0.0714826,('PDT000395041.1':0.0511065,'PDT000565908.1':0.0546865)'':0.0228639)'':0.0442168,('PDT000015797.3':0.114434,('PDT000310665.2':0.126097,('PDT000395297.1':0.100888,('PDT000051498.2':0.0898248,'PDT000495101.1':0.0821202)'':0.0112103)'':0.0146001)'':0.00812466)'':-0.00352719)'':0.0395523,((('PDT000395295.1':0.104029,('PDT000426701.1':0.103364,'PDT000357638.1':0.0892384)'':0.0161838)'':0.0447129,(('PDT000290530.2':0.128349,'PDT000497420.1':0.105744)'':0.00684369,('PDT000346873.1':0.0729429,('PDT000181790.2':0.0805428,'PDT000522688.1':0.0639862)'':0.00688907)'':0.0510236)'':0.0106444)'':0.000374713,(((('PDT000584495.1':0.0951566,('PDT000417112.1':0.0877971,'PDT000346829.1':0.0766469)'':0.00960937)'':0.0323338,(('PDT000095754.2':0.0718803,'PDT000409988.1':0.0693907)'':0.0189872,('PDT000551131.2':0.0803463,'PDT000251443.1':0.0762777)'':0.00324779)'':0.0219211)'':0.00431813,(('PDT000652540.1':0.0477413,('PDT000206247.3':0.0469454,'PDT000537807.1':0.0483428)'':0.0077101)'':0.0299537,(('PDT000537856.1':0.0418051,'PDT000716410.1':0.0425729)'':0.0500831,('PDT000200983.2':0.0685481,('PDT000403227.1':0.0657493,('PDT000494428.1':0.0503325,'PDT000535501.1':0.0571515)'':0.0173967)'':0.00292104)'':0.00458518)'':0.00084975)'':0.0205325)'':0.00269964,(('PDT000494422.1':0.106163,('PDT000346006.1':0.122062,(('PDT000557009.1':0.0645859,'PDT000250158.1':0.0614461)'':0.021195,('PDT000661344.1':0.0742862,('PDT000656943.1':0.0465861,'PDT000468902.1':0.0440434)'':0.0284781)'':0.0138459)'':0.0289287)'':0.0040418)'':0.00242478,('PDT000557740.2':0.134225,('PDT000046606.1':0.144619,((('PDT000218439.2':0.068078,'PDT000129872.2':0.081273)'':0.0120924,('PDT000188355.1':0.074004,('PDT000497351.1':0.0868311,'PDT000468927.1':0.0682189)'':0.0107035)'':0.00723242)'':0.0132694,('PDT000346882.1':0.126804,('PDT000411080.1':0.091972,('PDT000662400.1':0.0931094,('PDT000177245.1':0.0885163,('PDT000216158.2':0.0441863,'PDT000589393.1':0.0422465)'':0.0177048)'':0.0162127)'':0.0111854)'':0.0260167)'':-0.011397)'':0.0201735)'':0.00828763)'':-0.0211779)'':0.00413964)'':0.0220566)'':0.00372724)'':0.00158543,(('PDT000168720.1':0.179737,('PDT000171797.1':0.12131,(('PDT000621013.1':0.0744682,'PDT000346953.1':0.0692848)'':0.0336147,('PDT000309318.2':0.105805,('PDT000409995.1':0.108702,('PDT000208423.2':0.0771385,'PDT000346539.1':0.0806275)'':0.0211655)'':0.00757687)'':0.00422005)'':0.029272)'':0.0160342)'':0.00167204,(('PDT000652539.1':0.166861,('PDT000311265.2':0.0744461,('PDT000625963.1':0.0534185,('PDT000081744.1':0.0310056,('PDT000607485.1':0.0340865,'PDT000667727.2':0.0308637)'':0.00395147)'':0.0136387)'':0.00844853)'':0.0807108)'':-0.00767102,((('PDT000242954.1':0.103391,('PDT000294552.2':0.0643113,('PDT000467013.1':0.0445089,'PDT000714651.1':0.0484811)'':0.0158322)'':0.0109363)'':0.0324293,(('PDT000414198.1':0.132967,('PDT000242898.1':0.0936655,'PDT000346577.1':0.0692895)'':0.0169623)'':0.00270748,('PDT000345985.1':0.0709914,('PDT000242925.1':0.07043,'PDT000346038.1':0.077662)'':0.00580305)'':0.0159106)'':0.0203373)'':0.0267139,((('PDT000422102.1':0.0926443,'PDT000494438.1':0.0855677)'':0.00891187,('PDT000687997.1':0.134461,(('PDT000468323.1':0.0635618,'PDT000687998.1':0.0585592)'':0.0190141,('PDT000034368.1':0.0155477,'PDT000034371.1':0.0198505)'':0.0653803)'':0.0296863)'':-0.00241414)'':0.0197043,(((('PDT000652174.1':0.0600632,'PDT000250248.1':0.0517058)'':0.022011,('PDT000242885.1':0.00540453,'PDT000242890.1':0.00495197)'':0.0767665)'':0.0561896,('PDT000203404.2':0.120582,('PDT000372052.1':0.0751728,(('PDT000399978.1':0.0320722,'PDT000469959.1':0.0382342)'':0.00358675,('PDT000683606.1':0.030382,'PDT000720256.1':0.0455249)'':0.0100369)'':0.0182606)'':0.0417466)'':0.00487028)'':0.0114975,('PDT000565531.1':0.20377,((('PDT000235279.2':0.0831766,'PDT000354918.1':0.0815724)'':0.0306233,(('PDT000621064.1':0.154984,('PDT000317655.1':0.0896154,('PDT000471030.1':0.106128,(('PDT000432502.1':0.0933596,'PDT000510064.1':0.0836074)'':0.0053911,('PDT000399662.1':0.0721146,('PDT000234177.2':0.0682223,'PDT000719159.1':0.0552607)'':0.00302491)'':0.0148224)'':0.00463889)'':0.00400008)'':0.0186746)'':-0.00153774,('PDT000088259.2':0.125743,(('PDT000147432.4':0.0999662,('PDT000479640.1':0.0331936,'PDT000504792.1':0.0348713)'':0.0591808)'':-0.000143508,('PDT000248723.1':0.0752934,('PDT000425879.1':0.0894981,('PDT000553396.1':0.0667948,('PDT000626959.1':0.0376842,('PDT000592139.1':0.0322515,'PDT000593622.1':0.0279291)'':0.00403681)'':0.0121957)'':0.0120496)'':0.000505846)'':0.036005)'':0.011772)'':-0.000258573)'':0.00875156)'':-0.00325225,(('PDT000251404.1':0.069032,'PDT000289878.1':0.099459)'':0.0682323,((('PDT000095818.2':0.035248,'PDT000372524.1':0.0439187)'':0.0603962,('PDT000702368.1':0.107149,'PDT000149831.1':0.104446)'':6.92124e-05)'':0.0189944,('PDT000625082.1':0.15313,((('PDT000719240.1':0.107074,('PDT000376080.1':0.0606609,'PDT000393071.1':0.0759381)'':0.0227244)'':0.00643278,('PDT000003870.2':0.117496,('PDT000556877.1':0.0822457,(('PDT000645535.2':0.0613848,('PDT000345913.1':0.0509091,'PDT000346447.1':0.0483181)'':0.00716157)'':0.0195077,('PDT000468876.1':0.0778175,('PDT000242886.1':0.0402239,('PDT000095782.2':0.0168545,'PDT000691340.1':0.0155146)'':0.017403)'':0.0307002)'':-0.00220385)'':0.0148685)'':0.0193511)'':0.000472489)'':-0.00118105,(('PDT000644919.1':0.0802105,('PDT000228107.1':0.06066,'PDT000346950.1':0.068208)'':0.026517)'':0.0235073,('PDT000252498.1':0.116012,(('PDT000178555.2':0.0414199,'PDT000206134.2':0.0627671)'':0.0311692,('PDT000425856.1':0.0989051,('PDT000712630.1':0.085985,(('PDT000250188.1':0.0689171,('PDT000550331.1':0.0532397,('PDT000376082.1':0.0431503,('PDT000034390.1':0.0082226,'PDT000034391.1':0.0087653)'':0.0284358)'':0.00866499)'':0.0108732)'':-0.00175781,('PDT000237133.1':0.0786217,('PDT000306644.2':0.0578386,(('PDT000088300.2':0.0494727,'PDT000395310.1':0.0613693)'':0.00495903,(('PDT000095690.2':0.0193868,'PDT000566208.1':0.0254691)'':0.0189242,('PDT000494418.1':0.0286435,'PDT000660644.1':0.0352799)'':0.00447601)'':0.0147835)'':0.00879298)'':0.00892316)'':-0.00185124)'':0.00670493)'':0.0135813)'':-0.000900477)'':0.027354)'':-0.00144055)'':-0.00399637)'':0.0182365)'':-0.00319698)'':0.0202787)'':-0.0233214)'':0.0320468)'':-0.0275553)'':0.00370982)'':0.00507618)'':0.00507257)'':0.002539)'':-0.00402268)'':0.00367453)'':-0.00411062,(((((('PDT000228182.1':0.100826,'PDT000429736.1':0.0934267)'':0.00829778,(('PDT000443071.1':0.0305419,'PDT000251970.1':0.0237621)'':0.0809489,('PDT000174332.1':0.0915897,(('PDT000291203.2':0.0849547,'PDT000251492.1':0.0758453)'':0.0101239,('PDT000426028.1':0.087026,('PDT000537891.1':0.0250191,'PDT000428904.1':0.0293351)'':0.0577009)'':0.00466107)'':0.00273873)'':0.00828375)'':0.00234008)'':0.017897,(('PDT000713510.1':0.108225,'PDT000252576.1':0.0898785)'':0.0213,(('PDT000346549.1':0.0663719,'PDT000411288.1':0.0673921)'':0.0627884,(('PDT000286106.1':0.094062,(('PDT000557198.1':0.0411575,'PDT000566122.1':0.037884)'':0.0360117,('PDT000352377.1':0.039329,('PDT000124968.2':0.0227518,'PDT000496982.1':0.0299891)'':0.0137087)'':0.0324171)'':0.0104853)'':-0.000709561,(('PDT000557247.1':0.0818496,'PDT000584534.3':0.0806774)'':0.0119399,('PDT000250115.1':0.0983717,('PDT000463712.1':0.0545582,('PDT000181670.2':0.0622695,('PDT000411289.1':0.0431627,('PDT000251531.1':0.0222907,'PDT000424786.1':0.0288858)'':0.00812633)'':0.00680647)'':0.00473058)'':0.0166159)'':0.0115698)'':0.00285323)'':0.0225144)'':0.00715134)'':-0.00446165)'':0.00581813,((('PDT000719213.1':0.0625488,('PDT000557238.1':0.0236174,'PDT000249934.1':0.0306179)'':0.0343761)'':0.0429742,(('PDT000557275.1':0.0747197,'PDT000357605.1':0.0597143)'':0.0222868,(('PDT000047725.3':0.0304442,'PDT000357665.1':0.0348177)'':0.0555929,('PDT000489643.1':0.0915686,('PDT000325551.1':0.0437149,'PDT000248806.1':0.0388592)'':0.0389823)'':0.00198683)'':0.00785923)'':0.0086557)'':0.00605521,((('PDT000362883.1':0.0402758,'PDT000645015.1':0.0482937)'':0.0533111,(('PDT000366968.1':0.0552093,'PDT000111365.1':0.0471307)'':0.00294302,('PDT000566102.1':0.0346558,('PDT000035573.1':0.0300817,'PDT000345997.1':0.0391019)'':0.00263786)'':0.0114117)'':0.0376733)'':0.00482022,('PDT000652432.1':0.146767,((('PDT000565896.1':0.102214,('PDT000233297.1':0.0692374,'PDT000370993.1':0.0598926)'':0.0268736)'':0.000589769,(('PDT000415761.1':0.0714092,'PDT000718865.1':0.0588968)'':0.0167193,('PDT000566120.1':0.0821205,'PDT000691343.1':0.0785035)'':0.00348299)'':0.0119286)'':0.0149213,('PDT000444152.1':0.102097,(('PDT000463715.1':0.0925554,('PDT000319007.1':0.093123,(('PDT000354216.1':0.0743983,('PDT000566001.1':0.0488135,('PDT000682551.1':0.0309253,'PDT000150851.1':0.0360725)'':0.00918127)'':0.0282784)'':-0.000421777,(('PDT000109445.2':0.0707806,('PDT000067787.1':0.0191784,'PDT000011755.2':0.0205586)'':0.0445959)'':0.00294185,('PDT000575116.1':0.0809661,('PDT000501926.1':0.0353446,'PDT000111347.1':0.0551456)'':0.0200498)'':0.0122597)'':0.00289867)'':0.00936163)'':0.00501623)'':-0.000434336,(('PDT000565868.1':0.0836601,('PDT000594324.1':0.0146916,'PDT000645136.1':0.0135442)'':0.049337)'':0.01436,('PDT000298053.1':0.138444,(('PDT000557181.1':0.0816457,('PDT000051390.2':0.0396477,'PDT000645624.2':0.0385194)'':0.0346052)'':0.0154926,(('PDT000636358.1':0.0407847,'PDT000318990.1':0.0369219)'':0.035956,('PDT000181678.2':0.0732899,('PDT000081754.1':0.0849911,(('PDT000557262.1':0.0554983,'PDT000667705.2':0.0599447)'':0.0027509,('PDT000494412.1':0.0608362,('PDT000364519.2':0.0498047,'PDT000111376.1':0.0461264)'':0.0117242)'':0.00359867)'':0.00574275)'':0.00176055)'':0.00476258)'':0.00203824)'':0.0185561)'':-0.0126213)'':0.00487012)'':0.00517088)'':0.00153875)'':0.0254549)'':-0.0283103)'':0.0070324)'':0.0124263)'':0.00130586,((('PDT000675152.1':0.123436,(('PDT000129834.2':0.0801015,'PDT000429704.1':0.0788265)'':0.0101646,('PDT000468848.1':0.10732,('PDT000129915.2':0.0697208,('PDT000364517.2':0.0135663,'PDT000201783.1':0.00790147)'':0.0596468)'':0.0119074)'':0.0127162)'':0.00622853)'':0.0148767,(('PDT000600986.1':0.0901964,'PDT000267041.1':0.0939946)'':0.00999593,((('PDT000109380.2':0.0258614,'PDT000111370.1':0.0253781)'':0.0451426,('PDT000550374.1':0.0625663,'PDT000346942.1':0.0585057)'':0.0125336)'':0.0148415,('PDT000661120.1':0.0687448,('PDT000252500.1':0.0572319,('PDT000281774.2':0.0436693,('PDT000081736.1':0.0296081,'PDT000171791.1':0.0252504)'':0.0167425)'':0.0174162)'':0.00571139)'':0.0200902)'':0.00917843)'':0.00888777)'':0.00462974,(('PDT000252730.1':0.0809731,((('PDT000258457.2':0.020575,'PDT000502422.1':0.0370333)'':0.0193726,('PDT000275049.4':0.0560205,('PDT000651587.1':0.0343701,'PDT000251722.1':0.0292283)'':0.00955984)'':0.00592816)'':0.00071442,('PDT000584998.1':0.0498158,('PDT000081260.1':0.0308767,(('PDT000196616.3':0.0321782,('PDT000095248.3':0.0103711,'PDT000136900.3':0.0165213)'':0.0087316)'':0.00442526,(('PDT000542193.2':0.0313918,('PDT000147628.3':0.014948,'PDT000414583.2':0.0126396)'':0.0170188)'':0.000638213,(('PDT000501945.1':0.0269582,'PDT000150865.1':0.0265802)'':0.00436545,('PDT000190775.3':0.028434,('PDT000157292.3':0.0119,'PDT000343897.2':0.0117587)'':0.0103102)'':0.00108975)'':0.0024987)'':0.00656236)'':0.0055858)'':0.00465126)'':0.00806159)'':0.0506089)'':0.0190316,((('PDT000394900.1':0.0668692,('PDT000477134.1':0.0396212,'PDT000645094.1':0.0424691)'':0.0156727)'':0.0300814,('PDT000718844.1':0.0933854,(('PDT000598061.1':0.0595191,'PDT000227998.1':0.0580449)'':0.0112327,((('PDT000394914.1':0.0261547,'PDT000586257.1':0.0246231)'':0.0352397,('PDT000319739.1':0.0452887,('PDT000364420.1':0.0332376,'PDT000556907.1':0.0356133)'':0.0151731)'':0.006385)'':0.0102304,('PDT000548121.1':0.0465055,('PDT000444809.1':0.0224974,('PDT000239188.2':0.0291524,('PDT000271360.2':0.025876,'PDT000275530.2':0.027765)'':0.000730243)'':0.00492723)'':0.015057)'':0.0315799)'':0.00995119)'':0.0110175)'':-0.00406891)'':0.022479,(('PDT000517921.1':0.0864738,'PDT000228199.1':0.0794842)'':0.0291087,(('PDT000397941.1':0.115715,'PDT000346555.1':0.127506)'':0.00430404,(('PDT000032368.1':0.108674,('PDT000535314.1':0.0838694,('PDT000324639.1':0.0655221,('PDT000249956.1':0.0290717,('PDT000109422.2':0.0173344,'PDT000399975.1':0.0245344)'':0.00786268)'':0.0216286)'':0.0370427)'':0.0106451)'':-0.000945216,('PDT000488258.1':0.103372,('PDT000250126.1':0.111011,('PDT000312650.2':0.104056,((('PDT000720235.1':0.084197,('PDT000311264.2':0.0584882,('PDT000524056.1':0.0136924,'PDT000524098.1':0.0194248)'':0.0434032)'':0.0164501)'':0.00298978,('PDT000565951.1':0.0742242,(('PDT000161681.2':0.0579229,'PDT000714699.1':0.0437341)'':0.0104729,('PDT000228092.1':0.0571785,('PDT000111350.1':0.050819,('PDT000300827.2':0.0532215,'PDT000396605.1':0.0433469)'':0.0022138)'':0.00809249)'':0.00387537)'':0.00612977)'':0.000770075)'':0.0121767,(('PDT000672249.1':0.121388,('PDT000101301.1':0.0968215,('PDT000589810.1':0.0740184,(('PDT000129849.2':0.0494366,('PDT000306618.2':0.0273485,'PDT000129847.2':0.0357031)'':0.0109541)'':0.00190453,('PDT000191231.1':0.0458271,('PDT000609597.1':0.0236589,('PDT000500528.1':0.0166844,'PDT000573637.1':0.0130225)'':0.00984)'':0.00916053)'':0.00607478)'':0.00785893)'':0.0328928)'':0.0134219)'':-0.0121252,(('PDT000661813.1':0.0511471,'PDT000111341.1':0.037283)'':0.0448585,(('PDT000639899.1':0.070262,'PDT000017864.2':0.062952)'':0.0149998,('PDT000041345.1':0.0933807,('PDT000111355.1':0.0750589,('PDT000228237.1':0.0758625,('PDT000557203.1':0.0390852,('PDT000494423.1':0.0392742,('PDT000337142.1':0.0326632,('PDT000510053.1':0.020142,'PDT000510062.1':0.0247676)'':0.00928683)'':0.0071196)'':0.00536015)'':0.0288196)'':0.00290046)'':0.00288578)'':-0.00126653)'':0.00490014)'':0.000336374)'':0.00744572)'':0.0151872)'':0.0010908)'':0.00195835)'':-0.0156136)'':0.0172984)'':0.00335273)'':-0.014111)'':0.00773496)'':0.00420768)'':0.000703563)'':0.0134005,((('PDT000517935.2':0.136742,(('PDT000346550.1':0.068297,('PDT000526247.2':0.0597865,'PDT000412510.1':0.0655195)'':0.00721996)'':0.0116058,(('PDT000228347.1':0.0583026,('PDT000281238.2':0.0460695,('PDT000547545.1':0.0430758,'PDT000539068.2':0.0424847)'':0.0116581)'':0.0112277)'':0.0353179,(('PDT000380270.1':0.0742574,'PDT000557163.1':0.0705116)'':0.0137936,('PDT000604824.2':0.0715125,('PDT000233291.1':0.0578507,('PDT000345939.1':0.0355093,'PDT000369027.1':0.0419732)'':0.0121961)'':0.0166834)'':0.026188)'':0.00927019)'':0.000275659)'':0.0412324)'':-0.00696556,(('PDT000360139.1':0.13094,((('PDT000228131.1':0.0920529,'PDT000251738.1':0.0902891)'':0.0128608,(('PDT000394874.1':0.0670886,'PDT000298026.1':0.0591604)'':0.0107529,('PDT000290390.4':0.0822813,('PDT000594909.1':0.0535425,'PDT000639569.1':0.0422711)'':0.0143754)'':0.0239805)'':0.0161674)'':0.00508206,('PDT000468914.1':0.118426,(('PDT000405286.1':0.0763614,('PDT000425734.1':0.09207,('PDT000304274.1':0.0768605,('PDT000702847.1':0.078241,('PDT000626967.1':0.0501895,'PDT000639551.1':0.0536655)'':0.010006)'':0.00770052)'':0.0116707)'':0.00453266)'':0.00243881,(('PDT000652947.1':0.0965888,'PDT000107923.1':0.0741992)'':0.00986759,('PDT000687837.1':0.0882288,('PDT000312779.2':0.0723404,('PDT000394883.1':0.0784944,('PDT000034515.2':0.0583665,'PDT000630065.1':0.0603905)'':0.00993009)'':0.0026394)'':0.00909064)'':0.010728)'':0.000405938)'':0.0084138)'':0.000242638)'':0.00956538)'':-0.00389923,(('PDT000203414.2':0.0881037,('PDT000482519.1':0.0567972,('PDT000204883.2':0.0522623,('PDT000200208.2':0.0431562,('PDT000200197.4':0.0419418,'PDT000203106.4':0.040147)'':0.00384118)'':0.0144276)'':0.00424018)'':0.0406248)'':0.0359212,(('PDT000181801.2':0.111049,'PDT000424626.1':0.116286)'':0.0151154,((('PDT000600522.1':0.0776832,'PDT000710707.1':0.0809548)'':0.0103401,('PDT000396754.1':0.0943663,('PDT000292812.2':0.0718839,'PDT000032324.1':0.0599771)'':0.0228897)'':0.00834131)'':0.00688115,((('PDT000242926.1':0.0744033,'PDT000242933.1':0.0890577)'':0.0419874,(('PDT000566056.1':0.0956807,'PDT000617230.1':0.109737)'':0.00752494,(('PDT000556929.1':0.0625792,'PDT000346536.1':0.0746628)'':0.0129185,(('PDT000208416.2':0.0687769,'PDT000290375.2':0.0720611)'':0.00228942,('PDT000196647.2':0.0720475,('PDT000420661.1':0.0488038,'PDT000435591.1':0.0502307)'':0.0174817)'':0.000694257)'':0.00566509)'':0.0109061)'':0.0184589)'':-0.00620422,((('PDT000251941.1':0.0729414,'PDT000346961.1':0.0788316)'':0.00224769,('PDT000346840.1':0.0479207,('PDT000181814.2':0.0393808,'PDT000537863.1':0.0367829)'':0.00556573)'':0.0360828)'':0.00831265,(('PDT000524037.1':0.0640503,('PDT000109327.2':0.0228122,'PDT000109446.2':0.0233401)'':0.0487665)'':0.015799,(('PDT000233325.1':0.0649779,('PDT000242889.1':0.0104995,'PDT000242891.1':0.00840967)'':0.057906)'':0.00175435,('PDT000584553.2':0.084801,('PDT000144602.1':0.0698961,'PDT000346955.1':0.0597479)'':0.00904103)'':0.000918606)'':0.0155231)'':0.0167456)'':0.00798111)'':0.00783858)'':0.0033744)'':-0.00279181)'':0.000616489)'':0.00264675)'':0.0195089,((('PDT000417194.1':0.129865,('PDT000521050.1':0.0679512,('PDT000395112.1':0.0228403,('PDT000644868.1':0.00445209,'PDT000706048.1':0.00481732)'':0.0154112)'':0.0572924)'':0.0450778)'':-0.00344949,(('PDT000625452.1':0.040532,'PDT000383097.1':0.040003)'':0.0761097,('PDT000443382.1':0.098457,(('PDT000145166.2':0.0511487,'PDT000502922.1':0.0345036)'':0.0474572,(('PDT000525365.1':0.0703368,('PDT000243023.1':0.0357125,('PDT000109483.2':0.020204,'PDT000688606.1':0.0263719)'':0.013503)'':0.0333904)'':0.000598505,(('PDT000566167.1':0.0418561,'PDT000228090.1':0.0304968)'':0.0367631,('PDT000111346.1':0.0784799,('PDT000351842.1':0.0858769,('PDT000228096.1':0.0599004,'PDT000491112.1':0.0627616)'':0.00889913)'':0.00366369)'':0.00218976)'':0.00161506)'':0.00522237)'':0.0208292)'':0.00628585)'':0.00193529)'':0.00996263,(('PDT000168830.1':0.1476,(('PDT000132359.2':0.0494028,'PDT000574558.1':0.0459173)'':0.0937863,('PDT000357614.1':0.0627068,('PDT000426620.1':0.0724741,'PDT000248473.1':0.0644709)'':0.0115722)'':0.0566396)'':0.0104117)'':0.00879079,((('PDT000667728.2':0.19337,('PDT000652569.1':0.0881563,('PDT000132212.2':0.0525178,'PDT000652467.1':0.0714962)'':0.0164932)'':0.0211923)'':0.0117517,((('PDT000661122.1':0.0991115,(('PDT000566046.1':0.0399246,'PDT000568757.1':0.0443814)'':0.0525395,('PDT000588009.1':0.0682924,'PDT000346609.1':0.0792606)'':0.00952295)'':0.00670229)'':0.0121372,(('PDT000034501.2':0.0557509,('PDT000034493.2':0.0603238,'PDT000687690.1':0.0500712)'':0.0090471)'':0.0300239,('PDT000298038.1':0.11417,(('PDT000275051.4':0.0730303,'PDT000660399.1':0.0746117)'':0.0143225,(('PDT000034471.2':0.0602709,'PDT000178588.2':0.0635441)'':0.00908708,('PDT000409990.1':0.0766162,(('PDT000485256.1':0.0493287,'PDT000228317.1':0.0519803)'':0.00551295,('PDT000626961.1':0.0558556,'PDT000346863.1':0.0610844)'':0.00296305)'':0.00414477)'':0.000903156)'':0.00682517)'':0.0164974)'':-0.00579028)'':0.0165658)'':0.00542649,((('PDT000633321.1':0.0825934,'PDT000373309.1':0.0624206)'':0.0366792,('PDT000346858.1':0.137027,(('PDT000625988.1':0.103814,('PDT000720456.1':0.0795021,'PDT000468903.1':0.0515859)'':0.00853892)'':0.00634743,('PDT000712621.1':0.0761837,(('PDT000397623.1':0.0561401,('PDT000565953.1':0.0473814,'PDT000395164.1':0.0435016)'':0.00808936)'':0.0138313,('PDT000032332.1':0.00242237,('PDT000032214.1':0.00327262,'PDT000034396.1':0.00281278)'':0.000471597)'':0.0618344)'':0.0109763)'':-0.00134876)'':0.026103)'':0.00554271)'':-0.0103231,(('PDT000414197.1':0.0939853,('PDT000518000.1':0.066969,'PDT000557049.1':0.066484)'':0.0195842)'':0.00232957,('PDT000661173.1':0.124217,('PDT000034505.2':0.094396,('PDT000566140.1':0.10427,('PDT000298027.1':0.0973605,('PDT000291287.1':0.0869055,('PDT000145175.2':0.0655835,(('PDT000191871.2':0.0421274,'PDT000394966.1':0.0316607)'':0.00402401,('PDT000311252.2':0.0333248,'PDT000494433.1':0.0387612)'':0.00286844)'':0.00966399)'':0.020457)'':0.00748683)'':0.00168041)'':0.00507905)'':0.00568184)'':0.00300105)'':0.00660478)'':0.0103121)'':0.00752155)'':-0.0038217,((('PDT000026018.1':0.0585078,('PDT000444182.1':0.06236,('PDT000032219.1':0.00834605,('PDT000034411.1':0.00735899,'PDT000034412.1':0.00647051)'':0.00205355)'':0.0530471)'':0.00133745)'':0.0438477,(('PDT000394733.1':0.0199285,'PDT000459044.1':0.0164153)'':0.0190282,('PDT000532054.1':0.0158042,('PDT000522374.1':0.00914047,('PDT000533312.1':0.00757078,'PDT000669608.1':0.00653052)'':0.00112528)'':0.00976208)'':0.0212927)'':0.0721322)'':0.00265952,((('PDT000720213.1':0.0519567,'PDT000081739.1':0.0497573)'':0.0119631,('PDT000298071.1':0.0754767,('PDT000689962.1':0.0447675,'PDT000048125.1':0.0530051)'':0.00733097)'':0.0156175)'':0.0439405,(('PDT000463702.1':0.127726,(('PDT000134255.1':0.0736609,('PDT000535504.1':0.0379618,('PDT000600528.1':0.0377145,'PDT000712636.1':0.0327465)'':0.0105347)'':0.0299194)'':0.0256678,('PDT000557129.1':0.103507,('PDT000600527.1':0.0946552,(('PDT000712990.1':0.0653747,'PDT000543097.1':0.0732303)'':0.00265338,('PDT000497331.1':0.0804481,('PDT000432550.1':0.0521659,'PDT000205440.1':0.0409484)'':0.0144017)'':0.00629717)'':0.00676815)'':0.00820399)'':-0.00386979)'':0.0268794)'':-0.0106162,((('PDT000502916.1':0.0456109,('PDT000420676.1':0.017483,'PDT000577223.1':0.0312926)'':0.0301435)'':0.0392045,(('PDT000251417.1':0.0979154,(('PDT000271537.2':0.0207001,'PDT000048805.1':0.0208193)'':0.0381144,('PDT000109376.2':0.0538246,'PDT000415213.1':0.0454164)'':0.00694767)'':0.0215595)'':0.00174435,('PDT000290370.2':0.0966018,(('PDT000510162.1':0.0385416,'PDT000563584.1':0.0456431)'':0.00172928,('PDT000662183.1':0.0537036,('PDT000395433.1':0.0317977,('PDT000258341.2':0.0303052,'PDT000228001.1':0.0272747)'':0.00615742)'':0.00852257)'':0.00342561)'':0.0411041)'':-0.00578107)'':0.00963685)'':0.00417614,(('PDT000558448.1':0.104433,(('PDT000249864.1':0.0825542,('PDT000497423.1':0.0253042,'PDT000395389.1':0.0318527)'':0.0462028)'':-0.000936066,('PDT000651906.2':0.0785573,('PDT000095735.2':0.0538172,'PDT000250186.1':0.0544428)'':0.00961166)'':0.00872547)'':0.0304894)'':-0.00605686,(('PDT000327231.1':0.116003,(('PDT000691349.1':0.0916162,('PDT000150230.2':0.0667317,'PDT000288662.2':0.0674303)'':0.0117518)'':0.0174947,(('PDT000150241.2':0.0804815,('PDT000394829.1':0.0605548,'PDT000250019.1':0.0600372)'':0.013578)'':0.00513171,('PDT000661081.1':0.0875408,('PDT000716363.1':0.0708669,('PDT000594477.1':0.0617697,'PDT000483542.1':0.0501943)'':0.00494561)'':0.0118859)'':0.000935748)'':0.0155737)'':0.0104734)'':-0.00973534,(('PDT000397782.1':0.0631256,('PDT000706414.1':0.0676133,'PDT000706519.1':0.0705477)'':0.00967592)'':0.0211841,((((('PDT000352672.2':0.00239415,'PDT000140446.3':0.00260135)'':0.0167219,('PDT000096836.3':0.0211202,'PDT000708938.1':0.0193037)'':0.00467251)'':0.00781517,('PDT000587251.2':0.0239799,('PDT000132634.4':0.0287346,('PDT000227576.3':0.0189662,('PDT000215117.3':0.0137769,'PDT000630639.2':0.0141654)'':0.00625335)'':0.00296897)'':0.0056973)'':0.000113753)'':0.0536388,('PDT000242952.1':0.0968686,('PDT000652409.1':0.0944161,('PDT000092530.1':0.0655868,('PDT000426766.1':0.085195,(('PDT000081733.1':0.0602408,('PDT000358623.1':0.0283075,'PDT000218327.1':0.0337431)'':0.0184509)'':0.0120158,('PDT000145324.3':0.0827995,('PDT000109417.2':0.0568409,('PDT000714650.1':0.0643171,(('PDT000661358.1':0.0402946,'PDT000248499.1':0.0382768)'':0.0152858,('PDT000550351.1':0.0561336,('PDT000389646.1':0.049508,(('PDT000668478.2':0.0345175,'PDT000491102.1':0.0404127)'':0.000803152,('PDT000109306.2':0.0390106,('PDT000717106.1':0.0349997,'PDT000248820.1':0.0342071)'':0.00493049)'':0.00476872)'':0.00513484)'':0.00415624)'':0.00026588)'':0.00357716)'':0.0031756)'':0.00591175)'':-0.00708243)'':0.00987848)'':0.00154252)'':0.0033399)'':0.00175206)'':-0.00396618)'':0.00602993,(('PDT000490599.1':0.09384,('PDT000537516.1':0.0696114,('PDT000645499.2':0.0455203,'PDT000490172.1':0.0460828)'':0.0180576)'':0.0224891)'':-0.00398985,(('PDT000309263.1':0.073921,(('PDT000550379.1':0.0273979,'PDT000644889.1':0.0282065)'':0.0374122,('PDT000346886.1':0.0658703,(('PDT000346583.1':0.0503607,'PDT000346878.1':0.0495242)'':0.0012534,('PDT000346877.1':0.0523808,('PDT000346879.1':0.0371056,('PDT000294380.1':0.0236922,'PDT000346948.1':0.0231263)'':0.0097371)'':0.0110165)'':0.00833708)'':0.00507975)'':0.0101677)'':0.00831563)'':-0.000957869,(('PDT000415820.1':0.0632447,(('PDT000566039.1':0.0196492,'PDT000395219.1':0.0275374)'':0.0451979,('PDT000250245.1':0.0484148,('PDT000394929.1':0.0494482,'PDT000644924.1':0.034472)'':0.0156368)'':0.0101106)'':0.00567474)'':0.00017461,('PDT000251743.1':0.0749284,('PDT000683610.1':0.0781617,(('PDT000017890.2':0.0755212,('PDT000345983.1':0.0533229,('PDT000463612.1':0.0157856,'PDT000463648.1':0.0215818)'':0.0332354)'':0.0116541)'':-0.00287766,(('PDT000258721.2':0.0537965,'PDT000344878.1':0.0538405)'':0.00919729,((('PDT000557236.1':0.0334105,'PDT000228038.1':0.0254762)'':0.021026,('PDT000654628.1':0.0377159,'PDT000042444.1':0.0321154)'':0.00704957)'':0.00390127,(('PDT000062949.2':0.0249467,'PDT000602894.1':0.0451145)'':0.020059,('PDT000667751.2':0.0495016,('PDT000041341.1':0.0386246,'PDT000129852.2':0.0223474)'':0.0100745)'':0.000306921)'':0.00449592)'':0.0048669)'':-0.000303702)'':0.00750842)'':0.0108732)'':-0.0135418)'':0.00405891)'':0.0147297)'':0.00155777)'':0.00855598)'':-0.00606727)'':0.00797822)'':-0.0036492)'':0.00976115)'':0.00883607)'':-0.00407384)'':0.0022738)'':0.0285338)'':-0.0109967)'':-0.00617632)'':0.00635099)'':0.00979229)'':0.013011)'':0.0013695)'':0.0015464,((((('PDT000226135.1':0.0647485,('PDT000426334.1':0.0491482,('PDT000520991.1':0.038519,'PDT000668930.1':0.0488041)'':0.0117002)'':0.00707327)'':0.0059693,(('PDT000298057.1':0.0662518,('PDT000168848.1':0.0772138,('PDT000625090.1':0.0488799,'PDT000168759.1':0.0469961)'':0.0138262)'':0.00384159)'':0.0115468,(('PDT000708197.1':0.0312033,'PDT000168760.1':0.0400296)'':0.0322546,('PDT000168780.1':0.052435,'PDT000171809.1':0.0428489)'':0.00620597)'':0.0069723)'':0.0116906)'':0.0372403,((('PDT000202172.2':0.0674718,'PDT000616126.1':0.0691292)'':0.0255969,('PDT000651169.1':0.0767282,'PDT000661175.1':0.0750928)'':0.0203421)'':0.0105714,((('PDT000417164.1':0.080241,('PDT000417171.1':0.0865729,'PDT000627377.1':0.0804471)'':0.00315497)'':0.0262538,(('PDT000661176.1':0.0783655,((('PDT000535494.1':0.0539479,('PDT000233321.1':0.0606971,'PDT000495527.1':0.0535969)'':0.00318309)'':0.013663,('PDT000430714.1':0.0751106,('PDT000520803.1':0.0727443,('PDT000704938.1':0.0524973,('PDT000175912.2':0.0445528,'PDT000425772.1':0.0458589)'':0.0064455)'':0.00469434)'':0.0066091)'':0.000686946)'':0.000309319,((('PDT000639893.1':0.0460932,('PDT000459508.1':0.0331418,'PDT000448171.1':0.0285128)'':0.0111152)'':0.00435055,('PDT000604368.1':0.0502466,('PDT000208667.1':0.0211425,('PDT000633096.1':0.00869559,'PDT000634220.1':0.0142654)'':0.0131861)'':0.0190366)'':0.00119964)'':0.00447316,((('PDT000202607.2':0.0311447,'PDT000253661.1':0.028267)'':0.00429508,('PDT000438359.1':0.0146817,'PDT000417609.1':0.0193968)'':0.0206486)'':0.0039559,('PDT000163382.1':0.0414595,('PDT000675078.1':0.0334325,(('PDT000433746.1':0.0226666,'PDT000667747.2':0.0235564)'':0.00219944,('PDT000457708.1':0.0169602,('PDT000202698.2':0.0144981,'PDT000485946.1':0.0172949)'':0.00277196)'':0.00552568)'':0.00555622)'':0.00153565)'':0.0072635)'':0.0108096)'':0.0167011)'':0.00608492)'':-0.00238821,((('PDT000535488.1':0.0563112,('PDT000534874.1':0.0451663,('PDT000426735.1':0.0466963,('PDT000615810.1':0.0314023,'PDT000716807.1':0.0270847)'':0.0122074)'':0.00240667)'':0.00637664)'':-0.000659246,(('PDT000276015.4':0.0427965,'PDT000212815.1':0.0354391)'':0.0102854,(('PDT000136105.2':0.0434423,'PDT000675227.1':0.0364491)'':0.00036037,(('PDT000445102.1':0.00436756,'PDT000633212.1':0.00391009)'':0.0109436,('PDT000626912.1':0.0235467,('PDT000424801.1':0.0168893,('PDT000649370.1':0.00403335,'PDT000304289.1':0.00433986)'':0.00785873)'':0.00607675)'':0.00102027)'':0.0209037)'':0.00822052)'':0.00305824)'':0.0139283,((('PDT000459483.1':0.0340013,'PDT000398348.1':0.0318264)'':0.0195859,('PDT000477485.1':0.0476369,('PDT000637075.1':0.0167168,'PDT000400915.1':0.0176492)'':0.0281287)'':0.00560534)'':0.0180376,('PDT000625091.1':0.105807,(('PDT000535458.1':0.0464003,('PDT000042454.1':0.039464,('PDT000538580.1':0.0410798,('PDT000218136.2':0.0256758,('PDT000426201.1':0.019595,'PDT000673556.1':0.0233757)'':0.00674948)'':0.00456243)'':0.00567183)'':0.00924015)'':0.00575266,('PDT000346545.1':0.0585811,(('PDT000423938.1':0.0274752,('PDT000136095.2':0.0140471,'PDT000637076.1':0.0099904)'':0.0174741)'':0.00145161,('PDT000714564.1':0.0402009,('PDT000363557.1':0.033264,('PDT000520868.1':0.0156646,('PDT000443110.1':0.0171964,('PDT000520730.1':0.00983109,'PDT000252012.1':0.0168967)'':0.00459737)'':0.00351598)'':0.00472279)'':0.011579)'':-0.00352615)'':0.0259148)'':-0.00718542)'':0.0202601)'':-0.00766747)'':-0.00148834)'':0.0120494)'':0.018751)'':-0.0159916,('PDT000210721.1':0.100831,((((('PDT000181657.2':0.0462021,'PDT000425514.1':0.0472178)'':0.0217125,('PDT000181830.2':0.0511222,'PDT000535341.1':0.0595858)'':0.00984879)'':0.00836667,('PDT000427194.1':0.0909185,(('PDT000535425.1':0.0603383,('PDT000520872.1':0.0522039,'PDT000168462.1':0.0494011)'':0.0122567)'':0.00268593,('PDT000202591.2':0.0934527,(('PDT000228326.1':0.0527266,'PDT000417614.1':0.0444742)'':0.0127898,('PDT000019262.2':0.058565,('PDT000202056.2':0.0620255,(('PDT000668896.1':0.0376907,'PDT000668907.1':0.0325512)'':0.00361116,('PDT000425454.1':0.0377474,('PDT000423925.1':0.0125588,'PDT000423947.1':0.00880909)'':0.0231567)'':0.00418877)'':0.0146733)'':0.00532445)'':0.00076448)'':0.0138746)'':-0.00878506)'':0.0122515)'':-0.00258327)'':-0.00124184,(('PDT000291216.2':0.0648718,('PDT000615963.1':0.0714224,'PDT000271777.1':0.0639456)'':0.00276517)'':0.00634354,(('PDT000375417.1':0.100214,((('PDT000675631.1':0.0327017,'PDT000253679.1':0.0244832)'':0.0156463,('PDT000459434.1':0.0367418,'PDT000459435.1':0.0479622)'':0.00539654)'':0.0253533,('PDT000227985.1':0.0596704,('PDT000425617.1':0.0524453,(('PDT000437995.1':0.0275796,'PDT000248801.1':0.022047)'':0.00996303,('PDT000437910.1':0.021277,('PDT000340377.1':0.0118917,('PDT000340378.1':0.00230747,'PDT000340379.1':0.00317827)'':0.0138949)'':0.00987632)'':0.0168764)'':0.0135858)'':0.00430907)'':0.0135802)'':0.00907911)'':-0.00180881,(('PDT000615934.1':0.0333351,('PDT000437925.1':0.0367654,('PDT000716809.1':0.0310409,(('PDT000191591.1':0.0265811,'PDT000448177.1':0.0288382)'':0.00439401,('PDT000363554.1':0.0162904,'PDT000408002.1':0.0125272)'':0.00926284)'':0.00700403)'':0.00199951)'':0.00226028)'':0.0322023,(('PDT000239097.1':0.0534533,('PDT000426624.1':0.0408588,('PDT000042706.1':0.0333026,'PDT000218720.1':0.0404655)'':0.0081635)'':0.0109393)'':0.00574938,('PDT000233840.1':0.0594117,(('PDT000582019.1':0.0231465,'PDT000582120.1':0.0295783)'':0.0155506,('PDT000433775.1':0.0336677,('PDT000172049.4':0.0268716,('PDT000423844.1':0.0214791,'PDT000661313.1':0.0268253)'':0.0030041)'':0.00608066)'':0.00258961)'':0.0128946)'':0.0106613)'':0.00737505)'':0.00168154)'':0.00146867)'':-0.000582306)'':-7.98571e-05,(('PDT000391853.1':0.0628462,('PDT000433310.1':0.0646991,('PDT000675436.1':0.0618563,('PDT000210705.1':0.0543941,((('PDT000202636.2':0.0279464,'PDT000675578.1':0.0341165)'':0.0126681,(('PDT000107779.1':0.0155841,'PDT000340396.1':0.026581)'':0.0202017,('PDT000425968.1':0.0311634,('PDT000175957.2':0.0177588,'PDT000363563.1':0.0162871)'':0.0143804)'':0.00303928)'':0.00756658)'':-0.00195661,(('PDT000682316.1':0.0468963,('PDT000675444.1':0.0289392,('PDT000065733.2':0.0334239,('PDT000446001.1':0.0243176,'PDT000219963.1':0.0231477)'':0.00672766)'':0.00612942)'':0.00932811)'':-5.62291e-05,(('PDT000069696.1':0.0232497,'PDT000496660.1':0.0219804)'':0.012561,('PDT000615856.1':0.0354546,('PDT000425872.1':0.0311473,'PDT000437900.1':0.0299563)'':0.00428317)'':0.000346755)'':0.00372337)'':0.00349507)'':0.00952535)'':0.000285102)'':0.00846015)'':0.00186636)'':0.00634527,((('PDT000209466.1':0.0594612,('PDT000534800.1':0.0360682,'PDT000675409.1':0.0403071)'':0.0139057)'':0.0173296,('PDT000482606.1':0.0551835,('PDT000097041.1':0.0404674,'PDT000307275.2':0.0308699)'':0.0108621)'':0.0205861)'':0.00632165,((('PDT000201857.2':0.0455897,'PDT000346833.1':0.0478736)'':0.0175373,('PDT000092539.1':0.0618961,('PDT000363548.1':0.0522269,('PDT000426223.1':0.0490255,('PDT000616116.1':0.0387549,('PDT000423920.1':0.0235778,('PDT000437944.1':0.017894,('PDT000425681.1':0.00872688,'PDT000277851.1':0.01523)'':0.00267669)'':0.0106483)'':0.0141049)'':0.00316961)'':0.00177298)'':0.00925349)'':0.00276602)'':-0.0028415,(('PDT000431920.1':0.0585952,('PDT000378575.1':0.033002,'PDT000615853.1':0.0320701)'':0.0156652)'':0.001436,('PDT000616074.1':0.0768004,(('PDT000534795.1':0.0377326,'PDT000391854.1':0.0453144)'':0.0189525,('PDT000248450.1':0.0537161,(('PDT000352380.1':0.0308051,('PDT000202704.2':0.025061,'PDT000426352.1':0.0284181)'':0.0114244)'':0.00597201,(('PDT000137821.2':0.019101,('PDT000615948.1':0.0205049,('PDT000437963.1':0.0115865,'PDT000340383.1':0.00941687)'':0.00895451)'':0.00255049)'':0.0233018,('PDT000364530.1':0.0486458,('PDT000423857.1':0.0292911,('PDT000615886.1':0.0264239,('PDT000363564.1':0.0242541,('PDT000589031.1':0.0154685,('PDT000176209.2':0.015576,('PDT000202058.2':0.0104912,'PDT000124865.2':0.0145437)'':0.00230835)'':0.00120084)'':0.00302331)'':0.00743653)'':0.00534586)'':0.00591674)'':0.000409693)'':0.00267751)'':0.0100467)'':-0.00102105)'':0.0100809)'':-0.00718378)'':-0.000651176)'':0.0114881)'':0.00118906)'':0.00334907)'':0.0282651)'':-0.0240376)'':0.00978241)'':0.0158421)'':0.0635739,((((('PDT000435622.1':0.0577956,('PDT000397672.1':0.0538124,('PDT000714029.1':0.0510443,('PDT000067798.1':0.0401909,'PDT000250052.1':0.0398245)'':0.0103866)'':0.00533267)'':0.00915433)'':0.00138047,(('PDT000397609.1':0.0298583,('PDT000397812.1':0.0252864,'PDT000438123.1':0.0237255)'':0.00580386)'':0.0124924,('PDT000616117.1':0.0497791,('PDT000252474.1':0.0349054,('PDT000017842.2':0.0335701,('PDT000188349.1':0.00122283,'PDT000188358.1':0.00117453)'':0.027963)'':0.00879943)'':0.00404703)'':-0.000524706)'':0.0228961)'':0.0407106,(('PDT000482612.1':0.0830597,('PDT000259781.2':0.0158894,'PDT000571173.1':0.0191617)'':0.0554023)'':0.0188561,((('PDT000307437.2':0.0315113,'PDT000600867.1':0.0380038)'':0.041582,(('PDT000047196.3':0.0443613,'PDT000714012.1':0.0522241)'':0.00113738,('PDT000713560.1':0.0525197,('PDT000135967.2':0.0203019,'PDT000543327.1':0.0155393)'':0.0170659)'':0.0115349)'':0.03793)'':0.00224502,(('PDT000444013.1':0.0737631,('PDT000649264.1':0.060793,('PDT000188343.1':0.0231998,'PDT000017884.2':0.0221299)'':0.0407182)'':0.00526891)'':0.000252532,('PDT000652380.1':0.090844,('PDT000160754.2':0.0687821,('PDT000362451.1':0.0690143,('PDT000721004.1':0.0599457,'PDT000011723.2':0.0574163)'':0.00562172)'':0.0033893)'':0.00516681)'':-0.00162754)'':0.0260304)'':0.0150623)'':-0.00114673)'':0.0229412,((('PDT000363570.1':0.117834,('PDT000390953.1':0.0718353,('PDT000674780.1':0.025138,('PDT000407915.1':0.00812977,'PDT000488326.1':0.00725183)'':0.0106925)'':0.0439542)'':0.041143)'':0.0273718,((('PDT000160713.2':0.00436142,('PDT000278228.1':0.00116513,('PDT000278223.1':0.000532766,('PDT000278222.1':0.000322624,'PDT000278224.1':0.000377166)'':0.000366614)'':0.000577126)'':0.00397899)'':0.0132797,('PDT000278226.1':0.00720458,('PDT000190730.1':0.00410969,('PDT000483526.1':0.00198956,('PDT000278230.1':0.00108464,('PDT000142660.1':0.00109854,'PDT000278231.1':0.00110058)'':0.000213771)'':0.00166757)'':0.000620033)'':0.00544628)'':0.00520642)'':0.0957107,('PDT000357611.1':0.0994937,('PDT000411341.1':0.0835121,('PDT000319029.1':0.0513567,(('PDT000136111.2':0.0422939,('PDT000101130.2':0.0357329,'PDT000397901.1':0.0420696)'':0.00708954)'':0.00958388,('PDT000065069.1':0.0525622,('PDT000538410.1':0.0428943,('PDT000061127.2':0.0326354,('PDT000569047.1':0.0225341,'PDT000713615.1':0.0255654)'':0.0102666)'':0.00348088)'':0.0114782)'':-0.00595505)'':0.00792718)'':0.00506676)'':0.0358596)'':0.0144837)'':0.0100242)'':-0.00613397,(((('PDT000571152.1':0.0853695,'PDT000015291.2':0.0816665)'':0.0208441,((('PDT000425594.1':0.044349,'PDT000618788.1':0.0496846)'':0.0337831,('PDT000673494.1':0.042755,'PDT000362459.1':0.0566522)'':0.020418)'':0.0137521,('PDT000661796.1':0.05258,('PDT000711049.1':0.0467265,('PDT000443086.1':0.047762,'PDT000251979.1':0.0341168)'':0.00643259)'':0.00591888)'':0.0397535)'':0.012584)'':-0.00349125,(('PDT000205668.1':0.0918414,'PDT000251875.1':0.0800856)'':0.00696886,(('PDT000213250.2':0.0730155,('PDT000444936.1':0.0658418,('PDT000205430.1':0.0593063,'PDT000415836.1':0.0531157)'':0.00997472)'':0.0150074)'':0.00685412,('PDT000160838.2':0.0867307,(('PDT000344856.1':0.0580997,'PDT000252702.1':0.0671973)'':0.0175513,(('PDT000443074.1':0.0827824,('PDT000354193.1':0.0164476,'PDT000357667.1':0.0246075)'':0.0505961)'':0.00485938,('PDT000357737.1':0.0616697,(('PDT000131922.2':0.0580268,('PDT000644539.1':0.0502611,(('PDT000433455.1':0.0390322,'PDT000086327.1':0.0500999)'':0.00483232,((('PDT000163362.3':0.0128943,'PDT000713673.1':0.00900277)'':0.00492868,('PDT000163613.2':0.0145666,'PDT000142615.2':0.0111989)'':0.00486915)'':0.0155938,('PDT000248828.1':0.0314646,('PDT000034400.1':0.042822,('PDT000032359.1':0.00211597,'PDT000034398.1':0.0027723)'':0.0198178)'':0.00886124)'':0.0051508)'':0.0104675)'':0.00617786)'':0.00449263)'':0.00463056,(('PDT000579933.1':0.0572643,'PDT000713789.1':0.0510537)'':0.00500614,(('PDT000285333.1':0.0361616,(('PDT000296206.2':0.019826,'PDT000131899.2':0.02295)'':0.00688209,('PDT000433282.1':0.016122,'PDT000486044.1':0.0198421)'':0.00487666)'':0.00916652)'':0.00396515,(('PDT000425943.1':0.0232224,'PDT000636135.1':0.0258164)'':0.00705788,(('PDT000426109.1':0.0146015,'PDT000571150.1':0.0158478)'':0.0101439,('PDT000425523.1':0.0344061,('PDT000713310.1':0.0219684,'PDT000017899.2':0.0195507)'':0.00846498)'':0.000895927)'':0.00198506)'':0.00519854)'':0.0089267)'':0.00943988)'':0.010497)'':0.00734798)'':0.00421257)'':0.00875668)'':-0.00784165)'':0.0148287)'':0.00157893)'':-0.00109289,(('PDT000651400.1':0.0460565,('PDT000714176.1':0.0278738,'PDT000251930.1':0.0253552)'':0.0191776)'':0.0426373,(((('PDT000160940.2':0.0410997,('PDT000156652.2':0.0293557,'PDT000620165.1':0.0293704)'':0.011106)'':0.0185803,('PDT000006425.2':0.0599029,('PDT000571177.1':0.052716,('PDT000534949.1':0.0343315,'PDT000015339.2':0.027498)'':0.00464886)'':0.011816)'':0.0145591)'':0.00624262,(('PDT000574931.1':0.0691441,('PDT000635503.1':0.0661936,('PDT000631916.1':0.018602,(('PDT000631917.1':0.00554343,'PDT000631919.1':0.00552117)'':0.000864922,('PDT000631918.1':0.00808772,('PDT000631792.1':0.00270645,'PDT000635502.1':0.00417497)'':0.00166487)'':8.04164e-05)'':0.00569065)'':0.0320621)'':0.0120173)'':0.00276439,(('PDT000136000.2':0.072503,('PDT000652352.1':0.0571113,('PDT000188337.1':0.00319428,'PDT000188350.1':0.00549703)'':0.0446986)'':0.00751308)'':0.00322867,('PDT000162045.2':0.0649557,(('PDT000011752.2':0.0268848,'PDT000362455.1':0.0347892)'':0.0088466,('PDT000069624.1':0.0382347,'PDT000618848.1':0.0405596)'':0.00138922)'':0.0273381)'':-0.00261457)'':0.00161438)'':0.0122617)'':0.0122682,(('PDT000538408.1':0.0825494,('PDT000661734.1':0.0479655,('PDT000277852.1':0.0385553,('PDT000618825.1':0.0344646,'PDT000188369.1':0.0394256)'':0.00237855)'':0.00636311)'':0.0328073)'':0.0120515,((('PDT000661471.1':0.0519474,('PDT000661788.1':0.0502564,'PDT000206599.1':0.0561996)'':0.00450312)'':0.0202303,(('PDT000661365.1':0.0536909,('PDT000438186.1':0.0400382,'PDT000359683.1':0.0373656)'':0.00689772)'':0.0066159,('PDT000631838.1':0.0482499,('PDT000631923.1':0.0385531,('PDT000631812.1':0.0120532,('PDT000631815.1':0.00361996,('PDT000631810.1':0.004786,'PDT000631811.1':0.00397981)'':0.000721739)'':0.00247926)'':0.0112869)'':0.0246116)'':0.0129604)'':0.012048)'':-0.0010306,('PDT000161024.2':0.112586,('PDT000433316.1':0.0919513,((('PDT000167953.2':0.0441542,('PDT000587573.1':0.0428208,('PDT000631342.1':0.0295828,'PDT000604968.1':0.0338113)'':0.00693827)'':0.0029296)'':0.0141451,('PDT000673619.1':0.0618675,(('PDT000476883.1':0.046277,'PDT000448221.1':0.0418228)'':0.00444849,('PDT000009301.2':0.043843,('PDT000482858.1':0.0407112,(('PDT000618903.1':0.0416157,('PDT000502111.1':0.0323374,'PDT000371948.1':0.0307501)'':0.0065628)'':0.00214203,(('PDT000433460.1':0.0277318,'PDT000675474.1':0.0335126)'':0.00479391,('PDT000633933.1':0.038515,('PDT000535138.1':0.0200004,('PDT000534942.1':0.0112454,'PDT000618884.1':0.0184359)'':0.00460121)'':0.00634921)'':0.00426838)'':0.00760202)'':0.00306274)'':0.00310166)'':0.000551689)'':0.0101657)'':-0.00405495)'':0.00870336,((('PDT000132266.2':0.0517522,'PDT000469886.2':0.0439669)'':0.0177178,(('PDT000052502.1':0.0471814,('PDT000713973.1':0.047714,'PDT000238208.2':0.0514882)'':0.00672554)'':0.00723056,(('PDT000017856.2':0.0418683,('PDT000157496.2':0.0371442,'PDT000661521.1':0.0494776)'':0.00735332)'':0.00543098,('PDT000426537.1':0.0458677,('PDT000132103.2':0.0399931,'PDT000714060.1':0.0320922)'':0.00381519)'':0.0119156)'':0.00811563)'':0.00113584)'':0.000988914,('PDT000357632.1':0.0667794,(('PDT000443892.1':0.0341246,'PDT000443922.1':0.0317082)'':0.031795,('PDT000015310.2':0.0540264,(('PDT000611941.1':0.0210255,'PDT000571169.1':0.032131)'':0.0221473,(('PDT000238186.2':0.0366488,'PDT000251929.1':0.0489018)'':0.00856445,('PDT000577471.1':0.0421698,('PDT000161500.2':0.0240948,'PDT000713233.1':0.0229594)'':0.0150817)'':0.000230347)'':0.00133605)'':0.00525887)'':0.00206363)'':0.0120229)'':-0.00939744)'':0.000844664)'':0.0167757)'':0.0145821)'':-0.0286321)'':0.0157905)'':0.00268541)'':0.00482598)'':0.00880937)'':0.020777)'':0.00995511)'':0.00617353,((((('PDT000571180.1':0.0568218,'PDT000357627.1':0.0574452)'':0.0480042,(('PDT000032395.1':0.034633,'PDT000285342.1':0.0519967)'':0.0650692,('PDT000238190.2':0.0826622,('PDT000332691.1':0.0747468,'PDT000709920.1':0.0775862)'':0.00635178)'':0.0203677)'':0.0110665)'':0.0114634,(('PDT000017754.2':0.0612167,('PDT000633115.1':0.07384,('PDT000535513.1':0.0638686,'PDT000357668.1':0.0616284)'':0.00620299)'':0.013808)'':0.0332519,('PDT000287862.2':0.0926154,(('PDT000668941.1':0.0514452,'PDT000015294.2':0.0408357)'':0.0361975,(('PDT000354199.1':0.0638691,'PDT000357705.1':0.0659719)'':0.00599415,('PDT000716959.1':0.0586566,('PDT000448195.1':0.0481038,('PDT000610593.1':0.0324846,'PDT000661475.1':0.0376635)'':0.0102389)'':0.00569625)'':0.0228039)'':0.0165225)'':0.0102147)'':0.0241428)'':0.000411822)'':0.0177864,(((('PDT000251409.1':0.0846786,'PDT000357666.1':0.0827944)'':0.0327865,(('PDT000482841.1':0.0169769,('PDT000482842.1':0.00429872,('PDT000482844.1':0.00154966,'PDT000482849.1':0.00204854)'':0.00539378)'':0.00526608)'':0.0613871,(('PDT000537212.1':0.0740889,('PDT000660657.1':0.0624161,('PDT000466974.1':0.0446165,'PDT000556952.1':0.0462665)'':0.0336674)'':0.00700649)'':0.0126417,(('PDT000252689.1':0.0522335,'PDT000488273.1':0.0634375)'':0.0182355,(('PDT000502741.1':0.0547997,'PDT000395328.1':0.0533213)'':0.0101019,('PDT000557182.1':0.0539876,('PDT000644969.1':0.0374846,'PDT000650529.1':0.0478327)'':0.0140914)'':0.0149854)'':0.0122632)'':0.00089203)'':0.00358996)'':0.0191156)'':0.00231101,(('PDT000425904.1':0.105607,('PDT000094947.1':0.0684105,('PDT000111348.1':0.0489405,('PDT000395200.1':0.0412158,('PDT000309211.2':0.0412803,'PDT000395342.1':0.0367708)'':0.00380613)'':0.0035798)'':0.0121252)'':0.038884)'':-0.00429529,(('PDT000389647.1':0.0439309,('PDT000225864.2':0.048557,('PDT000517995.1':0.025517,(('PDT000372351.1':0.0115529,'PDT000395190.1':0.00864014)'':0.00484486,('PDT000468344.1':0.012679,'PDT000395338.1':0.011839)'':0.00608534)'':0.00633732)'':0.0111056)'':0.0131644)'':0.0301035,('PDT000205439.1':0.0934726,((('PDT000266388.2':0.0404645,'PDT000003766.2':0.0366293)'':0.0210504,('PDT000687397.1':0.0536443,('PDT000312789.2':0.0374754,'PDT000395097.1':0.0232861)'':0.0343919)'':0.000451126)'':0.013261,(('PDT000095831.2':0.0336575,'PDT000557321.1':0.0272487)'':0.0261072,(('PDT000228118.1':0.0495609,'PDT000252654.1':0.0488951)'':0.0148982,('PDT000251482.1':0.0684736,('PDT000032200.1':0.0523679,('PDT000424772.1':0.0327147,('PDT000642580.1':0.0400029,('PDT000389659.1':0.03772,'PDT000645118.1':0.0325629)'':0.00126619)'':0.0079813)'':0.00783414)'':0.0035638)'':0.000117443)'':0.00130172)'':0.00446702)'':0.0214118)'':-0.0153901)'':0.0137256)'':0.0162791)'':0.0120746,(('PDT000414211.1':0.124191,(('PDT000557185.1':0.0411764,'PDT000248846.1':0.0305305)'':0.0400149,('PDT000346584.1':0.0882905,('PDT000346843.1':0.0631954,('PDT000289871.1':0.0438254,'PDT000346591.1':0.0486712)'':0.00862882)'':0.0149923)'':0.0124084)'':0.0158632)'':0.0199099,((('PDT000221029.1':0.0716061,('PDT000221035.1':0.0432145,'PDT000352475.1':0.0525922)'':0.0110861)'':0.0486076,('PDT000046596.1':0.121359,('PDT000566018.1':0.111413,(('PDT000711732.1':0.060866,'PDT000711737.1':0.057895)'':0.0363777,('PDT000556865.1':0.0909201,('PDT000225861.2':0.0638788,'PDT000411286.1':0.0643722)'':0.0346034)'':0.00466561)'':0.0120639)'':0.00465596)'':0.00345079)'':0.00417279,(((('PDT000196650.2':0.0357297,'PDT000644918.1':0.0414323)'':0.0717133,('PDT000252711.1':0.0930144,('PDT000306645.2':0.0798718,('PDT000517996.1':0.0462308,'PDT000541621.1':0.0350621)'':0.0199057)'':0.0176402)'':0.00436136)'':0.0111096,(('PDT000397973.1':0.114982,('PDT000395084.1':0.052318,'PDT000619415.1':0.05286)'':0.0553239)'':0.00719977,((('PDT000603333.1':0.0720459,('PDT000397908.1':0.0788639,'PDT000565860.1':0.0738091)'':0.00473565)'':0.0142599,((('PDT000251925.1':0.0785241,'PDT000399655.1':0.0907729)'':0.000239639,('PDT000034485.2':0.079662,'PDT000346455.1':0.085218)'':0.00492661)'':0.00602742,('PDT000610219.1':0.0626543,(('PDT000395016.1':0.043921,'PDT000627601.1':0.0320122)'':0.00927866,('PDT000626539.1':0.0358601,'PDT000289920.1':0.0440583)'':0.000749814)'':0.0132685)'':0.0147912)'':0.00201147)'':0.0130477,(('PDT000648090.1':0.127717,'PDT000394881.1':0.112029)'':0.00777514,(('PDT000384275.1':0.0769523,('PDT000364522.2':0.0863663,'PDT000346520.1':0.0773027)'':0.00958466)'':0.00915943,(('PDT000372028.1':0.0794723,('PDT000644981.1':0.079198,'PDT000251544.1':0.063632)'':0.00818973)'':0.00912743,(('PDT000171801.1':0.0782685,('PDT000228072.1':0.0736403,('PDT000364402.1':0.0303453,'PDT000581175.1':0.0466837)'':0.0302027)'':0.00423929)'':0.011496,(('PDT000019456.2':0.0746074,(('PDT000115047.2':0.0519978,'PDT000687371.1':0.0496892)'':0.00805862,('PDT000566206.1':0.0636342,('PDT000659708.1':0.0189499,'PDT000720222.1':0.0255364)'':0.0201602)'':0.00593934)'':0.0081946)'':0.0123073,('PDT000200977.4':0.0890606,(('PDT000196617.2':0.058335,'PDT000095800.2':0.064193)'':0.0074785,(('PDT000034472.2':0.0679278,'PDT000396610.1':0.0636242)'':0.0055199,('PDT000571191.1':0.0521689,'PDT000252437.1':0.0626531)'':0.00509485)'':0.005414)'':0.00737783)'':0.000242243)'':0.00366497)'':0.0035997)'':0.0024612)'':0.00580117)'':-0.00521921)'':0.00618507)'':0.00222214)'':-0.00274705,(('PDT000417159.1':0.0710824,'PDT000289908.1':0.0788896)'':0.0456513,(('PDT000109787.2':0.0332848,'PDT000424778.1':0.0310702)'':0.0608505,('PDT000017891.2':0.0926281,(('PDT000228148.1':0.0905548,(('PDT000311269.2':0.062237,('PDT000251555.1':0.0448828,('PDT000228010.1':0.0280161,'PDT000346514.1':0.0318624)'':0.0141116)'':0.0136048)'':0.0105203,('PDT000439503.1':0.0555043,('PDT000470053.1':0.0521334,('PDT000109443.2':0.0308895,'PDT000566092.1':0.0351299)'':0.0117464)'':0.0144451)'':0.0153634)'':0.0123084)'':-0.00868932,(('PDT000128787.2':0.0903832,('PDT000150305.2':0.0852521,(('PDT000510061.1':0.0342922,'PDT000557239.1':0.0366674)'':0.0244259,('PDT000548076.1':0.0463828,('PDT000396810.1':0.0313526,'PDT000639530.1':0.0308493)'':0.0118933)'':0.0167369)'':0.00848127)'':0.0015978)'':0.00741232,(('PDT000312643.2':0.0598043,(('PDT000062950.2':0.0209798,'PDT000353625.1':0.0221644)'':0.0108841,('PDT000228251.1':0.0319125,'PDT000251802.1':0.0347153)'':0.00443299)'':0.0106602)'':0.00937907,((('PDT000251522.1':0.0584979,('PDT000081776.1':0.0416794,('PDT000565921.1':0.035119,'PDT000395340.1':0.0377524)'':0.00941822)'':0.00520624)'':0.000407589,(('PDT000644670.1':0.0377344,'PDT000111360.1':0.0510178)'':0.0170517,('PDT000109407.2':0.0368397,('PDT000510054.1':0.040787,'PDT000150895.1':0.0437718)'':0.00246455)'':0.0115881)'':0.000953894)'':0.0033336,((('PDT000109365.2':0.050971,'PDT000109375.2':0.056243)'':0.00310762,('PDT000111352.1':0.0461615,'PDT000019460.2':0.0485482)'':0.00557678)'':0.000958312,(('PDT000111362.1':0.0438851,('PDT000111364.1':0.0263624,'PDT000249951.1':0.0179471)'':0.0182085)'':0.00330537,('PDT000713327.1':0.0562041,(('PDT000150254.2':0.0272676,'PDT000111358.1':0.0255604)'':0.0128431,('PDT000109363.2':0.0345507,'PDT000109418.2':0.0320058)'':0.00891162)'':0.00446372)'':0.00191983)'':0.00647736)'':0.00291073)'':0.00422315)'':0.0166144)'':0.000596512)'':0.0116024)'':0.00332298)'':0.0109019)'':-0.0046288)'':0.0105715)'':0.00438228)'':-0.00254368)'':0.0106239)'':-0.0064062,((('PDT000583382.1':0.050097,('PDT000274552.2':0.0436027,'PDT000174315.1':0.0410193)'':0.00544002)'':0.0611907,(('PDT000441812.1':0.0370463,('PDT000590029.1':0.0207303,'PDT000675035.1':0.0256969)'':0.00962881)'':0.0664587,(((('PDT000616163.1':0.0733504,('PDT000434602.1':0.0500236,'PDT000674910.1':0.0356001)'':0.0125868)'':0.0176925,('PDT000136096.2':0.0579207,('PDT000417617.1':0.0488888,('PDT000675615.1':0.0328723,('PDT000145738.1':0.0330698,('PDT000632616.1':0.0188563,'PDT000304335.1':0.0148394)'':0.00986935)'':0.00801888)'':0.00789628)'':0.00470312)'':0.0359214)'':0.00424652,(((('PDT000702366.1':0.0398662,('PDT000445988.1':0.0305345,'PDT000297999.1':0.0274647)'':0.00946189)'':0.0183342,(('PDT000417142.1':0.042205,'PDT000417190.1':0.0541186)'':0.00768661,('PDT000709975.1':0.0395085,('PDT000592988.1':0.0326914,('PDT000545912.1':0.0263022,'PDT000277849.1':0.0229443)'':0.00721347)'':0.00636995)'':0.00495584)'':0.00920991)'':0.00601737,(('PDT000675305.1':0.0447274,('PDT000675377.1':0.0308789,'PDT000297998.1':0.0464757)'':0.0110099)'':0.00256268,('PDT000360975.1':0.0517545,('PDT000534807.1':0.0216797,('PDT000297995.1':0.0206986,('PDT000590022.1':0.0159651,('PDT000673447.1':0.0122483,'PDT000675167.1':0.00847091)'':0.00682868)'':0.00470175)'':0.00183424)'':0.0120146)'':0.013088)'':0.0148)'':0.000639564,(('PDT000632567.1':0.0478509,'PDT000298010.1':0.0574881)'':0.0141683,((('PDT000469694.1':0.0613163,('PDT000691321.1':0.0498867,'PDT000168839.1':0.0494668)'':0.0100079)'':0.020896,(('PDT000590018.1':0.039939,'PDT000216781.1':0.0370211)'':0.0132024,(('PDT000375427.1':0.0233339,'PDT000428839.1':0.0370926)'':0.0312381,(('PDT000431875.1':0.0127344,'PDT000431888.1':0.0206255)'':0.0390784,(('PDT000210708.1':0.0281223,'PDT000236050.1':0.0329359)'':0.0100185,('PDT000218721.1':0.0170812,'PDT000236048.1':0.0191592)'':0.0198187)'':0.00673552)'':0.00462561)'':0.00649774)'':0.00542961)'':-0.00430433,((('PDT000123365.1':0.068371,('PDT000523077.1':0.0301481,'PDT000276842.1':0.0310249)'':0.026392)'':0.00259481,('PDT000637954.1':0.0616754,(('PDT000087335.2':0.0469146,('PDT000431877.1':0.0332596,'PDT000298000.1':0.0343499)'':0.00727084)'':0.00168773,(('PDT000632698.1':0.0277322,'PDT000633988.1':0.0287045)'':0.00568589,('PDT000310772.1':0.0426179,('PDT000632628.1':0.0194396,'PDT000632708.1':0.0130678)'':0.0192337)'':0.00127714)'':0.00598196)'':0.019188)'':-0.00333786)'':0.00215739,((('PDT000444845.1':0.0157222,'PDT000216783.1':0.0233957)'':0.0167785,(('PDT000675051.1':0.0202906,'PDT000298008.1':0.0198586)'':0.00621486,('PDT000431887.1':0.00336523,('PDT000682324.1':0.00206168,'PDT000298007.1':0.00232866)'':0.00151871)'':0.0219188)'':0.0132079)'':0.0104298,(('PDT000283183.1':0.0476394,('PDT000717048.1':0.0468257,('PDT000675137.1':0.0412433,(('PDT000675255.1':0.0275387,'PDT000391851.1':0.0311646)'':0.0055458,('PDT000431908.1':0.0373896,('PDT000257785.1':0.0300079,('PDT000212799.1':0.0204438,'PDT000216789.1':0.0165093)'':0.0147771)'':0.00362163)'':0.00182949)'':0.00610678)'':0.00309015)'':0.0026811)'':-0.00198026,((('PDT000534778.1':0.0237035,'PDT000675505.1':0.0210682)'':0.0161347,('PDT000675027.1':0.0114831,('PDT000682323.1':0.00285333,'PDT000415845.1':0.00263405)'':0.00376277)'':0.0252973)'':0.0103959,('PDT000292880.1':0.0684176,('PDT000130444.2':0.0322233,('PDT000236051.1':0.0215424,('PDT000289949.3':0.0134742,('PDT000674996.1':0.00293524,('PDT000236049.1':0.00102035,'PDT000292877.1':0.00117855)'':0.002055)'':0.00945745)'':0.00420443)'':0.0183176)'':0.00953023)'':0.00506999)'':0.00252815)'':0.00336723)'':0.00136751)'':0.0038234)'':0.00552856)'':-0.00185225)'':0.0162313)'':0.0101,(('PDT000521066.1':0.0828996,('PDT000225238.1':0.0679747,(('PDT000716835.1':0.0204214,('PDT000520694.1':0.00657166,'PDT000111377.1':0.00669914)'':0.00963297)'':0.00963354,('PDT000714024.1':0.0333305,('PDT000067790.1':0.0249338,(('PDT000714566.1':0.00864648,'PDT000340380.1':0.00551672)'':0.0137795,('PDT000363558.1':0.0133168,('PDT000426414.1':0.0141291,'PDT000521003.1':0.0136707)'':0.00185672)'':0.00242566)'':0.00690223)'':0.00765628)'':-0.00457604)'':0.0205773)'':0.0402001)'':0.00167243,((('PDT000376260.1':0.0657158,('PDT000174336.1':0.0136382,'PDT000386996.1':0.0123539)'':0.0468797)'':0.00585833,('PDT000326062.1':0.0724375,('PDT000109301.2':0.0522344,(('PDT000193786.2':0.0387062,('PDT000102021.2':0.0224503,('PDT000674843.1':0.0196638,'PDT000675338.1':0.0222323)'':0.00958963)'':0.00793247)'':0.0189282,('PDT000307430.2':0.0502828,(('PDT000131849.2':0.0319491,('PDT000176204.2':0.036094,'PDT000006511.2':0.0369315)'':0.00168983)'':0.00112266,('PDT000426192.1':0.0388003,('PDT000212810.1':0.0312375,'PDT000294790.1':0.0212059)'':0.0119183)'':0.00350428)'':0.0116895)'':0.00152784)'':0.00779749)'':0.00675404)'':0.000899071)'':0.00536138,((('PDT000270536.1':0.063444,'PDT000292881.1':0.056621)'':0.0442611,(('PDT000215976.1':0.103588,('PDT000202720.2':0.0655542,'PDT000661826.1':0.0522738)'':0.0176604)'':0.002373,('PDT000662547.1':0.0891802,(('PDT000202024.2':0.0396555,'PDT000648044.1':0.0434925)'':0.0270361,('PDT000020945.1':0.0765006,('PDT000411325.1':0.053126,('PDT000363551.1':0.0436182,('PDT000111382.1':0.0467005,'PDT000363567.1':0.0434808)'':0.00278492)'':0.00804039)'':0.0181118)'':-0.00124513)'':0.0064091)'':0.0028042)'':0.0141432)'':-0.00719479,('PDT000244078.1':0.151305,((('PDT000442392.1':0.0641018,'PDT000443849.1':0.0691442)'':0.0107242,((('PDT000202743.2':0.0489504,('PDT000102983.1':0.0322758,'PDT000482863.1':0.0300174)'':0.00731698)'':0.0293259,(('PDT000437905.1':0.0780865,('PDT000343756.1':0.0773592,'PDT000426614.1':0.0511598)'':0.012086)'':-0.00168852,('PDT000042516.1':0.0566589,('PDT000426269.1':0.0481091,('PDT000175804.2':0.0488886,('PDT000354228.1':0.0398705,('PDT000437994.1':0.03194,'PDT000363574.1':0.0310445)'':0.00313212)'':0.0074064)'':0.00675943)'':0.00591166)'':0.00767142)'':0.00532039)'':0.00462181,(('PDT000427046.1':0.0757904,'PDT000042700.1':0.0857846)'':0.0106406,(('PDT000438362.1':0.059241,'PDT000633452.1':0.052263)'':0.0175954,(('PDT000228325.1':0.0219621,'PDT000448220.1':0.0207101)'':0.0491562,(('PDT000299993.2':0.0739393,('PDT000176205.2':0.0422623,'PDT000442394.1':0.0569305)'':0.0228963)'':-0.00178645,('PDT000441357.1':0.0653843,(('PDT000202702.2':0.0310047,'PDT000426149.1':0.0263289)'':0.0142784,('PDT000267156.1':0.0423728,'PDT000501874.1':0.0331875)'':0.00572839)'':0.0152563)'':0.00106997)'':0.00640813)'':0.00461401)'':0.00542059)'':-0.00693836)'':0.0119599)'':-0.00944231,(('PDT000168723.1':0.0892408,('PDT000668933.1':0.0574361,(('PDT000459425.1':0.0324981,('PDT000155059.1':0.0253774,'PDT000325635.1':0.0391612)'':0.00359671)'':0.00652261,('PDT000673465.1':0.0360397,(('PDT000523063.1':0.00962929,'PDT000621046.1':0.0160194)'':0.00347229,('PDT000426994.1':0.016439,('PDT000202761.2':0.0115437,'PDT000615939.1':0.00837073)'':0.00325878)'':7.20371e-05)'':0.0229169)'':0.00521853)'':0.00657501)'':0.0152943)'':-0.00227978,(('PDT000651167.1':0.0635033,'PDT000006420.2':0.0605317)'':0.0082399,((('PDT000423845.1':0.0501086,'PDT000615929.1':0.0444236)'':0.00773561,('PDT000426375.1':0.0492479,'PDT000616012.1':0.0481015)'':0.0108064)'':0.00701369,('PDT000267150.1':0.0729931,(('PDT000067791.1':0.0810251,('PDT000615811.1':0.0324192,('PDT000425735.1':0.0215703,'PDT000426619.1':0.0284642)'':0.00730136)'':0.0293901)'':-0.00509826,('PDT000212798.1':0.0640281,(('PDT000615904.1':0.0331195,('PDT000175801.2':0.0111111,'PDT000425614.1':0.0135839)'':0.0230861)'':0.00879714,('PDT000414007.1':0.0271103,('PDT000364527.1':0.033114,('PDT000615958.1':0.012416,'PDT000408005.1':0.0177333)'':0.00718058)'':0.00525991)'':0.0132647)'':0.00942789)'':0.0038924)'':0.00890821)'':-0.00516915)'':0.00489207)'':-0.00270237)'':0.00543283)'':0.0350733)'':-0.0253627)'':0.00138491)'':0.00416653)'':-0.00282381)'':0.00850008)'':0.0223144)'':0.00200155,((('PDT000432011.1':0.0698319,('PDT000181688.2':0.0465637,'PDT000543120.1':0.0400467)'':0.00854885)'':0.00494691,(('PDT000067779.1':0.0466459,(('PDT000423879.1':0.0390706,'PDT000340381.1':0.0321007)'':0.0015457,('PDT000519802.1':0.0454478,('PDT000634132.1':0.0344932,('PDT000535470.1':0.0265404,'PDT000566582.1':0.0232624)'':0.00187248)'':0.00516193)'':0.00370579)'':0.00288005)'':0.00368843,('PDT000424013.1':0.0679221,(('PDT000445992.1':0.053093,'PDT000713923.1':0.0356529)'':0.00467554,('PDT000510873.1':0.0488878,(('PDT000489645.2':0.0316266,('PDT000618971.1':0.0254888,'PDT000633268.1':0.0269164)'':0.00345929)'':0.00779675,(('PDT000181721.2':0.0343352,'PDT000286430.1':0.0369003)'':0.00453877,('PDT000191153.1':0.050305,((('PDT000633476.1':0.0327968,'PDT000583358.1':0.0287237)'':0.00458703,('PDT000674838.1':0.0047879,'PDT000297997.1':0.0075738)'':0.0374127)'':0.00172509,('PDT000423897.1':0.0418454,(('PDT000433707.1':0.0187678,'PDT000444877.1':0.0133432)'':0.00609736,('PDT000543094.1':0.0219621,('PDT000417619.1':0.0272301,('PDT000652346.1':0.0165423,(('PDT000425744.1':0.00944476,'PDT000652356.1':0.0112703)'':0.00474561,(('PDT000423872.1':0.00445759,'PDT000615902.1':0.00892101)'':0.00397144,('PDT000175993.2':0.00551378,'PDT000615940.1':0.00706732)'':0.00531323)'':0.00249987)'':0.00121813)'':0.00281038)'':0.0034756)'':-0.00168554)'':0.00729769)'':0.00352214)'':0.00298712)'':-0.002953)'':0.00152093)'':0.00608338)'':0.000339811)'':0.00999098)'':-0.0048438)'':0.00337964)'':0.0690069,(('PDT000298090.1':0.166416,((('PDT000425985.1':0.0683716,('PDT000463637.1':0.0380801,'PDT000463699.1':0.040389)'':0.0207278)'':0.00698445,('PDT000417143.1':0.074719,(('PDT000620988.1':0.0550678,'PDT000257160.1':0.0506862)'':0.000950205,('PDT000661177.1':0.0474099,('PDT000621050.1':0.0457558,'PDT000394411.1':0.0389142)'':0.00822156)'':0.00550598)'':0.00918227)'':0.000935125)'':0.0208992,((('PDT000673697.1':0.0441819,'PDT000407918.1':0.0358964)'':0.0278784,(('PDT000432552.1':0.0825738,(('PDT000720772.1':0.0486629,'PDT000270457.1':0.0534031)'':0.0101711,('PDT000425508.1':0.0562598,('PDT000218517.2':0.0480127,(('PDT000566233.1':0.0341821,'PDT000687464.1':0.0338425)'':0.0141956,('PDT000188157.2':0.0488683,('PDT000668932.1':0.0339761,'PDT000252532.1':0.0382897)'':0.00734997)'':0.00301972)'':0.00368145)'':0.00482023)'':0.00368293)'':0.00445916)'':-0.00324236,(('PDT000621008.1':0.0504311,'PDT000621009.1':0.0556399)'':0.00918295,(('PDT000346107.1':0.0647409,('PDT000656975.1':0.0688502,(('PDT000239130.2':0.0548293,('PDT000519808.1':0.0357476,'PDT000069701.1':0.0341568)'':0.00846441)'':-0.00106912,('PDT000542615.1':0.0342762,('PDT000708204.1':0.0309984,('PDT000583441.1':0.011368,'PDT000463603.1':0.0133874)'':0.010602)'':0.00428043)'':0.00848173)'':0.00817286)'':0.00272519)'':-0.00255027,((('PDT000181672.2':0.0548979,('PDT000656956.1':0.00520845,'PDT000656962.1':0.00806765)'':0.0358972)'':0.0119236,('PDT000645549.2':0.0566111,('PDT000294378.1':0.0411587,('PDT000327856.1':0.0249449,'PDT000411315.1':0.0271568)'':0.00791876)'':0.0130295)'':0.000421164)'':0.000952748,(('PDT000496889.1':0.0433253,'PDT000365436.1':0.0502571)'':0.00708263,('PDT000433484.1':0.0567918,((('PDT000521035.1':0.034568,'PDT000227976.1':0.0418195)'':0.0146607,('PDT000202617.2':0.0344681,('PDT000202051.2':0.0247153,'PDT000202600.2':0.0171836)'':0.0138773)'':0.00663466)'':0.00285271,(('PDT000639575.1':0.0447659,'PDT000689963.1':0.0376672)'':0.0119663,('PDT000661547.1':0.0432821,('PDT000521088.1':0.0422134,('PDT000202222.2':0.0451929,('PDT000137408.2':0.0294494,('PDT000425776.1':0.0281843,'PDT000425795.1':0.0213148)'':0.011005)'':0.00186)'':0.000968892)'':0.00328854)'':0.00107991)'':0.00095541)'':0.00561059)'':-0.00332885)'':0.000529815)'':0.003305)'':0.00490176)'':-0.00197727)'':0.0118599)'':-0.00355317,((('PDT000426663.1':0.0907257,'PDT000675662.1':0.0813473)'':0.0171321,(('PDT000672251.1':0.0499574,('PDT000360992.1':0.0425596,('PDT000645653.2':0.0429392,('PDT000645555.2':0.0328276,'PDT000645570.2':0.0294356)'':0.00791429)'':0.00668434)'':0.0126637)'':0.0172061,((('PDT000188190.2':0.0492819,'PDT000124650.1':0.048748)'':0.00204949,('PDT000703266.1':0.0376388,'PDT000186173.1':0.0506425)'':0.0103921)'':0.00920871,((('PDT000181837.2':0.048514,'PDT000651226.1':0.0478773)'':0.00906716,(('PDT000417161.1':0.0408264,'PDT000346865.1':0.0423707)'':0.0117337,('PDT000346855.1':0.046503,'PDT000346856.1':0.056609)'':0.00864561)'':0.00554968)'':0.00273735,((('PDT000283895.2':0.0427594,('PDT000661352.1':0.0233105,'PDT000485943.1':0.0265759)'':0.0098192)'':0.0167836,(('PDT000720460.1':0.0423671,'PDT000626960.1':0.0356436)'':0.0100838,(('PDT000365702.1':0.0341809,'PDT000128862.2':0.0324337)'':0.00647397,('PDT000202669.2':0.0337139,'PDT000426455.1':0.0259035)'':0.0109288)'':0.00468354)'':0.00459584)'':0.00040575,(('PDT000415660.1':0.0522052,'PDT000707308.1':0.0532628)'':0.00871956,(('PDT000685882.1':0.0586101,('PDT000499668.1':0.0231786,'PDT000125963.1':0.0312793)'':0.011736)'':0.00860609,('PDT000289921.1':0.0613359,('PDT000243011.1':0.0581292,(('PDT000069691.1':0.0421474,('PDT000411340.1':0.037559,('PDT000425640.1':0.0298759,'PDT000426604.1':0.0277902)'':0.00593764)'':0.00629016)'':-0.000323567,('PDT000598038.1':0.0406707,(('PDT000084300.3':0.0280298,'PDT000459679.1':0.0362587)'':0.00675254,('PDT000497390.1':0.0350464,('PDT000390645.1':0.0338832,'PDT000537884.1':0.025108)'':0.00384738)'':0.00315255)'':0.00387506)'':-0.00125982)'':0.00677345)'':0.00173042)'':-0.000462217)'':0.00281388)'':0.00103556)'':0.00372994)'':0.00315284)'':0.00198868)'':0.0116379)'':-0.0127877,('PDT000346846.1':0.117516,((('PDT000708218.1':0.0542489,('PDT000238529.1':0.0613482,('PDT000367300.1':0.0517886,('PDT000615917.1':0.0380086,('PDT000499454.1':0.0291469,'PDT000718654.1':0.0337841)'':0.00421998)'':0.00659521)'':0.0121399)'':0.00327805)'':0.00416036,('PDT000409994.1':0.0637195,(('PDT000248462.1':0.0481378,('PDT000604348.1':0.0367071,('PDT000520797.1':0.0362318,('PDT000024319.2':0.0309421,'PDT000241786.1':0.0281621)'':0.00454127)'':0.006617)'':0.00412084)'':0.00973219,('PDT000718298.1':0.0574346,(('PDT000128845.2':0.0275979,'PDT000595101.1':0.0306963)'':0.0103349,('PDT000425920.1':0.0328331,'PDT000384970.1':0.0292157)'':0.00591323)'':0.0101423)'':-0.000731724)'':0.00844136)'':0.00678035)'':-0.000290421,(('PDT000198196.3':0.0591026,'PDT000221999.3':0.0692494)'':0.0269387,(('PDT000619098.1':0.053694,(('PDT000397679.1':0.0507234,('PDT000321331.1':0.0477562,('PDT000426649.1':0.0476309,('PDT000426496.1':0.039896,'PDT000124710.1':0.0397218)'':0.0069652)'':0.00176922)'':0.00324877)'':0.000164107,(('PDT000128816.2':0.0498676,'PDT000652871.1':0.0524234)'':0.00404493,(('PDT000425513.1':0.0296957,'PDT000425623.1':0.0303334)'':0.0067494,('PDT000426212.1':0.0331891,'PDT000122112.1':0.0430135)'':0.00287765)'':0.00772784)'':0.00336815)'':0.00446341)'':-0.000325038,('PDT000414190.1':0.0795634,('PDT000189723.1':0.0610876,(('PDT000448222.1':0.054395,('PDT000202043.2':0.0390118,('PDT000716781.1':0.0253532,('PDT000202081.2':0.0102452,'PDT000352524.1':0.0145977)'':0.012252)'':0.0104346)'':0.0154258)'':0.000281764,((('PDT000672228.1':0.032549,'PDT000396751.1':0.0255837)'':0.0136076,('PDT000538575.1':0.0423035,(('PDT000423990.1':0.031448,'PDT000716782.1':0.0299838)'':0.010166,('PDT000674967.1':0.0364104,'PDT000620998.1':0.0408019)'':0.00095267)'':0.00304107)'':0.00446278)'':0.00017136,('PDT000129097.2':0.0528608,('PDT000346521.1':0.0467432,((('PDT000291147.2':0.0276888,'PDT000426667.1':0.0274224)'':0.0106611,('PDT000426276.1':0.0473161,('PDT000425488.1':0.0191694,'PDT000426482.1':0.0167311)'':0.0213485)'':5.02532e-05)'':0.00206892,('PDT000632288.1':0.0479138,(('PDT000128823.2':0.031778,'PDT000682552.1':0.0343)'':0.00421745,('PDT000525363.1':0.0408023,('PDT000101125.2':0.0168301,'PDT000425792.1':0.0160004)'':0.0187249)'':0.00100949)'':0.00323332)'':-0.00204135)'':0.00410412)'':0.00370037)'':-0.00425371)'':0.00283022)'':0.00455608)'':0.00687004)'':-0.0084657)'':0.0203029)'':-0.0116216)'':0.0409003)'':-0.0395453)'':0.00232463)'':0.0064287)'':0.053542)'':-0.0218783,((('PDT000459420.1':0.117688,((('PDT000238528.1':0.0896897,('PDT000188138.2':0.0774129,'PDT000397604.1':0.0736791)'':0.0112788)'':0.000499707,('PDT000434582.1':0.0728226,('PDT000618888.1':0.0587648,('PDT000136077.2':0.0538974,'PDT000423937.1':0.0422453)'':0.00239069)'':0.0112351)'':0.0263216)'':0.00922424,(('PDT000626936.1':0.080273,('PDT000309314.2':0.0644879,'PDT000556874.1':0.0693461)'':0.018897)'':0.0148037,(('PDT000636201.1':0.0717398,('PDT000657034.1':0.0334507,'PDT000019601.1':0.0349826)'':0.0253025)'':0.0280768,(('PDT000425506.1':0.0501835,'PDT000719077.1':0.0702145)'':0.0131851,(('PDT000425852.1':0.0459272,'PDT000257781.1':0.0492654)'':0.0180417,('PDT000248847.1':0.0523384,('PDT000202160.2':0.0219505,('PDT000425440.1':0.00321469,'PDT000425751.1':0.00527767)'':0.0209966)'':0.0306306)'':-0.0032224)'':0.00715693)'':0.0248321)'':0.00407622)'':0.00109643)'':0.00808564)'':-0.00538821,((('PDT000346944.1':0.0716009,('PDT000535479.1':0.052254,('PDT000150945.2':0.0495594,('PDT000675543.1':0.0403209,('PDT000634557.1':0.0413026,'PDT000212786.1':0.0394655)'':0.00455145)'':0.00287351)'':0.00737567)'':0.00451196)'':0.00800982,(('PDT000431885.1':0.00842508,'PDT000589865.1':0.00565102)'':0.0655041,(('PDT000430775.1':0.0430364,'PDT000535264.1':0.0398296)'':0.00688097,('PDT000632355.1':0.0449168,('PDT000522708.1':0.0296349,('PDT000427028.1':0.00884088,'PDT000340382.1':0.00717582)'':0.0237711)'':0.014051)'':0.00183004)'':0.0129761)'':-8.74568e-05)'':0.0201086,((((('PDT000500721.1':0.0410215,'PDT000107781.1':0.0300199)'':0.0266783,('PDT000425505.1':0.0273563,('PDT000675387.1':0.0206068,('PDT000434541.1':0.0142421,'PDT000652574.1':0.0179586)'':0.00213698)'':0.00656529)'':0.0410241)'':0.00244293,('PDT000633508.1':0.0509811,('PDT000191156.1':0.0645894,(('PDT000136157.2':0.0381424,'PDT000433737.1':0.0355854)'':0.0025512,('PDT000535466.1':0.00920358,'PDT000322913.1':0.00603702)'':0.0321384)'':0.00552412)'':0.00472379)'':0.0176776)'':0.00219865,(('PDT000423946.1':0.0677911,'PDT000285167.1':0.0470799)'':0.0111776,(('PDT000433763.1':0.0530208,('PDT000537829.1':0.0259133,('PDT000425495.1':0.0321206,('PDT000206252.3':0.00993874,'PDT000537938.1':0.0109627)'':0.0159756)'':0.0025152)'':0.0284926)'':0.000464005,(('PDT000656976.1':0.0218212,'PDT000675638.1':0.0205567)'':0.0139231,('PDT000669027.1':0.0080664,('PDT000188125.2':0.00730959,'PDT000615977.1':0.011975)'':0.00112455)'':0.0150437)'':0.0261874)'':0.0060547)'':0.000980265)'':0.00486991,(('PDT000648091.1':0.114431,('PDT000208669.1':0.0834271,('PDT000417193.1':0.0612029,'PDT000633273.1':0.0534311)'':0.0149619)'':0.00893037)'':0.0128606,((('PDT000589769.1':0.0598752,('PDT000203401.2':0.0482201,'PDT000426583.1':0.0435835)'':0.0101397)'':0.0157054,('PDT000633481.1':0.0496093,('PDT000633393.1':0.0329864,'PDT000396752.1':0.0410622)'':0.00445522)'':0.0292357)'':0.00221682,((('PDT000427239.1':0.0661665,('PDT000713931.1':0.0493525,('PDT000202631.2':0.0419549,('PDT000253658.1':0.0202745,'PDT000384972.1':0.0256378)'':0.0134249)'':0.0118755)'':0.0167245)'':0.00231866,('PDT000565514.2':0.074285,('PDT000291284.1':0.0739584,('PDT000433288.1':0.0535112,('PDT000443065.1':0.0413077,('PDT000425859.1':0.0363597,'PDT000502337.1':0.0357254)'':0.00382998)'':0.0156749)'':0.00738115)'':0.00384769)'':-0.00660263)'':0.00605858,(('PDT000535525.1':0.0810736,('PDT000433598.1':0.0601459,('PDT000069130.2':0.0441465,('PDT000175855.2':0.0297995,'PDT000203403.2':0.0284227)'':0.00536924)'':0.0215394)'':0.0101868)'':-0.00327445,(('PDT000346091.1':0.0642641,('PDT000042697.1':0.0614635,('PDT000709972.1':0.0410828,('PDT000241787.1':0.0328338,'PDT000285158.1':0.0312061)'':0.00586962)'':0.0108848)'':0.00810939)'':0.00649462,(('PDT000661286.1':0.0458118,('PDT000661108.1':0.0344221,('PDT000423928.1':0.0133403,'PDT000425703.1':0.0162193)'':0.0175315)'':0.0148449)'':0.0169557,(('PDT000425601.1':0.04229,('PDT000589034.1':0.0297258,'PDT000228338.1':0.0407966)'':0.010856)'':0.00398416,('PDT000111334.1':0.0610533,('PDT000360981.1':0.0438071,('PDT000175929.2':0.0176973,('PDT000615863.1':0.00747448,'PDT000363571.1':0.00614652)'':0.00739303)'':0.0270891)'':0.0102824)'':-0.00312892)'':0.00786911)'':0.00334115)'':0.00513936)'':-0.000880579)'':0.00600068)'':0.00850973)'':-0.00662096)'':0.00922894)'':0.0109562)'':0.00391135,(((('PDT000713591.1':0.0724137,('PDT000645534.2':0.055206,'PDT000111359.1':0.05563)'':0.0137493)'':0.0157052,(('PDT000648092.1':0.0716971,('PDT000202109.2':0.0213553,'PDT000502172.1':0.0190137)'':0.0288819)'':0.0114269,(('PDT000168866.1':0.0052972,('PDT000168784.1':0.00251099,'PDT000168808.1':0.00168565)'':0.000797316)'':0.0868696,('PDT000042564.1':0.0607032,('PDT000687957.1':0.0550112,('PDT000692400.1':0.0488011,('PDT000713432.1':0.0304264,('PDT000520984.1':0.0403876,(('PDT000202661.2':0.0205433,'PDT000020074.1':0.0237599)'':0.00272262,('PDT000713465.1':0.0132928,'PDT000482614.1':0.0123913)'':0.0053167)'':0.00689167)'':0.00665148)'':0.00475725)'':0.00710717)'':0.0149417)'':-0.00286996)'':0.00742349)'':0.00751677)'':0.0139944,((('PDT000617727.1':0.0560639,('PDT000305149.2':0.0437284,'PDT000312799.2':0.0415451)'':0.0372129)'':0.00659677,(('PDT000077139.2':0.0662201,'PDT000409823.1':0.0608199)'':0.00320597,('PDT000538566.1':0.0605129,('PDT000538609.1':0.0430114,('PDT000720218.1':0.0345826,('PDT000354226.1':0.0311042,('PDT000290519.2':0.0218768,'PDT000660658.1':0.0213967)'':0.00350042)'':0.00667833)'':0.00775562)'':0.0019146)'':0.0161121)'':0.00664275)'':0.0128104,(((('PDT000563589.1':0.0260147,'PDT000045614.1':0.0319492)'':0.03083,('PDT000045608.1':0.0434984,'PDT000720269.1':0.0495293)'':0.0140047)'':0.0107415,('PDT000714675.1':0.045876,('PDT000631843.1':0.0448991,('PDT000565962.1':0.0229669,'PDT000011788.2':0.0290505)'':0.0147801)'':0.00514255)'':0.0111394)'':0.0121811,((('PDT000337148.1':0.0437964,'PDT000496887.1':0.0498937)'':0.0258049,('PDT000202072.2':0.0530033,('PDT000485913.1':0.0250829,('PDT000439469.1':0.0217791,'PDT000616145.1':0.0159567)'':0.0188171)'':0.0267923)'':0.00944705)'':0.0123067,('PDT000257779.1':0.080146,('PDT000238526.1':0.0568883,('PDT000346066.1':0.0623212,(('PDT000305515.2':0.0231702,'PDT000310433.2':0.0228226)'':0.0264865,('PDT000403150.1':0.0515794,('PDT000271380.2':0.0328592,'PDT000398012.1':0.0394977)'':0.00939088)'':0.00113412)'':0.00289733)'':0.00990802)'':0.0135553)'':-0.000616403)'':0.00434184)'':0.00267049)'':0.0120569)'':0.000385766,((('PDT000661263.1':0.126969,((('PDT000346952.1':0.0542361,('PDT000709909.1':0.0309955,'PDT000111332.1':0.0228219)'':0.0200386)'':0.00714904,('PDT000355101.1':0.050448,('PDT000661339.1':0.0365376,('PDT000384994.1':0.00614254,('PDT000384981.1':0.005745,'PDT000384996.1':0.0054988)'':0.00239286)'':0.0324163)'':0.00873749)'':0.00139365)'':0.0114341,(('PDT000719048.1':0.0610951,('PDT000227990.1':0.0301144,('PDT000430824.1':0.0273728,'PDT000517266.1':0.0296336)'':0.00403357)'':0.0221519)'':-0.00304428,(('PDT000532398.1':0.0300474,'PDT000664461.1':0.0384108)'':0.0236675,(('PDT000275528.2':0.0310725,'PDT000655778.1':0.0278547)'':0.0123468,('PDT000499464.1':0.0143244,'PDT000662546.1':0.0148886)'':0.0173173)'':0.00718211)'':0.00412098)'':0.0226663)'':0.0495542)'':-0.0303571,((('PDT000583401.1':0.00931556,'PDT000583404.1':0.00448304)'':0.0736798,('PDT000425511.1':0.0740116,('PDT000172357.2':0.0690245,'PDT000326054.1':0.0749465)'':0.0109844)'':0.00798905)'':0.014019,(('PDT000668403.2':0.0401385,('PDT000466088.1':0.0268723,('PDT000649254.1':0.0310357,'PDT000352476.1':0.0215229)'':0.00147529)'':0.00493176)'':0.0337975,(('PDT000396796.1':0.0588279,('PDT000251759.1':0.0403147,'PDT000251953.1':0.0544978)'':0.0173969)'':0.00912708,('PDT000668358.2':0.0414003,('PDT000124598.1':0.048287,(('PDT000008734.2':0.0317886,'PDT000228203.1':0.0367019)'':0.00241685,('PDT000227995.1':0.0397488,('PDT000718331.1':0.0195047,('PDT000467018.1':0.0173475,'PDT000510060.1':0.0192523)'':0.00535481)'':0.0045396)'':0.00506711)'':0.00496445)'':0.000946845)'':0.0243098)'':0.00108279)'':0.017964)'':0.00320536)'':0.00244512,(((('PDT000675224.1':0.0756659,'PDT000675518.1':0.0701171)'':0.010525,('PDT000590056.1':0.0648702,('PDT000682350.1':0.0681713,'PDT000352482.1':0.0537537)'':0.00440579)'':0.00924125)'':0.0108492,('PDT000421801.1':0.0689198,(('PDT000445090.1':0.0544812,'PDT000682320.1':0.0451272)'':0.00609425,('PDT000675651.1':0.0505951,'PDT000712683.1':0.0459071)'':0.00761283)'':0.0209308)'':0.0186683)'':0.016186,((('PDT000520982.1':0.0537138,'PDT000240704.1':0.0722852)'':0.0541867,(('PDT000394922.1':0.0734711,('PDT000564919.1':0.0679067,('PDT000258731.2':0.0624504,'PDT000426618.1':0.0515196)'':0.00858978)'':0.00382979)'':0.0160063,('PDT000627591.1':0.0914742,(('PDT000433601.1':0.0770359,('PDT000397630.1':0.0286046,'PDT000645055.1':0.0373972)'':0.0458097)'':0.00299726,(('PDT000367276.1':0.0519843,('PDT000318978.1':0.0527032,'PDT000463701.1':0.0512138)'':0.00733619)'':0.00510451,('PDT000397570.1':0.0491859,('PDT000500485.1':0.0484198,('PDT000303123.2':0.0237835,'PDT000355107.1':0.0198194)'':0.0216775)'':0.0194256)'':0.00469127)'':0.0125937)'':0.00805906)'':-0.00298669)'':0.0309931)'':-0.0237276,((('PDT000719194.1':0.0924972,('PDT000425718.1':0.0788416,('PDT000557135.1':0.0478888,'PDT000714670.1':0.0494758)'':0.0250546)'':0.016677)'':-0.00332445,('PDT000439470.1':0.110244,(('PDT000216076.2':0.0759532,'PDT000045591.1':0.0690618)'':0.00510105,(('PDT000124616.1':0.0671652,'PDT000250058.1':0.0614758)'':0.00952799,('PDT000250071.1':0.0663276,(('PDT000636241.1':0.0535952,('PDT000380276.1':0.051474,'PDT000251918.1':0.052944)'':0.00658377)'':0.00282275,('PDT000354921.1':0.0597484,('PDT000396827.1':0.0605231,('PDT000719245.1':0.0368329,('PDT000510038.1':0.0385836,'PDT000249966.1':0.03066)'':0.00186779)'':0.00854009)'':0.00680922)'':0.00125852)'':0.00915072)'':0.00203721)'':0.0159373)'':0.00946281)'':-0.00159674)'':-0.00351662,(((('PDT000426609.1':0.0699507,(('PDT000181740.2':0.046892,'PDT000535477.1':0.0392648)'':0.0257909,('PDT000718299.1':0.0624654,('PDT000713284.1':0.0457967,'PDT000352526.1':0.0516987)'':0.00759188)'':0.00590597)'':0.0101073)'':0.000548089,(('PDT000425826.1':0.0555225,'PDT000124666.1':0.0622195)'':0.0350451,(('PDT000346832.1':0.0669916,('PDT000251584.1':0.0596466,('PDT000556896.1':0.0472234,'PDT000661152.1':0.0474403)'':0.0100365)'':0.00451991)'':0.00181487,(('PDT000362861.1':0.0479224,('PDT000644895.1':0.0471615,('PDT000541647.1':0.0305826,'PDT000639578.1':0.0320395)'':0.00259283)'':0.021352)'':0.0117007,(('PDT000689910.1':0.0480321,'PDT000008769.2':0.0508407)'':0.0102439,((('PDT000556886.1':0.0526958,'PDT000556994.1':0.0552992)'':0.000616042,('PDT000583466.1':0.0469221,'PDT000250164.1':0.0440835)'':0.00751116)'':0.00519313,('PDT000251854.1':0.0760058,('PDT000364521.1':0.0680179,(('PDT000632569.1':0.0434208,('PDT000482060.1':0.0279566,'PDT000248484.1':0.0443326)'':0.0100857)'':0.00671019,('PDT000466970.1':0.0378007,('PDT000380279.1':0.0310779,'PDT000556918.1':0.0381438)'':0.00298007)'':0.00788262)'':0.0104488)'':0.00092568)'':-0.000424257)'':0.00125644)'':0.000782456)'':0.00209818)'':0.0160166)'':-0.0101953)'':-0.000389112,(('PDT000537802.1':0.0605138,('PDT000704982.1':0.0560033,('PDT000181726.2':0.0125065,'PDT000459517.1':0.0103557)'':0.0422626)'':0.00136869)'':0.00793183,(('PDT000318981.1':0.0408758,('PDT000520979.1':0.0374288,'PDT000319038.1':0.0324904)'':0.0162945)'':0.023127,(('PDT000202001.2':0.0554405,'PDT000429808.1':0.0507295)'':0.00465207,((('PDT000177244.1':0.070557,'PDT000346868.1':0.059224)'':0.0105722,('PDT000212783.1':0.0501138,('PDT000568773.1':0.038657,'PDT000171419.1':0.0292442)'':0.0134772)'':0.0120972)'':0.0027089,((('PDT000423943.1':0.0508686,('PDT000423995.1':0.0420625,'PDT000046969.2':0.0345699)'':0.00523364)'':0.0141529,(('PDT000181820.2':0.0463435,('PDT000124640.1':0.0377858,('PDT000181813.2':0.0299192,'PDT000661165.1':0.0366244)'':0.00824003)'':0.00475912)'':-0.00276471,(('PDT000535656.1':0.0367408,'PDT000625989.1':0.028323)'':0.00692878,('PDT000124610.1':0.0338043,(('PDT000201873.2':0.00420469,'PDT000633223.1':0.00596781)'':0.0119782,('PDT000425829.1':0.0155113,'PDT000615892.1':0.0172594)'':0.00459937)'':0.0149461)'':-0.00124428)'':0.00775744)'':0.00406318)'':0.00187781,((('PDT000621005.1':0.044676,('PDT000619660.1':0.0267223,'PDT000543108.1':0.0332368)'':0.00648923)'':0.00291865,('PDT000460007.1':0.046235,(('PDT000128838.2':0.0264142,'PDT000426114.1':0.0314103)'':0.0122438,(('PDT000202076.2':0.0251942,'PDT000084297.3':0.0288731)'':0.00750517,('PDT000713395.1':0.0217283,'PDT000346956.1':0.0183584)'':0.0103493)'':0.00634526)'':0.00238598)'':0.000998795)'':0.00383707,(('PDT000448172.1':0.053372,('PDT000352471.1':0.0381769,('PDT000175982.2':0.0372394,'PDT000661405.1':0.0407768)'':0.00428177)'':0.0044955)'':0.0024433,('PDT000427517.1':0.0622304,('PDT000471015.1':0.0604494,(('PDT000202074.2':0.0354202,'PDT000425597.1':0.0351044)'':0.00295617,('PDT000425823.1':0.0495206,('PDT000615894.1':0.0295115,('PDT000202718.2':0.034171,('PDT000433243.1':0.0297277,('PDT000042715.1':0.027358,('PDT000425706.1':0.0197069,('PDT000426072.1':0.0199376,'PDT000426436.1':0.0220535)'':0.00114776)'':0.00408305)'':0.00146969)'':0.00706813)'':0.00175144)'':0.00348131)'':0.00166959)'':0.0128634)'':0.0016018)'':-0.00565062)'':0.00101373)'':0.00223567)'':0.00763552)'':0.00737241)'':-0.000868119)'':0.00653246)'':0.00735819)'':0.00807315,(((('PDT000588646.1':0.0759827,('PDT000443376.1':0.0750624,('PDT000488289.1':0.0531778,('PDT000032362.1':0.0362522,'PDT000319037.1':0.0372299)'':0.0129596)'':0.0079535)'':0.00166328)'':0.00567798,(('PDT000175927.2':0.0527954,'PDT000136097.2':0.0566226)'':0.0161462,('PDT000425883.1':0.0646618,(('PDT000656953.1':0.0365988,'PDT000645664.2':0.0462173)'':0.0264436,(('PDT000713784.1':0.0455877,'PDT000304362.1':0.0430818)'':0.00965212,('PDT000136100.2':0.0625921,('PDT000135954.2':0.0334888,('PDT000445101.1':0.0371253,('PDT000202584.2':0.0263708,'PDT000426763.1':0.0302884)'':0.00660395)'':0.00555021)'':0.0136467)'':-0.00131148)'':0.00550697)'':0.00335071)'':0.00251125)'':0.00745875)'':0.00173618,(('PDT000644688.1':0.0632104,('PDT000394817.1':0.0318125,'PDT000003869.3':0.02631)'':0.0403299)'':0.0171741,((('PDT000668917.1':0.0118236,'PDT000668929.1':0.0126294)'':0.0585062,(('PDT000492232.1':0.0425563,'PDT000583467.1':0.0570083)'':0.00425392,(('PDT000397718.1':0.0386666,'PDT000628864.1':0.0509001)'':0.00781618,('PDT000395009.1':0.04882,'PDT000397629.1':0.0474275)'':0.00600819)'':0.00393399)'':0.0141441)'':0.00488567,(('PDT000252158.1':0.07615,('PDT000651578.1':0.0636499,'PDT000645552.2':0.0645721)'':0.010495)'':0.00464886,(('PDT000346848.1':0.0235348,'PDT000346949.1':0.0261814)'':0.0421793,('PDT000271542.2':0.0462096,('PDT000479644.1':0.0395703,'PDT000639582.1':0.0398649)'':0.00443668)'':0.0154747)'':0.00626684)'':0.00554799)'':0.00526491)'':-0.00522939)'':0.00413772,(('PDT000639595.1':0.0785852,('PDT000213235.2':0.0345428,'PDT000397549.1':0.0483307)'':0.0302876)'':0.00829877,(('PDT000675250.1':0.101732,(('PDT000538602.1':0.0696379,'PDT000317408.2':0.0722941)'':0.00357762,('PDT000718833.1':0.0851408,(('PDT000202020.2':0.0307163,'PDT000433807.1':0.0366366)'':0.0353719,('PDT000408754.1':0.0412794,('PDT000323503.1':0.03655,'PDT000455209.1':0.0349683)'':0.0112426)'':0.0119677)'':0.0162505)'':0.000722653)'':0.0136369)'':-0.00896054,((('PDT000346851.1':0.0696316,('PDT000705246.1':0.0555891,'PDT000346087.1':0.0608199)'':0.0120154)'':0.0250042,(('PDT000382830.1':0.0650604,('PDT000251760.1':0.0409137,('PDT000242499.1':0.0122569,'PDT000248672.1':0.0118872)'':0.0444916)'':0.0134796)'':0.0132064,('PDT000346871.1':0.093685,('PDT000395428.1':0.0753102,('PDT000250060.1':0.0720609,'PDT000346106.1':0.0648541)'':0.00809431)'':0.01355)'':-0.00610228)'':0.00614274)'':-0.00134271,((((('PDT000137810.2':0.0563573,'PDT000498735.1':0.0588167)'':0.0087062,('PDT000252639.1':0.0712967,('PDT000354927.1':0.0195,'PDT000399443.1':0.0162214)'':0.0364536)'':0.00140588)'':0.0116582,(('PDT000470051.1':0.0493534,'PDT000228000.1':0.0375733)'':0.031158,((('PDT000354915.1':0.054808,'PDT000430679.1':0.05583)'':0.00699206,('PDT000664777.1':0.0693544,'PDT000720184.1':0.0497346)'':0.00206394)'':0.00909068,(('PDT000646912.1':0.0419599,('PDT000413219.1':0.0330026,'PDT000417412.1':0.030071)'':0.0092913)'':0.00356968,('PDT000720197.1':0.0412983,('PDT000258348.2':0.0294049,('PDT000477157.1':0.0290085,'PDT000644992.1':0.0250032)'':0.00147821)'':0.00714266)'':0.00405033)'':0.0157223)'':0.0089736)'':-0.00192968)'':0.00322033,(('PDT000346072.1':0.0745837,('PDT000644852.1':0.0630852,'PDT000586230.1':0.0505448)'':0.00452734)'':0.00765772,(('PDT000652363.1':0.0542484,'PDT000111336.1':0.0541776)'':0.0159098,(('PDT000231472.2':0.0801386,('PDT000346048.1':0.0647742,('PDT000425451.1':0.043657,'PDT000469966.1':0.062264)'':0.0198518)'':0.00231609)'':0.00133466,(('PDT000332136.1':0.0590801,'PDT000111378.1':0.0615539)'':0.00339616,('PDT000502532.1':0.054325,(('PDT000249863.1':0.0425303,'PDT000395505.1':0.0518185)'':0.00308942,(('PDT000520817.1':0.0551633,'PDT000521069.1':0.0375344)'':0.00709279,((('PDT000316098.1':0.0369793,'PDT000275050.2':0.0366905)'':0.00324867,('PDT000445080.1':0.0500569,'PDT000675266.1':0.0454503)'':0.000624784)'':0.0013712,('PDT000427822.1':0.0330103,('PDT000557658.2':0.0192175,('PDT000535521.1':0.0160089,'PDT000129706.1':0.020721)'':0.00199386)'':0.0137493)'':0.00438418)'':0.00646)'':0.00172861)'':0.00417011)'':-0.000107018)'':0.00606749)'':0.00421983)'':-0.000686964)'':0.00640421)'':0.00337454,((('PDT000667713.2':0.0292144,'PDT000343295.1':0.0380388)'':0.00763749,('PDT000565866.1':0.0444154,'PDT000008590.2':0.0373966)'':0.00261569)'':0.0534277,('PDT000123358.1':0.103129,((('PDT000463687.1':0.0848,('PDT000675450.1':0.0513587,'PDT000409989.1':0.0474718)'':0.0228193)'':-0.000524591,(('PDT000427816.1':0.0601601,'PDT000242915.1':0.0522809)'':0.0165761,('PDT000358748.1':0.0654089,('PDT000242495.1':0.0753306,(('PDT000618735.1':0.0530784,'PDT000646911.1':0.0624756)'':0.00287574,(('PDT000140451.2':0.0538161,('PDT000042604.1':0.0575732,('PDT000440490.1':0.0425254,'PDT000689887.1':0.0451939)'':0.00571462)'':0.00410806)'':0.00033196,('PDT000285160.1':0.0707489,('PDT000524053.1':0.0485114,('PDT000557152.1':0.0485001,('PDT000111323.1':0.0371336,('PDT000719212.1':0.0355685,('PDT000714683.1':0.0314812,'PDT000205427.1':0.0244684)'':0.00905827)'':0.00248635)'':0.005514)'':0.00474223)'':0.00282191)'':-6.07634e-05)'':0.00227441)'':0.00657936)'':0.0054612)'':-0.00533076)'':0.00307192)'':-0.00368052,((('PDT000425739.1':0.0439219,'PDT000500486.1':0.0540302)'':0.00668512,('PDT000644897.1':0.0550048,('PDT000425903.1':0.0510078,('PDT000240701.2':0.047202,'PDT000395280.1':0.0446904)'':0.011237)'':0.00290448)'':0.00378488)'':0.00266994,(('PDT000423344.1':0.0638683,('PDT000607584.1':0.0536296,('PDT000111337.1':0.044024,('PDT000644947.1':0.0395264,'PDT000586224.1':0.0372185)'':0.00244028)'':0.00316498)'':0.00726198)'':-0.00251395,(('PDT000346061.1':0.0504865,('PDT000069157.2':0.0110672,'PDT000399659.1':0.0114259)'':0.037345)'':0.00383154,('PDT000651592.1':0.0728236,(('PDT000003807.2':0.0445882,'PDT000024552.2':0.0379072)'':0.00438102,(('PDT000497360.1':0.0302513,'PDT000628865.1':0.0357763)'':0.0115026,(('PDT000360088.1':0.0234955,'PDT000548966.1':0.0251272)'':0.0109258,('PDT000497339.1':0.0357353,('PDT000395476.1':0.0394878,('PDT000524073.1':0.0228679,('PDT000397914.1':0.0175016,'PDT000395199.1':0.0120293)'':0.0101889)'':0.00357789)'':0.0036864)'':-0.00038467)'':0.00541484)'':0.00350268)'':0.0100996)'':-0.00476443)'':0.00568162)'':0.00418255)'':0.011733)'':0.0159269)'':0.0112074)'':-0.0207892)'':0.00535128)'':0.00119866)'':0.0040554)'':-0.00829317)'':0.00347)'':0.00456408)'':0.00271829)'':0.00564233)'':-0.00562881)'':0.00434521)'':0.00518224)'':0.00251162)'':0.0238633)'':-0.0134423)'':0.00835295)'':0.00873497)'':0.0106156)'':-0.00822072,((((('PDT000566052.1':0.0483516,('PDT000566071.1':0.0290022,'PDT000568762.1':0.02937)'':0.0213183)'':0.0285362,(('PDT000068832.2':0.0415919,'PDT000397997.1':0.0408167)'':0.0403996,('PDT000249939.1':0.0681852,('PDT000249960.1':0.0466875,'PDT000252656.1':0.052986)'':0.0142145)'':0.0107972)'':0.00266513)'':0.0605218,((('PDT000168823.1':0.0480834,'PDT000168855.1':0.0491109)'':0.0132345,(('PDT000652870.1':0.0587317,'PDT000652866.1':0.0407417)'':0.00814079,('PDT000167681.2':0.0360961,'PDT000652960.1':0.047497)'':0.0152488)'':0.00736707)'':0.0129057,('PDT000396760.1':0.0742814,(('PDT000249879.1':0.0643103,'PDT000252731.1':0.0627647)'':0.0113824,(('PDT000652885.1':0.0573112,'PDT000653028.1':0.0585448)'':0.00872686,('PDT000396781.1':0.0528125,('PDT000565859.1':0.0246718,('PDT000566005.1':0.015969,('PDT000341223.1':0.0111325,'PDT000610398.1':0.0147321)'':0.00271266)'':0.0126896)'':0.0290074)'':-0.00376699)'':0.0151847)'':0.00431439)'':-0.00502348)'':0.0742658)'':0.020742,(((('PDT000131908.2':0.0577327,'PDT000649136.1':0.0724853)'':0.0117434,('PDT000161526.2':0.0790558,('PDT000251491.1':0.0589125,('PDT000146067.2':0.0524311,('PDT000584166.1':0.0487043,'PDT000616945.1':0.0363979)'':0.0070123)'':0.0161265)'':0.0156965)'':0.00323748)'':0.0465864,(('PDT000581187.1':0.0498523,('PDT000571179.1':0.0298391,('PDT000314857.2':0.0229213,('PDT000542245.1':0.0161565,'PDT000042653.1':0.0183603)'':0.0026054)'':0.00387933)'':0.0176795)'':0.0772711,(('PDT000324137.1':0.0650503,('PDT000158020.2':0.0384731,'PDT000519441.1':0.041835)'':0.0188496)'':0.0209721,('PDT000354194.1':0.0757296,(('PDT000132325.2':0.0594072,('PDT000041342.1':0.0231593,('PDT000041344.1':0.0102587,'PDT000041346.1':0.00733574)'':0.00977306)'':0.0355455)'':0.00489409,('PDT000431721.1':0.0779332,('PDT000571182.1':0.0420848,('PDT000571151.1':0.0207463,'PDT000718445.1':0.0259643)'':0.0142102)'':0.0213709)'':0.000539781)'':0.0144032)'':0.00535317)'':0.0420165)'':0.00829025)'':0.00301402,((('PDT000655786.1':0.113722,('PDT000242955.1':0.0753501,'PDT000346101.1':0.0721799)'':0.0296896)'':-0.00358264,(('PDT000523911.1':0.0435288,('PDT000204898.2':0.0516051,'PDT000689810.1':0.0470697)'':0.0038464)'':0.0261805,('PDT000689808.1':0.0766881,('PDT000651900.1':0.0742333,'PDT000668338.2':0.0518087)'':0.00390488)'':0.00568486)'':0.0111443)'':0.0470287,(((('PDT000398138.1':0.0743891,'PDT000691292.1':0.0651279)'':0.05004,('PDT000196661.2':0.0838329,('PDT000327233.2':0.0790313,'PDT000205453.1':0.0683197)'':0.0123476)'':0.0157939)'':0.000579883,(('PDT000400032.2':0.108032,'PDT000524086.1':0.0985886)'':0.00194233,('PDT000675226.1':0.0934815,('PDT000250053.1':0.088548,('PDT000221027.1':0.057095,'PDT000250150.1':0.057687)'':0.0350505)'':0.000569844)'':0.00918228)'':0.00378289)'':0.00234848,((('PDT000495321.1':0.0701218,'PDT000066403.2':0.0766952)'':0.00764109,(('PDT000662600.1':0.020067,'PDT000663905.1':0.016451)'':0.043609,(('PDT000502884.1':0.0436564,'PDT000661241.1':0.0479363)'':0.00712971,('PDT000174357.1':0.0169529,('PDT000024646.1':0.00661944,'PDT000024647.1':0.00467496)'':0.0137785)'':0.0385287)'':0.00290595)'':0.0207855)'':0.0247354,((('PDT000645532.2':0.0762239,'PDT000228086.1':0.0756971)'':0.0211659,('PDT000065505.2':0.0819378,('PDT000238325.3':0.0820194,'PDT000526197.1':0.0754636)'':0.0119312)'':0.0105762)'':0.00308147,(('PDT000249750.1':0.0772863,('PDT000668836.1':0.0749054,'PDT000661239.1':0.0482216)'':0.0125327)'':0.00219568,('PDT000249938.1':0.0897473,('PDT000645478.2':0.0823147,('PDT000397816.1':0.0684806,'PDT000557322.1':0.0832594)'':0.00493827)'':0.00477298)'':0.000190967)'':0.0256081)'':0.00458005)'':0.00154598)'':0.00833278)'':0.00229485)'':0.0139252)'':0.00434566,((((('PDT000246523.2':0.0500978,('PDT000432484.1':0.031237,'PDT000228285.1':0.0340417)'':0.0171071)'':0.0557459,('PDT000541644.1':0.065889,('PDT000645487.2':0.0638124,'PDT000645489.2':0.0649936)'':0.027201)'':0.0363672)'':0.000586697,(('PDT000088263.2':0.0613934,'PDT000346959.1':0.0511916)'':0.045018,(('PDT000305148.2':0.0687188,('PDT000248491.1':0.0463839,('PDT000667723.2':0.0328635,('PDT000286930.2':0.0298946,('PDT000495040.1':0.0142179,'PDT000499657.1':0.0157831)'':0.0098857)'':0.0110906)'':0.000924457)'':0.0107878)'':0.0214783,(('PDT000566119.1':0.0801968,('PDT000321350.1':0.0267548,('PDT000693778.1':0.0167846,'PDT000237130.1':0.00830926)'':0.0087834)'':0.0360437)'':0.0102162,(('PDT000044306.1':0.0747229,('PDT000077058.2':0.0707287,('PDT000672632.1':0.0312011,'PDT000719162.1':0.0354914)'':0.017335)'':0.0137707)'':-0.00304253,('PDT000242932.1':0.0817219,(('PDT000517282.1':0.0550786,('PDT000691295.1':0.0521503,'PDT000250212.1':0.0456403)'':0.00334808)'':0.0121826,('PDT000397834.1':0.066459,(('PDT000395082.1':0.0468532,'PDT000399614.1':0.0421405)'':0.00190603,('PDT000397647.1':0.0106663,'PDT000399603.1':0.0163312)'':0.0359224)'':0.0116902)'':0.0081696)'':0.00756761)'':-0.00507004)'':0.00851847)'':0.00259222)'':0.0187081)'':0.0100839)'':0.0384614,((((('PDT000357618.1':0.0659246,'PDT000424788.1':0.0569104)'':0.0242909,('PDT000015305.2':0.0789811,('PDT000047748.3':0.0652331,'PDT000015304.2':0.0694579)'':0.0115804)'':0.00813014)'':0.0589065,(((('PDT000242878.1':0.0609869,('PDT000095720.2':0.0443311,'PDT000417125.1':0.0528655)'':0.0115928)'':5.63687e-05,('PDT000581036.1':0.0692298,('PDT000233278.3':0.0503454,'PDT000212780.1':0.0434863)'':0.0191244)'':0.00350563)'':0.0410951,(('PDT000661797.1':0.0631066,('PDT000497427.1':0.0542955,'PDT000557246.1':0.0650275)'':0.0080464)'':0.0147585,(('PDT000359334.1':0.0297719,'PDT000601256.1':0.0356523)'':0.0112013,('PDT000380306.1':0.0433721,('PDT000379415.1':0.0221848,'PDT000439461.1':0.0233099)'':0.011584)'':0.00764716)'':0.0388331)'':0.0151669)'':0.0125875,(((('PDT000318587.1':0.0741141,('PDT000356910.1':0.0347471,('PDT000095827.2':0.0147661,'PDT000251917.1':0.0145495)'':0.0232023)'':0.0284294)'':0.021202,('PDT000424766.1':0.0574196,('PDT000357708.1':0.0741467,('PDT000142216.2':0.0263676,'PDT000720260.1':0.031089)'':0.045305)'':0.00793696)'':0.0171366)'':0.0108842,(('PDT000495034.1':0.0576059,'PDT000346041.1':0.0677391)'':0.0523649,(('PDT000524100.1':0.0550778,'PDT000691369.1':0.0577512)'':0.0362173,('PDT000621016.1':0.109936,(('PDT000156002.2':0.0660785,'PDT000332085.1':0.0571745)'':0.00628039,('PDT000427802.1':0.0595003,'PDT000675595.1':0.0647047)'':0.00850436)'':0.0331511)'':-0.00486992)'':0.0136007)'':-0.00461022)'':0.00181449,('PDT000691342.1':0.135187,(('PDT000129823.2':0.0410258,'PDT000471043.1':0.0338035)'':0.0533688,((('PDT000495320.1':0.028484,'PDT000654576.1':0.0559071)'':0.0496149,(('PDT000150240.2':0.0535671,'PDT000111329.1':0.0599669)'':0.00972124,('PDT000283089.2':0.0615551,'PDT000346516.1':0.0590259)'':0.00241676)'':0.0157624)'':-0.00189971,(('PDT000708216.1':0.0726106,('PDT000032340.1':0.00867603,'PDT000032347.1':0.00480377)'':0.0721295)'':0.00082296,(('PDT000718977.1':0.0729507,('PDT000032380.1':0.0558101,('PDT000501896.1':0.0284585,('PDT000393070.1':0.0198937,'PDT000691356.1':0.0306475)'':0.0108207)'':0.00559318)'':0.0152069)'':0.00240264,(('PDT000380269.1':0.06241,'PDT000111351.1':0.05756)'':0.00261049,('PDT000443375.1':0.0580311,('PDT000661373.1':0.0457842,('PDT000271409.2':0.0472745,'PDT000551872.1':0.042097)'':0.0106042)'':0.00497126)'':0.00226903)'':0.00167126)'':0.00512204)'':0.00773714)'':0.0185937)'':0.0375124)'':-0.0260461)'':0.00967329)'':0.0185643)'':-0.0143739,(((('PDT000242996.1':0.0998781,('PDT000008772.2':0.0489126,'PDT000357713.1':0.0505947)'':0.0185393)'':0.00764953,(('PDT000466976.1':0.0659429,'PDT000242505.1':0.0672091)'':0.0128183,(('PDT000499498.2':0.0647497,'PDT000388472.1':0.0756573)'':0.0125382,(('PDT000671279.1':0.0377065,'PDT000672609.1':0.0522721)'':0.014395,('PDT000248785.1':0.0478623,('PDT000671280.1':0.0421669,'PDT000672692.1':0.0362131)'':0.0126769)'':0.00281785)'':0.0236413)'':0.0163474)'':0.00401266)'':0.00713122,((('PDT000124679.1':0.0370888,'PDT000124689.1':0.031813)'':0.038516,(('PDT000630266.1':0.0190354,'PDT000650014.2':0.0227751)'':0.0225188,('PDT000671286.1':0.0320169,('PDT000630267.2':0.0352574,('PDT000672685.1':0.0239355,('PDT000664460.1':0.0276385,('PDT000671278.1':0.0118399,('PDT000645150.2':0.00951844,'PDT000668384.2':0.0103584)'':0.00251891)'':0.0131086)'':0.002087)'':0.00380661)'':0.00138329)'':0.0094118)'':0.0444446)'':-0.0204008,(('PDT000547563.2':0.0130385,'PDT000671288.1':0.0112546)'':0.0248398,('PDT000009038.2':0.0436077,('PDT000672683.1':0.0410244,('PDT000557010.2':0.0194261,('PDT000668374.2':0.027209,('PDT000668339.2':0.0237092,(('PDT000630265.2':0.0110385,'PDT000640019.2':0.00796838)'':0.00716923,('PDT000671282.1':0.0159124,'PDT000008728.2':0.0163266)'':0.00116499)'':0.00211952)'':0.00189458)'':0.00541702)'':0.00458992)'':0.0110391)'':-0.00320716)'':0.0138202)'':0.0564762)'':0.0389262,(('PDT000662548.1':0.137356,((('PDT000384487.1':0.0809679,'PDT000688652.1':0.0702751)'':0.00818713,('PDT000719187.1':0.0574112,('PDT000645479.2':0.0347908,'PDT000252009.1':0.0363815)'':0.0188552)'':0.0164139)'':0.00289098,('PDT000225242.1':0.0989766,((('PDT000346844.1':0.0508616,('PDT000346602.1':0.0280955,'PDT000346834.1':0.0274818)'':0.0300768)'':0.00674746,(('PDT000417149.1':0.0569517,'PDT000417166.1':0.0586443)'':0.00547639,('PDT000661172.1':0.0441917,('PDT000124649.1':0.0378396,'PDT000291281.1':0.0374448)'':0.00700434)'':0.0151175)'':0.00882043)'':0.00288383,('PDT000124603.1':0.0953031,('PDT000124655.1':0.0695946,('PDT000298075.1':0.0761803,('PDT000417168.1':0.0687318,(('PDT000289905.1':0.054664,('PDT000237136.1':0.0353616,('PDT000107785.1':0.0308739,'PDT000225232.1':0.0345239)'':0.00907042)'':0.00487016)'':0.00782559,(('PDT000542480.1':0.0197618,'PDT000547554.1':0.0203244)'':0.0326159,('PDT000250069.1':0.0409673,('PDT000718944.1':0.0232602,'PDT000248848.1':0.0239032)'':0.0119068)'':0.00713395)'':0.00386994)'':0.00496356)'':0.004743)'':0.00601875)'':0.00500204)'':-0.00948177)'':0.0334835)'':-0.00913115)'':0.0259947)'':-0.000479814,((('PDT000537854.1':0.106322,('PDT000651195.1':0.100509,('PDT000181791.2':0.102615,('PDT000535403.1':0.0829042,'PDT000536920.1':0.0703228)'':0.0097063)'':0.00141386)'':0.0135048)'':-0.000659172,('PDT000395222.1':0.12078,('PDT000517272.1':0.0633289,('PDT000285933.2':0.095222,('PDT000479633.1':0.0714831,('PDT000706217.1':0.0570197,('PDT000191842.2':0.0411718,'PDT000645081.1':0.0438418)'':0.00395449)'':0.00617276)'':0.0136758)'':0.00693468)'':0.0251829)'':-0.000176736)'':0.00565676,((('PDT000011765.2':0.0710792,('PDT000221030.1':0.0749298,(('PDT000395035.1':0.0728705,'PDT000045630.1':0.0696535)'':0.000543051,('PDT000661237.1':0.0473811,'PDT000667731.2':0.0433275)'':0.0114501)'':0.0158112)'':0.00203331)'':0.0165569,(('PDT000250179.1':0.122759,(('PDT000160395.2':0.057003,('PDT000556975.1':0.0221155,'PDT000557082.1':0.0182215)'':0.0302455)'':0.0104287,(('PDT000288426.2':0.0363059,('PDT000694414.1':0.0347728,('PDT000343887.1':0.0217496,('PDT000691306.1':0.0261505,('PDT000556982.1':0.0101996,'PDT000008730.2':0.0122033)'':0.00870783)'':0.0043473)'':0.0068747)'':0.0120746)'':0.0151365,(('PDT000557043.1':0.0548054,('PDT000145916.2':0.0421414,'PDT000010315.2':0.0424386)'':0.00675012)'':0.0077857,('PDT000609623.1':0.062962,('PDT000649399.1':0.0432672,('PDT000577191.1':0.0272784,'PDT000577453.1':0.0267467)'':0.0130835)'':0.0240842)'':0.00638536)'':0.00224568)'':0.00743355)'':0.0480444)'':-0.0202275,('PDT000011772.2':0.11044,((('PDT000426318.1':0.0540178,'PDT000482074.1':0.0582492)'':0.00809442,('PDT000318995.1':0.0499316,'PDT000463728.1':0.0605194)'':0.00310133)'':0.0193338,(('PDT000252007.1':0.062149,('PDT000577031.1':0.0631119,('PDT000718816.1':0.0386642,'PDT000228048.1':0.0384522)'':0.0133154)'':0.0074276)'':0.000455626,('PDT000081751.1':0.0945486,(('PDT000469865.1':0.0622678,('PDT000132270.2':0.0492983,'PDT000661648.1':0.0438789)'':0.00725913)'':0.00343135,('PDT000495069.1':0.0514844,('PDT000135988.2':0.0274571,('PDT000148081.2':0.0207115,'PDT000015319.2':0.0195708)'':0.00280213)'':0.0300361)'':0.0200045)'':0.00995842)'':-0.00148446)'':0.00863428)'':0.0121697)'':-0.00328803)'':0.0101388)'':0.00151449,(((('PDT000621003.1':0.0795818,('PDT000188937.1':0.0629755,('PDT000346566.1':0.0758161,('PDT000212781.1':0.0536318,'PDT000233298.1':0.0616972)'':0.0062914)'':0.00594321)'':0.00105496)'':0.0197069,(('PDT000150266.2':0.0742308,'PDT000134252.1':0.0767322)'':0.0072923,('PDT000652841.1':0.0431206,('PDT000645497.2':0.0178334,('PDT000645685.2':0.0133829,'PDT000645692.2':0.0107327)'':0.00280901)'':0.0385038)'':0.0462501)'':0.0121474)'':0.0060062,((('PDT000397934.1':0.101225,'PDT000346065.1':0.0852311)'':0.00291502,('PDT000494516.1':0.084968,'PDT000714580.1':0.095419)'':0.00841698)'':0.00462803,(('PDT000426188.1':0.0915084,('PDT000719236.1':0.0867134,('PDT000437997.1':0.0512328,'PDT000525367.1':0.0462236)'':0.0135944)'':0.00707804)'':0.0064866,('PDT000411307.1':0.0848203,(('PDT000539273.1':0.0542514,'PDT000647843.1':0.0543826)'':0.00910496,(('PDT000250211.1':0.0483269,'PDT000345961.1':0.0593201)'':0.00399426,('PDT000389658.1':0.0478807,('PDT000277720.2':0.0366505,'PDT000466039.1':0.0395888)'':0.0184898)'':0.0144807)'':0.00869529)'':0.0115893)'':-0.00109359)'':0.0140421)'':-0.00150685)'':0.014243,((('PDT000520814.1':0.107784,('PDT000251965.1':0.106045,('PDT000358763.1':0.0788137,('PDT000718880.1':0.0711104,('PDT000372015.1':0.0668682,'PDT000710450.1':0.0609528)'':0.00594608)'':0.00163339)'':0.0122899)'':0.0132651)'':-0.000823166,(('PDT000421863.1':0.071559,'PDT000645486.2':0.097079)'':0.0232654,('PDT000290385.4':0.0726072,(('PDT000191699.2':0.0500357,'PDT000372153.1':0.0394237)'':0.0119865,('PDT000285334.1':0.0422209,('PDT000032196.1':0.00480637,'PDT000032205.1':0.00246962)'':0.0422585)'':0.00946861)'':0.0134344)'':0.0239226)'':0.00211677)'':0.00313651,(((('PDT000060275.2':0.0786116,'PDT000346504.1':0.0667024)'':0.00274347,('PDT000403070.1':0.0534985,'PDT000345924.1':0.0669715)'':0.025242)'':0.0174278,((('PDT000109362.2':0.0404338,'PDT000510041.1':0.0398863)'':0.05617,('PDT000238210.2':0.0668723,'PDT000417146.1':0.0863347)'':0.0268249)'':0.00323014,((('PDT000422540.1':0.00878711,'PDT000429768.1':0.00807969)'':0.00512199,('PDT000015764.3':0.020405,('PDT000429663.1':0.0111469,'PDT000429671.1':0.00761933)'':0.00343886)'':0.00391189)'':0.0357556,('PDT000679708.1':0.0484043,('PDT000015719.3':0.0307533,(('PDT000015792.3':0.0179728,'PDT000428895.1':0.0134227)'':0.00515468,('PDT000428857.1':0.0133464,'PDT000428903.1':0.00790472)'':0.0132813)'':0.00885145)'':0.0252643)'':-0.00202767)'':0.0496797)'':0.00119075)'':0.00524635,((('PDT000354221.1':0.0871684,'PDT000357721.1':0.0793106)'':0.0115171,(('PDT000264658.2':0.0493827,'PDT000587985.1':0.0420131)'':0.0346976,('PDT000111322.1':0.0226862,'PDT000252696.1':0.0255754)'':0.0480002)'':0.0121049)'':0.00359667,(('PDT000150884.1':0.0948491,('PDT000357664.1':0.0914643,('PDT000109360.2':0.0813704,(('PDT000502915.1':0.0622744,('PDT000510048.1':0.0660977,'PDT000357733.1':0.0620373)'':0.00098213)'':0.00267998,('PDT000289922.1':0.057609,('PDT000400017.1':0.0165315,('PDT000095708.2':0.00884222,'PDT000468342.1':0.0124577)'':0.00590981)'':0.0370393)'':0.01056)'':0.0179035)'':0.00233651)'':0.00768054)'':0.0011853,(('PDT000150705.2':0.105056,'PDT000720221.1':0.093809)'':0.017988,((('PDT000384313.2':0.0911795,('PDT000357690.1':0.0697656,('PDT000160993.2':0.065149,('PDT000109311.2':0.0500158,'PDT000017845.2':0.0590432)'':0.013506)'':0.00994937)'':0.00606732)'':-0.00415436,(('PDT000167679.2':0.0458285,'PDT000645583.2':0.0438412)'':0.0390541,(('PDT000355055.1':0.0374373,('PDT000178577.2':0.0214996,'PDT000307415.2':0.0221745)'':0.0152978)'':0.0253781,('PDT000490905.1':0.0564974,('PDT000496978.1':0.0292991,'PDT000394831.1':0.0248659)'':0.00947552)'':0.0209135)'':0.0184079)'':0.00357679)'':0.0047894,(('PDT000323533.1':0.0786737,'PDT000228204.1':0.0937693)'':0.0133324,(('PDT000129869.3':0.0364239,'PDT000064574.1':0.0443792)'':0.0553048,(('PDT000417386.1':0.0367887,'PDT000626559.1':0.0320384)'':0.0441265,(('PDT000228083.1':0.0643917,'PDT000228172.1':0.0684343)'':0.014624,(('PDT000045582.1':0.0365935,('PDT000109485.2':0.0209907,('PDT000323545.1':0.0199839,('PDT000095776.2':0.013425,'PDT000395395.1':0.0124413)'':0.00793449)'':0.00569455)'':0.00341115)'':0.0166118,(('PDT000639577.1':0.0128085,('PDT000417981.1':0.00714025,'PDT000577554.1':0.00797205)'':0.00229127)'':0.0210225,('PDT000227996.1':0.0332766,('PDT000150889.1':0.0176461,('PDT000111380.1':0.0123896,'PDT000150849.1':0.0115684)'':0.00531634)'':0.0090129)'':0.00971566)'':0.00512926)'':0.0330573)'':0.00319572)'':0.0027714)'':0.00890659)'':-0.00621894)'':0.0121878)'':-0.00563122)'':0.00244627)'':0.00418252)'':0.00179713)'':0.0017981)'':0.000901541)'':0.00273669)'':6.20704e-05)'':0.00912843)'':-0.00130943)'':0.00189961,((('PDT000242975.1':0.0751207,('PDT000547577.1':0.0869253,('PDT000542490.1':0.0601723,('PDT000186214.1':0.05525,('PDT000644888.1':0.0170651,('PDT000610087.1':0.0142774,'PDT000644848.1':0.0182849)'':0.00376378)'':0.0265848)'':0.00964539)'':0.00738346)'':0.00431788)'':0.0117599,(('PDT000668020.1':0.0567192,('PDT000644851.1':0.0470535,('PDT000719058.1':0.0286793,'PDT000252649.1':0.0414633)'':0.00749708)'':0.00893664)'':0.0109205,(('PDT000584532.1':0.0677332,'PDT000584539.3':0.0746828)'':0.0119019,('PDT000463717.1':0.0582129,('PDT000345973.1':0.0412546,('PDT000242978.1':0.0533338,'PDT000411297.1':0.0549252)'':0.00345202)'':0.00919425)'':0.0104202)'':0.00408194)'':0.0217362)'':0.0566964,(((('PDT000345964.1':0.108201,('PDT000145170.2':0.10033,'PDT000439433.1':0.105099)'':0.0145888)'':0.00117783,('PDT000111339.1':0.0618697,('PDT000504749.1':0.0466817,('PDT000228069.1':0.0368263,('PDT000095781.2':0.0316706,('PDT000258729.2':0.0338216,'PDT000095741.2':0.0256165)'':0.00848253)'':0.000499183)'':0.00417303)'':0.0307294)'':0.0337952)'':0.00372509,(('PDT000501833.1':0.0905861,('PDT000111379.1':0.0507581,('PDT000129558.1':0.050007,('PDT000111363.1':0.0183608,'PDT000345963.1':0.0302379)'':0.0106133)'':0.00260982)'':0.0431152)'':0.0060441,(('PDT000252707.1':0.0769546,('PDT000111367.1':0.0278873,('PDT000345935.1':0.0196515,'PDT000346050.1':0.0265918)'':0.00300588)'':0.0614052)'':0.00452824,('PDT000354952.2':0.050396,('PDT000500487.1':0.0366601,('PDT000249743.1':0.0234915,('PDT000342666.1':0.0232715,'PDT000559305.1':0.0231833)'':0.00406806)'':0.00744164)'':0.00872467)'':0.0381804)'':0.00715631)'':0.0264863)'':0.0284549,(((((('PDT000034468.2':0.0856631,'PDT000720237.1':0.0769459)'':0.0184173,('PDT000242866.1':0.0663071,('PDT000088283.2':0.040817,'PDT000639527.1':0.0459835)'':0.0144821)'':0.0263863)'':0.0321976,((('PDT000389634.1':0.0676904,('PDT000394992.1':0.0687362,'PDT000441060.2':0.0670308)'':0.0113471)'':0.00313354,('PDT000345915.1':0.0715048,('PDT000088270.2':0.0654912,'PDT000039239.1':0.0666248)'':0.00386221)'':0.00726109)'':0.0342232,('PDT000626563.1':0.104891,('PDT000617808.1':0.091815,(('PDT000345931.1':0.0260424,'PDT000346099.1':0.0246547)'':0.069356,(('PDT000425641.1':0.0658923,'PDT000719052.1':0.0649577)'':0.00840955,('PDT000367712.1':0.0682727,('PDT000346866.1':0.0383321,'PDT000346960.1':0.0366437)'':0.0315959)'':0.00668585)'':0.017973)'':0.0050766)'':0.0100051)'':-0.00844552)'':0.0257943)'':0.0011338,(('PDT000095722.2':0.0914156,(('PDT000235411.2':0.101457,'PDT000400233.1':0.0905184)'':0.0124005,('PDT000522376.1':0.098898,('PDT000602897.1':0.0866533,('PDT000203396.2':0.0544382,('PDT000645086.1':0.0408604,'PDT000716398.1':0.030923)'':0.0135457)'':0.00839167)'':0.0141139)'':0.0228325)'':0.00960778)'':0.00812696,('PDT000354192.1':0.11828,('PDT000400846.1':0.11243,('PDT000504793.1':0.0867897,('PDT000134254.1':0.095343,(('PDT000250024.1':0.0706836,('PDT000250174.1':0.0412574,('PDT000242942.1':0.0528441,'PDT000242987.1':0.0508049)'':0.00465642)'':0.0148467)'':0.00219129,(('PDT000281119.2':0.0345083,'PDT000228142.1':0.0434112)'':0.0152949,(('PDT000500477.1':0.0508305,'PDT000557193.1':0.0548935)'':0.00767721,(('PDT000500533.1':0.0647577,(('PDT000372332.1':0.0414027,'PDT000252472.1':0.0336118)'':0.00318032,(('PDT000241667.2':0.0338522,'PDT000642596.1':0.0420333)'':0.00271613,('PDT000266385.2':0.0293329,'PDT000399529.1':0.0298968)'':0.00972535)'':0.00301648)'':0.0109111)'':-0.008136,(('PDT000332424.1':0.0408687,'PDT000496979.1':0.0453679)'':0.00511843,(('PDT000149357.2':0.0401881,('PDT000266387.2':0.0341071,'PDT000566015.1':0.0311954)'':0.00168583)'':0.0098396,('PDT000069158.2':0.0408608,('PDT000360264.1':0.0286014,'PDT000424623.1':0.0306933)'':0.00216785)'':0.00543039)'':0.00558795)'':0.00215201)'':0.00908797)'':0.00228262)'':-0.00031635)'':0.0201035)'':0.00197256)'':0.0104345)'':0.0214012)'':-0.00482142)'':0.00401406)'':-4.63239e-05,(((('PDT000249882.1':0.0682033,('PDT000537359.1':0.0448536,'PDT000661243.1':0.0475947)'':0.0289326)'':0.0141831,(('PDT000357735.1':0.0897555,('PDT000510032.1':0.0629603,'PDT000326177.2':0.0678507)'':0.00942502)'':0.0102557,('PDT000150876.1':0.075599,(('PDT000249860.1':0.0509477,'PDT000399582.1':0.0470765)'':0.00493379,(('PDT000258351.2':0.0442627,('PDT000196606.2':0.0222102,'PDT000109461.2':0.028244)'':0.0148166)'':0.000816133,('PDT000645073.1':0.0350426,('PDT000395402.1':0.0366643,('PDT000490602.1':0.0235778,'PDT000717102.1':0.0202092)'':0.00696146)'':0.00903508)'':0.00994938)'':0.0260186)'':0.00419045)'':0.0100797)'':0.0132222)'':0.012076,((('PDT000675491.1':0.0739437,('PDT000654264.1':0.0698417,'PDT000463659.1':0.0598833)'':0.0246548)'':0.00200526,('PDT000537525.1':0.0451057,('PDT000111369.1':0.0220649,'PDT000249893.1':0.0242668)'':0.0106674)'':0.0458139)'':0.0204981,(('PDT000652457.1':0.104639,(('PDT000129876.2':0.0706657,'PDT000009579.2':0.0579713)'':0.0340017,(('PDT000509504.1':0.0185074,('PDT000393083.1':0.0203421,'PDT000548964.1':0.0242289)'':0.000965204)'':0.0473432,('PDT000251532.1':0.0370088,(('PDT000510036.1':0.0242188,'PDT000228054.1':0.0192595)'':0.0122416,('PDT000111326.1':0.0303561,('PDT000111371.1':0.0208758,'PDT000251800.1':0.0140726)'':0.0153971)'':0.00341717)'':0.0052591)'':0.024773)'':0.0307655)'':0.00302576)'':0.000679121,(('PDT000588450.1':0.0613824,'PDT000252442.1':0.0436156)'':0.0522569,('PDT000357622.1':0.100343,(('PDT000304952.2':0.0737329,'PDT000205431.1':0.0549421)'':0.0125041,(('PDT000687928.1':0.0315455,'PDT000228231.1':0.0554587)'':0.0287434,('PDT000627425.1':0.0723662,('PDT000153294.2':0.0542081,('PDT000502530.1':0.0428613,('PDT000111345.1':0.0276222,'PDT000501838.1':0.032793)'':0.0078793)'':0.00301996)'':0.013534)'':0.000264027)'':0.0140794)'':0.0168138)'':0.00227974)'':-0.000339819)'':0.00245928)'':0.00367121)'':0.00291102,(('PDT000258376.2':0.0914431,(('PDT000208425.2':0.0781118,('PDT000111327.1':0.0515566,'PDT000369031.1':0.0524024)'':0.0338682)'':0.00597015,(('PDT000323507.1':0.0706437,'PDT000711735.1':0.0599443)'':0.0180532,(('PDT000161670.3':0.047008,'PDT000063279.3':0.039431)'':0.0201685,(('PDT000323710.2':0.0332828,'PDT000556800.2':0.0268602)'':0.00320439,('PDT000638932.2':0.0299996,('PDT000329846.2':0.0147098,'PDT000494849.2':0.0273675)'':0.00953275)'':0.00420453)'':0.02765)'':0.0205155)'':0.00875701)'':0.00187091)'':0.0180563,((('PDT000167704.2':0.0599909,'PDT000352468.1':0.0506931)'':0.00505084,('PDT000309356.2':0.0559378,('PDT000710438.1':0.0497503,'PDT000713806.1':0.0540467)'':0.00287623)'':0.0114156)'':0.0468485,(('PDT000011767.2':0.0764307,('PDT000395443.1':0.0764067,('PDT000581010.1':0.0644185,('PDT000518563.1':0.0499085,(('PDT000280977.2':0.0200951,'PDT000006431.2':0.0260451)'':0.019931,('PDT000251486.1':0.0301852,'PDT000463618.1':0.0318483)'':0.00744447)'':0.0108685)'':0.0032704)'':0.00448167)'':0.0210362)'':0.02306,((('PDT000556942.1':0.062058,'PDT000619662.1':0.050921)'':0.0500889,(('PDT000557202.1':0.0024639,'PDT000228021.1':0.00342962)'':0.0850305,('PDT000355608.1':0.0887065,('PDT000395064.1':0.00819555,'PDT000048142.1':0.0130937)'':0.0727458)'':0.000179073)'':0.0169364)'':-0.00232183,(('PDT000372800.1':0.0467159,'PDT000711741.1':0.0632281)'':0.0458746,(('PDT000713352.1':0.0896139,('PDT000034740.1':0.0904324,(('PDT000323521.1':0.021508,'PDT000362886.1':0.0181549)'':0.00443113,('PDT000355626.1':0.0206161,('PDT000258347.2':0.0171842,'PDT000111375.1':0.0169895)'':0.00265318)'':0.00588859)'':0.0431304)'':0.00938272)'':0.000737909,(('PDT000251415.1':0.08492,('PDT000196658.2':0.0290462,'PDT000600644.1':0.0252953)'':0.0404912)'':0.00242889,('PDT000082130.2':0.076026,('PDT000129910.2':0.0540327,('PDT000062932.2':0.0280847,'PDT000042510.1':0.0325243)'':0.0178923)'':0.0114062)'':0.00275271)'':0.00906443)'':0.00975206)'':-0.00321875)'':0.0152618)'':0.00732257)'':-0.00340994)'':0.000107196)'':0.00359341)'':0.0012261,((('PDT000060299.2':0.103252,('PDT000399565.1':0.0915098,('PDT000490609.1':0.0539181,'PDT000546954.1':0.0430968)'':0.0190652)'':0.00995277)'':0.0243997,((('PDT000399577.1':0.0802129,('PDT000646918.1':0.0429748,'PDT000150883.1':0.0495405)'':0.0439929)'':0.00749887,(('PDT000343925.1':0.0694564,('PDT000497335.1':0.0535978,('PDT000610389.1':0.042398,'PDT000395134.1':0.0382783)'':0.0237875)'':0.0207632)'':0.0160962,(('PDT000497314.1':0.0735715,'PDT000714678.1':0.0604555)'':0.0105363,('PDT000168860.1':0.0575992,'PDT000346838.1':0.0461838)'':0.0315592)'':0.00769337)'':0.0104179)'':0.000920812,('PDT000617811.1':0.107743,('PDT000111325.1':0.106583,((('PDT000444995.1':0.0387164,'PDT000627340.1':0.0382809)'':0.00654576,('PDT000213206.2':0.0487861,'PDT000252434.1':0.0331262)'':0.00956601)'':0.0365123,((('PDT000346578.1':0.0516695,'PDT000346872.1':0.0486435)'':0.0106089,('PDT000290516.2':0.0562148,('PDT000081755.1':0.0411397,'PDT000463635.1':0.0522379)'':0.00603061)'':0.00108212)'':0.0164633,(('PDT000397801.1':0.0562275,('PDT000601909.1':0.0609446,'PDT000710221.1':0.0533604)'':0.00865103)'':0.0186139,(('PDT000712995.1':0.0218949,'PDT000691308.1':0.0267681)'':0.0314797,('PDT000196607.2':0.0634165,('PDT000415773.1':0.0461127,('PDT000565838.1':0.0273931,'PDT000251412.1':0.0269908)'':0.0161255)'':0.00630436)'':0.00572742)'':0.00481919)'':0.00298149)'':0.0051726)'':0.020225)'':0.00368498)'':-0.0180947)'':0.0131514)'':0.0134778,(('PDT000425783.1':0.09143,('PDT000029842.1':0.00928414,('PDT000032189.1':0.0058217,('PDT000025979.1':0.00231206,'PDT000026307.1':0.00337827)'':0.00213224)'':0.00936019)'':0.0643502)'':0.0395665,((('PDT000250144.1':0.0669539,('PDT000032207.1':0.00808,('PDT000032217.1':0.00445128,'PDT000034417.1':0.00471207)'':0.00205212)'':0.0732372)'':0.0540261,(('PDT000111331.1':0.0703072,('PDT000688725.1':0.0535236,'PDT000395385.1':0.0504524)'':0.0225353)'':0.0203882,('PDT000219689.1':0.111103,(('PDT000551883.1':0.0479621,('PDT000258740.2':0.0465017,(('PDT000106405.1':0.0130038,('PDT000357569.1':0.00798305,('PDT000176394.1':0.00289212,'PDT000313482.1':0.00279764)'':0.00175187)'':0.00445306)'':0.00635527,('PDT000321339.1':0.0226031,('PDT000229866.1':0.0185758,(('PDT000313487.1':0.00292832,('PDT000313486.1':0.00348174,('PDT000313483.1':0.00059924,'PDT000313485.1':0.0008002)'':0.00131525)'':0.00289946)'':0.00694896,('PDT000061496.1':0.0100804,('PDT000526191.1':0.0168922,('PDT000035300.1':0.00503508,'PDT000424764.1':0.00512222)'':0.004701)'':0.00279906)'':-0.000910265)'':0.00415182)'':0.00298354)'':-0.00406396)'':0.0192573)'':0.00358226)'':-0.00833503,((('PDT000357563.1':0.0313931,('PDT000288720.1':0.0122019,(('PDT000661073.1':0.00578721,'PDT000144345.1':0.00666669)'':0.00511427,('PDT000208155.1':0.00656384,'PDT000275550.1':0.00918576)'':0.00375548)'':0.00394938)'':0.00665258)'':0.00336523,('PDT000168834.1':0.0877848,(('PDT000321344.1':0.0271383,('PDT000576637.1':0.0140764,'PDT000321343.1':0.0160279)'':0.00448411)'':0.00115705,('PDT000281814.1':0.0380586,('PDT000627929.1':0.0179345,('PDT000627928.1':0.0110274,('PDT000627926.1':0.0114639,('PDT000627927.1':0.0013043,'PDT000499673.1':0.000594559)'':0.00750496)'':0.000687975)'':0.0045961)'':0.00918021)'':-0.000889181)'':0.0173973)'':-0.0122517)'':0.00266523,(('PDT000357576.1':0.0253831,('PDT000683131.1':0.0237867,('PDT000357578.1':0.0262586,('PDT000357641.1':0.0179355,('PDT000009307.2':0.0101574,('PDT000176419.1':0.00102447,('PDT000176414.1':0.000385855,'PDT000176417.1':0.000214145)'':0.000275079)'':0.00673235)'':0.00452756)'':0.00304208)'':0.00115265)'':0.00162861)'':0.00116859,('PDT000008911.2':0.0582055,('PDT000313456.1':0.0383946,(('PDT000082619.2':0.0140664,('PDT000045678.1':0.0119473,'PDT000621075.1':0.00857884)'':0.00754749)'':0.00017102,('PDT000081762.1':0.0398968,('PDT000008897.2':0.0290451,(('PDT000481261.1':0.00831639,'PDT000481263.1':0.00617061)'':0.0114685,('PDT000518045.1':0.0242587,(('PDT000518043.1':0.00136956,'PDT000518044.1':0.00112769)'':0.00770125,('PDT000307278.1':0.010575,(('PDT000273984.1':0.00290918,'PDT000343393.1':0.00358173)'':0.00786705,('PDT000114938.1':0.00652003,('PDT000020375.1':0.0059582,('PDT000061494.1':0.00531417,'PDT000007688.2':0.0042533)'':0.00100531)'':0.00171388)'':-0.000183564)'':0.00150081)'':0.00068668)'':0.00607275)'':0.000316911)'':0.00215739)'':0.00118161)'':-0.00189323)'':0.00541465)'':0.0126862)'':-0.0151339)'':0.00352181)'':0.0020469)'':0.0908971)'':-0.0373511)'':0.0167681)'':-0.013634,(((('PDT000652376.1':0.104532,('PDT000024485.2':0.0844548,'PDT000645629.2':0.0846602)'':0.0115274)'':0.00871759,((('PDT000469854.1':0.0752306,'PDT000720265.1':0.0613184)'':0.016236,('PDT000069173.2':0.0822458,('PDT000019604.1':0.0660662,('PDT000128867.2':0.072198,'PDT000652934.1':0.069982)'':0.00031781)'':0.0105376)'':0.00623877)'':0.000872075,(('PDT000652965.1':0.0223274,'PDT000645491.2':0.0316918)'':0.0730757,('PDT000149557.2':0.0821121,('PDT000653029.1':0.0761969,('PDT000652995.1':0.0599235,'PDT000369033.1':0.0622665)'':0.0222981)'':0.000871535)'':0.00973687)'':0.00430417)'':0.0110149)'':-0.00383617,(('PDT000354282.1':0.0999966,('PDT000645484.2':0.105026,('PDT000171436.1':0.0830456,'PDT000352546.1':0.0773104)'':0.00793537)'':0.00217344)'':0.017363,('PDT000520845.1':0.0964854,('PDT000312638.2':0.0970475,(('PDT000027743.2':0.0888761,('PDT000025506.2':0.0726257,'PDT000026891.2':0.0743233)'':0.0236209)'':0.0063367,((('PDT000353622.1':0.0801005,('PDT000167710.2':0.0595856,'PDT000346524.1':0.0654614)'':0.00499453)'':0.00669545,('PDT000662597.1':0.0777181,('PDT000645503.2':0.0756454,'PDT000228028.1':0.0706896)'':0.00257487)'':0.00485786)'':0.00458273,(('PDT000274856.2':0.0510217,'PDT000333680.1':0.0570603)'':0.0397748,(('PDT000019602.1':0.095378,('PDT000346092.1':0.0893834,('PDT000228223.1':0.0748104,('PDT000557186.1':0.0586407,'PDT000346117.1':0.0567983)'':0.00866208)'':0.00416907)'':0.00390225)'':0.00425065,(('PDT000150301.2':0.0588877,'PDT000003842.2':0.0786223)'':0.00934973,('PDT000688000.1':0.061022,('PDT000566076.1':0.0233197,'PDT000644993.1':0.0219207)'':0.0407878)'':0.00857176)'':0.000653046)'':0.00796278)'':-0.00299551)'':0.00640434)'':0.00542004)'':0.00639423)'':-0.00689563)'':-0.00128856)'':0.00313919,((('PDT000425665.1':0.0446896,'PDT000426515.1':0.0453296)'':0.0440193,(('PDT000250252.1':0.0944871,('PDT000483541.1':0.0855783,('PDT000645636.2':0.0827986,('PDT000367306.1':0.0533431,'PDT000444919.1':0.0579679)'':0.0159059)'':0.00532287)'':0.0126805)'':-0.000273589,(('PDT000345998.1':0.0869953,('PDT000357577.1':0.077184,('PDT000281483.2':0.0418316,'PDT000550393.1':0.051602)'':0.0257322)'':0.00883973)'':0.00438144,('PDT000709934.1':0.0743564,('PDT000167674.2':0.0581896,('PDT000687448.1':0.0581292,('PDT000399714.1':0.0486888,'PDT000629659.1':0.0521212)'':0.00520227)'':0.0037038)'':0.00315943)'':0.0130012)'':0.00649215)'':0.00462985)'':0.00323043,(('PDT000346488.1':0.0695842,(('PDT000343778.1':0.0630849,('PDT000372521.1':0.0523695,'PDT000124632.1':0.0431356)'':0.015817)'':0.00189225,('PDT000403077.1':0.0596772,('PDT000566199.1':0.0184427,'PDT000569521.1':0.0306902)'':0.0353383)'':0.00196614)'':0.0196618)'':0.0133036,(('PDT000410427.1':0.0800758,(('PDT000687777.2':0.0409543,'PDT000661221.1':0.0385682)'':0.0117447,('PDT000063364.2':0.051969,('PDT000566204.1':0.0654427,(('PDT000333776.1':0.0266362,'PDT000583903.1':0.0323435)'':0.0134373,('PDT000667997.1':0.0287916,'PDT000168850.1':0.0500486)'':0.0112406)'':0.00583846)'':0.0065719)'':-0.00084376)'':0.00867186)'':0.00558781,(('PDT000557237.1':0.0688638,(('PDT000131774.2':0.0667224,'PDT000081768.1':0.0614406)'':0.00272751,('PDT000517287.1':0.0631517,'PDT000228082.1':0.0738593)'':0.00247899)'':0.00567567)'':0.00282495,('PDT000619677.1':0.087906,(('PDT000395001.1':0.0822791,('PDT000034506.2':0.0870908,'PDT000394803.1':0.0664602)'':0.00514042)'':0.00207607,(('PDT000720186.1':0.0329349,'PDT000720223.1':0.0337706)'':0.0355008,(('PDT000150753.2':0.0434345,'PDT000313315.2':0.0418249)'':0.00547211,('PDT000196651.2':0.0472997,'PDT000494439.1':0.0424032)'':0.00934899)'':0.00951817)'':0.014143)'':0.00606795)'':-0.00177677)'':0.0047151)'':0.0111091)'':0.00240254)'':-0.000170683)'':0.00749092)'':0.0187158)'':-0.00330166)'':0.00273475)'':0.00721192)'':0.00597184)'':-0.00894796)'':0.0253701)'':-0.0220651,((((('PDT000129909.2':0.0374224,('PDT000649119.1':0.0236187,'PDT000083562.1':0.0201282)'':0.0124472)'':0.0418823,('PDT000463626.1':0.0873853,('PDT000357673.1':0.0592142,((('PDT000635771.1':0.0327825,'PDT000652375.1':0.0354081)'':0.0109583,('PDT000081285.1':0.03409,('PDT000083458.1':0.0203687,'PDT000083509.1':0.0261017)'':0.0100318)'':0.00967231)'':0.00335603,(('PDT000314195.2':0.0106794,('PDT000537826.1':0.00600239,'PDT000248480.1':0.00703631)'':0.00533783)'':0.0290711,(('PDT000687098.1':0.031244,'PDT000553180.1':0.0311214)'':0.00597281,('PDT000599132.1':0.0303068,('PDT000213249.2':0.0223714,'PDT000083556.1':0.0252616)'':0.00733287)'':0.00375357)'':0.00206664)'':0.005188)'':0.0222066)'':0.0266421)'':-0.0114148)'':0.0231643,((('PDT000444897.1':0.0180322,('PDT000588078.1':0.0141105,'PDT000322036.2':0.0143931)'':0.00674149)'':0.0460665,('PDT000289019.3':0.0782333,(('PDT000212847.3':0.02424,'PDT000141128.3':0.0202264)'':0.011513,(('PDT000148252.3':0.0177005,'PDT000081258.1':0.0161678)'':0.00380724,('PDT000320936.2':0.0170796,('PDT000344343.1':0.0157167,('PDT000444905.1':0.0126788,'PDT000015324.2':0.0128021)'':0.00057711)'':0.00377599)'':0.000637135)'':0.0118439)'':0.0363151)'':-0.00295776)'':0.0207607,(('PDT000319253.1':0.0377134,('PDT000083440.1':0.0270688,('PDT000540399.1':0.0316311,'PDT000576187.1':0.0212721)'':0.00862295)'':0.00504262)'':0.0527082,(('PDT000008476.2':0.0470817,'PDT000392509.1':0.0367836)'':0.0215737,(('PDT000619751.1':0.0405009,'PDT000392514.1':0.0439349)'':0.0146194,('PDT000083540.1':0.0472871,('PDT000083441.1':0.0345424,('PDT000083500.1':0.0406706,('PDT000247545.2':0.0284622,('PDT000083452.1':0.0257235,('PDT000032331.1':0.0169555,'PDT000083489.1':0.0147593)'':0.00549703)'':0.0102809)'':0.00361472)'':0.00467363)'':0.00689426)'':0.00134629)'':0.0199256)'':0.0192765)'':0.00316729)'':0.0350324)'':0.0281643,((((('PDT000571144.1':0.0665193,'PDT000718450.1':0.0683767)'':0.0448288,('PDT000354220.1':0.0488703,('PDT000672248.1':0.0333845,'PDT000571172.1':0.0439777)'':0.0218231)'':0.0475543)'':8.06073e-05,((('PDT000320926.1':0.0538675,'PDT000354198.1':0.0460368)'':0.0317319,('PDT000604198.1':0.0597023,('PDT000327168.1':0.0441754,'PDT000543329.1':0.0451637)'':0.0104356)'':0.0257152)'':0.0120323,(('PDT000260769.2':0.0358123,'PDT000672225.1':0.0362528)'':0.00248019,(('PDT000092538.1':0.0266028,'PDT000485904.1':0.0254948)'':0.00552402,('PDT000498155.1':0.0226058,('PDT000268758.2':0.017965,'PDT000571159.1':0.017247)'':0.00502339)'':0.0026642)'':0.00363691)'':0.0646195)'':0.00684605)'':0.0229097,(((('PDT000631995.1':0.0980725,('PDT000619959.1':0.0889855,('PDT000088278.2':0.0349285,'PDT000466965.1':0.0306118)'':0.0534428)'':0.012686)'':0.00782231,(('PDT000689826.1':0.0619341,'PDT000228053.1':0.0541029)'':0.00706593,('PDT000672630.1':0.0608994,('PDT000718325.1':0.0318746,'PDT000228379.1':0.0344825)'':0.0108566)'':0.0103335)'':0.0444752)'':0.00196205,(('PDT000242875.1':0.0714074,('PDT000668382.2':0.0626565,('PDT000655754.1':0.0511486,'PDT000228264.1':0.0571474)'':0.00734201)'':0.0133551)'':0.0113294,(('PDT000248826.1':0.0412382,('PDT000557248.1':0.0225899,'PDT000655789.1':0.0298971)'':0.0129269)'':0.0213733,('PDT000717612.1':0.0782781,('PDT000640026.1':0.053161,'PDT000672687.1':0.048292)'':0.00847741)'':0.00392393)'':0.00907231)'':0.0425165)'':0.00607288,((('PDT000426416.1':0.119874,('PDT000497323.1':0.0249463,'PDT000710218.1':0.0243412)'':0.0768958)'':0.00309027,(('PDT000124591.1':0.0470087,('PDT000069414.3':0.051034,'PDT000498187.1':0.0411853)'':0.00400564)'':0.0409337,('PDT000346954.1':0.0598443,('PDT000651029.1':0.0438138,'PDT000661083.1':0.0392375)'':0.00729358)'':0.0441712)'':0.0129035)'':0.00737734,(('PDT000636336.1':0.090307,('PDT000069147.2':0.0602973,'PDT000652930.1':0.0705827)'':0.00993354)'':0.0378444,(('PDT000569035.1':0.0869489,(('PDT000047729.3':0.00890432,'PDT000358752.1':0.0175739)'':0.0613099,('PDT000354200.1':0.0610287,'PDT000017916.2':0.0685623)'':0.0111088)'':0.0121765)'':0.0047062,('PDT000427570.1':0.111241,(('PDT000109482.2':0.033182,'PDT000150896.1':0.0419496)'':0.0459661,(('PDT000216128.2':0.074476,'PDT000217125.2':0.074082)'':0.00520482,('PDT000032199.1':0.0484386,'PDT000488282.1':0.0594514)'':0.0244032)'':0.0122506)'':0.00316756)'':0.00156138)'':0.00297054)'':0.00691466)'':0.00124568)'':0.00814985)'':0.00256425,(((('PDT000713504.1':0.0753053,'PDT000661799.1':0.0967747)'':0.0182454,('PDT000050614.1':0.0960857,('PDT000475563.2':0.0775961,('PDT000327265.2':0.0567584,('PDT000077176.2':0.0401015,('PDT000318839.2':0.0335527,'PDT000318888.2':0.0267737)'':0.00749421)'':0.00570428)'':0.016108)'':0.0323774)'':0.00286736)'':0.030742,((('PDT000672540.1':0.0828946,(('PDT000062977.3':0.0117821,'PDT000357629.1':0.0113416)'':0.0589656,('PDT000213267.2':0.0471199,('PDT000326180.1':0.0298242,('PDT000377021.1':0.0205558,'PDT000631693.1':0.0170775)'':0.0213413)'':0.00583256)'':0.0259013)'':0.0280859)'':0.0222511,((('PDT000555762.1':0.0443805,'PDT000399947.1':0.0471593)'':0.00810078,('PDT000577010.1':0.0342106,('PDT000213251.2':0.0181126,'PDT000445097.1':0.018115)'':0.0216281)'':0.0184453)'':0.0116373,('PDT000251987.1':0.040559,(('PDT000600189.1':0.0188489,('PDT000483971.1':0.0134546,'PDT000537058.1':0.0117455)'':0.0113527)'':0.00579904,(('PDT000486028.1':0.0196763,'PDT000579604.1':0.0195626)'':0.000887397,(('PDT000129828.2':0.0208955,('PDT000466058.1':0.0123944,'PDT000599903.1':0.0136995)'':0.00711908)'':0.00190207,(('PDT000414670.1':0.0146266,('PDT000678433.1':0.0141124,('PDT000386243.1':0.00617892,'PDT000408615.1':0.00818228)'':0.00794072)'':0.00317083)'':0.00051353,('PDT000649522.1':0.0131347,(('PDT000583064.1':0.00891248,'PDT000597354.1':0.00910822)'':0.00953162,('PDT000533169.1':0.0113145,('PDT000399961.1':0.0121704,('PDT000359189.1':0.00965007,('PDT000617239.1':0.00935121,'PDT000649270.1':0.00575959)'':0.000489428)'':0.00406721)'':0.00218455)'':-1.46353e-05)'':0.00399467)'':0.000335244)'':0.001659)'':0.0057324)'':0.0014674)'':0.017368)'':0.0242498)'':0.0482141)'':0.00642665,((('PDT000168797.1':0.145523,('PDT000161520.2':0.119506,(('PDT000522706.1':0.0745866,'PDT000713656.1':0.0767924)'':0.0143098,(('PDT000041264.1':0.119581,'PDT000208711.1':0.0904382)'':0.00322,('PDT000188364.1':0.101895,('PDT000426017.1':0.0857257,(('PDT000719276.1':0.084057,('PDT000149562.2':0.0464658,'PDT000233044.1':0.0418063)'':0.0325119)'':0.00447486,((('PDT000590050.1':0.0365464,'PDT000356184.2':0.0300225)'':0.0108824,('PDT000675605.1':0.0464073,'PDT000411319.1':0.0295258)'':0.00828224)'':0.0268948,(('PDT000403170.1':0.0485675,'PDT000714693.1':0.0392288)'':0.0159308,('PDT000683621.1':0.0514598,('PDT000006899.2':0.0379816,('PDT000487011.1':0.0306832,'PDT000197543.1':0.0342202)'':0.0100675)'':0.0130799)'':0.0075614)'':0.00727076)'':0.00808027)'':0.00301758)'':0.0102071)'':-0.00393279)'':0.00141921)'':0.00962346)'':0.0039418)'':-0.00756195,((('PDT000443059.1':0.0332034,('PDT000438183.1':0.0259287,'PDT000429749.1':0.0259775)'':0.00190955)'':0.070601,(('PDT000155624.2':0.0464294,'PDT000429794.1':0.0394472)'':0.0194701,('PDT000375770.1':0.0181805,('PDT000149886.2':0.0107574,'PDT000481015.1':0.0082883)'':0.0063313)'':0.0396309)'':0.0467377)'':0.0026096,((('PDT000713470.1':0.0528292,'PDT000215986.1':0.0569268)'':0.0111569,('PDT000426023.1':0.0578428,('PDT000155572.2':0.0369216,'PDT000357707.1':0.0325982)'':0.0210248)'':0.0221613)'':0.0156242,((('PDT000428887.1':0.0348021,('PDT000438192.1':0.0062582,'PDT000667746.2':0.0052898)'':0.0301957)'':0.0320326,('PDT000357678.1':0.0834397,('PDT000717573.1':0.0563932,('PDT000156803.2':0.0518197,'PDT000011781.2':0.0481903)'':0.0102098)'':0.0197604)'':-0.00112093)'':0.00450142,('PDT000438093.1':0.0621429,(('PDT000716926.1':0.0120862,'PDT000038890.1':0.0121592)'':0.0282414,('PDT000583931.1':0.0378521,('PDT000535447.1':0.0229893,('PDT000132322.2':0.0227207,'PDT000618833.1':0.0222539)'':0.00205857)'':0.0141132)'':0.00874716)'':0.0114783)'':0.0315864)'':0.00943789)'':0.00722752)'':0.00140439)'':0.0078955,((('PDT000711092.1':0.106423,('PDT000517823.1':0.00735055,'PDT000575187.1':0.00461355)'':0.0769948)'':-0.002948,('PDT000162104.1':0.0784118,('PDT000335802.1':0.0464392,('PDT000063375.2':0.0341724,'PDT000251546.1':0.0302771)'':0.0148005)'':0.0180263)'':0.0143215)'':0.0374769,((('PDT000538124.1':0.0686118,('PDT000366109.1':0.0537604,'PDT000463685.1':0.0566146)'':0.00530766)'':0.0186988,((('PDT000161998.2':0.0180204,('PDT000316546.1':0.0141626,'PDT000469387.1':0.0181434)'':0.00306221)'':0.0162724,(('PDT000155379.2':0.0138875,'PDT000711058.1':0.0109165)'':0.00697848,('PDT000445072.1':0.0102607,'PDT000625437.1':0.014314)'':0.012446)'':0.0142137)'':0.00489874,(('PDT000646113.1':0.00992778,('PDT000651200.1':0.00513156,'PDT000188341.1':0.00482662)'':0.00559803)'':0.0157693,('PDT000131915.2':0.0195576,(('PDT000386192.1':0.0113454,'PDT000716939.1':0.0151795)'':0.00411127,('PDT000408600.1':0.0047561,'PDT000611517.1':0.0070882)'':0.0101615)'':0.0111827)'':0.00104223)'':0.0114687)'':0.0520184)'':0.0073356,((('PDT000711122.1':0.107018,'PDT000484697.1':0.0928548)'':0.0103575,('PDT000629981.1':0.0337592,('PDT000253470.2':0.0182142,'PDT000379457.1':0.0209996)'':0.0103451)'':0.06917)'':-0.000665565,((('PDT000011675.3':0.0125096,'PDT000349621.1':0.0130755)'':0.0722935,('PDT000009347.2':0.0720093,('PDT000009348.2':0.056089,'PDT000357703.1':0.055531)'':0.00990269)'':0.0311531)'':0.00953773,(('PDT000280666.2':0.065471,('PDT000205081.2':0.0421517,(('PDT000625503.1':0.0283585,('PDT000635851.1':0.0165367,('PDT000148092.2':0.010701,'PDT000148097.2':0.0161239)'':0.0108069)'':0.00476426)'':0.00564802,(('PDT000626510.2':0.0274565,('PDT000205096.2':0.019359,'PDT000713930.1':0.0212531)'':0.00761529)'':0.0047497,(('PDT000586508.1':0.0196628,'PDT000649130.1':0.0228674)'':0.00106653,('PDT000205075.2':0.016072,('PDT000631701.1':0.0147564,'PDT000635984.1':0.0167032)'':0.00284563)'':0.00957699)'':0.00551727)'':0.00323395)'':0.0103984)'':0.0160977)'':0.0123334,(('PDT000110389.2':0.0126211,'PDT000357633.1':0.0123789)'':0.0662417,(('PDT000385032.1':0.0438807,('PDT000651651.1':0.0436354,('PDT000625372.1':0.0205385,('PDT000319250.1':0.0197891,(('PDT000668188.2':0.00331264,('PDT000671629.1':0.00243061,'PDT000011676.3':0.00186552)'':0.00136914)'':0.00193079,(('PDT000205076.2':0.00215077,'PDT000325787.1':0.00263869)'':0.00327808,('PDT000625409.1':0.00298695,'PDT000713337.1':0.00201155)'':0.00268934)'':0.00241725)'':0.00939953)'':0.00473138)'':0.0142486)'':0.00284744)'':-0.000537086,(('PDT000481329.1':0.0162359,('PDT000451175.1':0.0101769,'PDT000668184.2':0.0138367)'':0.0059965)'':0.00466518,(('PDT000384171.1':0.020682,('PDT000157814.2':0.0138465,('PDT000160300.2':0.00262532,'PDT000542128.1':0.00206389)'':0.00707162)'':0.00415528)'':0.00431363,('PDT000132075.2':0.0179541,('PDT000488015.1':0.0168465,('PDT000321601.1':0.00678583,('PDT000359190.1':0.00224936,'PDT000367190.1':0.00214756)'':0.00403831)'':0.00762043)'':0.00161421)'':0.00146314)'':0.00243876)'':0.0157823)'':0.0450303)'':-0.0249564)'':0.0378687)'':0.00383162)'':-0.00796315)'':0.0206805)'':-0.00599521)'':0.00277474)'':0.0109269)'':-0.00349078,((('PDT000588843.1':0.108296,(('PDT000368395.1':0.063539,'PDT000357672.1':0.062816)'':0.00520528,('PDT000322053.1':0.0153812,('PDT000601946.1':0.0198601,'PDT000660738.1':0.0190015)'':0.00173016)'':0.059672)'':0.0223708)'':0.0202927,((('PDT000129829.2':0.0457201,'PDT000428854.1':0.0465653)'':0.00353138,('PDT000472547.1':0.0249631,('PDT000131831.2':0.0186759,'PDT000649525.1':0.0178484)'':0.00283617)'':0.0291253)'':0.0347432,(('PDT000034407.1':0.0762552,('PDT000205065.2':0.022716,'PDT000658722.1':0.0187774)'':0.0559586)'':0.0133697,('PDT000496787.1':0.0314572,('PDT000635712.1':0.018722,'PDT000645777.1':0.0154703)'':0.0105337)'':0.0569324)'':0.00428)'':0.0393376)'':0.00741695,((('PDT000235276.2':0.0806512,('PDT000429622.1':0.0926345,(('PDT000158671.2':0.0241676,'PDT000585051.1':0.0248664)'':0.0598447,('PDT000600860.1':0.0348471,'PDT000662590.1':0.0274949)'':0.053891)'':0.00207217)'':0.0189289)'':0.0344402,(((('PDT000009350.2':0.0248301,'PDT000009351.2':0.0349223)'':0.0683136,('PDT000557077.1':0.0502883,('PDT000032377.1':0.0106811,('PDT000032375.1':0.00291389,'PDT000032376.1':0.00326942)'':0.0140894)'':0.039335)'':0.0401703)'':0.00665774,(('PDT000677027.1':0.0684657,('PDT000132345.2':0.0515568,('PDT000669118.1':0.0299829,('PDT000316910.1':0.0282588,'PDT000368349.1':0.0282824)'':0.0102781)'':0.0226072)'':0.0147846)'':0.00874102,('PDT000610188.1':0.0542276,(('PDT000319887.1':0.012281,'PDT000285789.2':0.0128107)'':0.0244733,('PDT000253139.2':0.0231256,'PDT000334213.1':0.0263071)'':0.00450397)'':0.0179919)'':0.0233798)'':0.0341766)'':0.00126908,(('PDT000600864.1':0.06553,'PDT000651796.1':0.074582)'':0.0155402,(('PDT000050586.1':0.0876175,('PDT000533184.1':0.0601845,('PDT000034423.1':0.00999097,('PDT000034414.1':0.00238298,('PDT000032220.1':0.00157831,'PDT000034415.1':0.00161754)'':0.000211135)'':0.00388766)'':0.056228)'':0.0157065)'':-0.0102084,('PDT000157992.2':0.065392,('PDT000490193.1':0.054712,(('PDT000032396.1':0.00860717,'PDT000032397.1':0.0118071)'':0.0101999,('PDT000032382.1':0.00630876,('PDT000032381.1':0.00956544,('PDT000032344.1':0.00518812,('PDT000029843.1':0.0054316,'PDT000032385.1':0.00392629)'':0.000399975)'':0.0024171)'':0.00166896)'':0.0180812)'':0.0314891)'':0.00530878)'':0.0196339)'':0.0186777)'':0.00303844)'':0.0174208)'':-0.00175036,((('PDT000610551.1':0.0883161,('PDT000420978.1':0.0551657,('PDT000285321.1':0.0486078,('PDT000397670.1':0.0311909,'PDT000421079.1':0.0286888)'':0.0057677)'':0.0128702)'':0.0414306)'':0.00467934,(('PDT000358749.1':0.0765159,('PDT000397765.1':0.0662069,'PDT000228211.1':0.0730571)'':0.0239096)'':0.00740518,('PDT000357670.1':0.103718,((('PDT000129893.2':0.0312465,'PDT000221776.2':0.041353)'':0.00101032,('PDT000034421.1':0.0111102,('PDT000034418.1':0.00538888,'PDT000034425.1':0.00486092)'':0.00243088)'':0.0222507)'':0.0295964,(('PDT000250134.1':0.0322576,('PDT000332806.1':0.0294646,'PDT000603508.1':0.0234361)'':0.0116965)'':0.0175708,('PDT000427562.1':0.0685175,('PDT000538572.1':0.0402437,('PDT000588029.1':0.0459074,('PDT000649666.1':0.0418608,'PDT000652350.1':0.0314025)'':0.00633212)'':0.00503794)'':0.00411616)'':0.00733766)'':0.00972292)'':0.0275294)'':0.00372416)'':0.00215513)'':0.0151949,((('PDT000346883.1':0.0755174,('PDT000252560.1':0.0555688,('PDT000298013.1':0.0568774,('PDT000425923.1':0.0262002,'PDT000221051.1':0.0194816)'':0.0150856)'':0.0132395)'':0.0073849)'':0.0277981,((('PDT000661682.1':0.0676634,('PDT000425755.1':0.0523578,'PDT000652344.1':0.0533372)'':0.00742663)'':0.00113482,(('PDT000497398.1':0.0601375,'PDT000644967.1':0.0474645)'':0.00349954,('PDT000718570.1':0.0202906,'PDT000395479.1':0.037792)'':0.0243095)'':0.00578754)'':0.00424433,(('PDT000571166.1':0.0730369,('PDT000129830.2':0.0539652,'PDT000250200.1':0.0441134)'':0.0108888)'':-0.00166471,(('PDT000566166.1':0.0611086,'PDT000688668.1':0.0610854)'':0.00239627,('PDT000161991.3':0.0566781,(('PDT000459043.1':0.0352298,'PDT000469389.1':0.0290566)'':0.00844434,('PDT000094031.1':0.0255429,('PDT000502732.2':0.0206129,'PDT000510677.1':0.0144801)'':0.0125524)'':0.00689733)'':0.0103708)'':0.00552487)'':0.00107848)'':0.00166698)'':0.0343175)'':0.0102559,(((('PDT000523983.1':0.103471,'PDT000532332.1':0.100609)'':0.0101633,(('PDT000488016.1':0.0670492,('PDT000655231.1':0.0233154,'PDT000050682.1':0.0318939)'':0.0236972)'':0.0259591,(('PDT000649267.1':0.0473772,('PDT000083174.2':0.0274154,'PDT000575724.1':0.027336)'':0.0188244)'':0.0308301,('PDT000161862.2':0.0410286,(('PDT000325455.1':0.0277322,'PDT000384146.1':0.025054)'':0.0263939,('PDT000600898.1':0.0414777,('PDT000034382.1':0.020863,('PDT000661113.1':0.0150755,('PDT000034375.1':0.0074323,('PDT000029845.1':0.00237818,'PDT000032209.1':0.00390799)'':0.00557522)'':0.00427464)'':0.0133233)'':0.00868452)'':-0.00163942)'':0.0151214)'':0.0221687)'':0.0104297)'':0.0115386)'':0.00190575,(('PDT000652276.1':0.0563385,('PDT000426566.1':0.0352475,'PDT000692373.1':0.0403672)'':0.0169581)'':0.0529299,((('PDT000630640.1':0.0698755,'PDT000357706.1':0.0710745)'':0.0182198,(('PDT000252726.1':0.0676228,'PDT000395163.1':0.0695102)'':0.0115175,('PDT000251963.1':0.0884776,('PDT000468246.1':0.0461535,((('PDT000667971.1':0.0146039,'PDT000469860.1':0.0168898)'':0.00370238,('PDT000357194.1':0.0197445,'PDT000629985.1':0.0199763)'':0.00536262)'':0.00737939,(('PDT000132108.2':0.0180372,'PDT000635769.1':0.0184355)'':0.000300596,('PDT000120872.2':0.0191436,('PDT000434179.1':0.00958093,('PDT000157944.2':0.0111199,'PDT000399954.1':0.0086712)'':0.00130212)'':0.00360425)'':0.00251498)'':0.00456003)'':0.0164234)'':0.0406198)'':0.00201486)'':0.00772622)'':0.00152692,(('PDT000661256.1':0.0768701,('PDT000709896.1':0.0288862,(('PDT000032391.1':0.00519291,'PDT000033990.1':0.00428434)'':0.00103775,('PDT000033993.1':0.00371795,('PDT000032378.1':0.0023311,'PDT000032379.1':0.00186094)'':0.000268877)'':0.00210767)'':0.0173509)'':0.0515037)'':0.0054264,(('PDT000250142.1':0.0807991,('PDT000188184.2':0.0243527,'PDT000535365.1':0.0251844)'':0.0482008)'':-0.0030508,(('PDT000160452.2':0.0365785,('PDT000319786.1':0.0369867,('PDT000131868.2':0.0188502,'PDT000500616.1':0.0171672)'':0.018594)'':0.00163777)'':0.00278915,(('PDT000625414.1':0.0173373,('PDT000156589.2':0.0161842,'PDT000641397.1':0.0147416)'':0.00309517)'':0.00758184,('PDT000716337.1':0.0165445,('PDT000328018.1':0.017877,'PDT000542764.1':0.015653)'':0.00321848)'':0.011049)'':0.00463131)'':0.0474436)'':-0.00376197)'':0.0247769)'':0.00602036)'':-0.00263462)'':0.00216875,(((('PDT000132097.2':0.0458706,'PDT000498770.1':0.048822)'':0.0219361,('PDT000032193.1':0.00503623,'PDT000032194.1':0.00830507)'':0.0725975)'':0.00477722,((('PDT000205089.2':0.0374499,'PDT000625486.1':0.0232941)'':0.0292433,('PDT000228297.1':0.0400704,('PDT000375954.1':0.0150191,'PDT000538640.1':0.0175141)'':0.0182379)'':0.0183995)'':0.01378,('PDT000571160.1':0.0608033,('PDT000213262.2':0.0371102,('PDT000583067.1':0.034197,('PDT000253439.2':0.0217899,('PDT000339392.1':0.0147441,('PDT000625497.1':0.016088,('PDT000469401.1':0.00397902,'PDT000667970.1':0.00539941)'':0.00549924)'':0.00432065)'':0.00532541)'':0.00962204)'':0.00750029)'':0.0228527)'':0.0080273)'':0.0124882)'':-0.000338226,((('PDT000359198.1':0.0575127,('PDT000399946.1':0.0350042,('PDT000518481.1':0.0301044,'PDT000668179.2':0.0298956)'':0.0106026)'':0.0259576)'':0.00685049,(('PDT000625401.1':0.0341049,('PDT000360400.1':0.00769291,('PDT000205064.2':0.0084855,'PDT000600944.1':0.0066223)'':0.000595744)'':0.0239026)'':0.016776,(('PDT000498955.1':0.0279225,('PDT000274718.2':0.021291,'PDT000549214.1':0.0198965)'':0.00729134)'':0.0203269,(('PDT000619167.1':0.0264699,'PDT000354219.1':0.0265782)'':0.00942282,('PDT000161863.2':0.0402003,('PDT000679761.1':0.0419306,('PDT000503004.1':0.0293303,('PDT000205069.2':0.0225206,'PDT000596027.1':0.0248658)'':0.00846155)'':0.0060813)'':0.00466985)'':0.000390891)'':0.00335978)'':0.00256064)'':0.0195896)'':0.0153985,(('PDT000357659.1':0.0807873,('PDT000205097.2':0.0545672,(('PDT000611527.1':0.0177727,'PDT000011778.2':0.0135734)'':0.00707469,(('PDT000677010.1':0.00870964,('PDT000579611.1':0.00695747,'PDT000318848.1':0.00854793)'':0.00421346)'':0.0104586,('PDT000328142.1':0.0270086,('PDT000084208.2':0.0170384,('PDT000148505.2':0.00965885,'PDT000387708.1':0.00698455)'':0.00732866)'':0.00629105)'':0.00200986)'':0.00254262)'':0.0162399)'':0.0424144)'':-0.00870372,((('PDT000382808.1':0.0361502,('PDT000381306.1':0.0343418,('PDT000421961.1':0.0176049,('PDT000228362.1':0.0128588,('PDT000451171.1':0.0133415,'PDT000228358.1':0.0132238)'':0.00339902)'':0.00209029)'':0.0099901)'':0.00749931)'':0.0253961,('PDT000357653.1':0.0431545,('PDT000651385.1':0.0305232,(('PDT000509693.1':0.0207146,'PDT000625405.1':0.0271713)'':0.00355858,('PDT000414402.1':0.0201417,('PDT000661600.1':0.011321,('PDT000132043.2':0.00972877,'PDT000451147.1':0.0100133)'':0.00279663)'':0.00783387)'':0.00263421)'':0.00567317)'':0.0177429)'':0.0124296)'':0.00353266,((('PDT000382803.1':0.0203044,('PDT000557148.1':0.0178635,('PDT000366118.1':0.0183029,'PDT000600981.1':0.0165097)'':0.00612852)'':0.00433038)'':0.0409453,('PDT000248802.1':0.0648493,('PDT000248503.1':0.0355576,('PDT000249842.1':0.0169529,('PDT000366302.1':0.0216882,'PDT000388879.1':0.0187845)'':0.00277076)'':0.0124077)'':0.0300058)'':0.00233521)'':0.00500976,((('PDT000625480.1':0.0317864,'PDT000017860.2':0.0344932)'':0.00538653,(('PDT000556602.1':0.0149297,'PDT000713730.1':0.0100104)'':0.0205547,('PDT000713256.1':0.0276418,('PDT000627210.1':0.00947428,'PDT000399953.1':0.00773792)'':0.0143359)'':0.0061092)'':0.0113826)'':0.0149965,(('PDT000160393.2':0.0695162,('PDT000631694.1':0.0296164,('PDT000378018.1':0.012722,'PDT000412826.1':0.0108074)'':0.0122995)'':0.0140395)'':0.00532048,(('PDT000274713.2':0.0308146,'PDT000274719.2':0.0329841)'':0.014509,(('PDT000253441.2':0.0258161,'PDT000132346.2':0.0265027)'':0.00524954,('PDT000438085.1':0.0271087,'PDT000141261.1':0.0218761)'':0.00440231)'':0.0147638)'':0.00136538)'':0.0080203)'':0.00975456)'':0.000142563)'':0.0113194)'':0.0103937)'':0.00350296)'':0.0151801)'':0.00620806)'':0.00250354)'':-0.00019062)'':0.00897121)'':-0.00450575)'':0.00730777)'':0.00393783)'':-0.00424185,((((('PDT000618725.1':0.0368413,'PDT000667714.2':0.0258129)'':0.0646695,((('PDT000390624.1':0.0290112,'PDT000499462.1':0.029229)'':0.00625712,('PDT000535472.1':0.0311163,('PDT000271714.2':0.0246172,'PDT000362445.1':0.0243134)'':0.0039157)'':0.000986477)'':0.03281,((('PDT000162963.3':0.03155,'PDT000520887.1':0.0371195)'':0.0100352,('PDT000226329.1':0.0370033,('PDT000619141.1':0.0329227,'PDT000346729.1':0.0201159)'':0.00412034)'':0.0148909)'':0.0229135,(('PDT000253484.3':0.0263398,('PDT000120903.3':0.0130583,('PDT000212316.3':0.00962704,'PDT000394075.2':0.00797616)'':0.0025274)'':0.00556037)'':0.0402597,(('PDT000363573.1':0.0697604,('PDT000537764.1':0.04159,'PDT000713281.1':0.0435593)'':0.0129049)'':-0.000893711,(('PDT000483531.1':0.0113895,'PDT000483544.1':0.00597055)'':0.0520488,('PDT000425556.1':0.0634405,(('PDT000352489.1':0.0426915,('PDT000713622.1':0.0418369,'PDT000150885.1':0.0301953)'':0.0057894)'':0.0035523,('PDT000672252.1':0.036145,('PDT000661390.1':0.0406317,('PDT000420675.1':0.00634912,'PDT000627603.1':0.00642538)'':0.0295177)'':0.00724501)'':0.000713478)'':0.00646732)'':-0.000210193)'':0.00184001)'':0.00134238)'':0.00558561)'':0.00194938)'':0.0235891)'':0.0212399,(('PDT000303283.2':0.0803826,'PDT000225236.1':0.0862534)'':0.0419512,((('PDT000132384.2':0.0757015,('PDT000594468.1':0.0695683,(('PDT000131905.2':0.0496729,'PDT000008692.2':0.0688281)'':0.00461775,('PDT000645780.1':0.00761289,'PDT000658751.1':0.0126446)'':0.049977)'':0.00522749)'':0.00180482)'':0.0322411,((('PDT000135989.2':0.0314143,'PDT000556863.1':0.0291591)'':0.0431449,('PDT000623233.1':0.0726923,'PDT000357731.1':0.0716237)'':0.00819769)'':0.00506673,(('PDT000497517.1':0.0606676,'PDT000429792.1':0.0660614)'':0.0218905,('PDT000319861.2':0.100487,('PDT000706400.1':0.0677521,('PDT000364400.1':0.0635358,('PDT000346589.1':0.0621424,('PDT000109364.2':0.0322865,'PDT000252545.1':0.0445911)'':0.0196178)'':0.00511534)'':0.00324555)'':0.010368)'':-0.00140916)'':0.00255219)'':0.0092002)'':0.0162668,((('PDT000392511.1':0.0732888,('PDT000619754.1':0.0469623,('PDT000084149.3':0.0484475,'PDT000720378.1':0.0534445)'':0.00458567)'':0.0164357)'':0.0123076,(('PDT000326252.1':0.0710481,('PDT000132134.2':0.051228,('PDT000619750.1':0.0194169,'PDT000663849.1':0.0238173)'':0.0109945)'':0.0347675)'':0.0156648,('PDT000278767.1':0.0552673,(('PDT000576186.1':0.0532596,'PDT000553133.1':0.0403446)'':0.00517319,('PDT000314176.3':0.0421491,('PDT000325690.1':0.0174186,'PDT000679617.1':0.0138345)'':0.0168411)'':0.0144765)'':0.0161212)'':0.0124551)'':0.00384612)'':0.0382438,('PDT000086970.1':0.159644,(('PDT000216307.2':0.0906923,'PDT000252677.1':0.129552)'':0.0156258,(((('PDT000500757.1':0.0690077,'PDT000661293.1':0.0587843)'':0.00379639,('PDT000288653.2':0.0563511,'PDT000620191.1':0.0575519)'':0.00711011)'':0.00080269,('PDT000228061.1':0.0642441,(('PDT000145179.2':0.060195,'PDT000249973.1':0.056229)'':0.00381167,('PDT000395050.1':0.067123,'PDT000631571.1':0.050541)'':0.00884958)'':0.00633351)'':0.00175683)'':0.00880562,((('PDT000357722.1':0.0587447,'PDT000358757.1':0.0444143)'':0.0275567,(('PDT000399563.1':0.0168901,'PDT000150902.1':0.0165547)'':0.0426939,('PDT000357698.1':0.0608361,('PDT000668481.2':0.0542673,('PDT000367259.1':0.0496152,('PDT000250014.1':0.0391784,('PDT000631945.1':0.0219828,'PDT000669489.1':0.0275105)'':0.0114344)'':0.00471863)'':0.000441959)'':0.00441942)'':0.00328538)'':0.00712863)'':-0.00461501,(('PDT000502554.1':0.0259039,('PDT000563583.1':0.0228703,'PDT000042459.1':0.0209017)'':0.0060278)'':0.030768,('PDT000652771.1':0.0680921,('PDT000494476.1':0.0420419,('PDT000502635.1':0.0526582,('PDT000501828.1':0.0298003,('PDT000375014.1':0.024812,('PDT000714053.1':0.033596,('PDT000557133.1':0.0269266,('PDT000161911.2':0.0151871,'PDT000557240.1':0.020833)'':0.00880645)'':0.00938992)'':0.000254526)'':0.00084218)'':0.00543273)'':0.00235481)'':0.00547722)'':0.00703632)'':0.00729405)'':0.00819622)'':0.0302329)'':0.0179298)'':-0.0244622)'':0.0142108)'':0.00639146)'':-0.00867542)'':-0.000303228,(('PDT000231461.2':0.109623,('PDT000188156.3':0.108099,('PDT000556888.1':0.0490436,'PDT000637311.1':0.0641774)'':0.024894)'':0.0147653)'':0.0257869,(((('PDT000637307.1':0.0741445,('PDT000346535.1':0.0746681,('PDT000346532.1':0.0407278,'PDT000346588.1':0.0355253)'':0.0174618)'':0.0367289)'':0.0277521,(('PDT000645490.2':0.0685756,'PDT000645512.2':0.0772814)'':0.0383169,(('PDT000687912.1':0.0735824,('PDT000251977.1':0.0162036,('PDT000034374.1':0.00459002,'PDT000034381.1':0.00417667)'':0.0128651)'':0.0473054)'':0.0125549,('PDT000652418.1':0.0812976,(('PDT000178592.2':0.0462482,'PDT000412045.1':0.0323309)'':0.0156478,('PDT000281477.2':0.0228428,'PDT000095733.2':0.0299391)'':0.0222713)'':0.0237778)'':0.010444)'':0.0124576)'':0.00348189)'':0.00108078,((('PDT000661224.1':0.0594612,'PDT000345957.1':0.0637928)'':0.0259111,('PDT000357584.1':0.0665149,'PDT000357689.1':0.0617361)'':0.00896939)'':0.0265975,((('PDT000426456.1':0.0757713,('PDT000281115.2':0.0722499,'PDT000320963.1':0.0610521)'':0.00311625)'':0.00555678,('PDT000346510.1':0.0693176,('PDT000124678.1':0.0582748,('PDT000389654.1':0.060249,'PDT000257773.1':0.050207)'':0.00874172)'':0.010525)'':0.00362854)'':0.0135299,((('PDT000145181.2':0.07716,'PDT000497320.1':0.058316)'':0.000927078,('PDT000298051.1':0.0670667,(('PDT000399450.1':0.0406669,'PDT000718906.1':0.0377211)'':0.00787626,('PDT000652414.1':0.0440047,'PDT000294384.1':0.0489606)'':0.00833326)'':0.00329695)'':0.00446804)'':0.0207454,(('PDT000551875.1':0.063168,'PDT000394859.1':0.054501)'':0.0234096,('PDT000496910.1':0.078362,(('PDT000088285.2':0.0412926,'PDT000346057.1':0.0613744)'':0.00259242,('PDT000111324.1':0.023895,('PDT000455219.1':0.0210348,'PDT000566183.1':0.0232478)'':0.00294483)'':0.0193035)'':0.0195811)'':0.014484)'':0.00350078)'':0.00634395)'':0.0133443)'':-0.00190965)'':-0.000857901,(((('PDT000290529.2':0.0556264,'PDT000395255.1':0.0661966)'':0.0052186,('PDT000281476.2':0.0715303,('PDT000274859.2':0.0488875,'PDT000490598.1':0.0435875)'':0.0145787)'':0.00339017)'':0.0228624,(('PDT000095718.2':0.0737959,'PDT000714659.1':0.0681441)'':0.006744,('PDT000132389.2':0.085732,('PDT000228056.1':0.0362394,'PDT000424762.1':0.0326827)'':0.0257894)'':0.00576251)'':0.00720354)'':0.0165776,((('PDT000441840.1':0.0851547,('PDT000319018.1':0.0944439,('PDT000652381.1':0.0486508,('PDT000427483.1':0.0301121,'PDT000618898.1':0.0254272)'':0.0173289)'':0.0352992)'':0.007677)'':0.014813,(('PDT000425820.1':0.111319,'PDT000345938.1':0.110078)'':0.00783278,('PDT000205002.2':0.0862261,('PDT000426765.1':0.102013,('PDT000226486.2':0.080452,('PDT000559256.1':0.0760905,(('PDT000322018.2':0.0470295,'PDT000248481.1':0.0412827)'':0.00992809,('PDT000675228.1':0.0600623,('PDT000303026.2':0.0307424,'PDT000661544.1':0.0315326)'':0.0112002)'':0.00400153)'':0.00967756)'':0.00840666)'':0.00275965)'':0.00287683)'':0.00600103)'':0.0092787)'':0.00278024,(((('PDT000206132.2':0.0906795,'PDT000251605.1':0.0558275)'':0.00759947,('PDT000411317.1':0.0603904,'PDT000482604.1':0.0717066)'':0.00814303)'':0.00461902,(('PDT000003800.2':0.0819908,('PDT000129905.2':0.0721584,'PDT000721989.1':0.0561426)'':0.0109987)'':-0.00251933,('PDT000427051.1':0.0920555,(('PDT000488277.1':0.019697,('PDT000398143.1':0.0190033,'PDT000233046.1':0.0164619)'':0.00228284)'':0.0338227,(('PDT000682390.1':0.0417792,'PDT000015341.2':0.0395248)'':0.00597711,(('PDT000356252.1':0.0520779,('PDT000714586.1':0.0459622,'PDT000205426.1':0.032215)'':0.00802248)'':0.00386575,(('PDT000510077.1':0.0424124,('PDT000357634.1':0.0346876,'PDT000424792.1':0.0354471)'':0.00829535)'':0.00315534,('PDT000617774.1':0.0411428,('PDT000424758.1':0.0433951,('PDT000399997.1':0.0384695,('PDT000683609.1':0.0228191,'PDT000011826.2':0.027924)'':0.013788)'':0.00341178)'':0.000352364)'':0.000876727)'':0.00298817)'':0.00216931)'':0.00290674)'':0.0129032)'':-0.00137588)'':0.00401948)'':0.0182585,(('PDT000275053.4':0.0733199,('PDT000397887.1':0.0550888,('PDT000718871.1':0.0233048,'PDT000252468.1':0.0282172)'':0.0151643)'':0.00484394)'':0.0363121,(('PDT000048131.1':0.0605742,('PDT000228059.1':0.0290441,'PDT000228070.1':0.0300141)'':0.0213199)'':0.0415946,(('PDT000664957.1':0.0270118,'PDT000664975.1':0.0295568)'':0.0724311,(('PDT000319832.1':0.0617591,(('PDT000083462.1':0.0126813,'PDT000083547.1':0.0074603)'':0.0299956,('PDT000082634.1':0.0340612,('PDT000081284.1':0.0500981,('PDT000083456.1':0.00987284,'PDT000083535.1':0.0139438)'':0.0119014)'':0.0128429)'':0.000182551)'':0.0165236)'':0.0126147,(('PDT000426726.1':0.0661306,('PDT000083486.1':0.0547591,('PDT000524025.1':0.0464532,('PDT000120935.2':0.0355836,('PDT000082636.1':0.0193395,'PDT000017889.2':0.0200368)'':0.00691098)'':0.0075988)'':0.00461938)'':0.00698094)'':0.01079,(('PDT000104682.2':0.0642614,'PDT000345974.1':0.0534366)'':0.0200217,((('PDT000672250.1':0.0307227,('PDT000553160.1':0.0164074,'PDT000553161.1':0.0225047)'':0.0122902)'':0.0221982,('PDT000083514.1':0.051081,(('PDT000083493.1':0.0431378,('PDT000619759.1':0.0104849,'PDT000619765.1':0.00935241)'':0.0244359)'':0.00367499,(('PDT000553210.1':0.0286582,'PDT000376243.1':0.0257656)'':0.0070221,('PDT000619757.1':0.0359882,'PDT000392507.1':0.0365517)'':0.00312603)'':0.00230253)'':0.0119454)'':0.00134246)'':0.00065351,((('PDT000557791.1':0.046365,'PDT000083435.1':0.0468359)'':0.00661085,(('PDT000705728.1':0.0454798,'PDT000132006.2':0.0523411)'':0.000830371,(('PDT000161715.2':0.0324157,'PDT000557264.1':0.0367868)'':0.0059926,('PDT000502587.1':0.0431069,'PDT000082651.1':0.0431929)'':0.00171152)'':0.00375808)'':0.0059516)'':-0.000413738,(('PDT000553179.1':0.034828,('PDT000346947.1':0.0277337,('PDT000553173.1':0.0258074,('PDT000502519.1':0.0134253,'PDT000553174.1':0.012776)'':0.00715072)'':0.000609284)'':0.0069576)'':0.0170531,(('PDT000316539.1':0.0402275,('PDT000016359.2':0.0323697,('PDT000016354.2':0.02084,'PDT000354183.1':0.0190186)'':0.0138884)'':0.00444904)'':9.94783e-05,('PDT000083531.1':0.034577,('PDT000132367.2':0.0299239,('PDT000524021.1':0.0217568,'PDT000394266.1':0.0219895)'':0.00744579)'':0.00237256)'':0.00804728)'':0.0107591)'':0.0050055)'':0.00400836)'':0.00941911)'':-0.00440231)'':0.00208818)'':0.0159245)'':0.00228132)'':0.00386259)'':0.00173044)'':0.00565581)'':0.00434367)'':-0.00295132)'':0.0184369)'':-0.0119254)'':3.43821e-05,((((('PDT000524500.1':0.0597917,'PDT000662631.1':0.0856823)'':0.00313662,('PDT000359341.1':0.0458036,('PDT000718842.1':0.0347864,('PDT000645012.1':0.0383257,('PDT000111333.1':0.0236701,('PDT000403159.1':0.0246574,'PDT000252469.1':0.0233167)'':0.00177387)'':0.00633172)'':0.00254813)'':0.00270323)'':0.0371259)'':0.0206902,((('PDT000359679.1':0.0412151,('PDT000249920.1':0.0458109,'PDT000356856.1':0.0407896)'':0.00253813)'':0.023178,('PDT000359677.1':0.0592976,('PDT000281775.4':0.0595499,('PDT000707328.1':0.0452257,(('PDT000637612.1':0.0344042,('PDT000155994.2':0.0349879,'PDT000161997.2':0.0284215)'':0.00799354)'':-0.000869855,('PDT000553124.1':0.0431292,('PDT000553125.1':0.0388125,('PDT000553127.1':0.0187552,'PDT000553131.1':0.0284897)'':0.0171418)'':0.00349034)'':0.00301346)'':0.00223491)'':0.00339606)'':0.0158984)'':0.00117514)'':0.00123416,((('PDT000148892.2':0.0285257,'PDT000553152.1':0.0290115)'':0.00613173,('PDT000384423.1':0.023617,('PDT000317182.1':0.0202772,'PDT000553151.1':0.0167342)'':0.00412199)'':0.00920842)'':0.0044828,('PDT000421833.1':0.0475162,('PDT000251803.1':0.0492159,('PDT000163310.3':0.0406525,(('PDT000150172.1':0.0317059,'PDT000150173.1':0.0313308)'':0.00450649,('PDT000553148.1':0.0319231,('PDT000150174.1':0.0340539,('PDT000010342.3':0.0345261,(('PDT000483237.1':0.0269768,('PDT000376176.1':0.0149045,('PDT000332902.1':0.0167968,'PDT000150169.1':0.02193)'':0.000527686)'':0.00643814)'':0.00151585,(('PDT000258486.2':0.0230542,(('PDT000115042.2':0.0171319,'PDT000351570.2':0.0184908)'':0.00374437,('PDT000378022.1':0.0238031,'PDT000648316.1':0.0228361)'':0.00212246)'':0.00243187)'':-0.00250937,('PDT000191352.3':0.0309078,('PDT000343826.1':0.0284204,('PDT000155345.2':0.0182585,('PDT000344005.1':0.018869,(('PDT000150171.1':0.00950754,'PDT000150175.1':0.00996206)'':0.00407148,('PDT000150170.1':0.0115263,('PDT000714581.1':0.012339,'PDT000008731.2':0.00990529)'':0.000822087)'':0.00152406)'':0.00504663)'':0.00197438)'':0.00298645)'':0.00116586)'':-0.00332038)'':0.00181795)'':0.00412023)'':0.00235323)'':0.00174725)'':-0.00165989)'':0.00606539)'':0.00264053)'':0.00194293)'':-0.00758979)'':0.0299563)'':0.0304563)'':0.00695146,(((('PDT000625415.1':0.0701974,'PDT000645648.2':0.0703826)'':0.0104317,('PDT000652837.1':0.0537812,('PDT000161640.2':0.028988,'PDT000625504.1':0.0333722)'':0.0159394)'':0.0276116)'':0.00586249,(('PDT000652900.1':0.0166858,'PDT000652984.1':0.0154198)'':0.0773296,('PDT000343641.1':0.0491069,('PDT000661096.1':0.0204579,('PDT000535018.1':0.00916189,'PDT000661128.1':0.00859431)'':0.00958327)'':0.0261065)'':0.0235226)'':0.00438329)'':0.00918696,(('PDT000132388.2':0.0818054,('PDT000205450.1':0.0766357,(('PDT000132326.2':0.0500245,'PDT000016372.2':0.0451999)'':0.0125823,('PDT000652987.1':0.0585587,(('PDT000426420.1':0.0164735,'PDT000019605.1':0.0207137)'':0.0244982,('PDT000653005.1':0.0582908,'PDT000019603.1':0.0549952)'':0.000654163)'':0.00663585)'':0.00856244)'':0.0103847)'':0.00801064)'':0.0110492,(('PDT000201853.2':0.0883448,('PDT000642308.1':0.0469913,('PDT000551916.1':0.0337929,'PDT000566174.1':0.0390066)'':0.019463)'':0.016779)'':0.0309092,((('PDT000566213.1':0.0609117,('PDT000399606.1':0.0413464,('PDT000399627.1':0.0292074,'PDT000468329.1':0.0276041)'':0.00769563)'':0.0187662)'':0.0182166,(('PDT000310431.2':0.0682221,('PDT000471009.1':0.0466349,'PDT000639592.1':0.0479494)'':0.0226882)'':0.00448316,(('PDT000204880.2':0.0550968,'PDT000539272.1':0.0559512)'':0.0123833,(('PDT000206117.2':0.0228648,'PDT000206133.2':0.0179354)'':0.0298623,('PDT000150298.2':0.0453017,(('PDT000399456.1':0.0196331,'PDT000477156.1':0.024434)'':0.00733618,('PDT000294546.2':0.0251566,('PDT000482513.1':0.0206637,'PDT000600989.1':0.019356)'':0.00721585)'':0.000443438)'':0.0242917)'':0.00805508)'':0.0108808)'':0.0100406)'':0.00213189)'':0.00550417,(('PDT000645477.2':0.0917846,(('PDT000322089.1':0.0584024,'PDT000652874.1':0.0631156)'':0.0104907,('PDT000132209.2':0.0436899,'PDT000408766.1':0.0402517)'':0.014218)'':0.0230999)'':-0.0124058,(('PDT000482821.1':0.0841497,('PDT000645483.2':0.032877,'PDT000645498.2':0.0364407)'':0.0493234)'':0.00104806,('PDT000625467.1':0.0815573,(('PDT000653024.1':0.0632578,('PDT000128827.2':0.0721952,('PDT000323298.1':0.0547741,'PDT000619112.1':0.0583299)'':0.00597776)'':0.00388459)'':0.00639081,((('PDT000625386.1':0.0399022,'PDT000346023.1':0.0432132)'':0.0245843,('PDT000625388.1':0.0557673,('PDT000625366.1':0.0309345,('PDT000346011.1':0.0185124,'PDT000346452.1':0.0143426)'':0.0122026)'':0.0313028)'':0.000362663)'':0.00840975,(('PDT000651549.1':0.0707379,('PDT000711022.1':0.0680742,('PDT000259568.2':0.0469921,'PDT000407370.1':0.0454786)'':0.014548)'':0.00801352)'':-0.00498516,(('PDT000208422.2':0.0608571,('PDT000652830.1':0.0468131,'PDT000652996.1':0.0519429)'':0.00744336)'':0.000605123,(('PDT000230508.2':0.0611793,'PDT000485435.1':0.0519087)'':0.0129419,(('PDT000496893.1':0.0540268,(('PDT000302817.2':0.03751,'PDT000364419.1':0.0402753)'':0.00286864,('PDT000435580.1':0.0440151,'PDT000645098.1':0.0418668)'':0.000424332)'':0.00376708)'':0.00244358,(('PDT000201851.2':0.0479182,'PDT000652989.1':0.0514221)'':0.00506236,('PDT000517943.1':0.0399023,('PDT000339379.1':0.0371929,(('PDT000259566.2':0.0247112,'PDT000062544.2':0.0211901)'':0.0039957,('PDT000386711.1':0.0307379,('PDT000303132.2':0.0181762,'PDT000471042.1':0.0177071)'':0.00726286)'':0.0022731)'':0.00570429)'':0.00830408)'':0.00981888)'':-0.00169664)'':0.00695535)'':0.00128855)'':0.00385954)'':0.00383315)'':0.00381723)'':0.00239765)'':0.00127991)'':-0.00313935)'':0.0127182)'':0.00973413)'':-0.00113093)'':0.00241686)'':0.0182064)'':0.00743523,(((('PDT000200125.4':0.0889289,'PDT000251733.1':0.0881011)'':0.0142856,('PDT000589390.1':0.0691158,'PDT000518041.1':0.0506582)'':0.0192589)'':0.0281054,((('PDT000290391.2':0.0556145,'PDT000228004.1':0.0608255)'':0.0566144,((('PDT000281072.2':0.103348,'PDT000616548.1':0.0847069)'':0.01435,(('PDT000652899.1':0.0707881,'PDT000228011.1':0.0672879)'':0.00779479,('PDT000627432.1':0.0655933,(('PDT000303268.2':0.0423824,'PDT000524502.1':0.0390621)'':0.00813699,('PDT000025500.2':0.0335422,'PDT000108224.2':0.0477219)'':0.00505634)'':0.0249394)'':0.00306606)'':0.0260962)'':0.00324658,(('PDT000568756.1':0.101925,('PDT000718872.1':0.0777069,('PDT000557351.1':0.0797208,'PDT000653008.1':0.0744462)'':0.00924058)'':0.00475498)'':0.0190123,(('PDT000556911.1':0.0778221,'PDT000718845.1':0.0741259)'':0.0165076,(('PDT000200195.2':0.0866832,'PDT000708211.1':0.0939918)'':0.0115241,(('PDT000395308.1':0.0740437,('PDT000396828.1':0.0459072,'PDT000228175.1':0.0416826)'':0.0171684)'':0.00530547,('PDT000550396.1':0.0458237,('PDT000397793.1':0.0408798,'PDT000397967.1':0.0296336)'':0.00981655)'':0.0214583)'':0.013299)'':0.0123379)'':0.004165)'':0.00576135)'':0.00741681)'':0.0024693,((('PDT000425694.1':0.0976565,('PDT000495057.1':0.0768227,('PDT000003806.2':0.094119,('PDT000399700.1':0.0555541,'PDT000395257.1':0.0482369)'':0.032908)'':0.00449468)'':0.0146478)'':0.00592011,(('PDT000422922.1':0.0523177,('PDT000124719.1':0.0438647,'PDT000495420.1':0.0366274)'':0.00683902)'':0.00697144,(('PDT000426009.1':0.025399,'PDT000414206.1':0.0329894)'':0.0300449,('PDT000532506.1':0.0282934,'PDT000495424.1':0.0230537)'':0.0372844)'':0.0117498)'':0.0459994)'':0.00662195,((('PDT000346574.1':0.0943754,(('PDT000518068.1':0.038253,'PDT000566023.1':0.0356592)'':0.0189778,(('PDT000228103.1':0.00778258,'PDT000228143.1':0.00567032)'':0.0417468,('PDT000557035.1':0.0271669,'PDT000252692.1':0.0335738)'':0.0160857)'':0.00974116)'':0.0194754)'':0.00987501,(('PDT000346003.1':0.120317,('PDT000667748.2':0.095051,('PDT000167709.2':0.0481569,'PDT000346507.1':0.040289)'':0.0173956)'':0.014212)'':-0.00424952,(('PDT000196644.2':0.0551787,'PDT000389650.1':0.0483853)'':0.0252815,(('PDT000510068.1':0.0650909,'PDT000537199.1':0.0606691)'':0.00882701,(('PDT000249877.1':0.0761988,('PDT000290388.2':0.019586,('PDT000394848.1':0.0197868,'PDT000252511.1':0.0274985)'':0.0042458)'':0.0400409)'':0.00202019,(('PDT000619755.1':0.0368413,'PDT000625368.1':0.0355565)'':0.0222597,('PDT000565996.1':0.0667558,('PDT000424785.1':0.0471521,('PDT000424780.1':0.0276925,('PDT000356903.1':0.0237851,'PDT000467000.1':0.0270291)'':0.00872861)'':0.0110039)'':0.00475175)'':0.00865779)'':0.00302179)'':0.00130327)'':0.0084433)'':-0.000905881)'':0.00316144)'':0.00310531,(('PDT000324190.2':0.098844,((('PDT000150087.3':0.010149,'PDT000377025.2':0.00988271)'':0.0267271,('PDT000395682.2':0.0286462,'PDT000123318.1':0.0176721)'':0.0240677)'':0.0371548,(('PDT000132044.3':0.0307605,('PDT000148173.3':0.0275402,'PDT000566853.2':0.0211947)'':0.00786172)'':0.0428605,('PDT000157942.3':0.0473807,('PDT000352662.2':0.00918911,'PDT000426863.2':0.00801799)'':0.0338137)'':0.0267618)'':0.0103916)'':0.012849)'':-0.00825064,(('PDT000251423.1':0.0924377,('PDT000160348.2':0.0761038,('PDT000557188.1':0.0673897,((('PDT000145160.2':0.0367919,'PDT000343960.1':0.0309799)'':0.0160948,('PDT000687978.1':0.0479721,'PDT000354218.1':0.0506843)'':0.000998411)'':0.00306256,(('PDT000318959.1':0.0484951,'PDT000017791.2':0.0343665)'':0.00543621,('PDT000346543.1':0.0318165,'PDT000346596.1':0.0349543)'':0.0310027)'':0.00562588)'':0.00793054)'':0.00740315)'':0.0052433)'':-0.00500807,('PDT000485852.2':0.0789757,(('PDT000502385.1':0.0660641,('PDT000641404.2':0.0229567,'PDT000251683.1':0.0273412)'':0.0322644)'':0.0010948,(('PDT000140580.4':0.015898,'PDT000140583.3':0.0177312)'':0.0174981,(('PDT000257293.3':0.0314494,('PDT000557525.1':0.00993277,'PDT000251611.1':0.00873363)'':0.0262207)'':0.000109289,('PDT000123303.1':0.0413323,('PDT000139175.3':0.0225025,(('PDT000327310.2':0.00444541,'PDT000060972.3':0.00543384)'':0.0120811,('PDT000190686.3':0.0121659,'PDT000434235.2':0.0114148)'':0.00295997)'':0.00254353)'':0.00847063)'':-0.000788844)'':0.00640777)'':0.0173569)'':0.028791)'':-0.00184295)'':0.0100856)'':0.000488862)'':0.0103247)'':0.00238782)'':0.00738253)'':-0.00374527,(((('PDT000110154.3':0.0440972,'PDT000115415.3':0.0452909)'':0.010224,(('PDT000319313.2':0.0404268,('PDT000711036.1':0.0283697,'PDT000498130.1':0.0253895)'':0.00897663)'':0.00572157,(('PDT000718584.1':0.0234849,'PDT000123346.1':0.0244516)'':0.0253405,('PDT000322439.2':0.0258951,('PDT000343962.2':0.00636224,'PDT000350548.2':0.00510566)'':0.0140412)'':0.0212675)'':0.00132221)'':0.0154396)'':0.0211643,((('PDT000318635.2':0.0564688,('PDT000326094.2':0.0144383,('PDT000322057.2':0.0157365,'PDT000542757.2':0.00983063)'':0.00637209)'':0.0192226)'':-0.00309862,('PDT000398525.2':0.0308651,('PDT000496087.2':0.0214023,(('PDT000319244.2':0.00890049,'PDT000378619.2':0.00856331)'':0.00596238,('PDT000332190.2':0.0165642,('PDT000532973.2':0.00352324,'PDT000635574.2':0.0035527)'':0.010978)'':0.00440617)'':0.00331694)'':0.00774172)'':0.0020521)'':0.0151136,(('PDT000550021.2':0.0394886,'PDT000651782.2':0.0387823)'':0.0081792,('PDT000332871.2':0.0549459,('PDT000108903.3':0.0408491,(('PDT000314201.3':0.0208484,('PDT000408661.2':0.0093045,('PDT000324083.2':0.0105284,'PDT000338960.2':0.00841255)'':0.00345285)'':0.00746814)'':0.0249536,(('PDT000349619.2':0.0454497,('PDT000498965.2':0.0179418,('PDT000316540.2':0.0171192,'PDT000319822.2':0.012843)'':0.00160068)'':0.0216571)'':-0.00191172,(('PDT000383913.2':0.0426202,'PDT000620328.2':0.0286052)'':0.00476346,(('PDT000322918.2':0.0361768,'PDT000623049.2':0.0330458)'':0.00270355,('PDT000324634.2':0.0380785,'PDT000550093.2':0.0374967)'':0.000920727)'':0.00194252)'':0.00342809)'':0.00206633)'':0.00752924)'':0.00377892)'':-0.00390535)'':0.01344)'':0.0118067)'':0.0308676,((('PDT000535373.1':0.0558078,'PDT000618882.1':0.0582922)'':0.0257496,('PDT000437882.1':0.0863747,('PDT000132278.2':0.0705425,(('PDT000465814.1':0.0227278,'PDT000485851.1':0.024316)'':0.0223558,('PDT000357643.1':0.0416892,('PDT000325811.1':0.0151725,'PDT000605101.1':0.0148611)'':0.0328332)'':0.00787933)'':0.0154374)'':0.0165814)'':-0.00394174)'':0.034832,((('PDT000065358.1':0.0362798,('PDT000200964.3':0.0233992,'PDT000200967.3':0.0342286)'':0.00894361)'':0.0255814,(('PDT000502344.1':0.052403,(('PDT000497747.1':0.0171771,('PDT000498170.1':0.0125132,'PDT000498188.1':0.0108763)'':0.00899935)'':0.00610074,(('PDT000502346.1':0.0249773,'PDT000048141.1':0.0288031)'':0.00603541,('PDT000502341.1':0.0193554,('PDT000190353.3':0.00754218,('PDT000707411.1':0.00804482,'PDT000571832.1':0.00773998)'':0.00133577)'':0.00520286)'':0.00751309)'':0.00165481)'':0.0157088)'':0.00319229,(('PDT000663943.1':0.0896356,('PDT000351879.2':0.0323434,('PDT000151070.3':0.0242989,('PDT000502471.1':0.0131901,('PDT000502299.1':0.0221756,('PDT000502342.1':0.00999342,'PDT000502361.1':0.0150166)'':0.00758354)'':0.00206072)'':0.00278182)'':0.0163142)'':0.0136735)'':-0.00490711,(('PDT000110443.3':0.00849462,'PDT000110444.3':0.0126699)'':0.0215224,(('PDT000110500.3':0.0174883,'PDT000123290.1':0.0260503)'':0.0218229,(('PDT000497921.1':0.0189577,'PDT000497935.1':0.0209443)'':0.00585495,('PDT000324027.2':0.0320834,('PDT000040367.4':0.0185032,('PDT000038854.1':0.0220262,('PDT000132314.3':0.00924779,('PDT000344870.2':0.00584655,'PDT000523436.2':0.00598835)'':0.00343396)'':0.00783008)'':0.00343148)'':0.00349949)'':-0.000897741)'':0.00721927)'':0.00543243)'':-0.00159903)'':0.00339575)'':0.010293)'':0.0228834,(('PDT000487468.2':0.0353765,('PDT000333295.2':0.040396,'PDT000038860.1':0.0383181)'':0.00941453)'':0.0475162,(('PDT000161611.3':0.0331077,'PDT000137388.3':0.0316153)'':0.0425466,((('PDT000332809.2':0.0155753,'PDT000600053.2':0.0218838)'':0.00964349,('PDT000132567.3':0.0109523,'PDT000135727.3':0.0116712)'':0.0130905)'':0.018129,(((('PDT000325670.2':0.0268661,'PDT000314218.3':0.0309916)'':0.00157217,(('PDT000275712.3':0.0200978,'PDT000051483.1':0.0212375)'':0.00570177,('PDT000376142.2':0.0190747,'PDT000624192.2':0.0193831)'':0.005829)'':0.00399174)'':0.00323251,('PDT000711059.1':0.0380881,(('PDT000254240.3':0.0314491,'PDT000532171.2':0.0303345)'':0.00348761,(('PDT000405454.2':0.0272865,'PDT000496089.2':0.0266518)'':0.00353408,(('PDT000131830.3':0.0257332,('PDT000319796.2':0.0233836,('PDT000323720.2':0.0260491,(('PDT000115452.3':0.0161129,'PDT000040402.4':0.0129085)'':0.00342711,('PDT000323321.2':0.0155555,('PDT000464160.2':0.00999773,'PDT000562134.2':0.00963737)'':0.00577552)'':0.0031446)'':0.00423554)'':0.00317624)'':0.00156217)'':-0.000901404,(('PDT000357215.2':0.0214297,'PDT000040394.4':0.0219252)'':0.00927417,(('PDT000325073.2':0.0161741,('PDT000161496.3':0.0126272,'PDT000321599.2':0.0109863)'':0.00332825)'':0.00621385,('PDT000451107.2':0.0182773,('PDT000325697.2':0.0173662,'PDT000479289.2':0.0174215)'':0.00461593)'':0.00302088)'':0.00581532)'':-0.00181505)'':0.00261492)'':0.00319997)'':0.00715401)'':-0.00661507)'':-0.000998268,('PDT000186216.1':0.0636051,((('PDT000145365.3':0.0190128,'PDT000319843.2':0.016986)'':0.00459807,(('PDT000132368.3':0.0218428,('PDT000332162.2':0.0149806,'PDT000548307.2':0.0174561)'':0.00237734)'':0.00123385,('PDT000386069.2':0.0295759,('PDT000603047.2':0.0107606,('PDT000532975.2':0.0134491,('PDT000503390.2':0.0102404,('PDT000316149.2':0.00756957,'PDT000317765.2':0.00782883)'':0.00155215)'':0.000743677)'':0.00226624)'':0.00641733)'':0.000494374)'':0.000335502)'':0.00264127,(('PDT000469887.2':0.0256367,('PDT000332873.2':0.0166783,('PDT000342237.2':0.0216485,(('PDT000417234.2':0.0167735,('PDT000466211.2':0.0134104,('PDT000327104.2':0.0074107,'PDT000384308.2':0.0108614)'':0.00158896)'':0.00277587)'':0.00250246,('PDT000661998.1':0.0233976,('PDT000687067.1':0.0129638,('PDT000641368.2':0.013854,('PDT000156990.3':0.0095412,('PDT000327128.2':0.00779509,('PDT000492127.2':0.00846751,('PDT000548387.2':0.00740004,('PDT000110462.3':0.00508441,('PDT000123328.1':0.00464222,'PDT000208681.1':0.00432997)'':0.000490021)'':0.00112771)'':9.79147e-05)'':0.00196194)'':0.00115517)'':0.00254007)'':0.00124001)'':0.00541452)'':-0.00513143)'':0.00384103)'':0.0033125)'':0.001891)'':-0.00411337,(('PDT000481469.2':0.0203118,('PDT000412488.2':0.0119915,('PDT000520393.2':0.0152865,('PDT000316851.2':0.00918545,('PDT000406798.2':0.00715181,('PDT000377937.2':0.00543435,('PDT000339397.2':0.00440857,'PDT000711397.1':0.0054569)'':0.000530715)'':0.00209668)'':0.000769249)'':0.00267219)'':0.00119102)'':0.00483539)'':0.00122806,('PDT000148230.3':0.0254726,(('PDT000333302.2':0.0200629,'PDT000667870.2':0.0143588)'':0.000258475,(('PDT000334470.2':0.0160838,('PDT000494510.2':0.00314458,'PDT000571225.2':0.00304243)'':0.0106645)'':0.00292159,('PDT000598448.2':0.0214565,('PDT000375977.2':0.0217539,(('PDT000625868.2':0.00864734,('PDT000155416.3':0.00794536,'PDT000619462.2':0.00755394)'':0.00136276)'':0.000361569,(('PDT000410704.2':0.0100634,('PDT000326097.2':0.00567733,'PDT000714267.1':0.00595867)'':0.00301928)'':0.00283975,(('PDT000327073.2':0.00961536,('PDT000326150.2':0.00552195,'PDT000408654.2':0.00564125)'':0.00220589)'':-0.000139998,(('PDT000562988.2':0.00466946,('PDT000387009.2':0.00446809,('PDT000156780.3':0.00309278,'PDT000314842.3':0.00279427)'':0.00140272)'':0.000850117)'':0.0008022,(('PDT000327160.2':0.00666644,'PDT000377068.2':0.00507716)'':0.000666736,('PDT000161596.3':0.00543002,('PDT000324050.2':0.00245958,'PDT000115413.3':0.0026277)'':0.00223377)'':0.00119061)'':0.000990316)'':0.00136407)'':0.000956887)'':0.00101288)'':0.00643053)'':0.000941205)'':-0.00156056)'':0.00516398)'':0.00252624)'':-0.00880362)'':0.00291296)'':0.0057794)'':0.0268284)'':-0.02502)'':0.0130614)'':0.0240881)'':0.0289869)'':-0.00314576)'':0.0258656)'':-0.00915994)'':0.0136788)'':0.00388155)'':0.000342572)'':0.00841042)'':0.00315672)'':0.0124714)'':-0.00513974)'':0.0213425)'':0.0174345)'':0.0109026)'':0.00268109)'':0.00278425)'':0.015879)'':0.021401,((('PDT000360134.1':0.20088,(((('PDT000160998.2':0.0619211,'PDT000426279.1':0.0746539)'':0.0416717,(('PDT000107243.2':0.0571124,'PDT000132313.2':0.0681976)'':0.0402997,('PDT000425944.1':0.0813492,'PDT000015331.2':0.0840618)'':0.00299607)'':0.0222223)'':0.0120877,(('PDT000619642.1':0.0939358,'PDT000675326.1':0.104547)'':0.0236388,('PDT000650447.2':0.0731892,('PDT000510040.1':0.0743729,(('PDT000510031.1':0.0435994,'PDT000719084.1':0.0457533)'':0.0215652,('PDT000352479.1':0.0552323,('PDT000580484.1':0.0522476,('PDT000510058.1':0.0319056,'PDT000661320.1':0.0455074)'':0.0183948)'':0.00362633)'':0.00631699)'':0.0162697)'':0.00596043)'':0.0324)'':0.00464302)'':0.0230477,(('PDT000372010.1':0.136395,('PDT000661452.1':0.125979,'PDT000319039.1':0.150933)'':0.00625617)'':0.00294593,(('PDT000398000.1':0.0656305,'PDT000568777.1':0.0838675)'':0.0797369,((('PDT000467023.1':0.0482487,'PDT000285168.1':0.0626473)'':0.062884,('PDT000345831.1':0.101005,('PDT000003840.2':0.0981154,'PDT000361000.1':0.0771756)'':0.00840272)'':0.00251211)'':0.0232511,('PDT000520729.1':0.138381,('PDT000654201.1':0.12917,('PDT000205442.1':0.0975443,('PDT000311245.2':0.0798402,('PDT000380303.1':0.0742691,'PDT000661627.1':0.0810709)'':0.00742226)'':0.00738843)'':0.0262519)'':0.00876111)'':0.0013252)'':0.0124661)'':0.00100746)'':-0.00756642)'':0.0633769)'':0.00952285,((('PDT000500482.1':0.04262,'PDT000600991.1':0.034032)'':0.00824568,(('PDT000259582.2':0.0363575,('PDT000384282.1':0.0143203,('PDT000294665.2':0.00409639,'PDT000594465.1':0.00447361)'':0.0142913)'':0.0187242)'':0.0101733,(('PDT000490595.1':0.0179085,'PDT000490601.1':0.0113715)'':0.0266083,('PDT000109460.2':0.0281175,('PDT000332090.1':0.0209956,('PDT000275527.2':0.0296576,'PDT000109429.2':0.0164901)'':0.00418279)'':0.00895528)'':0.00672896)'':0.00857202)'':0.00797961)'':0.134913,(('PDT000063348.2':0.11921,'PDT000571167.1':0.147315)'':0.0621332,((('PDT000714113.1':0.0743971,('PDT000135947.2':0.0513558,'PDT000137864.2':0.0494272)'':0.0114194)'':0.107137,(('PDT000644881.1':0.0158445,'PDT000714239.1':0.012577)'':0.0331264,('PDT000545606.1':0.030024,('PDT000394988.1':0.025604,'PDT000639553.1':0.019837)'':0.0123315)'':0.0114816)'':0.124897)'':0.00237275,((('PDT000252647.1':0.0563614,('PDT000566228.1':0.0604943,'PDT000619335.1':0.0643057)'':0.0103326)'':0.029511,(('PDT000051495.3':0.0276597,'PDT000497317.1':0.0406578)'':0.0615414,('PDT000160837.2':0.0939526,(('PDT000669481.1':0.0553743,('PDT000047685.4':0.0302639,'PDT000250102.1':0.0355926)'':0.0130773)'':0.0220281,(('PDT000258343.2':0.0304792,'PDT000566091.1':0.0342648)'':0.00929462,('PDT000034513.2':0.0432198,('PDT000594325.1':0.0282411,'PDT000395303.1':0.034162)'':0.00977678)'':0.00689622)'':0.0282597)'':0.0144204)'':-0.00309454)'':0.00181515)'':0.0718257,((('PDT000188104.2':0.118757,('PDT000373304.1':0.0768845,('PDT000716064.1':0.0651388,'PDT000395478.1':0.0622752)'':0.019587)'':0.0465786)'':-0.0203398,(('PDT000034509.2':0.0444581,('PDT000566189.1':0.0405286,'PDT000395212.1':0.0346922)'':0.0140602)'':0.0210618,(('PDT000395181.1':0.044679,('PDT000003947.2':0.0530915,'PDT000654248.1':0.0370861)'':0.00851766)'':0.0121904,(('PDT000644990.1':0.0489537,'PDT000662180.1':0.0472278)'':0.0206318,('PDT000660402.1':0.0677581,'PDT000395436.1':0.0658059)'':0.0121017)'':0.00770254)'':0.00891585)'':0.0367763)'':0.010577,(('PDT000018406.2':0.100403,(('PDT000425849.1':0.0415148,'PDT000521061.1':0.0377152)'':0.0251012,('PDT000444939.1':0.0414953,('PDT000319015.1':0.0359429,'PDT000483546.1':0.0296757)'':0.00212499)'':0.014024)'':0.031441)'':0.0480119,(((('PDT000426138.1':0.0973865,'PDT000691296.1':0.0682805)'':0.0364865,('PDT000425702.1':0.0949187,('PDT000107791.2':0.0753544,'PDT000645103.1':0.0675696)'':0.0176713)'':0.0231011)'':0.0052959,(('PDT000399640.1':0.0589486,('PDT000610085.1':0.0377428,'PDT000395334.1':0.0412884)'':0.0122193)'':0.0389902,(('PDT000378582.1':0.0688109,'PDT000644975.1':0.0706741)'':0.00822199,('PDT000394927.1':0.0761708,('PDT000566109.1':0.046894,'PDT000566161.1':0.0411569)'':0.0173817)'':0.0246133)'':0.00605493)'':0.0187691)'':0.00260846,('PDT000101297.1':0.119591,(('PDT000319032.1':0.128077,(('PDT000318979.1':0.113423,('PDT000034459.2':0.0763464,('PDT000716107.1':0.0740129,'PDT000358772.1':0.0802101)'':0.0137596)'':0.018113)'':-0.00052277,('PDT000009630.2':0.0776504,('PDT000395175.1':0.0525364,('PDT000395098.1':0.0533746,'PDT000690313.1':0.0460819)'':0.00766286)'':0.0194621)'':0.0237072)'':0.0218196)'':-0.00496853,('PDT000639528.1':0.097108,('PDT000716111.1':0.0865354,('PDT000716097.1':0.0988996,('PDT000716228.1':0.0786651,(('PDT000426833.1':0.0634949,'PDT000590034.1':0.0611271)'':0.0270786,('PDT000610207.1':0.0661693,(('PDT000131827.2':0.0469635,'PDT000716294.1':0.0432858)'':0.00837173,('PDT000032353.1':0.0452889,'PDT000318989.1':0.0381574)'':0.00539604)'':0.0135486)'':0.00525343)'':0.00523112)'':0.00817424)'':0.00383474)'':0.00668857)'':0.024153)'':0.00800784)'':-0.00990081)'':0.00669366)'':0.00611518)'':0.0529695)'':0.0223009)'':0.00790817)'':0.00417248)'':0.00094571)'':0.00878465,((('PDT000661752.1':0.13366,((('PDT000442391.1':0.0441705,('PDT000649268.1':0.0252877,('PDT000425593.1':0.00825741,'PDT000426034.1':0.0164056)'':0.00946531)'':0.0312037)'':0.0442365,('PDT000540519.1':0.0439882,('PDT000566078.1':0.0309668,('PDT000319742.1':0.0254134,('PDT000285391.2':0.0181679,'PDT000394887.1':0.0262304)'':0.00994587)'':0.00121973)'':0.0178252)'':0.0426372)'':0.0176823,(('PDT000213271.2':0.0799458,'PDT000520847.1':0.0727882)'':0.0342926,((('PDT000557339.1':0.0854193,'PDT000714019.1':0.0816597)'':0.0167087,('PDT000249899.1':0.0963331,((('PDT000714278.1':0.00877026,'PDT000174378.1':0.00829494)'':0.0440475,('PDT000174379.1':0.0291129,('PDT000662619.1':0.0260283,'PDT000250149.1':0.0176519)'':0.0106227)'':0.016864)'':0.0295071,('PDT000249765.1':0.0839182,(('PDT000687851.1':0.0736619,('PDT000289863.2':0.052827,'PDT000719124.1':0.052201)'':0.0133576)'':0.00662851,('PDT000250166.1':0.070594,(('PDT000509696.1':0.0489026,'PDT000687923.1':0.0520514)'':0.00801196,('PDT000137651.2':0.0393669,('PDT000620272.1':0.0268951,('PDT000104645.2':0.0204456,'PDT000502487.1':0.0280722)'':0.00514248)'':0.00234348)'':0.0068452)'':0.0209677)'':-0.00195949)'':0.00928)'':-0.00550977)'':0.0217148)'':-0.00236149)'':-0.0158307,(('PDT000251927.1':0.0853118,('PDT000252732.1':0.061577,'PDT000484665.1':0.073184)'':0.0129232)'':0.0117828,('PDT000015705.2':0.0484393,((('PDT000339393.1':0.0306631,'PDT000014149.3':0.0279261)'':0.0221723,('PDT000484674.1':0.0293718,'PDT000484709.1':0.0251345)'':0.014113)'':0.00190519,(('PDT000662576.1':0.039831,'PDT000662621.1':0.0354472)'':0.00894909,('PDT000015848.2':0.0171187,('PDT000014113.3':0.0126773,('PDT000014115.3':0.011468,('PDT000015793.3':0.0136899,(('PDT000014109.3':0.00889048,'PDT000014116.3':0.00847752)'':0.000385805,('PDT000015059.3':0.00915816,(('PDT000014114.2':0.0103568,('PDT000015817.3':0.00856326,'PDT000015808.2':0.00920174)'':0.000624914)'':0.000139221,('PDT000015811.3':0.0106273,('PDT000014110.3':0.00991565,('PDT000014112.3':0.00933627,('PDT000014137.3':0.00510497,'PDT000014108.3':0.00881433)'':0.000576127)'':0.000352343)'':6.98986e-05)'':0.000141823)'':0.00050359)'':-5.06323e-05)'':0.00204521)'':0.00106489)'':0.000815982)'':0.00118952)'':0.0329869)'':0.00311354)'':0.0119064)'':0.0290476)'':-0.00443969)'':0.0247965)'':0.000314162)'':0.0158718)'':0.0163745,(((((('PDT000346098.1':0.0930233,('PDT000646048.1':0.0877421,'PDT000252426.1':0.0650239)'':0.00520772)'':0.00590965,((('PDT000049401.1':0.0446922,('PDT000283026.2':0.0603446,'PDT000049400.1':0.0501934)'':0.00238522)'':0.0239412,('PDT000672689.1':0.0600505,('PDT000645155.1':0.0628514,('PDT000539036.1':0.0489044,('PDT000466968.1':0.0284963,'PDT000557137.1':0.0274607)'':0.00881437)'':0.0173339)'':0.00167868)'':0.0150515)'':0.00468893,((('PDT000124627.1':0.0495128,('PDT000718328.1':0.022107,'PDT000718839.1':0.025882)'':0.0147379)'':0.00121476,('PDT000280804.2':0.0324837,(('PDT000047695.3':0.0275649,'PDT000357594.1':0.0249061)'':0.00562244,('PDT000249833.1':0.0296256,'PDT000249834.1':0.0282947)'':0.00664318)'':0.0114081)'':-0.00135232)'':0.00665707,(('PDT000679715.2':0.036451,('PDT000536882.1':0.0389415,'PDT000646047.1':0.0244277)'':0.0103007)'':0.0202558,(('PDT000557063.1':0.0504769,('PDT000679716.2':0.0506526,('PDT000523912.1':0.0436092,'PDT000652080.1':0.0387957)'':0.0078873)'':0.0120738)'':-0.00110084,(('PDT000536887.1':0.0370332,'PDT000539042.1':0.0447859)'':0.0154721,('PDT000252450.1':0.0425509,(('PDT000645151.1':0.019272,'PDT000668367.2':0.0206611)'':0.0058755,('PDT000668340.2':0.0413068,(('PDT000672646.1':0.0200542,'PDT000655779.1':0.0258717)'':0.00063056,('PDT000518120.1':0.0210146,'PDT000645885.1':0.0250652)'':0.00564529)'':0.0119033)'':-0.00220338)'':0.00745819)'':0.000572864)'':0.00186002)'':0.00235254)'':0.00390112)'':0.0149646)'':0.0100785)'':-0.0101067,(((('PDT000645154.1':0.0565518,('PDT000655943.1':0.028499,('PDT000638923.1':0.0156674,'PDT000653437.1':0.0147433)'':0.0146373)'':0.0257413)'':0.000833049,('PDT000655794.1':0.0607282,('PDT000668312.2':0.0338751,(('PDT000518095.1':0.0212823,'PDT000644176.1':0.0218761)'':0.00310168,('PDT000655781.1':0.0270566,('PDT000439476.1':0.0187472,'PDT000644179.1':0.0127794)'':0.00465574)'':0.00367836)'':0.00992408)'':0.0107097)'':-0.00077439)'':0.00917685,((('PDT000718330.1':0.0498615,('PDT000557024.1':0.0298507,'PDT000717610.1':0.0324973)'':0.009172)'':-0.000250655,('PDT000556961.1':0.031294,(('PDT000466991.1':0.011971,'PDT000008773.2':0.0169915)'':0.0142882,(('PDT000556960.1':0.0199338,'PDT000718326.1':0.0157523)'':0.00243241,('PDT000523907.1':0.0159252,('PDT000557159.1':0.0107117,'PDT000228313.1':0.0121823)'':0.00422861)'':0.000835771)'':0.0127417)'':0.00776472)'':-0.000460589)'':0.00349967,(('PDT000637331.1':0.0359282,('PDT000624685.1':0.0418812,('PDT000518079.1':0.0358158,'PDT000714694.1':0.0259356)'':0.0100368)'':0.00253513)'':0.0141256,('PDT000714647.1':0.0548621,('PDT000557266.1':0.0321872,(('PDT000510238.1':0.0162442,'PDT000250172.1':0.0264594)'':0.00719298,('PDT000722590.1':0.0154969,('PDT000722589.1':0.00682366,('PDT000380305.1':0.00788978,('PDT000510075.1':0.00425598,'PDT000722593.1':0.00460503)'':0.00360281)'':0.00170284)'':0.00661861)'':0.00184131)'':0.0106381)'':0.0049754)'':0.000195423)'':0.00206273)'':0.00510026)'':0.0137605,((('PDT000251535.1':0.0482396,'PDT000227989.1':0.0348591)'':0.00993832,(('PDT000466961.1':0.0322869,'PDT000467007.1':0.0351766)'':0.00533517,('PDT000466999.1':0.0238927,'PDT000251716.1':0.0280451)'':0.0151829)'':0.00561632)'':0.0077545,('PDT000283087.2':0.0424534,((('PDT000467002.1':0.0140679,'PDT000557281.1':0.0146336)'':0.00232407,(('PDT000722591.1':0.0120543,'PDT000249975.1':0.0159525)'':0.0107868,('PDT000466985.1':0.0146788,('PDT000691214.1':0.00701935,'PDT000227997.1':0.0109411)'':0.00442618)'':0.00603023)'':0.00270429)'':0.00923387,(('PDT000282977.2':0.069029,('PDT000637584.1':0.0428226,('PDT000557263.1':0.0212224,(('PDT000312620.2':0.0120447,'PDT000719144.1':0.013467)'':0.00154116,('PDT000282887.2':0.00987633,('PDT000283074.2':0.00715954,'PDT000557190.1':0.0113867)'':0.00712262)'':0.00962315)'':0.0053627)'':0.00301304)'':0.014296)'':-0.0125412,(('PDT000638921.1':0.0159339,('PDT000689907.1':0.0110649,'PDT000714696.1':0.0103615)'':0.0032391)'':0.0175335,(('PDT000009355.2':0.0353111,('PDT000539046.1':0.0207432,('PDT000689793.1':0.0165046,'PDT000718824.1':0.0171237)'':0.0025291)'':0.0155835)'':0.00033315,('PDT000009356.2':0.0203184,(('PDT000282988.2':0.0240114,'PDT000009352.2':0.0176223)'':0.00693645,('PDT000009768.2':0.0241135,('PDT000400009.1':0.0283266,'PDT000009354.2':0.0200398)'':0.00445373)'':0.00144071)'':0.00359509)'':-0.000546372)'':0.00428316)'':0.00210634)'':0.0025657)'':0.0136203)'':0.00535384)'':0.0014984)'':0.00179414)'':0.052036,((('PDT000441814.1':0.0386294,'PDT000645515.2':0.0418459)'':0.0667005,(('PDT000276018.1':0.0960566,(('PDT000667862.2':0.054759,'PDT000365978.1':0.064519)'':0.0265568,('PDT000270582.3':0.0630313,('PDT000583481.1':0.0702479,'PDT000124643.1':0.0705031)'':0.0153597)'':0.00216781)'':0.0200212)'':0.0119861,(('PDT000652872.1':0.0756152,'PDT000501846.1':0.0734088)'':0.00727689,(('PDT000388878.1':0.0550149,'PDT000482531.1':0.0401443)'':0.0051415,('PDT000502622.1':0.00868405,('PDT000443380.1':0.00775378,'PDT000553420.1':0.00629002)'':0.0047336)'':0.0441146)'':0.0151995)'':0.0102382)'':0.0152879)'':0.00278021,((('PDT000719290.1':0.0720255,('PDT000439505.1':0.0774377,('PDT000320741.1':0.0657884,('PDT000346804.1':0.0453434,('PDT000482532.1':0.0345953,'PDT000590194.1':0.0468632)'':0.00329735)'':0.0170671)'':0.0044555)'':0.0100345)'':0.0107079,('PDT000228281.1':0.0849996,('PDT000557069.1':0.0515625,(('PDT000433487.1':0.0451003,'PDT000248523.1':0.0503214)'':0.00110066,('PDT000017644.2':0.032694,('PDT000425775.1':0.0148484,('PDT000713447.1':0.0070798,'PDT000661590.1':0.0060558)'':0.0051254)'':0.0284968)'':0.0135117)'':0.00841344)'':0.0214408)'':0.000257202)'':0.0133098,((('PDT000088290.2':0.0725582,('PDT000424628.1':0.0451794,'PDT000502608.1':0.0549556)'':0.0243943)'':0.0157048,('PDT000661824.1':0.0745711,('PDT000482848.1':0.0274654,('PDT000131753.2':0.023949,'PDT000661587.1':0.0223192)'':0.0034161)'':0.0246906)'':0.0333949)'':0.00773215,((('PDT000161880.3':0.0246195,('PDT000140578.4':0.00907049,'PDT000557526.1':0.0120429)'':0.0262853)'':0.0744001,(('PDT000589387.1':0.0872529,'PDT000248620.1':0.0482251)'':0.0225722,('PDT000178602.2':0.0709357,('PDT000400005.1':0.0342366,'PDT000502057.1':0.0439203)'':0.0315823)'':0.00585003)'':0.00921937)'':-0.00161692,((('PDT000520770.1':0.0778587,('PDT000175854.2':0.0479932,'PDT000425967.1':0.0438474)'':0.016611)'':0.00547565,(('PDT000251847.1':0.06586,('PDT000175796.2':0.0423146,'PDT000427695.1':0.0436304)'':0.024871)'':0.00305505,(('PDT000380285.1':0.049089,'PDT000719147.1':0.054554)'':0.0141813,('PDT000718925.1':0.0380057,'PDT000357600.1':0.0299232)'':0.0163355)'':0.00531125)'':0.00666132)'':0.019739,((('PDT000252719.1':0.0833852,('PDT000556954.1':0.055604,('PDT000721344.1':0.0374549,('PDT000228005.1':0.0167013,'PDT000250013.1':0.0130618)'':0.0164169)'':0.00945197)'':0.0314654)'':-0.000203514,(('PDT000557326.1':0.0402508,(('PDT000250237.1':0.0498425,'PDT000251559.1':0.041499)'':0.00708179,('PDT000017741.2':0.0298591,('PDT000128796.2':0.03098,'PDT000714026.1':0.0194018)'':0.0110502)'':0.00707012)'':0.00477789)'':0.0225841,((('PDT000396778.1':0.039624,'PDT000048147.1':0.0370696)'':0.0203329,(('PDT000307409.2':0.0154846,'PDT000396834.1':0.0123427)'':0.0148084,('PDT000354154.1':0.0199151,('PDT000309245.2':0.0143735,'PDT000557086.1':0.01144)'':0.0136232)'':0.000178726)'':0.0323468)'':0.0172597,(('PDT000277718.2':0.077981,('PDT000690065.1':0.0442375,'PDT000661687.1':0.0428441)'':0.0158302)'':0.000457431,('PDT000685907.1':0.0663156,('PDT000661579.1':0.0403819,('PDT000432486.1':0.0158925,'PDT000557153.1':0.0271611)'':0.0200664)'':0.0232167)'':0.0037182)'':0.00910029)'':0.00159863)'':0.00541552)'':0.00124632,((('PDT000132431.3':0.0770647,('PDT000338970.2':0.0576956,('PDT000151015.3':0.0379369,'PDT000354151.1':0.0355226)'':0.0138797)'':0.0204792)'':0.010311,(('PDT000426552.1':0.0454808,('PDT000619088.1':0.0331265,'PDT000212811.1':0.0416995)'':0.00761411)'':0.0324583,(('PDT000645007.1':0.0595376,('PDT000104644.2':0.0370116,'PDT000556870.1':0.0544175)'':0.00631382)'':0.0162574,(('PDT000482366.1':0.0734334,'PDT000319012.1':0.0498976)'':0.0024506,('PDT000252733.1':0.0546955,('PDT000015315.2':0.0368832,('PDT000188130.2':0.0276667,'PDT000621301.1':0.0310991)'':0.0145061)'':0.00607574)'':0.00842342)'':0.00298636)'':0.007724)'':0.00786643)'':-0.00414557,((('PDT000502188.1':0.081428,('PDT000175803.2':0.0377839,'PDT000222310.1':0.0332157)'':0.0372942)'':0.00407829,('PDT000497219.1':0.072174,(('PDT000003863.3':0.0556957,'PDT000713776.1':0.0568103)'':0.00404548,('PDT000539040.1':0.0580419,(('PDT000274853.2':0.0146471,'PDT000399686.1':0.0212091)'':0.02632,('PDT000003864.2':0.0219921,('PDT000619647.1':0.00934986,'PDT000017894.2':0.00822184)'':0.0126153)'':0.013805)'':0.0147631)'':0.00974109)'':0.00715029)'':0.0036552)'':0.00313072,((('PDT000131746.2':0.0570366,('PDT000088268.2':0.0295291,'PDT000426764.1':0.0227765)'':0.0209536)'':0.00418439,('PDT000228197.1':0.0559251,('PDT000396777.1':0.0429305,'PDT000619674.1':0.0409162)'':0.0116075)'':0.000876857)'':0.00550307,('PDT000520836.1':0.0506398,(('PDT000660665.1':0.0442046,'PDT000713280.1':0.0469828)'':0.00198375,(('PDT000305441.2':0.0225926,'PDT000518081.1':0.0210007)'':0.0242641,('PDT000557278.1':0.0284249,('PDT000619561.1':0.024446,'PDT000707605.1':0.0223405)'':0.00753562)'':0.0097163)'':0.00486313)'':0.00939004)'':-0.000578128)'':0.0121569)'':0.00914328)'':0.00426071)'':0.00360426)'':0.00904884)'':0.00200642)'':-0.00496188)'':0.0103571)'':0.0063458)'':0.00650391,((('PDT000171796.1':0.116172,('PDT000426830.1':0.0783764,('PDT000188098.2':0.0648067,'PDT000502690.1':0.0571263)'':0.00589506)'':0.0415103)'':0.0163974,('PDT000309309.2':0.0809725,('PDT000354263.1':0.0869044,(('PDT000352664.2':0.0393853,'PDT000140567.4':0.0365825)'':0.0365765,('PDT000063282.3':0.0688032,('PDT000501443.2':0.0371087,'PDT000125200.3':0.0299348)'':0.0291301)'':0.0100114)'':0.0109241)'':0.00427405)'':0.0491737)'':0.00828008,((((('PDT000397778.1':0.052247,('PDT000397996.1':0.03196,'PDT000003804.3':0.0415493)'':0.0106605)'':0.0430336,(('PDT000652848.1':0.0385831,'PDT000652362.1':0.0349991)'':0.0450345,(('PDT000426563.1':0.036862,(('PDT000661626.1':0.0209517,'PDT000402926.1':0.0215806)'':0.00617798,(('PDT000017825.2':0.0223637,'PDT000402925.1':0.0246859)'':0.00323452,('PDT000362461.1':0.0223623,('PDT000425771.1':0.0129108,'PDT000713603.1':0.0135897)'':0.00382698)'':0.00397024)'':0.0029529)'':0.0099079)'':0.0326312,(('PDT000326069.1':0.0233011,'PDT000718975.1':0.0217668)'':0.0491213,(('PDT000204896.2':0.0495888,'PDT000395021.1':0.0721442)'':0.0127057,(('PDT000557220.1':0.0527238,'PDT000539058.1':0.0648002)'':0.00380998,('PDT000015314.2':0.04118,('PDT000497890.1':0.0319457,('PDT000281776.2':0.0115141,'PDT000719082.1':0.00957921)'':0.0116882)'':0.0154737)'':0.00472185)'':0.00659691)'':0.0110438)'':0.00323395)'':0.00575768)'':0.00204274)'':0.0151395,(('PDT000188100.3':0.0829088,('PDT000627420.1':0.0683717,('PDT000389630.1':0.0693469,('PDT000497366.1':0.0318846,'PDT000548943.1':0.0328315)'':0.034673)'':0.00232466)'':0.0172883)'':-0.00666281,((('PDT000319314.1':0.0570067,'PDT000510181.1':0.0639893)'':0.00269046,('PDT000428308.1':0.0474833,'PDT000127568.1':0.0718777)'':0.0133448)'':0.00400878,('PDT000357599.1':0.0804524,((('PDT000128879.2':0.0343054,('PDT000202671.2':0.0291204,'PDT000520796.1':0.0294276)'':0.00333262)'':0.0112724,(('PDT000520782.1':0.0425135,'PDT000571154.1':0.0416768)'':0.0107066,('PDT000296215.2':0.0441025,('PDT000583379.1':0.0298004,'PDT000298024.1':0.0310216)'':0.0111785)'':0.00272993)'':0.00273912)'':0.00365806,('PDT000304337.1':0.0836484,((('PDT000427015.1':0.060062,('PDT000719136.1':0.0359886,('PDT000482215.1':0.0254002,'PDT000257780.1':0.0268561)'':0.00872641)'':0.00992455)'':-0.00158338,(('PDT000673468.1':0.0255064,('PDT000325043.1':0.020433,'PDT000707336.1':0.0175629)'':0.00805562)'':0.0133739,('PDT000289886.1':0.0406594,('PDT000586228.1':0.0365808,('PDT000708215.1':0.0271291,'PDT000171800.1':0.0358949)'':0.0033703)'':0.00376605)'':0.00323447)'':0.00477975)'':-0.000201584,('PDT000668918.1':0.0604065,('PDT000412139.2':0.0536447,('PDT000444912.1':0.0448148,('PDT000718296.1':0.0677759,('PDT000287624.1':0.0355287,('PDT000155842.2':0.0376766,('PDT000136112.2':0.0448936,(('PDT000148406.1':0.0261139,'PDT000422529.1':0.0348043)'':0.00357724,('PDT000221498.1':0.01809,('PDT000352492.1':0.0156281,('PDT000226146.1':0.0140335,('PDT000426184.1':0.0103948,'PDT000427729.1':0.00839625)'':0.00263053)'':0.00237795)'':0.00353094)'':0.00369817)'':0.00596524)'':0.00321558)'':0.00238818)'':0.00368103)'':0.00221035)'':0.00289725)'':0.0051273)'':-0.0058891)'':0.0159238)'':-0.0134798)'':0.0288235)'':-0.00428051)'':0.00423696)'':0.01376)'':0.00876336,(((('PDT000661636.1':0.0619217,('PDT000661403.1':0.0525378,('PDT000426771.1':0.0403337,'PDT000713924.1':0.0388494)'':0.0150331)'':0.00579859)'':0.0340351,(('PDT000425472.1':0.0848151,'PDT000425940.1':0.0696529)'':0.0214885,((('PDT000425658.1':0.0174114,'PDT000352518.1':0.0179431)'':0.0653761,('PDT000510079.1':0.033023,'PDT000557027.1':0.0290358)'':0.0531358)'':0.00701366,('PDT000619571.1':0.0110272,('PDT000484670.1':0.00889312,('PDT000484680.1':0.00219509,'PDT000484682.1':0.00199695)'':0.00865271)'':0.00564265)'':0.0682577)'':0.00857055)'':0.000620716)'':0.00546972,((('PDT000719089.1':0.0615276,('PDT000412042.1':0.0384352,'PDT000488247.1':0.0399054)'':0.0187586)'':0.0231383,('PDT000362453.1':0.0823527,('PDT000583475.1':0.0502405,'PDT000318969.1':0.0334868)'':0.0296741)'':0.0103453)'':0.00909119,('PDT000644995.1':0.103165,((('PDT000131758.2':0.0503129,'PDT000556893.1':0.0331845)'':0.0182693,(('PDT000362839.1':0.00933074,'PDT000375000.1':0.0118612)'':0.0416773,('PDT000300912.2':0.0380361,('PDT000286938.2':0.00414,'PDT000654614.1':0.00413682)'':0.0386006)'':0.00730225)'':0.00947952)'':0.0102922,((('PDT000354934.1':0.0433458,'PDT000439494.1':0.0363425)'':0.00857315,('PDT000713751.1':0.0549145,('PDT000397962.1':0.0526207,('PDT000248841.1':0.0373357,'PDT000357613.1':0.0271335)'':0.00882218)'':0.0104342)'':0.00530838)'':0.00180288,(('PDT000181815.2':0.0729745,(('PDT000131781.2':0.0321501,'PDT000592260.1':0.0365136)'':0.00493749,('PDT000128868.2':0.0371315,('PDT000321558.1':0.00868008,'PDT000652933.1':0.0118419)'':0.0151699)'':0.0111354)'':0.0216383)'':-0.00238204,('PDT000248844.1':0.091046,('PDT000131788.2':0.0643548,(('PDT000131770.2':0.0416904,'PDT000136020.2':0.0400108)'':0.00517456,('PDT000325578.1':0.0219168,('PDT000306639.2':0.0247556,('PDT000131750.2':0.0117763,'PDT000652958.1':0.0102811)'':0.0121804)'':0.00260946)'':0.0224669)'':0.000500179)'':0.0160452)'':-0.00629283)'':0.00454966)'':0.00471458)'':0.0246376)'':-0.000869133)'':0.0024822)'':0.00896721,((('PDT000034436.1':0.0321505,('PDT000650030.1':0.0246408,'PDT000702838.1':0.0344139)'':0.0159331)'':0.0570102,('PDT000556926.1':0.0237529,('PDT000719139.1':0.0180249,('PDT000396804.1':0.0116893,('PDT000702624.1':0.00881373,'PDT000228019.1':0.00609977)'':0.00235335)'':0.0122975)'':0.00627107)'':0.0565788)'':0.00958375,(('PDT000132366.2':0.100369,'PDT000221057.1':0.123046)'':0.00198829,('PDT000124623.1':0.107267,((('PDT000362845.1':0.0246289,'PDT000017690.2':0.0273421)'':0.0618867,('PDT000277736.2':0.0600016,(('PDT000477113.1':0.0419635,('PDT000389642.1':0.0521152,('PDT000423861.1':0.0415968,('PDT000202578.2':0.0309702,('PDT000607558.1':0.0229095,(('PDT000425576.1':0.0111851,'PDT000426214.1':0.0129423)'':0.0142392,('PDT000673415.1':0.0206029,('PDT000102782.2':0.0152344,('PDT000417499.1':0.0112557,'PDT000501510.1':0.0153686)'':0.00429626)'':0.003526)'':0.000181902)'':0.00607766)'':0.00214063)'':0.0021673)'':0.00791973)'':0.00255561)'':-0.00377227,('PDT000303269.2':0.050096,(('PDT000518545.1':0.0283723,('PDT000303138.2':0.0235913,('PDT000323536.1':0.0189391,'PDT000264648.2':0.022298)'':0.00493185)'':0.00522799)'':0.000832034,('PDT000563569.1':0.0388093,('PDT000393097.1':0.0303165,('PDT000341224.1':0.0196597,('PDT000042731.1':0.0177562,('PDT000230559.2':0.00426852,'PDT000645061.1':0.00651208)'':0.00761802)'':0.00545157)'':0.00431768)'':0.00579654)'':-0.00394069)'':0.00493106)'':0.00271879)'':0.0135532)'':0.0292747)'':-0.0209725,(('PDT000559252.1':0.0641797,'PDT000584750.1':0.103468)'':0.0168485,((('PDT000717027.1':0.0179437,'PDT000637082.1':0.0190055)'':0.0192333,('PDT000517997.1':0.0358522,('PDT000661730.1':0.0222277,('PDT000607578.1':0.0196559,('PDT000713255.1':0.0212658,('PDT000636696.1':0.00900839,'PDT000713402.1':0.00894261)'':0.00354545)'':0.00214439)'':0.000826537)'':0.0109873)'':7.53574e-05)'':0.01973,(('PDT000426678.1':0.0405812,'PDT000557223.1':0.0421913)'':0.0204527,('PDT000425785.1':0.0484469,(('PDT000618834.1':0.0419482,(('PDT000300999.2':0.0304908,'PDT000557334.1':0.0414516)'':0.00290633,('PDT000176037.2':0.0327677,('PDT000346134.1':0.0328247,('PDT000294548.2':0.0190167,'PDT000661495.1':0.0230167)'':0.00387573)'':0.00430678)'':0.000262924)'':0.0133741)'':-0.00551163,('PDT000250050.1':0.0627038,(('PDT000706203.1':0.0384722,('PDT000497409.1':0.0295126,'PDT000594471.1':0.0294718)'':0.00844588)'':0.00175416,('PDT000281481.2':0.0383191,('PDT000636339.1':0.0271493,('PDT000425989.1':0.0246038,('PDT000425936.1':0.0211434,'PDT000426012.1':0.0174735)'':0.0052082)'':0.00559199)'':0.00282373)'':0.011823)'':0.00410753)'':-0.004784)'':0.0126258)'':-0.00106174)'':0.0041468)'':0.0174304)'':-0.0104453)'':0.0330327)'':0.00577595)'':-0.0109626)'':0.0112177)'':0.000653525)'':0.00896967,((('PDT000524133.2':0.0568492,'PDT000115475.3':0.0408492)'':0.0615631,(('PDT000309594.3':0.0334887,('PDT000367750.2':0.019012,'PDT000667864.2':0.0159441)'':0.0106545)'':0.0037985,(('PDT000147748.3':0.0378041,'PDT000485318.2':0.0340266)'':0.00367211,(('PDT000377102.2':0.0289334,'PDT000443398.2':0.026214)'':0.00806897,(('PDT000318567.2':0.0210296,'PDT000332745.2':0.0224572)'':0.0200731,(('PDT000314832.3':0.0175687,'PDT000414585.2':0.0215358)'':0.0143442,(('PDT000500638.2':0.019063,('PDT000713224.1':0.00619338,'PDT000233104.1':0.00475732)'':0.00946781)'':0.00483372,((('PDT000615198.2':0.0267248,('PDT000288439.3':0.0197967,('PDT000324052.2':0.0186087,'PDT000389676.2':0.0159726)'':0.00327723)'':0.00518149)'':0.0013476,('PDT000711423.1':0.0188963,('PDT000147587.3':0.0141859,('PDT000322090.2':0.0114273,('PDT000667880.2':0.0126591,('PDT000677895.1':0.00884206,'PDT000132019.3':0.0111182)'':0.00305225)'':0.00279299)'':0.00365566)'':0.00439838)'':0.00303717)'':0.0016011,(((('PDT000332824.2':0.0125359,'PDT000332859.2':0.0123924)'':0.0036247,('PDT000549588.2':0.0111857,'PDT000658738.2':0.0119551)'':0.000696501)'':0.0126127,((('PDT000318683.2':0.0166133,'PDT000577001.2':0.0135429)'':0.00987217,('PDT000428763.2':0.0133936,'PDT000532067.2':0.0122703)'':0.00962206)'':0.00128231,(('PDT000624255.2':0.0191631,('PDT000583349.2':0.0122304,'PDT000716777.1':0.0138643)'':0.00774972)'':-3.31291e-05,('PDT000599123.2':0.0154605,('PDT000148160.3':0.00792863,'PDT000161682.3':0.0112391)'':0.00700812)'':0.00415727)'':0.00145795)'':0.0011744)'':0.000604716,((('PDT000314207.3':0.0179755,'PDT000624233.2':0.0157923)'':0.0044206,(('PDT000177082.3':0.0162179,'PDT000343965.2':0.0130977)'':0.00204602,('PDT000475747.2':0.011795,'PDT000517227.2':0.00914168)'':0.00560881)'':0.00404992)'':0.00110409,(('PDT000603601.2':0.021379,'PDT000137530.3':0.0151774)'':0.00347856,('PDT000638590.2':0.0205119,('PDT000718632.1':0.0298595,('PDT000377081.2':0.0218656,(('PDT000148263.3':0.0169893,'PDT000161652.3':0.0174697)'':0.00683048,('PDT000317194.2':0.0183206,(('PDT000314833.3':0.0139537,('PDT000464162.2':0.0102639,('PDT000314215.3':0.00954441,'PDT000603023.2':0.0118189)'':0.00216373)'':0.00188899)'':0.00109826,('PDT000494261.2':0.0182765,('PDT000147679.3':0.0120992,('PDT000161695.3':0.0103417,('PDT000351787.2':0.00461707,'PDT000472549.2':0.00386173)'':0.00330493)'':0.00141532)'':0.00397768)'':-0.00122481)'':0.00126154)'':0.00244466)'':0.00172365)'':0.00172895)'':0.00140569)'':0.000476299)'':-0.00234788)'':0.00275388)'':0.00232583)'':0.00275092)'':0.00538895)'':0.00153858)'':0.00068608)'':0.00392071)'':-0.00048813)'':0.083372)'':0.00350395,((('PDT000484711.1':0.027294,('PDT000484679.1':0.00626673,'PDT000484705.1':0.00665447)'':0.0209622)'':0.0880696,(('PDT000537855.1':0.0843291,('PDT000213265.2':0.0609571,'PDT000425750.1':0.0648529)'':0.0166114)'':0.0200722,(('PDT000694363.1':0.0674354,('PDT000636260.1':0.0453474,'PDT000520905.1':0.0430461)'':0.0177548)'':0.0238166,('PDT000251550.1':0.0733034,('PDT000250762.2':0.0479591,('PDT000499659.1':0.0487362,'PDT000526150.1':0.0577018)'':0.00934285)'':0.0134217)'':0.00527174)'':0.0201404)'':0.005827)'':-0.00286767,(((('PDT000556955.1':0.0311456,'PDT000251475.1':0.0457168)'':0.0479592,('PDT000309328.2':0.0647943,('PDT000129563.1':0.049304,('PDT000251749.1':0.0348547,'PDT000017717.2':0.0337556)'':0.00942643)'':0.0219574)'':0.0137247)'':0.00975096,(('PDT000352465.1':0.0953364,('PDT000196592.2':0.0370092,'PDT000356784.1':0.0396623)'':0.0416233)'':0.000127113,(('PDT000496900.1':0.0530517,('PDT000500494.1':0.0390225,('PDT000288614.2':0.0246579,'PDT000557287.1':0.0264584)'':0.00837099)'':0.0150134)'':0.00916783,('PDT000380300.1':0.0767443,(('PDT000306428.2':0.0421668,'PDT000557089.1':0.0300163)'':0.0124482,('PDT000275525.2':0.0395842,('PDT000286932.2':0.0325775,('PDT000310455.2':0.0279716,('PDT000496888.1':0.0182701,('PDT000246514.2':0.00524855,'PDT000479090.1':0.00609695)'':0.0103382)'':0.00679427)'':0.00440287)'':0.00199893)'':0.0102408)'':0.0249268)'':0.00168176)'':0.0169449)'':0.00898362)'':0.00696623,(((('PDT000181680.2':0.0783912,'PDT000417182.1':0.0851758)'':0.0228164,('PDT000131752.2':0.0729158,('PDT000249871.1':0.0486869,'PDT000252430.1':0.0564411)'':0.00418566)'':0.0251742)'':0.000174207,(('PDT000664965.1':0.0558634,('PDT000357567.1':0.0440573,'PDT000357655.1':0.0367067)'':0.00785112)'':0.0481291,((('PDT000399598.1':0.0880939,('PDT000397550.1':0.0696983,'PDT000616143.1':0.0626837)'':0.0148701)'':0.00680462,('PDT000396755.1':0.0836186,('PDT000557244.1':0.039651,'PDT000251526.1':0.0429222)'':0.0204378)'':0.00694976)'':0.00607908,(('PDT000354879.1':0.0971201,'PDT000140499.3':0.0847569)'':0.0135162,((('PDT000501884.1':0.0636375,('PDT000389664.1':0.0103521,('PDT000088275.2':0.0044886,'PDT000482843.1':0.00319526)'':0.00714604)'':0.0708022)'':-0.0030383,(('PDT000498985.1':0.0422429,'PDT000652970.1':0.0576991)'':0.0209867,(('PDT000653004.1':0.0600213,'PDT000716875.1':0.0521287)'':0.00697636,('PDT000175797.2':0.0542319,('PDT000535145.1':0.0297421,('PDT000004353.2':0.0126727,'PDT000639580.1':0.00988294)'':0.0142491)'':0.0188085)'':0.00812341)'':0.00662354)'':0.00642299)'':0.000572268,(('PDT000275529.2':0.0538774,'PDT000645087.1':0.0534316)'':0.0135096,(('PDT000496901.1':0.0605187,(('PDT000482839.1':0.0201329,'PDT000482865.1':0.0185895)'':0.0382832,('PDT000427511.1':0.0500796,('PDT000258747.2':0.0399636,'PDT000128789.2':0.0327018)'':0.0110176)'':0.00038361)'':0.0152031)'':0.000566428,('PDT000251502.1':0.0447626,(('PDT000645046.1':0.0539065,('PDT000145177.2':0.0304463,'PDT000397541.1':0.0395283)'':0.020233)'':0.0110158,('PDT000494440.1':0.0442579,(('PDT000131777.2':0.0382055,'PDT000131769.2':0.0476948)'':0.0037308,('PDT000393087.1':0.0214454,'PDT000248780.1':0.0210869)'':0.0153831)'':0.00424292)'':0.0104834)'':0.00247389)'':0.0055911)'':0.000912035)'':0.00981768)'':0.0175362)'':-0.0120141)'':0.01359)'':-0.0022772)'':-0.00220218,(('PDT000131764.2':0.0971626,('PDT000432500.1':0.0607135,((('PDT000425978.1':0.0375766,'PDT000652905.1':0.0362942)'':0.0152825,(('PDT000426616.1':0.0205354,'PDT000427477.1':0.01913)'':0.0228353,('PDT000395101.1':0.0412292,(('PDT000201856.2':0.00608554,'PDT000027900.2':0.00546136)'':0.0229946,('PDT000328672.1':0.0256431,'PDT000645536.2':0.0252341)'':0.00883792)'':0.0067002)'':0.00111269)'':0.0130242)'':-0.00418873,(('PDT000445096.1':0.037992,'PDT000092877.1':0.0364082)'':0.0112608,('PDT000535413.1':0.0651547,('PDT000362860.1':0.0570166,(('PDT000397821.1':0.0302695,('PDT000399679.1':0.0219438,'PDT000716783.1':0.0216751)'':0.00899286)'':0.00654581,(('PDT000645554.2':0.0350476,('PDT000587991.1':0.0227685,('PDT000060680.2':0.0252009,'PDT000397702.1':0.0193118)'':0.00754597)'':0.00417046)'':-0.000910342,(('PDT000397960.1':0.0254123,'PDT000713593.1':0.0306836)'':0.00616467,('PDT000030572.1':0.0295703,'PDT000251865.1':0.024452)'':0.00546098)'':0.00399551)'':0.00336843)'':0.00787649)'':0.00311285)'':0.00380632)'':-0.00411784)'':0.0171051)'':0.0142499)'':-0.0117628,((('PDT000425946.1':0.0100943,('PDT000425958.1':0.0024257,'PDT000426189.1':0.00396344)'':0.0103782)'':0.0576427,('PDT000521086.1':0.0593851,('PDT000520894.1':0.0388735,('PDT000251880.1':0.0302508,'PDT000395214.1':0.0240271)'':0.0168496)'':0.0114708)'':0.00349222)'':0.0113314,((('PDT000439472.1':0.0150152,'PDT000439478.1':0.0123888)'':0.0387163,('PDT000714125.1':0.0498206,(('PDT000380273.1':0.0308329,'PDT000557199.1':0.0362089)'':0.008489,('PDT000719137.1':0.0449016,('PDT000193071.2':0.025903,'PDT000251949.1':0.0263642)'':0.00724086)'':0.00944411)'':0.00817433)'':-6.42255e-05)'':0.00816055,((('PDT000557265.1':0.0717161,(('PDT000557277.1':0.0126748,'PDT000228051.1':0.0138905)'':0.0134,('PDT000557302.1':0.0236735,'PDT000249764.1':0.0271439)'':0.00200078)'':0.0302318)'':0.00548568,(('PDT000168467.1':0.0512378,('PDT000006415.2':0.00863693,'PDT000008490.2':0.00713307)'':0.0462902)'':0.0104985,(('PDT000202032.2':0.0416025,'PDT000713473.1':0.0485913)'':0.00938994,(('PDT000274861.2':0.0279788,'PDT000017782.2':0.0244988)'':0.0184612,(('PDT000716822.1':0.0355651,('PDT000661606.1':0.0162017,'PDT000661641.1':0.0129332)'':0.0169676)'':0.0160699,('PDT000521013.1':0.0407639,(('PDT000175989.2':0.0319281,'PDT000128798.2':0.0290819)'':0.0147501,('PDT000661311.1':0.0270725,('PDT000445019.1':0.0286015,('PDT000427220.1':0.0128062,'PDT000675069.1':0.0258579)'':0.011693)'':0.00427214)'':0.00554425)'':0.00310646)'':-0.000382243)'':0.004616)'':0.00259665)'':0.011025)'':0.0055737)'':0.00457712,('PDT000185859.1':0.0766797,((('PDT000240695.2':0.0190335,'PDT000713689.1':0.0270654)'':0.0269876,(('PDT000542501.1':0.0382593,'PDT000395165.1':0.029718)'':0.00861634,('PDT000713957.1':0.0465568,('PDT000397938.1':0.040259,('PDT000399718.1':0.0302665,('PDT000082618.1':0.0277899,('PDT000394843.1':0.0170261,'PDT000397704.1':0.0235585)'':0.00374881)'':0.00583593)'':0.0101667)'':0.00584173)'':-0.00229508)'':0.00735898)'':-0.00200419,('PDT000588046.1':0.0808399,('PDT000017658.2':0.0469524,(('PDT000395307.1':0.0395608,('PDT000397548.1':0.0231165,('PDT000132053.2':0.0192008,'PDT000689883.1':0.0202328)'':0.00369842)'':0.0137191)'':0.00162836,('PDT000131756.2':0.0268984,(('PDT000161517.2':0.0138134,'PDT000713497.1':0.00882552)'':0.0130008,('PDT000594461.1':0.0192196,('PDT000471036.1':0.0282465,(('PDT000367274.1':0.0127114,'PDT000395316.1':0.0146358)'':0.00742185,('PDT000201855.2':0.016084,('PDT000240700.2':0.00553108,'PDT000259583.2':0.00474532)'':0.0062172)'':0.000209196)'':0.00300082)'':0.00103649)'':0.0054023)'':0.00102821)'':0.0136422)'':0.00407773)'':0.0108442)'':-0.00703816)'':0.0164937)'':-0.00773507)'':0.00254486)'':0.00348755)'':0.00145937)'':0.0192873)'':0.0084502)'':0.0128323)'':0.000360846)'':0.00568429)'':0.0285503)'':-0.0287508)'':0.0166966,((((('PDT000354862.1':0.0418446,'PDT000672662.1':0.0498446)'':0.0878551,(('PDT000433451.1':0.0696147,('PDT000227136.2':0.0705137,'PDT000714667.1':0.0652933)'':0.0114083)'':0.00909749,('PDT000645156.1':0.061139,('PDT000565882.1':0.0478977,'PDT000395172.1':0.0286521)'':0.0136322)'':0.0216564)'':0.0133298)'':-0.00319225,(('PDT000227105.2':0.0535387,'PDT000497412.1':0.0500743)'':0.0113955,(('PDT000394999.1':0.0596421,'PDT000397761.1':0.0572839)'':0.00121467,(('PDT000216016.1':0.0159482,'PDT000249767.1':0.018099)'':0.00957197,('PDT000425921.1':0.0304773,'PDT000248470.1':0.0330796)'':0.00160463)'':0.0168469)'':0.00908119)'':0.0227765)'':0.00592584,((('PDT000645511.2':0.0982778,('PDT000538096.1':0.0421687,('PDT000675600.1':0.0284941,'PDT000297992.1':0.0305941)'':0.00436666)'':0.0344262)'':3.25419e-06,('PDT000383092.1':0.0727953,(('PDT000014125.3':0.038063,'PDT000015699.3':0.0352112)'':0.00743339,('PDT000015722.3':0.0167305,('PDT000014144.3':0.0114111,'PDT000015728.2':0.0151499)'':0.00278231)'':0.0338152)'':0.0346439)'':0.0179896)'':0.0107542,(((('PDT000277148.2':0.0630288,'PDT000225243.1':0.0568612)'':0.018022,((('PDT000427234.1':0.0559912,'PDT000402873.1':0.0498038)'':0.0147954,('PDT000427678.1':0.0504444,'PDT000716823.1':0.0446385)'':0.0154552)'':0.000947294,('PDT000712790.1':0.0697013,('PDT000535137.1':0.0527315,('PDT000181710.2':0.0518419,(('PDT000181703.2':0.0451872,'PDT000535292.1':0.0432677)'':0.0080634,('PDT000181806.2':0.0351161,'PDT000535127.1':0.0303988)'':0.0155812)'':0.00418449)'':0.00279889)'':0.00269176)'':4.86145e-05)'':0.0100936)'':-0.00394286,(('PDT000672216.1':0.0412945,'PDT000240291.1':0.0420069)'':0.0544874,((('PDT000713265.1':0.0474576,'PDT000441345.1':0.0376982)'':0.0108423,('PDT000604277.1':0.0552284,('PDT000616082.1':0.0401806,'PDT000046098.1':0.0471653)'':0.011633)'':0.00429502)'':0.00690896,(('PDT000537840.1':0.0795534,(('PDT000346887.1':0.059876,('PDT000535102.1':0.0454763,'PDT000388467.1':0.0465647)'':0.00620045)'':0.0161866,(('PDT000417104.1':0.0656716,'PDT000496894.1':0.0572774)'':0.00627896,('PDT000535002.1':0.0584543,'PDT000433490.1':0.0616967)'':0.00289204)'':0.00235145)'':0.0074791)'':-0.000811761,((('PDT000632577.1':0.0453438,'PDT000168872.1':0.0462797)'':0.0230257,('PDT000168847.1':0.0839148,('PDT000423867.1':0.0556985,'PDT000171442.1':0.0546645)'':0.0084232)'':0.000987114)'':-0.000193512,(('PDT000414181.1':0.072361,('PDT000661170.1':0.0440849,('PDT000417152.1':0.0417054,'PDT000124654.1':0.0387378)'':0.0031373)'':0.00542012)'':0.00911656,(('PDT000202159.2':0.0463151,'PDT000534983.1':0.0455167)'':0.00825546,('PDT000534952.1':0.0500756,'PDT000289880.1':0.0586944)'':0.00669381)'':0.00543171)'':0.001811)'':0.00381343)'':0.00210583)'':0.0137522)'':-0.0118287)'':0.00312333,((('PDT000402920.1':0.0817702,(('PDT000526190.1':0.0533746,'PDT000226148.1':0.0277981)'':0.0171215,(('PDT000618721.1':0.0208324,'PDT000632812.1':0.0222998)'':0.0182238,('PDT000431892.1':0.0412253,(('PDT000535311.1':0.0312325,'PDT000702365.1':0.0237818)'':0.00479726,('PDT000411334.1':0.0191553,('PDT000571195.2':0.013464,'PDT000691323.1':0.0185765)'':0.00257734)'':0.011121)'':0.0085223)'':0.00487401)'':0.00340124)'':0.0227167)'':-0.0163501,(((('PDT000433792.1':0.0314184,'PDT000340014.1':0.0393104)'':0.00466045,('PDT000411320.1':0.0403091,('PDT000102997.1':0.0235489,'PDT000452801.1':0.0255399)'':0.00674052)'':0.000419012)'':0.00343242,('PDT000426190.1':0.0500157,(('PDT000136074.2':0.0404008,'PDT000251381.1':0.034203)'':0.0131759,('PDT000202188.2':0.0382603,('PDT000615827.1':0.032864,('PDT000202711.2':0.012099,('PDT000202198.2':0.00965475,('PDT000202587.2':0.0047865,'PDT000129557.1':0.00360176)'':0.00598087)'':0.0062325)'':0.0176395)'':0.00261608)'':0.00179544)'':0.00622369)'':-0.00176751)'':0.00306142,(('PDT000202716.2':0.056627,((('PDT000202027.2':0.0315981,'PDT000202681.2':0.0350491)'':0.00174616,('PDT000063645.1':0.0305346,('PDT000520850.1':0.0184873,('PDT000175939.2':0.0132201,'PDT000176221.2':0.012683)'':0.00825855)'':0.00516109)'':0.00226889)'':0.00146572,('PDT000189438.1':0.0355327,('PDT000437904.1':0.0162197,('PDT000340395.1':0.0103385,('PDT000426308.1':0.00727615,('PDT000437990.1':0.00601828,'PDT000615998.1':0.00591612)'':0.0045095)'':0.00128208)'':0.00285182)'':0.0118688)'':0.000370084)'':0.0121389)'':0.00342779,((('PDT000136078.2':0.0350303,'PDT000425934.1':0.0292771)'':0.00734832,('PDT000368895.1':0.0245485,'PDT000471410.1':0.0204185)'':0.00824918)'':0.0177689,(('PDT000356853.1':0.0433254,('PDT000632226.1':0.026905,'PDT000542613.1':0.0210808)'':0.00517726)'':0.00820899,(('PDT000423992.1':0.0442278,'PDT000646844.1':0.0544494)'':0.00570668,((('PDT000253680.1':0.0259474,'PDT000215973.2':0.0204331)'':0.0103837,(('PDT000589799.1':0.030034,('PDT000445954.1':0.0345162,'PDT000589864.1':0.0253037)'':0.00563997)'':0.000661835,('PDT000402872.1':0.0242845,('PDT000471408.1':0.0258939,('PDT000222826.2':0.0180585,('PDT000669009.1':0.0156221,'PDT000448186.1':0.0168582)'':0.00410024)'':0.00229123)'':0.00321773)'':0.00514756)'':0.00296277)'':0.000732067,('PDT000633405.1':0.0338324,(('PDT000226152.1':0.0262186,('PDT000144557.3':0.0211964,'PDT000675514.1':0.0233965)'':0.00211169)'':0.00748849,('PDT000619077.1':0.0251418,('PDT000675068.1':0.0248083,((('PDT000673590.1':0.00924908,'PDT000720301.1':0.0119555)'':0.00680925,('PDT000674829.1':0.0167536,('PDT000310771.1':0.013512,'PDT000428840.1':0.0125508)'':0.00114075)'':0.00471062)'':0.00189239,('PDT000212804.1':0.0278102,('PDT000675156.1':0.0262374,('PDT000433247.1':0.0196487,('PDT000426525.1':0.0110472,'PDT000484653.1':0.0110451)'':0.0050525)'':0.00315316)'':0.00245846)'':-0.000588225)'':0.00194947)'':0.00287206)'':-0.00250697)'':0.00639918)'':-0.0018203)'':0.0070768)'':0.00556755)'':0.00354064)'':-0.00109912)'':0.00412002)'':0.00968268)'':0.0167952,(((('PDT000360974.1':0.0596126,('PDT000402927.1':0.0525219,('PDT000615971.1':0.0483154,('PDT000637080.1':0.0277845,'PDT000714559.1':0.0297148)'':0.0061142)'':0.0114575)'':0.00516)'':0.0102657,(('PDT000426128.1':0.0605939,'PDT000590051.1':0.0630021)'':0.00420979,('PDT000645509.2':0.0634863,('PDT000310774.1':0.0284533,'PDT000019257.2':0.0290831)'':0.0226075)'':0.00447681)'':0.0007443)'':0.000936828,(('PDT000097048.1':0.0867875,((('PDT000063647.1':0.0542009,('PDT000175830.2':0.0351117,'PDT000651991.1':0.0301714)'':0.00987881)'':0.00252992,('PDT000522793.1':0.0516935,('PDT000661426.1':0.0245601,('PDT000124856.2':0.0208382,'PDT000443902.1':0.0188982)'':0.00188757)'':0.0219166)'':0.00976106)'':0.00756324,(('PDT000633425.1':0.0289235,'PDT000396735.1':0.0332416)'':0.0206646,(('PDT000433467.1':0.0255387,('PDT000433762.1':0.0190798,'PDT000083424.1':0.0198544)'':0.00750035)'':0.00818172,('PDT000143954.3':0.0438659,('PDT000168465.1':0.0467079,('PDT000202165.2':0.0256786,('PDT000426743.1':0.00335648,'PDT000427565.1':0.00462676)'':0.0134464)'':0.0107621)'':0.00992548)'':-0.000173236)'':0.00870329)'':0.00334341)'':0.0127146)'':-0.00780625,('PDT000364531.1':0.0763685,(('PDT000675666.1':0.0307954,('PDT000291270.1':0.0233556,'PDT000499083.1':0.0318297)'':0.0114297)'':0.0109479,(('PDT000346889.1':0.0354188,'PDT000270533.3':0.0444176)'':0.0235103,(('PDT000426885.1':0.0290506,('PDT000402910.1':0.0333909,('PDT000402887.1':0.0160191,('PDT000675239.1':0.00980825,('PDT000402921.1':0.0106764,(('PDT000375421.1':0.000778235,'PDT000375426.1':0.000820645)'':2.03308e-05,('PDT000375423.1':0.000648144,'PDT000375425.1':0.000951216)'':0.000378864)'':0.0107584)'':0.00174704)'':0.000544573)'':0.0126402)'':0.00816198)'':-0.00737419,('PDT000495422.1':0.023453,('PDT000431880.1':0.0197479,('PDT000632925.1':0.0231714,('PDT000297994.1':0.00877511,('PDT000427612.1':0.00778663,(('PDT000675438.1':0.00858263,('PDT000431891.1':0.00456615,'PDT000691325.1':0.00550065)'':0.00320487)'':-3.68315e-06,('PDT000215977.2':0.00228047,('PDT000375420.1':0.000463028,'PDT000375424.1':0.000436612)'':0.00221053)'':0.00552811)'':0.0030855)'':0.00235959)'':0.00332953)'':0.00673506)'':0.0041183)'':-0.00697493)'':0.0183276)'':-0.00249406)'':0.0187176)'':0.00445892)'':0.00453857)'':-0.000225122,(('PDT000645630.2':0.0690195,(('PDT000426458.1':0.0648887,'PDT000270455.1':0.0590543)'':0.000323116,(('PDT000346892.1':0.0480946,('PDT000225235.1':0.0329162,'PDT000226331.1':0.0284179)'':0.00846885)'':0.00197261,(('PDT000535352.1':0.0285065,'PDT000651135.1':0.0359056)'':0.00454727,('PDT000568677.1':0.0312628,'PDT000346890.1':0.0275665)'':0.00663913)'':0.00603875)'':0.0129336)'':0.00386139)'':-0.000547408,(('PDT000136070.2':0.0576627,'PDT000704918.1':0.0864093)'':0.0139787,((('PDT000426596.1':0.0252186,'PDT000520841.1':0.022013)'':0.0185739,('PDT000615887.1':0.0423938,('PDT000534965.1':0.0314427,('PDT000713562.1':0.0238647,('PDT000425496.1':0.0260547,'PDT000143976.3':0.0288453)'':0.00446931)'':0.0020146)'':0.00892017)'':0.00364676)'':0.0100366,((('PDT000443091.1':0.0736363,('PDT000713205.1':0.0412631,'PDT000063644.1':0.0494461)'':0.00734614)'':-0.0002669,(('PDT000124690.1':0.029474,'PDT000459431.1':0.0415418)'':0.0134176,(('PDT000615838.1':0.0338264,'PDT000094041.1':0.0289947)'':0.00569564,(('PDT000272538.2':0.0321022,'PDT000162951.1':0.034383)'':0.00421786,('PDT000181817.2':0.0348727,(('PDT000145831.3':0.0214062,'PDT000030568.1':0.0255149)'':0.00135466,('PDT000482615.1':0.0168286,('PDT000488317.1':0.0246453,('PDT000713695.1':0.0173929,('PDT000423870.1':0.0150905,'PDT000430760.1':0.0129894)'':0.0034948)'':0.00122054)'':0.0022032)'':0.0025171)'':0.00485218)'':0.00732845)'':0.00171449)'':0.00190755)'':0.00701211)'':-0.00514676,(('PDT000622400.1':0.0420559,('PDT000445030.1':0.0192554,('PDT000423892.1':0.0196593,'PDT000402913.1':0.029688)'':0.0053954)'':0.00978082)'':0.0176393,(('PDT000526198.1':0.0635034,(('PDT000246494.2':0.0430953,('PDT000409029.1':0.0377796,('PDT000140747.2':0.021336,'PDT000434598.1':0.0248194)'':0.0113765)'':0.00812847)'':-0.00115439,(('PDT000181730.2':0.0360681,'PDT000536610.1':0.0345454)'':0.00281044,('PDT000426108.1':0.0308165,('PDT000411014.1':0.0197543,'PDT000672231.1':0.0218674)'':0.00795608)'':0.00516875)'':0.00439562)'':0.0110044)'':-0.00716553,(('PDT000297996.1':0.0338197,('PDT000632823.1':0.0256426,('PDT000675381.1':0.0230388,'PDT000402915.1':0.0264447)'':0.00520853)'':0.00502092)'':0.0102497,(('PDT000636385.1':0.0398081,('PDT000633248.1':0.0219761,('PDT000181788.2':0.016155,'PDT000433283.1':0.0166691)'':0.00508844)'':0.0147128)'':-0.000534428,('PDT000673520.1':0.0577375,(('PDT000534802.1':0.0476513,(('PDT000632811.1':0.0194416,'PDT000488882.1':0.0254701)'':0.00408911,('PDT000647175.1':0.0183647,('PDT000215984.2':0.0220676,('PDT000555537.1':0.0115494,'PDT000448179.1':0.0223641)'':0.00426965)'':0.00273553)'':0.00395191)'':0.0104587)'':-0.00292156,('PDT000616098.1':0.0344318,('PDT000656957.1':0.0451583,(('PDT000346534.1':0.0325994,('PDT000423931.1':0.0129034,'PDT000534894.1':0.0125111)'':0.0107061)'':0.00370081,(('PDT000423910.1':0.0277168,'PDT000191147.1':0.0250433)'':0.00453346,('PDT000618738.1':0.0302986,('PDT000352481.1':0.0270508,('PDT000202610.2':0.0212983,('PDT000675447.1':0.0272804,('PDT000441362.1':0.0355846,('PDT000618900.1':0.0188474,('PDT000615880.1':0.0221382,('PDT000083431.1':0.0165154,('PDT000457703.1':0.0123714,('PDT000426786.1':0.0152212,('PDT000083433.1':0.00804117,('PDT000423875.1':0.00951311,('PDT000202725.2':0.00617589,('PDT000423918.1':0.00712373,('PDT000426554.1':0.00357454,'PDT000713393.1':0.00470972)'':0.00101749)'':0.000688682)'':0.00144231)'':0.00130384)'':0.00142586)'':0.00258736)'':0.00127264)'':0.00050737)'':0.00256719)'':0.00169586)'':0.000722149)'':0.000979963)'':0.00178636)'':0.000578275)'':-0.000925065)'':0.000924002)'':0.00843091)'':0.00156657)'':-0.00716082)'':0.011449)'':-0.00440644)'':0.00239466)'':0.00178345)'':0.00731432)'':-0.00472816)'':0.00317804)'':0.0193589)'':-0.0136923)'':0.0104569)'':0.00401398)'':0.00307036)'':0.00991066)'':0.00964271)'':0.0102273,(((((('PDT000497421.1':0.0351212,'PDT000551871.1':0.0285375)'':0.0344544,('PDT000305508.2':0.0610849,('PDT000719121.1':0.0622077,('PDT000425626.1':0.022672,'PDT000411336.1':0.0167613)'':0.0448277)'':0.00482236)'':0.0048024)'':0.00172339,(('PDT000309297.2':0.0469145,'PDT000499493.1':0.0467518)'':0.0213304,(('PDT000354869.1':0.0319471,('PDT000354935.1':0.0104012,'PDT000499507.1':0.014736)'':0.0157175)'':0.0280519,(('PDT000586259.1':0.0560658,'PDT000228386.1':0.0494372)'':0.0120911,('PDT000557022.1':0.0419618,('PDT000204921.2':0.0293417,('PDT000557146.1':0.02113,'PDT000557175.1':0.0306026)'':0.0146661)'':0.00932117)'':0.00644154)'':0.00223234)'':0.00947737)'':0.0119293)'':0.0211748,((('PDT000318617.2':0.0691761,'PDT000322744.2':0.0675429)'':0.00372028,('PDT000409217.2':0.0718254,'PDT000124612.1':0.0753286)'':0.00334922)'':0.0271694,((('PDT000466993.1':0.0252617,'PDT000541650.1':0.0346439)'':0.0068747,('PDT000065388.1':0.0377591,('PDT000227994.1':0.0247952,'PDT000228261.1':0.0184518)'':0.013039)'':0.00110366)'':0.0607652,('PDT000228178.1':0.0939811,('PDT000309313.2':0.0784532,('PDT000713512.1':0.056673,('PDT000025985.1':0.065425,('PDT000617806.1':0.0452864,('PDT000129807.2':0.0366914,('PDT000713313.1':0.0331657,('PDT000346891.1':0.0246415,('PDT000354201.1':0.0133469,('PDT000132323.2':0.00675141,'PDT000661391.1':0.0108205)'':0.00723746)'':0.00591628)'':0.00641931)'':0.00961693)'':0.00933759)'':0.00451543)'':0.00744835)'':0.0102133)'':0.00208075)'':0.00475261)'':0.00332829)'':-0.000527757)'':0.000634796,(((('PDT000668171.2':0.0602966,'PDT000662589.1':0.0642094)'':0.0078947,('PDT000236380.2':0.0324751,'PDT000397669.1':0.028813)'':0.0507505)'':0.000442163,(('PDT000429719.1':0.0575245,('PDT000174314.1':0.0474289,('PDT000174317.1':0.0235311,'PDT000174319.1':0.0181849)'':0.0136382)'':0.0162117)'':0.00621731,('PDT000429715.1':0.0583953,('PDT000429750.1':0.0516718,(('PDT000010920.2':0.00307802,'PDT000248545.1':0.0026106)'':0.0307312,('PDT000174299.1':0.0312473,'PDT000174367.1':0.0250907)'':0.00530163)'':0.0139512)'':0.00210451)'':0.0111685)'':0.020637)'':0.016707,((('PDT000556986.1':0.0665795,'PDT000718924.1':0.0644645)'':0.00412774,(('PDT000132193.3':0.0397596,('PDT000559295.2':0.0285123,('PDT000160209.3':0.027463,('PDT000161541.3':0.029608,('PDT000205102.3':0.00947181,'PDT000679778.1':0.00867219)'':0.00937393)'':0.00646713)'':0.0047795)'':0.00697535)'':0.0265024,((('PDT000171536.3':0.0367827,('PDT000151501.3':0.0346736,'PDT000372126.2':0.0285008)'':0.00400541)'':0.0110429,('PDT000132410.3':0.0560407,('PDT000354872.2':0.0343391,'PDT000523346.2':0.0377233)'':0.00430213)'':0.00930646)'':0.00122103,('PDT000186213.1':0.0663052,(('PDT000649872.2':0.0363915,'PDT000651886.2':0.0313703)'':0.0141284,('PDT000038902.1':0.044097,(('PDT000319289.2':0.0397366,('PDT000156934.3':0.030167,('PDT000316258.2':0.0145507,'PDT000317053.2':0.0104542)'':0.0119257)'':0.00805559)'':-0.000433307,(('PDT000318866.2':0.0278691,'PDT000326207.2':0.0205578)'':0.0127476,('PDT000325135.2':0.0389338,('PDT000327179.2':0.0256814,'PDT000427851.2':0.0188857)'':0.0194345)'':1.67211e-05)'':0.0022588)'':0.00624236)'':-0.00174326)'':0.00804312)'':-0.00295873)'':0.0168721)'':0.00701561)'':0.012925,((('PDT000015721.3':0.0545283,'PDT000174360.1':0.0517657)'':0.0201491,('PDT000250133.1':0.033611,('PDT000502998.1':0.0297183,('PDT000660055.1':0.0191784,('PDT000316167.1':0.0131753,'PDT000503000.1':0.0111014)'':0.00304909)'':0.0129684)'':0.0178222)'':0.025145)'':0.00886163,(('PDT000583478.1':0.120017,('PDT000714679.1':0.0574545,'PDT000249901.1':0.0434905)'':0.0249746)'':0.00988308,(('PDT000250041.1':0.0407326,'PDT000304199.1':0.0500968)'':0.0422996,(('PDT000486040.1':0.00249241,'PDT000517801.1':0.00510303)'':0.0881867,(('PDT000510052.1':0.0701188,'PDT000719119.1':0.0713122)'':0.0103359,(('PDT000399207.1':0.0680256,'PDT000109309.2':0.0665594)'':0.00178654,(('PDT000468332.1':0.0564203,('PDT000359182.1':0.0412178,'PDT000386722.1':0.0438721)'':0.0204538)'':0.000896605,('PDT000719279.1':0.0676008,(('PDT000095766.2':0.0544332,('PDT000397759.1':0.0423088,'PDT000248497.1':0.0374589)'':0.00624195)'':0.00220207,('PDT000661299.1':0.0476962,(('PDT000228041.1':0.0377998,'PDT000250218.1':0.0384845)'':0.00815962,(('PDT000032348.1':0.0175337,'PDT000174356.1':0.0189159)'':0.0259335,('PDT000556924.1':0.0362503,('PDT000397891.1':0.013968,'PDT000498740.1':0.0153422)'':0.0129571)'':0.0025321)'':0.00805084)'':0.00417507)'':-0.00135263)'':0.00558272)'':0.00156904)'':0.00936078)'':0.00120246)'':0.00341567)'':0.00192825)'':0.00216504)'':-0.00510618)'':0.00421867)'':0.00682572)'':0.00421725)'':-7.56144e-05,((('PDT000014133.3':0.0567664,'PDT000248756.1':0.0471996)'':0.0627559,(('PDT000438083.1':0.0666704,'PDT000197014.1':0.0700356)'':0.0174205,(('PDT000221026.1':0.0315149,('PDT000221044.1':0.00606106,'PDT000221054.1':0.00627354)'':0.029093)'':0.0433069,('PDT000131838.2':0.0679851,('PDT000221022.1':0.0627308,('PDT000032323.1':0.00530031,(('PDT000026017.1':0.00243575,('PDT000032336.1':0.00193295,'PDT000026243.1':0.00195965)'':0.000259115)'':0.000204941,('PDT000029846.1':0.00338008,('PDT000032337.1':0.00230652,('PDT000032212.1':0.00194946,'PDT000026013.1':0.00244483)'':0.000386559)'':0.000145896)'':0.000186978)'':0.00432299)'':0.0567671)'':0.0127298)'':-1.63089e-05)'':0.0179901)'':0.0173558)'':-0.00831199,(((('PDT000131793.2':0.0427305,'PDT000619961.1':0.0426552)'':0.0271373,('PDT000124675.1':0.0567011,('PDT000526159.1':0.0519617,'PDT000248816.1':0.0640603)'':0.0105259)'':0.031117)'':0.0317394,((('PDT000134256.1':0.064008,('PDT000131765.2':0.0378776,'PDT000131779.2':0.0242268)'':0.0187378)'':0.0310085,('PDT000384977.1':0.10434,('PDT000398137.1':0.073907,'PDT000426303.1':0.060516)'':0.0156276)'':0.00444618)'':0.00618545,(((('PDT000131768.2':0.0314497,'PDT000689798.1':0.0223828)'':0.0409277,(('PDT000131759.2':0.0507675,'PDT000248504.1':0.0585325)'':0.0030727,('PDT000354244.1':0.0553361,('PDT000399643.1':0.0198538,'PDT000645030.1':0.0181694)'':0.0304165)'':0.00242101)'':0.0129676)'':0.015295,((('PDT000644910.1':0.0346853,'PDT000395439.1':0.0321442)'':0.00515634,('PDT000617730.1':0.0113721,'PDT000395376.1':0.0135509)'':0.0221341)'':0.0197285,((('PDT000109451.2':0.0211794,'PDT000248507.1':0.02415)'':0.0120102,('PDT000713935.1':0.0212471,('PDT000443896.1':0.0114097,'PDT000694510.1':0.00779805)'':0.00588057)'':0.0156413)'':0.00392419,(('PDT000202203.2':0.0143882,'PDT000354230.1':0.0221038)'':0.0167337,('PDT000175867.2':0.0394867,('PDT000176008.2':0.0170223,'PDT000435639.1':0.0184247)'':0.0176237)'':0.00372851)'':0.00429706)'':0.00699047)'':0.0376527)'':-0.00546907,((('PDT000588875.1':0.0543769,'PDT000242882.1':0.0607471)'':0.00285684,(('PDT000566113.1':0.0437112,'PDT000521030.1':0.0419976)'':0.00846708,('PDT000088273.2':0.0402753,('PDT000131772.2':0.0446366,('PDT000131763.2':0.00604029,('PDT000131748.2':0.00490445,'PDT000131775.2':0.00577205)'':0.00364116)'':0.0179897)'':0.0129004)'':0.00922043)'':0.0118566)'':0.0219909,((('PDT000095939.2':0.0362095,'PDT000470004.1':0.0263091)'':0.00427779,(('PDT000167642.2':0.0361096,('PDT000557290.1':0.0275677,'PDT000250016.1':0.0253723)'':0.00823887)'':0.00417067,('PDT000285403.2':0.043612,('PDT000275524.2':0.0276739,'PDT000251451.1':0.0324204)'':0.011435)'':0.0019932)'':0.00424303)'':0.0122603,(('PDT000396822.1':0.0458918,('PDT000389651.1':0.0487781,'PDT000251717.1':0.0423516)'':0.00355889)'':0.0132636,(('PDT000425909.1':0.0625737,('PDT000539060.1':0.053392,'PDT000521091.1':0.0409311)'':0.0120512)'':0.00503084,((('PDT000538567.1':0.029253,'PDT000538577.1':0.029222)'':0.0132675,('PDT000426376.1':0.0431201,(('PDT000167672.2':0.0177263,'PDT000137857.2':0.0214119)'':0.0126311,('PDT000226477.2':0.0170915,'PDT000661652.1':0.0180982)'':0.0151239)'':0.005861)'':0.000118587)'':0.0121525,((('PDT000648072.1':0.0513934,('PDT000714691.1':0.0481743,('PDT000719145.1':0.0297786,'PDT000353632.1':0.0308953)'':0.00798683)'':0.00883144)'':-0.0048166,(('PDT000003950.2':0.0443262,'PDT000069145.2':0.0340688)'':0.00288185,(('PDT000131857.2':0.0339994,('PDT000132206.2':0.0248562,'PDT000468916.1':0.0282654)'':0.00957994)'':0.00336389,('PDT000537224.1':0.0334133,('PDT000167671.2':0.0245711,('PDT000557073.1':0.00776512,('PDT000167648.2':0.0026672,'PDT000145731.1':0.00332322)'':0.00497892)'':0.00763407)'':0.0143994)'':-0.00277612)'':0.00438081)'':0.0120103)'':-0.00112232,((('PDT000652175.1':0.0485564,('PDT000202030.2':0.0190532,('PDT000425515.1':0.00722591,'PDT000652833.1':0.00451769)'':0.011192)'':0.0271495)'':0.0023773,('PDT000363555.1':0.0434646,('PDT000346021.1':0.0331998,('PDT000713857.1':0.0183612,('PDT000645052.1':0.0151905,('PDT000644875.1':0.00223341,'PDT000662182.1':0.00236016)'':0.0127607)'':0.0079232)'':0.0054476)'':0.0045008)'':0.0138928)'':0.00118601,(('PDT000713772.1':0.0298701,(('PDT000427457.1':0.0238865,'PDT000694450.1':0.0199378)'':0.00594844,('PDT000167682.2':0.0223521,('PDT000661808.1':0.00991537,'PDT000362468.1':0.0141355)'':0.00569476)'':0.00993606)'':0.00849872)'':0.0113911,('PDT000136013.2':0.0440313,(('PDT000389649.1':0.0340661,('PDT000095940.2':0.0289822,('PDT000425764.1':0.0226155,'PDT000645076.1':0.021446)'':0.00422218)'':0.00772229)'':-0.00278473,('PDT000167629.2':0.0242528,('PDT000475380.1':0.0290752,('PDT000644853.1':0.0166446,('PDT000024556.2':0.0141651,('PDT000003805.2':0.00315889,'PDT000714240.1':0.00302195)'':0.00532186)'':0.00658785)'':0.0024222)'':0.0055772)'':0.00119639)'':0.0192523)'':-0.00164827)'':0.00156567)'':0.00281061)'':0.00359667)'':0.00734323)'':0.00139357)'':-0.00554304)'':0.025956)'':-0.00481043)'':0.0120713)'':0.0114235)'':-0.0234697,(('PDT000468935.1':0.104782,((('PDT000286097.1':0.0261846,'PDT000469332.1':0.034188)'':0.00104972,('PDT000468865.1':0.0155008,('PDT000468845.1':0.00912278,'PDT000468863.1':0.00942332)'':0.0118236)'':0.0094025)'':0.0564865,(('PDT000175823.2':0.0700213,('PDT000397995.1':0.0336437,('PDT000312618.2':0.0356035,'PDT000399701.1':0.0319772)'':0.00337558)'':0.0238845)'':0.0131092,(('PDT000425970.1':0.0647067,'PDT000353627.1':0.0590563)'':0.0064911,('PDT000353635.1':0.0776303,('PDT000423883.1':0.04316,'PDT000423895.1':0.0394537)'':0.0221358)'':0.00689051)'':0.00633847)'':0.0051727)'':0.020789)'':-0.0156274,(((('PDT000397944.1':0.0539554,'PDT000399620.1':0.0489566)'':0.00760886,('PDT000565886.1':0.0395626,'PDT000566191.1':0.0308926)'':0.0235463)'':0.0145085,((('PDT000341208.1':0.0335333,'PDT000710534.1':0.0315907)'':0.00293228,('PDT000397683.1':0.0286652,('PDT000294550.2':0.0358909,'PDT000395074.1':0.0343478)'':0.00778095)'':0.0014486)'':0.00280473,('PDT000397824.1':0.0543943,('PDT000399656.1':0.0434802,(('PDT000397728.1':0.0399697,('PDT000004386.2':0.0334262,'PDT000026893.2':0.0281468)'':0.0104074)'':0.00503538,('PDT000399625.1':0.0383478,('PDT000024327.2':0.0258563,('PDT000246508.2':0.033364,('PDT000395481.1':0.0243761,('PDT000204877.2':0.0201564,'PDT000274097.2':0.0171445)'':0.00353775)'':0.00385818)'':0.00429637)'':0.00296117)'':0.00367366)'':0.00533512)'':0.00387339)'':-0.0028273)'':0.022956)'':0.0189501,(('PDT000505382.1':0.0951572,('PDT000346165.1':0.0853919,(('PDT000521041.1':0.0683419,('PDT000088262.2':0.0604527,'PDT000616153.1':0.0650663)'':0.00174464)'':0.00262608,('PDT000101292.1':0.0710796,(('PDT000029844.1':0.0108371,('PDT000026011.1':0.00631526,'PDT000026014.1':0.00592354)'':0.00820304)'':0.0456963,(('PDT000497208.1':0.0593485,(('PDT000426582.1':0.0410215,'PDT000720451.1':0.0417033)'':0.00437941,('PDT000537869.1':0.0469728,('PDT000444915.1':0.0295931,'PDT000125953.1':0.0344381)'':0.00689391)'':0.00189449)'':0.0116537)'':-0.00296177,(('PDT000661443.1':0.0437652,('PDT000425885.1':0.0382217,'PDT000618832.1':0.0414765)'':0.0137926)'':-0.00045824,(('PDT000359340.1':0.0326467,('PDT000490244.1':0.0408303,'PDT000716818.1':0.0313952)'':0.00260497)'':0.00610777,('PDT000720212.1':0.0481842,(('PDT000341207.1':0.0335002,'PDT000346182.1':0.0364278)'':0.00755311,('PDT000395090.1':0.0371354,('PDT000720193.1':0.0222114,('PDT000425810.1':0.021204,'PDT000352491.1':0.0205527)'':0.005537)'':0.00358629)'':0.00153072)'':0.00765503)'':0.000701265)'':0.00467384)'':0.000135793)'':0.00517711)'':0.0228845)'':-0.0161511)'':0.00744153)'':0.00251041)'':-0.00897859,(((('PDT000396814.1':0.0402286,('PDT000249902.1':0.032413,('PDT000069616.1':0.0327162,('PDT000102754.1':0.0226871,'PDT000501897.1':0.0252019)'':0.00392706)'':0.00247462)'':0.005807)'':0.00668137,(('PDT000034503.2':0.0513428,('PDT000397987.1':0.0272624,('PDT000027905.2':0.0153514,'PDT000397613.1':0.0194398)'':0.00304779)'':0.0115391)'':0.00725654,(('PDT000167636.2':0.036386,'PDT000565862.1':0.0403726)'':0.00636378,(('PDT000141380.2':0.0174817,'PDT000396830.1':0.020256)'':0.00886586,('PDT000694485.1':0.0152571,'PDT000468890.1':0.0198289)'':0.00994836)'':0.00880247)'':0.000459426)'':0.0112085)'':7.46413e-05,(('PDT000664956.1':0.0669952,('PDT000626691.1':0.0577841,('PDT000535369.1':0.0481197,('PDT000468872.1':0.040713,('PDT000521043.1':0.0226795,'PDT000133948.1':0.0209736)'':0.0170275)'':0.00876327)'':0.00826525)'':0.00362345)'':0.00209331,(('PDT000616128.1':0.0524151,('PDT000394816.1':0.0490865,('PDT000397911.1':0.0310753,'PDT000719126.1':0.032147)'':0.00921299)'':0.00119577)'':0.00588683,('PDT000372058.2':0.0556666,(('PDT000399599.1':0.0460596,'PDT000556900.1':0.0360895)'':0.00648935,('PDT000133946.1':0.0258286,('PDT000167632.2':0.0179881,'PDT000167647.2':0.0213416)'':0.00956349)'':0.0143534)'':0.00839163)'':0.00706888)'':0.0043968)'':0.00154804)'':0.0083979,((('PDT000584746.1':0.0634902,'PDT000584749.1':0.0394758)'':0.0263518,('PDT000718935.1':0.0270139,('PDT000661451.1':0.0293623,(('PDT000713659.1':0.0140227,'PDT000252435.1':0.0123659)'':0.00367056,('PDT000714032.1':0.0127161,'PDT000378617.1':0.00926521)'':0.00660741)'':0.00790018)'':0.00772677)'':0.0148061)'':0.0153876,(('PDT000251781.1':0.0634078,('PDT000556908.1':0.045205,'PDT000557136.1':0.046603)'':0.00977716)'':0.00232745,((('PDT000719153.1':0.0750728,('PDT000395383.1':0.0600027,('PDT000399685.1':0.0295563,('PDT000557221.1':0.0171426,'PDT000019613.1':0.0201899)'':0.0145348)'':0.0252939)'':0.00658704)'':-0.004571,(('PDT000394807.1':0.0275399,('PDT000589284.1':0.038384,'PDT000395071.1':0.0304768)'':0.00284519)'':0.014655,((('PDT000300819.2':0.0338285,'PDT000396785.1':0.0379733)'':0.00605493,('PDT000306686.2':0.0294235,('PDT000557031.1':0.0158513,'PDT000557300.1':0.0240532)'':0.0108171)'':0.0131645)'':0.00495314,(('PDT000124732.1':0.0328252,('PDT000652901.1':0.0273061,'PDT000152664.1':0.0152598)'':0.00835577)'':0.00528418,('PDT000019611.1':0.0512648,('PDT000135997.2':0.0357464,('PDT000299994.2':0.0283682,'PDT000248778.1':0.0302145)'':0.00728514)'':0.00452565)'':-0.000467922)'':0.00553861)'':0.0012602)'':0.0034351)'':0.00132256,(('PDT000095774.2':0.0633182,(('PDT000612015.1':0.0189531,'PDT000713435.1':0.0234223)'':0.00477448,('PDT000557158.1':0.0321662,('PDT000010481.2':0.0244809,('PDT000556917.1':0.022375,('PDT000010756.2':0.0127367,('PDT000557231.1':0.00841183,'PDT000010440.2':0.0104956)'':0.0025655)'':0.00859322)'':0.00300721)'':0.00530586)'':-0.00042316)'':0.0188649)'':-0.0107424,(('PDT000524034.1':0.0473792,'PDT000395354.1':0.0381631)'':0.00202983,(('PDT000719149.1':0.0410523,('PDT000639549.1':0.0379664,'PDT000644980.1':0.0390531)'':0.00359078)'':0.00380588,(('PDT000539353.1':0.0232343,'PDT000241773.1':0.0233117)'':0.00796018,('PDT000713660.1':0.0331554,('PDT000713980.1':0.0258348,('PDT000066397.2':0.0128808,('PDT000397723.1':0.0119087,'PDT000399605.1':0.0120043)'':0.00251112)'':0.00699189)'':0.00477476)'':-0.00119198)'':0.00748296)'':0.00499072)'':0.00206915)'':0.00259394)'':0.00418304)'':0.0040355)'':-0.00101174)'':0.0030069)'':0.00888064)'':0.0053094)'':-0.000529984)'':0.0204248)'':-0.00151394)'':0.0063714)'':0.0236302)'':0.00864545)'':0.0494844,(((((((('PDT000072768.3':0.0192049,('PDT000641495.2':0.00598544,('PDT000273536.3':0.00608149,'PDT000647178.2':0.00508051)'':0.000671708)'':0.0103148)'':0.00483697,(('PDT000487478.2':0.0196336,'PDT000366271.2':0.0272013)'':0.00499639,('PDT000285774.3':0.0212999,('PDT000242824.3':0.017422,('PDT000623979.2':0.011249,('PDT000281563.3':0.00899897,'PDT000641982.2':0.00732923)'':0.00318265)'':0.00821663)'':0.002218)'':0.000868497)'':8.28753e-05)'':0.0223633,(('PDT000707429.1':0.059862,('PDT000251873.1':0.0360019,('PDT000201311.3':0.02064,('PDT000646239.2':0.0124753,('PDT000485846.2':0.00661425,('PDT000336948.2':0.00730562,'PDT000586040.2':0.00642438)'':0.00192575)'':0.00427752)'':0.00375611)'':0.0131701)'':0.00673949)'':0.00507016,(('PDT000125502.3':0.050433,('PDT000384102.2':0.0350439,('PDT000209186.3':0.0218472,('PDT000163608.3':0.01606,'PDT000497905.1':0.0150665)'':0.00349782)'':0.009359)'':0.00420818)'':-0.00266556,('PDT000318597.2':0.0492274,('PDT000289832.3':0.0477213,(('PDT000226833.3':0.0220827,('PDT000305223.3':0.0196099,'PDT000603647.2':0.0195769)'':0.00402346)'':0.00407163,(('PDT000052688.1':0.0165226,'PDT000052689.1':0.0162384)'':0.00505543,(('PDT000324594.2':0.0222402,'PDT000373178.2':0.0210843)'':0.00781643,(('PDT000341789.2':0.0128846,('PDT000155819.3':0.0109558,'PDT000275670.3':0.013831)'':0.00437139)'':0.000638933,('PDT000113710.3':0.0173639,(('PDT000142630.3':0.0140963,'PDT000702468.1':0.0184272)'':0.00278511,(('PDT000288531.3':0.0135264,'PDT000547711.2':0.0100487)'':0.00267896,(('PDT000556837.2':0.0107964,'PDT000132436.3':0.0122035)'':0.00157241,('PDT000146492.3':0.0293613,(('PDT000215093.3':0.0088794,(('PDT000171583.3':0.00304283,'PDT000702890.1':0.00324146)'':0.000877291,('PDT000136892.3':0.00249961,'PDT000137113.3':0.00219195)'':0.00170549)'':0.00206302)'':0.00268896,('PDT000137153.3':0.0145665,(('PDT000336942.2':0.00897485,('PDT000672386.1':0.00933006,('PDT000537439.2':0.00593797,('PDT000372738.2':0.00537463,('PDT000602568.2':0.0028286,'PDT000081158.3':0.00235792)'':0.00169776)'':0.000402707)'':0.00121225)'':0.00105343)'':0.000433576,('PDT000051247.3':0.0164707,('PDT000136895.3':0.00929014,(('PDT000081121.3':0.00763229,('PDT000521241.2':0.00559618,('PDT000619310.2':0.00484702,'PDT000638821.2':0.00520378)'':0.000966874)'':0.00243356)'':-0.00105953,('PDT000498115.1':0.00884903,('PDT000105089.3':0.0112974,('PDT000244751.3':0.00656984,('PDT000603515.2':0.00437553,('PDT000366108.2':0.00497648,('PDT000185659.3':0.00390444,(('PDT000140571.4':0.00219103,'PDT000140518.3':0.00249912)'':0.000701048,('PDT000148500.3':0.00249891,('PDT000157504.3':0.00214809,('PDT000238398.3':0.00182678,'PDT000096845.3':0.00226504)'':0.00034781)'':4.76634e-05)'':0.000830549)'':0.000340654)'':0.000510986)'':0.000540721)'':0.000597009)'':0.00144707)'':0.00287425)'':-0.00241159)'':0.00372171)'':0.00110114)'':-0.00372602)'':0.00461265)'':-0.00294718)'':0.00555514)'':-0.00255725)'':0.000623014)'':0.00149534)'':0.00376363)'':-0.0034577)'':0.00407448)'':0.00191757)'':-0.00210524)'':0.0130884)'':0.00209637)'':-0.00999448)'':0.0140705)'':0.00121743)'':0.00716063,('PDT000619723.2':0.0728212,(('PDT000161653.3':0.0310492,('PDT000161549.3':0.0316736,'PDT000683262.1':0.0246256)'':0.00185736)'':0.0156911,(('PDT000146110.3':0.0404626,'PDT000637268.2':0.0550862)'':0.00456251,(('PDT000377149.2':0.0205942,('PDT000172085.3':0.0131424,'PDT000390824.2':0.0133608)'':0.00488389)'':0.023189,((('PDT000326194.2':0.0108934,'PDT000524123.2':0.0118407)'':0.0146363,('PDT000161641.3':0.0166756,('PDT000332810.2':0.0136826,('PDT000327887.2':0.00846703,('PDT000524127.2':0.00817741,'PDT000598083.2':0.00840819)'':0.00129822)'':0.00520781)'':0.00572767)'':0.00851999)'':0.00428374,(('PDT000601933.2':0.0298357,'PDT000498151.1':0.0241759)'':0.0199567,(('PDT000248626.1':0.0434237,(('PDT000321570.2':0.0301302,('PDT000549278.2':0.0102242,('PDT000372745.2':0.0092181,('PDT000521262.2':0.00662652,'PDT000532328.2':0.00830778)'':0.00310085)'':0.00145098)'':0.0098693)'':0.00276298,(('PDT000422020.2':0.0148254,('PDT000494932.2':0.0149121,('PDT000084014.3':0.0100103,('PDT000491406.2':0.00791061,'PDT000594708.2':0.00701629)'':0.00460111)'':0.00371589)'':0.00131937)'':0.00664546,(('PDT000609732.2':0.0236343,('PDT000227532.3':0.0247261,'PDT000649947.2':0.0175149)'':0.00484972)'':0.00138452,('PDT000537148.2':0.0292499,(('PDT000133831.3':0.0243564,('PDT000251822.1':0.0214906,('PDT000146042.3':0.0163034,('PDT000146569.3':0.0149748,'PDT000048656.3':0.0147994)'':0.00354924)'':0.00274584)'':0.00157309)'':-0.000556098,(('PDT000417816.2':0.0163355,('PDT000235351.3':0.0186323,'PDT000252877.3':0.0124717)'':0.00304816)'':0.00198889,('PDT000222033.3':0.0189447,(('PDT000649821.2':0.018414,(('PDT000336239.2':0.00859568,'PDT000644049.2':0.00946882)'':0.00195009,('PDT000366284.2':0.00838515,'PDT000414399.2':0.00840735)'':0.00212488)'':0.00282263)'':-0.00115041,(('PDT000504000.2':0.00927146,'PDT000577458.2':0.0105836)'':0.0019809,(('PDT000630327.2':0.014588,('PDT000366134.2':0.00826351,'PDT000136742.3':0.00791359)'':0.00286255)'':-0.000282025,('PDT000333157.2':0.00824768,('PDT000250869.3':0.00902559,(('PDT000353314.2':0.00490868,'PDT000102761.3':0.00505545)'':0.00127217,('PDT000225512.3':0.00499092,('PDT000655534.2':0.00424398,'PDT000120853.3':0.00383099)'':0.0014695)'':0.000939427)'':0.00193378)'':0.000472104)'':0.0010351)'':0.00244728)'':5.40226e-05)'':0.00374939)'':-0.000963643)'':0.00424074)'':0.00427059)'':-0.00413465)'':0.00188141)'':0.000320112)'':0.00822931)'':-0.00446689,((('PDT000123277.1':0.0184854,('PDT000161675.3':0.0194327,('PDT000325157.2':0.00964516,('PDT000148283.3':0.00931209,('PDT000148085.3':0.0054288,'PDT000148225.3':0.005523)'':0.00229526)'':0.000929239)'':0.00891753)'':0.00275168)'':0.00113563,('PDT000240310.1':0.0407449,('PDT000548552.2':0.0226834,(('PDT000250769.3':0.00489492,'PDT000707422.1':0.00457383)'':0.0133802,('PDT000679287.1':0.028682,('PDT000484011.2':0.0145965,('PDT000344438.2':0.0143663,('PDT000163012.4':0.00671126,'PDT000120848.3':0.00731994)'':0.00689362)'':0.0019891)'':0.00307173)'':-0.000287456)'':0.0029652)'':0.00528788)'':-0.004713)'':0.00116053,('PDT000702592.1':0.0540012,(('PDT000251622.1':0.0310244,'PDT000497769.1':0.0252556)'':0.00299774,(('PDT000465837.2':0.0168175,('PDT000492241.2':0.0144263,('PDT000587171.2':0.0126795,'PDT000717561.1':0.012323)'':0.00141755)'':0.00103409)'':0.00134275,(('PDT000426887.2':0.0228584,'PDT000544572.2':0.0173034)'':0.0024358,('PDT000367201.2':0.0278557,(('PDT000139192.3':0.0112858,('PDT000306759.3':0.0131656,(('PDT000142848.3':0.00879785,'PDT000707423.1':0.00977215)'':0.0010531,('PDT000683665.1':0.00777603,('PDT000660037.2':0.00584316,'PDT000711019.1':0.00618704)'':0.00168352)'':0.00205549)'':0.00156694)'':0.0016209)'':-0.000737185,(('PDT000338694.2':0.00669935,'PDT000592254.2':0.00593625)'':0.00542388,('PDT000319685.2':0.0140458,(('PDT000176661.3':0.0115533,('PDT000356265.2':0.00743726,'PDT000372106.2':0.00603044)'':0.00241711)'':0.00257942,(('PDT000517827.2':0.00557061,'PDT000706151.1':0.00576819)'':0.00511115,('PDT000366124.2':0.013113,('PDT000500456.2':0.00739221,('PDT000295887.3':0.00687661,'PDT000372133.2':0.00722609)'':0.000976942)'':0.00225524)'':0.000281395)'':0.000487179)'':0.00243972)'':0.000176668)'':-0.000476677)'':0.0076102)'':-0.00267057)'':-0.000193387)'':0.00481493)'':0.0127451)'':-0.00998126)'':0.00105681)'':0.00741408)'':-0.00235292)'':0.00690745)'':0.00763468)'':-0.00259179)'':0.0252889)'':-0.0198534)'':0.0450845,(((('PDT000377056.1':0.0467336,'PDT000548607.2':0.0446277)'':0.00434298,('PDT000158663.2':0.0373016,('PDT000325679.1':0.0330435,'PDT000108882.2':0.0223636)'':0.00584155)'':0.00558842)'':0.00689038,((('PDT000315907.2':0.0265415,('PDT000142632.3':0.012318,'PDT000573753.2':0.00978967)'':0.00727018)'':0.00130189,('PDT000492196.2':0.00809859,('PDT000492199.2':0.00365635,'PDT000081137.3':0.00372005)'':0.00498766)'':0.00765645)'':0.0283439,((('PDT000150713.3':0.0291824,'PDT000498073.1':0.0291841)'':0.00355617,('PDT000497897.1':0.0267584,('PDT000051481.1':0.0210541,(('PDT000351869.2':0.0207481,'PDT000137587.3':0.0167088)'':0.00235887,(('PDT000576878.2':0.0103641,('PDT000691346.1':0.00906034,'PDT000498006.1':0.0115297)'':0.00441474)'':-0.000611111,('PDT000524351.2':0.0131252,(('PDT000142646.3':0.00520251,('PDT000148504.3':0.00409847,'PDT000240595.3':0.00457551)'':0.00130919)'':0.00281699,('PDT000625716.2':0.00890434,('PDT000532190.2':0.00363236,'PDT000084030.3':0.00433736)'':0.0029255)'':0.00058812)'':0.00525766)'':5.07935e-05)'':0.00189183)'':0.0065252)'':0.00105814)'':0.0032893)'':0.00821651,((('PDT000337041.2':0.0285946,'PDT000050560.1':0.022753)'':0.00141011,('PDT000087351.3':0.0243458,('PDT000257989.3':0.0176229,('PDT000221744.3':0.014516,('PDT000271278.3':0.0118071,'PDT000127404.3':0.0124383)'':0.00173897)'':0.0032835)'':0.00536099)'':0.00461255)'':0.00045998,(('PDT000248578.1':0.0329711,('PDT000497883.1':0.0299929,('PDT000624935.2':0.0180241,('PDT000025756.1':0.0131798,(('PDT000173602.3':0.00154776,'PDT000203362.3':0.00154914)'':0.00546956,('PDT000083979.3':0.01422,('PDT000265281.3':0.00522971,('PDT000066888.5':0.00407385,('PDT000083238.3':0.00357938,('PDT000137670.3':0.00219467,(('PDT000198404.4':0.00174792,'PDT000410905.2':0.00144793)'':0.000176229,('PDT000492161.2':0.00153007,'PDT000062535.3':0.00126629)'':0.000373461)'':0.000573852)'':0.00196535)'':0.00160091)'':0.0019245)'':0.00177391)'':-0.000196606)'':0.00589834)'':0.00177813)'':0.00507125)'':0.00166062)'':-0.00501954,(('PDT000016358.2':0.0266167,('PDT000708636.1':0.0256787,('PDT000492207.2':0.0172712,('PDT000110183.3':0.0170495,('PDT000173970.3':0.0149575,('PDT000372695.2':0.0100006,'PDT000394053.2':0.00924345)'':0.00456658)'':0.000313577)'':0.00286156)'':0.00249005)'':0.00327181)'':0.00159131,('PDT000385900.2':0.0383168,('PDT000372128.2':0.037977,(('PDT000040683.2':0.0164157,('PDT000345971.1':0.0128145,(('PDT000153171.3':0.0102167,'PDT000524509.2':0.00740747)'':0.00640876,('PDT000498082.1':0.00738657,('PDT000162990.4':0.00597007,'PDT000583326.2':0.00352234)'':0.00445682)'':0.00249275)'':0.00390258)'':0.000302667)'':-0.00110465,((('PDT000365748.2':0.0101955,'PDT000539913.2':0.0101176)'':0.00253967,('PDT000523863.2':0.00947792,('PDT000084249.3':0.0119145,('PDT000248637.1':0.0105045,('PDT000489617.2':0.00572984,('PDT000216617.3':0.0051536,('PDT000040410.6':0.00333375,('PDT000257243.3':0.0012,'PDT000489619.2':0.00139688)'':0.00205123)'':0.00148821)'':0.00249164)'':0.00116638)'':0.00303949)'':0.000872819)'':-0.000417851)'':0.0020235,('PDT000139220.3':0.0184377,('PDT000545774.2':0.0295333,('PDT000142535.3':0.026008,('PDT000077012.3':0.0172577,(('PDT000262159.3':0.00929541,('PDT000226842.3':0.00880391,'PDT000645883.2':0.00580409)'':0.00326749)'':0.00341935,('PDT000086979.1':0.0156054,(('PDT000497891.1':0.00710745,'PDT000497940.1':0.00690425)'':0.00304466,('PDT000040687.3':0.0125285,('PDT000142591.3':0.0150285,('PDT000592579.2':0.00965146,('PDT000144063.3':0.0098199,(('PDT000250781.3':0.00668893,'PDT000576163.2':0.00772747)'':0.00100455,(('PDT000046587.3':0.00535328,'PDT000600748.2':0.00828672)'':0.00351042,('PDT000066900.3':0.00858161,(('PDT000300589.1':0.00663568,('PDT000248583.1':0.00359281,'PDT000251370.1':0.00348313)'':0.00125785)'':0.00108384,(('PDT000583327.2':0.00742153,'PDT000671660.1':0.00516207)'':0.00167667,(('PDT000671694.1':0.00334678,'PDT000107031.3':0.00363159)'':0.00187971,('PDT000708143.1':0.00746054,(('PDT000212451.3':0.00698589,'PDT000233646.3':0.00429601)'':0.00205242,('PDT000638771.2':0.00848235,('PDT000197036.3':0.00472509,(('PDT000162300.3':0.00514946,'PDT000133760.3':0.00470815)'':0.000295305,(('PDT000318811.2':0.00345046,'PDT000587803.2':0.00362972)'':0.00136853,('PDT000547689.2':0.00408876,('PDT000223294.3':0.00496122,('PDT000009620.2':0.00386095,(('PDT000230818.3':0.00281982,'PDT000549494.2':0.00157491)'':0.000939079,(('PDT000184482.3':0.00180256,'PDT000040702.3':0.00179241)'':0.000379346,(('PDT000054731.3':0.00201096,('PDT000184471.3':0.00159949,'PDT000135925.3':0.00169557)'':0.000384592)'':6.3477e-05,('PDT000135734.3':0.00210995,('PDT000147485.3':0.00153396,'PDT000492166.2':0.00126352)'':0.000583942)'':0.000233663)'':0.000190894)'':0.000741416)'':0.000620412)'':8.63744e-05)'':0.000420123)'':-0.000157407)'':0.000415924)'':0.00105421)'':0.000172044)'':-0.000270467)'':0.00172738)'':-0.00114983)'':0.000715379)'':-0.000384935)'':0.00205354)'':-0.00102121)'':0.000770419)'':0.00189705)'':0.000236006)'':0.000866281)'':0.00168681)'':-0.00195454)'':0.00241175)'':-0.00327699)'':0.00774733)'':0.00057943)'':0.000626522)'':0.00107544)'':-0.00849018)'':0.00342924)'':0.00952257)'':0.00613132)'':-0.0115069)'':-0.00115451)'':0.00351594)'':0.00912911)'':0.00701106)'':0.00559316)'':0.027131,(('PDT000103605.3':0.0403853,('PDT000359074.1':0.0335974,(('PDT000338460.1':0.013739,'PDT000251604.1':0.0175947)'':0.00468103,('PDT000248566.1':0.0132662,('PDT000107656.2':0.0123901,'PDT000251884.1':0.00996022)'':0.00584427)'':0.010848)'':0.00905498)'':0.00956946)'':0.0307119,(('PDT000251405.1':0.121254,(('PDT000394734.2':0.0314973,('PDT000344806.2':0.0287665,('PDT000251821.1':0.0417691,(('PDT000362208.2':0.0140577,(('PDT000063293.3':0.0182144,'PDT000591929.2':0.0096322)'':0.00109209,('PDT000226791.3':0.0109349,('PDT000168324.3':0.00517061,'PDT000063295.3':0.00538609)'':0.00425518)'':0.000859685)'':0.00441685)'':-0.0006892,('PDT000096848.3':0.0296956,('PDT000338949.2':0.0138437,(('PDT000326181.2':0.0095993,'PDT000498116.1':0.0103254)'':0.00200447,('PDT000616425.2':0.0119907,('PDT000539848.2':0.00515733,('PDT000558444.2':0.00388369,'PDT000107700.3':0.0036966)'':0.00110776)'':0.00644836)'':0.000105832)'':0.00248818)'':0.00620394)'':-0.00332755)'':0.010108)'':0.00149792)'':0.00445398)'':0.0137284,(('PDT000519163.2':0.0474687,(('PDT000278347.3':0.011562,('PDT000327868.2':0.00699505,'PDT000387003.2':0.00745435)'':0.00522557)'':0.000849776,('PDT000324680.2':0.00841177,('PDT000332188.2':0.0090807,'PDT000603025.2':0.0083762)'':0.000713181)'':0.00526145)'':0.0151854)'':0.0267445,(((('PDT000132008.2':0.0389714,'PDT000525461.2':0.0407596)'':0.00494499,('PDT000509707.2':0.0373581,('PDT000498148.1':0.0223678,('PDT000377968.1':0.0234048,'PDT000625482.1':0.0223577)'':0.00138416)'':0.00987955)'':0.00369227)'':-0.00123592,(('PDT000502613.1':0.0394158,('PDT000596031.2':0.0194302,'PDT000677479.1':0.0148368)'':0.0178744)'':0.00351837,(('PDT000063297.3':0.0311994,('PDT000288434.2':0.016808,'PDT000414091.1':0.0195071)'':0.0168592)'':9.03435e-05,('PDT000625384.1':0.0382743,((('PDT000063294.3':0.025212,('PDT000025405.1':0.00839963,('PDT000190688.3':0.00294589,'PDT000492205.2':0.00324854)'':0.0054876)'':0.00726485)'':0.00666837,('PDT000086173.3':0.0236713,('PDT000234046.3':0.018597,('PDT000235799.3':0.0122854,'PDT000083964.3':0.0116726)'':0.00435985)'':0.00294038)'':0.00440041)'':0.0031988,('PDT000197016.1':0.023729,('PDT000625513.1':0.0220457,(('PDT000637600.2':0.012508,('PDT000376141.2':0.012966,'PDT000248650.1':0.0121956)'':0.00327242)'':0.000467806,('PDT000328080.2':0.0121799,('PDT000702555.1':0.00772766,('PDT000417564.2':0.00584507,'PDT000629332.2':0.00629063)'':0.00209669)'':0.00576096)'':0.000986824)'':0.00790211)'':0.00112837)'':0.00460742)'':0.00748951)'':-0.00433062)'':0.00692499)'':0.000383307)'':0.00106003,(('PDT000468276.2':0.0146141,(('PDT000222574.3':0.0126544,'PDT000689099.1':0.0106406)'':0.000978427,('PDT000389744.2':0.00850437,'PDT000085591.3':0.00635403)'':0.00351832)'':0.0016141)'':0.00594613,('PDT000524332.2':0.0228585,('PDT000326114.2':0.0205487,(('PDT000327884.2':0.0133197,('PDT000321590.2':0.00894309,'PDT000487998.2':0.0102151)'':0.00317448)'':0.000510694,('PDT000547440.2':0.0154048,('PDT000610059.2':0.0247592,(('PDT000480454.2':0.00973767,('PDT000274057.3':0.00857989,'PDT000593086.1':0.00864261)'':0.000658329)'':0.000176069,(('PDT000311062.3':0.00685744,'PDT000344825.2':0.00849646)'':0.000833685,(('PDT000320214.2':0.00356099,('PDT000096904.3':0.00250865,('PDT000063292.3':0.00179649,('PDT000188055.3':0.00102042,'PDT000566466.2':0.000778502)'':0.00119816)'':0.00078337)'':0.000735553)'':0.00120642,('PDT000238408.3':0.00793875,('PDT000539002.2':0.00514237,('PDT000285873.3':0.00677093,(('PDT000502284.1':0.00250927,'PDT000133143.1':0.00278808)'':0.00170215,('PDT000587631.2':0.00695099,('PDT000325453.2':0.00364882,('PDT000227564.3':0.00364948,('PDT000309659.3':0.00259634,('PDT000622771.2':0.000781828,('PDT000602877.2':0.000762747,'PDT000664251.1':0.000536603)'':0.000317587)'':0.00148818)'':0.00039632)'':0.000651307)'':0.000544753)'':3.49016e-05)'':0.000905905)'':0.000158386)'':0.00108792)'':-0.00093199)'':0.00444572)'':0.000543172)'':0.00545906)'':0.000398085)'':-0.00372567)'':0.00386523)'':0.00443715)'':-0.00573122)'':0.0233716)'':0.0122216)'':-0.000367602)'':0.0477383)'':-0.0541748,((('PDT000274835.6':0.0439635,('PDT000054719.3':0.0250607,('PDT000390240.2':0.0164734,'PDT000551267.2':0.0166916)'':0.00341913)'':0.00804081)'':0.0170655,((('PDT000407925.2':0.0184543,'PDT000577022.2':0.0190703)'':0.00762436,('PDT000540389.2':0.01593,'PDT000123332.1':0.0152891)'':0.0206782)'':0.00883285,(('PDT000394879.1':0.030533,('PDT000107486.2':0.0257866,'PDT000502510.1':0.0301267)'':0.00569162)'':0.00780264,((('PDT000406995.1':0.0140422,('PDT000625874.2':0.00935826,'PDT000115478.3':0.00929434)'':0.00689421)'':0.0117338,(('PDT000600465.2':0.0241714,'PDT000300541.1':0.0224197)'':0.00315949,('PDT000643678.2':0.0269205,('PDT000201202.3':0.00670099,'PDT000313030.3':0.00653021)'':0.0141142)'':0.00432715)'':0.00378236)'':0.00606164,(('PDT000431662.2':0.0516035,('PDT000480455.2':0.0349846,(('PDT000190392.3':0.0225487,'PDT000487971.2':0.0123049)'':0.00485513,('PDT000480987.2':0.019623,'PDT000490334.2':0.0193406)'':0.00423982)'':0.00668541)'':0.00984239)'':-0.000392723,((('PDT000384045.2':0.0171255,'PDT000443255.2':0.0133762)'':0.0101458,('PDT000197035.3':0.028612,('PDT000313045.3':0.00777269,('PDT000480956.2':0.00538452,'PDT000503436.2':0.00546018)'':0.00558951)'':0.0061066)'':0.00448261)'':0.00359629,('PDT000658731.1':0.0266499,(('PDT000307863.3':0.0200007,'PDT000690143.1':0.0210305)'':0.00258913,('PDT000627997.2':0.0200097,(('PDT000602961.2':0.0121935,('PDT000296217.3':0.00855979,'PDT000702886.1':0.00688921)'':0.00619488)'':0.00259852,('PDT000598850.2':0.0178419,('PDT000334331.2':0.0144324,('PDT000623182.2':0.00916213,('PDT000102776.3':0.0116905,('PDT000307499.3':0.00610012,('PDT000465591.2':0.00586404,('PDT000362556.2':0.00388703,('PDT000367228.2':0.00385926,'PDT000372593.2':0.00422861)'':0.000799415)'':0.00106457)'':0.00112463)'':0.00206888)'':0.00079278)'':0.00361145)'':0.00269414)'':-0.00287712)'':0.00521535)'':-0.00177306)'':0.00747872)'':-0.000502831)'':0.00679051)'':0.000399066)'':0.00652586)'':-0.00128724)'':0.00504317)'':-0.00309062,((('PDT000625488.1':0.0382214,'PDT000332802.2':0.0343499)'':0.00261627,('PDT000621334.2':0.0394428,(('PDT000161982.3':0.0231668,'PDT000132356.2':0.0315615)'':0.00571611,('PDT000123357.1':0.0286994,('PDT000325803.2':0.0205579,'PDT000553175.1':0.0291249)'':0.00296513)'':0.0110882)'':0.00474059)'':0.0031766)'':0.000621751,(('PDT000407004.1':0.0388834,(('PDT000191701.3':0.0118844,'PDT000580140.2':0.0135788)'':0.00802813,('PDT000492075.2':0.0181411,('PDT000242607.2':0.0108276,'PDT000497754.1':0.0127018)'':0.00528269)'':0.00336174)'':0.0191961)'':-0.00237387,(('PDT000324189.1':0.0213999,'PDT000161995.2':0.0252694)'':0.0334972,((('PDT000161648.3':0.0209097,'PDT000468267.2':0.0234021)'':0.00717774,(('PDT000157734.2':0.0225804,'PDT000124989.3':0.0247766)'':0.0105424,('PDT000417912.2':0.0216903,'PDT000045607.1':0.0234329)'':0.00800764)'':0.00141094)'':0.00425976,(('PDT000197011.1':0.0256002,('PDT000367039.2':0.0349105,('PDT000706318.1':0.030748,('PDT000387572.2':0.0111355,('PDT000171590.3':0.0107482,('PDT000366309.2':0.000973043,'PDT000566465.2':0.000825877)'':0.00758216)'':0.00632172)'':0.00281948)'':0.0111935)'':0.0025779)'':-0.00368899,((('PDT000502374.1':0.0212659,('PDT000619511.2':0.0181161,('PDT000617430.2':0.0128736,('PDT000568658.2':0.00926324,('PDT000278340.3':0.00650539,'PDT000577017.2':0.00661721)'':0.00147591)'':0.00399258)'':0.00431849)'':0.00178481)'':0.00203917,(('PDT000323668.2':0.0296488,'PDT000414568.2':0.0384023)'':0.0158743,(('PDT000047678.4':0.0173456,('PDT000038897.1':0.00419132,'PDT000248642.1':0.00389493)'':0.0101905)'':0.00266372,('PDT000161878.3':0.0286001,(('PDT000624243.2':0.00898156,('PDT000544279.2':0.00595876,'PDT000123326.1':0.00688214)'':0.00272034)'':0.00448759,(('PDT000115474.3':0.00999841,'PDT000123293.1':0.00985269)'':0.00240708,('PDT000322790.2':0.00908236,(('PDT000322844.2':0.00556138,'PDT000523285.2':0.00650242)'':0.00204779,('PDT000394247.2':0.00539607,'PDT000651780.2':0.00615193)'':0.00265949)'':0.00120123)'':0.00264518)'':0.00105243)'':0.00709602)'':-0.00225128)'':0.0140803)'':-0.00643407)'':-0.000740749,(('PDT000397806.1':0.0254037,'PDT000498129.1':0.0276623)'':0.000608499,((('PDT000213211.2':0.017211,'PDT000250545.2':0.0208908)'':0.00124665,('PDT000275742.2':0.0110638,'PDT000355466.2':0.0138)'':0.00619865)'':0.00713797,(('PDT000147771.3':0.0135917,('PDT000161689.3':0.00359668,'PDT000565122.2':0.00407646)'':0.0112588)'':0.00631339,('PDT000496670.2':0.0205753,(('PDT000502581.1':0.0226581,(('PDT000040377.3':0.0121918,'PDT000251919.1':0.0103892)'':0.000377009,('PDT000502292.1':0.00718909,'PDT000304216.1':0.00691501)'':0.00657484)'':0.00431712)'':0.00160985,('PDT000544297.2':0.0142967,('PDT000532778.2':0.0148403,(('PDT000060948.3':0.00632074,'PDT000674763.1':0.00799336)'':0.00275022,('PDT000540394.2':0.00941091,('PDT000161561.3':0.00296223,('PDT000147568.3':0.00245537,'PDT000161906.3':0.00223619)'':0.000327602)'':0.00614013)'':0.000477046)'':0.00421548)'':0.00103773)'':0.0023639)'':0.00423541)'':-0.00300801)'':0.00485694)'':0.00157531)'':-0.00349833)'':0.00464984)'':0.00256953)'':0.0189167)'':-0.01082)'':-0.000923932)'':0.00383798)'':0.00159734)'':0.0252199)'':0.00269)'':0.0203431)'':0.0839241,(('PDT000249931.1':0.209315,((('PDT000097372.2':0.0349938,'PDT000358766.1':0.037428)'':0.0362568,('PDT000008479.2':0.0369875,('PDT000097363.2':0.0336689,'PDT000097366.2':0.0425335)'':0.0104581)'':0.0196179)'':0.108644,('PDT000470101.1':0.152389,(((('PDT000109067.2':0.049691,('PDT000137281.2':0.0158677,'PDT000610721.1':0.017082)'':0.0314589)'':0.0300585,('PDT000062966.2':0.067455,('PDT000098156.2':0.0573133,('PDT000147828.2':0.0527442,'PDT000097450.2':0.0434731)'':0.00161258)'':0.00629478)'':0.0235434)'':0.00171665,(('PDT000132336.2':0.0295984,'PDT000610711.1':0.0430803)'':0.0241009,(('PDT000423294.1':0.0180639,'PDT000610583.1':0.022852)'':0.0175998,(('PDT000157837.2':0.0220646,'PDT000097383.2':0.0303033)'':0.00203093,(('PDT000692372.1':0.0343053,('PDT000557794.1':0.00653591,'PDT000610764.1':0.00655099)'':0.0153592)'':0.003558,('PDT000097454.2':0.0313777,('PDT000610730.1':0.00706479,('PDT000322999.1':0.00435462,'PDT000157006.2':0.00321736)'':0.00251977)'':0.0204602)'':0.0001402)'':0.00247375)'':0.00652844)'':0.0269309)'':0.0233418)'':0.00153061,((('PDT000131985.2':0.0453024,'PDT000173338.1':0.0407871)'':0.0216308,((('PDT000432285.1':0.0342959,'PDT000519452.1':0.0272231)'':0.0118333,('PDT000600063.1':0.0363725,'PDT000610668.1':0.0361507)'':0.00556821)'':0.00990348,(('PDT000318880.1':0.0271046,('PDT000564281.1':0.0223624,'PDT000721271.1':0.0179602)'':0.00488568)'':0.0188133,('PDT000047712.3':0.043494,('PDT000173333.1':0.0296341,('PDT000132045.2':0.020907,('PDT000610670.1':0.0112748,'PDT000610829.1':0.0106455)'':0.00633579)'':0.00322375)'':0.0109577)'':0.00708173)'':0.00396678)'':0.013017)'':0.014266,('PDT000642466.1':0.0989968,('PDT000599152.1':0.0848681,('PDT000378087.1':0.0481971,(('PDT000343900.1':0.03457,(('PDT000173337.1':0.0273322,'PDT000249853.1':0.0181807)'':0.00741844,('PDT000357617.1':0.0133132,'PDT000357647.1':0.0245007)'':0.0135228)'':0.00650658)'':0.0114474,(('PDT000173335.1':0.0560576,('PDT000097382.2':0.0156094,'PDT000173329.1':0.011899)'':0.0225157)'':-0.00207012,('PDT000678052.1':0.0279319,('PDT000519335.1':0.0139798,('PDT000132204.2':0.00899042,('PDT000610684.1':0.00591169,'PDT000610718.1':0.00542931)'':0.00271528)'':0.00406826)'':0.00549978)'':0.0146306)'':0.0079601)'':0.0030396)'':0.0351796)'':0.0114546)'':-0.0239029)'':0.00311456)'':0.100526)'':-0.00699554)'':0.0235438)'':0.0136044,((((('PDT000196613.2':0.152607,'PDT000502693.1':0.15657)'':0.0326546,('PDT000062935.2':0.151254,('PDT000587254.1':0.104054,('PDT000150250.2':0.0630149,'PDT000356783.1':0.0572511)'':0.0380457)'':0.0723936)'':0.009598)'':0.00145361,(('PDT000610395.1':0.0394488,'PDT000718929.1':0.0343106)'':0.138747,(('PDT000459045.1':0.0339654,('PDT000707490.1':0.0173585,'PDT000128672.3':0.0136845)'':0.00920502)'':0.0287414,('PDT000069434.4':0.0254433,('PDT000293116.3':0.0241399,('PDT000465611.2':0.0185117,('PDT000163005.4':0.017146,('PDT000503809.2':0.0107965,('PDT000130550.3':0.0211028,('PDT000120845.3':0.0135372,('PDT000065519.3':0.00649746,('PDT000101087.3':0.00951107,('PDT000443211.2':0.00827819,('PDT000171532.3':0.00821024,('PDT000497857.1':0.00515483,('PDT000421163.2':0.00632615,(('PDT000486132.2':0.00188107,'PDT000040378.4':0.00261084)'':0.00101669,('PDT000086184.3':0.00223214,('PDT000264760.3':0.00126703,'PDT000556119.2':0.00163007)'':0.000760674)'':0.000392126)'':0.00106039)'':0.00192298)'':0.000827087)'':0.000354055)'':0.00132445)'':0.00113821)'':0.000638105)'':0.00144344)'':0.00061488)'':0.000671221)'':0.0011466)'':0.00149674)'':0.00632048)'':0.0312639)'':0.111145)'':0.00294209)'':-0.00337063,(((('PDT000316913.1':0.0981164,('PDT000289980.1':0.0651122,'PDT000380969.1':0.0621908)'':0.0585291)'':0.0573402,('PDT000637791.1':0.173465,(('PDT000711734.1':0.0959736,'PDT000711736.1':0.103057)'':0.037211,('PDT000267826.1':0.124151,('PDT000355777.2':0.0955743,('PDT000607484.1':0.0992756,'PDT000539846.1':0.0965024)'':0.00825071)'':0.00408912)'':0.00189654)'':0.0453751)'':0.00139589)'':0.0215822,(('PDT000285322.1':0.155,(('PDT000328112.2':0.0267605,('PDT000289050.3':0.0370099,'PDT000326206.2':0.0273314)'':0.00349665)'':0.105719,(('PDT000424794.1':0.0731886,('PDT000324025.1':0.0718428,'PDT000254355.2':0.0559342)'':0.0113494)'':0.00608613,('PDT000380296.1':0.0615152,('PDT000651901.1':0.0402515,'PDT000714702.1':0.0477671)'':0.0260445)'':0.0224929)'':0.0269527)'':0.0169826)'':-0.0147958,('PDT000571164.1':0.142091,(('PDT000675645.1':0.0754697,('PDT000358762.1':0.0530467,(('PDT000427821.1':0.0239467,'PDT000321342.1':0.024903)'':0.00598502,('PDT000520983.1':0.0150641,'PDT000521036.1':0.013598)'':0.0173136)'':0.0224333)'':0.0066735)'':0.0214357,('PDT000518005.1':0.066502,('PDT000584739.1':0.069885,(('PDT000467009.1':0.0269088,'PDT000714658.1':0.020729)'':0.00509362,('PDT000557232.1':0.0267083,'PDT000718329.1':0.0218269)'':0.0129636)'':0.0214741)'':0.0180406)'':0.0130531)'':0.033609)'':-0.00632406)'':0.0491901)'':-0.00525661,(('PDT000682346.1':0.0542835,('PDT000386252.2':0.0434371,('PDT000719043.1':0.0242565,('PDT000190945.2':0.0165729,'PDT000290363.3':0.0219954)'':0.00463419)'':0.0109608)'':0.00674172)'':0.0390378,(('PDT000390435.2':0.0682507,((('PDT000040370.6':0.0178657,'PDT000085578.4':0.0124102)'':0.0132534,('PDT000548491.2':0.0153557,('PDT000061294.4':0.0185064,'PDT000063316.4':0.014616)'':0.0038933)'':0.00205349)'':0.0196107,(('PDT000208735.1':0.0336991,(('PDT000052318.1':0.0132799,'PDT000498175.1':0.0100874)'':0.00909657,('PDT000101063.3':0.0235292,('PDT000619582.1':0.0147122,'PDT000321894.2':0.0155819)'':0.00352504)'':0.00395463)'':0.00429824)'':0.00924978,('PDT000252727.1':0.0304896,('PDT000285171.1':0.0328073,('PDT000625411.1':0.0194558,('PDT000502449.1':0.0157377,('PDT000129819.2':0.0196053,('PDT000063315.3':0.0124118,('PDT000638591.1':0.0118987,('PDT000625381.1':0.00992869,'PDT000625427.1':0.00721301)'':0.00271845)'':0.00159473)'':0.00123395)'':0.00363266)'':0.00061728)'':0.0058873)'':0.00580622)'':0.00227781)'':0.00560046)'':0.0147855)'':0.0128701,(((('PDT000184468.3':0.0185226,('PDT000492182.2':0.00816551,'PDT000124814.3':0.00910329)'':0.00525507)'':0.00793193,('PDT000443407.2':0.0230061,(('PDT000224261.3':0.015065,'PDT000334190.2':0.0144834)'':0.00404148,('PDT000120829.3':0.025022,('PDT000184496.3':0.0103429,('PDT000319676.2':0.00703913,'PDT000640711.2':0.00736577)'':0.00303009)'':0.00507095)'':-0.000324571)'':0.00374773)'':-0.00214357)'':0.00429959,(('PDT000569351.2':0.0135184,('PDT000497930.1':0.00953642,('PDT000547459.2':0.00820787,('PDT000281756.3':0.00244279,'PDT000542704.2':0.0028461)'':0.00226888)'':0.00485444)'':0.00458059)'':-0.000763756,(('PDT000135809.3':0.023901,('PDT000466054.2':0.0118374,('PDT000340797.2':0.00767778,('PDT000412005.2':0.0050017,('PDT000494826.2':0.00282637,'PDT000683388.1':0.00405986)'':0.0012202)'':0.00458105)'':0.00196471)'':0.00110674)'':0.00323464,(('PDT000503817.2':0.0185158,('PDT000184438.3':0.0110823,('PDT000209195.3':0.00758991,'PDT000501756.2':0.00790009)'':0.00125138)'':0.00998133)'':-2.04626e-05,(('PDT000405093.2':0.0164656,'PDT000135919.3':0.0167393)'':0.00012595,(('PDT000383987.2':0.0144266,'PDT000558542.2':0.0186198)'':0.00266968,((('PDT000148553.3':0.00996804,'PDT000585190.2':0.00822096)'':0.00177873,('PDT000356240.2':0.00936171,'PDT000300584.1':0.0113019)'':0.00131952)'':0.00105355,(('PDT000097329.3':0.00839086,('PDT000296223.3':0.00896954,('PDT000107726.3':0.0068105,('PDT000492109.2':0.00297986,'PDT000562722.2':0.004608)'':0.00187052)'':0.00110156)'':0.00261758)'':0.00151346,('PDT000492176.2':0.0152041,(('PDT000193137.3':0.00887724,'PDT000429519.2':0.00917476)'':0.00185905,('PDT000234201.3':0.011908,('PDT000065359.1':0.0165567,('PDT000235969.3':0.0084685,('PDT000221758.3':0.00739111,('PDT000501822.1':0.00959883,('PDT000575932.2':0.00623579,(('PDT000492163.2':0.0032939,'PDT000616660.2':0.00328636)'':0.00387025,('PDT000509944.2':0.00575394,('PDT000355522.2':0.00603325,(('PDT000404033.2':0.00414991,('PDT000583925.2':0.00259899,('PDT000362566.2':0.00388862,('PDT000192004.3':0.00218622,('PDT000065201.3':0.0015707,('PDT000163375.4':0.00127829,('PDT000592445.2':0.00100884,('PDT000574156.2':0.000803841,'PDT000708518.1':0.000995079)'':0.000189925)'':0.000223013)'':0.000267655)'':0.000451763)'':0.000965331)'':0.000458144)'':0.000837107)'':-0.000486771,('PDT000390974.2':0.00368679,('PDT000556167.2':0.00362309,('PDT000077528.1':0.00309275,((('PDT000235068.3':0.00126466,'PDT000492151.2':0.00143237)'':0.000522684,('PDT000205389.3':0.00139356,('PDT000125482.3':0.00120062,'PDT000134035.3':0.00129688)'':0.000304148)'':0.000473157)'':0.000191004,('PDT000623021.2':0.00285044,('PDT000083200.3':0.00186613,('PDT000239613.3':0.00147719,('PDT000357188.2':0.00111188,('PDT000184514.3':0.000944984,'PDT000384042.2':0.000953306)'':0.000286613)'':9.14823e-05)'':0.000321297)'':0.000424094)'':-7.0489e-05)'':0.000921455)'':0.000635966)'':0.000652071)'':-0.00102815)'':0.00198812)'':0.000860872)'':-0.00115688)'':0.00228841)'':0.000160758)'':0.000518212)'':0.00226421)'':0.000501725)'':0.000818802)'':-0.00238573)'':0.00474542)'':-0.00486406)'':0.00206833)'':0.00254919)'':0.00362486)'':-0.00312184)'':-8.81632e-05)'':0.00179)'':0.00047164)'':0.00101529,(('PDT000105639.3':0.0497426,'PDT000366283.1':0.0504194)'':0.00762916,(('PDT000392394.2':0.0542722,'PDT000051194.1':0.0358917)'':0.00658494,((('PDT000186913.3':0.0161651,'PDT000102803.3':0.0182053)'':0.00271643,(('PDT000114641.3':0.0170763,'PDT000120814.3':0.0206129)'':0.0043671,('PDT000498174.1':0.0187921,(('PDT000098723.3':0.0114929,('PDT000501783.1':0.00477877,'PDT000501811.1':0.00711413)'':0.00535123)'':0.00143929,((('PDT000342366.2':0.00921793,'PDT000098343.3':0.00841157)'':0.00600295,('PDT000501787.1':0.0130708,('PDT000082129.3':0.0113583,('PDT000146575.3':0.00637213,('PDT000063310.3':0.00399985,('PDT000239595.3':0.00273595,'PDT000125503.3':0.00255189)'':0.000496229)'':0.00139814)'':0.00470695)'':0.00152139)'':0.000567098)'':0.000606957,('PDT000497983.1':0.0249824,('PDT000120835.3':0.0140141,('PDT000251626.1':0.00773258,('PDT000487631.2':0.00784225,('PDT000150661.3':0.00675711,('PDT000124941.3':0.00779556,('PDT000368909.1':0.00619597,('PDT000234229.3':0.00418062,'PDT000124979.3':0.00310336)'':0.000882946)'':0.00295659)'':0.000272249)'':0.000394355)'':0.000607601)'':0.00454855)'':0.00325533)'':-0.00449197)'':0.00146975)'':0.00519689)'':-0.000305988)'':0.00225645)'':-0.00185706,((('PDT000110474.3':0.0214337,('PDT000366200.2':0.0206666,'PDT000492152.2':0.0185953)'':0.00263591)'':0.000287903,('PDT000498066.1':0.0166502,('PDT000107694.2':0.0181494,('PDT000120832.3':0.0134606,('PDT000046583.3':0.0131994,('PDT000291455.3':0.0135171,('PDT000553551.2':0.0103223,('PDT000213975.3':0.00387446,'PDT000649342.2':0.00429938)'':0.00668402)'':0.00180159)'':0.00120237)'':0.000513578)'':0.00207066)'':0.00281136)'':0.00150592)'':0.000655319,(('PDT000497985.1':0.0218539,('PDT000108019.3':0.0191022,('PDT000498150.1':0.0122233,('PDT000501785.1':0.0120204,('PDT000468594.2':0.00762136,('PDT000707594.1':0.00759824,('PDT000556680.2':0.00366946,'PDT000032321.1':0.00532065)'':0.000898007)'':0.00128799)'':0.00326457)'':0.0024484)'':0.00236798)'':0.00435548)'':-0.00482465,(('PDT000583305.2':0.0133271,(('PDT000131894.3':0.0103385,('PDT000386797.2':0.0125255,('PDT000632000.2':0.00837438,('PDT000137201.3':0.00680395,('PDT000226779.3':0.00606604,'PDT000356278.2':0.00498536)'':0.00183)'':0.0011248)'':0.00139444)'':0.00151625)'':0.00126456,(('PDT000107831.3':0.00965212,'PDT000598839.1':0.0132266)'':0.00298189,(('PDT000592330.2':0.00834605,'PDT000124792.3':0.00933725)'':0.00212454,('PDT000344388.2':0.0109495,('PDT000176690.3':0.00926065,'PDT000127414.3':0.0102869)'':0.000869936)'':0.000487935)'':0.00161301)'':0.000757542)'':0.00436081)'':-0.00341079,(('PDT000501814.1':0.0213191,('PDT000150701.3':0.00425429,'PDT000082083.3':0.00673141)'':0.00661652)'':0.00449367,(('PDT000469857.2':0.0110482,(('PDT000334325.2':0.00819258,'PDT000653299.2':0.00759062)'':0.00011228,('PDT000152689.3':0.00677917,('PDT000224231.3':0.00566476,'PDT000407682.2':0.00547394)'':0.00227743)'':0.00092756)'':0.00242101)'':-0.000334837,(('PDT000498067.1':0.00649055,('PDT000190684.3':0.00157571,'PDT000214073.3':0.00182055)'':0.00523447)'':0.00706539,(('PDT000480973.2':0.00601002,(('PDT000146554.3':0.00190783,'PDT000137157.3':0.00158658)'':0.00105279,('PDT000492065.2':0.00201735,'PDT000126453.3':0.00247277)'':0.000988607)'':0.00278448)'':0.0121076,(('PDT000025381.2':0.0162467,('PDT000173948.3':0.00640114,('PDT000547457.2':0.00182587,'PDT000125014.3':0.00236868)'':0.00167084)'':0.00547803)'':0.00360012,(('PDT000222023.3':0.0137795,('PDT000575646.2':0.00879391,'PDT000102792.3':0.00976509)'':0.00400824)'':0.0021242,(('PDT000107724.3':0.0162698,('PDT000208179.3':0.0121287,('PDT000353610.2':0.0085364,(('PDT000650733.2':0.00355108,('PDT000359251.2':0.00280614,'PDT000689065.1':0.00258093)'':0.00103183)'':0.001308,('PDT000321923.2':0.00511121,('PDT000222068.3':0.00354183,('PDT000153284.3':0.00313817,'PDT000221771.3':0.00383881)'':0.0002435)'':0.00217829)'':0.000200301)'':0.0028089)'':0.00117684)'':0.0020896)'':-0.00114013,((('PDT000414373.2':0.00346684,('PDT000242821.3':0.00144365,'PDT000107829.3':0.0017522)'':0.00292431)'':0.0017076,('PDT000421097.2':0.00770868,('PDT000248614.1':0.00683835,('PDT000234169.3':0.00767827,('PDT000366993.2':0.00329,('PDT000687939.1':0.00352332,('PDT000300571.1':0.00218816,'PDT000501790.1':0.00350273)'':0.00186886)'':0.00044917)'':0.00120857)'':0.00249067)'':0.000474107)'':-0.000738256)'':0.00221459,((('PDT000498061.1':0.00448189,'PDT000498081.1':0.00527907)'':0.00341372,('PDT000399877.2':0.00444619,('PDT000188039.3':0.00192243,('PDT000501797.1':0.00187009,('PDT000501815.1':0.000639742,'PDT000501819.1':0.000659998)'':0.000278515)'':0.00143215)'':0.0035585)'':0.00263209)'':0.00256961,(('PDT000107725.3':0.00742868,('PDT000583991.2':0.00207415,('PDT000167952.3':0.00158274,'PDT000103489.3':0.00181251)'':0.000421005)'':0.00254898)'':0.0066324,(('PDT000501788.1':0.00991057,('PDT000273527.3':0.00909758,('PDT000253182.3':0.00581686,('PDT000142633.3':0.00247176,'PDT000352810.2':0.00261552)'':0.00493565)'':0.00255481)'':0.00113662)'':0.00115541,(('PDT000213972.3':0.015262,'PDT000063232.3':0.0143039)'':0.000973384,(('PDT000250846.3':0.0117244,('PDT000107830.3':0.00308215,('PDT000189706.3':0.0028763,'PDT000668712.1':0.00231384)'':0.000708797)'':0.00628977)'':0.000212331,('PDT000375759.2':0.0246073,('PDT000641398.2':0.010124,(('PDT000498062.1':0.00777124,('PDT000498065.1':0.00376149,'PDT000498101.1':0.00371608)'':0.00218663)'':0.00278206,(('PDT000270681.3':0.00523633,'PDT000107434.3':0.00572417)'':0.00269706,(('PDT000355527.2':0.00295638,'PDT000492066.2':0.00272941)'':0.00825397,(('PDT000135762.3':0.00391307,('PDT000410938.2':0.00338379,('PDT000141645.3':0.00301737,('PDT000084021.3':0.00288251,'PDT000137342.3':0.00340303)'':0.000124254)'':0.00101683)'':0.000793817)'':0.000321442,(('PDT000414378.2':0.00604904,('PDT000151463.3':0.00563445,'PDT000409230.2':0.00531195)'':0.000819206)'':-0.000182495,('PDT000501789.1':0.02213,(('PDT000248664.1':0.00343836,('PDT000325454.2':0.00321238,('PDT000124812.3':0.00223522,'PDT000124883.3':0.00225714)'':0.000576675)'':0.00051964)'':0.00177533,('PDT000240600.3':0.0155744,(('PDT000142585.3':0.00732763,'PDT000209184.3':0.0100749)'':0.000722459,('PDT000384199.2':0.0061274,(('PDT000046585.3':0.00577799,('PDT000574366.2':0.00425996,('PDT000162975.4':0.00356269,'PDT000188045.3':0.00351255)'':0.00061691)'':0.00102759)'':-0.000477274,(('PDT000123486.3':0.00324938,('PDT000409203.2':0.00223775,('PDT000191664.3':0.00205093,'PDT000049088.3':0.00214236)'':0.000156626)'':0.000950618)'':0.00105298,(('PDT000173965.3':0.00212692,'PDT000238335.3':0.00196571)'':0.00708378,('PDT000208710.1':0.0070651,(('PDT000273347.3':0.00363249,('PDT000061164.3':0.00280398,'PDT000061317.3':0.00358516)'':0.000551948)'':0.00223908,(('PDT000066870.3':0.00758582,('PDT000235776.3':0.000992385,'PDT000124765.3':0.00110573)'':0.00551948)'':0.000654302,('PDT000305228.3':0.00568108,('PDT000421184.2':0.0102308,((('PDT000040729.3':0.00349705,'PDT000137664.3':0.00358878)'':0.000360906,('PDT000592635.2':0.00195969,'PDT000126268.3':0.00213213)'':0.00185069)'':0.000698016,((('PDT000157505.3':0.00206148,('PDT000580717.2':0.00163676,'PDT000135781.3':0.00215717)'':0.000631935)'':0.00110273,(('PDT000125474.3':0.00192494,('PDT000234220.3':0.00151626,'PDT000063306.3':0.00188)'':0.000270527)'':0.00252958,('PDT000137346.3':0.00906646,('PDT000429540.2':0.00685422,('PDT000108160.3':0.00323805,('PDT000145314.3':0.00315994,(('PDT000151506.3':0.0027226,'PDT000366991.2':0.00266393)'':0.000138049,('PDT000640614.2':0.0027223,('PDT000086307.3':0.00270008,('PDT000270676.3':0.00318386,('PDT000146574.3':0.00287635,('PDT000344802.2':0.00274883,(('PDT000190126.3':0.00220113,('PDT000124756.3':0.00185576,('PDT000413931.2':0.00160212,'PDT000101084.3':0.00189264)'':4.06513e-05)'':0.000148673)'':7.90418e-05,('PDT000583746.2':0.00186162,('PDT000537146.2':0.00189678,((('PDT000107723.3':0.00213746,('PDT000492155.2':0.0014802,'PDT000124946.3':0.00171724)'':0.000358366)'':4.58633e-05,('PDT000650734.2':0.001775,('PDT000496344.2':0.00176194,('PDT000152695.3':0.0014755,('PDT000124795.3':0.00110242,'PDT000124908.3':0.000996741)'':0.000222025)'':0.000161661)'':0.000279516)'':-2.04597e-05)'':0.000179775,('PDT000616666.2':0.00193165,(('PDT000148554.3':0.00158234,('PDT000302782.3':0.0013224,'PDT000714215.1':0.0011751)'':0.000115161)'':8.08535e-05,(('PDT000243706.3':0.00103892,'PDT000465879.2':0.00125878)'':0.00026966,('PDT000069932.3':0.00149114,('PDT000227582.3':0.00113767,'PDT000315837.2':0.00126017)'':0.000305822)'':9.43472e-05)'':0.000240101)'':0.000239671)'':-7.15929e-05)'':0.000311174)'':0.000210966)'':-0.000238371)'':0.000540828)'':0.00021558)'':6.68584e-05)'':4.64952e-05)'':0.000539593)'':-0.000605979)'':0.000627537)'':0.000206698)'':0.000456346)'':0.000864767)'':-0.00141171)'':0.00020655)'':-0.000557158,('PDT000302847.3':0.00731932,(('PDT000501782.1':0.00532415,('PDT000123360.1':0.00326016,('PDT000501821.1':0.00317892,(('PDT000248665.1':0.00205805,'PDT000251625.1':0.00213692)'':7.79371e-05,('PDT000008725.2':0.00236695,(('PDT000501823.1':0.001262,('PDT000313449.1':0.0010972,('PDT000273986.1':0.00115423,'PDT000300579.1':0.000744441)'':0.000201177)'':0.000170211)'':0.000374605,('PDT000598808.1':0.00201753,(('PDT000501781.1':0.00102435,'PDT000501784.1':0.00167295)'':0.000273764,('PDT000598701.1':0.00156857,(('PDT000501786.1':0.00132891,'PDT000501791.1':0.00106941)'':0.000307627,('PDT000300567.1':0.00132739,('PDT000497292.2':0.000800413,'PDT000501818.1':0.000798627)'':0.000470025)'':0.000234046)'':9.03493e-05)'':-9.39217e-06)'':0.000328045)'':-0.000111966)'':0.00040163)'':0.000561694)'':0.00111308)'':0.00055738)'':0.000273348)'':-0.000636484,(('PDT000608721.2':0.00564138,(('PDT000242835.3':0.00271719,(('PDT000273048.3':0.00161033,'PDT000307887.3':0.0016844)'':0.000356969,('PDT000336890.2':0.0017099,('PDT000235078.3':0.00212263,'PDT000384085.2':0.00147162)'':0.000187809)'':0.000212292)'':0.000704008)'':-4.60999e-05,('PDT000273509.3':0.00324332,('PDT000544278.2':0.00279858,('PDT000040726.3':0.0024128,(('PDT000222793.3':0.00164224,'PDT000250840.3':0.00175233)'':0.000498526,('PDT000234155.3':0.00152717,('PDT000312489.3':0.00174782,'PDT000051378.3':0.0014493)'':0.000369784)'':0.000110772)'':0.000580365)'':0.000706004)'':0.00012095)'':-0.000381167)'':0.00111425)'':-0.000667773,('PDT000120920.3':0.00401393,(('PDT000344824.2':0.00357249,('PDT000452722.2':0.00267015,'PDT000065193.3':0.00291756)'':0.000813223)'':-0.000244292,('PDT000216612.3':0.00535827,('PDT000171526.3':0.00308231,('PDT000105119.3':0.00381237,('PDT000135805.5':0.00431676,('PDT000492106.2':0.00354344,('PDT000367203.2':0.00299684,(('PDT000040727.3':0.00201692,'PDT000125005.3':0.00217721)'':0.000531594,('PDT000311093.3':0.00297063,('PDT000360405.2':0.00235141,(('PDT000549210.2':0.00206753,'PDT000124981.3':0.00192608)'':0.000542239,('PDT000082131.3':0.00210724,(('PDT000145352.3':0.00304183,(('PDT000120850.3':0.00187968,('PDT000083130.3':0.00149197,('PDT000240606.3':0.00124439,'PDT000547353.2':0.00155253)'':0.000307009)'':0.000214245)'':0.000113079,(('PDT000239596.3':0.0013661,'PDT000124931.3':0.0014325)'':0.00033843,('PDT000238401.3':0.00175755,('PDT000367204.2':0.0012294,'PDT000562718.2':0.00126835)'':0.000338525)'':0.00021316)'':5.55632e-05)'':0.000500986)'':-0.000349727,(('PDT000142627.3':0.00211818,('PDT000227563.3':0.00153038,'PDT000081120.3':0.0013673)'':0.000476738)'':9.75234e-05,('PDT000124873.3':0.00158405,('PDT000221961.3':0.00180096,('PDT000083150.3':0.00148143,('PDT000107698.3':0.00115637,('PDT000124802.3':0.00108383,('PDT000157502.3':0.00110332,('PDT000168315.3':0.000893162,'PDT000120912.3':0.000805818)'':9.53435e-05)'':0.000149494)'':0.000159859)'':0.000370313)'':0.000101083)'':0.000415353)'':-0.000109295)'':0.000163987)'':0.000360532)'':-0.000109862)'':0.000249622)'':0.000364819)'':-0.000357353)'':0.000461556)'':0.000369098)'':0.000119373)'':0.00034779)'':0.000102335)'':0.000180853)'':-0.000521527)'':0.00130986)'':-0.00164799)'':0.000187258)'':0.00132085)'':-0.001283)'':0.000672993)'':0.00232132)'':0.000134273)'':-0.00183062)'':0.000737104)'':0.00154153)'':-0.00124462)'':-0.000856)'':0.000298944)'':0.00266104)'':-0.00190755)'':0.0033422)'':-0.00331121)'':0.00551931)'':-0.00496973)'':-0.00070544)'':0.00273655)'':0.000132612)'':0.000642231)'':0.00183613)'':0.000880795)'':-0.00466068)'':0.00284573)'':-0.00262552)'':0.0007863)'':-2.93505e-05)'':-0.000371682)'':0.000831724)'':0.00288159)'':-0.000763162)'':-0.000611897)'':0.00288737)'':-0.0025966)'':0.00448324)'':-0.00345579)'':0.00412412)'':-2.25457e-06)'':-0.00199383)'':0.0155633)'':0.0333354)'':-0.0473746)'':0.0537377)'':0.0156974)'':0.0711731)'':0.0120787)'':-0.000937659,((('PDT000120873.2':0.0938791,'PDT000573885.1':0.0859849)'':0.0634551,(('PDT000144176.3':0.0228116,'PDT000372137.2':0.0137094)'':0.016678,(('PDT000661277.1':0.0168131,'PDT000498084.1':0.0173808)'':0.0046079,('PDT000315789.2':0.0184183,('PDT000672654.1':0.00732984,('PDT000375991.2':0.00441451,('PDT000366117.2':0.00332582,'PDT000563575.2':0.00325575)'':0.000661656)'':0.003632)'':0.00353608)'':0.011316)'':0.00506512)'':0.11977)'':0.0359591,(((('PDT000548532.2':0.0118082,'PDT000548602.2':0.0105199)'':0.0834745,('PDT000390574.1':0.105453,(('PDT000324068.1':0.0444673,'PDT000619589.2':0.0275658)'':0.0178357,(('PDT000161711.3':0.0208263,'PDT000332147.2':0.0196401)'':0.0160875,(('PDT000719051.1':0.0332666,('PDT000718978.1':0.0148023,'PDT000719109.1':0.0116098)'':0.0141054)'':0.00482143,('PDT000384975.1':0.0463965,('PDT000120932.2':0.0295731,('PDT000502388.1':0.0195758,('PDT000502458.1':0.0112022,('PDT000502352.1':0.00490115,'PDT000502464.1':0.00645345)'':0.00222337)'':0.00608572)'':0.0147747)'':0.00569764)'':0.00421546)'':0.00249085)'':0.00597133)'':0.0503739)'':0.00201245)'':0.0096833,(('PDT000410883.2':0.0290149,('PDT000708736.1':0.0208385,('PDT000288523.3':0.0227867,(('PDT000065377.1':0.0148252,('PDT000047686.4':0.0168625,'PDT000300585.1':0.0173483)'':0.00432596)'':0.000438916,('PDT000061193.3':0.0136197,('PDT000184374.3':0.0118166,('PDT000208159.3':0.0044206,'PDT000300537.1':0.0057621)'':0.00344283)'':0.00554344)'':0.00267127)'':0.00616057)'':0.00291513)'':0.00296299)'':0.0134314,(((('PDT000548258.2':0.0281525,'PDT000548383.2':0.0302395)'':0.00454335,('PDT000548386.2':0.0286909,('PDT000203386.3':0.0264742,('PDT000360370.2':0.0155588,'PDT000548384.2':0.0139604)'':0.00736382)'':0.00643056)'':-0.00044101)'':0.011846,(('PDT000619719.2':0.0399368,('PDT000161643.3':0.0371673,'PDT000190970.3':0.0302264)'':0.00418459)'':-0.000351282,(('PDT000107690.2':0.0208016,('PDT000040388.3':0.0215784,('PDT000384041.2':0.00573848,'PDT000497812.1':0.00513652)'':0.00807859)'':0.00421457)'':0.00985366,(('PDT000539009.2':0.0353487,('PDT000383990.2':0.0256656,('PDT000214022.3':0.0176606,'PDT000303570.3':0.0138954)'':0.00649038)'':0.00570938)'':0.000922783,('PDT000494940.2':0.0285406,('PDT000268720.3':0.0251081,((('PDT000061288.3':0.00833572,'PDT000131822.3':0.0124893)'':0.01183,('PDT000313235.3':0.020086,(('PDT000568859.2':0.00937215,('PDT000352807.2':0.00728456,('PDT000065241.3':0.00419604,('PDT000367234.2':0.00317199,'PDT000707473.1':0.0027133)'':0.000490812)'':0.00393129)'':0.00142925)'':0.00386669,(('PDT000524508.2':0.00816523,'PDT000248758.1':0.00732017)'':0.00300837,(('PDT000311420.3':0.00513271,'PDT000478268.2':0.00483241)'':0.00419487,('PDT000063274.3':0.00527099,'PDT000300582.1':0.00528891)'':0.00486572)'':0.000376492)'':0.0023662)'':0.00369439)'':0.0021838)'':-0.00295066,(('PDT000523418.2':0.0154321,('PDT000498118.1':0.00736186,('PDT000407928.2':0.00570781,('PDT000161580.3':0.00519932,'PDT000324130.2':0.00534478)'':0.000757343)'':0.00215741)'':0.00272371)'':0.000536773,(('PDT000384092.2':0.00950486,'PDT000141153.3':0.0101088)'':0.00992317,('PDT000638814.2':0.0181724,('PDT000329869.2':0.0187434,(('PDT000569430.2':0.00988947,'PDT000502349.1':0.0109645)'':0.00127143,('PDT000366178.2':0.0103296,('PDT000131033.3':0.00902252,('PDT000366255.2':0.00965805,('PDT000147501.3':0.00745588,('PDT000188037.3':0.00848908,('PDT000413935.2':0.00770548,('PDT000123347.1':0.00619605,('PDT000221756.3':0.00402918,('PDT000342355.2':0.00443168,'PDT000407530.2':0.00384184)'':5.40924e-05)'':0.00164802)'':0.000747794)'':0.000486332)'':0.000464104)'':0.000425969)'':0.000824225)'':0.000620956)'':-0.000484975)'':0.0060287)'':0.00107844)'':-0.00208425)'':-0.00142135)'':0.00318136)'':0.00355161)'':0.00185521)'':0.00270339)'':0.0063385)'':0.00531547)'':-0.00168535)'':-0.0076453,(('PDT000646238.2':0.0306681,('PDT000468292.2':0.0239753,('PDT000147480.3':0.0214289,'PDT000045588.1':0.0216999)'':0.000975909)'':0.0125633)'':0.00434632,(('PDT000492113.2':0.037537,(('PDT000383951.2':0.017695,('PDT000065221.3':0.0161323,'PDT000119753.3':0.0126433)'':0.00170117)'':0.00403093,('PDT000497898.1':0.0180384,(('PDT000394072.2':0.0186928,'PDT000399376.2':0.0134313)'':0.000871371,('PDT000146518.3':0.0243584,(('PDT000221732.3':0.014512,'PDT000431594.2':0.0125738)'':0.00194069,('PDT000329858.2':0.0113778,('PDT000243672.3':0.0114905,('PDT000251680.1':0.00521124,('PDT000201219.3':0.00706763,'PDT000574486.2':0.00588187)'':0.000570014)'':0.00334604)'':0.0033009)'':-0.000369647)'':0.00331125)'':-0.00133167)'':0.00269285)'':-0.000247211)'':0.0084924)'':-0.00388866,(('PDT000548385.2':0.0281643,('PDT000063278.3':0.0187514,('PDT000163035.4':0.0137743,'PDT000092875.1':0.0143989)'':0.00389199)'':0.00995931)'':-0.00127497,(('PDT000025425.1':0.0173884,('PDT000638820.2':0.01908,('PDT000107699.3':0.0125541,('PDT000550042.2':0.00980026,('PDT000146556.3':0.00804571,('PDT000151518.3':0.00643522,'PDT000394081.2':0.00709178)'':0.000510139)'':0.00259518)'':0.00118735)'':0.00367306)'':0.00231042)'':0.000234058,((('PDT000387736.2':0.0131711,'PDT000137524.3':0.0131837)'':0.00459677,('PDT000359294.2':0.0126398,('PDT000544625.2':0.00913547,'PDT000096846.3':0.0107773)'':0.00344813)'':0.000953837)'':0.00306072,(('PDT000063276.3':0.0132598,'PDT000707472.1':0.0186234)'':0.00261128,(('PDT000083988.3':0.0116253,'PDT000497900.1':0.00979541)'':0.00512381,((('PDT000366189.2':0.0063685,'PDT000383978.2':0.0068627)'':0.00184282,('PDT000325437.2':0.00894828,('PDT000580037.2':0.00732337,('PDT000168325.3':0.00761071,'PDT000665189.1':0.00719749)'':0.000959885)'':0.00113861)'':0.000711119)'':0.00346722,(('PDT000063277.3':0.0113245,('PDT000291454.3':0.00631807,('PDT000526243.2':0.00402267,'PDT000098745.3':0.00404506)'':0.00241951)'':0.00210535)'':0.001937,('PDT000300527.1':0.0174187,('PDT000295976.3':0.0118037,('PDT000102817.3':0.00646897,('PDT000317418.2':0.0061192,('PDT000410939.2':0.00404156,('PDT000451100.2':0.00337357,('PDT000052116.3':0.00386798,('PDT000163592.3':0.00363603,('PDT000356239.2':0.00228675,('PDT000148545.3':0.00200656,'PDT000307884.3':0.00248356)'':0.000406673)'':0.000287624)'':4.4858e-05)'':0.000840725)'':0.000385208)'':0.000215654)'':0.000746495)'':0.00414048)'':0.00227649)'':-0.00363604)'':0.00159824)'':0.00436402)'':0.00262448)'':-0.002043)'':2.14179e-05)'':0.0047137)'':9.39598e-05)'':0.00582725)'':-0.00238613)'':0.0169399)'':0.0754894)'':0.0309474,((('PDT000161519.3':0.0554992,(('PDT000047682.4':0.0345763,('PDT000576807.2':0.0277881,('PDT000248652.1':0.0251424,('PDT000316180.2':0.0315081,('PDT000147656.3':0.00923345,'PDT000501960.1':0.0104831)'':0.00864672)'':0.00358357)'':0.00785763)'':0.00892713)'':-0.00500735,(('PDT000465582.2':0.02273,('PDT000025428.1':0.015162,('PDT000147472.3':0.0103413,'PDT000051189.1':0.0103949)'':0.00562613)'':0.0062038)'':0.0040094,('PDT000410871.2':0.0353868,('PDT000069926.3':0.015379,('PDT000248528.1':0.0203317,('PDT000598446.2':0.0148787,('PDT000146462.3':0.00882408,'PDT000502901.1':0.00983042)'':0.00372846)'':0.00446036)'':0.0010255)'':0.0135472)'':-0.00399714)'':0.00743145)'':0.0184733)'':0.0399472,((('PDT000215112.3':0.0240456,('PDT000710904.1':0.0196206,('PDT000523330.2':0.00761307,'PDT000643156.2':0.00846953)'':0.00862596)'':0.00265456)'':0.0217231,(('PDT000362578.2':0.0168939,('PDT000421957.2':0.0107266,'PDT000502963.2':0.0104678)'':0.00431784)'':0.00590142,('PDT000523854.2':0.0107701,('PDT000147482.3':0.00778707,('PDT000257192.3':0.00917123,'PDT000555754.2':0.00699487)'':0.00192773)'':0.00391451)'':0.00419929)'':0.0184064)'':0.0649743,(('PDT000048674.3':0.00855074,'PDT000537155.2':0.0133056)'':0.0531083,((('PDT000351820.2':0.0170657,'PDT000492216.2':0.0172437)'':0.0265658,(('PDT000672403.1':0.0318575,('PDT000568979.2':0.0169254,('PDT000143063.3':0.0146567,'PDT000063272.3':0.0183407)'':0.00264714)'':0.0120783)'':0.000543855,('PDT000498085.1':0.029662,('PDT000492099.2':0.0231117,('PDT000650951.2':0.0151535,('PDT000312491.3':0.0115465,'PDT000049612.3':0.0124098)'':0.0038624)'':0.00385654)'':0.00499272)'':0.00223979)'':0.00629592)'':0.00565545,((('PDT000548306.2':0.0244941,'PDT000548315.2':0.0188853)'':0.00923927,(('PDT000598970.2':0.0232908,('PDT000411992.2':0.0200034,(('PDT000526043.2':0.00906114,'PDT000583303.2':0.0104709)'':0.0113297,('PDT000312468.3':0.02074,('PDT000481001.2':0.0192637,'PDT000669475.1':0.0126766)'':0.00164964)'':0.000130991)'':0.00790639)'':0.000963447)'':-0.00089569,('PDT000239349.1':0.0275217,('PDT000123325.1':0.0251895,('PDT000498128.1':0.0261791,(('PDT000579461.2':0.0147613,('PDT000213952.3':0.0123396,'PDT000275721.3':0.0137242)'':0.00177355)'':0.00272855,('PDT000540403.2':0.0193281,('PDT000148279.3':0.0111237,('PDT000161706.3':0.00957433,'PDT000333291.2':0.00975367)'':0.00202432)'':0.00567553)'':-0.00117033)'':0.00420255)'':0.00328231)'':0.000835213)'':-0.00207736)'':0.00637965)'':0.0112688,(('PDT000720366.1':0.0425265,('PDT000186920.3':0.0286548,('PDT000498005.1':0.0145475,('PDT000289015.3':0.0110711,'PDT000671483.1':0.0101212)'':0.00424708)'':0.018714)'':0.00233172)'':0.00220052,((('PDT000718576.1':0.0352594,('PDT000378604.2':0.0247123,'PDT000601939.2':0.0282071)'':0.00453761)'':7.97459e-05,((('PDT000342255.2':0.0147791,'PDT000276850.1':0.0129439)'':0.00337973,(('PDT000350574.2':0.0124481,'PDT000548305.2':0.0140184)'':0.00242257,('PDT000653300.2':0.0141766,('PDT000368363.2':0.0134317,('PDT000213232.3':0.00935906,'PDT000107472.3':0.00889124)'':0.00380836)'':0.00248643)'':0.000153306)'':0.00106207)'':0.00282636,(('PDT000143068.3':0.0214104,('PDT000525772.2':0.0223885,'PDT000548748.2':0.0208087)'':0.00264388)'':0.00387555,('PDT000562639.2':0.0249103,('PDT000163032.4':0.0137177,('PDT000247425.3':0.0161243,(('PDT000372193.2':0.00631893,'PDT000631695.2':0.00660357)'':0.0028457,('PDT000407643.2':0.00914032,'PDT000562842.2':0.00852018)'':0.0011125)'':0.00494994)'':0.00382548)'':0.00566595)'':-0.000515064)'':0.00136561)'':0.0134421)'':0.00335451,(('PDT000620936.2':0.0322717,'PDT000669812.1':0.037464)'':0.0061595,('PDT000551012.2':0.0416052,('PDT000300552.1':0.0298535,((('PDT000471143.2':0.022644,('PDT000273498.3':0.0169539,'PDT000384082.2':0.0156537)'':0.00306142)'':0.00331822,(('PDT000148512.3':0.0103716,'PDT000063179.3':0.00925763)'':0.00608271,('PDT000113776.3':0.0123928,('PDT000150704.3':0.007512,'PDT000366320.2':0.0079688)'':0.00269747)'':0.0031555)'':0.00389331)'':0.000664686,(('PDT000592442.2':0.0255438,'PDT000668169.2':0.0329283)'':0.00230984,('PDT000443409.2':0.0245336,(('PDT000710745.1':0.019103,('PDT000412102.2':0.0116021,'PDT000137199.3':0.0115456)'':0.00453294)'':0.00391148,('PDT000399408.2':0.0150199,('PDT000362227.2':0.0121416,'PDT000276848.1':0.0142213)'':0.00289576)'':0.00286344)'':0.00145274)'':0.00272451)'':-0.000356717)'':0.00643846)'':0.00796168)'':-0.00252318)'':0.00206054)'':0.00710572)'':-0.000938121)'':0.00028911)'':0.0288288)'':0.0361015)'':0.00309564)'':0.0259292,(('PDT000550767.1':0.173899,((('PDT000293431.1':0.0747584,('PDT000146500.3':0.0111532,('PDT000152943.3':0.00610798,'PDT000063165.3':0.00693202)'':0.00132716)'':0.0654867)'':0.00633837,('PDT000293432.1':0.0584361,('PDT000574473.1':0.0297117,('PDT000125001.3':0.0392281,(('PDT000300518.1':0.0156608,'PDT000300520.1':0.0169125)'':0.00644346,(('PDT000492221.2':0.0111324,'PDT000544784.2':0.0120336)'':0.00636287,('PDT000373242.1':0.0125867,('PDT000231291.3':0.0113035,('PDT000396197.1':0.0058047,('PDT000250209.1':0.00412752,'PDT000399315.1':0.00326213)'':0.00352363)'':0.0019754)'':0.000496881)'':0.00555327)'':0.000911532)'':0.00555496)'':0.00323232)'':0.0377646)'':0.0130792)'':0.000458036,(('PDT000289859.2':0.0203645,'PDT000252736.1':0.0147459)'':0.0470404,(('PDT000016361.2':0.0293064,'PDT000208702.1':0.0230811)'':0.0295156,((('PDT000307509.3':0.0142809,'PDT000248662.1':0.0200037)'':0.0144626,('PDT000368391.1':0.0296624,('PDT000104582.2':0.0127059,('PDT000120839.3':0.0078216,'PDT000032314.1':0.0081712)'':0.00741619)'':0.00820265)'':0.000248481)'':0.0118451,(('PDT000387580.1':0.0469375,('PDT000251993.1':0.0424831,('PDT000145533.3':0.0189689,('PDT000052315.1':0.021508,('PDT000294788.1':0.0263664,('PDT000151188.3':0.0151449,'PDT000050675.1':0.018673)'':0.00431258)'':0.0020896)'':0.00273414)'':0.0123406)'':0.00172536)'':-0.00393542,(('PDT000378791.2':0.0306062,'PDT000016368.2':0.0317416)'':0.00946144,(('PDT000157133.2':0.0208443,'PDT000157941.2':0.0220212)'':0.0066728,(('PDT000326183.1':0.0114745,'PDT000532068.2':0.0145047)'':0.00932356,('PDT000110496.3':0.0136487,'PDT000498177.1':0.0133276)'':0.0109343)'':0.00640809)'':0.0065813)'':0.00382382)'':0.0025726)'':0.0193463)'':0.0161479)'':0.00911015)'':0.0862935)'':-0.0648608,(('PDT000629979.1':0.0295719,'PDT000683161.1':0.0332614)'':0.074159,(((('PDT000168320.3':0.0108734,'PDT000298474.3':0.0117569)'':0.0160645,('PDT000562706.2':0.0306495,('PDT000084148.3':0.0246646,('PDT000440486.2':0.0340402,('PDT000376016.2':0.0289817,(('PDT000115008.3':0.0128515,('PDT000322449.2':0.0111419,('PDT000491385.2':0.00368869,'PDT000504883.2':0.00299527)'':0.0077411)'':0.000660405)'':0.00381891,(('PDT000085574.3':0.00789482,'PDT000707370.1':0.00798448)'':0.00441222,('PDT000300517.1':0.00888134,('PDT000706511.1':0.00824013,'PDT000105654.4':0.00659167)'':0.00270041)'':0.00146417)'':0.00251728)'':0.0115587)'':0.00192176)'':0.00170831)'':0.0012419)'':0.00128899)'':0.0104689,((('PDT000093886.1':0.0276047,'PDT000123361.1':0.0332115)'':0.00137506,(('PDT000306721.3':0.0221775,'PDT000611518.2':0.0271308)'':0.000914133,('PDT000123263.1':0.0263777,('PDT000086316.3':0.0201953,('PDT000485862.2':0.0210072,('PDT000311056.3':0.0185462,('PDT000502398.1':0.0154227,(('PDT000142617.3':0.00712404,'PDT000545558.2':0.00799586)'':0.00462552,('PDT000497924.1':0.0114726,('PDT000190948.3':0.0101639,('PDT000275668.3':0.007445,'PDT000600628.2':0.0070578)'':0.0019286)'':0.00249059)'':0.000937411)'':0.00120216)'':0.00132912)'':0.000714885)'':0.00385562)'':0.000708706)'':0.00182019)'':0.0037494)'':0.00336795,(('PDT000354941.2':0.0192345,'PDT000539925.2':0.0231175)'':0.0172144,('PDT000619732.2':0.0584066,(('PDT000160138.3':0.0382342,'PDT000093894.1':0.0334321)'':0.00237447,('PDT000548458.2':0.0305544,('PDT000319270.2':0.0378961,((('PDT000093889.1':0.0164703,'PDT000093893.1':0.015775)'':0.00517111,(('PDT000093884.1':0.0165887,('PDT000093882.1':0.0143144,('PDT000093887.1':0.0123179,'PDT000093888.1':0.0130174)'':0.00255258)'':0.000572882)'':0.00524038,('PDT000093883.1':0.0247417,(('PDT000494512.2':0.0101541,('PDT000147739.3':0.0108199,'PDT000161655.3':0.010745)'':0.00222784)'':0.00396162,('PDT000377118.2':0.013178,('PDT000147671.3':0.00932864,('PDT000161629.3':0.0085124,'PDT000161658.3':0.0082601)'':0.000971464)'':0.00557782)'':0.00381295)'':0.00534969)'':0.000194428)'':0.00209481)'':-0.00180586,((('PDT000542199.2':0.0101368,'PDT000672416.1':0.00948664)'':0.0078964,('PDT000248644.1':0.0134733,('PDT000707369.1':0.0111502,'PDT000008714.2':0.0131795)'':0.000779296)'':0.0086441)'':0.00657052,('PDT000093879.1':0.0266544,(('PDT000093890.1':0.0231722,('PDT000093891.1':0.0200373,'PDT000094869.1':0.0146602)'':0.00241964)'':-0.000300707,(('PDT000351786.2':0.0101824,('PDT000510666.2':0.0103435,'PDT000098734.3':0.0109628)'':0.0013791)'':0.00214126,(('PDT000239282.3':0.0188549,'PDT000497926.1':0.0164822)'':0.00192446,(('PDT000065224.3':0.00802559,'PDT000093892.1':0.00780951)'':0.00566568,('PDT000083235.3':0.0121403,('PDT000329243.2':0.0198843,('PDT000145375.3':0.0202726,(('PDT000375777.2':0.00657559,('PDT000139207.3':0.00626933,('PDT000216658.3':0.00510116,('PDT000163042.4':0.0041947,'PDT000083276.3':0.00417684)'':0.00136362)'':0.000659087)'':0.000429546)'':0.00200704,('PDT000587824.2':0.010641,('PDT000084078.3':0.0115264,((('PDT000253478.3':0.00584866,'PDT000359147.2':0.00511404)'':0.000606924,('PDT000296229.3':0.00447367,'PDT000468296.2':0.00439794)'':0.0019637)'':0.00117953,('PDT000084049.3':0.00689432,('PDT000150166.3':0.0100911,(('PDT000309599.3':0.00619319,'PDT000123297.1':0.00663881)'':0.000574341,('PDT000366206.2':0.00610317,('PDT000687695.1':0.00305647,('PDT000281554.3':0.00136439,'PDT000707431.1':0.00213107)'':0.00152774)'':0.0026686)'':-0.000132457)'':0.00130142)'':0.0010693)'':-0.00100076)'':0.00188818)'':0.00125815)'':-0.00208192)'':0.00452931)'':0.000710613)'':0.000435839)'':-0.00156736)'':0.00192877)'':-0.0010264)'':0.00418855)'':0.00732601)'':-0.00202876)'':-0.00311417)'':0.00724586)'':0.00303869)'':-0.00120979)'':0.00821835)'':-0.00494236)'':-0.00877684)'':0.00574757)'':0.00529531,((('PDT000146550.3':0.00333437,'PDT000086189.3':0.0039438)'':0.013931,('PDT000497540.2':0.0173546,'PDT000083247.3':0.0206401)'':0.00460234)'':0.0249492,((('PDT000269461.3':0.00883658,'PDT000293446.1':0.0112569)'':0.00115406,('PDT000362233.2':0.0178924,('PDT000497909.1':0.00911518,('PDT000544618.2':0.00919566,((('PDT000501467.2':0.00532092,('PDT000389750.2':0.00226673,'PDT000707676.1':0.00212405)'':0.00175059)'':0.000384914,(('PDT000243690.3':0.00219369,('PDT000190363.3':0.0019324,'PDT000193100.3':0.00206041)'':0.000501212)'':0.00111022,('PDT000119930.3':0.00334086,('PDT000491414.2':0.000671884,'PDT000707372.1':0.000527396)'':0.00214091)'':0.000366555)'':0.00106094)'':0.00150178,('PDT000133776.3':0.0074553,(('PDT000387695.2':0.00609496,('PDT000146573.3':0.00478138,('PDT000357243.2':0.00234414,'PDT000491413.2':0.00184957)'':0.00278411)'':0.000989711)'':0.000434029,(('PDT000412107.2':0.00457528,'PDT000421943.2':0.00448847)'':0.000573147,('PDT000300551.1':0.00536557,(('PDT000368379.2':0.00263194,'PDT000644519.2':0.00216183)'':0.00133029,('PDT000124944.3':0.00339159,('PDT000085579.3':0.00292272,(('PDT000547691.2':0.00203317,('PDT000555903.2':0.00161577,('PDT000208182.3':0.00223819,(('PDT000283372.3':0.000896783,'PDT000583099.2':0.000702257)'':0.00040244,('PDT000443271.2':0.000671556,'PDT000549485.2':0.000727744)'':0.000546245)'':0.000384793)'':0.000346288)'':0.000406603)'':-0.000607641,('PDT000137528.3':0.00149986,('PDT000124937.3':0.00114193,('PDT000533757.2':0.00173742,('PDT000491389.2':0.00042057,('PDT000707375.1':0.000304465,('PDT000150922.3':0.000293788,'PDT000150927.3':0.000206112)'':0.00019532)'':8.07016e-05)'':0.000720203)'':0.000240862)'':0.000325232)'':0.000111948)'':0.00102046)'':0.000641322)'':-0.00032897)'':0.0029581)'':-0.000421362)'':-0.000796191)'':0.00341506)'':-0.00202782)'':0.00268387)'':0.000834622)'':0.00756366)'':-0.00570799)'':0.0168996,((('PDT000579375.2':0.00647523,('PDT000333940.2':0.00470244,'PDT000115417.3':0.00446547)'':0.00286041)'':0.00818062,('PDT000093885.1':0.022708,('PDT000131997.3':0.0129332,('PDT000333928.2':0.00697158,('PDT000378580.2':0.00412478,'PDT000131882.3':0.0038505)'':0.00265618)'':0.00237802)'':0.00557713)'':-0.00200749)'':0.0175268,((('PDT000523241.2':0.00412242,'PDT000712266.1':0.00516365)'':0.0255606,('PDT000522464.2':0.0254745,(('PDT000366990.2':0.0103763,('PDT000299809.3':0.0101883,'PDT000351805.2':0.00933404)'':0.00177337)'':0.00256102,(('PDT000128468.3':0.00997745,('PDT000547696.2':0.00494086,('PDT000485858.2':0.00244938,'PDT000498981.2':0.00234439)'':0.00311326)'':0.00619014)'':0.000891503,(('PDT000146068.3':0.0100359,'PDT000580716.2':0.00753951)'':0.00460844,('PDT000352827.2':0.0204632,('PDT000160287.3':0.0212014,(('PDT000119987.3':0.00605196,('PDT000190675.3':0.00309167,'PDT000084181.3':0.00388739)'':0.00320806)'':0.00236299,('PDT000334541.2':0.0219962,('PDT000602435.2':0.00915531,('PDT000137569.3':0.0137408,('PDT000151500.3':0.0095505,('PDT000718817.1':0.00628627,('PDT000587163.2':0.0104841,(('PDT000426880.2':0.00456185,('PDT000146532.3':0.00425214,'PDT000663847.1':0.00422075)'':0.000226108)'':0.000794623,('PDT000635709.2':0.00468739,('PDT000139231.3':0.00383861,('PDT000201213.3':0.00265874,'PDT000137650.3':0.00292785)'':0.00104841)'':0.000714337)'':0.0004958)'':0.0014064)'':0.000566184)'':0.00267638)'':0.00079754)'':0.000427748)'':0.00199614)'':-0.00304838)'':0.00430745)'':0.00191767)'':-0.00204115)'':-0.00133533)'':0.00141517)'':0.00652202)'':0.00026243)'':-0.00241506,((('PDT000399402.2':0.0202343,('PDT000589726.2':0.00739684,'PDT000707427.1':0.00790826)'':0.00599473)'':0.00240928,('PDT000623026.2':0.0156318,((('PDT000266780.3':0.00390305,('PDT000303463.3':0.00234533,'PDT000521285.2':0.00314534)'':0.000976421)'':0.00243769,('PDT000066849.3':0.00429353,('PDT000503053.2':0.00224849,'PDT000547649.2':0.00254384)'':0.000596168)'':0.00116576)'':0.00657917,('PDT000549252.2':0.0133023,(('PDT000242825.3':0.00673436,'PDT000394047.2':0.00838854)'':0.00506393,('PDT000113658.3':0.00803349,(('PDT000237122.3':0.00285081,'PDT000542195.2':0.00284179)'':0.00329895,('PDT000222768.3':0.00724801,(('PDT000400850.2':0.00564455,'PDT000101014.3':0.00510815)'':0.000629987,('PDT000176771.3':0.00532651,('PDT000342387.2':0.00500004,('PDT000274049.3':0.00298424,'PDT000625725.2':0.003999)'':0.00116647)'':0.00127165)'':0.000740864)'':0.00114039)'':-0.00012877)'':0.00320117)'':0.00227405)'':0.00253556)'':-0.00334534)'':0.0047607)'':-0.00210346)'':0.00248057,('PDT000497770.1':0.0350638,(('PDT000375973.2':0.0219344,('PDT000385919.2':0.00884952,('PDT000497860.1':0.00289331,'PDT000497863.1':0.00269384)'':0.00660816)'':0.00397465)'':0.000401086,('PDT000084155.3':0.0349102,('PDT000296021.3':0.0324211,('PDT000499182.2':0.0166819,(('PDT000486048.2':0.00679446,'PDT000546295.2':0.00713034)'':0.00484427,(('PDT000226805.3':0.0105413,'PDT000125514.3':0.0116194)'':0.00661555,('PDT000300564.1':0.00968922,((('PDT000312470.3':0.00438244,('PDT000106997.3':0.00369143,('PDT000227578.3':0.00303559,('PDT000235044.3':0.00230925,'PDT000128441.3':0.00267878)'':0.000455724)'':0.000738199)'':0.00115811)'':-0.000648567,('PDT000344435.2':0.00328209,('PDT000221705.3':0.00271905,('PDT000083249.3':0.00284526,('PDT000203882.3':0.00214262,'PDT000533750.2':0.00295279)'':0.000544175)'':0.0001239)'':0.000668544)'':0.000566391)'':0.00221404,(('PDT000144021.3':0.00480348,'PDT000373174.2':0.00455721)'':6.32163e-05,('PDT000542187.2':0.00536436,(('PDT000501774.2':0.00197329,'PDT000547084.2':0.00182178)'':0.00190676,(('PDT000359169.2':0.00233924,'PDT000637493.2':0.00175421)'':0.000520576,('PDT000583352.2':0.00341378,('PDT000281745.3':0.0022064,('PDT000221995.3':0.0017323,'PDT000106999.3':0.00226291)'':0.000488209)'':0.000515351)'':-2.49689e-05)'':0.000710913)'':0.00103936)'':0.000804053)'':0.00101378)'':0.00261015)'':0.00207254)'':0.000866899)'':0.00663)'':0.00500061)'':0.000889761)'':-0.0107521)'':0.0128014)'':-0.00878866)'':0.00191099)'':0.011655)'':0.00250139)'':0.0039452)'':-0.000603476)'':0.0725278)'':-0.00190553)'':0.0153389)'':0.025904)'':0.0285122)'':0.00287941)'':0.0165153)'':-0.00788501)'':0.00463241,((((('PDT000341790.1':0.0804599,'PDT000095292.1':0.0693131)'':0.0578387,('PDT000548403.2':0.0760433,'PDT000627568.2':0.0838027)'':0.0316888)'':0.0321238,(('PDT000161694.2':0.0824621,('PDT000016391.2':0.0678047,('PDT000110451.2':0.0609795,((('PDT000107500.3':0.0286739,'PDT000016395.2':0.0295549)'':0.00608532,('PDT000157154.2':0.0415361,('PDT000394287.2':0.024708,('PDT000492222.2':0.0191953,('PDT000615215.2':0.0159986,('PDT000625416.1':0.0104829,'PDT000553172.1':0.0100569)'':0.00301434)'':0.00412219)'':0.00170594)'':0.00519)'':-0.000719425)'':0.0218374,(('PDT000576190.1':0.0328908,('PDT000161546.2':0.0285388,'PDT000707409.1':0.0265142)'':0.00618906)'':0.00184814,(('PDT000619761.1':0.0265278,'PDT000252542.1':0.0314913)'':0.0130508,('PDT000553123.1':0.0410503,(('PDT000132117.2':0.0181553,'PDT000498147.1':0.0192868)'':0.00806989,('PDT000155665.2':0.0320002,('PDT000568561.1':0.0183164,'PDT000625387.1':0.0168872)'':0.00846562)'':0.00174251)'':0.00589844)'':8.69484e-05)'':0.00162023)'':0.00478995)'':0.0104779)'':0.0193783)'':0.0112583)'':0.00570047,((('PDT000653459.1':0.0506075,('PDT000557124.1':0.034997,'PDT000251758.1':0.0469846)'':0.00963021)'':0.0159385,(('PDT000631999.1':0.0453175,'PDT000661672.1':0.0486298)'':0.0118552,('PDT000502582.1':0.057245,(('PDT000103578.2':0.0287258,'PDT000502513.1':0.0356297)'':0.00244237,('PDT000502364.1':0.0231437,'PDT000502502.1':0.0240335)'':0.00908248)'':0.0230666)'':0.00103165)'':0.00186022)'':0.0075576,((('PDT000502504.1':0.0539511,('PDT000339357.1':0.0340999,'PDT000509492.1':0.0387298)'':0.0084515)'':0.00845952,('PDT000547037.1':0.0916663,(('PDT000048126.1':0.0537752,('PDT000332418.1':0.0223109,'PDT000095700.2':0.0217372)'':0.0176609)'':0.000871588,('PDT000266389.2':0.0398324,(('PDT000286928.2':0.0304968,'PDT000105653.3':0.0362219)'':0.00652165,('PDT000375002.1':0.0260813,'PDT000455212.1':0.0260611)'':0.0130223)'':0.00252619)'':0.00274787)'':0.0125056)'':-0.00164695)'':0.00493196,(('PDT000360268.1':0.0393529,('PDT000517924.1':0.0205408,'PDT000502390.1':0.0151522)'':0.0105785)'':0.0147125,(('PDT000417485.1':0.0395469,'PDT000502572.1':0.0366242)'':0.00759009,('PDT000619580.1':0.0567533,('PDT000150870.1':0.0562965,('PDT000390718.1':0.047049,(('PDT000197017.1':0.0179859,('PDT000123480.2':0.0155593,'PDT000502461.1':0.0247192)'':0.00763138)'':0.00904339,(('PDT000502270.1':0.0177046,'PDT000502472.1':0.019107)'':0.00624654,('PDT000502670.1':0.0263962,('PDT000502516.1':0.0158238,'PDT000345944.1':0.0177631)'':0.0112072)'':0.00416419)'':0.00397035)'':0.00384663)'':0.00130928)'':0.00336469)'':0.00481618)'':0.000623891)'':0.00297798)'':0.0191712)'':0.0199408)'':0.0436423)'':0.0286528,(('PDT000132003.2':0.193071,('PDT000616943.1':0.017862,(('PDT000155708.2':0.00608649,'PDT000158735.2':0.00594491)'':0.00635218,('PDT000125472.2':0.0144531,'PDT000720396.1':0.0103082)'':0.00459777)'':0.00446392)'':0.164533)'':0.0155801,((('PDT000390632.1':0.013703,'PDT000714575.1':0.028276)'':0.102449,(('PDT000104792.2':0.0558284,('PDT000156819.2':0.0181477,'PDT000060838.2':0.0189949)'':0.0472528)'':0.0268123,(('PDT000553141.1':0.0743756,('PDT000161713.2':0.0351319,'PDT000011775.2':0.0408077)'':0.0167281)'':0.00857001,(('PDT000016381.2':0.019273,('PDT000426883.1':0.0212666,'PDT000248715.1':0.0152789)'':0.00796347)'':0.00887047,((('PDT000522389.1':0.0118106,'PDT000357696.1':0.016758)'':0.0066966,('PDT000579616.1':0.0139527,('PDT000466053.1':0.0108422,'PDT000466062.1':0.0137634)'':0.00238664)'':0.00803507)'':0.00232514,('PDT000291150.2':0.0139384,(('PDT000251952.1':0.00621992,('PDT000553185.1':0.00603575,'PDT000016390.2':0.00471585)'':0.000937229)'':0.00110692,('PDT000366998.1':0.00547164,('PDT000599902.1':0.00114388,'PDT000604152.1':0.00155396)'':0.00389679)'':0.000402457)'':0.00552383)'':0.00648571)'':0.00628886)'':0.0383543)'':0.0148541)'':0.043194)'':-0.0168579,(('PDT000372047.1':0.0615327,'PDT000390820.1':0.0458983)'':0.0371609,((('PDT000248720.1':0.0563465,('PDT000132009.2':0.0494711,'PDT000600094.1':0.0574869)'':0.00971958)'':0.000376174,(('PDT000316626.1':0.0418214,'PDT000314219.2':0.0436314)'':0.0015976,('PDT000410738.1':0.0373237,('PDT000502974.1':0.0193437,('PDT000553181.1':0.0162292,('PDT000156994.2':0.0184274,'PDT000161866.2':0.0153757)'':0.00165817)'':0.00209401)'':0.0121313)'':0.0129973)'':0.0108934)'':0.0131069,(('PDT000619832.1':0.0712423,('PDT000131834.2':0.0392999,'PDT000383187.1':0.0360638)'':0.0351414)'':0.00997021,((('PDT000084339.1':0.0872604,(('PDT000671994.1':0.0422371,'PDT000706489.1':0.0327751)'':0.00457635,('PDT000318625.1':0.0442797,('PDT000156513.2':0.0357712,'PDT000636722.1':0.0395487)'':0.0063044)'':0.00311225)'':0.037505)'':-0.00386425,(('PDT000120933.2':0.0322095,('PDT000253131.2':0.0205095,'PDT000120906.2':0.0128569)'':0.0103408)'':0.000552183,(('PDT000318890.1':0.025849,'PDT000651398.1':0.0235795)'':0.00592285,(('PDT000489900.1':0.016802,('PDT000131840.2':0.0123537,'PDT000132309.2':0.0149002)'':0.00408769)'':0.00215642,('PDT000349632.1':0.0193329,('PDT000132168.2':0.0168662,'PDT000414590.1':0.0177318)'':0.00240018)'':0.00390528)'':0.00374305)'':0.00126274)'':0.0359722)'':0.00442164,(('PDT000553212.1':0.0500832,('PDT000124819.2':0.0469743,'PDT000546869.1':0.0392107)'':0.0165328)'':0.00538064,(('PDT000578365.2':0.0324494,'PDT000683392.1':0.0254816)'':0.0336204,(('PDT000619762.1':0.0394177,('PDT000005934.3':0.0264724,'PDT000248747.1':0.0242288)'':0.0266655)'':0.000819554,('PDT000322980.1':0.0566312,(('PDT000625446.1':0.0220563,'PDT000083541.1':0.025146)'':0.0209436,(('PDT000625390.1':0.0267433,('PDT000651767.1':0.0307205,('PDT000132001.2':0.0119156,'PDT000645783.1':0.0196226)'':0.00865443)'':0.00813143)'':0.00535057,('PDT000553147.1':0.0352733,('PDT000083497.1':0.0144443,('PDT000625454.1':0.0107653,('PDT000711107.1':0.0124067,'PDT000081288.1':0.0114358)'':0.000563966)'':0.00321617)'':0.0175137)'':0.0028405)'':0.00586598)'':0.00670079)'':0.00384253)'':0.00334383)'':0.00431448)'':0.00518077)'':0.00636104)'':-0.000432459)'':0.0248345)'':0.0236556)'':0.0619426)'':-0.00765601)'':0.0025184,(((('PDT000373254.1':0.18228,'PDT000228227.1':0.18319)'':0.0048572,((('PDT000520717.1':0.0587128,('PDT000390714.1':0.0442022,('PDT000651507.1':0.0408407,'PDT000710220.1':0.0432687)'':0.00114157)'':0.0255932)'':0.0454683,('PDT000721099.1':0.143579,(('PDT000502662.1':0.0971304,('PDT000501948.1':0.0797227,(('PDT000292820.2':0.0476744,'PDT000565997.1':0.0461926)'':0.0059543,('PDT000310462.2':0.0231504,'PDT000095730.2':0.026928)'':0.0298599)'':0.0230647)'':0.00573538)'':0.00279607,(('PDT000502910.1':0.114957,('PDT000477120.1':0.0484191,'PDT000518006.1':0.0632379)'':0.0565893)'':-0.000723511,(('PDT000524055.1':0.0214759,('PDT000524079.1':0.0151909,'PDT000524094.1':0.0145963)'':0.00375752)'':0.00321547,('PDT000243054.1':0.0182733,('PDT000524080.1':0.00812585,('PDT000524049.1':0.00342994,'PDT000524107.1':0.00474309)'':0.00371793)'':0.0121846)'':4.7646e-05)'':0.0829486)'':0.00885521)'':0.027303)'':-0.0049379)'':0.0281323,(((('PDT000190952.3':0.0372797,'PDT000212417.3':0.0371371)'':0.0157139,('PDT000280673.3':0.0666081,('PDT000498883.2':0.0357556,'PDT000584546.2':0.0400064)'':0.0218989)'':0.00107202)'':0.0817622,(('PDT000248773.1':0.0761956,'PDT000017739.2':0.0759024)'':0.0159014,(('PDT000095830.2':0.0378031,'PDT000517942.1':0.0473385)'':0.0462057,('PDT000654619.1':0.0646211,('PDT000223034.2':0.0484052,'PDT000258448.2':0.0519768)'':0.00590736)'':0.0273217)'':0.0070604)'':0.0184841)'':0.0135227,(('PDT000711738.1':0.0934994,('PDT000306635.2':0.0816042,('PDT000271523.2':0.06928,(('PDT000312784.2':0.03183,('PDT000594475.1':0.0340855,'PDT000714572.1':0.0303147)'':0.00767297)'':0.0173213,('PDT000095714.2':0.0541306,(('PDT000150232.2':0.0409942,'PDT000181785.2':0.0510777)'':0.00125779,('PDT000588647.1':0.0336251,('PDT000502683.1':0.0239641,'PDT000502882.1':0.0272224)'':0.0122695)'':0.00614603)'':0.00605244)'':0.000105701)'':0.00666423)'':0.00579496)'':0.0182286)'':0.0334219,((('PDT000502686.1':0.03654,'PDT000228025.1':0.0264947)'':0.030708,(('PDT000644655.1':0.0300459,'PDT000502888.1':0.0340467)'':0.0225769,('PDT000109431.2':0.0342052,('PDT000372319.1':0.026249,'PDT000318992.1':0.0298705)'':0.00777671)'':0.018554)'':0.00760759)'':0.0149965,('PDT000557036.1':0.0599176,(('PDT000142168.2':0.0397809,('PDT000353631.1':0.0367638,('PDT000109316.2':0.0327454,('PDT000290408.2':0.021932,'PDT000468340.1':0.0229712)'':0.00891911)'':0.00305176)'':0.00131981)'':0.000843037,(('PDT000196603.2':0.0152714,'PDT000408774.1':0.0211967)'':0.0092004,(('PDT000362850.1':0.0120058,'PDT000162108.1':0.0129744)'':0.00817273,('PDT000494421.1':0.0130434,('PDT000359342.1':0.0104113,'PDT000468324.1':0.0127543)'':0.00237945)'':0.00490829)'':0.00966501)'':0.0117424)'':0.0137705)'':0.0201328)'':0.0404432)'':0.00214527)'':0.00463993)'':0.0254634)'':-0.00608627,(((('PDT000375001.1':0.0768917,'PDT000687855.1':0.0857773)'':0.019668,('PDT000236053.3':0.0177984,'PDT000382557.1':0.0350382)'':0.0868422)'':0.045944,((('PDT000290443.2':0.038445,('PDT000394993.1':0.0396784,'PDT000228145.1':0.0261409)'':0.00594525)'':0.00719567,(('PDT000628866.1':0.00893112,'PDT000645120.1':0.0202426)'':0.0237531,(('PDT000332419.1':0.0298649,'PDT000251827.1':0.0357956)'':0.000604024,('PDT000149339.2':0.0363598,(('PDT000129883.2':0.0184804,('PDT000532411.1':0.0137235,'PDT000395161.1':0.00811542)'':0.00705494)'':0.0100104,('PDT000601907.1':0.0218013,('PDT000415762.1':0.0188031,'PDT000556876.1':0.0255908)'':0.00781833)'':0.00326089)'':0.00287683)'':0.000519744)'':0.00201407)'':0.00740366)'':0.0154127,(('PDT000107795.2':0.0323875,('PDT000366111.1':0.0411184,('PDT000410864.1':0.0386906,'PDT000551014.1':0.0343376)'':0.00686239)'':0.0073233)'':0.030468,('PDT000228121.1':0.0743536,(('PDT000092544.1':0.0386952,('PDT000215939.1':0.0293514,'PDT000482862.1':0.0351962)'':0.00627337)'':-0.00104584,(('PDT000627594.1':0.0412953,('PDT000362873.1':0.0332824,('PDT000408755.1':0.0227985,('PDT000095687.2':0.0139917,'PDT000129816.2':0.00791189)'':0.00745425)'':0.00565295)'':0.00918094)'':-0.000306041,('PDT000687812.1':0.0351936,('PDT000636295.1':0.0417579,('PDT000495100.1':0.0283867,('PDT000364173.1':0.0225248,('PDT000149359.2':0.0123902,'PDT000252637.1':0.0127409)'':0.00760633)'':0.00319955)'':0.00324673)'':0.00295686)'':0.00186404)'':0.0022937)'':0.013198)'':0.00955174)'':-0.00564871)'':0.0749322)'':-0.0120748,(((((('PDT000321542.1':0.0365159,'PDT000083573.1':0.0323429)'':0.029328,(('PDT000157965.2':0.0245326,'PDT000081304.1':0.0214478)'':0.0267339,('PDT000081283.1':0.0287977,'PDT000101612.1':0.0225022)'':0.026795)'':0.00666099)'':0.0277465,('PDT000524028.1':0.0695961,(('PDT000571828.1':0.0546723,'PDT000357645.1':0.0570877)'':0.00757012,(('PDT000047718.3':0.0283018,'PDT000009769.2':0.0245054)'':0.0199442,('PDT000030349.1':0.0513006,'PDT000191937.1':0.0372447)'':0.00584465)'':0.0121547)'':0.0117097)'':-0.00343056)'':0.0070102,((('PDT000008478.2':0.0475611,('PDT000083487.1':0.0543205,'PDT000248731.1':0.0508575)'':0.00569287)'':0.00824767,(('PDT000030355.1':0.0441951,'PDT000081303.1':0.0351162)'':0.0265645,('PDT000213268.2':0.0612524,('PDT000082642.1':0.0133486,('PDT000083523.1':0.00122728,'PDT000083524.1':0.00147137)'':0.0172427)'':0.0280731)'':0.00351314)'':0.0069372)'':0.0154591,('PDT000357586.1':0.0702085,(('PDT000009382.2':0.0388514,('PDT000009378.2':0.021573,'PDT000357720.1':0.0389185)'':0.0219404)'':0.00150305,(('PDT000009379.2':0.035541,'PDT000009381.2':0.0340589)'':0.0118414,(('PDT000009380.2':0.0344667,'PDT000249875.1':0.0227916)'':0.0133448,(('PDT000345830.1':0.0474428,('PDT000051173.1':0.0163622,('PDT000485903.1':0.0114032,'PDT000357686.1':0.022662)'':0.0139244)'':0.0129449)'':0.00803198,('PDT000010608.3':0.0590817,('PDT000081289.1':0.0265529,('PDT000030352.1':0.0111129,('PDT000030351.1':0.00934602,('PDT000030350.1':0.00917501,'PDT000081271.1':0.00917089)'':0.000667584)'':0.00354093)'':0.0156493)'':0.017651)'':-0.00274395)'':0.00207245)'':0.00288327)'':0.000663995)'':0.0247138)'':0.00787961)'':0.000735093)'':0.0311564,((('PDT000123478.2':0.0370002,'PDT000259787.3':0.0456766)'':0.062637,(('PDT000123271.1':0.0756431,('PDT000168476.1':0.0583081,('PDT000107686.2':0.0501575,'PDT000168485.1':0.0522675)'':0.00865742)'':0.0036203)'':0.00249125,(('PDT000107955.3':0.0396489,'PDT000168484.1':0.0466598)'':0.036692,(('PDT000498048.1':0.0449627,('PDT000249001.3':0.0409188,'PDT000168483.1':0.0407727)'':0.0113677)'':0.00977142,(('PDT000008770.3':0.029895,'PDT000107655.2':0.036303)'':0.0137648,(('PDT000552689.2':0.0385324,('PDT000497865.1':0.0178784,'PDT000498164.1':0.0237554)'':0.0218746)'':0.00790294,(('PDT000136995.3':0.0342857,'PDT000497922.1':0.0235354)'':0.0033778,('PDT000252674.1':0.0294882,(('PDT000133950.1':0.0137893,'PDT000497867.1':0.0126935)'':0.00539205,('PDT000125532.3':0.0159328,('PDT000498028.1':0.00403497,'PDT000498030.1':0.00614363)'':0.0107309)'':0.00624973)'':0.00358068)'':0.00638076)'':0.0148395)'':0.00581081)'':0.00862197)'':0.00465966)'':-0.00221487)'':0.0440824)'':0.00846159,((('PDT000498132.1':0.0674099,('PDT000181803.2':0.0767611,('PDT000600191.2':0.0566593,('PDT000481331.2':0.0563321,('PDT000352099.1':0.026354,('PDT000303473.3':0.022639,'PDT000387733.2':0.0138364)'':0.0106217)'':0.00681277)'':0.00823749)'':0.019598)'':0.00745663)'':0.0367672,(('PDT000103590.2':0.0669278,(('PDT000646107.1':0.0239662,'PDT000016363.2':0.0227728)'':0.0324426,(('PDT000190950.2':0.0109195,'PDT000136733.3':0.0192082)'':0.0234254,('PDT000553209.1':0.0225515,'PDT000016371.2':0.0235572)'':0.0081764)'':0.0215559)'':0.00969007)'':0.0113515,(('PDT000396799.1':0.0302828,'PDT000005933.2':0.0327723)'':0.0612184,(('PDT000719090.1':0.0680125,'PDT000228154.1':0.0788535)'':0.00888471,('PDT000345926.1':0.102435,('PDT000519803.1':0.0485774,('PDT000313332.2':0.0422062,('PDT000364403.1':0.0310527,'PDT000475530.1':0.0208723)'':0.0262017)'':0.00707433)'':0.0318154)'':-0.00332516)'':0.00946055)'':0.00456963)'':0.00772091)'':-0.00200013,(('PDT000619806.1':0.0787698,(('PDT000009376.2':0.0223507,'PDT000009377.2':0.0305823)'':0.0440338,('PDT000553170.1':0.0408152,('PDT000040450.3':0.0305126,('PDT000120875.2':0.02136,'PDT000016373.2':0.0142127)'':0.00855448)'':0.0203501)'':0.0209341)'':0.0107941)'':0.00535617,((('PDT000719286.1':0.0644451,'PDT000250039.1':0.0528579)'':0.00852538,('PDT000008727.2':0.0504658,('PDT000250031.1':0.0372394,('PDT000718828.1':0.0154765,('PDT000396611.1':0.00690094,'PDT000252546.1':0.0105589)'':0.013414)'':0.0147303)'':0.00484306)'':0.0435358)'':0.014124,(('PDT000105650.3':0.0875003,((('PDT000103598.2':0.00790131,'PDT000691365.1':0.00756619)'':0.0252946,('PDT000502280.1':0.02967,'PDT000502476.1':0.0241671)'':0.00401087)'':0.0107664,(('PDT000104581.2':0.033956,('PDT000009374.2':0.0177451,'PDT000251889.1':0.017674)'':0.00593247)'':0.0057466,(('PDT000490182.1':0.0224657,'PDT000502626.1':0.0219809)'':0.0241318,(('PDT000271556.2':0.00956712,'PDT000333058.1':0.0101828)'':0.0116816,('PDT000362823.1':0.0143328,'PDT000502927.1':0.0171492)'':0.00307175)'':0.00675071)'':0.00758162)'':0.00810093)'':0.0120709)'':0.0059739,(('PDT000625377.1':0.0603479,(('PDT000150258.2':0.0553584,'PDT000396779.1':0.0467216)'':0.00190299,('PDT000502383.1':0.0496829,('PDT000218515.2':0.0561411,('PDT000228307.1':0.0306582,('PDT000104685.2':0.0288049,'PDT000105657.3':0.0348714)'':0.00577223)'':0.00548883)'':0.00599534)'':0.00637575)'':0.00878047)'':-0.00155826,(('PDT000396793.1':0.0477648,'PDT000625413.1':0.0529152)'':0.0062044,((('PDT000720310.1':0.0217534,'PDT000243046.1':0.0347404)'':0.0324934,('PDT000009373.2':0.0313321,(('PDT000539887.1':0.0195937,'PDT000016400.2':0.0204377)'':0.00701168,('PDT000161644.2':0.0258602,('PDT000016392.2':0.0149938,('PDT000557174.1':0.0107617,'PDT000651937.1':0.00929395)'':0.00524965)'':0.0111121)'':0.000504126)'':0.00236352)'':0.0122928)'':0.00391124,(('PDT000463675.1':0.0080746,'PDT000463678.1':0.00731)'':0.049869,(('PDT000413205.1':0.0315729,('PDT000199181.2':0.0179189,'PDT000171794.1':0.0198151)'':0.00914443)'':0.00592686,('PDT000519815.1':0.0633491,(('PDT000341202.1':0.0219187,'PDT000357244.1':0.0339219)'':0.00760127,(('PDT000390976.1':0.0176697,'PDT000502585.1':0.0118209)'':0.0111189,('PDT000625472.1':0.0208946,('PDT000342667.1':0.0196992,('PDT000557039.1':0.016322,('PDT000502588.1':0.0126415,('PDT000103584.2':0.00539563,'PDT000566179.1':0.00906387)'':0.00296231)'':0.00139496)'':0.0055525)'':0.00217633)'':0.00349566)'':0.00412794)'':0.0098432)'':-0.00547064)'':0.00955263)'':0.00693552)'':0.0069364)'':0.00110197)'':0.000139219)'':0.0171223)'':0.00325406)'':0.00362962)'':0.0109582)'':0.000287558)'':0.000132054,(((('PDT000691316.1':0.04201,('PDT000539903.1':0.0310784,(('PDT000065389.1':0.0112621,'PDT000191166.1':0.00835365)'':0.0209456,(('PDT000595664.1':0.0144446,'PDT000510679.2':0.0119391)'':0.0104426,('PDT000691366.1':0.0205984,('PDT000191162.1':0.0168375,'PDT000191163.1':0.0179571)'':0.0032823)'':0.0071071)'':0.00658053)'':0.00969681)'':0.00643394)'':-0.00123956,(('PDT000191170.1':0.0244883,'PDT000191171.1':0.0272796)'':0.00277402,(('PDT000162342.3':0.0197108,('PDT000502641.1':0.0220451,('PDT000191164.1':0.0121874,'PDT000191176.1':0.0148263)'':0.00276569)'':0.00664432)'':-0.00127354,('PDT000191165.1':0.0235341,('PDT000191172.1':0.0147687,('PDT000691370.1':0.0160814,'PDT000191168.1':0.0160875)'':0.00179696)'':0.00475987)'':0.00313656)'':0.0054173)'':0.0129893)'':0.0279839,(((('PDT000619673.1':0.0387955,('PDT000550350.1':0.027657,'PDT000600799.1':0.043433)'':0.00416275)'':0.0249662,(('PDT000161531.2':0.0583939,('PDT000016397.2':0.0441674,('PDT000502962.2':0.0483684,'PDT000083495.1':0.0302368)'':0.0137289)'':0.00777364)'':0.00287452,((('PDT000539878.1':0.0405933,'PDT000102806.3':0.0403188)'':0.00359927,('PDT000359248.2':0.0102892,'PDT000114521.3':0.0101617)'':0.0360012)'':0.00222439,('PDT000584744.1':0.0784263,('PDT000558595.2':0.0355313,('PDT000120855.2':0.0393362,('PDT000161551.2':0.020001,('PDT000501946.1':0.0123892,('PDT000556966.1':0.00985984,'PDT000251786.1':0.00600366)'':0.00607294)'':0.00407806)'':0.0153775)'':0.00205741)'':0.0195229)'':-0.00698795)'':0.0055072)'':0.0017972)'':-0.00196056,(('PDT000619774.1':0.0392067,(('PDT000619773.2':0.0304073,('PDT000497899.1':0.0200328,'PDT000498182.1':0.0213316)'':0.00762582)'':0.00137936,(('PDT000249870.1':0.0152346,'PDT000250250.1':0.00972564)'':0.00598732,('PDT000142536.3':0.0199983,('PDT000443364.2':0.0190202,('PDT000492070.2':0.0103757,('PDT000491953.2':0.012729,('PDT000146466.3':0.0073962,'PDT000707474.1':0.0084926)'':0.00678631)'':0.00199131)'':0.00252518)'':0.00264921)'':0.00016341)'':0.00972754)'':0.0101609)'':0.00531535,((('PDT000612026.2':0.0463149,('PDT000407582.2':0.0312362,('PDT000378063.2':0.0191108,('PDT000161972.3':0.0186322,'PDT000107688.2':0.01788)'':0.00185513)'':0.00211432)'':0.00759425)'':-0.000811445,('PDT000048650.3':0.0290714,(('PDT000110493.3':0.0251126,('PDT000546868.2':0.0224288,('PDT000317747.2':0.0125264,('PDT000161882.3':0.011814,('PDT000707496.1':0.00875336,'PDT000498181.1':0.00767764)'':0.00196926)'':0.00316005)'':0.00285209)'':0.00607153)'':-0.00156171,('PDT000115445.3':0.02583,(('PDT000334892.2':0.0154552,'PDT000123262.1':0.015987)'':0.00586105,('PDT000123266.1':0.0122452,(('PDT000327943.2':0.0146148,('PDT000157695.3':0.0122905,('PDT000148140.3':0.00870358,'PDT000066905.3':0.00776962)'':0.000465271)'':0.00247562)'':-0.000265414,('PDT000060877.3':0.0083338,('PDT000132411.3':0.00633496,('PDT000108900.3':0.0053892,'PDT000683135.1':0.0060856)'':0.000628186)'':0.00500236)'':0.00161279)'':0.00591879)'':-0.00108447)'':0.00645498)'':-0.00274881)'':0.00460426)'':-7.10784e-06)'':0.00122724,('PDT000320269.2':0.0356443,(('PDT000551984.2':0.0255335,'PDT000108928.3':0.0234417)'':0.00569882,(('PDT000115436.3':0.0110874,('PDT000107687.2':0.00714084,('PDT000123268.1':0.0080753,'PDT000123305.1':0.0074224)'':0.00341756)'':0.00258257)'':0.0108443,('PDT000320935.2':0.0240954,(('PDT000551322.2':0.0133956,('PDT000123292.1':0.00539633,'PDT000123356.1':0.00767617)'':0.00630378)'':0.00399927,(('PDT000324153.2':0.0160688,('PDT000038891.1':0.0118786,'PDT000208722.1':0.0118392)'':0.00812313)'':-0.000635491,('PDT000378083.2':0.0245846,(('PDT000316461.2':0.0214414,('PDT000050563.1':0.0139126,'PDT000208691.1':0.0194868)'':0.0051575)'':0.00291233,(('PDT000596032.2':0.0156081,('PDT000115442.3':0.0187449,'PDT000711057.1':0.0118054)'':0.00278272)'':-0.000484585,('PDT000160192.3':0.0190017,('PDT000061328.3':0.0202086,('PDT000115460.3':0.0109846,('PDT000669583.1':0.0162328,('PDT000576255.2':0.0149036,(('PDT000387010.2':0.011479,'PDT000500643.2':0.00919905)'':0.00130396,('PDT000324020.2':0.00771698,(('PDT000144014.3':0.00905254,'PDT000492103.2':0.00693236)'':0.00228732,('PDT000038880.1':0.00749941,('PDT000327936.2':0.00731303,('PDT000108914.3':0.007059,('PDT000314213.3':0.00598144,('PDT000155157.3':0.0052008,'PDT000322081.2':0.00475242)'':0.000886297)'':0.00134748)'':0.00101412)'':0.000377111)'':-0.000254313)'':0.00102478)'':0.000475176)'':0.00247756)'':0.000879577)'':0.00045251)'':0.000933544)'':0.00110318)'':-0.00082268)'':0.00433878)'':0.00205365)'':-0.00271118)'':0.000183831)'':0.0053523)'':-0.00301608)'':0.0032652)'':0.0204348)'':-0.0203668)'':0.0178398)'':0.00637864)'':0.00860502,(((('PDT000124709.1':0.0511596,('PDT000585181.2':0.0423855,(('PDT000285795.3':0.0119321,'PDT000668763.1':0.0123151)'':0.00586846,('PDT000368365.1':0.0178102,('PDT000373183.1':0.00477396,'PDT000625866.1':0.00687474)'':0.0139272)'':0.000941868)'':0.0263898)'':0.00421113)'':-0.000658582,(('PDT000315559.2':0.032066,('PDT000324678.2':0.0270008,('PDT000156536.2':0.0119901,'PDT000160443.3':0.00983753)'':0.0098128)'':0.00656972)'':0.00600241,(('PDT000132181.2':0.0255207,'PDT000253458.3':0.0233338)'':0.00153994,('PDT000221749.3':0.0193631,('PDT000253446.2':0.00954885,'PDT000718648.1':0.0110615)'':0.00436138)'':0.012631)'':0.00601482)'':0.00509667)'':0.00106024,((('PDT000337255.2':0.018694,'PDT000671614.1':0.0164127)'':0.0115736,(('PDT000161917.3':0.0214248,'PDT000320857.2':0.0224054)'':0.000828373,('PDT000714603.1':0.0184032,('PDT000446020.2':0.0114017,('PDT000500648.2':0.00940393,('PDT000496165.2':0.00805802,'PDT000706082.1':0.00988998)'':0.000520371)'':0.005592)'':0.000796334)'':0.00634127)'':0.00135563)'':0.0158857,('PDT000502563.1':0.0662285,(('PDT000711030.1':0.0512877,('PDT000317247.1':0.0378845,'PDT000444893.2':0.0330801)'':0.00503006)'':-0.000659826,(('PDT000645776.1':0.034568,'PDT000038852.1':0.0388772)'':0.00483639,('PDT000131863.2':0.045742,(('PDT000487867.1':0.0258818,'PDT000625507.1':0.0301738)'':0.0064095,('PDT000548263.2':0.0287448,('PDT000322043.2':0.0276255,(('PDT000332719.2':0.0261219,(('PDT000625376.1':0.00998556,'PDT000157976.3':0.0127417)'':0.00794174,('PDT000131976.3':0.0140863,('PDT000161990.2':0.0113363,'PDT000132353.2':0.0101178)'':0.00149588)'':0.0013177)'':0.00808176)'':0.00256737,('PDT000110185.3':0.0234405,(('PDT000580381.2':0.0208139,'PDT000285169.1':0.0185979)'':0.00262603,('PDT000132275.2':0.0214898,('PDT000161971.2':0.012483,'PDT000625318.1':0.0124059)'':0.00612749)'':0.00185373)'':0.00211183)'':0.000310938)'':0.00299962)'':0.00252368)'':-0.000356621)'':0.00520122)'':-0.00202661)'':0.0048904)'':0.0128556)'':-0.014366)'':0.00595572)'':0.0197234,(((('PDT000101367.3':0.0311373,((('PDT000123317.1':0.0259806,('PDT000065199.3':0.00920892,'PDT000125545.3':0.0118166)'':0.0100268)'':0.0115026,('PDT000082140.3':0.0262822,('PDT000264716.3':0.0248577,'PDT000498080.1':0.0285692)'':0.00368075)'':0.000328322)'':0.00457583,('PDT000184425.3':0.0309622,(('PDT000084230.3':0.0270251,('PDT000539813.2':0.0109088,'PDT000083152.3':0.00903957)'':0.0069895)'':0.00328924,(('PDT000273053.3':0.00838444,'PDT000098709.3':0.00988766)'':0.010383,('PDT000128460.3':0.0182182,('PDT000532415.2':0.0182087,('PDT000120919.3':0.00903523,('PDT000641710.2':0.00890217,(('PDT000271243.3':0.00970299,('PDT000222751.3':0.00438993,'PDT000137593.3':0.00408127)'':0.00318716)'':-0.000155218,('PDT000565000.2':0.00600658,('PDT000491895.2':0.00677316,('PDT000048657.3':0.00286793,'PDT000352669.2':0.00261726)'':0.00248374)'':0.00173811)'':0.0033646)'':0.00351773)'':0.000627818)'':0.00122849)'':0.00169612)'':-0.000760271)'':0.000321478)'':0.00473405)'':0.0107042)'':0.0010143)'':-0.00177187,('PDT000110452.3':0.0383397,((('PDT000144037.3':0.0259931,('PDT000189673.3':0.0161696,'PDT000083195.3':0.0110724)'':0.00406973)'':0.000579018,('PDT000005266.2':0.0137234,('PDT000258008.3':0.00817339,'PDT000384056.2':0.00811261)'':0.00732368)'':0.00243781)'':0.00576966,(('PDT000497782.1':0.0299306,('PDT000163049.4':0.0232175,('PDT000367056.2':0.0190241,('PDT000690012.1':0.0140151,'PDT000497919.1':0.0111265)'':0.00158508)'':0.00325037)'':0.00701855)'':-0.00346322,(('PDT000125023.3':0.0210512,('PDT000360449.2':0.0149076,(('PDT000267686.3':0.0110495,('PDT000673725.1':0.00818356,'PDT000102783.3':0.0123241)'':0.00337336)'':0.00212373,(('PDT000248628.1':0.00705405,'PDT000248713.1':0.00886635)'':0.00704111,('PDT000494391.2':0.00983672,('PDT000049084.3':0.00803824,'PDT000421157.2':0.0110853)'':0.00780278)'':0.0040354)'':0.00131763)'':0.00290041)'':0.00179006)'':-0.00109478,('PDT000496780.2':0.00792973,('PDT000491888.2':0.00646752,(('PDT000189681.3':0.00317432,('PDT000048216.3':0.00303097,('PDT000084228.3':0.00224912,'PDT000137735.3':0.00144829)'':0.00125432)'':0.000893162)'':0.000660531,('PDT000532322.2':0.00527522,('PDT000084000.3':0.00606176,('PDT000250824.3':0.00319328,('PDT000394740.2':0.00254789,'PDT000707440.1':0.00244014)'':0.000495704)'':0.00242485)'':0.000344299)'':-0.000329183)'':0.0025845)'':0.000938703)'':0.00592085)'':0.00301643)'':0.00060218)'':0.0149823)'':-0.00567836)'':0.00193512,((('PDT000040363.6':0.0262529,'PDT000136763.3':0.0246658)'':0.00288372,('PDT000081892.3':0.0170071,('PDT000366251.2':0.0241114,(('PDT000352671.2':0.0164109,'PDT000051383.3':0.00871122)'':0.00640325,('PDT000162279.3':0.00991377,('PDT000392145.2':0.00899197,'PDT000141667.3':0.00877473)'':0.00328308)'':0.00354774)'':0.0035664)'':0.00102109)'':0.00622182)'':6.77038e-05,((('PDT000233975.3':0.0248014,'PDT000235064.3':0.0204568)'':0.00232733,('PDT000562120.2':0.0278692,('PDT000240613.3':0.0286521,('PDT000250835.3':0.0148618,(('PDT000128475.3':0.00928358,('PDT000273837.3':0.00608249,('PDT000065188.3':0.00425607,'PDT000617953.2':0.00361873)'':0.00238801)'':0.00177292)'':0.000728861,('PDT000205384.4':0.0123828,('PDT000101089.3':0.0123698,('PDT000593736.2':0.00617082,(('PDT000173877.3':0.00535675,'PDT000251654.1':0.00450381)'':0.000442817,('PDT000281592.3':0.00285937,('PDT000084178.3':0.00240901,('PDT000322441.2':0.00205212,'PDT000141675.3':0.00204256)'':0.000983052)'':0.000367973)'':0.00234238)'':0.00388911)'':0.00150953)'':0.00134105)'':-0.00176838)'':0.00345297)'':0.00703541)'':0.00356741)'':-0.00268779)'':-0.000355104,(('PDT000135782.3':0.0243455,('PDT000707412.1':0.0106477,(('PDT000198192.3':0.0111781,('PDT000212526.3':0.0118485,'PDT000267474.3':0.00582323)'':0.0021913)'':0.000345267,('PDT000455182.2':0.00618923,('PDT000150137.3':0.00561833,('PDT000610111.2':0.00374961,'PDT000111544.3':0.00214332)'':0.00243018)'':0.00152648)'':0.00268363)'':0.00180685)'':0.00337213)'':0.008677,(('PDT000248597.1':0.0417065,('PDT000408625.2':0.0289808,('PDT000497299.2':0.0245193,('PDT000084154.3':0.0200352,('PDT000569700.2':0.0170177,('PDT000162291.3':0.0131706,'PDT000254227.3':0.0177695)'':0.00152799)'':0.00160015)'':0.00450698)'':0.00145452)'':0.00532126)'':0.00196184,(('PDT000040375.6':0.0351836,('PDT000252752.3':0.0224894,('PDT000205004.3':0.0287287,'PDT000312469.3':0.0163826)'':0.0050236)'':0.00758802)'':-0.00245442,(('PDT000150702.3':0.0358172,'PDT000125896.3':0.0285542)'':0.0122171,((('PDT000354955.2':0.0296338,'PDT000248594.1':0.0275257)'':0.0095197,('PDT000498194.1':0.0403294,('PDT000598846.2':0.0170334,('PDT000580721.2':0.0169405,(('PDT000167949.3':0.0119354,('PDT000216638.3':0.00480363,'PDT000708556.1':0.00566597)'':0.00348872)'':0.00262295,('PDT000233970.3':0.0205257,(('PDT000133832.3':0.00928088,('PDT000532746.2':0.00762706,('PDT000047666.4':0.00620601,('PDT000247549.3':0.00632416,'PDT000480362.2':0.00550604)'':0.000355887)'':0.00112635)'':0.00243294)'':-2.01746e-05,('PDT000491424.2':0.0215443,('PDT000032318.1':0.0164971,('PDT000233900.3':0.0125721,('PDT000083221.3':0.0115641,(('PDT000564999.2':0.00609669,('PDT000094746.3':0.00540382,('PDT000281616.3':0.00365436,'PDT000054723.3':0.00402644)'':0.00205068)'':0.000940281)'':-0.00029487,('PDT000011010.2':0.00532309,('PDT000394046.2':0.00492635,('PDT000158569.3':0.00435201,('PDT000049366.3':0.00399412,('PDT000238333.3':0.00117643,'PDT000137676.3':0.00132107)'':0.00119297)'':0.00126733)'':0.000624451)'':0.000459948)'':0.000764216)'':0.00292608)'':0.00112487)'':0.000773469)'':0.00153811)'':-0.00257924)'':0.00558323)'':-0.00393313)'':0.00492661)'':0.00467894)'':0.00840746)'':-0.00180212)'':-0.008537,(('PDT000254364.3':0.0146443,('PDT000356289.2':0.0133974,'PDT000085607.3':0.00924372)'':0.00308653)'':0.0039926,(('PDT000025349.1':0.0202523,(('PDT000127767.3':0.0116853,('PDT000421178.2':0.00370581,'PDT000465609.2':0.00357163)'':0.00557286)'':0.00516222,(('PDT000108012.3':0.0116316,'PDT000112516.3':0.0126258)'':0.00564788,('PDT000145374.3':0.0143657,('PDT000108015.3':0.0161813,('PDT000134034.3':0.0162794,('PDT000222748.3':0.00427861,(('PDT000384029.2':0.00443609,('PDT000260750.3':0.00303286,'PDT000594284.2':0.00255764)'':0.00183241)'':4.69196e-05,('PDT000040709.3':0.00506772,(('PDT000252937.3':0.00246513,'PDT000281609.3':0.00222455)'':0.000414405,('PDT000381333.2':0.00237759,'PDT000124966.3':0.00251458)'':0.00033157)'':0.00158028)'':-0.000113221)'':0.00148649)'':0.00343242)'':0.00132781)'':0.00184531)'':-0.000202549)'':-0.000196817)'':0.00985653)'':-0.00740559,('PDT000248532.1':0.0338484,('PDT000009375.2':0.0296881,('PDT000082091.3':0.0279509,((('PDT000083192.3':0.0169017,'PDT000679484.1':0.0192987)'':0.00565275,(('PDT000173972.3':0.0149871,'PDT000265140.3':0.0114175)'':0.00276324,('PDT000297755.3':0.00913026,'PDT000130570.3':0.0113973)'':0.00362151)'':0.00370159)'':-0.00118466,('PDT000499199.2':0.0190232,('PDT000093984.3':0.0219946,(('PDT000082117.3':0.0193607,('PDT000150135.3':0.0117891,'PDT000359253.2':0.0132554)'':0.0039501)'':0.00392282,('PDT000048215.4':0.018597,('PDT000025399.1':0.0147295,(('PDT000498178.1':0.0126529,'PDT000498179.1':0.011904)'':0.00152869,(('PDT000368897.1':0.0103638,('PDT000102252.3':0.00667803,'PDT000300578.1':0.00825927)'':0.00292265)'':0.000566508,('PDT000248577.1':0.0135384,('PDT000267684.3':0.0114844,(('PDT000115018.3':0.00781921,('PDT000151174.3':0.00673574,'PDT000049080.3':0.00569466)'':0.00199084)'':2.43059e-05,('PDT000491448.2':0.00947918,(('PDT000147943.3':0.00509045,('PDT000144050.3':0.00298291,'PDT000051227.3':0.00300034)'':0.000989802)'':0.00115552,('PDT000085617.3':0.0074008,(('PDT000120824.3':0.00625874,'PDT000251657.1':0.00567086)'':0.00131087,('PDT000359221.2':0.00687453,(('PDT000040708.3':0.00607463,'PDT000115002.3':0.00566667)'':0.000723066,('PDT000065514.3':0.00589599,('PDT000251653.1':0.00351372,('PDT000135754.3':0.00271041,('PDT000491369.2':0.0030123,('PDT000083959.3':0.00248782,('PDT000352809.2':0.00235455,('PDT000593818.2':0.00259439,(('PDT000086332.3':0.0012504,('PDT000049347.3':0.00136293,'PDT000082116.3':0.00163288)'':0.000147932)'':0.000169146,('PDT000050182.3':0.00148285,('PDT000051218.3':0.00103805,('PDT000145351.3':0.000535481,'PDT000504972.2':0.000663919)'':0.000460361)'':9.45125e-05)'':0.000538665)'':0.000310653)'':0.000387533)'':0.00019603)'':0.000235147)'':0.000246803)'':0.000483048)'':0.0012076)'':-0.000638567)'':0.00227136)'':-0.00114203)'':0.00160539)'':-0.000635166)'':0.00292081)'':-0.00139106)'':0.00208997)'':0.00206468)'':-0.000862652)'':0.00285342)'':0.00201179)'':0.00126941)'':-0.00409992)'':0.00545117)'':0.000799459)'':-0.0055877)'':0.0100015)'':0.000810783)'':0.00298074)'':-0.0129362)'':0.00510798)'':-0.00341737)'':0.00990516)'':-0.00409399)'':-0.000343893)'':-0.000372246)'':-0.000153169)'':0.002476)'':0.00783476)'':0.0119951,((('PDT000063259.3':0.0140119,('PDT000405088.2':0.0102721,'PDT000587554.2':0.0119346)'':0.00985828)'':0.0275764,('PDT000707405.1':0.0488164,('PDT000311072.3':0.0334003,('PDT000289056.3':0.0321319,('PDT000283356.3':0.0199509,'PDT000309663.3':0.0198486)'':0.00742318)'':0.00576028)'':0.00353427)'':0.00524093)'':0.0159831,((('PDT000706513.1':0.0164415,('PDT000417276.2':0.0118203,('PDT000177090.3':0.00494101,'PDT000101033.4':0.00582669)'':0.0047161)'':0.00384057)'':0.0294423,(('PDT000432424.2':0.0369198,(('PDT000533045.2':0.0184193,'PDT000593755.2':0.0167412)'':0.00848979,(('PDT000198189.3':0.0281399,('PDT000707443.1':0.0280546,('PDT000242601.3':0.0205549,'PDT000092281.3':0.0163393)'':0.00408784)'':0.00293978)'':0.000755401,('PDT000394064.2':0.0191126,('PDT000144038.3':0.0120503,('PDT000149222.3':0.00800153,('PDT000235052.3':0.00483242,'PDT000259810.3':0.0040507)'':0.00311196)'':0.00109663)'':0.00720091)'':0.00553317)'':0.0041123)'':0.0051786)'':-0.00347162,(('PDT000690533.1':0.0296842,('PDT000171529.3':0.0205215,('PDT000098757.3':0.0263809,('PDT000383993.2':0.0144431,'PDT000077506.1':0.0159797)'':0.00226487)'':0.00181166)'':0.0142855)'':-0.00109594,(('PDT000063152.3':0.0410768,('PDT000359156.2':0.0325129,'PDT000130561.3':0.0324954)'':0.00393939)'':0.00774322,('PDT000044722.3':0.0505611,(('PDT000126254.3':0.0410818,(('PDT000044719.3':0.0177077,'PDT000491886.2':0.0200021)'':0.0134632,(('PDT000153285.3':0.0219302,('PDT000557061.2':0.0181447,'PDT000566502.2':0.0163313)'':0.00564443)'':0.00142944,('PDT000569567.2':0.0177926,(('PDT000598835.1':0.0187422,('PDT000384018.2':0.0117895,'PDT000096931.3':0.011464)'':0.00559449)'':0.00108246,(('PDT000598722.1':0.0191972,('PDT000065220.3':0.0102063,'PDT000077527.1':0.00943271)'':0.00497939)'':0.00317609,('PDT000044506.3':0.00839657,('PDT000577459.2':0.00643538,('PDT000264708.3':0.0049935,'PDT000644542.2':0.00446206)'':0.00182369)'':0.00179559)'':0.00468178)'':0.00323122)'':0.000185868)'':0.00385475)'':0.00237377)'':0.00439774)'':-0.00456071,(('PDT000322000.2':0.0265184,('PDT000491981.2':0.0276284,('PDT000396585.2':0.0191148,('PDT000239601.3':0.0131642,'PDT000360435.2':0.0157238)'':0.002899)'':0.00540356)'':0.00800013)'':0.000924344,(('PDT000334240.2':0.028155,('PDT000071658.3':0.0290301,('PDT000587585.2':0.026636,'PDT000131126.3':0.0247061)'':0.00519425)'':0.0010997)'':0.000459671,(('PDT000455287.2':0.0327095,('PDT000081151.3':0.0245658,('PDT000247435.3':0.0209915,'PDT000334543.2':0.0191344)'':0.00498349)'':0.00423323)'':-0.000271954,(('PDT000221719.3':0.0229291,('PDT000707434.1':0.0187267,('PDT000101044.3':0.0147458,(('PDT000147963.3':0.0101034,'PDT000096926.3':0.0134389)'':0.00414374,('PDT000137566.3':0.0127751,('PDT000230811.3':0.0108087,'PDT000084001.3':0.0126752)'':0.00137748)'':0.00129958)'':0.0030801)'':0.000916681)'':0.00299574)'':-0.00100044,('PDT000554123.2':0.023269,('PDT000545161.2':0.0250659,('PDT000411090.2':0.0171152,('PDT000342367.2':0.0190338,(('PDT000624096.2':0.0171059,('PDT000443235.2':0.014594,('PDT000544557.2':0.0127173,('PDT000537065.2':0.00959298,'PDT000084048.3':0.0128454)'':0.00237605)'':0.00129214)'':0.00448504)'':-0.0019938,(('PDT000539843.2':0.0135408,('PDT000386288.2':0.0123658,'PDT000707442.1':0.0114484)'':0.00138719)'':0.00184995,('PDT000366316.2':0.0137008,((('PDT000163790.3':0.00858678,'PDT000352663.2':0.00887522)'':0.00532064,('PDT000579957.2':0.00972071,('PDT000375745.2':0.00734243,('PDT000388889.2':0.00413925,'PDT000085595.3':0.00463443)'':0.00136653)'':0.00071523)'':0.00191847)'':0.00133507,(('PDT000367778.2':0.0125339,('PDT000163788.3':0.00709409,'PDT000127483.3':0.00771411)'':0.00308944)'':0.000225219,('PDT000714519.1':0.00985763,('PDT000558664.2':0.00810445,(('PDT000234202.3':0.00566509,'PDT000069923.3':0.00638551)'':0.00188857,('PDT000491899.2':0.00679441,'PDT000071652.3':0.00574169)'':0.00116365)'':0.00111628)'':0.00137911)'':0.000979158)'':0.000150956)'':0.00455669)'':-0.00110968)'':0.00103418)'':0.00254595)'':0.000635403)'':0.00243826)'':0.0027047)'':-0.00406385)'':0.00282386)'':0.00370181)'':-0.000461725)'':-0.00239509)'':0.00901152)'':0.000473093)'':-0.00219399)'':-0.000815898)'':0.0135251)'':-0.0107718,(((('PDT000040407.3':0.0602853,'PDT000498180.1':0.0470377)'':0.0154014,('PDT000389727.2':0.0320405,('PDT000162969.4':0.0352098,'PDT000558647.2':0.0365578)'':0.0103273)'':0.0221014)'':0.00367205,('PDT000141066.3':0.0372995,(('PDT000309615.3':0.0284436,'PDT000470193.2':0.0279232)'':0.00353745,(('PDT000235328.3':0.0299973,'PDT000499258.2':0.0281195)'':0.00513339,((('PDT000147506.3':0.0208364,'PDT000359194.2':0.0204624)'':0.00315064,('PDT000609537.2':0.0166772,'PDT000085605.3':0.0139989)'':0.00755186)'':0.00128519,(('PDT000203845.3':0.0284689,'PDT000624036.2':0.0253199)'':0.00140732,(('PDT000491963.2':0.0242797,('PDT000581262.2':0.0107572,'PDT000604637.2':0.00831308)'':0.00760021)'':0.00790511,(('PDT000250812.3':0.0153277,'PDT000130544.3':0.0179497)'':0.001374,(('PDT000048209.3':0.0109281,'PDT000133833.3':0.00894266)'':0.00626311,(('PDT000275633.3':0.0108359,'PDT000360465.2':0.016823)'':0.00373444,('PDT000604554.2':0.00945825,'PDT000669162.1':0.011023)'':0.00276198)'':0.000892853)'':0.00342169)'':0.00188753)'':0.0036148)'':-0.000213502)'':0.00438933)'':0.00279525)'':0.0116024)'':0.0081079)'':-0.0119038,(('PDT000334171.2':0.0338055,('PDT000316177.2':0.0279012,('PDT000465764.2':0.0128887,'PDT000540023.2':0.0142132)'':0.00713675)'':0.00480217)'':0.0100514,((('PDT000203849.3':0.0272343,('PDT000617677.2':0.0175227,('PDT000480974.2':0.0118279,'PDT000626977.2':0.0108971)'':0.00755955)'':0.00364452)'':0.0020525,(('PDT000356245.2':0.0255638,('PDT000360454.2':0.0179958,'PDT000076963.3':0.0188915)'':0.0030729)'':0.00415073,((('PDT000598509.2':0.0102936,'PDT000133819.3':0.0151133)'':0.00991554,('PDT000616426.2':0.017258,('PDT000588258.2':0.0103421,'PDT000375718.2':0.010256)'':0.00583348)'':0.00185656)'':0.000374821,('PDT000404013.2':0.0222749,(('PDT000307895.3':0.0123477,'PDT000131114.3':0.0119654)'':0.00523534,(('PDT000329304.2':0.0126398,'PDT000124958.3':0.0134266)'':0.00313536,('PDT000275640.3':0.0121093,('PDT000388890.2':0.00559604,'PDT000086164.3':0.00416783)'':0.00500532)'':0.000976475)'':0.00126702)'':0.00778626)'':-0.000984463)'':0.0017838)'':0.00136787)'':0.00261817,(((('PDT000209197.3':0.00985366,'PDT000133750.3':0.0103112)'':0.0172624,('PDT000480926.2':0.0276106,(('PDT000145406.3':0.0134196,'PDT000394074.2':0.0142238)'':0.00320824,('PDT000381309.2':0.0140977,'PDT000386249.2':0.0139074)'':0.00447481)'':0.00455754)'':0.00233144)'':0.00259492,(('PDT000368336.2':0.0232104,'PDT000440550.2':0.0191792)'':0.00837679,(('PDT000264711.3':0.0149612,'PDT000501769.2':0.0131814)'':0.00343921,('PDT000359245.2':0.0189747,('PDT000520302.2':0.0156989,('PDT000329305.2':0.0144049,'PDT000579257.2':0.0147979)'':0.00822875)'':0.00124406)'':0.00201102)'':0.00222021)'':0.00884128)'':0.00160499,((('PDT000491371.2':0.031005,('PDT000151179.3':0.0188933,(('PDT000383980.2':0.0141357,'PDT000399871.2':0.00891593)'':0.00759333,('PDT000356227.2':0.0119523,('PDT000532421.2':0.0077627,('PDT000222760.3':0.0057741,'PDT000356234.2':0.0057705)'':0.00288655)'':0.00172806)'':0.00817419)'':0.0044468)'':0.0031102)'':0.00520112,(('PDT000069911.3':0.0292887,(('PDT000525267.2':0.0103497,'PDT000688102.1':0.00962051)'':0.00656683,(('PDT000499201.2':0.0150031,'PDT000549343.2':0.0176585)'':0.00463709,('PDT000688116.1':0.0195151,('PDT000243675.3':0.0156511,'PDT000126003.3':0.0144932)'':0.003491)'':0.000887893)'':0.00149739)'':0.00333746)'':0.00324908,(('PDT000396595.2':0.0440713,(('PDT000559325.2':0.0230165,'PDT000570328.2':0.0212861)'':0.00873321,(('PDT000243731.3':0.016649,'PDT000443264.2':0.0155168)'':0.00575128,(('PDT000564128.2':0.0131399,'PDT000616410.2':0.0147263)'':0.0032133,('PDT000491949.2':0.0128654,(('PDT000421069.2':0.0136361,'PDT000546203.2':0.0129739)'':0.00468425,(('PDT000537385.2':0.0120901,('PDT000205386.3':0.0128651,'PDT000103589.3':0.00901461)'':0.00177865)'':0.00143384,('PDT000430571.2':0.0164823,(('PDT000510809.2':0.00663575,'PDT000132457.3':0.00629575)'':0.00585787,('PDT000355542.2':0.00569526,('PDT000191189.1':0.00467159,('PDT000556655.2':0.00354697,('PDT000221967.3':0.00278472,'PDT000366157.2':0.0032033)'':0.000338746)'':0.000482039)'':0.00271084)'':0.00111067)'':0.00412172)'':-0.000729382)'':0.000718263)'':0.00502886)'':-0.00308851)'':0.00157878)'':0.007338)'':0.006964)'':-0.00645785,('PDT000690092.1':0.0388356,(('PDT000065185.3':0.0192282,'PDT000133413.3':0.0159621)'':0.00617658,((('PDT000120818.3':0.0269598,'PDT000536930.2':0.0197719)'':0.00319873,('PDT000193103.3':0.0126838,'PDT000048206.3':0.0124225)'':0.00348882)'':0.00554481,(('PDT000144011.3':0.0133221,('PDT000562536.2':0.0107414,('PDT000373188.2':0.00297171,'PDT000599879.2':0.00380863)'':0.00535527)'':0.00105665)'':0.00935231,(('PDT000201858.3':0.0207128,('PDT000641705.2':0.0173896,'PDT000191219.1':0.0221256)'':0.00244704)'':0.00625546,((('PDT000173961.3':0.00741813,'PDT000554102.2':0.0134256)'':0.0107765,('PDT000081134.3':0.00900798,('PDT000066893.3':0.00976043,'PDT000127702.3':0.00793167)'':0.00173667)'':0.00451119)'':0.00348872,(('PDT000151418.3':0.0104053,('PDT000544558.2':0.0077716,'PDT000555746.2':0.0093004)'':0.00377697)'':0.00266815,(('PDT000225299.3':0.00909982,'PDT000598081.2':0.00855888)'':0.00165982,('PDT000069916.3':0.00680908,('PDT000342327.2':0.00557375,'PDT000708513.1':0.00566895)'':0.00374347)'':0.00417828)'':0.00344695)'':0.00281803)'':0.00305147)'':0.000168435)'':0.0018241)'':0.00116452)'':0.0096003)'':-0.00996104)'':0.00268272)'':0.00247158)'':-0.00280622,(((('PDT000162275.3':0.00690791,'PDT000653394.2':0.0149724)'':0.0112939,('PDT000668639.1':0.0286477,(('PDT000545113.2':0.0141449,('PDT000480976.2':0.0119913,'PDT000627451.2':0.0105599)'':0.00278975)'':-0.000467343,(('PDT000062539.3':0.0107595,'PDT000580518.2':0.0100243)'':0.00385134,(('PDT000566592.2':0.00353415,('PDT000357231.2':0.00338384,'PDT000102255.3':0.00289669)'':5.56837e-05)'':0.00478389,(('PDT000302804.3':0.00818048,'PDT000112519.3':0.00828332)'':0.00359379,('PDT000273839.3':0.0152252,('PDT000127482.3':0.00516524,('PDT000234154.3':0.00473135,('PDT000151479.3':0.00358635,'PDT000562543.2':0.00418567)'':0.000442455)'':0.00191147)'':0.00297043)'':-3.16777e-05)'':0.000286169)'':0.00195292)'':0.00369941)'':0.00574883)'':-0.00178365)'':0.000796164,(('PDT000066282.3':0.0285863,('PDT000318462.2':0.0202716,('PDT000142909.3':0.0152522,'PDT000491921.2':0.0187849)'':0.00666056)'':0.00340124)'':0.0012235,('PDT000527188.2':0.0233146,(('PDT000385925.2':0.027053,('PDT000242799.3':0.0170227,('PDT000356270.2':0.0123289,('PDT000190139.3':0.00841863,('PDT000041509.3':0.00594765,'PDT000394755.2':0.00641035)'':0.00228137)'':0.00123617)'':0.00949293)'':0.00297487)'':0.00037687,('PDT000221441.3':0.0218075,(('PDT000084150.3':0.0189715,(('PDT000238403.3':0.0115118,('PDT000264709.3':0.00836335,('PDT000333629.2':0.00775573,'PDT000071656.3':0.00605867)'':0.000758301)'':0.0039398)'':-0.00114407,('PDT000072446.3':0.0124124,('PDT000083199.3':0.0053209,('PDT000052593.3':0.00459387,('PDT000240614.3':0.00458483,'PDT000098755.3':0.00398859)'':0.000679126)'':0.00150682)'':0.00200363)'':0.00264909)'':0.00468757)'':-0.00220697,('PDT000086213.3':0.0221678,((('PDT000147964.3':0.00514464,'PDT000085611.3':0.00641836)'':0.0024363,('PDT000131128.3':0.0101961,('PDT000083278.3':0.00376628,'PDT000085614.3':0.00361159)'':0.00465758)'':0.000907088)'':0.00156135,(('PDT000375968.2':0.014725,'PDT000707493.1':0.00742628)'':0.00281357,('PDT000510665.2':0.00867462,('PDT000227570.3':0.00630816,'PDT000083268.3':0.00572804)'':0.00144273)'':0.00221233)'':0.00102024)'':0.00550607)'':-0.00244512)'':0.00763131)'':-0.00164509)'':0.00399765)'':-0.00232804)'':0.0030789)'':-0.000863045,((('PDT000503450.2':0.0207163,('PDT000098347.3':0.0177154,('PDT000707449.1':0.0103591,'PDT000102811.3':0.00948415)'':0.00143736)'':0.00394022)'':0.0027759,('PDT000480981.2':0.0234766,(('PDT000569454.2':0.0151384,('PDT000540121.2':0.0160187,('PDT000281572.3':0.00278113,'PDT000683507.1':0.00270406)'':0.00777)'':0.00329095)'':5.48241e-05,('PDT000065226.3':0.0198528,('PDT000503994.2':0.0130653,('PDT000279444.3':0.0110126,('PDT000367237.2':0.00672598,('PDT000314303.3':0.00898858,'PDT000362567.2':0.00644812)'':0.000243721)'':0.00229949)'':0.00401626)'':0.00108225)'':0.000693744)'':0.00565621)'':-0.00358768)'':0.0089073,(('PDT000275057.3':0.0311281,('PDT000082148.3':0.0362206,('PDT000356228.2':0.0216245,(('PDT000481284.2':0.0201557,'PDT000066942.3':0.0185667)'':0.00217231,('PDT000102880.3':0.0230399,('PDT000147914.3':0.0191218,'PDT000127269.3':0.0164302)'':0.00416558)'':0.001546)'':0.00352956)'':0.0018537)'':0.00204465)'':0.000932435,((('PDT000344394.2':0.0274139,'PDT000532201.2':0.0240416)'':0.00414016,(('PDT000421012.2':0.0188706,('PDT000146473.3':0.0139871,'PDT000329847.2':0.00877418)'':0.00436915)'':0.0024657,('PDT000496454.2':0.0207143,('PDT000136746.3':0.0327494,('PDT000386158.2':0.0158185,(('PDT000259811.3':0.0113186,'PDT000051837.3':0.0127084)'':0.0034962,('PDT000146039.3':0.00856739,('PDT000051235.3':0.0085994,('PDT000491935.2':0.00617143,('PDT000084174.3':0.00562,'PDT000707587.1':0.0060148)'':0.00219087)'':0.00281031)'':0.000239687)'':0.00197935)'':0.00242802)'':0.00656166)'':0.00112862)'':-0.00253639)'':0.00576371)'':-0.000561227,((('PDT000231232.3':0.0229096,'PDT000471140.2':0.0177216)'':0.0055733,('PDT000233969.3':0.0093503,('PDT000343615.2':0.01202,('PDT000184450.3':0.00751982,'PDT000322429.2':0.00846348)'':0.00206595)'':0.0016161)'':0.00903017)'':0.00307587,(((('PDT000372751.2':0.0154067,'PDT000592645.2':0.0203111)'':0.0013456,('PDT000344362.2':0.0187763,(('PDT000390210.2':0.0176792,('PDT000366129.2':0.00851692,'PDT000669353.1':0.00964868)'':0.00665556)'':0.00222203,('PDT000049882.3':0.0207077,(('PDT000135756.3':0.0128913,('PDT000390214.2':0.0104624,('PDT000275626.3':0.00865726,'PDT000359223.2':0.00841804)'':0.00110892)'':0.00237191)'':0.00037301,(('PDT000227277.3':0.00958258,'PDT000641712.2':0.00933952)'':0.00124783,(('PDT000410925.2':0.00838804,'PDT000133198.3':0.00828856)'':0.00196141,('PDT000718617.1':0.00681132,('PDT000198193.3':0.0041234,'PDT000329842.2':0.00444148)'':0.00133829)'':0.00160622)'':0.0033476)'':0.00161773)'':0.00404804)'':0.000850898)'':0.00213417)'':0.000913615)'':-0.00268468,(('PDT000101017.3':0.0259953,('PDT000147955.3':0.0192371,'PDT000247414.3':0.0229941)'':0.00132484)'':0.00283596,('PDT000313300.3':0.0259467,((('PDT000551376.2':0.01838,'PDT000137694.3':0.0151711)'':0.00815044,(('PDT000342279.2':0.0119451,'PDT000387592.2':0.0135105)'':0.00529309,(('PDT000367739.2':0.00668069,'PDT000397514.2':0.00841511)'':0.00540774,('PDT000389789.2':0.0238402,(('PDT000191174.1':0.00486593,'PDT000191177.1':0.00750317)'':0.006717,('PDT000150127.3':0.013219,(('PDT000146387.3':0.00733585,('PDT000299962.3':0.00498325,'PDT000405107.2':0.00428616)'':0.00378825)'':0.00043346,(('PDT000171640.3':0.00602855,'PDT000707487.1':0.00640305)'':0.00342661,('PDT000242732.3':0.00526163,'PDT000520096.2':0.00680577)'':0.00080169)'':0.00154473)'':0.00126121)'':0.000421532)'':0.00408231)'':-0.00133058)'':0.0025967)'':0.00334295)'':-0.00237314,((('PDT000351862.2':0.0100151,'PDT000545702.2':0.00942363)'':0.00439308,('PDT000303507.3':0.0104656,('PDT000125989.3':0.00878135,('PDT000151173.3':0.00648668,'PDT000047672.3':0.00788602)'':0.00333625)'':0.00295011)'':0.0019756)'':0.000778701,(('PDT000315882.2':0.0156769,'PDT000137571.3':0.0197562)'':0.00615761,(('PDT000242802.3':0.0160318,'PDT000355840.2':0.0138735)'':0.000333383,('PDT000569014.2':0.00908469,('PDT000191672.3':0.0122685,('PDT000387713.2':0.0113612,('PDT000375733.2':0.00616414,('PDT000151428.3':0.00767427,(('PDT000271240.3':0.00425185,'PDT000360430.2':0.00411802)'':0.000283058,('PDT000098752.3':0.00383092,'PDT000191213.1':0.00435598)'':0.00124172)'':0.00171909)'':0.000631789)'':0.000439649)'':0.000607688)'':0.00134144)'':0.00384134)'':0.00142948)'':9.96261e-05)'':0.00172609)'':0.0059672)'':-0.0032819)'':-0.000987212)'':0.000413353,((('PDT000191209.1':0.0171264,'PDT000191214.1':0.0269502)'':0.0175704,(('PDT000083267.3':0.00936219,'PDT000132636.3':0.0126813)'':0.00337968,(('PDT000115004.3':0.010114,('PDT000388876.2':0.00963501,'PDT000076956.3':0.00591829)'':0.0021527)'':0.00181398,(('PDT000083208.3':0.00627572,'PDT000707438.1':0.00769778)'':0.00328493,(('PDT000072717.3':0.00455035,'PDT000669529.1':0.00580515)'':0.00199457,(('PDT000191187.1':0.005621,'PDT000191218.1':0.0063134)'':0.00246703,('PDT000131015.3':0.00647857,(('PDT000491903.2':0.00440379,'PDT000120029.3':0.00475956)'':0.00085336,('PDT000098751.3':0.00417215,('PDT000491929.2':0.00146596,'PDT000708509.1':0.00133096)'':0.00249788)'':0.00168076)'':0.00085493)'':0.00035555)'':0.00221515)'':0.000967888)'':0.000174662)'':0.00413072)'':0.00838048)'':-0.0064344,(('PDT000483969.2':0.0237025,(('PDT000153166.3':0.010712,('PDT000086188.3':0.0080318,'PDT000114692.3':0.0062155)'':0.0027299)'':0.00649878,('PDT000551273.2':0.0191673,('PDT000258570.3':0.0133452,('PDT000281555.3':0.0120882,('PDT000679122.1':0.0102152,('PDT000295878.3':0.0082682,(('PDT000224226.3':0.00676552,'PDT000260735.3':0.00696308)'':0.000908665,('PDT000504879.2':0.00669038,'PDT000096867.3':0.00672372)'':0.00132598)'':0.001027)'':0.00265648)'':0.00103649)'':0.000880791)'':0.00482468)'':-0.00198521)'':0.00617161)'':-0.004423,(('PDT000523857.2':0.0245902,('PDT000366141.2':0.0062821,'PDT000587796.2':0.0089721)'':0.012285)'':0.000635435,(('PDT000563744.2':0.0368656,('PDT000404035.2':0.0107485,('PDT000553489.2':0.0111056,('PDT000428453.2':0.00804752,('PDT000575803.2':0.00830262,('PDT000399903.2':0.00507057,('PDT000357183.2':0.00336781,('PDT000408380.2':0.00259376,'PDT000104688.3':0.00289581)'':0.000816539)'':0.00108999)'':0.00300291)'':0.000704268)'':0.00234081)'':0.00245788)'':0.00613212)'':-0.0038545,(('PDT000137591.3':0.0235156,(('PDT000163283.4':0.0106366,'PDT000566650.2':0.00935542)'':0.000355196,(('PDT000451084.2':0.00717303,'PDT000125990.3':0.00776127)'':0.00207338,('PDT000112510.3':0.0105923,(('PDT000146450.3':0.00485596,'PDT000491911.2':0.00641584)'':0.00229545,(('PDT000238410.3':0.00363469,'PDT000499834.2':0.00434138)'':0.000584065,('PDT000393040.2':0.00444852,('PDT000266297.3':0.00370917,('PDT000188054.3':0.00223594,'PDT000221760.3':0.00205805)'':0.00192054)'':0.000440095)'':0.000101949)'':0.00189528)'':0.00208762)'':0.00124258)'':0.000573893)'':0.00498809)'':-0.00423596,('PDT000499237.2':0.0323307,('PDT000644052.2':0.0152029,('PDT000065204.3':0.0199564,('PDT000604424.2':0.0115335,('PDT000295857.3':0.0106583,('PDT000085622.3':0.0112039,('PDT000083968.3':0.00895128,('PDT000394052.2':0.00716655,('PDT000083138.3':0.00749093,(('PDT000646730.2':0.00479217,'PDT000707489.1':0.00456758)'':0.000490688,('PDT000602012.2':0.00618789,(('PDT000544782.2':0.00257722,'PDT000707678.1':0.00262018)'':0.00240455,('PDT000279550.3':0.00587849,('PDT000191184.1':0.00557131,(('PDT000250877.3':0.00335551,('PDT000137657.3':0.00227111,'PDT000710304.1':0.00252074)'':0.00153514)'':0.000209667,('PDT000307856.3':0.00591741,('PDT000367587.2':0.00451779,('PDT000130545.3':0.00369609,('PDT000144721.3':0.00307351,('PDT000539915.2':0.00297212,('PDT000102790.3':0.00275202,('PDT000616672.2':0.00273958,('PDT000069411.3':0.00226999,'PDT000136760.3':0.00212167)'':0.000351806)'':8.94045e-05)'':0.000349599)'':0.000247483)'':0.00017521)'':0.000694057)'':0.000174869)'':-0.000363133)'':0.00100527)'':0.00101225)'':-0.00078606)'':0.00119595)'':-0.000576056)'':0.0018851)'':0.000649599)'':0.00016824)'':0.000497352)'':0.00231047)'':0.000489296)'':0.00250143)'':0.0009496)'':0.00684682)'':-0.0101648)'':0.00273679)'':0.00336734)'':-0.00260948)'':0.00048239)'':0.000897261)'':0.00183711)'':0.000623018)'':0.00249975)'':-0.000433887)'':8.19895e-05)'':0.00143602)'':0.00167824)'':0.0022728)'':0.00761522)'':-0.00482556)'':-0.00327553)'':0.0222194)'':-0.00324599)'':0.0122561)'':0.00835691)'':0.00561321)'':-0.00626549,((((('PDT000342262.2':0.0353273,'PDT000465226.2':0.0380164)'':0.00957382,(('PDT000321550.2':0.021631,'PDT000652803.2':0.0204155)'':0.0157605,('PDT000532057.2':0.0490198,(('PDT000491483.2':0.0237419,('PDT000296046.3':0.0194149,'PDT000645949.2':0.0191906)'':0.00679079)'':-0.000667511,('PDT000510664.2':0.0315435,('PDT000327161.2':0.0146412,'PDT000446025.2':0.0206483)'':0.0051395)'':0.00455365)'':0.0125415)'':0.00246541)'':0.00506455)'':0.000670306,(('PDT000160211.3':0.0515363,'PDT000481590.2':0.0496087)'':0.0123014,((('PDT000481582.2':0.0225339,'PDT000523437.2':0.0211798)'':0.0183362,('PDT000069880.3':0.0237513,('PDT000124951.3':0.0207963,('PDT000161889.3':0.0111239,'PDT000311074.3':0.0136153)'':0.00659253)'':0.00248089)'':0.0143162)'':0.000513123,((('PDT000468264.2':0.0119523,'PDT000544260.2':0.0127953)'':0.0171672,(('PDT000285869.3':0.0274851,'PDT000625850.2':0.0200395)'':0.013248,('PDT000429539.2':0.0226628,(('PDT000160335.3':0.00692333,'PDT000464166.2':0.00717937)'':0.00693576,('PDT000616196.2':0.00898877,'PDT000641406.2':0.00846463)'':0.00176717)'':0.0113648)'':0.00724734)'':0.00480122)'':0.00142168,(('PDT000408655.2':0.0311107,('PDT000523446.2':0.0128903,('PDT000316488.2':0.0111719,'PDT000598408.2':0.0115306)'':0.00405966)'':0.00654075)'':0.000333774,(('PDT000315903.2':0.0130035,'PDT000328125.2':0.0154045)'':0.00905659,('PDT000484194.2':0.0224251,('PDT000319860.2':0.0259671,(('PDT000146541.3':0.0169259,'PDT000052314.1':0.0178661)'':0.0027143,('PDT000149639.3':0.0208512,('PDT000327109.2':0.0137426,'PDT000571614.2':0.01147)'':0.00316172)'':0.00377912)'':0.00307773)'':0.00135548)'':0.00140528)'':0.00389301)'':0.00620637)'':0.0044936)'':0.0267874)'':-0.0122675)'':0.0269774,(((('PDT000221986.3':0.0598375,('PDT000168363.3':0.0447038,'PDT000072708.3':0.0464311)'':0.0144875)'':-0.00101478,(('PDT000176762.3':0.0209155,'PDT000574474.2':0.0239249)'':0.0197274,('PDT000216648.3':0.0404343,('PDT000156960.3':0.0276087,'PDT000227274.3':0.0313699)'':0.00406645)'':0.00452607)'':0.00560761)'':0.0110949,(('PDT000546136.2':0.0453274,('PDT000146007.3':0.034507,'PDT000193134.3':0.0332089)'':0.00804472)'':-0.00163729,(('PDT000525856.2':0.0348588,(('PDT000564125.2':0.0182377,('PDT000145348.3':0.0197924,'PDT000372608.2':0.0187559)'':0.00237401)'':0.00364133,('PDT000469787.2':0.0230264,('PDT000213971.3':0.0184583,'PDT000334177.2':0.0295623)'':0.00126379)'':0.00341328)'':0.00432764)'':0.00305397,((('PDT000083998.3':0.0477385,('PDT000105121.3':0.0235057,('PDT000142643.3':0.0234908,'PDT000583683.2':0.026426)'':0.00422536)'':0.00885213)'':-0.0021546,(('PDT000239307.3':0.0220521,'PDT000404015.2':0.0229832)'':0.00281002,((('PDT000400034.2':0.0209516,'PDT000708506.1':0.017582)'':0.00266374,('PDT000518480.2':0.016806,('PDT000145373.3':0.0132264,('PDT000491946.2':0.0140713,('PDT000216630.3':0.0105042,'PDT000485870.2':0.0104755)'':0.00107218)'':0.00171842)'':0.00272679)'':0.000510471)'':-0.000354977,(('PDT000083976.3':0.0157413,('PDT000158571.3':0.0104857,'PDT000085593.3':0.0096492)'':0.00680111)'':0.001173,(('PDT000083963.3':0.0157745,'PDT000390262.2':0.0142947)'':0.000234293,('PDT000344816.2':0.0134679,('PDT000126294.3':0.0119941,('PDT000501747.2':0.0102887,('PDT000142865.3':0.00957191,'PDT000097091.3':0.00810089)'':0.00242701)'':0.00113254)'':0.00138641)'':0.00287032)'':0.000694873)'':0.000840468)'':0.00863519)'':-0.000621197)'':0.000125909,(('PDT000083222.3':0.0271196,'PDT000088591.3':0.021104)'':0.0196448,((('PDT000580818.2':0.0286398,'PDT000115419.3':0.0269429)'':0.00240033,(('PDT000147948.3':0.0249346,'PDT000545704.2':0.0259311)'':0.00135112,('PDT000553336.2':0.0224513,'PDT000136729.3':0.0233547)'':0.00336506)'':0.00251182)'':0.00415399,(('PDT000101115.3':0.0329067,(('PDT000525264.2':0.0206516,'PDT000542726.2':0.0177116)'':0.00249284,('PDT000213953.3':0.017621,('PDT000581245.2':0.0204581,'PDT000086170.3':0.014353)'':0.00403725)'':0.001954)'':0.00870027)'':-0.00160117,(('PDT000069437.4':0.0289002,('PDT000668182.2':0.0139375,('PDT000209872.3':0.00961354,'PDT000235362.3':0.00921886)'':0.00281029)'':0.0113671)'':0.00208438,(('PDT000497780.1':0.0243505,('PDT000707400.1':0.01963,('PDT000683653.1':0.022664,('PDT000242841.3':0.0185406,'PDT000389737.2':0.0185692)'':0.00266433)'':0.0010027)'':0.0040663)'':-0.00136475,(('PDT000417803.2':0.0256366,('PDT000234149.3':0.0183554,'PDT000280686.3':0.0171167)'':0.00438842)'':0.00239501,(('PDT000569066.2':0.0320972,('PDT000661995.2':0.0246739,('PDT000366105.2':0.0252965,(('PDT000152920.3':0.00594036,'PDT000105075.3':0.00707634)'':0.00955335,(('PDT000144421.3':0.0143263,'PDT000083283.3':0.0134186)'':0.0021075,(('PDT000307530.3':0.0109838,('PDT000238406.3':0.00986309,'PDT000265291.3':0.0123569)'':0.00195123)'':0.000385245,('PDT000150147.3':0.00965654,('PDT000488862.2':0.00824414,'PDT000081153.3':0.00735756)'':0.00177156)'':0.00254703)'':0.00217506)'':0.00273535)'':0.00438787)'':0.00264018)'':0.00343109)'':-0.00574736,('PDT000410943.2':0.0291158,(('PDT000216602.3':0.0158688,'PDT000584821.2':0.0143415)'':0.00387718,('PDT000491954.2':0.0241826,((('PDT000124963.3':0.0137286,('PDT000556770.2':0.0154903,'PDT000120014.3':0.0136945)'':2.75512e-05)'':0.00143793,('PDT000136752.3':0.019248,('PDT000045894.3':0.0180471,('PDT000267475.3':0.00721966,'PDT000303568.3':0.00768944)'':0.0047669)'':0.0026275)'':0.00069858)'':0.00108108,('PDT000647200.2':0.0181346,('PDT000135807.3':0.0164805,('PDT000072754.3':0.0143696,(('PDT000144046.3':0.0113119,'PDT000491905.2':0.0109565)'':0.000662735,('PDT000707392.1':0.01174,('PDT000209189.3':0.0109218,'PDT000108014.3':0.00987233)'':0.00129053)'':0.000774337)'':0.000950609)'':0.00269301)'':0.00248841)'':-0.00046715)'':0.00282299)'':-0.00142917)'':0.00598855)'':-0.00481468)'':0.002519)'':-0.000761568)'':0.00262969)'':-0.00074283)'':0.0019995)'':0.00509846)'':-0.00586965)'':0.00219044)'':0.0102676)'':0.00343321)'':-0.00917183,(((('PDT000707439.1':0.0331589,('PDT000145415.3':0.0340227,('PDT000147944.3':0.0236699,'PDT000342236.2':0.0197185)'':0.00792102)'':0.00374993)'':0.00191859,(('PDT000098722.3':0.0216331,('PDT000051242.5':0.0152822,'PDT000524432.2':0.0188233)'':0.00394136)'':0.00749907,(('PDT000334475.2':0.0198416,'PDT000407551.2':0.0202919)'':0.00279972,(('PDT000389803.2':0.0283016,('PDT000151483.3':0.0233381,('PDT000082134.3':0.0182478,('PDT000707445.1':0.0142379,('PDT000197842.3':0.0132193,'PDT000376051.2':0.0128702)'':0.00270519)'':0.00432085)'':0.00466172)'':0.00335602)'':-0.00343698,('PDT000491931.2':0.0320655,((('PDT000227339.3':0.0147177,'PDT000399427.2':0.0163443)'':0.00175729,('PDT000216608.3':0.0184248,'PDT000084053.3':0.0135934)'':0.00195379)'':0.00155539,('PDT000156843.3':0.0170436,('PDT000189679.3':0.0165399,('PDT000497513.2':0.013079,('PDT000404051.2':0.0125094,(('PDT000215101.3':0.00798524,'PDT000127709.3':0.00840656)'':0.000793048,('PDT000247158.3':0.00973322,'PDT000366290.2':0.0101776)'':0.000515227)'':0.00267823)'':0.000807762)'':0.0016149)'':0.000734352)'':-0.000190415)'':0.0125003)'':-0.00613588)'':0.00238387)'':0.0034029)'':0.000577461)'':-0.00297765,((('PDT000173941.3':0.0188773,'PDT000137561.4':0.0169706)'':0.00360815,('PDT000313028.3':0.0244917,('PDT000673735.1':0.0253488,'PDT000136734.3':0.0240062)'':0.00123967)'':0.000933573)'':0.00741069,(('PDT000077020.3':0.028745,('PDT000498042.1':0.0226277,('PDT000085619.3':0.015474,'PDT000707463.1':0.0127785)'':0.00488367)'':0.00983098)'':0.00058376,(('PDT000126002.3':0.0299266,('PDT000244742.3':0.0292247,('PDT000296050.3':0.0231724,('PDT000048671.6':0.0230171,(('PDT000338862.2':0.00861998,'PDT000388894.2':0.00814752)'':0.00654691,('PDT000669115.1':0.0162805,('PDT000082109.3':0.0105719,('PDT000329838.2':0.00702714,('PDT000491926.2':0.00223724,'PDT000072760.3':0.00235312)'':0.00631583)'':0.00425693)'':0.00257033)'':0.00122532)'':0.00183858)'':0.00141916)'':0.00277344)'':0.00855988)'':-0.00896784,('PDT000566407.2':0.034625,('PDT000548754.2':0.0219785,((('PDT000270011.3':0.0142766,'PDT000097030.3':0.0118782)'':0.00608616,('PDT000303457.3':0.0183771,'PDT000355510.2':0.0189738)'':0.00232944)'':0.00155034,('PDT000152925.3':0.0286997,(('PDT000242816.3':0.0193532,'PDT000722015.1':0.0198529)'':0.00210013,(('PDT000295979.3':0.023092,('PDT000351584.2':0.0156728,'PDT000602670.2':0.0165948)'':0.0050764)'':-0.000616676,('PDT000559293.2':0.0193694,(('PDT000190114.3':0.0133328,'PDT000491887.2':0.0179636)'':0.000281752,(('PDT000399371.2':0.0106204,'PDT000580803.2':0.0106583)'':0.00175151,('PDT000465870.2':0.0144374,('PDT000520103.2':0.00913743,('PDT000299791.3':0.00794135,('PDT000198411.4':0.00716376,'PDT000072712.3':0.00645184)'':0.000893704)'':0.00174762)'':0.00097412)'':0.000213819)'':0.00373097)'':0.00277744)'':-0.000359222)'':0.00102085)'':0.00379026)'':-0.0040018)'':0.00455883)'':0.0065448)'':-0.00941514)'':0.00652486)'':-0.000326511)'':-0.002006)'':0.00512467,(((('PDT000086315.3':0.0369151,('PDT000322431.2':0.0233707,'PDT000351836.2':0.0261362)'':0.00905794)'':0.00100173,(('PDT000275761.3':0.0247616,('PDT000627228.2':0.0235147,('PDT000367193.2':0.0239069,('PDT000247392.3':0.0116239,('PDT000653355.2':0.00542386,(('PDT000190987.3':0.00309994,'PDT000394060.2':0.00318686)'':0.0004398,('PDT000162999.4':0.00404009,('PDT000242804.3':0.00248483,'PDT000319656.2':0.00250419)'':0.000740977)'':0.00104306)'':0.00262148)'':0.00231159)'':0.0140715)'':0.00247407)'':0.00488223)'':0.000944454,(('PDT000270037.3':0.0397343,(('PDT000239284.3':0.0176539,'PDT000668686.1':0.0138001)'':0.00382456,('PDT000360385.2':0.018899,('PDT000707386.1':0.0199941,('PDT000340802.2':0.00439401,'PDT000372754.2':0.0044723)'':0.0134866)'':0.00170083)'':0.000162471)'':0.00836893)'':-0.00110046,(('PDT000147949.3':0.0359004,('PDT000708613.1':0.0218412,('PDT000240661.3':0.0173928,'PDT000125895.3':0.0138788)'':0.00629064)'':0.00405383)'':0.00282911,('PDT000351825.2':0.0216075,('PDT000689989.1':0.0156357,('PDT000152929.3':0.0129206,'PDT000216606.3':0.0138448)'':0.00506308)'':0.00246549)'':0.00641952)'':0.00142407)'':0.00213045)'':0.00547855)'':0.00018358,(('PDT000491988.2':0.0237826,('PDT000250830.3':0.0273261,((('PDT000334382.2':0.0120451,'PDT000127376.3':0.0125962)'':0.0072249,('PDT000313052.3':0.014482,('PDT000389788.2':0.0137733,('PDT000244732.3':0.00674673,'PDT000630453.2':0.00885037)'':0.00195089)'':0.00218801)'':0.00122211)'':-0.000211121,('PDT000265426.3':0.0152744,(('PDT000305199.3':0.0135808,'PDT000384087.2':0.00995804)'':0.00160389,('PDT000673718.1':0.0131012,('PDT000575648.2':0.0141593,('PDT000584141.2':0.0120808,('PDT000189536.3':0.0093193,('PDT000360433.2':0.00543506,('PDT000430546.2':0.00238396,('PDT000480450.2':0.00161814,'PDT000659074.2':0.00147814)'':0.00120939)'':0.00287773)'':0.0030923)'':0.00245964)'':0.000472758)'':0.00183598)'':-0.00143596)'':0.00369424)'':0.000103015)'':0.00312952)'':0.00449332)'':0.000116788,((('PDT000637278.2':0.0301101,'PDT000706526.1':0.037046)'':0.00273942,('PDT000383995.2':0.029668,('PDT000325164.2':0.0244619,'PDT000050483.3':0.0290972)'':0.002468)'':0.00270793)'':0.00538839,((('PDT000350411.2':0.0245475,'PDT000485440.2':0.0191193)'':0.00104802,(('PDT000101082.3':0.0186694,(('PDT000107433.3':0.0115286,('PDT000163623.3':0.00858316,('PDT000139808.3':0.00684979,('PDT000367730.2':0.00473801,'PDT000051371.3':0.00362769)'':0.00218211)'':0.0015382)'':0.0030068)'':-2.42577e-05,('PDT000410921.2':0.0115014,(('PDT000359185.2':0.00800637,('PDT000384116.2':0.00619922,'PDT000084040.3':0.00417798)'':0.000251984)'':0.0013855,('PDT000677072.1':0.00495077,('PDT000163611.3':0.00541047,('PDT000481030.2':0.006848,('PDT000491480.2':0.00404188,'PDT000616518.2':0.00294485)'':0.00101788)'':0.000514281)'':0.000246882)'':0.0031446)'':0.00240068)'':0.000917158)'':0.0062135)'':-0.00309527,((('PDT000145370.3':0.0108325,'PDT000707488.1':0.0102503)'':0.000208446,('PDT000086317.3':0.00983703,('PDT000235771.3':0.00872429,'PDT000083324.4':0.00813911)'':0.00234882)'':0.00182482)'':0.00221782,((('PDT000045891.3':0.0114582,'PDT000295945.3':0.0150843)'':0.00152912,(('PDT000311104.3':0.00517627,'PDT000394069.2':0.00596243)'':0.00170785,('PDT000663639.1':0.00162793,'PDT000669448.1':0.00156919)'':0.00255151)'':0.00712347)'':0.00217658,('PDT000499238.2':0.0172532,('PDT000587562.2':0.0106991,('PDT000687218.1':0.012252,(('PDT000156959.3':0.00421455,'PDT000136769.3':0.00495793)'':0.00208443,('PDT000171538.3':0.00445249,'PDT000289022.3':0.00411751)'':0.00236335)'':0.00140853)'':0.0013757)'':0.00671019)'':-0.00326367)'':0.00112039)'':-0.00014424)'':0.0076211)'':-0.00432473,(('PDT000040706.2':0.0152171,(('PDT000616667.2':0.00840373,'PDT000706093.1':0.00779427)'':0.005071,(('PDT000242613.3':0.00943695,'PDT000124965.3':0.00709365)'':0.0032989,('PDT000040690.3':0.0149913,'PDT000275660.3':0.0105836)'':0.00122015)'':0.00222742)'':0.00596651)'':-0.00199972,(('PDT000634631.2':0.0282279,('PDT000152699.3':0.0239626,'PDT000630381.2':0.0205364)'':0.00216649)'':0.00721772,(('PDT000115437.3':0.0210074,('PDT000604200.2':0.0159561,('PDT000065198.3':0.0124016,('PDT000163045.4':0.0110737,'PDT000644447.2':0.0114827)'':0.00337142)'':0.00152307)'':0.00156133)'':0.00265552,(('PDT000112877.3':0.017006,('PDT000108918.3':0.0166087,(('PDT000283367.3':0.0086246,'PDT000095669.3':0.0080454)'':0.00313875,('PDT000084138.3':0.0103817,('PDT000143272.3':0.00588108,'PDT000150124.3':0.011195)'':0.00305596)'':0.00118256)'':0.00241761)'':0.00302087)'':0.000910659,(((('PDT000366334.1':0.00941152,'PDT000417213.2':0.00944518)'':0.00204739,('PDT000227554.3':0.00897663,'PDT000227549.3':0.00966857)'':0.00243049)'':0.000791038,('PDT000137579.3':0.0203218,('PDT000139213.3':0.00910596,('PDT000234189.3':0.0115331,(('PDT000677692.1':0.00581275,'PDT000712539.1':0.00672455)'':0.00162983,('PDT000491920.2':0.00932414,('PDT000336938.2':0.00529545,'PDT000624688.2':0.00338718)'':0.00159629)'':0.00385248)'':0.00262673)'':0.000820749)'':0.00210343)'':0.000173902)'':0.0010724,('PDT000234199.3':0.0173094,(('PDT000221993.3':0.00703355,('PDT000048212.3':0.00762497,'PDT000649569.2':0.00652123)'':0.00102735)'':0.00560205,(('PDT000069403.4':0.0166555,('PDT000231248.3':0.0115395,'PDT000101022.3':0.0098594)'':0.00366964)'':0.000521893,('PDT000702570.1':0.015364,('PDT000106272.3':0.015348,('PDT000399385.2':0.0140273,('PDT000072790.3':0.0151235,(('PDT000400829.2':0.0100658,'PDT000566408.2':0.0112614)'':0.00142819,('PDT000233966.3':0.0130628,((('PDT000083997.3':0.00423528,'PDT000098744.3':0.00453228)'':0.00574683,('PDT000083995.3':0.00692317,'PDT000510646.2':0.00861313)'':0.000347538)'':0.00194031,(('PDT000669378.1':0.00982653,'PDT000086222.3':0.00871417)'':0.00113275,('PDT000101045.3':0.010417,(('PDT000495027.2':0.00761121,'PDT000502717.2':0.00767869)'':0.00259214,(('PDT000494256.2':0.00503387,('PDT000257230.3':0.00442957,'PDT000357224.2':0.00523853)'':0.00102872)'':0.00199917,(('PDT000127406.3':0.00831489,('PDT000431735.2':0.00783203,'PDT000088590.3':0.00971537)'':0.000605107)'':0.000320696,('PDT000176792.3':0.0133032,('PDT000624560.2':0.00855105,(('PDT000717143.1':0.00815303,(('PDT000242779.3':0.00297836,'PDT000197047.3':0.00450966)'':0.00129549,('PDT000690097.1':0.00651379,('PDT000083053.3':0.00486408,('PDT000147986.3':0.00379051,('PDT000625618.2':0.00318093,'PDT000121853.3':0.00329898)'':0.0010957)'':0.000938904)'':0.000633646)'':0.000196115)'':0.00267338)'':-0.00180525,(('PDT000163609.3':0.00457668,'PDT000141678.3':0.00698982)'':0.0046506,('PDT000257981.3':0.00763695,('PDT000283347.3':0.00594865,('PDT000385998.2':0.00359482,('PDT000146286.3':0.00499275,('PDT000585951.2':0.00321064,('PDT000243704.3':0.00248444,('PDT000135780.3':0.00256082,('PDT000250855.3':0.00177347,('PDT000412831.2':0.00139283,'PDT000083227.3':0.00100549)'':0.000621805)'':0.000722464)'':0.000546077)'':0.000470216)'':0.000245345)'':0.000593677)'':0.000377821)'':0.00142401)'':0.000421029)'':-0.000280946)'':0.00129267)'':0.001648)'':-0.00130241)'':0.000465552)'':0.000837841)'':0.00208409)'':-0.00112777)'':-0.000457439)'':0.00249052)'':-0.0013699)'':0.00242773)'':0.000874184)'':0.000330147)'':0.00136904)'':-0.00348198)'':0.000585227)'':0.0040748)'':-0.00387265)'':0.00151942)'':0.00244466)'':0.00400971)'':-0.00216427)'':-0.00193581)'':0.00828367)'':-0.00524653)'':0.00365715)'':-0.0029357,((('PDT000289813.3':0.042422,'PDT000126266.3':0.0399718)'':0.00496286,(('PDT000486053.2':0.030045,('PDT000086309.5':0.0253669,'PDT000071648.3':0.0210876)'':0.0103873)'':-0.000295558,(('PDT000102251.3':0.0233397,'PDT000125029.3':0.0371825)'':0.00693718,('PDT000069893.3':0.0232276,('PDT000296226.3':0.0210865,'PDT000127409.4':0.0196677)'':0.00461942)'':0.00412357)'':0.00434491)'':0.00455439)'':0.0108355,(((('PDT000254229.3':0.0116153,'PDT000130987.3':0.00856364)'':0.00824049,('PDT000198187.3':0.0150496,('PDT000189680.3':0.00799046,'PDT000212604.3':0.00498484)'':0.00697526)'':0.000302155)'':0.0158139,(('PDT000580729.2':0.0212958,('PDT000569697.2':0.01629,'PDT000084159.3':0.010627)'':0.00391075)'':0.00334549,('PDT000008721.2':0.023599,('PDT000306750.3':0.0171603,('PDT000150130.3':0.0113808,('PDT000491883.2':0.00874961,('PDT000707469.1':0.00809816,('PDT000225440.3':0.00631037,('PDT000084069.6':0.00565454,'PDT000063233.3':0.00442836)'':0.00158833)'':0.000981966)'':0.00131084)'':0.00238346)'':0.0030441)'':0.00815649)'':-0.00541469)'':0.003489)'':0.00468418,(((('PDT000148544.3':0.0228248,'PDT000239600.3':0.0240946)'':0.000173058,('PDT000600788.2':0.023652,('PDT000142631.3':0.0204067,'PDT000594289.2':0.0178205)'':0.0021744)'':0.00307254)'':0.00508508,(('PDT000221990.3':0.0325076,'PDT000388604.2':0.0296957)'':0.000599696,(('PDT000146297.3':0.0197712,'PDT000066850.3':0.0252303)'':0.00745432,('PDT000306533.3':0.0195304,('PDT000150133.3':0.0163382,(('PDT000677630.1':0.0141808,'PDT000251624.1':0.0103956)'':0.00215135,('PDT000491950.2':0.0138368,('PDT000594138.2':0.0118481,('PDT000366296.2':0.0131651,('PDT000221717.4':0.0084276,'PDT000404040.2':0.0074643)'':0.00414189)'':0.000994513)'':0.000845255)'':-5.86241e-05)'':0.00172828)'':0.00187613)'':0.00635283)'':0.00172146)'':-0.00171851)'':0.00367478,((('PDT000498038.1':0.038739,('PDT000498021.1':0.030799,(('PDT000201395.3':0.0188693,'PDT000212522.3':0.0195354)'':0.0047397,('PDT000069435.4':0.0223225,('PDT000163600.3':0.0262072,('PDT000141673.3':0.0214918,('PDT000238331.3':0.0148468,('PDT000071649.3':0.0137063,'PDT000101917.3':0.0144113)'':0.00213578)'':0.00629389)'':0.0031939)'':0.000705267)'':0.0010029)'':0.00449552)'':0.00124802)'':0.00102484,(('PDT000157513.3':0.0533198,('PDT000188035.3':0.0287435,(('PDT000149640.3':0.0194492,('PDT000327157.2':0.0165475,('PDT000083054.3':0.0183819,('PDT000083290.3':0.0124468,('PDT000312472.3':0.0137371,('PDT000233657.3':0.00887137,('PDT000102884.3':0.00806179,'PDT000708557.1':0.00751671)'':0.00022428)'':0.00182756)'':0.00285562)'':0.00239497)'':0.00128522)'':0.00373194)'':-0.00122606,('PDT000564148.2':0.0371961,(('PDT000559693.2':0.014076,'PDT000566609.2':0.0122372)'':0.00188894,('PDT000128458.3':0.0175967,('PDT000594286.2':0.0167256,('PDT000392132.2':0.0137281,('PDT000414391.2':0.0110517,('PDT000171602.3':0.00640311,('PDT000124771.3':0.00410869,('PDT000575829.2':0.00327169,'PDT000668682.1':0.00321145)'':0.000768913)'':0.000965927)'':0.00419117)'':0.00309135)'':0.000898013)'':0.0021696)'':-0.000440609)'':0.008429)'':-0.00613583)'':0.00919478)'':0.00852344)'':-0.0118993,(('PDT000501610.2':0.0279006,(('PDT000495248.2':0.0150511,'PDT000499252.2':0.00895047)'':0.00853277,('PDT000082092.3':0.0428341,('PDT000289815.3':0.0119613,('PDT000365939.2':0.00682446,('PDT000240601.3':0.004387,'PDT000544398.2':0.00359146)'':0.00240231)'':0.0022762)'':0.0137109)'':0.000386488)'':0.00961914)'':-0.00516891,((('PDT000499233.2':0.0156687,('PDT000372533.2':0.0134461,('PDT000086308.3':0.00939975,('PDT000319686.2':0.00886412,'PDT000114687.3':0.00821798)'':0.00181575)'':0.00451221)'':0.00169253)'':-0.000890696,(('PDT000443265.2':0.0160178,('PDT000108021.3':0.00865973,'PDT000141140.3':0.0107221)'':0.00341361)'':0.00174687,('PDT000334351.2':0.0153167,('PDT000492248.2':0.0130208,(('PDT000163791.3':0.0118534,'PDT000072713.3':0.00893846)'':0.000583309,(('PDT000190103.3':0.00866942,('PDT000146017.3':0.00672425,'PDT000491910.2':0.00503345)'':0.00320753)'':0.00378153,('PDT000281581.3':0.00999265,(('PDT000146288.3':0.00622176,('PDT000321164.2':0.0032727,'PDT000050466.3':0.00321044)'':0.00135604)'':0.00203189,(('PDT000190105.3':0.00453301,'PDT000289052.3':0.00773879)'':0.00376556,('PDT000575799.2':0.00938919,(('PDT000150669.3':0.00338962,'PDT000594674.2':0.00388492)'':0.00272242,('PDT000235357.3':0.00553518,('PDT000066867.3':0.00571923,('PDT000321932.2':0.00362797,'PDT000509040.2':0.00335388)'':0.000961652)'':0.000746398)'':0.000183737)'':0.00218874)'':1.66109e-05)'':5.80046e-05)'':0.00260509)'':-0.00097218)'':0.00124416)'':0.00212546)'':0.00500149)'':-0.00110325)'':-0.000970361)'':-0.000214332,(('PDT000469846.2':0.0113997,(('PDT000243724.3':0.00793162,('PDT000545700.2':0.00683733,'PDT000668527.2':0.00551207)'':0.00269428)'':0.00279631,('PDT000523849.2':0.0133097,('PDT000083984.3':0.0123621,('PDT000128745.3':0.0125712,(('PDT000362748.2':0.006635,'PDT000125513.3':0.0057058)'':0.00115209,(('PDT000392120.2':0.00544757,'PDT000102763.3':0.00499813)'':0.00172152,('PDT000693817.1':0.00876188,'PDT000128453.3':0.00628542)'':0.00064831)'':0.0014)'':0.00194976)'':0.000759872)'':0.00260923)'':-0.00191189)'':0.00308912)'':-0.000137888,('PDT000340508.2':0.0273785,('PDT000586444.2':0.0217693,('PDT000062525.3':0.0167957,(('PDT000521275.2':0.0120618,('PDT000209193.3':0.0102159,('PDT000465812.2':0.00657048,(('PDT000218521.3':0.00672355,'PDT000065200.3':0.00778365)'':0.00094561,('PDT000198184.3':0.00736395,('PDT000086219.3':0.00808096,(('PDT000376012.2':0.00726348,'PDT000649705.2':0.00549062)'':0.000421788,('PDT000381321.2':0.0085415,('PDT000051213.3':0.00519957,('PDT000584843.2':0.00474143,('PDT000491938.2':0.00169951,'PDT000107822.3':0.00219698)'':0.00133836)'':0.00168428)'':0.000625407)'':-0.000110459)'':0.00127483)'':0.000879829)'':-0.000847107)'':0.000765211)'':0.000706267)'':0.00133704)'':-0.00162826,(('PDT000385927.2':0.00863495,'PDT000509620.2':0.00811815)'':0.00396472,(('PDT000682500.1':0.00647958,('PDT000521246.2':0.00493093,'PDT000136765.3':0.00355719)'':0.00306046)'':0.00278114,(('PDT000312466.3':0.00698395,'PDT000072442.3':0.00555335)'':0.00260768,('PDT000646909.2':0.0160038,(('PDT000171589.3':0.00512825,'PDT000247519.3':0.00762715)'':0.00188466,('PDT000200972.3':0.00665142,(('PDT000040385.4':0.0107513,(('PDT000356237.2':0.00297047,'PDT000386245.2':0.00192317)'':0.00378653,('PDT000707491.1':0.00396093,('PDT000275754.3':0.0039614,('PDT000394735.2':0.00243912,'PDT000052124.3':0.00365357)'':0.0010159)'':0.00126023)'':0.000460094)'':0.0012993)'':-0.00108895,(('PDT000555873.2':0.00318945,('PDT000240577.3':0.00112435,'PDT000279477.3':0.00137315)'':0.0020015)'':0.00213,('PDT000227347.3':0.00402276,('PDT000501745.2':0.00368665,('PDT000480393.2':0.00434327,(('PDT000215056.3':0.00241883,'PDT000404008.2':0.00187216)'':0.000916842,('PDT000082086.3':0.00206933,('PDT000547417.2':0.00139224,'PDT000126284.3':0.00140524)'':0.0011239)'':3.81595e-05)'':0.000682763)'':0.000944572)'':0.000671365)'':-0.000424497)'':0.000561327)'':0.00202272)'':-0.000231695)'':0.00342832)'':-0.0023443)'':0.000519208)'':0.00278753)'':-0.00230195)'':0.00905791)'':0.000851386)'':0.00030829)'':-0.00821551)'':0.00159126)'':0.00995709)'':-0.00457954)'':0.00499672)'':-0.00283666,((('PDT000377960.2':0.0285524,('PDT000051243.3':0.0266514,('PDT000044503.3':0.0261835,('PDT000242807.3':0.0266261,('PDT000672560.1':0.0157446,'PDT000137668.3':0.0277943)'':0.00510312)'':0.00279193)'':0.00276678)'':0.00274612)'':-0.00192043,('PDT000161534.3':0.0304484,('PDT000498045.1':0.0292033,(('PDT000316451.2':0.0186306,'PDT000498072.1':0.0175913)'':0.00376552,(('PDT000248192.3':0.0230511,'PDT000549661.2':0.0160269)'':0.00492396,('PDT000414690.2':0.0208064,('PDT000077042.3':0.0269566,('PDT000201672.3':0.0192756,('PDT000094748.3':0.0125238,'PDT000065187.3':0.0164358)'':0.00137163)'':0.00408483)'':0.00210468)'':0.000382238)'':0.00115261)'':0.00576498)'':0.0028014)'':-0.00369577)'':0.0073307,(((('PDT000498016.1':0.0201658,'PDT000498185.1':0.0161026)'':0.00115166,('PDT000044781.3':0.0150944,('PDT000009621.2':0.017041,('PDT000069882.3':0.0128317,('PDT000051169.1':0.00792633,('PDT000066883.3':0.00763999,('PDT000107153.3':0.00887978,'PDT000124954.3':0.00819392)'':0.00165651)'':0.000781505)'':0.00197016)'':0.00151201)'':0.00039873)'':0.00199119)'':0.00245375,('PDT000498083.1':0.0290112,('PDT000115249.3':0.02625,('PDT000689974.1':0.0202708,('PDT000260784.3':0.0185796,('PDT000063244.3':0.0215883,('PDT000137689.3':0.0103685,('PDT000248529.1':0.0136899,('PDT000497781.1':0.00973746,('PDT000101370.3':0.00836568,('PDT000076994.3':0.01039,('PDT000683648.1':0.00928972,('PDT000222750.3':0.00785547,(('PDT000417221.2':0.00643269,'PDT000137679.3':0.00611091)'':0.000441636,(('PDT000344795.2':0.00480053,('PDT000044508.3':0.00376709,'PDT000120808.3':0.00390835)'':0.0012644)'':0.000720259,('PDT000248537.1':0.0068134,('PDT000081066.1':0.00472562,('PDT000497296.2':0.00396373,('PDT000191194.1':0.00432775,'PDT000008720.2':0.00315579)'':0.00121754)'':0.000823316)'':0.000526646)'':0.000814846)'':0.00048016)'':0.00103038)'':0.000582542)'':0.000213375)'':0.000281779)'':0.00117293)'':0.00104096)'':0.000612892)'':0.0015097)'':0.000496212)'':0.0030517)'':0.00373419)'':0.00117077)'':-0.00592983)'':0.00434417,(((('PDT000552692.2':0.016448,'PDT000391618.2':0.0181407)'':0.00417392,('PDT000123344.1':0.0107808,('PDT000598486.2':0.00981548,'PDT000653286.2':0.0124753)'':0.00147048)'':0.0070563)'':0.00342898,(('PDT000065380.1':0.0221563,('PDT000212533.3':0.00909294,'PDT000082123.3':0.012765)'':0.00599738)'':0.00469167,(('PDT000143252.3':0.0224344,'PDT000351929.2':0.0224775)'':0.00234189,(('PDT000307498.3':0.0172463,'PDT000412114.2':0.0130867)'':0.00460686,('PDT000201220.3':0.0178515,('PDT000212454.3':0.0172377,('PDT000191197.1':0.00962113,'PDT000223305.1':0.0126694)'':0.00262865)'':0.00310637)'':0.00210948)'':0.00496814)'':0.000865593)'':0.00113774)'':2.08647e-05,(('PDT000672395.1':0.0321999,(('PDT000580736.2':0.0208937,('PDT000389740.2':0.0138638,'PDT000127277.4':0.0119599)'':0.00183804)'':0.0045058,('PDT000491933.2':0.0209384,(('PDT000399894.2':0.0172044,'PDT000084207.3':0.0130148)'':0.00212415,(('PDT000191138.3':0.0135844,'PDT000319669.2':0.0136399)'':0.00174946,('PDT000264724.3':0.0144026,'PDT000124936.3':0.0137317)'':0.000933515)'':0.00250902)'':0.00176279)'':-0.000882603)'':0.00707976)'':-0.000453226,(('PDT000162982.4':0.0158321,('PDT000112514.3':0.0202278,('PDT000259549.3':0.00532975,('PDT000264683.3':0.00452751,('PDT000085625.3':0.00348388,'PDT000102894.3':0.00319941)'':0.00154076)'':0.00153328)'':0.00393707)'':0.00421765)'':0.0067015,((('PDT000312500.3':0.0220093,(('PDT000214070.3':0.0200876,('PDT000365941.2':0.010954,'PDT000084020.3':0.00935911)'':0.00721825)'':-0.000558151,(('PDT000148499.3':0.011906,'PDT000520007.2':0.0170968)'':0.0018356,('PDT000575962.2':0.0151377,('PDT000213934.3':0.0137112,('PDT000085609.3':0.0110148,('PDT000259526.3':0.0100472,('PDT000084217.3':0.00817202,('PDT000259785.3':0.00737551,'PDT000407646.2':0.00567229)'':0.00144038)'':0.0022601)'':0.000585398)'':0.00256501)'':0.000479511)'':0.000764593)'':0.00107547)'':0.00377239)'':-0.00223036,(('PDT000270093.3':0.0199059,('PDT000137577.3':0.0217225,('PDT000147960.3':0.0118142,'PDT000127711.3':0.0117268)'':0.00582534)'':0.0037672)'':0.000938641,(('PDT000163044.4':0.0227867,('PDT000163568.3':0.0181821,('PDT000295974.3':0.0130895,'PDT000356225.2':0.0129295)'':0.00327171)'':0.00187334)'':0.00103496,(('PDT000083330.11':0.0073987,'PDT000086169.3':0.0064363)'':0.00607808,(('PDT000203827.3':0.0128099,'PDT000137720.3':0.00860664)'':0.00641203,('PDT000135357.3':0.0160272,(('PDT000634609.2':0.0138014,'PDT000084232.3':0.0110895)'':0.000926486,('PDT000085608.3':0.0140093,(('PDT000069878.3':0.0138326,('PDT000491928.2':0.00637408,('PDT000579989.2':0.00544822,'PDT000682951.1':0.00648968)'':0.00158642)'':0.0013732)'':0.000835255,(('PDT000598823.1':0.0076102,('PDT000596948.2':0.00745477,('PDT000367236.2':0.00269927,('PDT000491940.2':0.00162706,'PDT000525242.2':0.00156783)'':0.00149448)'':0.00112993)'':0.00121823)'':0.0010765,('PDT000189669.3':0.0166998,(('PDT000145274.3':0.0107413,'PDT000387614.2':0.0103395)'':0.000757328,(('PDT000147844.3':0.00681322,('PDT000318464.2':0.00474442,'PDT000616671.2':0.00533038)'':0.00307418)'':-0.000309449,('PDT000193098.3':0.016537,('PDT000351572.2':0.00549921,(('PDT000184517.3':0.00313591,'PDT000641699.2':0.00295435)'':0.00269199,('PDT000216671.3':0.00432338,(('PDT000707435.1':0.00394478,'PDT000096922.3':0.00451967)'':0.000721389,('PDT000284043.3':0.00390322,('PDT000108013.3':0.00371759,('PDT000645929.2':0.00233843,('PDT000451081.2':0.00194216,('PDT000233881.3':0.00212043,('PDT000145774.3':0.00187544,'PDT000540014.2':0.00172025)'':0.000375092)'':6.24517e-05)'':0.000777248)'':0.000832031)'':0.000804997)'':-0.000420723)'':0.000813128)'':3.93812e-05)'':0.000519764)'':0.0030317)'':-0.00191439)'':0.00180675)'':0.00188281)'':-0.00259662)'':0.00180846)'':0.00315746)'':-0.000713429)'':0.00491277)'':-0.00333236)'':0.00153179)'':0.000781244)'':0.000681594)'':-0.00132381)'':0.00302439,(('PDT000479291.2':0.027385,((('PDT000201160.3':0.0146102,'PDT000357229.2':0.0163431)'':0.00262073,('PDT000539923.2':0.00906256,'PDT000141132.3':0.0108779)'':0.00802289)'':0.000697446,('PDT000123252.1':0.0196262,('PDT000045531.3':0.0153954,('PDT000224222.3':0.0120636,('PDT000151495.3':0.0073439,'PDT000081888.3':0.008365)'':0.00476454)'':0.00273984)'':0.00197201)'':0.000367233)'':0.00921152)'':-0.00319048,(('PDT000706357.1':0.023373,('PDT000366315.2':0.0223493,(('PDT000188005.3':0.0126775,'PDT000159011.3':0.0133261)'':4.0487e-05,('PDT000575647.2':0.0142119,('PDT000431674.2':0.0103847,('PDT000551393.2':0.0134195,('PDT000088592.3':0.00768112,('PDT000344810.2':0.00673907,('PDT000296017.3':0.00655187,('PDT000096895.3':0.00660201,('PDT000647780.2':0.00422435,('PDT000599895.2':0.00562457,('PDT000216676.3':0.0032512,'PDT000329286.2':0.00323259)'':0.000765245)'':0.000303414)'':0.00151153)'':0.00118251)'':0.000458463)'':0.00022114)'':0.00152862)'':0.000537791)'':0.000452786)'':0.00203673)'':0.00598858)'':0.00131257)'':-0.00150211,((('PDT000267468.3':0.0209367,('PDT000303550.3':0.0181152,'PDT000532758.2':0.0184678)'':0.00715475)'':0.000491713,('PDT000176665.3':0.0169028,(('PDT000355473.2':0.0146029,('PDT000499301.2':0.0115398,('PDT000254232.3':0.0106589,'PDT000386258.2':0.0106135)'':0.000623202)'':0.00236118)'':0.000202235,('PDT000161636.3':0.0152049,('PDT000216628.3':0.0128314,('PDT000115414.3':0.0145735,('PDT000130998.3':0.0111963,(('PDT000351803.2':0.00997359,'PDT000432442.2':0.00886631)'':0.00139871,('PDT000072427.3':0.0100463,('PDT000188046.3':0.00943977,('PDT000125025.3':0.00965186,('PDT000083252.3':0.00907787,('PDT000491908.2':0.00823505,('PDT000176756.3':0.00695524,'PDT000081160.3':0.00539166)'':0.000570502)'':0.00124675)'':0.0010547)'':0.000391528)'':0.000634128)'':0.000204121)'':0.00027202)'':0.00179283)'':0.00041529)'':0.00205355)'':-0.000613139)'':0.00198223)'':0.00610062)'':-0.00492356,(('PDT000311426.3':0.0136972,'PDT000125996.3':0.0122075)'':0.0127209,(('PDT000564216.2':0.0209368,('PDT000707385.1':0.0191142,(('PDT000250834.3':0.0111556,'PDT000426870.2':0.00959292)'':0.00274664,('PDT000150676.3':0.0112814,('PDT000152923.3':0.00850922,'PDT000101055.3':0.00795908)'':0.00286836)'':0.000121984)'':0.0035098)'':0.00560085)'':-0.00534549,(('PDT000467104.2':0.0184255,(('PDT000190665.3':0.0111601,'PDT000360422.2':0.00969813)'':0.00324219,('PDT000083987.3':0.015324,('PDT000251632.1':0.0112956,('PDT000157470.3':0.00706176,'PDT000491936.2':0.0109499)'':0.00355258)'':0.0021549)'':0.000626845)'':0.0047408)'':-0.00246527,((('PDT000426895.2':0.0130104,('PDT000084035.3':0.0117844,('PDT000336798.2':0.00846474,'PDT000342388.2':0.00869456)'':0.00226154)'':0.00177705)'':0.0083039,('PDT000532759.2':0.0250596,(('PDT000672750.1':0.00771278,'PDT000121849.3':0.00973662)'':0.00185073,('PDT000048675.3':0.0135251,('PDT000625636.2':0.0142963,('PDT000637182.2':0.014774,('PDT000465826.2':0.00895257,(('PDT000598851.2':0.00519353,('PDT000096888.3':0.00466047,'PDT000110748.3':0.00519812)'':0.000977934)'':0.00307706,('PDT000386153.2':0.00984193,('PDT000218537.3':0.00820929,(('PDT000486014.2':0.00746118,('PDT000125505.3':0.00512772,('PDT000503059.2':0.00463819,('PDT000120028.3':0.0046017,('PDT000191630.3':0.00457837,('PDT000465820.2':0.00250951,('PDT000147945.3':0.00241098,'PDT000150685.3':0.00198024)'':0.000482887)'':0.000713411)'':0.000645745)'':0.000468589)'':0.00102099)'':0.000306196)'':4.38187e-05,(('PDT000365742.2':0.00618136,'PDT000580730.2':0.00495964)'':0.00117492,('PDT000532773.2':0.00858915,(('PDT000277735.3':0.00541458,'PDT000050476.3':0.00454161)'':0.00122678,('PDT000190668.3':0.0102723,('PDT000101083.3':0.00548658,('PDT000396653.2':0.00473664,('PDT000658820.2':0.00624426,((('PDT000177066.3':0.00236249,'PDT000491939.2':0.0020296)'':0.000526893,('PDT000114532.3':0.0019638,'PDT000125546.3':0.00222907)'':0.000468577)'':0.000417154,('PDT000571794.2':0.00351423,('PDT000259777.3':0.003008,('PDT000062536.3':0.00214511,('PDT000275643.3':0.00226002,('PDT000494486.2':0.00135806,('PDT000225443.3':0.00163228,'PDT000107752.3':0.00176195)'':0.000242674)'':0.000452734)'':0.000331969)'':0.000815337)'':0.000754268)'':-0.000378564)'':0.00135832)'':0.000527965)'':0.000528013)'':0.00032346)'':-0.000327966)'':0.00101983)'':-0.00022379)'':-0.00082113)'':0.0023652)'':0.000135955)'':-0.00105765)'':0.00208941)'':0.00108546)'':0.00118533)'':0.00048364)'':-0.002256)'':0.0105706)'':-0.00754554)'':-0.00257066,(('PDT000096902.3':0.0217581,('PDT000388602.2':0.0121455,('PDT000235783.3':0.0123125,'PDT000343555.2':0.0101549)'':0.00165162)'':0.00650877)'':-0.00416683,(('PDT000051369.3':0.0218323,'PDT000648855.2':0.0187523)'':0.00254813,(('PDT000283340.3':0.0124681,'PDT000466061.2':0.0128509)'':0.00258915,(('PDT000214031.3':0.0114245,'PDT000271207.3':0.0121282)'':0.00144406,('PDT000132175.3':0.0106096,('PDT000431719.2':0.0118258,(('PDT000366304.2':0.0088188,('PDT000486874.2':0.00190732,'PDT000650494.2':0.00208589)'':0.00529189)'':0.000742163,((('PDT000356216.2':0.00576275,('PDT000197038.3':0.00561073,'PDT000342322.2':0.00723277)'':0.000310797)'':0.00163155,('PDT000342246.2':0.00600869,(('PDT000144496.3':0.00557954,('PDT000394068.2':0.00415509,'PDT000126371.3':0.00402283)'':0.000883201)'':0.000706859,('PDT000660094.2':0.00709311,('PDT000640182.2':0.00382295,('PDT000069876.3':0.00315361,('PDT000221716.3':0.00277396,'PDT000083281.3':0.00241359)'':0.000632772)'':0.000789587)'':0.000871834)'':0.000567172)'':0.00104985)'':0.000976317)'':0.000132012,(('PDT000279465.3':0.00575751,('PDT000315822.2':0.0029119,'PDT000191179.1':0.0035829)'':0.00345034)'':0.0019441,(('PDT000226813.3':0.00479491,'PDT000390970.2':0.00427245)'':0.00224461,('PDT000598830.1':0.0111853,(('PDT000432449.2':0.00358928,'PDT000191186.1':0.00460007)'':0.00180368,('PDT000247430.3':0.0109749,('PDT000334384.2':0.00561706,('PDT000625138.2':0.00466299,('PDT000714184.1':0.00589959,('PDT000707441.1':0.00418101,(('PDT000539908.2':0.0024281,'PDT000123340.1':0.00256641)'':0.000432723,('PDT000264755.3':0.00278266,('PDT000188049.3':0.00135732,'PDT000133762.3':0.0015392)'':0.00150718)'':0.000389821)'':0.000962991)'':0.000874784)'':0.000873209)'':0.000742843)'':0.000272822)'':-0.000383206)'':0.00130388)'':-0.000296382)'':-3.37453e-05)'':-4.80508e-05)'':0.000912449)'':0.00136454)'':0.00161527)'':-0.000713686)'':0.00070836)'':0.00353289)'':-0.000373272)'':-0.00307169)'':0.00207829)'':0.00138458)'':0.00830031)'':-0.00757773)'':0.00200629)'':0.00176823)'':-0.00278419)'':0.00310242)'':0.000491761)'':-2.36464e-05)'':-0.000349545)'':0.00698405)'':-0.00629747)'':0.00129215)'':0.00384616)'':0.0056005)'':-0.00982301)'':0.0038715)'':0.00071489)'':0.0384001)'':-0.0211215,(('PDT000251471.1':0.0827555,(((('PDT000465604.2':0.0294469,(('PDT000150678.3':0.0101143,'PDT000134069.3':0.0110839)'':0.00430804,(('PDT000469027.2':0.00991221,'PDT000540021.2':0.00963149)'':0.00083773,('PDT000683664.1':0.0100387,('PDT000537348.2':0.00847197,('PDT000587626.2':0.00550931,'PDT000645911.2':0.00663969)'':0.00207613)'':0.00412723)'':0.000156687)'':0.00178259)'':0.00853343)'':0.00129926,(('PDT000146041.3':0.0204888,'PDT000158566.3':0.0184697)'':0.0086285,('PDT000333925.2':0.0216673,('PDT000600709.2':0.0141627,('PDT000385975.2':0.0196629,('PDT000569572.2':0.0145462,('PDT000373149.2':0.0124684,('PDT000184463.3':0.0111529,'PDT000235303.3':0.00808363)'':0.00203723)'':0.00194814)'':0.00152112)'':0.00265085)'':0.00115065)'':0.000691605)'':0.00225155)'':0.00133501,(('PDT000481320.2':0.0265181,('PDT000044720.3':0.0302318,('PDT000156850.3':0.0128489,(('PDT000491960.2':0.00738336,'PDT000101026.3':0.00585574)'':0.00632784,('PDT000643047.2':0.0118286,('PDT000537333.2':0.0107636,('PDT000671177.1':0.00575831,('PDT000259534.3':0.00447197,'PDT000066864.3':0.00730333)'':0.00131239)'':0.00445397)'':0.00147155)'':0.000253162)'':0.00285671)'':0.00112401)'':0.0113578)'':-0.00993554,(('PDT000303475.3':0.0328692,('PDT000271189.2':0.0221189,('PDT000237101.3':0.0121714,('PDT000309575.3':0.00539393,'PDT000131129.3':0.00545507)'':0.00325386)'':0.00551255)'':0.00763684)'':-0.00288955,(('PDT000265132.3':0.0317499,'PDT000303555.3':0.033981)'':0.0127316,(('PDT000375792.2':0.0227498,('PDT000621130.2':0.0221349,'PDT000679497.1':0.0182274)'':0.00400159)'':0.00324167,(('PDT000084183.3':0.0228748,('PDT000312506.3':0.0158119,'PDT000569403.2':0.0221715)'':0.00122815)'':0.0106364,((('PDT000259536.3':0.0244661,'PDT000360398.2':0.0236042)'':0.0033205,(('PDT000491901.2':0.0117346,'PDT000630375.2':0.00738701)'':0.00392436,('PDT000135796.3':0.0148533,(('PDT000682521.1':0.0102156,('PDT000505501.2':0.00985705,('PDT000537369.2':0.0104193,'PDT000575819.2':0.00670875)'':0.0038833)'':0.000546469)'':0.00297909,(('PDT000222572.3':0.0123471,('PDT000120010.3':0.0134819,('PDT000102878.3':0.0101877,('PDT000396581.2':0.00665025,'PDT000491930.2':0.00656255)'':0.00121407)'':0.000516881)'':0.000553998)'':0.000165658,('PDT000083177.3':0.0152259,(('PDT000522419.2':0.00766399,'PDT000707389.1':0.00762741)'':0.00353885,('PDT000520106.2':0.0101378,('PDT000227360.3':0.00747795,('PDT000574070.2':0.0092595,('PDT000150144.3':0.00408767,('PDT000386108.2':0.00569318,'PDT000429531.2':0.00565352)'':0.000610704)'':0.00125371)'':0.00115238)'':0.000471843)'':0.000433664)'':0.00260829)'':-0.000806708)'':0.00096977)'':0.00369658)'':-0.00139086)'':0.00612645)'':-0.00181763,(('PDT000494841.2':0.0206503,('PDT000112497.3':0.0243447,('PDT000707589.1':0.0115817,('PDT000342318.2':0.00986351,('PDT000556701.2':0.00769608,'PDT000131017.3':0.00870392)'':0.00107139)'':0.00075118)'':0.0108871)'':0.00216361)'':-0.00104174,((('PDT000649664.2':0.0123882,'PDT000707383.1':0.0146656)'':0.00520251,('PDT000490191.2':0.0178287,'PDT000313236.3':0.0219899)'':0.00122119)'':0.00292219,(('PDT000289039.3':0.0218442,('PDT000289125.3':0.0136306,('PDT000303456.3':0.00816774,'PDT000501000.2':0.010108)'':0.00535023)'':0.00509008)'':-0.00164251,('PDT000359215.2':0.0203114,((('PDT000459010.2':0.00873909,'PDT000197121.3':0.00951121)'':0.000125147,('PDT000481324.2':0.00666558,('PDT000189655.3':0.00537574,'PDT000396563.2':0.00657406)'':0.00112652)'':0.00180801)'':0.00271075,(('PDT000082157.3':0.0169528,('PDT000281560.3':0.0110598,'PDT000486337.2':0.0110216)'':0.00254619)'':0.00108891,('PDT000334401.2':0.0167267,(('PDT000131131.3':0.011555,('PDT000144023.3':0.00809335,('PDT000213923.3':0.0103284,('PDT000637262.2':0.00556759,'PDT000682695.1':0.00479511)'':0.00131695)'':0.00270295)'':0.00227934)'':0.000999086,(('PDT000355458.2':0.0148408,('PDT000336943.2':0.0122459,('PDT000157472.3':0.0074022,'PDT000160125.3':0.0081825)'':0.00240905)'':0.00281675)'':0.000413906,(('PDT000375698.2':0.0184125,'PDT000690530.1':0.0125337)'':0.0025331,('PDT000084166.3':0.0191329,('PDT000309570.3':0.0151494,(('PDT000205377.3':0.00905287,(('PDT000151519.3':0.0063647,'PDT000062610.3':0.0079808)'':0.000734556,('PDT000084044.3':0.00613958,('PDT000221623.3':0.00583665,'PDT000383974.2':0.00531655)'':0.000919015)'':0.00161746)'':0.00189213)'':-0.000849816,(('PDT000233634.3':0.0125206,'PDT000235313.3':0.00643388)'':0.00128054,(('PDT000281525.3':0.00414816,'PDT000443220.2':0.00334135)'':0.00199488,('PDT000548713.2':0.0120268,('PDT000466066.2':0.0110098,('PDT000366076.2':0.00730049,('PDT000312461.3':0.0115693,(('PDT000334500.2':0.00310508,('PDT000096889.3':0.00227828,('PDT000566651.2':0.00167663,'PDT000690043.1':0.00161876)'':0.000616052)'':0.000677083)'':0.00112619,('PDT000316162.2':0.0066703,('PDT000285842.3':0.00432287,(('PDT000404028.2':0.00388148,('PDT000101037.3':0.00308226,'PDT000708751.1':0.00430444)'':0.000601906)'':7.90405e-05,(('PDT000275629.3':0.00487823,'PDT000309674.3':0.00310182)'':0.000249328,('PDT000491958.2':0.00293285,'PDT000136738.3':0.0022604)'':0.00194131)'':0.00024681)'':0.000563977)'':0.00196107)'':-0.00133956)'':0.00201419)'':0.00025868)'':0.000598229)'':0.000413404)'':-0.00120279)'':0.00160179)'':-0.000933788)'':0.00449436)'':0.000816539)'':-0.00207438)'':-0.000887926)'':0.000312227)'':0.00332834)'':-0.00145535)'':0.000168657)'':0.00476539)'':-0.00274151)'':0.00240528)'':0.000938324)'':-0.00320847)'':0.00484893)'':0.000877298)'':0.0039409)'':-0.00409582)'':-0.00111834)'':0.00222252)'':0.00979535,((('PDT000145380.3':0.0230859,'PDT000119988.3':0.0217589)'':0.00642837,('PDT000557533.1':0.0106654,'PDT000248743.1':0.00915928)'':0.0286856)'':0.00791749,((('PDT000550870.2':0.0487172,('PDT000133142.2':0.0233787,('PDT000547695.2':0.01564,('PDT000707396.1':0.00542523,'PDT000096841.3':0.00622237)'':0.0145773)'':0.00565401)'':0.0070992)'':0.0013719,(('PDT000274286.3':0.032716,('PDT000451150.2':0.0193965,('PDT000545698.2':0.014168,(('PDT000190735.3':0.0100872,'PDT000368369.2':0.0114898)'':0.00405368,('PDT000359179.2':0.00831441,('PDT000306767.3':0.00794261,'PDT000500464.2':0.00823449)'':0.00041924)'':0.00428987)'':0.00283568)'':0.00457184)'':0.00382128)'':0.00474099,((('PDT000499193.2':0.0247143,'PDT000235049.3':0.0228163)'':0.00355272,('PDT000163377.4':0.0144873,'PDT000132452.3':0.0135131)'':0.00830096)'':0.00336054,('PDT000389754.2':0.0304517,('PDT000147975.3':0.0253968,('PDT000266788.3':0.02361,('PDT000097027.3':0.0160786,('PDT000227575.3':0.0100085,'PDT000480977.2':0.00615761)'':0.00656019)'':0.00278825)'':0.00314597)'':0.00386763)'':0.00266845)'':0.00392374)'':0.0021173)'':0.00149481,((('PDT000115411.3':0.0221746,('PDT000101369.3':0.0164666,('PDT000030574.1':0.019078,('PDT000242778.3':0.00828292,('PDT000465617.2':0.0143563,('PDT000387606.2':0.000956002,'PDT000492056.2':0.0010428)'':0.00668487)'':0.00306331)'':0.00737589)'':0.0015194)'':0.00228574)'':0.00591666,(((('PDT000123476.3':0.0166325,('PDT000063286.3':0.00939941,'PDT000124800.3':0.0126048)'':0.00422351)'':0.00620308,(('PDT000672179.1':0.0152234,'PDT000705931.1':0.0150467)'':0.000460369,('PDT000677290.1':0.0168388,'PDT000718485.1':0.012272)'':0.00360838)'':0.00813137)'':0.00373093,(('PDT000600716.2':0.0204902,'PDT000130554.3':0.0219454)'':0.00487433,(('PDT000084045.3':0.0108923,('PDT000198057.3':0.00822061,'PDT000662029.1':0.00904819)'':0.00576714)'':0.00390041,(('PDT000077508.1':0.0179529,('PDT000151456.3':0.0164604,('PDT000384077.2':0.00785989,'PDT000082120.3':0.0152799)'':0.00252097)'':0.00422232)'':0.00152042,('PDT000085623.3':0.0155563,('PDT000651894.2':0.0132293,('PDT000084961.3':0.0107783,('PDT000084098.3':0.0105081,'PDT000532766.2':0.0115026)'':0.00119245)'':0.00323652)'':0.000336865)'':0.00145153)'':0.00177016)'':0.00608756)'':0.000500281)'':0.00035005,(('PDT000499148.2':0.048527,('PDT000253174.3':0.026631,(('PDT000465601.2':0.0262998,('PDT000573756.2':0.016511,'PDT000651808.2':0.00821243)'':0.00772637)'':0.00213404,('PDT000071664.3':0.0139325,('PDT000708841.1':0.00817918,('PDT000319662.2':0.0106413,('PDT000295972.3':0.00748433,'PDT000571224.2':0.00458067)'':0.00437814)'':0.00228143)'':0.00404089)'':-0.000210384)'':0.00720803)'':0.0113655)'':-0.00671777,('PDT000625315.2':0.0314843,('PDT000329851.2':0.0220234,('PDT000429534.2':0.0193923,(('PDT000071657.3':0.0204105,('PDT000120803.3':0.0137747,('PDT000385876.2':0.0080837,'PDT000487639.2':0.0076236)'':0.00332621)'':0.00232202)'':-0.000304128,('PDT000389781.2':0.0179051,(('PDT000102898.3':0.0106148,('PDT000576033.2':0.00315695,'PDT000096917.3':0.00323155)'':0.00851176)'':0.00235735,(('PDT000430565.2':0.0091012,'PDT000465785.2':0.0083273)'':0.000952505,('PDT000212341.3':0.0127187,('PDT000311112.3':0.0117603,('PDT000259779.3':0.00495748,'PDT000120011.3':0.00431101)'':0.00336462)'':0.00201715)'':-0.00025207)'':0.00146799)'':0.00265201)'':0.00155204)'':0.00448298)'':0.00320895)'':0.00143059)'':-0.00351606)'':0.00141458)'':0.00230062)'':0.000304706,(('PDT000307504.3':0.0393379,('PDT000396657.2':0.0374154,('PDT000225524.3':0.0270691,(('PDT000081119.3':0.0249728,('PDT000151509.3':0.0178998,'PDT000569384.2':0.0177689)'':0.00654668)'':-0.000805292,('PDT000569488.2':0.0388534,('PDT000123410.3':0.0246209,(('PDT000319647.2':0.0196198,('PDT000191669.3':0.0101098,'PDT000268676.3':0.00961074)'':0.00673833)'':0.00171069,(('PDT000426871.2':0.0143567,'PDT000494180.2':0.0104523)'':0.00263833,('PDT000305215.3':0.0118496,('PDT000212337.3':0.00358748,'PDT000707585.1':0.00299409)'':0.00365592)'':0.0038)'':0.00293355)'':0.00507712)'':0.00519328)'':-0.00091448)'':0.00499524)'':0.002888)'':0.0013151)'':-0.000723569,(('PDT000311073.3':0.0367439,'PDT000386851.2':0.0281346)'':0.00624703,((('PDT000203853.3':0.0209946,'PDT000490183.2':0.0203739)'':0.00308484,(('PDT000319660.2':0.0146181,'PDT000137696.3':0.01995)'':0.00148322,('PDT000410749.2':0.00968162,('PDT000296000.3':0.00299498,'PDT000562544.2':0.00309163)'':0.00381097)'':0.0103548)'':0.00483171)'':0.00327299,(('PDT000084223.3':0.0179457,('PDT000222083.3':0.016109,'PDT000373182.2':0.0116384)'':0.00383619)'':0.00424176,(('PDT000146001.3':0.0281442,('PDT000281768.3':0.0109047,('PDT000191668.3':0.0179046,('PDT000451145.2':0.010632,(('PDT000708044.1':0.00689291,('PDT000620106.2':0.00747174,('PDT000490361.2':0.00755066,'PDT000687894.1':0.00499164)'':0.00164911)'':0.00159323)'':0.00150097,('PDT000645954.2':0.0072729,('PDT000409274.2':0.00486997,('PDT000521242.2':0.00346978,('PDT000510676.2':0.00296956,'PDT000691083.1':0.00401786)'':0.00182409)'':0.00109437)'':0.00185269)'':0.000300978)'':0.000708545)'':0.00175636)'':0.00304782)'':0.00784696)'':-0.00735499,(('PDT000481069.2':0.0239038,('PDT000351790.2':0.0176101,('PDT000214058.3':0.0126576,'PDT000048654.3':0.0157554)'':0.00323317)'':0.00215666)'':0.0041254,('PDT000547441.2':0.0179554,(('PDT000410936.2':0.015126,'PDT000412118.2':0.0117509)'':0.00180234,(('PDT000083246.3':0.00912433,('PDT000208166.3':0.00181278,'PDT000268684.3':0.00208021)'':0.00477187)'':0.00137625,('PDT000468250.2':0.0150969,('PDT000707391.1':0.0095785,('PDT000357181.2':0.00899105,('PDT000198485.3':0.00562929,('PDT000537342.2':0.00525856,('PDT000235725.3':0.00446669,'PDT000314339.3':0.00509697)'':0.000512706)'':0.000461736)'':0.000755472)'':0.00184139)'':0.0025099)'':0.00112958)'':0.00321535)'':0.00525095)'':-0.00281416)'':0.000100994)'':0.00693117)'':-0.00209178)'':0.0106046)'':-0.00660385)'':0.00159552)'':0.00244216)'':0.00888521)'':-0.00622925)'':0.0393853)'':-0.034467,((('PDT000443272.2':0.0475409,(('PDT000189532.3':0.0109231,'PDT000494840.2':0.0135178)'':0.00542251,('PDT000198197.3':0.0143222,('PDT000227559.3':0.00601398,'PDT000587029.2':0.00832152)'':0.00536059)'':0.00879892)'':0.0111592)'':0.00246565,((('PDT000431747.2':0.0265728,('PDT000494252.2':0.0204314,'PDT000604700.2':0.0286629)'':0.00279076)'':0.000830406,(('PDT000145275.3':0.0189193,'PDT000306722.3':0.0188427)'':0.00216369,(('PDT000359260.2':0.0142727,'PDT000504690.2':0.0105584)'':0.00261496,('PDT000275699.3':0.0211984,('PDT000238391.3':0.0191668,('PDT000198054.3':0.0119933,('PDT000285866.3':0.0141643,(('PDT000548731.2':0.0125916,'PDT000578983.2':0.0100432)'':0.0008055,('PDT000303479.3':0.00929614,(('PDT000213863.3':0.00510447,'PDT000496475.2':0.00437278)'':0.00294566,('PDT000280699.3':0.00655464,('PDT000150687.3':0.00368544,'PDT000133761.3':0.00509524)'':0.00228017)'':0.00106732)'':0.00228457)'':-0.000357322)'':0.00305332)'':0.00126128)'':0.00211704)'':0.00125039)'':-0.00159282)'':0.00326821)'':0.00368806)'':0.00434553,(('PDT000387693.2':0.0393988,('PDT000558467.2':0.0383279,('PDT000190685.3':0.027615,('PDT000354912.2':0.0292708,('PDT000417249.2':0.0262707,('PDT000254230.3':0.020582,(('PDT000360410.2':0.0162623,'PDT000687622.1':0.0134602)'':0.0029729,('PDT000501766.2':0.0131339,('PDT000191667.3':0.0130304,'PDT000714480.1':0.0149119)'':0.00229825)'':0.000675878)'':0.00182773)'':0.0012797)'':0.00450454)'':0.00126827)'':0.00712801)'':0.00198958)'':-0.00547641,((('PDT000481024.2':0.0223486,'PDT000598068.2':0.0282881)'':0.00375431,('PDT000344779.2':0.044131,(('PDT000677279.1':0.0217319,'PDT000114541.3':0.0212989)'':0.00268488,(('PDT000503055.2':0.0178339,'PDT000590228.2':0.0184671)'':0.00664171,('PDT000555753.2':0.0160569,('PDT000285766.3':0.0161422,('PDT000225295.3':0.00521379,'PDT000366978.2':0.00463892)'':0.00582403)'':0.00481584)'':0.00252309)'':0.00265158)'':0.00434111)'':0.000828227)'':0.000159768,(((('PDT000306732.3':0.0111535,'PDT000050463.3':0.0106287)'':0.00570434,('PDT000649162.2':0.0247798,(('PDT000281528.3':0.0156852,'PDT000313041.3':0.0143514)'':0.000856356,('PDT000465791.2':0.0117506,('PDT000440547.2':0.010683,'PDT000545557.2':0.00879037)'':0.00244395)'':0.00341808)'':0.00213982)'':-0.000167795)'':0.00123666,('PDT000430564.2':0.0303485,('PDT000221766.3':0.0199092,('PDT000687887.1':0.0295072,('PDT000212370.3':0.0155256,(('PDT000671529.1':0.00953224,'PDT000677120.1':0.00900646)'':0.00386718,('PDT000387022.2':0.0137587,(('PDT000212848.3':0.0136146,'PDT000083239.3':0.0118587)'':0.000813761,('PDT000054732.3':0.00961614,('PDT000338845.2':0.00900777,('PDT000653452.2':0.00720474,('PDT000344338.2':0.00607903,('PDT000707561.1':0.00516577,('PDT000405079.2':0.00378476,'PDT000132571.3':0.00458594)'':0.000709724)'':0.0018269)'':0.000725676)'':0.00137721)'':0.00101472)'':0.00178195)'':0.00172021)'':-0.000642683)'':0.00414236)'':0.00140188)'':0.00134882)'':0.00558513)'':-0.00689035)'':0.00111549,((('PDT000443215.2':0.0254899,'PDT000640311.2':0.0261836)'':0.00312403,((('PDT000281518.3':0.00428136,'PDT000040421.4':0.00448445)'':0.0107266,('PDT000238326.3':0.0185832,'PDT000329861.2':0.0138091)'':0.00176528)'':0.00163749,('PDT000546087.2':0.0243722,('PDT000499200.2':0.0132131,('PDT000707681.1':0.0108706,('PDT000083169.3':0.00861066,'PDT000693886.1':0.00955494)'':0.00236417)'':0.000532509)'':0.00588067)'':7.45435e-05)'':0.0060661)'':0.0024222,(('PDT000538645.2':0.0260944,('PDT000128474.3':0.0136992,('PDT000564134.2':0.0137198,('PDT000499823.2':0.00889357,'PDT000563578.2':0.00791233)'':0.000963828)'':0.00318692)'':0.00501657)'':-0.00168918,(('PDT000569476.2':0.0206154,('PDT000565075.2':0.0168103,('PDT000712912.1':0.0149571,(('PDT000679115.1':0.00877642,('PDT000213931.3':0.00664681,'PDT000238402.3':0.00696199)'':0.00177843)'':0.000678622,('PDT000309582.3':0.0151722,('PDT000295984.3':0.00940418,('PDT000556381.2':0.00886077,'PDT000717594.1':0.00841483)'':0.00170817)'':0.00129972)'':-0.00017977)'':0.0037067)'':0.000851894)'':0.00529773)'':-0.00444441,(('PDT000587496.2':0.0178897,('PDT000375960.2':0.0125717,'PDT000520016.2':0.0106634)'':0.00476768)'':-0.00133023,(('PDT000303565.3':0.0193431,'PDT000563574.2':0.0219499)'':0.00150239,(('PDT000544556.2':0.0102029,('PDT000147508.3':0.00264242,('PDT000337032.2':0.00149018,'PDT000540225.2':0.00140692)'':0.00284888)'':0.00405992)'':0.00161977,('PDT000587203.2':0.014992,(('PDT000578984.2':0.0132253,'PDT000671557.1':0.0117599)'':0.00146057,((('PDT000238413.3':0.00550384,'PDT000265137.3':0.00554096)'':0.00176603,(('PDT000171582.3':0.00345913,'PDT000451163.2':0.00471879)'':0.00116123,('PDT000306784.3':0.00834676,'PDT000690189.1':0.00461434)'':0.00187855)'':0.000960047)'':6.51385e-05,('PDT000653336.2':0.0144074,('PDT000497972.1':0.00952043,('PDT000451144.2':0.00938655,('PDT000315885.2':0.0142467,(('PDT000280681.3':0.00568153,'PDT000311078.3':0.00576467)'':0.00180712,('PDT000717593.1':0.00784663,(('PDT000216647.3':0.00897884,('PDT000679126.1':0.00576637,('PDT000285779.3':0.00364168,('PDT000492198.2':0.00155541,'PDT000119998.3':0.0014407)'':0.00134566)'':0.000886778)'':0.00153043)'':0.000767476,('PDT000384049.2':0.00664145,('PDT000306782.3':0.00754431,('PDT000188060.3':0.00625482,(('PDT000492104.2':0.00392009,('PDT000268752.3':0.00210383,'PDT000663796.1':0.0024879)'':0.00126165)'':0.00163739,('PDT000163583.3':0.00475792,('PDT000202828.3':0.00446934,('PDT000707680.1':0.00628354,('PDT000549846.2':0.00558411,(('PDT000201222.3':0.00237705,'PDT000562875.2':0.00341137)'':0.000273686,(('PDT000277152.3':0.00280442,'PDT000568856.2':0.00258588)'':0.000159016,(('PDT000492061.2':0.00135048,'PDT000492137.2':0.00164592)'':0.000939949,('PDT000313014.3':0.00222757,'PDT000242840.3':0.00256237)'':0.000452654)'':0.000462186)'':0.000283503)'':0.00106727)'':0.000533627)'':0.000345455)'':0.000391839)'':-0.000858215)'':0.0013904)'':0.000229249)'':0.000185372)'':-0.00100387)'':0.00320118)'':-0.0015633)'':0.00218864)'':0.000419552)'':0.000189917)'':0.00215498)'':-0.00419222)'':0.004591)'':0.00368467)'':-0.00616711)'':0.00557586)'':-0.000604265)'':-0.00163713)'':0.00350188)'':0.000674815)'':-0.000328806)'':0.00372548)'':0.000584936)'':-0.00109147)'':0.00942701)'':0.00454008,((('PDT000643999.2':0.0515937,(('PDT000689068.1':0.0194277,('PDT000259860.3':0.0143578,('PDT000494582.2':0.00786727,'PDT000084212.3':0.00885913)'':0.00300422)'':0.00695842)'':0.00852249,(('PDT000487566.2':0.0161905,('PDT000687788.1':0.0118808,'PDT000134042.3':0.0089007)'':0.00762202)'':0.00552192,('PDT000040372.6':0.0143752,('PDT000306764.3':0.0100839,'PDT000482575.2':0.0078016)'':0.0061194)'':0.00577862)'':0.00141177)'':0.0297526)'':-0.0133291,(('PDT000650316.2':0.0178857,('PDT000276406.3':0.0122211,'PDT000096842.3':0.0182843)'':0.00424981)'':0.0183168,(('PDT000497889.1':0.0236954,'PDT000498183.1':0.0233692)'':0.00668828,(('PDT000082089.3':0.0229621,('PDT000480997.2':0.0189609,'PDT000126001.3':0.0152931)'':0.00515087)'':0.00339958,('PDT000502345.1':0.0289196,(('PDT000502508.1':0.0153579,('PDT000143251.3':0.00828912,'PDT000127749.3':0.00724098)'':0.00719575)'':-0.000473752,('PDT000407515.2':0.00937012,('PDT000357155.2':0.0046232,('PDT000230567.3':0.00398462,'PDT000486042.2':0.00378818)'':0.00115371)'':0.00530194)'':0.00640802)'':0.00405021)'':-0.000289659)'':0.00691663)'':0.00963988)'':0.0108647)'':0.0117653,((('PDT000619449.2':0.0466474,('PDT000641880.2':0.0325469,'PDT000127743.3':0.0329379)'':0.0170726)'':0.0107487,(('PDT000373150.2':0.0373901,(('PDT000302835.3':0.0314131,'PDT000571221.2':0.0227978)'':0.00272321,('PDT000203379.3':0.0233668,('PDT000193099.3':0.0163321,'PDT000708507.1':0.0120128)'':0.00668314)'':0.00253906)'':0.00862892)'':0.00017515,(('PDT000124926.3':0.0226089,('PDT000151424.3':0.0181516,('PDT000387025.2':0.00819986,'PDT000491902.2':0.00910154)'':0.00528267)'':0.00623487)'':0.00193424,(('PDT000352799.2':0.0293887,(('PDT000366305.2':0.0170608,'PDT000683526.1':0.0118254)'':0.00383177,('PDT000135812.3':0.0122196,('PDT000359243.2':0.00912931,('PDT000414362.2':0.00592826,'PDT000125038.3':0.00610194)'':0.00498979)'':0.00314339)'':0.00202759)'':0.00853816)'':-0.00229574,(('PDT000145409.3':0.0234763,('PDT000161429.3':0.0136622,('PDT000137155.3':0.00345371,'PDT000708523.1':0.00303073)'':0.0102821)'':0.00457389)'':0.00857694,(('PDT000546150.2':0.0329982,((('PDT000393811.2':0.0283841,('PDT000144497.3':0.0123953,'PDT000366346.2':0.0175413)'':0.0060516)'':0.00065069,('PDT000432445.2':0.016929,('PDT000225454.3':0.00977866,'PDT000491948.2':0.0126317)'':0.00315909)'':0.00349771)'':0.00109653,('PDT000359150.2':0.0400422,(('PDT000430572.2':0.0248204,('PDT000342258.2':0.0168625,('PDT000295971.3':0.00680028,('PDT000484050.2':0.00570737,'PDT000689302.1':0.00662113)'':0.00361492)'':0.00153447)'':0.00320153)'':-0.000982022,('PDT000201125.3':0.0133383,('PDT000687896.1':0.00960088,('PDT000285787.3':0.00636061,('PDT000480418.2':0.00402521,('PDT000260745.3':0.00400471,'PDT000311688.3':0.0035718)'':0.00125439)'':0.00100196)'':0.0015419)'':0.00237933)'':0.000859273)'':0.0121729)'':-0.00755337)'':0.00985438)'':-0.00828488,('PDT000096825.3':0.0442839,(('PDT000198978.3':0.014499,'PDT000542625.2':0.0170446)'':0.00378822,(('PDT000533380.2':0.0249041,('PDT000556320.2':0.0101826,('PDT000295834.3':0.00666897,'PDT000532296.2':0.00519903)'':0.00554993)'':0.00787232)'':0.00357558,(('PDT000384078.2':0.0118603,('PDT000124984.3':0.0151166,('PDT000264768.3':0.0124567,('PDT000163023.4':0.00847952,'PDT000556298.2':0.00869508)'':0.000190964)'':0.00266298)'':0.00185951)'':-5.39388e-06,('PDT000065506.3':0.0256777,('PDT000490336.2':0.0171551,(('PDT000491882.2':0.0109246,'PDT000491916.2':0.0118027)'':0.00654956,('PDT000309834.3':0.0133634,('PDT000267467.3':0.0123316,('PDT000417253.2':0.015814,(('PDT000222047.3':0.00347632,('PDT000215136.3':0.0035987,'PDT000690261.1':0.00407521)'':0.000921103)'':0.00142423,('PDT000136737.3':0.00905382,('PDT000212391.3':0.00866168,('PDT000313402.3':0.00396385,('PDT000338807.2':0.00302224,'PDT000625712.2':0.00336307)'':0.000814754)'':0.0015518)'':0.00187736)'':-0.000810986)'':0.00255042)'':0.00124121)'':0.00128607)'':-0.000458666)'':0.00203665)'':0.00750188)'':-0.00750517)'':0.00263934)'':0.00372695)'':0.00783074)'':-0.00816636)'':0.00208908)'':-0.000609515)'':0.00342681)'':0.003155)'':0.0155151)'':-0.00793979,(('PDT000157515.3':0.0543106,('PDT000102249.3':0.0325108,('PDT000321998.2':0.0319826,'PDT000359338.2':0.0200198)'':0.00270996)'':0.0131846)'':0.0101999,((('PDT000156927.3':0.0379871,'PDT000127375.3':0.0255252)'':0.0175079,(('PDT000142546.3':0.0418463,((('PDT000368362.2':0.0315759,('PDT000499227.2':0.0218523,('PDT000193093.3':0.0129764,'PDT000242818.3':0.0114839)'':0.00882088)'':0.00429296)'':0.00159373,((('PDT000551483.2':0.0247107,('PDT000538886.2':0.0164494,'PDT000587567.2':0.0165513)'':0.00505972)'':0.00664915,('PDT000640539.2':0.0257931,(('PDT000496763.2':0.0144026,'PDT000141148.3':0.014135)'':0.00539822,('PDT000602389.2':0.0139225,('PDT000302794.3':0.0210632,('PDT000144417.3':0.0166262,('PDT000492116.2':0.00660631,'PDT000137568.3':0.00882269)'':0.00562637)'':0.00175688)'':0.00119631)'':-0.000243117)'':0.00626796)'':0.0016031)'':0.000736806,('PDT000594846.2':0.0312493,(('PDT000409248.2':0.0218846,('PDT000641596.2':0.0272719,('PDT000302774.3':0.0182444,('PDT000306534.3':0.0162804,'PDT000677016.1':0.0221469)'':0.00315157)'':0.0040616)'':0.00241619)'':-0.00142989,(('PDT000083180.3':0.0191635,'PDT000101062.3':0.0279397)'':0.00359633,(('PDT000273508.3':0.0119697,'PDT000131912.3':0.0109197)'':0.00737637,('PDT000431660.2':0.0199142,('PDT000564177.2':0.0134609,('PDT000376047.2':0.0117347,('PDT000149201.3':0.0131002,'PDT000459064.2':0.00783493)'':0.00357623)'':0.00174422)'':0.00323692)'':-0.0010303)'':0.00433296)'':0.000393009)'':0.00356338)'':-0.00209592)'':0.00400276)'':0.00258071,(('PDT000135794.3':0.0408443,('PDT000144172.3':0.0298852,'PDT000502442.1':0.0352855)'':0.00810932)'':0.0152023,((('PDT000101116.3':0.0315906,'PDT000131034.3':0.0362474)'':0.00445521,('PDT000144006.3':0.0425175,('PDT000491894.2':0.0304689,(('PDT000225465.3':0.0226685,'PDT000480905.2':0.0236839)'':0.00379908,('PDT000662034.1':0.0143438,'PDT000669294.1':0.0205838)'':0.00940464)'':0.00843911)'':0.00324618)'':-0.0014508)'':0.000642569,('PDT000271281.3':0.0358065,('PDT000481311.2':0.0418103,(('PDT000267464.3':0.017638,'PDT000643929.2':0.0141002)'':0.00561335,((('PDT000146579.3':0.0152133,'PDT000669316.1':0.0142508)'':0.0124776,('PDT000097104.3':0.0207988,('PDT000520254.2':0.0205288,'PDT000289834.3':0.0151926)'':0.000110133)'':0.00470217)'':0.00169807,((('PDT000482574.2':0.013081,'PDT000496523.2':0.00911913)'':0.00666247,('PDT000311061.3':0.0181188,('PDT000303769.3':0.0125884,'PDT000687620.1':0.0134728)'':0.00193531)'':0.00112233)'':0.00229572,('PDT000096894.3':0.0226118,('PDT000707444.1':0.0260969,('PDT000342337.2':0.0155418,('PDT000565102.2':0.013495,('PDT000240651.3':0.0077689,('PDT000564097.2':0.00317524,'PDT000690145.1':0.0043083)'':0.00463303)'':0.00254041)'':0.00321992)'':0.0034248)'':0.00118533)'':0.00129663)'':0.00360129)'':0.000686596)'':0.0093868)'':0.00155024)'':-0.00570135)'':0.00825412)'':-0.0059747)'':0.00620202)'':-0.00313342,((('PDT000101069.3':0.0323201,((('PDT000247556.3':0.0268515,('PDT000372741.2':0.011124,'PDT000394714.2':0.00844683)'':0.0117696)'':0.000862177,('PDT000128669.3':0.0212154,('PDT000224225.3':0.0105028,('PDT000672422.1':0.00941583,'PDT000707380.1':0.00612977)'':0.00460034)'':0.00591487)'':0.00110013)'':0.00295538,(('PDT000362221.2':0.0176939,('PDT000131000.3':0.00649372,('PDT000299800.3':0.00384725,'PDT000086183.3':0.0029385)'':0.003102)'':0.00737027)'':0.00332819,(('PDT000302784.3':0.0163451,'PDT000105066.3':0.0140867)'':0.0038571,('PDT000084024.3':0.0131382,('PDT000545121.2':0.0135366,('PDT000143248.3':0.0133458,('PDT000190989.3':0.00748051,'PDT000386159.2':0.00703679)'':0.00403449)'':0.00104975)'':0.000367009)'':0.00210753)'':0.00327643)'':0.000937668)'':0.0107067)'':-0.00174632,(('PDT000176753.3':0.0231238,(('PDT000394743.2':0.0202893,('PDT000189529.3':0.010014,'PDT000247186.3':0.00961522)'':0.00431082)'':0.00667893,('PDT000559714.2':0.0266333,('PDT000637099.2':0.0165599,('PDT000130573.3':0.0131319,('PDT000148628.3':0.00559367,'PDT000221976.3':0.00664753)'':0.00535323)'':0.0047164)'':0.00286152)'':0.00160536)'':0.00162415)'':0.00701683,(('PDT000549303.2':0.0319257,('PDT000142540.3':0.0187912,'PDT000156853.3':0.0157562)'':0.00795091)'':-0.00166247,((('PDT000146279.3':0.00870237,'PDT000101068.3':0.00735103)'':0.017348,('PDT000487996.2':0.0184026,'PDT000708639.1':0.0113741)'':0.0045073)'':0.00349649,('PDT000569798.2':0.0288519,(('PDT000532714.2':0.00992295,('PDT000265290.3':0.011591,'PDT000537372.2':0.00811046)'':0.0050848)'':0.0069316,('PDT000571820.2':0.0240231,('PDT000495720.2':0.025799,('PDT000367742.2':0.0187044,('PDT000231228.3':0.0161078,(('PDT000124949.3':0.00985464,('PDT000551412.2':0.00828364,('PDT000214051.3':0.00649503,'PDT000564149.2':0.0101567)'':0.00162336)'':0.000558438)'':0.00501603,('PDT000065516.3':0.0149803,('PDT000707930.1':0.013073,('PDT000227275.3':0.0127139,('PDT000145200.3':0.0110603,('PDT000135735.3':0.00596344,('PDT000707384.1':0.00747855,('PDT000084016.3':0.00470661,('PDT000209883.3':0.00365419,'PDT000429548.2':0.00522273)'':0.00106223)'':0.000946786)'':0.000854927)'':0.00214945)'':0.00196338)'':0.000749677)'':0.00209457)'':-0.00262398)'':0.00520006)'':0.00227289)'':0.00174767)'':0.000707966)'':-0.00219799)'':0.0100079)'':-0.00848751)'':0.00177962)'':-0.000348163)'':0.000294053)'':0.00552828,(('PDT000190734.3':0.0407072,(('PDT000237100.3':0.00821953,'PDT000084147.3':0.00661967)'':0.00673189,(('PDT000235275.3':0.0151769,'PDT000289671.3':0.0191969)'':0.00522994,('PDT000407526.2':0.0161659,('PDT000451209.2':0.0118182,'PDT000576227.2':0.00968609)'':0.00542896)'':0.00106494)'':0.00128568)'':0.00839764)'':-0.00403029,(('PDT000124929.3':0.0180199,('PDT000359180.2':0.0105752,('PDT000457350.2':0.010162,('PDT000600717.2':0.00992703,('PDT000465614.2':0.0133903,('PDT000480906.2':0.00815859,((('PDT000662032.1':0.00337403,'PDT000107435.3':0.00490527)'':0.00122187,('PDT000163036.4':0.00335347,('PDT000317582.2':0.00289821,'PDT000137589.3':0.0029906)'':0.000534445)'':0.000994964)'':0.000959176,('PDT000637507.2':0.00560026,('PDT000549493.2':0.00452634,(('PDT000243715.3':0.00421048,'PDT000114998.3':0.00405892)'':0.000270442,('PDT000718783.1':0.00296594,('PDT000417220.2':0.0028596,'PDT000643649.2':0.00252801)'':0.00052432)'':0.000755693)'':0.00108851)'':0.00038978)'':-0.000532971)'':0.00307323)'':0.000560738)'':0.000289542)'':0.00185696)'':0.00109757)'':0.00433198)'':-0.00291819,((('PDT000624198.2':0.0131802,('PDT000190542.3':0.0135672,'PDT000295870.3':0.0102944)'':0.007475)'':-0.000532618,(('PDT000270025.3':0.0244026,'PDT000623900.2':0.0143371)'':0.00445039,('PDT000494949.2':0.0156269,('PDT000151477.3':0.00984904,'PDT000549486.2':0.00707316)'':0.00688484)'':0.00280512)'':0.00146034)'':0.00246223,(('PDT000383977.2':0.0174557,('PDT000556805.2':0.00457305,'PDT000113659.3':0.00699915)'':0.0071657)'':0.00492432,('PDT000387623.2':0.0287692,('PDT000491499.2':0.0240441,(('PDT000310653.3':0.0259289,('PDT000559841.2':0.0164926,'PDT000693866.1':0.0127824)'':0.00547611)'':0.00131546,('PDT000662023.1':0.0232492,(('PDT000495599.2':0.0135406,'PDT000628676.2':0.0173413)'':0.00332149,((('PDT000280712.3':0.0135922,'PDT000468183.2':0.0141141)'':0.00344494,('PDT000671726.1':0.0130009,('PDT000285769.3':0.0133971,('PDT000235278.3':0.00886883,'PDT000367752.2':0.00862967)'':0.00170556)'':0.00151739)'':0.00112429)'':0.000748566,(('PDT000570143.2':0.0145672,('PDT000492183.2':0.00862053,'PDT000668185.2':0.0117593)'':0.00351454)'':0.00234704,(('PDT000490332.2':0.0207823,('PDT000321951.2':0.0118594,('PDT000321935.2':0.0102779,'PDT000691215.1':0.0102192)'':0.0010437)'':0.00564091)'':-0.000963295,('PDT000649169.2':0.0208851,('PDT000566475.2':0.0135292,('PDT000394244.2':0.0131627,('PDT000176663.3':0.0241487,(('PDT000289131.3':0.00980216,'PDT000084055.3':0.00863194)'':0.00354288,('PDT000687900.1':0.0113381,(('PDT000721340.1':0.00919766,('PDT000367192.2':0.006733,('PDT000649668.2':0.00721594,('PDT000151505.3':0.00560386,'PDT000235719.3':0.00484184)'':0.000694413)'':0.00143218)'':0.000817991)'':-0.000451289,('PDT000707467.1':0.0207388,('PDT000242614.3':0.00800692,('PDT000173984.3':0.00831838,('PDT000669460.1':0.0125941,(('PDT000504878.2':0.00606357,('PDT000184478.3':0.00475583,'PDT000205739.3':0.00440843)'':0.0021844)'':0.000283073,(('PDT000257985.3':0.00620886,'PDT000356212.2':0.00553474)'':0.000250662,('PDT000573318.2':0.00913139,((('PDT000233654.3':0.00612859,'PDT000712144.1':0.00463271)'':0.000703609,('PDT000307892.3':0.00332892,('PDT000306768.3':0.00357409,'PDT000307489.3':0.00330802)'':0.000709779)'':0.000626493)'':0.00100434,('PDT000492123.2':0.00775535,('PDT000594139.2':0.00947536,('PDT000239608.3':0.00461602,(('PDT000242754.3':0.00318877,'PDT000492184.2':0.00220476)'':0.00244395,('PDT000705836.1':0.00678295,('PDT000598086.2':0.00435125,('PDT000114999.3':0.00437271,(('PDT000084219.3':0.0024577,('PDT000559161.2':0.0014737,'PDT000707460.1':0.0014234)'':0.00113413)'':5.87008e-05,('PDT000224242.3':0.00267895,('PDT000490197.2':0.00212825,('PDT000230810.3':0.00342305,'PDT000707455.1':0.00186901)'':0.000669752)'':0.000324092)'':0.000284932)'':0.000598036)'':0.000319201)'':0.000679034)'':-0.000471204)'':0.00124423)'':0.000558959)'':0.000479233)'':-0.000826172)'':0.00155473)'':-0.00101416)'':0.000617387)'':0.00160111)'':0.000572426)'':0.000299205)'':0.00196381)'':-0.00298071)'':0.00242888)'':-0.000967693)'':0.0042996)'':0.000614981)'':0.000355656)'':8.31801e-05)'':-0.00183155)'':0.00341394)'':-0.00215167)'':0.0031477)'':0.00385457)'':-0.00440653)'':0.0043861)'':0.0029498)'':-0.0057733)'':-0.00251061)'':-0.000422584)'':0.00421538)'':0.00585685)'':0.00212866)'':0.00785303)'':-0.00821128,(((('PDT000571610.2':0.0206628,'PDT000115466.3':0.0220237)'':0.00276638,('PDT000588390.2':0.0166423,'PDT000132114.3':0.0152042)'':0.0161892)'':0.00914896,(('PDT000344872.2':0.0246602,'PDT000115428.3':0.0244897)'':0.00396531,(('PDT000325168.2':0.0175101,'PDT000408382.2':0.0192845)'':0.00314972,('PDT000321588.2':0.0210256,('PDT000115440.3':0.0131463,('PDT000497783.1':0.0142176,('PDT000327949.2':0.013629,('PDT000324581.2':0.0120749,(('PDT000465896.2':0.00940736,('PDT000480955.2':0.00880032,'PDT000132025.3':0.00867738)'':0.00330334)'':0.00160995,('PDT000162964.4':0.00968212,('PDT000326152.2':0.00826307,('PDT000233974.3':0.00673177,'PDT000132399.3':0.00552773)'':0.00302968)'':0.000998066)'':0.00205323)'':0.000877325)'':0.00119249)'':0.000564019)'':0.00171499)'':0.00102241)'':0.000427585)'':0.00536747)'':0.00331293)'':0.0112051,((('PDT000279544.3':0.0247579,('PDT000414685.2':0.018282,(('PDT000243707.3':0.0147853,'PDT000305200.3':0.0172439)'':0.00494471,(('PDT000494968.2':0.014502,'PDT000127708.3':0.0214279)'':0.00492243,('PDT000594135.2':0.0153888,('PDT000357187.2':0.0087882,'PDT000672425.1':0.0125963)'':0.00318222)'':0.00235416)'':0.00100737)'':0.00546774)'':0.00172671)'':0.00875876,(('PDT000101113.3':0.0323438,(('PDT000237123.3':0.0211231,'PDT000112492.3':0.0252222)'':0.00303915,('PDT000132462.3':0.0194984,('PDT000491909.2':0.0159538,'PDT000625230.2':0.0115591)'':0.00438936)'':0.00526815)'':0.00635593)'':-0.000296288,(('PDT000242819.3':0.0335547,('PDT000491941.2':0.0233439,('PDT000338830.2':0.01714,'PDT000126421.3':0.0165833)'':0.00424251)'':0.0090846)'':-0.000675895,((('PDT000334426.2':0.0126369,'PDT000191199.1':0.0139267)'':0.00666851,((('PDT000190551.3':0.00904134,'PDT000247563.3':0.00919276)'':0.00345639,('PDT000054726.3':0.0156368,('PDT000145391.3':0.0104757,'PDT000111287.3':0.0107019)'':0.002408)'':0.00111629)'':0.00430083,(('PDT000468288.2':0.0129949,'PDT000092462.3':0.0122202)'':0.00108062,(('PDT000375987.2':0.0115949,'PDT000480991.2':0.00975779)'':0.000277236,('PDT000468229.2':0.00803253,('PDT000312547.3':0.00516944,'PDT000191193.1':0.00688836)'':0.00309297)'':0.00113232)'':0.00363904)'':-0.000289855)'':0.00601215)'':-3.25604e-05,(('PDT000270104.3':0.0211271,'PDT000066894.3':0.0199352)'':0.0168034,((('PDT000354860.1':0.00776756,'PDT000682207.1':0.00744274)'':0.00351203,('PDT000556599.2':0.0157976,('PDT000430535.2':0.0064445,'PDT000092283.3':0.0071256)'':0.0046106)'':0.0014058)'':0.00430869,('PDT000247153.3':0.0234306,((('PDT000520015.2':0.0133882,'PDT000600712.2':0.0153696)'':0.0011979,('PDT000360356.2':0.0117114,('PDT000498979.2':0.00602738,('PDT000337010.2':0.00915676,'PDT000539016.2':0.00966634)'':0.000916024)'':0.00403979)'':0.00090978)'':-0.000327205,('PDT000069402.4':0.00930917,('PDT000389757.2':0.00977945,(('PDT000198225.3':0.0100154,'PDT000649567.2':0.00709387)'':0.00165788,('PDT000669497.1':0.00854926,('PDT000208171.3':0.00634651,'PDT000412017.2':0.00499789)'':0.00119029)'':0.000382244)'':0.00173027)'':0.00225928)'':0.00201145)'':0.00259358)'':0.00020705)'':0.00974759)'':-0.00614465)'':0.00300466)'':0.00479151)'':0.00434563)'':-0.00589731,(('PDT000216660.4':0.044787,'PDT000049610.3':0.0396305)'':0.0185664,((('PDT000396597.2':0.0308407,('PDT000383920.2':0.0304041,('PDT000213966.3':0.0184574,('PDT000224480.3':0.0149632,('PDT000587621.2':0.0169284,('PDT000312546.3':0.0149959,'PDT000137565.3':0.0111894)'':0.00236756)'':0.00170285)'':0.00393183)'':0.00972308)'':0.00273285)'':-0.00275059,(((('PDT000145379.3':0.0162517,'PDT000247362.3':0.0188343)'':0.00262889,('PDT000215130.3':0.0222022,'PDT000136736.3':0.0202274)'':0.000672684)'':0.00303656,('PDT000112490.3':0.0385837,('PDT000421080.2':0.0166284,('PDT000171533.3':0.0165322,'PDT000367054.2':0.015325)'':0.00334886)'':0.00941308)'':-0.0011702)'':0.00281988,(('PDT000480449.2':0.0246837,'PDT000710951.1':0.029967)'':0.0012805,('PDT000355541.2':0.0345717,(('PDT000201312.3':0.0214713,('PDT000145342.3':0.0173812,'PDT000081152.3':0.0147716)'':0.00494874)'':0.00311165,('PDT000367024.2':0.026186,('PDT000556697.2':0.0205,('PDT000162299.3':0.0185471,('PDT000242602.3':0.0102967,'PDT000083992.3':0.0113886)'':0.00543631)'':0.00258778)'':0.00150194)'':0.000129508)'':0.00322362)'':8.30005e-05)'':0.00356382)'':0.000438458)'':0.00501265,((('PDT000221610.3':0.0300972,'PDT000125997.3':0.0335046)'':0.0151258,((('PDT000344348.2':0.00513828,'PDT000499035.2':0.00384734)'':0.0069571,('PDT000509769.2':0.0186695,'PDT000564131.2':0.0145583)'':0.00257634)'':0.00845356,('PDT000084076.3':0.0323797,(('PDT000197119.3':0.0127064,('PDT000329845.2':0.0110704,'PDT000594859.2':0.00628789)'':0.00690234)'':0.00575928,('PDT000084136.3':0.0140182,('PDT000722047.1':0.0086429,('PDT000541604.2':0.00956017,'PDT000096924.3':0.0148373)'':0.0011814)'':0.00729165)'':0.000750724)'':0.00343718)'':-4.33949e-05)'':0.00708207)'':0.000653611,((('PDT000252762.3':0.0410913,('PDT000480922.2':0.031176,((('PDT000221605.3':0.0143897,'PDT000517318.2':0.0151873)'':0.00797019,('PDT000273530.3':0.0176172,'PDT000069895.3':0.0148574)'':0.00244561)'':0.00136175,(('PDT000342301.2':0.0245968,'PDT000409213.2':0.0293406)'':0.00268776,('PDT000184451.3':0.0185822,('PDT000344798.2':0.0167809,('PDT000372744.2':0.0154577,'PDT000414389.2':0.0170899)'':0.00133534)'':0.00340631)'':0.00343961)'':0.000297762)'':0.00489634)'':0.00714714)'':-0.00846754,('PDT000566729.2':0.0366511,(('PDT000253062.3':0.0202691,'PDT000498136.1':0.019018)'':0.00840264,(('PDT000151460.3':0.0174268,('PDT000556278.2':0.0129218,'PDT000569713.2':0.0146229)'':0.00357812)'':0.0036231,(('PDT000707492.1':0.0288351,'PDT000105109.3':0.0165337)'':0.00347257,('PDT000488037.2':0.0219675,('PDT000040694.3':0.013925,(('PDT000285811.3':0.00781516,'PDT000086329.3':0.00846434)'':0.00221232,('PDT000365725.2':0.0081853,'PDT000443243.2':0.0070189)'':0.00278151)'':0.00313631)'':0.00580365)'':-0.000125407)'':1.81783e-05)'':0.00603104)'':0.00434464)'':-0.00256464)'':0.00207854,(((('PDT000221989.3':0.0331571,'PDT000707484.1':0.0373458)'':0.00963236,('PDT000500778.2':0.030142,(('PDT000386150.2':0.0235392,'PDT000131125.3':0.0285452)'':0.00135405,(('PDT000087350.3':0.027218,('PDT000216610.3':0.018829,('PDT000069884.3':0.021803,'PDT000141069.3':0.0171785)'':0.00126164)'':0.00425518)'':0.000421691,(('PDT000329835.2':0.0219059,('PDT000163595.3':0.0175526,'PDT000494617.2':0.02965)'':0.000811956)'':0.00499648,('PDT000238428.3':0.0166132,('PDT000662017.1':0.0178866,'PDT000671528.1':0.0170853)'':0.00228858)'':0.00605277)'':0.0013041)'':0.00233467)'':0.00389528)'':-0.000311886)'':0.0021319,((('PDT000250872.3':0.0239456,'PDT000466055.2':0.0236967)'':0.00269948,((('PDT000266338.3':0.0140303,'PDT000536778.2':0.0141915)'':0.00198259,('PDT000555795.2':0.0177983,'PDT000581255.2':0.0137337)'':0.00281096)'':0.00197482,('PDT000537256.2':0.0309025,(('PDT000390447.2':0.0150277,'PDT000591770.2':0.0139635)'':0.00144733,('PDT000498677.2':0.0162318,('PDT000167945.3':0.00700863,('PDT000150926.3':0.00981931,'PDT000130995.3':0.00644329)'':0.00255582)'':0.0042894)'':0.00202462)'':0.00615928)'':-0.00236432)'':0.00503496)'':0.00252678,(((('PDT000334197.2':0.023902,'PDT000337042.2':0.0315933)'':0.00732441,('PDT000137728.3':0.0240771,('PDT000478269.2':0.0251358,'PDT000482585.2':0.0208955)'':0.00329886)'':0.00886072)'':0.00346687,(('PDT000126267.3':0.0262737,('PDT000302840.3':0.0297328,'PDT000352682.2':0.0307025)'':0.00142346)'':0.00249572,('PDT000597985.2':0.0364532,(('PDT000307903.3':0.0241705,'PDT000480934.2':0.019012)'':0.00543723,((('PDT000114637.3':0.0253274,(('PDT000253503.3':0.0102936,'PDT000682719.1':0.00912204)'':0.00333029,('PDT000556501.2':0.010294,('PDT000707588.1':0.0106518,('PDT000289040.3':0.0129831,'PDT000306752.3':0.0118536)'':0.000876494)'':0.00127866)'':0.00189497)'':0.00850851)'':-0.00280174,(('PDT000340505.2':0.0108727,('PDT000417208.2':0.00134846,'PDT000687945.1':0.00134911)'':0.00644723)'':0.00360132,('PDT000421164.2':0.0125111,('PDT000672398.1':0.0109702,('PDT000421950.2':0.00601831,('PDT000491516.2':0.00542964,('PDT000484010.2':0.0033784,'PDT000672418.1':0.00370178)'':0.00192847)'':0.000858741)'':0.00163066)'':0.00308251)'':0.000141359)'':0.00458138)'':0.000511687,(('PDT000542112.2':0.0145342,('PDT000495598.2':0.0141664,('PDT000127729.3':0.0175766,(('PDT000687624.1':0.0113564,'PDT000113709.3':0.00736676)'':0.00543445,('PDT000107489.3':0.00980354,('PDT000201218.3':0.00786668,'PDT000307876.3':0.0111043)'':0.00166326)'':0.00348565)'':0.00282966)'':0.002902)'':0.000861193)'':0.00216639,('PDT000387587.2':0.0220466,(('PDT000101104.3':0.0182619,('PDT000679461.1':0.0224495,('PDT000315900.2':0.0174892,('PDT000151504.3':0.0199187,('PDT000707408.1':0.0148602,('PDT000482572.2':0.00756418,('PDT000328158.2':0.00973684,('PDT000156868.3':0.0054056,(('PDT000509939.2':0.00746455,'PDT000125285.4':0.00716235)'':0.000562803,('PDT000649211.2':0.00651021,('PDT000225296.3':0.00750631,('PDT000289057.3':0.00442337,'PDT000492089.2':0.00484696)'':0.00104228)'':0.000232922)'':0.000660329)'':0.000347336)'':0.00268178)'':0.000618019)'':0.00220365)'':0.00102713)'':0.00127445)'':0.00366762)'':0.00211745)'':-0.00717607,('PDT000491517.2':0.0113976,('PDT000717507.1':0.0150211,('PDT000466049.2':0.013226,('PDT000375797.2':0.0125345,('PDT000302810.3':0.012091,('PDT000191661.3':0.0105017,('PDT000482590.2':0.00836877,('PDT000321143.2':0.00826997,('PDT000599888.2':0.00966603,('PDT000532708.2':0.00768639,('PDT000702882.1':0.00686099,('PDT000362574.2':0.00524041,('PDT000222566.3':0.00352991,('PDT000201099.3':0.00323821,'PDT000600803.2':0.0028484)'':0.00075233)'':0.00100554)'':0.000834688)'':0.000565203)'':0.00111323)'':0.00058624)'':0.000585155)'':0.000426446)'':0.00022289)'':0.000946681)'':0.00212757)'':0.0019764)'':0.000518104)'':-0.00188031)'':0.00860802)'':-0.00479818)'':-0.000295048)'':0.00448611)'':0.00655062)'':-9.86359e-05)'':-0.00188637)'':-0.00387355,((('PDT000603513.2':0.0173682,('PDT000604192.2':0.0219678,'PDT000627567.2':0.0176303)'':0.00190219)'':0.00246496,('PDT000102764.3':0.0333392,(('PDT000499216.2':0.0130802,('PDT000357281.2':0.0107939,'PDT000501164.2':0.0135168)'':0.000672555)'':0.00101917,('PDT000101110.3':0.0147173,('PDT000082102.3':0.013319,('PDT000598181.2':0.0122796,('PDT000081124.5':0.00994316,'PDT000360353.2':0.00903534)'':0.00173885)'':0.00216654)'':0.000417)'':0.00133951)'':0.0107847)'':-0.00427684)'':-0.00140127,((('PDT000106311.3':0.0105278,'PDT000368898.1':0.00917572)'':0.0180098,(('PDT000662472.1':0.00360754,'PDT000687881.1':0.00447871)'':0.0101208,('PDT000227290.3':0.00743662,('PDT000338824.2':0.00663578,('PDT000132437.3':0.00594364,('PDT000146486.3':0.00514937,('PDT000212339.3':0.00519515,'PDT000277162.3':0.00387854)'':0.000915418)'':0.000607594)'':0.00147521)'':0.00129499)'':0.00130547)'':0.0127559)'':-0.00373326,((('PDT000366160.2':0.0300655,'PDT000107000.3':0.0275781)'':0.00412524,(('PDT000315823.2':0.0159218,(('PDT000367222.2':0.00551937,'PDT000547091.2':0.00522693)'':0.00290773,(('PDT000221983.3':0.00342397,'PDT000247146.3':0.00335569)'':0.00483521,('PDT000083240.3':0.0106826,'PDT000636592.2':0.012074)'':0.00117061)'':0.00311529)'':0.00431494)'':0.00171951,(('PDT000303569.3':0.0156464,'PDT000307879.3':0.00827704)'':0.00194764,('PDT000616805.2':0.0147738,((('PDT000313027.3':0.0101982,'PDT000351797.2':0.00984548)'':0.000230488,('PDT000306773.3':0.00911916,('PDT000145198.3':0.0052833,'PDT000205380.3':0.006259)'':0.00260934)'':0.0010599)'':0.000340504,(('PDT000587028.2':0.00955437,('PDT000066896.3':0.00994683,('PDT000083087.3':0.00671558,'PDT000134063.3':0.00681272)'':0.00243847)'':0.000260244)'':0.000727864,(('PDT000240623.3':0.00869584,'PDT000273528.3':0.0118078)'':0.0013935,('PDT000281511.3':0.010531,('PDT000708510.1':0.00965788,('PDT000491956.2':0.00657744,'PDT000127543.3':0.00653736)'':0.00185172)'':0.0013218)'':0.000323539)'':0.000206373)'':0.00139789)'':0.00330471)'':0.00266718)'':-0.000920942)'':0.010125)'':-0.0089726,(('PDT000112489.3':0.0208634,(('PDT000325494.2':0.0178202,'PDT000351806.2':0.0132351)'':0.00235084,('PDT000503521.2':0.0157924,('PDT000309892.3':0.018055,'PDT000421074.2':0.0115496)'':0.00421456)'':8.9905e-05)'':0.00787442)'':-0.00396953,('PDT000307891.3':0.0245289,('PDT000277153.3':0.022631,(('PDT000191212.1':0.0128457,('PDT000302796.3':0.00918735,('PDT000191665.3':0.00621566,'PDT000235045.3':0.00533464)'':0.00202565)'':0.00774676)'':-0.000792934,('PDT000580720.2':0.0186084,(('PDT000128473.3':0.0265634,('PDT000309593.3':0.0171395,('PDT000125009.3':0.0109824,('PDT000230813.3':0.00687174,'PDT000547088.2':0.00674656)'':0.00428824)'':0.00308706)'':0.00439969)'':-0.00192617,('PDT000238407.3':0.0225472,(('PDT000189539.3':0.00980689,'PDT000691410.1':0.011109)'':0.00097163,('PDT000678083.1':0.0175779,('PDT000407576.2':0.0130509,('PDT000051252.3':0.0103326,('PDT000171576.3':0.00749089,('PDT000303750.3':0.00757979,(('PDT000311663.3':0.00459614,'PDT000390220.2':0.00466773)'':0.00206381,(('PDT000522606.2':0.00524497,'PDT000062607.3':0.00581413)'':0.00037435,('PDT000247439.3':0.00938687,('PDT000221737.3':0.00723926,(('PDT000381334.2':0.00586645,('PDT000157475.3':0.00423778,'PDT000084237.3':0.00304983)'':0.000896149)'':0.00131625,('PDT000162994.4':0.00423335,('PDT000491927.2':0.00454448,'PDT000707437.1':0.0040187)'':0.000157695)'':0.00201762)'':0.000495302)'':0.000797195)'':-0.000379644)'':0.00130619)'':0.000378847)'':0.00077481)'':0.000620031)'':0.0014373)'':0.00120756)'':-0.00162707)'':0.00546022)'':-0.00211812)'':0.00533794)'':-0.00268553)'':0.00676)'':0.00167806)'':-0.00647955)'':-0.000900288)'':0.00297444)'':0.000565337)'':0.000824269)'':0.00418917)'':3.50071e-05)'':-0.00215651,((('PDT000366057.2':0.0285751,('PDT000303775.3':0.0263878,'PDT000491898.2':0.0272098)'':0.00673074)'':-0.000478346,('PDT000240658.3':0.0278695,('PDT000396593.2':0.0183238,('PDT000156644.3':0.0235641,(('PDT000483990.2':0.00851225,('PDT000289809.3':0.00860818,'PDT000065186.3':0.0112903)'':0.000782653)'':0.00613844,('PDT000054722.3':0.00794243,('PDT000366158.2':0.00804526,'PDT000137564.3':0.00835304)'':0.00172502)'':0.0041304)'':0.00932107)'':0.000790579)'':0.000761936)'':0.00410132)'':0.00423394,(('PDT000566734.2':0.0165123,('PDT000244660.3':0.0173679,(('PDT000537375.2':0.00953566,'PDT000616682.2':0.0107366)'':0.00337272,('PDT000556039.2':0.00959804,('PDT000145317.3':0.0165714,('PDT000257960.3':0.00512549,('PDT000153283.3':0.00403503,'PDT000486068.2':0.00334284)'':0.00114924)'':0.00546261)'':0.00169995)'':-0.000197874)'':0.00197854)'':0.0017108)'':0.00750209,((((('PDT000213231.3':0.0283647,'PDT000539316.2':0.033522)'':0.00157142,('PDT000549601.2':0.0197497,('PDT000571214.2':0.0259435,('PDT000440539.2':0.016494,('PDT000285793.3':0.0146846,('PDT000707387.1':0.00789572,'PDT000131104.3':0.00955248)'':0.00398153)'':0.00491306)'':0.00305691)'':0.00122403)'':0.00565521)'':-0.000196578,((('PDT000225432.3':0.0271602,'PDT000587569.2':0.0162274)'':0.0063662,('PDT000627903.2':0.0264068,('PDT000052448.3':0.0235027,'PDT000665178.1':0.0188526)'':0.00449004)'':0.00465732)'':0.000319099,('PDT000105079.3':0.0265589,('PDT000321966.2':0.027479,(('PDT000432275.2':0.0126575,'PDT000575916.2':0.0125227)'':0.000847626,(('PDT000359117.2':0.0105593,'PDT000682820.1':0.0144705)'':0.0017658,(('PDT000084033.3':0.0163053,('PDT000221942.3':0.00927467,('PDT000342241.2':0.00557169,'PDT000356281.2':0.00917231)'':0.00220533)'':0.00403218)'':0.00200163,(('PDT000559300.2':0.00961756,('PDT000324571.2':0.00798725,('PDT000190998.3':0.00614126,'PDT000098730.3':0.00579664)'':0.0010434)'':0.00282589)'':0.00143044,('PDT000385887.2':0.0148726,('PDT000355478.2':0.0103746,('PDT000707378.1':0.0123818,(('PDT000644210.2':0.00931176,('PDT000213225.3':0.00366663,'PDT000571507.2':0.00390762)'':0.00266597)'':0.00191842,('PDT000044782.3':0.0131203,(('PDT000313043.3':0.00622128,'PDT000650024.2':0.00640672)'':0.00159154,('PDT000150094.3':0.00803424,(('PDT000102814.3':0.00596449,('PDT000221736.3':0.00480059,'PDT000360390.2':0.00376856)'':0.00193324)'':0.000426346,('PDT000113702.3':0.00617657,('PDT000491907.2':0.00420635,'PDT000542088.2':0.00348135)'':0.00211958)'':0.000838538)'':0.00131153)'':-0.000161144)'':0.00263537)'':-0.00192416)'':0.00261792)'':0.000491356)'':0.00268849)'':-0.00264518)'':0.000561044)'':0.00242201)'':0.000606498)'':0.00762787)'':0.00415289)'':-0.00600458)'':0.0014094)'':-0.00223166,(('PDT000457351.2':0.0354572,('PDT000187999.3':0.0184497,('PDT000125281.4':0.0152816,'PDT000191200.1':0.0132503)'':0.00396854)'':0.00680845)'':0.00131307,(('PDT000132011.3':0.0431187,('PDT000562620.2':0.0138378,('PDT000250857.3':0.0178975,('PDT000212530.3':0.0137014,('PDT000634611.2':0.00870978,('PDT000672412.1':0.00747117,('PDT000375793.2':0.00760024,('PDT000231267.3':0.0054196,'PDT000520372.2':0.0053224)'':0.00232306)'':0.00142421)'':0.00061471)'':0.000793951)'':0.00464313)'':0.000776666)'':0.0102132)'':-0.0084339,(('PDT000084131.3':0.0265629,'PDT000113775.3':0.0211827)'':0.0137415,(('PDT000491880.2':0.0168376,('PDT000120023.3':0.0100388,('PDT000273515.3':0.00563175,('PDT000708535.1':0.00562186,('PDT000198198.3':0.00248891,'PDT000417250.2':0.00240473)'':0.00352877)'':0.000498333)'':0.00358879)'':0.000505191)'':0.000498207,(('PDT000193138.3':0.021997,'PDT000101016.3':0.0196369)'':0.00081501,(('PDT000128442.3':0.00842955,('PDT000144418.3':0.00638255,'PDT000708512.1':0.00644685)'':0.00191965)'':0.00531035,('PDT000243720.3':0.0239128,('PDT000135788.3':0.0129398,('PDT000465878.2':0.0103179,('PDT000412006.2':0.007922,(('PDT000167943.3':0.00686838,'PDT000124935.3':0.00440682)'':0.00174432,('PDT000566525.2':0.0168922,('PDT000539012.2':0.012669,('PDT000707433.1':0.00743552,('PDT000300521.1':0.00800541,('PDT000190938.3':0.00515886,(('PDT000160131.3':0.00630904,(('PDT000616653.2':0.00318727,'PDT000669471.1':0.00349469)'':0.000843346,('PDT000247180.3':0.00274968,('PDT000669465.1':0.00175319,'PDT000125282.3':0.00174192)'':0.00154114)'':0.000568056)'':0.00194341)'':-0.00107235,(('PDT000048655.3':0.00186399,'PDT000545779.2':0.00163042)'':0.00174978,('PDT000136735.3':0.00360622,('PDT000115054.3':0.00518232,(('PDT000146050.3':0.00159638,'PDT000627219.2':0.00180056)'':0.000796342,('PDT000147497.3':0.00195047,'PDT000126369.3':0.00164414)'':0.000846498)'':0.000735686)'':0.000453634)'':-3.01365e-05)'':0.0014403)'':0.00176788)'':0.000546771)'':0.000204452)'':0.000365694)'':0.000810461)'':-0.000865921)'':0.00208838)'':0.000488253)'':0.00279817)'':0.00074666)'':-0.00284635)'':0.00656246)'':-0.00399689)'':0.00866417)'':-0.00737107)'':0.00346702)'':-0.000611084)'':0.00125263,((('PDT000167947.3':0.0160199,'PDT000257971.3':0.0153963)'':0.0164246,(('PDT000547702.2':0.0224162,('PDT000280698.3':0.0287998,'PDT000096844.3':0.0237956)'':0.00428186)'':0.00406829,(('PDT000189523.3':0.021639,'PDT000342253.2':0.0203844)'':0.0023332,('PDT000222055.3':0.0193469,(('PDT000235325.3':0.0169339,('PDT000655557.2':0.0137837,('PDT000212508.3':0.0120878,'PDT000707447.1':0.0137104)'':0.00551627)'':0.00129699)'':-1.31296e-05,(('PDT000362189.2':0.00866322,'PDT000360389.2':0.0102498)'':0.00197263,('PDT000239607.3':0.00902041,'PDT000135813.3':0.0101569)'':0.00384234)'':0.00203423)'':0.00703184)'':-0.00145757)'':0.000720443)'':0.00494924)'':-0.00103715,((('PDT000264784.3':0.0170772,'PDT000128448.3':0.0232589)'':0.00168607,(('PDT000132568.3':0.0124443,('PDT000307875.3':0.00984737,'PDT000112839.3':0.0113424)'':0.0057067)'':0.0014855,(('PDT000124806.3':0.00557129,('PDT000309585.3':0.00402324,'PDT000672399.1':0.0044429)'':0.00429079)'':0.00695901,('PDT000234236.3':0.0146972,('PDT000257227.3':0.00871814,(('PDT000245721.3':0.00450696,'PDT000281218.3':0.0036677)'':0.00504316,('PDT000315866.2':0.00663287,'PDT000069924.3':0.00572753)'':0.00177399)'':0.0023657)'':0.00326663)'':-0.00119031)'':0.00224396)'':0.00631427)'':-0.000897268,(('PDT000196473.3':0.0311305,('PDT000551448.2':0.0187676,('PDT000344365.2':0.01333,('PDT000647819.2':0.0116581,('PDT000128668.3':0.010087,('PDT000082146.3':0.00987461,('PDT000624934.2':0.00916881,('PDT000239598.3':0.00565512,'PDT000384065.2':0.00559538)'':0.00281234)'':0.00165573)'':0.000950697)'':0.00365018)'':0.000971274)'':0.00281621)'':0.00310018)'':0.000358252,(('PDT000405478.2':0.0223545,'PDT000101081.3':0.0210682)'':0.00356143,((('PDT000414369.2':0.0184172,'PDT000102795.3':0.0168525)'':0.0035776,('PDT000084007.3':0.0198808,('PDT000498096.1':0.0122487,(('PDT000114699.3':0.00841321,('PDT000466042.2':0.01048,'PDT000508967.2':0.00798151)'':0.00246259)'':0.00117188,(('PDT000491944.2':0.00808207,'PDT000707486.1':0.00635443)'':0.00301393,('PDT000253190.3':0.00884159,('PDT000280709.3':0.0118144,'PDT000311121.3':0.00937996)'':0.000409907)'':0.000743256)'':0.000461558)'':0.0035988)'':0.000355555)'':0.00669793)'':-0.00296432,(('PDT000394063.2':0.0230873,(('PDT000492159.2':0.00794871,('PDT000235340.3':0.00649996,('PDT000598745.1':0.0078583,('PDT000616652.2':0.00448451,'PDT000707601.1':0.00477752)'':0.00130993)'':0.000770837)'':0.00184157)'':0.00173686,('PDT000658617.2':0.0169923,('PDT000487622.2':0.0143966,('PDT000672402.1':0.00715717,(('PDT000421152.2':0.00685466,'PDT000682716.1':0.00776934)'':0.000237458,('PDT000356269.2':0.00519334,('PDT000490341.2':0.00619075,('PDT000307527.3':0.00345035,'PDT000490325.2':0.00472675)'':0.0015754)'':0.00151758)'':0.00109799)'':0.0020872)'':0.00170952)'':0.00437423)'':-0.00418343)'':0.00929505)'':-0.00631115,((('PDT000209213.3':0.0155885,'PDT000351861.2':0.0146548)'':0.00504655,('PDT000040716.3':0.0116917,('PDT000211628.3':0.00992561,'PDT000114697.3':0.0125084)'':0.00428801)'':0.00626435)'':0.00093215,(('PDT000114686.3':0.0198625,('PDT000163578.3':0.0119725,'PDT000171597.3':0.0107953)'':0.00717444)'':-0.00181664,('PDT000273532.3':0.0159632,((('PDT000151426.3':0.00813564,'PDT000352797.2':0.00962756)'':0.00285622,('PDT000544722.2':0.012345,'PDT000603397.2':0.0138278)'':0.00047601)'':0.00118719,(('PDT000385955.2':0.0123158,('PDT000324567.2':0.0085711,'PDT000590265.2':0.0077068)'':0.00430175)'':0.000480287,(('PDT000592628.2':0.01557,'PDT000630871.2':0.0129338)'':0.00153722,(('PDT000283359.3':0.0106491,'PDT000579646.2':0.0121187)'':0.00162274,(('PDT000321914.2':0.0171858,'PDT000385894.2':0.0175994)'':0.00557444,((('PDT000365944.2':0.00793139,('PDT000360359.2':0.0089654,'PDT000491881.2':0.0052819)'':0.00114051)'':0.0016244,('PDT000342386.2':0.00882948,('PDT000307510.3':0.00729146,('PDT000105122.3':0.00460832,('PDT000387702.2':0.002849,'PDT000583002.2':0.00303922)'':0.00131206)'':0.00312973)'':0.00241892)'':-0.000620049)'':0.000989872,(('PDT000269966.3':0.0135959,('PDT000351784.2':0.00801257,'PDT000125004.3':0.00747593)'':0.00356468)'':0.00176368,('PDT000498074.1':0.014103,('PDT000224481.3':0.012483,(('PDT000162978.4':0.00786499,'PDT000201196.3':0.00811511)'':0.00257785,(('PDT000177084.3':0.00978836,('PDT000630875.2':0.00762699,('PDT000305227.3':0.00569043,'PDT000539007.2':0.00525487)'':0.00150001)'':0.00263777)'':-0.00051971,(('PDT000312082.3':0.0103794,('PDT000215132.3':0.00689891,'PDT000707382.1':0.00842909)'':0.00127605)'':0.00367913,(('PDT000147487.3':0.00698919,'PDT000167624.3':0.00712891)'':0.00108866,(('PDT000215103.3':0.0141837,(('PDT000622988.2':0.00887859,'PDT000120016.3':0.00712701)'':0.00100072,(('PDT000238431.3':0.00630089,('PDT000468277.2':0.00266923,'PDT000708614.1':0.00342103)'':0.00107708)'':0.000892164,(('PDT000176833.3':0.00485299,('PDT000049615.3':0.00476466,'PDT000084185.3':0.00609624)'':0.000727856)'':0.000117793,('PDT000303770.3':0.00653949,('PDT000491924.2':0.00525809,('PDT000224273.3':0.00383032,('PDT000309595.3':0.0034444,(('PDT000238429.3':0.00309061,'PDT000387684.2':0.00260085)'':0.000659693,('PDT000707485.1':0.00286092,('PDT000491961.2':0.00224786,'PDT000708511.1':0.00264187)'':0.000329245)'':0.000920535)'':7.98198e-05)'':0.00023939)'':0.000482707)'':0.000978837)'':-0.000386157)'':0.000948728)'':0.0017755)'':0.00416108)'':-0.00426333,('PDT000502720.2':0.0120064,('PDT000142533.3':0.00819407,(('PDT000496403.2':0.00448275,('PDT000491952.2':0.00590794,'PDT000136771.3':0.0040721)'':0.000791424)'':0.00105334,(('PDT000233641.3':0.00944539,'PDT000491925.2':0.00812631)'':0.00191792,(('PDT000145377.3':0.00498359,'PDT000532158.2':0.00518391)'':0.000436124,(('PDT000230809.3':0.00532729,('PDT000198055.3':0.00579284,'PDT000520369.2':0.00504966)'':0.000340457)'':0.000645302,('PDT000626985.2':0.00690315,('PDT000707451.1':0.00926027,('PDT000365730.2':0.00633412,(('PDT000357129.2':0.00496272,('PDT000227342.3':0.00443368,'PDT000069887.3':0.00442821)'':0.000718655)'':-9.19299e-05,('PDT000077523.1':0.00558918,('PDT000542730.2':0.00375359,('PDT000152934.3':0.00455569,('PDT000683650.1':0.00389454,('PDT000212529.3':0.00323921,('PDT000127271.3':0.00391775,('PDT000227586.3':0.00288721,('PDT000065207.3':0.00278069,('PDT000126264.3':0.00227972,('PDT000562719.2':0.00189026,('PDT000707381.1':0.00216361,('PDT000708612.1':0.00163982,('PDT000147475.3':0.000916085,'PDT000491955.2':0.000882655)'':0.00055627)'':0.000479428)'':0.000189264)'':0.00013039)'':0.000400581)'':0.000508353)'':0.000263805)'':0.00025119)'':0.000346887)'':0.000176443)'':0.000124456)'':0.00101195)'':-0.000825333)'':0.000636378)'':0.0011681)'':0.000716271)'':-0.0014525)'':0.000729621)'':0.000178542)'':0.000156454)'':0.00267332)'':0.00127416)'':-0.00338661)'':0.00172621)'':0.000987553)'':0.000177731)'':0.0014735)'':0.000913988)'':0.00346221)'':-0.00514005)'':-0.00104016)'':0.00531813)'':0.000376246)'':0.000245135)'':-0.00151812)'':-0.000229085)'':0.00311825)'':0.00144512)'':-0.00259406)'':-0.00201143)'':0.00148485)'':0.00757809)'':-0.00597447)'':-0.000545144)'':0.00385924)'':-0.00181232)'':0.00285106)'':0.00371406)'':-0.00410785)'':0.00327196)'':0.0026931)'':-0.0015571)'':0.0100268)'':-0.0102954)'':0.0148956)'':-0.0143484)'':0.0144764)'':-0.00922199)'':0.018577)'':-0.0170418)'':0.0039715)'':0.0120278)'':0.0153257)'':0.0352585)'':0.0391541)'':0.0238149)'':-0.00273531,(((('PDT000396805.1':0.0377275,('PDT000069171.2':0.0172357,'PDT000395353.1':0.0162058)'':0.0259894)'':0.0598838,('PDT000392506.1':0.0596077,(('PDT000086177.2':0.0793076,'PDT000251834.1':0.0564644)'':0.00650661,(('PDT000642297.1':0.0480253,('PDT000553201.1':0.0443832,'PDT000083589.1':0.048478)'':0.00578967)'':0.00280839,(('PDT000616198.1':0.0193055,'PDT000332135.1':0.0250213)'':0.0220636,('PDT000016380.2':0.0453525,('PDT000083455.1':0.0170799,('PDT000083491.1':0.0174578,'PDT000083561.1':0.0170046)'':0.00228028)'':0.0198639)'':0.0024096)'':0.00686477)'':0.00576826)'':0.00940255)'':0.0253395)'':0.0293076,(((('PDT000083496.1':0.055896,'PDT000016365.2':0.052104)'':0.0672445,(('PDT000691448.1':0.0404341,('PDT000083515.1':0.00294766,'PDT000083560.1':0.00333726)'':0.035298)'':0.0545165,('PDT000390575.1':0.0886843,(('PDT000625490.1':0.0423749,'PDT000248684.1':0.0431425)'':0.0135677,('PDT000625485.1':0.042678,('PDT000083563.1':0.023854,('PDT000034410.1':0.0193923,'PDT000017885.2':0.0165888)'':0.00931895)'':0.0229557)'':0.00681742)'':0.0226244)'':0.00843666)'':0.0210772)'':-0.00131341,((('PDT000249858.1':0.0298275,('PDT000625474.1':0.00887497,'PDT000016393.2':0.0112065)'':0.0200964)'':0.0151911,('PDT000386989.2':0.04766,('PDT000040393.3':0.0398246,('PDT000082647.1':0.00871082,'PDT000083459.1':0.00806008)'':0.0360372)'':0.00243283)'':0.0041231)'':0.042054,(('PDT000083532.1':0.0826386,(('PDT000032188.1':0.00199137,'PDT000032204.1':0.0018018)'':0.00412991,('PDT000083583.1':0.00377052,'PDT000083590.1':0.00622149)'':0.00288361)'':0.0665071)'':0.0101352,(('PDT000324207.1':0.0639732,('PDT000083446.1':0.0449786,('PDT000577038.2':0.035898,('PDT000679766.1':0.0211529,('PDT000083461.1':0.00766732,'PDT000083538.1':0.0127164)'':0.0125114)'':0.0189727)'':0.0086862)'':0.0253832)'':0.00247475,((('PDT000253474.2':0.048828,('PDT000161690.2':0.0321824,'PDT000083584.1':0.0280352)'':0.0130508)'':0.00971836,(('PDT000132274.2':0.0334641,'PDT000619758.1':0.0349748)'':0.00529443,('PDT000375716.1':0.0287319,'PDT000553126.1':0.019913)'':0.0229734)'':0.00882304)'':0.00191515,(('PDT000641373.1':0.0483874,('PDT000322929.1':0.0258561,'PDT000083530.1':0.0205483)'':0.0158964)'':-0.00139222,(('PDT000132014.2':0.0310847,'PDT000382848.1':0.0344667)'':0.00500021,('PDT000625371.1':0.0380066,('PDT000410730.1':0.0155865,('PDT000083575.1':0.00971038,('PDT000083513.1':0.00913725,'PDT000083586.1':0.00854605)'':0.00501067)'':0.00379164)'':0.00952997)'':0.00718749)'':0.00604564)'':0.0131594)'':0.0201594)'':0.0134066)'':0.0118705)'':0.00307026)'':0.00211452,(((('PDT000646098.2':0.0210204,('PDT000157098.3':0.016039,('PDT000519166.2':0.0163284,'PDT000641369.2':0.0134412)'':0.00147409)'':0.0106701)'':0.019353,(('PDT000050619.1':0.030707,('PDT000161663.3':0.0163586,'PDT000239350.1':0.019897)'':0.0121387)'':0.010176,('PDT000300553.1':0.0210347,('PDT000651391.2':0.00838619,'PDT000131936.3':0.0123083)'':0.017347)'':0.00502841)'':0.00887568)'':0.0120008,(('PDT000422646.2':0.0360982,'PDT000571569.2':0.0262729)'':0.022863,(('PDT000372699.2':0.0366512,(('PDT000139186.3':0.0469584,'PDT000048658.3':0.0351465)'':0.00761634,(('PDT000610060.2':0.0241232,('PDT000338844.2':0.0302968,('PDT000101034.4':0.0220465,('PDT000264877.3':0.0138491,('PDT000481326.2':0.00993359,'PDT000137640.3':0.0113134)'':0.00515933)'':0.00504348)'':0.00294793)'':0.00218117)'':0.00161714,('PDT000576864.1':0.022545,((('PDT000503995.2':0.0169553,('PDT000239588.3':0.0115635,'PDT000650828.2':0.0125413)'':0.00135592)'':0.00195635,('PDT000558477.2':0.0134938,('PDT000299942.3':0.00967304,'PDT000325493.2':0.0100749)'':0.00348125)'':0.00233401)'':0.00367389,('PDT000240584.3':0.0357224,((('PDT000128451.3':0.014506,'PDT000722049.1':0.01404)'':0.00231744,('PDT000084160.3':0.0168677,('PDT000540024.2':0.0124401,('PDT000707580.1':0.00533593,'PDT000125002.3':0.00661627)'':0.00326329)'':0.00553212)'':-0.000358286)'':0.00288848,('PDT000580239.2':0.015662,('PDT000250866.3':0.0124465,('PDT000253482.3':0.0096119,('PDT000189658.3':0.0117944,('PDT000443357.2':0.00950359,('PDT000060078.3':0.0067178,'PDT000521239.2':0.0073989)'':0.00128401)'':0.00147918)'':0.00138349)'':0.00190644)'':0.00464966)'':-0.00286293)'':0.00635871)'':-0.00185464)'':0.00603998)'':-0.00330658)'':0.0139936)'':0.00580738)'':-0.00383523,((('PDT000496683.2':0.0271894,'PDT000523427.2':0.02697)'':0.00152155,('PDT000147513.3':0.0171461,'PDT000139176.3':0.0150832)'':0.0101964)'':0.00444392,((('PDT000142644.3':0.0283395,'PDT000620779.2':0.0328137)'':0.00556679,(('PDT000225457.3':0.0246565,'PDT000384051.2':0.0243287)'':0.00555198,('PDT000625228.2':0.0323443,(('PDT000602675.2':0.0113051,('PDT000197041.3':0.00877981,'PDT000300592.1':0.00661249)'':0.00251771)'':0.00150438,('PDT000497553.2':0.0107773,('PDT000478257.2':0.00675788,('PDT000227363.3':0.004312,'PDT000303774.3':0.0087658)'':0.00303737)'':0.00253384)'':0.000320672)'':0.00739618)'':0.0033788)'':0.00539196)'':-0.00492468,(('PDT000222075.3':0.015728,'PDT000125530.3':0.0191385)'':0.00915598,(('PDT000384081.2':0.0186832,'PDT000131100.3':0.0248466)'':0.0103724,(('PDT000576861.2':0.013306,('PDT000124994.3':0.0127411,'PDT000127545.3':0.00986901)'':0.00348468)'':0.00466246,(('PDT000150123.3':0.0115527,('PDT000357285.2':0.00441222,'PDT000389773.2':0.00426262)'':0.00463891)'':0.00389071,(('PDT000289864.3':0.00969974,'PDT000372722.2':0.00929576)'':0.00390064,('PDT000259542.3':0.0147962,('PDT000213928.3':0.017852,(('PDT000296207.3':0.0110163,('PDT000198972.3':0.0076244,'PDT000576211.2':0.0071956)'':0.00205039)'':0.00112423,(('PDT000137578.3':0.00533773,'PDT000300557.1':0.00442809)'':0.00358676,('PDT000721966.1':0.0178448,('PDT000375765.2':0.00689454,('PDT000553496.2':0.00555734,('PDT000589706.2':0.00488146,('PDT000296203.3':0.00371457,('PDT000692222.1':0.00281712,('PDT000145318.3':0.00206215,'PDT000399852.2':0.00203007)'':0.000676742)'':0.00118005)'':0.000114818)'':0.00163937)'':0.00104261)'':0.00488644)'':-0.0019339)'':0.00101107)'':0.00328497)'':0.00217694)'':-0.00224188)'':0.000173283)'':0.00224034)'':0.00387403)'':0.000951697)'':0.00354871)'':0.00992682)'':-0.00266646)'':0.0116479)'':0.00872191)'':0.0294513,(((((('PDT000244753.3':0.0398285,('PDT000579384.2':0.0404271,('PDT000107485.3':0.0268661,('PDT000532267.2':0.0268817,('PDT000316120.2':0.0242881,'PDT000544628.2':0.0176477)'':0.00867598)'':0.00389755)'':0.00212172)'':0.0107584)'':-0.00133132,(('PDT000114536.3':0.00897743,'PDT000130563.3':0.00906027)'':0.0319026,('PDT000252781.3':0.026984,('PDT000107823.3':0.0262594,('PDT000628842.2':0.0119669,('PDT000221952.3':0.0162752,'PDT000302791.3':0.0162051)'':0.00278978)'':0.00943968)'':0.00446856)'':0.0092246)'':-0.00191613)'':0.0110669,((('PDT000235774.3':0.0238694,'PDT000127408.3':0.021968)'':0.0118205,('PDT000139216.3':0.0336229,('PDT000150157.3':0.0297147,('PDT000190544.3':0.0124983,'PDT000386071.2':0.0174543)'':0.0108257)'':0.00648897)'':0.0026507)'':0.004514,('PDT000136888.3':0.042126,('PDT000145360.3':0.0262993,('PDT000497893.1':0.028947,(('PDT000235319.3':0.0265861,('PDT000072752.3':0.0164136,'PDT000124764.3':0.0174914)'':0.00584148)'':-0.00041325,(('PDT000579633.2':0.0113987,'PDT000623022.2':0.0149459)'':0.00627454,('PDT000125028.3':0.014868,('PDT000544620.2':0.0156931,'PDT000679747.1':0.00931192)'':0.00143502)'':0.00100551)'':0.0071848)'':0.00627833)'':0.00102584)'':0.00566539)'':0.000376955)'':0.00594073)'':0.00682097,((('PDT000484014.2':0.0228122,('PDT000306757.3':0.0182894,'PDT000082132.3':0.0197834)'':0.00277796)'':0.00989691,(('PDT000082088.3':0.0306364,('PDT000394693.2':0.0190564,'PDT000574065.2':0.0191858)'':0.00530594)'':0.00286671,(('PDT000252161.1':0.00786027,'PDT000498176.1':0.00611733)'':0.02115,('PDT000357227.2':0.026482,('PDT000146537.3':0.0176683,'PDT000208730.1':0.0180249)'':0.00672632)'':0.00212071)'':0.00499169)'':0.00756577)'':0.00101885,((('PDT000204928.3':0.0299925,'PDT000010365.2':0.028207)'':0.00581898,('PDT000564054.2':0.04701,(('PDT000421278.2':0.0228192,('PDT000137639.3':0.0203841,('PDT000378532.2':0.0174142,'PDT000422014.2':0.016929)'':0.00419376)'':0.00249013)'':0.00465619,('PDT000498099.1':0.0331484,(('PDT000346685.1':0.0243508,'PDT000248534.1':0.0251223)'':0.00230965,('PDT000110504.3':0.0162617,('PDT000252164.1':0.00706436,'PDT000498184.1':0.00620384)'':0.00900258)'':0.00679674)'':0.00563546)'':-0.000599692)'':0.0083455)'':-0.000490014)'':0.000996535,((('PDT000083147.3':0.0361069,'PDT000498063.1':0.0290167)'':0.00736346,(('PDT000327260.2':0.021642,'PDT000490403.2':0.0215284)'':0.00365719,('PDT000365922.2':0.0291656,(('PDT000212409.3':0.0222526,'PDT000465863.2':0.0219961)'':0.00612718,('PDT000161428.3':0.025408,'PDT000631698.2':0.0261696)'':0.00159869)'':0.00130856)'':0.00250609)'':0.00302625)'':0.00150268,('PDT000084153.3':0.0260674,((('PDT000310661.3':0.0186592,'PDT000334343.2':0.0125254)'':0.00417653,('PDT000481027.2':0.0181764,'PDT000492149.2':0.0160648)'':0.00721442)'':0.00331588,(('PDT000243700.3':0.014811,'PDT000107088.3':0.0124941)'':0.00394791,('PDT000146461.3':0.0268206,('PDT000253512.3':0.0119196,('PDT000212388.3':0.0107004,'PDT000303459.3':0.0130667)'':0.00121119)'':0.0041606)'':0.00113909)'':0.00196833)'':0.00318381)'':0.00197736)'':0.00473303)'':0.00183198)'':0.00407908)'':0.00189576,(((('PDT000563706.2':0.0431503,('PDT000619686.1':0.0232582,'PDT000619690.2':0.0296212)'':0.0135944)'':0.00669802,(('PDT000005268.2':0.0295133,'PDT000252177.1':0.0219333)'':0.00234581,(('PDT000101372.3':0.0179258,'PDT000124881.3':0.0129549)'':0.00150381,('PDT000026314.2':0.0198882,('PDT000108207.2':0.0148687,'PDT000329815.1':0.0110591)'':0.00315158)'':0.0019259)'':0.00848029)'':0.00961847)'':0.0159109,(('PDT000225311.3':0.0375326,'PDT000127278.3':0.0323296)'':0.0195951,((('PDT000557538.1':0.023095,'PDT000248712.1':0.0228442)'':0.011783,(((('PDT000562140.2':0.0165614,'PDT000587570.2':0.0113456)'':0.0088419,('PDT000108920.3':0.0208658,('PDT000322961.2':0.0112073,'PDT000377135.2':0.0113558)'':0.0111407)'':0.00353773)'':0.011605,('PDT000619816.2':0.0390531,(('PDT000157798.3':0.0201327,'PDT000378596.2':0.02173)'':0.00820513,('PDT000619742.2':0.0194423,('PDT000619687.2':0.0206889,'PDT000346682.1':0.0198331)'':0.00330742)'':0.00217205)'':0.00628026)'':-0.00266792)'':0.00131333,(('PDT000498002.1':0.0355001,('PDT000252174.1':0.0334666,('PDT000110438.3':0.0300829,('PDT000548485.2':0.0210818,('PDT000110447.3':0.0131974,'PDT000110483.3':0.0167374)'':0.00567496)'':0.0054224)'':0.00424079)'':0.00184322)'':-0.000433203,(('PDT000157721.3':0.00962846,'PDT000123336.1':0.0105504)'':0.0291567,('PDT000098724.3':0.0285315,(('PDT000252167.1':0.0185885,'PDT000497999.1':0.0144538)'':0.00514064,('PDT000120915.3':0.0215437,('PDT000248575.1':0.0172714,'PDT000252166.1':0.0218127)'':0.00417431)'':5.49075e-05)'':0.00737091)'':0.00215268)'':0.00627667)'':0.002178)'':0.00712507)'':-0.0043355,((('PDT000208683.1':0.0290022,('PDT000367197.2':0.0307934,('PDT000115021.3':0.0140562,'PDT000115035.3':0.018889)'':0.0161014)'':0.00479558)'':0.00434341,(('PDT000108931.3':0.0312047,('PDT000115027.3':0.0245141,'PDT000363547.1':0.029885)'':0.00671025)'':0.00226213,(('PDT000344147.2':0.0316572,'PDT000384054.2':0.0265496)'':0.0076442,('PDT000062530.3':0.0329442,('PDT000355579.2':0.0289733,'PDT000008722.2':0.0298788)'':0.00504168)'':0.00116468)'':0.00123152)'':0.003231)'':0.00785165,(('PDT000325685.2':0.0424853,(('PDT000238392.3':0.0226934,'PDT000253016.3':0.0183145)'':0.0091041,(('PDT000110441.3':0.0184882,'PDT000248739.1':0.0147762)'':0.012543,('PDT000049368.3':0.0233082,('PDT000157471.3':0.0269917,'PDT000562758.1':0.0216575)'':0.00218144)'':0.00513308)'':0.00283594)'':0.0118144)'':-0.00979522,('PDT000110468.3':0.0323756,(('PDT000259507.3':0.0149743,'PDT000112513.3':0.010761)'':0.0172463,(('PDT000127800.3':0.0259064,'PDT000252176.1':0.0222177)'':0.00622706,('PDT000469864.2':0.0296948,('PDT000359254.2':0.0263637,('PDT000051372.3':0.0237029,'PDT000682985.1':0.0219784)'':0.00453595)'':0.00119806)'':0.00278215)'':0.00291484)'':0.00369793)'':0.00328458)'':0.0027967)'':0.00269999)'':0.00925559)'':-0.00044401)'':-0.0045072,((('PDT000376027.2':0.0312423,'PDT000517638.2':0.0428938)'':0.0155361,(('PDT000582456.2':0.0430921,'PDT000112507.3':0.032255)'':0.0201714,('PDT000619693.2':0.0460332,('PDT000366080.2':0.0415772,'PDT000367785.2':0.0478741)'':0.00576298)'':0.0127921)'':0.00299747)'':0.00290999,((('PDT000679486.1':0.0538336,('PDT000637299.2':0.0253143,('PDT000564160.2':0.0318753,('PDT000238433.3':0.0274264,(('PDT000239611.3':0.0223257,('PDT000718294.1':0.0128812,('PDT000687880.1':0.00914262,('PDT000062613.3':0.0082197,'PDT000544407.2':0.0084569)'':0.00277633)'':0.000942556)'':0.00714905)'':0.00717527,('PDT000137692.3':0.0211321,(('PDT000081142.3':0.0128696,('PDT000146300.3':0.0131386,'PDT000574088.2':0.0122029)'':0.00108318)'':0.0044505,('PDT000498064.1':0.0124555,('PDT000247144.3':0.0127403,('PDT000247548.3':0.00902315,'PDT000107702.3':0.00765175)'':0.00173279)'':0.0029128)'':0.00113851)'':0.00714686)'':-0.0025406)'':0.00462504)'':0.00808255)'':0.00349559)'':0.0113552)'':-0.00495345,(('PDT000582452.2':0.0382637,(('PDT000162995.4':0.0407334,('PDT000486055.2':0.0232587,'PDT000107718.3':0.0207597)'':0.00424602)'':0.00142891,('PDT000047176.3':0.0341201,('PDT000225509.3':0.0194566,('PDT000142595.3':0.0141179,('PDT000619134.2':0.00953129,'PDT000110476.3':0.00842501)'':0.00387092)'':0.00210667)'':0.0107943)'':0.00248426)'':0.00609208)'':-0.00154787,(('PDT000372098.2':0.0161968,'PDT000628851.2':0.0180714)'':0.0186156,((('PDT000315814.2':0.0289543,'PDT000721085.1':0.0288351)'':0.00692487,('PDT000324579.2':0.0364944,('PDT000465590.2':0.0232721,'PDT000517745.2':0.0174858)'':0.0103367)'':0.000732859)'':0.003777,('PDT000072723.3':0.0268715,('PDT000424797.1':0.0380798,('PDT000226793.3':0.0201849,(('PDT000227541.3':0.0209262,'PDT000048662.3':0.0188846)'':0.00155553,('PDT000499169.2':0.0164124,'PDT000083203.3':0.0159791)'':0.00258897)'':0.00184655)'':0.00544042)'':0.00354422)'':0.000595687)'':0.00616241)'':-0.00315433)'':0.00470355)'':0.00176659,((('PDT000213948.3':0.0329767,('PDT000184361.3':0.0181122,'PDT000682395.1':0.0260954)'':0.00537042)'':0.0193766,('PDT000095910.1':0.0617268,(('PDT000115025.3':0.0459768,('PDT000352801.2':0.0225675,'PDT000501174.2':0.0173026)'':0.00886549)'':0.011417,('PDT000430544.2':0.0400277,(('PDT000227579.3':0.0257945,'PDT000662025.1':0.0181094)'':0.0113186,((('PDT000491932.2':0.02587,'PDT000498069.1':0.0251047)'':0.00366319,('PDT000107087.3':0.029854,('PDT000225526.3':0.022029,'PDT000396640.2':0.0200959)'':0.00944928)'':0.00140723)'':0.00126354,(('PDT000225307.3':0.0328337,'PDT000407581.2':0.029812)'':0.00413818,(('PDT000201166.3':0.014605,'PDT000573279.2':0.020837)'':0.00879675,('PDT000222758.3':0.0254054,'PDT000105068.3':0.0213392)'':0.0043292)'':0.00671115)'':0.00355042)'':0.00485564)'':0.0069748)'':-0.00339915)'':0.0134925)'':-0.0031054)'':-0.00290827,((('PDT000672426.1':0.0350457,('PDT000048217.3':0.0227454,'PDT000309638.3':0.0279729)'':0.00907296)'':0.0065597,(('PDT000126253.3':0.0328007,(('PDT000227510.3':0.0113736,'PDT000247187.3':0.0102255)'':0.0113582,('PDT000574526.2':0.0109958,'PDT000101042.3':0.00908372)'':0.00664614)'':0.0083507)'':0.000198408,((('PDT000127267.3':0.0215614,'PDT000137719.3':0.0188397)'':0.00345383,('PDT000532180.2':0.0257815,('PDT000077009.3':0.0232458,('PDT000201666.3':0.0258549,('PDT000107199.3':0.0220059,('PDT000054727.3':0.0149328,('PDT000145395.3':0.00650666,'PDT000146536.3':0.00883654)'':0.00527694)'':0.00300335)'':0.00426644)'':0.00119545)'':0.00243708)'':-0.00128094)'':0.00400954,('PDT000107093.3':0.0282103,((('PDT000184428.3':0.0110899,('PDT000490320.2':0.00618278,'PDT000125512.3':0.00725002)'':0.00844061)'':0.00872935,('PDT000265417.3':0.0325086,('PDT000069913.3':0.020717,('PDT000257223.3':0.0163497,('PDT000163008.4':0.00562117,'PDT000355467.2':0.00621373)'':0.0122567)'':0.00249151)'':0.00506933)'':-0.00172739)'':0.00344613,('PDT000300576.1':0.0259367,('PDT000233637.3':0.0246446,('PDT000085583.3':0.0243974,('PDT000231303.3':0.0162692,('PDT000357211.2':0.0196617,('PDT000400851.2':0.00978578,('PDT000394756.2':0.00810146,'PDT000072707.3':0.00768484)'':0.00306697)'':0.00226448)'':0.00270924)'':0.00330652)'':0.00209425)'':0.000737071)'':0.00143835)'':0.00705626)'':-0.0043292)'':0.00129731)'':0.00679531)'':-0.000940506,((('PDT000108886.3':0.0395479,('PDT000203372.3':0.0182115,'PDT000707593.1':0.0229673)'':0.0217613)'':0.00213386,('PDT000497746.1':0.0423341,('PDT000248742.1':0.0265026,'PDT000248862.1':0.0259085)'':0.0203073)'':0.00347215)'':0.00234739,(('PDT000105111.3':0.0239375,('PDT000142543.3':0.0240215,('PDT000302818.3':0.0162115,('PDT000114693.3':0.0189394,('PDT000149631.3':0.0122405,('PDT000250788.3':0.00449624,'PDT000457375.2':0.00667246)'':0.00544503)'':0.00357236)'':0.000471411)'':0.00311349)'':0.00786304)'':0.00896852,((('PDT000150159.3':0.02753,('PDT000387582.2':0.025248,'PDT000115030.3':0.0248399)'':0.00355292)'':0.00576346,('PDT000362558.2':0.0440292,(('PDT000351828.2':0.0369474,'PDT000489616.2':0.0391869)'':0.0054647,(('PDT000152691.3':0.0391234,('PDT000373153.2':0.0278838,(('PDT000646245.2':0.008916,'PDT000101072.3':0.0065299)'':0.00435874,('PDT000145327.3':0.0203154,('PDT000136739.3':0.00579828,('PDT000226892.3':0.00573913,'PDT000309641.3':0.00482917)'':0.000265675)'':0.00670112)'':-0.000406384)'':0.0154912)'':0.00356384)'':-0.00612263,(('PDT000451223.2':0.00653157,'PDT000481068.2':0.00639093)'':0.0207146,('PDT000640315.2':0.0298772,('PDT000558544.2':0.0204643,(('PDT000225448.3':0.0133259,'PDT000107750.3':0.0165234)'':0.000808772,(('PDT000160128.3':0.00862484,'PDT000239297.3':0.00709346)'':0.0040842,('PDT000239618.3':0.00784609,('PDT000616681.2':0.00788911,('PDT000707498.1':0.00570533,'PDT000708520.1':0.00732947)'':0.00221389)'':0.000751615)'':0.00312091)'':0.00248278)'':0.00391041)'':0.00928292)'':-0.00164836)'':-0.00361332)'':0.0100583)'':0.00926661)'':-0.00887037)'':0.00187133,((('PDT000404046.2':0.0156349,('PDT000242846.3':0.0107652,'PDT000566655.2':0.0108361)'':0.00403009)'':0.0129297,('PDT000399381.2':0.0367256,('PDT000481053.2':0.0230432,'PDT000050190.3':0.0245151)'':0.00565157)'':0.000164104)'':0.00540688,((('PDT000062523.3':0.037868,('PDT000424796.1':0.0311503,('PDT000115028.3':0.0209509,('PDT000069889.3':0.0149493,'PDT000683640.1':0.0151955)'':0.0052947)'':0.0053378)'':0.00546801)'':-0.00180443,('PDT000084211.3':0.0253847,('PDT000083142.3':0.0218799,('PDT000190717.3':0.0193205,('PDT000235790.3':0.00998635,'PDT000051249.3':0.0127365)'':0.00519278)'':0.00590816)'':0.00812041)'':-0.000586166)'':0.00304362,(('PDT000144058.3':0.00335953,'PDT000050482.3':0.00482901)'':0.0341739,('PDT000492114.2':0.0260185,(('PDT000487637.2':0.0176665,('PDT000588834.2':0.0127811,'PDT000081146.3':0.015164)'':0.00329024)'':0.00186912,(('PDT000221746.3':0.00706247,'PDT000676471.1':0.00629083)'':0.00757211,('PDT000285780.3':0.00911866,('PDT000492088.2':0.00811683,'PDT000386780.2':0.00883907)'':0.000412942)'':0.00378596)'':0.00502488)'':0.00878904)'':0.0107099)'':0.00170394)'':0.00148361)'':0.00214938)'':0.00182833)'':0.00904359)'':-0.00745532)'':0.00557065)'':0.000940533)'':0.0096873)'':-0.00628104)'':0.00117482)'':0.00613589,(((('PDT000128447.3':0.061799,('PDT000399378.2':0.0341205,'PDT000107821.3':0.0299796)'':0.00573635)'':0.00664694,(('PDT000110473.3':0.0273534,('PDT000134036.3':0.0153119,'PDT000711094.1':0.0194356)'':0.00230815)'':0.00742717,('PDT000399423.2':0.0515285,('PDT000521278.2':0.0312858,('PDT000321623.2':0.0278736,('PDT000283219.3':0.0222179,'PDT000060931.3':0.0226853)'':0.0006882)'':0.00350791)'':0.00584773)'':0.00057759)'':0.009712)'':0.00116708,(((('PDT000503054.2':0.0213271,'PDT000132347.3':0.0187279)'':0.00888664,('PDT000548579.2':0.00884818,'PDT000548610.2':0.00565462)'':0.0203028)'':0.00416908,((('PDT000320873.2':0.0327606,'PDT000252162.1':0.0338735)'':0.000588068,('PDT000136754.3':0.0238854,'PDT000009361.2':0.0277949)'':0.00444611)'':0.00273595,(('PDT000038870.1':0.0249994,('PDT000496155.2':0.0159533,'PDT000682737.1':0.0125024)'':0.0104468)'':0.00705629,('PDT000522448.2':0.0248427,(('PDT000465830.2':0.0117155,'PDT000548429.2':0.0119426)'':0.00987861,(('PDT000319323.2':0.0143249,'PDT000497938.1':0.0131926)'':0.0048253,('PDT000503391.2':0.0135625,'PDT000519343.2':0.0180862)'':0.00673573)'':0.00260825)'':0.00224751)'':-0.000328054)'':0.00327926)'':0.00298112)'':-0.000321293,((('PDT000317135.2':0.017231,'PDT000326214.2':0.0138959)'':0.0171169,('PDT000638589.2':0.0206084,'PDT000128470.3':0.0166904)'':0.00575043)'':0.00904828,(('PDT000421956.2':0.0221473,('PDT000083229.3':0.0138496,'PDT000655217.2':0.0146089)'':0.00717983)'':0.00862954,(('PDT000151461.3':0.0217667,('PDT000251623.1':0.0248481,('PDT000271163.3':0.0242835,('PDT000362576.2':0.0156788,'PDT000665109.1':0.011865)'':0.00447386)'':6.66239e-05)'':0.00199885)'':0.00670782,('PDT000500457.2':0.0386036,(('PDT000320850.2':0.0330295,'PDT000421143.2':0.0288131)'':0.0097845,((('PDT000203866.3':0.0122876,'PDT000131875.3':0.0168684)'':0.010694,('PDT000325059.2':0.0188948,('PDT000702532.1':0.0125886,('PDT000635767.2':0.00920529,'PDT000683074.1':0.0119081)'':0.000250831)'':0.00526286)'':0.00156222)'':0.00319908,('PDT000682696.1':0.0297956,('PDT000674747.1':0.036507,(('PDT000150129.3':0.0220247,'PDT000589220.2':0.0163105)'':0.00598373,('PDT000291456.3':0.0193308,('PDT000628002.2':0.0134763,('PDT000660053.2':0.0121919,'PDT000674770.1':0.0100831)'':0.00262179)'':0.00232859)'':0.00396083)'':0.00198448)'':0.00507035)'':-0.00110831)'':0.00526383)'':0.00214539)'':0.000426854)'':0.00175662)'':0.00240696)'':0.00125739)'':0.00385005)'':0.00629875,((('PDT000009358.2':0.0224378,(('PDT000548585.2':0.0266073,('PDT000548489.2':0.0150406,('PDT000548486.2':0.00675314,'PDT000548582.2':0.00804186)'':0.00935224)'':0.00404091)'':0.000350856,(('PDT000548606.2':0.0208013,('PDT000548483.2':0.0153177,('PDT000548427.2':0.0129151,(('PDT000548433.2':0.00785952,'PDT000548481.2':0.00851278)'':0.000884601,('PDT000548441.2':0.00574633,'PDT000548442.2':0.00529627)'':0.0031234)'':0.00177937)'':0.00563147)'':0.00169421)'':0.000224768,(('PDT000135819.3':0.015428,('PDT000212963.1':0.00618693,'PDT000018055.2':0.00767837)'':0.0111209)'':0.00123744,('PDT000548609.2':0.0123906,(('PDT000548488.2':0.0128994,'PDT000548583.2':0.0104141)'':0.00181586,('PDT000548428.2':0.0123499,('PDT000548584.2':0.0121602,('PDT000548430.2':0.00657881,'PDT000120913.3':0.00715659)'':0.00199891)'':0.00328487)'':0.00106723)'':0.000655464)'':0.00265587)'':0.000530294)'':0.00521178)'':0.0070486)'':0.0165605,((('PDT000051244.3':0.0437975,('PDT000557531.1':0.01105,'PDT000009357.2':0.0103658)'':0.0268526)'':0.00762434,(('PDT000373159.2':0.0457115,'PDT000130996.3':0.0368034)'':0.0120471,('PDT000492058.2':0.0420653,'PDT000251785.1':0.0429835)'':0.00518074)'':0.00321088)'':0.0158297,((('PDT000566570.2':0.0315174,('PDT000481462.2':0.0286671,(('PDT000540222.2':0.0254193,('PDT000465620.2':0.0114555,'PDT000481005.2':0.012413)'':0.00946394)'':0.00100773,('PDT000287894.3':0.0186935,('PDT000135778.3':0.0203053,('PDT000271176.3':0.0176738,'PDT000375671.2':0.0149628)'':0.00474832)'':0.00262907)'':-0.000146053)'':0.0102143)'':0.00283378)'':-0.00725991,((('PDT000107706.3':0.0177787,'PDT000123289.1':0.0231726)'':0.0108898,('PDT000295953.3':0.0237106,('PDT000397318.2':0.0227354,'PDT000050475.3':0.0265673)'':0.00206883)'':0.00333619)'':0.000196342,((('PDT000084221.3':0.0194704,('PDT000259764.3':0.0151037,'PDT000264783.3':0.0212923)'':0.00774897)'':0.00149946,('PDT000084213.3':0.0246394,('PDT000105126.3':0.0226368,('PDT000469862.2':0.0152357,('PDT000383989.2':0.0101675,'PDT000107828.3':0.00964629)'':0.0037971)'':0.00754153)'':0.00210432)'':0.00133747)'':0.00128136,((('PDT000483997.2':0.00917748,'PDT000544966.2':0.0117821)'':0.00380689,('PDT000465786.2':0.0111396,('PDT000390253.2':0.00957426,'PDT000491508.2':0.0111208)'':0.00235867)'':0.00294141)'':0.00758747,(('PDT000494618.2':0.0227995,'PDT000555706.2':0.0271712)'':0.00218027,('PDT000097102.3':0.0248836,('PDT000399391.2':0.0175144,('PDT000101020.4':0.014898,('PDT000624724.2':0.0134433,(('PDT000309826.3':0.0102063,('PDT000176670.3':0.00925009,'PDT000044718.3':0.0100951)'':0.000981545)'':0.00132363,('PDT000134048.3':0.0104108,('PDT000283362.3':0.0094452,'PDT000496509.2':0.0099)'':0.00258183)'':0.000549193)'':0.00220132)'':0.000595537)'':0.000919192)'':0.0017806)'':0.00338744)'':-0.00138808)'':0.00265683)'':0.0035824)'':-0.000729021)'':0.00595608,(('PDT000264803.3':0.0341392,'PDT000519346.2':0.0355441)'':0.0100449,(('PDT000107701.3':0.0471626,('PDT000204952.3':0.0379632,((('PDT000146562.3':0.0259154,'PDT000124788.3':0.019911)'':0.00369698,('PDT000368337.2':0.0244012,('PDT000329330.2':0.0134155,'PDT000137576.3':0.00974549)'':0.00944514)'':0.00105243)'':0.00163447,(('PDT000149247.3':0.0184303,'PDT000562836.2':0.0244043)'':0.00511465,(('PDT000639824.2':0.0189221,('PDT000271306.3':0.0141659,'PDT000051379.3':0.0113023)'':0.00388832)'':0.00281765,('PDT000407677.2':0.0165489,('PDT000280716.3':0.0132014,'PDT000083155.3':0.0121226)'':0.00359568)'':0.00375144)'':0.00541482)'':-0.000179412)'':0.00766918)'':0.00698949)'':-0.0120884,((('PDT000072432.3':0.023903,'PDT000107705.3':0.0257105)'':0.00482576,('PDT000545350.2':0.0250893,'PDT000365746.2':0.0287478)'':0.00119849)'':0.00678997,(('PDT000359193.2':0.0262134,('PDT000163366.4':0.0321195,('PDT000421024.2':0.0221759,('PDT000365902.2':0.021029,('PDT000267472.3':0.0165073,'PDT000309616.3':0.0224113)'':0.00321971)'':0.00147917)'':0.00198312)'':0.00555396)'':-0.00221279,(('PDT000481025.2':0.0290351,(('PDT000184447.3':0.0196193,'PDT000307523.3':0.0267272)'':0.00263042,('PDT000235333.3':0.0194911,'PDT000394736.2':0.0218028)'':0.0025794)'':0.00355417)'':-0.00131516,(('PDT000405098.2':0.0280335,(('PDT000239180.3':0.0222666,('PDT000641594.2':0.016897,(('PDT000367018.2':0.0059288,'PDT000564914.2':0.0062069)'':0.000582416,('PDT000305214.3':0.00499363,'PDT000107717.3':0.00565737)'':0.00159886)'':0.010837)'':0.00497645)'':-0.000622844,('PDT000054720.3':0.0200349,(('PDT000386301.2':0.0158886,(('PDT000153295.3':0.00830945,'PDT000708525.1':0.00798625)'':0.00450999,('PDT000046591.3':0.0112225,'PDT000120019.3':0.0102563)'':0.00195753)'':0.00263088)'':-0.000897274,('PDT000082080.3':0.0181288,('PDT000107488.3':0.00770652,('PDT000404027.2':0.00795914,('PDT000150668.3':0.00469111,('PDT000238419.3':0.00534255,'PDT000125481.3':0.00481175)'':0.000580904)'':0.000702388)'':0.00261125)'':0.00556183)'':0.000394666)'':0.00307835)'':0.000740305)'':0.00731896)'':-0.0061348,('PDT000139179.3':0.029806,(('PDT000653334.2':0.02482,('PDT000317579.2':0.0112074,('PDT000101059.3':0.00825381,('PDT000305219.3':0.00544028,'PDT000392138.2':0.00670142)'':0.00224774)'':0.00444829)'':0.0101825)'':0.000532246,(('PDT000126370.3':0.0245373,('PDT000671301.1':0.0175723,('PDT000149649.3':0.0094254,'PDT000046381.3':0.0109281)'':0.00610162)'':0.00200947)'':0.000405916,(('PDT000251631.1':0.0201119,('PDT000115033.3':0.0153253,('PDT000163605.3':0.0134344,('PDT000107720.3':0.0060092,'PDT000137583.3':0.0044678)'':0.00364603)'':0.00471305)'':0.00336635)'':0.000207441,(('PDT000541581.2':0.0146945,'PDT000679768.1':0.0160596)'':0.00093292,('PDT000107094.3':0.0122389,('PDT000579617.2':0.0151672,('PDT000147920.3':0.0040387,'PDT000547083.2':0.00563808)'':0.00483945)'':0.00335017)'':0.004667)'':0.000550101)'':0.00329599)'':0.000988969)'':0.00759356)'':-0.00664138)'':0.00198546)'':0.000536814)'':0.00311896)'':-0.00227961)'':0.0175764)'':-0.0151123)'':0.0148361)'':-0.00280162)'':-0.00382494,(((('PDT000718777.1':0.0385698,('PDT000481983.2':0.0325316,('PDT000486039.2':0.0393377,'PDT000107092.3':0.0360503)'':0.00284266)'':0.00555798)'':0.00199689,('PDT000430562.2':0.0359178,('PDT000544301.2':0.0364007,('PDT000147988.3':0.0232155,(('PDT000486338.2':0.0153861,'PDT000260738.3':0.0119433)'':0.00299519,('PDT000242760.3':0.0231536,('PDT000232786.3':0.0086086,'PDT000580137.2':0.0088586)'':0.00455775)'':0.00341919)'':0.00645335)'':0.00164247)'':0.00398503)'':0.00493755)'':-0.00035516,((('PDT000334276.2':0.0298328,('PDT000190304.3':0.0243968,'PDT000222094.3':0.021733)'':0.00750706)'':0.00314587,(('PDT000373171.2':0.0145966,('PDT000146284.3':0.00932065,'PDT000571222.2':0.00908185)'':0.0105266)'':0.0082769,('PDT000498068.1':0.0204354,('PDT000274054.3':0.00872322,'PDT000084975.3':0.00897938)'':0.00804653)'':0.0022637)'':0.00626051)'':0.00821046,(('PDT000086331.3':0.0316889,('PDT000492165.2':0.0304401,'PDT000119992.3':0.0298453)'':0.0061471)'':0.00596631,('PDT000188000.3':0.044604,(('PDT000135345.3':0.00876173,('PDT000048653.3':0.00549101,'PDT000097106.3':0.00606049)'':0.00282992)'':0.00696411,('PDT000312548.3':0.0322447,('PDT000209174.3':0.0221397,('PDT000227546.3':0.0195559,('PDT000197034.3':0.0123103,'PDT000663643.1':0.0136907)'':0.00162253)'':0.00217048)'':0.00632874)'':-0.00221764)'':0.0154447)'':0.000562021)'':0.00707885)'':0.0019814)'':0.00456858,(((('PDT000107714.3':0.0452318,('PDT000360404.2':0.0500494,'PDT000602883.2':0.0302303)'':0.00626673)'':0.0039969,('PDT000062526.5':0.0402152,(('PDT000315849.2':0.0251524,('PDT000151177.3':0.0285766,('PDT000482231.2':0.034194,('PDT000598809.1':0.0228453,('PDT000295937.3':0.014167,('PDT000176766.3':0.0108104,'PDT000227557.3':0.0146912)'':0.000868833)'':0.00761849)'':0.00187904)'':0.00334688)'':0.00155189)'':0.00179382,(('PDT000459003.2':0.0237562,(('PDT000375784.2':0.0155635,'PDT000503811.2':0.0154751)'':0.00529508,('PDT000259759.3':0.0105683,('PDT000152275.3':0.00577629,'PDT000280721.3':0.00498821)'':0.00959546)'':0.00358385)'':0.00186333)'':0.00228094,('PDT000457378.2':0.0407199,('PDT000457383.2':0.0252496,('PDT000142531.3':0.0209976,('PDT000390426.2':0.0156713,('PDT000637300.2':0.0125938,('PDT000245726.3':0.0117342,'PDT000359231.2':0.0142848)'':0.000919553)'':0.00238069)'':0.00371995)'':0.00482744)'':0.00727065)'':-0.003492)'':0.0041332)'':0.011182)'':-0.00215182)'':0.00165827,((('PDT000377972.2':0.0276607,('PDT000381329.2':0.00723209,'PDT000683380.1':0.00600961)'':0.0157909)'':0.00861759,((('PDT000533414.2':0.0200581,'PDT000610014.2':0.0157554)'':0.00370092,('PDT000157468.3':0.0230722,('PDT000480902.2':0.0127304,('PDT000107825.3':0.0118857,('PDT000566614.2':0.0124968,'PDT000677273.1':0.0118283)'':0.00181389)'':0.00189669)'':0.00269848)'':0.000947991)'':0.00540163,('PDT000584993.2':0.0300728,(('PDT000465779.2':0.0257863,('PDT000146504.3':0.0165141,'PDT000191654.3':0.0170429)'':0.00358615)'':0.00420544,('PDT000189665.3':0.0286761,('PDT000481010.2':0.0277243,('PDT000214008.3':0.02187,(('PDT000201224.3':0.0162603,('PDT000355517.2':0.0141766,('PDT000163378.4':0.0111593,(('PDT000407565.2':0.00402484,'PDT000520367.2':0.00475672)'':0.00451678,('PDT000521250.2':0.00731349,'PDT000124844.3':0.00744081)'':0.00174487)'':0.00152381)'':0.000844305)'':0.00204254)'':-0.000639195,(('PDT000300523.1':0.0158935,('PDT000247159.3':0.0131682,('PDT000198489.3':0.00917155,'PDT000617024.2':0.0100784)'':0.00233673)'':0.00299199)'':0.00166813,('PDT000151467.3':0.0166187,('PDT000114647.3':0.0130402,('PDT000148893.3':0.0100435,('PDT000386013.2':0.00710857,(('PDT000480387.2':0.00594738,'PDT000687883.1':0.00699052)'':0.00113433,('PDT000240655.3':0.00911017,('PDT000115052.3':0.0060911,('PDT000102794.3':0.00574143,('PDT000208187.3':0.00374569,'PDT000641703.2':0.00342991)'':0.00181864)'':0.000619106)'':0.000816671)'':5.039e-05)'':0.000265044)'':0.00232229)'':0.00102691)'':0.00129673)'':0.000364156)'':-0.000101567)'':0.00345916)'':0.00633451)'':0.00116142)'':-0.00439667)'':0.00640127)'':-0.00721518)'':0.00415697)'':0.00416002,(((('PDT000250833.3':0.0379929,'PDT000549495.2':0.031612)'':0.00916841,(('PDT000492172.2':0.0193602,'PDT000346686.1':0.0211419)'':0.00373999,(('PDT000259802.3':0.0211428,'PDT000315870.2':0.0231951)'':0.00360143,((('PDT000044721.3':0.00731025,'PDT000107840.3':0.00689455)'':0.0118565,('PDT000352826.2':0.0158454,'PDT000086178.3':0.0154293)'':0.00158091)'':0.00289016,('PDT000344768.2':0.0286807,('PDT000719998.1':0.0187483,(('PDT000230118.3':0.0192261,'PDT000134066.3':0.0189711)'':0.00166675,('PDT000619148.2':0.0161001,('PDT000124955.3':0.0184564,('PDT000381332.2':0.0115271,('PDT000417217.2':0.012377,((('PDT000227500.3':0.00747974,'PDT000355602.2':0.00721596)'':0.00232198,('PDT000546081.2':0.00769237,('PDT000247417.3':0.00669187,'PDT000130980.3':0.00792043)'':0.00244393)'':0.000755564)'':0.00217413,('PDT000062599.3':0.0115613,('PDT000361212.2':0.011517,('PDT000289137.3':0.0106464,('PDT000202816.3':0.00880397,'PDT000107711.3':0.00825103)'':0.00142511)'':0.000784765)'':0.00124097)'':0.000474407)'':0.000814723)'':0.000999578)'':0.00163043)'':0.00249236)'':-0.00159003)'':0.00554741)'':0.0016887)'':-0.0055408)'':0.00371158)'':0.00126224)'':0.0128276)'':-0.0127532,(('PDT000677506.1':0.0205498,('PDT000146452.3':0.00652166,'PDT000387581.2':0.00770574)'':0.0113247)'':0.000160869,(('PDT000451247.2':0.0205883,('PDT000221969.3':0.0157938,'PDT000342321.2':0.0190934)'':0.00354127)'':-0.000845062,(('PDT000227548.3':0.0164031,('PDT000576146.2':0.0124565,'PDT000688106.1':0.0137344)'':0.00133378)'':0.00183099,('PDT000480414.2':0.0200117,((('PDT000390407.2':0.0154391,'PDT000096899.3':0.0148609)'':0.00565103,('PDT000575822.2':0.0144621,('PDT000431653.2':0.0109882,('PDT000214063.3':0.0102752,('PDT000247173.3':0.00795233,('PDT000098738.3':0.00689491,('PDT000417813.2':0.00625815,('PDT000237161.3':0.00592521,('PDT000144415.3':0.0047985,'PDT000708617.1':0.00376638)'':0.0018222)'':0.000548855)'':0.00125859)'':0.000336854)'':0.00138411)'':0.00130995)'':0.00189306)'':0.00354983)'':-0.0027426,(('PDT000375673.2':0.0121878,'PDT000107709.3':0.0132628)'':0.00252296,('PDT000316155.2':0.0220631,(('PDT000212361.3':0.0143754,'PDT000498044.1':0.01255)'':0.000722367,(('PDT000252754.3':0.0149782,('PDT000144494.3':0.00874283,'PDT000160127.3':0.00890887)'':0.00509557)'':-1.4655e-05,(('PDT000279505.3':0.0117031,'PDT000126452.3':0.00990252)'':0.000266448,('PDT000492143.2':0.010356,('PDT000083957.5':0.0074576,'PDT000124760.3':0.0113673)'':0.000540249)'':0.00239634)'':0.000914982)'':0.000582181)'':0.00221073)'':-0.00107331)'':6.47818e-05)'':0.00425637)'':-0.00277567)'':0.00301935)'':0.00147225)'':-2.45451e-05)'':0.00423283,((('PDT000150697.3':0.0236854,'PDT000471098.2':0.0248877)'':0.008359,(('PDT000595794.2':0.0222744,'PDT000687899.1':0.0187512)'':0.00953466,('PDT000394038.2':0.0267509,('PDT000227294.3':0.0231058,(('PDT000150671.3':0.0062291,'PDT000399897.2':0.0085193)'':0.00568099,('PDT000215097.3':0.00935212,'PDT000050485.3':0.00908018)'':0.00168871)'':0.00665066)'':0.00574311)'':-0.000352936)'':0.00214963)'':0.0021397,((('PDT000047183.3':0.0225429,('PDT000366069.2':0.0175596,'PDT000400840.2':0.0207925)'':0.00151376)'':0.0035034,(('PDT000189511.3':0.029489,'PDT000233638.3':0.0233393)'':0.00575895,('PDT000672651.1':0.0232767,('PDT000072443.3':0.0164868,('PDT000399907.2':0.0154629,('PDT000709755.1':0.0177308,(('PDT000176759.3':0.0104565,'PDT000238328.3':0.0107295)'':0.00252631,(('PDT000277150.3':0.00966936,'PDT000312474.3':0.00819664)'':0.00309101,('PDT000366288.2':0.0123166,('PDT000324588.2':0.0153992,('PDT000307522.3':0.0126246,((('PDT000190136.3':0.00605795,'PDT000547081.2':0.00519815)'':0.00220807,('PDT000243721.3':0.00651729,'PDT000133753.3':0.00572881)'':0.00219893)'':0.00113816,(('PDT000317570.2':0.00638553,'PDT000105100.3':0.00794427)'':0.00212587,(('PDT000556488.2':0.00581582,('PDT000151417.3':0.00409771,('PDT000143250.3':0.00433345,'PDT000492076.2':0.00482534)'':0.000739276)'':0.00188629)'':0.000324112,('PDT000242756.3':0.00715824,('PDT000098710.3':0.00727881,('PDT000146293.3':0.00521365,('PDT000443418.2':0.00333731,'PDT000492133.2':0.00384473)'':0.000662757)'':0.00186298)'':0.00176339)'':-0.001191)'':0.00163739)'':0.000465189)'':0.0012561)'':0.000443732)'':0.00131227)'':-0.00147905)'':0.000836986)'':0.00307584)'':0.00168258)'':0.000982468)'':0.000514809)'':0.00115196)'':-2.26529e-05)'':-0.00133807,((('PDT000503050.2':0.00721463,'PDT000510662.2':0.00908427)'':0.00380439,('PDT000187998.3':0.0116828,'PDT000147953.3':0.0176939)'':0.00139828)'':0.00747311,(('PDT000566472.2':0.0233844,('PDT000466034.2':0.0155425,('PDT000315852.2':0.0116595,'PDT000250810.5':0.0112813)'':0.00406278)'':0.00417121)'':0.00129688,(('PDT000574064.2':0.0193992,('PDT000302795.3':0.0216926,('PDT000150089.3':0.0121879,('PDT000396648.2':0.0130691,('PDT000239620.3':0.00898594,(('PDT000351809.2':0.008774,('PDT000163624.3':0.00594404,'PDT000501765.2':0.00558906)'':0.00342015)'':-7.56369e-05,('PDT000282452.3':0.00543186,('PDT000208174.3':0.00387827,'PDT000393031.2':0.0041967)'':0.00182771)'':0.00219564)'':0.00208037)'':0.00167611)'':0.00152808)'':0.00306022)'':0.00182215)'':-0.00367296,(('PDT000484004.2':0.0182953,('PDT000144399.3':0.00991742,('PDT000316142.2':0.0103748,'PDT000600804.2':0.00802031)'':0.00253808)'':0.00523649)'':0.00329837,('PDT000150136.3':0.0366547,(('PDT000599108.2':0.0116661,(('PDT000243670.3':0.00626778,'PDT000455238.2':0.00664822)'':0.00121806,('PDT000481330.2':0.00701471,('PDT000264804.3':0.00531994,('PDT000305209.3':0.00610028,(('PDT000355500.2':0.00384008,'PDT000396616.2':0.00424375)'':0.000644473,('PDT000566497.2':0.00249905,'PDT000672401.1':0.0027909)'':0.000942595)'':0.000965318)'':0.000671627)'':0.0013845)'':0.000646839)'':0.0040404)'':-0.00248087,('PDT000510691.2':0.0184677,(('PDT000233658.3':0.0127,'PDT000495716.2':0.0124741)'':0.00356238,('PDT000603493.2':0.0142048,(('PDT000387574.2':0.00954921,'PDT000627565.2':0.0104567)'':0.000924548,(('PDT000563932.2':0.012978,('PDT000492154.2':0.00866006,'PDT000643525.2':0.00820004)'':0.00252406)'':0.00122626,(('PDT000576012.2':0.00890063,('PDT000375744.2':0.00737081,'PDT000410891.2':0.00906179)'':0.00190737)'':-7.64826e-05,(('PDT000287895.3':0.00683205,'PDT000351583.2':0.00630355)'':0.00188375,(('PDT000574529.2':0.00897418,'PDT000107713.3':0.0102031)'':0.00266641,('PDT000115022.3':0.00848961,(('PDT000135746.3':0.0058931,('PDT000594291.2':0.00529274,'PDT000679119.1':0.00574986)'':0.000279954)'':0.00170023,('PDT000300531.1':0.00879039,('PDT000344784.2':0.0100745,('PDT000121848.3':0.0106805,('PDT000643956.2':0.0112872,('PDT000492121.2':0.015441,('PDT000579634.2':0.00697266,('PDT000084250.3':0.00634673,(('PDT000321953.2':0.00699325,(('PDT000622882.2':0.00450325,'PDT000135749.3':0.00588425)'':0.000228179,('PDT000153280.3':0.00466053,('PDT000492158.2':0.00372658,'PDT000583331.2':0.00295405)'':0.000812591)'':0.000721321)'':0.00104108)'':-0.000384262,('PDT000368902.1':0.00778524,(('PDT000677073.1':0.00442546,('PDT000125523.3':0.0038942,('PDT000298470.3':0.00287337,'PDT000124807.3':0.00301133)'':0.000890572)'':0.00105806)'':0.000378838,('PDT000594871.2':0.00721934,('PDT000143257.3':0.00460868,(('PDT000128445.3':0.00645147,('PDT000191642.3':0.00282293,('PDT000492168.2':0.0028629,'PDT000135923.3':0.00272704)'':0.000468206)'':0.00197871)'':-0.000217916,(('PDT000501048.2':0.00548929,('PDT000132580.3':0.00311057,('PDT000238414.3':0.00339903,'PDT000124779.3':0.00278674)'':0.000483232)'':0.00112821)'':0.000780101,('PDT000102902.3':0.00458373,(('PDT000257983.3':0.00491861,'PDT000086191.3':0.00425387)'':0.000245725,('PDT000247178.3':0.00434113,('PDT000143058.3':0.00467989,(('PDT000469873.2':0.00350601,'PDT000640312.2':0.00376998)'':0.000983303,(('PDT000465600.2':0.00355385,('PDT000344391.2':0.00199542,'PDT000569706.2':0.00219662)'':0.00143247)'':-0.000109192,('PDT000270031.3':0.00378727,(('PDT000538639.2':0.00298038,('PDT000190671.3':0.00298446,'PDT000690179.1':0.00270303)'':0.000609092)'':0.000292509,(('PDT000356218.2':0.00269228,'PDT000551427.2':0.00229724)'':0.000431162,('PDT000366088.2':0.00360321,('PDT000708514.1':0.00193767,('PDT000167582.3':0.0016916,('PDT000492170.2':0.00134259,'PDT000124777.3':0.0013539)'':0.000604622)'':0.000388489)'':0.000901385)'':0.000233424)'':0.000210675)'':0.000150958)'':-4.95338e-05)'':0.000133285)'':0.000767806)'':0.000245055)'':-0.000338732)'':0.000623638)'':-0.000427958)'':0.000130356)'':0.00064322)'':0.000911966)'':-0.00136498)'':0.00186397)'':-0.00155685)'':0.00190755)'':0.000157138)'':0.00131325)'':0.000114109)'':6.20568e-05)'':8.81552e-05)'':0.000720597)'':-0.0025799)'':0.0030703)'':-0.00185109)'':0.000211523)'':0.000140578)'':0.000342759)'':0.0019672)'':0.00203763)'':-0.000829254)'':0.00380357)'':-0.00730395)'':0.0101762)'':-0.00422118)'':-0.00296102)'':0.00433543)'':0.00351675)'':-0.00539151)'':0.00509165)'':-0.00108671)'':0.00123223)'':0.00914563)'':-0.00144623,((('PDT000009666.2':0.0722284,(('PDT000072750.3':0.0463622,('PDT000235499.3':0.0508688,'PDT000083213.3':0.0389072)'':0.00744121)'':0.00475134,(('PDT000325504.2':0.0401131,'PDT000050473.3':0.0321983)'':0.0196276,('PDT000634592.2':0.0555442,'PDT000137526.3':0.0441491)'':0.0068031)'':0.00816436)'':0.00700759)'':0.00768143,(((('PDT000190355.3':0.0260897,'PDT000285785.3':0.026336)'':0.00672588,('PDT000346684.1':0.0354805,('PDT000120842.3':0.0295373,(('PDT000050488.3':0.0171242,'PDT000682454.1':0.0248484)'':0.00805822,('PDT000465777.2':0.0220912,('PDT000134047.3':0.0168601,('PDT000071660.3':0.00996841,'PDT000081150.3':0.0131215)'':0.00338717)'':0.00428653)'':0.00343194)'':0.0057923)'':0.00222694)'':-0.00202791)'':0.00178243,(('PDT000573322.2':0.0317276,('PDT000491891.2':0.0182026,('PDT000145277.3':0.0189735,('PDT000329852.2':0.00342093,'PDT000556406.2':0.00375825)'':0.0118178)'':0.00257743)'':0.0107092)'':0.000191678,('PDT000297043.3':0.0361719,(('PDT000617676.2':0.011125,('PDT000173944.3':0.0101393,(('PDT000260749.3':0.00604945,'PDT000404009.2':0.00659865)'':0.00247525,('PDT000564164.2':0.00262345,'PDT000124809.3':0.00456577)'':0.00423844)'':0.00180542)'':0.00681571)'':0.00515455,('PDT000478264.2':0.025399,('PDT000108892.3':0.0174846,(('PDT000184457.3':0.0119291,'PDT000583841.2':0.00731323)'':0.00309241,('PDT000271206.3':0.00683204,'PDT000492212.2':0.00984126)'':0.00162439)'':0.00773442)'':0.00263456)'':-7.4108e-05)'':0.0103229)'':-0.0044621)'':0.000452254)'':0.00133259,((('PDT000375725.2':0.0330136,'PDT000540020.2':0.0306404)'':0.00349841,('PDT000491624.2':0.0307989,('PDT000176696.3':0.0342472,('PDT000150150.3':0.0108742,'PDT000190100.3':0.0140835)'':0.00813822)'':0.00730682)'':-0.000154239)'':0.00824174,(('PDT000628834.2':0.0449076,(('PDT000152684.3':0.0243312,'PDT000268726.3':0.0231576)'':0.00217826,('PDT000025697.2':0.0215665,('PDT000386831.2':0.0258995,('PDT000107704.3':0.0223881,'PDT000107826.3':0.0203218)'':0.00115695)'':0.00155782)'':0.00323251)'':0.00994932)'':-0.00153103,(('PDT000084135.3':0.035116,(('PDT000367756.2':0.0199507,'PDT000598706.1':0.0219419)'':0.00655313,(('PDT000250885.3':0.0310471,'PDT000125022.3':0.0207078)'':0.00300622,(('PDT000599899.2':0.0122498,'PDT000626978.2':0.0112528)'':0.00346372,('PDT000359291.2':0.0262873,(('PDT000048659.3':0.017248,'PDT000084027.3':0.0174688)'':0.00307049,(('PDT000231244.3':0.0123452,'PDT000086318.3':0.0105563)'':0.00261976,('PDT000233642.3':0.0128072,'PDT000346681.1':0.0121829)'':0.000411864)'':0.0035728)'':0.00176389)'':0.00191495)'':0.00127373)'':0.00121325)'':0.00283408)'':-0.00313568,((('PDT000054728.3':0.0303831,'PDT000410914.2':0.0195031)'':0.00102163,('PDT000548755.2':0.0193416,('PDT000112512.3':0.0210877,('PDT000319655.2':0.0177697,('PDT000334604.2':0.0096077,'PDT000083274.3':0.0068298)'':0.00849324)'':0.00269095)'':0.00188771)'':0.00308439)'':-0.000334673,('PDT000431635.2':0.0264774,(('PDT000151474.4':0.0126831,('PDT000592640.2':0.0103647,'PDT000008480.2':0.0101405)'':0.00763243)'':-0.000518631,('PDT000148542.3':0.017689,('PDT000609733.2':0.0124459,('PDT000525271.2':0.0069662,('PDT000141668.3':0.00811967,('PDT000298477.3':0.00725555,('PDT000124880.3':0.00471539,('PDT000214009.3':0.0020093,'PDT000404070.2':0.00158567)'':0.00186362)'':0.00212236)'':0.00194503)'':0.0023503)'':0.00150474)'':0.00246853)'':-0.000115064)'':0.0109676)'':-0.000947826)'':0.00152976)'':0.00506473)'':0.00339507)'':-0.00398175)'':0.0185088)'':-0.00886372,(((('PDT000189722.3':0.0149704,'PDT000247393.3':0.0273392)'':0.0243044,('PDT000150132.3':0.0345399,('PDT000708526.1':0.0380377,(('PDT000150978.3':0.0117588,'PDT000107719.3':0.0169489)'':0.00868893,('PDT000054729.3':0.0115364,('PDT000216605.3':0.00972845,'PDT000712540.1':0.00950995)'':0.00467723)'':0.00266836)'':0.0059809)'':0.000946078)'':0.00591482)'':0.00242568,(('PDT000019928.2':0.0743272,(('PDT000191650.3':0.0296817,('PDT000044511.3':0.0237706,'PDT000408396.2':0.0193024)'':0.00598734)'':-0.000599066,(('PDT000388877.2':0.026844,'PDT000410924.2':0.0263776)'':0.00411616,(('PDT000144045.3':0.0175741,('PDT000491481.2':0.0117311,(('PDT000235070.3':0.0134124,'PDT000077505.1':0.00971872)'':0.000894254,('PDT000137985.3':0.00924571,('PDT000184356.3':0.00479346,('PDT000508952.2':0.00416996,('PDT000234228.3':0.00309912,'PDT000546197.2':0.00378093)'':0.000608399)'':0.0010455)'':0.00121547)'':0.00120725)'':0.00351128)'':0.00576591)'':-0.0018764,(('PDT000548750.2':0.0122385,'PDT000097025.3':0.0138768)'':0.00319686,(('PDT000025344.2':0.012366,('PDT000025730.2':0.0100749,'PDT000025368.1':0.0105416)'':0.00182732)'':0.00297722,('PDT000239622.3':0.0137984,(('PDT000107716.3':0.0127479,'PDT000133757.3':0.00943014)'':0.00105663,('PDT000525263.2':0.0139385,('PDT000062615.3':0.0108659,'PDT000300559.1':0.011233)'':0.00213999)'':4.40049e-05)'':0.00361549)'':0.000747573)'':0.00204835)'':0.00459646)'':0.00324547)'':0.00880039)'':0.0167562)'':-0.0134966,(((('PDT000235053.3':0.0106874,('PDT000176685.3':0.0123897,('PDT000189517.3':0.00434463,'PDT000687642.1':0.0043233)'':0.00301062)'':0.00168348)'':0.0128008,('PDT000115014.3':0.0147534,('PDT000719900.1':0.01994,('PDT000372130.2':0.0115828,'PDT000533411.2':0.0104629)'':0.00240424)'':0.00582155)'':0.00454038)'':0.00142688,(('PDT000048660.3':0.0104484,'PDT000383962.2':0.0145938)'':0.00665665,('PDT000319666.2':0.0238214,('PDT000201313.3':0.0276029,('PDT000587023.2':0.0119857,(('PDT000147622.3':0.00893416,'PDT000719938.1':0.00764314)'':0.000768557,('PDT000190658.3':0.0140642,('PDT000198974.3':0.0106987,('PDT000124811.3':0.00793684,('PDT000203377.3':0.00502756,'PDT000265136.3':0.00534544)'':0.00221921)'':0.00102375)'':0.00216595)'':-0.000566194)'':0.00275919)'':0.00580982)'':0.00139706)'':-0.000699153)'':0.000336768)'':0.00523973,(('PDT000326261.2':0.02252,('PDT000270105.3':0.00697574,'PDT000307914.3':0.00735116)'':0.0196496)'':0.00361501,((('PDT000190682.3':0.0116861,'PDT000574082.2':0.0166897)'':0.00531777,('PDT000077516.1':0.0170994,('PDT000522478.2':0.0104844,('PDT000332920.2':0.00937832,('PDT000151516.3':0.00798205,'PDT000520091.2':0.00741785)'':0.00162803)'':0.00132231)'':0.00321494)'':0.00303044)'':0.00325726,((('PDT000280719.3':0.0255473,'PDT000134075.3':0.0229173)'':0.00578064,('PDT000455264.2':0.00947526,('PDT000317581.2':0.00521012,'PDT000115026.3':0.00553938)'':0.00390959)'':0.0119666)'':8.91011e-05,((('PDT000315788.2':0.0199609,('PDT000488029.2':0.0137679,'PDT000707506.1':0.0129764)'':0.00309402)'':0.00287314,('PDT000584614.2':0.0145032,('PDT000492220.2':0.0134187,'PDT000518082.2':0.0140988)'':0.00127884)'':0.00316733)'':0.00169325,('PDT000409220.2':0.0182629,(('PDT000215058.3':0.0104407,'PDT000492206.2':0.0140172)'':0.00283018,('PDT000134073.3':0.0148851,('PDT000300532.1':0.0131714,('PDT000243614.3':0.0110482,(('PDT000125521.3':0.0086544,('PDT000355459.2':0.00264468,'PDT000492148.2':0.00244768)'':0.00724712)'':0.00107842,('PDT000106269.3':0.00683454,('PDT000113772.3':0.0065858,'PDT000130552.3':0.0053652)'':0.00201846)'':0.00242684)'':0.00191317)'':0.00234329)'':0.000427037)'':-0.000528149)'':0.00271574)'':0.00125714)'':0.000900738)'':0.000268013)'':0.00615589)'':-0.00377557)'':0.00411738)'':0.00435879)'':-0.00182864,((('PDT000344867.2':0.0270739,('PDT000252163.1':0.0121127,('PDT000051174.1':0.00680363,'PDT000497297.2':0.00496577)'':0.00780486)'':0.0179621)'':0.00221227,((('PDT000115045.3':0.0311024,('PDT000275627.3':0.0240005,'PDT000252168.1':0.0303591)'':0.0062617)'':0.00291733,('PDT000144059.3':0.0267864,(('PDT000431696.2':0.0139641,'PDT000008764.2':0.0147426)'':0.00493075,('PDT000094918.4':0.0218354,'PDT000009665.2':0.0190395)'':0.000535797)'':0.00434033)'':0.00397544)'':-8.93746e-05,(('PDT000226795.3':0.0255594,('PDT000321943.2':0.0128021,('PDT000049364.3':0.0106586,'PDT000120005.3':0.012837)'':0.00688354)'':0.00397034)'':0.0032497,(('PDT000218514.3':0.037019,(('PDT000144012.3':0.0208302,('PDT000581288.2':0.0252971,('PDT000518638.2':0.0171968,('PDT000213862.3':0.0164545,('PDT000214013.3':0.00776628,'PDT000581136.2':0.00749552)'':0.00346676)'':0.0025285)'':0.00352784)'':0.00142101)'':-0.00140062,(('PDT000329319.2':0.0102209,'PDT000352825.2':0.0127585)'':0.0107436,('PDT000143057.3':0.00726746,('PDT000141671.3':0.00569356,('PDT000107824.3':0.00523695,'PDT000120916.3':0.00762575)'':0.00129679)'':0.00285953)'':0.00583631)'':0.000637931)'':0.00966859)'':-0.00709116,('PDT000471094.2':0.0405317,(('PDT000410902.2':0.0184461,(('PDT000107760.3':0.010383,'PDT000065370.1':0.0105329)'':0.00460954,('PDT000602359.2':0.0111421,'PDT000443262.2':0.0113164)'':0.00253573)'':0.00574999)'':-0.00115014,(('PDT000522413.2':0.022273,'PDT000649107.2':0.0289434)'':0.00330884,(('PDT000249809.1':0.0206134,('PDT000575645.2':0.0143263,('PDT000125478.3':0.0110112,('PDT000315825.2':0.00737704,'PDT000671480.1':0.00823396)'':0.000864143)'':0.00272788)'':0.00316624)'':0.000364071,(('PDT000120911.3':0.0177623,(('PDT000545781.2':0.00908971,'PDT000115024.3':0.0139045)'':0.00106777,('PDT000107708.3':0.0053984,'PDT000114522.3':0.007842)'':0.00506838)'':0.00176479)'':0.00105428,(('PDT000139215.3':0.0134856,('PDT000338860.2':0.0112207,'PDT000545699.2':0.0114279)'':0.00149736)'':0.00326225,(('PDT000281576.3':0.00627544,'PDT000429542.2':0.00704336)'':0.00364123,('PDT000498026.1':0.0139149,('PDT000227561.3':0.0106689,('PDT000227538.3':0.0131856,(('PDT000564141.2':0.0112746,'PDT000135747.3':0.00957106)'':0.000520323,('PDT000083141.3':0.0089905,(('PDT000396569.2':0.00583477,'PDT000139889.3':0.00541913)'':0.000313564,('PDT000142583.3':0.00633187,(('PDT000189678.3':0.00283807,'PDT000124763.3':0.00324975)'':0.00198063,('PDT000298483.3':0.00232536,'PDT000307518.3':0.00196863)'':0.00239714)'':0.000978873)'':0.000394782)'':0.00137402)'':0.00190488)'':0.00346807)'':0.000458482)'':0.000526453)'':-0.00136663)'':0.00129757)'':0.000719978)'':0.00247847)'':0.00474231)'':-0.00375356)'':0.01396)'':-0.0128533)'':0.00476965)'':0.00427488)'':0.0060546)'':-0.00824507,(('PDT000570063.2':0.0354424,((('PDT000189664.3':0.0194666,'PDT000126262.3':0.0206002)'':0.00382188,('PDT000252160.1':0.0267045,('PDT000066853.3':0.0149396,'PDT000580804.2':0.0202472)'':0.00366173)'':0.00219197)'':0.00105821,((('PDT000429547.2':0.00566464,'PDT000137675.3':0.00468666)'':0.0081976,('PDT000384098.2':0.0142613,'PDT000707598.1':0.0204368)'':0.00158067)'':0.00174143,('PDT000430531.2':0.0211542,('PDT000343383.1':0.0180718,('PDT000141136.3':0.0121891,(('PDT000590249.2':0.0156628,('PDT000376052.2':0.0128674,('PDT000107712.3':0.00474698,'PDT000141669.3':0.00711152)'':0.00187753)'':0.00280183)'':0.00100615,(('PDT000152921.3':0.00718437,('PDT000264734.3':0.00789753,'PDT000683638.1':0.00573977)'':0.00105028)'':0.000501931,('PDT000357149.2':0.0100818,('PDT000492173.2':0.00635041,('PDT000161421.3':0.00383563,'PDT000125475.3':0.00403525)'':0.00419485)'':0.00155539)'':0.0015444)'':0.00212331)'':0.00491985)'':0.00365034)'':0.00109798)'':-0.00506931)'':0.00502226)'':0.0123325)'':-0.00184882,(('PDT000158004.3':0.0263638,('PDT000160325.3':0.0285139,'PDT000464149.2':0.0373912)'':0.00824299)'':0.0129562,(((('PDT000625664.2':0.0263575,('PDT000264878.3':0.0225537,'PDT000599120.2':0.0169174)'':0.00682211)'':0.000271743,(('PDT000300522.1':0.0194687,'PDT000102887.3':0.027091)'':0.000449472,(('PDT000252760.3':0.0132188,'PDT000271620.3':0.0136613)'':0.00338661,('PDT000094753.3':0.0175829,('PDT000146066.3':0.00879624,('PDT000145333.3':0.00297099,'PDT000124793.3':0.00441571)'':0.00143091)'':0.0117284)'':0.00188632)'':0.00452677)'':0.00212841)'':0.00509602,((('PDT000311099.3':0.0137282,('PDT000253517.3':0.00791855,'PDT000490379.2':0.00718025)'':0.00509207)'':0.00378054,(('PDT000209841.3':0.0176957,'PDT000708881.1':0.0174215)'':0.00486554,(('PDT000242823.3':0.0128381,'PDT000399373.2':0.0124409)'':0.000706948,(('PDT000212429.3':0.0135588,'PDT000227529.3':0.0213041)'':0.00217018,('PDT000562642.2':0.0100418,('PDT000119778.3':0.00808891,('PDT000268715.3':0.0106286,(('PDT000173940.3':0.0076171,('PDT000227364.3':0.00610963,'PDT000708041.1':0.00674157)'':0.000241398)'':0.000641636,('PDT000710818.1':0.0071596,('PDT000384150.2':0.0043237,('PDT000375766.2':0.0024141,'PDT000136740.3':0.00287479)'':0.00155959)'':0.000849275)'':0.00155653)'':0.00282193)'':0.000485715)'':0.001417)'':0.00149472)'':0.00114597)'':0.00329944)'':-0.000751068)'':0.00637806,(('PDT000351576.2':0.024058,'PDT000556619.2':0.029943)'':0.00415338,('PDT000081125.3':0.0292046,(('PDT000152690.3':0.0142789,'PDT000367768.2':0.0178221)'':0.00183504,('PDT000559166.2':0.0228117,((('PDT000357176.2':0.00694452,'PDT000545079.2':0.00591818)'':0.00396918,('PDT000572644.2':0.0102929,('PDT000247183.3':0.0145039,'PDT000556268.2':0.0118066)'':0.00204048)'':0.00248814)'':0.00220176,('PDT000488246.1':0.0200757,(('PDT000150711.3':0.0134349,'PDT000084170.3':0.0130133)'':0.00106261,(('PDT000359167.2':0.0100722,'PDT000065372.1':0.00998135)'':0.00244932,(('PDT000538181.2':0.00866508,(('PDT000545665.2':0.0110285,'PDT000559329.2':0.00809314)'':0.000326503,('PDT000387731.2':0.0101124,'PDT000216636.3':0.00964731)'':0.000220547)'':0.00231579)'':0.00169682,(('PDT000465584.2':0.00856129,'PDT000559312.2':0.00899301)'':0.00643426,('PDT000189512.3':0.00959569,(('PDT000139228.3':0.00611829,('PDT000149632.3':0.00421068,('PDT000084955.3':0.00249027,'PDT000125479.3':0.00290326)'':0.00286213)'':0.00110676)'':0.00137119,('PDT000115046.3':0.00684707,('PDT000269458.3':0.00689751,('PDT000285891.3':0.00619042,('PDT000604944.2':0.0044022,('PDT000148905.3':0.00556684,('PDT000098148.3':0.00342074,('PDT000582455.2':0.00331152,('PDT000134077.3':0.00241557,('PDT000147496.3':0.0017664,'PDT000432452.2':0.00202715)'':0.000728081)'':0.000948136)'':0.000300519)'':0.0004145)'':0.000920943)'':0.00119672)'':0.000935187)'':0.000629498)'':-0.000928573)'':0.0020675)'':0.00234316)'':-0.00129056)'':0.00378403)'':0.000441031)'':0.00248567)'':-0.00383551)'':0.00625823)'':-0.00222163)'':0.00435316)'':0.00337722)'':-0.005073)'':0.00260715)'':-0.000831344,((('PDT000480945.2':0.024599,('PDT000400831.2':0.018093,('PDT000306761.3':0.0112551,'PDT000385918.2':0.00983606)'':0.0110134)'':0.00480405)'':0.00225842,(('PDT000296221.3':0.0130244,'PDT000307536.3':0.0144803)'':0.0120006,(('PDT000315847.2':0.0156546,'PDT000545101.2':0.0136758)'':0.00370348,(('PDT000677272.1':0.0181931,'PDT000702597.1':0.0172497)'':0.001147,(('PDT000247168.3':0.00985473,'PDT000431701.2':0.0100779)'':0.0100904,(('PDT000717384.1':0.0114175,('PDT000051841.3':0.00892631,('PDT000212487.3':0.00717701,'PDT000247558.3':0.00704619)'':0.00113344)'':0.00372252)'':0.00242244,((('PDT000145341.3':0.0102661,'PDT000499172.2':0.0120969)'':0.00125628,('PDT000201091.3':0.00949446,('PDT000221981.3':0.00755975,'PDT000227348.3':0.00674585)'':0.00194059)'':0.0019888)'':0.000101132,('PDT000576899.2':0.00883033,(('PDT000201101.3':0.00697357,'PDT000212467.3':0.00995703)'':0.000617365,('PDT000139214.3':0.00989917,('PDT000486342.2':0.00941616,('PDT000505043.2':0.00323226,'PDT000098718.3':0.00424904)'':0.00155184)'':0.00236594)'':0.000105922)'':0.00082754)'':0.000857283)'':0.00141195)'':0.00317404)'':0.00219184)'':0.000961979)'':0.00235237)'':0.00337859)'':-0.00110352,((('PDT000108881.3':0.0202187,('PDT000227556.3':0.0180141,'PDT000085604.3':0.0201974)'':0.00489137)'':0.000801989,(('PDT000338963.2':0.0195839,'PDT000252173.1':0.0245701)'':0.000495719,(('PDT000707606.1':0.0149726,'PDT000124964.3':0.0162898)'':0.00456425,('PDT000203370.3':0.0129836,('PDT000235792.3':0.0123163,('PDT000082654.3':0.0125851,('PDT000209834.3':0.00976508,'PDT000107090.3':0.00916452)'':0.00187038)'':0.0012667)'':0.00144957)'':0.00252586)'':0.00217018)'':0.00134294)'':-0.0004025,('PDT000317689.2':0.0250809,((('PDT000375963.2':0.0236111,(('PDT000208160.3':0.0168856,'PDT000586496.2':0.0142189)'':0.00164017,('PDT000083277.3':0.0185072,('PDT000026312.1':0.0138303,('PDT000195699.3':0.0109628,'PDT000307541.3':0.0119802)'':0.00364587)'':0.00253225)'':0.00146514)'':0.00424404)'':-0.00304591,('PDT000115020.3':0.0258947,(('PDT000252171.1':0.0159,('PDT000065357.1':0.0187216,('PDT000225438.3':0.00915817,('PDT000303455.3':0.00600706,'PDT000362570.2':0.00515274)'':0.00107823)'':0.00798478)'':0.00102262)'':5.19327e-05,(('PDT000051250.3':0.0114046,'PDT000443412.2':0.00753675)'':0.00552704,('PDT000721063.1':0.0131112,('PDT000430539.2':0.0148144,('PDT000381319.2':0.00974672,('PDT000147967.3':0.00735098,('PDT000338827.2':0.00597185,('PDT000352819.2':0.00543962,'PDT000120844.3':0.00742308)'':0.0021612)'':0.000954743)'':0.00109591)'':0.00222536)'':0.00211943)'':-0.000463076)'':0.000384773)'':0.00649548)'':-0.00333228)'':0.000184207,('PDT000421969.2':0.0227992,(('PDT000443341.2':0.0129932,(('PDT000184466.3':0.00951684,'PDT000359207.2':0.0105209)'':0.000785652,('PDT000545664.2':0.00734745,'PDT000679124.1':0.00766695)'':0.00556)'':0.00230529)'':0.000463158,((('PDT000197876.3':0.00872582,'PDT000247552.3':0.00728448)'':0.00729656,('PDT000237111.3':0.0080571,'PDT000137341.3':0.0073397)'':0.00420094)'':0.00128181,('PDT000492174.2':0.021631,(('PDT000190655.3':0.0165729,'PDT000597508.2':0.0140212)'':0.00239357,(('PDT000112509.3':0.0117649,('PDT000150154.3':0.0070934,'PDT000559247.2':0.0094109)'':0.0023526)'':0.00556289,('PDT000429538.2':0.0117094,('PDT000107827.3':0.0119201,((('PDT000086983.1':0.00793156,('PDT000682544.1':0.00770485,'PDT000313454.1':0.00729165)'':0.000916386)'':0.000588478,('PDT000399893.2':0.00867069,(('PDT000396591.2':0.00676827,'PDT000072724.3':0.00676403)'':0.000338639,('PDT000124940.3':0.00668712,'PDT000135793.3':0.00594468)'':0.000696986)'':0.00325857)'':-0.000860667)'':0.000533829,(('PDT000050478.3':0.0106393,('PDT000147494.3':0.00763412,'PDT000191663.3':0.00745418)'':0.00231855)'':0.00138714,(('PDT000599115.2':0.00552663,'PDT000115036.3':0.00522817)'':0.00171245,('PDT000225525.3':0.00814336,(('PDT000157503.3':0.00430537,'PDT000136761.3':0.00397063)'':0.00157527,('PDT000313232.3':0.00284866,'PDT000050474.3':0.00293572)'':0.00209332)'':0.00248665)'':-0.000310171)'':0.00354003)'':0.000544092)'':0.000787191)'':0.00123035)'':-0.000195089)'':0.00157055)'':0.00317332)'':-0.0039811)'':0.000131601)'':0.00821736)'':-0.00611955)'':0.00546592)'':-0.00270251)'':0.00226939)'':-0.000390254)'':0.0140744)'':-0.0118625)'':-0.000137338)'':0.00165269)'':0.00434006)'':-0.00102301)'':0.00345833)'':-0.00200737)'':0.0120058)'':-0.00503448)'':0.0461433)'':0.0189768)'':0.0127596)'':-0.0143313,(((('PDT000133953.1':0.0868883,('PDT000430776.1':0.0969077,('PDT000248530.1':0.080009,'PDT000248735.1':0.06313)'':0.00739633)'':0.00435124)'':0.0234665,(('PDT000251766.1':0.0544844,('PDT000344822.2':0.0114334,'PDT000432460.2':0.012103)'':0.0504669)'':0.0229285,(('PDT000539971.1':0.0624803,(('PDT000359090.1':0.0201829,('PDT000502467.1':0.0109105,'PDT000249923.1':0.012764)'':0.00205825)'':0.0151022,('PDT000050607.1':0.0346311,('PDT000619710.1':0.0129668,'PDT000720390.1':0.0167097)'':0.0136463)'':-0.000229013)'':0.0134086)'':0.0100944,((('PDT000502384.1':0.0351085,('PDT000277169.3':0.0178881,'PDT000360442.2':0.0257576)'':0.00941827)'':-0.00214929,(('PDT000533171.2':0.0180188,('PDT000297758.3':0.0171603,'PDT000329307.2':0.0244565)'':0.00328294)'':0.00343104,('PDT000144053.3':0.0360953,('PDT000273379.3':0.0268221,('PDT000400027.2':0.0161904,'PDT000581138.2':0.0150041)'':0.00385456)'':0.00321757)'':0.00379073)'':0.011989)'':0.0173482,(('PDT000355780.1':0.0486301,('PDT000672198.1':0.0173431,('PDT000620737.1':0.00843478,'PDT000624686.1':0.00667302)'':0.00948767)'':0.0124011)'':-0.00042655,('PDT000502900.1':0.0596736,(('PDT000502289.1':0.0379936,('PDT000338393.1':0.0232279,('PDT000396198.1':0.0247073,('PDT000396182.1':0.0122055,'PDT000568555.1':0.014048)'':0.00777858)'':0.00363323)'':0.00261198)'':0.00203475,('PDT000251888.1':0.0363112,('PDT000502574.1':0.0475753,('PDT000502387.1':0.0317658,(('PDT000502348.1':0.00694164,'PDT000502394.1':0.00777886)'':0.0159736,('PDT000244763.3':0.0213856,(('PDT000502297.1':0.00772645,'PDT000251619.1':0.00727905)'':0.00273229,(('PDT000581013.1':0.0111274,'PDT000502902.1':0.0085467)'':0.000769405,('PDT000502271.1':0.00396412,'PDT000502286.1':0.00672198)'':0.00358314)'':0.000659446)'':0.00559132)'':-0.000319629)'':0.00735809)'':0.00589548)'':0.00315962)'':-0.00787165)'':0.0100616)'':-0.00532859)'':0.0135343)'':0.0142563)'':0.0262082)'':0.00923634)'':-0.00667357,((('PDT000319687.1':0.0240146,'PDT000539888.1':0.0212476)'':0.0161023,('PDT000111161.1':0.0195586,('PDT000111160.1':0.025423,('PDT000040395.3':0.014998,'PDT000111146.1':0.0117256)'':0.0124896)'':0.00688111)'':0.000931773)'':0.0625013,(('PDT000249865.1':0.0563238,('PDT000009372.2':0.0448353,'PDT000228186.1':0.0300826)'':0.0409768)'':0.0308241,(('PDT000502375.1':0.0600555,('PDT000324642.1':0.0655348,('PDT000359335.1':0.0399501,('PDT000380282.1':0.0320088,'PDT000009365.2':0.0192328)'':0.00914879)'':0.00600512)'':0.0165049)'':0.000292793,((('PDT000553224.1':0.0469962,('PDT000009362.2':0.030589,'PDT000009366.2':0.0269817)'':0.0060667)'':0.00306676,('PDT000111108.1':0.0568003,(('PDT000111150.1':0.00652637,('PDT000111131.1':0.00228598,'PDT000111149.1':0.00320359)'':0.00793914)'':0.0121218,('PDT000111127.1':0.0342042,('PDT000111137.1':0.0157819,('PDT000384094.1':0.00914228,('PDT000111123.1':0.00353834,'PDT000111162.1':0.00304061)'':0.0140017)'':0.00150496)'':0.00605909)'':-0.000973003)'':0.0250768)'':0.00017739)'':0.00963254,((('PDT000111147.1':0.0154408,('PDT000111134.1':0.017314,'PDT000111152.1':0.012683)'':0.0046257)'':0.0318914,(('PDT000502656.1':0.0398411,'PDT000111142.1':0.0381641)'':0.00379118,('PDT000111125.1':0.0256122,('PDT000248854.1':0.0253788,('PDT000111132.1':0.0232938,'PDT000111145.1':0.0227588)'':0.00188181)'':0.00333352)'':0.0234711)'':0.00333604)'':0.0132158,(('PDT000009364.2':0.0434367,('PDT000009363.2':0.0375819,(('PDT000357675.1':0.0106714,'PDT000357730.1':0.0143806)'':0.010895,('PDT000067878.2':0.0189793,'PDT000067879.2':0.0297069)'':0.00739912)'':0.0169455)'':0.00419503)'':-0.00270218,(('PDT000502446.1':0.0158284,'PDT000720254.1':0.028228)'':0.0259799,(('PDT000156946.2':0.0323065,'PDT000502706.1':0.0364853)'':0.00752511,(('PDT000645179.1':0.0335488,((('PDT000285772.2':0.0147532,'PDT000677504.1':0.011638)'':0.0050067,('PDT000492126.1':0.0198354,'PDT000571485.1':0.0148855)'':0.00130165)'':0.0126499,('PDT000392512.1':0.0287082,('PDT000444927.1':0.0226096,('PDT000264496.1':0.0134118,'PDT000264512.1':0.0160579)'':0.00584102)'':0.00531198)'':0.0015348)'':0.0109392)'':-0.001962,(('PDT000619709.1':0.0537821,('PDT000468561.1':0.0317053,('PDT000468455.1':0.022259,('PDT000468470.1':0.0109269,'PDT000468548.1':0.0115417)'':0.0120415)'':0.0149388)'':0.0104408)'':-0.00201088,(('PDT000641392.1':0.0274105,('PDT000553223.1':0.0185741,('PDT000669702.1':0.0131746,'PDT000067877.2':0.0191833)'':0.00363038)'':0.010163)'':-0.000551265,(('PDT000190398.2':0.0417182,('PDT000468488.1':0.0276077,('PDT000468588.1':0.0218318,('PDT000468505.1':0.0133617,'PDT000468536.1':0.0143114)'':0.00691847)'':0.00698)'':0.00435648)'':-0.000198915,(('PDT000146591.1':0.0199219,'PDT000067874.2':0.0200702)'':0.0213186,('PDT000491522.1':0.0319847,('PDT000016389.2':0.0277051,(('PDT000016356.2':0.0137628,'PDT000264297.1':0.0133203)'':0.00180308,('PDT000111154.1':0.0132125,'PDT000286435.1':0.0154493)'':0.00177459)'':0.00716446)'':0.00346314)'':0.00344907)'':-0.00065904)'':0.00418377)'':0.00271048)'':-0.000855946)'':0.00373167)'':0.00177903)'':0.00762275)'':0.00113971)'':0.0120665)'':0.00502885)'':0.028736)'':-0.00840271)'':0.0124521)'':0.00501976,(((('PDT000084133.3':0.0304687,('PDT000188059.3':0.00787979,'PDT000327972.2':0.0103959)'':0.0170861)'':0.0617113,('PDT000619766.1':0.0622847,('PDT000387014.2':0.0198041,('PDT000317780.2':0.022988,('PDT000505406.2':0.0113982,'PDT000714705.1':0.01811)'':0.0037669)'':0.00448434)'':0.03512)'':0.0290638)'':0.00819592,('PDT000047727.4':0.106571,(('PDT000661086.1':0.0645295,(('PDT000463643.1':0.0176748,'PDT000463679.1':0.0159146)'':0.0518115,(('PDT000244630.3':0.0343466,'PDT000294710.2':0.0459255)'':0.0231549,('PDT000718736.1':0.0505988,('PDT000298037.1':0.061483,'PDT000298058.1':0.062245)'':0.00461373)'':0.00759301)'':0.00154662)'':0.00860499)'':0.00955985,('PDT000248822.1':0.0916952,(('PDT000088267.2':0.0470862,('PDT000602781.1':0.0423646,'PDT000242983.1':0.0338259)'':0.026595)'':0.00503977,('PDT000242863.1':0.0560642,('PDT000646045.1':0.0327904,('PDT000309242.2':0.045298,'PDT000251453.1':0.0326711)'':0.00364913)'':0.0113463)'':0.010398)'':0.0232151)'':-0.0156677)'':0.0381348)'':-0.0081562)'':0.00664696,((('PDT000498977.1':0.0650222,(('PDT000452718.1':0.0358533,('PDT000286402.3':0.0264733,('PDT000051382.3':0.0245862,('PDT000156861.3':0.0162324,('PDT000127703.3':0.0106519,'PDT000051384.3':0.00881194)'':0.00146427)'':0.0119519)'':0.0023929)'':0.0107377)'':-0.00778943,('PDT000553227.1':0.0405048,(('PDT000538185.2':0.0299338,'PDT000548913.2':0.0271869)'':0.00827887,(('PDT000131006.3':0.0228804,('PDT000329318.2':0.00840871,'PDT000135739.3':0.0112304)'':0.00511861)'':0.0121062,(('PDT000403997.2':0.0155813,'PDT000313461.1':0.0153425)'':0.00195779,(('PDT000576031.2':0.0147402,(('PDT000250286.3':0.00974477,'PDT000501619.2':0.00783223)'':0.00500332,('PDT000181333.3':0.0108695,('PDT000107757.3':0.0118226,('PDT000388643.2':0.00702743,('PDT000501475.2':0.00589984,'PDT000072775.3':0.00554066)'':0.00174282)'':0.00472734)'':0.000543612)'':0.000246512)'':0.00343796)'':-0.00125189,(('PDT000300569.1':0.0101243,'PDT000313464.1':0.0114406)'':0.00722836,((('PDT000410890.2':0.00482516,'PDT000098149.3':0.0046304)'':0.00344991,('PDT000104583.3':0.0101523,'PDT000125040.3':0.00761575)'':0.00100496)'':0.00247577,('PDT000579993.2':0.0183003,('PDT000115034.3':0.0116429,('PDT000137574.3':0.0077032,('PDT000240662.3':0.00593511,'PDT000098753.3':0.00570089)'':0.0018143)'':0.000496383)'':0.00478563)'':0.00192695)'':0.00153766)'':0.0015322)'':0.00283979)'':0.00258539)'':0.0150936)'':0.00468163)'':-0.0156115)'':0.0301948)'':-0.024254,((('PDT000259204.1':0.0225489,'PDT000259218.1':0.0184104)'':0.0210044,('PDT000366338.1':0.0189639,('PDT000252737.1':0.00948922,'PDT000259215.1':0.00943288)'':0.00685176)'':0.00559145)'':0.00123804,(('PDT000259212.1':0.0434394,(('PDT000242606.2':0.0210561,'PDT000325476.2':0.0299421)'':0.00535538,('PDT000523888.1':0.0191306,('PDT000525776.2':0.0221942,('PDT000228034.1':0.0193087,('PDT000125476.2':0.016954,('PDT000619707.1':0.0110239,('PDT000259197.1':0.000947063,'PDT000286431.1':0.000751917)'':0.00887846)'':0.0039495)'':0.00161962)'':0.00200486)'':0.00143889)'':0.00204117)'':0.00742259)'':-0.00535798,('PDT000205373.2':0.0362497,(('PDT000502440.1':0.0204228,('PDT000225463.3':0.0181817,('PDT000111153.1':0.0133533,('PDT000111128.1':0.00498465,'PDT000271765.1':0.00808925)'':0.00359346)'':0.00518283)'':0.00190359)'':0.000953544,('PDT000168361.3':0.0279784,('PDT000066899.5':0.0283952,('PDT000323969.2':0.0180207,(('PDT000176927.3':0.0115545,'PDT000533042.2':0.013198)'':0.00205349,('PDT000084974.3':0.025968,('PDT000538832.2':0.0186064,('PDT000225444.3':0.0156736,('PDT000077014.3':0.00912593,(('PDT000300568.1':0.0119282,'PDT000480722.1':0.00613395)'':0.0021909,(('PDT000372107.2':0.00714668,('PDT000289020.3':0.00678401,('PDT000168318.3':0.00680188,('PDT000072757.3':0.00334592,('PDT000189672.3':0.00241046,'PDT000107755.3':0.00278175)'':0.000838619)'':0.00163723)'':0.00212568)'':0.000319931)'':-0.000354572,('PDT000077017.3':0.00771909,('PDT000469988.2':0.0109034,(('PDT000227571.3':0.00810094,'PDT000426872.2':0.00555806)'':0.000709265,(('PDT000212525.3':0.00332972,'PDT000267881.3':0.00374552)'':0.0013434,('PDT000213951.3':0.00368607,('PDT000289814.3':0.00358208,'PDT000319692.2':0.00380389)'':0.000546394)'':0.00130106)'':0.000844285)'':0.00145554)'':0.00127705)'':-0.000617749)'':0.000384874)'':0.00268786)'':0.00309478)'':0.00180994)'':0.000619639)'':-0.00158365)'':0.00289579)'':0.00232532)'':0.00195834)'':-0.00262692)'':0.00607022)'':-0.00263299)'':0.00400235)'':-0.000478422)'':0.0358655,((('PDT000016349.2':0.0691004,('PDT000155364.2':0.0349602,'PDT000278344.3':0.0332504)'':0.0339608)'':0.00948434,('PDT000189648.2':0.0651636,(('PDT000323635.1':0.0429126,('PDT000669704.1':0.0414778,'PDT000711025.1':0.0347901)'':0.00935534)'':0.0132307,(('PDT000481315.2':0.0659024,('PDT000625506.1':0.0396851,('PDT000083436.1':0.0404973,'PDT000124705.1':0.0349212)'':0.00573063)'':0.006453)'':0.000369005,(('PDT000156722.2':0.0340495,'PDT000060853.3':0.0368121)'':0.0117097,(('PDT000131935.2':0.0278508,'PDT000625489.1':0.0243325)'':0.00698517,('PDT000625494.1':0.0341908,'PDT000317186.2':0.037776)'':0.0050511)'':0.00348198)'':0.00401768)'':0.0106207)'':0.0155397)'':0.00197302)'':0.00869893,((((('PDT000243729.3':0.0135593,'PDT000063240.3':0.0179196)'':0.0365645,('PDT000455185.2':0.0551532,('PDT000069921.3':0.0531307,('PDT000063263.3':0.0395685,('PDT000123473.3':0.0341738,('PDT000163030.4':0.022675,'PDT000107753.3':0.0137699)'':0.0107681)'':0.0138818)'':0.00729199)'':0.00235356)'':0.0070495)'':0.00813687,(('PDT000497952.1':0.0318746,('PDT000583951.2':0.0222591,('PDT000532402.2':0.0218982,('PDT000563030.2':0.029301,('PDT000146495.3':0.0169649,('PDT000238430.3':0.00310355,('PDT000340817.2':0.00262751,'PDT000573747.2':0.002364)'':0.000285598)'':0.00796356)'':0.00559455)'':0.000744205)'':0.00398318)'':0.00621444)'':0.000851101,(('PDT000378669.2':0.036585,('PDT000295956.3':0.0355998,'PDT000452772.2':0.0372204)'':0.00382817)'':0.00237016,('PDT000597900.2':0.0337554,('PDT000123337.1':0.0458421,(('PDT000063239.3':0.0179744,('PDT000336946.2':0.0175025,('PDT000407575.2':0.00783625,('PDT000359142.2':0.00284548,'PDT000392379.2':0.00274557)'':0.00446508)'':0.00314746)'':0.00368154)'':0.000528028,('PDT000719105.1':0.0249934,('PDT000407549.2':0.0140529,(('PDT000253487.3':0.00455921,'PDT000491420.2':0.00440315)'':0.00381707,('PDT000051222.3':0.00656601,'PDT000082153.3':0.00981269)'':0.00119664)'':0.00325058)'':0.00834658)'':-0.0030719)'':0.0104793)'':0.0018274)'':0.000940715)'':-0.000415448)'':0.0122179)'':-0.00108599,(((('PDT000583699.2':0.0290704,('PDT000387729.2':0.026224,'PDT000483980.2':0.0285437)'':0.00413975)'':0.00344737,((('PDT000365729.2':0.0226622,'PDT000133749.3':0.0224498)'':0.00351782,('PDT000389682.2':0.0110114,('PDT000112487.3':0.00702827,'PDT000300548.1':0.00731293)'':0.00358976)'':0.00797593)'':0.000628707,(('PDT000146457.3':0.0150352,('PDT000146547.3':0.00945784,'PDT000518074.2':0.00848426)'':0.00344998)'':0.00239092,('PDT000687893.1':0.0183594,(('PDT000268759.3':0.00682867,'PDT000414394.2':0.010009)'':0.00198695,('PDT000208189.3':0.012503,('PDT000352700.2':0.00509437,'PDT000359134.2':0.00447692)'':0.00374253)'':0.00269744)'':0.00422303)'':4.80072e-05)'':0.00363356)'':0.00657431)'':-9.82732e-06,(('PDT000063261.3':0.0455027,('PDT000060077.3':0.0479369,('PDT000063265.3':0.0167028,('PDT000197045.3':0.00671243,'PDT000124945.3':0.00829897)'':0.0112124)'':0.012036)'':0.0116021)'':-0.000159174,('PDT000307877.3':0.0373569,(('PDT000325469.2':0.0277941,('PDT000660097.2':0.0219436,('PDT000338820.2':0.0175723,'PDT000494262.2':0.0160337)'':0.0057897)'':0.0070164)'':-0.00122081,(('PDT000268754.3':0.025288,('PDT000386160.2':0.00656776,('PDT000144489.3':0.00732189,'PDT000480929.2':0.00543351)'':0.00178049)'':0.0116816)'':0.0014621,('PDT000492101.2':0.0307939,(('PDT000496472.2':0.0105272,'PDT000404079.2':0.0151291)'':0.00156364,('PDT000145408.3':0.00973251,'PDT000587134.2':0.00988119)'':0.00661146)'':0.00688456)'':-0.00224557)'':0.00271127)'':0.00386738)'':0.00554149)'':-0.00200565)'':0.0120154,((('PDT000025725.1':0.063651,(('PDT000386162.2':0.0328722,(('PDT000319659.2':0.0291376,'PDT000719941.1':0.0276573)'':0.00311272,('PDT000275725.3':0.0245235,(('PDT000285792.3':0.012703,'PDT000127386.3':0.0127936)'':0.00214917,(('PDT000201090.3':0.0150654,'PDT000366167.2':0.0167698)'':0.00307116,('PDT000306765.3':0.0146541,('PDT000198181.3':0.0110036,('PDT000273490.3':0.0102434,'PDT000491913.2':0.0116638)'':0.00147904)'':0.00471562)'':-8.47831e-05)'':0.001087)'':0.00687677)'':5.33982e-05)'':0.0066802)'':-0.00411205,(('PDT000214042.3':0.0158527,'PDT000032313.1':0.0190218)'':0.00159413,('PDT000235041.3':0.0277188,(('PDT000309807.3':0.0125551,('PDT000287848.3':0.0149479,'PDT000260757.3':0.0130276)'':0.000984293)'':0.000229327,(('PDT000598817.1':0.0114024,('PDT000313394.3':0.00457604,'PDT000496508.2':0.00579076)'':0.00310162)'':0.00178242,('PDT000084134.3':0.0145383,('PDT000491552.2':0.0103467,('PDT000176666.3':0.00682769,('PDT000333322.2':0.00829992,'PDT000352838.2':0.0100697)'':0.00107891)'':0.00206387)'':0.00189867)'':-0.00101767)'':0.00561896)'':0.0059659)'':-0.000880958)'':0.00127543)'':0.0350221)'':-0.0301642,((('PDT000040409.6':0.0404247,('PDT000222749.3':0.0391967,('PDT000611695.2':0.0275518,('PDT000355554.2':0.0279794,('PDT000146542.3':0.0214684,'PDT000494936.2':0.0214265)'':0.00370372)'':0.00307394)'':0.00853177)'':0.000465513)'':0.000189977,(('PDT000063281.3':0.0160707,('PDT000264782.3':0.018563,'PDT000323362.2':0.0139387)'':0.00230034)'':0.00177222,('PDT000563577.2':0.0122069,('PDT000491381.2':0.0173283,('PDT000351793.2':0.0213442,('PDT000040420.4':0.013577,('PDT000356323.2':0.0056397,('PDT000524421.2':0.00417246,('PDT000426866.2':0.00380925,('PDT000302783.3':0.00194962,'PDT000708500.1':0.00274334)'':0.00147262)'':0.000892357)'':0.00116077)'':0.00368872)'':0.00247772)'':0.00191322)'':0.00091674)'':0.0047126)'':0.00591407)'':0.0012502,((('PDT000101019.4':0.0213688,'PDT000285848.3':0.0170548)'':0.0020266,(('PDT000102259.3':0.0129228,('PDT000149635.3':0.00928794,'PDT000085594.3':0.00852656)'':0.00247084)'':0.00190226,('PDT000050562.2':0.0247324,(('PDT000365896.2':0.0156755,('PDT000521266.2':0.0142506,('PDT000213222.3':0.00734028,'PDT000662018.1':0.00799222)'':0.00215668)'':0.00244004)'':-0.000714009,('PDT000259807.3':0.0113262,('PDT000040418.4':0.0126078,('PDT000414396.2':0.00801656,('PDT000356314.2':0.00506632,'PDT000386168.2':0.00449639)'':0.00141983)'':0.00211352)'':0.00380515)'':-0.000507055)'':0.00620472)'':-0.00286662)'':0.00228997)'':0.000914588,('PDT000049881.3':0.0243424,(('PDT000216611.3':0.0249733,('PDT000683625.1':0.0178105,(('PDT000315872.2':0.011518,'PDT000712538.1':0.0115467)'':0.00184504,('PDT000533046.2':0.0107803,('PDT000085610.3':0.00809711,('PDT000485336.2':0.00650821,'PDT000671880.1':0.00782159)'':0.00131864)'':0.00349063)'':0.00134776)'':0.00589864)'':0.00115842)'':-0.00245827,('PDT000430558.2':0.0279666,(('PDT000049085.3':0.00576282,'PDT000086215.3':0.00597848)'':0.00623515,('PDT000624856.2':0.0215539,('PDT000351578.2':0.0141184,(('PDT000227329.3':0.00872403,'PDT000102842.3':0.00795087)'':0.000224447,('PDT000302789.3':0.00864939,'PDT000574309.2':0.00647951)'':0.00117005)'':0.00448581)'':0.00215642)'':-0.000932835)'':0.00613529)'':-0.00104381)'':0.00445093)'':-0.000120338)'':0.00831456)'':0.00139179)'':0.00326505,(((((('PDT000389802.2':0.0330391,('PDT000270118.3':0.025244,('PDT000311127.3':0.0108188,'PDT000063238.3':0.0112604)'':0.00950937)'':0.00633552)'':-0.00148302,('PDT000366977.2':0.0223487,('PDT000082107.3':0.0136358,('PDT000266781.3':0.00240428,'PDT000312079.3':0.00198694)'':0.0103109)'':0.00566806)'':0.0115144)'':0.00588626,(('PDT000295868.3':0.052264,('PDT000127748.3':0.033278,('PDT000198984.3':0.0251232,('PDT000480425.2':0.0190821,'PDT000669402.1':0.0163859)'':0.00159262)'':0.00849968)'':0.00920449)'':-0.000686093,(('PDT000234150.3':0.0233469,'PDT000293088.3':0.0158454)'':0.00168451,('PDT000491417.2':0.0349589,('PDT000285764.3':0.0192852,'PDT000065225.3':0.0165383)'':0.00723756)'':0.001271)'':0.00875982)'':0.0100652)'':0.00139108,(((('PDT000114529.3':0.0344782,'PDT000557766.2':0.0300695)'':0.00047815,('PDT000132455.3':0.0315498,('PDT000542632.2':0.0264873,'PDT000102875.3':0.0247797)'':0.00111717)'':0.00674258)'':0.00249402,(('PDT000338842.2':0.0317488,'PDT000072445.3':0.0254191)'':0.00125239,(('PDT000576249.2':0.0207261,'PDT000025379.2':0.027712)'':0.00245646,(('PDT000311069.3':0.016994,('PDT000208177.3':0.0143294,'PDT000135748.3':0.0102665)'':0.00486388)'':0.00361495,('PDT000208176.3':0.0238669,('PDT000040423.4':0.0173033,('PDT000146544.3':0.0159169,'PDT000191655.3':0.0148888)'':0.00094607)'':0.00537982)'':0.00256688)'':0.0027891)'':0.0052417)'':0.0020943)'':0.00249652,(('PDT000344840.2':0.0345106,('PDT000144052.3':0.0247327,'PDT000540219.2':0.0223851)'':0.00892268)'':0.000741142,('PDT000025744.1':0.0392994,(('PDT000225434.3':0.0411275,('PDT000351564.2':0.0307438,('PDT000144414.3':0.026676,'PDT000386794.2':0.0302767)'':0.00595283)'':0.00351639)'':-0.00199355,(('PDT000083190.3':0.0151047,('PDT000643930.2':0.0106119,'PDT000689219.1':0.0104136)'':0.00671431)'':0.013414,('PDT000497815.1':0.0349103,('PDT000485992.2':0.0239942,(('PDT000549946.2':0.0165908,('PDT000545076.2':0.0136314,('PDT000244730.3':0.0127579,'PDT000417251.2':0.0113575)'':0.00284445)'':0.00112203)'':0.00447433,('PDT000498078.1':0.0263424,(('PDT000300536.1':0.0197264,('PDT000360386.2':0.022623,('PDT000071663.3':0.00957477,'PDT000650493.2':0.010298)'':0.00441232)'':0.00510614)'':0.00021789,(('PDT000351846.2':0.019833,('PDT000143054.3':0.0156116,'PDT000065212.3':0.0173933)'':0.00575568)'':-8.74558e-05,(('PDT000235359.3':0.016484,'PDT000604202.2':0.0176147)'':0.000748343,('PDT000065189.3':0.0145691,('PDT000107754.3':0.0142249,('PDT000429533.2':0.0106094,('PDT000197815.3':0.00831491,'PDT000580395.2':0.00805409)'':0.00231634)'':0.00153877)'':0.000878061)'':0.00365016)'':0.0025407)'':0.000598959)'':0.00545597)'':-0.0032482)'':0.00270089)'':0.00929208)'':-0.0076368)'':0.000996497)'':0.00892778)'':-0.00687751)'':0.00189488)'':0.00182369)'':0.000364161,(('PDT000521619.2':0.0480495,('PDT000426867.2':0.0506842,('PDT000139188.3':0.0409824,('PDT000412106.2':0.0325551,'PDT000065222.3':0.0303037)'':0.00621908)'':0.00375901)'':0.00576514)'':0.0093548,(('PDT000497551.2':0.0436414,('PDT000367246.2':0.0318844,(('PDT000494990.2':0.0195382,('PDT000163576.3':0.0174524,'PDT000225301.3':0.0116998)'':0.00343652)'':0.00614071,(('PDT000190951.3':0.0135056,'PDT000201103.3':0.00899993)'':0.00843543,(('PDT000171578.3':0.021094,'PDT000227312.3':0.0184212)'':0.00529332,('PDT000188057.3':0.0137496,'PDT000311079.3':0.0119497)'':0.00557551)'':0.00616298)'':0.00280518)'':0.00550218)'':0.00895102)'':-0.00663744,(('PDT000496364.2':0.0455363,('PDT000481289.2':0.025594,(('PDT000381341.2':0.0199894,(('PDT000443281.2':0.0190834,'PDT000414355.2':0.0164866)'':0.00190762,(('PDT000313245.3':0.0136761,'PDT000315890.2':0.014269)'':0.00252327,('PDT000490355.2':0.0152128,'PDT000598063.2':0.0150484)'':0.00184291)'':0.00268011)'':0.00312887)'':-0.00225974,('PDT000113708.3':0.0174387,('PDT000600467.2':0.0134609,(('PDT000537368.2':0.0120671,('PDT000710940.1':0.00882138,('PDT000384187.2':0.00883596,('PDT000148514.3':0.00579089,'PDT000547708.2':0.00445791)'':0.00402524)'':0.00156001)'':0.0011471)'':0.000312069,('PDT000544420.2':0.0117323,('PDT000649116.2':0.0119163,(('PDT000604989.2':0.00924126,'PDT000141638.3':0.00980444)'':0.000450064,('PDT000457373.2':0.00652821,'PDT000628849.2':0.00678259)'':0.00294189)'':0.00145434)'':0.00040668)'':-0.000235693)'':0.0013923)'':0.0025561)'':0.00194178)'':0.00376621)'':0.0058352)'':-0.00519943,(('PDT000360354.2':0.0250614,((('PDT000480382.2':0.00858596,'PDT000520282.2':0.00897534)'':0.00298913,(('PDT000551670.2':0.00753911,'PDT000084157.3':0.00706739)'':0.00642594,('PDT000167955.3':0.0069855,('PDT000238313.3':0.00522953,'PDT000283233.3':0.00664077)'':0.0015003)'':0.00340233)'':0.00186746)'':0.00147826,('PDT000302792.3':0.0224469,('PDT000189697.3':0.0146647,('PDT000269469.3':0.01201,('PDT000222073.3':0.0127725,('PDT000491435.2':0.0087088,('PDT000198410.4':0.00849997,'PDT000212469.3':0.00786903)'':0.000480549)'':0.0025071)'':0.00205763)'':0.00174343)'':0.00240881)'':-0.00283131)'':0.00833638)'':-0.00487356,(('PDT000235723.3':0.021996,'PDT000626867.2':0.0200605)'':9.33533e-05,('PDT000289061.3':0.0166575,(('PDT000351898.2':0.0189744,('PDT000573164.2':0.00902169,'PDT000677624.1':0.00813761)'':0.00489786)'':-0.000199163,(('PDT000141137.3':0.0173689,('PDT000412832.2':0.0109439,('PDT000275740.3':0.00488767,'PDT000105067.3':0.004967)'':0.00316654)'':0.00232152)'':0.00172208,('PDT000386799.2':0.0116336,('PDT000266303.3':0.0109964,('PDT000708638.1':0.00956504,('PDT000637264.2':0.00778242,'PDT000640125.2':0.00759918)'':0.00114311)'':0.00112104)'':0.00223091)'':-6.38299e-05)'':0.00150864)'':0.00314997)'':0.000941587)'':-0.00241194)'':0.0025066)'':0.00789797)'':0.0133189)'':-0.007264)'':0.00123187,(((('PDT000603054.2':0.0167298,'PDT000682992.1':0.0124118)'':0.00346137,('PDT000203368.3':0.0177422,('PDT000322440.2':0.00674162,'PDT000499164.2':0.00697198)'':0.00602067)'':0.00633063)'':0.0117683,(('PDT000589265.2':0.0233322,(('PDT000451104.2':0.019117,'PDT000491374.2':0.0181129)'':0.00457388,('PDT000190366.3':0.0105999,('PDT000360397.2':0.012359,'PDT000587791.2':0.0162547)'':0.00300039)'':0.00245024)'':0.00911283)'':-0.00171405,(('PDT000584839.2':0.0217554,('PDT000227501.3':0.0156675,'PDT000687885.1':0.0156164)'':0.00686136)'':0.00493172,(('PDT000575952.2':0.0219544,(('PDT000533051.2':0.013127,('PDT000523852.2':0.01165,'PDT000579963.2':0.0116312)'':0.000702851)'':0.000630494,(('PDT000250875.3':0.0129706,'PDT000351581.2':0.0148457)'':0.000737027,('PDT000491387.2':0.0132134,('PDT000124967.3':0.0120869,('PDT000146467.3':0.0142032,('PDT000491372.2':0.0117686,('PDT000065218.3':0.00983691,(('PDT000390429.2':0.00811067,'PDT000344801.2':0.00816083)'':0.00104348,(('PDT000465828.2':0.00660174,'PDT000677505.1':0.00613336)'':0.00213045,(('PDT000105112.3':0.00984416,'PDT000137594.3':0.00850354)'':0.000608316,(('PDT000662020.1':0.00635698,'PDT000687898.1':0.00666492)'':0.000768893,('PDT000608079.2':0.00698864,('PDT000585184.2':0.00457777,('PDT000190357.3':0.00390112,'PDT000432457.2':0.00498555)'':0.000695727)'':0.00224246)'':-5.5908e-07)'':0.00122395)'':0.000260597)'':0.00107318)'':0.00106331)'':0.000804985)'':0.000344004)'':0.000556402)'':0.00247871)'':-0.00223025)'':0.00130934)'':0.00530663)'':-0.0027061,((('PDT000430532.2':0.0102357,'PDT000571554.2':0.0115013)'':0.0014823,('PDT000486058.2':0.015019,('PDT000049883.3':0.0107088,('PDT000575654.2':0.00651618,'PDT000084240.3':0.00769572)'':0.0031826)'':0.00137373)'':0.000283767)'':0.00560495,('PDT000576856.2':0.0250107,(('PDT000482589.2':0.0151129,('PDT000309607.3':0.0104482,'PDT000525261.2':0.0141184)'':0.00515017)'':0.00402347,(('PDT000409268.2':0.00855011,'PDT000582450.2':0.00803379)'':0.00570817,('PDT000496783.2':0.0185692,((('PDT000270060.3':0.0121041,'PDT000555717.2':0.0111817)'':0.0023843,('PDT000465884.2':0.0147441,'PDT000106265.3':0.0111642)'':0.0012347)'':0.00025173,('PDT000562733.2':0.0128969,(('PDT000235782.3':0.0107279,('PDT000130431.3':0.0134485,('PDT000399324.2':0.00527398,'PDT000690311.1':0.00618132)'':0.00345985)'':0.00353316)'':-6.17048e-06,('PDT000289672.3':0.0198847,(('PDT000465925.2':0.0111137,('PDT000267886.3':0.00500511,'PDT000399319.2':0.00574219)'':0.00318741)'':0.000722994,('PDT000139187.3':0.00991365,(('PDT000242812.3':0.00780581,('PDT000281588.3':0.00455861,'PDT000707578.1':0.00531769)'':0.00152199)'':0.000701084,(('PDT000163625.3':0.00552011,'PDT000305207.3':0.00504189)'':0.00109593,('PDT000571517.2':0.0065354,(('PDT000573452.2':0.00434304,('PDT000242805.3':0.00396957,'PDT000387626.2':0.00350278)'':0.000541646)'':0.000643246,('PDT000510649.2':0.00465376,('PDT000221720.3':0.00380197,'PDT000334469.2':0.00357222)'':0.00151576)'':0.000639012)'':0.00194436)'':-0.000367655)'':0.000142201)'':0.0039921)'':-0.00229365)'':0.00306382)'':-0.000721683)'':0.00147462)'':0.000773549)'':0.00191685)'':0.000465603)'':0.000548133)'':0.00492769)'':-0.00596351)'':-0.00101034)'':0.0051431)'':0.00577176)'':0.00270539)'':-0.0036355,((('PDT000375672.2':0.0399546,'PDT000141403.3':0.0356087)'':0.00928279,('PDT000250116.1':0.0238054,('PDT000248602.1':0.0169703,'PDT000250023.1':0.0180932)'':0.00870836)'':0.0141821)'':0.00624946,((('PDT000691138.1':0.0216956,('PDT000135737.3':0.0227245,('PDT000359132.2':0.0128202,'PDT000637162.2':0.0123707)'':0.00304949)'':0.00407043)'':0.00459455,(('PDT000298482.3':0.0254517,('PDT000688097.1':0.0218624,('PDT000625174.2':0.0157739,'PDT000687904.1':0.00876958)'':0.00880009)'':0.00155131)'':0.00267143,('PDT000234195.3':0.0136807,('PDT000388898.2':0.016125,('PDT000383983.2':0.00949649,('PDT000234164.3':0.00603559,'PDT000365926.2':0.00629411)'':0.00434076)'':0.00323406)'':0.00335034)'':0.00542488)'':0.00108283)'':0.00474398,((('PDT000222020.3':0.0307302,('PDT000238298.3':0.0184399,'PDT000465800.2':0.0248369)'':0.0114945)'':0.00201133,(('PDT000316128.2':0.025548,'PDT000490271.2':0.0236372)'':0.00409935,('PDT000176664.3':0.0408549,('PDT000209851.3':0.0173361,('PDT000190346.3':0.0110326,'PDT000315864.2':0.0150509)'':0.00666708)'':0.0077932)'':-0.000937405)'':0.00165054)'':0.0017286,('PDT000198408.4':0.039264,(('PDT000321940.2':0.0268914,('PDT000414692.2':0.0150074,(('PDT000566481.2':0.00867533,'PDT000086311.3':0.00867377)'':0.004576,(('PDT000235046.3':0.00917954,'PDT000355484.2':0.00741266)'':0.0055368,('PDT000568860.2':0.0085476,'PDT000648185.2':0.0095097)'':0.00690782)'':0.00133423)'':0.00508127)'':0.00407636)'':-0.00273172,(('PDT000389747.2':0.035479,'PDT000587825.2':0.0270271)'':0.0104308,('PDT000147488.3':0.0320704,(('PDT000147481.3':0.0298522,('PDT000394073.2':0.0177236,('PDT000235764.3':0.0141454,'PDT000553505.2':0.0141863)'':0.00400387)'':0.00451006)'':0.00199465,((('PDT000672565.1':0.0141835,('PDT000142611.3':0.0126421,'PDT000130574.3':0.0135204)'':0.00162667)'':0.00379082,((('PDT000066895.3':0.0186073,'PDT000641499.2':0.0139138)'':0.00323801,('PDT000708534.1':0.0130634,(('PDT000584140.2':0.00889401,'PDT000709100.1':0.00596139)'':0.00281076,('PDT000259775.3':0.00718612,('PDT000390266.2':0.00739528,'PDT000097092.3':0.00650882)'':0.00185018)'':0.00119107)'':0.00327258)'':0.0024729)'':-0.000533204,('PDT000313059.3':0.0183826,(('PDT000465679.2':0.00856421,('PDT000407586.2':0.00413859,'PDT000105110.3':0.00473125)'':0.00434072)'':0.00568268,('PDT000270108.3':0.0123248,(('PDT000163287.4':0.0104635,'PDT000137557.3':0.0110216)'':0.00035011,('PDT000368384.2':0.00683784,'PDT000120003.3':0.00630566)'':0.00387976)'':0.00267941)'':-0.000762397)'':0.00403804)'':-0.00146936)'':0.00165101)'':-5.84987e-05,(('PDT000163047.4':0.0229964,(('PDT000505220.2':0.0279913,(('PDT000417908.2':0.0110732,'PDT000553490.2':0.0111204)'':0.000322098,('PDT000501173.2':0.00917134,'PDT000106266.3':0.00956376)'':0.00136705)'':0.00880894)'':-0.00558185,((('PDT000367000.2':0.0127152,'PDT000587020.2':0.0181319)'':0.0019026,('PDT000520152.2':0.00687011,(('PDT000276414.3':0.00303979,'PDT000431677.2':0.0027469)'':0.004728,(('PDT000285790.3':0.00416729,'PDT000047671.3':0.00419924)'':0.00066258,('PDT000499254.2':0.0043072,('PDT000280697.3':0.00345481,'PDT000357221.2':0.00352147)'':0.000772898)'':0.000449669)'':0.00165116)'':0.002059)'':0.0042187)'':-0.00235925,(('PDT000487636.2':0.0145143,('PDT000626712.2':0.00783117,('PDT000190485.3':0.00739385,('PDT000197043.3':0.00663384,(('PDT000243687.3':0.00476557,('PDT000682991.1':0.00432654,('PDT000321904.2':0.00304367,'PDT000491890.2':0.00324438)'':0.00114641)'':0.000704118)'':0.000966608,(('PDT000267882.3':0.00555296,'PDT000324587.2':0.00579254)'':0.000837617,('PDT000300360.3':0.00505017,'PDT000065231.3':0.00480156)'':0.000201668)'':0.000766127)'':0.00121369)'':0.000174973)'':0.000927367)'':0.00286276)'':-0.00204849,('PDT000430566.2':0.0178467,(('PDT000302841.3':0.00828441,'PDT000407564.2':0.00838059)'':0.00150809,('PDT000324570.2':0.0106916,('PDT000343554.2':0.00891596,('PDT000679517.1':0.0118337,(('PDT000296211.3':0.00790226,('PDT000669404.1':0.00597088,'PDT000719815.1':0.00556682)'':0.00223429)'':-4.29538e-05,('PDT000583742.2':0.00642675,('PDT000176817.3':0.00532212,('PDT000679118.1':0.00535376,('PDT000267858.3':0.00436053,'PDT000329276.2':0.00400767)'':0.000831101)'':0.000599173)'':0.0014465)'':-0.000151218)'':0.00151876)'':0.000399545)'':0.000415541)'':0.000673734)'':0.00281194)'':-0.00314658)'':0.00134726)'':0.00410568)'':0.0051604)'':-0.00503227,((('PDT000644468.2':0.00894585,'PDT000669456.1':0.011568)'':0.00676101,('PDT000083233.3':0.012373,('PDT000491404.2':0.0147704,('PDT000387576.2':0.0119517,('PDT000201113.3':0.0102297,'PDT000271167.3':0.0108397)'':0.00147028)'':0.00182532)'':0.000558783)'':0.00150034)'':0.0027731,((('PDT000366199.2':0.0145838,('PDT000065234.3':0.00871444,('PDT000142647.3':0.00888896,('PDT000215106.3':0.00560739,'PDT000357152.2':0.00475941)'':0.00330379)'':0.00063631)'':0.00590251)'':-0.00274158,(('PDT000573394.2':0.00609696,'PDT000582883.2':0.00545104)'':0.0013563,('PDT000570993.2':0.00575605,('PDT000259769.3':0.0031662,'PDT000360374.2':0.0036182)'':0.00140403)'':0.0013972)'':0.00266733)'':0.000657179,(('PDT000408801.2':0.0111469,'PDT000498107.1':0.0114073)'':0.00200512,('PDT000469883.2':0.013665,('PDT000558674.2':0.0161553,('PDT000275634.3':0.0177138,('PDT000215123.3':0.0160784,('PDT000387570.2':0.0120601,('PDT000296040.3':0.0110094,(('PDT000525105.2':0.00752658,('PDT000359272.2':0.0065458,'PDT000588382.2':0.0075583)'':0.000243423)'':0.00106365,('PDT000355468.2':0.0111307,(('PDT000579969.2':0.00882388,('PDT000278235.3':0.00471083,('PDT000234221.3':0.00465284,('PDT000383984.2':0.00379045,'PDT000687943.1':0.00438176)'':0.000527396)'':0.001393)'':0.00333523)'':-0.00115056,((('PDT000300529.1':0.00817532,'PDT000300590.1':0.00588388)'':0.000623175,('PDT000566527.2':0.00713086,('PDT000508997.2':0.00347374,('PDT000150925.3':0.00312628,('PDT000145354.3':0.00231207,'PDT000107003.3':0.00268094)'':0.000563208)'':0.000691337)'':0.00239413)'':0.00137772)'':-0.000294876,('PDT000362746.2':0.00700067,('PDT000315844.2':0.00791619,('PDT000137562.3':0.00515072,('PDT000315829.2':0.00516674,('PDT000247179.3':0.0029531,('PDT000569439.2':0.00328713,('PDT000121851.3':0.00302762,'PDT000137586.3':0.00355198)'':0.00020341)'':0.00087383)'':0.000629157)'':0.00122682)'':0.000896411)'':0.00136485)'':-0.000870046)'':0.000703246)'':0.00192371)'':-0.000637945)'':0.00176181)'':0.000872657)'':0.000206652)'':0.000256195)'':0.000806355)'':0.00074833)'':-0.000165946)'':-0.00261983)'':0.00300169)'':-0.00109469)'':0.001446)'':0.00348249)'':0.00505021)'':0.00230172)'':-0.0047462)'':0.00828168)'':-0.00610591)'':0.000850176)'':0.0185327)'':-0.0216351)'':0.00673395)'':0.00211462)'':0.00629747)'':0.00182483)'':0.00819505,((((('PDT000009667.2':0.0603715,('PDT000252169.1':0.0141003,('PDT000497818.1':0.0090096,('PDT000497772.1':0.00655187,('PDT000497820.1':0.00513369,('PDT000497771.1':0.00546267,'PDT000497775.1':0.00509723)'':0.000736358)'':0.0012211)'':0.00311519)'':0.0015953)'':0.0411839)'':0.00428385,(('PDT000009622.2':0.0190986,'PDT000010364.2':0.0265852)'':0.0101773,('PDT000009623.2':0.0500347,(('PDT000009668.2':0.024302,('PDT000557530.1':0.0207311,'PDT000009359.2':0.0191301)'':0.0121658)'':0.000792634,('PDT000252180.1':0.026757,('PDT000251648.1':0.0189686,'PDT000252170.1':0.0162246)'':0.014437)'':0.00386385)'':0.0101983)'':-3.99602e-05)'':0.000241417)'':0.0313723,((('PDT000502577.1':0.0541113,('PDT000325445.2':0.0405431,'PDT000072729.3':0.0345952)'':0.00882604)'':0.0117572,(('PDT000359195.2':0.0427675,('PDT000105108.3':0.0371956,('PDT000280706.3':0.0244199,('PDT000242776.3':0.0228135,'PDT000083261.3':0.0191625)'':0.00531199)'':0.00350153)'':0.00885208)'':-0.00357224,(('PDT000096856.3':0.0273507,('PDT000360409.2':0.0164376,('PDT000111646.6':0.0116341,('PDT000394076.2':0.0126414,'PDT000139887.3':0.011517)'':0.00197154)'':0.00334852)'':0.00729995)'':-0.00330703,('PDT000081065.1':0.0404243,(('PDT000235269.3':0.019111,'PDT000134062.3':0.0166808)'':0.0111393,(('PDT000404061.2':0.0234921,'PDT000722081.1':0.0151951)'':0.00281055,(('PDT000490348.2':0.0233459,'PDT000574068.2':0.0231521)'':5.72223e-05,((('PDT000040693.3':0.00585012,'PDT000707583.1':0.00568528)'':0.00671633,('PDT000286162.3':0.0121771,(('PDT000218536.3':0.0103291,'PDT000191169.1':0.0116925)'':0.000376423,('PDT000387711.2':0.00692778,'PDT000137666.3':0.00660722)'':0.00381545)'':0.00242936)'':0.000230217)'':0.00169752,(('PDT000619216.2':0.0182716,('PDT000598806.1':0.0191255,('PDT000221938.3':0.0094701,'PDT000315868.2':0.0100091)'':0.00292682)'':0.00529384)'':-0.000533289,(('PDT000102247.3':0.0188499,(('PDT000279560.3':0.0100799,'PDT000362205.2':0.00936449)'':0.00120973,('PDT000547035.2':0.0125008,('PDT000289121.3':0.0166848,('PDT000669317.1':0.00767674,('PDT000325436.2':0.00392309,('PDT000127405.3':0.00341695,'PDT000137663.3':0.00306943)'':0.000756755)'':0.0022299)'':0.00456068)'':0.000790648)'':-0.00106981)'':0.00320915)'':-0.00162691,('PDT000628839.2':0.0189883,('PDT000296013.3':0.0174044,(('PDT000173604.3':0.0053386,'PDT000084210.3':0.00461165)'':0.00726443,('PDT000408603.2':0.00748815,('PDT000352684.2':0.0154994,('PDT000494629.2':0.00893383,('PDT000313040.3':0.00760296,('PDT000077050.3':0.00566726,(('PDT000097090.3':0.00457007,('PDT000259782.3':0.00287405,'PDT000620945.2':0.00360974)'':0.00160034)'':0.00025086,('PDT000609740.2':0.00927942,('PDT000227358.3':0.00467053,('PDT000083983.3':0.0035997,('PDT000124882.3':0.00287093,('PDT000195766.3':0.00162761,'PDT000351827.2':0.00186645)'':0.00092143)'':0.00137148)'':0.000808622)'':0.00110998)'':-0.00033526)'':0.0020028)'':0.000685821)'':0.00136038)'':0.000904923)'':0.000379763)'':-0.000490292)'':0.00366212)'':0.00174591)'':-0.00202426)'':0.00187744)'':-0.000394056)'':0.00762084)'':0.000513344)'':0.000398949)'':0.00729553)'':-0.00409515)'':0.00532196)'':0.00664049)'':-0.00954537,('PDT000619692.1':0.0366241,('PDT000162950.3':0.0281382,(((('PDT000273060.3':0.0115323,'PDT000400844.2':0.00931551)'':0.00808061,('PDT000251371.1':0.0233516,('PDT000051835.3':0.0134908,'PDT000051840.3':0.0147282)'':0.0114492)'':0.00228184)'':0.00865092,((('PDT000519147.2':0.00718577,'PDT000533172.2':0.00731993)'':0.0104826,('PDT000102918.3':0.011093,('PDT000264792.3':0.00649865,'PDT000630159.2':0.00694625)'':0.00193733)'':0.00551152)'':0.000495064,((('PDT000362582.2':0.0137596,('PDT000388891.2':0.00401472,('PDT000189720.3':0.00334405,'PDT000125039.3':0.00353463)'':0.000273452)'':0.00452516)'':0.00137692,('PDT000407561.2':0.0105182,('PDT000253201.3':0.0123328,('PDT000227337.3':0.00937799,('PDT000354135.2':0.00627973,'PDT000482573.2':0.00695407)'':0.000797358)'':0.00507814)'':0.00066619)'':0.000734557)'':0.00450693,(('PDT000497868.1':0.0112152,(('PDT000321146.2':0.00599029,'PDT000549944.2':0.00594171)'':0.00365635,('PDT000120834.3':0.00700421,'PDT000051180.1':0.00789299)'':0.00128645)'':0.00305855)'':0.00399742,(('PDT000321149.2':0.0119709,'PDT000598707.1':0.0111928)'':0.00619358,('PDT000564142.2':0.00749927,(('PDT000120006.3':0.00440952,'PDT000125892.3':0.00436328)'':0.00138126,('PDT000077526.1':0.00745095,('PDT000653392.2':0.00828512,('PDT000144411.3':0.00481477,('PDT000126456.3':0.00473158,('PDT000190538.3':0.00347371,'PDT000491937.2':0.00321092)'':0.00183408)'':0.000412139)'':0.00122125)'':0.000754022)'':-0.000617223)'':0.00271437)'':0.00330278)'':0.00295177)'':0.00128748)'':0.00109269)'':0.00549707)'':-0.000890359,(('PDT000539926.2':0.0313782,('PDT000356313.2':0.0168205,'PDT000408611.2':0.0216983)'':0.0164063)'':0.00897194,((('PDT000213924.3':0.0364123,('PDT000265415.3':0.022069,'PDT000381338.2':0.0208091)'':0.0100919)'':0.00711775,(('PDT000549305.2':0.0195951,('PDT000208161.3':0.0160743,('PDT000404020.2':0.00918059,'PDT000719825.1':0.00793041)'':0.00759383)'':0.00356477)'':0.00312892,('PDT000266341.3':0.0347522,(('PDT000640331.2':0.0167662,('PDT000421085.2':0.0176042,('PDT000480448.2':0.0171691,('PDT000281086.3':0.0104801,('PDT000112496.3':0.00933767,(('PDT000281520.3':0.00865123,'PDT000101077.3':0.00725177)'':0.000780898,('PDT000480366.2':0.00728957,('PDT000373172.2':0.00384156,'PDT000375660.2':0.00383311)'':0.00337095)'':0.000579975)'':0.00213638)'':0.00173794)'':0.00132544)'':0.00305943)'':0.00361542)'':-0.00617172,(('PDT000556593.2':0.0141121,('PDT000630383.2':0.0070045,('PDT000209831.3':0.00357895,'PDT000227276.3':0.00410261)'':0.00301687)'':0.00800706)'':0.0035744,(('PDT000417233.2':0.0179798,'PDT000491959.2':0.0136376)'':0.000300554,('PDT000394086.2':0.0103625,('PDT000501767.2':0.00582712,('PDT000627259.2':0.00911981,('PDT000559327.2':0.00415562,('PDT000076955.3':0.00299,('PDT000542731.2':0.0026943,'PDT000629871.2':0.00269331)'':0.000399483)'':0.000532787)'':0.00216315)'':0.00110838)'':0.00134781)'':0.00266736)'':0.00135741)'':0.00129446)'':0.00607723)'':-0.00117108)'':0.00517972)'':-0.00490634,(('PDT000108878.3':0.0247801,('PDT000189530.3':0.0194556,'PDT000077049.3':0.0155523)'':0.00712652)'':-0.000394987,(('PDT000093292.3':0.014996,('PDT000486059.2':0.0184873,'PDT000630377.2':0.0187792)'':0.00124028)'':0.00264691,(('PDT000097328.3':0.0132241,('PDT000559710.2':0.0144084,('PDT000351844.2':0.0106468,'PDT000120000.3':0.00901174)'':0.00243631)'':0.00186253)'':0.00126204,((('PDT000069416.3':0.0113836,('PDT000708504.1':0.00980092,'PDT000141154.3':0.00816608)'':0.0029445)'':0.00207646,(('PDT000375962.2':0.0103815,'PDT000392134.2':0.012096)'':0.00303057,('PDT000344376.2':0.0126769,('PDT000096897.3':0.00658449,('PDT000283223.3':0.00501762,'PDT000359171.2':0.00455271)'':0.00347195)'':0.00477028)'':0.000333402)'':0.00070184)'':0.000233529,('PDT000466064.2':0.0108022,('PDT000580041.2':0.0116008,('PDT000429521.2':0.0132927,('PDT000366997.2':0.0127583,('PDT000499153.2':0.00639468,(('PDT000242788.3':0.00372864,'PDT000127713.3':0.0035517)'':0.000977442,(('PDT000300560.1':0.00310208,'PDT000313450.1':0.00297967)'':0.00230232,('PDT000355526.2':0.00325281,('PDT000227082.3':0.00265141,'PDT000302807.3':0.00293964)'':0.000434922)'':0.000856794)'':0.000587349)'':0.00172333)'':0.00242435)'':0.000680968)'':0.000832753)'':0.000564352)'':0.000527829)'':0.0011789)'':0.00125007)'':0.00205855)'':0.000109724)'':0.0104473)'':-0.00372389)'':0.00554684)'':0.00605653)'':-0.0105797)'':0.0254157)'':-0.0071721,(((('PDT000616310.2':0.0297087,'PDT000718639.1':0.0253462)'':0.0157977,((('PDT000191645.3':0.0120652,'PDT000702531.1':0.0108765)'':0.01024,('PDT000387583.2':0.012362,('PDT000152924.3':0.00679228,'PDT000247172.3':0.00791802)'':0.0094465)'':0.00307592)'':0.0207109,('PDT000072721.3':0.045742,('PDT000146464.3':0.0303488,('PDT000619738.2':0.0213455,('PDT000619706.2':0.020024,('PDT000295813.3':0.0165781,('PDT000146385.3':0.0130876,'PDT000480723.1':0.016695)'':0.0032549)'':0.00233048)'':0.000657964)'':0.00472724)'':0.00669775)'':-0.0024491)'':0.00580122)'':-0.00174714,((('PDT000548216.2':0.0406259,'PDT000548248.2':0.0475613)'':0.0222676,((('PDT000548173.2':0.0153125,'PDT000548262.2':0.019647)'':0.00553881,('PDT000548321.2':0.0145705,('PDT000548382.2':0.0106804,'PDT000286442.1':0.0103347)'':0.00307282)'':0.00524552)'':0.00928815,(('PDT000548510.2':0.024051,('PDT000548186.2':0.0154679,'PDT000548224.2':0.0155314)'':0.00231018)'':0.00546139,(('PDT000548225.2':0.0117493,('PDT000548534.2':0.0079077,'PDT000128675.3':0.0095156)'':0.00339221)'':0.00635648,('PDT000548519.2':0.0311044,((('PDT000548185.2':0.00997585,'PDT000548500.2':0.00850215)'':0.00499894,('PDT000548316.2':0.0125371,'PDT000611694.2':0.012658)'':0.00421693)'':0.00401081,('PDT000009368.2':0.0169874,('PDT000264351.1':0.0162329,(('PDT000548151.2':0.0101144,'PDT000706223.1':0.0118963)'':0.00087131,('PDT000548515.2':0.00828211,('PDT000548227.2':0.00630403,'PDT000548518.2':0.00641967)'':0.00256509)'':0.00131284)'':0.00393721)'':0.00443331)'':-0.0008268)'':0.00703799)'':-0.00474495)'':0.000352445)'':0.00105696)'':0.0221349)'':-0.0146786,(((('PDT000548266.2':0.0239203,('PDT000548219.2':0.0178936,'PDT000712380.1':0.0183967)'':0.00120961)'':0.00238125,('PDT000548511.2':0.0255897,(('PDT000548211.2':0.0215518,'PDT000548218.2':0.0177602)'':0.00188331,('PDT000548517.2':0.00901422,'PDT000072732.4':0.00816038)'':0.0118646)'':0.00125238)'':0.00188523)'':0.0104115,(('PDT000468511.2':0.00948936,('PDT000468492.2':0.0172789,('PDT000468507.2':0.00231674,'PDT000468515.2':0.00217607)'':0.00637933)'':0.0026729)'':0.0150836,('PDT000286460.1':0.0396714,('PDT000137678.3':0.0455989,('PDT000336919.2':0.0335505,('PDT000443401.2':0.0251478,('PDT000356328.2':0.0280771,(('PDT000050594.1':0.0243262,('PDT000377195.2':0.0162776,('PDT000264488.1':0.0185903,('PDT000052127.3':0.0159189,('PDT000270115.3':0.014358,('PDT000271218.3':0.00937207,('PDT000662016.1':0.00797278,'PDT000691233.1':0.00909412)'':0.00327573)'':0.000626872)'':0.00314913)'':0.00223336)'':0.00129354)'':0.0022362)'':-0.00271642,(('PDT000465607.2':0.0152283,'PDT000114539.3':0.0124725)'':0.00271511,('PDT000595081.2':0.0186447,(('PDT000225298.3':0.00854928,('PDT000149223.3':0.00476971,'PDT000388900.2':0.00478633)'':0.00112975)'':0.00729702,(('PDT000133752.3':0.0177617,('PDT000708558.1':0.0150458,('PDT000273516.3':0.00589635,'PDT000084004.3':0.00614465)'':0.00647232)'':0.00236557)'':-0.000241673,('PDT000443266.2':0.0159135,('PDT000599122.2':0.00768804,('PDT000083220.3':0.00354123,('PDT000270117.3':0.00301495,'PDT000465848.2':0.0028721)'':0.00124326)'':0.00203379)'':0.00318369)'':0.00559636)'':0.000795608)'':0.00220804)'':0.000377094)'':0.000503626)'':0.00683559)'':0.000866513)'':0.00188052)'':0.00798507)'':0.00118297)'':-0.0063384)'':-1.84966e-05)'':-5.40852e-05,(('PDT000548459.2':0.0411384,(('PDT000131858.3':0.0199095,'PDT000132248.3':0.0327897)'':0.00832146,(('PDT000468471.2':0.0252931,'PDT000468590.2':0.0238023)'':0.00386008,('PDT000559119.2':0.0351657,('PDT000468454.2':0.0187867,(('PDT000253510.3':0.0132874,'PDT000317564.2':0.0135098)'':0.00398803,('PDT000184422.3':0.0155324,'PDT000539352.2':0.0163982)'':0.00662852)'':0.00539255)'':0.00360836)'':-0.000130168)'':0.00343432)'':0.00763949)'':-0.00191593,(('PDT000492146.2':0.0259273,'PDT000553225.1':0.0267402)'':0.00639055,(((('PDT000468458.2':0.00475825,'PDT000468560.2':0.00609505)'':0.0144682,((('PDT000468451.2':0.00759019,'PDT000548516.2':0.00714641)'':0.0118294,('PDT000468485.2':0.00783694,'PDT000468551.2':0.0119779)'':0.00299955)'':0.000441572,(('PDT000468553.2':0.0117798,('PDT000468508.2':0.0110876,'PDT000248744.1':0.0102964)'':0.00224728)'':0.000185077,('PDT000468453.2':0.00987776,('PDT000468497.2':0.00933409,'PDT000468499.2':0.00970031)'':0.00223304)'':0.0037018)'':0.00422976)'':0.00362801)'':0.00149399,(('PDT000468552.2':0.0125764,(('PDT000443244.2':0.00860016,'PDT000468480.2':0.00749844)'':0.00538587,(('PDT000468467.2':0.00398301,'PDT000468550.2':0.00489037)'':0.00306779,('PDT000468557.2':0.0117254,('PDT000468464.2':0.00380581,'PDT000468569.2':0.0036725)'':0.00548508)'':0.000329594)'':0.00254299)'':0.00399226)'':0.00297511,(('PDT000468572.2':0.0123428,'PDT000468579.2':0.0177346)'':0.00732877,('PDT000468466.2':0.0120583,('PDT000468556.2':0.00997041,(('PDT000468445.2':0.00371466,'PDT000468535.2':0.00396231)'':0.000590029,('PDT000468506.2':0.00499814,('PDT000468489.2':0.00434434,'PDT000468525.2':0.0041277)'':0.000676295)'':0.000213779)'':0.0015859)'':0.00496167)'':0.01093)'':0.00405185)'':0.00118478)'':0.00438064,(('PDT000259225.1':0.0239715,('PDT000132260.3':0.0150568,'PDT000259222.1':0.0238746)'':0.00119785)'':0.0118262,((('PDT000156068.3':0.0207837,'PDT000264472.1':0.026591)'':0.013867,('PDT000227531.3':0.00953879,('PDT000337257.2':0.00735608,('PDT000147582.3':0.00515667,'PDT000212375.3':0.00680743)'':0.00169682)'':0.00231274)'':0.0105213)'':-0.0018488,(('PDT000264419.1':0.0254732,('PDT000707418.1':0.0121925,('PDT000372114.2':0.00468065,('PDT000417215.2':0.00413188,'PDT000131122.3':0.00383942)'':0.00129481)'':0.0043642)'':0.0102396)'':0.00159611,((('PDT000465588.2':0.0168898,(('PDT000221753.3':0.0117558,'PDT000264430.1':0.0148131)'':0.00177249,('PDT000239303.3':0.0138108,'PDT000257303.3':0.0125206)'':0.00480966)'':0.000375988)'':0.00111773,('PDT000134043.3':0.0210236,('PDT000644516.2':0.0188504,('PDT000124933.3':0.0124764,('PDT000490368.2':0.00709437,('PDT000485535.2':0.00930904,('PDT000156840.3':0.00458819,('PDT000289048.3':0.00262533,'PDT000086171.3':0.00326172)'':0.00187684)'':0.00311949)'':0.0016828)'':0.00108412)'':0.00493379)'':0.00141097)'':-0.000820748)'':0.00256531,(('PDT000322966.2':0.0166134,'PDT000714269.1':0.0179509)'':0.003174,(('PDT000468446.2':0.0190091,('PDT000468516.2':0.0110579,('PDT000107673.2':0.013484,('PDT000468457.2':0.0124487,('PDT000468501.2':0.0130979,'PDT000468524.2':0.0169706)'':0.00152279)'':0.000312041)'':0.0017064)'':0.00236225)'':0.00017191,('PDT000077018.4':0.0220464,((('PDT000248538.1':0.00730569,'PDT000264446.1':0.00833491)'':0.00280092,('PDT000264494.1':0.0117291,(('PDT000468476.2':0.00686921,('PDT000548512.2':0.00766906,('PDT000468477.2':0.00358591,'PDT000468513.2':0.00349074)'':0.00344586)'':0.00131178)'':0.000229248,('PDT000303551.3':0.00848647,('PDT000163282.4':0.00934983,('PDT000082156.4':0.00557585,('PDT000146580.3':0.0040567,'PDT000253171.3':0.00411144)'':0.00479933)'':0.000624921)'':0.000549243)'':0.00175347)'':0.00113001)'':0.00302365)'':-0.00135761,(('PDT000264534.1':0.0220968,('PDT000258005.3':0.0110265,('PDT000359200.2':0.00497799,('PDT000267728.3':0.00550902,'PDT000595790.2':0.00406608)'':0.00109239)'':0.0051613)'':0.00361848)'':-0.0029021,('PDT000468472.2':0.013908,('PDT000120924.3':0.0102194,('PDT000488256.1':0.0133644,('PDT000468475.2':0.00703384,('PDT000102821.3':0.00687109,('PDT000468527.2':0.00550479,'PDT000677097.1':0.00455601)'':0.00146601)'':0.00228809)'':0.00172302)'':0.000947676)'':0.000674769)'':-6.89109e-05)'':0.00233615)'':0.00783515)'':-0.00621738)'':0.00230964)'':-0.000423112)'':0.00120485)'':0.00192272)'':0.0036841)'':-0.0024796)'':0.0098433)'':-0.00547259)'':-0.000854387)'':0.003077)'':0.00949391)'':0.00568455,((('PDT000405101.2':0.0270313,('PDT000190678.3':0.0286362,'PDT000484027.2':0.0272922)'':0.00827041)'':0.0225238,(('PDT000492119.2':0.0210445,('PDT000631605.2':0.0203798,('PDT000197537.3':0.0194627,'PDT000693930.1':0.0202191)'':0.00700168)'':0.0050921)'':0.00688757,((('PDT000486035.2':0.0170394,('PDT000574284.2':0.012211,'PDT000114997.3':0.0178844)'':0.00514893)'':0.00303241,('PDT000707502.1':0.0226763,('PDT000235709.3':0.0147619,('PDT000189683.3':0.0164483,('PDT000146557.3':0.0142874,('PDT000150096.3':0.0101553,'PDT000307514.3':0.00966237)'':0.00298035)'':0.00174591)'':0.00256572)'':0.00448226)'':0.00367395)'':0.00607027,(('PDT000150987.3':0.0387666,('PDT000130546.3':0.027419,(('PDT000397513.2':0.0198893,'PDT000105113.3':0.0195017)'':0.00529765,(('PDT000400835.2':0.0107838,'PDT000451184.2':0.0116355)'':0.00230325,('PDT000312540.3':0.0136681,('PDT000163787.3':0.0134306,('PDT000336812.2':0.00958646,'PDT000501440.2':0.0105604)'':0.00486092)'':0.00270484)'':0.00136816)'':0.00266899)'':0.00415424)'':0.00327852)'':-0.00408648,((('PDT000040440.4':0.0250894,'PDT000137552.3':0.0264358)'':0.00572,(('PDT000465612.2':0.0173463,'PDT000673702.1':0.0114357)'':0.0089908,('PDT000113665.3':0.0249489,(('PDT000683080.1':0.0189644,('PDT000198051.3':0.0179527,('PDT000362197.2':0.0118647,'PDT000522493.2':0.0161912)'':0.00514494)'':0.00190845)'':-3.36702e-05,('PDT000096866.3':0.0148219,('PDT000532967.2':0.0160622,('PDT000334200.2':0.0122942,'PDT000604550.2':0.0156792)'':0.000860141)'':0.00142522)'':0.00107314)'':0.00327658)'':-0.00094052)'':0.00567311)'':-0.00316583,(('PDT000383938.2':0.0156202,'PDT000566483.2':0.021058)'':0.00148434,((('PDT000389814.2':0.0119678,('PDT000707497.1':0.0111284,('PDT000672397.1':0.00944534,'PDT000141126.3':0.0108759)'':0.000550544)'':0.00350632)'':0.00138397,('PDT000452714.2':0.018629,('PDT000432454.2':0.0153051,('PDT000227335.3':0.00918129,('PDT000468213.2':0.00460739,('PDT000163034.4':0.00328961,'PDT000492215.2':0.00259685)'':0.00235996)'':0.00211578)'':0.00399841)'':0.000992467)'':-0.000222904)'':0.00134482,('PDT000025345.1':0.0174647,('PDT000498104.1':0.0203613,('PDT000630826.2':0.0129662,('PDT000289042.3':0.0159998,('PDT000573245.2':0.0153763,(('PDT000492200.2':0.00776332,'PDT000492189.2':0.00684758)'':0.00216278,('PDT000430574.2':0.0119507,('PDT000488036.2':0.00445426,'PDT000693927.1':0.00471639)'':0.0031583)'':0.000757939)'':0.00242552)'':0.000721647)'':0.000728814)'':0.00529255)'':0.00338568)'':-0.00471413)'':0.00587627)'':-0.00268636)'':0.00135904)'':0.00369555)'':0.0027266)'':0.0163373)'':-0.00655322,(((('PDT000311417.3':0.0264623,('PDT000032315.1':0.0217397,('PDT000556868.2':0.0123209,'PDT000251845.1':0.0139243)'':0.012838)'':0.00718366)'':0.00413458,('PDT000213216.3':0.01682,('PDT000431709.2':0.0157901,('PDT000040419.4':0.0101963,('PDT000678188.1':0.00843818,'PDT000107028.3':0.00937102)'':0.00210509)'':0.00421278)'':0.00241912)'':0.0197723)'':0.0120416,(('PDT000394070.2':0.0390588,'PDT000619734.2':0.0391234)'':0.0121075,(('PDT000161514.3':0.03111,'PDT000248560.1':0.0408028)'':0.00917827,(('PDT000376009.2':0.030949,'PDT000647878.2':0.030558)'':0.0162914,((('PDT000360363.2':0.0208381,'PDT000494275.2':0.0158282)'':0.0183857,(('PDT000594136.2':0.0271558,'PDT000044790.4':0.0184523)'':0.00682786,(('PDT000414364.2':0.0129017,'PDT000603505.2':0.0124936)'':0.00679431,('PDT000049606.3':0.0238066,('PDT000173939.3':0.0134899,'PDT000492110.2':0.0128546)'':0.0063809)'':0.00034357)'':0.00423728)'':0.00763286)'':-0.00501285,(('PDT000248671.1':0.0471535,('PDT000525266.2':0.0252468,('PDT000336952.2':0.0189222,('PDT000307493.3':0.017981,('PDT000671556.1':0.0147838,('PDT000283220.3':0.012385,'PDT000509120.2':0.0114528)'':0.00272113)'':0.00292145)'':0.00492159)'':0.000854472)'':0.00976953)'':-0.0020924,('PDT000556866.2':0.0286244,(('PDT000040373.4':0.0250431,('PDT000366325.2':0.0165738,('PDT000669117.1':0.0142429,('PDT000257239.3':0.0164591,('PDT000387593.2':0.0150725,('PDT000430554.2':0.0129873,('PDT000147946.3':0.00971768,'PDT000084173.3':0.0104983)'':0.00285928)'':0.00146411)'':0.00267472)'':0.00118351)'':0.0011822)'':0.00302862)'':0.000352307,('PDT000366205.2':0.0287491,('PDT000610900.2':0.0262039,(('PDT000498012.1':0.0214515,('PDT000132459.3':0.0141721,'PDT000025747.2':0.0138801)'':0.00450138)'':0.00142199,(('PDT000152686.3':0.0111685,'PDT000040422.4':0.0148505)'':0.00553717,(('PDT000295855.3':0.0112557,'PDT000555677.2':0.0122004)'':0.00105865,(('PDT000276427.3':0.00978452,'PDT000390440.2':0.00877078)'':0.00244118,('PDT000521255.2':0.0151367,(('PDT000399328.2':0.00942188,'PDT000480405.2':0.00960682)'':0.00102263,('PDT000595783.2':0.00873385,('PDT000065192.3':0.00769409,(('PDT000221607.3':0.00877502,'PDT000518107.2':0.00577408)'':0.000564941,('PDT000227362.3':0.00466268,'PDT000544857.2':0.0047036)'':0.00157634)'':0.00141782)'':0.00155417)'':0.00173297)'':0.00292322)'':-0.00132718)'':0.00141376)'':0.00430547)'':0.000231463)'':0.00481551)'':0.000966388)'':-0.00434533)'':0.00804086)'':-0.00457406)'':0.00102894)'':0.00695113)'':0.00225791)'':0.0024286)'':0.000636003)'':-0.00614792,(('PDT000492111.2':0.0345759,('PDT000260765.3':0.0267294,(('PDT000373157.2':0.0128879,('PDT000548747.2':0.0136289,'PDT000107496.3':0.0145889)'':0.000538272)'':0.00162367,('PDT000283344.3':0.0179175,('PDT000086336.3':0.0124756,('PDT000303487.3':0.00936512,('PDT000227352.3':0.00577109,'PDT000096921.3':0.00647011)'':0.00295463)'':0.00181907)'':0.00169984)'':0.00251564)'':0.00286328)'':0.00203938)'':0.00497617,((('PDT000323302.2':0.0240577,'PDT000095911.2':0.0483048)'':0.00980379,(('PDT000299814.3':0.00867031,'PDT000623897.2':0.00568379)'':0.00788235,('PDT000357168.2':0.014762,('PDT000151480.3':0.00947589,'PDT000311115.3':0.00759441)'':0.00358722)'':0.00961001)'':0.0128335)'':-0.00300662,((('PDT000494825.2':0.00536564,'PDT000551465.2':0.00459948)'':0.0275049,(('PDT000377142.2':0.0267158,'PDT000443342.2':0.0260038)'':0.00581699,('PDT000377146.2':0.0287777,(('PDT000316477.2':0.0183966,'PDT000248629.1':0.0183413)'':0.00250721,((('PDT000244682.3':0.00727554,'PDT000469976.2':0.00743036)'':0.00548564,('PDT000247381.3':0.0109103,('PDT000359234.2':0.0085799,('PDT000623275.2':0.00732272,('PDT000504002.2':0.00746604,('PDT000151493.3':0.00369985,'PDT000205385.3':0.00387364)'':0.00425057)'':0.000508396)'':0.000222236)'':0.00438393)'':0.000385536)'':0.00120083,('PDT000307519.3':0.011973,('PDT000492139.2':0.013872,('PDT000083161.3':0.00911878,('PDT000645877.2':0.0133441,('PDT000465596.2':0.00974302,('PDT000161574.3':0.00689955,('PDT000209187.3':0.00485606,('PDT000144174.3':0.00428497,'PDT000082114.3':0.00438469)'':0.00101097)'':0.00110793)'':0.000477582)'':0.00127381)'':0.000605832)'':0.0023143)'':0.000815538)'':0.00280266)'':0.00265862)'':0.00518446)'':0.00174338)'':0.00124752)'':-0.00660547,(('PDT000459106.2':0.0265804,(('PDT000553451.2':0.00555516,('PDT000307878.3':0.00490361,'PDT000481003.2':0.0035659)'':0.00140529)'':0.00847517,('PDT000548715.2':0.0212224,('PDT000607724.2':0.0113985,'PDT000653372.2':0.0097766)'':0.00789106)'':0.00111138)'':0.00903162)'':0.00157856,(('PDT000619743.2':0.0321982,('PDT000248604.1':0.0265933,'PDT000248657.1':0.0144685)'':0.0107304)'':0.0043698,(('PDT000521249.2':0.0255176,(('PDT000372587.2':0.0171064,('PDT000081067.1':0.0137686,('PDT000617027.2':0.00877452,'PDT000598742.1':0.00951578)'':0.00365114)'':0.0032689)'':6.33648e-05,(('PDT000386149.2':0.0141623,'PDT000492186.2':0.0111742)'':0.0072115,('PDT000221731.3':0.0103276,('PDT000300540.1':0.00700749,('PDT000356316.2':0.00641484,'PDT000480992.2':0.00712826)'':0.00015821)'':0.00380314)'':0.00375042)'':0.00070811)'':0.00889703)'':-0.00721982,(('PDT000533746.2':0.0232062,'PDT000569558.2':0.0201826)'':0.00312616,('PDT000060841.3':0.0349504,((('PDT000443369.2':0.00914927,'PDT000517744.2':0.0110888)'':0.0066878,('PDT000222037.3':0.0108774,(('PDT000214027.3':0.00950494,('PDT000334273.2':0.00621564,'PDT000414694.2':0.00542036)'':0.00200431)'':0.000643433,('PDT000637272.2':0.00764014,('PDT000130562.3':0.00697917,('PDT000384243.2':0.00628968,'PDT000300561.1':0.00629272)'':0.00135128)'':0.000503383)'':3.50602e-05)'':0.00167392)'':0.00142713)'':0.00275585,(('PDT000200973.3':0.016745,'PDT000540233.2':0.0142451)'':0.00128273,(('PDT000375942.2':0.0151884,'PDT000499240.2':0.0182893)'':0.000630403,('PDT000282446.3':0.0190352,(('PDT000549340.2':0.0144632,('PDT000430568.2':0.00703467,('PDT000247216.3':0.00698515,('PDT000351855.2':0.00518329,('PDT000375965.2':0.00425605,('PDT000340812.2':0.00329072,('PDT000147479.3':0.00245005,'PDT000264746.3':0.00273905)'':0.0006982)'':0.000556309)'':0.00157129)'':0.0020704)'':0.000462464)'':0.00330269)'':0.000352382,('PDT000025728.1':0.017165,('PDT000502970.2':0.0191133,('PDT000580240.2':0.017849,(('PDT000163620.3':0.0090219,'PDT000227282.3':0.00818)'':0.00193561,(('PDT000248555.1':0.00898566,('PDT000151637.3':0.0070009,'PDT000551446.2':0.00792)'':0.00104694)'':0.00245873,('PDT000621131.2':0.0119923,('PDT000399329.2':0.0112243,('PDT000549288.2':0.01313,('PDT000267888.3':0.00877044,(('PDT000344818.2':0.00704403,('PDT000598727.1':0.00774166,(('PDT000227300.3':0.00358642,'PDT000231249.3':0.00309754)'':0.000539777,('PDT000496488.2':0.00552089,('PDT000250871.3':0.00340193,('PDT000189704.3':0.00306678,(('PDT000389730.2':0.00210174,'PDT000102784.3':0.00179359)'':0.00048107,('PDT000066941.3':0.00127977,('PDT000392393.2':0.00111026,'PDT000431759.2':0.00128734)'':0.000417507)'':0.00100779)'':0.000519407)'':0.000800633)'':0.000870196)'':-0.00025526)'':0.00129237)'':0.0010037)'':-0.000944141,('PDT000386191.2':0.00769219,(('PDT000225461.3':0.00632792,'PDT000235281.3':0.00492478)'':0.000431386,('PDT000142599.3':0.00555443,('PDT000502711.2':0.00557005,('PDT000563704.2':0.00588128,('PDT000126249.3':0.0066269,(('PDT000259543.3':0.00455684,('PDT000145773.3':0.00410157,('PDT000147498.3':0.00672021,('PDT000212460.3':0.002904,('PDT000151170.3':0.00275948,(('PDT000375689.2':0.00311196,('PDT000215127.3':0.00201809,'PDT000309823.3':0.00187529)'':0.000481109)'':0.000347339,('PDT000562537.2':0.00322972,('PDT000163587.3':0.00223808,('PDT000146512.3':0.00198559,('PDT000146548.3':0.00152587,'PDT000492097.2':0.00226768)'':0.000508239)'':0.000288553)'':0.000430622)'':-3.53833e-05)'':0.000129438)'':0.000459302)'':0.00116684)'':0.000232942)'':0.000797303)'':-0.000966269,(('PDT000491978.2':0.00449498,'PDT000130549.3':0.00426647)'':0.000823773,('PDT000430542.2':0.00570049,(('PDT000235050.3':0.00262101,'PDT000603745.2':0.00276875)'':0.00130684,('PDT000387682.2':0.00498958,('PDT000145316.3':0.00333773,(('PDT000635820.2':0.00203313,('PDT000234182.3':0.00205033,'PDT000623901.2':0.00184266)'':0.000462717)'':0.000169749,('PDT000313035.3':0.00296686,('PDT000404052.2':0.00295296,('PDT000259529.3':0.00264772,('PDT000230821.3':0.00168744,('PDT000267863.3':0.00153209,('PDT000265284.3':0.0011932,'PDT000066868.3':0.0018038)'':0.000364216)'':0.000187403)'':0.000267278)'':0.000606873)'':0.000397207)'':-0.000431634)'':0.00100626)'':0.00102077)'':-0.000533239)'':0.00198012)'':-0.000741368)'':-0.00108355)'':0.0013994)'':6.28815e-05)'':0.000226812)'':0.000475167)'':-0.000198927)'':0.00308824)'':-0.00336445)'':0.00398931)'':0.00215544)'':0.000584655)'':4.89967e-05)'':-0.00397456)'':0.00216783)'':0.0045243)'':0.00203819)'':0.00210684)'':-0.00845299)'':0.0053736)'':0.000614968)'':0.000521535)'':-0.00633803)'':0.0158557)'':-0.00771902)'':-0.00474497)'':0.0101342)'':-0.00472581)'':-0.00288848)'':0.00688739)'':0.00221099)'':-0.00163822)'':0.00651359)'':0.0106941)'':0.00106014)'':0.00394393,((('PDT000251569.1':0.0615442,('PDT000556922.1':0.0316578,'PDT000016387.2':0.0343094)'':0.00752772)'':0.0230278,(('PDT000468491.1':0.020333,('PDT000468486.1':0.00848407,'PDT000468586.1':0.00949013)'':0.0122232)'':0.0245036,('PDT000627375.1':0.0461947,('PDT000521257.1':0.0389787,('PDT000526251.1':0.0231167,('PDT000626583.1':0.0180265,('PDT000444910.1':0.0195114,('PDT000132098.2':0.0111186,('PDT000444926.1':0.0116884,('PDT000444908.1':0.0019667,'PDT000444922.1':0.00272814)'':0.0054535)'':0.00480181)'':0.00604064)'':0.0024487)'':0.00720748)'':0.00611792)'':0.0106573)'':0.00183997)'':0.0274511)'':-0.000496133,((('PDT000157051.2':0.0373812,(('PDT000444901.1':0.0151124,'PDT000357688.1':0.0196788)'':0.00564821,('PDT000110467.3':0.0278576,'PDT000358750.1':0.0201227)'':0.00165942)'':0.00938967)'':0.013036,(('PDT000279489.3':0.0269436,'PDT000404824.2':0.0343108)'':0.00312799,('PDT000063269.3':0.0434801,(('PDT000157685.3':0.0257465,'PDT000063270.3':0.0277603)'':0.00303219,(('PDT000157981.3':0.0225835,'PDT000161555.3':0.0151)'':0.00892987,('PDT000619718.1':0.0345587,(('PDT000161616.3':0.0166312,'PDT000325750.2':0.0209576)'':0.00157625,('PDT000480989.2':0.0137333,('PDT000316531.2':0.00664972,('PDT000115410.3':0.00320737,'PDT000136982.3':0.00367268)'':0.0023854)'':0.00430282)'':0.00723175)'':0.00321145)'':-0.000106489)'':0.00566991)'':0.00320319)'':-0.000591794)'':0.015413)'':0.0127295,(((('PDT000444994.1':0.0490423,(('PDT000468461.1':0.014694,'PDT000468487.1':0.0150579)'':0.0155598,('PDT000548166.2':0.030376,('PDT000468582.1':0.0225098,('PDT000110157.2':0.0179751,('PDT000468503.1':0.0181335,(('PDT000468482.1':0.0102464,'PDT000468490.1':0.0123542)'':0.000853971,('PDT000468547.1':0.00573737,'PDT000468593.1':0.00632403)'':0.00349025)'':0.002409)'':0.00545806)'':0.00119047)'':0.0052429)'':-0.0012795)'':0.018898)'':-0.00203535,(('PDT000377153.2':0.0362152,('PDT000264501.1':0.035091,('PDT000155237.3':0.014426,'PDT000327903.2':0.0206027)'':0.0118011)'':0.00456456)'':0.0166345,((('PDT000669709.1':0.0283626,('PDT000496158.2':0.0113288,'PDT000600091.2':0.0131679)'':0.0148696)'':0.00490129,(('PDT000619713.1':0.0266411,'PDT000669703.1':0.0341504)'':0.00192295,('PDT000619708.2':0.0298612,('PDT000132040.2':0.0171017,('PDT000147981.3':0.016934,('PDT000184441.3':0.00927813,'PDT000389756.2':0.00850097)'':0.00326664)'':0.00914008)'':0.00481002)'':0.00179874)'':0.00273892)'':0.00410813,('PDT000322946.2':0.0414766,((('PDT000132406.3':0.0281068,('PDT000615206.2':0.0165318,'PDT000615214.2':0.0157358)'':0.00769857)'':0.00135824,((('PDT000550019.2':0.0170592,'PDT000664180.1':0.0198466)'':0.00195793,('PDT000343825.2':0.0137049,('PDT000335801.2':0.0161413,('PDT000716355.1':0.00796577,('PDT000520378.2':0.00598978,'PDT000645773.2':0.00594222)'':0.00175993)'':0.00256213)'':0.00200265)'':0.00252515)'':0.000277688,('PDT000110465.3':0.0189076,('PDT000376234.2':0.0229644,(('PDT000403493.2':0.00992174,'PDT000334863.1':0.00916946)'':0.00299943,('PDT000618546.2':0.0136412,('PDT000519363.2':0.00991763,('PDT000440489.2':0.00861069,'PDT000569755.2':0.00799801)'':0.00204607)'':0.00220242)'':0.00255116)'':0.00433509)'':0.00225988)'':-0.0048577)'':0.0113268)'':0.00205488,(('PDT000553121.1':0.0270111,('PDT000553226.1':0.024577,'PDT000392508.1':0.0227018)'':0.00657078)'':0.00284932,((('PDT000110442.3':0.00967771,'PDT000110501.3':0.0121933)'':0.0175159,(('PDT000318881.2':0.0151207,'PDT000521222.2':0.0145989)'':0.0101372,(('PDT000067876.2':0.0134766,('PDT000702598.1':0.0128446,('PDT000615209.2':0.0101798,('PDT000320887.2':0.00838054,'PDT000619461.2':0.00950496)'':0.00219432)'':0.00314732)'':0.000985883)'':0.00231752,((('PDT000324085.2':0.0174737,'PDT000485314.2':0.0159813)'':0.00158112,('PDT000390812.2':0.0157078,'PDT000132170.3':0.0138056)'':0.00360938)'':0.00014839,(('PDT000132240.3':0.0149853,(('PDT000377926.2':0.0107286,'PDT000716369.1':0.0095644)'':0.00350618,('PDT000619549.2':0.0186204,('PDT000376188.2':0.010324,'PDT000264485.1':0.0110776)'':0.00196209)'':0.000563976)'':0.0026859)'':0.000844555,('PDT000208682.1':0.0126856,('PDT000334847.1':0.0174278,('PDT000110448.3':0.0158113,('PDT000669701.1':0.00822522,('PDT000326220.2':0.00801526,'PDT000251678.1':0.00842224)'':0.000752181)'':0.00291353)'':0.00153079)'':0.000805232)'':0.000334531)'':0.001368)'':0.000579536)'':0.00471985)'':0.000821753)'':-0.00217273,(('PDT000264497.1':0.000692644,'PDT000264498.1':0.00100651)'':0.0443729,(('PDT000072726.3':0.0415269,('PDT000110494.3':0.0308113,('PDT000534847.2':0.0319186,('PDT000127745.3':0.0258218,('PDT000586035.2':0.0181148,('PDT000238421.2':0.0115411,'PDT000125522.2':0.0112042)'':0.0069434)'':0.00223285)'':0.00203445)'':0.000676483)'':0.00254784)'':-0.00149083,(('PDT000151515.3':0.017599,(('PDT000148559.3':0.0092862,'PDT000551433.2':0.0079329)'':0.00234633,('PDT000444842.2':0.00735502,'PDT000264299.1':0.00707428)'':0.00358092)'':0.00549332)'':0.00159584,(('PDT000334866.1':0.0323192,('PDT000598737.1':0.0171053,('PDT000257956.3':0.013564,'PDT000490181.2':0.0134336)'':0.00197644)'':0.0098151)'':0.00118943,('PDT000549271.2':0.0336122,('PDT000687717.1':0.0212088,((('PDT000394737.2':0.0138179,'PDT000542707.2':0.0188599)'':0.0041282,('PDT000356320.2':0.0208647,('PDT000359149.2':0.0169974,'PDT000430576.2':0.0172512)'':0.00130138)'':0.00185134)'':0.000924736,(('PDT000501952.1':0.0242464,('PDT000275700.3':0.0158884,'PDT000403962.2':0.014899)'':0.00664903)'':0.0015539,(('PDT000313468.1':0.015363,('PDT000669710.1':0.00874063,'PDT000208734.1':0.00767727)'':0.00427602)'':0.00166189,('PDT000496154.1':0.0129077,('PDT000340497.1':0.0116666,(('PDT000417487.2':0.0116184,('PDT000465608.2':0.00115622,'PDT000487638.2':0.00114194)'':0.0097648)'':0.000877905,(('PDT000081161.3':0.00784827,('PDT000147917.3':0.00726643,'PDT000218532.3':0.00752407)'':0.00167618)'':0.00189823,('PDT000602651.2':0.012446,('PDT000480985.2':0.00894726,(('PDT000272580.3':0.00526662,('PDT000237294.5':0.00388807,'PDT000362571.2':0.00437804)'':0.00109443)'':0.00152813,('PDT000559696.2':0.00823014,(('PDT000617849.2':0.00558004,('PDT000540025.2':0.00391619,'PDT000585188.2':0.00455585)'':0.00127734)'':-9.34883e-05,('PDT000146070.3':0.00497651,(('PDT000221936.3':0.00405128,'PDT000240649.3':0.00361957)'':0.000986632,('PDT000126252.3':0.00406316,('PDT000151416.3':0.00310703,'PDT000481002.2':0.0031735)'':0.00101572)'':0.000955329)'':0.000683521)'':0.000438628)'':0.00167466)'':0.000180944)'':0.00156318)'':0.00309112)'':-0.0032791)'':0.000760672)'':0.00379174)'':0.00131752)'':0.00212911)'':0.00120834)'':-0.00144562)'':0.00464268)'':0.00237635)'':-0.00427194)'':-0.000294239)'':0.00241973)'':0.00529143)'':-0.00652361)'':0.00989526)'':-0.00478609)'':0.00693205)'':-0.00597696)'':0.00596657)'':0.00743563)'':-0.000494579,((('PDT000050259.1':0.11404,(('PDT000110471.3':0.0299663,'PDT000357616.1':0.0297193)'':0.000214965,('PDT000332713.2':0.0171935,('PDT000161892.3':0.0213441,('PDT000378627.2':0.0142999,('PDT000157081.3':0.0103271,'PDT000708524.1':0.00961927)'':0.00349002)'':0.00143559)'':0.00463705)'':0.0093887)'':0.0392695)'':-0.0278406,(('PDT000081143.3':0.0614846,'PDT000619689.2':0.0552154)'':0.0122461,((('PDT000708503.1':0.0385816,('PDT000264468.1':0.0392821,'PDT000286468.1':0.03068)'':0.00684178)'':0.0183841,('PDT000264302.1':0.0413881,('PDT000264489.1':0.0144791,'PDT000286458.1':0.0125077)'':0.0130646)'':0.0134118)'':0.00152503,((('PDT000517643.2':0.0329761,'PDT000264357.1':0.0356913)'':0.00404351,(('PDT000386263.2':0.0230043,('PDT000148904.3':0.0229584,'PDT000672383.1':0.0188805)'':0.00621618)'':-0.000220779,('PDT000264388.1':0.035157,('PDT000107734.3':0.029104,'PDT000259194.1':0.0221629)'':0.00267569)'':0.00130934)'':0.00456801)'':0.0040818,((('PDT000373208.2':0.0267874,('PDT000627120.2':0.0114235,('PDT000344396.2':0.0113406,'PDT000366318.2':0.0100795)'':0.00215908)'':0.0156542)'':0.000134861,('PDT000264508.1':0.0307721,(('PDT000259227.1':0.0283283,('PDT000267479.3':0.0126846,'PDT000540089.2':0.0102913)'':0.0151554)'':0.00161966,('PDT000259207.1':0.0201439,('PDT000480359.2':0.0229539,'PDT000542093.2':0.0187743)'':0.004006)'':0.00264629)'':0.00643929)'':-0.00231635)'':0.00462837,((('PDT000334867.1':0.0349195,('PDT000264452.1':0.0149474,'PDT000300555.1':0.0231962)'':0.0167677)'':0.000173234,('PDT000264380.1':0.0314963,('PDT000259208.1':0.0314714,('PDT000264319.1':0.0179898,('PDT000264404.1':0.0191355,'PDT000264477.1':0.0212636)'':0.00406817)'':0.00349441)'':0.00361561)'':0.00383107)'':0.00370681,(((('PDT000313477.1':0.0178398,('PDT000313480.1':0.0109788,'PDT000313481.1':0.0110778)'':0.00863825)'':0.00234672,('PDT000313479.1':0.0209,('PDT000313473.1':0.00765617,('PDT000120815.3':0.00675365,'PDT000313474.1':0.0107483)'':0.00088963)'':0.00675201)'':0.00141997)'':0.00697482,('PDT000708610.1':0.0207,('PDT000264563.1':0.0226223,('PDT000203381.3':0.0184038,('PDT000151180.3':0.012488,('PDT000387679.2':0.00690082,('PDT000623903.2':0.00307075,'PDT000124785.3':0.00380724)'':0.00237023)'':0.00414257)'':0.0023497)'':0.00480387)'':0.00310419)'':-0.000727804)'':0.00919009,(('PDT000557536.1':0.0246862,'PDT000248706.1':0.0235835)'':0.0113478,(('PDT000040383.3':0.0378172,('PDT000184519.3':0.0192176,('PDT000148407.1':0.016284,'PDT000264424.1':0.0163139)'':0.00241048)'':0.00692581)'':-0.00157403,((('PDT000258000.3':0.0190231,'PDT000412000.2':0.0187702)'':0.00407273,('PDT000522490.2':0.0200485,('PDT000580807.2':0.0092882,'PDT000082078.3':0.0083828)'':0.0108433)'':0.00857263)'':0.00228766,((('PDT000243678.3':0.0257979,'PDT000264387.1':0.0275458)'':0.00522421,('PDT000264313.1':0.0204696,('PDT000044732.4':0.0121603,('PDT000264450.1':0.013929,('PDT000264906.3':0.00861096,'PDT000125012.3':0.00866294)'':0.00447387)'':0.00209763)'':0.0049522)'':0.00694548)'':-0.000799075,('PDT000264303.1':0.0307704,((('PDT000264525.1':0.0276981,('PDT000259812.3':0.0186927,'PDT000096864.3':0.0166583)'':0.00146209)'':0.00255836,('PDT000264324.1':0.0194864,('PDT000264379.1':0.0145082,'PDT000264416.1':0.0165963)'':0.00467569)'':0.00573314)'':0.000593964,(('PDT000324072.2':0.0165795,('PDT000349618.2':0.00973494,'PDT000594137.2':0.01187)'':0.00502878)'':0.00627865,('PDT000264427.1':0.0199634,(('PDT000051839.3':0.0116178,'PDT000712141.1':0.0171635)'':0.00828754,(('PDT000264376.1':0.0108932,'PDT000286450.1':0.0169398)'':0.00985349,((('PDT000131103.3':0.0134391,('PDT000135354.3':0.0166023,('PDT000707482.1':0.00841835,('PDT000494828.2':0.00807755,'PDT000264329.1':0.0115421)'':0.000747754)'':0.00387523)'':0.00508858)'':-0.00245992,('PDT000113673.3':0.0153094,('PDT000128673.3':0.0126192,(('PDT000264463.1':0.00434035,'PDT000286456.1':0.00773545)'':0.0049459,('PDT000580070.2':0.00686054,'PDT000214066.3':0.0100751)'':0.00216965)'':0.0034172)'':0.00175093)'':0.000231706)'':0.000623129,('PDT000168314.3':0.0259441,('PDT000085580.3':0.0160687,('PDT000248740.1':0.0132626,('PDT000150923.3':0.0137434,('PDT000498102.1':0.0124203,('PDT000706094.1':0.0122831,(('PDT000224264.3':0.00791817,'PDT000307859.3':0.00826693)'':0.00258531,('PDT000135825.3':0.0108372,('PDT000142650.3':0.00986657,('PDT000366980.2':0.00633381,('PDT000492246.2':0.00637678,(('PDT000501441.2':0.00528853,('PDT000559697.2':0.00525073,('PDT000143632.3':0.00363098,'PDT000421284.2':0.00364356)'':0.0012236)'':0.00144955)'':-7.91374e-05,('PDT000491431.2':0.00485066,('PDT000049608.3':0.00317479,('PDT000663860.1':0.00283629,('PDT000221991.3':0.00236322,'PDT000334433.2':0.00262531)'':0.00075817)'':0.00055858)'':0.00072506)'':0.00158193)'':0.000589353)'':0.000852452)'':0.000850747)'':0.00198256)'':-0.000570693)'':0.00286902)'':0.000552224)'':0.00101238)'':0.00125375)'':0.000784589)'':0.00135933)'':-0.00468928)'':0.00438093)'':0.000529273)'':0.00531454)'':-0.00532366)'':0.000848151)'':0.00577515)'':-0.00560027)'':0.00319686)'':0.00206411)'':0.004237)'':-0.0043424)'':0.00197692)'':0.00169567)'':0.0017081)'':0.00433765)'':0.01339)'':-0.00745063)'':-0.000652892,((((('PDT000108899.3':0.0320294,('PDT000465824.2':0.0306803,'PDT000549603.2':0.0274264)'':0.00658864)'':0.00346851,('PDT000264479.1':0.0478128,(('PDT000155255.3':0.0213718,('PDT000351913.2':0.0158866,'PDT000264492.1':0.0125242)'':0.00990175)'':0.00570361,('PDT000148002.3':0.0180771,('PDT000669700.1':0.0150468,('PDT000069399.8':0.0101439,'PDT000123327.1':0.0106832)'':0.00359479)'':0.00312721)'':0.00639914)'':0.0156356)'':-0.00809986)'':0.00332334,('PDT000332878.2':0.0408954,(('PDT000316843.2':0.0265316,'PDT000645927.2':0.025152)'':0.00674463,(('PDT000123335.1':0.0206643,('PDT000253188.3':0.0270452,('PDT000417236.2':0.0226786,'PDT000660099.2':0.0212171)'':0.00190935)'':0.00256903)'':0.00263008,('PDT000135816.3':0.0251047,('PDT000377170.2':0.02881,('PDT000148066.3':0.0203638,('PDT000161612.3':0.0186743,('PDT000147710.3':0.00987509,'PDT000060080.3':0.00963561)'':0.00319609)'':0.00240135)'':0.00284974)'':0.000829818)'':0.00212882)'':0.00289527)'':0.00374806)'':-0.000226319)'':0.00813026,((('PDT000150681.3':0.023792,('PDT000376150.2':0.0192512,('PDT000586499.2':0.0135698,('PDT000322804.2':0.016525,'PDT000351568.2':0.0146264)'':0.000440888)'':0.0034717)'':0.00237582)'':0.000235807,(('PDT000049884.3':0.0195051,('PDT000619714.2':0.0246293,'PDT000619822.2':0.017525)'':0.00125279)'':0.00352819,(('PDT000619740.2':0.0243176,'PDT000248625.1':0.0209366)'':0.00202379,(('PDT000108885.3':0.0166979,'PDT000108926.3':0.0139843)'':0.00208222,(('PDT000144419.3':0.0110551,'PDT000707394.1':0.00706515)'':0.00788317,('PDT000647815.2':0.0184308,(('PDT000564136.2':0.00585553,'PDT000571026.2':0.00840167)'':0.00553589,('PDT000617069.2':0.00798472,('PDT000593758.2':0.00634775,'PDT000115461.3':0.00571605)'':0.00124828)'':0.00283978)'':0.00596458)'':-0.000672799)'':0.000694862)'':0.00111854)'':0.000695594)'':0.00114003)'':0.00870257,((('PDT000155240.3':0.0301962,'PDT000637603.2':0.0305042)'':0.00812318,('PDT000318835.2':0.0172847,('PDT000389671.2':0.0156345,'PDT000711031.1':0.0120276)'':0.00519216)'':0.0215351)'':0.00169113,((('PDT000366268.2':0.0258028,'PDT000626869.2':0.0194461)'':0.00852252,('PDT000161604.3':0.029918,('PDT000320907.2':0.0183585,'PDT000708517.1':0.0214628)'':0.0070323)'':0.00781666)'':0.000276941,(('PDT000314175.3':0.0254627,('PDT000619819.2':0.0249163,('PDT000264486.1':0.011634,('PDT000123284.1':0.00691703,'PDT000286459.1':0.00774047)'':0.0021781)'':0.00967344)'':0.00450286)'':0.0112857,(('PDT000376236.2':0.0288133,('PDT000161595.3':0.0255412,('PDT000317239.2':0.0138275,'PDT000491623.2':0.0137054)'':0.00494609)'':0.00408913)'':0.00230917,('PDT000598077.2':0.0372121,(('PDT000604797.2':0.0236033,'PDT000069418.4':0.0221183)'':0.00425384,('PDT000629336.2':0.0278895,(('PDT000398145.2':0.0251288,('PDT000264301.1':0.0114781,'PDT000264484.1':0.0145708)'':0.0119614)'':-0.000182465,('PDT000108907.3':0.0163794,('PDT000383985.2':0.0188064,('PDT000444902.2':0.0125146,'PDT000120015.3':0.0139833)'':0.00356406)'':0.00176031)'':0.00197627)'':0.00406711)'':9.08945e-05)'':0.011938)'':-0.0101639)'':0.000956889)'':0.00135286)'':0.00843601)'':-0.00220506)'':0.00524291)'':0.00238593,((('PDT000038855.1':0.0161208,'PDT000051480.2':0.0296649)'':0.0157045,('PDT000619814.2':0.0237364,('PDT000161554.3':0.0206083,'PDT000251677.1':0.0278958)'':0.00361497)'':0.00776891)'':0.0112151,(((('PDT000619824.2':0.0310979,'PDT000619827.2':0.0282896)'':0.00495429,(('PDT000157182.3':0.0146225,'PDT000351923.2':0.0210317)'':0.020642,('PDT000517244.2':0.033149,('PDT000496151.2':0.0241038,'PDT000225428.2':0.0269846)'':0.00506212)'':0.00148184)'':0.00406658)'':0.0040087,((('PDT000562991.2':0.0416728,('PDT000524024.1':0.017114,'PDT000016351.2':0.0206908)'':0.0166614)'':0.00167277,(('PDT000146539.3':0.0318851,'PDT000491375.2':0.0209848)'':0.00969756,(('PDT000152685.3':0.0225619,'PDT000275607.3':0.0266426)'':0.00514915,('PDT000115481.3':0.0378772,('PDT000669498.1':0.0291159,(('PDT000134079.3':0.0265456,('PDT000160126.3':0.0192854,'PDT000198409.4':0.0216572)'':0.00332577)'':0.0031269,('PDT000264895.3':0.0226618,('PDT000655505.2':0.0127615,('PDT000269982.3':0.0102615,'PDT000399320.2':0.0145122)'':0.00465802)'':0.00342941)'':0.00425641)'':0.0016628)'':0.00379512)'':-0.0014816)'':0.00614851)'':0.0032378)'':-0.0016273,(('PDT000040430.4':0.0384056,(('PDT000431667.2':0.0300191,('PDT000407545.2':0.0206147,('PDT000497748.1':0.0175152,('PDT000619711.1':0.0149337,'PDT000110460.3':0.0181397)'':0.000859503)'':0.00185541)'':0.00204108)'':0.000275612,(('PDT000619828.2':0.0253153,('PDT000286436.1':0.0197152,('PDT000334845.1':0.0156124,'PDT000334856.1':0.0163718)'':0.00608528)'':0.004355)'':0.000715563,('PDT000502266.1':0.0276304,(('PDT000247455.3':0.021183,'PDT000655782.2':0.0221804)'':0.00220212,(('PDT000142638.3':0.0142401,'PDT000491380.2':0.0143849)'':0.00334218,('PDT000555974.2':0.0187348,('PDT000491453.2':0.0158064,('PDT000225520.3':0.00574628,'PDT000307849.3':0.00669402)'':0.003428)'':0.00307345)'':0.00163813)'':0.0017006)'':0.00447698)'':-4.78233e-05)'':0.000378127)'':0.01619)'':-0.01285,((('PDT000532976.2':0.0250931,'PDT000110506.3':0.0253361)'':0.00271522,('PDT000146534.3':0.0252649,(('PDT000190659.3':0.0179195,'PDT000065197.3':0.0170228)'':0.00300294,('PDT000205382.3':0.027062,('PDT000498943.2':0.010538,('PDT000107834.3':0.00592826,'PDT000718650.1':0.00610554)'':0.00228225)'':0.00856416)'':0.00037021)'':0.00319546)'':0.00766817)'':-0.0011387,('PDT000687895.1':0.0439648,(('PDT000188006.3':0.0213285,'PDT000598184.2':0.0205847)'':0.00965349,(('PDT000720834.1':0.0192468,('PDT000485848.2':0.0180658,('PDT000148719.3':0.0145083,'PDT000085602.3':0.0118855)'':0.0016463)'':0.00186432)'':0.00413694,('PDT000065512.4':0.0167962,(('PDT000687888.1':0.0105654,'PDT000124924.3':0.0103278)'':0.006117,('PDT000500813.2':0.0195083,('PDT000139217.3':0.0172687,('PDT000318460.2':0.00900125,'PDT000722021.1':0.00592705)'':0.00417915)'':0.0028497)'':0.00118994)'':0.00152977)'':0.00100824)'':0.00373066)'':0.00747966)'':-0.0056682)'':0.00228676)'':0.00260972)'':0.00886442)'':-0.00349028,(('PDT000602889.1':0.0669742,(('PDT000108876.3':0.0383074,('PDT000404823.2':0.0225677,('PDT000333948.2':0.0223502,'PDT000444923.2':0.0189802)'':0.00123188)'':0.0162656)'':1.4501e-05,(('PDT000259224.1':0.0272115,('PDT000327206.2':0.0218543,('PDT000619729.2':0.0214525,'PDT000646125.2':0.0206238)'':0.00428853)'':0.00609292)'':-0.000182638,(('PDT000161703.3':0.0169092,'PDT000120828.3':0.0222311)'':0.00774554,('PDT000327877.2':0.025076,(('PDT000160233.3':0.0186816,'PDT000598404.2':0.0220747)'':0.00182742,('PDT000721946.1':0.0213932,('PDT000326125.2':0.0136674,'PDT000132354.3':0.0147681)'':0.00748524)'':0.0018956)'':0.00301967)'':-0.00114837)'':0.00382103)'':0.00359035)'':0.0168932)'':-0.0099148,((('PDT000491594.2':0.0389374,('PDT000384305.2':0.0126679,'PDT000635983.2':0.0184176)'':0.0154671)'':0.00870554,(('PDT000132413.3':0.0364921,('PDT000267353.4':0.0275793,'PDT000378656.2':0.0287354)'':0.0115429)'':0.00696084,('PDT000619724.2':0.0346932,('PDT000410946.2':0.0279624,(('PDT000235708.3':0.0216622,'PDT000130792.3':0.0237568)'':0.00666631,(('PDT000145399.3':0.0169033,'PDT000484202.2':0.0170768)'':0.00953945,('PDT000038859.1':0.025082,('PDT000579458.2':0.0140953,('PDT000157983.3':0.0154777,'PDT000115480.3':0.014024)'':0.00149641)'':0.00636472)'':0.00053875)'':0.00260265)'':0.00105122)'':0.00345998)'':0.00397097)'':0.00332715)'':-0.000945624,((('PDT000110486.3':0.0295441,('PDT000523348.2':0.0202839,'PDT000619715.2':0.018344)'':0.0026516)'':0.00238751,(('PDT000375735.2':0.0219815,('PDT000156054.3':0.0188466,'PDT000132133.3':0.0193543)'':0.00375604)'':0.00088338,('PDT000630478.2':0.0227444,('PDT000578364.2':0.016061,('PDT000641419.2':0.0125863,('PDT000338974.2':0.0111381,'PDT000114766.3':0.0106506)'':0.000932249)'':0.00269044)'':0.00341637)'':0.000792444)'':0.00103231)'':0.00724044,((('PDT000313465.1':0.0185794,'PDT000313469.1':0.021125)'':0.0210233,('PDT000108897.2':0.0293791,(('PDT000212331.3':0.0200812,'PDT000669707.1':0.017113)'':0.00641335,('PDT000417807.2':0.00917748,'PDT000620105.2':0.00895752)'':0.0129811)'':0.00656062)'':0.00401864)'':2.89002e-05,((('PDT000352666.2':0.0217892,('PDT000259226.1':0.0154456,'PDT000264304.1':0.0240281)'':0.00593995)'':0.0162921,(('PDT000221618.3':0.0250518,('PDT000135755.3':0.0160118,('PDT000201172.3':0.0151286,'PDT000084037.3':0.020561)'':0.00207491)'':0.00471057)'':0.00740454,((('PDT000548260.2':0.0228158,'PDT000264487.1':0.026838)'':0.00593107,('PDT000548256.2':0.0310338,('PDT000548535.2':0.0167228,(('PDT000548513.2':0.016945,('PDT000548210.2':0.0115171,('PDT000548174.2':0.00763776,'PDT000548208.2':0.00864664)'':0.00385618)'':0.00286859)'':0.000223969,(('PDT000548251.2':0.0101638,('PDT000548318.2':0.00919415,'PDT000548380.2':0.00996215)'':0.00172156)'':0.000786833,(('PDT000548317.2':0.00688188,'PDT000264466.1':0.00785622)'':0.0100827,('PDT000548381.2':0.00563939,('PDT000548314.2':0.00275623,'PDT000548379.2':0.00333038)'':0.00261335)'':0.00552975)'':0.000953567)'':0.00380619)'':0.00291903)'':0.00959265)'':-0.0037391)'':0.00251219,(('PDT000517246.2':0.0303021,'PDT000132373.3':0.0349153)'':0.00327448,(('PDT000416431.2':0.0277449,('PDT000156597.3':0.0255246,'PDT000522450.2':0.0230627)'':0.00461219)'':0.00342702,(('PDT000239344.1':0.0256839,('PDT000702586.1':0.0177262,('PDT000071662.3':0.0163939,'PDT000128444.3':0.0155911)'':0.0014305)'':0.0046653)'':0.00332046,(('PDT000110458.3':0.0228805,('PDT000264482.1':0.0174693,'PDT000498134.1':0.0141977)'':0.00596353)'':0.00578862,('PDT000498039.1':0.0230914,((('PDT000161425.3':0.0144908,'PDT000042359.3':0.0130539)'':0.00427187,('PDT000264483.1':0.016438,('PDT000366218.2':0.014418,(('PDT000147616.3':0.00988737,'PDT000326262.2':0.00924423)'':0.00198404,('PDT000551629.2':0.00844277,'PDT000637185.2':0.00794743)'':0.00271816)'':0.00302828)'':0.00189989)'':-0.000708772)'':0.00224645,('PDT000038869.1':0.0207829,(('PDT000161601.3':0.0153646,('PDT000160267.3':0.0113918,('PDT000222315.1':0.0103204,('PDT000157224.3':0.00678685,'PDT000311064.3':0.00674275)'':0.00213338)'':0.002121)'':0.00329934)'':0.00122321,('PDT000343821.2':0.02323,(('PDT000234215.3':0.013669,'PDT000110461.3':0.0145361)'':0.00181845,(('PDT000332794.2':0.0137238,('PDT000317154.2':0.0132883,('PDT000108909.3':0.0089185,('PDT000319807.2':0.00788297,('PDT000108896.3':0.00673851,'PDT000108927.3':0.00678439)'':0.00125488)'':0.00146223)'':0.00225883)'':0.00186545)'':-0.00255962,('PDT000264491.1':0.0125882,(('PDT000480335.2':0.00570252,'PDT000490185.2':0.00504588)'':0.00177425,('PDT000468263.2':0.0057066,('PDT000386291.2':0.00609534,('PDT000264490.1':0.00738273,('PDT000163566.3':0.00533165,'PDT000242761.3':0.00433741)'':0.000394137)'':0.00113628)'':0.000714164)'':5.56864e-05)'':0.00166716)'':0.00239762)'':0.00294801)'':0.00622288)'':-0.00566895)'':0.00832859)'':-0.00783995)'':0.00991826)'':-0.00558467)'':0.00091647)'':0.00217409)'':0.00472653)'':-0.00292312)'':0.00322918)'':0.00546567)'':-0.00970603,((('PDT000375751.2':0.0140108,'PDT000264481.1':0.0188102)'':0.00748028,('PDT000576245.2':0.0171371,('PDT000108933.3':0.0200774,'PDT000714273.1':0.019793)'':0.00151121)'':0.00466024)'':0.00537576,((('PDT000327125.2':0.0178,('PDT000115407.3':0.0119704,('PDT000318831.2':0.011676,'PDT000647768.2':0.008318)'':0.00242005)'':0.00232462)'':0.00404248,('PDT000384320.2':0.0203388,(('PDT000161994.3':0.0122578,('PDT000328169.2':0.0119399,'PDT000110507.3':0.0120886)'':0.000575736)'':0.00609943,('PDT000377112.2':0.0198582,('PDT000377084.2':0.00907593,('PDT000156043.3':0.0100501,'PDT000353589.2':0.0096338)'':0.00129972)'':0.0067565)'':0.00411289)'':0.000540122)'':-0.00200162)'':0.00424449,((('PDT000157132.3':0.0244422,('PDT000132188.3':0.012173,('PDT000322806.2':0.00989075,'PDT000711017.1':0.0104345)'':0.00383699)'':0.00219466)'':0.00511337,('PDT000157210.3':0.0259332,(('PDT000328094.2':0.0126174,'PDT000598468.2':0.0118042)'':0.00534347,('PDT000377050.2':0.0143984,('PDT000322919.2':0.0123862,('PDT000324115.2':0.0111627,('PDT000327242.2':0.00882007,'PDT000565133.2':0.00914523)'':0.00225356)'':0.000958278)'':0.000425524)'':0.00230432)'':0.00346021)'':0.00132028)'':0.000275322,((('PDT000327954.2':0.0196317,'PDT000619818.2':0.0252312)'':0.00361515,('PDT000322088.2':0.0195424,('PDT000264300.1':0.0166081,('PDT000562143.2':0.00946552,('PDT000444899.2':0.00401822,'PDT000616353.2':0.00425036)'':0.00564564)'':0.00406077)'':0.00249535)'':0.00327646)'':-0.000207199,(('PDT000325171.2':0.0123475,'PDT000603060.2':0.0125657)'':0.00823899,(('PDT000558545.2':0.0216669,('PDT000329312.2':0.0171799,('PDT000240660.3':0.0110521,'PDT000410933.2':0.012668)'':0.000881436)'':0.00553808)'':-0.0013071,(('PDT000108894.3':0.0213943,((('PDT000616513.2':0.0121255,'PDT000132196.3':0.0116438)'':0.00420956,('PDT000114661.3':0.00682767,'PDT000497294.2':0.00751783)'':0.00581804)'':0.000715857,(('PDT000148073.3':0.0110252,('PDT000327963.2':0.0118288,('PDT000257236.3':0.00864278,('PDT000156612.3':0.00718833,'PDT000444838.2':0.00944677)'':0.000917368)'':0.00210679)'':0.00153633)'':-0.000423845,(('PDT000668170.2':0.00688259,'PDT000205742.3':0.00885131)'':0.00103075,('PDT000669712.1':0.0067879,('PDT000149243.3':0.00608503,'PDT000669711.1':0.00528997)'':0.000759496)'':0.00251442)'':0.000981437)'':0.00310983)'':0.0110753)'':-0.00932659,(('PDT000519451.2':0.0151724,('PDT000601096.2':0.0122112,(('PDT000379498.2':0.0102528,'PDT000115406.3':0.0102626)'':0.00157159,('PDT000327174.2':0.0115828,('PDT000378636.2':0.0106017,('PDT000157183.3':0.00617058,'PDT000314178.3':0.00687852)'':0.00212468)'':0.000898792)'':0.000809526)'':0.00136936)'':0.0017258)'':0.000741752,(('PDT000573297.2':0.0119275,'PDT000651397.2':0.0127907)'':0.00223009,('PDT000378015.2':0.0122765,('PDT000277147.5':0.0152306,(('PDT000319266.2':0.0130331,'PDT000324026.2':0.0104448)'':0.000545697,('PDT000160569.3':0.00791386,('PDT000503126.2':0.00864439,('PDT000161608.3':0.00542335,'PDT000061303.3':0.00522345)'':0.00287011)'':0.00118084)'':0.000742533)'':0.00228225)'':0.000921898)'':0.000769808)'':0.00131972)'':9.41679e-05)'':0.00183349)'':0.00337795)'':-0.00204842)'':-0.000207273)'':0.000632196)'':0.00820924)'':-0.0026491)'':0.0049197)'':-0.00129201)'':0.00281322)'':0.000225987)'':0.00302967)'':0.00319185)'':-0.00266948)'':0.00255125)'':0.00315532)'':-0.00135945,((((('PDT000160439.3':0.0189766,'PDT000324707.2':0.0219579)'':0.00629309,('PDT000322972.2':0.0125213,'PDT000388885.2':0.0183122)'':0.00629589)'':0.00203832,('PDT000241717.3':0.0345461,('PDT000038868.1':0.0206182,('PDT000621342.2':0.0108328,('PDT000316113.2':0.00724853,('PDT000155926.3':0.00444901,'PDT000158931.3':0.00510798)'':0.00191877)'':0.00192775)'':0.0123974)'':0.00360238)'':-0.00153071)'':0.00389458,((('PDT000110454.3':0.0232291,(('PDT000038900.1':0.0154022,'PDT000052321.1':0.0136905)'':0.00563622,('PDT000334846.1':0.0228861,'PDT000334852.1':0.017383)'':0.00459073)'':0.00542645)'':0.00301433,('PDT000313462.1':0.0282106,('PDT000132203.3':0.0171438,('PDT000251676.1':0.0168597,('PDT000259221.1':0.00850576,('PDT000251679.1':0.00811592,'PDT000334854.1':0.00896788)'':0.00252969)'':0.00124341)'':0.00545291)'':0.00282912)'':0.000882673)'':0.00268219,(('PDT000325091.2':0.027833,'PDT000641411.2':0.0272836)'':0.0050373,('PDT000148254.3':0.025714,('PDT000702615.1':0.0201326,(('PDT000334850.1':0.00739254,'PDT000334862.1':0.00603366)'':0.0125452,('PDT000334853.1':0.0128104,('PDT000334861.1':0.0148939,('PDT000334840.1':0.00639635,('PDT000052322.1':0.005963,(('PDT000313460.1':0.00484322,'PDT000334841.1':0.00621368)'':0.00332749,('PDT000334836.1':0.00564337,('PDT000067875.2':0.00631357,('PDT000334858.1':0.00278806,'PDT000334860.1':0.00270151)'':0.00176094)'':0.0007963)'':0.00075792)'':0.00187573)'':0.00177938)'':0.000676174)'':0.00266071)'':-0.00221119)'':0.0079958)'':0.00532733)'':-3.81901e-06)'':-0.00120954)'':0.00691494)'':0.0201467,((((('PDT000480451.2':0.0293925,('PDT000430527.2':0.0197305,('PDT000152114.3':0.0181748,'PDT000087346.3':0.015542)'':0.00346412)'':0.00445356)'':-0.000785565,(('PDT000230795.3':0.0276354,(('PDT000499253.2':0.0147598,'PDT000707426.1':0.0163329)'':0.0012268,(('PDT000302849.3':0.0189125,'PDT000081118.3':0.0126101)'':0.0014752,('PDT000469868.2':0.0158767,(('PDT000306539.3':0.00719895,'PDT000084086.3':0.00729955)'':0.000826119,(('PDT000533418.2':0.003845,('PDT000176662.3':0.00263095,'PDT000523243.2':0.00246039)'':0.0015325)'':0.00150671,('PDT000151638.3':0.00546688,('PDT000062527.3':0.00428662,('PDT000480349.2':0.00349517,('PDT000509955.2':0.00127628,'PDT000627223.2':0.00142156)'':0.00138872)'':0.000503751)'':0.00257118)'':-0.000905844)'':0.00154996)'':0.00521099)'':-0.00110416)'':0.00237875)'':0.00732322)'':-0.0044461,('PDT000096831.3':0.0240144,(('PDT000360462.2':0.0202893,(('PDT000271287.3':0.0108858,'PDT000491393.2':0.0171497)'':0.0045098,('PDT000287906.3':0.0152756,'PDT000508995.2':0.0214906)'':0.00387955)'':0.002445)'':-0.000762709,(('PDT000112499.3':0.0262071,(('PDT000201231.3':0.00921596,'PDT000407626.2':0.0113197)'':0.00257021,('PDT000197822.3':0.012335,('PDT000051373.3':0.0115458,('PDT000355519.2':0.00833917,('PDT000233456.3':0.00758832,'PDT000235322.3':0.00694048)'':0.00139563)'':0.00223173)'':0.000811757)'':0.00125034)'':0.00509498)'':-0.00162024,(('PDT000589617.2':0.0123859,('PDT000082085.3':0.00728546,'PDT000083975.3':0.00702584)'':0.00481891)'':0.00267539,('PDT000628607.2':0.0114377,('PDT000137725.3':0.00809864,('PDT000222582.3':0.00814118,('PDT000214028.3':0.00487372,'PDT000390206.2':0.00498389)'':0.00266182)'':0.00197105)'':0.00199899)'':0.0017019)'':0.000427075)'':0.00175791)'':0.00579911)'':-0.00367534)'':0.00721788)'':-0.00379805,((('PDT000600810.2':0.0184247,'PDT000722636.1':0.0157861)'':0.00867039,('PDT000148894.3':0.0186425,('PDT000188002.3':0.0138576,'PDT000539812.2':0.0166616)'':0.00108559)'':0.00328966)'':0.00963452,((('PDT000417211.2':0.0242795,('PDT000133756.3':0.0167428,('PDT000533416.2':0.0168314,('PDT000313034.3':0.0105268,('PDT000221608.3':0.00987088,('PDT000208163.3':0.00524931,'PDT000594852.2':0.00510309)'':0.00156997)'':0.00227778)'':0.00214473)'':0.000760441)'':0.0078306)'':-0.00165761,((('PDT000214049.3':0.00949544,('PDT000224234.3':0.00743126,'PDT000225515.3':0.0105787)'':0.00232476)'':0.00348251,('PDT000281562.3':0.0174722,('PDT000430536.2':0.00995319,('PDT000176660.3':0.0100274,'PDT000562730.2':0.00689808)'':0.00279326)'':0.00162977)'':-0.000473764)'':0.00191382,(('PDT000235071.3':0.0130247,'PDT000242809.3':0.0125173)'':0.000909532,('PDT000148906.3':0.0107073,(('PDT000225435.3':0.00435022,'PDT000396647.2':0.0046354)'':0.00123854,('PDT000216673.3':0.00472588,('PDT000585376.2':0.00391148,('PDT000389794.2':0.00399945,'PDT000659069.2':0.00397106)'':0.000375555)'':0.00049197)'':0.000191013)'':0.00231526)'':0.00336683)'':-0.000667135)'':0.00367778)'':0.00161927,((('PDT000357225.2':0.0169335,'PDT000429520.2':0.0193568)'':0.00701799,('PDT000451216.2':0.0242826,('PDT000198191.3':0.013995,'PDT000404042.2':0.0140129)'':0.00317787)'':0.00238701)'':0.00259169,(('PDT000010358.3':0.0387192,('PDT000227269.3':0.0176729,('PDT000386790.2':0.0196512,(('PDT000366331.2':0.0152222,'PDT000137529.3':0.0150241)'':0.00163006,('PDT000679468.1':0.0171763,('PDT000420999.2':0.0135329,('PDT000125027.3':0.0112929,(('PDT000385882.2':0.00773868,'PDT000124808.3':0.00619032)'':0.00222779,(('PDT000545707.2':0.00680854,'PDT000065190.3':0.00710246)'':0.00126634,('PDT000565039.2':0.00579977,'PDT000641713.2':0.00545633)'':0.00189701)'':0.000419327)'':0.00106837)'':0.00337774)'':0.00102274)'':0.000916856)'':0.0015878)'':0.000304666)'':0.0118666)'':-0.0106154,(('PDT000093291.3':0.0196052,'PDT000102780.3':0.0160739)'':0.00602596,('PDT000128466.3':0.0216475,('PDT000498110.1':0.018664,('PDT000189707.3':0.0173364,('PDT000190695.3':0.0189652,(('PDT000523858.2':0.0127205,('PDT000153281.3':0.00838459,('PDT000114524.3':0.00714885,'PDT000128450.3':0.00617525)'':0.00276701)'':0.00285317)'':0.000103818,(('PDT000497549.2':0.0104809,(('PDT000570324.2':0.00695542,('PDT000520281.2':0.00451761,'PDT000583924.2':0.00456151)'':0.00187907)'':0.00145775,('PDT000287912.3':0.0126446,('PDT000124950.3':0.00712129,('PDT000264549.1':0.00699257,('PDT000140525.3':0.00648959,('PDT000431644.2':0.00487517,('PDT000149200.3':0.00419108,('PDT000499178.2':0.00421455,('PDT000616854.2':0.00323638,('PDT000238305.3':0.00244374,'PDT000707359.1':0.00304583)'':0.00104672)'':0.000257742)'':0.00125078)'':0.000704364)'':0.00093498)'':0.000678458)'':0.00110473)'':0.000605287)'':-0.000733536)'':0.00201584)'':-0.00130223,(('PDT000242757.3':0.00992441,('PDT000221951.3':0.00541476,('PDT000270038.3':0.00618262,('PDT000590106.2':0.00434468,('PDT000173960.3':0.00299715,('PDT000334406.2':0.0018549,'PDT000481028.2':0.00154035)'':0.000691856)'':0.00103411)'':0.00204122)'':0.000365017)'':0.00250693)'':-0.000169921,(('PDT000101043.3':0.00739526,('PDT000637244.2':0.00691202,'PDT000139177.3':0.00631128)'':0.00132929)'':0.000235273,('PDT000133830.3':0.00872294,(('PDT000430551.2':0.0066238,'PDT000108011.3':0.006212)'':0.000771478,('PDT000687892.1':0.00349992,'PDT000707470.1':0.00307903)'':0.00308827)'':0.0013313)'':-0.000456296)'':0.000754438)'':0.000905709)'':0.00146769)'':0.00401362)'':0.000625168)'':0.00186235)'':0.00189423)'':-0.000789499)'':-0.00354376)'':0.00412276)'':-0.00273068)'':0.00495273)'':-0.00342994)'':0.00478597,(((('PDT000386795.2':0.0262623,('PDT000367009.2':0.0221631,'PDT000628835.2':0.0154591)'':0.00490567)'':0.000743497,(('PDT000707494.1':0.0155022,('PDT000385881.2':0.0178342,'PDT000420989.2':0.018308)'':0.00333539)'':0.00441695,(('PDT000390442.2':0.012968,'PDT000148629.3':0.0114681)'':0.00203612,('PDT000132569.3':0.0168508,(('PDT000132637.3':0.0156884,('PDT000132629.3':0.00988601,'PDT000264321.1':0.012387)'':0.00407669)'':-0.000116382,('PDT000234166.3':0.0131314,('PDT000190971.3':0.0134921,('PDT000197823.3':0.00804243,'PDT000221950.3':0.00905327)'':0.00470346)'':0.00330569)'':0.00183059)'':0.00179912)'':-0.000925577)'':0.00485753)'':0.000539549)'':0.000307731,(('PDT000497861.1':0.0182948,'PDT000498146.1':0.0219961)'':0.00597509,('PDT000223296.3':0.042503,(('PDT000149242.3':0.0179868,'PDT000576251.2':0.0150129)'':0.00393747,(('PDT000627119.2':0.0199937,(('PDT000130977.5':0.0116988,'PDT000233656.3':0.0111499)'':0.00120653,('PDT000235072.3':0.00668148,'PDT000566789.2':0.00644502)'':0.00377724)'':0.00960846)'':-0.00134547,(('PDT000264520.1':0.0226249,(('PDT000135804.3':0.0203932,('PDT000230074.3':0.0154868,'PDT000072720.3':0.0145735)'':0.00320743)'':8.18972e-05,('PDT000134045.3':0.0173935,('PDT000247523.3':0.0147067,('PDT000176909.3':0.00995066,'PDT000707397.1':0.00792954)'':0.00461974)'':0.0023573)'':0.000583676)'':0.00402614)'':-0.00190947,((('PDT000162301.3':0.0117182,'PDT000509938.2':0.010836)'':0.00430432,('PDT000101112.3':0.0119855,('PDT000046382.3':0.0114145,'PDT000132633.3':0.0179239)'':0.000627292)'':0.00161335)'':0.00390149,(('PDT000258014.3':0.0107325,'PDT000315879.2':0.0108731)'':0.00485936,('PDT000062617.5':0.0151329,('PDT000417237.2':0.0116918,'PDT000492210.2':0.00931288)'':0.000916899)'':0.00340606)'':0.00284108)'':0.00103565)'':0.00149611)'':0.00213792)'':0.00527562)'':-0.0022493)'':-0.00355137)'':0.00663422,((('PDT000173622.3':0.0140533,'PDT000270654.3':0.0199413)'':0.00595981,(('PDT000334368.2':0.0196209,('PDT000215057.3':0.0104107,'PDT000410913.2':0.0135901)'':0.00756502)'':0.00607838,('PDT000533179.2':0.0285883,(('PDT000559309.2':0.0141449,'PDT000668307.2':0.0128821)'':0.00582616,(('PDT000309643.3':0.00914618,'PDT000368360.2':0.00999872)'':0.00187838,('PDT000176788.3':0.00885476,('PDT000359136.2':0.00706595,'PDT000388880.2':0.00985955)'':0.00233049)'':0.00108239)'':0.00331312)'':0.00478976)'':-0.000653203)'':0.00317282)'':0.00475468,((('PDT000112517.3':0.0225874,('PDT000010363.2':0.0172544,('PDT000264428.1':0.0130227,('PDT000008735.2':0.0149203,('PDT000009369.2':0.0140353,('PDT000469990.2':0.00933041,'PDT000009671.2':0.00892899)'':0.0014288)'':0.00165239)'':0.000859567)'':0.00115249)'':0.00379905)'':0.0036833,('PDT000009360.2':0.0625311,((('PDT000114646.3':0.0237365,('PDT000156613.3':0.0216811,('PDT000319663.2':0.0174691,('PDT000208186.3':0.0145845,'PDT000132577.3':0.016232)'':0.00211262)'':0.00317784)'':0.00298118)'':0.00316309,('PDT000009367.2':0.0265647,(('PDT000239619.3':0.0215515,'PDT000325468.2':0.0284583)'':0.0027557,('PDT000040384.3':0.019284,(('PDT000355494.2':0.0137912,('PDT000150710.3':0.0122918,(('PDT000372805.2':0.00844601,'PDT000082128.3':0.0115939)'':8.82632e-05,('PDT000465586.2':0.0053331,'PDT000124922.3':0.00462408)'':0.00279342)'':0.00245226)'':0.00126898)'':-0.000945541,(('PDT000334458.2':0.00930672,('PDT000046467.3':0.00692052,('PDT000060079.3':0.0053083,'PDT000082142.3':0.0054465)'':0.00171178)'':0.00164842)'':0.00127011,(('PDT000257980.3':0.00617377,('PDT000313026.3':0.00345758,'PDT000492164.2':0.00401477)'':0.00248396)'':0.00105332,('PDT000359271.2':0.00698664,('PDT000081135.3':0.00534541,'PDT000125519.3':0.00520399)'':0.00135711)'':0.0010327)'':0.00258357)'':0.000811086)'':0.00744692)'':0.000720404)'':0.0113912)'':-0.0103436)'':-0.000841048,((('PDT000451189.2':0.0158087,'PDT000629887.2':0.0163507)'':0.00758266,(('PDT000049613.3':0.0191292,'PDT000289835.3':0.0118971)'':0.000857644,('PDT000579077.2':0.0113571,('PDT000278623.3':0.00853332,'PDT000480721.1':0.00974968)'':0.00139033)'':0.00271201)'':0.00758427)'':-0.000106561,(('PDT000137700.3':0.0187367,(('PDT000299782.3':0.0111064,'PDT000491396.2':0.00877425)'':0.00180781,('PDT000120021.3':0.00631626,'PDT000125565.3':0.00690044)'':0.00399749)'':0.00937989)'':-0.00385426,(('PDT000587140.2':0.0159878,('PDT000264561.1':0.0176787,('PDT000227314.3':0.00860191,'PDT000562720.2':0.0119038)'':0.00373899)'':0.0032594)'':0.00131839,(('PDT000264315.1':0.0144904,('PDT000264420.1':0.013167,('PDT000151175.3':0.00754623,('PDT000264445.1':0.00743732,('PDT000264363.1':0.00558454,('PDT000135810.3':0.0044356,'PDT000264435.1':0.00523635)'':0.000678386)'':0.00111176)'':0.000748607)'':0.0034755)'':0.00526171)'':-0.0016448,(('PDT000083270.3':0.0129129,('PDT000544568.2':0.0108865,'PDT000721339.1':0.00981265)'':0.00391346)'':0.00165459,(('PDT000373213.2':0.0106151,'PDT000085592.3':0.00915445)'':0.00260248,(('PDT000212432.3':0.0101358,'PDT000679489.1':0.0164426)'':0.00056,((('PDT000410878.2':0.00851831,'PDT000130427.3':0.00680659)'':0.000272678,('PDT000126237.3':0.0111479,('PDT000145345.3':0.00431246,('PDT000338836.2':0.00316226,'PDT000719917.1':0.00402193)'':0.00218033)'':0.00230917)'':0.00124637)'':0.000196282,(('PDT000344337.2':0.00712343,'PDT000084141.3':0.00806867)'':0.00441534,(('PDT000342247.2':0.00673976,'PDT000417259.2':0.00523034)'':0.00114382,(('PDT000150138.3':0.00785051,'PDT000142601.4':0.00572779)'':0.000978867,(('PDT000271183.3':0.00491612,('PDT000176783.3':0.00405272,'PDT000492034.2':0.00431631)'':0.000770242)'':0.00118211,('PDT000212497.3':0.00377616,('PDT000625200.2':0.00401576,('PDT000302803.3':0.00161514,'PDT000655727.2':0.00158134)'':0.00186131)'':0.000689882)'':0.00128879)'':0.00134618)'':0.000858135)'':0.000832407)'':-0.000779456)'':0.00327382)'':0.000714024)'':0.000645474)'':0.000489274)'':0.00187244)'':0.000150213)'':0.00169346)'':-0.000278374)'':0.0351234)'':-0.0299301)'':-0.00239233,(((('PDT000569780.2':0.0223611,'PDT000630469.2':0.019551)'':0.00174691,(('PDT000486344.2':0.0168668,('PDT000151459.3':0.0108457,'PDT000307866.3':0.011558)'':0.00240004)'':0.00442527,('PDT000357279.2':0.0200156,('PDT000366264.2':0.0156358,'PDT000480971.2':0.0181178)'':0.00594381)'':0.00213942)'':0.0035244)'':-0.000329586,(('PDT000525101.2':0.0253646,('PDT000490346.2':0.0115519,'PDT000549251.2':0.0139573)'':0.00771321)'':0.000541205,(('PDT000230055.3':0.0211085,'PDT000588236.2':0.0184246)'':0.00137184,('PDT000594847.2':0.0235103,('PDT000096819.3':0.0223384,('PDT000146069.3':0.0161716,(('PDT000184354.3':0.00893793,('PDT000264441.1':0.00395427,'PDT000264515.1':0.00452876)'':0.00404516)'':0.00455758,(('PDT000491376.2':0.0122764,('PDT000390208.2':0.014026,('PDT000152937.3':0.00770817,('PDT000212447.3':0.00472697,'PDT000311055.3':0.00604393)'':0.00220318)'':0.0018655)'':0.00181263)'':0.00200307,('PDT000594542.2':0.0100486,('PDT000189559.3':0.0145313,('PDT000082082.3':0.0149211,('PDT000189516.3':0.010065,('PDT000707416.1':0.00606592,('PDT000600973.2':0.00458479,('PDT000299784.3':0.00385629,'PDT000719821.1':0.00431511)'':0.000489771)'':0.00205693)'':0.00113563)'':0.00205503)'':0.00088566)'':0.00166704)'':-0.00134997)'':0.00290751)'':0.00235039)'':0.000731345)'':0.00513105)'':-0.00339729)'':0.00022491)'':0.000360897)'':0.00500771,(('PDT000492150.2':0.0378099,(('PDT000271228.3':0.0239748,'PDT000422019.2':0.0246818)'':0.00340236,(('PDT000148493.3':0.0178875,'PDT000682717.1':0.0168368)'':0.0027117,('PDT000351850.2':0.0162907,('PDT000101374.3':0.0133627,('PDT000579076.2':0.0131459,'PDT000264503.1':0.011172)'':0.00303572)'':0.00414229)'':0.00143595)'':0.00393128)'':0.00281127)'':-0.000123762,((('PDT000227361.3':0.0166268,'PDT000524510.2':0.0164408)'':0.00431304,('PDT000617850.2':0.0142046,(('PDT000213259.3':0.00738493,'PDT000264527.1':0.00652327)'':0.00721705,('PDT000459004.2':0.0146674,(('PDT000226926.3':0.00764242,('PDT000161422.3':0.00907837,'PDT000124783.3':0.00737553)'':0.00221983)'':0.000546784,('PDT000492082.2':0.00904502,('PDT000351779.2':0.0103861,'PDT000135751.3':0.0109031)'':0.00185263)'':0.00137611)'':0.00287568)'':0.000593689)'':0.0035803)'':0.00453811)'':0.00259195,(('PDT000189510.3':0.0201062,('PDT000336815.2':0.0213908,'PDT000578986.2':0.0226331)'':0.0078057)'':0.00345793,(('PDT000494833.2':0.0302868,(('PDT000157476.3':0.013757,'PDT000048214.3':0.0175678)'':0.00478313,('PDT000264453.1':0.02145,(('PDT000221745.3':0.0139407,('PDT000707367.1':0.0113444,('PDT000147965.3':0.00510966,'PDT000481290.2':0.00694454)'':0.00646189)'':0.00201419)'':0.00266887,(('PDT000108930.3':0.0150118,'PDT000110463.3':0.0180289)'':0.00186029,('PDT000303467.3':0.021549,('PDT000579988.2':0.0127203,(('PDT000142613.3':0.00822728,'PDT000356321.2':0.00661632)'':0.00499337,('PDT000184455.3':0.0129797,('PDT000072447.3':0.013197,('PDT000270004.3':0.011945,(('PDT000496335.2':0.00814403,'PDT000114642.3':0.00851937)'':0.000852359,('PDT000086179.3':0.00800451,('PDT000158535.3':0.00840796,('PDT000235048.3':0.00921239,('PDT000142582.3':0.00518368,'PDT000264771.3':0.00447575)'':0.00247499)'':0.00113371)'':0.000715071)'':-0.000187303)'':0.00199214)'':0.00185176)'':0.000425291)'':-0.00044153)'':0.00230001)'':0.00291786)'':-0.00158566)'':-0.000395017)'':0.00514352)'':-0.00238163)'':0.00847742)'':-0.00875494,(('PDT000083223.3':0.0187614,('PDT000491451.2':0.0105607,'PDT000098707.3':0.00818124)'':0.00445812)'':0.00391442,(('PDT000242810.3':0.0226531,'PDT000321903.2':0.0185973)'':0.0019504,(('PDT000264451.1':0.0125049,'PDT000264551.1':0.0142066)'':0.00236015,('PDT000355460.2':0.0179676,('PDT000285826.3':0.0138425,(('PDT000112491.3':0.0112593,('PDT000357278.2':0.00962375,'PDT000669263.1':0.0111787)'':0.00130353)'':0.00120978,(('PDT000214030.3':0.00840777,('PDT000491439.2':0.00975174,'PDT000286464.1':0.00753416)'':0.000881234)'':0.00130745,('PDT000498040.1':0.0111761,(('PDT000171520.3':0.0086229,('PDT000221609.3':0.00841659,'PDT000355456.2':0.00772851)'':0.00108815)'':0.000844973,('PDT000145344.3':0.010711,(('PDT000148627.3':0.00735962,'PDT000491377.2':0.0103996)'':0.000864441,('PDT000287854.3':0.006228,('PDT000096892.3':0.00671859,(('PDT000084091.3':0.00586371,'PDT000132596.3':0.00478409)'':0.00123579,('PDT000285775.3':0.00537081,('PDT000227357.3':0.0040447,'PDT000123415.3':0.0066318)'':0.00134152)'':0.00100119)'':0.000285076)'':0.000271541)'':0.00120012)'':0.00182415)'':-0.000553057)'':0.00308876)'':-0.00217742)'':0.0021326)'':0.00120669)'':0.00204638)'':-0.00207686)'':0.00279344)'':0.000282157)'':-4.64565e-05)'':0.00595001)'':-0.00355221)'':0.00323142)'':-0.00138949)'':0.00118056)'':0.00409598)'':-0.00152398)'':-0.000320067)'':0.00151803,(((('PDT000162270.3':0.0301274,'PDT000125473.3':0.0366624)'':0.0315047,('PDT000250861.3':0.0226337,'PDT000082079.3':0.0195504)'':0.0274169)'':0.00445739,(('PDT000146615.1':0.0539496,'PDT000286433.1':0.0479894)'':0.00978378,(('PDT000063177.3':0.0285222,('PDT000025735.1':0.0253885,(('PDT000252193.1':0.0164434,'PDT000252194.1':0.0183144)'':0.00510668,('PDT000248553.1':0.0317835,('PDT000259206.1':0.0296493,('PDT000264368.1':0.0243108,('PDT000248541.1':0.0115178,'PDT000259211.1':0.0135193)'':0.0129464)'':0.000865868)'':0.00290308)'':-0.00028452)'':0.00613269)'':0.00110479)'':0.00337531,((('PDT000502743.2':0.0186731,('PDT000708571.1':0.0198178,('PDT000285763.3':0.00377346,'PDT000127544.3':0.00380305)'':0.00899188)'':0.00402557)'':0.00448254,('PDT000009626.2':0.0330687,(('PDT000305229.3':0.0212531,('PDT000149192.3':0.016762,'PDT000721965.1':0.0168862)'':0.00229191)'':-0.000737861,('PDT000259196.1':0.0104509,('PDT000504039.2':0.00622494,('PDT000520230.2':0.0046418,('PDT000306786.3':0.00530692,'PDT000125993.3':0.00415617)'':0.00033346)'':0.00127766)'':0.00335149)'':0.00469555)'':0.0112855)'':-0.00283379)'':-0.00122178,(('PDT000394055.2':0.0153288,'PDT000136767.3':0.016273)'':0.00949325,(('PDT000481013.2':0.0150355,('PDT000227272.3':0.00527149,'PDT000532706.2':0.00628571)'':0.0124867)'':0.00244569,('PDT000592145.2':0.014358,('PDT000342251.2':0.0156138,('PDT000403999.2':0.0124336,(('PDT000254253.3':0.0138792,'PDT000360394.2':0.0132625)'':0.000621964,('PDT000381340.2':0.0131264,('PDT000275620.3':0.006961,'PDT000360459.2':0.0073645)'':0.0034725)'':0.00176035)'':0.00137713)'':0.000867873)'':0.00343617)'':0.000182199)'':0.00713828)'':0.00187696)'':0.00708089)'':0.0111603)'':-0.00255342)'':0.00333648,((('PDT000338858.1':0.0391498,'PDT000311108.3':0.0399016)'':0.0213742,((((('PDT000315853.2':0.0294015,('PDT000190365.3':0.0192222,'PDT000336940.2':0.0177057)'':0.00599321)'':0.00212111,('PDT000492083.2':0.026205,('PDT000504871.2':0.0244445,('PDT000086296.3':0.016558,('PDT000184513.3':0.0102748,'PDT000549199.2':0.0117687)'':0.00356137)'':0.00656856)'':0.00319027)'':2.74009e-05)'':0.00167934,('PDT000619699.2':0.0491734,(('PDT000189719.3':0.0203995,('PDT000066907.3':0.0177616,('PDT000198983.3':0.0107311,'PDT000134059.3':0.0107583)'':0.00410168)'':0.00139373)'':0.00937205,(('PDT000081147.3':0.0265768,('PDT000144455.3':0.0233796,'PDT000227573.3':0.0209453)'':0.00953613)'':0.000487665,(('PDT000375657.2':0.0114499,'PDT000492060.2':0.013008)'':0.0114183,('PDT000101375.3':0.0236848,('PDT000146036.3':0.0232649,('PDT000412103.2':0.0265278,((('PDT000621129.2':0.0142145,'PDT000705938.1':0.0115766)'':0.00110761,('PDT000134058.3':0.0119835,(('PDT000085631.3':0.00717716,'PDT000707508.1':0.00803764)'':0.00223648,('PDT000066909.3':0.00963728,('PDT000602560.2':0.0066229,'PDT000084175.3':0.0071915)'':0.00262822)'':0.00153682)'':0.0042943)'':-0.000986054)'':0.00420746,(('PDT000235323.3':0.0170576,(('PDT000492081.2':0.0134405,'PDT000264433.1':0.0179394)'':0.00027259,('PDT000264353.1':0.0141368,('PDT000707581.1':0.00981794,'PDT000051185.1':0.0107823)'':0.00456552)'':0.000411137)'':0.00447827)'':-6.97722e-05,(('PDT000611533.2':0.0177468,('PDT000234203.3':0.00855754,'PDT000668167.2':0.00959546)'':0.00761002)'':0.000458047,('PDT000547415.2':0.015255,(('PDT000368345.2':0.0145961,('PDT000197879.3':0.0111842,'PDT000084038.3':0.0118965)'':0.00172141)'':7.42267e-05,('PDT000084075.3':0.0118137,('PDT000273524.3':0.00350079,'PDT000491445.2':0.00418384)'':0.00795997)'':0.00104498)'':0.00262598)'':0.0005611)'':0.00181251)'':0.0016177)'':0.00380648)'':0.0017237)'':0.000855567)'':-0.00173389)'':0.000726134)'':-6.46751e-05)'':0.0169492)'':-0.0149523)'':0.000785837,(((('PDT000497869.1':0.0163583,'PDT000498079.1':0.0185449)'':0.00941043,('PDT000707362.1':0.0276016,(('PDT000384069.2':0.0159151,'PDT000141142.3':0.0233628)'':0.00436618,('PDT000469843.2':0.0200598,'PDT000708502.1':0.0197675)'':0.00287417)'':0.004893)'':-0.00056716)'':0.00101547,(('PDT000372131.2':0.0219054,('PDT000532757.2':0.0172009,'PDT000682353.1':0.0219447)'':0.00382145)'':0.00130031,('PDT000325466.2':0.0296813,('PDT000083285.3':0.0133457,('PDT000389796.2':0.00971455,'PDT000114525.3':0.00816745)'':0.00281242)'':0.0144123)'':0.000420781)'':0.000320599)'':0.00431582,(('PDT000157294.3':0.0319459,('PDT000130999.3':0.0294182,('PDT000189531.3':0.0230537,('PDT000144487.3':0.0205356,('PDT000201216.3':0.00962391,'PDT000128364.3':0.00806469)'':0.00921088)'':0.0015524)'':0.00455801)'':0.00309658)'':0.000109357,((('PDT000214033.3':0.0180741,'PDT000520267.2':0.0161772)'':0.00624569,('PDT000383935.2':0.0205068,'PDT000562728.2':0.0294833)'':0.00213484)'':0.00353116,(('PDT000114657.3':0.0242339,('PDT000112502.3':0.0170791,('PDT000646247.2':0.0100472,'PDT000107201.3':0.0105048)'':0.00424689)'':0.0033852)'':0.00184474,(('PDT000468215.2':0.0216228,'PDT000102799.3':0.0260009)'':0.00282728,(('PDT000491518.2':0.0201592,'PDT000570327.2':0.0178929)'':0.000500947,(('PDT000505013.2':0.0173754,'PDT000121933.3':0.0184128)'':0.00269437,('PDT000480958.2':0.0272427,(('PDT000533039.2':0.0157648,(('PDT000221973.3':0.0143468,'PDT000065223.3':0.0117031)'':0.00194757,('PDT000163376.4':0.0119649,('PDT000307847.3':0.00567887,'PDT000121055.3':0.00576053)'':0.00555834)'':0.00155241)'':0.00212879)'':0.00310743,(('PDT000523966.2':0.0180803,('PDT000214053.3':0.0138009,('PDT000329320.2':0.00322995,'PDT000501755.2':0.0025579)'':0.00872813)'':0.00158467)'':0.000712226,('PDT000458999.2':0.0172582,('PDT000234211.3':0.0119622,('PDT000630382.2':0.0110411,('PDT000208162.3':0.00582731,'PDT000133754.3':0.00682209)'':0.00348641)'':0.00195827)'':0.00403489)'':-0.00138034)'':0.00114489)'':0.00420631)'':-0.000467555)'':0.00135697)'':0.00169417)'':-0.00154114)'':0.00233914)'':0.000157579)'':0.00106254)'':0.00172799)'':0.00164802,((('PDT000065517.3':0.0202186,('PDT000156865.3':0.0178616,('PDT000135353.3':0.0215401,(('PDT000244735.3':0.00955605,('PDT000344842.2':0.0115685,('PDT000214011.3':0.00396836,'PDT000275759.3':0.00561057)'':0.00303132)'':0.00132499)'':0.00232225,('PDT000355504.2':0.0146516,('PDT000315732.2':0.0105204,('PDT000269985.3':0.00646797,'PDT000357197.2':0.00788033)'':0.00361119)'':0.00452154)'':-0.00116555)'':0.00629325)'':0.00608307)'':0.00230296)'':-0.00237428,('PDT000498196.1':0.0355776,(('PDT000602360.2':0.0174781,('PDT000113669.4':0.0173944,('PDT000085632.3':0.0127838,'PDT000264339.1':0.0124014)'':0.00635277)'':0.00397132)'':-0.000791195,(('PDT000120836.3':0.0215779,(('PDT000139809.3':0.0114407,('PDT000234219.3':0.00950343,'PDT000616516.2':0.00701237)'':0.00613767)'':0.00315355,(('PDT000222030.3':0.00712103,'PDT000491368.2':0.00688657)'':0.00500533,('PDT000189699.3':0.0134539,'PDT000202820.3':0.00865555)'':0.00303822)'':0.00175296)'':0.0114413)'':-0.00705757,(('PDT000264308.1':0.0196002,('PDT000264440.1':0.012627,'PDT000264528.1':0.0166592)'':0.00425318)'':0.0089435,(('PDT000375748.2':0.019171,('PDT000497544.2':0.0107706,'PDT000636721.2':0.0179118)'':0.00586523)'':0.00121828,(('PDT000344370.2':0.0192943,'PDT000491382.2':0.0161074)'':0.00609949,(('PDT000566529.2':0.0133533,('PDT000405111.2':0.00966829,'PDT000510684.2':0.00896221)'':0.00287936)'':0.00173504,((('PDT000173950.3':0.0124248,'PDT000264344.1':0.0131883)'':0.00242837,('PDT000551634.2':0.014752,('PDT000690310.1':0.00756424,'PDT000708611.1':0.00732556)'':0.00639352)'':0.00100224)'':0.00345022,(('PDT000097100.3':0.0198268,('PDT000250765.3':0.0173246,'PDT000132578.3':0.0122272)'':0.0060435)'':-0.000605447,((('PDT000287901.3':0.00565573,'PDT000357132.2':0.00699997)'':0.00349853,('PDT000707579.1':0.0100807,('PDT000431690.2':0.00798898,'PDT000707476.1':0.00929002)'':0.000521981)'':0.00168154)'':0.00394356,(('PDT000491459.2':0.0136008,'PDT000264514.1':0.00955563)'':0.00604111,(('PDT000222584.3':0.0181525,(('PDT000478253.2':0.0215688,('PDT000153282.3':0.00911814,'PDT000588385.2':0.00868926)'':0.00299427)'':9.59286e-05,('PDT000336793.2':0.00855611,('PDT000264519.1':0.00576479,('PDT000240582.3':0.00470144,'PDT000264457.1':0.00546096)'':0.00109066)'':0.00266902)'':0.0033138)'':0.00285688)'':-0.00189318,(('PDT000133961.1':0.00815818,('PDT000492071.2':0.0113006,('PDT000338869.2':0.00744144,('PDT000133773.3':0.00439582,('PDT000368344.2':0.00266399,'PDT000372112.2':0.00272685)'':0.0014856)'':0.00235411)'':0.00212651)'':0.00126249)'':0.000389523,('PDT000424798.1':0.0276246,(('PDT000264314.1':0.00351125,'PDT000264317.1':0.0033791)'':0.00619107,(('PDT000125015.3':0.00882463,(('PDT000222586.3':0.00613895,'PDT000491449.2':0.00668535)'':0.000137294,('PDT000227820.3':0.00803547,('PDT000286437.1':0.00554582,('PDT000491441.2':0.00384553,'PDT000264539.1':0.00362757)'':0.000636076)'':0.00219684)'':0.000137091)'':0.00201742)'':0.00292273,('PDT000072725.3':0.0127447,('PDT000105052.3':0.00885768,('PDT000334336.2':0.00843095,('PDT000216651.3':0.00791153,('PDT000092459.3':0.00512072,('PDT000597581.2':0.00527684,('PDT000152113.3':0.00548388,('PDT000707501.1':0.00483143,('PDT000707366.1':0.00282943,'PDT000130564.3':0.00265685)'':0.00134764)'':0.000112069)'':0.000444143)'':0.000848845)'':0.00110267)'':0.00177337)'':0.00223352)'':0.000973618)'':-0.00263858)'':0.0020052)'':0.00641353)'':-0.00528326)'':0.000944049)'':0.00302601)'':-0.00228092)'':0.00218842)'':-0.000422939)'':0.00268549)'':0.00169741)'':-0.000404603)'':0.00152837)'':-0.00293436)'':0.00435211)'':0.0148392)'':-0.0157505)'':0.00560556,(('PDT000259201.1':0.0319826,('PDT000084196.3':0.0139845,('PDT000264493.1':0.018162,('PDT000259200.1':0.0141891,('PDT000317729.2':0.0108208,'PDT000491489.2':0.0124373)'':0.0029433)'':0.00333481)'':0.00471964)'':0.00280042)'':0.0140728,(((('PDT000157921.3':0.0141062,'PDT000540088.2':0.0115882)'':0.0161706,('PDT000544730.2':0.0271278,('PDT000257974.3':0.028765,('PDT000264383.1':0.0273664,(('PDT000130551.3':0.0101519,'PDT000286443.1':0.0136126)'':0.00391952,(('PDT000482565.2':0.0143086,'PDT000537374.2':0.0108673)'':0.00162651,('PDT000490189.2':0.00929407,('PDT000366148.2':0.00690493,'PDT000492171.2':0.00741777)'':0.00418203)'':0.00390218)'':0.00401318)'':0.00652423)'':0.0015543)'':0.00794274)'':-0.00251859)'':0.000632482,(('PDT000498191.1':0.0371297,('PDT000086175.3':0.0183831,'PDT000286444.1':0.018945)'':0.0164071)'':-0.00025496,(('PDT000264473.1':0.0455702,(('PDT000066889.3':0.0189199,('PDT000336800.2':0.0186219,'PDT000492039.2':0.0197604)'':0.00280669)'':0.0084974,('PDT000412827.2':0.0268452,('PDT000235360.3':0.0132949,'PDT000355493.2':0.00984774)'':0.0130888)'':3.49045e-05)'':0.00581787)'':-0.00426948,(('PDT000273537.3':0.0259102,('PDT000271154.3':0.0176597,'PDT000653431.2':0.0138785)'':0.00818903)'':0.00155847,(('PDT000375969.2':0.0331994,(('PDT000357254.2':0.0159871,'PDT000602363.2':0.0227872)'':0.00288107,(('PDT000386156.2':0.0179125,('PDT000392136.2':0.0150232,('PDT000083234.3':0.0121975,('PDT000114538.3':0.0095521,('PDT000082150.3':0.00838105,'PDT000108016.3':0.0126014)'':0.000953902)'':0.00285256)'':0.00220037)'':0.00339253)'':4.50406e-05,(('PDT000367735.2':0.0144535,'PDT000107494.3':0.0154091)'':0.00169767,('PDT000366084.2':0.0203743,('PDT000071665.3':0.0142011,((('PDT000146454.3':0.0081838,'PDT000579070.2':0.0109973)'':0.00117032,('PDT000163370.4':0.00778355,('PDT000066847.3':0.0058272,'PDT000082121.3':0.006016)'':0.0006542)'':0.00357064)'':0.000408044,('PDT000420995.2':0.0111751,(('PDT000285892.3':0.00555197,'PDT000107729.3':0.00532843)'':0.00443809,('PDT000485838.2':0.00967979,('PDT000360445.2':0.00752878,'PDT000127480.3':0.00846882)'':0.00132046)'':8.91869e-05)'':0.00247827)'':0.000362088)'':0.00466537)'':0.00134049)'':-0.00124412)'':-0.000804124)'':0.00242115)'':0.00998979)'':-0.00695452,(('PDT000082147.3':0.0226093,'PDT000647198.2':0.0201215)'':0.00880834,(('PDT000286465.1':0.0200635,(('PDT000315881.2':0.018661,'PDT000101011.3':0.0170885)'':0.00280513,('PDT000564917.2':0.011825,(('PDT000203876.3':0.00659075,'PDT000355599.2':0.00418015)'':0.00463284,('PDT000264406.1':0.00804772,('PDT000359224.2':0.00558921,'PDT000082138.3':0.00525659)'':0.00131833)'':0.0010521)'':0.0050176)'':0.00388168)'':0.00150319)'':0.00275732,('PDT000208689.1':0.0415868,((('PDT000264422.1':0.0190717,('PDT000254242.3':0.0155214,'PDT000264358.1':0.0174033)'':0.00320183)'':0.0010435,(('PDT000410935.2':0.0166694,('PDT000329828.2':0.0106015,'PDT000651810.2':0.0115877)'':0.00300544)'':0.00182892,('PDT000707675.1':0.0248743,('PDT000562534.2':0.0264943,('PDT000110449.3':0.0171824,('PDT000120008.3':0.0196794,('PDT000366335.2':0.0185179,('PDT000102895.3':0.0171035,(('PDT000390230.2':0.00807949,'PDT000597583.2':0.00749901)'':0.00488771,('PDT000271151.3':0.0147483,('PDT000163368.4':0.0137186,(('PDT000201165.3':0.0127929,'PDT000707430.1':0.0152981)'':0.00092117,('PDT000580732.2':0.0127371,(('PDT000707462.1':0.0102657,'PDT000119774.3':0.0100716)'':0.00167537,('PDT000498801.2':0.00595148,('PDT000191137.3':0.00346172,'PDT000524331.2':0.00321957)'':0.00418292)'':0.0046269)'':0.00130528)'':0.000365147)'':0.000460835)'':0.000733351)'':0.000445112)'':0.00160984)'':0.000617014)'':0.000709518)'':0.00110321)'':0.00195668)'':0.000606043)'':-0.00292141)'':0.00362904)'':-0.00362938,(('PDT000492091.2':0.0151631,('PDT000048669.3':0.0123281,('PDT000102893.3':0.011322,('PDT000162345.3':0.0108823,('PDT000480930.2':0.00887071,('PDT000190108.3':0.00612491,'PDT000540115.2':0.00952659)'':0.00256219)'':0.000708868)'':0.00203187)'':0.000592675)'':0.000909602)'':0.000852135,(('PDT000366269.2':0.00798985,'PDT000069883.3':0.0114693)'':0.00589741,('PDT000508987.2':0.0153599,('PDT000532422.2':0.0181201,('PDT000673674.1':0.0151676,(('PDT000146011.3':0.00887555,'PDT000359217.2':0.00832295)'':0.00318595,('PDT000086333.3':0.0151716,('PDT000104755.10':0.0136979,(('PDT000359230.2':0.00533434,'PDT000430559.2':0.00669466)'':0.000354483,('PDT000604946.2':0.00707797,(('PDT000184477.3':0.00595847,'PDT000386185.2':0.00490243)'':0.000223797,('PDT000387609.2':0.00571022,('PDT000532404.2':0.00402336,('PDT000498869.2':0.0034962,('PDT000580727.2':0.00302206,'PDT000708615.1':0.00385662)'':9.08956e-05)'':0.00123004)'':0.000583336)'':-1.39768e-05)'':0.00178588)'':-0.000428233)'':0.00300334)'':0.000630752)'':-0.000564157)'':0.00273305)'':0.00327652)'':0.00109531)'':-0.00179715)'':-0.000856854)'':0.00422541)'':0.0164428)'':-0.0134718)'':0.00973667)'':-0.00862875)'':0.00567798)'':-0.00215539)'':0.00672488)'':-0.00506795)'':-0.00170439,(('PDT000264356.1':0.0261386,('PDT000707478.1':0.023244,'PDT000264459.1':0.0267315)'':0.00149236)'':0.0116798,(('PDT000521259.2':0.0348615,((('PDT000072786.3':0.0170675,('PDT000127731.3':0.0130912,'PDT000248603.1':0.0127751)'':0.00604686)'':-0.000256,('PDT000624101.2':0.0206849,('PDT000135750.3':0.0206815,('PDT000144491.3':0.0111495,'PDT000202818.3':0.0110156)'':0.00539766)'':0.00242467)'':0.00216123)'':0.00333037,('PDT000619728.2':0.0211584,(('PDT000137685.3':0.0129691,('PDT000147957.3':0.0165591,'PDT000083196.3':0.0106775)'':0.00266861)'':0.00277229,('PDT000264541.1':0.0185969,(('PDT000250760.3':0.009604,'PDT000280671.3':0.0103246)'':0.00442009,('PDT000251633.1':0.0141283,('PDT000259531.3':0.00968144,('PDT000492044.2':0.00734911,'PDT000544875.2':0.00581139)'':0.00367361)'':0.00298846)'':0.000223909)'':0.00109226)'':0.000299475)'':0.00909989)'':-0.00224327)'':0.00849083)'':-0.00941983,(('PDT000619695.2':0.0365483,((('PDT000707509.1':0.0198793,'PDT000286453.1':0.0160358)'':0.00467915,('PDT000492098.2':0.0167303,('PDT000691313.1':0.0117882,(('PDT000146472.3':0.00689227,'PDT000222095.3':0.00674633)'':0.001519,('PDT000465762.2':0.00676241,'PDT000491419.2':0.00479479)'':0.00424307)'':0.00200714)'':0.00599727)'':0.00108657)'':-0.000350731,('PDT000497830.1':0.0232772,(('PDT000123285.1':0.0174776,('PDT000264462.1':0.0120265,'PDT000264471.1':0.0141484)'':0.00502019)'':0.00699577,(('PDT000497944.1':0.0144139,('PDT000264377.1':0.00979277,'PDT000264553.1':0.0101338)'':0.00682871)'':-0.000130939,(('PDT000215104.3':0.00847411,('PDT000235765.3':0.00345648,'PDT000139218.3':0.00452437)'':0.00297631)'':0.00735637,(('PDT000381313.2':0.0121155,('PDT000209844.3':0.0100849,('PDT000144047.3':0.00614671,'PDT000133777.3':0.00696679)'':0.00306327)'':0.00111223)'':0.000747773,(('PDT000237159.3':0.00421208,('PDT000317576.2':0.00261807,'PDT000547447.2':0.00267188)'':0.00351936)'':0.00793131,('PDT000259195.1':0.00983464,('PDT000237165.3':0.00704082,('PDT000557840.2':0.00744129,('PDT000542152.2':0.00315539,('PDT000334236.2':0.00233995,'PDT000645175.2':0.00215017)'':0.00182894)'':0.00284188)'':0.000475374)'':0.000873054)'':0.00199984)'':0.000912334)'':0.00303663)'':0.000775552)'':0.00244224)'':0.00961223)'':-0.00848277)'':0.0185705)'':-0.0165669,('PDT000231236.3':0.026828,((('PDT000163580.3':0.0112498,'PDT000544322.2':0.011902)'':0.00534177,('PDT000072755.3':0.0151914,('PDT000540113.2':0.0119454,('PDT000264529.1':0.010403,('PDT000556682.2':0.00897481,'PDT000617924.2':0.00986879)'':0.000684157)'':0.00228989)'':0.00182288)'':0.00330636)'':0.00344136,(('PDT000155301.3':0.0211002,'PDT000189541.3':0.0214133)'':0.00386359,(('PDT000163581.3':0.0238242,'PDT000616424.2':0.0177804)'':0.00101659,('PDT000321895.2':0.0254112,(('PDT000107211.3':0.0218831,('PDT000149221.3':0.0134832,'PDT000247369.3':0.0170028)'':0.00904369)'':0.00158682,((('PDT000050487.3':0.0228013,'PDT000130566.3':0.0246176)'':0.00193452,('PDT000084209.3':0.0209847,('PDT000146487.3':0.0140807,('PDT000247195.3':0.0135631,('PDT000240619.3':0.00875739,('PDT000500461.2':0.0062067,'PDT000587813.2':0.0094463)'':0.00238481)'':0.00159279)'':0.00177895)'':0.00534155)'':0.00103432)'':-0.001003,(('PDT000404022.2':0.0151466,'PDT000125525.3':0.0129932)'':0.00304303,(('PDT000163284.4':0.00587718,('PDT000421062.2':0.00538687,('PDT000142614.3':0.00478894,'PDT000598744.1':0.00608926)'':0.000741681)'':0.00105748)'':0.0134311,(('PDT000273487.3':0.0172252,'PDT000264385.1':0.0143817)'':0.00461577,((('PDT000235722.3':0.0111381,'PDT000281527.3':0.0113127)'':0.00300339,('PDT000357115.2':0.0126984,'PDT000449069.2':0.0124128)'':0.00130559)'':0.00145414,(('PDT000125018.3':0.0137018,(('PDT000132156.3':0.0135817,('PDT000173886.3':0.0132482,('PDT000307841.3':0.00911295,('PDT000491397.2':0.00744641,('PDT000147951.3':0.00535467,'PDT000124791.3':0.00678823)'':0.00103509)'':0.00168782)'':0.00134274)'':0.00418391)'':-0.00108369,(('PDT000211639.3':0.00956639,'PDT000274058.3':0.00928651)'':0.00332787,('PDT000112505.3':0.0157666,('PDT000283373.3':0.0114646,('PDT000230812.3':0.00864719,('PDT000225287.3':0.00638129,('PDT000491367.2':0.00654315,'PDT000084010.3':0.00521335)'':0.00136856)'':0.00137124)'':0.00266482)'':0.0025455)'':0.000719608)'':0.000561487)'':0.001779)'':0.000296294,(('PDT000342334.2':0.0115399,'PDT000405489.2':0.00888859)'':0.00675606,(('PDT000287904.3':0.0121536,'PDT000048672.3':0.0120583)'':0.00635443,('PDT000334457.2':0.0137348,(('PDT000233971.3':0.0117562,'PDT000707424.1':0.0118931)'':0.00111781,('PDT000492115.2':0.0128511,(('PDT000176667.3':0.00903423,'PDT000469989.2':0.0102042)'':0.00110359,(('PDT000247176.3':0.00672296,'PDT000367249.2':0.00799314)'':0.00302565,('PDT000501776.2':0.00953266,(('PDT000101079.3':0.00857583,'PDT000120027.3':0.00872727)'':0.000845651,('PDT000366143.2':0.00697824,('PDT000137567.3':0.00490737,'PDT000141647.3':0.00466583)'':0.00319646)'':0.00190405)'':0.00140916)'':-0.000688018)'':0.000891102)'':0.0027886)'':-0.00106506)'':0.00171886)'':0.0011906)'':0.000408955)'':-0.00165963)'':0.0016835)'':0.000845915)'':-0.000276577)'':0.00200563)'':-0.00116017)'':0.00333127)'':0.00285493)'':-0.00272009)'':0.000901189)'':-0.0020389)'':0.00404067)'':-0.00250676)'':0.00100477)'':0.00649655)'':-0.00562612)'':0.013452)'':-0.00941898)'':0.00278488)'':0.0175744)'':-0.0160873,(('PDT000264455.1':0.0594305,(('PDT000336811.2':0.0351175,('PDT000108908.3':0.029936,'PDT000259203.1':0.0245156)'':0.00479184)'':0.00361517,(('PDT000242503.1':0.0322894,('PDT000145393.3':0.0150591,'PDT000081896.3':0.0137861)'':0.0120362)'':-0.00124319,(('PDT000158864.3':0.0198009,(('PDT000707461.1':0.0119536,'PDT000707600.1':0.0108996)'':0.00657115,(('PDT000259527.3':0.0158222,'PDT000264390.1':0.0170213)'':0.000416608,('PDT000264403.1':0.0129323,('PDT000143041.3':0.0126596,'PDT000264298.1':0.0117427)'':0.00418694)'':0.00176439)'':0.00249291)'':0.0023971)'':0.00213161,('PDT000421026.2':0.0261999,(('PDT000277077.3':0.0221956,(('PDT000329324.2':0.0129483,'PDT000372701.2':0.0132349)'':0.0068736,('PDT000707406.1':0.0150717,('PDT000225230.3':0.0101169,'PDT000521314.2':0.012931)'':0.00387129)'':0.00207199)'':0.00267401)'':0.00164793,(('PDT000146455.3':0.0170311,('PDT000503437.2':0.0134035,(('PDT000143042.3':0.0141147,'PDT000049090.3':0.0125783)'':0.00104993,('PDT000227271.3':0.0104438,'PDT000337028.2':0.00941318)'':0.00502637)'':0.00145809)'':0.000881125)'':0.00073249,('PDT000120022.3':0.0194534,(('PDT000107477.3':0.0103487,'PDT000286461.1':0.0101463)'':0.00527865,('PDT000270058.5':0.0188408,('PDT000072428.3':0.0140203,'PDT000107733.3':0.011565)'':0.00630541)'':0.000111451)'':0.00229039)'':-0.000508404)'':0.00160978)'':0.00738117)'':-0.00585824)'':0.00242888)'':0.00759018)'':0.0171047)'':-0.0191651,(('PDT000150134.3':0.032199,(('PDT000141599.1':0.0288289,('PDT000313056.3':0.0183902,'PDT000687282.1':0.0152565)'':0.0129766)'':-0.00264013,(('PDT000307525.3':0.0285451,(('PDT000264414.1':0.0186017,'PDT000264542.1':0.0124653)'':0.00622705,('PDT000264437.1':0.0190851,('PDT000286447.1':0.0185799,('PDT000146509.3':0.0125054,'PDT000396634.2':0.0125491)'':0.00455023)'':0.000650218)'':0.000202703)'':0.00674752)'':-0.00032714,(('PDT000120837.3':0.0290824,('PDT000384004.2':0.0281998,('PDT000102839.3':0.0158519,'PDT000124762.3':0.0139311)'':0.0116138)'':0.000605246)'':-0.00196831,(('PDT000146510.3':0.0161204,'PDT000193090.3':0.0146564)'':0.00328417,('PDT000251609.1':0.0242032,(('PDT000208190.3':0.0206411,'PDT000264398.1':0.0231746)'':0.00118809,('PDT000212445.3':0.0157479,('PDT000066913.3':0.0116013,('PDT000549254.2':0.0141859,('PDT000257954.3':0.00705395,'PDT000082155.3':0.00813665)'':0.00326092)'':0.00175131)'':0.00391614)'':0.000959231)'':0.00194297)'':-0.00120296)'':0.00247971)'':0.0032475)'':0.00309016)'':0.00498333)'':-0.00302629,(((('PDT000271244.3':0.0376377,('PDT000600207.2':0.0257424,('PDT000227338.3':0.0179502,'PDT000135356.3':0.0192179)'':0.00305999)'':0.0111789)'':-0.00256481,(('PDT000539314.2':0.0182643,('PDT000086301.3':0.0202203,('PDT000227311.3':0.0106813,('PDT000287886.3':0.00867605,'PDT000540323.2':0.00724115)'':0.0033531)'':0.00418944)'':0.00285413)'':0.000959921,(('PDT000264352.1':0.02899,'PDT000264381.1':0.0285248)'':0.00305661,(('PDT000598065.2':0.0263368,('PDT000107739.3':0.0169801,'PDT000124801.3':0.0160021)'':0.00571676)'':-0.000255252,(('PDT000286446.1':0.0305461,(('PDT000520073.2':0.0172225,('PDT000051381.3':0.0104752,('PDT000150160.3':0.00649456,'PDT000287900.3':0.00535564)'':0.00376255)'':0.00197072)'':0.00339859,('PDT000264343.1':0.0117887,('PDT000259213.1':0.00877223,('PDT000102774.3':0.00621594,('PDT000107731.3':0.00376422,('PDT000264776.3':0.00165709,'PDT000492209.2':0.00183941)'':0.000929397)'':0.00251755)'':0.00333859)'':0.00371324)'':0.00225964)'':0.00799247)'':-0.00595432,('PDT000156862.3':0.0233018,((('PDT000189712.3':0.00737851,'PDT000136753.3':0.00772329)'':0.00829061,('PDT000275715.3':0.0140666,('PDT000356305.2':0.0099019,'PDT000065504.3':0.0106155)'':0.00272312)'':0.000781302)'':0.00175089,(('PDT000491437.2':0.0115937,'PDT000526246.2':0.0127651)'':0.00610715,('PDT000102810.3':0.0185295,(('PDT000569726.2':0.00757969,('PDT000046386.3':0.00304538,'PDT000052118.3':0.00293966)'':0.00567984)'':0.00333709,(('PDT000468266.2':0.0122538,('PDT000142651.3':0.0110254,('PDT000267727.3':0.00994271,'PDT000107759.3':0.0105869)'':0.000737444)'':0.00183286)'':0.000759723,(('PDT000197839.3':0.00845734,('PDT000146578.3':0.00691607,'PDT000558469.2':0.00708873)'':0.00398096)'':0.00197982,('PDT000084081.3':0.0103549,('PDT000538681.2':0.00956836,('PDT000422024.2':0.00766056,('PDT000491443.2':0.00443179,('PDT000576043.2':0.00335129,'PDT000611843.2':0.00362638)'':0.0012441)'':0.00286356)'':0.00249852)'':0.00227227)'':-0.000249049)'':0.000980049)'':7.15593e-05)'':0.00451382)'':-0.000790342)'':-0.00114964)'':0.00371857)'':-0.0019338)'':0.00234385)'':0.00522424)'':-0.00422623)'':0.00480529)'':0.0021759,((('PDT000009672.2':0.0382488,('PDT000044757.3':0.037127,'PDT000248722.1':0.0320903)'':0.00691754)'':0.000302228,('PDT000313466.1':0.0403933,('PDT000264513.1':0.0262974,('PDT000390461.2':0.0208792,('PDT000313478.1':0.0268794,('PDT000264373.1':0.0158847,('PDT000264391.1':0.0177149,('PDT000264350.1':0.0115546,'PDT000264518.1':0.0118713)'':0.00440615)'':0.00202297)'':0.00319799)'':0.00158529)'':0.00269658)'':0.00654376)'':0.0083536)'':-0.00419152,((('PDT000280679.3':0.0131007,'PDT000405081.2':0.0150086)'':0.0125263,(('PDT000375990.2':0.0202331,'PDT000264517.1':0.021454)'':0.00515656,(('PDT000257952.3':0.0173564,'PDT000114659.3':0.0145196)'':0.00619035,(('PDT000264545.1':0.0221349,('PDT000238395.3':0.0277996,('PDT000264311.1':0.0157725,('PDT000173946.3':0.0168964,('PDT000243640.3':0.0130023,('PDT000113672.3':0.00650515,'PDT000557528.1':0.00770105)'':0.00231159)'':0.00515904)'':0.00145026)'':0.00178022)'':0.00190886)'':-0.00209134,('PDT000286455.1':0.0215973,('PDT000537438.2':0.0205415,('PDT000264323.1':0.0218783,(('PDT000046387.3':0.0132554,('PDT000072766.3':0.0165719,('PDT000221620.3':0.0102146,('PDT000421282.2':0.00919185,(('PDT000142636.3':0.00532897,('PDT000491415.2':0.00649524,'PDT000102914.3':0.00486046)'':0.00156288)'':0.000282774,('PDT000526244.2':0.00372797,('PDT000146295.3':0.00294282,'PDT000042363.3':0.00264489)'':0.00115779)'':0.00166511)'':0.00300786)'':0.000977284)'':0.00180348)'':0.00171782)'':-0.00178115,('PDT000135352.3':0.0118911,('PDT000457362.2':0.0126615,('PDT000264442.1':0.0142605,('PDT000264386.1':0.0083885,(('PDT000120925.3':0.007666,'PDT000264325.1':0.0081094)'':0.0011748,('PDT000062521.3':0.00495233,('PDT000264552.1':0.0059181,(('PDT000491418.2':0.0039209,'PDT000523345.2':0.00425865)'':0.000103542,('PDT000494938.2':0.0037246,'PDT000286441.1':0.0039562)'':0.000437618)'':0.00121805)'':0.00158105)'':0.00198736)'':0.00210189)'':0.000336925)'':0.000656127)'':0.00105326)'':-0.00173605)'':0.00639127)'':0.000467787)'':0.00266152)'':-0.0040649)'':0.00492853)'':0.00249994)'':0.00140164)'':-0.00424793,(('PDT000379555.2':0.0264525,(('PDT000368905.1':0.0157471,('PDT000264447.1':0.013339,('PDT000025430.2':0.00718257,('PDT000249846.1':0.00988158,'PDT000264443.1':0.00645142)'':0.000698376)'':0.00348473)'':0.00120259)'':0.000989569,('PDT000145315.3':0.0190479,(('PDT000150703.3':0.00904794,'PDT000616906.2':0.00871356)'':0.000555163,('PDT000611675.2':0.00793833,'PDT000137344.3':0.00805447)'':0.000812987)'':0.00812257)'':-0.00386806)'':0.0089781)'':-0.00422036,((('PDT000126255.3':0.0146781,('PDT000114691.3':0.0149885,'PDT000130428.3':0.0144233)'':0.0018342)'':0.00610167,(('PDT000066891.3':0.0114051,('PDT000086303.3':0.0176452,('PDT000142629.3':0.00547695,'PDT000587160.2':0.00428401)'':0.00686413)'':0.00256096)'':0.00508565,('PDT000008477.2':0.0219798,('PDT000375742.2':0.0214311,(('PDT000133960.1':0.0142825,('PDT000264331.1':0.00827775,'PDT000286457.1':0.0100088)'':0.00369969)'':0.000858129,('PDT000682349.1':0.0148113,(('PDT000041508.3':0.0106035,'PDT000046586.3':0.00846302)'':0.00198564,(('PDT000120821.3':0.00544434,'PDT000368901.1':0.00727806)'':0.00821491,('PDT000072759.3':0.0118436,(('PDT000412003.2':0.0083058,'PDT000126297.3':0.0116185)'':0.000828512,('PDT000357190.2':0.0100776,('PDT000492122.2':0.0101875,'PDT000162297.3':0.00733185)'':0.00125091)'':0.000642185)'':0.000766141)'':0.000636599)'':0.000767211)'':0.00178498)'':-0.00058456)'':0.00643924)'':0.00266383)'':-0.00520857)'':0.00205789)'':-0.00234972,('PDT000150712.3':0.0238699,((('PDT000712378.1':0.0198215,('PDT000533997.2':0.0141173,('PDT000412119.2':0.0100869,'PDT000707576.1':0.0106123)'':0.00301985)'':0.0086995)'':-0.00109396,(('PDT000592658.2':0.0195224,'PDT000264509.1':0.0197913)'':0.00471783,(('PDT000556533.2':0.016735,('PDT000491457.2':0.00931048,(('PDT000264421.1':0.00399753,'PDT000264431.1':0.00407905)'':0.00301241,('PDT000190360.3':0.00679309,('PDT000195709.3':0.00534681,'PDT000259199.1':0.00541769)'':0.000951812)'':0.00123168)'':0.00320209)'':0.00296485)'':-0.000817915,((('PDT000264502.1':0.0125458,('PDT000643599.2':0.00736385,('PDT000344815.2':0.00705419,('PDT000096860.3':0.00569061,('PDT000344372.2':0.00462604,('PDT000226799.3':0.00430746,'PDT000616965.2':0.00566462)'':0.00115156)'':0.000632162)'':0.00158251)'':0.00169282)'':0.00390179)'':-0.00301949,(('PDT000338821.2':0.00619438,'PDT000124877.3':0.00684822)'':0.00209648,(('PDT000189518.3':0.009128,('PDT000289667.3':0.00708763,('PDT000235730.3':0.00421053,'PDT000491945.2':0.00406052)'':0.00186304)'':0.00112787)'':0.000637365,(('PDT000150161.3':0.00539599,('PDT000491379.2':0.00380256,'PDT000707428.1':0.00407146)'':0.00197526)'':0.000974459,('PDT000286451.1':0.00716013,('PDT000173953.3':0.002105,'PDT000707413.1':0.0021907)'':0.00426277)'':-1.32833e-05)'':0.00115383)'':0.00139011)'':0.000651469)'':0.000646185,((('PDT000264307.1':0.0137055,'PDT000264337.1':0.00882237)'':0.0049933,('PDT000421982.2':0.0122921,('PDT000336796.2':0.00825458,'PDT000491400.2':0.00864742)'':0.00388312)'':0.00179191)'':0.000957945,(('PDT000536779.2':0.0202504,('PDT000052447.3':0.010937,('PDT000327658.2':0.00739392,('PDT000520271.2':0.00700896,'PDT000604740.2':0.00750974)'':0.00145758)'':0.001199)'':0.00625676)'':-0.00286803,(('PDT000041847.3':0.0101208,'PDT000459012.2':0.0105784)'':0.000117818,(('PDT000147983.3':0.00556701,'PDT000306751.3':0.00577289)'':0.000801654,('PDT000602697.2':0.00657959,(('PDT000244737.3':0.00494737,'PDT000051933.3':0.00500882)'':0.000499766,('PDT000664348.1':0.00486073,('PDT000271245.3':0.00398993,(('PDT000072710.3':0.00339197,'PDT000083157.3':0.00378864)'':0.000764092,('PDT000573757.2':0.00421238,('PDT000267875.3':0.0029524,('PDT000587231.2':0.00295784,'PDT000646753.2':0.00302481)'':0.000140153)'':0.000326042)'':0.000488131)'':0.000789139)'':0.000625352)'':0.000293764)'':0.000897201)'':-0.000390502)'':0.0039296)'':0.00145524)'':0.00103218)'':-0.00101264)'':0.00210099)'':0.00581094)'':0.00391226)'':-0.00710055,(('PDT000478262.2':0.0194238,('PDT000221752.3':0.00931619,'PDT000264465.1':0.00719961)'':0.00679726)'':0.000870974,((('PDT000497874.1':0.0256451,(('PDT000497827.1':0.00822617,'PDT000498015.1':0.0104765)'':0.00215099,('PDT000498109.1':0.010485,'PDT000498139.1':0.0143412)'':0.00118086)'':0.00567782)'':0.000106131,(('PDT000407594.2':0.0222031,('PDT000264532.1':0.0122701,('PDT000365940.2':0.00994027,('PDT000184429.3':0.00796711,('PDT000189708.3':0.00556996,'PDT000471093.2':0.00758534)'':0.00174789)'':0.00232603)'':0.00339656)'':0.00314385)'':-0.00164601,(('PDT000156807.3':0.011733,('PDT000366195.2':0.0103699,('PDT000411982.2':0.00586864,('PDT000293101.3':0.00439281,'PDT000125008.3':0.00426994)'':0.00189458)'':0.00337461)'':0.00222084)'':0.000622042,(('PDT000608718.2':0.00765203,'PDT000653416.2':0.00726147)'':0.00318642,('PDT000624104.2':0.00992495,('PDT000480388.2':0.00965016,('PDT000238306.3':0.00454937,'PDT000264341.1':0.00441924)'':0.00452664)'':0.0018625)'':0.000529134)'':0.00238291)'':0.00116029)'':0.0037359)'':-0.000511524,('PDT000603823.2':0.0225653,(('PDT000334464.2':0.00661721,'PDT000072718.3':0.00886269)'':0.00452508,(('PDT000306726.3':0.0126965,('PDT000300577.1':0.00799516,('PDT000604635.2':0.0108398,('PDT000628852.2':0.00967739,('PDT000128677.3':0.00454688,('PDT000373161.2':0.00470648,(('PDT000674818.1':0.00395216,('PDT000321929.2':0.0039026,'PDT000576166.2':0.00577128)'':0.000490267)'':0.000269901,('PDT000372598.2':0.00449988,('PDT000107745.3':0.00302257,('PDT000491458.2':0.00466158,('PDT000281512.3':0.00234864,('PDT000491434.2':0.00177968,'PDT000586167.2':0.00201387)'':0.000344704)'':0.000818542)'':0.000302395)'':0.00131502)'':-0.000387117)'':0.000697656)'':0.000805146)'':0.00202461)'':0.000441195)'':0.00100819)'':0.00298851)'':-0.00283954,(('PDT000518064.2':0.0055755,('PDT000232788.3':0.00887297,'PDT000394716.2':0.00547963)'':0.00175135)'':0.00193448,(('PDT000259214.1':0.0126463,('PDT000235791.3':0.00726641,'PDT000123282.1':0.00835239)'':0.00228483)'':0.00159143,('PDT000387595.2':0.020246,(('PDT000131016.3':0.0117397,('PDT000491402.2':0.00983914,'PDT000566468.2':0.0101887)'':0.00165745)'':0.00110269,((('PDT000235775.3':0.00633013,'PDT000240593.3':0.00424767)'':0.00101474,(('PDT000151458.3':0.00300989,'PDT000124798.3':0.00407948)'':0.00104013,('PDT000359211.2':0.00372266,('PDT000050486.3':0.00261392,(('PDT000491383.2':0.001455,'PDT000124943.3':0.0013422)'':0.000679694,('PDT000102917.3':0.00164277,('PDT000491455.2':0.00133094,'PDT000492064.2':0.00106738)'':0.000753427)'':0.000165742)'':0.000953943)'':0.00175946)'':-0.000558401)'':0.00219833)'':0.000115549,('PDT000282455.3':0.011154,('PDT000362581.2':0.00682408,(('PDT000198403.4':0.00464288,'PDT000494947.2':0.00601772)'':0.000905929,('PDT000538650.2':0.0058591,('PDT000317574.2':0.00581162,('PDT000176708.3':0.00510724,('PDT000351785.2':0.00349459,('PDT000167509.3':0.00391343,('PDT000655733.2':0.0037953,('PDT000124776.3':0.00320464,('PDT000662019.1':0.00178585,('PDT000491384.2':0.00151461,'PDT000707365.1':0.00108279)'':0.000708723)'':0.000516307)'':0.000298502)'':0.00041213)'':0.00121925)'':0.000499209)'':0.00114412)'':0.00042531)'':-0.000132311)'':0.00180301)'':0.00235407)'':-0.003391)'':0.00327766)'':0.0044732)'':-0.00359969)'':0.000885852)'':-0.000585095)'':0.00362125)'':0.0045712)'':-0.0021787)'':0.00174028)'':-0.00329114)'':0.00774737)'':-0.00566338)'':0.00177083)'':-0.00030283)'':0.00982748)'':-0.00734129)'':0.000916862,((('PDT000444896.2':0.0394267,('PDT000123288.1':0.0352485,('PDT000378696.2':0.0174638,'PDT000707425.1':0.01331)'':0.00905145)'':0.00585493)'':0.000438829,('PDT000322774.2':0.0517765,((('PDT000320249.2':0.0214751,'PDT000444929.2':0.0173417)'':0.0077532,('PDT000444836.2':0.0223555,'PDT000616903.2':0.0190252)'':0.00585258)'':0.00117043,('PDT000379450.2':0.033079,('PDT000316153.2':0.0180174,('PDT000378700.2':0.0177596,(('PDT000148549.3':0.0079994,'PDT000123274.1':0.0105247)'':0.00246674,(('PDT000598439.2':0.0105614,('PDT000147816.3':0.00966189,'PDT000378036.2':0.00998491)'':0.00152674)'':-4.33021e-05,('PDT000378060.2':0.00952524,('PDT000339391.2':0.00750579,('PDT000147689.3':0.00747289,'PDT000161683.3':0.00721981)'':0.00125126)'':0.00222032)'':6.29533e-05)'':0.000911505)'':0.00397557)'':0.00191453)'':0.00488413)'':-0.00236292)'':0.0110475)'':0.00225438)'':0.00423125,(((('PDT000115416.3':0.0147585,('PDT000468462.2':0.0139028,('PDT000619746.2':0.0102413,'PDT000264504.1':0.0119147)'':0.002013)'':0.00295517)'':0.0024441,('PDT000146566.3':0.0144086,('PDT000072435.3':0.00692809,('PDT000252777.3':0.0043225,'PDT000357204.2':0.00464075)'':0.00342254)'':0.00498782)'':0.00699714)'':0.00192952,('PDT000286452.1':0.027083,(('PDT000072733.3':0.0182122,'PDT000264426.1':0.0288673)'':0.00697796,((('PDT000313470.1':0.0200354,'PDT000313476.1':0.0226601)'':0.00589404,(('PDT000264395.1':0.0102255,'PDT000264397.1':0.0145077)'':0.00667228,('PDT000264425.1':0.0105601,('PDT000264369.1':0.0110235,'PDT000264375.1':0.00919252)'':0.00609457)'':0.00203321)'':0.00629496)'':6.6733e-05,('PDT000498141.1':0.0295288,(('PDT000082115.3':0.0268542,'PDT000264411.1':0.0225273)'':0.000230693,(('PDT000468566.2':0.0171057,('PDT000468563.2':0.0124204,'PDT000468570.2':0.00980841)'':0.00890153)'':0.000471823,(('PDT000264461.1':0.0188421,('PDT000264399.1':0.0225499,('PDT000264354.1':0.0121684,'PDT000264365.1':0.013536)'':0.00568789)'':0.00241754)'':0.000910733,('PDT000286432.1':0.0122837,('PDT000264384.1':0.010565,(('PDT000264355.1':0.00670685,'PDT000264458.1':0.00592115)'':0.00613693,('PDT000264367.1':0.0128449,('PDT000286445.1':0.00714986,('PDT000264361.1':0.00551548,'PDT000264389.1':0.00601882)'':0.000197145)'':0.00257195)'':0.00110325)'':0.00128902)'':0.00442794)'':0.000847664)'':0.00234554)'':0.00281099)'':0.00607225)'':-0.00339589)'':0.00196206)'':0.00435301)'':-0.00520327)'':0.00795931,(((('PDT000243617.3':0.0360541,('PDT000212405.3':0.0347739,('PDT000225514.3':0.00769049,'PDT000234194.3':0.00790501)'':0.0160092)'':0.00468971)'':-0.00111069,(('PDT000146565.3':0.0110081,'PDT000491433.2':0.00877117)'':0.00458491,('PDT000102897.3':0.0185944,('PDT000158567.3':0.0159015,('PDT000146291.3':0.00709938,('PDT000242817.3':0.00556756,'PDT000510667.2':0.00508024)'':0.00105537)'':0.00528756)'':0.00402352)'':0.000648037)'':0.005892)'':0.00196125,(('PDT000376013.2':0.018815,'PDT000286462.1':0.0218091)'':0.00470088,('PDT000584547.2':0.0180654,('PDT000372036.2':0.0165267,('PDT000525366.2':0.0157162,('PDT000264429.1':0.0105367,(('PDT000264740.3':0.00873843,('PDT000344836.2':0.00521947,'PDT000083136.3':0.00483433)'':0.00408737)'':0.00114415,(('PDT000311675.3':0.00342894,'PDT000357202.2':0.00345042)'':3.60273e-05,('PDT000238312.3':0.00341195,(('PDT000144410.3':0.0019322,'PDT000707591.1':0.0021637)'':0.00123084,('PDT000548673.2':0.00314755,'PDT000705904.1':0.00303576)'':0.000809668)'':0.000302449)'':0.000277027)'':0.00427094)'':0.00383217)'':0.000666264)'':0.00478605)'':0.00143615)'':0.002597)'':0.00634243)'':0.00188037,((((('PDT000338720.2':0.0231243,('PDT000184487.3':0.0159482,'PDT000284935.3':0.0162463)'':0.00365091)'':0.00167701,('PDT000084056.3':0.0182207,('PDT000602962.2':0.0184254,'PDT000135790.3':0.0181011)'':0.00150337)'':0.00159095)'':0.00391107,(('PDT000500642.2':0.0269085,('PDT000264540.1':0.0247445,('PDT000498189.1':0.0186602,('PDT000598469.2':0.0160241,('PDT000161588.3':0.0109303,'PDT000365723.2':0.0114615)'':0.00570137)'':0.00284736)'':0.00367332)'':0.00167846)'':0.00375296,('PDT000313472.1':0.0376497,(('PDT000264347.1':0.0110186,'PDT000264400.1':0.0180622)'':0.0175787,('PDT000110436.3':0.0229136,(('PDT000359238.2':0.0161468,('PDT000047174.3':0.0154301,'PDT000491442.2':0.0153891)'':0.00197968)'':0.00119912,(('PDT000390975.2':0.0179215,'PDT000576158.2':0.0174027)'':0.00122375,('PDT000367047.2':0.0162222,'PDT000264555.1':0.0217938)'':0.000112075)'':0.0019569)'':0.00510097)'':0.000655444)'':0.00351612)'':-0.00214103)'':0.00133887)'':0.00171043,(('PDT000352806.2':0.0413505,('PDT000247395.3':0.0339096,(('PDT000124995.3':0.0230518,('PDT000566482.2':0.0171029,('PDT000247526.3':0.016675,'PDT000306545.3':0.0147722)'':0.00231662)'':0.00273544)'':0.00145755,(('PDT000573464.2':0.0267602,('PDT000235785.3':0.0175146,'PDT000708501.1':0.0190503)'':0.00288722)'':0.00098161,(('PDT000356319.2':0.019274,'PDT000492136.2':0.0150824)'':0.00406648,(('PDT000548988.2':0.0119055,'PDT000038876.1':0.0150543)'':0.00358703,(('PDT000555671.2':0.0147568,('PDT000631006.2':0.016825,('PDT000355453.2':0.00747026,'PDT000491446.2':0.00633444)'':0.00502079)'':0.00415478)'':-0.00148075,('PDT000592630.2':0.0279135,(('PDT000259220.1':0.0152781,('PDT000355508.2':0.01211,'PDT000127744.3':0.0136837)'':0.00341437)'':0.00031678,(('PDT000227316.3':0.00726496,'PDT000321954.2':0.00843144)'':0.00444012,('PDT000366984.2':0.0124934,('PDT000411986.2':0.0167623,('PDT000181358.3':0.00714485,('PDT000146471.3':0.0053311,'PDT000505042.2':0.0055222)'':0.00119815)'':0.00334506)'':0.0030849)'':0.000668915)'':0.000724419)'':0.00464)'':-0.00224585)'':0.00137594)'':0.00118043)'':0.00206605)'':0.000405072)'':0.00651777)'':0.00268323)'':-0.00711233,((('PDT000110751.3':0.00829889,'PDT000259202.1':0.0103945)'':0.00422149,('PDT000224233.3':0.0144007,(('PDT000101021.4':0.00582392,'PDT000084152.3':0.00763438)'':0.00530508,('PDT000471160.2':0.0112625,'PDT000128443.3':0.0120738)'':0.00204104)'':0.00197438)'':0.00265163)'':0.00242124,(('PDT000323625.2':0.0304847,(('PDT000288524.3':0.01929,'PDT000355465.2':0.0171612)'':0.00050325,(('PDT000151430.3':0.0162453,'PDT000205379.3':0.0133123)'':0.00363223,(('PDT000108911.3':0.0122791,'PDT000264438.1':0.012307)'':0.00324566,(('PDT000171581.3':0.01345,('PDT000491440.2':0.00912605,'PDT000573250.2':0.00970265)'':0.00400428)'':0.000258003,(('PDT000084158.3':0.0107911,('PDT000082087.3':0.0090204,'PDT000264480.1':0.0091236)'':0.00309006)'':0.000418606,('PDT000083275.3':0.0112101,('PDT000719826.1':0.0122962,('PDT000066910.3':0.00997794,('PDT000356326.2':0.00873802,('PDT000366286.2':0.0072895,'PDT000672409.1':0.0085961)'':0.00147353)'':0.000686435)'':0.00112276)'':0.000982377)'':0.000378952)'':0.000837827)'':0.00214649)'':0.0019436)'':0.00315652)'':0.00730313)'':-0.00950121,(('PDT000459008.2':0.0290942,('PDT000235337.3':0.0182763,('PDT000491378.2':0.0139176,('PDT000603568.2':0.0176094,('PDT000275630.3':0.0107465,'PDT000048205.3':0.0137214)'':0.00336291)'':0.00185472)'':0.00192068)'':0.00408443)'':0.000844082,(('PDT000325784.2':0.0154786,(('PDT000355524.2':0.0146891,(('PDT000366289.2':0.00626085,'PDT000084025.3':0.00666285)'':0.00176568,('PDT000083181.3':0.00394792,'PDT000120854.3':0.00423245)'':0.00204883)'':0.00779974)'':-0.00109056,('PDT000510821.2':0.0161035,(('PDT000360434.2':0.00685789,'PDT000549881.2':0.00675361)'':0.00166052,('PDT000492050.2':0.00677476,('PDT000336807.2':0.00409099,('PDT000145411.3':0.00293203,'PDT000289044.3':0.00265512)'':0.00109252)'':0.000874158)'':0.00403054)'':0.00344796)'':-0.00163069)'':0.00444883)'':-0.00193575,(((('PDT000357148.2':0.00599373,'PDT000481038.2':0.00792697)'':0.00362044,('PDT000594288.2':0.00599849,'PDT000135729.3':0.00575101)'':0.00244191)'':0.00130759,('PDT000264412.1':0.0150132,('PDT000025441.2':0.00793153,(('PDT000253186.3':0.00514373,'PDT000087349.3':0.00582887)'':0.00161014,('PDT000188023.3':0.00804904,('PDT000509940.2':0.00501397,'PDT000707364.1':0.00503983)'':0.00136761)'':0.000206178)'':0.00105303)'':0.00141726)'':0.00114129)'':0.00186708,(('PDT000145355.3':0.0139525,('PDT000102881.3':0.0178875,('PDT000264359.1':0.0123985,(('PDT000265285.3':0.00538402,'PDT000579958.2':0.00536118)'':0.00406862,('PDT000682698.1':0.00875659,('PDT000132456.3':0.00945019,(('PDT000163373.4':0.0050043,'PDT000491366.2':0.00445408)'':0.00108378,('PDT000702537.1':0.00165503,'PDT000141643.3':0.00144095)'':0.00437213)'':0.00184866)'':0.000982469)'':-0.000237666)'':0.00286985)'':0.00319756)'':0.000847111)'':1.61277e-05,(('PDT000146506.3':0.017748,(('PDT000196476.3':0.00729863,'PDT000344819.2':0.00802167)'':0.00337639,(('PDT000707507.1':0.0134412,('PDT000400028.2':0.00717467,('PDT000131117.3':0.0078835,('PDT000145407.3':0.00679545,'PDT000340517.2':0.00583505)'':0.00074465)'':0.00146329)'':0.00175118)'':0.000267752,('PDT000293094.3':0.0153949,('PDT000549592.2':0.00853216,('PDT000209811.3':0.0136034,('PDT000492253.2':0.00824764,(('PDT000176754.3':0.00594504,('PDT000443368.2':0.00400962,'PDT000264322.1':0.00356236)'':0.00102167)'':0.00150609,('PDT000299808.3':0.00665218,('PDT000190660.3':0.00733801,('PDT000234159.3':0.00594888,('PDT000259772.3':0.00377675,('PDT000222585.3':0.00303038,('PDT000146451.3':0.00202992,'PDT000086176.3':0.00276624)'':0.000808292)'':0.000723612)'':0.00123057)'':0.000907898)'':0.00124046)'':-0.000883167)'':0.00191561)'':0.00127776)'':0.000438564)'':0.00459791)'':-0.00429638)'':0.00141069)'':0.00310646)'':-0.00279679,(('PDT000629984.2':0.0170217,('PDT000598841.1':0.0090114,('PDT000355520.2':0.00728173,'PDT000525159.2':0.00429507)'':0.00443115)'':0.0036572)'':0.00551154,((('PDT000569569.2':0.0190105,(('PDT000420996.2':0.0115441,'PDT000544469.2':0.0118477)'':0.00114337,(('PDT000221943.3':0.0121566,'PDT000376079.2':0.00935405)'':0.00143333,('PDT000281519.3':0.0102245,'PDT000465603.2':0.0101857)'':0.00247222)'':0.00160443)'':0.00723913)'':-0.00583681,(('PDT000577468.2':0.00996195,('PDT000550976.2':0.00772827,'PDT000615287.2':0.00749563)'':0.00272155)'':0.00345183,('PDT000492217.2':0.0341967,('PDT000134070.3':0.0211003,(('PDT000264511.1':0.0125591,('PDT000144061.3':0.0105797,'PDT000683636.1':0.0108086)'':0.00244851)'':0.00220165,('PDT000313471.1':0.0188635,((('PDT000198488.3':0.0053919,'PDT000125562.3':0.0053554)'':0.00497612,('PDT000081154.3':0.0100405,'PDT000102879.3':0.0100585)'':0.00156963)'':0.000249602,('PDT000285800.3':0.011072,(('PDT000556630.2':0.0147491,('PDT000144056.3':0.00983481,'PDT000096896.3':0.0119432)'':0.00204148)'':-0.000295353,(('PDT000710813.1':0.00982317,('PDT000547101.2':0.00846779,('PDT000492185.2':0.00821184,('PDT000386789.2':0.00680904,(('PDT000244740.3':0.00426812,'PDT000130986.3':0.00449595)'':0.00232965,(('PDT000311120.3':0.00487462,'PDT000360413.2':0.00668028)'':0.000739982,('PDT000532420.2':0.00530533,('PDT000492178.2':0.00310478,'PDT000096905.3':0.00258157)'':0.00254615)'':0.000269161)'':0.000283173)'':0.000748394)'':0.000580625)'':0.00212854)'':0.0012496)'':-0.00163997,('PDT000193104.3':0.019081,(('PDT000253175.3':0.00741524,('PDT000705872.1':0.00655908,('PDT000143270.3':0.00521308,('PDT000367191.2':0.00428092,'PDT000610262.2':0.00498664)'':0.000455649)'':0.00181434)'':0.00101615)'':0.000325564,(('PDT000329306.2':0.00964239,'PDT000707465.1':0.00920311)'':0.000892726,('PDT000038862.1':0.0106655,(('PDT000360403.2':0.00550379,(('PDT000145273.3':0.00351154,'PDT000254255.3':0.00386265)'':0.00168323,('PDT000342379.2':0.00509251,('PDT000707471.1':0.00294175,('PDT000171639.3':0.00251751,'PDT000084005.3':0.00237271)'':0.000646238)'':0.00158917)'':0.00035481)'':0.000861643)'':0.000775076,('PDT000355518.2':0.0107242,('PDT000451195.2':0.00683254,('PDT000637261.2':0.00543565,('PDT000137732.3':0.004234,(('PDT000163789.3':0.00350932,'PDT000086330.3':0.00326899)'':0.000633073,('PDT000469977.2':0.00365433,'PDT000066887.3':0.00362093)'':0.000184917)'':0.000657947)'':0.00120241)'':0.000638301)'':0.00130872)'':-0.000676822)'':0.00301163)'':-0.000900314)'':-0.000799379)'':0.00418581)'':-0.00286911)'':0.00172213)'':0.00188471)'':-0.00190115)'':0.00652395)'':-0.00461745)'':0.005269)'':0.00301253)'':-0.00642697)'':-0.000202859)'':0.000214319,(('PDT000676472.1':0.0184953,('PDT000469991.2':0.00663725,'PDT000558527.2':0.0108191)'':0.00655081)'':0.00188254,((('PDT000386275.2':0.0113427,'PDT000485839.2':0.0115958)'':0.00166758,('PDT000093293.3':0.0117519,('PDT000400799.2':0.0128104,'PDT000389723.2':0.0114254)'':0.000960566)'':0.00154031)'':-1.2329e-05,((('PDT000340522.2':0.00866824,'PDT000123409.3':0.0109201)'':0.0023338,('PDT000490410.2':0.00531874,'PDT000628674.2':0.00742776)'':0.00645215)'':0.00336206,('PDT000552691.2':0.0161916,(('PDT000368338.2':0.00997291,'PDT000110601.3':0.0157576)'':0.00260215,('PDT000480411.2':0.0131675,(('PDT000604634.2':0.0146705,(('PDT000351856.2':0.00678753,'PDT000690188.1':0.00702817)'':0.00379851,(('PDT000213973.3':0.00924654,('PDT000153293.3':0.00808129,('PDT000147915.3':0.00561851,'PDT000168362.3':0.00660809)'':0.00134361)'':0.00323419)'':-0.00047924,(('PDT000394059.2':0.00619797,('PDT000235066.3':0.00545382,'PDT000083260.3':0.00391433)'':0.0013752)'':0.000834906,('PDT000307487.3':0.00683471,('PDT000222861.3':0.0046391,'PDT000549491.2':0.00442104)'':0.00268727)'':0.000260021)'':0.0016182)'':0.00235633)'':0.00406287)'':-0.00539894,(('PDT000648243.2':0.0123276,('PDT000569796.2':0.0128447,('PDT000451142.2':0.00907621,('PDT000273062.3':0.00759817,('PDT000381327.2':0.00772328,'PDT000546089.2':0.00708782)'':0.000536175)'':0.00172599)'':0.00205825)'':0.000939501)'':-0.00175876,(('PDT000081162.3':0.0121439,'PDT000264464.1':0.0137028)'':0.0030554,(('PDT000135753.3':0.0102038,(('PDT000086306.3':0.00535194,'PDT000663490.1':0.00649706)'':0.00142557,('PDT000125574.3':0.00913033,('PDT000325509.2':0.00508878,('PDT000230057.3':0.00749238,('PDT000354930.2':0.00490002,('PDT000264346.1':0.00572463,('PDT000707682.1':0.00391764,('PDT000707577.1':0.00303619,('PDT000492085.2':0.00287413,('PDT000142642.3':0.00332815,('PDT000707361.1':0.00250457,'PDT000124787.3':0.00238419)'':0.000557772)'':0.000529136)'':0.000446993)'':0.000391052)'':0.000932835)'':0.0010243)'':0.000620492)'':9.87241e-05)'':0.000862361)'':0.000758791)'':0.00311919)'':-0.0031288,(('PDT000038888.1':0.00855705,'PDT000498022.1':0.00803675)'':0.00077636,(('PDT000252826.3':0.00740119,'PDT000488038.2':0.00768711)'':0.00434407,(('PDT000221778.3':0.0121878,'PDT000366974.2':0.00811884)'':0.00056823,('PDT000325190.2':0.00895362,(('PDT000271289.3':0.00593655,'PDT000580421.2':0.00442825)'':0.00164271,('PDT000203878.3':0.00917263,('PDT000146507.3':0.00821747,(('PDT000329872.2':0.00532126,'PDT000465594.2':0.00605714)'':0.000948646,('PDT000248554.1':0.00728784,('PDT000105637.3':0.00801627,((('PDT000602358.2':0.0041904,'PDT000084028.3':0.00368597)'':0.00219575,('PDT000366107.2':0.00644818,('PDT000049611.3':0.00425317,('PDT000102812.3':0.00283316,('PDT000539969.2':0.00265402,'PDT000083993.3':0.00342834)'':0.000464274)'':0.000639326)'':0.00137509)'':6.2897e-05)'':0.000122406,('PDT000547354.2':0.00929484,('PDT000385930.2':0.00771944,(('PDT000594813.2':0.00573391,'PDT000114690.3':0.00641389)'':0.00107607,('PDT000119764.3':0.00652832,(('PDT000189647.3':0.00627953,('PDT000184453.3':0.00472306,'PDT000189557.3':0.00384438)'':0.000558405)'':0.000732838,(('PDT000069912.3':0.00475764,('PDT000128667.3':0.00571871,('PDT000289053.3':0.00397856,('PDT000271302.3':0.00396772,(('PDT000325450.2':0.0041219,'PDT000554121.2':0.00296747)'':0.000278094,('PDT000069406.3':0.0037261,('PDT000383971.2':0.00248228,'PDT000126257.3':0.00210716)'':0.00095645)'':9.74974e-05)'':0.000969361)'':0.000662216)'':0.000237538)'':0.000141331)'':-0.000506593,(('PDT000555939.2':0.00465515,('PDT000137573.3':0.0046328,('PDT000492068.2':0.00295225,'PDT000096843.3':0.0027341)'':0.000953166)'':0.00122801)'':0.000167081,(('PDT000048204.3':0.00453465,('PDT000049363.3':0.00373303,('PDT000101109.3':0.00276956,'PDT000130440.3':0.00272001)'':0.000750227)'':0.000202919)'':0.000164384,(('PDT000583927.2':0.00272641,'PDT000139227.3':0.00275933)'':0.00167878,('PDT000264374.1':0.00466134,(('PDT000510816.2':0.00371619,'PDT000124962.3':0.00326566)'':0.000256383,('PDT000488239.1':0.00369227,('PDT000313389.3':0.00392728,(('PDT000630994.2':0.00225471,'PDT000112515.3':0.00223541)'':0.000504415,('PDT000491416.2':0.00222285,('PDT000707360.1':0.00245854,('PDT000124875.3':0.00180793,('PDT000492179.2':0.00111323,'PDT000492192.2':0.00108501)'':0.000886829)'':0.000140199)'':0.00030236)'':0.00010951)'':0.0010929)'':0.000564087)'':-0.000229466)'':0.000974464)'':-0.000690331)'':-3.83119e-05)'':0.00100627)'':-0.000205904)'':0.000280336)'':0.00180869)'':-0.000718397)'':0.000601511)'':0.000251447)'':-0.00152422)'':0.00199541)'':0.00044102)'':-0.00135697)'':0.000612595)'':0.00143598)'':-0.00100398)'':0.00178065)'':-0.00112872)'':0.000855593)'':0.00110007)'':-0.001928)'':0.004211)'':-0.00124558)'':-0.00135942)'':0.00657594)'':-0.00281959)'':0.00391042)'':-0.00493588)'':0.00184116)'':0.00322624)'':-0.0049068)'':0.00341045)'':-0.00373927)'':0.000880074)'':0.000619382)'':-6.10811e-05)'':0.00571495)'':-0.00292324)'':0.00297136)'':-8.00702e-05)'':0.00303691)'':-0.00442176,(('PDT000573766.2':0.0477772,('PDT000264537.1':0.0294974,('PDT000492145.2':0.0233029,(('PDT000492069.2':0.0130886,'PDT000262770.1':0.0100517)'':0.00153885,('PDT000150695.3':0.0129661,('PDT000151172.3':0.0115297,('PDT000051239.3':0.00585482,'PDT000492062.2':0.00501038)'':0.00172175)'':0.00318271)'':0.00325392)'':0.00450658)'':0.00709232)'':0.00215286)'':-0.000193873,((('PDT000171588.3':0.0299935,'PDT000124990.3':0.0234461)'':0.00309047,(('PDT000237115.3':0.0185062,'PDT000285867.3':0.0198603)'':0.00481907,(((('PDT000468509.2':0.014273,'PDT000264327.1':0.017484)'':0.00382829,('PDT000598733.1':0.0231641,('PDT000309332.3':0.012506,('PDT000242800.3':0.017148,('PDT000280729.3':0.00981634,('PDT000250886.3':0.0110376,'PDT000469984.2':0.00786043)'':0.00233656)'':0.00106578)'':0.00215679)'':0.00429405)'':0.000837797)'':0.00227567,(('PDT000135914.3':0.0244362,(('PDT000600715.2':0.018345,'PDT000309668.3':0.0144403)'':0.00402503,('PDT000190672.3':0.0112902,'PDT000300558.1':0.0104985)'':0.0089765)'':0.00343099)'':0.000750407,('PDT000566266.2':0.0299992,(('PDT000205415.1':0.0135116,(('PDT000351826.2':0.0125588,'PDT000366207.2':0.0125475)'':0.000598453,('PDT000619736.2':0.0114696,('PDT000239610.3':0.00867289,('PDT000329256.2':0.00541347,'PDT000602567.2':0.00463833)'':0.00195606)'':0.00334979)'':0.000520211)'':0.00261937)'':-0.000397656,(('PDT000564124.2':0.0133924,('PDT000468223.2':0.0120718,('PDT000221935.3':0.00963968,'PDT000485994.2':0.0110493)'':0.000619502)'':0.00220116)'':0.000415451,(('PDT000532753.2':0.0126556,'PDT000066897.3':0.012894)'':0.00470903,(('PDT000082141.3':0.0126182,('PDT000065513.3':0.01078,'PDT000125017.3':0.0116975)'':0.000740266)'':0.00398499,((('PDT000214050.3':0.013275,'PDT000082139.3':0.0112892)'':0.00077377,('PDT000227568.3':0.00777399,('PDT000244041.3':0.00476375,'PDT000375952.2':0.00578875)'':0.00269856)'':0.00352278)'':0.000831392,('PDT000234184.3':0.0111761,(('PDT000386247.2':0.00879831,'PDT000607746.2':0.00905179)'':0.0020193,('PDT000492051.2':0.0152993,('PDT000357185.2':0.0119531,('PDT000430569.2':0.0103652,('PDT000124772.3':0.00862803,('PDT000221711.3':0.0114708,('PDT000468174.2':0.00730372,('PDT000125484.3':0.0073373,('PDT000146453.3':0.00721402,('PDT000222815.3':0.0062519,(('PDT000620715.2':0.00559941,'PDT000096869.3':0.00644999)'':0.000576721,('PDT000231226.3':0.00454137,('PDT000029747.3':0.00497383,('PDT000307517.3':0.00340384,('PDT000212422.3':0.0043405,('PDT000620183.2':0.00339232,('PDT000145390.3':0.00274079,('PDT000386848.2':0.00226447,'PDT000125557.3':0.0029236)'':0.000652108)'':0.000112423)'':0.000644648)'':0.000259073)'':0.000739644)'':0.000317837)'':0.000166962)'':0.000856058)'':0.000455882)'':0.00125961)'':0.000330756)'':0.000807649)'':0.000314726)'':0.0014957)'':0.0011937)'':0.000575361)'':-0.0022771)'':0.00316844)'':-0.00240186)'':0.00256603)'':0.00265904)'':-0.00227694)'':-0.00064842)'':0.0116826)'':-0.00623095)'':-0.000463286)'':-0.00116556,(((('PDT000492259.2':0.0172063,'PDT000238308.3':0.0159565)'':0.00107108,('PDT000286440.1':0.0176372,(('PDT000300535.1':0.0131634,'PDT000313453.1':0.0186377)'':0.00277015,('PDT000594872.2':0.00982405,'PDT000616656.2':0.00805435)'':0.00502645)'':0.00229249)'':0.00256328)'':0.00256999,((('PDT000405473.2':0.0165494,('PDT000127481.3':0.0110154,'PDT000264392.1':0.0095122)'':0.00595898)'':0.000749303,(('PDT000148557.3':0.0102591,'PDT000250790.3':0.00964976)'':0.00592672,('PDT000102903.3':0.0130358,('PDT000101373.3':0.0112174,('PDT000184469.3':0.00973702,'PDT000127387.3':0.0111624)'':0.00321153)'':0.00124193)'':0.000587788)'':0.00209037)'':0.000163818,('PDT000264320.1':0.0245289,((('PDT000548313.2':0.0124816,('PDT000146538.3':0.00522774,'PDT000644012.2':0.00502306)'':0.00454788)'':0.00125443,('PDT000357154.2':0.0123464,('PDT000162298.3':0.00884991,('PDT000309620.3':0.00682088,'PDT000362586.2':0.0101754)'':0.00221519)'':0.00549757)'':-0.000505785)'':0.00136567,(('PDT000141155.3':0.0176818,('PDT000691362.1':0.0132689,('PDT000048666.3':0.0119873,('PDT000394730.2':0.0089981,'PDT000708184.1':0.0065305)'':0.0041024)'':0.00116793)'':0.00266538)'':-0.000735315,('PDT000042362.3':0.00953992,('PDT000707404.1':0.0108222,('PDT000264558.1':0.00639221,('PDT000086299.3':0.00620692,'PDT000707417.1':0.00621848)'':0.00100864)'':0.00238857)'':0.000221735)'':0.00278577)'':0.0010286)'':0.0102202)'':-0.00726982)'':0.00193647)'':-0.0001904,(('PDT000311094.3':0.0172284,('PDT000539921.2':0.00965804,'PDT000083982.3':0.0101597)'':0.00410283)'':0.00286866,(('PDT000532756.2':0.00862034,'PDT000545775.2':0.0102833)'':0.0047626,('PDT000250880.3':0.0172552,('PDT000639707.2':0.0180106,(((('PDT000201226.3':0.00853506,'PDT000137592.3':0.0102578)'':0.00103942,('PDT000225455.3':0.00839942,('PDT000285874.3':0.00612484,('PDT000051377.3':0.00630375,'PDT000488026.2':0.00534155)'':0.00171786)'':0.00200083)'':0.00123426)'':0.0013034,(('PDT000588257.2':0.00798454,'PDT000113773.3':0.00695566)'':0.00409492,(('PDT000168316.3':0.0103963,'PDT000319682.2':0.00908677)'':0.00115536,('PDT000679467.1':0.00576472,('PDT000212376.3':0.0051131,'PDT000571040.2':0.00476517)'':0.00143679)'':0.00284153)'':0.00174042)'':-0.00018589)'':0.00154139,(('PDT000038865.1':0.0145842,('PDT000366214.2':0.00939304,('PDT000107737.3':0.0082541,'PDT000128440.3':0.0059733)'':0.00161206)'':0.00700901)'':-0.00253594,(('PDT000084051.3':0.0129214,'PDT000705906.1':0.0123675)'':0.000311304,(('PDT000303554.3':0.012598,('PDT000351590.2':0.00708163,'PDT000490333.2':0.00844077)'':0.00358414)'':0.000879608,('PDT000491923.2':0.0109421,('PDT000414395.2':0.0100107,('PDT000593603.2':0.00977759,('PDT000127769.3':0.0117741,('PDT000465592.2':0.009695,(('PDT000492059.2':0.00507537,'PDT000488244.1':0.00579423)'':0.00151911,(('PDT000708528.1':0.00668609,'PDT000264510.1':0.00643391)'':0.00116378,(('PDT000492162.2':0.00515388,'PDT000492211.2':0.00529592)'':0.0012322,('PDT000392377.2':0.00618018,'PDT000679520.1':0.00663782)'':0.00150522)'':0.000252047)'':0.000456165)'':0.0014974)'':0.000500116)'':0.000568509)'':0.000676956)'':0.000402885)'':0.000519573)'':0.00194065)'':-0.00071766)'':0.000280643)'':0.00315071)'':0.000504072)'':-0.000671374)'':-0.000391453)'':0.0011845)'':0.00123502)'':0.00409378)'':0.000649753)'':-0.00250792,((('PDT000538887.2':0.0215271,'PDT000668116.2':0.0187321)'':0.00585575,(('PDT000373215.2':0.0216474,('PDT000110749.3':0.0188239,('PDT000097023.3':0.0182475,('PDT000047529.3':0.0170866,('PDT000144501.3':0.0150671,('PDT000583542.2':0.0127598,('PDT000274047.3':0.00896468,('PDT000224227.3':0.00672565,'PDT000386827.2':0.00679995)'':0.00224937)'':0.00455702)'':0.00136596)'':0.00199238)'':0.00137523)'':0.000844306)'':0.000342791)'':-0.000827901,(('PDT000645957.2':0.0284933,('PDT000579636.2':0.0225205,(('PDT000307882.3':0.0056884,'PDT000564056.2':0.0068477)'':0.00543957,('PDT000646926.2':0.013546,('PDT000114650.3':0.0111868,('PDT000421007.2':0.00525351,'PDT000128449.3':0.00790959)'':0.00453088)'':0.0014432)'':0.00208001)'':0.00615795)'':0.00540803)'':-0.00539118,(('PDT000213933.3':0.0224748,('PDT000260785.3':0.0150729,'PDT000707464.1':0.0159738)'':0.0115439)'':0.00678818,('PDT000111106.1':0.033052,(('PDT000163019.4':0.015021,('PDT000564861.2':0.0126812,'PDT000032319.1':0.0140806)'':0.00316486)'':0.00247336,(('PDT000584823.2':0.0198978,((('PDT000083262.3':0.0114602,'PDT000006785.2':0.012373)'':0.00320645,('PDT000264401.1':0.0154563,'PDT000264516.1':0.0148467)'':0.000483029)'':0.000568363,('PDT000264326.1':0.0164442,('PDT000264531.1':0.0160627,('PDT000237108.3':0.00842206,('PDT000707421.1':0.00603816,'PDT000135815.3':0.00492674)'':0.00268529)'':0.00318871)'':0.00393011)'':0.000608865)'':0.00429886)'':-0.00271428,(('PDT000190350.3':0.0183607,('PDT000334179.2':0.0134983,'PDT000351794.2':0.0132407)'':0.00229207)'':0.00504252,((('PDT000221931.3':0.00688642,'PDT000052119.3':0.00857958)'':0.00389372,('PDT000224223.3':0.0142828,('PDT000523859.2':0.0132926,('PDT000510831.2':0.00875821,('PDT000316116.2':0.00599474,('PDT000362562.2':0.00540995,('PDT000708616.1':0.00471293,('PDT000184434.3':0.00423646,'PDT000359229.2':0.00334232)'':0.000814034)'':0.00139319)'':0.000817531)'':0.0011439)'':0.00349093)'':0.00218591)'':-0.00123623)'':0.00179265,(('PDT000498000.1':0.017245,('PDT000541950.2':0.00938765,('PDT000580715.2':0.00707591,('PDT000264546.1':0.00568481,('PDT000125526.3':0.00389816,('PDT000708555.1':0.00373304,'PDT000264560.1':0.0037408)'':0.000989294)'':0.00158309)'':0.00101599)'':0.00226808)'':0.00495393)'':-0.00294962,((('PDT000264407.1':0.0113656,('PDT000264536.1':0.00655316,('PDT000264393.1':0.0033793,'PDT000264554.1':0.00319965)'':0.00338357)'':0.00194803)'':0.00513264,('PDT000498004.1':0.00916936,('PDT000113670.4':0.00743267,('PDT000125270.4':0.00597063,('PDT000264526.1':0.00449178,('PDT000264439.1':0.00313986,('PDT000048673.3':0.00239144,('PDT000286449.1':0.00211272,'PDT000286466.1':0.00198032)'':0.000701096)'':0.00104265)'':0.00101324)'':0.0015244)'':0.00208737)'':0.00301978)'':-0.00075682)'':0.00131683,(('PDT000231227.3':0.0163516,('PDT000146533.3':0.00732462,'PDT000351788.2':0.00602868)'':0.00871066)'':0.0025936,(('PDT000213221.3':0.00902179,'PDT000492219.2':0.0118239)'':0.0164404,(('PDT000481012.2':0.00836396,'PDT000125518.3':0.011039)'':0.00540517,(('PDT000637184.2':0.013641,('PDT000492188.2':0.00684658,(('PDT000492086.2':0.0086271,('PDT000163369.4':0.00372474,'PDT000108018.3':0.00434862)'':0.00125177)'':0.00219539,('PDT000312487.3':0.00618868,('PDT000264523.1':0.00586688,'PDT000313451.1':0.00547192)'':0.000790073)'':0.00125525)'':0.000709897)'':0.00285435)'':-0.00122907,('PDT000238320.3':0.0113511,(('PDT000234226.3':0.00858955,('PDT000405470.2':0.00581319,'PDT000707454.1':0.00674791)'':0.0023094)'':0.000913585,('PDT000451179.2':0.0106641,((('PDT000523239.2':0.00547689,('PDT000411990.2':0.00521111,('PDT000492042.2':0.00350063,'PDT000492197.2':0.0036764)'':0.00175787)'':0.00106438)'':0.00137652,('PDT000222766.3':0.0076018,('PDT000313475.1':0.00854003,('PDT000235058.3':0.00520245,(('PDT000352821.2':0.00584663,('PDT000231271.3':0.00382193,'PDT000212336.3':0.00276621)'':0.000918663)'':0.00022773,('PDT000639328.2':0.00322151,('PDT000556736.2':0.00319359,('PDT000264495.1':0.00398455,(('PDT000209176.3':0.00220913,'PDT000593806.2':0.00268158)'':0.000100336,('PDT000492041.2':0.00228587,'PDT000107738.3':0.00210842)'':0.000844489)'':0.000779818)'':0.00024987)'':0.000411391)'':0.000667484)'':0.000722362)'':0.00133389)'':0.00113588)'':-0.00157685)'':0.00086259,('PDT000510683.2':0.012712,(('PDT000523284.2':0.0066082,'PDT000264460.1':0.0072226)'':0.00165624,(('PDT000216667.3':0.00524798,('PDT000486046.2':0.00448321,'PDT000646241.2':0.00518296)'':0.00232264)'':0.00139266,('PDT000564856.2':0.00941804,(('PDT000146564.3':0.00648272,'PDT000253177.3':0.00507678)'':0.00207528,('PDT000264432.1':0.00838436,(('PDT000226798.3':0.00879632,('PDT000113778.3':0.00393394,('PDT000214046.3':0.00311907,('PDT000550068.2':0.0030937,('PDT000492112.2':0.00394983,'PDT000494611.2':0.00332616)'':0.000598511)'':0.000276463)'':0.00045941)'':0.00113025)'':9.91134e-05,('PDT000132638.3':0.00542656,('PDT000708527.1':0.00794101,('PDT000300519.1':0.00497352,('PDT000492040.2':0.00410127,('PDT000569973.2':0.00396964,('PDT000492194.2':0.00426664,(('PDT000238397.3':0.00173223,'PDT000491922.2':0.00236163)'':0.000534697,('PDT000264470.1':0.0027548,('PDT000494842.2':0.00238641,'PDT000124983.3':0.00220349)'':0.000441464)'':4.7292e-05)'':0.000707963)'':0.000634797)'':0.000208477)'':0.0010844)'':0.000381371)'':0.000262622)'':-5.07611e-05)'':0.00180593)'':-0.000515644)'':0.00337039)'':-0.00257603)'':0.000401911)'':0.00374117)'':-0.00389166)'':0.00420066)'':-0.00238133)'':0.00377181)'':-0.00387815)'':0.0040359)'':0.00106651)'':0.000149446)'':-0.00399)'':0.000535387)'':0.00145199)'':0.00527938)'':-0.00318437)'':0.00212565)'':0.0084648)'':-0.00414545)'':-0.00372092)'':0.000493885)'':0.00380736)'':-0.00547973,((('PDT000319303.2':0.0211079,'PDT000503118.2':0.0218586)'':0.00330444,('PDT000275554.1':0.024282,('PDT000275552.1':0.0238429,'PDT000275553.1':0.0167818)'':0.00276288)'':0.00449073)'':0.00270829,((('PDT000319233.2':0.0221661,'PDT000327085.2':0.0207931)'':0.0051968,(('PDT000190327.3':0.0139409,'PDT000707456.1':0.0132599)'':0.00278841,('PDT000407676.2':0.0207723,('PDT000110453.3':0.0232816,((('PDT000242773.3':0.00928244,'PDT000307857.3':0.0110126)'':0.00149733,(('PDT000125024.3':0.00957445,('PDT000222031.3':0.00819961,('PDT000386817.2':0.00606508,'PDT000451212.2':0.00716742)'':0.00320134)'':0.00125301)'':0.00190867,('PDT000123283.1':0.00983743,('PDT000384015.2':0.0080894,(('PDT000113668.4':0.00648318,'PDT000124923.3':0.00545232)'':0.00090737,('PDT000503422.2':0.00680719,('PDT000131113.3':0.0054316,('PDT000492046.2':0.00586353,('PDT000367744.2':0.00543182,'PDT000417287.2':0.00742188)'':0.000801171)'':0.000443144)'':0.000716032)'':-0.000164225)'':0.00137566)'':0.00139796)'':-0.000696909)'':0.0018659)'':-7.34977e-05,(('PDT000566523.2':0.00872463,'PDT000084224.3':0.0116673)'':0.00314039,(('PDT000114765.3':0.00809768,('PDT000264434.1':0.0083163,'PDT000264469.1':0.0077695)'':0.000848268)'':0.00101848,(('PDT000639742.2':0.00811733,('PDT000214065.3':0.00735878,('PDT000209172.3':0.00719393,'PDT000119763.3':0.00662177)'':0.000578665)'':0.000999401)'':0.000584835,('PDT000038858.1':0.00969983,(('PDT000242751.3':0.00687218,'PDT000102890.3':0.00586032)'':0.00181739,('PDT000147969.3':0.00623765,(('PDT000264309.1':0.0065023,'PDT000264448.1':0.0050423)'':0.00062337,('PDT000264338.1':0.0043194,'PDT000264360.1':0.00376201)'':0.00147234)'':0.00144901)'':0.00107567)'':0.00279596)'':-0.0017168)'':0.000788265)'':0.00464636)'':-0.00313711)'':0.00647983)'':0.0012799)'':-0.00343558)'':0.00667608)'':-0.00788499,(('PDT000562993.2':0.0249174,('PDT000492037.2':0.0144634,'PDT000542700.2':0.0155198)'':0.00913186)'':-0.000881453,((('PDT000135730.3':0.0203484,('PDT000600460.2':0.0188824,('PDT000494260.2':0.0117213,('PDT000641591.2':0.00531594,('PDT000312077.3':0.0050726,('PDT000564138.2':0.00291767,('PDT000247557.3':0.00219854,'PDT000125035.3':0.00239228)'':0.00057101)'':0.00163198)'':0.00182507)'':0.0033892)'':0.00579451)'':0.00119603)'':-0.00411809,('PDT000083265.3':0.0211984,('PDT000509948.2':0.0172193,('PDT000264727.3':0.0183598,('PDT000233430.3':0.0106468,('PDT000264371.1':0.0189316,('PDT000342316.2':0.0130178,('PDT000631010.2':0.0073199,(('PDT000042360.3':0.00693052,'PDT000264550.1':0.00707428)'':0.0034833,('PDT000360364.2':0.00981062,('PDT000083228.3':0.00748162,('PDT000710842.1':0.0058345,(('PDT000631011.2':0.00365499,('PDT000223285.3':0.00340475,'PDT000137655.3':0.00357919)'':0.000528165)'':0.000949687,('PDT000491438.2':0.00476439,('PDT000329338.2':0.00476834,('PDT000133839.3':0.00571866,('PDT000065508.3':0.00519433,('PDT000544421.2':0.00349432,('PDT000336818.2':0.00429191,('PDT000708559.1':0.0044196,('PDT000491500.2':0.00251604,('PDT000227518.3':0.00255592,('PDT000577466.2':0.00250019,('PDT000150985.3':0.00172624,'PDT000707377.1':0.00156915)'':0.00118834)'':0.000173849)'':0.000207057)'':0.000505865)'':0.000553546)'':0.000191351)'':0.000202211)'':0.000757457)'':0.000224061)'':0.00054871)'':-0.000658015)'':0.00125918)'':0.00193832)'':0.000313621)'':-0.00111753)'':0.00172802)'':0.00124821)'':0.00172298)'':0.00032229)'':0.0017335)'':0.000951503)'':0.00171261)'':-0.00338285)'':0.00164134,(('PDT000497777.1':0.0230232,(('PDT000497768.1':0.00905493,'PDT000497819.1':0.00810607)'':0.00292846,('PDT000498190.1':0.0136322,('PDT000497773.1':0.0119684,('PDT000498076.1':0.00807537,'PDT000498077.1':0.00732603)'':0.00349356)'':0.000777355)'':0.00122737)'':0.00619908)'':-0.00228962,(('PDT000457359.2':0.014397,'PDT000534068.2':0.0151193)'':0.00967707,(('PDT000409250.2':0.0144218,('PDT000108884.3':0.0126197,'PDT000123296.1':0.0115053)'':0.00183601)'':0.00256675,('PDT000718917.1':0.0149309,(('PDT000286463.1':0.00957137,('PDT000157465.3':0.00952577,'PDT000195713.3':0.00949543)'':0.00113078)'':0.000403975,(('PDT000223300.3':0.0126234,('PDT000344833.2':0.0123136,(('PDT000097093.3':0.00606072,('PDT000150125.3':0.00408704,'PDT000225336.3':0.0043791)'':0.00139081)'':0.00277956,('PDT000123294.1':0.0121379,('PDT000264330.1':0.00949697,('PDT000254243.3':0.00798931,('PDT000123338.1':0.0066055,(('PDT000205376.3':0.00597624,('PDT000491429.2':0.005523,'PDT000264467.1':0.0056168)'':0.000889511)'':0.000592868,('PDT000538643.2':0.00668575,('PDT000407652.2':0.00659659,('PDT000366147.2':0.00491982,('PDT000146469.3':0.00572497,'PDT000190130.3':0.00521703)'':0.000370977)'':0.000800021)'':0.000581063)'':0.000152153)'':0.00103018)'':0.000411348)'':0.000647284)'':0.000751902)'':-0.000905507)'':0.00152423)'':0.00279323)'':-0.00302325,(('PDT000702606.1':0.00994727,('PDT000536999.2':0.0107069,'PDT000599143.2':0.010987)'':0.000932482)'':0.00168424,('PDT000226886.3':0.0113715,(('PDT000570407.2':0.00608845,('PDT000679285.1':0.00465407,'PDT000096910.3':0.00541473)'':0.00165415)'':0.00119298,(('PDT000598725.1':0.00416366,'PDT000598804.1':0.00430163)'':0.00406887,('PDT000141679.3':0.0113496,('PDT000115434.3':0.0105204,(('PDT000221628.3':0.0064199,'PDT000565055.2':0.005723)'':0.000426859,(('PDT000417223.2':0.00574667,'PDT000465610.2':0.00451433)'':0.0015543,('PDT000707415.1':0.0105606,('PDT000289825.3':0.0050907,((('PDT000240631.3':0.00338599,'PDT000521361.2':0.00449195)'':0.000404135,('PDT000264538.1':0.00396171,('PDT000124886.3':0.0041891,('PDT000707414.1':0.00186323,'PDT000707458.1':0.00192994)'':0.00109526)'':0.000651645)'':0.000276045)'':3.20301e-05,('PDT000189540.3':0.00528806,(('PDT000610106.2':0.00275092,'PDT000660091.2':0.0027381)'':0.000326271,('PDT000386286.2':0.00245245,('PDT000188048.3':0.00180913,'PDT000616514.2':0.00218568)'':0.000342923)'':0.0012184)'':0.000847681)'':2.28696e-05)'':0.00178241)'':0.00033815)'':-0.000592525)'':0.00120312)'':0.00279128)'':0.00106868)'':-0.00117511)'':-0.000197541)'':0.00368342)'':-0.00100119)'':-0.00216664)'':0.00168535)'':0.00680317)'':-0.00310916)'':0.00367307)'':-0.00191453)'':-0.00240282)'':0.00534148)'':-0.00485741)'':0.011477)'':-0.009664)'':0.00147666)'':0.00758693)'':-0.00803589)'':0.00637527)'':0.00161076)'':0.00923936)'':-0.0143244)'':0.00419537)'':-0.000235033)'':0.0011112)'':0.0120546)'':-0.0111716)'':0.0191702)'':-0.00607396)'':0.0219318)'':0.0044356)'':-0.0189786)'':0.00106384)'':0.0251738)'':-0.00935586)'':0.0314612)'':0.00126032)'':0.00980914)'':0.0600074)'':0.0148705)'':0.000744528)'':0.00660609,(((((('PDT000181769.2':0.0712246,('PDT000077081.2':0.0673096,'PDT000444814.1':0.0536444)'':0.0128904)'':0.103626,('PDT000538618.1':0.170425,('PDT000548265.2':0.147286,(('PDT000257768.2':0.0982008,'PDT000126250.2':0.0839742)'':0.0011796,('PDT000486057.2':0.0559222,'PDT000523083.2':0.0665148)'':0.0311391)'':0.0591153)'':0.014384)'':0.0272911)'':0.0130586,(('PDT000537833.1':0.0562156,('PDT000626012.1':0.044462,('PDT000689948.1':0.0155259,'PDT000361011.1':0.0106667)'':0.0214342)'':0.00577445)'':0.0735649,('PDT000394944.1':0.0960478,('PDT000719110.1':0.0793634,(('PDT000417957.1':0.0377287,'PDT000395432.1':0.0383698)'':0.0101274,('PDT000639510.1':0.042651,'PDT000395221.1':0.0373913)'':0.0153941)'':0.0375692)'':0.00897685)'':0.0255808)'':0.0711818)'':0.0080299,((('PDT000661148.1':0.121546,('PDT000423996.1':0.102486,'PDT000535497.1':0.0763064)'':0.0239596)'':-0.00354704,((('PDT000014103.3':0.0910979,('PDT000583384.1':0.026531,'PDT000332168.1':0.0307064)'':0.0539214)'':0.00396527,((('PDT000014129.3':0.0328905,('PDT000014107.3':0.00685222,'PDT000014146.2':0.00559428)'':0.0331582)'':0.0597711,('PDT000656964.1':0.088148,(('PDT000652387.1':0.0615832,('PDT000202179.2':0.057035,'PDT000637079.1':0.053607)'':0.0082393)'':0.00139622,(('PDT000215770.2':0.0298305,'PDT000206241.3':0.0371965)'':0.00495357,('PDT000206255.3':0.0200809,('PDT000208413.2':0.0184217,'PDT000215769.2':0.029004)'':0.00313568)'':0.0163539)'':0.0261127)'':0.00787737)'':0.00795184)'':-0.0021116,(('PDT000645528.2':0.0389893,'PDT000645538.2':0.0391773)'':0.0422684,('PDT000201002.2':0.0789273,('PDT000215765.2':0.0736817,(('PDT000706522.1':0.0312867,'PDT000285341.1':0.0237492)'':0.0390304,('PDT000202687.2':0.0577737,(('PDT000216265.2':0.0496385,('PDT000215763.2':0.0350607,'PDT000534976.1':0.0400816)'':0.00979039)'':0.0053428,('PDT000571157.1':0.0573242,(('PDT000215760.2':0.0398086,'PDT000557659.2':0.0452362)'':0.00899067,('PDT000616558.1':0.0345944,'PDT000656999.1':0.0378134)'':0.0150624)'':0.00353381)'':0.000204851)'':0.00479057)'':0.000236836)'':0.00453214)'':0.00301077)'':0.000267224)'':0.000759931)'':0.00787512)'':-0.00308892,(('PDT000048128.1':0.0818443,('PDT000423932.1':0.0662669,'PDT000429813.1':0.0570651)'':0.00588817)'':0.0230618,(('PDT000485920.1':0.0951051,('PDT000203395.2':0.0676331,'PDT000290515.2':0.0720799)'':0.0229864)'':-0.00574836,(((('PDT000537874.1':0.0374499,('PDT000548979.1':0.0352233,'PDT000345921.1':0.0318641)'':0.00814663)'':0.010507,('PDT000712637.1':0.101263,('PDT000712629.1':0.0353535,('PDT000212793.1':0.0361538,('PDT000289875.1':0.0177646,'PDT000289891.1':0.0113075)'':0.00784479)'':0.0126375)'':0.0261892)'':0.00539399)'':0.00822174,(('PDT000188084.2':0.0578686,'PDT000536573.1':0.0618734)'':0.003067,('PDT000672227.1':0.0600427,(('PDT000346556.1':0.0230156,'PDT000414208.1':0.0418638)'':0.0223802,('PDT000233276.2':0.0404361,('PDT000346608.1':0.0410653,('PDT000520846.1':0.0300507,'PDT000124656.1':0.0287786)'':0.00555585)'':0.00220371)'':0.00507631)'':0.00946382)'':-0.000171334)'':-0.000150308)'':0.0114219,('PDT000617961.1':0.082949,('PDT000423985.1':0.0825875,('PDT000448182.1':0.0682768,((('PDT000188082.2':0.048025,'PDT000346875.1':0.0472268)'':0.00755175,(('PDT000171803.1':0.0551586,('PDT000030556.1':0.0367388,'PDT000319000.1':0.0360183)'':0.00630577)'':0.00404212,('PDT000720093.1':0.04455,('PDT000673698.1':0.0381554,('PDT000321767.1':0.0180404,'PDT000372596.1':0.0247685)'':0.00883597)'':0.00411248)'':0.008572)'':0.00338989)'':-0.000790677,(('PDT000337004.1':0.05365,'PDT000045615.1':0.054117)'':0.00213208,('PDT000208414.2':0.0359727,('PDT000181774.2':0.0482152,('PDT000443867.1':0.0486589,('PDT000535156.1':0.0301397,('PDT000104797.2':0.0260559,('PDT000290526.2':0.0232942,'PDT000502401.1':0.0182865)'':0.00491735)'':0.00325342)'':0.00436967)'':0.00183114)'':0.00312613)'':0.00629784)'':-0.000645223)'':0.00854159)'':0.00557093)'':0.0148203)'':-0.0153373)'':0.00633519)'':0.0120603)'':-0.00885303)'':0.0115733)'':0.0590374,((('PDT000644843.1':0.114342,(('PDT000034504.2':0.0407001,'PDT000399669.1':0.0460096)'':0.0336331,('PDT000656952.1':0.0965583,('PDT000517941.1':0.0356907,'PDT000562139.1':0.0266186)'':0.045056)'':0.00019919)'':0.0310026)'':0.059146,('PDT000252487.1':0.0758689,('PDT000502232.1':0.0591184,(('PDT000234087.2':0.0291308,'PDT000544783.1':0.0331809)'':0.0204275,('PDT000619641.1':0.0393949,'PDT000675435.1':0.0454046)'':0.00301728)'':0.0170104)'':0.0099614)'':0.0858633)'':0.0031639,((('PDT000109424.2':0.136351,'PDT000125962.1':0.126999)'':0.00109483,((('PDT000289895.1':0.0593004,'PDT000346912.1':0.0456476)'':0.0468988,('PDT000709811.1':0.0998282,('PDT000168851.1':0.100571,('PDT000656941.1':0.0876906,('PDT000716820.1':0.0841008,('PDT000064571.1':0.0654747,('PDT000627359.1':0.0731735,('PDT000346910.1':0.0546151,'PDT000346911.1':0.0546259)'':0.0016695)'':0.00525879)'':0.00651885)'':0.0072342)'':0.00230899)'':0.006196)'':0.0178772)'':-0.00827558,(((('PDT000668940.1':0.0407537,('PDT000321764.1':0.0309449,'PDT000106251.2':0.0306876)'':0.0158033)'':0.0045349,('PDT000673694.1':0.0443887,('PDT000675344.1':0.0343834,('PDT000632325.1':0.0443371,('PDT000448178.1':0.0362302,('PDT000421807.1':0.0241391,'PDT000589808.1':0.027193)'':0.00626507)'':0.00436002)'':0.00292883)'':0.00357316)'':0.00192954)'':0.0360992,(('PDT000329024.1':0.0862236,'PDT000329057.1':0.0830124)'':0.0128855,(('PDT000651105.1':0.0947717,('PDT000622419.1':0.0601368,('PDT000202710.2':0.0518638,('PDT000362412.1':0.0457341,'PDT000716824.1':0.0445736)'':0.00239186)'':0.00633918)'':0.0100676)'':0.00515708,('PDT000328686.1':0.0876175,(('PDT000072464.1':0.082168,'PDT000457709.1':0.063944)'':0.00205984,(('PDT000289919.1':0.0726169,('PDT000637073.1':0.053816,'PDT000712183.1':0.050212)'':0.0135761)'':-0.00103262,(('PDT000336920.1':0.0602222,('PDT000150955.2':0.0436137,'PDT000535529.1':0.0397518)'':0.017368)'':0.00444533,('PDT000396756.1':0.0584191,('PDT000584551.1':0.0571656,('PDT000535471.1':0.0404016,'PDT000636388.1':0.0402745)'':0.00929783)'':0.00335246)'':0.00271419)'':0.00208209)'':0.00465468)'':0.00745325)'':-0.00526032)'':0.00930389)'':-0.00458075)'':-5.19403e-05,(('PDT000468924.1':0.0641024,('PDT000627211.1':0.0472542,(('PDT000479850.1':0.00418625,'PDT000549868.1':0.00689825)'':0.0135985,('PDT000719206.1':0.0172227,'PDT000502482.1':0.0196577)'':0.003291)'':0.0132559)'':0.030954)'':0.00358844,('PDT000568689.1':0.0964607,(('PDT000517927.1':0.0374152,('PDT000042548.1':0.0219209,('PDT000287695.2':0.0172469,'PDT000427042.1':0.0192891)'':0.00246464)'':0.0174966)'':0.0247907,((('PDT000551929.1':0.0579411,'PDT000720092.1':0.0472319)'':0.0171062,('PDT000708234.1':0.06139,('PDT000202013.2':0.0474431,'PDT000425861.1':0.042698)'':0.0151309)'':0.00576034)'':0.00176353,(('PDT000621069.1':0.069723,('PDT000414005.1':0.0588769,('PDT000397864.1':0.0522095,('PDT000543135.1':0.0336703,'PDT000669032.1':0.0381819)'':0.0038354)'':0.0157219)'':0.00384977)'':0.00152692,((('PDT000589973.1':0.0692333,('PDT000426381.1':0.0600158,('PDT000521040.1':0.043922,'PDT000251697.1':0.0437687)'':0.00193481)'':0.0166246)'':0.0167218,('PDT000557058.1':0.0823212,('PDT000274551.2':0.057506,('PDT000488274.1':0.0483281,('PDT000408014.1':0.0461568,('PDT000102986.1':0.0446319,('PDT000632792.1':0.0354246,'PDT000346187.1':0.0444036)'':0.00350408)'':0.000587984)'':0.00468083)'':0.00243129)'':0.0101393)'':0.00256665)'':0.00229395,(('PDT000203335.1':0.0768101,'PDT000203677.1':0.0512309)'':0.0242769,(('PDT000346567.1':0.0740275,('PDT000417121.1':0.0521747,'PDT000661166.1':0.0541213)'':0.024334)'':0.00146675,((('PDT000283187.1':0.063819,('PDT000645544.2':0.0513809,'PDT000325631.1':0.0561941)'':0.011107)'':0.00650508,('PDT000233275.2':0.0768939,('PDT000175840.2':0.0511777,'PDT000135946.2':0.0630773)'':0.00528006)'':0.00446245)'':0.00213284,('PDT000563599.2':0.0855263,(('PDT000704988.1':0.0719773,('PDT000535340.1':0.0661456,(('PDT000201007.2':0.0614253,('PDT000468830.1':0.0613506,('PDT000661125.1':0.0420453,('PDT000216244.2':0.0348887,'PDT000433749.1':0.0258512)'':0.0109473)'':0.00891938)'':0.00125075)'':0.00134927,((('PDT000523079.1':0.03727,'PDT000714568.1':0.0447793)'':0.0107872,('PDT000637792.1':0.0379154,'PDT000252533.1':0.0414202)'':0.00540745)'':0.00248679,(('PDT000534988.1':0.0342774,'PDT000675554.1':0.0545003)'':0.00823317,('PDT000705070.1':0.0461793,('PDT000633056.1':0.0388187,('PDT000346158.1':0.0367819,('PDT000171787.1':0.0253163,'PDT000460546.1':0.0342641)'':0.00248707)'':0.00501191)'':0.00692919)'':-0.00160764)'':0.00347485)'':0.00298499)'':0.00755305)'':0.00326301)'':-0.00767874,(('PDT000423924.1':0.0543862,'PDT000702737.1':0.0408379)'':0.0141829,(('PDT000426623.1':0.0603709,'PDT000048130.1':0.0578021)'':0.0051278,(('PDT000719020.1':0.0511651,(('PDT000607471.1':0.0437443,'PDT000445122.1':0.041143)'':0.00261176,('PDT000019319.2':0.0511968,('PDT000216288.2':0.0427284,'PDT000425845.1':0.047816)'':0.00151837)'':0.00403096)'':0.00297237)'':9.06759e-05,('PDT000521143.1':0.0677117,(('PDT000340391.1':0.033333,('PDT000438305.1':0.0367905,'PDT000363565.1':0.0379327)'':0.00470943)'':0.0118672,('PDT000251689.1':0.05466,('PDT000128831.2':0.0442941,('PDT000252582.1':0.0439722,('PDT000425954.1':0.0381843,'PDT000718875.1':0.0372325)'':0.00187103)'':0.00311717)'':0.00424939)'':0.00354306)'':0.00445026)'':-0.00210217)'':0.0058812)'':0.00110683)'':-0.00156267)'':0.0132498)'':-0.00855211)'':0.00720504)'':0.00721593)'':-0.0135027)'':0.00327664)'':0.000409844)'':0.0022583)'':0.0198333)'':-0.00429355)'':-0.0072387)'':0.00835938)'':0.0483879)'':0.00877534,(((('PDT000323513.1':0.120289,'PDT000249916.1':0.110123)'':0.00442611,(('PDT000455217.1':0.0897607,'PDT000661573.1':0.0883803)'':0.00586646,('PDT000607561.1':0.11579,(('PDT000142214.2':0.0183646,'PDT000216691.1':0.0278319)'':0.0535175,('PDT000459787.1':0.069138,('PDT000333765.1':0.0648197,(('PDT000501920.1':0.0453328,('PDT000397867.1':0.0350926,'PDT000252671.1':0.030615)'':0.00961337)'':0.00831363,(('PDT000502245.1':0.026798,('PDT000235842.2':0.0223333,'PDT000598053.1':0.0237058)'':0.00397641)'':0.0146683,('PDT000668779.1':0.0423595,('PDT000350430.1':0.0356415,('PDT000477263.1':0.0282725,('PDT000469963.1':0.00893199,'PDT000502434.1':0.0105524)'':0.0207694)'':0.00425846)'':0.00470345)'':0.00224311)'':0.00896383)'':0.00571787)'':0.00558747)'':-0.0060695)'':0.0188991)'':0.0034281)'':0.0442902)'':0.00165607,(('PDT000485563.1':0.0347663,'PDT000642685.1':0.0426065)'':0.0636599,(('PDT000502185.1':0.0804319,'PDT000171832.1':0.0905681)'':0.0123177,((('PDT000445049.1':0.0680109,('PDT000358634.1':0.0295207,'PDT000409021.1':0.0219433)'':0.0191981)'':0.0406645,(('PDT000417127.1':0.0567533,'PDT000457702.1':0.0574027)'':0.00594883,('PDT000426439.1':0.0394725,('PDT000694493.1':0.025865,'PDT000243051.1':0.0444918)'':0.00810387)'':0.00907528)'':0.0411561)'':0.00100967,('PDT000233295.2':0.0979438,((('PDT000656998.1':0.0741636,('PDT000123372.1':0.0447729,'PDT000124664.1':0.0402415)'':0.0127037)'':0.0252622,('PDT000589449.1':0.0809087,('PDT000394989.1':0.0794331,('PDT000325552.1':0.0403444,'PDT000129561.1':0.0400678)'':0.0160298)'':0.00659201)'':0.00378975)'':0.00646072,(('PDT000475398.1':0.0595434,'PDT000651589.1':0.0358693)'':0.0422462,((('PDT000664894.1':0.02876,'PDT000664906.1':0.0296448)'':0.00076176,('PDT000664919.1':0.0300297,'PDT000664948.1':0.0344864)'':0.00804039)'':0.013232,('PDT000664941.1':0.0259066,('PDT000664920.1':0.00518299,('PDT000664897.1':0.00171987,'PDT000664915.1':0.00237562)'':0.00170127)'':0.0109787)'':0.0140077)'':0.0502774)'':0.0085776)'':0.00926599)'':0.00444406)'':0.0113904)'':0.00180461)'':0.0191699)'':0.00619392,((('PDT000415796.1':0.114477,('PDT000583399.1':0.040077,('PDT000423962.1':0.0420993,'PDT000424005.1':0.0416759)'':0.00572454)'':0.0760602)'':0.0035602,(('PDT000691087.1':0.0943444,('PDT000104987.4':0.0243026,'PDT000362384.1':0.0367428)'':0.0617684)'':0.0127968,(('PDT000425558.1':0.018932,'PDT000668901.1':0.0101604)'':0.0447492,('PDT000356183.1':0.0486432,'PDT000448198.1':0.0479951)'':0.00832772)'':0.0564106)'':0.0104939)'':0.0132928,((('PDT000553176.1':0.0794727,('PDT000395499.1':0.0855364,'PDT000656949.1':0.102033)'':0.0109683)'':0.0135256,(('PDT000323294.1':0.000982464,'PDT000205086.2':0.00131501)'':0.0502133,('PDT000025977.1':0.00604149,('PDT000025981.1':0.00282712,'PDT000025983.1':0.00425306)'':0.000619921)'':0.0388572)'':0.0571945)'':0.0102072,((('PDT000651140.1':0.0736705,('PDT000705908.1':0.0651143,'PDT000294406.1':0.0453927)'':0.0172845)'':0.00384481,('PDT000599900.1':0.0637242,('PDT000294405.1':0.0464294,('PDT000393056.1':0.0237798,'PDT000661114.1':0.020519)'':0.0225401)'':0.0077304)'':0.011956)'':0.0116953,('PDT000583445.1':0.131957,(((('PDT000168728.1':0.0364139,'PDT000168758.1':0.0468866)'':0.0624219,('PDT000298028.1':0.0754273,('PDT000417174.1':0.0718047,('PDT000721938.1':0.0328484,'PDT000721939.1':0.0392843)'':0.0362775)'':0.0120116)'':0.00656126)'':0.00702928,((('PDT000128877.2':0.0192223,'PDT000652354.1':0.0274603)'':0.0375383,('PDT000519817.1':0.0359952,'PDT000551893.1':0.0471027)'':0.0204282)'':0.00195053,('PDT000652340.1':0.0879414,('PDT000397626.1':0.0659287,('PDT000423951.1':0.0516295,'PDT000535270.1':0.0474548)'':0.0117737)'':0.0111428)'':-0.00425495)'':0.0121135)'':0.00768499,((('PDT000250802.2':0.0706191,'PDT000497426.1':0.0803209)'':0.00671509,('PDT000322977.2':0.0835189,('PDT000482829.1':0.0534302,('PDT000661109.1':0.0162703,'PDT000395273.1':0.0160259)'':0.021066)'':0.0181009)'':-0.000735241)'':0.00632679,(('PDT000644921.1':0.0283106,'PDT000252452.1':0.0400697)'':0.0389667,(('PDT000672652.1':0.0694834,(('PDT000439502.1':0.0427615,('PDT000343874.1':0.0251567,'PDT000228349.1':0.0292604)'':0.0199417)'':0.00711931,('PDT000025492.2':0.0536892,('PDT000688711.1':0.024187,('PDT000557669.1':0.0162846,'PDT000502160.1':0.013878)'':0.00922741)'':0.0118275)'':0.012265)'':0.0101328)'':-0.00117595,((('PDT000149348.2':0.0498418,'PDT000566218.1':0.0550472)'':0.0181126,('PDT000494265.1':0.0433206,('PDT000583450.1':0.0226366,('PDT000583449.1':0.00654581,'PDT000720246.1':0.00687499)'':0.00902092)'':0.028272)'':0.0134282)'':0.00154203,('PDT000048134.1':0.0730721,(('PDT000703275.1':0.0428527,('PDT000483198.1':0.0343006,'PDT000553184.1':0.0399592)'':0.0141211)'':-0.000450059,(('PDT000143597.1':0.037746,'PDT000249907.1':0.0367108)'':0.00828432,('PDT000372173.1':0.0393761,('PDT000438289.1':0.0418713,('PDT000420666.1':0.0183858,'PDT000518555.1':0.0183441)'':0.0172053)'':0.00331997)'':0.00290602)'':0.0042979)'':0.00903826)'':-0.00052089)'':0.00135899)'':0.010892)'':-0.00250551)'':0.00865639)'':0.0269357)'':-0.020539)'':0.0227098)'':0.0197004)'':0.000366829)'':0.000630495)'':0.025124)'':-0.0174615)'':0.0378607)'':-0.0138176,(((('PDT000061087.3':0.0182455,('PDT000320267.2':0.00941785,'PDT000457515.2':0.0103654)'':0.00318437)'':0.131152,(('PDT000519337.2':0.0183205,'PDT000136999.3':0.019495)'':0.0182629,(('PDT000158910.3':0.0172654,('PDT000305206.3':0.0092554,('PDT000239306.3':0.00529821,'PDT000281583.3':0.00796329)'':0.00222115)'':0.0113693)'':0.00177666,('PDT000548536.2':0.0214221,('PDT000597901.2':0.0182912,('PDT000324061.2':0.0147221,('PDT000503398.2':0.0104434,'PDT000060808.3':0.0115629)'':0.00282436)'':0.000618234)'':0.011164)'':-0.00281839)'':0.0064785)'':0.12303)'':0.00415977,((('PDT000661607.1':0.148127,'PDT000238069.1':0.132818)'':0.00471198,(('PDT000250153.1':0.0789914,'PDT000304202.1':0.0728666)'':0.0169805,('PDT000293547.1':0.0886704,(('PDT000720175.1':0.0613041,'PDT000497238.1':0.0657999)'':0.010302,('PDT000251480.1':0.046588,('PDT000135935.2':0.0137597,'PDT000661716.1':0.022535)'':0.0268046)'':0.011287)'':0.0221372)'':-0.0036405)'':0.0463101)'':-0.0100544,((((('PDT000557125.1':0.0386762,'PDT000228150.1':0.0404536)'':0.0106585,('PDT000228156.1':0.0334056,'PDT000228159.1':0.0311824)'':0.0288713)'':0.0127624,(('PDT000479629.1':0.051473,'PDT000594472.1':0.05147)'':0.0186545,(('PDT000600995.1':0.061834,('PDT000367258.1':0.0351548,'PDT000367710.1':0.0348849)'':0.0259982)'':0.0055946,('PDT000632007.1':0.0520163,('PDT000485416.1':0.0482405,('PDT000294549.2':0.0302105,'PDT000342669.1':0.033609)'':0.0202558)'':0.00226514)'':0.00795573)'':0.0112039)'':0.000653878)'':0.0035171,((('PDT000228008.1':0.0583631,'PDT000228032.1':0.0573989)'':0.00939013,(('PDT000045620.1':0.0167207,'PDT000048129.1':0.0178333)'':0.0491344,(('PDT000290406.2':0.0469004,'PDT000228117.1':0.0443562)'':0.0108175,('PDT000304956.2':0.037949,'PDT000045585.1':0.0415789)'':0.0154732)'':0.0135116)'':0.00671983)'':0.00189079,(('PDT000150260.2':0.0333471,('PDT000477200.1':0.026548,('PDT000417387.1':0.0266612,'PDT000626547.1':0.0263706)'':0.00399807)'':0.00721534)'':0.0196056,(('PDT000509485.1':0.0290586,'PDT000617803.1':0.0296094)'':0.0086728,('PDT000423359.1':0.0442266,('PDT000191848.2':0.0312461,'PDT000566234.1':0.0289832)'':0.0107164)'':0.00173583)'':0.0108897)'':0.0226043)'':0.00112028)'':0.0114775,('PDT000501827.1':0.124485,((('PDT000235609.2':0.0701121,'PDT000502428.1':0.0518979)'':0.0152826,(('PDT000485433.1':0.0365441,'PDT000505389.1':0.0303361)'':0.0225048,(('PDT000364427.1':0.0357662,'PDT000502227.1':0.0501113)'':0.00111056,('PDT000509510.1':0.00376666,'PDT000645047.1':0.00382423)'':0.0317075)'':0.0207234)'':0.0113386)'':0.00579664,(('PDT000691350.1':0.0841582,(('PDT000417494.1':0.0598067,('PDT000323561.1':0.0462604,'PDT000285939.2':0.0429518)'':0.00737317)'':0.00149008,('PDT000323537.1':0.0498613,('PDT000271536.2':0.040028,('PDT000285397.2':0.0227644,('PDT000317610.1':0.00936988,'PDT000386740.1':0.01066)'':0.00794096)'':0.0153522)'':0.0169548)'':-5.84509e-05)'':0.0160101)'':0.00622027,(('PDT000362881.1':0.0656624,('PDT000497374.1':0.0586939,('PDT000095826.2':0.0277149,'PDT000525369.1':0.0296279)'':0.0354817)'':0.00512661)'':0.0028812,((('PDT000691358.1':0.0514329,('PDT000196628.2':0.0347894,'PDT000060528.2':0.0393063)'':0.00654271)'':0.00907378,('PDT000372335.1':0.0551731,('PDT000435595.1':0.0412214,('PDT000325537.1':0.0269252,('PDT000408757.1':0.0223108,'PDT000395148.1':0.0208526)'':0.00842597)'':0.00410966)'':0.0142487)'':-0.00024537)'':0.00641079,(('PDT000251849.1':0.0529934,('PDT000502317.1':0.0370853,('PDT000095799.2':0.0260783,'PDT000497377.1':0.0269542)'':0.00508901)'':0.0101527)'':0.00489546,(('PDT000306436.2':0.0342178,'PDT000502325.1':0.0272769)'':0.0132764,(('PDT000502320.1':0.0428217,('PDT000502676.1':0.02681,'PDT000171786.1':0.0276025)'':0.0149388)'':0.00697182,(('PDT000313335.2':0.0408367,('PDT000644994.1':0.0295465,('PDT000310461.2':0.015179,'PDT000551908.1':0.0179369)'':0.0183301)'':0.00399078)'':0.00140197,('PDT000557206.1':0.0474158,('PDT000502553.1':0.0358542,(('PDT000372336.1':0.00628359,'PDT000509502.1':0.00956091)'':0.0154766,('PDT000479857.1':0.0175144,('PDT000565973.1':0.0120274,'PDT000395388.1':0.00954102)'':0.00914651)'':0.0038076)'':0.00462306)'':0.0119581)'':-0.00343165)'':0.00855669)'':0.00499679)'':-0.00121122)'':0.00729259)'':0.00866488)'':0.00188944)'':0.0033134)'':0.0403176)'':-0.0334886)'':0.0409659)'':0.0263364)'':0.0214694,((('PDT000565998.1':0.101819,('PDT000252580.1':0.0734899,('PDT000003802.2':0.0577332,('PDT000305143.2':0.0711983,'PDT000188090.2':0.0634867)'':0.000579847)'':0.0212308)'':0.0145472)'':0.0428997,(((('PDT000228295.1':0.0264562,('PDT000354909.1':0.03316,'PDT000498775.1':0.0379924)'':0.00750576)'':0.0109361,('PDT000334501.2':0.0456509,('PDT000298488.3':0.0367499,('PDT000399366.2':0.0214383,('PDT000623895.2':0.0298734,('PDT000136731.3':0.0214039,('PDT000205399.3':0.00796307,'PDT000126282.3':0.00703793)'':0.00647591)'':0.00398238)'':0.00430116)'':0.00325962)'':0.00745883)'':0.0166522)'':0.028655,(('PDT000204989.2':0.0665658,'PDT000661110.1':0.0633392)'':0.0169519,((('PDT000405258.1':0.0263256,'PDT000425460.1':0.0347461)'':0.0326033,('PDT000557118.1':0.0722691,('PDT000627424.1':0.0521731,(('PDT000336712.1':0.0193818,'PDT000343738.1':0.0292824)'':0.0336612,('PDT000459781.1':0.0383227,('PDT000395024.1':0.0333424,'PDT000619953.1':0.0309817)'':0.00493027)'':0.00598045)'':0.00843693)'':0.0104147)'':0.010084)'':0.000428085,(('PDT000030564.1':0.0751486,('PDT000548300.2':0.0394867,('PDT000503642.2':0.0442782,('PDT000310651.3':0.0371337,('PDT000190955.3':0.0314666,'PDT000103594.3':0.0272363)'':0.00424012)'':0.00378068)'':0.0110625)'':0.024265)'':-0.00251512,(('PDT000147430.3':0.0540294,'PDT000315897.1':0.0615906)'':0.00184491,('PDT000548285.2':0.045188,('PDT000098202.1':0.0418487,(('PDT000663981.1':0.033798,('PDT000687979.1':0.0230462,'PDT000407003.1':0.0222173)'':0.00699768)'':0.00942204,(('PDT000439444.2':0.0291988,('PDT000140471.3':0.0195787,('PDT000150675.3':0.00698175,'PDT000062517.3':0.00536635)'':0.00595909)'':0.0110708)'':0.00533603,(('PDT000389793.2':0.0248592,'PDT000081058.1':0.0345997)'':0.00209775,('PDT000717630.1':0.0235698,('PDT000124992.3':0.0175499,'PDT000502454.1':0.0156911)'':0.00282424)'':0.00773217)'':0.00289092)'':0.00770582)'':0.00529693)'':0.000476349)'':0.0170221)'':0.0038904)'':0.0108509)'':0.00748072)'':0.00111644)'':0.043289,(((('PDT000443073.1':0.0772452,'PDT000522772.1':0.0871928)'':0.0113828,('PDT000251449.1':0.0513095,('PDT000688697.1':0.0256854,('PDT000710821.1':0.0147494,('PDT000485407.1':0.0113168,'PDT000395159.1':0.0151351)'':0.00333895)'':0.00877497)'':0.0427687)'':0.0301727)'':0.00938053,(('PDT000501995.1':0.0444192,('PDT000469980.1':0.0160654,'PDT000624696.1':0.0316533)'':0.0218992)'':0.0162301,(('PDT000687929.1':0.0410941,'PDT000661130.1':0.0489776)'':0.0062443,('PDT000553401.1':0.0272535,('PDT000645125.1':0.0278653,'PDT000395483.1':0.0204072)'':0.0139114)'':0.0194003)'':0.0138395)'':0.0233479)'':0.0048397,((('PDT000292798.2':0.0188065,('PDT000565926.1':0.0223478,('PDT000395345.1':0.0169507,'PDT000395251.1':0.0109706)'':0.00480105)'':0.00574558)'':0.0519608,(('PDT000281737.2':0.0724087,('PDT000395293.1':0.0527647,('PDT000251782.1':0.0475799,('PDT000051398.2':0.0237853,('PDT000714585.1':0.00394827,'PDT000714590.1':0.00163776)'':0.0163144)'':0.0132577)'':0.00381603)'':0.0182055)'':-0.0050932,(('PDT000546952.1':0.048833,('PDT000083994.2':0.0243626,'PDT000415841.1':0.0293682)'':0.0121208)'':0.00896357,('PDT000285377.2':0.0365591,('PDT000518042.1':0.0576987,('PDT000042500.1':0.0389001,'PDT000645541.2':0.0401458)'':0.00459596)'':0.00412734)'':0.00885767)'':0.00386253)'':0.012817)'':0.00830301,(('PDT000052518.3':0.0854215,('PDT000397796.1':0.0773843,('PDT000466051.1':0.0633222,('PDT000621300.1':0.0344366,'PDT000644845.1':0.034265)'':0.0204305)'':0.0109348)'':0.0168592)'':0.00611517,(('PDT000325582.1':0.0619264,(('PDT000394945.1':0.0448262,('PDT000566201.1':0.0314582,'PDT000604447.1':0.0328358)'':0.00970048)'':0.0097168,('PDT000517803.1':0.0602764,('PDT000719028.1':0.0497593,('PDT000583499.1':0.0437869,('PDT000399663.1':0.0392119,('PDT000499435.1':0.0329442,'PDT000719068.1':0.0204697)'':0.0112234)'':0.00197935)'':0.00428752)'':0.0038791)'':0.00124809)'':0.00130912)'':0.00269327,('PDT000631557.1':0.0532744,(('PDT000095761.2':0.0449721,('PDT000719262.1':0.0387266,('PDT000128860.2':0.0257387,'PDT000426341.1':0.0201329)'':0.00999191)'':0.00922623)'':0.00397878,(('PDT000395289.1':0.0354211,('PDT000425913.1':0.0313035,'PDT000521582.1':0.0215735)'':0.00160012)'':0.00868289,('PDT000145846.3':0.0372249,('PDT000395438.1':0.024869,('PDT000025150.2':0.0144644,'PDT000395141.1':0.0161509)'':0.00763489)'':0.00567972)'':0.0148173)'':0.00238279)'':0.00909117)'':0.00149564)'':0.0246557)'':0.00184918)'':0.0192724)'':0.0265936)'':0.0125135)'':0.0114903,(((('PDT000105019.2':0.0696717,('PDT000285418.2':0.0582833,'PDT000095780.2':0.0407441)'':0.0273406)'':0.0157787,('PDT000181821.2':0.116682,(('PDT000711105.1':0.0838829,('PDT000190657.2':0.0611507,('PDT000520772.1':0.0681994,('PDT000137848.2':0.0504883,'PDT000095764.2':0.0487775)'':0.00378717)'':0.0041141)'':0.00532868)'':2.73185e-05,(('PDT000085572.3':0.0412648,('PDT000300575.1':0.035857,('PDT000171577.3':0.0294491,'PDT000498119.1':0.0222361)'':0.00540083)'':0.00901797)'':0.0204803,(('PDT000251563.1':0.050684,('PDT000188153.2':0.0395436,'PDT000706221.1':0.0413663)'':0.0069967)'':0.00358969,('PDT000661764.1':0.038417,('PDT000718997.1':0.042032,('PDT000721985.1':0.0357133,'PDT000502879.1':0.0321867)'':0.00806162)'':0.00201282)'':0.00512002)'':0.00451259)'':0.00189003)'':0.0365494)'':-0.00724316)'':0.0355579,('PDT000395198.1':0.176897,(('PDT000147464.3':0.131398,('PDT000327300.1':0.0437624,(('PDT000107032.3':0.0161109,('PDT000142223.3':0.00148768,'PDT000216694.1':0.00151022)'':0.0172195)'':0.00647534,('PDT000498013.1':0.0232897,('PDT000142225.3':0.0206941,(('PDT000142227.3':0.00244367,'PDT000216689.1':0.00284786)'':0.0126598,(('PDT000669447.1':0.0153839,'PDT000045929.1':0.00745665)'':0.00113497,('PDT000142224.2':0.00138948,'PDT000216696.1':0.00180796)'':0.00710453)'':0.00219705)'':0.00283962)'':0.00364341)'':0.000155008)'':0.0350112)'':0.0721856)'':-0.0112875,((('PDT000311257.2':0.0585523,'PDT000502014.1':0.0681837)'':0.0380222,('PDT000497308.1':0.073766,'PDT000584497.1':0.076544)'':0.0285956)'':0.00882501,(('PDT000205079.2':0.0607548,'PDT000032322.1':0.0659312)'':0.0538098,('PDT000654429.1':0.100655,(('PDT000011736.2':0.0717044,('PDT000502583.1':0.0582013,'PDT000345979.1':0.0585627)'':0.0168011)'':0.0263091,(('PDT000102024.2':0.0290288,'PDT000683611.1':0.0434978)'':0.0177718,('PDT000671619.1':0.0377042,'PDT000252621.1':0.0299149)'':0.0137691)'':0.0413282)'':0.00795073)'':0.00174519)'':0.00256241)'':0.00685261)'':0.0218164)'':-0.00998787)'':0.00533813,(((('PDT000661645.1':0.138637,('PDT000604842.1':0.0782852,(('PDT000095738.2':0.038835,'PDT000399674.1':0.0321232)'':0.0109949,('PDT000396518.1':0.0413364,'PDT000397785.1':0.0451439)'':0.0189821)'':0.00751195)'':0.0741942)'':0.00209608,(((('PDT000134031.2':0.0545217,('PDT000281212.2':0.048076,'PDT000499666.1':0.0340463)'':0.0203151)'':0.0541351,('PDT000473277.1':0.105999,(('PDT000426069.1':0.0312318,'PDT000619635.1':0.035124)'':0.0615267,('PDT000228324.1':0.0665824,('PDT000357652.1':0.085159,('PDT000191807.2':0.0685941,'PDT000141475.2':0.0771659)'':0.00445196)'':0.00429523)'':0.0117262)'':0.0208077)'':-0.0067158)'':0.000595976,('PDT000171820.1':0.108298,(('PDT000025356.1':0.0353505,('PDT000249885.1':0.0133544,('PDT000428426.2':0.0148822,'PDT000691353.1':0.0142134)'':0.0044015)'':0.0201595)'':0.0167298,('PDT000471029.1':0.0701528,(('PDT000479861.2':0.0378943,'PDT000620941.2':0.0284197)'':0.0220724,(('PDT000683389.1':0.0440394,'PDT000065384.1':0.0396075)'':0.00376254,(('PDT000150752.3':0.0387772,'PDT000413927.2':0.0245452)'':0.00843016,('PDT000313322.3':0.0358229,'PDT000691360.1':0.0287466)'':0.00639069)'':0.00289739)'':0.00749189)'':0.00937381)'':-0.00574299)'':0.0648381)'':-0.00104072)'':0.0144029,((('PDT000494528.1':0.0768414,'PDT000566061.1':0.0684836)'':0.0146211,('PDT000631951.1':0.063863,'PDT000640346.1':0.112573)'':0.0127094)'':0.0184682,((('PDT000204931.2':0.0853819,'PDT000645042.1':0.0761471)'':0.0190567,(('PDT000644882.1':0.09211,('PDT000679718.1':0.0650338,('PDT000589278.1':0.0526948,'PDT000081766.1':0.0542372)'':0.0172852)'':0.0109149)'':0.0026272,(('PDT000556878.1':0.0367753,'PDT000631954.1':0.0340049)'':0.028772,('PDT000687846.1':0.0762008,('PDT000477466.1':0.0428736,('PDT000188182.2':0.0288137,'PDT000583822.1':0.0341336)'':0.00448081)'':0.0106914)'':0.0121086)'':0.00787276)'':0.0104536)'':-0.000610396,((('PDT000235349.3':0.0508848,'PDT000131101.3':0.03855)'':0.00410275,('PDT000527191.2':0.0293839,'PDT000242521.1':0.0338738)'':0.0113542)'':0.0395339,(('PDT000307407.2':0.0487304,('PDT000394986.1':0.0226707,'PDT000411095.1':0.0265822)'':0.024872)'':0.00708645,(('PDT000323913.1':0.047374,'PDT000109476.2':0.0411301)'':0.00964016,(('PDT000066406.2':0.0272263,'PDT000395309.1':0.0269753)'':0.0139232,('PDT000095728.2':0.0325863,'PDT000477064.1':0.0329066)'':0.0055508)'':0.00973603)'':0.00366981)'':0.0158884)'':0.0201139)'':0.0060955)'':0.0198443)'':0.0212995)'':-0.0221379,(('PDT000188114.2':0.139701,'PDT000081749.1':0.132438)'':0.0147252,('PDT000249748.1':0.145244,((('PDT000104643.2':0.0415875,'PDT000251514.1':0.0419296)'':0.045165,('PDT000417991.1':0.0665145,('PDT000384286.1':0.0605031,('PDT000548629.2':0.0611382,('PDT000196591.2':0.0190768,'PDT000482516.1':0.016222)'':0.0194569)'':0.0075061)'':0.0125874)'':0.0268174)'':0.00352771,(((('PDT000482220.1':0.0667164,'PDT000637137.1':0.0518496)'':0.00563126,('PDT000535651.1':0.0696987,('PDT000559806.1':0.0454708,'PDT000156073.1':0.0376857)'':0.026941)'':0.01281)'':0.00440681,(('PDT000577076.1':0.0432406,'PDT000594458.1':0.0308662)'':0.0154854,(('PDT000265422.2':0.0464442,'PDT000095707.2':0.0415612)'':0.00465598,('PDT000471019.1':0.0236747,('PDT000323525.1':0.00860947,'PDT000178591.2':0.00995313)'':0.0212397)'':0.0150219)'':0.0014943)'':0.024091)'':0.00652075,((('PDT000671283.1':0.0455857,('PDT000672778.1':0.0294558,'PDT000689816.1':0.0444572)'':0.00252484)'':0.0911288,(('PDT000355097.1':0.0511045,('PDT000557342.1':0.0421861,'PDT000502880.1':0.0337781)'':0.0176781)'':0.0135313,('PDT000626963.1':0.109101,(('PDT000584498.1':0.0509302,'PDT000248734.1':0.043862)'':0.00666624,('PDT000455207.1':0.0681292,('PDT000178567.2':0.0500137,(('PDT000484127.1':0.0374504,'PDT000622550.1':0.035082)'':0.00780394,('PDT000610077.1':0.0450208,('PDT000248546.1':0.0280995,('PDT000373243.1':0.0216286,'PDT000509155.1':0.0221269)'':0.00201097)'':0.00987427)'':0.0014352)'':0.00641839)'':0.00358553)'':-0.00159131)'':0.0270944)'':-0.0127911)'':0.0181388)'':-0.00991163,((('PDT000395215.1':0.0359061,('PDT000313318.2':0.0182795,('PDT000504753.1':0.0230819,('PDT000323543.1':0.0193414,'PDT000455215.1':0.0200109)'':0.00199268)'':0.00120169)'':0.00298665)'':0.0404459,('PDT000550395.1':0.0566787,('PDT000051391.2':0.0516147,(('PDT000167676.2':0.0376468,'PDT000502252.1':0.0285377)'':0.00463696,('PDT000652923.1':0.00942921,'PDT000652964.1':0.01732)'':0.0375248)'':0.0053016)'':0.00366779)'':0.0107442)'':0.0151233,(('PDT000228202.1':0.0615012,('PDT000104649.2':0.0651374,('PDT000030547.1':0.0427891,('PDT000393080.1':0.0227287,'PDT000706397.1':0.0317826)'':0.0248987)'':0.00227706)'':0.0090482)'':0.00461589,(('PDT000299920.2':0.0606077,'PDT000502410.1':0.0530853)'':0.00848556,('PDT000206108.2':0.0852963,('PDT000163827.2':0.0255157,('PDT000509501.1':0.0156951,'PDT000604863.1':0.0146557)'':0.00701532)'':0.0307734)'':0.00251642)'':0.00613172)'':0.00639)'':0.00356034)'':0.00359234)'':0.0189936)'':0.0342951)'':0.00707854)'':-0.0303345)'':0.0152701,((('PDT000502005.1':0.0613061,'PDT000652455.1':0.0724359)'':0.0531211,((('PDT000427602.1':0.0591469,'PDT000233102.1':0.0613341)'':0.028128,('PDT000069955.1':0.0366652,('PDT000494529.1':0.0168388,'PDT000319036.1':0.0127857)'':0.0326737)'':0.0507938)'':0.0102841,('PDT000502909.1':0.104511,(('PDT000661134.1':0.0648259,('PDT000673584.1':0.0527023,('PDT000602919.1':0.0507269,('PDT000644932.1':0.0466436,('PDT000395131.1':0.0332371,('PDT000426120.1':0.0266467,'PDT000535532.1':0.0275866)'':0.00477188)'':0.00556482)'':0.00718371)'':0.00135784)'':0.0105704)'':0.0313036,('PDT000719242.1':0.117883,(('PDT000069958.1':0.0811404,('PDT000415755.1':0.0424233,'PDT000060442.1':0.0475574)'':0.0225058)'':0.0204858,('PDT000631953.1':0.110152,('PDT000248834.1':0.0812294,('PDT000203174.3':0.0321439,'PDT000312492.3':0.0227167)'':0.0596558)'':0.0036543)'':0.00466761)'':0.0102933)'':-0.00268353)'':0.0151398)'':-0.00645697)'':0.0128321)'':-2.43869e-05,((('PDT000652439.1':0.126656,('PDT000652446.1':0.0224607,'PDT000652456.1':0.0268882)'':0.0714418)'':0.00131238,((('PDT000639566.1':0.0467821,('PDT000502485.1':0.0390687,'PDT000720170.1':0.034648)'':0.0173141)'':0.00776365,('PDT000298098.1':0.0672911,('PDT000027741.2':0.0335057,('PDT000482517.1':0.0337569,('PDT000688656.1':0.0219842,('PDT000201009.2':0.0207181,'PDT000494555.1':0.0200502)'':0.00517866)'':0.00642226)'':0.000758766)'':0.0202981)'':0.00434414)'':0.0223412,((('PDT000025505.2':0.0754828,('PDT000069133.2':0.0545805,'PDT000584494.1':0.0396075)'':0.00512318)'':0.00760777,(('PDT000395156.1':0.0288235,'PDT000645032.1':0.0261731)'':0.0629654,('PDT000626001.1':0.0577543,('PDT000652896.1':0.0372288,'PDT000652908.1':0.0379541)'':0.0274132)'':0.0169729)'':0.00490041)'':0.0291349,(('PDT000498870.2':0.0733658,('PDT000253025.3':0.0520858,'PDT000663642.1':0.0408496)'':0.025935)'':0.0388594,((('PDT000181765.2':0.0618144,'PDT000619093.1':0.0565976)'':0.016485,('PDT000502634.1':0.0756238,'PDT000588637.1':0.0896012)'':0.00762679)'':0.00537873,(('PDT000491103.1':0.0806203,('PDT000624697.1':0.0734393,('PDT000310619.2':0.065594,'PDT000251897.1':0.052905)'':0.0043977)'':0.00597211)'':-0.000718259,(('PDT000426105.1':0.0514391,('PDT000298022.1':0.0655359,('PDT000616556.1':0.0550558,'PDT000124680.1':0.0412296)'':0.00717989)'':0.00473808)'':0.0134007,(('PDT000196643.2':0.0845965,'PDT000124660.1':0.0855535)'':0.00563593,('PDT000675564.1':0.0856944,('PDT000539053.1':0.0761988,'PDT000718885.1':0.0538812)'':0.0111811)'':0.00184872)'':0.000457081)'':0.0074079)'':0.00399991)'':0.01038)'':0.00122642)'':0.0029334)'':0.0108586)'':0.00276846,(('PDT000242883.1':0.158373,(('PDT000661550.1':0.0550176,'PDT000661761.1':0.0380169)'':0.0653308,((('PDT000323322.1':0.0525072,'PDT000298042.1':0.0624278)'':0.0191007,('PDT000233842.1':0.0857641,('PDT000271760.1':0.0757694,('PDT000720445.1':0.0248246,('PDT000396745.1':0.0261978,'PDT000414213.1':0.0209515)'':0.017374)'':0.0171406)'':0.0267589)'':0.00544215)'':0.010868,(('PDT000228327.1':0.0684691,('PDT000425572.1':0.0436004,'PDT000547546.1':0.0324028)'':0.0330863)'':0.0113732,('PDT000468831.1':0.0747457,(('PDT000397776.1':0.076704,'PDT000435634.1':0.096538)'':0.0030341,('PDT000298015.1':0.0766664,('PDT000202768.2':0.0446399,'PDT000675191.1':0.0556011)'':0.0131146)'':0.00959187)'':0.00619798)'':-0.000702131)'':0.0090167)'':0.0139576)'':0.0329863)'':-0.0214471,((('PDT000208424.2':0.0904377,(('PDT000611515.2':0.0349449,'PDT000140500.3':0.0404592)'':0.0439395,('PDT000548490.2':0.0688843,('PDT000459514.1':0.040155,'PDT000470446.1':0.0558552)'':0.0115046)'':0.00708091)'':0.0270184)'':-0.00148048,(('PDT000171825.1':0.0561332,('PDT000081773.1':0.0372705,(('PDT000494539.1':0.0305618,'PDT000494557.1':0.0340327)'':0.00526694,('PDT000509761.1':0.0182787,'PDT000171817.1':0.0191118)'':0.00550384)'':0.0134861)'':0.0105591)'':0.0516506,((('PDT000290409.2':0.0922661,'PDT000556910.1':0.0827269)'':0.00607765,('PDT000439474.1':0.0564042,'PDT000252738.1':0.0453218)'':0.0287439)'':0.0132938,(('PDT000484651.1':0.0713453,('PDT000394820.1':0.0497979,'PDT000640027.1':0.0583261)'':0.0227707)'':0.0209479,('PDT000656968.1':0.0937881,('PDT000060208.2':0.0530419,('PDT000497301.1':0.0404415,('PDT000639547.1':0.0269693,'PDT000702831.1':0.0361484)'':0.00359296)'':0.00216468)'':0.0484411)'':0.00772108)'':0.00535511)'':0.00220029)'':0.000308442)'':0.00425372,((('PDT000557090.1':0.0587609,'PDT000008948.2':0.0660141)'':0.0299285,(('PDT000495056.1':0.0793605,'PDT000620257.1':0.0750195)'':0.011519,('PDT000346599.1':0.0846004,('PDT000711740.1':0.0776901,('PDT000208403.2':0.0477727,('PDT000652334.1':0.056146,('PDT000069623.1':0.047874,('PDT000426091.1':0.0373509,'PDT000550345.1':0.0303278)'':0.00751253)'':0.00241084)'':0.00702523)'':0.00869611)'':0.004499)'':-0.000863006)'':0.0148335)'':0.00389508,((('PDT000509697.1':0.00311329,'PDT000583850.1':0.00647712)'':0.0548117,('PDT000718962.1':0.0653781,'PDT000249963.1':0.0366199)'':0.00948832)'':0.0197973,((('PDT000650531.1':0.0674358,'PDT000562122.1':0.0807932)'':0.0138908,('PDT000714680.1':0.0679196,('PDT000136150.2':0.0479819,'PDT000445048.1':0.0564061)'':0.00513035)'':0.00834808)'':0.000994639,(('PDT000651145.1':0.0487805,'PDT000242488.1':0.0526505)'':0.0193687,('PDT000633567.1':0.0735637,('PDT000228244.1':0.0424734,('PDT000718876.1':0.0120231,'PDT000228236.1':0.0143267)'':0.0235737)'':0.0322541)'':0.00153666)'':0.00390888)'':0.0027449)'':0.0107404)'':0.00787451)'':0.00977306)'':0.00183929)'':0.00675057)'':0.0187303)'':0.00217547)'':0.0175762)'':0.00846831)'':0.00143184)'':0.0146044,((('PDT000566112.1':0.10827,((('PDT000395351.1':0.0173885,('PDT000150265.2':0.0196895,('PDT000565840.1':0.00938938,('PDT000355616.1':0.00591486,'PDT000550380.1':0.00434824)'':0.00979572)'':0.00518656)'':0.00448794)'':0.0317462,('PDT000661140.1':0.0585034,(('PDT000279583.2':0.0346576,('PDT000204922.2':0.032473,'PDT000206395.2':0.0262872)'':0.00785266)'':0.00860535,('PDT000720203.1':0.0460502,(('PDT000258407.2':0.0351084,('PDT000500484.1':0.0106522,'PDT000502311.1':0.00940142)'':0.0246427)'':0.00596895,('PDT000652564.1':0.0341502,('PDT000502425.1':0.0338335,'PDT000124597.1':0.0366638)'':0.00420049)'':0.00345491)'':0.00538361)'':-0.000213086)'':0.015019)'':-0.00361098)'':0.00895015,(('PDT000617805.1':0.0549156,('PDT000557171.1':0.0521637,(('PDT000557213.1':0.0189514,'PDT000691357.1':0.020085)'':0.0156186,('PDT000420664.1':0.0159491,('PDT000502248.1':0.0228111,('PDT000309338.2':0.0157828,'PDT000565906.1':0.0206132)'':0.00340037)'':0.00198401)'':0.00259291)'':0.0102627)'':0.0197992)'':0.00540197,((('PDT000286925.2':0.0191844,'PDT000095807.2':0.0144557)'':0.00549404,('PDT000364423.1':0.0129194,'PDT000417398.1':0.0140006)'':0.0137321)'':0.0237675,('PDT000664972.1':0.0677933,(('PDT000563865.1':0.0457831,'PDT000502684.1':0.0361554)'':0.00613062,('PDT000497431.1':0.0281592,('PDT000131795.2':0.0277163,'PDT000664961.1':0.0332569)'':0.00320972)'':0.00429024)'':0.00855881)'':0.00626093)'':0.00264502)'':0.00211926)'':0.0453055)'':0.0563414,((('PDT000667724.2':0.084928,'PDT000667745.2':0.158194)'':0.0218358,(('PDT000615872.1':0.0463642,('PDT000425561.1':0.0442482,('PDT000221040.1':0.038198,('PDT000425571.1':0.0380575,('PDT000443034.1':0.031801,'PDT000557298.1':0.0238147)'':0.00386495)'':0.00855842)'':0.00340004)'':0.00506727)'':0.0365472,(('PDT000249814.1':0.0708133,'PDT000357636.1':0.0794197)'':0.00572664,('PDT000221058.1':0.0536635,('PDT000250163.1':0.0443847,('PDT000443077.1':0.025147,('PDT000129853.2':0.0230845,('PDT000425900.1':0.0104665,'PDT000441348.1':0.0161765)'':0.00486928)'':0.0121027)'':0.0128065)'':0.00667857)'':0.0318067)'':0.00310706)'':0.0272772)'':0.0402714,(('PDT000646914.1':0.0805222,((('PDT000200965.3':0.00812505,'PDT000140512.3':0.010803)'':0.00851058,('PDT000483983.2':0.0222085,('PDT000443283.2':0.00966167,'PDT000587577.2':0.0130158)'':0.00647128)'':0.00119661)'':0.0478398,(('PDT000428378.1':0.0448036,'PDT000718976.1':0.0413901)'':0.00959614,('PDT000309259.2':0.0524425,('PDT000556921.1':0.0442425,('PDT000178996.2':0.0260838,'PDT000258395.2':0.0241508)'':0.0246694)'':0.00562324)'':0.00120446)'':0.00963786)'':0.00937697)'':0.00471616,('PDT000228318.1':0.120268,(('PDT000598844.1':0.0525097,'PDT000077514.1':0.0672023)'':0.0111194,(('PDT000660093.2':0.0425289,('PDT000386830.2':0.0472385,('PDT000494251.2':0.0375826,('PDT000275037.3':0.0345298,('PDT000537066.2':0.0156807,'PDT000673703.1':0.0188191)'':0.00671085)'':0.00600743)'':0.00669259)'':0.00394489)'':0.00305411,('PDT000140568.4':0.0720809,(('PDT000494273.2':0.0373434,'PDT000497767.1':0.0410575)'':0.00220412,('PDT000343471.2':0.0367064,(('PDT000504864.2':0.0269535,('PDT000439578.2':0.0276695,'PDT000486525.1':0.0262081)'':0.000913331)'':0.000240078,('PDT000244677.3':0.00910164,('PDT000140514.3':0.00978847,('PDT000336081.2':0.00579087,'PDT000501707.2':0.00488033)'':0.00269358)'':0.00312082)'':0.0114981)'':0.0143535)'':0.00113871)'':0.00841341)'':-0.00273187)'':0.0130036)'':0.0458704)'':-0.0300162)'':0.0770915)'':0.0230406)'':0.0119322,((((('PDT000034481.2':0.147087,'PDT000404059.2':0.148617)'':0.0116133,('PDT000289889.1':0.0606218,(('PDT000656997.1':0.0624178,'PDT000298039.1':0.0760972)'':0.00390785,('PDT000396740.1':0.0575781,('PDT000656950.1':0.053504,'PDT000129704.1':0.047204)'':0.00871542)'':0.00832734)'':0.0104137)'':0.0916184)'':0.0203795,(((('PDT000253466.3':0.0399345,('PDT000208718.1':0.00466763,'PDT000498097.1':0.00469772)'':0.0400089)'':0.004934,('PDT000222046.3':0.0194103,('PDT000354139.2':0.00777886,('PDT000501571.2':0.00436934,'PDT000501573.2':0.00609506)'':0.00331909)'':0.0125652)'':0.0286452)'':0.00501943,(('PDT000065375.1':0.0327242,('PDT000568710.2':0.0145904,'PDT000109066.3':0.018488)'':0.00857229)'':0.0171111,('PDT000343951.2':0.0508856,(('PDT000492242.2':0.0391739,'PDT000502459.1':0.0417588)'':0.0023417,('PDT000288590.3':0.0391538,(('PDT000497882.1':0.0359825,('PDT000719876.1':0.0215704,('PDT000502441.1':0.00673371,'PDT000502562.1':0.00769989)'':0.0173159)'':0.00661796)'':0.000717362,(('PDT000288571.3':0.0193403,('PDT000258479.4':0.0155042,'PDT000486496.1':0.0185631)'':0.00111049)'':0.00698827,('PDT000200960.3':0.0301263,('PDT000691403.1':0.0120914,('PDT000140225.3':0.00534779,'PDT000502351.1':0.00580091)'':0.00365833)'':0.020384)'':0.000961104)'':0.00242757)'':0.00813912)'':-0.000724632)'':0.00881008)'':0.00884526)'':-0.001685)'':0.108991,('PDT000181810.2':0.152863,(('PDT000357679.1':0.121038,(('PDT000253452.2':0.0268296,('PDT000466065.1':0.0178935,'PDT000649269.1':0.0185525)'':0.0074899)'':0.0200344,('PDT000131916.2':0.033948,('PDT000253459.2':0.0196317,'PDT000332869.1':0.0126616)'':0.0107557)'':0.02353)'':0.0642474)'':0.0168323,(('PDT000248817.1':0.12473,('PDT000228354.1':0.167421,('PDT000400002.1':0.12744,((('PDT000557210.1':0.102317,'PDT000691351.1':0.0920739)'':0.019929,('PDT000502339.1':0.0350486,'PDT000502424.1':0.0337801)'':0.0669017)'':0.00275998,(('PDT000307441.2':0.0311064,'PDT000466982.1':0.0214991)'':0.0769212,(('PDT000095704.2':0.0701362,'PDT000228094.1':0.0741668)'':0.00142695,('PDT000095726.2':0.0897722,(('PDT000550389.1':0.0306605,('PDT000502255.1':0.0220776,'PDT000294408.1':0.0188069)'':0.0101178)'':0.0392927,(('PDT000417971.1':0.0447508,'PDT000358765.1':0.0379879)'':0.0126862,('PDT000510074.1':0.0294209,'PDT000687917.1':0.0381268)'':0.0157605)'':0.00655962)'':0.0123926)'':-0.00326527)'':0.011801)'':0.00256645)'':0.0189227)'':0.0140064)'':0.000726746)'':-0.0182672,((('PDT000523439.2':0.0417399,'PDT000346914.1':0.0414016)'':0.0405416,(('PDT000355765.1':0.0470637,'PDT000619598.1':0.0501347)'':0.0033479,('PDT000216618.3':0.0456515,('PDT000565891.1':0.0228298,'PDT000587992.1':0.0293244)'':0.00651902)'':0.0115631)'':0.0282662)'':0.0357262,(((('PDT000702604.1':0.0575657,('PDT000251901.1':0.0429706,('PDT000240656.3':0.00657347,'PDT000712686.1':0.00586313)'':0.0398809)'':0.00807402)'':0.00227093,(('PDT000281548.3':0.039887,'PDT000304208.1':0.0403331)'':0.00905054,('PDT000124629.1':0.0211955,('PDT000158878.3':0.0130654,'PDT000498149.1':0.0126997)'':0.0132726)'':0.0137743)'':0.0185173)'':0.0301272,(('PDT000214068.3':0.0737427,'PDT000404822.2':0.0614263)'':0.0295901,((('PDT000465418.2':0.0121604,'PDT000480978.2':0.00937814)'':0.0483334,('PDT000148165.3':0.0607732,('PDT000156563.3':0.0441658,('PDT000320894.2':0.0450017,'PDT000465218.2':0.0439955)'':0.00464214)'':0.0162318)'':0.00483654)'':0.00225845,('PDT000314193.3':0.0470346,(('PDT000548546.2':0.0168905,'PDT000558755.2':0.012836)'':0.0319465,('PDT000124707.1':0.0460965,('PDT000314856.3':0.0231016,'PDT000625613.2':0.0306809)'':0.020034)'':0.0079462)'':0.00279125)'':0.00877794)'':0.0342001)'':-0.0149444)'':0.0145177,((('PDT000668725.1':0.0536507,('PDT000213945.3':0.0370589,('PDT000367007.2':0.0297121,'PDT000300528.1':0.0205689)'':0.00515836)'':0.019105)'':0.00180094,('PDT000376191.2':0.0665034,(('PDT000176701.3':0.0180738,'PDT000221706.3':0.0253743)'':0.0114783,('PDT000319689.2':0.0277747,'PDT000025736.1':0.0296653)'':6.17181e-05)'':0.0265473)'':-0.0017091)'':0.0202688,((('PDT000321135.2':0.0226705,('PDT000250817.3':0.0102775,'PDT000281571.3':0.0128628)'':0.00906036)'':0.00893908,('PDT000248641.1':0.0302645,('PDT000339390.2':0.0163425,'PDT000248630.1':0.0110222)'':0.0156082)'':-0.00151738)'':0.0155036,(('PDT000038883.1':0.0547177,('PDT000168310.3':0.0197747,'PDT000342277.2':0.0189483)'':0.0223257)'':0.00726671,((('PDT000040392.6':0.0145961,('PDT000203841.3':0.00788149,'PDT000312549.3':0.0115563)'':0.0106062)'':0.0112735,('PDT000123319.1':0.0289998,('PDT000160193.3':0.021522,('PDT000155917.3':0.0150544,'PDT000123329.1':0.0165622)'':0.00516144)'':0.00696933)'':0.00248832)'':0.00339904,(('PDT000160249.3':0.0317443,'PDT000457504.2':0.0306215)'':0.00677227,(('PDT000027608.1':0.0333102,'PDT000008767.2':0.0248214)'':0.00699876,(('PDT000157104.3':0.0309653,'PDT000287884.3':0.0222052)'':0.00283712,('PDT000248666.1':0.0377137,('PDT000040685.4':0.0206005,'PDT000497895.1':0.024433)'':0.00853909)'':0.00255105)'':0.0037142)'':0.0021812)'':0.000723401)'':0.00363936)'':0.00768256)'':0.00988285)'':0.0269195)'':0.0293273)'':0.010616)'':0.00398773)'':0.0267821)'':0.0157168)'':0.0105457)'':-0.0076775,(((('PDT000396794.1':0.131197,(('PDT000400870.2':0.0425368,('PDT000548531.2':0.0354253,'PDT000107002.3':0.0234273)'':0.0101701)'':0.0180358,(('PDT000280677.3':0.0129764,('PDT000213970.3':0.0190412,('PDT000281566.3':0.0111116,(('PDT000147468.3':0.0132904,'PDT000083971.3':0.00947527)'':0.00317753,('PDT000133826.3':0.0119773,('PDT000163031.4':0.0134887,('PDT000156638.3':0.0069288,('PDT000134326.7':0.0064821,'PDT000484007.2':0.0070341)'':0.00275405)'':0.0019525)'':0.00186469)'':-0.00019692)'':0.00217916)'':0.00058165)'':0.00447853)'':0.0184146,(('PDT000161687.3':0.0289436,'PDT000679863.1':0.0417862)'':0.00386079,(('PDT000574396.2':0.0307652,'PDT000123264.1':0.0360958)'':0.00248002,((('PDT000410714.2':0.0281909,'PDT000414686.2':0.0281665)'':0.00908378,('PDT000366116.2':0.0238132,('PDT000510647.2':0.017234,('PDT000237874.5':0.0197663,('PDT000201210.3':0.00780583,'PDT000275044.3':0.00940637)'':0.00517857)'':0.00248789)'':0.0096812)'':0.00747299)'':0.00192401,(('PDT000378683.2':0.0343845,'PDT000110153.3':0.0315655)'':0.00720141,(('PDT000156576.3':0.0129475,'PDT000107674.2':0.0140075)'':0.00906002,(('PDT000296212.3':0.0131545,'PDT000066904.3':0.0117685)'':0.00180052,('PDT000171641.3':0.0130232,'PDT000497920.1':0.0144092)'':0.00402835)'':0.00449023)'':0.00738553)'':0.00246662)'':0.00225484)'':0.00123641)'':0.00188251)'':0.0200521)'':0.084008)'':0.0239889,((('PDT000588635.1':0.088392,('PDT000150303.2':0.0651209,'PDT000565839.1':0.0534831)'':0.038085)'':0.0243175,(('PDT000095747.2':0.0206211,'PDT000563567.1':0.0154114)'':0.0960919,('PDT000502256.1':0.0280023,'PDT000501839.1':0.0298827)'':0.0704351)'':0.00971528)'':0.0151101,((('PDT000124704.1':0.0691988,(('PDT000316908.2':0.0403089,'PDT000716357.1':0.0465926)'':0.00359971,('PDT000641504.2':0.036395,('PDT000051178.1':0.00414625,'PDT000498169.1':0.00472271)'':0.0386436)'':0.0121608)'':0.013506)'':0.000819294,((('PDT000101032.3':0.0210953,('PDT000326133.2':0.0126083,'PDT000648189.2':0.0197816)'':0.00168132)'':0.0264343,('PDT000636725.2':0.0426041,(('PDT000322076.2':0.0212686,'PDT000032320.1':0.0198865)'':0.00801241,('PDT000410739.2':0.0242779,'PDT000496551.2':0.0236298)'':0.00268582)'':0.0194678)'':0.00430316)'':0.000882396,(((('PDT000359283.2':0.0256541,'PDT000569728.2':0.0169679)'':0.00203531,('PDT000546865.2':0.017265,'PDT000123291.1':0.0189555)'':0.0100861)'':0.00180044,(('PDT000598462.2':0.018754,'PDT000651389.2':0.0194507)'':0.00382925,('PDT000526250.2':0.0200171,('PDT000409202.2':0.0140045,'PDT000668723.1':0.0138254)'':0.00802128)'':0.0011087)'':0.000472643)'':0.00677628,('PDT000085621.3':0.024569,('PDT000693928.1':0.0360556,(('PDT000317334.2':0.029442,((('PDT000316600.2':0.0163986,'PDT000321575.2':0.0163527)'':0.00100156,('PDT000156636.3':0.014397,'PDT000327114.2':0.0145799)'':0.00197309)'':0.0050232,('PDT000123259.1':0.0277106,(('PDT000320265.2':0.0218301,'PDT000137656.3':0.0250772)'':0.000711076,(('PDT000323312.2':0.0153796,'PDT000333269.2':0.0128729)'':0.00276849,(('PDT000556155.2':0.0149025,'PDT000157949.3':0.0150057)'':0.000393562,('PDT000157880.3':0.00943191,'PDT000110437.3':0.0125919)'':0.00524539)'':0.0011498)'':0.00224316)'':0.00882854)'':-0.00618192)'':0.00511727)'':-0.00442582,(('PDT000155326.3':0.0115987,'PDT000063288.3':0.0123356)'':0.0112502,(('PDT000621337.2':0.0187678,('PDT000148256.3':0.0123249,'PDT000327091.2':0.0119373)'':0.00322013)'':0.0051275,('PDT000534073.2':0.0166512,('PDT000324091.2':0.00936925,('PDT000386782.2':0.00957939,(('PDT000615201.2':0.00860038,'PDT000669466.1':0.0102074)'':0.00166123,(('PDT000161709.3':0.00743671,'PDT000327226.2':0.00463069)'':0.00158025,('PDT000377206.2':0.00906191,'PDT000494503.2':0.00792089)'':8.0549e-05)'':0.0017004)'':0.00295543)'':0.000100299)'':0.00856267)'':-0.00481831)'':0.00628834)'':0.00226184)'':0.0074584)'':0.00147056)'':-0.00334674)'':0.0238489)'':0.0181686)'':0.00784134,(((('PDT000324701.2':0.0289696,('PDT000366063.2':0.0203562,(('PDT000333256.2':0.00949458,'PDT000353583.2':0.0126856)'':0.0032215,('PDT000321534.2':0.0120431,('PDT000132179.3':0.0159554,('PDT000637606.2':0.00950005,'PDT000711010.1':0.0150966)'':0.0010972)'':0.000445683)'':0.00105084)'':0.00874517)'':0.0113274)'':-0.00418733,('PDT000497933.1':0.0417046,(('PDT000689092.1':0.0196237,'PDT000131874.3':0.0244114)'':0.00354073,(('PDT000378577.2':0.0235697,('PDT000317136.2':0.0104069,'PDT000332788.2':0.0115842)'':0.00902964)'':0.00197043,('PDT000668119.2':0.0230195,(('PDT000156677.3':0.0235781,'PDT000328086.2':0.0215228)'':0.00189945,(('PDT000404044.2':0.024892,('PDT000317773.2':0.0162305,'PDT000333938.2':0.0141595)'':0.00277959)'':-1.18825e-05,('PDT000334436.2':0.029922,(('PDT000160565.3':0.0129713,'PDT000189685.3':0.0157217)'':0.00604642,('PDT000399854.2':0.0156916,('PDT000333277.2':0.0166688,(('PDT000157265.3':0.0112137,'PDT000063287.3':0.010364)'':0.00153481,('PDT000428312.2':0.00789362,('PDT000517242.2':0.00644616,'PDT000647764.2':0.00818944)'':0.000733434)'':0.00293108)'':0.00187995)'':0.000824231)'':0.00120571)'':0.00282467)'':-0.0018611)'':0.00326192)'':0.00350067)'':-0.00410234)'':0.00236202)'':0.0160795)'':-0.00967926)'':0.0146842,((('PDT000334417.2':0.0211718,'PDT000593817.2':0.0249184)'':0.00732193,('PDT000257948.3':0.035388,('PDT000468259.2':0.0303488,('PDT000319648.2':0.0251246,(('PDT000497871.1':0.0185429,('PDT000146553.3':0.0139316,'PDT000163372.4':0.0124364)'':0.0085873)'':0.0012788,('PDT000112484.3':0.0207137,('PDT000161825.3':0.0190161,('PDT000147959.3':0.015095,'PDT000086305.3':0.0151632)'':0.00780479)'':0.000933295)'':0.00169494)'':0.0033179)'':0.0023874)'':0.00777918)'':0.00331931)'':0.00632585,(('PDT000132081.3':0.0390982,'PDT000038881.1':0.0378842)'':0.0104195,('PDT000208703.1':0.0429775,(('PDT000132277.3':0.0499188,('PDT000549599.2':0.0207099,('PDT000132573.3':0.0180474,('PDT000063289.3':0.0099647,'PDT000646252.2':0.0098904)'':0.004322)'':0.00331345)'':0.0195136)'':0.00179664,((('PDT000040447.4':0.0380887,('PDT000421050.2':0.0203958,'PDT000497942.1':0.0182809)'':0.00663601)'':0.00859163,(('PDT000311116.3':0.0314498,'PDT000548562.2':0.0253729)'':0.00219131,('PDT000469880.2':0.0269277,('PDT000105651.4':0.0158948,('PDT000222747.3':0.0245037,('PDT000146299.3':0.0130832,'PDT000289030.3':0.0141841)'':0.00269781)'':0.00336964)'':0.00242899)'':0.00556929)'':0.00314656)'':0.000939847,((('PDT000296228.3':0.0271195,'PDT000107207.3':0.0220239)'':0.00509138,('PDT000481006.2':0.0229424,(('PDT000492191.2':0.0104863,'PDT000120878.3':0.0125921)'':0.00864175,('PDT000669473.1':0.0148937,('PDT000264911.3':0.0187634,(('PDT000289034.3':0.0174549,('PDT000287856.3':0.00822618,'PDT000443260.2':0.0103826)'':0.00420162)'':0.00440863,('PDT000189548.3':0.0130997,('PDT000243616.3':0.0152977,('PDT000522449.2':0.00722784,'PDT000497894.1':0.00649936)'':0.00307081)'':0.00228207)'':-0.000360108)'':0.00285395)'':0.00248911)'':0.00137947)'':0.00421223)'':-0.00239356)'':0.00156275,(('PDT000270030.3':0.0273565,('PDT000557179.2':0.0206217,'PDT000120866.3':0.0195393)'':0.00635377)'':0.001018,(('PDT000410748.2':0.0231541,'PDT000061169.3':0.0193552)'':0.00236139,(('PDT000084984.3':0.0105929,'PDT000127741.3':0.0107513)'':0.00761475,('PDT000548184.2':0.0172348,('PDT000537063.2':0.0231371,(('PDT000548494.2':0.00699012,'PDT000616354.2':0.00841748)'':0.00769991,('PDT000356220.2':0.0103433,('PDT000548495.2':0.00618368,'PDT000548553.2':0.00895732)'':0.00436058)'':0.0014632)'':0.00414727)'':0.00262069)'':-0.00191486)'':0.00447181)'':0.000268547)'':0.000865406)'':0.00565075)'':0.0072114)'':0.010922)'':-0.00569615)'':-0.00476888)'':0.0138455)'':0.000962469,((('PDT000333248.2':0.058982,('PDT000542072.2':0.0605724,('PDT000268704.3':0.0460804,('PDT000157045.3':0.0524277,'PDT000186545.1':0.0558043)'':0.00392228)'':0.00270858)'':0.00267074)'':0.00188089,((('PDT000155701.3':0.0251841,'PDT000008726.2':0.037279)'':0.00797916,('PDT000540229.2':0.0188578,('PDT000327924.2':0.016807,'PDT000107677.2':0.014721)'':0.00426363)'':0.0147017)'':0.0072899,('PDT000386992.2':0.0439242,('PDT000327946.2':0.0293577,('PDT000576254.2':0.0210165,('PDT000323305.2':0.0111585,('PDT000320831.2':0.00685171,'PDT000625873.2':0.00667119)'':0.00252139)'':0.00730579)'':0.018285)'':0.00271365)'':0.0030396)'':0.0127014)'':-0.00157488,(('PDT000281564.3':0.0499728,('PDT000379525.2':0.0270612,'PDT000115009.3':0.0316585)'':0.00849865)'':-0.000552979,(((('PDT000398524.2':0.013,('PDT000674752.1':0.0114268,'PDT000702595.1':0.00969584)'':0.00393091)'':0.0103,(('PDT000319827.2':0.0150205,'PDT000136959.3':0.0125405)'':0.00161914,('PDT000349633.2':0.0146649,('PDT000322026.2':0.0103788,('PDT000320246.2':0.0100267,'PDT000394343.2':0.00728854)'':0.00276904)'':0.000764153)'':0.00176116)'':0.00487458)'':0.0138931,('PDT000115443.3':0.0182967,((('PDT000316856.2':0.0165868,'PDT000131852.3':0.0173115)'':0.00624125,(('PDT000588070.2':0.0181851,('PDT000108556.3':0.0108321,'PDT000115479.3':0.0104571)'':0.00518936)'':0.000973978,((('PDT000343818.2':0.0139674,'PDT000414574.2':0.0106358)'':0.00160427,('PDT000629326.2':0.0138769,'PDT000651766.2':0.0125274)'':0.00229596)'':0.00314057,('PDT000649126.2':0.0213977,(('PDT000332117.2':0.0119875,'PDT000424701.2':0.00990885)'':0.00281029,('PDT000315532.2':0.0139052,('PDT000641422.2':0.0129836,'PDT000679871.1':0.0130828)'':0.000637574)'':0.000415484)'':0.0060842)'':-0.00290611)'':0.00170148)'':0.00445409)'':-0.00118451,((('PDT000635559.2':0.0134054,('PDT000316580.2':0.0107713,'PDT000344891.2':0.00880935)'':0.00181622)'':0.0042521,('PDT000131850.3':0.0116767,('PDT000378585.2':0.0102501,'PDT000496081.2':0.010862)'':0.00355992)'':0.00319863)'':0.00175327,('PDT000123308.1':0.0248348,('PDT000317175.2':0.0254785,((('PDT000440488.2':0.0108152,('PDT000333965.2':0.0103471,'PDT000123331.1':0.0123847)'':0.000408679)'':0.00198741,('PDT000115412.3':0.0129033,('PDT000384328.2':0.00947369,('PDT000332855.2':0.0075749,'PDT000131886.3':0.0097869)'':0.00161296)'':0.00377879)'':0.00154317)'':0.00318613,('PDT000498158.1':0.017086,('PDT000628871.2':0.0146375,('PDT000332716.2':0.0165686,('PDT000390825.2':0.0075534,(('PDT000332112.2':0.00725085,'PDT000619453.2':0.00798365)'':0.000210128,('PDT000326129.2':0.0101515,('PDT000552694.2':0.00499147,'PDT000132140.3':0.00766423)'':0.00221692)'':0.000981981)'':0.00251425)'':0.0037473)'':0.000741343)'':0.00190034)'':-0.00187239)'':0.00553341)'':0.000358135)'':-0.00371708)'':0.000556812)'':0.00345206)'':0.0121302)'':-0.00166012,((('PDT000322034.2':0.0119956,'PDT000546871.2':0.00967683)'':0.00688904,('PDT000341802.2':0.0225483,('PDT000658742.2':0.0105052,('PDT000611947.2':0.0155859,('PDT000326187.2':0.00881185,'PDT000131836.3':0.0108096)'':0.00224615)'':0.00327389)'':0.00395676)'':0.00175763)'':0.0089895,('PDT000131826.3':0.0552284,(('PDT000148276.3':0.0241761,('PDT000148224.3':0.0235176,('PDT000494269.2':0.0217927,('PDT000161712.3':0.0126511,'PDT000161974.3':0.0151817)'':0.00756126)'':0.00505382)'':0.00197288)'':0.0055002,(('PDT000548284.2':0.0406319,('PDT000548276.2':0.0179892,('PDT000394225.2':0.0161382,('PDT000317756.2':0.0143634,('PDT000548287.2':0.0114062,'PDT000548560.2':0.0144969)'':0.000763207)'':0.00317304)'':0.00178102)'':0.0114227)'':-0.00529964,('PDT000333956.2':0.053342,(('PDT000384047.2':0.0293205,(('PDT000316528.2':0.0171668,'PDT000576998.2':0.0181366)'':0.00777271,(('PDT000682780.1':0.018076,('PDT000333951.2':0.00575798,'PDT000552703.2':0.00698342)'':0.00944095)'':-9.97694e-05,('PDT000320281.2':0.0277987,(('PDT000414573.2':0.0059422,('PDT000160244.3':0.00536661,'PDT000123273.1':0.00629719)'':0.00327595)'':0.00505901,('PDT000325756.2':0.0186153,('PDT000314836.3':0.0102334,(('PDT000532982.2':0.00492907,'PDT000115441.3':0.00563183)'':0.00196205,('PDT000161615.3':0.00835634,'PDT000519426.2':0.00991576)'':0.0015923)'':0.00319541)'':0.00129986)'':-0.00178297)'':0.00464301)'':0.000246129)'':0.00324157)'':0.00635338)'':-0.00298796,((('PDT000407930.2':0.0244391,'PDT000410709.2':0.0208413)'':0.00208839,(('PDT000157179.3':0.0101488,('PDT000377994.2':0.00825563,'PDT000416433.2':0.0138362)'':0.00102788)'':0.00575673,('PDT000316498.2':0.0176312,('PDT000320251.2':0.012383,('PDT000519388.2':0.0162878,('PDT000324646.2':0.00989448,('PDT000155606.3':0.0107973,('PDT000155295.3':0.008,'PDT000326119.2':0.008574)'':0.00165431)'':0.000667232)'':0.00263257)'':0.00188193)'':0.00221795)'':-0.00216614)'':0.0065305)'':0.00253248,('PDT000325730.2':0.0240139,('PDT000319808.2':0.0242259,('PDT000645793.2':0.0378355,('PDT000548556.2':0.0249232,(('PDT000716360.1':0.0164749,'PDT000497947.1':0.0202069)'':0.00117374,(('PDT000319866.2':0.0126966,('PDT000548286.2':0.0137245,('PDT000548554.2':0.011146,('PDT000548235.2':0.00809851,'PDT000123316.1':0.00932999)'':0.000622284)'':0.00449059)'':0.00106051)'':-0.000463303,(('PDT000326179.2':0.0119583,'PDT000548600.2':0.0126376)'':0.00387402,('PDT000327923.2':0.0132756,('PDT000319248.2':0.0112965,'PDT000548559.2':0.00939037)'':0.00132447)'':0.00120119)'':0.000468152)'':0.00410155)'':0.00304693)'':0.00612172)'':0.00173486)'':0.00317069)'':-0.00996147)'':0.000599033)'':0.015491)'':-0.0126546)'':0.00371415)'':0.0178259)'':-0.0164194)'':0.000409718)'':0.00334244)'':0.0188488)'':-0.00431834)'':0.0278677)'':0.0849265)'':0.00632518)'':-0.00267371,(('PDT000719251.1':0.150036,(('PDT000332420.1':0.0400265,('PDT000200207.2':0.0488861,'PDT000663438.1':0.040555)'':0.00993981)'':0.000283209,('PDT000222311.1':0.0268342,('PDT000317619.1':0.0244647,'PDT000109303.2':0.0217147)'':0.00771175)'':0.00722385)'':0.100785)'':0.00585329,((('PDT000042509.1':0.0351868,'PDT000227999.1':0.0246876)'':0.124437,((('PDT000149345.2':0.0218066,'PDT000373239.1':0.0145065)'':0.0688706,('PDT000251579.1':0.0628019,('PDT000145272.2':0.0502986,('PDT000563561.1':0.0438398,'PDT000502544.1':0.0404012)'':0.0106144)'':0.0118349)'':0.0199459)'':0.016904,((('PDT000123255.1':0.044566,('PDT000674765.1':0.0379699,'PDT000716350.1':0.0440987)'':0.00451824)'':0.00665131,(('PDT000429553.2':0.0358736,'PDT000469925.2':0.0437125)'':0.0103237,(('PDT000132387.3':0.0391621,('PDT000108890.3':0.0231942,'PDT000108891.3':0.0229732)'':0.00594308)'':0.00166293,(('PDT000623048.2':0.0323534,('PDT000123330.1':0.0324377,('PDT000464151.2':0.0152108,'PDT000123343.1':0.0159314)'':0.00842719)'':0.00453027)'':0.00331945,('PDT000123278.1':0.0429754,(('PDT000322830.2':0.0193832,'PDT000123267.1':0.0225412)'':0.00825803,(('PDT000161639.3':0.00976227,'PDT000123304.1':0.0108911)'':0.00698776,('PDT000123353.1':0.00854329,('PDT000123272.1':0.00884947,'PDT000123352.1':0.00648913)'':0.00178741)'':0.0065262)'':0.0102202)'':0.0175975)'':-0.00591069)'':0.0056744)'':0.0121653)'':-0.00138503)'':0.00730171,(('PDT000123342.1':0.0969827,((('PDT000579642.2':0.0474129,'PDT000682784.1':0.0414159)'':0.00369875,('PDT000318877.2':0.0466886,'PDT000532073.2':0.0442466)'':0.00306065)'':0.00431687,(('PDT000225429.3':0.0302015,'PDT000668202.2':0.022793)'':0.0280713,('PDT000038878.1':0.0512184,'PDT000123348.1':0.0451136)'':0.00572043)'':0.000688609)'':0.0211384)'':-0.0159769,(('PDT000025748.1':0.0523884,('PDT000108880.3':0.0490825,(('PDT000108932.3':0.035939,('PDT000328099.2':0.0289854,'PDT000339401.2':0.032761)'':0.00470463)'':0.00440935,('PDT000658612.2':0.0358147,(('PDT000491386.2':0.0350545,'PDT000123355.1':0.0411443)'':0.00184564,(('PDT000108879.3':0.0298548,'PDT000123349.1':0.0255975)'':0.00494482,('PDT000123301.1':0.0210043,'PDT000123362.1':0.0189486)'':0.0111086)'':0.00585383)'':0.0017102)'':0.00555571)'':0.00925213)'':0.00104228)'':-0.00601859,(('PDT000706338.1':0.0558107,(('PDT000383996.2':0.0320019,'PDT000071659.3':0.0240563)'':0.012722,('PDT000264913.3':0.046787,('PDT000497886.1':0.0355915,(('PDT000191656.3':0.0264829,'PDT000264695.3':0.029051)'':0.00434243,('PDT000242609.3':0.02534,'PDT000360372.2':0.019212)'':0.00909152)'':0.00772899)'':0.00257891)'':0.00319947)'':0.00989322)'':-0.00679556,(((('PDT000184383.3':0.0322847,'PDT000441058.2':0.0424835)'':0.00856979,('PDT000163001.4':0.0499891,('PDT000497872.1':0.0327455,('PDT000193097.3':0.0341102,('PDT000441061.2':0.0170203,'PDT000497881.1':0.0156794)'':0.0156553)'':0.00173999)'':0.0146357)'':-0.000466674)'':0.00123749,(('PDT000619583.2':0.0426874,'PDT000108893.3':0.0523165)'':0.00158812,('PDT000145363.3':0.032254,('PDT000228254.1':0.0354038,('PDT000368342.2':0.0340561,('PDT000426907.2':0.0275323,('PDT000426892.2':0.0225866,'PDT000659130.2':0.0153856)'':0.00609295)'':0.0042715)'':0.00280191)'':0.00179469)'':0.00845238)'':0.00342558)'':0.000789473,((('PDT000137523.3':0.0409791,('PDT000630467.2':0.0241028,('PDT000496401.2':0.0230994,'PDT000710908.1':0.020847)'':0.00483846)'':0.0104956)'':-0.0031573,(('PDT000107751.3':0.0288737,('PDT000227507.3':0.0329735,('PDT000065240.3':0.0243249,('PDT000227303.3':0.0203734,('PDT000190977.3':0.0180786,('PDT000281759.3':0.00962616,'PDT000441067.2':0.00880794)'':0.00635029)'':0.00215111)'':0.00239709)'':0.00739364)'':0.00209498)'':-0.00207062,('PDT000198053.3':0.0454658,(('PDT000146470.3':0.0193056,'PDT000489615.2':0.0149146)'':0.00232945,(('PDT000440487.2':0.013091,'PDT000441062.2':0.0143239)'':0.00133504,('PDT000662027.1':0.0148868,('PDT000545354.2':0.011178,('PDT000604986.2':0.00821967,'PDT000040446.4':0.00816233)'':0.00391879)'':0.00178362)'':0.0048698)'':0.0040041)'':0.0111303)'':-0.00862109)'':0.00714581)'':0.0021592,((('PDT000400783.2':0.00811081,'PDT000579640.2':0.0109747)'':0.0267686,(('PDT000221960.3':0.0156806,('PDT000351843.2':0.00930396,'PDT000098344.3':0.0105886)'':0.00564871)'':0.00149711,(('PDT000234237.3':0.0133011,'PDT000084073.3':0.0140353)'':0.00319059,('PDT000585178.2':0.0101699,'PDT000690096.1':0.00907232)'':0.00407131)'':0.00210362)'':0.0167861)'':-0.00561647,(('PDT000421188.2':0.0425228,('PDT000566406.2':0.0345666,('PDT000421068.2':0.0393451,('PDT000488238.1':0.0196469,('PDT000566567.2':0.0211663,('PDT000289816.3':0.0215048,('PDT000480432.2':0.0151524,('PDT000626776.2':0.0163389,('PDT000342252.2':0.0131242,('PDT000163591.3':0.00880951,('PDT000510685.2':0.00949724,'PDT000102767.3':0.00942306)'':0.00142624)'':0.00337944)'':0.000610082)'':0.0021991)'':0.00165054)'':0.00402507)'':0.00257494)'':0.00464463)'':0.00229136)'':0.0129712)'':-0.0109255,(('PDT000690059.1':0.0320934,('PDT000579986.2':0.00969079,('PDT000547273.2':0.00594902,'PDT000587879.2':0.00599248)'':0.00277656)'':0.0150244)'':8.77663e-05,(('PDT000334476.2':0.0167056,'PDT000524414.2':0.0221437)'':0.00110368,(('PDT000243697.3':0.0167848,'PDT000136748.3':0.015314)'':0.00317481,('PDT000146456.3':0.0210243,(('PDT000213877.3':0.0125709,'PDT000491409.2':0.0145955)'':0.00570141,('PDT000291458.3':0.026465,('PDT000651893.2':0.0183542,('PDT000496506.2':0.012218,('PDT000040364.4':0.00738879,'PDT000488262.1':0.00710681)'':0.00725647)'':0.00214332)'':0.00157034)'':0.00013578)'':0.00170366)'':-0.000181809)'':0.00377486)'':0.00105669)'':0.00308823)'':0.00463248)'':0.0015156)'':0.0066017)'':0.0024293)'':0.00844726)'':0.00518111)'':0.00162377)'':0.0636485)'':0.0472685)'':-0.0690685,(((('PDT000367002.2':0.0393962,'PDT000136713.3':0.0366487)'':0.00925232,(('PDT000360350.2':0.0293695,('PDT000627273.2':0.019899,(('PDT000084163.3':0.0123651,'PDT000040380.5':0.0133039)'':0.00333214,('PDT000638768.2':0.014265,'PDT000133141.1':0.0149033)'':0.00110678)'':0.00302418)'':0.00468618)'':0.00683953,(('PDT000417261.2':0.0206941,('PDT000161651.3':0.0224411,'PDT000323311.2':0.0191104)'':0.00140867)'':0.00420647,(('PDT000015278.2':0.0201422,'PDT000498142.1':0.0184477)'':0.00623395,('PDT000147977.3':0.0282314,('PDT000227280.3':0.00980568,'PDT000576262.2':0.00799642)'':0.0152204)'':0.000177217)'':0.00704974)'':0.00178178)'':0.0147588)'':0.01629,('PDT000314311.3':0.076374,(('PDT000499445.2':0.0284799,('PDT000267469.3':0.0128939,('PDT000486010.2':0.00715044,('PDT000336951.2':0.00357062,'PDT000366100.2':0.00361071)'':0.0030763)'':0.00595809)'':0.00454538)'':0.00393455,(('PDT000366123.2':0.0104861,'PDT000101031.3':0.0108873)'':0.00622593,(('PDT000307869.3':0.00730889,'PDT000309576.3':0.00984311)'':0.00363827,('PDT000102800.3':0.0160274,('PDT000096877.3':0.0134121,('PDT000096893.3':0.00721291,('PDT000421054.2':0.00742839,('PDT000504995.2':0.00535711,((('PDT000190535.3':0.00346743,'PDT000112884.5':0.00341056)'':0.000734123,('PDT000238389.3':0.0035901,'PDT000259786.3':0.0037863)'':0.00125922)'':0.00146829,(('PDT000390321.2':0.00302415,'PDT000480441.2':0.00346029)'':0.00188736,('PDT000136730.3':0.00487343,('PDT000146531.3':0.00488324,'PDT000253072.3':0.00428833)'':0.00119174)'':0.00104474)'':0.000373625)'':0.000874854)'':0.000220344)'':0.000263985)'':0.00464327)'':0.000721733)'':-0.0017352)'':0.000859153)'':0.00345878)'':0.0542861)'':0.00675006)'':0.0132716,((((('PDT000306783.3':0.0358966,'PDT000300530.1':0.034043)'':0.00302591,('PDT000708735.1':0.0315411,('PDT000214025.3':0.0100236,'PDT000105090.3':0.00830315)'':0.0225428)'':0.0111781)'':0.0106234,('PDT000716364.1':0.0394872,(('PDT000319265.2':0.0293539,('PDT000344382.2':0.0258895,('PDT000155160.3':0.0215015,'PDT000131914.3':0.0250193)'':0.00544984)'':0.0034899)'':0.0102883,(('PDT000155197.3':0.0353804,('PDT000155242.3':0.0153978,'PDT000325771.2':0.0183566)'':0.0173269)'':0.000227487,('PDT000320242.2':0.0427757,('PDT000607889.2':0.0306683,('PDT000160196.3':0.0193099,('PDT000332206.2':0.0118134,'PDT000588085.2':0.0147739)'':0.00690752)'':0.0092672)'':0.00154218)'':0.00273424)'':0.0042651)'':0.00776909)'':-8.13078e-06)'':0.00463575,((('PDT000546437.2':0.0332202,('PDT000322762.2':0.0163689,'PDT000377064.2':0.0209019)'':0.00625918)'':-0.00032865,('PDT000379560.2':0.028297,('PDT000155135.3':0.0242857,('PDT000320912.2':0.0151609,'PDT000491598.2':0.0177715)'':0.00358265)'':0.00262937)'':0.00355326)'':0.00812109,(('PDT000300563.1':0.0345294,('PDT000378608.2':0.0438702,((('PDT000316877.2':0.0161646,'PDT000578363.2':0.0207292)'':0.00995004,('PDT000588069.2':0.0180657,'PDT000618556.2':0.0251381)'':0.00474589)'':0.00208126,((('PDT000468060.2':0.0249771,'PDT000132058.3':0.0252382)'':0.00216628,('PDT000320236.2':0.0309719,('PDT000326157.2':0.0236164,('PDT000520398.2':0.0249784,('PDT000716343.1':0.0168961,('PDT000491602.2':0.0174237,'PDT000607596.2':0.0168384)'':0.00160412)'':0.00478675)'':0.00104935)'':0.00305913)'':-0.000367515)'':0.00168195,(('PDT000316212.2':0.0292576,'PDT000132369.3':0.0273022)'':0.00127907,('PDT000155596.3':0.0269888,('PDT000610289.2':0.0296318,('PDT000324094.2':0.0249532,((('PDT000320245.2':0.010618,'PDT000494504.2':0.0107554)'':0.00818705,('PDT000571535.2':0.0146382,'PDT000711128.1':0.0188139)'':0.0048208)'':0.00340468,(('PDT000603034.2':0.0245608,('PDT000155392.3':0.0143971,'PDT000316849.2':0.0177816)'':0.0049352)'':0.000436551,(('PDT000641389.2':0.0218153,('PDT000651382.2':0.0155892,('PDT000155972.3':0.0190233,('PDT000328087.2':0.0152103,('PDT000576246.2':0.0104812,'PDT000136986.3':0.011884)'':0.00303605)'':0.00367003)'':0.000969404)'':0.00269772)'':-0.00146659,(('PDT000160526.3':0.0162969,'PDT000544885.2':0.0202848)'':0.00200091,('PDT000689074.1':0.0188098,('PDT000599125.2':0.0246314,('PDT000562918.2':0.0188274,('PDT000621347.2':0.0148693,('PDT000317229.2':0.0128581,('PDT000323700.2':0.00762761,'PDT000327911.2':0.00739279)'':0.00275303)'':0.000449645)'':0.00346509)'':0.00225075)'':0.00109453)'':-0.000475949)'':0.000158307)'':0.00233992)'':0.0017464)'':0.00126172)'':0.00172114)'':0.000966982)'':1.78464e-05)'':-0.00210508)'':0.00103856)'':0.00994441)'':0.00930758)'':-0.0141318,(('PDT000325053.2':0.0313718,(('PDT000378526.2':0.0151624,('PDT000389681.2':0.0134598,'PDT000131835.3':0.0137726)'':0.00559701)'':0.00550517,('PDT000332881.2':0.0248999,(('PDT000332849.2':0.0147093,'PDT000110528.3':0.0151828)'':0.00186696,('PDT000132308.3':0.0141116,'PDT000132392.3':0.0137155)'':0.00171374)'':0.00514751)'':-0.00160757)'':0.00748874)'':-0.00285683,((('PDT000325251.2':0.0029048,'PDT000332799.2':0.00327974)'':0.023156,('PDT000325742.2':0.0269319,(('PDT000155777.3':0.0206972,'PDT000131846.3':0.0160109)'':0.00359449,('PDT000641379.2':0.0175013,('PDT000161907.3':0.0164767,('PDT000160134.3':0.0147093,'PDT000519386.2':0.0179691)'':0.00599394)'':0.00045124)'':0.0039756)'':0.00271873)'':0.00160332)'':-0.000233807,('PDT000161516.3':0.0348176,((('PDT000318876.2':0.022612,('PDT000325051.2':0.0180193,'PDT000496077.2':0.0199043)'':0.00466476)'':0.000992311,(('PDT000323284.2':0.014968,'PDT000323653.2':0.0134961)'':0.00407099,('PDT000334883.2':0.017738,'PDT000485302.2':0.0190289)'':0.00143891)'':0.00294037)'':0.00205768,(('PDT000322736.2':0.0276914,'PDT000332097.2':0.0268027)'':0.00226033,(('PDT000332144.2':0.024604,('PDT000452775.2':0.026036,('PDT000716358.1':0.0270097,('PDT000710995.1':0.0158385,('PDT000158717.3':0.0153227,'PDT000321566.2':0.0141152)'':0.00273716)'':0.00197575)'':0.00365496)'':0.00340994)'':-0.00260883,(('PDT000332151.2':0.0258656,'PDT000540441.2':0.0213245)'':0.00200527,(('PDT000376131.2':0.0310097,('PDT000156972.3':0.0235026,(('PDT000615217.2':0.0167114,('PDT000410856.2':0.0149737,'PDT000601947.2':0.0177841)'':0.00142813)'':0.00215893,('PDT000398527.2':0.02461,(('PDT000161899.3':0.0167345,'PDT000532983.2':0.0150068)'':0.00215259,('PDT000131813.3':0.0158495,('PDT000324014.2':0.014851,('PDT000316163.2':0.0105132,'PDT000517226.2':0.0111979)'':0.0029347)'':0.00215622)'':-8.60939e-05)'':0.00540454)'':-0.000906035)'':0.00315578)'':0.00228163)'':-0.00157742,(((('PDT000390959.2':0.0214066,'PDT000624231.2':0.0148603)'':0.00312717,('PDT000394239.2':0.0159709,('PDT000649135.2':0.0160904,'PDT000132259.3':0.0140182)'':0.00468881)'':0.000818413)'':0.00145486,(('PDT000157744.3':0.0209815,('PDT000317162.2':0.0169054,('PDT000160450.3':0.016551,'PDT000316895.2':0.0120488)'':0.00226393)'':0.00309909)'':0.00154854,(('PDT000212517.3':0.014395,'PDT000689424.1':0.0126831)'':0.00986819,('PDT000378728.2':0.0184547,('PDT000320291.2':0.0179196,'PDT000629337.2':0.0144307)'':0.0038121)'':0.00327239)'':0.000227025)'':0.00099191)'':0.000310741,(('PDT000325078.2':0.0131924,'PDT000720386.1':0.0154807)'':0.0053034,(('PDT000160199.3':0.00831207,'PDT000326134.2':0.0107962)'':0.00961342,(('PDT000324142.2':0.0206626,('PDT000428311.2':0.0143753,'PDT000660042.2':0.0144937)'':0.00211941)'':0.000623861,(('PDT000390952.2':0.0186771,'PDT000615207.2':0.0194069)'':0.000520523,(('PDT000318628.2':0.0130839,'PDT000108888.3':0.0116465)'':0.00415879,(('PDT000132378.3':0.0160985,('PDT000123350.1':0.0135644,('PDT000332937.2':0.00869332,'PDT000394241.2':0.0102065)'':0.00156903)'':0.000803921)'':0.00169482,(('PDT000324690.2':0.0133832,'PDT000598487.2':0.0129456)'':0.0019137,('PDT000157967.3':0.0103904,('PDT000161635.3':0.00729375,'PDT000132395.3':0.00622645)'':0.00276657)'':0.00316557)'':0.00018931)'':0.00313986)'':0.00195179)'':0.000198143)'':0.00200797)'':0.000527853)'':-0.000973044)'':0.00135491)'':0.00337744)'':-0.00163557)'':0.00455602)'':-0.00323701)'':0.00974268)'':-0.0087303)'':0.000136926)'':0.00490443)'':0.00567615)'':0.00728228)'':0.00357911,(((('PDT000548498.2':0.0391869,('PDT000317244.2':0.0231486,('PDT000322737.2':0.023146,'PDT000641416.2':0.0218133)'':0.00270231)'':0.0115328)'':0.00132025,('PDT000123280.1':0.0369784,(('PDT000147705.3':0.0202306,'PDT000319283.2':0.0165442)'':0.0130276,('PDT000293422.3':0.0218272,'PDT000607586.2':0.0248186)'':0.010156)'':0.00178605)'':0.00075577)'':0.00141232,((('PDT000303476.3':0.0253484,'PDT000520380.2':0.0227238)'':0.00939543,(('PDT000271233.3':0.0143863,'PDT000342360.2':0.0180954)'':0.014814,(('PDT000203387.3':0.0155789,'PDT000247446.3':0.0166378)'':0.00977431,('PDT000317565.2':0.0226706,'PDT000525759.2':0.0241567)'':0.00203699)'':0.00645942)'':0.00350593)'':0.00542339,(('PDT000326073.2':0.0265818,('PDT000326115.2':0.0246855,('PDT000384306.2':0.0172127,(('PDT000473632.2':0.0206699,'PDT000598433.2':0.0159359)'':0.00182561,('PDT000323007.2':0.0178661,(('PDT000319888.2':0.0124759,'PDT000497794.2':0.012695)'':0.00246319,('PDT000469498.2':0.0173758,('PDT000320914.2':0.0117698,'PDT000322739.2':0.0135391)'':0.00220429)'':0.000728766)'':0.00235257)'':-5.74914e-05)'':0.0018286)'':0.00285156)'':0.00420532)'':-0.00208939,('PDT000316876.2':0.0505458,(('PDT000115451.3':0.0311914,('PDT000160206.3':0.0306106,('PDT000316915.2':0.0308066,('PDT000318643.2':0.0195828,('PDT000599138.2':0.0155556,('PDT000344014.2':0.0147108,'PDT000389692.2':0.0161175)'':0.00182071)'':0.00343958)'':0.00463542)'':0.000905923)'':0.00298512)'':-0.00263065,('PDT000327248.2':0.0272626,(('PDT000333949.2':0.0184779,'PDT000607589.2':0.0171799)'':0.00120604,(('PDT000378697.2':0.0169527,('PDT000326169.2':0.0161929,'PDT000702629.1':0.0146006)'':0.00180226)'':0.000695358,('PDT000147807.3':0.0149219,('PDT000061013.3':0.0171928,('PDT000155258.3':0.0118847,'PDT000689067.1':0.00993197)'':0.00394535)'':0.000194588)'':0.00279286)'':0.00187252)'':0.00394584)'':-0.00018203)'':0.0156233)'':-0.0122474)'':0.00988958)'':0.00409874)'':0.0183165,(((('PDT000717188.1':0.0346818,(('PDT000568974.2':0.0238932,'PDT000110491.3':0.0273689)'':0.00156443,('PDT000327190.2':0.0242846,('PDT000271223.3':0.0242472,'PDT000248582.1':0.0210602)'':0.0076782)'':0.0031616)'':0.00539753)'':-0.000810391,('PDT000591258.2':0.0455662,('PDT000417242.2':0.0343208,('PDT000131847.3':0.0249871,(('PDT000616668.2':0.0213192,('PDT000143268.3':0.0109949,'PDT000190138.3':0.0145826)'':0.0040187)'':0.00298221,('PDT000145385.3':0.0219106,('PDT000277165.3':0.0224459,('PDT000414684.2':0.021872,(('PDT000184363.3':0.0145975,'PDT000293105.3':0.0165724)'':0.00227654,('PDT000212382.3':0.0150951,('PDT000400842.2':0.0117511,'PDT000644533.2':0.0144684)'':0.00218671)'':0.00217166)'':0.00255254)'':0.00427498)'':0.00110575)'':-0.00144967)'':0.00793374)'':0.003042)'':0.00550243)'':-0.00816215)'':0.00432743,('PDT000549597.2':0.0302057,(('PDT000084089.3':0.0285499,(('PDT000156525.3':0.0275236,('PDT000378646.2':0.021161,'PDT000711401.1':0.0208864)'':0.00365026)'':0.00156545,('PDT000096874.3':0.0361338,(('PDT000639326.2':0.0248181,'PDT000040396.4':0.0277775)'':0.0054304,(('PDT000362565.2':0.0159936,'PDT000407547.2':0.0194117)'':0.00774259,('PDT000708185.1':0.016947,'PDT000101105.3':0.0170679)'':0.00637561)'':0.00307129)'':0.00317084)'':0.00142139)'':0.00783317)'':-0.00410702,('PDT000161869.3':0.0377033,((('PDT000416435.2':0.0207617,('PDT000314200.3':0.0193643,'PDT000318589.2':0.0230385)'':0.00286121)'':0.00484513,('PDT000623053.2':0.0280526,('PDT000548293.2':0.0233321,('PDT000649129.2':0.021231,(('PDT000316453.2':0.00899371,'PDT000326089.2':0.0100302)'':0.00225785,('PDT000327961.2':0.0144727,'PDT000131991.3':0.012223)'':0.000585346)'':0.00515229)'':0.00419425)'':0.00197239)'':-0.00275062)'':0.00465096,(('PDT000132315.3':0.0285936,(('PDT000465792.2':0.0150073,'PDT000008719.2':0.00966196)'':0.0111271,('PDT000332196.2':0.0199321,('PDT000317283.2':0.0143212,'PDT000327997.2':0.0160655)'':0.00194857)'':0.00912197)'':0.00261642)'':8.45607e-05,('PDT000326228.2':0.0287479,('PDT000123315.1':0.0289012,('PDT000321598.2':0.0248822,('PDT000320844.2':0.0184181,('PDT000318824.2':0.0199054,('PDT000160468.3':0.0140282,'PDT000316595.2':0.0136975)'':0.00172783)'':0.00315634)'':0.00467272)'':0.00203682)'':0.000472879)'':-0.00011975)'':0.00174826)'':0.00454604)'':-0.0012292)'':0.00443461)'':-0.00104155)'':0.00935912,(((('PDT000653367.2':0.0176291,'PDT000137563.3':0.0190201)'':0.00952162,('PDT000270079.3':0.0257359,('PDT000264774.3':0.0184281,'PDT000655540.2':0.0175148)'':0.00661505)'':0.00328415)'':0.00187032,(('PDT000025359.1':0.022142,('PDT000275741.3':0.0191557,('PDT000190369.3':0.0141374,'PDT000275646.3':0.0131111)'':0.00558511)'':0.00947983)'':-0.0031155,(('PDT000409196.2':0.0197563,'PDT000082143.3':0.0158106)'':0.00408173,(('PDT000357299.2':0.0268592,'PDT000417849.2':0.0189925)'':0.000259571,(('PDT000152697.3':0.0111992,'PDT000386836.2':0.0099171)'':0.00466212,('PDT000630995.2':0.0144744,('PDT000237109.3':0.0140895,'PDT000669462.1':0.0107866)'':0.00245861)'':0.00282841)'':0.00346456)'':0.00296677)'':0.00407842)'':0.00202691)'':0.0208853,(('PDT000641381.2':0.0555934,('PDT000324069.2':0.0371265,(('PDT000327985.2':0.027199,'PDT000546875.2':0.0309439)'':0.00677907,(((('PDT000161677.3':0.014295,'PDT000649570.2':0.015603)'':0.00484744,('PDT000491617.2':0.0107468,'PDT000619454.2':0.0105403)'':0.0119776)'':0.00684334,('PDT000389384.2':0.0247523,(('PDT000317236.2':0.0168951,('PDT000387016.2':0.0166698,'PDT000404827.2':0.0179656)'':0.00337068)'':-0.000125299,('PDT000714284.1':0.0217014,('PDT000314212.3':0.0221865,('PDT000611939.2':0.0193879,('PDT000320838.2':0.0156635,(('PDT000319910.2':0.0141143,('PDT000390568.2':0.0116107,'PDT000524131.2':0.00934684)'':0.00175546)'':0.00161215,('PDT000607592.2':0.0169841,('PDT000115473.3':0.0085554,'PDT000131937.3':0.0082038)'':0.00392821)'':0.00180619)'':0.00237397)'':0.000879066)'':0.00102711)'':0.0026856)'':-0.00167174)'':0.00435469)'':0.00196486)'':-0.000199594,(('PDT000324078.2':0.0195896,('PDT000332153.2':0.0138733,('PDT000503394.2':0.00966554,'PDT000583560.2':0.0123539)'':0.0045543)'':0.00323346)'':0.000137654,(('PDT000324669.2':0.0174004,('PDT000325085.2':0.0100319,'PDT000452368.2':0.00880614)'':0.00998634)'':-0.000488321,((('PDT000526252.2':0.0140145,'PDT000131837.3':0.0140432)'':0.00203465,('PDT000445192.2':0.0127837,('PDT000641265.2':0.0134778,('PDT000377189.2':0.011874,('PDT000132013.3':0.0102944,'PDT000132070.3':0.0108093)'':0.0023481)'':0.00189996)'':0.000914449)'':0.00168131)'':9.53788e-05,(('PDT000519090.2':0.0153702,'PDT000711093.1':0.0143084)'':0.0021825,(('PDT000115448.3':0.00966394,('PDT000320832.2':0.0101376,('PDT000333283.2':0.00640473,'PDT000503395.2':0.00535407)'':0.00213815)'':0.00210126)'':0.000612125,('PDT000132167.3':0.0144176,('PDT000115467.3':0.0121835,('PDT000115468.3':0.00973166,('PDT000115425.3':0.0100812,'PDT000131869.3':0.00839318)'':0.00364729)'':0.000737242)'':0.00186621)'':0.000297998)'':0.00214155)'':0.000876465)'':0.00267591)'':-3.25425e-06)'':0.00210711)'':0.00820533)'':0.0105891)'':0.00122603)'':-0.0109135,((('PDT000161504.3':0.0258442,'PDT000562145.2':0.0291802)'':0.00199597,('PDT000607754.2':0.0332504,('PDT000646028.2':0.0214686,('PDT000084956.3':0.0270524,('PDT000277149.3':0.0180894,'PDT000407567.2':0.0183582)'':0.00983481)'':0.00195348)'':0.00867392)'':0.00236047)'':0.00243217,('PDT000184424.3':0.078371,('PDT000038872.1':0.0431418,(('PDT000332795.2':0.0222246,'PDT000384311.2':0.0254739)'':0.00674645,(('PDT000253491.3':0.0281972,'PDT000487993.2':0.0285535)'':0.0072334,('PDT000479292.2':0.04542,(('PDT000326178.2':0.021185,('PDT000390957.2':0.0152199,'PDT000674771.1':0.0163476)'':0.0147161)'':0.00638326,('PDT000325060.2':0.0265773,('PDT000309685.3':0.0257023,((('PDT000546862.2':0.028033,('PDT000623757.2':0.0279601,('PDT000314188.3':0.0210442,('PDT000465420.2':0.0193818,('PDT000354937.2':0.0172594,'PDT000548444.2':0.0137805)'':0.00438364)'':0.000222278)'':0.00202231)'':0.00596743)'':-0.000743758,(('PDT000278343.3':0.0180968,'PDT000548548.2':0.0184562)'':0.00243238,('PDT000061101.3':0.0250257,('PDT000415190.2':0.0241581,('PDT000271201.3':0.0213395,'PDT000604988.2':0.0182108)'':0.00356645)'':0.0011846)'':0.00134761)'':0.00184684)'':0.00118697,(('PDT000325713.2':0.0268587,('PDT000326154.2':0.0234445,('PDT000588076.2':0.0152135,('PDT000517247.2':0.0135007,'PDT000131926.3':0.0128099)'':0.0076005)'':0.00298058)'':0.00371409)'':0.000175182,('PDT000110439.3':0.0441289,(('PDT000571577.2':0.0175729,('PDT000548178.2':0.00898753,'PDT000601935.2':0.00969287)'':0.00562106)'':0.00682807,('PDT000221738.3':0.0260086,(('PDT000248589.1':0.0274392,(('PDT000299799.3':0.0146702,'PDT000623982.2':0.0145413)'':0.0048972,('PDT000498173.1':0.019669,('PDT000498172.1':0.0215316,('PDT000273539.3':0.011263,'PDT000421070.2':0.00976037)'':0.0103424)'':0.0031879)'':0.000608041)'':0.00362794)'':0.000223013,('PDT000533675.2':0.0160516,('PDT000548164.2':0.0196319,(('PDT000399851.2':0.0133028,'PDT000643645.2':0.0119886)'':0.003381,('PDT000548292.2':0.0152549,('PDT000548299.2':0.0121524,'PDT000610015.2':0.0105568)'':0.00110324)'':0.00386321)'':0.00171349)'':0.000529479)'':0.00268291)'':0.00262506)'':-0.00140926)'':0.00859073)'':-0.00652275)'':0.000405178)'':0.0042434)'':0.00154748)'':-0.000322402)'':0.00697281)'':-0.00694826)'':0.00159951)'':0.0117604)'':0.0264849)'':-0.0335053)'':-0.00353671)'':0.0191822)'':-0.00485977)'':0.00764083)'':0.00212326)'':0.040078)'':0.0173386)'':0.0664522)'':-0.0120686)'':0.0158706,((((('PDT000325518.2':0.0370025,('PDT000310638.3':0.0225071,('PDT000682699.1':0.0243274,('PDT000140453.3':0.0231159,('PDT000430795.2':0.00860897,('PDT000417852.2':0.00833902,'PDT000313452.1':0.00887838)'':0.00491693)'':0.00676691)'':0.00243834)'':0.00884139)'':0.0153431)'':0.0446212,((('PDT000258743.2':0.0767233,'PDT000500476.1':0.0761827)'':0.013106,('PDT000285937.2':0.0432062,'PDT000485545.1':0.0357005)'':0.0446521)'':0.0148726,(('PDT000309385.2':0.0236312,('PDT000309388.2':0.020143,'PDT000565965.1':0.0129058)'':0.00711095)'':0.0359181,('PDT000502638.1':0.0756891,('PDT000357677.1':0.0360293,('PDT000266425.2':0.0232686,'PDT000249904.1':0.0223908)'':0.0222029)'':0.0170941)'':-0.00367307)'':0.0189633)'':0.00688005)'':0.0137031,(((('PDT000431845.2':0.091782,'PDT000599144.2':0.084764)'':0.000726318,('PDT000321655.2':0.031474,'PDT000679753.1':0.039753)'':0.0551762)'':0.00100398,((('PDT000406992.1':0.0222723,('PDT000311091.3':0.0177578,'PDT000132165.3':0.01909)'':0.00606723)'':0.0239229,('PDT000616197.2':0.0450278,('PDT000573791.2':0.031548,'PDT000120852.3':0.0411258)'':0.0118052)'':0.00319327)'':0.00727562,(('PDT000083257.3':0.0509423,(('PDT000317706.2':0.0317308,('PDT000503392.2':0.0253895,'PDT000679490.1':0.0267589)'':0.00947473)'':0.0145684,('PDT000353586.2':0.0441305,('PDT000495245.2':0.0286168,'PDT000406996.1':0.0203156)'':0.0103384)'':0.0162087)'':0.00548778)'':-0.00172903,('PDT000314853.3':0.041108,('PDT000132099.3':0.0327744,(('PDT000366110.2':0.0214639,'PDT000499250.2':0.0214088)'':0.0113282,('PDT000404039.2':0.0249807,('PDT000297759.3':0.0127226,'PDT000487994.2':0.0142937)'':0.0095394)'':0.00573851)'':0.00203557)'':0.0079853)'':0.0202043)'':0.00577251)'':0.0197482)'':0.0224984,('PDT000322438.2':0.114624,(((('PDT000258397.2':0.0358116,'PDT000576974.1':0.0697314)'':0.032441,('PDT000326053.1':0.058891,'PDT000502400.1':0.052505)'':0.010916)'':0.00199125,('PDT000415739.1':0.0974605,('PDT000482527.1':0.0747184,(('PDT000257700.2':0.0562645,'PDT000394857.1':0.0629895)'':0.00846729,('PDT000372003.1':0.0422019,'PDT000248853.1':0.0390427)'':0.0122096)'':0.00876944)'':0.0104998)'':-0.00425363)'':0.00378703,(('PDT000665168.1':0.0797645,'PDT000252455.1':0.0858755)'':0.0025204,(('PDT000288611.2':0.0825036,('PDT000565863.1':0.0590241,('PDT000397585.1':0.0482393,('PDT000714593.1':0.0245607,('PDT000470075.1':0.0108076,'PDT000668479.2':0.010391)'':0.00523926)'':0.0149331)'':0.015671)'':0.0132333)'':0.00038926,('PDT000343776.1':0.085348,(('PDT000332088.1':0.0424475,('PDT000339377.1':0.0402537,('PDT000150235.2':0.0253673,('PDT000539302.1':0.0227428,'PDT000251885.1':0.0198541)'':0.00649784)'':0.00544129)'':0.0100991)'':0.027689,(('PDT000178605.2':0.0428689,'PDT000346971.1':0.0462419)'':0.00788244,('PDT000502429.1':0.0549064,('PDT000150310.2':0.00958441,'PDT000626545.1':0.0126091)'':0.0484284)'':0.00482298)'':0.00115291)'':0.0104897)'':-0.00200967)'':0.0027051)'':6.47722e-05)'':0.0225279)'':-0.0126099)'':0.0100425)'':0.0265185,(('PDT000230077.2':0.124026,('PDT000502556.1':0.0287353,('PDT000354214.1':0.0371216,('PDT000395017.1':0.0279168,'PDT000354182.1':0.020103)'':0.00501631)'':0.00357525)'':0.0860031)'':0.00954097,((('PDT000003865.2':0.0770002,('PDT000645542.2':0.104974,'PDT000352439.1':0.0795447)'':0.00529876)'':0.05208,((('PDT000235077.3':0.0281974,'PDT000668194.2':0.0240047)'':0.0477743,(('PDT000556981.1':0.0636303,('PDT000386714.1':0.0515269,'PDT000641342.1':0.0506081)'':0.00856774)'':0.00263952,(('PDT000046039.2':0.0575042,'PDT000354181.1':0.0464418)'':0.00642537,('PDT000326067.1':0.0590403,'PDT000691089.1':0.0492067)'':0.00385926)'':0.00213941)'':0.0110226)'':0.0420588,(('PDT000594469.1':0.029797,'PDT000346496.1':0.0223547)'':0.0733359,('PDT000497207.1':0.121529,('PDT000129820.2':0.0681243,('PDT000250233.1':0.0539361,((('PDT000502187.1':0.042001,('PDT000339364.1':0.0284184,'PDT000095804.2':0.032282)'':0.00333452)'':0.00146682,('PDT000637354.1':0.0372431,('PDT000375013.1':0.0301215,('PDT000365300.1':0.0318582,'PDT000471004.1':0.0253881)'':0.00225146)'':0.00583364)'':0.000405823)'':0.00244628,((('PDT000399454.1':0.0352975,'PDT000714676.1':0.0313175)'':0.00586551,('PDT000069643.1':0.018241,'PDT000373315.1':0.0181388)'':0.0150259)'':0.0019039,('PDT000309384.2':0.0221402,('PDT000720167.1':0.0213917,('PDT000346509.1':0.00620013,('PDT000191841.2':0.00732488,'PDT000471007.1':0.00561822)'':0.00361887)'':0.00725664)'':0.00329851)'':0.0172871)'':0.00410081)'':0.00852165)'':0.00462545)'':0.0425441)'':0.00381066)'':0.00290781)'':0.0173753)'':-0.0151554,(('PDT000604782.2':0.142787,('PDT000156507.3':0.0588422,('PDT000548296.2':0.0350174,'PDT000040404.4':0.0351478)'':0.0233792)'':0.0866324)'':0.00595675,(((('PDT000189666.3':0.0616137,'PDT000674820.1':0.0606933)'':0.00301058,((('PDT000171601.3':0.0104797,'PDT000198413.3':0.015493)'':0.00236911,('PDT000190362.3':0.0107858,'PDT000651887.2':0.00781926)'':0.0118537)'':0.00561183,('PDT000299941.3':0.0216683,(('PDT000584616.2':0.0163183,'PDT000115001.3':0.0134706)'':0.00614499,('PDT000449052.2':0.0150482,('PDT000384006.2':0.010414,('PDT000412837.2':0.00696465,'PDT000139226.3':0.00645755)'':0.0051824)'':0.00206259)'':0.00229131)'':0.00489525)'':0.0019346)'':0.034177)'':0.0171383,((('PDT000228013.1':0.0812158,'PDT000289932.1':0.0715902)'':0.00882221,('PDT000362879.1':0.0501853,('PDT000150278.2':0.0333464,'PDT000691339.1':0.0260644)'':0.0098899)'':0.0314442)'':0.00245093,(('PDT000320795.1':0.0560079,('PDT000626529.1':0.0364272,('PDT000661782.1':0.0248872,'PDT000017777.2':0.018364)'':0.00821695)'':0.012754)'':0.00530969,('PDT000641737.2':0.0801381,('PDT000706404.1':0.042286,('PDT000323918.1':0.0163704,'PDT000502050.1':0.020619)'':0.0170297)'':0.0152976)'':-0.00296869)'':0.0122691)'':0.000644903)'':0.0342603,((('PDT000557180.1':0.0553067,('PDT000290383.4':0.0332765,'PDT000645088.1':0.0380131)'':0.015629)'':0.00437532,((('PDT000497348.1':0.0333501,'PDT000251456.1':0.0464003)'':0.0109138,('PDT000652464.1':0.0300793,'PDT000228385.1':0.0247809)'':0.0205309)'':0.00299441,(('PDT000088266.2':0.036883,'PDT000251551.1':0.0353441)'':0.00592037,(('PDT000306620.2':0.0402588,'PDT000644841.1':0.0407339)'':0.00627099,('PDT000345928.1':0.0491241,('PDT000558420.1':0.0428387,'PDT000510088.1':0.0460971)'':0.00898453)'':0.00104375)'':0.000403053)'':0.00309234)'':0.00487734)'':0.0242241,((('PDT000226496.2':0.0417446,'PDT000051179.1':0.0357673)'':0.0069897,(('PDT000557341.1':0.0290033,'PDT000345982.1':0.0314309)'':0.00389454,('PDT000355738.1':0.0221272,('PDT000394876.1':0.00627831,'PDT000626554.1':0.00635729)'':0.0120073)'':0.0175267)'':0.00921514)'':0.00669421,(('PDT000239208.2':0.0565598,'PDT000248769.1':0.0512642)'':0.00399028,(('PDT000285408.2':0.0474498,'PDT000403088.1':0.0473835)'':0.00344089,('PDT000060762.2':0.0444556,('PDT000235736.2':0.0380085,('PDT000496899.1':0.0120396,'PDT000538826.1':0.0152925)'':0.0332913)'':0.00998145)'':0.00823209)'':0.00224803)'':0.00128683)'':0.0249936)'':0.0289095)'':0.017268)'':-0.0109222)'':0.0249293)'':-0.0193342)'':0.028894,(((((('PDT000285478.2':0.107386,'PDT000228332.1':0.112161)'':0.00836853,(('PDT000668474.2':0.0634249,'PDT000713655.1':0.0620721)'':0.0321116,('PDT000719191.1':0.0833818,('PDT000149342.2':0.0421554,'PDT000258374.2':0.0525024)'':0.0208163)'':0.0220165)'':0.0197514)'':-0.00281099,((('PDT000082145.2':0.04808,'PDT000637081.1':0.0480977)'':0.00476006,('PDT000604309.1':0.0435672,('PDT000233042.1':0.0240027,'PDT000010580.2':0.0234858)'':0.0117448)'':0.00397132)'':0.0499141,(('PDT000216333.5':0.0865943,('PDT000270472.1':0.0878788,('PDT000687356.1':0.0282538,('PDT000566327.1':0.0206747,'PDT000657010.2':0.0298593)'':0.00738476)'':0.0299547)'':0.00139488)'':0.0330135,(('PDT000372051.1':0.0414057,'PDT000501514.1':0.0560697)'':0.020269,('PDT000661142.1':0.0486601,('PDT000618883.1':0.0570645,'PDT000661123.1':0.0454625)'':0.00542671)'':0.00874564)'':0.0324737)'':0.00574814)'':0.00705888)'':0.0123935,(((('PDT000662361.1':0.0486596,('PDT000215126.3':0.0479176,('PDT000360418.2':0.0393637,'PDT000649098.2':0.0366845)'':0.00642118)'':0.004342)'':0.00579196,(('PDT000162974.4':0.0570797,'PDT000190966.3':0.0453113)'':0.00473051,(('PDT000189537.3':0.043943,('PDT000377046.2':0.0354576,'PDT000413934.2':0.0331383)'':0.0010765)'':0.0106663,('PDT000709056.1':0.0516984,('PDT000548336.2':0.0208789,('PDT000548269.2':0.0115474,'PDT000548337.2':0.00849248)'':0.00469345)'':0.0356937)'':0.00475917)'':0.0066688)'':0.000216453)'':0.0182699,(('PDT000607562.1':0.0777643,('PDT000233305.1':0.0498955,('PDT000181816.2':0.049458,('PDT000607575.1':0.0585637,'PDT000661144.1':0.0481233)'':0.00361874)'':0.00385273)'':0.0121276)'':0.0117751,('PDT000661728.1':0.0742878,(('PDT000538554.1':0.0652042,('PDT000501992.1':0.047503,('PDT000339380.1':0.0337045,('PDT000468044.1':0.028095,('PDT000397948.1':0.0190722,'PDT000557254.1':0.0171718)'':0.00897799)'':0.00460814)'':0.0131001)'':0.00446379)'':-0.00252294,('PDT000300821.2':0.0641608,(('PDT000501862.1':0.047108,('PDT000034502.2':0.0110577,'PDT000242523.1':0.0164286)'':0.0196206)'':0.00225324,('PDT000636816.1':0.0222121,('PDT000604289.1':0.0200125,'PDT000574939.1':0.0256713)'':0.00490955)'':0.00978386)'':0.0101292)'':0.00182652)'':0.029767)'':-0.0125458)'':-0.00130262)'':0.0422968,((('PDT000228049.1':0.105897,('PDT000652431.1':0.078038,'PDT000171822.1':0.072107)'':0.0209124)'':0.020145,(('PDT000652537.1':0.103343,('PDT000619775.1':0.0549126,('PDT000228271.1':0.034584,('PDT000323904.1':0.0221585,'PDT000042675.1':0.0262681)'':0.002566)'':0.0310588)'':0.0217058)'':-0.00334282,((('PDT000469992.1':0.0415057,('PDT000557293.1':0.0226067,'PDT000251783.1':0.028683)'':0.00687399)'':0.00484291,('PDT000589391.1':0.0523197,('PDT000083483.1':0.0398822,('PDT000651940.1':0.0234653,'PDT000359643.1':0.024512)'':0.0104093)'':0.00373839)'':0.000304063)'':0.0102597,(('PDT000142215.2':0.00142534,'PDT000216687.1':0.00127277)'':0.0541132,('PDT000258447.2':0.0454367,('PDT000216684.1':0.0423191,('PDT000620253.1':0.0347543,('PDT000065362.1':0.00697581,('PDT000307436.2':0.0102673,'PDT000077522.1':0.00761474)'':0.00335999)'':0.0136071)'':0.00379463)'':0.0147821)'':0.0101124)'':0.00246356)'':0.0213351)'':0.0246778)'':-0.00583059,(('PDT000297776.2':0.0625058,('PDT000566038.1':0.0429882,'PDT000644940.1':0.0471875)'':0.0126289)'':0.0565753,(('PDT000651996.1':0.0771546,'PDT000652265.1':0.0935564)'':0.00514067,(('PDT000518476.1':0.0454865,('PDT000367763.1':0.0420037,('PDT000459048.1':0.0242245,('PDT000556685.1':0.0129227,'PDT000627215.1':0.0159197)'':0.00234737)'':0.00767498)'':0.00331284)'':0.0347118,(('PDT000649271.1':0.0543422,('PDT000388893.1':0.0384864,'PDT000480949.1':0.0480372)'':0.00698)'':0.0140367,('PDT000661133.1':0.0799065,('PDT000167718.2':0.0704612,(('PDT000399675.1':0.065401,('PDT000041490.5':0.0341356,'PDT000395336.1':0.0402333)'':0.0124551)'':0.00176619,('PDT000355551.1':0.044236,('PDT000354875.1':0.0444476,('PDT000502322.1':0.0273855,('PDT000356919.1':0.0102399,'PDT000479855.1':0.00977804)'':0.0180327)'':0.00844741)'':0.00433156)'':0.00696715)'':0.00681064)'':0.00345374)'':-0.00275023)'':0.00057073)'':0.0242264)'':-0.00292419)'':0.0201608)'':0.0155138)'':0.0100078)'':0.0130771,(('PDT000198975.3':0.101891,(('PDT000519327.1':0.0186623,('PDT000691232.1':0.00690245,'PDT000610796.1':0.00894365)'':0.00468081)'':0.0705752,(('PDT000174374.1':0.0209348,'PDT000428886.1':0.0218999)'':0.00987524,('PDT000174323.1':0.0287312,'PDT000174376.1':0.0295155)'':0.00157169)'':0.0484469)'':0.0162575)'':0.0628913,(((('PDT000498734.1':0.124242,('PDT000672220.1':0.0905863,('PDT000584741.1':0.0839866,(('PDT000391858.1':0.0648507,('PDT000425695.1':0.0220007,'PDT000535104.1':0.015267)'':0.0258004)'':0.00108862,(('PDT000633487.1':0.0299546,'PDT000297981.1':0.0338399)'':0.0300535,('PDT000675619.1':0.0400553,('PDT000675204.1':0.0143763,'PDT000415844.1':0.0103077)'':0.0166549)'':0.00747462)'':0.013289)'':0.00937101)'':0.00729134)'':0.0526946)'':0.0119355,(('PDT000661285.1':0.0457588,('PDT000181761.2':0.0528854,(('PDT000600525.1':0.0233886,('PDT000600521.1':0.0091219,'PDT000476469.1':0.0098115)'':0.0115801)'':0.0180109,('PDT000101140.2':0.0552635,('PDT000426497.1':0.0232769,('PDT000175992.2':0.0249771,'PDT000720276.1':0.0190957)'':0.0104899)'':0.014019)'':-0.00260904)'':0.00776236)'':0.00267039)'':0.00454434,((('PDT000425858.1':0.0120209,'PDT000206603.1':0.010274)'':0.00905453,('PDT000589780.1':0.021091,'PDT000672240.1':0.0226528)'':0.0021212)'':0.047624,(('PDT000675533.1':0.0510027,'PDT000682322.1':0.0512493)'':0.00960305,(('PDT000691331.1':0.0467661,('PDT000672294.1':0.0277616,'PDT000675407.1':0.0253064)'':0.010984)'':-5.81848e-05,('PDT000417608.1':0.0331707,('PDT000426478.1':0.0184286,'PDT000535505.1':0.0293932)'':0.00708074)'':0.00408753)'':0.0216743)'':0.00157857)'':0.00531764)'':0.0727903)'':-0.000529397,((('PDT000618827.1':0.0847522,('PDT000421764.1':0.0620675,('PDT000434738.1':0.0571782,'PDT000535650.1':0.0684828)'':0.010173)'':0.0203026)'':-0.000680488,(('PDT000083503.1':0.0439524,('PDT000625514.1':0.0311748,'PDT000641823.1':0.0369385)'':0.01668)'':0.0254984,(('PDT000426066.1':0.0332667,'PDT000019238.2':0.038773)'':0.0158026,(('PDT000082641.1':0.0135641,'PDT000083576.1':0.0171945)'':0.0138953,('PDT000675287.1':0.0111939,('PDT000155685.2':0.00684767,'PDT000538560.1':0.00560003)'':0.00123591)'':0.0105865)'':0.0283155)'':0.0216625)'':0.0133751)'':0.0300034,((('PDT000537530.1':0.11633,('PDT000570874.1':0.0728595,'PDT000429818.1':0.0994395)'':0.0211903)'':0.00227792,(('PDT000720468.1':0.0431941,'PDT000340090.1':0.0447296)'':0.0325876,('PDT000423994.1':0.0957577,(('PDT000014156.3':0.0048395,'PDT000015861.2':0.0103602)'':0.059059,('PDT000551623.1':0.0285448,('PDT000645089.1':0.0145279,'PDT000672660.1':0.00932203)'':0.005707)'':0.0516839)'':0.00786523)'':-0.00137057)'':0.0301431)'':-0.00626403,((('PDT000484685.1':0.0454658,('PDT000484663.1':0.0158988,'PDT000484707.1':0.0213358)'':0.0255552)'':0.0168892,('PDT000484688.1':0.0605507,(('PDT000484660.1':0.0361184,'PDT000484676.1':0.0299111)'':0.00881548,(('PDT000484669.1':0.0249132,'PDT000484684.1':0.0246169)'':0.00816071,('PDT000484677.1':0.0248204,(('PDT000484708.1':0.00851429,'PDT000484714.1':0.00945101)'':0.0100142,('PDT000484675.1':0.0103829,('PDT000484687.1':0.00980641,'PDT000484706.1':0.00775309)'':0.00502371)'':0.000409578)'':0.0144066)'':0.00569049)'':0.00579351)'':0.010367)'':0.00238365)'':0.00912228,(('PDT000710996.1':0.0595534,('PDT000357642.1':0.0356779,('PDT000428868.1':0.0239886,('PDT000014151.3':0.0125412,'PDT000264296.1':0.0116892)'':0.0118965)'':0.0205821)'':0.00853154)'':0.00743996,((('PDT000010917.2':0.0535334,('PDT000034366.1':0.0189799,('PDT000034367.1':0.0168811,'PDT000034380.1':0.0127339)'':0.011513)'':0.0179854)'':0.010555,('PDT000428864.1':0.0616843,((('PDT000558672.1':0.0368585,'PDT000428867.1':0.0420328)'':0.00195502,('PDT000429696.1':0.0330794,(('PDT000429734.1':0.00271758,'PDT000429743.1':0.00296764)'':0.00634823,('PDT000429745.1':0.00450518,'PDT000429777.1':0.00666802)'':0.00176209)'':0.0214383)'':0.00755332)'':0.00710268,(('PDT000429706.1':0.0335982,('PDT000422543.1':0.0214326,('PDT000429773.1':0.0148724,(('PDT000429769.1':0.0128102,'PDT000429771.1':0.0107752)'':0.00377665,('PDT000429805.1':0.0118536,('PDT000429733.1':0.00751056,'PDT000435632.1':0.00936624)'':0.00284775)'':0.00115782)'':0.00566182)'':0.00453408)'':0.00424765)'':0.00202707,(('PDT000428901.1':0.0289808,('PDT000428876.1':0.0142051,'PDT000428890.1':0.0161069)'':0.00527753)'':0.0119065,(('PDT000429692.1':0.0255296,('PDT000429677.1':0.0127314,'PDT000429695.1':0.0136964)'':0.00367401)'':0.0110062,('PDT000429675.1':0.0266477,('PDT000250151.1':0.0208409,('PDT000429672.1':0.0156505,'PDT000429689.1':0.0177815)'':0.00511315)'':0.0034493)'':0.00116272)'':0.00485534)'':0.00263411)'':0.007316)'':0.00827625)'':0.000125803)'':-0.00267928,((('PDT000015700.3':0.0255524,'PDT000017247.2':0.0222646)'':0.00307086,('PDT000422534.1':0.0228701,('PDT000017244.3':0.0168844,('PDT000017246.3':0.0157112,('PDT000017245.3':0.0139763,('PDT000015923.2':0.0139717,(('PDT000014134.3':0.00670901,'PDT000015784.3':0.00989139)'':0.00135666,('PDT000015928.3':0.0136202,(('PDT000015780.3':0.0109509,'PDT000015924.3':0.0119921)'':0.000280616,('PDT000015925.3':0.0121055,('PDT000015927.3':0.00997312,'PDT000015754.3':0.00827538)'':0.000540079)'':0.000454541)'':0.0005306)'':-0.00018671)'':0.00124531)'':0.000922)'':0.000338518)'':0.00410035)'':0.00983945)'':0.00281457)'':0.00661741,('PDT000429791.1':0.0307122,(('PDT000429761.1':0.00969218,'PDT000429762.1':0.00894752)'':0.0321957,((('PDT000429718.1':0.0233742,'PDT000429737.1':0.029118)'':0.00912669,(('PDT000015777.3':0.0252753,'PDT000015809.3':0.0168373)'':0.0110385,('PDT000015759.3':0.0272885,'PDT000429670.1':0.0200784)'':0.0015279)'':0.00276316)'':0.000494905,('PDT000023664.2':0.0357081,('PDT000014130.3':0.0288414,('PDT000015767.2':0.0331496,('PDT000429731.1':0.0251598,('PDT000015776.3':0.0216474,('PDT000422541.1':0.0101486,('PDT000015801.3':0.00891274,'PDT000240800.1':0.00814976)'':0.00377791)'':0.00800894)'':0.00621243)'':0.00168929)'':0.00136409)'':0.000851727)'':0.000985977)'':0.00292681)'':0.00666915)'':-0.00311659)'':0.0113981)'':0.00878232)'':0.0101736)'':0.0377523)'':0.00490525)'':0.0222356)'':0.00472265,(((('PDT000252463.1':0.0616902,('PDT000502460.1':0.0802358,('PDT000288548.3':0.0184251,('PDT000258499.3':0.0135116,'PDT000486492.1':0.00840287)'':0.00542046)'':0.0417753)'':0.01398)'':0.0244982,(('PDT000347483.2':0.0238787,'PDT000503679.2':0.0220972)'':0.00286272,('PDT000372142.2':0.0213624,('PDT000501720.2':0.0191048,('PDT000347490.2':0.0189011,'PDT000501591.2':0.0155271)'':0.00201411)'':0.00139646)'':0.00958847)'':0.0875691)'':0.0203458,((('PDT000661143.1':0.0841216,'PDT000383093.1':0.0653494)'':0.021939,('PDT000003793.2':0.047977,(('PDT000443389.1':0.0284533,'PDT000395132.1':0.0223475)'':0.00495984,('PDT000617733.1':0.0239559,('PDT000417382.1':0.017404,'PDT000395205.1':0.0107889)'':0.00886472)'':0.00641741)'':0.0218968)'':0.0520493)'':0.00126049,(('PDT000287996.1':0.0887302,('PDT000228207.1':0.0705483,('PDT000034377.1':0.00711009,('PDT000034379.1':0.00928257,'PDT000034384.1':0.00654773)'':0.00399581)'':0.0792214)'':0.00693064)'':0.0102772,(('PDT000675548.1':0.00340181,'PDT000212803.1':0.00318436)'':0.0555375,(('PDT000306739.3':0.0234329,'PDT000063252.3':0.0420741)'':0.0275163,('PDT000600790.2':0.0155637,('PDT000385912.2':0.00971058,'PDT000063172.3':0.011585)'':0.00731886)'':0.0433739)'':0.00491952)'':0.0302095)'':0.00235663)'':0.0144916)'':0.0195521,(((('PDT000081775.1':0.0837635,('PDT000502022.1':0.0676299,('PDT000500525.1':0.0663804,('PDT000443384.1':0.00868885,'PDT000518003.1':0.00852335)'':0.0502845)'':0.0120722)'':0.0278632)'':-0.00880531,('PDT000174369.1':0.0675539,(('PDT000174316.1':0.0433964,'PDT000174318.1':0.0489646)'':0.0110595,('PDT000015714.3':0.0144919,(('PDT000015710.2':0.0133106,'PDT000015753.2':0.0137325)'':0.00212311,('PDT000015804.3':0.0129207,('PDT000015816.3':0.0124613,('PDT000015803.2':0.0113636,('PDT000015788.3':0.0107138,('PDT000015827.3':0.00986066,('PDT000014138.3':0.00418635,'PDT000015806.3':0.00784865)'':0.00144059)'':0.000383275)'':0.00206498)'':0.000713503)'':0.00167146)'':-0.000139589)'':0.00171625)'':0.0456697)'':0.0151058)'':0.0258489)'':0.0281695,(('PDT000683605.1':0.104717,(('PDT000691337.1':0.0922415,('PDT000258494.3':0.102862,'PDT000655878.1':0.0723567)'':0.0153815)'':0.00159715,('PDT000152944.3':0.110059,(('PDT000557051.1':0.0728284,('PDT000495048.1':0.0543904,'PDT000251424.1':0.0391012)'':0.0131693)'':0.0204047,(('PDT000397926.1':0.0744238,'PDT000556755.1':0.0522702)'':0.0109617,('PDT000537527.1':0.0752318,('PDT000140530.3':0.0351038,(('PDT000147512.3':0.00637418,'PDT000298478.3':0.0121221)'':0.0137902,('PDT000296010.3':0.0207611,('PDT000215122.3':0.0151026,('PDT000336894.2':0.0130622,('PDT000140558.3':0.00656483,('PDT000150921.3':0.00479934,('PDT000146571.3':0.00423236,'PDT000407548.2':0.0033502)'':0.000374142)'':0.00185074)'':0.00194819)'':0.00420186)'':0.00397836)'':0.00403715)'':0.0158426)'':0.0293289)'':0.0149737)'':0.0101035)'':0.0180505)'':-0.00373213)'':0.0102287)'':-0.00165143,((('PDT000717154.1':0.0745108,('PDT000189718.3':0.0288287,'PDT000300533.1':0.0383851)'':0.0414273)'':-0.000849025,('PDT000548541.2':0.0310743,('PDT000491538.2':0.0202314,('PDT000227534.3':0.0187899,'PDT000525270.2':0.0161007)'':0.00241359)'':0.0107656)'':0.0334808)'':0.0315086,((('PDT000188173.2':0.0821499,('PDT000502883.1':0.0653109,('PDT000544286.1':0.0254742,'PDT000262731.1':0.0193862)'':0.0272024)'':0.0175927)'':-0.00274561,((('PDT000556019.2':0.0522256,'PDT000501947.1':0.042875)'':0.00585177,(('PDT000367050.2':0.0395259,'PDT000105029.3':0.0252586)'':0.00666763,('PDT000502291.1':0.000854253,'PDT000502293.1':0.000844557)'':0.0418745)'':0.0165756)'':0.0062431,(('PDT000216229.3':0.0208456,('PDT000147473.3':0.0195033,'PDT000691406.1':0.0163167)'':0.00456224)'':0.0271644,('PDT000082137.3':0.0443458,(('PDT000598496.2':0.035565,('PDT000140488.3':0.0217223,('PDT000336575.2':0.0161517,('PDT000542107.2':0.00825112,'PDT000542116.2':0.00735988)'':0.0098766)'':0.00574138)'':0.0115454)'':-0.00628703,('PDT000502579.1':0.0240178,(('PDT000193062.3':0.0230034,'PDT000351897.2':0.0183793)'':0.0060035,('PDT000502363.1':0.0255583,('PDT000366959.2':0.0169444,('PDT000542110.2':0.0140619,('PDT000140216.3':0.0136521,('PDT000247440.3':0.014246,'PDT000542113.2':0.00691799)'':0.00567248)'':0.00294319)'':0.00257616)'':0.00667904)'':-0.00156421)'':0.00246507)'':-0.000798607)'':0.0261066)'':-0.00866902)'':0.0203669)'':0.0220168)'':0.010036,((('PDT000548264.2':0.0773899,'PDT000346025.1':0.0756811)'':0.0421314,(('PDT000547320.2':0.0186404,'PDT000625739.2':0.0127649)'':0.0332871,('PDT000486520.1':0.0415447,(('PDT000653284.2':0.0367308,(('PDT000459097.2':0.0149192,'PDT000502353.1':0.0164356)'':0.00571025,('PDT000288562.3':0.0217659,('PDT000242497.1':0.00718027,('PDT000288549.3':0.00404481,'PDT000486522.1':0.00374273)'':0.00198021)'':0.00836833)'':0.00994855)'':0.00505)'':0.00478958,(('PDT000261047.1':0.0387538,('PDT000203416.3':0.0239771,('PDT000502593.1':0.00993343,'PDT000502595.1':0.00989407)'':0.0161678)'':0.00857541)'':-0.00516071,('PDT000551251.2':0.0256615,(('PDT000403984.2':0.015351,'PDT000137560.3':0.0110559)'':0.00669032,('PDT000140556.3':0.0113865,('PDT000568556.2':0.00678973,('PDT000140486.3':0.00718259,('PDT000305224.3':0.00593794,'PDT000135355.3':0.00414896)'':0.00135576)'':0.00260183)'':0.00569287)'':-0.00089891)'':0.0100442)'':0.00645867)'':0.00662445)'':0.00478626)'':0.00320596)'':0.0722095)'':-0.028822,(('PDT000251700.1':0.0376597,('PDT000222010.3':0.037995,(('PDT000253115.3':0.0260834,('PDT000150143.3':0.0153429,('PDT000316178.2':0.0190599,('PDT000216632.3':0.00616627,'PDT000101024.3':0.00739843)'':0.00585385)'':0.00198009)'':0.00262875)'':0.00443013,('PDT000142532.3':0.0239313,(('PDT000494263.2':0.0199618,'PDT000538882.2':0.0151714)'':0.00473469,('PDT000280757.3':0.0198244,('PDT000468210.2':0.0153133,('PDT000235335.3':0.0107925,'PDT000285831.3':0.00912431)'':0.0031723)'':0.0021039)'':0.00179956)'':0.00479095)'':0.00118611)'':0.0145266)'':0.00521056)'':-0.00101731,('PDT000281761.3':0.0738171,('PDT000688104.1':0.0478447,('PDT000573486.2':0.0713397,((('PDT000620711.2':0.0348718,('PDT000280723.3':0.0134123,'PDT000307506.3':0.0148681)'':0.0105889)'':0.00655696,(('PDT000140459.3':0.0210456,('PDT000066901.3':0.016838,'PDT000617914.2':0.0170036)'':0.0125071)'':0.00643093,(('PDT000273522.3':0.0135458,'PDT000125467.3':0.0197244)'':0.00736683,('PDT000096820.3':0.017774,('PDT000209198.3':0.0128918,'PDT000707450.1':0.00965711)'':0.00406006)'':0.00618377)'':0.00354605)'':0.00174679)'':-6.83243e-05,(('PDT000235320.3':0.0226614,('PDT000351895.2':0.0203078,'PDT000366287.2':0.0143148)'':0.00523161)'':0.00718779,(('PDT000271241.3':0.0236365,('PDT000648856.2':0.0136257,'PDT000140503.3':0.015324)'':0.0134797)'':0.00183689,('PDT000366131.2':0.029848,(('PDT000719829.1':0.0226093,('PDT000321896.2':0.0211093,'PDT000502300.1':0.0151104)'':0.0052698)'':0.00309109,(('PDT000598300.2':0.0153098,('PDT000707584.1':0.0126022,('PDT000236265.3':0.00674546,'PDT000103588.3':0.00990294)'':0.00768443)'':0.00120455)'':0.00105749,('PDT000045889.3':0.0169654,(('PDT000312490.3':0.00959345,'PDT000388910.2':0.0102351)'':0.00470769,('PDT000300570.1':0.00635103,('PDT000271226.3':0.00355584,'PDT000523855.2':0.00433154)'':0.00188917)'':0.00355278)'':0.00423877)'':0.00268528)'':0.00443494)'':0.0107863)'':-0.00444735)'':0.0026529)'':0.000473234)'':0.0134304)'':0.00466913)'':0.00377011)'':-0.00372375)'':0.0373554)'':0.00451192)'':0.00889463)'':0.0132204)'':0.00643992)'':0.00317327,((((('PDT000395456.1':0.0635488,('PDT000167640.2':0.0513337,'PDT000439451.1':0.0309152)'':0.0127518)'':0.0155842,((('PDT000109373.2':0.0371467,'PDT000306271.1':0.0414317)'':0.000832044,('PDT000714245.1':0.0374575,'PDT000395189.1':0.0406826)'':0.00949013)'':0.0220804,('PDT000250242.1':0.0766666,('PDT000627419.1':0.043108,('PDT000721005.1':0.0335701,'PDT000424781.1':0.0383597)'':0.00978191)'':0.00968085)'':0.00479747)'':0.00786697)'':0.00174075,(('PDT000644965.1':0.0850348,('PDT000250627.2':0.0394659,'PDT000313329.2':0.0349495)'':0.0448065)'':0.00658608,(('PDT000394701.2':0.0573637,('PDT000652503.1':0.0602091,('PDT000550333.1':0.0583358,'PDT000619672.1':0.0543702)'':0.00169336)'':0.000606054)'':0.00596703,('PDT000566002.1':0.0746419,(('PDT000003783.2':0.035846,'PDT000395350.1':0.053438)'':0.0247957,(('PDT000259587.2':0.0503245,'PDT000645099.1':0.0569895)'':0.00698626,(('PDT000566043.1':0.0476793,'PDT000566104.1':0.0483939)'':0.00150048,('PDT000650533.1':0.0405426,('PDT000003808.2':0.042793,('PDT000564926.1':0.0272891,'PDT000395403.1':0.0295524)'':0.00535769)'':0.0127956)'':0.000486871)'':0.00472036)'':0.00590844)'':0.0109676)'':0.00148863)'':0.00676592)'':0.00290358)'':0.0312417,((('PDT000556879.1':0.109397,('PDT000346079.1':0.0709989,'PDT000357694.1':0.0641671)'':0.0144099)'':-0.000210578,(('PDT000353581.2':0.0844356,('PDT000373262.1':0.0596063,'PDT000501902.1':0.0618967)'':0.0074799)'':0.000744085,(('PDT000548586.2':0.0324822,('PDT000548505.2':0.037421,'PDT000548587.2':0.0374825)'':0.00221227)'':0.0310192,('PDT000548530.2':0.0615992,(('PDT000579996.2':0.0484432,'PDT000635817.2':0.041099)'':0.00848019,('PDT000110440.3':0.0445621,('PDT000227506.3':0.0113492,('PDT000502586.1':0.00700761,'PDT000077517.1':0.00572109)'':0.00349328)'':0.034193)'':0.00537861)'':0.0102385)'':0.0030346)'':0.0138086)'':0.00205611)'':0.00318209,((('PDT000131973.2':0.0182788,'PDT000132063.2':0.0141231)'':0.00518494,(('PDT000132055.2':0.0163826,'PDT000557792.1':0.0124264)'':0.00581922,('PDT000718587.1':0.013088,('PDT000600198.1':0.0110937,('PDT000620837.1':0.00833661,'PDT000610738.1':0.00719039)'':0.00216621)'':0.00213086)'':0.00383663)'':0.00132045)'':0.00581761,('PDT000648093.1':0.105698,('PDT000610614.1':0.0376173,(('PDT000610665.1':0.0112223,(('PDT000132113.2':0.00436251,'PDT000498966.1':0.00351072)'':0.0055332,('PDT000064972.1':0.00905179,'PDT000376167.1':0.00823411)'':0.000964856)'':0.00359751)'':0.00188493,(('PDT000566586.1':0.00978772,('PDT000278450.2':0.00554502,'PDT000278457.2':0.00642508)'':0.00518683)'':0.00206017,('PDT000610623.1':0.0137892,('PDT000610556.1':0.00991218,('PDT000641371.1':0.00636172,('PDT000655230.1':0.0044235,'PDT000610733.1':0.00555256)'':0.00217585)'':0.000675362)'':0.00333754)'':0.00273465)'':0.00178526)'':0.0108696)'':0.0213298)'':-0.0243636)'':0.0713121)'':0.025151)'':0.00428336,(((('PDT000689423.1':0.0636079,('PDT000258450.2':0.0557652,'PDT000642313.1':0.0485718)'':0.0191446)'':0.0102352,('PDT000107233.2':0.0689626,('PDT000644905.1':0.0474236,'PDT000368904.1':0.0318435)'':0.0214633)'':0.022111)'':0.015158,(('PDT000672708.1':0.0409694,('PDT000672707.1':0.0212546,'PDT000672709.1':0.0199582)'':0.0127516)'':0.0589548,('PDT000124634.1':0.0824573,('PDT000396811.1':0.104563,('PDT000045613.1':0.0829306,('PDT000343726.1':0.0675513,('PDT000720244.1':0.0257592,('PDT000131778.2':0.00856788,'PDT000102988.1':0.013526)'':0.0110815)'':0.0348513)'':0.00962107)'':0.00753441)'':0.00478742)'':0.000204839)'':0.018251)'':0.010246,((('PDT000032388.1':0.000933201,('PDT000032387.1':0.000805599,'PDT000034397.1':0.000893381)'':6.58339e-05)'':0.0814986,(('PDT000407464.1':0.0595715,('PDT000355625.1':0.0561861,('PDT000258428.2':0.0413329,'PDT000579038.1':0.0250239)'':0.0232765)'':0.0153632)'':0.0188582,('PDT000150901.1':0.0793733,((('PDT000332092.1':0.0228235,('PDT000646921.1':0.0197729,'PDT000150852.1':0.0181718)'':0.00821513)'':0.015552,('PDT000557141.1':0.0550556,(('PDT000150854.1':0.00225756,'PDT000150861.1':0.00452143)'':0.0258422,(('PDT000619591.1':0.0236127,'PDT000150904.1':0.0219264)'':0.00444184,('PDT000008591.2':0.0290429,('PDT000109493.2':0.020402,'PDT000627430.1':0.0225308)'':0.0047104)'':0.00169994)'':0.00339167)'':0.00872346)'':-0.00103619)'':0.00416611,((('PDT000382832.1':0.0373919,'PDT000661726.1':0.0478454)'':0.00115674,('PDT000150903.1':0.0466977,('PDT000355756.1':0.0291094,('PDT000661832.1':0.0173692,'PDT000051554.1':0.012483)'':0.00947211)'':0.0079474)'':0.000557161)'':0.00352278,(('PDT000395447.1':0.0279954,('PDT000196642.2':0.0259591,'PDT000557068.1':0.0224896)'':0.00649227)'':0.00380711,('PDT000332423.1':0.0267873,('PDT000374995.1':0.0268029,'PDT000541640.1':0.0322622)'':0.00140364)'':0.00613587)'':0.0110807)'':0.00270236)'':0.0248899)'':0.0155919)'':0.00164479)'':0.00657536,(((('PDT000397963.1':0.0638527,'PDT000497367.1':0.0424223)'':0.0361229,('PDT000345980.1':0.0822534,('PDT000412763.1':0.0701969,'PDT000204785.1':0.0647851)'':0.00785759)'':0.000803224)'':0.00617173,(('PDT000526209.1':0.0303674,'PDT000526210.1':0.0258322)'':0.0724463,('PDT000150892.1':0.0378244,('PDT000251761.1':0.0261644,'PDT000252529.1':0.0247326)'':0.0130277)'':0.0257146)'':0.0222319)'':0.00229551,((('PDT000536996.2':0.0641746,'PDT000065368.1':0.0543934)'':0.00758037,(('PDT000331996.2':0.0455479,('PDT000288552.3':0.0211713,'PDT000486499.1':0.0169689)'':0.0160474)'':0.00764934,('PDT000497786.1':0.036957,(('PDT000439439.2':0.00586008,'PDT000468293.2':0.00548542)'':0.00145966,('PDT000497778.1':0.00590106,'PDT000497875.1':0.00714024)'':0.00026614)'':0.0142117)'':0.026289)'':0.0159436)'':0.0224457,(('PDT000502438.1':0.0929922,'PDT000502557.1':0.0848018)'':0.00660015,('PDT000352431.1':0.104241,(('PDT000644839.1':0.0874335,'PDT000395377.1':0.0699495)'':0.001625,('PDT000095682.2':0.0619357,('PDT000205436.1':0.0760808,(('PDT000583393.1':0.0431279,'PDT000318988.1':0.0394577)'':0.00844705,('PDT000430715.1':0.0244213,('PDT000470084.1':0.0164808,'PDT000501059.1':0.0193006)'':0.0139006)'':0.00835351)'':0.0138891)'':0.00469495)'':0.00299184)'':0.0135226)'':-0.00666558)'':0.0034125)'':0.00515808)'':0.00173515)'':0.0142283)'':0.00808671)'':0.00700966)'':0.0108167)'':-0.00227937)'':0.0258677)'':-0.0166907)'':0.00683693,(((('PDT000132162.3':0.0314941,('PDT000325745.2':0.00846994,('PDT000432286.2':0.00680228,'PDT000626581.2':0.00962872)'':0.00235771)'':0.02233)'':0.00312418,(('PDT000377038.2':0.0208474,'PDT000635562.2':0.0206764)'':0.00990463,(('PDT000389672.2':0.0305945,'PDT000649809.2':0.0303039)'':0.00253396,(('PDT000360490.2':0.0250722,('PDT000491885.2':0.0091837,('PDT000332201.2':0.00904532,('PDT000147810.3':0.00365634,'PDT000416439.2':0.00423025)'':0.00337018)'':0.00749384)'':0.00461306)'':0.00821874,(('PDT000317269.2':0.0307236,'PDT000637608.2':0.0305445)'':0.00605184,(('PDT000548452.2':0.0310958,('PDT000532174.2':0.0183167,'PDT000598414.2':0.0193932)'':0.00769169)'':-0.00242901,('PDT000328084.2':0.0277513,(('PDT000481065.2':0.0171892,'PDT000556616.2':0.0192284)'':0.00138878,('PDT000107668.2':0.0127026,('PDT000583546.2':0.0195228,('PDT000619473.2':0.0141115,('PDT000378723.2':0.00898006,('PDT000421174.2':0.00862654,'PDT000587169.2':0.0101496)'':0.00220014)'':0.000630506)'':0.00249424)'':0.00320221)'':0.000528141)'':0.00413025)'':0.00113618)'':0.00439182)'':-0.00219341)'':0.00389656)'':0.000633435)'':0.00587523)'':0.128023,((('PDT000250033.1':0.0880043,('PDT000359660.1':0.0526087,('PDT000687930.1':0.0301822,'PDT000251561.1':0.0274406)'':0.0149843)'':0.0133634)'':0.0101366,(('PDT000408657.2':0.0493404,(('PDT000498946.2':0.0212377,'PDT000656079.2':0.0181857)'':0.0166406,('PDT000161666.3':0.0404888,('PDT000250105.1':0.0232708,('PDT000500618.2':0.00849253,'PDT000631678.2':0.00966587)'':0.0101874)'':0.00847933)'':0.014715)'':0.00870247)'':-0.00396495,('PDT000237865.5':0.0523285,(('PDT000252768.3':0.0280564,('PDT000312538.3':0.0108381,'PDT000098346.3':0.0118394)'':0.00881185)'':0.00874467,('PDT000643077.2':0.010215,('PDT000311085.3':0.00581003,'PDT000337040.2':0.0114641)'':0.00415815)'':0.0168615)'':0.0164171)'':0.00954872)'':0.046512)'':0.0565325,((('PDT000251718.1':0.0684915,('PDT000486907.1':0.0241987,'PDT000384031.2':0.0300176)'':0.0339943)'':0.010851,('PDT000502895.1':0.0875453,('PDT000200974.3':0.0590747,('PDT000313316.2':0.0554935,'PDT000502912.1':0.0660555)'':0.00773777)'':0.0155283)'':0.00980752)'':0.0278968,((('PDT000499232.2':0.05109,(('PDT000132069.3':0.0538432,('PDT000465424.2':0.020618,('PDT000573017.2':0.0174234,'PDT000651403.2':0.0146152)'':0.0040007)'':0.0162927)'':-0.0045682,('PDT000189552.3':0.0303667,('PDT000311065.3':0.0291603,('PDT000319785.2':0.018472,('PDT000288597.3':0.0139323,('PDT000288528.3':0.0105525,'PDT000293084.3':0.0142854)'':0.00349337)'':0.00662173)'':0.00414635)'':0.00496864)'':0.0038328)'':0.0163844)'':-0.00146573,(('PDT000707373.1':0.0509929,(('PDT000443373.2':0.0269601,('PDT000333926.2':0.0169258,(('PDT000267879.3':0.00488384,'PDT000651812.2':0.00788166)'':0.00968523,('PDT000719141.1':0.00923367,'PDT000008717.2':0.012028)'':0.00238599)'':0.00593697)'':0.00219763)'':0.0035269,('PDT000641377.2':0.0322856,('PDT000494857.2':0.023306,('PDT000101028.3':0.0214518,('PDT000107689.2':0.0227121,('PDT000155137.3':0.0225837,('PDT000540385.2':0.0185666,'PDT000646094.2':0.0150336)'':0.00352288)'':0.00229597)'':0.00205197)'':0.00129994)'':0.00248237)'':-0.00122448)'':0.025922)'':-0.00989315,((('PDT000532974.2':0.0334271,('PDT000161679.3':0.0339081,'PDT000314203.3':0.0351241)'':0.00511044)'':0.00291026,('PDT000131909.3':0.0514222,('PDT000389666.2':0.0371844,'PDT000464158.2':0.0342095)'':0.00184196)'':0.00171439)'':0.00206372,(('PDT000268707.3':0.0301743,'PDT000123302.1':0.034466)'':0.00600214,((('PDT000318889.2':0.0109503,('PDT000316493.2':0.00395603,'PDT000702580.1':0.00392113)'':0.0031502)'':0.0190056,(('PDT000198976.3':0.0112205,('PDT000417811.2':0.00748014,'PDT000714481.1':0.00905706)'':0.00568922)'':0.0158537,('PDT000672406.1':0.0310669,('PDT000147507.3':0.0256638,'PDT000498124.1':0.0236996)'':0.00251468)'':0.00365271)'':0.00147357)'':0.00554485,('PDT000155219.3':0.045021,(('PDT000327948.2':0.00851251,'PDT000546438.2':0.00864509)'':0.0230496,('PDT000587501.2':0.0395739,('PDT000395697.2':0.0278368,('PDT000519172.2':0.0230122,(('PDT000230058.3':0.0224066,'PDT000650731.2':0.0210759)'':0.00255504,(('PDT000157763.3':0.0110582,'PDT000546420.2':0.00898936)'':0.006654,('PDT000377964.2':0.0146904,'PDT000394268.2':0.0124487)'':0.0027712)'':0.00315301)'':0.00241078)'':0.0020652)'':0.00669561)'':-6.05729e-05)'':0.00743359)'':-0.000197399)'':0.00276489)'':-0.000124948)'':0.00010539)'':0.00530782)'':-0.00218986,((((('PDT000457519.2':0.028291,'PDT000132180.3':0.0272374)'':0.00497946,('PDT000322768.2':0.0240233,('PDT000161865.3':0.0164812,('PDT000132386.3':0.0153655,('PDT000160218.3':0.0164958,'PDT000115476.3':0.0171955)'':0.00177278)'':0.00513367)'':0.00350788)'':0.00630111)'':-0.000283416,(('PDT000395673.2':0.0276433,('PDT000148125.3':0.0181943,'PDT000383190.2':0.0192063)'':0.00709355)'':-0.000876476,('PDT000320828.2':0.0219717,('PDT000332791.2':0.0224378,('PDT000157181.3':0.0203189,('PDT000326106.2':0.0152439,'PDT000115459.3':0.0165568)'':0.00181712)'':0.00369448)'':0.00157355)'':0.00373325)'':0.00301464)'':0.00332278,((('PDT000496094.2':0.024649,('PDT000626586.2':0.0278484,('PDT000314858.3':0.0214223,('PDT000322067.2':0.0131568,'PDT000132109.3':0.0116431)'':0.00625895)'':0.00438476)'':0.00236519)'':0.000711756,('PDT000047184.3':0.0268491,(('PDT000542758.2':0.0157626,'PDT000378581.2':0.0184617)'':0.000391196,('PDT000157807.3':0.0225035,('PDT000131876.3':0.0168362,('PDT000155891.3':0.0146963,'PDT000325746.2':0.0120894)'':0.00232126)'':0.00385289)'':0.000537184)'':0.00348451)'':0.00354257)'':0.00427447,(('PDT000481056.2':0.0117015,'PDT000655219.2':0.0163638)'':0.0156565,('PDT000316502.1':0.0256409,('PDT000190537.3':0.0270773,(('PDT000323286.2':0.0287482,'PDT000523442.2':0.0285754)'':0.000755686,('PDT000327129.2':0.0213546,(('PDT000155516.3':0.0239086,'PDT000038857.1':0.0232962)'':0.00140017,(('PDT000479290.2':0.0214606,('PDT000544883.2':0.0174926,('PDT000148017.3':0.00893614,'PDT000540382.2':0.0120047)'':0.00304274)'':0.00261601)'':0.00135509,(('PDT000333262.2':0.0130618,'PDT000690264.1':0.0132462)'':0.00431685,('PDT000140577.4':0.0120423,'PDT000110433.3':0.0162801)'':0.0028227)'':0.00112208)'':0.00341604)'':0.00279464)'':-0.0019852)'':0.00276564)'':0.00074008)'':0.00134563)'':-0.00106625)'':0.00422783)'':0.00117906,((('PDT000377997.2':0.0217701,('PDT000603044.2':0.0146611,'PDT000132398.3':0.0168731)'':0.00281571)'':0.00757106,('PDT000264879.3':0.0358615,(('PDT000144028.3':0.0148941,'PDT000589624.2':0.0170523)'':0.00979881,('PDT000250813.3':0.02149,'PDT000422015.2':0.0214091)'':0.00419269)'':0.00410683)'':0.00422236)'':0.000217467,(('PDT000107511.3':0.032993,('PDT000316190.2':0.0352831,(('PDT000342303.2':0.0232223,('PDT000222765.3':0.0160514,('PDT000623052.2':0.0155472,('PDT000575866.2':0.00640787,'PDT000687889.1':0.0101474)'':0.00720714)'':0.00229172)'':0.00467784)'':0.00203655,(('PDT000161872.3':0.0215159,('PDT000635985.2':0.0230426,('PDT000410858.2':0.0149169,('PDT000325132.2':0.0121937,'PDT000519351.2':0.015788)'':0.00502401)'':0.00288806)'':0.00266421)'':-0.000772681,(('PDT000137279.3':0.0157686,('PDT000163374.4':0.0229819,('PDT000687903.1':0.0101989,('PDT000215102.3':0.010957,'PDT000128455.3':0.0111507)'':0.00197591)'':0.00456058)'':0.00289069)'':5.83349e-05,('PDT000123265.1':0.0172055,('PDT000365732.2':0.00963275,('PDT000147486.3':0.014689,('PDT000273525.3':0.00624964,'PDT000083970.3':0.00929746)'':0.00333981)'':0.000965301)'':0.00417892)'':0.000838481)'':0.00175661)'':0.000528876)'':0.0048975)'':0.00355138)'':-0.00384293,((('PDT000384014.2':0.0360192,('PDT000188001.3':0.0198058,'PDT000311083.3':0.0215173)'':0.00686656)'':0.00140661,(('PDT000273034.3':0.0184943,'PDT000486320.2':0.019585)'':0.00130718,('PDT000313392.3':0.018657,'PDT000671526.1':0.0124347)'':0.00337352)'':0.00386432)'':0.00355714,(((('PDT000491904.2':0.0237578,'PDT000604040.2':0.0190161)'':0.00369558,('PDT000338719.2':0.0229889,('PDT000408651.2':0.0200439,'PDT000128671.3':0.0142305)'':0.0019398)'':0.00212425)'':0.00210511,('PDT000707475.1':0.0279371,('PDT000490917.2':0.0271831,(('PDT000190722.3':0.018505,'PDT000646088.2':0.0256301)'':0.00355873,('PDT000486346.2':0.0215408,(('PDT000143043.3':0.0163952,'PDT000148494.3':0.0194744)'':0.0011039,(('PDT000237866.5':0.0155765,('PDT000270096.3':0.0085984,'PDT000563532.2':0.0102639)'':0.00250603)'':0.00323834,(('PDT000280732.3':0.0110744,'PDT000619143.2':0.0118959)'':0.00400802,('PDT000504859.2':0.0139268,('PDT000338829.2':0.00480431,'PDT000665167.1':0.00645739)'':0.013652)'':0.00105366)'':0.000341758)'':0.000351357)'':0.00313077)'':0.00030235)'':0.00555723)'':0.00260878)'':-0.00325889)'':0.00105134,((('PDT000295837.3':0.0249935,('PDT000548323.2':0.0191527,'PDT000548626.2':0.022967)'':0.00126141)'':0.000692541,('PDT000328079.2':0.0193338,(('PDT000335805.2':0.0122654,'PDT000548322.2':0.013269)'':0.00517851,('PDT000319220.2':0.0161042,(('PDT000161891.3':0.0124692,'PDT000316111.2':0.010231)'':0.00509985,('PDT000061033.3':0.0105942,'PDT000124617.1':0.0126224)'':0.00475742)'':0.00139332)'':0.000328363)'':0.00305369)'':0.00275919)'':-0.00112888,('PDT000604530.2':0.0315491,(('PDT000491957.2':0.028083,('PDT000351815.2':0.0252833,('PDT000275708.3':0.0181643,'PDT000390233.2':0.0172365)'':0.0029931)'':0.00329245)'':0.0008196,(('PDT000306748.3':0.0190729,('PDT000275716.3':0.0220126,('PDT000649214.2':0.0140286,('PDT000551631.2':0.00869845,'PDT000707481.1':0.00712245)'':0.00275895)'':0.00399298)'':0.00324113)'':0.00412187,((('PDT000317575.2':0.00753866,'PDT000084225.3':0.0117859)'':0.00370689,('PDT000546086.2':0.0125808,('PDT000269991.3':0.0129792,('PDT000151434.3':0.0128685,'PDT000325438.2':0.0147334)'':0.00235445)'':0.000899547)'':0.0012051)'':0.00268665,(('PDT000309621.3':0.0230638,'PDT000679464.1':0.0172795)'':0.00260704,('PDT000366130.2':0.0372788,((('PDT000577478.2':0.0204145,('PDT000322448.2':0.017931,'PDT000624037.2':0.0144228)'':0.00153633)'':0.00369422,(('PDT000214044.3':0.0141391,'PDT000469906.2':0.0183394)'':0.00499499,('PDT000315880.2':0.0182079,'PDT000303016.3':0.0193646)'':0.000447938)'':0.000527071)'':0.000186691,('PDT000598067.2':0.0217208,(('PDT000127707.3':0.0203741,('PDT000213866.3':0.012823,('PDT000221721.3':0.0107347,'PDT000540197.2':0.00861824)'':0.00224903)'':0.00500594)'':-0.000430446,((('PDT000544296.2':0.00789107,'PDT000594259.2':0.00789053)'':0.0056765,(('PDT000366272.2':0.00339979,'PDT000584144.2':0.00437688)'':0.00140691,('PDT000085630.3':0.00880717,('PDT000407541.2':0.00482184,('PDT000147474.3':0.00176177,'PDT000096852.3':0.00273373)'':0.00244016)'':0.00102091)'':4.1305e-05)'':0.00235902)'':0.00106417,(('PDT000114660.3':0.0142859,'PDT000712824.1':0.0130883)'':0.00204589,('PDT000344792.2':0.015126,('PDT000316133.2':0.00973577,('PDT000466040.2':0.00771015,('PDT000279541.3':0.00549895,('PDT000545120.2':0.00715978,('PDT000710746.1':0.00612086,('PDT000252517.1':0.00447013,(('PDT000146514.3':0.00293121,'PDT000587156.2':0.00325086)'':3.36974e-05,('PDT000041857.3':0.00244262,'PDT000303497.3':0.00245151)'':0.000417455)'':0.000685073)'':0.00130876)'':0.000583989)'':0.0003678)'':0.000440367)'':0.000500124)'':0.00411701)'':-0.000964843)'':0.000395561)'':0.00145565)'':0.00390086)'':-0.000928837)'':0.00727002)'':-0.00231209)'':-0.00256707)'':0.00141648)'':0.00180523)'':0.0100861)'':-0.0091623)'':-0.000706399)'':0.00220854)'':-0.00084761)'':0.00243038)'':0.00392015)'':0.00974239)'':0.0880279)'':0.0168697)'':0.00594299)'':0.00253437,(((('PDT000646923.1':0.0323411,'PDT000672195.1':0.0344161)'':0.0692324,((('PDT000625420.1':0.0397825,'PDT000083577.1':0.036419)'':0.00890134,(('PDT000625451.1':0.0173706,'PDT000553177.1':0.0161302)'':0.0120673,('PDT000082643.1':0.0198588,'PDT000083480.1':0.0214975)'':0.019039)'':0.00421898)'':0.0072493,('PDT000147511.3':0.0428263,(('PDT000104687.2':0.0380487,'PDT000104686.2':0.0460924)'':0.00354862,('PDT000228335.1':0.0310404,('PDT000105638.3':0.0172159,('PDT000539882.1':0.0159068,'PDT000539895.1':0.0166119)'':0.00477963)'':0.00809205)'':0.00427289)'':0.00367738)'':0.00499172)'':0.0512617)'':0.0242566,(((('PDT000422087.1':0.0326432,('PDT000356790.1':0.0323529,'PDT000485556.1':0.0330459)'':0.00119067)'':0.0305803,('PDT000350401.1':0.058033,(('PDT000150231.2':0.042476,'PDT000494429.1':0.0499666)'':0.00425529,('PDT000502619.1':0.0436008,('PDT000355751.1':0.0183603,'PDT000382196.1':0.0199233)'':0.00953259)'':0.0110715)'':0.0123656)'':0.00768231)'':0.0014761,('PDT000343988.2':0.0865734,((('PDT000107261.2':0.0113965,'PDT000250051.1':0.0240433)'':0.0407581,('PDT000373234.1':0.0454619,('PDT000196619.2':0.052398,'PDT000131975.2':0.0440619)'':0.000541732)'':0.00231874)'':0.00897744,('PDT000502681.1':0.0829199,('PDT000502319.1':0.0678294,(('PDT000573612.1':0.0515188,('PDT000443351.1':0.0433207,('PDT000196609.2':0.0205,'PDT000502914.1':0.0238131)'':0.0158031)'':0.00915135)'':0.0121075,(('PDT000620250.1':0.0587716,'PDT000251825.1':0.0560784)'':0.00299915,(('PDT000150312.2':0.0342683,'PDT000332091.1':0.0381997)'':0.010263,('PDT000600641.1':0.0352878,'PDT000227993.1':0.0354232)'':0.0176225)'':0.00569819)'':0.000996791)'':0.0105986)'':0.000536053)'':-0.00399441)'':0.0176865)'':-0.017321)'':0.05504,(('PDT000274717.2':0.0382506,(('PDT000625443.1':0.0186893,('PDT000394255.1':0.0159191,('PDT000205082.2':0.0147365,'PDT000679755.1':0.0200487)'':0.00433113)'':0.000967652)'':0.00304812,(('PDT000404003.1':0.0165777,'PDT000468244.1':0.0220164)'':0.00656822,('PDT000702602.1':0.0169343,'PDT000652373.1':0.0118276)'':0.0150342)'':0.00158347)'':0.0161816)'':0.0763715,(((('PDT000509508.1':0.0469275,'PDT000557001.1':0.0435943)'':0.0476607,('PDT000719163.1':0.0809568,('PDT000337223.1':0.0435228,(('PDT000408777.1':0.0472371,('PDT000258371.2':0.02331,'PDT000577481.1':0.0308107)'':0.0151878)'':0.000381322,('PDT000251799.1':0.0206478,('PDT000399983.1':0.0128909,'PDT000566215.1':0.0165617)'':0.00820325)'':0.0109404)'':0.00606911)'':0.0285698)'':0.00427357)'':0.0114342,(('PDT000719075.1':0.114776,(('PDT000047703.3':0.0691854,'PDT000250210.1':0.0658386)'':0.00286346,('PDT000136741.2':0.0352356,'PDT000375006.1':0.0271723)'':0.0292903)'':0.0179539)'':-0.0103145,(('PDT000372316.1':0.0681161,'PDT000248588.1':0.0668669)'':0.0287478,(('PDT000367256.1':0.0382874,'PDT000395504.1':0.029136)'':0.0288316,(('PDT000293476.1':0.076392,('PDT000557045.1':0.0525111,('PDT000619661.1':0.0470628,'PDT000502701.1':0.0503485)'':0.00808907)'':0.0184048)'':-0.00300744,(('PDT000659705.1':0.0408411,('PDT000468321.1':0.0183656,('PDT000455213.1':0.0154756,'PDT000568754.1':0.0151769)'':0.00682317)'':0.0208927)'':0.00811827,('PDT000626533.1':0.0455609,(('PDT000196630.2':0.0369007,('PDT000250113.1':0.0115987,'PDT000251874.1':0.0134955)'':0.0143643)'':0.0117678,('PDT000395143.1':0.0260982,('PDT000306486.2':0.0169922,'PDT000337220.1':0.0162028)'':0.00480471)'':0.0199855)'':0.00632383)'':-0.00462131)'':0.0187779)'':0.00924376)'':0.00754624)'':0.00199883)'':0.0122554)'':0.00685248,((('PDT000222771.2':0.0702429,'PDT000535526.1':0.0574981)'':0.00647183,('PDT000283341.2':0.0486246,'PDT000538612.1':0.0525764)'':0.0113554)'':0.0405367,(('PDT000665114.1':0.0926879,'PDT000005858.2':0.0935851)'':0.0198667,((('PDT000124974.3':0.0937023,('PDT000242796.3':0.0120738,'PDT000312497.3':0.0139169)'':0.0648838)'':0.0114673,('PDT000566004.1':0.0741789,('PDT000250705.1':0.0498446,('PDT000174344.1':0.0350547,'PDT000261033.1':0.0477665)'':0.0103843)'':0.0241907)'':0.0210767)'':0.00603901,((('PDT000181691.2':0.124945,(('PDT000409253.2':0.0839337,'PDT000251974.1':0.0719003)'':0.00309461,((('PDT000619810.1':0.0265159,('PDT000161634.2':0.0194371,'PDT000103603.2':0.0198055)'':0.00949577)'':0.00530369,('PDT000105658.3':0.0370064,('PDT000105649.3':0.0128922,'PDT000105652.3':0.0177961)'':0.0145967)'':0.00934407)'':0.0102997,(('PDT000553153.1':0.0276223,('PDT000600200.1':0.0228572,'PDT000619804.1':0.0175629)'':0.00675667)'':0.00314929,('PDT000107660.1':0.0246755,('PDT000083527.1':0.0199964,'PDT000083571.1':0.0159329)'':0.00531124)'':0.00307814)'':0.0094554)'':0.0296696)'':0.0139341)'':-0.00964291,((('PDT000719244.1':0.019546,'PDT000251536.1':0.0229611)'':0.0619951,(('PDT000439456.1':0.00998446,'PDT000526145.1':0.00599154)'':0.047608,('PDT000589392.1':0.0626686,'PDT000346976.1':0.0738274)'':0.00626079)'':0.00861909)'':0.00919376,(('PDT000647177.2':0.0773556,('PDT000517934.1':0.0428954,'PDT000594968.1':0.0507176)'':0.0179849)'':0.00905192,(('PDT000617813.1':0.0741396,('PDT000150302.2':0.0252024,'PDT000395348.1':0.0187688)'':0.0532118)'':0.00256222,(('PDT000417392.1':0.0631896,('PDT000502890.1':0.0483879,'PDT000543099.1':0.0437669)'':0.00623404)'':0.00577477,('PDT000634351.1':0.0282562,('PDT000719228.1':0.0302498,'PDT000251513.1':0.0176099)'':0.0155838)'':0.0295064)'':0.00435073)'':0.00194298)'':0.0076104)'':-0.000195269)'':0.00105856,(('PDT000652483.1':0.0942818,(('PDT000252481.1':0.0797089,('PDT000572850.1':0.0558146,(('PDT000397733.1':0.0433725,'PDT000551911.1':0.0484701)'':0.00835986,('PDT000395247.1':0.0419404,('PDT000396807.1':0.0346952,'PDT000396841.1':0.0333122)'':0.00489189)'':0.00417344)'':0.00958799)'':0.00968102)'':-0.00610518,(('PDT000081779.1':0.0605892,'PDT000319022.1':0.0544248)'':0.00338572,(('PDT000411088.1':0.0513121,('PDT000141479.2':0.0220788,'PDT000395014.1':0.0314468)'':0.0162292)'':0.00336277,('PDT000251730.1':0.052586,('PDT000251980.1':0.0460808,('PDT000171813.1':0.0271426,('PDT000526115.1':0.0205763,'PDT000566704.1':0.0219122)'':0.00667711)'':0.00535974)'':0.00534657)'':0.000622169)'':0.0022765)'':0.00427579)'':0.0195637)'':-0.00995469,('PDT000038429.1':0.0533285,('PDT000110505.3':0.0640482,(('PDT000131859.3':0.025496,'PDT000186211.1':0.0346796)'':0.0119067,('PDT000258478.3':0.047061,(('PDT000354924.2':0.022567,'PDT000040399.4':0.028642)'':0.0191261,(('PDT000107763.3':0.0271952,'PDT000486516.1':0.0273201)'':0.00591591,('PDT000288573.3':0.0381168,(('PDT000123322.1':0.0251024,'PDT000497295.2':0.0182987)'':0.00743327,((('PDT000288591.3':0.0187363,('PDT000486513.1':0.016119,'PDT000486515.1':0.0193805)'':0.0019834)'':0.0111612,(('PDT000313246.3':0.0259332,'PDT000032316.1':0.0150879)'':0.00216015,('PDT000372132.2':0.0157706,('PDT000663863.1':0.0060108,('PDT000288538.3':0.000578105,'PDT000288546.3':0.000921145)'':0.00492587)'':0.00908709)'':0.00258163)'':0.00450016)'':0.000798154,(('PDT000108935.3':0.0224974,('PDT000307919.3':0.0158382,('PDT000235306.3':0.0154646,('PDT000065365.1':0.00911663,'PDT000486514.1':0.0101256)'':0.0029558)'':0.00450534)'':0.00763976)'':-0.00180713,(('PDT000564055.2':0.0218537,'PDT000040371.4':0.020206)'':0.00137293,(('PDT000498009.1':0.0279144,('PDT000602263.3':0.0209921,('PDT000176775.3':0.0144019,('PDT000248535.1':0.0118915,'PDT000498171.1':0.00993389)'':0.00548906)'':0.00453854)'':0.00339507)'':-0.000131694,('PDT000248755.1':0.0273783,(('PDT000544424.2':0.013003,'PDT000258779.1':0.0149538)'':0.00255394,(('PDT000133962.1':0.0101752,('PDT000619749.2':0.00896423,('PDT000571829.1':0.00586665,'PDT000248587.1':0.00431605)'':0.00425672)'':0.00182703)'':0.00169855,('PDT000487610.2':0.0190964,('PDT000140550.3':0.0142079,('PDT000176688.3':0.00900511,('PDT000222003.3':0.00947931,'PDT000498046.1':0.00868449)'':0.00144184)'':0.00115634)'':0.00130018)'':0.000761681)'':0.00378041)'':0.00328188)'':-0.00189603)'':0.00253569)'':-0.000954566)'':0.00327182)'':0.00288269)'':0.00336879)'':-0.00261846)'':0.00329766)'':0.00832681)'':0.00107154)'':0.00767016)'':0.00457857)'':0.0137586)'':0.0127925)'':0.00928994)'':0.0150016)'':-0.00240196)'':-0.0115203)'':0.0151084)'':0.0136683)'':0.00377651)'':-0.0215298,((('PDT000128452.3':0.0593281,'PDT000501847.1':0.0571269)'':0.0666355,((('PDT000140552.3':0.0614676,(('PDT000208733.1':0.0427657,('PDT000008783.2':0.0345697,('PDT000145996.3':0.0126022,'PDT000619144.2':0.0103304)'':0.0287514)'':0.00216337)'':0.0189488,('PDT000497906.1':0.0189594,('PDT000103604.3':0.0139847,('PDT000682955.1':0.010052,('PDT000707407.1':0.00827911,'PDT000140219.3':0.00736769)'':0.00527815)'':0.00232627)'':0.0042254)'':0.034652)'':0.00781957)'':-2.87676e-05,(('PDT000063242.3':0.0442199,('PDT000107471.3':0.0234654,'PDT000140565.3':0.027593)'':0.00713914)'':0.0292469,(('PDT000569717.2':0.0357697,('PDT000689148.1':0.0343035,('PDT000645782.2':0.0412758,(('PDT000558466.2':0.0202501,('PDT000562141.2':0.0120634,'PDT000711072.1':0.0124984)'':0.00299624)'':0.00231658,('PDT000711403.1':0.0252157,('PDT000469849.2':0.0132223,('PDT000629951.2':0.0115077,'PDT000040426.4':0.00879541)'':0.00835745)'':0.00373166)'':0.00273847)'':0.006134)'':0.00176847)'':0.0114599)'':0.0101061,('PDT000679117.1':0.0476392,((('PDT000415189.2':0.00654387,'PDT000657048.2':0.00851543)'':0.00633752,(('PDT000146558.3':0.0107458,'PDT000624085.2':0.0118039)'':0.013213,('PDT000639475.2':0.0147951,('PDT000285798.3':0.019192,('PDT000155966.3':0.012086,'PDT000161871.3':0.00877631)'':0.00436587)'':0.00143827)'':0.000117198)'':0.00221414)'':0.00497719,('PDT000285821.3':0.0435706,('PDT000188027.3':0.0355339,('PDT000366263.2':0.0213731,('PDT000137335.3':0.0198042,(('PDT000523347.2':0.00614159,'PDT000716356.1':0.00545031)'':0.00275312,('PDT000248636.1':0.0108278,('PDT000123354.1':0.00490349,('PDT000040433.4':0.00216282,'PDT000248581.1':0.00163035)'':0.00315103)'':0.0017409)'':0.00320214)'':0.00694748)'':0.00163172)'':0.00785258)'':0.00386002)'':-0.00786466)'':0.00903341)'':0.0135702)'':0.00778769)'':0.00209987)'':0.0101597,((('PDT000321901.2':0.0283581,('PDT000248663.1':0.0162608,('PDT000258521.3':0.0071599,'PDT000309684.3':0.0050048)'':0.00578383)'':0.00495774)'':0.0399691,(('PDT000303557.3':0.0118577,'PDT000548182.2':0.0160193)'':0.0136556,(('PDT000430821.2':0.0213335,('PDT000497959.1':0.00692458,'PDT000498010.1':0.00777982)'':0.0061997)'':0.00545156,('PDT000373198.2':0.0228847,(('PDT000548183.2':0.012036,'PDT000140511.3':0.0116438)'':0.000398061,('PDT000176695.3':0.0108473,('PDT000548179.2':0.00652151,('PDT000148509.3':0.00479454,'PDT000275600.3':0.00407)'':0.00172002)'':0.00110832)'':0.0012297)'':0.0112723)'':-0.00507176)'':0.0157584)'':0.0255042)'':0.00752843,((('PDT000421040.2':0.0321586,'PDT000300542.1':0.0300503)'':0.0044124,(('PDT000161672.3':0.0219053,'PDT000502899.1':0.0134938)'':0.0118682,(('PDT000497337.2':0.0135914,'PDT000497937.1':0.0163948)'':0.00396226,('PDT000289031.3':0.0198846,('PDT000027609.1':0.0168097,('PDT000488043.2':0.0137519,'PDT000248674.1':0.0120284)'':0.00487147)'':0.00244039)'':0.00303506)'':0.00279403)'':0.00259307)'':0.0176794,(('PDT000502327.1':0.0633794,'PDT000502330.1':0.0389966)'':0.0475555,(('PDT000311422.3':0.0323734,'PDT000321136.2':0.0454305)'':0.004734,((('PDT000188044.3':0.0248005,'PDT000365752.2':0.0204385)'':0.0197592,('PDT000420985.2':0.0371817,('PDT000625729.2':0.0183605,('PDT000618232.2':0.0166619,('PDT000151508.3':0.00878117,'PDT000193538.3':0.012614)'':0.00844504)'':0.00453169)'':0.011135)'':0.00107298)'':0.000794266,((('PDT000248540.1':0.0238894,'PDT000251887.1':0.019387)'':0.00472428,('PDT000288575.3':0.0186093,('PDT000288550.3':0.0148339,('PDT000248590.1':0.00985091,'PDT000486494.1':0.0116726)'':0.00343516)'':0.00452672)'':0.0091665)'':0.00585335,('PDT000123306.1':0.028282,('PDT000239352.1':0.0480758,('PDT000631667.2':0.033392,('PDT000502381.1':0.0353769,('PDT000672690.1':0.0269601,('PDT000498041.1':0.0266686,('PDT000443399.2':0.0157971,('PDT000502354.1':0.0183216,'PDT000502377.1':0.0199775)'':0.0093975)'':0.0021373)'':0.00494204)'':0.00179456)'':0.00160181)'':0.00359729)'':0.00138339)'':0.00226389)'':0.00977912)'':0.00768023)'':0.0165466)'':-0.0138766)'':0.0194688)'':0.00604167)'':0.0588374)'':-0.0304523,((('PDT000565119.2':0.0361841,'PDT000708039.1':0.0412015)'':0.0408712,('PDT000271297.3':0.0844203,('PDT000201161.3':0.0554077,'PDT000550372.2':0.0475283)'':0.0201597)'':0.0170082)'':0.00929902,(((('PDT000135917.3':0.0590515,('PDT000163018.4':0.0541402,('PDT000481064.2':0.0491705,('PDT000373251.2':0.0283888,'PDT000576164.2':0.0242222)'':0.0157081)'':0.0050825)'':0.0163776)'':0.0189836,('PDT000216620.3':0.0895738,(('PDT000260209.2':0.0279872,'PDT000272316.3':0.0287469)'':0.037507,('PDT000300526.1':0.0317481,('PDT000201118.3':0.0185469,'PDT000206386.3':0.0206109)'':0.0160151)'':0.0359075)'':0.0166576)'':-0.00554134)'':0.00696817,(('PDT000399307.2':0.0545951,('PDT000251890.1':0.0440504,('PDT000526241.2':0.0295625,'PDT000208713.1':0.0324326)'':0.00956496)'':0.00177199)'':0.0069587,(('PDT000322003.2':0.0312844,('PDT000389762.2':0.029347,'PDT000481032.2':0.0317765)'':0.00794512)'':0.00628435,('PDT000494259.2':0.0564794,('PDT000372140.2':0.0520085,(('PDT000213242.3':0.0360258,'PDT000248623.1':0.0186065)'':0.00762818,(('PDT000212355.3':0.0296064,'PDT000538888.2':0.0272615)'':0.000923145,('PDT000259525.3':0.0123788,'PDT000040432.5':0.0139526)'':0.0219304)'':0.00591114)'':0.00477541)'':0.0070144)'':0.000479346)'':0.0075388)'':0.019257)'':0.00195354,((('PDT000132557.1':0.0607539,('PDT000519175.2':0.0580992,('PDT000148226.3':0.0491651,'PDT000412847.2':0.0432137)'':0.0213819)'':0.0159798)'':-0.00328834,(('PDT000133955.1':0.0312775,'PDT000248645.1':0.0226513)'':0.0672629,(('PDT000085590.3':0.0546178,((('PDT000264743.3':0.0467477,('PDT000641261.2':0.0193204,'PDT000682347.1':0.00994498)'':0.035893)'':0.00991432,(('PDT000417907.2':0.0383586,'PDT000469891.2':0.0383869)'':0.00911199,(('PDT000242806.3':0.0263879,'PDT000631684.2':0.0260334)'':0.0147605,('PDT000548277.2':0.0311237,'PDT000110874.3':0.0367817)'':0.00352994)'':0.0143207)'':0.00582721)'':-0.000731172,(('PDT000098704.3':0.0329327,'PDT000140448.3':0.0188051)'':0.0103807,(('PDT000421156.2':0.0332708,'PDT000040431.4':0.0378045)'':0.00549646,('PDT000621227.2':0.0315484,(('PDT000556561.2':0.0202506,'PDT000098714.3':0.0225757)'':0.000685372,('PDT000227816.3':0.0213151,('PDT000257220.3':0.00996706,'PDT000267466.3':0.01461)'':0.00901308)'':0.00218505)'':0.00843167)'':0.00547473)'':0.00391507)'':0.00865233)'':0.0120442)'':-0.00647502,('PDT000652800.2':0.0659104,(('PDT000142544.3':0.0348838,'PDT000251784.1':0.0470208)'':0.00500349,('PDT000147476.3':0.0547789,(('PDT000240653.3':0.0282292,('PDT000494931.2':0.0198067,'PDT000105123.3':0.0131658)'':0.00919007)'':-0.000469137,('PDT000098719.3':0.0218989,(('PDT000388896.2':0.0161524,('PDT000279549.3':0.0103638,'PDT000137684.3':0.0108411)'':0.00788296)'':0.000250615,('PDT000581141.2':0.0147984,('PDT000083967.3':0.00902275,('PDT000504687.2':0.00633597,('PDT000235308.3':0.00425134,(('PDT000198190.3':0.00313841,'PDT000499251.2':0.00304675)'':0.00111858,('PDT000198402.4':0.00260736,('PDT000385890.2':0.00207018,'PDT000718291.1':0.00212102)'':0.000384322)'':0.00065242)'':0.00112631)'':0.000830367)'':0.00350434)'':0.00350397)'':0.00819528)'':0.00457201)'':0.00409894)'':0.0249762)'':-0.00576169)'':0.0190185)'':-0.00493902)'':0.0136303)'':-0.00223781)'':-0.00129156,(('PDT000222076.3':0.0945718,(('PDT000470333.2':0.0179846,'PDT000648084.1':0.0252999)'':0.0571554,('PDT000123475.3':0.0323758,('PDT000624090.2':0.0228973,'PDT000251628.1':0.0221204)'':0.00646972)'':0.0507027)'':0.0118167)'':0.00508924,((('PDT000208679.1':0.0675652,(('PDT000576037.2':0.02862,('PDT000107652.2':0.0244444,('PDT000151184.3':0.0158229,'PDT000132407.3':0.0154425)'':0.00893992)'':0.0118628)'':0.00525353,('PDT000328053.1':0.0401996,('PDT000444840.2':0.0303247,('PDT000161893.3':0.0260465,'PDT000326238.2':0.0248374)'':0.00041788)'':0.00741303)'':0.00776279)'':0.0193196)'':-0.0155605,((('PDT000492142.2':0.0144428,('PDT000319667.2':0.00867724,'PDT000711090.1':0.00726056)'':0.00995302)'':0.0214997,(('PDT000435600.2':0.0436174,('PDT000189564.3':0.0270011,(('PDT000311066.3':0.0223854,'PDT000410719.2':0.0214984)'':0.00304006,('PDT000624256.2':0.0236396,'PDT000132197.3':0.0215293)'':0.000881939)'':0.00663661)'':0.00705129)'':-0.00483199,('PDT000594749.2':0.0295716,('PDT000378065.2':0.0309249,('PDT000328004.2':0.0214541,('PDT000598480.2':0.0140829,('PDT000161667.3':0.0150376,'PDT000324660.2':0.0152804)'':0.00203373)'':0.00359581)'':0.00575382)'':0.00168315)'':0.00377015)'':0.0043017)'':-0.00167169,(('PDT000148089.3':0.0468941,'PDT000505361.2':0.0342846)'':0.0107533,(('PDT000317762.2':0.0311208,('PDT000257966.3':0.0284113,('PDT000140576.4':0.0307838,('PDT000405083.2':0.0198148,'PDT000422029.2':0.0183486)'':0.00921088)'':0.00107201)'':0.0035351)'':-0.000548596,(('PDT000422211.2':0.0243379,('PDT000645769.2':0.019754,('PDT000324030.2':0.0169388,'PDT000325093.2':0.0166011)'':0.000807487)'':0.00366097)'':0.00523917,(('PDT000443217.2':0.0296913,(('PDT000317202.2':0.0269696,'PDT000123307.1':0.028716)'':0.000721465,('PDT000617080.2':0.0195679,('PDT000449068.2':0.0331229,('PDT000710867.1':0.0109127,('PDT000388613.2':0.00695275,('PDT000362323.2':0.00500489,'PDT000623028.2':0.00635311)'':0.00113255)'':0.00223914)'':0.00901741)'':0.00328105)'':0.00589639)'':0.00193138)'':-0.00187987,('PDT000032317.1':0.0305233,(('PDT000555889.2':0.0225981,'PDT000603301.2':0.0197252)'':0.00689687,('PDT000496542.2':0.0279118,(('PDT000455263.2':0.0127602,'PDT000682162.1':0.00822648)'':0.00499767,('PDT000711400.1':0.0144024,(('PDT000480422.2':0.00635242,'PDT000480446.2':0.00598098)'':0.00466848,('PDT000630376.2':0.0109981,('PDT000443421.2':0.0082835,'PDT000509947.2':0.0087817)'':0.00301841)'':0.00194608)'':0.00300432)'':0.000645735)'':0.00958404)'':-0.00108363)'':0.00929296)'':-0.00868592)'':0.00792853)'':0.000696957)'':0.0113968)'':-0.00621389)'':0.0052287)'':0.00332315,(((('PDT000377095.2':0.0284371,'PDT000553678.2':0.0243127)'':0.00920402,('PDT000314182.3':0.0482482,((('PDT000325735.2':0.0179585,('PDT000131885.3':0.0160132,('PDT000324631.2':0.00987823,'PDT000523419.2':0.00997687)'':0.00549647)'':0.00622264)'':0.00419926,('PDT000323620.2':0.0174256,('PDT000464167.2':0.0160754,('PDT000131825.3':0.0209211,('PDT000324197.2':0.00945659,'PDT000658746.2':0.0113499)'':0.00556351)'':0.00214028)'':0.00544934)'':0.00260357)'':0.00104078,(('PDT000645788.2':0.0141573,('PDT000341774.2':0.0155626,('PDT000378654.2':0.0141083,('PDT000537447.2':0.0128841,'PDT000624280.2':0.0106616)'':0.00359053)'':0.00235456)'':0.0026564)'':-0.00135138,('PDT000467231.2':0.0255577,(('PDT000377983.2':0.0189828,('PDT000325142.2':0.0110105,'PDT000326201.2':0.0138083)'':0.00501356)'':0.00195593,('PDT000157925.3':0.0208852,(('PDT000491608.2':0.0177863,'PDT000573550.2':0.0196374)'':0.00108113,('PDT000155695.3':0.012695,(('PDT000326102.2':0.0229026,(('PDT000319263.2':0.00961303,'PDT000325706.2':0.00951477)'':0.00147933,('PDT000379494.2':0.0100172,('PDT000316886.2':0.0138869,('PDT000316911.2':0.0122542,(('PDT000390823.2':0.00674234,'PDT000603035.2':0.00707616)'':0.000716875,(('PDT000157228.3':0.00662654,'PDT000702582.1':0.00534596)'':8.1904e-05,('PDT000377172.2':0.00543469,'PDT000378039.2':0.00530731)'':0.00115482)'':0.00117884)'':0.00159016)'':0.00113727)'':0.000285375)'':0.000800732)'':0.00415222)'':-0.00243282,(('PDT000321547.2':0.011986,('PDT000318878.2':0.00683377,'PDT000641366.2':0.00796853)'':0.00469885)'':0.000448301,(('PDT000155848.3':0.00821546,('PDT000157898.3':0.0078396,'PDT000588075.2':0.0067539)'':0.00149899)'':0.00258246,('PDT000325709.2':0.0102065,(('PDT000324114.2':0.00940938,('PDT000160216.3':0.00862823,'PDT000410740.2':0.00793427)'':0.00128937)'':0.00115353,('PDT000323622.2':0.00789613,('PDT000485298.2':0.00785954,('PDT000160234.3':0.00849361,('PDT000278348.3':0.00617005,'PDT000387004.2':0.00754625)'':0.000827443)'':0.000889333)'':0.00240449)'':0.00111414)'':0.00243379)'':-0.000145885)'':0.000887889)'':-0.00068359)'':0.00386507)'':0.00236954)'':0.00365774)'':-0.00447989)'':0.0117237)'':-0.0110993)'':0.00342727)'':0.0158169)'':-0.0027618)'':-0.00922799,((('PDT000332099.2':0.045849,('PDT000669596.1':0.0201774,('PDT000316945.2':0.0222881,('PDT000565128.2':0.015652,('PDT000415209.2':0.0183968,'PDT000532061.2':0.0180902)'':0.00100289)'':0.00387728)'':0.00317491)'':0.0124747)'':-0.00613981,(('PDT000156752.3':0.0252899,'PDT000711394.1':0.0234238)'':0.0031905,('PDT000155195.3':0.0282181,('PDT000658729.2':0.0198976,('PDT000155422.3':0.0131659,('PDT000323337.2':0.0108333,'PDT000040444.4':0.0136419)'':0.00326763)'':0.00535621)'':0.00452195)'':0.00554142)'':0.00206431)'':0.00259854,(('PDT000598471.2':0.0277419,('PDT000161579.3':0.0241348,'PDT000320900.2':0.0189057)'':0.00979682)'':0.000870287,(('PDT000394280.2':0.0237381,('PDT000658748.2':0.0269411,('PDT000156025.3':0.0165556,('PDT000711051.1':0.0102233,('PDT000377155.2':0.00684009,'PDT000398522.2':0.00668951)'':0.00525498)'':0.00622964)'':0.00374127)'':0.00410844)'':0.00254404,(('PDT000380339.2':0.0358536,('PDT000621346.2':0.0193868,('PDT000327213.2':0.00990821,'PDT000576243.2':0.0135075)'':0.0106982)'':0.00344737)'':0.00346516,(('PDT000426942.2':0.0184801,('PDT000457520.2':0.0143281,('PDT000410716.2':0.0131017,('PDT000160498.3':0.00854862,'PDT000243668.3':0.0105331)'':0.00209147)'':0.00232463)'':0.00382562)'':0.008173,(('PDT000376159.2':0.0217219,('PDT000156510.3':0.014341,'PDT000332167.2':0.0218239)'':0.00431225)'':0.00751857,((('PDT000378522.2':0.0159773,('PDT000314831.3':0.0150784,'PDT000549596.2':0.0097972)'':0.00389438)'':5.85975e-05,('PDT000378561.2':0.0189623,('PDT000702564.1':0.0162283,('PDT000318624.2':0.0136791,('PDT000160231.3':0.0141582,('PDT000414566.2':0.00898402,'PDT000496090.2':0.00849708)'':0.00216879)'':0.00175903)'':0.00093227)'':0.00287534)'':0.000802637)'':0.00205656,('PDT000645791.2':0.0261141,('PDT000157834.3':0.0341015,(('PDT000327259.2':0.00816859,'PDT000115446.3':0.00919771)'':0.00538406,('PDT000546872.2':0.0161604,('PDT000326083.2':0.0205932,('PDT000377949.2':0.016422,((('PDT000325064.2':0.010036,'PDT000327132.2':0.00857838)'':0.00327415,('PDT000432281.2':0.00863801,'PDT000569744.2':0.00985469)'':0.00228645)'':0.00158968,('PDT000333971.2':0.0162363,('PDT000679859.1':0.0155083,('PDT000320218.2':0.0122815,(('PDT000160261.3':0.00977218,'PDT000160269.3':0.0101327)'':0.000182871,(('PDT000155628.3':0.00986239,'PDT000115422.3':0.00887831)'':0.000778484,('PDT000157270.3':0.00660641,('PDT000115430.3':0.00726867,('PDT000160147.3':0.00511243,'PDT000491615.2':0.00584917)'':0.000873784)'':0.000427509)'':0.00134712)'':0.00195838)'':0.00186131)'':0.000773943)'':0.0014864)'':-0.0023214)'':0.0030468)'':0.000669542)'':0.00165054)'':-0.00111386)'':0.00392399)'':0.000861242)'':-0.00420034)'':0.00538455)'':0.00118134)'':0.00247971)'':0.00045506)'':0.00412792)'':-0.00349206)'':0.000652456)'':0.00301365,((('PDT000367238.2':0.0309634,(('PDT000235721.3':0.0146092,'PDT000125554.3':0.0152297)'':0.00647046,('PDT000318590.2':0.0152936,'PDT000647769.2':0.0182309)'':0.00412816)'':0.00730471)'':0.00287222,(('PDT000160140.3':0.0291723,'PDT000333297.2':0.035836)'':0.00322142,('PDT000629328.2':0.0469097,(('PDT000302814.3':0.0307049,('PDT000571029.2':0.0201256,'PDT000623057.2':0.0177383)'':0.00516169)'':0.0049671,('PDT000546870.2':0.0244471,('PDT000657051.2':0.0228768,(('PDT000176786.3':0.0144044,'PDT000523430.2':0.0119322)'':0.0047709,('PDT000132317.3':0.00910352,'PDT000123351.1':0.00977568)'':0.00876955)'':0.00413235)'':0.00574074)'':0.00313599)'':0.00777977)'':-0.00119673)'':-0.00125504)'':0.0072385,((('PDT000326239.2':0.0407131,'PDT000115469.3':0.0472954)'':0.00816741,('PDT000157893.3':0.0285848,'PDT000682788.1':0.0288305)'':0.018544)'':0.00926129,(('PDT000332143.2':0.0235571,('PDT000319879.2':0.0154521,'PDT000596028.2':0.0128433)'':0.00486787)'':0.0215821,(('PDT000485309.2':0.0347031,('PDT000038889.1':0.0362528,('PDT000148168.3':0.0229368,('PDT000155569.3':0.0241853,(('PDT000148177.3':0.016124,('PDT000155772.3':0.0129186,'PDT000160417.3':0.0123314)'':0.0032768)'':0.00344373,(('PDT000148098.3':0.0122609,'PDT000148221.3':0.0135858)'':0.00753789,('PDT000444916.2':0.0198067,'PDT000115439.3':0.0173655)'':0.00300886)'':0.00200394)'':0.0040236)'':0.000944261)'':0.00778381)'':0.00977197)'':-0.0113478,((('PDT000387001.2':0.0281849,'PDT000586500.2':0.0333445)'':0.00536497,('PDT000711056.1':0.0347964,('PDT000333327.2':0.0216047,('PDT000333326.2':0.0154435,'PDT000491613.2':0.0141277)'':0.00679771)'':0.00826786)'':0.00309196)'':-0.000242243,((('PDT000711405.1':0.0277776,('PDT000323287.2':0.0283995,('PDT000131929.3':0.0186915,('PDT000338955.2':0.0212271,'PDT000410736.2':0.0167901)'':0.00334952)'':0.00422243)'':0.00152466)'':0.0022448,(('PDT000629673.2':0.0217267,('PDT000317299.2':0.0142244,'PDT000327268.2':0.0138028)'':0.00598888)'':0.00496343,(('PDT000394237.2':0.0197641,('PDT000615211.2':0.0103226,('PDT000496153.2':0.0118879,'PDT000651384.2':0.00993748)'':0.001059)'':0.00256754)'':0.0102514,(('PDT000484191.2':0.0090569,('PDT000377944.2':0.00711249,'PDT000702613.1':0.00851411)'':0.00268165)'':0.00975321,((('PDT000394259.2':0.0125629,'PDT000540384.2':0.0172079)'':0.0042235,('PDT000588066.2':0.0138661,'PDT000132141.3':0.0177901)'':0.00416582)'':0.00102191,(('PDT000317179.2':0.0217388,'PDT000132238.3':0.0184147)'':0.00219152,('PDT000161894.3':0.0219841,('PDT000060822.3':0.0257446,('PDT000333953.2':0.0187084,('PDT000410713.2':0.0200463,(('PDT000671621.1':0.0118089,(('PDT000156598.3':0.0107026,'PDT000123281.1':0.0133507)'':0.00130831,('PDT000161528.3':0.0144297,'PDT000316905.2':0.0132851)'':0.00174524)'':0.000870577)'':0.000479552,('PDT000115431.3':0.0187463,(('PDT000316881.2':0.00809608,'PDT000323624.2':0.00925982)'':0.0023167,('PDT000132400.3':0.0144829,(('PDT000378690.2':0.00935652,'PDT000641837.2':0.00996958)'':0.000126667,('PDT000132377.3':0.00926091,('PDT000161524.3':0.00569617,'PDT000161559.3':0.00524583)'':0.00224624)'':0.00291907)'':0.00230187)'':-0.000404985)'':0.00247674)'':-0.0012885)'':0.00428981)'':0.001966)'':0.000814884)'':0.000541536)'':-0.000542222)'':-0.000785354)'':0.0038923)'':-0.000394833)'':0.00182064)'':-0.000201257)'':-0.00108541,(('PDT000317778.2':0.0344544,('PDT000157042.3':0.0333624,('PDT000496082.2':0.0182233,'PDT000600086.2':0.0144306)'':0.0119169)'':0.00083934)'':0.00021583,('PDT000332132.2':0.0344564,((('PDT000132348.3':0.0187524,('PDT000583567.2':0.0153047,('PDT000332710.2':0.011469,('PDT000395670.2':0.00718453,'PDT000603037.2':0.00872007)'':0.0030905)'':0.00186403)'':0.00633859)'':-2.17039e-05,('PDT000641390.2':0.0214471,(('PDT000160194.3':0.0120363,'PDT000546430.2':0.0172474)'':0.00169395,('PDT000465205.2':0.0149277,('PDT000629335.2':0.0112337,(('PDT000327209.2':0.0097956,('PDT000325098.2':0.00811445,'PDT000569734.2':0.00718765)'':0.0019863)'':-2.85652e-05,('PDT000115477.3':0.00956225,('PDT000317687.2':0.00443273,'PDT000647765.2':0.00543864)'':0.00209947)'':0.00143411)'':0.000961333)'':0.00146693)'':0.00240385)'':0.00397334)'':0.000457515)'':-0.000922749,(('PDT000319215.2':0.0187197,'PDT000532978.2':0.0153321)'':0.0081792,(('PDT000161587.3':0.0163325,(('PDT000378001.2':0.0161552,'PDT000641378.2':0.0155627)'':0.00050389,('PDT000132023.3':0.0106963,('PDT000325743.2':0.0146803,('PDT000389673.2':0.0121052,('PDT000160364.3':0.00927652,'PDT000578362.2':0.0126677)'':0.00248725)'':0.00179463)'':0.00100987)'':0.00155083)'':0.00420354)'':-0.00218439,('PDT000689107.1':0.0265447,((('PDT000132147.3':0.0148377,('PDT000395675.2':0.0152334,('PDT000388955.2':0.00863482,('PDT000328141.2':0.00787449,'PDT000519383.2':0.00713391)'':0.000584885)'':0.00285024)'':0.00165249)'':-0.000234206,('PDT000651760.2':0.0198576,('PDT000378651.2':0.00964601,(('PDT000157268.3':0.00952405,'PDT000115424.3':0.00796755)'':0.00204585,('PDT000377952.2':0.00969688,'PDT000503122.2':0.00953012)'':0.00162438)'':0.00101363)'':0.00335353)'':0.00040298)'':0.00416208,(('PDT000156897.3':0.0239309,('PDT000414572.2':0.0168551,'PDT000447366.2':0.0165209)'':0.00355536)'':-0.000198075,(('PDT000414550.2':0.0151935,'PDT000621315.2':0.0165165)'':0.00423446,(('PDT000316223.2':0.0148447,'PDT000711064.1':0.0134814)'':0.00427589,('PDT000326139.2':0.0175843,('PDT000422643.2':0.0172342,('PDT000319859.2':0.0132718,(('PDT000517231.2':0.00834394,'PDT000588068.2':0.00686636)'':0.00312133,('PDT000491884.2':0.00927518,('PDT000394231.2':0.00782,'PDT000131988.3':0.0092587)'':0.00257057)'':0.00128026)'':0.00244842)'':0.00124729)'':0.00217027)'':-5.63235e-05)'':0.00108442)'':0.000124889)'':-0.000342166)'':0.00679335)'':-0.00454887)'':0.00364968)'':-0.00360493)'':0.0105623)'':-0.00413109)'':-0.00434628)'':0.00688877)'':-0.00134487)'':0.0116154)'':0.00779296)'':-0.00399761)'':-0.0102685)'':0.010621)'':0.0464699)'':-0.0214266)'':0.010966)'':0.0254964)'':0.0156576)'':0.00586236)'':0.0274784)'':0.0116926)'':0.00474438)'':0.00216427)'':0.00220504)'':0.00393688,(((('PDT000709940.1':0.113066,('PDT000345920.1':0.0595827,('PDT000242868.1':0.0644958,('PDT000285331.1':0.0365386,('PDT000517263.1':0.0100017,('PDT000553397.1':0.00553883,'PDT000625321.1':0.00471807)'':0.00502647)'':0.0144869)'':0.0148717)'':0.00718888)'':0.0337468)'':0.0936832,(('PDT000015844.3':0.00867408,('PDT000015849.3':0.00816587,('PDT000015846.2':0.00782859,('PDT000015835.3':0.00798579,('PDT000015850.3':0.00804484,'PDT000015839.3':0.00664646)'':0.000149857)'':0.000171576)'':0.000206745)'':0.00018776)'':0.11354,(('PDT000386730.1':0.0742989,'PDT000346012.1':0.0799921)'':0.0377625,('PDT000501942.1':0.0434756,('PDT000252492.1':0.0433118,('PDT000719111.1':0.0215645,('PDT000556923.1':0.0225481,(('PDT000040689.2':0.0143553,'PDT000230079.2':0.0113366)'':0.00860166,('PDT000124733.1':0.0109336,'PDT000293511.1':0.00938557)'':0.0105479)'':0.00370075)'':0.00458312)'':0.00557154)'':0.00621141)'':0.0530782)'':0.0142642)'':0.0482362)'':0.000541816,(((((('PDT000719018.1':0.0664104,('PDT000309336.2':0.0306169,'PDT000251505.1':0.0277489)'':0.0128622)'':0.0332554,('PDT000129900.2':0.100859,('PDT000251757.1':0.0563516,'PDT000251770.1':0.0659594)'':0.0133365)'':0.000585854)'':0.0558672,(('PDT000188093.2':0.0646167,'PDT000243025.1':0.0729893)'':0.0159894,('PDT000439486.1':0.0600552,('PDT000482526.1':0.0300897,('PDT000500475.1':0.0416245,('PDT000375010.1':0.0348154,'PDT000243004.1':0.0351585)'':0.00291687)'':0.0033211)'':0.0238004)'':0.0212212)'':0.0469231)'':0.00348098,(('PDT000321341.1':0.104888,(('PDT000634715.1':0.0598008,('PDT000291155.2':0.043679,'PDT000631208.1':0.0548785)'':0.00885444)'':0.00557038,('PDT000290517.2':0.0731215,('PDT000355106.1':0.0592507,('PDT000550361.1':0.0387128,'PDT000581171.1':0.0416392)'':0.0112252)'':0.0116215)'':0.00382314)'':0.0118712)'':0.00862991,(('PDT000354946.2':0.0690207,('PDT000502465.1':0.0392568,('PDT000502473.1':0.0356841,('PDT000139206.3':0.0158967,'PDT000486500.1':0.0164313)'':0.00926876)'':0.00877173)'':0.0395104)'':0.0251669,(('PDT000131767.2':0.0508836,'PDT000228268.1':0.0456734)'':0.0111533,(('PDT000425802.1':0.062044,'PDT000345999.1':0.053757)'':0.00693079,('PDT000131780.2':0.052373,'PDT000150862.1':0.0461615)'':0.00397271)'':0.00294431)'':0.0288551)'':0.00705253)'':0.0415048)'':0.00919666,((('PDT000557333.1':0.137201,('PDT000502026.1':0.112292,('PDT000345930.1':0.101471,(('PDT000048146.1':0.0677167,('PDT000258718.2':0.0236579,'PDT000346063.1':0.026347)'':0.0300664)'':0.0126851,(('PDT000661116.1':0.0570909,'PDT000345933.1':0.0615451)'':0.00432528,('PDT000345951.1':0.0506548,('PDT000240301.1':0.0507774,('PDT000375007.1':0.0148981,'PDT000240311.1':0.0203819)'':0.026217)'':0.00550988)'':0.00665276)'':0.00728615)'':0.0146078)'':0.034506)'':0.0259524)'':-0.00189401,((('PDT000502335.1':0.10251,('PDT000600642.1':0.0617599,'PDT000687820.1':0.0651161)'':0.0432767)'':0.0100472,(('PDT000235749.2':0.0569928,'PDT000718921.1':0.0745782)'':0.0333279,(('PDT000495086.1':0.0440056,'PDT000396743.1':0.046503)'':0.0346212,(('PDT000003613.2':0.0626449,'PDT000150878.1':0.0593041)'':0.00586067,('PDT000566142.1':0.0512437,'PDT000568701.1':0.0537183)'':0.0320911)'':0.0047512)'':0.0181888)'':0.00633843)'':0.0311344,((('PDT000289913.1':0.0532023,'PDT000412507.1':0.0659957)'':0.0346448,('PDT000174326.1':0.106634,('PDT000627339.1':0.0880842,('PDT000020080.1':0.0887504,('PDT000557139.1':0.0785736,('PDT000312794.2':0.071406,('PDT000509494.1':0.0536087,'PDT000716404.1':0.0502253)'':0.0117815)'':0.00872212)'':0.00379492)'':0.00413957)'':0.010819)'':-0.00124078)'':0.0142375,('PDT000661132.1':0.121241,((('PDT000719132.1':0.0352897,'PDT000502537.1':0.0455776)'':0.00981653,('PDT000150890.1':0.0567333,'PDT000250103.1':0.0638087)'':0.00273232)'':0.023468,('PDT000257784.1':0.10692,(('PDT000399980.1':0.0487032,'PDT000346454.1':0.0474323)'':0.0185658,(('PDT000109480.2':0.0612923,'PDT000687925.1':0.0480377)'':0.00252446,('PDT000432494.1':0.0286225,'PDT000557208.1':0.02209)'':0.0279443)'':0.0111187)'':0.0254462)'':-0.00557859)'':0.0275968)'':-0.00310951)'':0.0195794)'':0.000535684)'':0.00499559,((('PDT000174346.1':0.0882228,(('PDT000188151.3':0.0280208,'PDT000339363.1':0.0274265)'':0.0116904,('PDT000455221.1':0.0272873,'PDT000719085.1':0.0240973)'':0.00705841)'':0.044388)'':0.0363997,(('PDT000004909.2':0.0937559,'PDT000221018.1':0.0789261)'':0.0214111,(('PDT000325532.1':0.0884909,('PDT000718904.1':0.0589694,'PDT000228106.1':0.0698136)'':0.0172866)'':0.00560234,((('PDT000247164.2':0.0501164,'PDT000346491.1':0.0438401)'':0.0145183,('PDT000354944.1':0.0537605,('PDT000548099.1':0.0334694,'PDT000718941.1':0.0272582)'':0.0189637)'':0.00975148)'':0.0210077,('PDT000162003.3':0.06379,('PDT000309340.2':0.0530564,('PDT000502491.1':0.0458584,(('PDT000129818.2':0.0152414,'PDT000252601.1':0.0152831)'':0.00407857,('PDT000468333.1':0.0205645,'PDT000318972.1':0.0199002)'':0.0025153)'':0.0169859)'':0.00460126)'':0.0107127)'':0.0166963)'':0.00959901)'':0.0047802)'':0.00721404)'':-0.000547906,(('PDT000346576.1':0.0707663,('PDT000250157.1':0.0755349,('PDT000385067.1':0.0482215,'PDT000438015.1':0.0521785)'':0.0107356)'':0.00669667)'':0.00916471,('PDT000354867.1':0.0914242,(('PDT000580125.2':0.0779807,'PDT000406999.1':0.0616593)'':0.0169931,(('PDT000502911.1':0.0621931,('PDT000502926.1':0.0428315,'PDT000553119.1':0.0444402)'':0.019459)'':0.0189997,(('PDT000077122.2':0.0973322,(('PDT000346919.1':0.0693437,('PDT000088282.2':0.0533626,'PDT000518112.1':0.0588884)'':0.00657634)'':0.000135361,('PDT000200963.3':0.0700858,('PDT000359661.1':0.0441922,('PDT000258446.2':0.0318219,'PDT000332094.1':0.0375326)'':0.00652886)'':0.0169007)'':0.000315833)'':0.00898395)'':-0.00721059,('PDT000488315.1':0.0689174,('PDT000502329.1':0.0695264,('PDT000550376.1':0.0518406,('PDT000502524.1':0.0622647,(('PDT000373241.1':0.0457388,'PDT000646916.1':0.0453874)'':0.00634399,('PDT000631950.1':0.0369243,('PDT000095824.2':0.0308715,'PDT000396547.1':0.0434827)'':0.00398445)'':0.000958132)'':0.00318778)'':0.00417952)'':0.00418526)'':0.00212432)'':0.00519948)'':0.00503146)'':0.00570599)'':0.00397834)'':-0.00328134)'':0.0214687)'':0.0265118)'':0.00622437)'':-0.000358165,(((('PDT000357661.1':0.0736545,('PDT000441064.2':0.018764,'PDT000497934.1':0.0147373)'':0.0695523)'':-0.00335032,(('PDT000637329.1':0.0544084,('PDT000559260.2':0.0482148,('PDT000367032.2':0.0258957,'PDT000487995.2':0.0278205)'':0.0207807)'':0.00540678)'':0.00710173,('PDT000228270.1':0.0655213,(('PDT000490909.2':0.0552074,('PDT000420673.1':0.0452535,('PDT000150757.3':0.0367405,'PDT000242516.1':0.0351922)'':0.00626316)'':0.0116638)'':-0.00254328,(('PDT000714666.1':0.0368589,'PDT000249903.1':0.0308721)'':0.00691914,('PDT000468034.1':0.0489253,('PDT000228095.1':0.014854,('PDT000228078.1':0.0145316,'PDT000228097.1':0.0120832)'':0.00387692)'':0.0224816)'':0.00276325)'':0.00117456)'':0.00934681)'':-0.00347678)'':0.0252741)'':0.0355165,((('PDT000337206.1':0.021239,'PDT000577109.1':0.0269916)'':0.0508216,(('PDT000619968.1':0.0584377,'PDT000645069.1':0.0495953)'':0.00393371,('PDT000671616.1':0.0760077,(('PDT000557355.1':0.0325562,'PDT000025384.2':0.0284776)'':0.00213089,('PDT000557217.1':0.0334408,('PDT000305454.2':0.0265129,('PDT000312647.2':0.0138952,'PDT000556950.1':0.0137783)'':0.00864887)'':0.00768648)'':0.00959882)'':0.0266481)'':0.00504971)'':0.0041976)'':0.00056809,((('PDT000539063.1':0.0639367,'PDT000583480.1':0.0749253)'':0.00123518,('PDT000624695.1':0.0680415,('PDT000653443.1':0.0416407,'PDT000689823.1':0.03265)'':0.0261432)'':0.00994731)'':0.00695735,((('PDT000312629.2':0.0312489,'PDT000482523.1':0.0366403)'':0.0109541,('PDT000025383.1':0.0618659,('PDT000305154.2':0.0257168,('PDT000296181.2':0.0108733,'PDT000642582.1':0.0174117)'':0.0102338)'':0.034976)'':-0.00158159)'':0.0216305,((('PDT000300978.2':0.0298435,'PDT000354883.1':0.0282599)'':0.0400391,('PDT000242998.1':0.0629172,('PDT000242860.1':0.0517223,('PDT000242861.1':0.010922,('PDT000242867.1':0.00320147,'PDT000242900.1':0.00357954)'':0.00500439)'':0.0271265)'':0.0352078)'':0.00370599)'':0.00364612,(('PDT000637314.1':0.0681843,('PDT000639985.1':0.0524239,'PDT000689856.1':0.0485681)'':0.0055052)'':0.0200729,('PDT000551910.1':0.0626625,('PDT000523913.1':0.0634423,('PDT000294661.2':0.0300067,'PDT000668347.2':0.0387366)'':0.017466)'':0.00936198)'':0.0136152)'':0.00119976)'':0.00575552)'':0.00479943)'':0.00442768)'':0.0318605)'':0.0182105,(((('PDT000547570.1':0.0953962,('PDT000439407.1':0.0288727,('PDT000307424.2':0.0250941,'PDT000249944.1':0.0178495)'':0.00902439)'':0.0627798)'':0.0108119,((('PDT000477112.1':0.0897808,('PDT000006428.2':0.0743748,('PDT000395104.1':0.0578467,('PDT000656983.1':0.0626066,'PDT000645548.2':0.0618964)'':0.0151128)'':0.00785342)'':0.00526658)'':-0.00688346,(('PDT000718950.1':0.0558792,('PDT000425690.1':0.0427934,'PDT000426725.1':0.0419589)'':0.0112646)'':0.0260297,(('PDT000624699.1':0.0724895,('PDT000556936.1':0.0581541,'PDT000602779.1':0.0566109)'':0.0108835)'':0.00211188,(('PDT000242937.1':0.0612587,'PDT000251756.1':0.0696873)'':0.00551876,('PDT000258732.2':0.0579152,('PDT000718825.1':0.0276197,'PDT000354202.1':0.0235998)'':0.0187725)'':0.00600182)'':0.00321253)'':0.00675605)'':0.00461572)'':0.00358192,((('PDT000034517.2':0.0769605,'PDT000069135.2':0.0575735)'':0.0159281,('PDT000720232.1':0.0865751,('PDT000034478.2':0.0776858,('PDT000707864.1':0.055714,('PDT000646908.1':0.0569866,'PDT000395240.1':0.0521324)'':0.0105455)'':0.00928784)'':0.00408729)'':-0.000627062)'':0.00136916,(('PDT000639584.1':0.0344062,(('PDT000394895.1':0.024626,'PDT000565855.1':0.0267539)'':0.00240851,('PDT000497372.1':0.0310652,('PDT000395305.1':0.0219212,('PDT000394800.1':0.0197966,('PDT000610396.1':0.0153356,'PDT000637367.1':0.0169956)'':0.00155559)'':0.00434626)'':0.00668516)'':-0.00018845)'':0.00497833)'':0.0449087,('PDT000556957.1':0.050135,('PDT000501512.1':0.0739496,(('PDT000682394.1':0.0508039,('PDT000095716.2':0.0339449,'PDT000644850.1':0.0365654)'':0.00520146)'':0.00802936,('PDT000557018.1':0.0473541,('PDT000566021.1':0.0551038,('PDT000249905.1':0.0374895,('PDT000248694.1':0.0302324,'PDT000250249.1':0.0318118)'':0.00187338)'':0.00886409)'':0.00279734)'':-0.00151406)'':0.00742121)'':0.0012604)'':-0.000684794)'':0.00991132)'':0.00903853)'':0.0198721)'':0.00268523,(('PDT000228374.1':0.0958451,('PDT000228276.1':0.0796547,(('PDT000425952.1':0.0607657,('PDT000006421.2':0.0487268,'PDT000017711.2':0.0452149)'':0.0101234)'':0.00383159,(('PDT000341203.1':0.0548083,'PDT000425499.1':0.0532867)'':0.00640654,(('PDT000619569.1':0.0522203,'PDT000644999.1':0.0505857)'':0.00827502,(('PDT000426152.1':0.0507051,'PDT000427734.1':0.0413218)'':0.00899817,('PDT000131786.2':0.0531896,('PDT000040556.1':0.047642,'PDT000661615.1':0.0388209)'':0.00860085)'':0.00319624)'':0.00447618)'':0.000936024)'':0.000961123)'':0.02076)'':0.00131465)'':0.0034906,(((('PDT000290009.1':0.0562573,(('PDT000535221.1':0.0543428,'PDT000627296.1':0.0551452)'':0.0029949,('PDT000425980.1':0.0658048,('PDT000633565.1':0.0392997,'PDT000721996.1':0.0429556)'':0.00804556)'':0.00342543)'':0.00409149)'':0.000653103,('PDT000482832.1':0.0647109,(('PDT000345919.1':0.0656433,('PDT000346572.1':0.0489641,'PDT000346909.1':0.0430528)'':0.0183367)'':0.000657967,(('PDT000088295.2':0.0518375,'PDT000202005.2':0.0563745)'':0.000691079,('PDT000426598.1':0.0531316,('PDT000425996.1':0.0471671,'PDT000426591.1':0.0433592)'':0.00107185)'':0.00657772)'':0.0095545)'':0.00467955)'':-0.00447379)'':0.00514072,((('PDT000242984.1':0.0440785,'PDT000319019.1':0.0555436)'':0.00493224,('PDT000566116.1':0.0826826,('PDT000266391.2':0.0530662,('PDT000249872.1':0.0473478,'PDT000250171.1':0.0388775)'':0.00771264)'':0.00760684)'':-0.000374735)'':0.0068119,('PDT000354223.1':0.0923221,('PDT000535322.1':0.0618149,((('PDT000426013.1':0.0336582,'PDT000661484.1':0.0362265)'':0.0115973,(('PDT000417087.1':0.0341213,'PDT000520819.1':0.04327)'':0.0104615,('PDT000202623.2':0.0574467,'PDT000653020.1':0.0630063)'':0.00133763)'':0.0023454)'':0.00680813,('PDT000521017.1':0.0794003,(('PDT000426191.1':0.045654,('PDT000656973.1':0.0355554,('PDT000426816.1':0.031626,'PDT000619609.1':0.0240166)'':0.0117306)'':0.00684434)'':0.000466693,('PDT000586229.1':0.0645525,(('PDT000425948.1':0.0316422,'PDT000425995.1':0.0313973)'':0.0062495,('PDT000128847.2':0.0338098,('PDT000426677.1':0.0390289,'PDT000520985.1':0.0329695)'':0.00373576)'':0.00175398)'':0.0165149)'':-0.00315707)'':0.00941029)'':-0.00605171)'':0.00753776)'':0.0191825)'':-0.0130367)'':-2.42799e-05)'':0.0166874,((('PDT000011194.2':0.0719083,('PDT000128797.2':0.0877805,('PDT000720452.1':0.0685833,'PDT000468849.1':0.0714927)'':0.00675949)'':0.00120217)'':0.00927188,(('PDT000025494.2':0.0260638,'PDT000395396.1':0.0250133)'':0.0366327,('PDT000636210.1':0.0389193,'PDT000346743.1':0.0482657)'':0.00098774)'':0.030425)'':0.00186565,(((('PDT000205452.1':0.071954,('PDT000318994.1':0.0548462,('PDT000535329.1':0.0548252,'PDT000008839.2':0.0490328)'':0.00799176)'':0.0110904)'':0.00209512,('PDT000249959.1':0.0676036,('PDT000497399.1':0.0606167,('PDT000319023.1':0.0655078,('PDT000088306.2':0.028368,('PDT000200991.2':0.0237415,'PDT000425975.1':0.0266491)'':0.00641588)'':0.0192847)'':0.00829526)'':0.005121)'':0.00347903)'':0.00359515,(('PDT000251477.1':0.0626009,('PDT000535320.1':0.05326,('PDT000713813.1':0.0468604,'PDT000252438.1':0.0460248)'':0.00904044)'':0.0199584)'':0.00311176,(('PDT000520789.1':0.0575653,('PDT000713537.1':0.0413399,('PDT000426046.1':0.0394719,'PDT000556968.1':0.0218197)'':0.00667207)'':0.00925762)'':0.011031,(('PDT000713300.1':0.0230571,'PDT000543331.1':0.0408422)'':0.0109851,('PDT000042448.1':0.018778,'PDT000501906.1':0.0225993)'':0.0129014)'':0.0104304)'':0.0166954)'':-1.25112e-05)'':0.00273669,((('PDT000619622.1':0.0660168,'PDT000482838.1':0.0649592)'':0.00541479,('PDT000444931.1':0.0424797,('PDT000394867.1':0.0294941,'PDT000566098.1':0.0392247)'':0.0073859)'':0.0317771)'':0.00463398,(('PDT000216210.2':0.0964198,('PDT000081758.1':0.0460921,'PDT000396759.1':0.0412931)'':0.0258666)'':-0.00195933,(('PDT000481337.1':0.0630187,('PDT000663933.1':0.0627905,('PDT000713909.1':0.0339251,'PDT000429787.1':0.0240002)'':0.0238354)'':0.0030995)'':0.00463692,('PDT000399945.1':0.0891562,('PDT000352533.1':0.0683432,('PDT000664462.1':0.0787377,('PDT000639557.1':0.0615742,(('PDT000395487.1':0.0723755,('PDT000535185.1':0.0296326,('PDT000482062.1':0.030423,'PDT000557655.1':0.0264107)'':0.00373787)'':0.0166045)'':-0.00215262,(('PDT000645152.1':0.0457545,'PDT000249940.1':0.0428082)'':0.00239786,(('PDT000557155.1':0.0447354,'PDT000661156.1':0.0533019)'':0.00172672,('PDT000174295.1':0.0115572,'PDT000174297.1':0.017352)'':0.0344798)'':0.00127315)'':0.00343105)'':0.00567267)'':0.00339403)'':0.00290757)'':0.00407035)'':-0.00630062)'':0.00424347)'':-0.000679869)'':0.00324648)'':0.00225493)'':0.00133388)'':0.00284135)'':0.0256432)'':0.00757522,((('PDT000228170.1':0.108956,('PDT000557360.1':0.0729639,(('PDT000317613.1':0.0357952,'PDT000642299.1':0.0340307)'':0.0160131,('PDT000367706.1':0.0617811,'PDT000631536.1':0.0604219)'':0.00285724)'':0.022859)'':0.0182715)'':0.0128484,(('PDT000290410.4':0.0988764,('PDT000455222.1':0.0506356,('PDT000386759.1':0.0228149,('PDT000280984.2':0.0198765,'PDT000281105.2':0.0143395)'':0.00420643)'':0.0223874)'':0.062926)'':0.0154233,(('PDT000181809.2':0.0803768,('PDT000398140.1':0.0489655,'PDT000650029.1':0.0385089)'':0.023811)'':0.0550698,(('PDT000396831.1':0.056275,'PDT000228075.1':0.048983)'':0.0660378,((('PDT000215759.2':0.0740871,('PDT000150249.2':0.0564886,('PDT000241671.2':0.0406735,'PDT000124726.1':0.0389026)'':0.0119263)'':0.0143715)'':0.03283,((('PDT000587795.1':0.083888,'PDT000652577.1':0.074466)'':0.00480016,('PDT000200969.3':0.0620493,('PDT000040369.4':0.0262846,'PDT000497829.1':0.0285367)'':0.0235535)'':0.0394735)'':0.00533145,((('PDT000557121.1':0.077322,'PDT000607574.1':0.073464)'':0.00569225,('PDT000252622.1':0.0544635,'PDT000252665.1':0.0463105)'':0.0315278)'':0.00322898,('PDT000509486.1':0.100701,('PDT000343908.1':0.0554633,('PDT000196635.2':0.0510793,'PDT000235756.2':0.0533097)'':0.0178832)'':0.0116789)'':0.0132578)'':0.0068782)'':0.00518797)'':0.00030299,((('PDT000720299.1':0.037671,'PDT000248477.1':0.0367857)'':0.0730955,(('PDT000502695.1':0.0545934,'PDT000221045.1':0.0609866)'':0.0178285,('PDT000713616.1':0.0632122,'PDT000384489.1':0.0600758)'':0.00613523)'':0.0170614)'':0.00249879,((('PDT000287322.2':0.0496184,'PDT000426631.1':0.0527906)'':0.0205622,('PDT000427013.1':0.0889864,'PDT000710222.1':0.0823006)'':0.00206559)'':0.0136814,(('PDT000228080.1':0.0651386,'PDT000228314.1':0.0869264)'':0.031902,(('PDT000689817.1':0.0854503,('PDT000305523.2':0.0757584,'PDT000393064.1':0.0687896)'':0.0120802)'':0.000625574,(('PDT000395429.1':0.0970372,('PDT000174296.1':0.010881,'PDT000174300.1':0.00753979)'':0.0672834)'':0.0100018,(('PDT000343872.1':0.052155,'PDT000661135.1':0.057602)'':0.0019535,('PDT000250123.1':0.0341273,('PDT000714283.1':0.0278131,('PDT000383923.1':0.0233344,('PDT000318871.1':0.0123419,('PDT000579619.1':0.0189881,('PDT000332775.1':0.0103516,'PDT000048013.1':0.00824433)'':0.00369392)'':0.00240055)'':0.00206345)'':0.011247)'':0.0145156)'':0.0153412)'':0.031309)'':0.004054)'':0.00631649)'':1.15277e-05)'':0.00665335)'':-0.000117503)'':0.00410178)'':0.00208675)'':-0.000229654)'':0.000835374)'':-0.000624364,((((('PDT000181818.2':0.0556725,'PDT000712622.1':0.0762195)'':0.0199896,('PDT000181836.2':0.0637222,('PDT000712638.1':0.0595914,('PDT000181669.2':0.0487552,'PDT000556931.1':0.0541408)'':0.00492457)'':0.0084589)'':0.00486438)'':0.0140653,(('PDT000124677.1':0.0646215,('PDT000221659.2':0.0485227,('PDT000345954.1':0.0401981,'PDT000020081.1':0.0393188)'':0.00220989)'':0.00728942)'':0.0119901,('PDT000630424.1':0.0592272,('PDT000661077.1':0.0482009,('PDT000650530.1':0.0388519,('PDT000619972.1':0.034383,'PDT000346204.1':0.0407991)'':0.00479305)'':0.00607802)'':0.00943003)'':0.00259131)'':0.0135109)'':0.00432446,('PDT000518546.1':0.0972832,(('PDT000181758.2':0.0459058,'PDT000346962.1':0.0532184)'':0.0376008,((('PDT000060666.2':0.032978,'PDT000566178.1':0.0392572)'':0.0199991,('PDT000566152.1':0.0608915,('PDT000178609.2':0.0387183,'PDT000395335.1':0.0327934)'':0.0117756)'':0.0058465)'':0.0222746,('PDT000181713.2':0.0888991,(('PDT000242487.1':0.0883202,('PDT000181748.2':0.0738026,'PDT000346907.1':0.0708414)'':0.0154213)'':0.0044078,((('PDT000181787.2':0.0670163,'PDT000346598.1':0.0640527)'':0.00324583,(('PDT000362831.1':0.048705,'PDT000131792.2':0.0464694)'':0.00659752,('PDT000559259.1':0.0540709,('PDT000226474.2':0.0415329,'PDT000626564.1':0.0353376)'':0.00493353)'':0.00543546)'':0.00748311)'':0.00133088,('PDT000659703.1':0.0752974,(('PDT000504752.1':0.0622352,'PDT000248782.1':0.0543758)'':0.00617494,('PDT000046097.1':0.0654044,(('PDT000069156.2':0.0539929,('PDT000046605.1':0.0582273,('PDT000240296.1':0.0449478,('PDT000626558.1':0.0343138,'PDT000463683.1':0.0348084)'':0.007299)'':0.00600715)'':0.00263019)'':0.00228264,(('PDT000463631.1':0.00919876,'PDT000463641.1':0.0105159)'':0.0336151,('PDT000654230.1':0.0303095,('PDT000500521.1':0.037928,'PDT000714689.1':0.0365149)'':0.00278254)'':0.0059864)'':0.00434716)'':0.0096602)'':-0.00395136)'':0.0124193)'':-0.00728216)'':0.00729609)'':0.0250999)'':-0.0156884)'':0.00573217)'':0.015102)'':-0.0118216)'':0.0219996,(('PDT000228185.1':0.0688686,('PDT000206126.4':0.0472553,('PDT000206112.4':0.0370878,('PDT000200196.4':0.021916,'PDT000203105.4':0.0256751)'':0.00821599)'':0.0177879)'':0.0375702)'':0.044581,((('PDT000248813.1':0.054818,(('PDT000421827.1':0.0239717,'PDT000466086.1':0.0217594)'':0.0164183,('PDT000399940.1':0.0213437,(('PDT000532330.1':0.0252212,'PDT000619653.1':0.0205856)'':0.00324844,('PDT000687482.1':0.0164984,'PDT000251612.1':0.012104)'':0.00388481)'':0.00228)'':0.0123518)'':0.0330211)'':0.02457,(('PDT000619963.1':0.0384616,('PDT000426734.1':0.0461761,'PDT000253675.1':0.0346954)'':0.00889143)'':0.0484056,('PDT000661080.1':0.10216,('PDT000583496.1':0.105182,(('PDT000557336.1':0.0787141,('PDT000145174.2':0.0584657,'PDT000577515.1':0.0679053)'':0.0109919)'':0.00175044,('PDT000374993.1':0.0379735,('PDT000375004.1':0.0276013,'PDT000566074.1':0.029617)'':0.0103117)'':0.0400017)'':0.00723663)'':0.00528423)'':-0.00393251)'':0.00240966)'':0.00868512,(('PDT000503003.1':0.0766449,'PDT000285324.1':0.0937601)'':0.0331207,(('PDT000652242.1':0.0971209,(('PDT000642597.1':0.0334513,'PDT000642598.1':0.0279061)'':0.0215375,('PDT000708757.1':0.0503532,('PDT000200105.2':0.0312793,('PDT000095767.2':0.0134253,'PDT000428415.1':0.0113164)'':0.0324313)'':0.018943)'':0.00428425)'':0.0509284)'':0.00987447,(('PDT000565883.1':0.114763,('PDT000590160.1':0.0971509,'PDT000711733.1':0.0827821)'':0.00853319)'':0.0147704,((('PDT000466978.2':0.0968071,(('PDT000691352.1':0.0614711,('PDT000637312.1':0.0505366,'PDT000048139.1':0.0477959)'':0.0155751)'':0.0115149,(('PDT000300824.2':0.0150878,'PDT000409822.1':0.0195043)'':0.0384183,('PDT000396838.1':0.0247786,'PDT000539052.1':0.0274149)'':0.031687)'':0.00851539)'':0.0115324)'':0.0100473,(('PDT000213942.2':0.116073,('PDT000590205.1':0.0708185,'PDT000251948.1':0.0471145)'':0.0302734)'':0.00105155,((('PDT000399578.1':0.058945,'PDT000346585.1':0.075718)'':0.0246598,(('PDT000305465.3':0.0398248,'PDT000439457.1':0.035337)'':0.0289625,('PDT000109449.2':0.0785255,'PDT000502678.1':0.0732435)'':0.00269414)'':0.00906074)'':-0.000535754,('PDT000411085.1':0.117934,(('PDT000251724.1':0.0909283,(('PDT000535527.1':0.0507526,'PDT000548978.1':0.0660474)'':0.0159085,('PDT000251902.1':0.0577137,('PDT000637330.1':0.0362806,('PDT000088257.2':0.0350705,'PDT000517940.1':0.0301448)'':0.00559563)'':0.0203282)'':0.00669426)'':0.00761243)'':-0.00119003,('PDT000399692.1':0.0919558,('PDT000485413.1':0.0743224,(('PDT000160956.2':0.0446623,'PDT000083258.2':0.0561487)'':0.00519136,('PDT000556915.1':0.0691347,('PDT000228029.1':0.0595988,'PDT000319003.1':0.0519332)'':0.00563085)'':0.00137702)'':0.00877916)'':0.0177365)'':-0.00809746)'':0.020757)'':-0.0101939)'':0.00217238)'':0.00527992)'':-0.000346064,(('PDT000556909.1':0.0849564,'PDT000032334.1':0.0735226)'':0.0160416,(('PDT000650302.1':0.0861875,(('PDT000251485.1':0.045933,('PDT000719248.1':0.0260901,'PDT000251539.1':0.0210242)'':0.0139238)'':0.0454723,(('PDT000557354.1':0.0790682,('PDT000203073.2':0.0636532,'PDT000482806.1':0.0717628)'':0.0156443)'':0.0115847,('PDT000421981.1':0.0923034,('PDT000168487.1':0.0821939,('PDT000591968.1':0.0710701,('PDT000136050.2':0.0555458,('PDT000521580.1':0.0386108,'PDT000251538.1':0.0399469)'':0.0149124)'':0.0143075)'':0.00193367)'':0.00525614)'':-0.000762963)'':0.00215292)'':0.00641908)'':-0.00194418,('PDT000248490.1':0.0879146,((('PDT000482079.1':0.0755048,'PDT000174341.1':0.0611712)'':0.0079876,('PDT000485434.1':0.0401895,'PDT000252669.1':0.0384687)'':0.0294551)'':0.00309563,(('PDT000228208.1':0.0937286,('PDT000556970.1':0.0529632,('PDT000714643.1':0.019253,'PDT000319031.1':0.0233922)'':0.0223394)'':0.020359)'':0.0110392,(('PDT000482836.1':0.075329,('PDT000463620.1':0.0143833,'PDT000463650.1':0.0165364)'':0.0582241)'':0.0105363,(('PDT000497316.1':0.0613234,('PDT000557048.1':0.0338179,('PDT000399604.1':0.00433751,'PDT000627605.1':0.00444317)'':0.0323925)'':0.0195734)'':0.0124293,('PDT000104648.2':0.0880336,(('PDT000093015.2':0.0463244,'PDT000540594.1':0.0601746)'':0.0124043,('PDT000457706.1':0.0692847,('PDT000718850.1':0.0554673,('PDT000354291.1':0.0588682,('PDT000346487.1':0.0517212,('PDT000218133.2':0.0392565,'PDT000251540.1':0.0328765)'':0.0053509)'':0.00495414)'':0.00320009)'':0.00327925)'':0.000874467)'':0.00681465)'':-0.0017239)'':0.00995311)'':0.00133033)'':0.000844923)'':0.00990525)'':-0.00641295)'':0.00349871)'':0.00558038)'':0.00410864)'':0.000317408)'':0.00878504)'':-0.0109146)'':0.00761587)'':-0.00418723)'':0.00620043)'':0.00133009)'':0.0126351)'':0.020762)'':0.024481)'':-0.00138707,((((('PDT000132255.2':0.0715237,(('PDT000005194.2':0.0426911,('PDT000205052.2':0.0291354,'PDT000011098.2':0.0253924)'':0.0230549)'':0.0115162,(('PDT000107667.2':0.0382789,'PDT000208719.1':0.041238)'':0.00392978,(('PDT000223015.2':0.0272959,('PDT000278350.3':0.0313972,('PDT000132157.2':0.0164888,'PDT000032330.1':0.0239794)'':0.0120697)'':0.00150675)'':0.000497618,(('PDT000678441.1':0.0180039,('PDT000600859.1':0.0204806,'PDT000571830.1':0.0162311)'':0.00379567)'':0.00868963,('PDT000008850.2':0.0181267,('PDT000008836.2':0.0227321,('PDT000008851.2':0.0150388,'PDT000370994.1':0.023136)'':0.00395421)'':0.00355487)'':0.00906537)'':0.00285612)'':0.0127275)'':0.0204425)'':0.00107019)'':-0.00882434,((('PDT000205054.2':0.0462458,'PDT000278345.2':0.0492962)'':0.00384687,(('PDT000280664.3':0.0317965,('PDT000324187.1':0.0117459,'PDT000323327.1':0.0138595)'':0.010272)'':0.0084665,(('PDT000392144.1':0.0281024,'PDT000189713.3':0.0291265)'':0.0051548,('PDT000132005.2':0.0275379,('PDT000266784.2':0.0143569,'PDT000131854.2':0.0137524)'':0.00940165)'':0.00355512)'':0.00939756)'':0.00291086)'':0.00112891,((('PDT000324037.1':0.0132037,'PDT000637692.1':0.011905)'':0.0131051,('PDT000415294.1':0.0290286,('PDT000481336.1':0.0219767,('PDT000332191.1':0.0285477,('PDT000205060.2':0.0127449,'PDT000604743.1':0.0146402)'':0.00549176)'':0.00149654)'':0.00412695)'':0.000210648)'':0.0121462,(('PDT000611526.1':0.0346557,'PDT000107740.3':0.0294569)'':0.00523623,('PDT000132101.2':0.0464675,(('PDT000327315.1':0.0297863,('PDT000318595.1':0.020954,'PDT000339394.1':0.0215783)'':0.00649467)'':-0.000227113,(('PDT000010996.2':0.0243511,('PDT000010997.2':0.0153539,('PDT000093880.1':0.00313794,'PDT000008858.2':0.00454286)'':0.00491705)'':0.00386436)'':0.0140063,('PDT000368907.1':0.025338,('PDT000368908.1':0.00505656,('PDT000093881.1':0.00134838,'PDT000008193.2':0.00135027)'':0.00163295)'':0.0123966)'':0.00516567)'':0.0078871)'':0.00781403)'':-0.00276154)'':0.00363662)'':0.00447969)'':0.0118673)'':0.095703,(((('PDT000011794.2':0.0705039,('PDT000150754.2':0.0377379,'PDT000344855.2':0.0360318)'':0.0287042)'':0.0221765,('PDT000142218.2':0.0855105,('PDT000228167.1':0.0689397,('PDT000228139.1':0.0305169,'PDT000396471.1':0.0234517)'':0.0229765)'':0.0169215)'':0.0156407)'':0.0182362,(('PDT000293479.1':0.037978,'PDT000293523.1':0.0379007)'':0.0820612,(('PDT000637072.1':0.0487406,'PDT000248833.1':0.0442112)'':0.012319,(('PDT000107175.3':0.00356632,('PDT000504863.2':0.00348677,'PDT000655384.2':0.00379067)'':0.000621104)'':0.000663853,('PDT000140573.3':0.00551864,('PDT000548696.2':0.00239612,('PDT000668168.2':0.0014428,'PDT000140473.3':0.00145315)'':0.00150072)'':0.00148927)'':0.000319185)'':0.0688216)'':0.0322307)'':0.00881456)'':0.00709387,((('PDT000499509.1':0.0407293,('PDT000333674.1':0.0063454,'PDT000496902.1':0.0054848)'':0.0278449)'':0.0630543,('PDT000095822.2':0.112406,('PDT000251497.1':0.0730473,(('PDT000581024.1':0.0645164,('PDT000239219.2':0.0218641,('PDT000428357.1':0.00441429,'PDT000620249.1':0.00535445)'':0.01605)'':0.0326843)'':0.00506331,('PDT000539898.1':0.0488964,('PDT000045602.1':0.0473157,('PDT000444817.1':0.0241275,'PDT000617809.1':0.0144375)'':0.0187472)'':0.0183233)'':-0.000831108)'':0.0180065)'':0.0227169)'':-0.00842334)'':0.00348684,('PDT000652580.1':0.122182,((('PDT000329290.2':0.0320296,('PDT000147504.3':0.0195946,'PDT000127286.3':0.0173804)'':0.0274231)'':0.00789867,('PDT000140208.3':0.020467,('PDT000502367.1':0.0322716,('PDT000501956.1':0.0195173,('PDT000502295.1':0.0227295,'PDT000502296.1':0.0222485)'':0.00441228)'':0.00550839)'':0.00256823)'':0.0272155)'':0.027746,(('PDT000718939.1':0.0622956,'PDT000228350.1':0.0617874)'':0.0351416,('PDT000425696.1':0.0823027,('PDT000711739.1':0.0933729,('PDT000252465.1':0.0611626,('PDT000471028.1':0.0676353,(('PDT000720230.1':0.0263528,('PDT000355839.1':0.0146979,'PDT000379445.1':0.0209917)'':0.00410825)'':0.0294484,('PDT000395026.1':0.0226958,('PDT000368350.2':0.026365,(('PDT000645923.2':0.0143567,'PDT000690494.1':0.0115377)'':0.0129906,('PDT000573344.2':0.028723,('PDT000171572.3':0.0149603,('PDT000085582.3':0.00580446,'PDT000140561.3':0.00465784)'':0.00285328)'':0.00676413)'':0.000660343)'':0.00838365)'':0.00515635)'':0.0224303)'':0.00622385)'':0.00375025)'':0.00905655)'':0.0012495)'':0.00433132)'':-0.00639084)'':0.0177092)'':0.00727083)'':0.0206296)'':0.0158396)'':0.0184975,((((('PDT000132138.2':0.0416256,'PDT000520726.1':0.0355439)'':0.00223951,('PDT000424802.1':0.0264863,('PDT000248729.1':0.00907144,'PDT000248733.1':0.00918616)'':0.0198792)'':0.0097596)'':0.0395667,(('PDT000213266.2':0.0345606,(('PDT000205087.2':0.0137588,'PDT000625397.1':0.0114787)'':0.0094546,('PDT000253142.2':0.0166887,'PDT000625445.1':0.0279169)'':0.00577775)'':0.0197065)'':-0.00391383,(('PDT000132127.2':0.0215991,('PDT000131816.2':0.0198987,'PDT000131879.2':0.0233743)'':0.0115076)'':0.00212,('PDT000428900.1':0.0230007,('PDT000131900.2':0.0204216,'PDT000645787.1':0.0145151)'':0.00489054)'':0.0060472)'':0.0194682)'':0.0339088)'':0.0338068,(((('PDT000317157.1':0.0303622,'PDT000377115.1':0.0311673)'':0.0217074,('PDT000328013.1':0.0150652,'PDT000379564.1':0.0125361)'':0.0260496)'':0.0139899,(('PDT000162000.2':0.0297409,'PDT000205062.2':0.0393493)'':0.0159929,('PDT000549211.1':0.0438794,(('PDT000161977.2':0.0122283,'PDT000253418.2':0.0123749)'':0.00251637,('PDT000365933.1':0.00638619,('PDT000486032.1':0.00503753,'PDT000640695.1':0.00484074)'':0.00107108)'':0.00899476)'':0.0185625)'':0.00837087)'':0.0141505)'':0.0515274,('PDT000683508.1':0.124477,((('PDT000610637.1':0.0844211,('PDT000162048.2':0.0307913,('PDT000253427.2':0.0349708,'PDT000711416.1':0.0240318)'':0.00485665)'':0.0266531)'':-0.000378479,('PDT000645801.1':0.0498082,('PDT000502236.1':0.0448253,(('PDT000520303.1':0.0192865,'PDT000667973.1':0.027686)'':0.000792592,('PDT000480424.1':0.014824,('PDT000238209.2':0.0162351,'PDT000411993.1':0.0156481)'':0.00276265)'':0.00649979)'':0.0138942)'':0.0105835)'':0.01484)'':0.0195611,((('PDT000577024.1':0.0202172,'PDT000332741.1':0.0215272)'':0.0406154,('PDT000274714.2':0.0513473,('PDT000635778.1':0.0483702,('PDT000205067.2':0.00507532,'PDT000490345.1':0.00388615)'':0.0355986)'':0.0100004)'':0.0125192)'':0.00818914,('PDT000248725.1':0.0723433,(('PDT000132015.2':0.0311544,('PDT000558677.1':0.0190646,('PDT000322011.1':0.0068856,('PDT000674764.1':0.00347601,('PDT000496152.1':0.00295874,'PDT000711074.1':0.0030257)'':0.00140182)'':0.00319033)'':0.0129568)'':0.00472098)'':0.0365123,(('PDT000517456.1':0.0243904,('PDT000132079.2':0.0276157,('PDT000160288.2':0.011634,'PDT000625404.1':0.0125803)'':0.00781583)'':0.00727722)'':0.00181139,('PDT000660737.1':0.0405657,('PDT000324039.1':0.0138311,('PDT000131860.2':0.0146316,'PDT000674762.1':0.00858708)'':0.00566902)'':0.00860256)'':0.0126327)'':0.0291114)'':0.0134597)'':-0.00167826)'':0.00624877)'':0.0381726)'':-0.0160249)'':0.000376789)'':0.0212258,(('PDT000376137.2':0.115146,('PDT000667767.2':0.143199,('PDT000252718.1':0.064616,('PDT000556997.1':0.0481568,('PDT000250236.1':0.0454606,'PDT000251940.1':0.0613284)'':0.00495002)'':0.00461409)'':0.0572319)'':0.00645395)'':0.0399371,((('PDT000221065.1':0.0411106,(('PDT000478551.1':0.0164626,'PDT000017866.2':0.017429)'':0.000397625,('PDT000221032.1':0.0184146,'PDT000221053.1':0.0104296)'':0.0120491)'':0.0172974)'':0.064734,(('PDT000174306.1':0.0869975,('PDT000568776.1':0.0792955,('PDT000673501.1':0.0617438,('PDT000573609.1':0.0444105,'PDT000687365.1':0.043791)'':0.00915748)'':0.00474306)'':0.00739181)'':0.0167708,(('PDT000372013.1':0.0686738,('PDT000583376.1':0.0652875,'PDT000221048.1':0.0688805)'':0.00140022)'':0.0130325,(('PDT000483197.1':0.0629914,'PDT000248770.1':0.0797576)'':0.00535187,(('PDT000522495.1':0.0612235,'PDT000384973.1':0.0517795)'':0.0108936,('PDT000249874.1':0.0802625,('PDT000652429.1':0.0565154,('PDT000346001.1':0.0745158,('PDT000426686.1':0.0416058,('PDT000510047.1':0.0324743,'PDT000520972.1':0.0436169)'':0.00304454)'':0.00763822)'':0.00706506)'':0.0047227)'':0.00254148)'':0.00367604)'':0.00860781)'':9.71938e-05)'':0.00649667)'':0.00516118,(('PDT000395312.1':0.06176,('PDT000200118.2':0.0153993,('PDT000206115.2':0.00888201,'PDT000215585.2':0.00751799)'':0.0128719)'':0.052834)'':0.0698726,(('PDT000404832.2':0.0672666,('PDT000497334.1':0.0497522,'PDT000577094.1':0.0499806)'':0.0134055)'':0.0353482,((('PDT000174309.1':0.0677932,('PDT000652262.1':0.0826023,'PDT000174365.1':0.0484137)'':0.0210218)'':0.0420687,(('PDT000171795.1':0.0755235,'PDT000228319.1':0.0615675)'':0.0510392,('PDT000060933.3':0.0882226,((('PDT000651493.1':0.049498,'PDT000221016.1':0.055054)'':0.0134134,('PDT000133499.1':0.080777,('PDT000565857.1':0.0334528,'PDT000619553.1':0.0414962)'':0.028453)'':0.0136979)'':0.00281387,((('PDT000150313.2':0.0285897,'PDT000407303.1':0.0307999)'':0.0128964,('PDT000216078.2':0.0430494,('PDT000271540.2':0.0310739,'PDT000496886.1':0.0337983)'':0.0146784)'':0.00240898)'':0.00176973,('PDT000661103.1':0.0550189,('PDT000147456.3':0.0353674,('PDT000471048.1':0.0378758,((('PDT000346014.1':0.0343709,('PDT000258733.2':0.0234653,'PDT000409817.1':0.0231532)'':0.0031105)'':0.00100247,(('PDT000394825.1':0.0276623,'PDT000397772.1':0.0300956)'':0.00124968,('PDT000645019.1':0.0173454,'PDT000395470.1':0.0140131)'':0.00936999)'':0.00413224)'':0.00151201,('PDT000258738.2':0.02773,('PDT000651498.1':0.0339047,(('PDT000541639.1':0.0194927,('PDT000364426.1':0.0115841,'PDT000577092.1':0.00893177)'':0.00603903)'':0.00477548,('PDT000399651.1':0.0367151,('PDT000281104.2':0.0232391,('PDT000178597.2':0.0204247,('PDT000215565.2':0.0127779,'PDT000565898.1':0.0147441)'':0.00870582)'':0.00134067)'':0.0037649)'':-0.000906258)'':0.00524333)'':0.00480663)'':-0.00276758)'':0.00341735)'':0.00186866)'':0.0159265)'':-0.0106212)'':0.0443277)'':0.00487664)'':0.00736898)'':0.0112439)'':-0.0144973,('PDT000425800.1':0.0999199,(('PDT000150863.1':0.0918519,('PDT000228329.1':0.0941067,('PDT000490239.1':0.0672972,('PDT000228288.1':0.067551,(('PDT000178608.2':0.0621975,'PDT000221034.1':0.0401935)'':0.0149578,('PDT000150246.2':0.0537897,('PDT000652259.1':0.0522678,('PDT000190131.2':0.0213,'PDT000502535.1':0.0246534)'':0.0318672)'':0.00402145)'':0.00611726)'':0.00663211)'':0.00646325)'':0.00268802)'':0.00640901)'':-0.00643275,((('PDT000115049.3':0.0698357,('PDT000285794.3':0.0631113,'PDT000407670.2':0.0649727)'':0.0133973)'':0.00958025,('PDT000161921.3':0.0289105,('PDT000137659.3':0.0338578,('PDT000140228.3':0.029739,'PDT000502560.1':0.027784)'':0.00343828)'':0.00396745)'':0.0373641)'':0.0109344,(('PDT000309637.3':0.0740223,('PDT000470105.1':0.0308352,'PDT000526116.1':0.0303397)'':0.0503822)'':0.00548916,((('PDT000082159.3':0.0315601,('PDT000362186.2':0.0392594,'PDT000391442.2':0.0269612)'':0.00693501)'':0.00136619,('PDT000491906.2':0.0512056,(('PDT000147503.3':0.0197758,'PDT000289656.3':0.0224224)'':0.00802687,('PDT000148513.3':0.0397704,('PDT000502269.1':0.0271671,'PDT000502276.1':0.0286863)'':0.00783473)'':0.00210005)'':0.00319344)'':0.00127055)'':0.0251424,(('PDT000161975.3':0.0391324,(('PDT000568711.2':0.0135539,'PDT000140560.3':0.0118329)'':0.01635,('PDT000548335.2':0.0149916,('PDT000324617.2':0.00962033,'PDT000548334.2':0.00876567)'':0.00575685)'':0.00712368)'':0.015656)'':0.00623153,(('PDT000253445.3':0.0485402,'PDT000050589.2':0.0377423)'':0.00449032,(('PDT000270797.3':0.0154869,'PDT000140489.3':0.0105853)'':0.00733126,('PDT000084164.3':0.0208733,(('PDT000302828.3':0.00812179,'PDT000542075.2':0.00667611)'':0.00632236,('PDT000274863.3':0.0145832,(('PDT000084235.12':0.00434709,'PDT000497859.1':0.00391902)'':0.001316,('PDT000533751.2':0.00655997,'PDT000566501.2':0.00626433)'':0.000499141)'':0.0045855)'':-0.000744573)'':0.00342257)'':-0.000341675)'':0.0292355)'':0.00340489)'':0.00770142)'':0.0228903)'':-0.00368125)'':0.00575744)'':0.0103204)'':-0.00782749)'':0.00841856)'':0.0036434)'':-0.00328282)'':0.0527538)'':-0.0321854)'':0.0387952)'':0.00240464,(((('PDT000625464.1':0.174854,((('PDT000088230.1':0.0433498,('PDT000131993.2':0.0449448,'PDT000649802.1':0.036765)'':0.00934165)'':0.0100758,('PDT000038898.1':0.0673857,((('PDT000611514.1':0.0247491,(('PDT000649519.1':0.0133881,'PDT000244073.1':0.0128184)'':0.000869923,('PDT000412830.1':0.0108476,('PDT000384157.1':0.0116771,('PDT000537057.1':0.00498403,'PDT000629983.1':0.00626087)'':0.00583818)'':0.000529447)'':0.00381945)'':0.010257)'':-0.00613049,('PDT000551008.1':0.0257853,(('PDT000469863.1':0.00550282,'PDT000558676.1':0.00484848)'':0.00593955,(('PDT000421167.1':0.00758186,'PDT000575756.1':0.00656864)'':0.00303027,('PDT000600733.1':0.00969771,'PDT000631665.1':0.00963989)'':0.00137875)'':0.00442339)'':0.00982784)'':-0.00551472)'':0.0132504,(('PDT000009392.2':0.0154874,('PDT000557534.1':0.0100774,('PDT000009393.2':0.00516696,('PDT000009394.2':0.00324726,('PDT000009395.2':0.0031259,'PDT000009396.2':0.00345305)'':0.000844726)'':0.000726903)'':0.00576362)'':0.00478992)'':0.00344679,('PDT000205110.2':0.0258226,(('PDT000394739.1':0.0123849,'PDT000599887.1':0.0135634)'':0.00218634,(('PDT000625396.1':0.00824352,('PDT000321556.1':0.00615082,'PDT000425814.1':0.00667858)'':0.00189173)'':0.00738163,('PDT000520827.1':0.00823054,('PDT000157147.2':0.00237477,'PDT000131895.2':0.00241565)'':0.0091276)'':-0.000234222)'':0.00497482)'':0.00924165)'':-0.00549601)'':0.00649978)'':0.0239655)'':0.00624874)'':0.0501422,(((('PDT000390836.1':0.0200081,'PDT000034409.1':0.0253394)'':0.00979487,('PDT000332827.1':0.0278296,('PDT000157692.2':0.0259574,('PDT000205092.2':0.010018,'PDT000588351.1':0.0110765)'':0.00842844)'':0.00422962)'':0.0077395)'':0.041798,(('PDT000709770.1':0.0511035,(('PDT000156662.2':0.0391477,'PDT000334194.1':0.0332567)'':0.00752363,('PDT000399958.1':0.0356438,('PDT000160320.2':0.0352202,(('PDT000283368.2':0.0149076,'PDT000132232.2':0.0216488)'':0.00388325,('PDT000328029.1':0.012612,'PDT000610631.1':0.01628)'':0.00285632)'':0.00850868)'':0.00818159)'':0.00486008)'':0.0133504)'':-0.0144519,((('PDT000132052.2':0.0103938,('PDT000155877.2':0.00893112,'PDT000205088.2':0.00717548)'':0.0041811)'':0.00178536,('PDT000344861.1':0.0122173,('PDT000132148.2':0.00963183,'PDT000674748.1':0.0152715)'':0.00242453)'':0.00228502)'':0.00995439,(('PDT000678051.1':0.0196914,('PDT000625492.1':0.00475816,'PDT000322950.1':0.00698784)'':0.00586469)'':0.00809656,('PDT000551009.1':0.0186109,('PDT000421948.1':0.0150598,('PDT000131928.2':0.00651939,'PDT000500623.1':0.0100909)'':0.0037361)'':0.0025277)'':0.00609501)'':0.00546601)'':0.0136303)'':0.0318798)'':0.00143761,((('PDT000557769.1':0.0391216,(('PDT000619169.1':0.00644974,'PDT000580073.2':0.00680126)'':0.00735921,('PDT000600194.1':0.0134325,('PDT000366104.1':0.010378,'PDT000388610.1':0.0115336)'':0.00350064)'':0.00270711)'':0.00729418)'':0.0231146,('PDT000017832.2':0.0420541,(('PDT000465854.1':0.00582151,'PDT000496786.1':0.00624349)'':0.015429,(('PDT000332715.1':0.00978923,('PDT000421955.1':0.0111303,'PDT000718580.1':0.00799322)'':0.00320502)'':0.00137013,(('PDT000205061.2':0.0105647,'PDT000625456.1':0.00866986)'':0.00589695,('PDT000060993.2':0.00949602,('PDT000324145.1':0.00551654,'PDT000132365.2':0.00690886)'':0.00328758)'':0.00181538)'':0.000442755)'':0.00862477)'':0.0329074)'':0.000880937)'':0.0125556,('PDT000205078.2':0.0608256,((('PDT000498878.1':0.0342939,'PDT000559222.2':0.0425688)'':0.00131188,('PDT000509631.1':0.0217115,('PDT000205070.2':0.0127227,('PDT000579608.1':0.0152351,'PDT000660057.1':0.0111383)'':0.00358018)'':0.00630802)'':0.0185352)'':0.00905739,((('PDT000274720.2':0.0204474,'PDT000542155.1':0.031154)'':0.00678515,('PDT000359131.1':0.0205156,'PDT000468191.1':0.0313283)'':0.0050317)'':0.0140591,('PDT000564185.1':0.0480064,('PDT000047684.3':0.0454007,('PDT000486910.1':0.0185545,('PDT000378568.1':0.0127998,('PDT000156787.2':0.011315,('PDT000451165.1':0.0101667,('PDT000161560.2':0.0025039,'PDT000532331.1':0.00328915)'':0.00631447)'':0.00226236)'':0.00185892)'':0.0074399)'':0.0121798)'':0.0135453)'':-0.00917236)'':0.00704037)'':0.0043899)'':0.00592791)'':0.0136546)'':0.0408176)'':0.0511482)'':-0.0366696,(((('PDT000123479.3':0.130553,('PDT000040701.3':0.0675336,('PDT000388601.2':0.0501652,('PDT000340520.2':0.0216003,('PDT000198486.3':0.0177927,'PDT000066885.3':0.0123361)'':0.0053184)'':0.018733)'':0.0239002)'':0.0776216)'':-0.00300919,(('PDT000252728.1':0.0964604,(('PDT000548714.1':0.0563896,('PDT000149350.2':0.0400526,'PDT000470104.1':0.0424404)'':0.0175714)'':0.00452467,('PDT000252664.1':0.03643,('PDT000557020.1':0.0110992,'PDT000687840.1':0.0172477)'':0.0194868)'':0.0172518)'':0.0149042)'':-0.0056571,('PDT000373237.1':0.0768781,(('PDT000470085.1':0.0670572,('PDT000470103.1':0.0453135,'PDT000494254.1':0.0513923)'':0.0080074)'':0.00903349,('PDT000719172.1':0.0104077,('PDT000248824.1':0.00548879,'PDT000251446.1':0.00349594)'':0.00660883)'':0.0495628)'':0.00531959)'':0.00514017)'':0.0351908)'':0.000646953,('PDT000040704.2':0.107458,((('PDT000439458.1':0.0820756,'PDT000252566.1':0.0762774)'':0.00567704,('PDT000573562.1':0.108307,(('PDT000249753.1':0.0842277,('PDT000251986.1':0.0364525,('PDT000372786.1':0.0171859,'PDT000228174.1':0.0142737)'':0.0155631)'':0.0441302)'':-0.00333768,('PDT000081735.1':0.0865596,('PDT000312226.2':0.0524152,('PDT000293462.1':0.0396308,'PDT000293550.1':0.0341149)'':0.032188)'':0.0112868)'':0.0044185)'':0.0160846)'':0.000729364)'':-0.0021442,(('PDT000160263.3':0.0355102,'PDT000107653.2':0.0283529)'':0.0110017,(('PDT000711023.1':0.0471307,('PDT000151531.3':0.0285483,'PDT000156821.3':0.0261499)'':0.0168576)'':0.00446792,('PDT000487927.1':0.0506488,(('PDT000264718.3':0.0310969,'PDT000328041.2':0.0375572)'':0.00214602,('PDT000324716.2':0.0329393,('PDT000160132.3':0.0148613,'PDT000714280.1':0.0183239)'':0.0095395)'':0.00351678)'':0.00731844)'':0.0050103)'':0.00503051)'':0.0466062)'':0.00230107)'':0.00955179)'':0.00264359,(('PDT000661124.1':0.0984289,('PDT000562854.1':0.0881376,('PDT000430688.1':0.0487744,'PDT000501831.1':0.0432854)'':0.02052)'':0.0159843)'':0.032679,(('PDT000637722.1':0.0967781,(('PDT000438232.1':0.070624,'PDT000032349.1':0.063306)'':0.0186328,('PDT000663992.1':0.076731,'PDT000248760.1':0.06583)'':0.0102305)'':0.00601704)'':0.0160754,(('PDT000556337.2':0.0730878,(('PDT000589717.2':0.0275182,'PDT000710373.1':0.0252571)'':0.0209375,(('PDT000545075.2':0.0431134,('PDT000443396.2':0.038421,('PDT000098699.3':0.0260523,('PDT000156796.3':0.011152,'PDT000303491.3':0.00994762)'':0.0129041)'':0.00339864)'':0.00330692)'':0.000373959,((('PDT000615196.2':0.0191933,'PDT000618547.2':0.0157192)'':0.00769367,('PDT000708839.1':0.0103426,('PDT000171553.3':0.00634738,('PDT000158536.3':0.00632286,'PDT000719828.1':0.00831854)'':0.00348607)'':0.00108076)'':0.0102858)'':0.00940424,('PDT000156534.3':0.0432141,(('PDT000521263.2':0.0286626,'PDT000124622.1':0.0307953)'':0.00846651,('PDT000105656.4':0.0308864,('PDT000156802.3':0.0118747,('PDT000145366.3':0.00830484,'PDT000196471.3':0.0128234)'':0.00409008)'':0.00992471)'':0.000495136)'':0.00729491)'':-0.00205916)'':0.00113643)'':0.00697263)'':0.0126058)'':0.0433269,(('PDT000484146.1':0.0804952,(('PDT000619800.1':0.0568691,('PDT000063285.2':0.0246876,'PDT000405210.1':0.0253417)'':0.0182766)'':0.0147146,('PDT000083482.1':0.0733684,((('PDT000227558.2':0.0202363,'PDT000542699.1':0.016841)'':0.0294646,(('PDT000120840.3':0.0189468,'PDT000120867.3':0.0200805)'':0.0369707,('PDT000645774.1':0.0411386,('PDT000161505.3':0.0113769,('PDT000120820.3':0.00277724,'PDT000120863.3':0.00132194)'':0.0150712)'':0.0236032)'':0.00477474)'':0.0138844)'':0.0100566,(('PDT000717307.1':0.0481538,('PDT000687934.1':0.0253101,'PDT000553194.1':0.038082)'':0.019917)'':0.0023721,(('PDT000372056.1':0.0344112,'PDT000501991.1':0.029635)'':0.00185511,(('PDT000253432.2':0.0207433,'PDT000016370.2':0.0180744)'':0.00224065,('PDT000161665.2':0.0127994,('PDT000161714.2':0.0124003,'PDT000327290.1':0.0105473)'':0.00469502)'':0.0065727)'':0.00346327)'':0.0242376)'':0.00811962)'':0.00977431)'':-0.00237615)'':0.0176763)'':-0.0144433,((('PDT000083457.1':0.0299265,('PDT000120907.2':0.0279401,'PDT000625459.1':0.0238987)'':0.00384679)'':0.014524,(('PDT000323669.1':0.038409,'PDT000379486.1':0.0358651)'':0.0218483,(('PDT000617170.1':0.039334,('PDT000083447.1':0.000757961,'PDT000083549.1':0.000740989)'':0.0335737)'':0.0071325,('PDT000661105.1':0.0360815,('PDT000502996.1':0.0366222,('PDT000082638.1':0.0227307,('PDT000083521.1':0.0229885,'PDT000083536.1':0.0176026)'':0.00574557)'':0.00698814)'':0.00682022)'':-0.000423754)'':0.00590767)'':0.000727106)'':0.00888521,('PDT000637171.1':0.068738,(('PDT000157833.2':0.0357178,'PDT000463707.1':0.0364342)'':0.00989271,('PDT000518656.1':0.0554052,('PDT000668482.2':0.0422571,('PDT000083597.1':0.0414808,('PDT000651020.1':0.0345176,('PDT000679692.1':0.0314064,(('PDT000352558.1':0.0420991,('PDT000386248.1':0.00498751,'PDT000252670.1':0.00368993)'':0.0187937)'':0.00437773,(('PDT000131815.2':0.020707,('PDT000120870.2':0.0097545,'PDT000553169.1':0.0107629)'':0.0124021)'':0.00207258,(('PDT000585919.1':0.0243482,('PDT000131892.2':0.0164795,'PDT000367782.2':0.0148261)'':0.00459747)'':0.00281504,('PDT000161647.2':0.0197488,('PDT000625476.1':0.0112052,'PDT000083539.1':0.0153858)'':0.00546778)'':0.00321241)'':0.0043577)'':0.00176198)'':0.00335829)'':0.000883917)'':0.00153095)'':0.00423418)'':0.00515776)'':-0.00511424)'':0.0156039)'':-0.00484019)'':0.017723)'':0.015344)'':0.0135141)'':0.0125224)'':-0.012145)'':0.0104204)'':0.0450198,(((('PDT000398878.2':0.0412027,('PDT000260740.3':0.0265463,'PDT000625858.2':0.0231981)'':0.00905692)'':7.21378e-05,('PDT000551606.2':0.0215144,('PDT000601940.2':0.0278914,('PDT000325692.2':0.0139677,('PDT000319896.2':0.00672176,'PDT000576252.2':0.00600954)'':0.00380578)'':0.00543638)'':0.0066819)'':0.00593198)'':0.122242,(('PDT000258389.2':0.127846,(('PDT000502549.1':0.0796564,('PDT000360104.1':0.0459305,'PDT000395211.1':0.0386663)'':0.0460082)'':0.00975187,('PDT000009391.2':0.0491014,(('PDT000663917.1':0.0216437,'PDT000009390.2':0.0170532)'':0.0209777,('PDT000009371.2':0.0284108,'PDT000009389.2':0.040531)'':0.00855842)'':0.00734067)'':0.0500057)'':0.0192074)'':0.0135947,((('PDT000400013.2':0.0303451,'PDT000077525.1':0.024508)'':0.00500197,('PDT000077519.1':0.0197655,('PDT000216685.2':0.0139643,('PDT000065381.1':0.00937396,'PDT000216682.1':0.00617004)'':0.00367525)'':0.00438882)'':0.0208737)'':0.0299171,(('PDT000155915.3':0.0523281,('PDT000038853.1':0.0444821,'PDT000486511.1':0.0361004)'':0.0153827)'':0.00320122,(('PDT000050623.1':0.044417,('PDT000333286.2':0.0430634,('PDT000142226.3':0.0034033,'PDT000216692.1':0.00488592)'':0.0457599)'':0.0019242)'':0.00703862,(('PDT000163601.3':0.0272993,'PDT000105645.4':0.0167234)'':0.0201515,(('PDT000377090.2':0.0220218,'PDT000208705.1':0.0171206)'':0.0062277,(('PDT000157282.3':0.0166677,'PDT000325804.2':0.0141349)'':0.0108225,('PDT000332782.2':0.00909946,'PDT000553662.2':0.00978154)'':0.0122348)'':0.00308592)'':0.0164171)'':0.00320057)'':0.0108444)'':0.00287059)'':0.0733493)'':0.0178874)'':-0.00161385,((('PDT000250241.1':0.132968,(('PDT000252598.1':0.0216834,'PDT000278218.1':0.029104)'':0.00696969,('PDT000551879.1':0.0322529,('PDT000428398.1':0.0199948,'PDT000502659.1':0.0200485)'':0.00820075)'':0.00771466)'':0.0787534)'':0.017613,('PDT000093003.2':0.156661,((('PDT000293536.1':0.0956997,'PDT000395249.1':0.0993863)'':0.0083426,(('PDT000226491.2':0.0437148,'PDT000720187.1':0.0445177)'':0.00565854,('PDT000313319.2':0.0410876,'PDT000479630.1':0.0399857)'':0.0138053)'':0.0482957)'':0.031067,('PDT000248772.1':0.0828826,('PDT000413215.1':0.0821998,('PDT000415756.1':0.0616587,(('PDT000150300.2':0.033993,'PDT000420665.1':0.0392841)'':0.00161568,(('PDT000258744.2':0.0245847,'PDT000252183.1':0.0279902)'':0.00430906,('PDT000203411.2':0.0207865,('PDT000288658.2':0.0237341,('PDT000259613.2':0.0180934,'PDT000479092.1':0.0170998)'':0.00417046)'':0.00476376)'':7.47274e-05)'':0.00486114)'':0.0264143)'':0.0269255)'':0.0164423)'':0.0254172)'':0.0312292)'':-0.0101162)'':0.00710118,(((('PDT000703129.1':0.122241,('PDT000386181.1':0.106779,'PDT000250028.1':0.115494)'':0.0120916)'':-0.00145505,((('PDT000335804.1':0.0237321,('PDT000711097.1':0.0140794,'PDT000412014.1':0.0133476)'':0.00402838)'':0.0585669,('PDT000649795.1':0.0632584,(('PDT000434182.1':0.0381487,'PDT000480960.1':0.0192618)'':0.0223128,('PDT000357610.1':0.0407305,('PDT000285783.2':0.0221189,'PDT000492140.1':0.0268852)'':0.0182242)'':0.00225026)'':0.014683)'':0.012892)'':0.0121836,(('PDT000424761.1':0.063959,('PDT000423935.1':0.0299155,('PDT000253440.2':0.0245578,'PDT000629322.1':0.0255836)'':0.00716553)'':0.0443157)'':0.00406877,(('PDT000157188.2':0.0445845,'PDT000357669.1':0.0461853)'':0.0166079,(('PDT000625501.1':0.0114747,'PDT000682798.1':0.0154985)'':0.0184531,('PDT000539013.1':0.0172804,('PDT000468235.1':0.00919669,('PDT000533049.1':0.00614689,'PDT000668175.2':0.00953581)'':0.00484621)'':0.00521948)'':0.0112089)'':0.0343172)'':0.00937337)'':0.00269872)'':0.0308332)'':0.00828292,(((('PDT000305483.2':0.076264,(('PDT000645085.1':0.0183025,'PDT000502534.1':0.0165927)'':0.0110062,('PDT000644927.1':0.0307043,('PDT000565928.1':0.0223803,'PDT000566044.1':0.021824)'':0.00319834)'':0.000410516)'':0.0373235)'':-0.00480839,(('PDT000485572.1':0.0372052,'PDT000600747.1':0.0394804)'':0.0157274,(('PDT000480348.1':0.0303421,'PDT000496984.1':0.0251699)'':0.0249587,(('PDT000420677.1':0.0265602,('PDT000095797.2':0.0186246,'PDT000443349.1':0.0161374)'':0.00288543)'':0.0143286,('PDT000191811.2':0.0308922,('PDT000479854.1':0.027128,'PDT000395267.1':0.0280269)'':0.00517067)'':0.00852206)'':0.00765297)'':0.00597243)'':0.00400226)'':0.00574246,('PDT000249896.1':0.0863798,((('PDT000259579.2':0.0260111,'PDT000336917.1':0.0306643)'':0.0287168,((('PDT000364421.1':0.0369409,'PDT000632009.1':0.0387259)'':0.00806916,('PDT000644660.1':0.0292303,'PDT000395271.1':0.0347635)'':0.0216408)'':0.0128078,(('PDT000509503.1':0.0267842,'PDT000620267.1':0.0377066)'':0.0166659,(('PDT000352094.1':0.0405914,'PDT000645004.1':0.0400739)'':0.00406778,('PDT000565938.1':0.0392893,('PDT000497333.1':0.020867,'PDT000519140.1':0.0243948)'':0.0204771)'':0.000535714)'':0.010679)'':0.00960923)'':0.000449471)'':-0.00044958,(('PDT000235738.2':0.0651075,'PDT000045597.1':0.0736375)'':0.0066299,(('PDT000628870.1':0.0598445,('PDT000312793.2':0.0437296,'PDT000646901.1':0.0513577)'':0.0175813)'':0.00499672,(('PDT000337141.1':0.0262228,('PDT000399417.1':0.0223788,'PDT000468338.1':0.0192468)'':0.0233584)'':0.0190738,((('PDT000045604.1':0.0471199,'PDT000258339.2':0.0330331)'':0.00998825,('PDT000228023.1':0.044789,('PDT000248808.1':0.0366102,'PDT000395302.1':0.0417718)'':0.016887)'':0.00157758)'':0.00571229,(('PDT000485445.1':0.044289,('PDT000364424.1':0.0340155,'PDT000565867.1':0.0297171)'':0.00691673)'':0.0088983,(('PDT000069128.2':0.05266,'PDT000548115.1':0.052829)'':0.00610777,('PDT000600992.1':0.0643317,('PDT000311262.2':0.0352962,'PDT000095811.2':0.0330666)'':0.0176574)'':0.00151489)'':0.00254738)'':0.00304794)'':0.00390351)'':0.00152286)'':0.00348288)'':-0.00373705)'':0.0190878)'':-0.0174155)'':0.0377669,((('PDT000428852.1':0.0667663,(('PDT000009383.2':0.00580321,'PDT000009384.2':0.00904039)'':0.0510786,('PDT000009387.2':0.0241527,('PDT000009385.2':0.0282107,'PDT000009386.2':0.0332214)'':0.00601576)'':0.0245393)'':0.00250145)'':0.0325346,(('PDT000637745.1':0.0406681,('PDT000287702.2':0.00421913,'PDT000619768.1':0.00355599)'':0.0147985)'':0.0403839,(('PDT000535434.1':0.0489499,'PDT000638538.1':0.0380823)'':0.00706174,('PDT000394750.1':0.0312879,('PDT000579620.1':0.0267217,('PDT000415800.1':0.0151313,('PDT000564178.1':0.0116544,'PDT000585053.1':0.0135315)'':0.00380297)'':0.00338206)'':0.01183)'':0.0172427)'':0.0176708)'':0.0191777)'':-0.00150682,(((('PDT000174355.1':0.0199227,('PDT000083442.1':0.0137985,'PDT000174343.1':0.0136164)'':0.00747792)'':0.0272978,(('PDT000174330.1':0.0555543,'PDT000015762.2':0.0374622)'':0.00514767,(('PDT000237127.1':0.02649,('PDT000237128.1':0.00561655,'PDT000237129.1':0.00465575)'':0.0161162)'':0.00986716,('PDT000390577.1':0.0366573,(('PDT000083507.1':0.0183301,'PDT000083517.1':0.0172963)'':0.00769127,('PDT000083479.2':0.00790534,('PDT000083473.1':0.00597962,'PDT000083522.1':0.00745718)'':0.00345421)'':0.0178904)'':0.00740788)'':-0.000740256)'':0.00545079)'':0.0142608)'':0.0137287,((('PDT000081319.1':0.0229944,('PDT000083476.1':0.0246238,'PDT000083492.1':0.0229673)'':0.00819333)'':0.0112739,('PDT000083460.1':0.0220517,('PDT000083449.1':0.014952,'PDT000083478.1':0.0217107)'':0.00564143)'':0.0193454)'':0.0124701,('PDT000081290.1':0.09039,(('PDT000155893.2':0.0329575,'PDT000625487.1':0.0438981)'':0.0117362,(('PDT000619807.1':0.0303476,'PDT000619809.1':0.0245975)'':0.00940141,(('PDT000083450.1':0.0361531,('PDT000524027.1':0.0322109,'PDT000083438.1':0.0212566)'':0.00529036)'':0.000285236,('PDT000619802.1':0.032051,('PDT000151012.2':0.0281493,(('PDT000120902.2':0.0223778,'PDT000083580.1':0.0206403)'':0.00167478,('PDT000034408.1':0.0218422,'PDT000081325.1':0.0196111)'':0.00318522)'':0.00497282)'':0.00252662)'':-0.000253156)'':0.00406053)'':0.00725071)'':0.0151608)'':-0.00471358)'':0.00645037)'':0.0147053,((('PDT000081266.1':0.0644497,'PDT000198905.1':0.0461983)'':0.0178155,('PDT000081391.1':0.0683938,('PDT000081296.1':0.0200954,('PDT000081276.1':0.00984368,('PDT000081275.1':0.00870602,'PDT000081280.1':0.00982728)'':0.000656216)'':0.0104445)'':0.0140425)'':0.0419424)'':0.0086998,((('PDT000627438.1':0.0414567,'PDT000495365.1':0.0450578)'':0.00119887,(('PDT000246517.2':0.0305663,('PDT000203094.2':0.020317,'PDT000375005.1':0.0235022)'':0.00588663)'':0.00917985,(('PDT000395292.1':0.0381961,('PDT000417992.1':0.0338956,('PDT000200192.2':0.0322519,'PDT000313323.2':0.0245574)'':0.00512904)'':0.00505404)'':0.00133536,('PDT000566150.1':0.0322751,('PDT000367711.1':0.0243881,('PDT000497352.1':0.0227008,'PDT000551882.1':0.0188312)'':0.00349385)'':0.00927261)'':0.006057)'':0.000760481)'':0.00238694)'':0.0366009,('PDT000251435.1':0.0823041,('PDT000396791.1':0.0765924,((('PDT000162001.2':0.0622184,('PDT000103576.2':0.0161768,'PDT000123477.2':0.0255725)'':0.020534)'':0.00663207,('PDT000604203.1':0.076315,('PDT000161649.2':0.0253382,'PDT000083544.1':0.0224538)'':0.017004)'':0.00614692)'':0.00453355,('PDT000426536.1':0.0785895,(('PDT000248738.1':0.057743,('PDT000161973.2':0.0539694,'PDT000357726.1':0.0528186)'':0.0019855)'':0.00376754,(('PDT000319829.1':0.0529148,('PDT000016374.2':0.0353233,'PDT000251809.1':0.0341064)'':0.00549853)'':0.0101899,('PDT000619763.1':0.021316,('PDT000120905.2':0.0162412,'PDT000083439.1':0.0183333)'':0.00460668)'':0.0255546)'':0.006665)'':0.00406876)'':-0.000927203)'':0.00763959)'':0.00293194)'':-0.00123)'':0.00671222)'':-0.00230921)'':0.0121143)'':0.0125844)'':0.0137175)'':0.00358057,((('PDT000720208.1':0.0884456,'PDT000251416.1':0.0968624)'':0.00533158,(('PDT000719164.1':0.0688922,('PDT000518007.1':0.0573697,('PDT000294545.2':0.014972,'PDT000411087.1':0.0187137)'':0.0395669)'':0.0251697)'':0.0143305,((('PDT000294697.2':0.0633832,'PDT000718907.1':0.0524368)'':0.0184071,(('PDT000380313.1':0.0233212,'PDT000557274.1':0.0244798)'':0.0507496,(('PDT000439482.1':0.059582,'PDT000250238.1':0.074734)'':0.00859259,('PDT000248531.1':0.0696162,('PDT000718980.1':0.0510137,('PDT000482518.1':0.0387904,('PDT000305145.2':0.0204518,('PDT000294544.2':0.0248155,'PDT000305498.2':0.0275436)'':0.00510905)'':0.00706644)'':0.0224761)'':0.00628566)'':0.00354164)'':0.00986348)'':0.0029835)'':0.00550216,((('PDT000218520.2':0.068435,('PDT000396782.1':0.0334318,'PDT000557301.1':0.0286969)'':0.0282817)'':0.0112583,(('PDT000380272.1':0.0452203,('PDT000272555.2':0.0125409,'PDT000292791.2':0.0187681)'':0.0448239)'':0.0153942,('PDT000248779.1':0.073898,(('PDT000310432.2':0.00485072,'PDT000408756.1':0.00573548)'':0.0534656,('PDT000617793.1':0.0493586,'PDT000252523.1':0.0573384)'':0.00392809)'':0.00569834)'':0.00798686)'':0.00380953)'':0.000405696,(('PDT000718932.1':0.0441698,'PDT000228383.1':0.0508369)'':0.0319635,(('PDT000339375.1':0.0546268,'PDT000228124.1':0.0458152)'':0.0128044,((('PDT000430780.1':0.0370752,'PDT000499430.1':0.0435262)'':0.00698001,('PDT000332421.1':0.042649,'PDT000519811.1':0.0418904)'':0.0126337)'':0.0098981,('PDT000691341.1':0.0580373,('PDT000372000.2':0.0600376,('PDT000500488.1':0.0390245,'PDT000228050.1':0.0416426)'':0.00889333)'':0.00830923)'':-0.00300038)'':0.00240636)'':0.00701183)'':0.00571532)'':0.00107112)'':0.00750806)'':0.0153985)'':0.029994,((('PDT000719045.1':0.113143,('PDT000317702.1':0.101371,('PDT000501939.1':0.0825684,('PDT000258441.2':0.0602319,'PDT000251552.1':0.0674731)'':0.0114841)'':0.00937371)'':0.00213326)'':0.0141531,(('PDT000407048.1':0.113758,'PDT000667761.2':0.108017)'':0.0180907,(('PDT000424625.1':0.0724069,('PDT000532122.1':0.0688914,('PDT000047673.3':0.0549158,('PDT000251904.1':0.0457596,(('PDT000150255.2':0.035686,'PDT000248809.1':0.0351763)'':0.00773675,('PDT000178580.2':0.0333632,'PDT000645038.1':0.0388742)'':0.00508267)'':0.00249429)'':0.007955)'':0.00640655)'':0.0249099)'':-0.0092213,(('PDT000157765.3':0.00783378,'PDT000384323.2':0.00454142)'':0.0372413,((('PDT000352824.2':0.00923279,'PDT000101029.3':0.00749861)'':0.00760138,('PDT000627325.2':0.00540668,'PDT000679483.1':0.00564692)'':0.00510552)'':0.00997749,(('PDT000145336.3':0.0163077,'PDT000247550.3':0.0185726)'':0.0211806,(('PDT000285827.3':0.0226593,(('PDT000558718.2':0.00803178,'PDT000648987.2':0.00796262)'':0.00473843,('PDT000386270.2':0.0202004,('PDT000248829.1':0.0104825,('PDT000580036.2':0.00862887,('PDT000163575.3':0.0095325,('PDT000132551.1':0.0112699,('PDT000668593.1':0.00163834,'PDT000121057.3':0.00175623)'':0.0037102)'':0.00141237)'':0.00087016)'':0.00279144)'':0.000777779)'':-0.00039063)'':0.0056381)'':-0.00138111,('PDT000150667.3':0.0191457,('PDT000115041.3':0.0249838,('PDT000063312.3':0.0244484,('PDT000321941.2':0.0244572,('PDT000051234.3':0.0197423,(('PDT000120810.3':0.0127087,'PDT000136726.3':0.0105425)'':0.00426368,(('PDT000571490.2':0.009117,('PDT000523314.2':0.0045426,'PDT000532711.2':0.00414871)'':0.00176055)'':0.00466682,('PDT000203840.3':0.0167663,('PDT000120809.2':0.0113997,(('PDT000264897.3':0.0124014,'PDT000248643.1':0.0096355)'':0.000867022,(('PDT000499044.2':0.00504741,'PDT000650735.2':0.00563229)'':0.00573322,('PDT000306755.3':0.0100742,(('PDT000718967.1':0.0121779,('PDT000235042.3':0.00683086,('PDT000132547.1':0.00406756,('PDT000102874.3':0.00181806,'PDT000304215.1':0.00207804)'':0.000824237)'':0.00243173)'':0.00151658)'':-0.000619069,(('PDT000509951.2':0.00994962,('PDT000336885.2':0.00374565,('PDT000086313.3':0.00407502,('PDT000283234.3':0.00221266,'PDT000132548.1':0.00197979)'':0.00100863)'':0.00115263)'':0.00319049)'':-0.00150173,('PDT000143053.3':0.00531089,('PDT000328062.2':0.00359711,('PDT000132550.1':0.00356793,('PDT000313239.3':0.00208397,'PDT000556576.2':0.00260946)'':0.000624318)'':0.0013531)'':0.00214316)'':0.00123241)'':0.00145609)'':0.00281875)'':-0.00047578)'':0.000721535)'':0.00198187)'':0.000371315)'':-0.00226585)'':0.00127688)'':0.00268421)'':0.00101171)'':0.0013447)'':0.000299606)'':0.00992796)'':-0.0135423)'':0.00714562)'':-0.00027055)'':0.014909)'':0.0344469)'':0.0238961)'':0.000581567)'':0.0154141,(((('PDT000399436.1':0.0446233,'PDT000417495.1':0.0381273)'':0.0866431,((('PDT000238071.1':0.0490657,('PDT000251571.1':0.0376453,('PDT000171819.1':0.0104067,'PDT000171824.1':0.00941493)'':0.026293)'':0.0281563)'':0.054775,(('PDT000015873.3':0.0253528,'PDT000015783.3':0.0180348)'':0.0884973,('PDT000679707.1':0.0805534,('PDT000502908.1':0.061579,(('PDT000445112.1':0.0574232,('PDT000502633.1':0.0402555,'PDT000346512.1':0.0367569)'':0.0061277)'':0.00274016,('PDT000150306.2':0.0441591,('PDT000627437.1':0.0483889,('PDT000333679.1':0.0214921,('PDT000147454.3':0.0113405,'PDT000565922.1':0.0081109)'':0.011683)'':0.021154)'':0.00187587)'':0.00653596)'':0.00541309)'':0.0185002)'':0.0141531)'':-0.00151148)'':-0.00312656,(('PDT000395985.1':0.0812873,('PDT000479858.1':0.0469041,('PDT000502260.1':0.0173083,'PDT000399950.1':0.0167401)'':0.0371404)'':0.0119009)'':0.0216867,(('PDT000648085.1':0.106238,('PDT000502503.1':0.0294509,('PDT000502505.1':0.0185495,'PDT000667709.2':0.0175441)'':0.00497514)'':0.0671333)'':-0.00623059,(('PDT000251840.1':0.0717114,('PDT000251573.1':0.0478729,('PDT000399216.1':0.0351854,'PDT000251984.1':0.0247143)'':0.0155584)'':0.0108994)'':0.00282007,(('PDT000259517.2':0.0494494,'PDT000417482.1':0.0613076)'':0.0151554,(('PDT000584653.1':0.0581721,('PDT000632011.1':0.0365348,'PDT000045599.1':0.0427543)'':0.0151594)'':0.00480796,(('PDT000293472.1':0.0517829,('PDT000251438.1':0.0514055,('PDT000150273.2':0.0474493,('PDT000228074.1':0.0365568,'PDT000294404.1':0.0405008)'':0.00295047)'':0.00340425)'':0.00668053)'':-0.000950766,('PDT000502919.1':0.0537356,('PDT000045598.1':0.0412948,('PDT000294785.1':0.0346735,('PDT000095810.2':0.024777,'PDT000399974.1':0.0215175)'':0.0123928)'':0.00776326)'':0.0193933)'':0.00620287)'':0.00165317)'':0.0118391)'':-0.000286356)'':0.0114589)'':0.010006)'':-0.00334061)'':0.0304534)'':-0.0234429,(((('PDT000719926.1':0.0627145,'PDT000124701.1':0.0551565)'':0.0349598,(('PDT000115409.3':0.0350739,'PDT000716353.1':0.0228385)'':0.0403993,(('PDT000193072.3':0.0521231,'PDT000634791.2':0.0400851)'':0.0264412,(('PDT000333275.2':0.0336781,('PDT000235717.3':0.0313942,'PDT000584139.2':0.0341907)'':0.00362946)'':0.00788238,(('PDT000161716.3':0.0258146,'PDT000324665.2':0.0241366)'':0.00450301,('PDT000327182.2':0.00563691,'PDT000362506.1':0.00590879)'':0.0196516)'':0.00654033)'':0.0221524)'':0.00394793)'':0.0297279)'':-0.0126637,('PDT000047681.4':0.0632037,(('PDT000548255.2':0.0310613,('PDT000205098.3':0.0297295,('PDT000398044.2':0.021077,('PDT000332846.2':0.0163646,'PDT000548181.2':0.0178226)'':0.00597423)'':0.00431382)'':0.00215785)'':-0.000428813,(('PDT000716339.1':0.0167625,('PDT000524132.2':0.00565322,'PDT000554025.2':0.00697978)'':0.00402659)'':0.00326071,('PDT000325208.2':0.0192877,('PDT000517236.2':0.016959,('PDT000523434.2':0.00770992,'PDT000588065.2':0.00582508)'':0.00508712)'':0.000459223)'':0.00454459)'':0.00776726)'':0.0329249)'':0.0158602)'':0.0294152,((('PDT000097362.2':0.107585,('PDT000343725.1':0.0691883,('PDT000188186.2':0.0498059,('PDT000502525.1':0.0379689,'PDT000150868.1':0.0403962)'':0.0108928)'':0.0185528)'':0.0237542)'':-0.00285637,('PDT000228098.1':0.0676788,('PDT000109307.2':0.0666485,('PDT000501825.1':0.0468716,('PDT000395110.1':0.0281229,('PDT000172355.2':0.0195234,'PDT000345996.1':0.0200213)'':0.00916292)'':0.00690425)'':0.0195718)'':0.0047213)'':0.039659)'':0.00373087,(('PDT000557242.1':0.0672338,('PDT000502326.1':0.0807792,('PDT000502531.1':0.0305002,('PDT000095706.2':0.0235208,'PDT000095712.2':0.0180017)'':0.00770863)'':0.0223182)'':0.0158931)'':0.00901153,((('PDT000140478.2':0.0647139,'PDT000540525.1':0.0688441)'':0.016498,('PDT000537190.1':0.0690765,'PDT000488266.1':0.0570295)'':0.0094118)'':0.0012824,((('PDT000632013.1':0.0294399,'PDT000502698.1':0.0296581)'':0.0263374,('PDT000651994.1':0.041746,('PDT000258398.2':0.00434927,'PDT000639508.1':0.00812703)'':0.0466719)'':0.0161874)'':0.000985019,(('PDT000710415.1':0.0567936,(('PDT000262161.2':0.0210783,'PDT000045625.1':0.0221909)'':0.01364,('PDT000332076.1':0.0283364,('PDT000216079.2':0.0249187,'PDT000241662.2':0.0198736)'':0.015717)'':0.00740256)'':0.0131147)'':0.00614126,('PDT000426371.1':0.0525276,('PDT000720198.1':0.0602473,('PDT000720257.1':0.0423947,('PDT000428348.1':0.0211075,('PDT000479557.1':0.0186743,'PDT000252491.1':0.0220675)'':0.00493601)'':0.00582655)'':0.011643)'':0.00535176)'':-0.0031369)'':0.00749602)'':0.0107149)'':0.000503866)'':0.0165565)'':0.00350868)'':0.00165177)'':0.0161659,((('PDT000502392.1':0.121215,(('PDT000485551.1':0.0637016,('PDT000015393.2':0.043065,('PDT000199186.2':0.038237,'PDT000539284.1':0.0345978)'':0.00741951)'':0.0182402)'':0.00771269,(('PDT000291256.1':0.0521085,'PDT000482833.1':0.0721305)'':0.000135972,(('PDT000346538.1':0.0609166,('PDT000161707.2':0.0312613,'PDT000625471.1':0.0395317)'':0.0103187)'':0.0133838,('PDT000242880.1':0.0491087,('PDT000325169.1':0.0455604,('PDT000660655.1':0.045892,('PDT000313317.2':0.0341821,'PDT000557359.1':0.0342031)'':0.015254)'':0.000158112)'':0.00302646)'':0.00666259)'':0.00383914)'':0.00914935)'':0.0401754)'':-0.0327041,((('PDT000677507.1':0.0739963,('PDT000326121.2':0.046635,'PDT000406786.2':0.0461308)'':0.0097453)'':0.00901637,(('PDT000103587.2':0.0220448,('PDT000285768.2':0.0147967,'PDT000575842.1':0.012396)'':0.0118469)'':0.00133309,('PDT000551356.2':0.0159316,('PDT000161578.3':0.0107153,'PDT000464168.2':0.0132148)'':0.0041405)'':0.014909)'':0.0311942)'':0.00723243,(('PDT000502590.1':0.0770662,('PDT000486062.2':0.0544436,('PDT000502456.1':0.0291926,'PDT000502589.1':0.0226132)'':0.0141812)'':0.0156086)'':-0.00539292,((('PDT000062529.3':0.0197666,'PDT000637100.2':0.0191256)'':0.00577184,('PDT000409249.2':0.0315594,('PDT000231288.3':0.0126522,'PDT000281526.3':0.0133038)'':0.010297)'':0.000329663)'':0.00467095,(('PDT000540054.2':0.0543423,('PDT000407571.2':0.0253461,'PDT000106996.3':0.0209393)'':0.0159464)'':-0.00238442,('PDT000385950.2':0.0362545,('PDT000146551.3':0.0238366,('PDT000677271.1':0.022601,('PDT000306763.3':0.0159435,('PDT000162976.4':0.0102843,('PDT000558649.2':0.00482644,'PDT000502447.1':0.00563906)'':0.0036845)'':0.00721079)'':0.00308206)'':0.000864999)'':0.00899043)'':-0.00135742)'':0.00300413)'':0.0262216)'':0.00806188)'':0.0145642)'':0.0335902,((('PDT000615197.2':0.102364,('PDT000322073.2':0.0841783,('PDT000242612.3':0.056598,'PDT000251668.1':0.0356526)'':0.0471249)'':0.00579151)'':0.00271007,(('PDT000346064.1':0.0576312,('PDT000340987.1':0.0431618,'PDT000045627.1':0.0629192)'':0.0213463)'':0.0435146,('PDT000251767.1':0.0846909,('PDT000418653.1':0.0904948,('PDT000716405.1':0.061629,('PDT000564182.1':0.0581921,('PDT000632012.1':0.047802,(('PDT000095701.2':0.0384131,'PDT000502240.1':0.0412935)'':0.00474218,('PDT000017858.2':0.0229108,('PDT000303274.2':0.0138592,('PDT000394870.1':0.0167851,'PDT000645048.1':0.015578)'':0.00408591)'':0.00730962)'':0.0140054)'':0.00738872)'':0.00648353)'':0.0135863)'':0.00491001)'':0.00422807)'':0.00829293)'':0.00190534)'':0.00983316,((('PDT000248690.1':0.0425433,('PDT000390182.1':0.0318455,'PDT000261041.1':0.0191396)'':0.0202483)'':0.00886241,('PDT000625428.1':0.0605376,('PDT000254353.2':0.0529893,(('PDT000250127.1':0.0282463,('PDT000389391.1':0.0265726,'PDT000444992.1':0.017777)'':0.00848566)'':0.00505703,('PDT000486034.1':0.0384883,('PDT000491596.1':0.0425364,(('PDT000484199.1':0.004439,'PDT000011804.2':0.0056348)'':0.0238987,(('PDT000150983.2':0.0118507,'PDT000309679.2':0.00960419)'':0.0245257,('PDT000410859.1':0.0258096,('PDT000157793.2':0.0154606,'PDT000309612.2':0.0110249)'':0.0116478)'':0.000967506)'':0.00409702)'':0.0082109)'':0.00227959)'':0.000291118)'':0.0040717)'':0.00588425)'':-0.00470513)'':0.058041,((('PDT000156552.2':0.0854323,'PDT000253419.2':0.0971767)'':0.00778883,(('PDT000120939.2':0.0404556,(('PDT000171798.1':0.025893,('PDT000553182.1':0.0306126,('PDT000303490.2':0.0195492,('PDT000351877.1':0.00923435,'PDT000131121.2':0.00978685)'':0.00684807)'':0.0102429)'':0.00228847)'':0.00125821,('PDT000627347.1':0.0386809,('PDT000628000.1':0.0333482,('PDT000566519.1':0.0210127,('PDT000126256.2':0.0160402,'PDT000553203.1':0.0218905)'':0.0028839)'':0.00315216)'':0.0039873)'':0.00133174)'':0.0051256)'':0.00578142,('PDT000526120.1':0.0402611,(('PDT000600046.1':0.0391362,('PDT000553190.1':0.02317,('PDT000553188.1':0.0186569,('PDT000553139.1':0.0143819,'PDT000252676.1':0.018842)'':0.00272303)'':0.00539844)'':0.0109713)'':0.0037277,(('PDT000161593.3':0.0247039,'PDT000579457.2':0.0230927)'':0.013436,('PDT000553137.1':0.0529022,('PDT000398009.1':0.0277479,('PDT000439507.1':0.0201479,'PDT000553138.1':0.0244658)'':0.00406495)'':0.00659517)'':0.00015941)'':0.00428588)'':0.00256224)'':-0.00174656)'':0.0531044)'':-0.000913291,('PDT000709947.1':0.255641,(('PDT000494407.1':0.0414259,'PDT000610182.1':0.0450261)'':0.0620232,((('PDT000069897.3':0.0074483,'PDT000069953.3':0.0074637)'':0.100202,('PDT000557272.1':0.0838172,('PDT000425730.1':0.0710254,(('PDT000510070.1':0.0349606,('PDT000109448.2':0.0445235,('PDT000048143.1':0.02088,'PDT000251991.1':0.0214315)'':0.0148623)'':0.00574042)'':0.0194566,(('PDT000228091.1':0.0648002,'PDT000196653.2':0.0572628)'':0.0023521,(('PDT000252663.1':0.0453195,('PDT000095779.2':0.00972316,'PDT000228370.1':0.0109801)'':0.0336213)'':0.00955698,('PDT000417506.1':0.0317505,('PDT000428458.1':0.0265236,('PDT000246519.2':0.0294969,'PDT000266392.2':0.0254363)'':0.00234393)'':0.0164056)'':0.00845871)'':0.00695368)'':0.00600365)'':0.00596834)'':0.0171386)'':0.00788834)'':-0.00528783,(('PDT000502407.1':0.0914756,'PDT000346497.1':0.0638954)'':0.0238949,(('PDT000568557.1':0.0411117,('PDT000008638.2':0.0178398,('PDT000502463.1':0.0135986,'PDT000502565.1':0.0138257)'':0.00138694)'':0.0294176)'':0.0363954,('PDT000249921.1':0.0734076,(('PDT000326035.1':0.0495541,'PDT000150860.1':0.0545089)'':0.0171423,(('PDT000439459.1':0.0468612,'PDT000502660.1':0.0557448)'':0.0234875,(('PDT000233841.1':0.065111,('PDT000178610.2':0.039665,'PDT000502885.1':0.0424951)'':0.0127895)'':-0.000549504,('PDT000661408.1':0.0485644,('PDT000394819.1':0.0393241,('PDT000150259.2':0.0216712,'PDT000660645.1':0.0297371)'':0.00758738)'':0.0096906)'':0.0212027)'':0.00665921)'':0.00482425)'':0.00941992)'':-0.00265188)'':0.0144672)'':-0.00958759)'':0.0190826)'':0.0193153)'':-0.0281043)'':0.00382182)'':0.00121764)'':0.00695959)'':0.00433523)'':0.00125667)'':0.00529803)'':-0.00470403)'':0.016176)'':0.0109984)'':0.0167178)'':0.00030536,((((('PDT000691302.1':0.153386,(('PDT000625881.1':0.0789275,('PDT000557025.1':0.0704722,('PDT000501943.1':0.0369523,'PDT000252679.1':0.0441655)'':0.0121429)'':0.0183726)'':0.0104996,(('PDT000467005.1':0.0496058,'PDT000557147.1':0.0513112)'':0.0452436,('PDT000305146.2':0.077586,('PDT000088296.2':0.0390417,'PDT000584550.1':0.0416112)'':0.020729)'':0.0159046)'':0.00901611)'':0.046405)'':-0.00849872,(('PDT000488283.1':0.0590463,('PDT000017913.2':0.0518059,('PDT000477476.1':0.0385248,'PDT000228218.1':0.0368812)'':0.00375875)'':0.00559723)'':0.012052,((('PDT000305522.2':0.0444678,'PDT000364395.1':0.0330425)'':0.0087238,(('PDT000501516.1':0.0236936,('PDT000518564.1':0.0235017,'PDT000659707.1':0.0254911)'':0.00326839)'':0.0101272,(('PDT000417958.1':0.0291365,'PDT000477124.1':0.0330236)'':0.00996323,(('PDT000311261.2':0.0205474,('PDT000479099.1':0.025199,('PDT000538731.1':0.00998559,'PDT000577081.1':0.0128727)'':0.0100721)'':0.00267864)'':0.00172284,('PDT000428364.1':0.0267501,('PDT000382203.1':0.0194455,('PDT000178584.2':0.0155519,'PDT000430842.1':0.0234555)'':0.00123046)'':0.00575061)'':0.00119439)'':0.00553005)'':0.00454347)'':0.00667795)'':0.00726071,('PDT000228024.1':0.0687814,((('PDT000281752.2':0.0265588,'PDT000045592.1':0.0322878)'':0.0140017,('PDT000303129.2':0.0350646,'PDT000095713.2':0.0322637)'':0.00990973)'':0.00249084,(('PDT000417373.1':0.0295085,(('PDT000479554.1':0.00912689,'PDT000501513.1':0.0122661)'':0.00538797,('PDT000644955.1':0.0156105,('PDT000563576.1':0.0160869,('PDT000288618.2':0.00584227,'PDT000509507.1':0.00859563)'':0.00323276)'':0.00289136)'':0.000315472)'':0.0142758)'':0.0171785,(('PDT000282205.2':0.0146208,'PDT000563556.1':0.0173737)'':0.0219534,('PDT000500489.1':0.0349871,(('PDT000259595.2':0.0217928,'PDT000502331.1':0.0291181)'':0.00396626,('PDT000455224.1':0.0243783,('PDT000312787.2':0.0212563,'PDT000467087.1':0.0278596)'':0.00129949)'':0.00774949)'':0.00139266)'':0.00731505)'':-0.000180968)'':0.00599027)'':0.020482)'':-0.0151649)'':0.0252796)'':0.0501435)'':0.033371,(('PDT000249925.1':0.121508,('PDT000494547.1':0.0795957,('PDT000145182.2':0.0330787,('PDT000095749.2':0.0234395,'PDT000551913.1':0.0290016)'':0.00228504)'':0.047251)'':0.0578419)'':0.00519573,((((('PDT000014152.2':0.0234786,'PDT000428883.1':0.0178912)'':0.00481443,('PDT000429664.1':0.0120853,('PDT000429665.1':0.0126869,('PDT000429666.1':0.0102035,('PDT000428877.1':0.00460816,'PDT000429660.1':0.00416902)'':0.00794267)'':0.0021275)'':0.00312534)'':0.00219567)'':0.031306,(('PDT000015781.3':0.0513242,('PDT000422533.1':0.0204985,('PDT000428859.1':0.0163998,'PDT000428905.1':0.0162207)'':0.0124934)'':0.016229)'':0.00494475,((('PDT000428879.1':0.0321243,'PDT000428884.1':0.0307883)'':0.00989616,('PDT000428892.1':0.0193396,'PDT000429661.1':0.0281473)'':0.00907682)'':0.0167335,('PDT000261037.1':0.0393509,(('PDT000429708.1':0.0292756,('PDT000429685.1':0.0107818,'PDT000429757.1':0.0182196)'':0.0219863)'':0.00433261,('PDT000248750.1':0.0238302,('PDT000390189.1':0.0137027,'PDT000261027.1':0.0106827)'':0.0196775)'':0.00634703)'':0.00979235)'':-0.00159278)'':0.00241776)'':0.0156314)'':0.0422253,(('PDT000663914.1':0.0475928,('PDT000484689.1':0.0399517,('PDT000484712.1':0.0299239,('PDT000484693.1':0.0205375,('PDT000484691.1':0.0090195,'PDT000484702.1':0.0099741)'':0.00490418)'':0.012151)'':0.011963)'':0.00223397)'':0.0239189,(('PDT000251972.1':0.0506442,(('PDT000484700.1':0.0102435,'PDT000484701.1':0.0115324)'':0.02995,('PDT000014148.3':0.0340513,('PDT000250173.1':0.0209706,('PDT000010918.2':0.00328449,'PDT000248598.1':0.00191499)'':0.0185412)'':0.00897057)'':0.00277595)'':0.0123154)'':-0.00296526,((('PDT000484671.1':0.0224094,('PDT000484664.1':0.0091311,'PDT000484698.1':0.0104475)'':0.00529546)'':0.0169821,(('PDT000484667.1':0.0229463,('PDT000316585.1':0.0200397,'PDT000484673.1':0.0231265)'':0.00406757)'':0.00222722,('PDT000484668.1':0.0297494,('PDT000484678.1':0.0189439,('PDT000261031.1':0.0156852,'PDT000261032.1':0.00882167)'':0.00430221)'':0.00993382)'':-0.00161004)'':0.00116029)'':0.00273666,(('PDT000015717.3':0.0163852,('PDT000015855.3':0.016141,'PDT000015860.3':0.0138807)'':0.00156711)'':0.00844476,(('PDT000484704.1':0.0328847,('PDT000015832.3':0.0194017,'PDT000014131.3':0.0200487)'':0.0114133)'':-0.00177631,('PDT000015713.3':0.0221659,('PDT000240801.1':0.0122603,('PDT000015822.2':0.0129566,(('PDT000015814.3':0.0102998,'PDT000015831.3':0.0109135)'':0.000316284,(('PDT000015838.3':0.00908674,'PDT000015857.2':0.00758986)'':0.00078848,('PDT000014126.3':0.0105636,('PDT000015847.2':0.00819144,'PDT000015833.2':0.00798076)'':0.00221049)'':0.000278823)'':0.000713852)'':0.00156303)'':0.00328181)'':0.00602222)'':0.000781163)'':0.00322059)'':0.00634888)'':0.0152361)'':0.0152882)'':0.0379323)'':0.00860899,(('PDT000125553.3':0.0945157,'PDT000718990.1':0.0918223)'':0.0465638,((('PDT000196626.2':0.04043,'PDT000566587.1':0.0300091)'':0.0351956,('PDT000599852.1':0.0700731,('PDT000216274.2':0.0635388,'PDT000652273.1':0.0527472)'':0.0132244)'':0.00464356)'':0.00602412,(('PDT000288587.3':0.0651226,(('PDT000317811.2':0.018863,'PDT000123279.1':0.0195188)'':0.0322865,(('PDT000679740.1':0.0301247,('PDT000644218.2':0.0209936,('PDT000162979.4':0.0124208,('PDT000200953.3':0.0109868,'PDT000616515.2':0.0121769)'':0.00115187)'':0.00409128)'':0.00452549)'':0.00659804,('PDT000038428.1':0.0341559,('PDT000641407.2':0.0255408,('PDT000481031.2':0.02056,('PDT000617237.2':0.0170654,(('PDT000266562.3':0.0110728,'PDT000682154.1':0.0092746)'':0.00600592,('PDT000197888.3':0.00894163,(('PDT000197116.3':0.0011234,'PDT000295581.3':0.00137435)'':0.00277828,('PDT000492130.2':0.00585644,'PDT000707596.1':0.00499586)'':0.000779979)'':0.0027901)'':0.00360908)'':0.00274464)'':0.00176441)'':0.00421772)'':0.00722203)'':-0.00756202)'':0.0210806)'':0.0188573)'':-0.003647,(('PDT000102896.3':0.0758816,('PDT000081770.1':0.0298456,('PDT000548298.2':0.0336624,'PDT000047679.5':0.0283554)'':0.0127973)'':0.0108542)'':0.0152126,(('PDT000143271.3':0.0243341,('PDT000432450.2':0.0234458,'PDT000115006.3':0.0288033)'':0.00389239)'':0.000622734,((('PDT000722076.1':0.016814,('PDT000386808.2':0.0126442,('PDT000486133.2':0.0137462,'PDT000533755.2':0.0119964)'':0.00204022)'':0.00236864)'':0.00298125,(('PDT000443417.2':0.00540973,'PDT000616902.2':0.00376916)'':0.0077189,('PDT000112498.3':0.0114348,('PDT000485845.2':0.00533344,'PDT000490322.2':0.00443627)'':0.00562908)'':0.0039965)'':0.00297873)'':0.00988909,(('PDT000443253.2':0.0490899,('PDT000342309.2':0.0214951,'PDT000130434.3':0.0175005)'':0.00976529)'':0.00718175,(('PDT000708637.1':0.0302588,(('PDT000144013.3':0.0156898,'PDT000375685.2':0.0226267)'':0.00267946,('PDT000352656.2':0.0209555,('PDT000238324.3':0.0178063,('PDT000150924.3':0.0103639,'PDT000394748.2':0.0118275)'':0.0059575)'':0.00265506)'':0.00103969)'':0.00996174)'':-0.0064249,('PDT000300546.1':0.0284807,(('PDT000583069.2':0.0188177,('PDT000140555.3':0.0196897,('PDT000359220.2':0.0198834,('PDT000490343.2':0.0115379,'PDT000300543.1':0.011104)'':0.00382339)'':0.0024499)'':0.00113269)'':0.000173537,('PDT000407516.2':0.0181987,('PDT000495250.2':0.0250157,('PDT000298471.3':0.00925376,('PDT000494174.2':0.0099115,('PDT000500985.2':0.00812527,'PDT000539902.2':0.00894163)'':0.000788403)'':0.0024096)'':0.00326862)'':0.00255599)'':0.000385854)'':0.00297983)'':-0.000146774)'':0.00607511)'':-0.00258535)'':0.00422488)'':0.0369756)'':-0.00815659)'':0.0242031)'':0.0588431)'':-0.043279)'':0.0430618)'':0.0138965)'':0.00406262,(((('PDT000042538.1':0.028876,'PDT000042571.1':0.0314445)'':0.0500553,('PDT000346967.1':0.0975737,('PDT000417128.1':0.0585145,'PDT000294383.1':0.0493555)'':0.0579168)'':0.0095292)'':0.0646823,(((('PDT000032206.1':0.0119086,'PDT000034429.1':0.0135925)'':0.086528,('PDT000502263.1':0.0721669,(('PDT000630060.1':0.0279999,'PDT000242522.1':0.0453381)'':0.0114724,(('PDT000441059.2':0.0208603,'PDT000593801.2':0.0300171)'':0.00668009,('PDT000428393.1':0.0402801,('PDT000242514.1':0.00625826,('PDT000242518.1':0.00655902,('PDT000441063.2':0.00404478,'PDT000510668.2':0.00374276)'':0.00161446)'':0.00450555)'':0.0231304)'':0.000950832)'':0.00719649)'':0.0327478)'':0.00568945)'':0.0413557,(('PDT000712997.1':0.0260963,'PDT000228212.1':0.0214152)'':0.060131,(('PDT000135956.2':0.0756709,('PDT000361010.1':0.0464952,('PDT000042660.1':0.00450694,'PDT000042661.1':0.00376411)'':0.037212)'':0.0420921)'':-0.00684611,(('PDT000610215.1':0.0643557,('PDT000132332.2':0.0206365,('PDT000548995.1':0.0184602,('PDT000579613.1':0.0202717,'PDT000323274.1':0.0142008)'':0.00430022)'':0.00422462)'':0.0384361)'':-0.0119841,('PDT000159987.2':0.0421783,(('PDT000188345.1':0.00281746,('PDT000188336.1':0.00147869,'PDT000188356.1':0.00111845)'':0.000874993)'':0.0367889,('PDT000465827.1':0.0297551,('PDT000407914.1':0.0221555,(('PDT000579610.1':0.0189093,'PDT000407916.1':0.0182409)'':0.0024252,('PDT000316524.1':0.0159121,'PDT000631691.1':0.0180349)'':0.00626668)'':0.00433306)'':0.00770911)'':0.00316926)'':0.0160584)'':-0.0017261)'':0.0241124)'':0.0267836)'':0.0336479)'':0.0183483,('PDT000638920.1':0.135703,((('PDT000714275.1':0.0926308,('PDT000032328.1':0.0870712,'PDT000251499.1':0.0757148)'':0.00652817)'':0.00597513,('PDT000709946.1':0.11234,('PDT000115353.3':0.0833671,(('PDT000557219.1':0.0510608,'PDT000589388.1':0.0427223)'':0.0115608,('PDT000688003.1':0.0607977,('PDT000277724.2':0.0317688,('PDT000323534.1':0.0151659,'PDT000372317.1':0.0225064)'':0.00172993)'':0.0332633)'':0.00622298)'':0.0115332)'':0.0237336)'':0.0104578)'':0.0210166,('PDT000687809.1':0.148395,(('PDT000395502.1':0.0991722,('PDT000047691.3':0.0746758,('PDT000557329.1':0.0742735,('PDT000290381.2':0.0309166,'PDT000706403.1':0.033349)'':0.0414862)'':0.00359808)'':0.0183539)'':-0.00632345,('PDT000327200.2':0.0910244,((('PDT000497542.2':0.0275366,'PDT000669554.1':0.0189085)'':0.00517593,(('PDT000548633.2':0.0130657,'PDT000622735.2':0.0139853)'':0.00408396,('PDT000163024.4':0.0185896,('PDT000421067.2':0.00998506,'PDT000490397.2':0.00945364)'':0.00376063)'':0.00818206)'':0.00221698)'':0.0225838,(('PDT000548496.2':0.0325969,('PDT000258506.3':0.0285623,'PDT000163386.1':0.0240434)'':0.00736947)'':0.00248972,((('PDT000264770.3':0.0119421,'PDT000566476.2':0.0144966)'':0.0160733,('PDT000242507.1':0.035275,('PDT000140498.3':0.030265,('PDT000140504.3':0.0286157,'PDT000502277.1':0.0295679)'':0.00261535)'':0.00190103)'':0.000340028)'':0.00589422,(('PDT000251882.1':0.0205025,('PDT000497974.1':0.0127521,'PDT000497998.1':0.0122928)'':0.00985032)'':0.00531615,(('PDT000548497.2':0.0191612,('PDT000497901.1':0.0175636,('PDT000497892.1':0.0117228,('PDT000140449.3':0.0116374,'PDT000065376.1':0.0111259)'':0.003135)'':0.00526319)'':0.00125955)'':0.00467226,('PDT000259540.3':0.0240327,(('PDT000203363.3':0.0163548,('PDT000501949.1':0.00877854,('PDT000502573.1':0.00790319,('PDT000502455.1':0.00402282,'PDT000502475.1':0.00745768)'':0.00741761)'':0.00158413)'':0.00566479)'':0.00372984,('PDT000242494.1':0.0184524,(('PDT000306762.3':0.0130203,('PDT000277157.3':0.0120253,'PDT000486495.1':0.0105646)'':0.00451771)'':-9.82116e-05,('PDT000396587.2':0.0182411,(('PDT000216394.3':0.0116484,'PDT000502469.1':0.0108179)'':0.00357185,('PDT000065361.1':0.0107679,('PDT000190670.3':0.010249,'PDT000264751.3':0.0106835)'':0.00138617)'':0.00194301)'':0.00237317)'':-0.000643859)'':0.00438911)'':-0.00175645)'':0.00320456)'':-0.0013603)'':0.00433605)'':0.00175925)'':0.0160213)'':-0.00672609)'':0.0296132)'':0.024314)'':0.0317875)'':-0.00364104)'':0.0181673)'':0.00507551)'':0.0166969)'':-0.00815314,((('PDT000551367.2':0.073897,('PDT000306269.1':0.0378373,(('PDT000486049.2':0.0430518,'PDT000669455.1':0.0360563)'':0.00290732,(('PDT000140529.3':0.0605219,('PDT000146111.3':0.0276279,'PDT000102762.3':0.0203737)'':0.00787596)'':0.00028741,(('PDT000250763.3':0.0335657,'PDT000157506.3':0.0297699)'':0.00344285,('PDT000372753.2':0.0353349,('PDT000140210.3':0.027114,'PDT000140226.3':0.0265583)'':0.00626863)'':0.00978432)'':0.00207441)'':0.00336701)'':0.0176188)'':0.0130773)'':-0.0226689,((('PDT000136897.3':0.0218749,('PDT000309603.3':0.0186725,('PDT000609735.2':0.0214979,'PDT000712832.1':0.0120116)'':0.00636305)'':0.0100581)'':0.00684246,(('PDT000352816.2':0.0189853,'PDT000626859.2':0.0192833)'':0.00735026,('PDT000115350.3':0.0260139,(('PDT000146540.3':0.0133497,'PDT000618234.2':0.0187709)'':0.00192706,('PDT000407544.2':0.0138572,'PDT000707419.1':0.0157724)'':0.00248819)'':0.00365398)'':0.00344465)'':0.0104425)'':0.0023213,((('PDT000163582.3':0.0273564,'PDT000627097.2':0.0230281)'':0.0134551,('PDT000563742.2':0.0277497,('PDT000216121.3':0.0194286,'PDT000082111.3':0.0211986)'':0.00183882)'':0.00609783)'':0.000412709,('PDT000295986.3':0.0259245,(('PDT000140519.3':0.0269853,('PDT000707721.1':0.0208778,'PDT000140477.3':0.017781)'':0.0092713)'':0.00239949,('PDT000399377.2':0.0386795,(('PDT000177065.3':0.00772942,'PDT000242772.3':0.00727158)'':0.00814008,('PDT000465605.2':0.0193844,('PDT000234867.3':0.00645062,'PDT000501468.2':0.00452628)'':0.00888171)'':0.00327345)'':0.00733204)'':-0.00129654)'':0.00516086)'':8.53754e-05)'':0.000470142)'':0.0154239)'':0.0790802,(((('PDT000196615.2':0.107271,(('PDT000128844.2':0.0769402,(('PDT000683164.1':0.0250769,('PDT000411318.1':0.0366738,('PDT000131902.2':0.0215555,'PDT000502497.1':0.0266325)'':0.00147354)'':0.00404304)'':0.000377402,('PDT000661379.1':0.0436542,('PDT000549145.1':0.0267592,('PDT000426164.1':0.0177702,'PDT000430804.1':0.0200231)'':0.00716086)'':0.010462)'':0.000970429)'':0.0278859)'':-0.000972425,(('PDT000445120.1':0.0345807,('PDT000720199.1':0.018096,('PDT000716912.1':0.00724846,'PDT000543149.1':0.0121747)'':0.0045922)'':0.0107465)'':0.0241707,('PDT000616473.1':0.0622341,(('PDT000433767.1':0.0358112,('PDT000444872.1':0.0203604,'PDT000501834.1':0.0124624)'':0.0112772)'':0.0057134,(('PDT000206253.3':0.039623,'PDT000661616.1':0.0443789)'':0.00494131,('PDT000019243.2':0.0313996,('PDT000128806.2':0.0232239,'PDT000427219.1':0.0302022)'':0.00708115)'':0.00432686)'':0.00313702)'':0.00712349)'':0.00510459)'':0.00204623)'':0.0266509)'':0.00735313,(((('PDT000258496.3':0.0153152,'PDT000603567.2':0.0179589)'':0.0210005,('PDT000486524.1':0.054931,('PDT000469875.2':0.0381245,('PDT000119929.3':0.024359,('PDT000486523.1':0.0320848,('PDT000148108.3':0.0154377,'PDT000140480.3':0.0204359)'':0.00651324)'':0.00143224)'':0.00430183)'':0.00481843)'':0.000509025)'':0.0550487,(('PDT000468877.1':0.0950508,('PDT000661093.1':0.0590334,('PDT000482214.1':0.0512212,(('PDT000437892.1':0.0207769,'PDT000448173.1':0.0207992)'':0.00478346,('PDT000438286.1':0.0200869,'PDT000618938.1':0.0170411)'':0.00487459)'':0.0179083)'':0.0206185)'':0.0172125)'':-0.00746407,(('PDT000496908.1':0.0628967,('PDT000292829.2':0.0133224,'PDT000504796.1':0.0134563)'':0.041089)'':0.0268398,('PDT000672653.1':0.0819538,(('PDT000537949.1':0.0368108,'PDT000538608.1':0.0351776)'':0.0320767,((('PDT000167696.2':0.0667143,'PDT000438175.1':0.0532507)'':0.0105437,('PDT000151507.2':0.0563015,('PDT000466059.1':0.0496856,'PDT000488260.1':0.0462991)'':0.00478069)'':0.00603874)'':0.00232178,('PDT000502040.1':0.0503769,(('PDT000498774.1':0.0356812,'PDT000557297.1':0.0338131)'':0.00531536,('PDT000600635.1':0.0474822,('PDT000125010.2':0.0435558,'PDT000689899.1':0.0375567)'':0.00484582)'':0.000325414)'':0.00990888)'':-0.0023757)'':0.00660558)'':0.00881186)'':-0.00223872)'':0.00229384)'':0.00536598)'':0.00287672,((('PDT000226473.2':0.0153811,'PDT000395391.1':0.010426)'':0.0669196,(('PDT000644978.1':0.0558612,'PDT000675441.1':0.043278)'':0.00713757,('PDT000565958.1':0.0387588,(('PDT000395079.1':0.00685265,'PDT000645029.1':0.00511145)'':0.0186514,('PDT000399602.1':0.0120169,'PDT000479793.1':0.0174007)'':0.0104108)'':0.0195061)'':0.0164734)'':0.00776399)'':-0.00282176,((('PDT000051389.2':0.035615,'PDT000584793.1':0.0513288)'':0.0161966,(('PDT000257743.2':0.0334134,'PDT000479791.1':0.0368862)'':0.0161423,('PDT000397708.1':0.0327011,'PDT000497405.1':0.0422843)'':0.0112469)'':0.0145454)'':0.0118635,(((('PDT000218527.2':0.0431619,'PDT000505390.1':0.0484808)'':0.0181986,('PDT000566051.1':0.0230667,('PDT000393074.1':0.0129394,'PDT000617729.1':0.0121545)'':0.0129208)'':0.0372994)'':0.000212147,(('PDT000639550.1':0.0483201,('PDT000395370.1':0.00572829,'PDT000395324.1':0.00424279)'':0.0391197)'':-0.000313443,(('PDT000565841.1':0.0113993,'PDT000395230.1':0.0108317)'':0.035213,('PDT000497306.1':0.0375871,('PDT000497305.1':0.0352208,('PDT000565856.1':0.0122225,('PDT000395455.1':0.0117901,('PDT000417489.1':0.0110862,('PDT000249851.1':0.00946375,('PDT000395124.1':0.00819639,('PDT000397940.1':0.00550329,'PDT000627602.1':0.00645481)'':0.00102791)'':0.000964775)'':0.00071756)'':0.00188642)'':0.00603467)'':0.00548437)'':0.0113105)'':0.00609166)'':0.00583353)'':0.0125194)'':0.00231242,(('PDT000471014.1':0.0419557,('PDT000285416.2':0.0211083,'PDT000587229.1':0.0210073)'':0.00681084)'':0.0260428,((('PDT000565964.1':0.0263746,'PDT000395152.1':0.0260314)'':0.017027,('PDT000637365.1':0.0444313,('PDT000069118.2':0.0365074,'PDT000661411.1':0.0392048)'':0.0152698)'':0.00804232)'':0.00288512,(('PDT000241775.1':0.0516911,('PDT000394799.1':0.0242942,('PDT000003664.2':0.00723968,'PDT000566703.1':0.00746182)'':0.0278351)'':0.0179603)'':0.0107062,(('PDT000639567.1':0.050326,('PDT000550355.1':0.0158689,'PDT000629656.1':0.031541)'':0.0124585)'':0.0102947,('PDT000468255.1':0.0496726,('PDT000644914.1':0.0399004,('PDT000398014.1':0.022695,'PDT000644877.1':0.0156639)'':0.0156722)'':0.00509893)'':0.000123303)'':0.00617874)'':0.000655961)'':0.00825164)'':0.00428709)'':0.00490352)'':-0.000306616)'':0.0282911)'':0.00418338)'':0.0108953,((('PDT000319779.1':0.0797402,('PDT000618785.1':0.0476822,('PDT000107796.2':0.0444524,(('PDT000201898.2':0.0285102,'PDT000668947.1':0.0390732)'':0.00793908,('PDT000432062.1':0.0319102,'PDT000452802.1':0.0228103)'':0.00972227)'':0.00343606)'':0.00354826)'':0.0259581)'':0.0488965,((('PDT000425452.1':0.0888318,('PDT000200204.4':0.0832472,('PDT000024330.2':0.0633412,'PDT000565525.1':0.0604148)'':0.0134853)'':0.00457663)'':0.0161763,(('PDT000206106.2':0.0901852,'PDT000196593.2':0.0839798)'':0.0108826,('PDT000556965.1':0.0665565,'PDT000602935.1':0.0611175)'':0.0199511)'':0.0036995)'':0.00403269,(('PDT000145169.2':0.018561,'PDT000275669.2':0.0101125)'':0.0409906,(('PDT000639589.1':0.0601837,'PDT000395467.1':0.0326182)'':0.0326529,(('PDT000563942.1':0.0573311,('PDT000417989.1':0.0345308,'PDT000688671.1':0.0554272)'':0.00939112)'':0.00304501,('PDT000426145.1':0.0492564,('PDT000499479.1':0.0362782,('PDT000395133.1':0.0506467,('PDT000394804.1':0.0206569,('PDT000395153.1':0.0252044,('PDT000397584.1':0.0282513,'PDT000397894.1':0.0196079)'':0.00377048)'':0.00108155)'':0.00780315)'':0.00144862)'':0.0187148)'':-0.00351002)'':0.0134783)'':0.00789708)'':0.0173421)'':0.0186082)'':-0.00887775,(((('PDT000274083.2':0.0509285,'PDT000141124.3':0.0701985)'':0.0129629,('PDT000145183.2':0.0468462,(('PDT000713547.1':0.0269262,'PDT000294407.1':0.031483)'':0.00706586,('PDT000399970.1':0.0306238,('PDT000687927.1':0.0320637,'PDT000251909.1':0.0278048)'':0.00326983)'':0.000604861)'':0.00814201)'':0.00939108)'':-0.00207429,(('PDT000036657.3':0.058523,('PDT000397798.1':0.0321011,'PDT000395301.1':0.0374041)'':0.0119659)'':0.00930765,('PDT000661102.1':0.0691561,('PDT000550397.1':0.025805,('PDT000632006.1':0.0219231,('PDT000642307.1':0.0158036,'PDT000394958.1':0.0109874)'':0.00559885)'':0.0136408)'':0.0187583)'':0.000297134)'':0.00453297)'':0.0249318,(((('PDT000601912.1':0.0669457,'PDT000710447.1':0.0880893)'':0.00723378,('PDT000537188.1':0.0671323,('PDT000566050.1':0.0520225,('PDT000045609.1':0.0445511,('PDT000500479.1':0.0395822,('PDT000501055.1':0.0190908,'PDT000719135.1':0.016514)'':0.0145286)'':0.00304868)'':0.00820936)'':0.0125765)'':0.005592)'':-0.00319331,(('PDT000654334.1':0.077442,'PDT000640338.1':0.064848)'':0.00450636,(('PDT000555966.2':0.0637555,('PDT000362475.1':0.061584,('PDT000719196.1':0.0448346,'PDT000069630.1':0.0380819)'':0.00693375)'':0.00502252)'':0.00186416,(('PDT000566185.1':0.0489922,'PDT000251421.1':0.0489814)'':0.00716611,(('PDT000713761.1':0.0253234,'PDT000716940.1':0.0313062)'':0.0152711,('PDT000341114.1':0.0368875,('PDT000309352.2':0.0215349,'PDT000498767.1':0.0213671)'':0.0128216)'':0.00570648)'':0.00462726)'':0.00411102)'':0.00484748)'':-0.000992461)'':0.00469431,((('PDT000620186.1':0.0604269,'PDT000660403.1':0.0756901)'':0.00341232,('PDT000652873.1':0.085027,((('PDT000260223.2':0.0272913,'PDT000107797.2':0.0263672)'':0.0159853,('PDT000556977.1':0.0383811,'PDT000673414.1':0.0386014)'':0.0065759)'':0.00256275,('PDT000325140.2':0.0554666,('PDT000277154.3':0.0325863,('PDT000551374.2':0.0395645,('PDT000421170.2':0.0208156,('PDT000558673.2':0.0126396,'PDT000631704.2':0.0144034)'':0.00702112)'':0.0117894)'':0.0037894)'':0.0172806)'':-0.00214553)'':0.0137266)'':-0.000833113)'':0.00275517,(('PDT000081764.1':0.0702764,('PDT000399671.1':0.0705312,('PDT000719103.1':0.0569992,(('PDT000356781.1':0.0190542,'PDT000250718.1':0.0305905)'':0.00873565,(('PDT000430796.1':0.0175054,'PDT000491097.1':0.0214377)'':0.00579644,('PDT000677288.1':0.0228962,('PDT000600638.1':0.00901486,'PDT000660404.1':0.0116905)'':0.00954861)'':0.00917762)'':0.00436659)'':0.0113458)'':0.0049355)'':0.0150584)'':-0.0063341,((('PDT000497419.1':0.0508178,('PDT000317615.1':0.047449,('PDT000687833.1':0.0420937,'PDT000248815.1':0.0262188)'':0.00600762)'':0.00607017)'':0.00415293,('PDT000295989.3':0.059071,(('PDT000352675.2':0.0490775,'PDT000296061.3':0.0392178)'':0.00429865,(('PDT000149196.3':0.0388571,'PDT000115354.3':0.0346352)'':0.00303716,('PDT000712345.1':0.0424099,('PDT000239609.3':0.0265382,'PDT000451174.2':0.0265609)'':0.00593686)'':0.00272122)'':0.0151316)'':0.0125363)'':-0.00183145)'':0.0055784,('PDT000251682.1':0.065126,(('PDT000492244.2':0.0475825,((('PDT000281575.3':0.0206197,'PDT000309828.3':0.0163954)'':0.00981147,('PDT000296213.3':0.0161119,'PDT000677119.1':0.0206252)'':0.00364848)'':0.00756013,('PDT000284045.3':0.0527933,('PDT000257212.3':0.037259,('PDT000176689.3':0.0154394,'PDT000209177.3':0.0297797)'':0.00540527)'':0.00489903)'':-0.0017357)'':0.00417994)'':5.95505e-05,(('PDT000641978.2':0.0185121,'PDT000718966.1':0.0208681)'':0.0215417,(('PDT000149228.3':0.035117,'PDT000602251.2':0.0362978)'':0.00710634,(('PDT000499045.2':0.0234906,'PDT000646092.2':0.0190521)'':0.0103723,(('PDT000409256.2':0.0198682,('PDT000630160.2':0.0198315,'PDT000101060.3':0.0140752)'':0.00389529)'':0.000822391,('PDT000289833.3':0.0182293,('PDT000203879.3':0.0175506,('PDT000465907.2':0.0052617,'PDT000708940.1':0.00391536)'':0.00683117)'':0.00246046)'':0.00374101)'':0.00911111)'':0.005443)'':0.00407856)'':-0.000790423)'':0.0144552)'':0.0105768)'':0.00506006)'':0.00185944)'':0.00412466)'':0.0129367)'':0.00856719)'':0.00836004)'':0.00555898,(((('PDT000155615.2':0.0538271,'PDT000325660.1':0.0473259)'':0.0444031,('PDT000444958.1':0.0438532,('PDT000422544.1':0.0121834,('PDT000015826.2':0.00697496,('PDT000015859.3':0.00475463,'PDT000015862.2':0.00351477)'':0.00185894)'':0.0156307)'':0.0152254)'':0.0519451)'':0.00764703,((('PDT000132028.2':0.0271599,('PDT000553146.1':0.0191006,('PDT000553145.1':0.0110237,'PDT000083570.1':0.0127622)'':0.0115574)'':0.00849679)'':0.0393684,('PDT000524030.1':0.0789299,(('PDT000083467.1':0.0393979,(('PDT000625439.1':0.011921,'PDT000652365.1':0.0103916)'':0.0276376,('PDT000083454.2':0.0235539,'PDT000016383.2':0.0197827)'':0.0039141)'':0.020944)'':0.01337,(('PDT000625449.1':0.0441786,(('PDT000502975.1':0.0486421,'PDT000632871.1':0.0452726)'':0.00347318,('PDT000553163.1':0.0274193,'PDT000553165.1':0.0256737)'':0.0220529)'':0.0067659)'':-0.00152238,(('PDT000171829.1':0.0651746,('PDT000346537.1':0.0483989,('PDT000083553.1':0.0265914,('PDT000324194.1':0.0266418,(('PDT000524023.1':0.0202469,'PDT000283581.1':0.0181616)'':0.00201361,('PDT000083445.1':0.00549837,('PDT000083443.1':0.0029804,'PDT000083463.1':0.0020151)'':0.00364518)'':0.0128238)'':0.00770573)'':0.00310908)'':0.00630744)'':0.0150758)'':0.000318075,((('PDT000083568.2':0.0348975,('PDT000132402.2':0.0157368,'PDT000710985.1':0.0163403)'':0.0121253)'':0.00794127,(('PDT000332166.1':0.0179854,'PDT000652382.1':0.0203121)'':0.0258474,('PDT000553136.1':0.037182,'PDT000553162.1':0.031854)'':0.00862122)'':0.0029652)'':0.00809978,('PDT000435630.1':0.0435737,(('PDT000157923.2':0.03443,'PDT000324205.1':0.0343562)'':0.0152847,('PDT000625448.1':0.0562609,('PDT000083537.1':0.0365045,('PDT000652367.1':0.0300451,('PDT000132002.2':0.0338932,'PDT000083564.1':0.0388959)'':0.00443488)'':0.0058199)'':0.00832362)'':-0.000563742)'':0.00492286)'':-0.000107804)'':0.0025116)'':0.00121489)'':0.0049881)'':0.00743217)'':0.00596936)'':0.00183949,((('PDT000621427.1':0.0672374,(('PDT000325665.1':0.0179001,('PDT000082633.1':0.011329,'PDT000082637.1':0.0115332)'':0.00606584)'':0.0164459,('PDT000083481.1':0.0411304,('PDT000083534.1':0.0327813,'PDT000248607.1':0.0231316)'':0.00535812)'':0.0052566)'':0.027522)'':0.00214334,('PDT000083434.1':0.0695356,((('PDT000129886.2':0.0301459,'PDT000640735.1':0.03379)'':0.00530348,('PDT000103582.2':0.0331314,'PDT000421816.1':0.0333727)'':0.000898548)'':0.0106724,(('PDT000522666.1':0.0226249,'PDT000553192.1':0.018902)'':0.016227,('PDT000661336.1':0.0283922,(('PDT000417205.2':0.0137457,'PDT000324031.1':0.0152827)'':0.00986487,(('PDT000161704.2':0.0166968,'PDT000016378.2':0.0197297)'':0.00175151,('PDT000553186.1':0.00745915,'PDT000553189.1':0.010076)'':0.0101693)'':0.0014434)'':0.00981383)'':0.00421415)'':0.0144612)'':0.00737236)'':0.00192671)'':-0.000937365,((('PDT000083453.1':0.0345336,('PDT000083592.1':0.0334694,('PDT000467240.1':0.0392957,'PDT000721026.1':0.0276064)'':0.00663226)'':0.00271187)'':0.0190153,(('PDT000502389.1':0.0450564,'PDT000502397.1':0.0398913)'':0.0200901,('PDT000395093.1':0.0551346,(('PDT000161988.2':0.0269788,'PDT000637602.2':0.025967)'':0.000565527,('PDT000476882.2':0.0150808,('PDT000319802.2':0.0161099,('PDT000097085.3':0.0229553,('PDT000063284.3':0.0107517,('PDT000156767.3':0.0117989,'PDT000621339.2':0.00834667)'':0.00302699)'':0.00397653)'':0.00186165)'':0.00361662)'':0.0031088)'':0.0277375)'':0.00813789)'':-0.000789644)'':0.00339895,(('PDT000553178.1':0.0447726,'PDT000250143.1':0.0337731)'':0.0175448,((('PDT000081307.1':0.0497142,('PDT000625374.1':0.022273,'PDT000625383.1':0.029843)'':0.00411564)'':0.00412105,(('PDT000553166.1':0.00954657,'PDT000011108.2':0.0133905)'':0.00568833,('PDT000553167.1':0.00785536,'PDT000553168.1':0.0101349)'':0.0165318)'':0.00965503)'':0.014401,('PDT000619769.1':0.0509808,(('PDT000131884.2':0.0467486,('PDT000083542.1':0.0104048,('PDT000082639.1':0.0100282,'PDT000083543.1':0.0110902)'':0.00142063)'':0.0164158)'':0.0109818,('PDT000502372.1':0.0346369,('PDT000317744.1':0.034614,('PDT000645790.1':0.0175344,('PDT000667877.2':0.0139232,'PDT000553215.1':0.0159541)'':0.00320569)'':0.00789008)'':0.00762254)'':0.00361508)'':0.0133901)'':-0.00672287)'':0.00898734)'':0.0047573)'':0.0041351)'':0.00860068)'':0.0206879)'':0.016398,((((('PDT000520371.2':0.0741598,('PDT000345977.1':0.0545201,('PDT000228153.1':0.0532054,('PDT000470093.1':0.0354034,'PDT000502309.1':0.0268763)'':0.0220724)'':0.00711721)'':0.01533)'':0.00908326,(('PDT000480720.1':0.0413011,('PDT000150274.2':0.0327424,'PDT000397635.1':0.0377115)'':0.0088025)'':0.00875367,('PDT000163046.4':0.0603642,('PDT000633403.1':0.0522495,('PDT000466964.1':0.00584761,'PDT000706268.1':0.00489969)'':0.0287073)'':0.00771581)'':0.00271152)'':0.0223009)'':0.0104569,((('PDT000622371.1':0.0724273,'PDT000647676.1':0.0624947)'':0.0134006,(('PDT000232214.2':0.0689568,('PDT000303112.2':0.0630439,'PDT000397552.1':0.0564751)'':0.00898724)'':0.00781776,(('PDT000025251.2':0.0528939,'PDT000644984.1':0.0383534)'':0.0117955,('PDT000422095.1':0.06026,'PDT000566035.1':0.0378801)'':0.00747707)'':0.00132635)'':0.00776236)'':0.00608249,(('PDT000382211.1':0.0892433,('PDT000626567.1':0.0560285,('PDT000301009.2':0.051595,('PDT000354886.1':0.0146922,'PDT000251839.1':0.00943776)'':0.0328065)'':0.0113238)'':0.0161527)'':-0.00835414,(('PDT000411287.1':0.0710111,(('PDT000432473.1':0.0146699,'PDT000556892.1':0.0103623)'':0.0446767,('PDT000631993.1':0.0340271,'PDT000251444.1':0.035447)'':0.0130372)'':0.00597264)'':-4.6442e-05,(('PDT000290521.2':0.0544367,('PDT000687484.1':0.035074,'PDT000714235.1':0.0318643)'':0.00922795)'':0.00505842,('PDT000498054.1':0.0582492,('PDT000566158.1':0.0495975,('PDT000519098.2':0.0203916,'PDT000623899.2':0.0216631)'':0.0275156)'':0.00271932)'':0.00307067)'':0.00165199)'':0.00628041)'':0.00614216)'':0.013236)'':0.00843536,((('PDT000063258.3':0.0663219,('PDT000175377.2':0.0600378,'PDT000706195.1':0.0546202)'':0.00226712)'':0.00917864,((('PDT000336782.1':0.0488556,'PDT000250257.1':0.0341924)'':0.015407,('PDT000645023.1':0.0137451,'PDT000395415.1':0.0141724)'':0.0565785)'':0.00889717,(('PDT000502418.1':0.0840354,('PDT000395065.1':0.0247165,'PDT000501938.1':0.0448479)'':0.0304339)'':0.00149757,(('PDT000502920.1':0.0447772,'PDT000251747.1':0.0476115)'':0.00337242,('PDT000502494.1':0.0559053,('PDT000253456.2':0.0439506,'PDT000251582.1':0.043811)'':0.00988388)'':0.00783667)'':0.00235627)'':0.00872307)'':0.00609177)'':0.00630463,(((('PDT000651053.1':0.0559798,'PDT000710701.1':0.0405344)'':0.00840344,('PDT000346974.1':0.051747,('PDT000442830.1':0.0569659,('PDT000124618.1':0.0407609,'PDT000411278.1':0.0463256)'':0.0103408)'':0.00552294)'':0.00114831)'':0.0302022,(('PDT000095756.2':0.093945,'PDT000645018.1':0.078348)'':0.0107825,(('PDT000627429.1':0.0844754,('PDT000150261.2':0.0636099,('PDT000354908.1':0.0408333,'PDT000620260.1':0.0368234)'':0.0269292)'':0.0120262)'':-0.00115315,(('PDT000396765.1':0.0653724,('PDT000709898.1':0.0773106,(('PDT000150276.2':0.0174587,'PDT000485224.1':0.0220359)'':0.0140758,('PDT000375009.1':0.0174448,'PDT000395243.1':0.0157245)'':0.019162)'':0.0212998)'':0.00345952)'':-0.00275901,('PDT000428456.1':0.0866059,(('PDT000069408.3':0.0394132,('PDT000481014.2':0.0224197,('PDT000394754.2':0.0320971,'PDT000672842.1':0.0240069)'':0.00475549)'':0.00340401)'':0.02869,('PDT000095705.2':0.0664193,(('PDT000322024.2':0.0319662,'PDT000327353.2':0.0388225)'':0.00510154,('PDT000341781.2':0.0398655,('PDT000389386.2':0.0281346,'PDT000584996.2':0.0303164)'':0.00978868)'':0.00159768)'':0.0268083)'':-0.000681805)'':0.00914644)'':-0.0040976)'':0.00651599)'':0.00386178)'':0.00164244)'':-0.00230536,(('PDT000123311.1':0.0846118,('PDT000081055.1':0.0438914,'PDT000406986.1':0.0423287)'':0.0161386)'':0.00357891,(('PDT000161873.3':0.0148946,'PDT000488240.1':0.0151034)'':0.032588,((('PDT000426862.2':0.031521,('PDT000192048.3':0.017029,'PDT000199184.3':0.0204475)'':0.0082216)'':-0.000405193,(('PDT000271304.3':0.0258385,'PDT000103606.3':0.0208905)'':0.00900576,('PDT000366193.2':0.0143879,'PDT000063320.3':0.0110316)'':0.0103302)'':0.00256437)'':0.00364408,(('PDT000162968.4':0.0245751,'PDT000274866.3':0.0180285)'':0.0158377,(('PDT000146549.3':0.0110549,('PDT000258493.3':0.00751256,'PDT000488255.1':0.00718014)'':0.00521437)'':0.0119834,(('PDT000497970.1':0.0202867,(('PDT000539856.2':0.00666471,'PDT000140564.3':0.0110838)'':0.00656789,('PDT000200971.3':0.0136013,'PDT000103573.3':0.0122428)'':0.000486032)'':0.00601008)'':0.00417929,(('PDT000140524.3':0.0271877,(('PDT000487986.2':0.0146234,('PDT000426860.2':0.0143192,'PDT000502273.1':0.0120851)'':0.00503228)'':0.00333206,('PDT000497948.1':0.0199876,('PDT000354954.2':0.0121667,('PDT000279466.3':0.0093931,'PDT000497779.1':0.0066316)'':0.00619019)'':0.00343778)'':0.000644815)'':0.00289254)'':0.00165811,(('PDT000502448.1':0.0236352,('PDT000289819.3':0.0247972,('PDT000498140.1':0.0142273,('PDT000103571.3':0.00867476,'PDT000497962.1':0.00920194)'':0.00693013)'':0.00153557)'':0.0025597)'':0.000828902,('PDT000488261.1':0.0239109,('PDT000354911.2':0.0164228,('PDT000008715.2':0.0107613,('PDT000081056.1':0.00877775,'PDT000497945.1':0.00823045)'':0.00218637)'':0.00333253)'':0.00490894)'':0.00151583)'':0.000932458)'':0.00245634)'':0.00140494)'':0.000817608)'':-0.000694539)'':0.0184688)'':0.0142206)'':0.0153533)'':0.0042244)'':0.0163757)'':0.00184416,(((('PDT000188132.2':0.0535383,(('PDT000060221.2':0.0398324,'PDT000604382.1':0.0258785)'':0.0160631,('PDT000645132.1':0.0398369,('PDT000537187.1':0.0282381,'PDT000550384.1':0.0210597)'':0.0165666)'':0.00661693)'':0.0112127)'':0.0237912,(('PDT000149338.2':0.0399576,('PDT000005195.2':0.0290374,'PDT000019130.2':0.0357602)'':0.00448144)'':0.027112,(('PDT000343660.1':0.0654626,'PDT000660675.1':0.0526244)'':0.026763,('PDT000395346.1':0.0475004,('PDT000627427.1':0.0446958,'PDT000345981.1':0.0510017)'':0.0076019)'':0.0281439)'':0.00108625)'':0.0107269)'':0.0151015,((('PDT000359333.1':0.065521,('PDT000426257.1':0.0568865,('PDT000239092.1':0.0305138,'PDT000346527.1':0.0282353)'':0.0126223)'':0.0157836)'':0.000191458,('PDT000653022.1':0.0581137,('PDT000571190.1':0.0417681,('PDT000098370.2':0.0467759,('PDT000713637.1':0.0261914,'PDT000150899.1':0.0264196)'':0.00566168)'':0.00463473)'':0.00923719)'':0.0172726)'':0.0307411,((('PDT000417379.1':0.0255578,'PDT000518554.1':0.0265633)'':0.0598541,(('PDT000311271.2':0.019399,'PDT000312801.2':0.0221653)'':0.0159493,('PDT000379443.1':0.0399705,('PDT000352058.1':0.0232482,('PDT000409827.1':0.0173791,'PDT000660640.1':0.010965)'':0.00761244)'':0.0135919)'':0.000190548)'':0.0470827)'':0.0229633,((('PDT000250789.3':0.0884358,('PDT000633447.1':0.00116821,'PDT000634215.1':0.00102981)'':0.0639562)'':0.0099028,(('PDT000126420.3':0.0634179,('PDT000196594.2':0.0395099,'PDT000251469.1':0.0566892)'':0.0152625)'':0.00753699,('PDT000196654.2':0.0707248,('PDT000199449.3':0.0641482,('PDT000556934.1':0.0509749,'PDT000661690.1':0.0384401)'':0.0110103)'':0.00731028)'':0.00337025)'':0.0181781)'':-0.00340392,((('PDT000644835.1':0.0790175,('PDT000656946.1':0.0141554,'PDT000656947.1':0.0101571)'':0.0644173)'':0.0250582,(('PDT000246507.2':0.0470068,'PDT000719054.1':0.0544162)'':0.0359677,(('PDT000588886.1':0.0482995,('PDT000643505.1':0.0260634,('PDT000395209.1':0.0105661,'PDT000395088.1':0.0094815)'':0.0206616)'':0.0118173)'':0.0350787,('PDT000719247.1':0.0710021,(('PDT000285631.2':0.00805833,'PDT000395058.1':0.00704047)'':0.052724,('PDT000305448.2':0.022131,'PDT000312637.2':0.0172375)'':0.0390471)'':0.00675325)'':0.00169116)'':0.0131067)'':0.0045787)'':-0.00408088,(('PDT000673763.1':0.09374,(('PDT000157731.3':0.0709863,'PDT000658732.2':0.0586747)'':0.00640546,('PDT000465895.2':0.0625497,('PDT000557349.1':0.0569693,'PDT000672711.1':0.0597017)'':0.00340475)'':0.00207695)'':0.00892855)'':-0.00462557,((('PDT000565942.1':0.0145993,'PDT000488284.1':0.0129009)'':0.0358195,('PDT000645049.1':0.0680858,('PDT000309345.2':0.0298868,'PDT000399623.1':0.0381678)'':0.00966383)'':0.0118023)'':0.00153948,('PDT000227533.3':0.0584678,((('PDT000689960.1':0.0497395,'PDT000252483.1':0.0536435)'':0.00177783,('PDT000354861.1':0.0346814,('PDT000556983.1':0.0187816,'PDT000718847.1':0.0183975)'':0.0155719)'':0.0167819)'':0.00517731,(('PDT000598147.2':0.0351172,('PDT000360426.2':0.0168032,'PDT000421046.2':0.0228037)'':0.0210106)'':0.0203898,(('PDT000682352.1':0.0200788,'PDT000682554.1':0.00929767)'':0.0390985,('PDT000631998.1':0.0572088,(('PDT000589394.1':0.0205447,'PDT000502404.1':0.0189316)'':0.0143066,('PDT000477472.1':0.0495842,('PDT000034514.2':0.014615,'PDT000343746.1':0.01573)'':0.0312324)'':0.000304122)'':0.00583646)'':0.000161133)'':0.0026754)'':0.00112649)'':0.00367753)'':0.0011722)'':0.00289093)'':0.00649675)'':0.00823682)'':0.00591393)'':0.00307772)'':0.00945723)'':-0.00838446,(('PDT000239245.2':0.0876258,('PDT000290514.2':0.0721335,('PDT000607585.1':0.0516525,('PDT000687853.1':0.0503565,'PDT000395117.1':0.0604885)'':0.00174303)'':0.00401963)'':0.0162725)'':0.0256559,(('PDT000336560.1':0.0765341,('PDT000427079.1':0.0710359,('PDT000656995.1':0.0737194,(('PDT000457700.1':0.0285615,'PDT000457701.1':0.0307374)'':0.0194731,('PDT000181702.2':0.0394145,('PDT000223022.2':0.0273762,(('PDT000445118.1':0.0256585,'PDT000562760.1':0.0226621)'':0.00269589,('PDT000626757.1':0.0262717,('PDT000538559.1':0.0165991,('PDT000610221.1':0.013895,('PDT000632776.1':0.00625475,('PDT000434476.1':0.0012518,'PDT000444988.1':0.00114652)'':0.00292775)'':0.00658869)'':0.00754057)'':0.0014036)'':-0.000370438)'':0.00596423)'':0.00800415)'':0.00907155)'':0.0122021)'':0.00782068)'':0.00653666)'':0.0193686,((('PDT000167705.2':0.0430294,'PDT000051502.2':0.054399)'':0.0121069,('PDT000396839.1':0.0650201,'PDT000510081.1':0.0472639)'':0.0196507)'':0.037958,((('PDT000168721.1':0.0672003,('PDT000168785.1':0.0602139,('PDT000417111.1':0.040262,'PDT000417170.1':0.0413844)'':0.0199499)'':0.0072533)'':-0.00111215,('PDT000414182.1':0.0699894,('PDT000346917.1':0.0487822,('PDT000508912.1':0.0390895,'PDT000394410.1':0.0347758)'':0.00817953)'':0.00429925)'':0.0048287)'':0.0277853,(('PDT000668926.1':0.0982953,(('PDT000411323.1':0.0559958,('PDT000332417.1':0.0339157,('PDT000499506.1':0.0184123,'PDT000717300.1':0.0166945)'':0.00469617)'':0.0192309)'':-0.00586575,(('PDT000271752.2':0.0184725,('PDT000372147.1':0.0301232,('PDT000477459.1':0.0255833,('PDT000719933.1':0.017032,'PDT000042502.1':0.0206035)'':0.00387137)'':0.00198767)'':0.00154173)'':0.00692291,(('PDT000602260.1':0.0306663,('PDT000687476.1':0.0165486,('PDT000243746.2':0.0195744,('PDT000499485.1':0.0169729,'PDT000687358.1':0.0113768)'':0.00549293)'':0.00362932)'':0.00722048)'':0.00506368,(('PDT000426360.1':0.0343928,('PDT000426794.1':0.0144418,'PDT000661500.1':0.0196084)'':0.00493114)'':0.00873666,(('PDT000136045.2':0.0217592,'PDT000687453.1':0.0258692)'':0.00275456,('PDT000581873.1':0.0143807,'PDT000018734.1':0.0108169)'':0.00772801)'':0.00410149)'':0.00266217)'':0.00216106)'':0.0030685)'':0.0455749)'':-0.0223977,(('PDT000645082.1':0.0922602,(('PDT000399717.1':0.0467275,'PDT000621302.1':0.0347329)'':0.0214991,('PDT000397990.1':0.0390161,('PDT000645517.2':0.0261004,'PDT000346049.1':0.027748)'':0.0154525)'':0.0159756)'':0.0214345)'':-0.0160547,((('PDT000289923.1':0.0469486,('PDT000675463.1':0.0170353,'PDT000468922.1':0.00907849)'':0.0356038)'':0.00990396,('PDT000652349.1':0.0610758,('PDT000661136.1':0.0459803,('PDT000128815.2':0.0467164,('PDT000661680.1':0.0280632,('PDT000425660.1':0.0134149,('PDT000202754.2':0.0201138,('PDT000372176.1':0.00843096,('PDT000426172.1':0.0133937,'PDT000661805.1':0.00849685)'':0.00355424)'':0.00282807)'':0.00214172)'':0.00807825)'':0.00636139)'':0.00439365)'':0.0167295)'':-0.00156337)'':0.00092148,(('PDT000645550.2':0.0624051,('PDT000128829.2':0.053073,'PDT000445052.1':0.032793)'':0.00545958)'':0.0160934,(('PDT000566139.1':0.0315196,('PDT000384290.1':0.0222253,'PDT000517998.1':0.0164646)'':0.00793288)'':0.026257,(('PDT000720771.1':0.0740389,(('PDT000124606.1':0.0452444,'PDT000352531.1':0.0399747)'':0.00489944,('PDT000652912.1':0.0502041,('PDT000425494.1':0.0410294,'PDT000713960.1':0.0386209)'':0.00313582)'':0.00197539)'':0.010208)'':-0.00302502,(('PDT000289883.1':0.0452152,('PDT000656974.1':0.0407434,'PDT000171432.1':0.0386415)'':0.00454911)'':0.00844515,('PDT000188127.2':0.0645236,('PDT000497256.1':0.0421082,('PDT000616524.1':0.0349225,('PDT000652520.1':0.0348581,('PDT000128869.2':0.0173494,'PDT000716882.1':0.0138634)'':0.0130754)'':0.00366037)'':0.00274382)'':0.00735433)'':3.10825e-06)'':0.00152719)'':0.00640841)'':0.00233023)'':-0.00150808)'':0.00964259)'':-0.00396212)'':0.0120068)'':0.026957)'':-0.0100559)'':0.0198918)'':-0.0134579)'':0.00588306)'':0.00456375)'':0.00296078)'':0.0156812)'':0.0221435)'':0.0108555)'':-0.00128557,(((((('PDT000397546.1':0.0273366,'PDT000399638.1':0.0323386)'':0.0483587,(('PDT000051516.2':0.0482483,'PDT000572834.1':0.0376778)'':0.0141096,('PDT000532150.1':0.046232,('PDT000556916.1':0.0521548,('PDT000305443.2':0.0399375,'PDT000009388.2':0.0369482)'':0.00826743)'':0.00301802)'':0.00904379)'':0.011)'':0.00537182,(('PDT000710112.1':0.0342325,('PDT000143906.3':0.0201131,'PDT000636812.1':0.0159786)'':0.0150342)'':0.0442695,('PDT000430677.1':0.0492964,((('PDT000305084.2':0.0348617,'PDT000176457.1':0.0395353)'':0.000616405,('PDT000390647.3':0.0322994,'PDT000719927.1':0.0320978)'':0.00552174)'':0.00302647,('PDT000651983.1':0.0358709,('PDT000522696.1':0.0353714,('PDT000269701.2':0.0272593,'PDT000604274.1':0.0312778)'':0.00453639)'':0.000840268)'':0.00368268)'':0.00720291)'':0.0196005)'':0.0208329)'':0.0274776,(((('PDT000548603.2':0.0286272,('PDT000465223.2':0.0233288,('PDT000326241.2':0.0163826,'PDT000332165.2':0.0144079)'':0.00444191)'':0.0157728)'':0.0137759,('PDT000682797.1':0.0466839,('PDT000115455.3':0.0523767,('PDT000545347.2':0.0370114,('PDT000161656.3':0.0321975,('PDT000321596.2':0.0172657,'PDT000412490.2':0.0162583)'':0.00954992)'':0.00276963)'':0.00626941)'':0.0101389)'':0.0133182)'':0.0380064,(('PDT000285893.3':0.0374648,(('PDT000390973.2':0.0224452,'PDT000712343.1':0.021246)'':0.0109759,(('PDT000288572.3':0.0228855,('PDT000288539.3':0.00259269,'PDT000288558.3':0.00319573)'':0.00969783)'':0.00768679,('PDT000151457.3':0.0260793,('PDT000288536.3':0.0132127,'PDT000288563.3':0.0127228)'':0.0103948)'':0.00160592)'':0.010141)'':0.0136758)'':0.0206591,((('PDT000063280.3':0.0558615,'PDT000038884.1':0.0537355)'':0.00486045,('PDT000558719.2':0.033167,'PDT000639515.1':0.0213875)'':0.0283906)'':0.00906266,((('PDT000392398.2':0.0497371,('PDT000539869.2':0.0230069,'PDT000548333.2':0.0161545)'':0.0265734)'':0.0121635,('PDT000288565.3':0.0551802,(('PDT000620090.2':0.0480096,'PDT000486497.1':0.0434031)'':0.000762969,((('PDT000151475.3':0.0188998,'PDT000198979.6':0.0100394)'':0.0121617,('PDT000103570.3':0.02329,'PDT000140557.3':0.0257536)'':0.00250151)'':0.0100838,(('PDT000190090.3':0.0382737,'PDT000647784.2':0.0253027)'':0.0104214,('PDT000386033.2':0.0413431,'PDT000047680.4':0.0316061)'':0.00445559)'':0.0016947)'':0.00593632)'':0.00971983)'':0.00447569)'':0.00341866,(((('PDT000548332.2':0.0263837,'PDT000548461.2':0.0309741)'':0.0111509,('PDT000548331.2':0.044046,('PDT000568627.2':0.0314891,(('PDT000502302.1':0.0188063,'PDT000300587.1':0.0163384)'':0.00337241,('PDT000110191.3':0.0210149,'PDT000340512.2':0.0205001)'':0.00258257)'':0.00471656)'':0.0123522)'':0.000420727)'':-0.00119904,('PDT000323278.2':0.0456539,('PDT000548330.2':0.0525819,((('PDT000160436.3':0.0339127,'PDT000491572.2':0.0300249)'':0.00292378,('PDT000288560.3':0.00651284,'PDT000486493.1':0.00722536)'':0.0283003)'':0.000894771,(('PDT000163027.4':0.0122132,'PDT000063159.3':0.0100466)'':0.020262,('PDT000556333.2':0.0293173,('PDT000161498.3':0.0322555,(('PDT000443363.2':0.0181891,'PDT000621250.2':0.0145605)'':0.010164,(('PDT000215111.3':0.017646,'PDT000280748.3':0.0213485)'':0.003338,('PDT000063157.3':0.0143832,'PDT000123471.3':0.0167648)'':0.00723708)'':0.00300872)'':0.000850996)'':0.00389613)'':0.000115519)'':0.00180459)'':0.010348)'':0.00530663)'':-0.00588311)'':-0.000679445,(('PDT000063160.3':0.0430287,('PDT000289841.3':0.0399892,('PDT000336566.2':0.0345444,('PDT000140559.3':0.0284358,(('PDT000152694.3':0.0180346,'PDT000198981.3':0.0191665)'':0.00459689,('PDT000238399.3':0.0220301,'PDT000248812.1':0.0215461)'':0.00531241)'':0.00221579)'':0.00304229)'':0.00535689)'':0.0050125)'':-0.0051529,(('PDT000235324.3':0.0127091,('PDT000142584.3':0.0102052,'PDT000491983.2':0.0189991)'':0.00386556)'':0.0170382,(('PDT000137582.3':0.0209592,('PDT000084161.3':0.0207474,('PDT000063158.3':0.015611,('PDT000171579.3':0.0160878,'PDT000282453.3':0.0141764)'':0.00175528)'':0.00100636)'':0.00229116)'':0.00198678,('PDT000627895.2':0.043299,(('PDT000287910.3':0.0256732,'PDT000376018.2':0.0259114)'':0.00396813,('PDT000494278.2':0.0315909,((('PDT000486007.2':0.0147437,('PDT000367770.2':0.0125753,'PDT000672388.1':0.0151887)'':0.000519677)'':0.00252792,(('PDT000365738.2':0.0176602,'PDT000140227.3':0.0147952)'':0.00127647,('PDT000086214.3':0.015116,'PDT000140520.3':0.0142521)'':0.00120145)'':0.00113049)'':0.00245567,(('PDT000608813.2':0.0285715,'PDT000692283.1':0.0261519)'':0.0113989,(('PDT000517837.2':0.0203777,'PDT000556744.2':0.029943)'':0.00395459,(('PDT000140458.3':0.0184523,(('PDT000562864.2':0.0113828,'PDT000502468.1':0.0134908)'':0.00570218,('PDT000502466.1':0.0205297,('PDT000273369.3':0.0103989,'PDT000140506.3':0.00772318)'':0.00470624)'':0.000241809)'':0.00130888)'':0.0026204,(('PDT000502457.1':0.0155513,'PDT000065373.1':0.0117646)'':0.00542441,(('PDT000106334.4':0.010956,('PDT000268721.3':0.0035361,'PDT000106331.3':0.00343949)'':0.00335833)'':0.00484705,('PDT000603530.2':0.015455,('PDT000154950.3':0.0174172,('PDT000234172.3':0.0126544,('PDT000498122.1':0.0073795,('PDT000147469.3':0.00693117,'PDT000140570.4':0.00727653)'':0.000786703)'':0.00176866)'':0.00148904)'':0.00072676)'':-0.00043524)'':0.00280346)'':0.00144916)'':0.00280167)'':0.00421872)'':-0.00503452)'':0.00521878)'':-0.000158825)'':0.00546009)'':-0.0070473)'':0.00777925)'':-0.0018799)'':0.0122916)'':0.0202075)'':0.00731958)'':-0.000274785)'':0.017368)'':0.000692965,((('PDT000492229.1':0.059774,'PDT000637793.1':0.062613)'':0.0174097,('PDT000656696.2':0.03476,'PDT000721702.1':0.0318035)'':0.0412238)'':0.0589905,(('PDT000372045.1':0.0281116,'PDT000499327.1':0.0238962)'':0.0847143,(((('PDT000034483.2':0.0586992,'PDT000645599.2':0.0595078)'':0.0115447,('PDT000285386.2':0.0543908,('PDT000167658.2':0.0474893,('PDT000258735.2':0.0296394,'PDT000027911.2':0.0256822)'':0.00961322)'':0.0061415)'':0.00523508)'':0.0267544,(('PDT000485241.1':0.078104,('PDT000604841.1':0.0666076,'PDT000359673.1':0.0446014)'':0.011574)'':0.000896982,('PDT000241772.1':0.0477094,('PDT000566068.1':0.0466954,('PDT000025496.2':0.0566687,('PDT000572846.1':0.0111692,'PDT000572847.1':0.00969118)'':0.0266934)'':0.000701854)'':0.0134484)'':0.0168363)'':0.00557604)'':0.0124449,((('PDT000082672.4':0.0841929,('PDT000239124.2':0.0583004,('PDT000535463.1':0.044771,'PDT000353621.1':0.0298412)'':0.0165505)'':0.00454186)'':0.0218983,(('PDT000203427.2':0.0748009,'PDT000497389.1':0.0653011)'':0.00137967,('PDT000285934.2':0.0465916,('PDT000095677.2':0.0162956,'PDT000669449.1':0.0110867)'':0.0144115)'':0.0125312)'':0.0311197)'':0.000787086,((('PDT000652438.1':0.0810112,('PDT000357264.2':0.0512381,('PDT000131924.2':0.0559129,'PDT000426187.1':0.0421529)'':0.00541425)'':0.0137966)'':0.0032264,(('PDT000425468.1':0.0598852,('PDT000183024.3':0.0367714,'PDT000208429.2':0.0507736)'':0.0220138)'':-0.000719547,(('PDT000202782.2':0.0444241,'PDT000502481.1':0.0372225)'':0.00561777,(('PDT000550367.1':0.0536561,('PDT000352411.1':0.00108513,'PDT000352426.1':0.00111377)'':0.0440854)'':0.00162753,('PDT000645064.1':0.0486403,('PDT000399622.1':0.0367911,'PDT000473279.1':0.036825)'':0.0089876)'':0.00624027)'':0.00161563)'':0.00502164)'':0.0193582)'':0.00402558,(((('PDT000376085.1':0.0204153,'PDT000502878.1':0.0250795)'':0.0315444,('PDT000662249.1':0.0856276,('PDT000714574.1':0.0457968,('PDT000290380.2':0.030736,'PDT000672277.1':0.0352271)'':0.00862348)'':0.0152377)'':0.00124835)'':0.00748001,(('PDT000400019.1':0.0572777,'PDT000228302.1':0.0312064)'':0.0305471,(('PDT000288589.3':0.0540546,('PDT000359656.1':0.0622936,('PDT000671190.1':0.0541661,('PDT000557662.1':0.0460857,('PDT000266016.4':0.0309402,'PDT000714583.1':0.0192582)'':0.0197388)'':0.00697766)'':0.00459197)'':0.00446884)'':0.00281838,('PDT000386753.1':0.0722598,('PDT000687936.1':0.0455304,('PDT000346103.1':0.0611337,(('PDT000566182.1':0.0460974,'PDT000672193.1':0.0354456)'':0.00557703,('PDT000689851.1':0.0394627,('PDT000645116.1':0.0165311,'PDT000719274.1':0.0232354)'':0.0136419)'':0.00212421)'':0.00536491)'':0.00552195)'':0.00984286)'':0.000999744)'':0.00561988)'':-0.000203404)'':0.00131898,((('PDT000706412.1':0.0612936,('PDT000248793.1':0.0500286,'PDT000251832.1':0.0375962)'':0.0158085)'':0.00648311,('PDT000645558.2':0.111309,('PDT000317663.1':0.0521582,('PDT000713583.1':0.0517,('PDT000391670.1':0.0521176,('PDT000556899.1':0.0292228,('PDT000098758.2':0.0175913,('PDT000134071.2':0.0169918,'PDT000590201.1':0.010706)'':0.0124877)'':0.00490232)'':0.00695494)'':0.00503578)'':0.00325026)'':0.0178805)'':-0.000851672)'':0.00308757,(('PDT000355105.1':0.095889,'PDT000095750.2':0.08074)'':0.00688646,(((('PDT000258349.2':0.021319,'PDT000394863.1':0.0229842)'':0.0366909,('PDT000510897.1':0.0548332,('PDT000355622.1':0.0474483,'PDT000502332.1':0.0518323)'':0.00622451)'':0.00356739)'':0.00063693,(('PDT000539279.1':0.0402031,'PDT000396556.1':0.0423277)'':0.0216797,('PDT000092543.1':0.0624837,('PDT000249820.1':0.0276661,'PDT000251820.1':0.0262847)'':0.0269299)'':0.00335699)'':0.00399538)'':0.00671279,((('PDT000445115.1':0.0666697,('PDT000042501.1':0.040863,'PDT000359657.1':0.0524664)'':0.0141506)'':-0.000322598,('PDT000675135.1':0.0716842,('PDT000566055.1':0.0638171,('PDT000120811.2':0.040996,('PDT000307429.2':0.0328299,'PDT000251384.1':0.0429671)'':0.00752011)'':0.00805512)'':0.00515633)'':0.00454705)'':0.00480801,(('PDT000424770.1':0.0338525,('PDT000250253.1':0.0305965,('PDT000411019.1':0.0272257,'PDT000445003.1':0.0261037)'':0.00403033)'':0.0154021)'':0.0285294,((('PDT000191806.2':0.0228178,'PDT000227116.2':0.0224315)'':0.0322042,('PDT000339376.1':0.0298903,('PDT000477144.1':0.018659,'PDT000490906.1':0.0202255)'':0.0101046)'':0.0181349)'':0.0156351,('PDT000607489.1':0.114832,(('PDT000266430.2':0.066573,('PDT000042582.1':0.0646404,('PDT000672201.1':0.0358303,'PDT000252478.1':0.0398627)'':0.0186971)'':0.00263845)'':-0.000137825,((('PDT000502495.1':0.0432378,'PDT000124729.1':0.0541116)'':0.00497903,(('PDT000312776.2':0.0345289,'PDT000482218.1':0.0432607)'':0.00891258,(('PDT000421189.2':0.0622869,('PDT000069166.2':0.0238652,'PDT000397863.1':0.026042)'':0.0223085)'':0.00209153,('PDT000712684.1':0.0486463,(('PDT000432498.1':0.0296564,'PDT000694534.1':0.0375894)'':0.00682194,('PDT000121601.1':0.0205845,('PDT000430802.1':0.00536628,'PDT000430820.1':0.00899352)'':0.0133919)'':0.0266212)'':0.00441108)'':0.00124002)'':0.00193974)'':0.00635067)'':-0.00357211,(('PDT000624692.1':0.0606989,('PDT000042658.1':0.0390984,('PDT000178578.2':0.0415002,'PDT000672854.1':0.045855)'':0.0071652)'':0.0081928)'':0.000104783,('PDT000661088.1':0.0809568,(('PDT000497418.1':0.04026,'PDT000346696.1':0.0486696)'':0.0168251,(('PDT000405236.1':0.0372166,'PDT000171792.1':0.0345427)'':0.00431129,(('PDT000279749.2':0.0165822,'PDT000250118.1':0.0205206)'':0.0234675,('PDT000499446.1':0.0245042,('PDT000502433.1':0.0260229,('PDT000325615.1':0.0198239,'PDT000470107.1':0.0217896)'':0.00439497)'':0.00125745)'':0.0116422)'':0.00169502)'':0.00742151)'':0.0071493)'':-0.00439761)'':-0.00155704)'':0.00242096)'':0.0197224)'':-0.0149285)'':0.00355481)'':-0.000650759)'':0.000775037)'':0.00866988)'':-0.00714344)'':-0.000358121)'':0.00145482)'':0.00565595)'':0.00199189)'':0.026533)'':0.0184121)'':-0.0421464)'':0.0134504)'':0.0522124,((((('PDT000504742.1':0.109783,'PDT000640116.3':0.0995968)'':0.00365857,(('PDT000396821.1':0.0742449,'PDT000396837.1':0.0632551)'':0.00417081,('PDT000042667.1':0.073435,('PDT000426841.1':0.0791593,(('PDT000619601.1':0.0318924,('PDT000088274.2':0.0207482,'PDT000131796.2':0.0241173)'':0.0210331)'':0.0195966,('PDT000150905.1':0.0405489,('PDT000128804.2':0.0389661,'PDT000397618.1':0.0346822)'':0.011681)'':0.00281356)'':0.0177824)'':0.00378418)'':-0.00117165)'':0.0263783)'':0.0388117,((('PDT000316533.2':0.0435896,'PDT000502898.1':0.041576)'':0.0236547,('PDT000098698.3':0.0704145,('PDT000161976.3':0.0419645,'PDT000052117.3':0.0377877)'':0.0218679)'':0.000158129)'':0.0834131,(((('PDT000155465.3':0.0279595,'PDT000351559.2':0.0269952)'':0.00429669,('PDT000674806.1':0.0265617,('PDT000158952.3':0.0152378,'PDT000469487.2':0.0154026)'':0.00620995)'':0.0101643)'':0.00697483,('PDT000128674.3':0.0514248,('PDT000619331.2':0.0463737,(('PDT000224274.3':0.0238237,'PDT000602433.2':0.03154)'':0.00139975,('PDT000306745.3':0.0271018,('PDT000375684.2':0.0189848,'PDT000501046.2':0.0245966)'':0.00188344)'':0.00566186)'':0.00910416)'':0.0016774)'':-0.000317415)'':0.073384,(('PDT000604515.1':0.0665706,'PDT000672229.1':0.0808454)'':0.0175478,(('PDT000147478.3':0.0138817,('PDT000492072.2':0.00812854,'PDT000040445.4':0.00775396)'':0.00256864)'':0.028519,(('PDT000637352.2':0.0380295,('PDT000253094.3':0.01892,'PDT000315875.2':0.0196789)'':0.0130671)'':0.00764498,('PDT000063153.3':0.016306,('PDT000287845.3':0.0219837,('PDT000300565.1':0.0110716,('PDT000084198.3':0.0124062,'PDT000040390.4':0.0151404)'':0.00204622)'':0.00305766)'':0.00271207)'':0.0120402)'':0.00312479)'':0.0439686)'':0.0157814)'':0.0425567)'':0.00258157)'':0.00595814,((('PDT000425480.1':0.0892201,'PDT000557041.1':0.0781689)'':0.0708568,('PDT000468131.1':0.0959038,('PDT000359648.1':0.0627977,(('PDT000713812.1':0.0644692,('PDT000031975.2':0.0460116,'PDT000250114.1':0.0322415)'':0.0198872)'':7.18263e-05,(('PDT000248799.1':0.00667703,'PDT000346725.1':0.0108407)'':0.0626744,('PDT000490241.1':0.0381816,'PDT000482357.1':0.0672424)'':0.00489289)'':0.00150844)'':0.00410371)'':0.00893334)'':0.0820936)'':0.00491189,((((('PDT000384283.1':0.0778258,('PDT000661418.1':0.0614027,('PDT000607570.1':0.060505,'PDT000687958.1':0.061171)'':0.0159218)'':0.00762707)'':0.00332064,(('PDT000240694.2':0.0620067,('PDT000709963.1':0.0519439,('PDT000250182.1':0.029965,('PDT000380292.1':0.0202537,'PDT000664970.1':0.0281665)'':0.00591302)'':0.0124298)'':0.0137885)'':0.0172169,('PDT000604571.1':0.0771196,('PDT000485319.2':0.0610093,('PDT000714571.1':0.0624944,('PDT000228304.1':0.0483449,('PDT000380287.1':0.0236418,'PDT000251774.1':0.0226827)'':0.0109489)'':0.00720091)'':0.010494)'':0.0133744)'':-0.00156966)'':0.00918122)'':-6.74328e-06,(('PDT000502324.1':0.0818068,(('PDT000310538.3':0.0329405,'PDT000396586.2':0.0396291)'':0.0227687,('PDT000645013.1':0.0484347,'PDT000251894.1':0.0551433)'':0.000785521)'':0.0195315)'':-0.00200745,(('PDT000009302.2':0.0912454,('PDT000354211.1':0.0553384,('PDT000124599.1':0.00901644,'PDT000124692.1':0.0162381)'':0.0465563)'':0.0115344)'':0.00769937,('PDT000193059.2':0.0712237,('PDT000485406.1':0.0505874,('PDT000068833.2':0.022635,('PDT000345990.1':0.015777,('PDT000258435.2':0.0179791,('PDT000323544.1':0.0108594,'PDT000422101.1':0.0118971)'':0.00469545)'':0.00361281)'':0.000584393)'':0.0329044)'':0.012774)'':0.0180213)'':0.00388018)'':0.00716283)'':0.00448938,((('PDT000109330.2':0.0763145,(('PDT000149358.2':0.0718059,('PDT000556991.1':0.0318273,'PDT000718971.1':0.0376441)'':0.0203369)'':0.0176446,('PDT000227992.1':0.0689187,('PDT000566017.1':0.0570063,'PDT000345952.1':0.0516827)'':0.0119983)'':0.0102499)'':0.0129218)'':0.00861949,('PDT000539300.1':0.0932237,('PDT000251798.1':0.0918422,(('PDT000504788.1':0.0838446,('PDT000494427.1':0.0588826,('PDT000150243.2':0.0468689,('PDT000150268.3':0.0452047,'PDT000497971.1':0.0256357)'':0.00774039)'':0.033779)'':0.00954241)'':0.00166433,(('PDT000251997.1':0.0492971,('PDT000565879.1':0.0478564,'PDT000654617.1':0.0346889)'':0.00825167)'':0.00458105,(('PDT000379434.1':0.0436584,('PDT000428387.1':0.0194206,'PDT000660653.1':0.0253614)'':0.0129315)'':0.00947003,('PDT000663977.1':0.0566846,('PDT000502666.1':0.0361945,('PDT000502905.1':0.0232739,('PDT000502687.1':0.0104186,'PDT000502688.1':0.0151332)'':0.0112715)'':0.00567589)'':0.0214834)'':0.000268988)'':0.018272)'':0.0165479)'':0.00439696)'':0.00119777)'':0.00366832)'':-0.00105595,((('PDT000109368.2':0.0250648,('PDT000502546.1':0.0124984,'PDT000233075.1':0.0122174)'':0.00452765)'':0.0420409,('PDT000395409.1':0.0394126,('PDT000251908.1':0.0263081,('PDT000109468.2':0.0245485,'PDT000252657.1':0.0243534)'':0.000538042)'':0.012423)'':0.0220358)'':0.0185964,((('PDT000576976.1':0.0512092,('PDT000394942.1':0.0355499,('PDT000150242.2':0.0349983,'PDT000204923.2':0.0315328)'':0.013994)'':0.00327897)'':0.0191856,('PDT000386728.1':0.0688261,(('PDT000312802.2':0.0558028,'PDT000409833.1':0.0433531)'':0.00327717,('PDT000333194.1':0.0205272,'PDT000333195.1':0.0147844)'':0.0425921)'':0.00374016)'':0.00087878)'':0.0122993,(('PDT000289935.1':0.0673346,'PDT000501835.1':0.0737364)'':0.00610777,('PDT000248810.1':0.0913203,('PDT000048135.1':0.0752834,((('PDT000106253.2':0.0375819,'PDT000502904.1':0.0459432)'':0.010424,('PDT000094032.1':0.0465522,'PDT000251710.1':0.0410637)'':0.00720563)'':0.0134561,('PDT000258719.2':0.0556258,('PDT000563585.1':0.034372,('PDT000251706.1':0.022982,('PDT000286934.2':0.032396,('PDT000710817.1':0.0259996,(('PDT000095771.2':0.019239,('PDT000191810.2':0.0114196,'PDT000390711.1':0.0149066)'':0.00559045)'':0.00167213,('PDT000405490.1':0.0194707,('PDT000258378.2':0.0129512,'PDT000394977.1':0.0155806)'':0.00436881)'':0.00505514)'':0.00425295)'':0.0080334)'':0.00219342)'':0.00282821)'':0.0229181)'':-0.000194569)'':0.0133787)'':0.00396045)'':-0.00177775)'':-0.00248475)'':0.0132486)'':-0.00305914)'':0.00823373)'':0.0180195,(('PDT000713307.1':0.0438635,(('PDT000034395.1':0.00861838,'PDT000034399.1':0.0111393)'':0.0228801,(('PDT000443043.1':0.0150456,'PDT000033995.1':0.0152216)'':0.00378291,('PDT000033983.1':0.00857463,'PDT000034405.1':0.00804067)'':0.00584397)'':0.00747497)'':0.0159188)'':0.0906933,((('PDT000132412.3':0.0827367,('PDT000109426.2':0.0301367,'PDT000557204.1':0.0420626)'':0.0330811)'':0.0152012,(('PDT000034518.3':0.0458407,'PDT000477049.1':0.0314692)'':0.0342542,('PDT000594907.1':0.0448351,('PDT000709791.1':0.0411903,'PDT000395127.1':0.0441428)'':0.00772036)'':0.0167036)'':0.0121967)'':0.0189355,(((('PDT000149347.2':0.0708137,'PDT000354180.1':0.0639883)'':0.0271243,('PDT000311266.2':0.0903221,('PDT000149356.2':0.0254848,'PDT000557083.1':0.0199074)'':0.0784008)'':0.0101244)'':0.004665,((('PDT000572851.1':0.0878529,('PDT000128803.2':0.0981057,('PDT000395004.1':0.0875515,('PDT000095739.2':0.050268,'PDT000718822.1':0.056759)'':0.00935955)'':0.00494209)'':0.00627551)'':0.0121928,(('PDT000524493.1':0.0922964,'PDT000583483.1':0.0717846)'':0.00173851,('PDT000557154.1':0.0622083,('PDT000312639.2':0.069174,('PDT000652508.1':0.0740417,('PDT000498762.1':0.0532299,('PDT000510086.1':0.05782,('PDT000649253.1':0.038757,'PDT000251729.1':0.0452317)'':0.00929895)'':0.0028192)'':0.00629254)'':0.00416199)'':0.00563873)'':-0.0020608)'':0.0111445)'':-0.00200394,((('PDT000065367.1':0.0259781,('PDT000525774.2':0.00962957,'PDT000690008.1':0.00691743)'':0.0105062)'':0.0200421,(('PDT000227577.3':0.0281468,('PDT000184511.3':0.0186821,'PDT000538654.2':0.0130087)'':0.00946071)'':0.0107863,('PDT000490337.2':0.0254223,('PDT000357174.2':0.019448,('PDT000503691.2':0.0121023,'PDT000065363.1':0.0114341)'':0.00385273)'':0.00515804)'':0.0102811)'':0.0169671)'':0.0335529,(('PDT000324662.2':0.0653962,'PDT000665113.1':0.0614578)'':0.0358992,((('PDT000317751.2':0.0355891,('PDT000502395.1':0.00230658,'PDT000502512.1':0.00248862)'':0.0328663)'':-0.00219784,(('PDT000045899.3':0.0308102,'PDT000315876.2':0.0269462)'':0.00123333,('PDT000317586.2':0.0172139,'PDT000548635.2':0.0154227)'':0.00752949)'':0.0113458)'':0.0257749,('PDT000383098.1':0.106227,(('PDT000714279.1':0.0593619,'PDT000008766.2':0.0498151)'':0.0259526,('PDT000661141.1':0.0604801,((('PDT000189368.3':0.0299703,('PDT000248649.1':0.0207931,('PDT000396184.2':0.0185362,('PDT000146581.3':0.00661262,'PDT000481066.2':0.00960788)'':0.00915658)'':0.000985074)'':0.00344781)'':0.00971963,('PDT000354907.2':0.0222191,('PDT000288529.3':0.0187793,('PDT000186215.1':0.0202793,('PDT000534060.2':0.0110343,'PDT000497876.1':0.0103624)'':0.00526001)'':0.00690811)'':0.00275356)'':0.00742918)'':0.0040838,(('PDT000548561.2':0.0143571,'PDT000140494.3':0.0153742)'':0.0297766,(('PDT000008724.2':0.0327122,('PDT000400797.2':0.0163569,('PDT000288566.3':0.0101708,'PDT000288580.3':0.0109079)'':0.0111482)'':0.0182216)'':-0.00675626,(('PDT000598845.1':0.0329957,('PDT000234217.3':0.0269571,('PDT000176757.3':0.0194583,('PDT000204950.3':0.0182938,('PDT000251883.1':0.0163316,('PDT000133956.1':0.0110082,('PDT000193091.3':0.00883539,('PDT000248539.1':0.00476456,'PDT000300550.1':0.00419869)'':0.00306889)'':0.00157989)'':0.00200174)'':0.00402374)'':0.0010659)'':0.00224002)'':0.0140106)'':-0.00808719,('PDT000548446.2':0.0295836,('PDT000556116.2':0.0558228,('PDT000465606.2':0.0279462,('PDT000498007.1':0.0422748,(('PDT000225436.3':0.0178324,('PDT000040403.4':0.0142999,('PDT000240575.3':0.00718452,'PDT000115005.3':0.0101704)'':0.00432824)'':0.00336324)'':0.00456111,(('PDT000550066.2':0.0220021,'PDT000502267.1':0.020918)'':0.00519227,((('PDT000133957.1':0.018972,'PDT000133958.1':0.0283133)'':0.00189467,(('PDT000322447.2':0.018437,('PDT000502973.2':0.0101886,'PDT000566335.2':0.0119366)'':0.00190887)'':0.00425812,(('PDT000135351.3':0.00998035,('PDT000502580.1':0.00927315,'PDT000497989.1':0.010072)'':0.0047478)'':0.00355468,('PDT000502596.1':0.0190393,('PDT000502274.1':0.0172133,('PDT000498186.1':0.0105095,('PDT000271229.3':0.0117287,(('PDT000145775.3':0.00872685,'PDT000548388.2':0.00845285)'':0.00115839,(('PDT000249954.1':0.0112281,'PDT000497744.1':0.00885139)'':0.000654523,('PDT000502272.1':0.0113822,('PDT000400854.2':0.00844062,('PDT000077511.1':0.00569698,'PDT000497290.1':0.00534782)'':0.000898276)'':0.00126354)'':0.00100228)'':0.000958546)'':0.00091168)'':0.00135249)'':0.000271965)'':0.00306304)'':-0.0013908)'':0.00164242)'':0.00387253)'':-0.0029562,((('PDT000271198.3':0.0115487,'PDT000333178.2':0.0160471)'':0.00598508,('PDT000274845.3':0.0140375,'PDT000321850.1':0.0135835)'':0.00599794)'':0.00312753,(('PDT000714214.1':0.0213172,('PDT000131018.3':0.0164839,(('PDT000176686.3':0.00907414,'PDT000214059.3':0.00994326)'':0.00158005,('PDT000157464.3':0.00906669,('PDT000276349.3':0.00719427,'PDT000590278.2':0.00720783)'':0.00304161)'':0.0010776)'':0.00337462)'':0.00574646)'':-0.00265997,((('PDT000246449.1':0.0121317,('PDT000145389.3':0.0103795,'PDT000610179.2':0.00820708)'':0.00296445)'':0.00182915,('PDT000038904.1':0.0185955,(('PDT000157467.3':0.0138058,'PDT000140542.3':0.00970137)'':0.00214509,('PDT000040374.5':0.00556832,'PDT000502281.1':0.00597628)'':0.00612634)'':0.00532118)'':-0.00170322)'':0.00115634,((('PDT000568700.2':0.00578944,'PDT000663845.1':0.00946636)'':0.00524516,('PDT000677275.1':0.0062154,'PDT000710968.1':0.0082297)'':0.00876572)'':0.00413796,('PDT000248701.1':0.015731,(('PDT000140536.3':0.0110273,('PDT000532709.2':0.0114084,('PDT000142592.3':0.00714917,'PDT000214057.3':0.00471643)'':0.00215422)'':0.0027713)'':0.000498744,('PDT000299804.3':0.00870634,(('PDT000140496.3':0.00419537,'PDT000140549.3':0.00618903)'':0.00708688,(('PDT000150165.3':0.00621215,'PDT000538680.2':0.00494105)'':0.00142681,('PDT000062612.3':0.00776684,'PDT000140474.3':0.00731996)'':0.000406712)'':0.00189261)'':0.00062077)'':0.000852249)'':0.00248044)'':0.000486284)'':0.00287814)'':0.000616468)'':0.00194571)'':-0.000314162)'':0.00166427)'':0.000393497)'':0.00970476)'':0.00118484)'':0.00236294)'':0.00101147)'':-0.0091524)'':0.00689678)'':0.0116579)'':-0.00841968)'':0.0235842)'':0.0167685)'':0.0130373)'':-0.0260876)'':0.0260093)'':-0.010623)'':0.00900612)'':0.00862468)'':-0.00468715,(((('PDT000459068.2':0.023375,'PDT000098759.3':0.0247133)'':0.00196427,(('PDT000490362.2':0.00691548,'PDT000136768.3':0.00807662)'':0.00809609,('PDT000107487.3':0.0211834,('PDT000213252.3':0.0164702,'PDT000411010.2':0.0160421)'':0.00591198)'':0.000342919)'':0.0153063)'':0.0627792,((('PDT000501759.2':0.0524589,('PDT000250258.1':0.0382538,'PDT000354176.1':0.0258879)'':0.0126735)'':0.00242646,('PDT000430305.2':0.0609885,('PDT000148297.3':0.0540266,'PDT000148511.3':0.0434968)'':0.0139818)'':0.00403098)'':0.0103732,(('PDT000482558.2':0.0591931,('PDT000679783.1':0.0590179,'PDT000502060.1':0.0711731)'':0.00971494)'':-0.000480787,(('PDT000172094.1':0.02121,'PDT000172095.1':0.0388797)'':0.0217608,('PDT000172096.1':0.0422767,'PDT000186546.1':0.0470279)'':0.00512385)'':0.0224084)'':0.0135955)'':0.00576404)'':0.0178548,((('PDT000385502.1':0.0503504,'PDT000248771.1':0.0468705)'':0.0144371,('PDT000568771.1':0.0742906,('PDT000248823.1':0.0277876,'PDT000251856.1':0.0287536)'':0.0200748)'':0.00910109)'':0.0234741,(((('PDT000500536.1':0.0597083,'PDT000550370.1':0.0610027)'':0.0310507,('PDT000251768.1':0.0791359,'PDT000357620.1':0.0740261)'':0.0110293)'':0.00380825,((('PDT000718832.1':0.0663425,('PDT000124593.1':0.0535246,'PDT000250203.1':0.0468364)'':0.012478)'':0.000895125,('PDT000557337.1':0.0545848,('PDT000249914.1':0.0729418,('PDT000502528.1':0.0636355,('PDT000305482.2':0.045658,('PDT000251463.1':0.0422917,('PDT000557215.1':0.0308651,'PDT000250195.1':0.0472569)'':0.0107906)'':0.00973255)'':0.00419032)'':0.00398869)'':0.00338391)'':-0.000370689)'':0.00442511,(('PDT000502313.1':0.0354548,'PDT000502426.1':0.0380519)'':0.0407795,(('PDT000548941.1':0.0380019,('PDT000497350.1':0.038405,('PDT000299884.2':0.0152019,'PDT000289946.1':0.0147636)'':0.0236333)'':0.005274)'':0.00830503,('PDT000720206.1':0.0619568,(('PDT000719046.1':0.0414228,('PDT000336785.1':0.0231145,'PDT000479860.1':0.0240794)'':0.02004)'':0.00665044,(('PDT000271378.2':0.0344437,'PDT000494431.1':0.0369918)'':0.00400837,('PDT000310445.2':0.0371048,('PDT000225859.2':0.0308273,'PDT000355750.1':0.0256362)'':0.00694808)'':0.00207349)'':0.00417799)'':0.0112536)'':-0.0033427)'':0.0254729)'':-0.00521098)'':0.0156067)'':0.00462198,(('PDT000672650.1':0.0291838,('PDT000565983.1':0.0228803,('PDT000394837.1':0.0170859,('PDT000340986.1':0.0148067,'PDT000394994.1':0.00888902)'':0.00313378)'':0.00200915)'':0.00392101)'':0.0593321,(((('PDT000664973.1':0.0582163,'PDT000664983.1':0.0643997)'':0.00703079,('PDT000426129.1':0.0549794,'PDT000713383.1':0.0548616)'':0.00713321)'':0.01479,('PDT000479089.1':0.092418,((('PDT000202019.2':0.0412933,'PDT000426500.1':0.0291306)'':0.0433056,('PDT000315733.1':0.0630012,'PDT000345912.1':0.0740678)'':0.0132734)'':0.00264177,(('PDT000664979.1':0.0447468,('PDT000664984.1':0.0424093,'PDT000124645.1':0.0315887)'':0.00203953)'':0.0214548,('PDT000252695.1':0.0509481,('PDT000526176.1':0.0833867,('PDT000145178.2':0.0431782,('PDT000309224.2':0.0194249,'PDT000687434.1':0.0214195)'':0.0183456)'':0.0137564)'':0.00214336)'':-0.000679281)'':0.0136614)'':0.00739078)'':-0.00605562)'':0.00432002,((('PDT000131751.2':0.0328765,(('PDT000380297.1':0.00325044,'PDT000242502.1':0.00413553)'':0.0145049,('PDT000651904.1':0.00885302,'PDT000380162.1':0.00553258)'':0.0128859)'':0.00878499)'':0.0301612,(((('PDT000158950.3':0.0256859,'PDT000486502.1':0.0226143)'':0.00952784,('PDT000288596.3':0.0238304,'PDT000486506.1':0.0228858)'':0.0121707)'':0.00422361,(('PDT000051175.1':0.0347762,'PDT000497884.1':0.0307464)'':0.00263443,('PDT000186217.1':0.0418256,('PDT000498156.1':0.0339318,('PDT000486505.1':0.0150488,'PDT000486508.1':0.010982)'':0.017806)'':0.00517746)'':0.000253301)'':0.0083965)'':0.000909481,(('PDT000709069.1':0.02173,('PDT000406953.2':0.0272503,'PDT000112469.3':0.027051)'':0.00250542)'':0.00512108,(('PDT000499086.1':0.0239074,('PDT000501959.1':0.0154339,('PDT000497907.1':0.00644257,'PDT000497908.1':0.00945093)'':0.00761074)'':0.00254793)'':0.00690331,('PDT000113660.3':0.0146111,('PDT000400043.2':0.0247395,('PDT000040443.4':0.0196972,('PDT000300524.1':0.0137578,('PDT000148560.3':0.0189598,('PDT000376001.2':0.00832159,('PDT000200959.3':0.0106826,'PDT000077512.1':0.00637306)'':0.00212251)'':0.00322411)'':0.00137273)'':0.000741055)'':0.00404773)'':0.00162496)'':0.000840036)'':0.0036972)'':0.011922)'':0.025222)'':0.0063132,(('PDT000305453.2':0.00927333,'PDT000228066.1':0.00638127)'':0.0855241,((('PDT000362849.1':0.0439536,'PDT000251947.1':0.0467548)'':0.0108512,('PDT000124684.1':0.0492512,('PDT000306575.2':0.0343212,'PDT000354179.1':0.0270255)'':0.0176029)'':0.00595373)'':0.00987985,((('PDT000275042.3':0.0490547,'PDT000424793.1':0.0333134)'':0.0200275,(('PDT000196655.2':0.0488292,('PDT000468327.1':0.0359962,'PDT000594474.1':0.0389145)'':0.00586749)'':0.0153353,('PDT000500473.1':0.062798,('PDT000510049.1':0.0342571,('PDT000390710.1':0.027419,('PDT000109467.2':0.00648036,'PDT000251796.1':0.00525974)'':0.0182064)'':0.00442081)'':0.0234881)'':-0.00813442)'':0.0171516)'':0.00254309,(('PDT000604476.1':0.0676954,(('PDT000341201.1':0.0403666,'PDT000501842.1':0.0416069)'':0.00779173,('PDT000251530.1':0.0562075,('PDT000557106.1':0.0173263,'PDT000719155.1':0.0204188)'':0.0199615)'':0.00209843)'':0.0053302)'':0.0101459,('PDT000249924.1':0.0821452,('PDT000687999.1':0.0620979,('PDT000556938.1':0.050514,((('PDT000196657.2':0.0382338,('PDT000250255.1':0.0321702,'PDT000395417.1':0.021224)'':0.00968675)'':0.00417495,('PDT000110190.3':0.0472814,('PDT000110450.3':0.0342056,('PDT000332756.2':0.0051853,'PDT000532977.2':0.00439458)'':0.0168778)'':0.0159738)'':0.0048373)'':0.00730459,(('PDT000428374.1':0.0559237,'PDT000627592.1':0.0307443)'':0.0126887,('PDT000197966.2':0.0469543,('PDT000713871.1':0.0524139,('PDT000233050.1':0.0349124,(('PDT000150248.2':0.0156997,'PDT000248777.1':0.0210592)'':0.00753387,('PDT000107086.2':0.0181546,'PDT000081724.1':0.0202126)'':0.00364601)'':0.00376012)'':0.00593539)'':0.00485266)'':0.00288002)'':0.00403456)'':0.005428)'':0.00676736)'':0.0101531)'':-0.0153248)'':0.00049529)'':0.00344681)'':0.0178309)'':-0.0169474)'':0.00999184)'':0.00287898)'':0.00186531)'':0.00825094)'':-0.0059315)'':0.00423638)'':0.0123426)'':0.00967975)'':-0.00131277)'':0.0408867)'':-0.0132895)'':-0.0187287,(((('PDT000598484.2':0.088213,('PDT000610399.1':0.0524244,'PDT000719011.1':0.0490996)'':0.02833)'':0.0284871,(('PDT000495247.2':0.100861,('PDT000395005.1':0.0698911,('PDT000656996.1':0.051896,'PDT000396753.1':0.059872)'':0.00573544)'':0.0278121)'':-0.00665873,('PDT000248500.1':0.054308,(('PDT000003957.2':0.0205549,'PDT000496884.1':0.0214586)'':0.0233605,('PDT000566072.1':0.0362331,('PDT000003668.2':0.0264957,'PDT000397958.1':0.0261719)'':0.0113714)'':0.00965314)'':0.0292801)'':0.0201629)'':0.0270032)'':0.0404184,(('PDT000206243.3':0.122831,('PDT000625994.1':0.0965723,'PDT000396742.1':0.0798377)'':0.0107285)'':0.00940696,(((((('PDT000003791.2':0.0722153,'PDT000583410.1':0.0566777)'':0.0139217,('PDT000141130.2':0.0583286,'PDT000308402.1':0.0599724)'':0.0105643)'':0.00467478,('PDT000505150.1':0.0771289,('PDT000497416.1':0.101318,('PDT000662578.1':0.0865312,(('PDT000441811.1':0.0807495,'PDT000713272.1':0.0673495)'':0.00163546,(('PDT000128801.2':0.0711961,'PDT000605106.2':0.0740269)'':0.00293179,(('PDT000285626.2':0.0520141,'PDT000313007.2':0.0399208)'':0.00936223,(('PDT000498769.1':0.0204287,'PDT000568708.1':0.0320888)'':0.0248047,(('PDT000482073.1':0.0206408,'PDT000496790.1':0.024742)'':0.0163749,(('PDT000246492.2':0.0159456,'PDT000397596.1':0.013957)'':0.0207413,('PDT000502654.1':0.0267229,('PDT000687849.1':0.0281359,('PDT000292832.2':0.0225306,('PDT000498779.1':0.020383,'PDT000069670.1':0.0214601)'':0.00921543)'':0.00127261)'':0.0009837)'':0.00711519)'':0.00221889)'':0.00352743)'':0.00290132)'':0.0099848)'':0.00115655)'':0.00343855)'':0.00429896)'':0.0120247)'':-0.0182155)'':0.0319802,(('PDT000048651.3':0.101677,(('PDT000032355.1':0.00134859,'PDT000032356.1':0.00204666)'':0.0427038,('PDT000274727.2':0.0706183,('PDT000438312.1':0.0539592,('PDT000428896.1':0.00617108,('PDT000600192.1':0.00434423,('PDT000604125.1':0.00425857,('PDT000490283.1':0.0022696,'PDT000490340.1':0.00272241)'':0.000738264)'':0.000223424)'':0.00394604)'':0.0215011)'':0.0344295)'':-0.00464712)'':0.0323079)'':0.00273618,(('PDT000015708.3':0.0216789,('PDT000015854.3':0.0114173,('PDT000015830.3':0.0104957,('PDT000015818.3':0.00945341,('PDT000015836.2':0.0108366,('PDT000015834.2':0.00852718,('PDT000015851.3':0.00761839,'PDT000015874.2':0.00392161)'':0.000603222)'':0.000672482)'':0.000877686)'':0.000205048)'':0.00436564)'':0.00689728)'':0.0745597,('PDT000618837.1':0.0969049,('PDT000583362.1':0.0677526,(('PDT000174337.1':0.055554,('PDT000174368.1':0.0185632,'PDT000174384.1':0.0339863)'':0.0372148)'':0.0102619,('PDT000242798.2':0.0665846,(('PDT000333964.1':0.0229692,'PDT000711418.1':0.0334062)'':0.0276087,('PDT000174345.1':0.0255429,('PDT000652384.1':0.0237463,'PDT000174331.1':0.0210857)'':0.00742908)'':0.0281202)'':0.0148803)'':0.000533993)'':0.0055832)'':0.0163779)'':-7.32357e-05)'':0.0063569)'':0.00626271)'':0.00197609,((((('PDT000556927.1':0.0771948,('PDT000251392.1':0.0673924,'PDT000252000.1':0.0519776)'':0.0368372)'':0.00244787,('PDT000251763.1':0.0488766,('PDT000718855.1':0.0579183,('PDT000557157.1':0.0129523,'PDT000250038.1':0.0181923)'':0.0449324)'':0.0068315)'':0.0238238)'':0.0140638,(('PDT000687283.1':0.0630997,'PDT000289956.1':0.0691993)'':0.00290751,(('PDT000036658.3':0.061582,'PDT000395223.1':0.053054)'':0.0108628,('PDT000644942.1':0.0667913,('PDT000183029.3':0.0590622,('PDT000294683.2':0.040218,('PDT000640128.1':0.0500184,'PDT000395378.1':0.0448175)'':0.00487471)'':0.00930399)'':0.018231)'':0.00610599)'':0.00341748)'':0.0140187)'':0.0102604,(('PDT000683620.1':0.0763454,('PDT000003795.2':0.0532782,'PDT000413208.1':0.0444341)'':0.0155069)'':0.0137751,('PDT000298036.1':0.143045,(('PDT000346900.1':0.0664753,('PDT000362832.1':0.0477335,(('PDT000088258.2':0.0473388,'PDT000504798.1':0.0335934)'':0.00467234,('PDT000426300.1':0.0297911,'PDT000535094.1':0.0366801)'':0.0135322)'':0.00301599)'':0.0107043)'':-0.0010846,(('PDT000644912.1':0.0566203,('PDT000397885.1':0.0604006,('PDT000610081.1':0.0478411,'PDT000652835.1':0.0535189)'':0.0135059)'':0.00413155)'':0.000495149,(('PDT000346902.1':0.0551569,('PDT000426625.1':0.0495091,'PDT000519821.1':0.0513839)'':0.00462957)'':0.00281006,('PDT000258727.2':0.0557803,('PDT000403076.1':0.0398026,'PDT000386754.1':0.0420524)'':0.0175027)'':0.0101075)'':0.00377152)'':0.00554008)'':0.0311059)'':-0.00701028)'':0.0113985)'':0.00586536,((((('PDT000557667.2':0.0780028,'PDT000346904.1':0.0762782)'':0.00359742,('PDT000251981.1':0.0861859,('PDT000661546.1':0.0819481,('PDT000437936.1':0.0777665,(('PDT000652531.1':0.0537519,'PDT000626901.1':0.0444035)'':0.00346979,('PDT000535424.1':0.0433629,'PDT000252448.1':0.0394335)'':0.00843646)'':0.0109935)'':0.00172713)'':0.0132295)'':-0.00269591)'':0.00265976,(('PDT000500713.1':0.0884188,('PDT000661316.1':0.0864856,('PDT000352484.1':0.073844,('PDT000707297.1':0.0577498,('PDT000424775.1':0.0553644,('PDT000694544.1':0.051762,'PDT000346501.1':0.0442318)'':0.00419216)'':0.0068597)'':0.00413967)'':0.00482582)'':0.00992125)'':-0.00160527,((('PDT000384281.1':0.044888,'PDT000395360.1':0.076057)'':0.0237016,(('PDT000421962.1':0.0535088,('PDT000502500.1':0.0445444,'PDT000251841.1':0.0352573)'':0.0124666)'':0.00263022,('PDT000719095.1':0.0335027,('PDT000500535.1':0.0265311,'PDT000683165.1':0.0363328)'':0.00479281)'':0.0143796)'':0.0103009)'':-0.00145176,('PDT000397760.1':0.0787572,(('PDT000498741.1':0.0635973,'PDT000174359.1':0.0646757)'':0.00205808,('PDT000465585.2':0.0570732,('PDT000396848.1':0.0477161,('PDT000129815.2':0.0332135,('PDT000668485.2':0.020775,('PDT000598046.1':0.00943042,'PDT000502405.1':0.0121273)'':0.00681273)'':0.0168562)'':0.00428544)'':0.00660613)'':0.00481052)'':0.00640548)'':0.00120775)'':0.00894175)'':0.0040462)'':0.00116253,((('PDT000653002.1':0.0400409,'PDT000252638.1':0.0559094)'':0.0545096,('PDT000354868.1':0.0762801,('PDT000619623.1':0.0481455,'PDT000046604.1':0.0498488)'':0.0431902)'':0.00412528)'':0.0036235,(('PDT000124626.1':0.0815085,('PDT000652548.1':0.0770284,('PDT000034512.2':0.0735954,('PDT000494563.1':0.0548387,('PDT000367275.1':0.0743887,('PDT000669533.1':0.050969,('PDT000355612.1':0.0225629,'PDT000471012.1':0.0272693)'':0.02341)'':0.00535986)'':0.00471222)'':0.0026682)'':0.0136857)'':0.00374281)'':0.00273661,(('PDT000393057.1':0.0774348,'PDT000675290.1':0.0722142)'':0.0138288,(('PDT000312644.2':0.0816892,'PDT000354247.1':0.0793158)'':0.0162872,(('PDT000027735.2':0.0791509,('PDT000034462.2':0.0784672,('PDT000128848.2':0.0538783,'PDT000566229.1':0.0433118)'':0.0110033)'':0.0154365)'':0.00602455,(('PDT000358640.1':0.0598633,'PDT000556939.1':0.0620807)'':0.0133214,(('PDT000589282.1':0.0666949,'PDT000594899.1':0.0561371)'':0.00899587,((('PDT000719097.1':0.055353,'PDT000353634.1':0.046692)'':0.00420796,('PDT000498772.1':0.0453054,('PDT000004016.2':0.0353621,'PDT000249827.1':0.0437432)'':0.00746893)'':0.0109725)'':0.00437765,('PDT000620807.1':0.0773854,(('PDT000631560.1':0.0280393,'PDT000655877.1':0.0239521)'':0.0263366,('PDT000337204.1':0.0370562,('PDT000311260.2':0.024137,'PDT000417502.1':0.0259021)'':0.0147688)'':0.00335748)'':0.0268884)'':-0.00340767)'':0.00379658)'':0.00306926)'':-0.000680743)'':0.00770171)'':0.00545442)'':-0.00344658)'':0.00533109)'':-0.000201811)'':0.00513299,(((('PDT000240706.1':0.0896153,('PDT000204869.2':0.0750807,'PDT000215551.2':0.0705003)'':0.0203062)'':0.00231572,(('PDT000594910.1':0.0880659,(('PDT000204878.2':0.0754014,'PDT000645657.2':0.0790626)'':0.00102908,('PDT000346562.1':0.07182,'PDT000346898.1':0.058192)'':0.0187699)'':0.0141179)'':0.00524511,(('PDT000524033.1':0.0884856,('PDT000201013.2':0.0659044,'PDT000584492.1':0.0620706)'':0.00827886)'':0.00892423,('PDT000504741.1':0.0799738,('PDT000353637.1':0.0732631,'PDT000468844.1':0.0774329)'':0.0071817)'':0.0140836)'':0.00310232)'':0.00603716)'':0.00968685,(('PDT000397641.1':0.0924723,((('PDT000289939.1':0.0490437,('PDT000375008.1':0.0237946,'PDT000557251.1':0.03204)'':0.015458)'':0.0178781,('PDT000216178.2':0.0637852,('PDT000217102.2':0.0525617,('PDT000618775.1':0.0331504,'PDT000289911.1':0.048697)'':0.0230126)'':0.0127797)'':0.00652973)'':0.00829483,(('PDT000206250.3':0.0786595,'PDT000468892.1':0.0802115)'':0.00343548,('PDT000616423.1':0.0660987,('PDT000200987.2':0.0779691,('PDT000200976.2':0.0611486,('PDT000163825.2':0.0483012,'PDT000203410.2':0.0574958)'':0.00988537)'':0.0103371)'':0.00694359)'':0.00566626)'':0.000377862)'':0.00623794)'':-0.00341902,(('PDT000355611.1':0.0547913,('PDT000303139.2':0.0396256,('PDT000240674.2':0.0211311,'PDT000566146.1':0.0355233)'':0.0146759)'':0.0153281)'':0.000962062,(('PDT000706398.1':0.0348839,('PDT000627606.1':0.0376777,('PDT000277716.2':0.0422479,('PDT000491869.1':0.0287891,'PDT000628869.1':0.0334048)'':0.00753161)'':0.00286305)'':0.00174184)'':0.00627348,('PDT000346044.1':0.0650111,('PDT000051870.2':0.0597577,(('PDT000688496.1':0.0194128,'PDT000252563.1':0.0169347)'':0.0175235,('PDT000395051.1':0.025709,('PDT000319754.1':0.0322588,'PDT000274084.2':0.0315256)'':0.00595376)'':0.0032263)'':0.0175609)'':0.00347637)'':-0.00252719)'':0.00133345)'':0.0266868)'':0.0100208)'':-0.000977753,((((('PDT000525370.1':0.0552904,'PDT000672191.1':0.0363533)'':0.016005,('PDT000557654.2':0.0477146,'PDT000346905.1':0.0574934)'':0.00695208)'':0.0147326,('PDT000573300.1':0.0976236,('PDT000643455.1':0.0742547,('PDT000672278.1':0.0423698,('PDT000354863.1':0.0283586,'PDT000584489.1':0.0307135)'':0.0225435)'':0.011226)'':0.0138845)'':0.00173758)'':0.00858253,(('PDT000329047.1':0.0838212,'PDT000291288.1':0.0696698)'':0.00280313,(('PDT000024553.2':0.069008,('PDT000425530.1':0.0445009,('PDT000645044.1':0.0555785,(('PDT000306643.2':0.0310833,'PDT000027915.2':0.0379592)'':0.00761304,('PDT000128824.2':0.0453782,'PDT000652968.1':0.037139)'':0.00780206)'':0.00842223)'':0.0041259)'':0.00704588)'':0.00368433,('PDT000644879.1':0.0573627,(('PDT000714553.1':0.0417796,('PDT000669188.1':0.0365214,('PDT000333671.1':0.0241713,'PDT000395503.1':0.0191183)'':0.0213011)'':0.0109443)'':-0.00346049,(('PDT000644952.1':0.0587918,('PDT000281215.2':0.0479684,('PDT000333780.1':0.0259054,'PDT000393084.1':0.0323804)'':0.00574863)'':0.0110827)'':-0.000536161,('PDT000604367.1':0.0530467,('PDT000497355.1':0.042769,('PDT000407297.1':0.0428312,(('PDT000566137.1':0.0287948,('PDT000155177.2':0.0305504,'PDT000060267.2':0.0188067)'':0.00574896)'':0.00345835,('PDT000395419.1':0.0275972,('PDT000566164.1':0.0247829,('PDT000645130.1':0.0202485,'PDT000645559.2':0.0166092)'':0.00627481)'':0.00379842)'':0.00546566)'':0.00129562)'':0.00233864)'':0.00898828)'':-0.00385059)'':0.00478001)'':0.00820316)'':0.00416091)'':0.0118426)'':0.0205159)'':0.00526633,((('PDT000550325.1':0.0361474,'PDT000682541.1':0.0438022)'':0.00838098,('PDT000397587.1':0.0193747,('PDT000024328.2':0.0156991,'PDT000661814.1':0.0097449)'':0.0103621)'':0.0388039)'':0.0244244,((('PDT000163826.2':0.0552595,'PDT000163828.2':0.0603705)'':0.015307,('PDT000201008.2':0.0276505,'PDT000394868.1':0.0271854)'':0.0460908)'':0.00624956,((('PDT000645668.2':0.0726133,('PDT000206251.3':0.0481037,('PDT000485399.1':0.0268034,('PDT000226171.2':0.030175,'PDT000639555.1':0.0326966)'':0.00642933)'':0.0141843)'':0.0044834)'':0.0212441,(('PDT000124864.2':0.0648297,'PDT000468851.1':0.0648433)'':0.000469371,('PDT000468832.1':0.0640191,('PDT000633621.1':0.0279308,('PDT000720473.1':0.00644059,'PDT000720474.1':0.00778681)'':0.0304132)'':0.019513)'':0.00254719)'':0.0154404)'':0.00258899,(('PDT000417097.1':0.0726692,'PDT000720461.1':0.0609938)'':0.0132694,('PDT000329056.1':0.0969013,(('PDT000204788.1':0.0624776,('PDT000720469.1':0.0716194,'PDT000191590.1':0.0654096)'':0.00395989)'':0.00671315,((('PDT000353623.1':0.0867782,('PDT000675017.1':0.077897,'PDT000271762.1':0.074207)'':0.0102208)'':0.00365426,(('PDT000468885.1':0.0527977,('PDT000468901.1':0.0514633,('PDT000468867.1':0.0492793,'PDT000468909.1':0.048392)'':0.00964652)'':0.00315806)'':0.0106552,(('PDT000346896.1':0.0645592,'PDT000468864.1':0.0640098)'':0.00895758,('PDT000201016.2':0.0608594,('PDT000468888.1':0.070155,('PDT000206244.3':0.0679647,('PDT000425554.1':0.0567138,('PDT000716815.1':0.047371,'PDT000468835.1':0.0510864)'':0.00297499)'':0.00564788)'':0.0050189)'':0.00362818)'':0.00621552)'':-0.000285436)'':0.00372252)'':-0.00216805,((('PDT000645576.2':0.0494738,('PDT000208428.2':0.0299545,'PDT000636387.1':0.0321627)'':0.0128272)'':0.00617691,('PDT000669190.1':0.0416448,('PDT000524498.1':0.040043,('PDT000285434.2':0.0202752,'PDT000548086.1':0.015386)'':0.00741771)'':0.00996952)'':0.0106946)'':0.00765642,('PDT000469958.1':0.0746992,('PDT000594969.1':0.0719538,('PDT000636384.1':0.073301,(('PDT000286096.1':0.0647061,('PDT000124600.1':0.0551403,'PDT000468852.1':0.0449277)'':0.00431442)'':0.00992991,('PDT000488280.1':0.0546308,((('PDT000307443.2':0.031846,'PDT000675029.1':0.0460722)'':0.00319015,('PDT000632990.1':0.0466083,'PDT000653019.1':0.0429268)'':0.00670675)'':0.00608641,('PDT000346860.1':0.054066,(('PDT000430777.1':0.0383392,('PDT000633080.1':0.037015,'PDT000290011.1':0.0338321)'':0.00682841)'':0.00618093,('PDT000604970.1':0.0393503,('PDT000468868.1':0.02608,'PDT000468925.1':0.0329522)'':0.0108124)'':0.00422431)'':0.00223884)'':0.0016233)'':0.008316)'':-0.00571436)'':0.00868035)'':0.000973382)'':0.00461511)'':-0.0117795)'':0.00606936)'':0.00328022)'':0.00786645)'':-0.00387874)'':-0.00302528)'':0.0114093)'':0.00165475)'':0.00365773)'':0.00093716)'':0.00484889)'':0.00754209)'':0.000543533)'':-0.000451534,((((('PDT000251958.1':0.0843696,'PDT000024645.1':0.0715584)'':0.00750662,('PDT000321320.1':0.0742931,('PDT000319745.1':0.0651722,'PDT000102816.3':0.0720338)'':0.0216204)'':0.0082034)'':0.00998241,('PDT000405302.1':0.102037,(('PDT000354929.1':0.07754,(('PDT000286926.2':0.0286586,'PDT000422151.1':0.0321476)'':0.00521677,('PDT000319744.1':0.0261579,('PDT000367305.1':0.0221738,'PDT000289945.1':0.0220402)'':0.00403106)'':0.00419248)'':0.0291816)'':0.0181513,(('PDT000557207.1':0.0436919,'PDT000566136.1':0.041885)'':0.00791761,('PDT000045618.1':0.0564794,('PDT000240668.2':0.0322713,('PDT000150234.2':0.0235984,'PDT000095829.2':0.0248313)'':0.0195971)'':0.00906128)'':0.00795993)'':0.019011)'':0.010982)'':0.00578641)'':0.00336658,((('PDT000178562.2':0.0506448,'PDT000397790.1':0.0535592)'':0.0237616,('PDT000610212.1':0.0547689,('PDT000661095.1':0.02884,('PDT000032370.1':0.0156251,'PDT000032398.1':0.00802518)'':0.0114525)'':0.0196876)'':0.00854356)'':0.0268779,((('PDT000285339.1':0.0712402,('PDT000394990.1':0.056282,('PDT000068831.2':0.0548512,'PDT000626562.1':0.0509038)'':0.00494899)'':0.0157701)'':-0.0031046,('PDT000369037.1':0.066155,('PDT000509495.1':0.0487504,('PDT000025249.2':0.0357497,'PDT000577091.1':0.0312927)'':0.00862202)'':0.00348375)'':0.0182171)'':0.0400425,(('PDT000661137.1':0.0737663,(('PDT000518001.1':0.0419577,('PDT000191808.2':0.0361338,'PDT000645036.1':0.0336381)'':0.00730237)'':0.0208191,(('PDT000060709.2':0.0398495,'PDT000600746.1':0.0504398)'':0.00720347,('PDT000644874.1':0.0251518,'PDT000502674.1':0.0202176)'':0.0185267)'':0.00776468)'':0.0146601)'':0.00505586,('PDT000543294.1':0.100694,(('PDT000288569.2':0.0486229,('PDT000556885.1':0.0338955,'PDT000718862.1':0.0349528)'':0.0133968)'':0.000621467,(('PDT000362866.1':0.0383502,('PDT000248788.1':0.0107454,('PDT000147457.3':0.0135092,('PDT000565987.1':0.00791249,'PDT000502307.1':0.00970291)'':0.00376548)'':0.00143005)'':0.0265034)'':0.0124546,(('PDT000281482.2':0.0193656,'PDT000443390.1':0.0194292)'':0.0274994,('PDT000095743.2':0.0331211,('PDT000095740.2':0.0361317,'PDT000095819.2':0.0451207)'':0.00591302)'':0.0103033)'':0.00579277)'':0.00183423)'':0.0337175)'':-0.000120095)'':0.0102305)'':0.00291466)'':0.0024412)'':0.00139799,((((('PDT000275549.1':0.0638513,('PDT000251830.1':0.0423227,('PDT000565925.1':0.0218523,'PDT000395285.1':0.014007)'':0.0326947)'':0.0126012)'':0.0289366,(('PDT000275036.2':0.0737213,'PDT000509490.1':0.0816027)'':0.00995848,('PDT000305488.2':0.0933437,(('PDT000264630.2':0.0695154,'PDT000566064.1':0.0693786)'':0.00932673,('PDT000341204.1':0.0668861,('PDT000397895.1':0.0630626,'PDT000395234.1':0.0877574)'':0.000928425)'':0.0122386)'':0.00347768)'':0.00110071)'':0.00534102)'':0.00359864,((('PDT000307410.3':0.0872336,'PDT000396746.1':0.0873924)'':0.00544941,(('PDT000283185.1':0.0541908,('PDT000285932.2':0.0418734,'PDT000661337.1':0.048992)'':0.0109945)'':0.0281176,('PDT000360977.1':0.072828,('PDT000027736.2':0.0472871,'PDT000249937.1':0.0519181)'':0.0195014)'':0.00960454)'':0.00905365)'':-0.00137242,(('PDT000312623.2':0.0752318,('PDT000285630.2':0.0715223,('PDT000264655.2':0.0238944,'PDT000095692.2':0.0316776)'':0.0469202)'':0.00793032)'':0.00308736,('PDT000251725.1':0.0641653,('PDT000718840.1':0.0645522,('PDT000397689.1':0.0117986,'PDT000519108.1':0.0170552)'':0.0410939)'':0.00281447)'':0.021981)'':0.00712814)'':0.00344581)'':0.00577353,('PDT000276019.1':0.152737,((('PDT000024331.2':0.110478,('PDT000661121.1':0.0872695,('PDT000394998.1':0.0416234,'PDT000525373.1':0.0520541)'':0.0354133)'':0.0172059)'':0.00721941,(('PDT000346606.1':0.0579456,('PDT000201015.2':0.0279734,'PDT000557786.2':0.0334079)'':0.0169117)'':0.0569857,(('PDT000203402.2':0.0669564,'PDT000066400.2':0.0718856)'':0.0255009,(('PDT000203400.2':0.076169,('PDT000543302.1':0.0443556,'PDT000395169.1':0.0646164)'':0.019809)'':0.0134808,('PDT000228195.1':0.0863057,('PDT000644846.1':0.0747584,('PDT000497215.1':0.0728415,('PDT000200979.2':0.0609095,'PDT000201006.2':0.0587755)'':0.00980001)'':0.00230639)'':0.010993)'':0.00412661)'':0.00416749)'':0.0081877)'':0.000905625)'':-0.00317056,(((('PDT000589907.1':0.0747441,('PDT000399952.1':0.0624521,('PDT000556962.1':0.0371313,'PDT000228228.1':0.037597)'':0.0135938)'':0.00990627)'':0.002863,('PDT000661157.1':0.106405,(('PDT000544285.1':0.0525921,'PDT000566003.1':0.0447445)'':0.0142732,('PDT000444907.1':0.0730133,('PDT000642579.1':0.0721933,(('PDT000325584.1':0.0471127,'PDT000408765.1':0.0506165)'':0.00196428,('PDT000372177.1':0.0212617,'PDT000716907.1':0.0238489)'':0.0135771)'':0.0100593)'':0.0011657)'':0.00159478)'':0.0238357)'':-0.00357237)'':0.0102764,((('PDT000594977.1':0.0551925,'PDT000703199.1':0.0408499)'':0.049401,('PDT000203407.2':0.0868209,('PDT000687415.1':0.0707331,'PDT000346901.1':0.0834839)'':0.00394006)'':0.00543691)'':0.0155444,('PDT000251397.1':0.127755,((('PDT000636357.1':0.0858429,('PDT000346966.1':0.0802599,'PDT000354284.1':0.0586181)'':0.0127711)'':0.00467564,('PDT000600655.1':0.085179,('PDT000215758.2':0.0709339,('PDT000566317.1':0.0615611,'PDT000654278.1':0.0565169)'':0.00912658)'':0.01173)'':0.00249748)'':0.00611374,(('PDT000510151.2':0.0866724,(('PDT000536888.1':0.0491042,'PDT000251474.1':0.0437151)'':0.00540653,(('PDT000646907.1':0.0317071,'PDT000713770.1':0.039832)'':0.0216041,('PDT000688630.1':0.0476949,('PDT000167684.4':0.0543761,('PDT000203145.2':0.0265305,'PDT000346492.1':0.0179711)'':0.0213475)'':0.00428294)'':0.00609957)'':0.00121917)'':0.0115977)'':0.0035407,(('PDT000251864.1':0.0819316,('PDT000588654.1':0.0532612,('PDT000204871.2':0.0524363,'PDT000216022.1':0.0471611)'':0.0108506)'':0.00637044)'':0.0138373,(('PDT000212784.1':0.0863997,('PDT000343779.1':0.0666571,'PDT000032383.1':0.0716609)'':0.0107403)'':-0.000909621,(('PDT000720176.1':0.0834312,('PDT000426313.1':0.0557499,('PDT000520680.1':0.0371985,'PDT000661151.1':0.0606599)'':0.0105616)'':0.0111546)'':0.00330016,(('PDT000081763.1':0.0529071,('PDT000572833.1':0.0548925,'PDT000428836.1':0.0503355)'':0.00234144)'':0.000714868,('PDT000174335.1':0.0520032,('PDT000289944.1':0.0508873,('PDT000710411.1':0.0363571,'PDT000346894.1':0.0416776)'':0.0018818)'':0.00784975)'':0.00609677)'':0.0081197)'':0.00604147)'':0.000991601)'':0.00115061)'':0.00328617)'':0.0145993)'':-0.0093524)'':-0.000778042)'':0.00110009,((('PDT000251769.1':0.107102,('PDT000188120.2':0.0821804,('PDT000250067.1':0.0904777,('PDT000025253.2':0.0849981,'PDT000604478.1':0.0680569)'':0.0108823)'':0.0137882)'':0.00454345)'':-0.00331824,('PDT000150088.3':0.0796227,('PDT000640337.1':0.0919749,('PDT000095814.2':0.0718555,(('PDT000687440.1':0.033447,('PDT000600800.1':0.0251435,'PDT000662250.1':0.024959)'':0.00800365)'':0.0033875,(('PDT000346105.1':0.0203247,('PDT000397721.1':0.0199092,'PDT000395372.1':0.0197609)'':0.00644518)'':0.00277171,('PDT000519824.1':0.0249947,('PDT000565899.1':0.0264785,('PDT000565930.1':0.0138891,'PDT000619348.1':0.0163483)'':0.00206206)'':0.00146716)'':0.00487049)'':0.00636204)'':0.0363662)'':0.00723377)'':0.0045475)'':0.00276531)'':0.00969032,((('PDT000583451.1':0.107032,'PDT000583453.1':0.0767029)'':0.0262842,(('PDT000556988.1':0.0732984,'PDT000202060.2':0.0852336)'':0.00866166,(('PDT000213218.3':0.0643988,('PDT000200981.2':0.0498414,'PDT000425731.1':0.0638456)'':0.0018717)'':0.00412573,('PDT000626955.1':0.0369267,(('PDT000583426.1':0.00294919,'PDT000583456.1':0.00313378)'':0.0162032,('PDT000583452.1':0.00730787,('PDT000583444.1':0.00426401,'PDT000583455.1':0.00392862)'':0.00205616)'':0.00606614)'':0.0176133)'':0.0176778)'':0.0214186)'':0.00910063)'':-0.00228353,(((('PDT000174305.1':0.00721108,'PDT000174312.1':0.0118839)'':0.0601611,('PDT000574938.1':0.0590774,('PDT000427561.1':0.0432765,'PDT000248693.1':0.0412063)'':0.0088442)'':0.0134739)'':0.00122007,(('PDT000399706.1':0.0544052,('PDT000188118.2':0.0533219,'PDT000638776.1':0.0585711)'':0.00193877)'':0.0163759,('PDT000426546.1':0.0587732,('PDT000631539.1':0.0503187,('PDT000252682.1':0.028685,('PDT000251699.1':0.0233699,'PDT000252464.1':0.0352681)'':0.00490336)'':0.024178)'':0.0161612)'':0.000934245)'':0.00456)'':0.00363557,((('PDT000313334.2':0.0278669,('PDT000288615.2':0.0229814,'PDT000428391.1':0.0229055)'':0.0053161)'':0.0245591,('PDT000524050.1':0.0648169,('PDT000319021.1':0.0378191,('PDT000188091.2':0.0287243,'PDT000491101.1':0.0255514)'':0.0114923)'':0.00608148)'':0.000465324)'':0.00725492,('PDT000526208.1':0.0868486,('PDT000212921.2':0.0610665,('PDT000251669.1':0.0698581,('PDT000346581.1':0.0553413,(('PDT000394850.1':0.0387942,('PDT000226151.1':0.043557,'PDT000252499.1':0.031815)'':0.00304675)'':0.00674772,('PDT000535342.1':0.0371664,('PDT000411059.1':0.0149865,'PDT000485486.1':0.0195744)'':0.00887233)'':0.0058791)'':0.0104935)'':0.00400445)'':0.0121504)'':0.00119633)'':-0.00395912)'':0.0167492)'':0.00587396)'':0.00591158)'':0.000512034)'':0.00212616)'':0.0376482)'':-0.0388624)'':0.00191073,((('PDT000203419.2':0.120512,('PDT000494554.1':0.0178404,'PDT000494560.1':0.0167239)'':0.0852266)'':0.00873386,(('PDT000242496.1':0.038722,'PDT000242510.1':0.0443698)'':0.0150332,(('PDT000626549.2':0.0256194,('PDT000414403.2':0.0148107,('PDT000242509.1':0.00698845,'PDT000242519.1':0.00733705)'':0.0086602)'':0.0170369)'':0.00602442,('PDT000242524.1':0.0161745,('PDT000242517.1':0.00302875,('PDT000242513.1':0.00358379,'PDT000250698.1':0.0030975)'':0.000961353)'':0.0071499)'':0.0160468)'':0.0146918)'':0.0667383)'':0.00370463,((('PDT000645530.2':0.0619674,('PDT000398010.1':0.0445907,('PDT000034497.2':0.0172632,'PDT000026911.2':0.0232853)'':0.00908433)'':0.00887649)'':0.0574025,(('PDT000006339.2':0.0729725,('PDT000556912.1':0.0719892,('PDT000394851.1':0.0674829,'PDT000249922.1':0.0602191)'':0.00541984)'':0.00665293)'':0.0041511,(('PDT000200117.2':0.0835931,'PDT000468900.1':0.0629859)'':0.0110901,(('PDT000399702.1':0.0703253,('PDT000399612.1':0.0308936,'PDT000624691.1':0.0326587)'':0.0250165)'':-0.000443371,('PDT000557317.1':0.0334918,('PDT000384288.1':0.0365536,('PDT000196662.2':0.021882,'PDT000420674.1':0.0157487)'':0.0134648)'':0.0143927)'':0.0216547)'':0.011659)'':0.00505111)'':0.00846675)'':0.00734253,((('PDT000161881.3':0.0768889,'PDT000502493.1':0.0635411)'':0.0205242,(('PDT000174358.1':0.0733493,('PDT000221055.1':0.0235344,'PDT000221059.1':0.024108)'':0.016506)'':0.0360432,('PDT000577083.1':0.0788957,('PDT000426435.1':0.0408414,(('PDT000349626.1':0.0337095,'PDT000631946.1':0.0275865)'':0.00794113,('PDT000438127.1':0.0343058,'PDT000658723.1':0.0295429)'':0.00269382)'':0.0033031)'':0.0344889)'':-0.000530835)'':0.0109377)'':0.00393653,(('PDT000273356.3':0.119228,('PDT000603063.2':0.0866405,'PDT000346586.1':0.0922945)'':0.0159042)'':0.00276636,((('PDT000318976.1':0.0977895,('PDT000346541.1':0.0841455,(('PDT000589386.1':0.0636371,'PDT000290372.4':0.0765309)'':0.00359541,('PDT000669026.1':0.0825059,('PDT000669007.1':0.0249998,'PDT000225237.1':0.0288795)'':0.0403769)'':0.00993456)'':0.00351716)'':0.00292688)'':0.0153345,(('PDT000428437.1':0.030225,'PDT000517933.1':0.0229686)'':0.079744,((('PDT000662567.1':0.0477963,'PDT000662568.1':0.0507832)'':0.0403028,('PDT000014127.2':0.0825886,'PDT000015757.2':0.0930594)'':0.00710541)'':0.0132875,(('PDT000252600.1':0.0562728,('PDT000327435.1':0.0369131,'PDT000600857.1':0.0356253)'':0.027344)'':0.0147097,(('PDT000188122.2':0.0660014,('PDT000341225.1':0.0450372,'PDT000468248.1':0.0515646)'':0.0103487)'':0.00554088,(('PDT000095736.2':0.0557733,'PDT000228014.1':0.0567747)'':0.0106369,('PDT000631573.1':0.0585553,(('PDT000095795.2':0.0432966,('PDT000250190.1':0.041315,('PDT000380302.1':0.0260781,'PDT000251493.1':0.0289435)'':0.00460308)'':0.0135715)'':0.00648192,('PDT000620192.1':0.0351545,('PDT000386712.1':0.0371636,('PDT000104801.2':0.0193541,'PDT000501060.1':0.0235567)'':0.00728261)'':0.00520118)'':0.0153859)'':0.00262023)'':0.00208603)'':0.00435864)'':0.0021241)'':0.010035)'':0.00708318)'':-0.00401912)'':-0.00169996,((('PDT000651544.1':0.0816069,('PDT000429700.1':0.0807533,('PDT000228180.1':0.0567407,('PDT000425899.1':0.0347136,'PDT000248835.1':0.0404904)'':0.00480882)'':0.0101603)'':0.0101892)'':0.0217914,('PDT000664988.1':0.0856744,('PDT000006906.2':0.0641009,(('PDT000664963.1':0.0490402,'PDT000664969.1':0.0411222)'':0.0375578,('PDT000228273.1':0.0640052,('PDT000205441.1':0.0551344,('PDT000556990.1':0.0342739,'PDT000250062.1':0.03787)'':0.0161897)'':0.00810291)'':0.00162164)'':0.00335892)'':0.00779765)'':0.0121775)'':-0.00174478,('PDT000015704.3':0.117208,(((('PDT000718882.1':0.0590373,'PDT000719008.1':0.0587817)'':0.0175912,(('PDT000524082.1':0.0169466,'PDT000524087.1':0.0175769)'':0.00292383,('PDT000524043.1':0.00964717,('PDT000524048.1':0.00889286,(('PDT000524044.1':0.00689751,'PDT000524068.1':0.00740519)'':0.000258822,('PDT000524040.1':0.00647363,('PDT000524046.1':0.00557294,('PDT000524058.1':0.00387611,'PDT000267185.1':0.00231955)'':0.00159751)'':0.00137413)'':0.000169567)'':0.00127352)'':0.000157367)'':0.00612515)'':0.0530763)'':-0.000313358,('PDT000702571.1':0.100119,(('PDT000252526.1':0.0596734,('PDT000396803.1':0.0266922,'PDT000345942.1':0.0428281)'':0.00769488)'':0.0273953,('PDT000196660.2':0.0793451,('PDT000439429.1':0.0541451,(('PDT000277721.2':0.00566899,'PDT000305525.2':0.00610981)'':0.0319737,(('PDT000311259.2':0.0358071,'PDT000384289.1':0.039495)'':0.00620784,('PDT000081728.1':0.0283795,'PDT000228367.1':0.035256)'':0.00255476)'':0.00253674)'':0.0127577)'':0.00626775)'':0.00836841)'':0.0135277)'':-0.0114875)'':0.000592027,(('PDT000482525.1':0.0811902,('PDT000652421.1':0.0371609,'PDT000652566.1':0.0256653)'':0.0642222)'':0.00708524,((('PDT000251967.1':0.0858652,('PDT000502417.1':0.0769103,'PDT000395341.1':0.0827477)'':0.0123563)'':0.00251868,(('PDT000275054.4':0.0536363,('PDT000337205.1':0.0333181,'PDT000364418.1':0.035106)'':0.0242961)'':0.0137083,(('PDT000357219.2':0.0373539,'PDT000029743.3':0.0291632)'':0.0403006,('PDT000672710.1':0.0636927,'PDT000497965.1':0.0708273)'':0.0073003)'':0.00306823)'':0.00467675)'':0.00124143,((('PDT000617960.1':0.0566237,('PDT000432504.1':0.0480542,'PDT000141601.1':0.0524928)'':0.00941332)'':0.00977973,('PDT000136024.2':0.0802246,('PDT000131782.2':0.0644458,'PDT000293549.1':0.0540852)'':0.0070729)'':0.00428603)'':0.00351869,((('PDT000408771.1':0.0247744,'PDT000661368.1':0.0301912)'':0.040638,('PDT000432499.1':0.0411306,'PDT000250201.1':0.0518138)'':0.0214392)'':0.00244231,((('PDT000661159.1':0.054653,('PDT000350423.1':0.0268794,'PDT000646913.1':0.0200298)'':0.0109289)'':0.0233297,(('PDT000384287.1':0.0387196,'PDT000720216.1':0.0345667)'':0.0234939,('PDT000502254.1':0.0708653,('PDT000192050.3':0.0588814,('PDT000069956.1':0.0554712,(('PDT000204892.2':0.0383148,'PDT000230494.1':0.0362804)'':0.00810492,(('PDT000470050.1':0.0340009,'PDT000581020.1':0.0426586)'':0.00980588,('PDT000644950.1':0.0493687,('PDT000196604.2':0.0371459,('PDT000312634.2':0.0278459,'PDT000677994.1':0.036395)'':0.00429757)'':0.0046413)'':0.00277174)'':0.00111176)'':0.00133049)'':0.00214333)'':0.00392984)'':-0.00184114)'':0.00358978)'':0.00799811,(('PDT000619592.1':0.0612517,(('PDT000465157.1':0.0171102,'PDT000571495.1':0.0158687)'':0.034077,('PDT000129814.2':0.045425,('PDT000343959.1':0.0206829,('PDT000344002.1':0.0241129,('PDT000343946.1':0.0123396,'PDT000178922.1':0.0155374)'':0.00139949)'':0.00547337)'':0.00554607)'':0.021516)'':0.0133206)'':0.00561086,(('PDT000109410.2':0.0705369,'PDT000652250.1':0.0876931)'':0.00752056,('PDT000557353.1':0.0512724,(('PDT000433711.1':0.0378725,'PDT000318986.1':0.0341561)'':0.003382,('PDT000081742.1':0.0400754,('PDT000713577.1':0.0271388,('PDT000501057.1':0.0232119,('PDT000485222.1':0.0182322,'PDT000252687.1':0.0193773)'':0.00583923)'':0.00530484)'':0.00304701)'':0.00148347)'':0.00786898)'':0.0165216)'':-0.000339148)'':0.00179533)'':0.00170349)'':0.00204119)'':0.00364188)'':0.00385052)'':-0.007325)'':0.0180952)'':-0.00975645)'':-0.00158736)'':0.0147329)'':-0.00970007)'':0.000271306)'':0.0108816)'':-0.00916767)'':0.00330138)'':0.00324202)'':0.00812184)'':0.0400117)'':-0.0390596,(((('PDT000356789.1':0.0763717,'PDT000086878.2':0.0847743)'':0.0378193,(('PDT000095796.2':0.0315926,'PDT000642255.1':0.0306037)'':0.0180768,(('PDT000590199.1':0.0289553,'PDT000687489.1':0.0310156)'':0.00166179,('PDT000685902.1':0.0309904,'PDT000359698.1':0.0235266)'':0.00823401)'':0.0152122)'':0.0343417)'':0.0570656,((((('PDT000309305.2':0.0910495,'PDT000709976.1':0.0659125)'':0.0131533,(('PDT000320776.1':0.0949586,'PDT000556887.1':0.0760004)'':0.0018117,(('PDT000473251.1':0.0609176,('PDT000312224.2':0.0339479,'PDT000397677.1':0.0279689)'':0.045704)'':-0.000566477,('PDT000251440.1':0.0600413,('PDT000557160.1':0.0422309,('PDT000380290.1':0.0333643,'PDT000248776.1':0.0318173)'':0.0048607)'':0.0190887)'':0.0132414)'':0.0136339)'':0.00748229)'':0.00166785,(('PDT000631566.1':0.0660502,('PDT000690222.1':0.0755156,('PDT000397714.1':0.0343979,('PDT000329826.1':0.0239074,'PDT000346899.1':0.0231641)'':0.0100181)'':0.0175862)'':0.0183653)'':0.0204148,(('PDT000651093.1':0.0892445,('PDT000425949.1':0.0592542,'PDT000346590.1':0.0636158)'':0.0261425)'':0.00625336,(('PDT000522633.1':0.063292,'PDT000111115.1':0.069009)'':0.0151348,((('PDT000285782.2':0.0634286,'PDT000081729.1':0.0902054)'':0.0120578,('PDT000251672.1':0.0386723,('PDT000142179.2':0.0294408,'PDT000251671.1':0.0126918)'':0.0204266)'':0.023587)'':0.00400558,('PDT000433666.1':0.0763176,(('PDT000659710.1':0.0311976,'PDT000253673.1':0.0375433)'':0.0252705,('PDT000409810.1':0.0463667,('PDT000550383.1':0.0241793,'PDT000672200.1':0.0479476)'':0.0257072)'':0.00122537)'':0.0116738)'':2.20219e-05)'':0.00582025)'':0.000861181)'':0.00276167)'':0.00457845)'':0.000652707,((('PDT000346893.1':0.0820218,('PDT000497341.1':0.0612866,('PDT000332414.1':0.027714,'PDT000343919.1':0.0362492)'':0.0161758)'':0.00818274)'':0.0226699,(('PDT000651807.2':0.021737,'PDT000672859.1':0.0259101)'':0.0561562,('PDT000660092.2':0.061822,('PDT000619512.2':0.0546984,('PDT000481059.2':0.0124224,'PDT000497552.2':0.0152924)'':0.0350899)'':0.0068047)'':0.0241366)'':0.00284433)'':0.0222708,((('PDT000249861.1':0.0634669,('PDT000535023.1':0.0628512,('PDT000411277.1':0.0637312,('PDT000063155.2':0.0474238,'PDT000140472.2':0.0341604)'':0.00813273)'':0.0112068)'':0.00583025)'':0.00550481,('PDT000620269.1':0.0781999,(('PDT000668178.2':0.0311907,('PDT000376008.2':0.0306127,('PDT000548724.2':0.0122392,'PDT000682081.1':0.0130798)'':0.00680505)'':0.00546126)'':0.010736,((('PDT000643651.2':0.0298998,('PDT000502452.1':0.0261,('PDT000709079.1':0.0193261,('PDT000144062.3':0.0158608,'PDT000048210.3':0.016219)'':0.00538009)'':0.00906192)'':0.00370608)'':0.0103655,('PDT000168317.3':0.0463057,(('PDT000267868.3':0.0245665,'PDT000540198.2':0.0260305)'':0.00448988,('PDT000148495.2':0.0179959,'PDT000063290.3':0.0223457)'':0.00649422)'':0.0107634)'':-0.000258698)'':0.00168676,(('PDT000086300.3':0.0386268,('PDT000400040.2':0.0129293,'PDT000248655.1':0.0114055)'':0.00991553)'':0.0166989,(('PDT000343441.2':0.0321302,'PDT000650911.2':0.022725)'':0.00684596,(('PDT000189643.3':0.0310412,'PDT000131099.3':0.0342526)'':0.00561413,(('PDT000502301.1':0.0251448,('PDT000502514.1':0.0136423,'PDT000502515.1':0.0144108)'':0.0192372)'':0.00495786,(('PDT000203852.3':0.0191283,'PDT000708936.1':0.018416)'':0.00252551,('PDT000486045.2':0.0286031,('PDT000289661.3':0.0109224,('PDT000167944.3':0.00580232,'PDT000279484.3':0.00702838)'':0.00410437)'':0.00819311)'':0.00282418)'':0.00908956)'':0.00480458)'':0.00162692)'':0.00265878)'':0.00345756)'':0.00623127)'':0.0323367)'':0.00194265)'':-0.00629432,('PDT000161674.3':0.0942433,(('PDT000646087.2':0.0719011,(('PDT000576801.2':0.0409837,('PDT000248677.1':0.0163134,'PDT000248745.1':0.0094834)'':0.0183575)'':0.00731815,(('PDT000486526.1':0.0247249,('PDT000288598.3':0.013414,('PDT000288541.3':0.0104263,('PDT000288530.3':0.00730112,'PDT000288561.3':0.00634018)'':0.00415505)'':0.00273169)'':0.00574756)'':0.0124132,(('PDT000288576.3':0.0283504,(('PDT000499181.2':0.00546615,'PDT000712835.1':0.00419809)'':0.0119824,('PDT000336562.2':0.0162446,('PDT000366233.2':0.0123547,'PDT000096876.3':0.0126898)'':0.00596752)'':0.00160827)'':0.00342398)'':0.00789672,('PDT000140223.3':0.0348715,(('PDT000098728.3':0.0295381,'PDT000557177.2':0.0283905)'':0.000177575,('PDT000498120.1':0.0317026,('PDT000096868.3':0.0230077,('PDT000233978.3':0.0171751,'PDT000077520.1':0.0165527)'':0.00215261)'':0.00521498)'':0.00527794)'':0.00536106)'':-0.000123896)'':0.00112052)'':0.00473058)'':0.0346221)'':-0.0084358,((('PDT000201190.3':0.0232218,'PDT000480433.2':0.0378032)'':0.0105526,(('PDT000546192.2':0.0310143,'PDT000580719.2':0.0400245)'':0.0126872,(('PDT000486064.2':0.0370793,'PDT000689063.1':0.0289639)'':0.000537608,('PDT000451105.2':0.0356256,('PDT000638753.2':0.0313237,('PDT000257211.3':0.0296619,'PDT000710374.1':0.0247945)'':0.00544997)'':0.00213093)'':0.0018128)'':0.00289326)'':0.00999757)'':0.0155602,(('PDT000548294.2':0.0520529,'PDT000710824.1':0.0389182)'':0.0216541,(('PDT000227317.3':0.0401226,('PDT000248647.1':0.0235901,('PDT000501614.2':0.0224017,'PDT000008729.2':0.0196663)'':0.00922266)'':0.00769106)'':0.00549594,(('PDT000502453.1':0.0337794,('PDT000365733.2':0.0246579,'PDT000542085.2':0.0302375)'':0.00630684)'':0.00465926,((('PDT000496792.2':0.030365,'PDT000040439.5':0.0273273)'':0.0073975,('PDT000081062.1':0.0258798,('PDT000248634.1':0.0154977,'PDT000251867.1':0.0144855)'':0.00268459)'':0.00845544)'':0.00818768,(('PDT000502278.1':0.00767167,'PDT000498003.1':0.00724333)'':0.0150776,('PDT000162970.4':0.0270847,('PDT000081060.1':0.0215676,('PDT000203877.3':0.0202872,'PDT000258413.3':0.0196461)'':0.00288029)'':0.00468301)'':0.00160801)'':0.00254208)'':0.00263785)'':0.00295218)'':0.0348145)'':-0.0101077)'':0.00338105)'':0.0235446)'':-0.0126428)'':0.0198111)'':0.000779898)'':0.00394794,(('PDT000557358.1':0.0986736,(('PDT000230560.2':0.0875111,'PDT000568667.1':0.0783629)'':0.0132665,(('PDT000709942.1':0.0304867,'PDT000092541.1':0.0364137)'':0.0633528,('PDT000553118.1':0.0881379,('PDT000501585.1':0.0779269,('PDT000386724.1':0.072156,('PDT000485420.1':0.0427554,'PDT000485547.1':0.0620956)'':0.012379)'':0.0220817)'':0.0140031)'':0.00460106)'':0.0023618)'':0.0137609)'':0.0230414,((((('PDT000720251.1':0.0568187,('PDT000150871.1':0.0330409,'PDT000150893.1':0.0348232)'':0.0377913)'':0.00584881,('PDT000502887.1':0.0645327,('PDT000150874.1':0.0600836,('PDT000346078.1':0.0503222,('PDT000584499.1':0.0314781,'PDT000488296.1':0.0404728)'':0.0187748)'':0.00636948)'':0.00253983)'':0.0011185)'':0.0259442,(('PDT000250189.1':0.0889639,('PDT000502314.1':0.0646333,('PDT000150881.1':0.0694929,(('PDT000337133.1':0.0265886,'PDT000494408.1':0.0305067)'':0.00321423,('PDT000150872.1':0.0254807,'PDT000252608.1':0.0319285)'':0.00432612)'':0.0190596)'':0.00382324)'':0.0158008)'':-0.00637929,(('PDT000502648.1':0.0418437,('PDT000109416.2':0.0305,'PDT000109425.2':0.0284437)'':0.0146987)'':0.0248217,(('PDT000109408.2':0.0534733,'PDT000510037.1':0.0445829)'':0.00991502,('PDT000228077.1':0.0315821,('PDT000399998.1':0.0287701,'PDT000249976.1':0.0339865)'':0.00790971)'':0.00805216)'':0.00733536)'':0.00723874)'':0.0110738)'':0.0083102,((('PDT000095823.2':0.051985,'PDT000390712.1':0.0455014)'':0.0503622,(('PDT000201854.2':0.0441682,'PDT000251511.1':0.0513805)'':0.0271442,('PDT000253667.1':0.0689425,('PDT000594506.1':0.0525402,('PDT000147429.3':0.0459933,'PDT000095815.2':0.036813)'':0.00649549)'':0.0132523)'':0.0148407)'':0.0101759)'':0.00023293,('PDT000283585.2':0.130332,(('PDT000006904.2':0.0702899,(('PDT000150756.2':0.0573622,'PDT000258377.2':0.0554388)'':0.00290697,('PDT000403082.1':0.056074,'PDT000556925.1':0.058009)'':0.00523453)'':0.0133846)'':0.0100914,(('PDT000109463.2':0.0307184,'PDT000600524.1':0.0432108)'':0.0431401,('PDT000518587.1':0.0640698,(('PDT000109495.2':0.026084,'PDT000218936.1':0.0366317)'':0.0329773,(('PDT000228015.1':0.0533688,('PDT000502430.1':0.0514832,'PDT000502705.1':0.0459542)'':0.00491498)'':0.00736928,(('PDT000252646.1':0.0441916,'PDT000394957.1':0.0530869)'':0.00174484,('PDT000631845.1':0.0442712,('PDT000231464.2':0.0258038,'PDT000482511.1':0.020669)'':0.0232564)'':0.00899755)'':0.00251605)'':0.00387319)'':0.00679847)'':0.0041496)'':-0.00472812)'':0.0164176)'':-0.0104973)'':0.00655263)'':0.00437615,((('PDT000228339.1':0.0804613,'PDT000251893.1':0.0707767)'':0.0286126,('PDT000250141.1':0.101075,(('PDT000556905.1':0.0642775,'PDT000251879.1':0.0467475)'':0.028838,(('PDT000589286.1':0.0527232,('PDT000444953.1':0.0423655,'PDT000604153.2':0.0492182)'':0.00253194)'':0.00913841,(('PDT000203412.2':0.0347946,'PDT000667712.2':0.0303759)'':0.0192794,('PDT000651604.1':0.0303517,'PDT000011809.2':0.0209379)'':0.0225831)'':0.00698814)'':0.0113377)'':0.0201537)'':0.00125927)'':0.00823041,((((('PDT000134021.2':0.0923546,'PDT000566326.1':0.0857404)'':0.00545696,('PDT000289860.2':0.0696585,'PDT000290364.2':0.0744005)'':0.0172918)'':0.0022369,('PDT000556951.1':0.0823409,('PDT000228380.1':0.0607844,('PDT000625982.1':0.0583392,'PDT000289933.1':0.0641988)'':0.00101962)'':0.00904485)'':0.00869676)'':0.0100098,(('PDT000181667.2':0.0872091,('PDT000590170.1':0.0840226,('PDT000420668.1':0.0565354,'PDT000672194.1':0.0692636)'':0.00525386)'':0.0151588)'':0.0140485,(('PDT000664981.1':0.0735511,('PDT000502399.1':0.0470165,'PDT000502420.1':0.0504371)'':0.0314311)'':0.0152477,('PDT000248751.1':0.117595,(('PDT000333180.1':0.0633841,'PDT000664968.1':0.0877329)'':0.00609141,('PDT000664960.1':0.0765112,(('PDT000502340.1':0.0659177,('PDT000557167.1':0.031907,'PDT000502594.1':0.0511141)'':0.0249657)'':0.000796071,(('PDT000664986.1':0.0151326,('PDT000557116.1':0.010686,'PDT000379435.1':0.0152392)'':0.00848138)'':0.0283011,('PDT000150755.2':0.0470619,('PDT000251869.1':0.0170258,('PDT000422114.1':0.0126813,'PDT000124721.1':0.0186462)'':0.00357289)'':0.0264165)'':-0.00284308)'':0.0162333)'':0.0191299)'':0.00377454)'':0.00829313)'':-0.00563218)'':0.0033591)'':0.00589335)'':-0.0003374,((('PDT000548116.1':0.047038,('PDT000719275.1':0.0473054,('PDT000355744.1':0.0199521,'PDT000386716.1':0.021551)'':0.029042)'':0.00170046)'':0.00641838,('PDT000251503.1':0.0630717,('PDT000557267.1':0.0410148,('PDT000131785.2':0.0396294,('PDT000132271.2':0.0313392,'PDT000642312.1':0.0330636)'':0.0034637)'':0.0061098)'':0.00776933)'':0.00069281)'':0.0253553,((((('PDT000175384.2':0.0449759,'PDT000303784.2':0.0470539)'':0.00294147,('PDT000275533.2':0.0268662,'PDT000496903.1':0.0180487)'':0.0353361)'':0.0171644,('PDT000417405.1':0.0760754,(('PDT000173726.3':0.0337244,'PDT000228027.1':0.0415119)'':0.016787,('PDT000409860.1':0.0532202,(('PDT000602925.1':0.0315492,'PDT000318961.1':0.0277968)'':0.0101462,('PDT000577078.1':0.0424321,('PDT000107260.2':0.00651207,'PDT000372011.1':0.00467673)'':0.0361309)'':0.00230339)'':0.0074141)'':0.00275981)'':0.0113751)'':-0.00202917)'':-3.04222e-05,('PDT000128812.2':0.0984507,(('PDT000027894.2':0.0714265,(('PDT000565873.1':0.0337208,'PDT000251566.1':0.0464809)'':0.0171316,('PDT000343600.1':0.0587905,('PDT000258380.2':0.0482954,('PDT000551914.1':0.0312405,'PDT000644956.1':0.0484954)'':0.0102137)'':0.013899)'':0.000202756)'':0.00359348)'':0.00400604,((('PDT000502039.1':0.0390131,'PDT000720229.1':0.060594)'':0.00714987,('PDT000521144.1':0.0133117,'PDT000526211.1':0.0316277)'':0.0352793)'':0.0114269,(('PDT000386733.1':0.0516203,'PDT000607466.1':0.0497797)'':0.0064862,('PDT000095770.2':0.0510489,'PDT000563557.1':0.0467866)'':0.0051088)'':0.00343356)'':0.00207226)'':0.0146168)'':-0.0133062)'':0.00717536,((('PDT000150864.1':0.0686956,('PDT000660648.1':0.0258736,('PDT000355059.1':0.0198708,'PDT000571176.1':0.0256509)'':0.00837557)'':0.0319526)'':0.0121592,('PDT000710443.1':0.0663519,(('PDT000502437.1':0.0523065,('PDT000109489.2':0.0390646,('PDT000235603.2':0.028242,'PDT000281475.2':0.0284279)'':0.00759646)'':0.00749691)'':0.00556635,('PDT000045589.1':0.0622169,('PDT000581018.1':0.0370938,('PDT000332416.1':0.0277119,'PDT000069126.2':0.0374626)'':0.00308016)'':0.0255329)'':-0.00456895)'':0.0118123)'':0.0131671)'':0.00272887,(('PDT000395421.1':0.0723299,('PDT000193058.2':0.0713252,'PDT000264633.2':0.0526858)'':0.00388558)'':0.016228,((('PDT000636386.1':0.0786526,'PDT000134152.1':0.0965684)'':0.0170707,('PDT000502650.1':0.102564,('PDT000109450.2':0.0617389,(('PDT000188142.2':0.0495569,('PDT000635713.1':0.0186239,('PDT000095696.2':0.0146329,'PDT000095744.2':0.0150223)'':0.00640023)'':0.0292922)'':0.00292359,('PDT000248800.1':0.0601346,('PDT000362834.1':0.0347242,('PDT000501841.1':0.0284171,('PDT000400006.1':0.0208558,('PDT000487585.1':0.0216299,('PDT000485262.1':0.0165768,'PDT000190082.1':0.0186199)'':0.00202761)'':0.000480233)'':0.00424845)'':0.00873569)'':0.00354772)'':0.00712976)'':0.0164852)'':0.013541)'':0.0110378)'':-0.0105871,(('PDT000312800.2':0.0428406,'PDT000502547.1':0.0576064)'':0.030045,(('PDT000661490.1':0.0784194,'PDT000010932.2':0.0461136)'':0.0109647,(('PDT000662556.1':0.0948107,('PDT000150886.1':0.0461235,('PDT000627433.1':0.0457856,'PDT000228030.1':0.0546664)'':0.00783734)'':0.0171505)'':-0.00780796,(('PDT000095759.2':0.0523729,'PDT000420681.1':0.0543911)'':0.0122947,('PDT000251519.1':0.0601302,('PDT000095725.2':0.0565544,(('PDT000250035.1':0.0378034,('PDT000482076.1':0.0232508,'PDT000319048.1':0.0209689)'':0.010336)'':0.017308,('PDT000346906.1':0.0447186,('PDT000556880.1':0.0323228,'PDT000150879.1':0.0350297)'':0.00587289)'':0.00662607)'':0.00481504)'':0.00307185)'':0.00466359)'':-0.0004823)'':0.00263613)'':0.00367512)'':0.00939704)'':0.00146552)'':-0.00323533)'':0.00204161)'':0.00119902)'':0.00474261)'':0.00865475)'':-0.00710373)'':0.0112225)'':-0.00436093)'':0.0412681)'':-0.0378275,((((('PDT000619087.1':0.0466162,('PDT000188123.3':0.0313957,'PDT000135992.2':0.0282431)'':0.00802589)'':0.0114957,('PDT000425520.1':0.0663172,('PDT000168874.1':0.0483157,('PDT000426306.1':0.0452682,'PDT000677993.1':0.0432163)'':0.00109275)'':0.0059636)'':0.000366451)'':0.0113231,('PDT000661091.1':0.0657131,('PDT000106261.2':0.0444381,(('PDT000311239.2':0.0248366,('PDT000281484.2':0.0119245,'PDT000571175.1':0.0155257)'':0.010744)'':0.0063942,('PDT000067794.1':0.0281862,('PDT000411268.1':0.010343,'PDT000411281.1':0.0158657)'':0.0334317)'':0.00544705)'':0.0132572)'':0.0238834)'':-0.0182291)'':0.0312679,(('PDT000404835.1':0.0869125,('PDT000015734.3':0.0958174,('PDT000015712.3':0.0363357,('PDT000015770.3':0.0185372,('PDT000015768.3':0.0178891,(('PDT000015747.3':0.0116889,'PDT000015740.2':0.0124218)'':0.00522494,('PDT000015758.2':0.0122811,('PDT000015765.3':0.0133918,('PDT000015756.3':0.0112817,'PDT000015763.3':0.0110021)'':0.00119377)'':0.000890902)'':0.00646285)'':0.00154762)'':0.00342157)'':0.0182039)'':0.050019)'':0.0113303)'':0.0109097,(('PDT000429775.1':0.0725411,('PDT000261040.1':0.0450237,('PDT000343838.1':0.0423095,'PDT000216019.1':0.0443168)'':0.00553312)'':0.0183955)'':0.0190598,(('PDT000174386.1':0.0697499,('PDT000497407.1':0.0437076,'PDT000661274.1':0.0350441)'':0.0195087)'':8.3485e-05,('PDT000524078.1':0.0604092,('PDT000252476.1':0.0455955,('PDT000228160.1':0.0258892,('PDT000520897.1':0.0152375,'PDT000042623.1':0.0122361)'':0.0156581)'':0.0140209)'':0.0129038)'':0.00693663)'':0.0112728)'':0.00882585)'':0.00259207)'':0.0164015,(((('PDT000642586.1':0.125937,('PDT000430743.1':0.0473512,('PDT000382198.1':0.0220233,('PDT000557346.1':0.0124064,'PDT000010933.3':0.0161054)'':0.00220702)'':0.0220605)'':0.0587495)'':0.0069406,(('PDT000322080.1':0.0256838,'PDT000303064.2':0.0327334)'':0.0424649,('PDT000289941.1':0.0762008,('PDT000048127.1':0.0535332,('PDT000471002.1':0.033698,'PDT000045605.1':0.0373408)'':0.0124054)'':0.0358535)'':0.00445519)'':0.0209394)'':0.00220452,((('PDT000488279.1':0.0741243,('PDT000499504.1':0.0764595,'PDT000709937.1':0.0810465)'':0.00552967)'':0.0127646,('PDT000312648.2':0.0600777,('PDT000258452.2':0.0515651,('PDT000304928.2':0.0210145,'PDT000532400.1':0.0196638)'':0.0235236)'':0.0192044)'':0.0197468)'':0.00257042,(('PDT000718815.1':0.0539946,('PDT000228002.1':0.0650408,('PDT000482322.1':0.0510903,'PDT000250068.1':0.0583147)'':0.00392072)'':0.00177671)'':0.0345372,(('PDT000250240.1':0.07119,('PDT000312805.2':0.0701876,'PDT000502478.1':0.0536834)'':0.01664)'':0.00504442,('PDT000718956.1':0.0817169,(('PDT000557057.1':0.0470393,'PDT000357701.1':0.0429999)'':0.0237391,('PDT000714144.1':0.0475049,('PDT000485439.1':0.0228761,(('PDT000259598.2':0.0170002,('PDT000258746.2':0.00825129,'PDT000548108.1':0.0133579)'':0.00646353)'':0.00378541,('PDT000519806.1':0.0135178,('PDT000188135.2':0.0063177,'PDT000275531.2':0.0083544)'':0.00688311)'':0.00655127)'':0.00473302)'':0.0223553)'':0.0259041)'':0.00428925)'':-0.000386866)'':0.00338912)'':0.00866389)'':0.0111367)'':0.00342931,(('PDT000534980.1':0.0957119,('PDT000433632.1':0.0599582,('PDT000537881.1':0.0601233,('PDT000426111.1':0.054311,'PDT000019619.1':0.054677)'':0.00316472)'':0.0149132)'':0.0303857)'':0.0176196,(((('PDT000095793.2':0.0120666,'PDT000248821.1':0.00658304)'':0.0633669,('PDT000252686.1':0.075753,('PDT000124722.1':0.05694,(('PDT000509487.1':0.0325325,'PDT000384476.1':0.024298)'':0.00760737,('PDT000319010.1':0.0368314,('PDT000537876.1':0.030295,('PDT000346813.1':0.0219952,('PDT000682396.1':0.0166965,('PDT000240686.2':0.0131181,'PDT000565874.1':0.0146487)'':0.0037662)'':0.00704055)'':0.00239015)'':0.00697385)'':-0.00314403)'':0.0266487)'':0.00822065)'':0.00165773)'':0.00426388,((('PDT000181662.2':0.0315925,'PDT000535402.1':0.0217844)'':0.0479435,('PDT000364394.1':0.0571857,('PDT000292893.1':0.0482217,('PDT000425910.1':0.0538769,(('PDT000246512.2':0.0497585,'PDT000640129.1':0.0349541)'':0.00618552,('PDT000539043.1':0.03789,'PDT000651572.1':0.0355794)'':0.00914785)'':0.000841528)'':0.0119168)'':0.00242294)'':0.0174752)'':0.00693732,(('PDT000645043.1':0.0611183,('PDT000258725.2':0.047718,'PDT000250251.1':0.0486262)'':0.00912159)'':0.0227856,(('PDT000161686.2':0.0531766,((('PDT000535040.1':0.0452837,'PDT000713962.1':0.0407905)'':0.0049031,('PDT000584488.1':0.0387192,('PDT000618459.1':0.0237565,'PDT000661442.1':0.0315085)'':0.0083282)'':0.0088957)'':0.00849194,('PDT000352055.1':0.0618206,(('PDT000412055.1':0.0337619,('PDT000303131.2':0.0200085,'PDT000395326.1':0.0258874)'':0.0077099)'':0.0106551,('PDT000318997.1':0.0474493,('PDT000288652.2':0.0444754,('PDT000364428.1':0.0245839,'PDT000417390.1':0.0280011)'':0.00753619)'':0.00731552)'':-0.00283788)'':0.0111604)'':-0.000231681)'':0.00932793)'':0.00988763,(('PDT000242865.1':0.0744462,(('PDT000231462.2':0.0547797,'PDT000455200.1':0.04325)'':0.00202321,('PDT000482840.1':0.044671,('PDT000259592.2':0.0390317,(('PDT000346043.1':0.0527827,('PDT000262173.2':0.0395346,('PDT000535160.1':0.0273886,'PDT000564918.1':0.0290457)'':0.00626847)'':0.00641392)'':-0.00245097,('PDT000336962.1':0.0429741,(('PDT000355053.1':0.0198425,'PDT000417496.1':0.0217419)'':0.00608966,('PDT000355759.1':0.0294234,('PDT000479093.1':0.0141812,'PDT000015290.2':0.0192742)'':0.00757701)'':0.00135976)'':0.00628714)'':-0.00110693)'':0.0058185)'':0.00124468)'':-0.00133557)'':0.00623139)'':0.0123007,('PDT000652860.1':0.0828229,(('PDT000669469.1':0.0651559,('PDT000460560.1':0.0630784,(('PDT000426101.1':0.0195133,'PDT000019599.1':0.0242176)'':0.00537194,('PDT000509488.1':0.0232328,('PDT000312262.2':0.0219939,'PDT000501515.1':0.0240136)'':0.00438401)'':0.00641904)'':0.0281272)'':0.0133153)'':-0.010138,(('PDT000556012.2':0.0482584,('PDT000497424.1':0.0313387,'PDT000521048.1':0.0407659)'':0.00869495)'':0.00201533,('PDT000409835.1':0.0585388,(('PDT000397777.1':0.0572273,'PDT000397803.1':0.0467177)'':0.00277663,(('PDT000644963.1':0.0350722,('PDT000191872.2':0.0247999,'PDT000566103.1':0.0316984)'':0.00582746)'':0.00503486,(('PDT000003866.2':0.00561523,'PDT000604846.1':0.0102594)'':0.0282266,('PDT000274088.2':0.0149558,'PDT000644928.1':0.0159364)'':0.0107283)'':0.00398557)'':0.00522997)'':0.00585361)'':-0.00462786)'':0.0135206)'':0.0214712)'':-0.0189738)'':0.00770037)'':0.00843205)'':0.00034601)'':0.00312979)'':-6.70314e-05,(((((('PDT000191809.2':0.0418087,'PDT000003611.2':0.0308438)'':0.0250539,('PDT000617224.1':0.0400848,'PDT000644886.1':0.0480397)'':0.0163143)'':0.00330224,('PDT000393068.1':0.0401369,('PDT000652842.1':0.03413,('PDT000395484.1':0.0261215,('PDT000652956.1':0.0293984,'PDT000656966.1':0.0366172)'':0.0106938)'':0.00135829)'':0.0186538)'':0.00254335)'':0.0138113,('PDT000637581.1':0.0385699,(('PDT000395327.1':0.0249617,'PDT000395256.1':0.0266562)'':0.0038548,('PDT000482837.1':0.0346896,(('PDT000382200.1':0.00715357,'PDT000588636.1':0.0110877)'':0.00939177,('PDT000305499.2':0.0211338,('PDT000323546.1':0.016487,'PDT000077091.2':0.0126747)'':0.00849919)'':0.000584915)'':0.00622069)'':0.000484797)'':0.0156209)'':0.0118245)'':0.00603356,((('PDT000246520.2':0.0460859,('PDT000343918.1':0.0323906,'PDT000642587.1':0.0391637)'':0.0282903)'':0.0299466,('PDT000271361.2':0.0634455,(('PDT000427754.1':0.0253409,'PDT000425611.1':0.0209892)'':0.0111126,(('PDT000713997.1':0.00530075,'PDT000661644.1':0.00708315)'':0.0264946,('PDT000019598.1':0.041031,('PDT000496907.1':0.0220261,'PDT000248795.1':0.0175097)'':0.00925733)'':0.00478002)'':0.00507482)'':0.0212824)'':0.00691923)'':-0.00414966,((('PDT000467099.1':0.0534925,('PDT000296286.2':0.0303116,('PDT000196611.2':0.0420909,'PDT000413227.1':0.0365792)'':0.00181862)'':0.013925)'':0.0112983,('PDT000590166.1':0.0500859,(('PDT000498731.1':0.0500172,'PDT000719041.1':0.0498877)'':0.00507853,('PDT000661621.1':0.0474538,('PDT000358624.1':0.0343149,('PDT000161532.2':0.0327602,('PDT000397845.1':0.0212876,'PDT000642576.1':0.0209576)'':0.00741855)'':0.00594742)'':0.0039874)'':-0.00020874)'':0.0119888)'':0.0144863)'':-0.000805463,('PDT000520780.1':0.0917651,('PDT000661576.1':0.107131,(((('PDT000497400.1':0.0152617,'PDT000639517.1':0.0179763)'':0.0486529,('PDT000550390.1':0.0537272,'PDT000581174.1':0.0495338)'':0.0234001)'':0.00244579,(('PDT000566129.1':0.031366,'PDT000644855.1':0.039274)'':0.0307611,(('PDT000409782.1':0.0384155,('PDT000024551.2':0.0295484,'PDT000494435.1':0.0293784)'':0.00277913)'':0.0129031,('PDT000494410.1':0.0471483,(('PDT000017812.2':0.0351287,('PDT000662549.1':0.0273297,'PDT000248478.1':0.0240827)'':0.00718019)'':0.00505889,(('PDT000661572.1':0.0214273,('PDT000352098.1':0.00868401,'PDT000661535.1':0.011503)'':0.00897727)'':0.00132312,('PDT000346108.1':0.0234633,('PDT000463605.1':0.00554246,('PDT000292793.2':0.00439732,'PDT000602256.1':0.00488412)'':0.00482902)'':0.0103035)'':0.00229267)'':0.0108207)'':0.0120542)'':-0.00559039)'':0.00841348)'':0.00910131)'':-0.00260616,(((('PDT000101126.2':0.0472506,'PDT000583468.1':0.0542074)'':0.00919994,(('PDT000281216.2':0.0434528,'PDT000426486.1':0.0626252)'':0.00758031,('PDT000426648.1':0.0534745,('PDT000266403.2':0.0435947,('PDT000584737.1':0.0302741,('PDT000240697.2':0.0226195,'PDT000642300.1':0.0196215)'':0.00817025)'':0.00876022)'':0.00970202)'':0.00357141)'':0.00143244)'':-0.00125819,('PDT000652330.1':0.0505099,(('PDT000426315.1':0.0346978,('PDT000566121.1':0.00809814,'PDT000594323.1':0.00868776)'':0.0326644)'':0.00322355,('PDT000352517.1':0.0341606,('PDT000496986.1':0.0212183,('PDT000108223.2':0.0102516,('PDT000319756.1':0.00678043,'PDT000258748.2':0.00654097)'':0.0015476)'':0.00879072)'':0.0100374)'':0.00495879)'':0.0153666)'':0.0011142)'':0.00314961,((((('PDT000426280.1':0.0390776,'PDT000463979.1':0.0386082)'':0.00119532,('PDT000275046.4':0.0221775,'PDT000706521.1':0.0334275)'':0.0102685)'':0.011634,('PDT000661635.1':0.051645,(('PDT000566070.1':0.0333994,'PDT000661809.1':0.032222)'':0.0025912,('PDT000463680.1':0.0210408,'PDT000463703.1':0.0121199)'':0.0236539)'':0.00610124)'':-0.000396894)'':0.00541326,(('PDT000157514.2':0.0506498,'PDT000395193.1':0.041649)'':0.00936638,(('PDT000426759.1':0.0324088,'PDT000694518.1':0.0295089)'':0.0131588,(('PDT000326047.1':0.0281797,'PDT000395465.1':0.0253307)'':0.0133011,('PDT000417395.1':0.0455564,('PDT000306434.2':0.0207171,'PDT000426155.1':0.0267153)'':0.0146739)'':0.00312552)'':0.00229776)'':0.00676906)'':-0.000587438)'':0.00146132,((('PDT000246522.2':0.0384245,'PDT000264664.2':0.030947)'':0.0198276,('PDT000427624.1':0.0468626,'PDT000661784.1':0.0574334)'':0.00565997)'':0.00653333,((('PDT000332415.1':0.0466454,'PDT000103103.2':0.0585356)'':0.0160547,(('PDT000286929.2':0.0227462,('PDT000281213.2':0.0198847,'PDT000296287.2':0.0128989)'':0.00751782)'':0.0181075,('PDT000276029.1':0.050599,('PDT000509680.1':0.0360479,('PDT000285402.2':0.026429,('PDT000427008.1':0.0262702,('PDT000631991.1':0.0160755,'PDT000714546.1':0.013687)'':0.00376773)'':0.00440529)'':0.00904818)'':0.00436989)'':0.00245273)'':0.00694118)'':-0.002344,(('PDT000136032.2':0.025146,'PDT000524104.1':0.0347166)'':0.0199083,(('PDT000426383.1':0.0367701,('PDT000271452.2':0.0202111,'PDT000477085.1':0.0219789)'':0.00432244)'':0.00983704,('PDT000414186.1':0.04518,('PDT000509498.1':0.0445469,(('PDT000365699.1':0.0263831,'PDT000479552.1':0.0186228)'':0.00214149,('PDT000471024.1':0.025197,('PDT000426569.1':0.0235002,'PDT000251835.1':0.0205367)'':0.00420256)'':0.00595441)'':0.00792655)'':0.00370348)'':-0.000735732)'':0.00808953)'':0.00139578)'':0.00247338)'':-0.000518897)'':0.00118288)'':0.00139208)'':0.0212869)'':0.00692924)'':-0.0213451)'':0.00462673)'':0.00210189)'':0.00789088,((('PDT000332422.1':0.0809195,('PDT000346897.1':0.0848102,('PDT000003799.2':0.0534684,'PDT000426100.1':0.0351429)'':0.0334602)'':0.00700753)'':0.00133416,((('PDT000428377.1':0.0350404,'PDT000548130.1':0.0511886)'':0.0237986,('PDT000517931.1':0.0478994,('PDT000588008.1':0.0346524,('PDT000095732.2':0.0128801,'PDT000639590.1':0.0145526)'':0.0138)'':0.0234435)'':0.00710153)'':0.00364577,(('PDT000629657.1':0.0433643,('PDT000262171.2':0.0281117,'PDT000497345.1':0.0212589)'':0.0373309)'':0.000973865,(('PDT000397730.1':0.0270472,'PDT000639571.1':0.0279289)'':0.0289173,('PDT000566690.1':0.0341975,'PDT000644896.1':0.0283515)'':0.0185565)'':0.010041)'':0.0099954)'':0.0154397)'':-0.000538232,((('PDT000425663.1':0.0695429,'PDT000563609.1':0.0617231)'':0.0113162,(('PDT000060175.2':0.0626974,('PDT000471045.1':0.0568101,('PDT000343620.1':0.0370843,(('PDT000428425.1':0.0283209,'PDT000395167.1':0.0234809)'':0.00781935,(('PDT000130979.2':0.0266144,'PDT000517928.1':0.0214266)'':0.00408007,(('PDT000620259.1':0.013973,'PDT000714589.1':0.016312)'':0.0047989,('PDT000395313.1':0.0275302,('PDT000366967.1':0.0141368,'PDT000345910.1':0.00953073)'':0.0077847)'':0.00202107)'':0.000699009)'':0.00447791)'':0.00119812)'':0.0145199)'':0.00190127)'':0.0114283,(('PDT000271469.2':0.0117309,'PDT000312786.2':0.0196561)'':0.0421614,(('PDT000395114.1':0.0454807,('PDT000642592.1':0.0348464,('PDT000669530.1':0.0319153,('PDT000612018.1':0.00906706,'PDT000639552.1':0.0108536)'':0.0123776)'':0.0139411)'':0.0117249)'':0.00556786,(('PDT000272549.2':0.0429184,'PDT000294393.1':0.0429906)'':0.0114401,((('PDT000523288.1':0.0418545,('PDT000661325.1':0.00873334,'PDT000107540.1':0.00910606)'':0.0223279)'':0.0086856,('PDT000131889.2':0.0388509,('PDT000503832.1':0.0437904,('PDT000367299.1':0.0303284,'PDT000636383.1':0.0318809)'':0.00784381)'':0.00199753)'':0.00361832)'':0.00185956,(('PDT000395238.1':0.0449337,('PDT000425591.1':0.033291,'PDT000636290.1':0.0355611)'':0.00727662)'':-0.00116399,(('PDT000661610.1':0.0171515,'PDT000352534.1':0.023608)'':0.0215805,('PDT000709922.1':0.0239757,'PDT000395286.1':0.02262)'':0.00880796)'':0.00328898)'':0.00937672)'':0.00365718)'':0.000202248)'':0.00892141)'':-0.00148371)'':0.0121034)'':-0.00181607,(('PDT000661506.1':0.0916647,('PDT000430761.1':0.0637169,(('PDT000425788.1':0.024637,'PDT000426652.1':0.0239694)'':0.0294039,('PDT000019256.2':0.0297697,('PDT000661727.1':0.0429695,(('PDT000714120.1':0.00934286,'PDT000661307.1':0.0115236)'':0.00670526,('PDT000661314.1':0.0246866,('PDT000661571.1':0.0120561,'PDT000483534.1':0.0116663)'':0.00516713)'':0.00264158)'':0.0158838)'':0.00112729)'':0.0126466)'':0.00960558)'':0.0148233)'':0.00231089,(('PDT000584743.1':0.096577,('PDT000656980.1':0.0724837,(('PDT000714105.1':0.0435863,'PDT000714110.1':0.0434078)'':0.0187058,((('PDT000720457.1':0.0504945,'PDT000283184.1':0.0537615)'':0.000806213,('PDT000425527.1':0.0455041,('PDT000425997.1':0.0423068,'PDT000720472.1':0.0400711)'':0.00448353)'':0.00730086)'':0.00781096,('PDT000713897.1':0.0499399,('PDT000267829.1':0.052741,('PDT000425725.1':0.0471189,('PDT000619973.1':0.0248701,('PDT000713646.1':0.0174711,'PDT000519730.1':0.0131442)'':0.00598644)'':0.0131988)'':0.00928741)'':0.00242066)'':0.0032737)'':0.00284526)'':0.00680171)'':0.00774954)'':-0.0064477,(((('PDT000661146.1':0.0926196,('PDT000426162.1':0.0733517,'PDT000645525.2':0.0636173)'':0.0211134)'':-0.00113219,((('PDT000661350.1':0.0404298,('PDT000535440.1':0.0177988,('PDT000409830.1':0.0165979,'PDT000426672.1':0.00838457)'':0.00993182)'':0.0105279)'':0.00896662,('PDT000661308.1':0.024049,('PDT000636198.1':0.0198764,('PDT000661427.1':0.0193585,('PDT000694470.1':0.0135797,'PDT000708228.1':0.0164322)'':0.00249804)'':0.00152654)'':0.00928361)'':0.00485833)'':0.00653353,(('PDT000311241.2':0.0394941,'PDT000466092.1':0.0368759)'':0.00733446,(('PDT000168488.1':0.0113986,'PDT000448181.1':0.0182856)'':0.0229243,(('PDT000352532.1':0.0344103,('PDT000367298.1':0.0323265,'PDT000720465.1':0.0294582)'':0.00711638)'':0.00422357,('PDT000589793.1':0.0355104,('PDT000668943.1':0.04609,('PDT000389645.1':0.0248144,('PDT000661519.1':0.0149041,'PDT000395246.1':0.0138055)'':0.00336386)'':0.00827771)'':0.0034606)'':0.000490411)'':0.00489182)'':0.0082556)'':-0.00549523)'':0.0239205)'':-0.0119413,((('PDT000310459.2':0.0421757,'PDT000687931.1':0.0264353)'':0.0271774,((('PDT000398015.1':0.0423706,'PDT000645003.1':0.039408)'':0.00270577,('PDT000497397.1':0.0423073,'PDT000557012.1':0.0363815)'':0.00365393)'':0.0109357,('PDT000627344.1':0.0667719,('PDT000019620.1':0.0620552,('PDT000397957.1':0.0419535,('PDT000520947.1':0.0397734,('PDT000425643.1':0.0315628,'PDT000395374.1':0.0316522)'':0.000656153)'':0.00735921)'':0.00411185)'':0.0041734)'':-0.00205027)'':0.00512046)'':0.00188328,(('PDT000475443.1':0.0489277,('PDT000395196.1':0.0492221,('PDT000500493.1':0.0445806,('PDT000565927.1':0.0337812,'PDT000645091.1':0.0402676)'':0.00750853)'':0.00447992)'':0.00644759)'':0.00477152,('PDT000645142.1':0.054722,(('PDT000717191.1':0.045459,(('PDT000274867.2':0.00411203,'PDT000644972.1':0.00746717)'':0.020494,('PDT000565905.1':0.0200853,'PDT000566689.1':0.0241147)'':0.00797145)'':0.0154122)'':-0.0029875,(('PDT000566060.1':0.028923,('PDT000394933.1':0.000971472,'PDT000566063.1':0.00102713)'':0.0204953)'':0.00782629,('PDT000395154.1':0.0436042,(('PDT000588814.1':0.0246287,'PDT000642591.1':0.0232374)'':0.0103528,('PDT000589283.1':0.0322726,('PDT000565956.1':0.0231237,'PDT000631534.1':0.0152615)'':0.00463422)'':0.0110184)'':0.00457823)'':-0.00409176)'':0.00396802)'':0.0149329)'':0.00168199)'':0.00403604)'':0.000961412)'':0.007464,((('PDT000521581.1':0.0358747,'PDT000395191.1':0.0498676)'':0.0147832,(('PDT000264649.2':0.017168,'PDT000171802.1':0.0233351)'':0.0331487,((('PDT000323902.1':0.0311053,'PDT000337227.1':0.0311576)'':0.0101017,('PDT000409840.1':0.0425022,'PDT000499517.1':0.0392643)'':0.00187734)'':0.00772287,('PDT000124668.1':0.0614046,(('PDT000251586.1':0.033878,('PDT000044123.1':0.0237743,'PDT000293444.1':0.022028)'':0.0139847)'':0.0110443,(('PDT000286927.2':0.0415919,'PDT000642303.1':0.0442619)'':0.00464527,('PDT000262155.2':0.0184036,('PDT000313327.2':0.0173609,'PDT000397622.1':0.0216322)'':0.00406441)'':0.020023)'':0.00423999)'':0.00499719)'':-0.00219947)'':0.00362714)'':0.0081676)'':-0.00471342,((('PDT000248471.1':0.0638379,(('PDT000102991.1':0.045412,('PDT000196648.2':0.0316638,'PDT000397780.1':0.0327992)'':0.0136795)'':0.00556684,('PDT000477125.1':0.0457226,('PDT000471037.1':0.0218529,('PDT000500474.1':0.0234463,('PDT000259580.2':0.00858443,'PDT000395018.1':0.00891577)'':0.0103163)'':0.00628811)'':0.0167069)'':-0.00204753)'':0.0113996)'':-0.00206307,('PDT000252672.1':0.0644373,(('PDT000497417.1':0.0309347,('PDT000395434.1':0.0171702,('PDT000394996.1':0.016849,'PDT000627596.1':0.0119588)'':0.0045773)'':0.0108983)'':0.000120635,('PDT000594473.1':0.0263724,('PDT000517925.1':0.027519,('PDT000477057.1':0.0187332,'PDT000525371.1':0.0173295)'':0.00542484)'':0.00349041)'':0.00555244)'':0.0316346)'':-0.00530338)'':0.00721681,(('PDT000409829.1':0.0543727,('PDT000426252.1':0.04668,'PDT000557243.1':0.044674)'':0.0153938)'':0.00551887,(('PDT000632003.1':0.0470995,('PDT000292807.2':0.0380979,('PDT000031976.2':0.0275664,('PDT000312780.2':0.0194536,'PDT000051513.2':0.0191367)'':0.00776053)'':0.01304)'':0.0142188)'':0.0085312,('PDT000594460.1':0.0682776,(('PDT000471033.1':0.0438718,'PDT000497235.1':0.0581632)'':0.0102087,('PDT000210706.1':0.049482,(('PDT000362501.1':0.0229313,('PDT000281480.2':0.00673282,'PDT000497336.1':0.0110352)'':0.00535525)'':0.0245747,(('PDT000345929.1':0.0364266,('PDT000145168.2':0.0232182,'PDT000425650.1':0.0255478)'':0.0105851)'':0.00565196,('PDT000088293.2':0.0210938,('PDT000294711.2':0.0227678,('PDT000258739.2':0.0283518,'PDT000372347.1':0.025911)'':0.00195117)'':0.00363147)'':0.00856362)'':0.00310555)'':0.0111337)'':0.00803161)'':0.00415553)'':-0.00908932)'':0.00319363)'':-0.00131785)'':0.0010887)'':0.00195752)'':0.0015337)'':0.00366902)'':0.000152763)'':0.00930057)'':-0.00477327)'':0.00910476)'':0.0211369)'':-0.00148619)'':0.00294356)'':0.000486533)'':0.00562078)'':0.0116106)'':0.0306065)'':0.00609198,((((((('PDT000395357.1':0.0599502,'PDT000395129.1':0.0602688)'':0.010267,('PDT000675171.1':0.0675898,('PDT000438319.1':0.0288812,'PDT000667718.2':0.0320335)'':0.0213754)'':0.000514279)'':0.00584163,(('PDT000535389.1':0.0449391,'PDT000675509.1':0.0412996)'':0.00120487,('PDT000500714.1':0.0383689,('PDT000590059.1':0.0415268,'PDT000417615.1':0.0554036)'':0.00184671)'':0.00900955)'':0.0233709)'':0.092391,((('PDT000027898.2':0.0619143,('PDT000109473.2':0.0530336,'PDT000566030.1':0.0527944)'':0.00415024)'':0.00722783,(('PDT000304934.2':0.0288982,'PDT000644988.1':0.0266952)'':0.00181873,(('PDT000235731.2':0.0260294,'PDT000548125.1':0.0222532)'':0.00664107,('PDT000566088.1':0.0212442,('PDT000077313.2':0.0190668,'PDT000409811.1':0.0121863)'':0.00336577)'':0.00854709)'':0.00315926)'':0.0462588)'':0.0693432,((('PDT000502265.1':0.0252683,'PDT000502288.1':0.0192813)'':0.0952795,('PDT000191804.2':0.0618558,('PDT000129102.2':0.0257367,'PDT000683617.1':0.0201711)'':0.0422323)'':0.0359977)'':0.0340033,(('PDT000213261.3':0.022133,'PDT000502536.1':0.0265591)'':0.0272258,('PDT000324086.2':0.0119223,('PDT000205105.3':0.0115007,('PDT000155680.3':0.00631867,'PDT000158942.3':0.00862153)'':0.00378415)'':0.00318485)'':0.0427239)'':0.090555)'':0.00609122)'':0.0126493)'':0.000943795,((('PDT000498884.1':0.0320865,('PDT000032392.1':0.00253826,'PDT000032393.1':0.00255105)'':0.0292596)'':0.0667561,('PDT000566134.1':0.081369,(('PDT000586249.1':0.0411729,('PDT000617814.1':0.0412601,'PDT000482818.1':0.0374391)'':0.00284809)'':0.0196614,('PDT000397969.1':0.0390935,('PDT000551867.1':0.0286102,('PDT000397637.1':0.0302729,('PDT000634724.1':0.0275524,('PDT000397642.1':0.0193189,'PDT000566232.1':0.0177072)'':0.0116026)'':0.00204892)'':0.00361032)'':0.00274397)'':0.0228093)'':0.021958)'':0.00655063)'':0.0650218,(('PDT000399964.2':0.189518,('PDT000373256.1':0.111765,'PDT000485974.1':0.119121)'':0.0550574)'':0.00312061,((('PDT000649797.1':0.0517414,'PDT000293474.1':0.0525266)'':0.0606872,('PDT000395192.1':0.0708727,('PDT000433784.1':0.0503897,'PDT000221056.1':0.0490277)'':0.0091281)'':0.063666)'':0.024797,((((('PDT000539282.1':0.0468935,'PDT000251427.1':0.0628805)'':0.00244379,('PDT000409478.1':0.0611407,('PDT000620263.1':0.0411288,('PDT000413222.1':0.0290114,('PDT000178589.2':0.0185989,'PDT000372358.1':0.0166165)'':0.0137)'':0.0065052)'':0.010506)'':0.0161721)'':0.0136485,(('PDT000706183.1':0.0458537,'PDT000253672.1':0.0568733)'':0.00654916,(('PDT000277722.2':0.0374634,'PDT000252559.1':0.0360646)'':0.0154351,('PDT000389640.1':0.0366435,('PDT000294543.2':0.0174383,'PDT000594459.1':0.0224311)'':0.0243022)'':0.00795335)'':0.00566873)'':0.0159967)'':0.0196111,(('PDT000362887.1':0.0477124,'PDT000346570.1':0.0644306)'':0.044776,('PDT000630426.1':0.137929,(('PDT000414196.1':0.128562,'PDT000463644.1':0.0904278)'':0.00594277,(('PDT000425633.1':0.0732525,('PDT000264647.2':0.0472029,('PDT000496898.1':0.0435325,('PDT000717105.1':0.0280215,('PDT000149354.2':0.0303977,'PDT000355760.1':0.0294364)'':0.000869844)'':0.00630406)'':0.00327942)'':0.0178772)'':-8.6465e-05,(('PDT000646922.1':0.0573223,('PDT000541628.1':0.0389077,('PDT000403145.1':0.0282933,'PDT000632010.1':0.0303176)'':0.0164208)'':0.00792025)'':0.000600405,('PDT000389663.1':0.0456973,('PDT000251773.1':0.0444711,('PDT000258720.2':0.038879,'PDT000129882.2':0.0298765)'':0.0126905)'':0.00253975)'':0.00342799)'':0.0022845)'':0.0227029)'':0.0324545)'':-0.0120779)'':-0.0173032)'':0.0334306,((('PDT000411314.1':0.0684975,(('PDT000129888.2':0.0486858,'PDT000228206.1':0.0549512)'':0.00714248,(('PDT000160511.2':0.0255344,('PDT000251797.1':0.0211329,'PDT000395195.1':0.023192)'':0.00710509)'':0.0191412,(('PDT000109442.2':0.0456065,'PDT000501054.1':0.0439999)'':0.00843488,('PDT000318963.1':0.0433383,('PDT000413207.1':0.0398915,('PDT000045581.1':0.0359422,('PDT000109484.2':0.0310221,'PDT000243041.1':0.0425102)'':0.0111141)'':0.00156304)'':0.00453944)'':0.00111078)'':0.00171639)'':0.00919897)'':0.0060319)'':0.0419936,('PDT000691303.1':0.162928,(('PDT000228196.1':0.0134002,'PDT000228213.1':0.0139335)'':0.0851081,('PDT000667896.2':0.0679325,('PDT000396786.1':0.0716528,('PDT000566080.1':0.0639215,(('PDT000455208.1':0.0527316,('PDT000171811.1':0.0340464,'PDT000360991.1':0.040751)'':0.00695415)'':0.00340615,('PDT000502411.1':0.0407819,('PDT000399971.1':0.028633,('PDT000577080.1':0.0219096,('PDT000095803.2':0.0212377,'PDT000565854.1':0.0204576)'':0.00527701)'':0.00225739)'':0.00707281)'':0.020883)'':0.00493029)'':0.00458579)'':0.00151903)'':0.0116708)'':0.0393231)'':-0.0108219)'':0.00411048,('PDT000384319.1':0.121284,((('PDT000502427.1':0.0665057,(('PDT000325575.1':0.0187178,'PDT000637366.1':0.0207935)'':0.0165258,('PDT000551912.1':0.0442405,('PDT000264663.2':0.0110458,'PDT000337225.1':0.00893234)'':0.0311241)'':0.00304102)'':0.0207365)'':0.0145242,(('PDT000288567.2':0.0481821,('PDT000109458.2':0.0275377,'PDT000501837.1':0.021817)'':0.0201606)'':0.0149112,('PDT000313321.2':0.0702954,(('PDT000337221.1':0.0248068,'PDT000109456.2':0.0273722)'':0.0148147,('PDT000413210.1':0.0480232,('PDT000619581.1':0.0244376,'PDT000501832.1':0.0247427)'':0.0243296)'':0.00192602)'':0.0128244)'':0.00534197)'':0.00656826)'':0.00485011,(('PDT000425960.1':0.0541854,('PDT000290518.2':0.0281508,('PDT000271527.2':0.0192329,'PDT000485565.1':0.0237779)'':0.00460987)'':0.0264415)'':0.0177322,(('PDT000557156.1':0.0519973,'PDT000637784.1':0.042214)'':0.0135171,((('PDT000409826.1':0.0252708,'PDT000632002.1':0.0254128)'':0.00901607,('PDT000339359.1':0.0296868,('PDT000259565.2':0.0215874,'PDT000303094.2':0.0167642)'':0.0146767)'':0.000161006)'':0.0255978,('PDT000346490.1':0.0423637,('PDT000296293.2':0.0496399,('PDT000355767.1':0.044312,('PDT000150263.2':0.0342513,('PDT000675642.1':0.0278779,('PDT000277714.2':0.0160844,'PDT000435597.1':0.0160307)'':0.00394226)'':0.00690587)'':0.0063169)'':0.0032678)'':0.00579882)'':-0.00122886)'':0.0155373)'':5.3897e-05)'':0.0116311)'':0.0353527)'':0.000983431)'':0.00534951)'':0.0307163)'':0.00817043)'':-0.00766842)'':0.0084396)'':-0.00160649,((('PDT000289858.2':0.0755403,'PDT000228133.1':0.0778847)'':0.00759128,(('PDT000290386.2':0.0475519,'PDT000557123.1':0.0578071)'':0.0241519,('PDT000290378.2':0.0669646,'PDT000290407.2':0.0469714)'':0.0108206)'':0.0132652)'':0.116906,((('PDT000373240.1':0.0754571,('PDT000129890.2':0.0472708,('PDT000502680.1':0.0338164,'PDT000502691.1':0.043586)'':0.0120142)'':0.0278287)'':0.013852,(('PDT000408656.1':0.0476486,('PDT000008765.2':0.0279065,('PDT000492160.1':0.0150172,'PDT000016364.2':0.0116018)'':0.010549)'':0.0267103)'':0.0231726,('PDT000086879.2':0.0666721,('PDT000095723.2':0.0559766,('PDT000691307.1':0.0529171,'PDT000369028.1':0.0677459)'':0.00582537)'':0.00461509)'':0.00781885)'':0.0242599)'':0.0089192,((('PDT000284918.2':0.0852987,'PDT000400036.2':0.0937653)'':0.0427466,((('PDT000247217.3':0.0637967,('PDT000273368.3':0.0334549,('PDT000672172.1':0.0296523,('PDT000497745.1':0.0279062,('PDT000065374.1':0.0214863,'PDT000497824.1':0.0224276)'':0.00295002)'':0.00528522)'':0.00390446)'':0.0328333)'':-0.000678386,(('PDT000439501.2':0.0306059,('PDT000151643.3':0.0251875,'PDT000140479.3':0.0276394)'':0.0119984)'':0.00337022,('PDT000081064.1':0.031057,('PDT000129889.3':0.0236047,'PDT000486517.1':0.0238159)'':0.0033794)'':0.00836786)'':0.0264729)'':0.0105627,(('PDT000627209.2':0.0212107,'PDT000636726.2':0.0330276)'':0.00442315,('PDT000487596.2':0.0445541,('PDT000200961.2':0.0190551,(('PDT000184462.3':0.0176731,('PDT000069419.3':0.0117256,'PDT000248606.1':0.012372)'':0.00250566)'':0.00434948,(('PDT000497774.1':0.0228706,('PDT000302829.3':0.0115748,'PDT000248611.1':0.0144133)'':0.00383646)'':0.000975877,(('PDT000451108.2':0.0164725,('PDT000498135.1':0.0114649,(('PDT000655771.2':0.0065694,'PDT000502303.1':0.0069616)'':0.00328717,('PDT000321138.2':0.00467724,('PDT000163000.4':0.00332113,'PDT000140469.3':0.00305909)'':0.00201099)'':0.00148502)'':0.0042916)'':0.000696019)'':0.000980935,('PDT000497784.1':0.0349171,('PDT000248616.1':0.0173247,('PDT000523222.2':0.00862973,(('PDT000588825.2':0.0064029,'PDT000497896.1':0.0064278)'':0.00117755,(('PDT000336891.2':0.0094129,('PDT000465774.2':0.00619512,'PDT000660259.2':0.00594418)'':0.000320947)'':0.0008172,('PDT000248654.1':0.0057161,('PDT000251871.1':0.00463134,('PDT000523968.2':0.00553159,'PDT000077509.1':0.00592141)'':0.00034876)'':0.00142418)'':0.000167051)'':0.000798102)'':0.00188471)'':0.00060116)'':0.0087683)'':-0.00666292)'':0.00343545)'':0.000502204)'':0.0128048)'':0.0117093)'':-0.00860262)'':0.0339829)'':0.028688)'':-0.00672895,((('PDT000706406.1':0.102209,(('PDT000266386.2':0.0461005,'PDT000644849.1':0.0410563)'':0.0401686,('PDT000203071.4':0.0600537,('PDT000235022.5':0.0565265,'PDT000397975.1':0.0486965)'':0.00467877)'':0.029691)'':0.0202334)'':0.00522638,((('PDT000215938.1':0.0334567,('PDT000650532.1':0.0053202,'PDT000687816.1':0.0048371)'':0.0418572)'':0.0234783,('PDT000645539.2':0.0607137,('PDT000352057.1':0.0343205,('PDT000509491.1':0.0280934,'PDT000610397.1':0.027251)'':0.00699159)'':0.0148896)'':0.0118779)'':0.00669624,('PDT000661155.1':0.109629,(('PDT000645671.2':0.119052,('PDT000426549.1':0.0572104,('PDT000289931.1':0.0413697,'PDT000346088.1':0.0385799)'':0.0112123)'':0.0365418)'':-0.00537808,((('PDT000345989.1':0.0109243,'PDT000346486.1':0.00665276)'':0.0432258,('PDT000645531.2':0.0583985,('PDT000395343.1':0.0522364,('PDT000497318.1':0.0500956,('PDT000095721.2':0.0187817,'PDT000644663.1':0.0222723)'':0.0162352)'':0.0023942)'':0.0101829)'':-0.000772687)'':0.00579417,(('PDT000228169.1':0.0465043,'PDT000346921.1':0.0501961)'':0.00346816,(('PDT000463666.1':0.0444283,('PDT000129894.2':0.0400527,('PDT000088276.2':0.0404048,'PDT000250256.1':0.0312952)'':0.00261816)'':0.00656458)'':-0.000439704,('PDT000556993.1':0.0513671,(('PDT000137687.2':0.0346242,'PDT000346508.1':0.0357012)'':0.00397087,('PDT000249909.1':0.0362403,('PDT000584792.1':0.0291023,'PDT000293503.1':0.0304855)'':0.00273666)'':0.00429052)'':0.00618819)'':-0.00191888)'':0.00364564)'':0.00428002)'':0.0154283)'':0.0113269)'':-0.0135228)'':0.0123135)'':0.00420159,(('PDT000470070.1':0.103734,('PDT000300923.2':0.0880986,'PDT000343853.1':0.0653214)'':0.0281693)'':0.0105423,((('PDT000259576.2':0.0577165,'PDT000423339.1':0.0651845)'':0.0406185,(('PDT000651579.1':0.0767479,('PDT000539299.1':0.0185363,('PDT000226476.2':0.0127025,('PDT000235759.2':0.0205395,'PDT000277738.2':0.0145326)'':0.00389052)'':0.00203656)'':0.0577724)'':0.00287442,(('PDT000548537.2':0.0488871,'PDT000112878.3':0.0382175)'':0.00820349,('PDT000213219.3':0.0415527,('PDT000190956.3':0.042678,'PDT000081068.1':0.0479594)'':0.00401107)'':0.0155268)'':0.0104279)'':0.0108979)'':0.00741244,((('PDT000150264.2':0.0694232,'PDT000228265.1':0.0740198)'':0.0243986,('PDT000109475.2':0.0848798,('PDT000124609.1':0.0586659,('PDT000150262.2':0.0234287,'PDT000720219.1':0.0354065)'':0.0476835)'':0.0149925)'':0.00326736)'':0.005843,(('PDT000393096.1':0.0633729,('PDT000364411.1':0.0554738,'PDT000568770.1':0.0613982)'':0.0187796)'':0.0245828,('PDT000703271.1':0.109514,(('PDT000230568.2':0.0930499,('PDT000502924.1':0.0821547,'PDT000346026.1':0.0768353)'':0.0103171)'':0.00754191,((('PDT000095828.2':0.048233,'PDT000589759.1':0.067698)'':0.0185779,('PDT000532121.1':0.00025977,'PDT000532148.1':0.00024013)'':0.0682822)'':0.00926231,((('PDT000444811.1':0.0731952,('PDT000502253.1':0.0552318,'PDT000502673.1':0.0528742)'':0.00923278)'':0.00171539,(('PDT000468031.1':0.0428473,'PDT000566049.1':0.0405021)'':0.0174419,('PDT000494417.1':0.0216869,'PDT000626569.1':0.0234244)'':0.03205)'':0.00847539)'':0.00185869,(('PDT000617792.1':0.0700166,((('PDT000312783.2':0.0440322,'PDT000443405.1':0.0476855)'':0.00259798,('PDT000365312.1':0.030158,'PDT000376081.1':0.0289285)'':0.0171854)'':0.00293819,('PDT000307401.2':0.05915,('PDT000719156.1':0.0441533,('PDT000443387.1':0.0488408,('PDT000645080.1':0.0431215,('PDT000147453.4':0.0441307,('PDT000566231.1':0.0341816,'PDT000599863.1':0.0299371)'':0.00491004)'':0.0038612)'':0.000335895)'':0.00279558)'':0.00410203)'':-0.00178073)'':0.0111038)'':-0.0011693,((('PDT000350390.1':0.0313568,'PDT000403075.1':0.0328252)'':0.0195281,('PDT000156012.3':0.0532086,('PDT000147459.3':0.0318028,'PDT000311254.2':0.0323097)'':0.0162107)'':0.00284609)'':0.00469769,(('PDT000644860.1':0.0604598,('PDT000252596.1':0.0356766,(('PDT000135929.2':0.0257062,'PDT000346518.1':0.0198243)'':0.00507085,('PDT000577099.1':0.023306,'PDT000251872.1':0.0268112)'':0.00285083)'':0.00595377)'':0.0153094)'':-0.00200171,(('PDT000502694.1':0.0408951,('PDT000351858.1':0.0273425,'PDT000627440.1':0.0351089)'':0.0136597)'':0.00592339,('PDT000517937.1':0.036086,('PDT000646904.1':0.0247263,('PDT000502490.1':0.0240968,'PDT000501843.1':0.0306207)'':0.00289411)'':0.00860011)'':0.0144931)'':0.00409907)'':0.00267708)'':0.00587871)'':0.00341687)'':0.00498767)'':0.00530765)'':0.00413295)'':-0.00262231)'':-2.63032e-05)'':-0.000682594)'':0.00874933)'':-0.00600352)'':0.00293304)'':0.0132558)'':0.0577215)'':-0.00754483)'':0.00085821,(((('PDT000271249.3':0.0632308,('PDT000539855.2':0.0221956,(('PDT000209210.3':0.0087602,'PDT000336250.2':0.0091855)'':0.00134962,('PDT000404036.2':0.00959037,'PDT000649341.2':0.00845263)'':0.00151723)'':0.00482781)'':0.0467793)'':0.0372751,(('PDT000306780.3':0.0189533,'PDT000505404.2':0.0141915)'':0.0214061,('PDT000083956.3':0.0359162,('PDT000060969.3':0.0284868,('PDT000449058.2':0.0207286,'PDT000702634.1':0.0217871)'':0.00429708)'':0.00638893)'':0.00936699)'':0.0802046)'':0.0663254,(((('PDT000095788.2':0.0363912,'PDT000500531.1':0.0362658)'':0.0358628,('PDT000357598.1':0.0482637,('PDT000226489.2':0.0527482,'PDT000435443.1':0.0488348)'':0.00939648)'':0.0120974)'':0.0817933,(('PDT000243710.3':0.051892,('PDT000495225.2':0.0347301,'PDT000077504.1':0.0250434)'':0.0379022)'':0.00862856,('PDT000109414.2':0.0725797,('PDT000353602.2':0.0780112,(('PDT000258505.3':0.0193858,'PDT000288570.3':0.023643)'':0.00617683,(('PDT000184445.3':0.0259395,'PDT000400845.2':0.0322455)'':0.00191664,(('PDT000488042.2':0.032331,'PDT000501957.1':0.023861)'':0.00122968,(('PDT000115003.3':0.00846904,'PDT000137635.3':0.0122754)'':0.0102804,('PDT000251870.1':0.0195039,(('PDT000488249.1':0.0161143,'PDT000488254.1':0.018226)'':0.00109968,('PDT000249806.1':0.0266069,(('PDT000488251.1':0.0101032,'PDT000488257.1':0.00808582)'':0.00833775,('PDT000354931.2':0.015142,'PDT000247539.3':0.0108744)'':0.00194525)'':0.00369803)'':-3.08182e-06)'':0.00473762)'':0.000179904)'':0.00102128)'':0.00437165)'':0.00141904)'':0.034092)'':0.0067618)'':-0.0173198)'':0.0954456)'':-0.000779843,(((('PDT000399982.1':0.0408225,'PDT000551909.1':0.050281)'':0.0365943,(('PDT000600865.2':0.0381232,'PDT000705905.1':0.0270559)'':0.00484676,(('PDT000501950.1':0.0162206,'PDT000502592.1':0.0174152)'':0.00560099,(('PDT000457356.2':0.0290072,'PDT000105054.3':0.01927)'':0.00116083,('PDT000140501.3':0.013882,('PDT000366295.2':0.0180105,('PDT000236267.3':0.0133188,('PDT000107836.3':0.0121087,'PDT000140517.3':0.0112878)'':0.00107475)'':0.00118054)'':0.00164673)'':0.00327494)'':0.00440186)'':0.00360819)'':0.051417)'':0.0115217,((('PDT000425779.1':0.0356408,('PDT000417512.1':0.025588,'PDT000557169.1':0.0184731)'':0.0117001)'':0.0323839,(('PDT000167641.2':0.0577736,'PDT000501824.1':0.0555144)'':0.00566734,('PDT000494562.1':0.0702003,('PDT000095787.2':0.0458739,'PDT000386720.1':0.0493415)'':0.012768)'':0.00546442)'':0.00329816)'':0.00302495,(('PDT000710997.1':0.0263369,('PDT000123312.1':0.0250704,('PDT000132320.3':0.0210792,('PDT000040710.3':0.0115636,'PDT000674781.1':0.0115978)'':0.00384608)'':0.00388358)'':0.0022435)'':0.00560472,(('PDT000040381.4':0.0189305,('PDT000040713.3':0.0183088,'PDT000127291.3':0.0131951)'':0.00556664)'':0.00912276,(('PDT000327083.2':0.0147275,'PDT000501186.2':0.0168837)'':0.00610256,('PDT000711086.1':0.0176688,'PDT000027606.1':0.0170411)'':0.00312679)'':0.00645701)'':0.00318832)'':0.048218)'':0.0129235)'':0.0457457,((('PDT000537064.2':0.0542566,('PDT000638586.2':0.0170759,('PDT000665174.1':0.0116108,'PDT000252668.1':0.0186681)'':0.00609461)'':0.0439544)'':0.0676536,(((('PDT000158779.3':0.0316592,'PDT000491620.2':0.0256147)'':0.0206986,('PDT000160350.3':0.0301617,('PDT000402765.2':0.0190999,('PDT000600060.2':0.0137966,('PDT000599124.2':0.0109374,('PDT000588072.2':0.00585966,'PDT000131872.3':0.00570104)'':0.00281731)'':0.00304136)'':0.00142432)'':0.00657159)'':0.0242753)'':0.0111851,('PDT000676473.1':0.0742327,(('PDT000160221.3':0.0571256,'PDT000316938.2':0.0519564)'':0.00945683,('PDT000491985.2':0.0470655,('PDT000412104.2':0.00768778,('PDT000163785.3':0.00827288,('PDT000333300.2':0.0060219,'PDT000540386.2':0.0056396)'':0.00264107)'':0.00108381)'':0.0312264)'':0.0111923)'':0.0100523)'':0.00375078)'':0.018795,(('PDT000452730.2':0.0325359,(('PDT000542084.2':0.026531,'PDT000542089.2':0.0247836)'':0.0111534,('PDT000542092.2':0.0256958,('PDT000542058.2':0.0156634,'PDT000542063.2':0.0132246)'':0.00979325)'':0.0120488)'':0.00848012)'':0.0476111,(('PDT000557173.2':0.0637704,('PDT000548180.2':0.0215322,'PDT000548275.2':0.0150133)'':0.0285729)'':0.017368,(('PDT000289068.3':0.0462506,'PDT000542629.2':0.0575324)'':0.00285482,(('PDT000645919.2':0.0256025,'PDT000703142.1':0.0320879)'':0.00283732,('PDT000509941.2':0.0398991,(('PDT000190359.3':0.02162,'PDT000443247.2':0.0258871)'':0.00821246,('PDT000572653.2':0.025127,('PDT000487629.2':0.0194978,('PDT000617057.2':0.00992068,('PDT000439410.2':0.001392,'PDT000537432.2':0.00150568)'':0.00639253)'':0.00562873)'':0.00375406)'':0.00279155)'':0.0109463)'':0.00470258)'':0.0162946)'':0.011944)'':0.0086338)'':-0.000611291)'':0.0418458)'':-0.0165656,((('PDT000501724.1':0.0687765,('PDT000252681.1':0.0239771,('PDT000470011.1':0.0192485,'PDT000717590.1':0.0218299)'':0.00370585)'':0.0263284)'':0.00774182,(('PDT000408768.1':0.043155,'PDT000661382.1':0.0503577)'':0.0262205,('PDT000167655.2':0.0734693,('PDT000645119.1':0.0486195,'PDT000719013.1':0.0467015)'':0.0170162)'':0.00137338)'':0.0048915)'':0.0177441,(('PDT000526142.1':0.0657219,('PDT000150271.2':0.0222975,'PDT000637322.1':0.0136473)'':0.0480507)'':0.026935,(('PDT000718972.1':0.136564,(('PDT000346052.1':0.0978717,('PDT000216266.2':0.0393439,('PDT000577093.1':0.0343033,('PDT000282897.2':0.0271061,('PDT000095786.2':0.0254295,('PDT000566024.1':0.0117187,'PDT000617843.1':0.00721882)'':0.018221)'':0.00499803)'':0.00429519)'':0.0038632)'':0.0514472)'':0.0082714,(('PDT000719049.1':0.0525837,('PDT000426211.1':0.0445387,'PDT000228384.1':0.0353642)'':0.00891383)'':0.0135599,('PDT000160604.2':0.0659645,(('PDT000470043.1':0.047823,('PDT000617796.1':0.0490652,('PDT000178596.2':0.0451514,'PDT000251436.1':0.0374821)'':0.0069628)'':0.00226118)'':-0.000823472,('PDT000252610.1':0.0530562,('PDT000712955.1':0.0469566,(('PDT000393054.1':0.0250457,'PDT000304201.1':0.0377319)'':0.0111636,('PDT000409815.1':0.0349963,('PDT000557319.1':0.0108568,'PDT000718853.1':0.00624231)'':0.0271826)'':0.00377094)'':0.00755935)'':0.00299316)'':0.00115544)'':0.010159)'':0.00325092)'':0.0058551)'':0.0228095)'':-0.0153365,(('PDT000208419.2':0.0201977,'PDT000206248.3':0.0187735)'':0.0906715,((('PDT000003954.2':0.044437,'PDT000557042.1':0.0524852)'':0.0236584,('PDT000288525.2':0.0521714,('PDT000095808.2':0.0024253,'PDT000395135.1':0.00276691)'':0.0482545)'':0.020028)'':0.0094852,(('PDT000274332.2':0.0426602,'PDT000109455.2':0.0499752)'':0.0530561,('PDT000248832.1':0.102837,(('PDT000034499.2':0.0345242,'PDT000502542.1':0.0340937)'':0.0436374,(('PDT000557252.1':0.0581051,(('PDT000336966.1':0.00626642,'PDT000504794.1':0.00840428)'':0.0414452,('PDT000191844.2':0.0616215,('PDT000394913.1':0.0282942,'PDT000565902.1':0.0204263)'':0.0298127)'':0.00607004)'':0.00314558)'':0.0105479,(('PDT000109465.2':0.0451076,'PDT000717588.1':0.0536685)'':0.00495409,(('PDT000412040.1':0.027433,'PDT000346028.1':0.0313391)'':0.029065,('PDT000502689.1':0.0409666,('PDT000502893.1':0.0393743,'PDT000250040.1':0.0338861)'':0.00428002)'':0.0106127)'':0.00504215)'':0.0116652)'':0.0065995)'':0.0118006)'':-0.00225473)'':-0.00529434)'':0.0149758)'':-0.00755669)'':0.0127066)'':-0.0091147)'':0.00769877)'':0.0401415)'':0.0118624)'':0.0149163)'':-0.00520195,((((('PDT000577480.1':0.0803059,('PDT000339378.1':0.0553223,'PDT000468325.1':0.0578317)'':0.0401096)'':0.0518647,('PDT000713662.1':0.0620795,('PDT000081317.1':0.0547549,('PDT000081264.1':0.027803,'PDT000081311.1':0.02231)'':0.0309191)'':0.00413884)'':0.0700134)'':0.0538549,((('PDT000497212.1':0.146721,('PDT000559258.1':0.0555199,(('PDT000600654.1':0.0246789,'PDT000638770.1':0.0390838)'':0.0145996,('PDT000397886.1':0.0265151,('PDT000397732.1':0.0208142,('PDT000544284.1':0.00819163,'PDT000639908.1':0.00838897)'':0.0134212)'':0.00164663)'':0.00648592)'':0.0191435)'':0.0948799)'':-0.0306819,(('PDT000455218.1':0.0464439,('PDT000336995.1':0.0314829,('PDT000240665.2':0.017231,'PDT000594456.1':0.0130332)'':0.0152977)'':0.0156073)'':0.0900799,(('PDT000479624.1':0.052134,'PDT000565871.1':0.06324)'':0.0386892,('PDT000718903.1':0.0994325,(('PDT000544964.1':0.0620208,'PDT000502409.1':0.0800222)'':0.00150731,('PDT000251420.1':0.0616605,(('PDT000228177.1':0.0125406,'PDT000359701.1':0.0197625)'':0.0220386,('PDT000502925.1':0.0256526,('PDT000645002.1':0.00683987,'PDT000687914.1':0.011409)'':0.0172569)'':0.0180228)'':0.0123355)'':0.00565742)'':0.0159233)'':-0.00389269)'':0.0187757)'':0.0067645)'':0.0501004,(('PDT000395142.1':0.0955483,(('PDT000312646.2':0.0504604,('PDT000502658.1':0.0341481,'PDT000252441.1':0.035248)'':0.0132229)'':0.0240819,('PDT000664976.1':0.0634035,('PDT000252562.1':0.0516528,'PDT000252625.1':0.0694252)'':0.0135055)'':0.01184)'':0.0139808)'':0.0588452,((('PDT000168831.1':0.140656,('PDT000332154.1':0.100143,((('PDT000644962.1':0.0603123,'PDT000081743.1':0.0441477)'':0.0136277,('PDT000397611.1':0.0675076,('PDT000568680.1':0.020952,'PDT000251966.1':0.0190559)'':0.0392405)'':0.0132774)'':0.0041708,(('PDT000718557.1':0.0362276,('PDT000332717.2':0.0195169,'PDT000135922.3':0.0225769)'':0.00958224)'':0.0313637,('PDT000417490.1':0.0894627,(('PDT000003867.2':0.0557277,('PDT000399564.1':0.0525091,'PDT000318998.1':0.0494929)'':0.00173977)'':0.00486239,('PDT000566057.1':0.0591788,('PDT000354890.1':0.0646728,('PDT000430794.1':0.0331564,('PDT000510034.1':0.0242038,'PDT000557276.1':0.0278662)'':0.0105585)'':0.0180444)'':0.00427008)'':-0.00312531)'':0.00737059)'':-0.00439239)'':0.00993981)'':0.00951304)'':0.0345213)'':-0.023736,(('PDT000538825.1':0.0748756,('PDT000502304.1':0.0397627,'PDT000502432.1':0.0319582)'':0.0405235)'':0.0198122,(('PDT000035299.1':0.0472852,(('PDT000429739.1':0.0321758,('PDT000556704.1':0.0222074,'PDT000649517.1':0.0218122)'':0.00916635)'':0.00148622,(('PDT000390181.1':0.00838082,'PDT000050498.1':0.00653718)'':0.0230042,('PDT000429759.1':0.03292,('PDT000662583.1':0.0186072,('PDT000663937.1':0.023388,'PDT000250125.1':0.0144735)'':0.00124309)'':0.00435549)'':0.0011527)'':0.00912554)'':0.00553373)'':0.0263135,('PDT000250152.1':0.0551741,(('PDT000250154.1':0.0276677,'PDT000357593.1':0.0407591)'':0.00262536,(('PDT000027924.1':0.0231334,('PDT000309680.2':0.00999587,('PDT000404006.1':0.00314648,'PDT000422548.1':0.00404058)'':0.0034314)'':0.00635679)'':0.000980476,(('PDT000428872.1':0.00418028,('PDT000428912.1':0.00318702,'PDT000428917.1':0.00389669)'':0.000796164)'':0.0156892,(('PDT000015796.3':0.0161259,'PDT000428869.1':0.0152451)'':0.00254137,('PDT000422546.1':0.0102567,('PDT000422539.1':0.00786278,'PDT000422547.1':0.0162252)'':0.001213)'':0.000517635)'':0.00268628)'':0.00475961)'':0.0328342)'':0.0217788)'':0.00440216)'':0.027392)'':0.0102018)'':0.00906031,((('PDT000265098.2':0.0848097,(('PDT000423986.1':0.0607404,'PDT000431886.1':0.0701426)'':0.00661886,('PDT000675527.1':0.0607636,'PDT000492383.1':0.0464994)'':0.0147369)'':0.0125489)'':0.0330725,((('PDT000423989.1':0.0501887,'PDT000226136.1':0.0465824)'':0.0142519,(('PDT000633533.1':0.030935,'PDT000295537.1':0.0472332)'':0.0112583,('PDT000589763.1':0.0394877,('PDT000534801.1':0.0238593,'PDT000590004.1':0.0167554)'':0.0272497)'':0.00575707)'':0.00496459)'':0.016285,('PDT000589897.1':0.0588674,(('PDT000675104.1':0.0446827,'PDT000355786.1':0.0578863)'':0.0196304,('PDT000495429.1':0.0583022,('PDT000633162.1':0.0493663,('PDT000720294.1':0.0363917,('PDT000411329.1':0.0373156,('PDT000675164.1':0.0284239,('PDT000424019.1':0.0240767,'PDT000674855.1':0.0234818)'':0.00627743)'':0.00589364)'':0.0022077)'':0.00622769)'':0.00298199)'':0.00847788)'':0.0128497)'':0.0124241)'':0.0142776)'':0.00254441,((('PDT000289075.2':0.0745723,'PDT000171833.1':0.0582807)'':0.0295261,(('PDT000427035.1':0.0899377,('PDT000443097.1':0.0915561,(('PDT000248483.1':0.0492991,'PDT000267827.1':0.0409841)'':0.0048023,(('PDT000672226.1':0.0609038,'PDT000006337.2':0.0604512)'':0.000798199,('PDT000694378.1':0.0527387,('PDT000534871.1':0.0359327,'PDT000672146.1':0.0439074)'':0.00916508)'':0.00586451)'':0.00201673)'':0.0169155)'':0.00539181)'':-0.0113094,(('PDT000136076.3':0.086354,('PDT000522765.1':0.054226,'PDT000206606.1':0.060084)'':0.0202945)'':0.00162003,('PDT000633667.1':0.0422322,('PDT000426826.1':0.0317579,(('PDT000442385.1':0.0135813,'PDT000534875.1':0.0213567)'':0.00430014,('PDT000067795.1':0.0190629,('PDT000460398.1':0.00330845,'PDT000460400.1':0.00366853)'':0.0194854)'':0.00747474)'':0.00767162)'':0.00450732)'':0.0383136)'':0.000126048)'':0.014953)'':-0.00101266,(('PDT000316917.2':0.0845303,('PDT000146570.3':0.0652585,('PDT000199116.3':0.0390968,'PDT000234924.3':0.0435502)'':0.0245655)'':0.0258532)'':0.00104685,(('PDT000129559.1':0.0467626,('PDT000482607.1':0.0428919,('PDT000604326.1':0.0100432,'PDT000485908.1':0.00996079)'':0.0283945)'':0.00487427)'':0.0161166,((('PDT000535359.1':0.0452715,'PDT000600520.1':0.0463535)'':0.00634438,('PDT000128855.2':0.0342964,('PDT000433799.1':0.00726078,'PDT000433803.1':0.00758872)'':0.0464713)'':0.0200795)'':0.00544975,('PDT000664966.1':0.082416,(('PDT000584540.1':0.0799206,('PDT000128870.2':0.0444524,('PDT000175908.2':0.0221486,'PDT000716817.1':0.0168176)'':0.0191586)'':0.0161228)'':0.00187319,('PDT000501909.1':0.0681498,('PDT000426401.1':0.0270534,('PDT000425816.1':0.0333679,('PDT000136025.2':0.0126127,'PDT000720168.1':0.0164732)'':0.00779106)'':0.00308479)'':0.034343)'':-0.00719248)'':0.0115982)'':-0.000916271)'':0.00319076)'':0.0222213)'':-0.0087156)'':0.00432225)'':0.0268409)'':0.0319779)'':0.0184735)'':0.00495197)'':-0.00292458,(((('PDT000095757.2':0.129738,'PDT000228310.1':0.119561)'':0.0210528,('PDT000354904.1':0.122662,('PDT000484141.1':0.0719822,'PDT000228247.1':0.0873748)'':0.0403893)'':0.000756652)'':0.0147965,('PDT000124696.1':0.113128,((('PDT000639585.1':0.0358132,'PDT000502192.1':0.0357886)'':0.0598899,('PDT000319026.1':0.0609305,('PDT000644892.1':0.0389262,('PDT000565933.1':0.0410038,'PDT000395253.1':0.0371664)'':0.0234359)'':0.0154738)'':0.0283247)'':0.0166874,(('PDT000719059.1':0.112719,('PDT000642298.1':0.0737304,('PDT000217113.2':0.0136273,('PDT000551919.1':0.0115551,'PDT000590162.1':0.01814)'':0.00382151)'':0.0878854)'':0.00894497)'':0.00892813,('PDT000566706.1':0.109131,('PDT000439412.1':0.0964671,('PDT000357621.1':0.0882619,('PDT000149355.2':0.0879129,'PDT000502692.1':0.0738681)'':0.0127211)'':0.00353954)'':0.0035926)'':0.0031422)'':0.00407186)'':0.0130346)'':0.0251977)'':0.0238136,(((((('PDT000190545.3':0.0258383,'PDT000123314.1':0.0356335)'':0.0499315,('PDT000125552.3':0.0642273,('PDT000228201.1':0.0479104,'PDT000251407.1':0.0521846)'':0.0178532)'':0.009262)'':0.018424,(('PDT000619550.1':0.107049,('PDT000399967.1':0.0786995,('PDT000627362.1':0.0566337,'PDT000711742.1':0.0518373)'':0.030866)'':0.0175232)'':-0.00264148,((('PDT000095717.2':0.0247885,'PDT000420680.1':0.014591)'':0.00560022,('PDT000399628.1':0.0191268,'PDT000395469.1':0.017783)'':0.00533563)'':0.053117,(('PDT000652985.1':0.0743266,'PDT000645485.2':0.0683004)'':0.00519747,(('PDT000459497.1':0.053187,'PDT000346070.1':0.045268)'':0.0264619,('PDT000346030.1':0.0643536,(('PDT000342665.1':0.0141444,'PDT000627442.1':0.0131947)'':0.00745067,('PDT000563587.1':0.0175127,'PDT000622368.1':0.0141802)'':0.00354861)'':0.0405572)'':0.00221257)'':0.00257378)'':0.0148103)'':0.00962028)'':0.00641261)'':0.0499105,(('PDT000270003.3':0.0890994,(('PDT000557103.1':0.06559,'PDT000414683.2':0.083518)'':0.0498987,((('PDT000233899.2':0.0323705,'PDT000345960.1':0.0276509)'':0.0254233,('PDT000369036.1':0.0691775,('PDT000372776.1':0.0265663,'PDT000086974.1':0.0326944)'':0.0260291)'':0.00554071)'':0.00264256,(('PDT000566159.1':0.0465923,'PDT000252191.1':0.0443081)'':0.0226683,('PDT000191217.1':0.0563595,('PDT000496980.1':0.0426141,'PDT000191227.1':0.0414405)'':0.00726994)'':0.0276887)'':0.00371986)'':0.0209013)'':0.00289688)'':0.0555642,('PDT000691371.1':0.0918038,((('PDT000644869.1':0.0492977,('PDT000313314.2':0.0462178,'PDT000417420.1':0.0312733)'':0.00751338)'':0.000715355,('PDT000178606.2':0.0329672,('PDT000471018.1':0.0187423,'PDT000642590.1':0.0176231)'':0.0132723)'':0.00990653)'':0.00404455,(('PDT000617795.1':0.0575283,('PDT000501931.1':0.0376504,('PDT000455226.1':0.0376166,'PDT000488301.1':0.0316546)'':0.002724)'':0.0149591)'':-0.000393748,(('PDT000624698.1':0.0652606,('PDT000627441.1':0.0475402,('PDT000500480.1':0.0425241,('PDT000502703.1':0.0413449,('PDT000147431.3':0.0351865,'PDT000497432.1':0.0309477)'':0.00671225)'':0.00176513)'':0.00172584)'':0.00761169)'':-0.00120391,('PDT000466089.1':0.0617602,(('PDT000417511.1':0.0282372,('PDT000502685.1':0.0224722,('PDT000550348.1':0.0197772,'PDT000612014.1':0.0216156)'':0.00464493)'':0.00431405)'':0.0070551,(('PDT000541624.1':0.0205547,'PDT000604573.1':0.0208925)'':0.0170503,(('PDT000337002.1':0.0300315,'PDT000384279.1':0.0356576)'':0.00171085,('PDT000412041.1':0.0340571,('PDT000034476.3':0.028485,(('PDT000500739.1':0.0169722,'PDT000620251.1':0.0155905)'':0.0108791,('PDT000403073.1':0.0218134,('PDT000333767.1':0.0164145,'PDT000471010.1':0.0132758)'':0.010116)'':0.00449946)'':0.00663743)'':0.000476162)'':0.00456749)'':0.000629165)'':-0.000360092)'':0.0071154)'':-0.00315593)'':0.00436358)'':-0.000600177)'':0.0168819)'':0.0776906)'':0.0018037)'':0.00236754,(((('PDT000675558.1':0.071987,('PDT000618901.1':0.0126249,'PDT000234609.1':0.0129121)'':0.0575725)'':0.00281059,((('PDT000427470.1':0.0496816,'PDT000411338.1':0.0741224)'':0.00945652,('PDT000445953.1':0.0216584,'PDT000675102.1':0.029497)'':0.0337303)'':0.0106354,('PDT000675281.1':0.0527462,('PDT000691322.1':0.057949,('PDT000427034.1':0.0354238,'PDT000589960.1':0.0277714)'':0.0106134)'':0.0136896)'':0.0162911)'':0.000972804)'':0.0430176,('PDT000399979.1':0.0951495,((('PDT000662592.1':0.0899686,('PDT000250128.1':0.0602676,'PDT000390802.1':0.0663444)'':0.0112964)'':0.0127732,('PDT000174322.1':0.0800345,('PDT000174364.1':0.00834006,'PDT000429688.1':0.00472724)'':0.0387454)'':0.0481153)'':0.0115755,(('PDT000212812.1':0.0850452,('PDT000675170.1':0.0642436,('PDT000649099.1':0.0639456,'PDT000675584.1':0.0543234)'':0.00454844)'':0.013155)'':0.00292349,(('PDT000667706.2':0.0224561,'PDT000667715.2':0.0165823)'':0.0356218,(('PDT000433324.1':0.0469636,'PDT000682317.1':0.0408821)'':0.00700782,('PDT000425927.1':0.0243136,'PDT000675049.1':0.0289321)'':0.0234972)'':0.00501295)'':0.0219127)'':0.0218866)'':0.00670709)'':-0.00558455)'':0.0471267,(('PDT000610535.1':0.0959892,('PDT000249883.1':0.0773859,(('PDT000407933.1':0.050832,('PDT000160273.2':0.0253884,'PDT000322985.1':0.0239108)'':0.00893467)'':0.0217862,('PDT000132073.2':0.0536698,('PDT000650970.1':0.0316343,'PDT000610613.1':0.0268315)'':0.0206393)'':0.0025099)'':0.0284983)'':0.0161501)'':0.052915,((('PDT000655777.1':0.0849925,('PDT000718955.1':0.0397361,'PDT000124671.1':0.0528655)'':0.0151347)'':0.0250901,(('PDT000150226.2':0.0563947,'PDT000188155.2':0.0535683)'':0.016587,('PDT000128799.2':0.0454888,'PDT000425625.1':0.0422677)'':0.0293492)'':0.00850536)'':0.00351466,(('PDT000518067.1':0.079751,(('PDT000312619.2':0.0588097,'PDT000539285.1':0.0542233)'':0.00312923,('PDT000109459.2':0.0282686,'PDT000386725.1':0.0301177)'':0.0349724)'':0.0186984)'':0.00624928,((('PDT000277737.2':0.0437989,'PDT000610089.1':0.0316267)'':0.0153572,('PDT000345976.1':0.0650212,(('PDT000495085.1':0.0302372,('PDT000397594.1':0.0274492,'PDT000496985.1':0.02694)'':0.00380061)'':0.00890745,('PDT000720207.1':0.0466625,('PDT000539298.1':0.0443494,('PDT000369025.1':0.0458157,('PDT000600657.1':0.0333117,('PDT000062940.2':0.0318361,'PDT000645072.1':0.0378929)'':0.00178315)'':0.00419154)'':0.00107483)'':0.00575572)'':-0.00398452)'':0.00959492)'':-0.00347489)'':0.014054,('PDT000060875.2':0.0967071,(('PDT000309331.2':0.0846876,('PDT000343610.1':0.0498597,('PDT000264624.2':0.0485759,'PDT000634743.1':0.0516621)'':0.0145463)'':0.0130833)'':0.000494543,((('PDT000145171.3':0.00134605,'PDT000396836.1':0.00115195)'':0.0447815,('PDT000337226.1':0.0361039,('PDT000109361.2':0.0349977,'PDT000346027.1':0.0395799)'':0.00870921)'':0.00781763)'':0.0050772,('PDT000661527.1':0.0626491,(('PDT000250710.1':0.00973428,('PDT000250712.1':0.00682678,'PDT000250715.1':0.00375842)'':0.00182807)'':0.0492432,('PDT000672169.1':0.0460497,(('PDT000396844.1':0.0350456,'PDT000395054.1':0.0368043)'':0.00389557,('PDT000384160.1':0.0412662,('PDT000518559.1':0.0203335,'PDT000710303.1':0.0316543)'':0.00908086)'':0.00497129)'':0.00484942)'':0.00053752)'':0.0090239)'':-0.00339511)'':0.00998425)'':0.028977)'':-0.0286831)'':0.0110095)'':0.0104361)'':0.026369)'':0.0167551)'':0.0141998)'':0.00417995,(((('PDT000343757.2':0.0926819,(('PDT000539351.1':0.0581595,'PDT000656955.1':0.0734695)'':0.0127828,(('PDT000196633.2':0.0117415,'PDT000718985.1':0.0195054)'':0.041067,(('PDT000196624.2':0.0448639,'PDT000475407.1':0.0487203)'':0.00753866,('PDT000251712.1':0.0459203,('PDT000249906.1':0.0372498,('PDT000304200.1':0.0219271,('PDT000150309.2':0.0143759,'PDT000305494.2':0.0166548)'':0.00404802)'':0.00894234)'':0.003853)'':0.00760683)'':0.00635162)'':0.00975713)'':0.0123831)'':0.0695849,(('PDT000417167.1':0.059287,'PDT000346972.1':0.04435)'':0.0645485,(('PDT000712318.1':0.0950014,('PDT000617222.1':0.0704626,'PDT000250703.1':0.0724484)'':0.00179605)'':0.0040799,('PDT000714562.1':0.102167,(('PDT000032399.1':0.0378102,'PDT000221031.1':0.0456776)'':0.0244708,(('PDT000570974.1':0.0530189,('PDT000027739.2':0.0346648,'PDT000637355.1':0.0400697)'':0.0205598)'':0.0053585,(('PDT000397993.1':0.0248331,'PDT000497410.1':0.0243601)'':0.029102,(('PDT000394991.1':0.0358612,'PDT000566153.1':0.0343668)'':0.00892605,('PDT000395425.1':0.0286532,('PDT000060538.2':0.0203219,'PDT000399637.1':0.0220269)'':0.00921986)'':0.0058583)'':0.0106008)'':0.00171672)'':0.021866)'':0.0143542)'':0.000321096)'':0.0112449)'':0.0545756)'':0.00149311,((('PDT000003670.2':0.126807,(('PDT000667763.2':0.121333,'PDT000293513.1':0.0965974)'':0.0103498,('PDT000252505.1':0.00404125,'PDT000252555.1':0.00393641)'':0.0755549)'':0.028262)'':0.00902488,(('PDT000271352.2':0.0785324,(('PDT000497383.1':0.0198006,'PDT000639898.1':0.0256946)'':0.054914,(('PDT000497300.1':0.0482079,'PDT000565976.1':0.045748)'':0.0225717,('PDT000565885.1':0.0492896,('PDT000568766.1':0.0156871,'PDT000687426.1':0.0184419)'':0.00934686)'':0.0288997)'':0.00936054)'':0.00547317)'':0.0154387,((('PDT000502279.1':0.016055,'PDT000065360.1':0.0161364)'':0.0123938,('PDT000557214.2':0.0267712,('PDT000081778.1':0.0228796,('PDT000502451.1':0.016846,'PDT000498117.1':0.0155267)'':0.00341503)'':0.0109703)'':0.00397232)'':0.0463136,('PDT000235271.3':0.110039,(('PDT000502584.1':0.0241446,('PDT000499433.2':0.0170492,'PDT000502444.1':0.00891837)'':0.00498487)'':0.0139258,(('PDT000307862.3':0.016839,'PDT000592613.2':0.0122313)'':0.0220736,('PDT000065385.1':0.00876305,('PDT000366961.2':0.012756,(('PDT000329293.2':0.00493783,'PDT000065236.3':0.00585457)'':0.00402882,(('PDT000084074.3':0.00441945,'PDT000010399.3':0.00335644)'':0.00421621,('PDT000497751.1':0.00405288,('PDT000387588.2':0.00194023,'PDT000481008.2':0.0018537)'':0.00221788)'':0.00130567)'':0.00101171)'':0.0017542)'':0.00231015)'':0.00864571)'':0.00271377)'':0.0489901)'':-0.00282098)'':0.0121263)'':0.0342184)'':0.0181652,(((('PDT000426721.1':0.0441955,'PDT000645556.2':0.0510928)'':0.0161585,('PDT000718867.1':0.0459101,'PDT000228071.1':0.047876)'':0.013289)'':0.0270162,(('PDT000584535.3':0.0659077,'PDT000645482.2':0.0807523)'':0.037548,(('PDT000396829.1':0.0331373,'PDT000718823.1':0.0354702)'':0.0391091,(('PDT000619663.1':0.0376644,'PDT000395260.1':0.0486504)'':0.0393885,(('PDT000281773.2':0.0235676,'PDT000488297.1':0.0335556)'':0.0274339,(('PDT000196631.2':0.0505606,'PDT000566065.1':0.0620474)'':0.00248121,('PDT000251462.1':0.0542092,('PDT000395202.1':0.0460756,('PDT000566195.1':0.0377995,('PDT000257710.2':0.00764048,'PDT000645141.1':0.00677592)'':0.0283668)'':0.00605179)'':0.0103997)'':0.00287901)'':0.00685395)'':0.00750072)'':0.00236149)'':0.00683923)'':-0.00132162)'':0.00135552,(('PDT000281478.2':0.110698,((('PDT000518558.1':0.029394,'PDT000609592.1':0.0309104)'':0.0259968,('PDT000502628.1':0.059733,'PDT000150866.1':0.050837)'':0.000148219)'':0.0280252,(('PDT000411068.1':0.0505112,('PDT000628174.1':0.0406335,'PDT000628189.1':0.0441823)'':0.00422683)'':0.0193539,('PDT000403086.1':0.0713124,('PDT000178600.2':0.0644009,('PDT000251583.1':0.0347693,'PDT000252466.1':0.0246521)'':0.0345739)'':0.00195961)'':-0.000510323)'':0.00474886)'':0.0142897)'':-0.00963008,(('PDT000336923.1':0.0607908,('PDT000230513.2':0.046709,'PDT000488286.1':0.0444006)'':0.0121564)'':0.0164964,(('PDT000652925.1':0.105742,(('PDT000687505.1':0.0621168,('PDT000644863.1':0.0500351,'PDT000248825.1':0.0524829)'':0.0147872)'':-0.000336331,('PDT000292815.2':0.0553281,('PDT000251762.1':0.0509473,('PDT000656994.1':0.0444204,'PDT000395424.1':0.0373034)'':0.00616534)'':0.0036447)'':0.0125135)'':0.00912026)'':-0.00465582,(('PDT000399648.1':0.0240717,'PDT000687452.1':0.0336929)'':0.0466802,((('PDT000394995.1':0.0645547,'PDT000258430.2':0.0533123)'':0.00141106,('PDT000497330.1':0.0511306,'PDT000005662.2':0.0415622)'':0.012802)'':0.00499922,(('PDT000668919.1':0.0465299,('PDT000168821.1':0.0521387,'PDT000396757.1':0.0422394)'':0.00648535)'':0.0217224,('PDT000547549.1':0.0768769,('PDT000399716.1':0.0524758,('PDT000399607.1':0.0465498,('PDT000719127.1':0.0637328,(('PDT000095765.2':0.0403703,'PDT000395239.1':0.0350538)'':0.00207913,('PDT000501935.1':0.0337,('PDT000322947.1':0.0253474,'PDT000644668.1':0.0343246)'':0.00544178)'':0.000681679)'':0.00909718)'':0.00304565)'':0.0030567)'':0.0100513)'':0.000134461)'':0.00272099)'':0.00354557)'':-0.00479284)'':0.00436859)'':0.00671616)'':0.00131237)'':0.0624267)'':0.00952234)'':0.00412818,(((('PDT000252607.1':0.129122,('PDT000659701.1':0.0487621,'PDT000429816.1':0.0494187)'':0.0869102)'':0.00189761,(('PDT000581015.1':0.0296676,'PDT000502501.1':0.0318066)'':0.0656083,('PDT000339396.2':0.0346305,('PDT000373191.2':0.0368287,'PDT000710978.1':0.0399889)'':0.00301076)'':0.0588004)'':0.039302)'':0.00663226,(('PDT000651041.1':0.11802,('PDT000345986.1':0.132292,('PDT000551915.1':0.100804,'PDT000345908.1':0.0772084)'':0.0384246)'':0.0032971)'':0.0174813,(('PDT000713468.1':0.0615797,(('PDT000423849.1':0.0404599,'PDT000248472.1':0.0439777)'':0.0152339,(('PDT000710991.1':0.0462663,('PDT000120901.2':0.0325653,'PDT000538592.1':0.0562582)'':0.00790756)'':0.00268077,('PDT000213214.2':0.0287117,('PDT000294547.2':0.0203935,('PDT000342668.1':0.00866538,'PDT000499661.1':0.0105845)'':0.00743437)'':0.0222235)'':0.00771827)'':0.00193038)'':0.00600124)'':0.0520131,(('PDT000652288.1':0.150845,('PDT000664959.1':0.130771,('PDT000499449.1':0.090459,(('PDT000709899.1':0.00953741,'PDT000709904.1':0.00621529)'':0.093778,('PDT000485560.1':0.0840475,('PDT000661295.1':0.0674576,('PDT000062956.2':0.0117722,'PDT000357660.1':0.0215216)'':0.0473955)'':0.00585207)'':0.00872351)'':0.00682551)'':0.0090828)'':0.00181321)'':-0.00751225,('PDT000251525.1':0.121195,('PDT000009631.2':0.107865,('PDT000251932.1':0.0931682,('PDT000627355.1':0.0850946,(('PDT000557052.1':0.0285222,('PDT000250065.1':0.023395,'PDT000251452.1':0.0207601)'':0.00664071)'':0.0129095,(('PDT000303065.2':0.0331409,('PDT000420682.1':0.0301009,'PDT000720171.1':0.030767)'':0.00275641)'':0.00310194,(('PDT000411084.1':0.0242021,'PDT000443379.1':0.0253637)'':0.00643834,('PDT000332079.1':0.0332554,(('PDT000366957.1':0.0182949,'PDT000641729.1':0.0135649)'':0.00365931,('PDT000482514.1':0.0136842,'PDT000395375.1':0.0241893)'':0.00142254)'':0.00568425)'':0.0038808)'':0.012235)'':0.0114051)'':0.0193706)'':0.00238796)'':0.0106021)'':0.00895205)'':-0.00773553)'':0.0104484)'':0.0246374)'':0.0100527)'':0.0160846,(((((('PDT000546873.2':0.0286426,'PDT000486510.1':0.0280237)'':0.0221959,('PDT000017985.3':0.0163052,'PDT000249895.1':0.0201356)'':0.032373)'':0.0888769,('PDT000360979.1':0.13992,(('PDT000254249.3':0.0236151,'PDT000502474.1':0.0227736)'':0.00667468,('PDT000334391.2':0.0199957,'PDT000106264.3':0.0186056)'':0.00619029)'':0.0992619)'':0.0190697)'':0.0108405,((('PDT000354858.1':0.080893,'PDT000334508.2':0.079369)'':0.0027602,('PDT000168319.3':0.0292664,'PDT000620799.2':0.0283568)'':0.0551477)'':0.0520944,(('PDT000011808.2':0.101348,(('PDT000501937.1':0.0950729,('PDT000395086.1':0.0491373,'PDT000494533.1':0.0595857)'':0.0263891)'':-0.00148489,(('PDT000667737.2':0.0584725,'PDT000081725.1':0.0682965)'':0.00618376,('PDT000102785.3':0.0235659,('PDT000501951.1':0.0221962,('PDT000602257.2':0.00792136,('PDT000128423.3':0.00593785,'PDT000502275.1':0.00501615)'':0.00130479)'':0.00725778)'':0.00469862)'':0.04668)'':0.0162917)'':0.0362655)'':-0.0277664,(('PDT000160246.3':0.0276695,'PDT000411316.1':0.0318589)'':0.0338736,(('PDT000326213.2':0.0323977,'PDT000519360.2':0.0260566)'':0.028955,(('PDT000328012.2':0.0480635,(('PDT000161586.3':0.0299241,('PDT000161507.3':0.0292151,'PDT000466037.2':0.0336988)'':0.00686742)'':0.00154999,('PDT000205738.3':0.0288303,(('PDT000145330.3':0.0247466,'PDT000687838.1':0.0192755)'':0.00213875,('PDT000231287.3':0.0288196,'PDT000708505.1':0.025216)'':0.00494663)'':0.00610802)'':0.00850985)'':0.014193)'':-0.00260485,(('PDT000123341.1':0.0541969,('PDT000327084.2':0.0419619,('PDT000253462.3':0.0217761,('PDT000629333.2':0.0165474,'PDT000648354.2':0.0206214)'':0.00187656)'':0.0115587)'':0.0184333)'':0.00366575,('PDT000408652.2':0.0561658,(('PDT000038866.1':0.0329375,'PDT000050615.2':0.0442702)'':0.0086658,('PDT000372102.2':0.0547402,('PDT000161999.3':0.0349603,'PDT000051486.1':0.0362402)'':0.011735)'':0.00261555)'':0.00461165)'':0.000526087)'':0.00399688)'':0.000243445)'':0.00997776)'':0.0489197)'':0.0435833)'':0.0132199)'':-0.0028065,(((('PDT000672417.1':0.0142398,('PDT000492252.2':0.0161967,('PDT000214072.3':0.00452014,'PDT000603527.2':0.00345673)'':0.0115312)'':0.00680126)'':0.0301472,('PDT000399500.2':0.0359106,('PDT000497761.1':0.0279196,('PDT000375997.2':0.0238983,('PDT000275714.3':0.0151627,'PDT000490339.2':0.0150745)'':0.00331554)'':0.00573275)'':0.00626894)'':0.0135656)'':0.0269932,(('PDT000343729.1':0.0755749,'PDT000045631.1':0.0736561)'':0.01125,(('PDT000203147.2':0.0821369,'PDT000107787.2':0.0795831)'':0.00494292,(('PDT000672703.1':0.0857018,('PDT000372490.1':0.0333951,('PDT000477093.1':0.0287715,'PDT000009576.2':0.0294809)'':0.00999029)'':0.0262079)'':-0.00369459,('PDT000131890.2':0.055654,((('PDT000196649.2':0.0338862,'PDT000502669.1':0.033799)'':0.0065904,('PDT000617802.1':0.0267372,'PDT000502251.1':0.025992)'':0.0166859)'':0.00560496,('PDT000622543.1':0.0438454,('PDT000652422.1':0.037829,(('PDT000390629.1':0.0203897,'PDT000413217.1':0.0142024)'':0.00532813,('PDT000332078.1':0.025312,('PDT000687933.1':0.0252839,('PDT000372191.1':0.018568,'PDT000249932.1':0.0158395)'':0.00278009)'':0.00591853)'':-0.00036825)'':0.00789558)'':0.00279697)'':0.00460781)'':0.00452847)'':0.00807265)'':0.0149678)'':0.00603734)'':-0.0126216)'':0.0343673,(('PDT000583877.1':0.12164,('PDT000444961.1':0.0949374,('PDT000438266.1':0.0584098,'PDT000687961.1':0.0408976)'':0.0468329)'':0.00308087)'':0.0137752,(((('PDT000502323.1':0.0223485,'PDT000502421.1':0.0272586)'':0.065018,('PDT000502639.1':0.0857325,('PDT000502649.1':0.0924482,'PDT000248761.1':0.0610138)'':0.00341802)'':0.0130952)'':-2.2699e-05,(('PDT000352446.1':0.0944345,('PDT000181773.2':0.0748131,('PDT000720174.1':0.0555722,('PDT000616416.1':0.0306325,('PDT000566007.1':0.0282689,('PDT000671187.1':0.0175351,'PDT000502149.1':0.0171379)'':0.00739267)'':0.00617)'':0.0215658)'':0.00426245)'':0.00752253)'':0.00706871,('PDT000126259.2':0.056717,(('PDT000716798.1':0.0636669,('PDT000203143.2':0.0290876,'PDT000206246.4':0.0342516)'':0.016491)'':0.00526045,('PDT000306473.2':0.058731,('PDT000202067.2':0.0484327,('PDT000241778.1':0.0280511,('PDT000175799.2':0.0144847,'PDT000716819.1':0.0162724)'':0.00809045)'':0.0111389)'':0.0151076)'':-0.00281162)'':0.0217417)'':0.0141371)'':0.00585403)'':0.0122679,((('PDT000065229.3':0.0795958,'PDT000607565.1':0.0883192)'':0.0160421,('PDT000661330.1':0.0868296,('PDT000175376.2':0.0772643,('PDT000312641.2':0.043151,'PDT000424011.1':0.0550115)'':0.0255984)'':0.0185592)'':0.00158372)'':0.00586495,(('PDT000495418.1':0.0810754,(('PDT000395258.1':0.0768057,('PDT000502243.1':0.0234868,'PDT000081774.1':0.0320361)'':0.0235378)'':0.00836369,('PDT000703272.1':0.0596941,('PDT000631994.1':0.0493295,('PDT000710423.1':0.06242,'PDT000251578.1':0.037501)'':0.00518837)'':0.0118477)'':0.00532946)'':0.0178734)'':0.00490624,(('PDT000181849.2':0.0768326,('PDT000661273.1':0.0406624,('PDT000645520.2':0.0501009,'PDT000340375.1':0.0371533)'':0.00836102)'':0.0270838)'':0.00689575,(('PDT000206123.2':0.105069,('PDT000645134.1':0.0308555,('PDT000178603.2':0.025474,'PDT000645063.1':0.0207176)'':0.0130943)'':0.0506528)'':0.0101293,(('PDT000167692.2':0.0739533,'PDT000426209.1':0.0607377)'':0.0296498,(('PDT000372172.1':0.0549748,'PDT000399611.1':0.0547642)'':0.0287737,(('PDT000188105.2':0.0515081,'PDT000607577.1':0.0576299)'':0.022994,(('PDT000652239.1':0.0644044,('PDT000557331.1':0.0465725,'PDT000719133.1':0.0531004)'':0.0129116)'':0.00257296,(('PDT000025247.2':0.046836,'PDT000248852.1':0.075521)'':0.0108188,('PDT000426131.1':0.0608976,('PDT000645641.2':0.0573729,('PDT000312626.2':0.0393535,'PDT000233040.1':0.0433692)'':0.00104604)'':0.0135665)'':0.00129734)'':0.00331797)'':0.00639694)'':0.00451)'':0.00374623)'':-0.00114068)'':0.00397587)'':0.00889432)'':0.00353218)'':-0.000121571)'':0.018666)'':-0.01094)'':0.0364185)'':0.00170359,(((('PDT000568623.1':0.0664756,('PDT000249778.1':0.0517441,(('PDT000129858.2':0.0242489,'PDT000553135.1':0.022781)'':0.00658059,('PDT000129885.2':0.0248622,'PDT000539881.1':0.0172587)'':0.0106203)'':0.0222398)'':0.0102985)'':0.0906359,((('PDT000181719.2':0.104791,(('PDT000248208.2':0.0559306,'PDT000081734.1':0.0468694)'':0.036122,('PDT000355829.2':0.0456735,'PDT000703273.1':0.0389657)'':0.0313111)'':0.01725)'':0.0376223,((('PDT000569502.2':0.0895732,('PDT000424704.1':0.0465916,('PDT000365309.1':0.0222801,'PDT000589379.1':0.0233168)'':0.0136583)'':0.0483734)'':0.0100991,('PDT000610259.1':0.0811274,('PDT000502492.1':0.0716681,(('PDT000490605.1':0.0281207,'PDT000557070.1':0.0379448)'':0.0036651,('PDT000131783.2':0.0508705,(('PDT000332077.1':0.0177418,'PDT000352093.1':0.0207349)'':0.00897568,(('PDT000393094.1':0.0158536,'PDT000541636.1':0.0187853)'':0.00440382,(('PDT000303099.2':0.0171773,'PDT000102256.2':0.0160738)'':0.000674164,('PDT000235958.2':0.00789615,('PDT000313326.2':0.00721834,('PDT000266492.2':0.00290896,'PDT000281132.2':0.00337533)'':0.00255772)'':0.00304456)'':0.00430972)'':0.00430823)'':0.00604308)'':0.0175821)'':-0.00481374)'':0.0393171)'':0.0244127)'':0.0126007)'':-0.00192647,((('PDT000689958.1':0.08125,'PDT000045596.1':0.091536)'':0.0142961,('PDT000709894.1':0.0695684,('PDT000661126.1':0.0739958,('PDT000590206.1':0.0694624,('PDT000488269.1':0.0467997,(('PDT000624687.1':0.0377335,'PDT000626532.1':0.0320725)'':0.00184741,('PDT000281098.2':0.0344827,('PDT000424630.1':0.0261207,'PDT000656733.1':0.0281414)'':0.00573956)'':0.00415672)'':0.013237)'':0.014529)'':0.00142308)'':0.00280153)'':0.0122346)'':0.00401495,((('PDT000294551.2':0.0471207,'PDT000518008.1':0.0545483)'':0.0149796,('PDT000496909.1':0.074896,('PDT000354913.1':0.0436741,'PDT000497401.1':0.0507036)'':0.0232717)'':0.00201174)'':0.019939,((('PDT000504795.1':0.0709046,'PDT000171793.1':0.0472654)'':0.0157575,('PDT000228344.1':0.0814499,('PDT000178560.2':0.0521146,'PDT000081723.1':0.0560014)'':0.0223986)'':0.00354042)'':0.00289938,(('PDT000652436.1':0.0379492,('PDT000397820.1':0.0294328,'PDT000032333.1':0.032439)'':0.00604138)'':0.0204156,(('PDT000703270.1':0.0516393,'PDT000360999.1':0.0563447)'':0.000790842,('PDT000133501.1':0.0462385,('PDT000399676.1':0.0221718,'PDT000425712.1':0.0221351)'':0.0136679)'':0.0177003)'':0.0035159)'':0.00852351)'':0.00428146)'':0.00330337)'':0.0228072)'':0.0357368)'':0.00468503,(((('PDT000250178.1':0.0370686,'PDT000289926.1':0.0464515)'':0.0201757,('PDT000661090.1':0.0803337,('PDT000208430.2':0.0604285,'PDT000290413.2':0.0509595)'':0.0130208)'':0.000142948)'':0.0251058,(('PDT000397909.1':0.0798397,'PDT000497361.1':0.0894903)'':0.013424,(('PDT000714577.1':0.0435756,('PDT000564936.1':0.0419773,'PDT000253655.1':0.0332275)'':0.00372555)'':0.0309052,(('PDT000639556.1':0.0529382,('PDT000128795.2':0.0442229,'PDT000497327.1':0.0529216)'':0.0158925)'':0.0139993,(('PDT000557088.1':0.025222,'PDT000718913.1':0.031671)'':0.0401388,(('PDT000642306.1':0.0457111,('PDT000413221.1':0.0514594,'PDT000627434.1':0.0385067)'':0.00905463)'':0.0183471,('PDT000252583.1':0.0394846,('PDT000717103.1':0.0311195,'PDT000162103.1':0.044161)'':0.00434166)'':0.0143513)'':0.000174887)'':0.00451446)'':0.00236148)'':0.0118259)'':0.00594821)'':-0.00414579,((('PDT000619655.1':0.08222,('PDT000488265.1':0.0491033,('PDT000719273.1':0.0375807,'PDT000249754.1':0.0321867)'':0.00752563)'':0.0144504)'':0.0150992,('PDT000081752.1':0.0788419,('PDT000661260.1':0.0616234,('PDT000556891.1':0.0428023,'PDT000359625.1':0.0528306)'':0.0104127)'':0.0115952)'':0.00473774)'':0.00547622,('PDT000172092.1':0.150961,(('PDT000541625.1':0.0690758,'PDT000709953.1':0.0868912)'':0.0133398,((('PDT000539050.1':0.0560145,'PDT000502336.1':0.0384814)'':0.0158358,('PDT000501053.1':0.0709813,('PDT000718888.1':0.0505695,(('PDT000557313.1':0.0334849,'PDT000395118.1':0.0361765)'':0.00870364,('PDT000497396.1':0.0412094,'PDT000501936.1':0.0387751)'':0.00177059)'':0.0062023)'':0.00494028)'':-0.000669517)'':0.0152305,(('PDT000661245.1':0.121264,(('PDT000258392.2':0.068549,'PDT000503641.1':0.071617)'':0.0142256,('PDT000250111.1':0.0667188,(('PDT000305444.2':0.0534392,('PDT000687834.1':0.0323712,'PDT000687916.1':0.0344966)'':0.0193893)'':0.00280176,('PDT000502431.1':0.0714424,('PDT000502415.1':0.0411077,'PDT000502523.1':0.0305247)'':0.0141814)'':0.00855123)'':0.00873682)'':0.00420496)'':0.0103217)'':-0.0127308,((('PDT000557132.1':0.064426,('PDT000501848.1':0.0488948,('PDT000556969.1':0.0177676,'PDT000557040.1':0.0273706)'':0.0222384)'':0.00397073)'':0.00899453,(('PDT000109479.2':0.0276554,'PDT000248837.1':0.0276733)'':0.0279972,('PDT000150869.1':0.0444226,('PDT000346053.1':0.0422228,('PDT000500738.1':0.0298816,'PDT000502555.1':0.0377361)'':0.00895267)'':0.00633739)'':0.00345049)'':0.00879449)'':0.00160056,('PDT000656985.1':0.092664,(('PDT000303265.2':0.0363732,(('PDT000594331.1':0.0284822,'PDT000619416.1':0.0327942)'':0.00736563,('PDT000550340.1':0.0225941,('PDT000662178.1':0.0249479,('PDT000395325.1':0.0250335,('PDT000682549.1':0.026902,('PDT000285396.2':0.0185857,('PDT000231470.2':0.0104639,'PDT000600658.1':0.0205802)'':0.00367915)'':0.00365419)'':0.00494643)'':0.00106433)'':0.00628952)'':-0.00116901)'':0.00561635)'':0.028858,(('PDT000663972.1':0.10197,('PDT000494420.1':0.065902,'PDT000228226.1':0.052434)'':0.0109571)'':-0.00261935,(('PDT000399630.1':0.0278306,'PDT000566111.1':0.0347552)'':0.0402624,((('PDT000249866.1':0.0339571,'PDT000357607.1':0.0213443)'':0.0238945,('PDT000409814.1':0.0427232,(('PDT000040103.1':0.0146561,'PDT000007391.2':0.0120633)'':0.0189692,('PDT000007709.2':0.0223738,('PDT000629249.2':0.0071911,'PDT000017076.2':0.0028039)'':0.0148242)'':0.00854628)'':0.0206494)'':0.00425743)'':0.00442254,('PDT000510240.1':0.0555913,('PDT000650292.1':0.0647584,((('PDT000140497.2':0.0561102,'PDT000395106.1':0.0268506)'':0.0102061,(('PDT000150252.2':0.0259874,'PDT000619401.1':0.036433)'':0.00799514,(('PDT000285940.2':0.0258849,'PDT000497428.1':0.0252939)'':0.00238286,('PDT000627595.1':0.0240576,('PDT000563939.1':0.00810409,'PDT000586935.1':0.00679901)'':0.0160816)'':0.000901829)'':0.00414231)'':0.0152937)'':0.00354209,(('PDT000095813.2':0.0243482,'PDT000557120.1':0.0285355)'':0.0139806,('PDT000411011.1':0.0390639,(('PDT000564928.1':0.0261134,'PDT000251819.1':0.0295559)'':0.00394863,('PDT000372337.1':0.0280546,('PDT000395037.1':0.0228814,('PDT000499476.1':0.0108571,'PDT000251585.1':0.00819617)'':0.00833354)'':0.00490045)'':0.0036993)'':0.00412498)'':0.00207802)'':0.00542226)'':0.00718261)'':0.000495401)'':-0.00460694)'':0.00699717)'':0.00536278)'':-0.00432174)'':0.0173457)'':-0.0132885)'':0.00295032)'':0.00212335)'':0.0187927)'':0.0219922)'':-0.0261033)'':-0.00615399)'':0.0591914)'':0.00974325)'':-0.00696796,((((('PDT000459518.1':0.0995901,'PDT000644671.1':0.0964409)'':0.020592,(('PDT000702840.1':0.0660679,'PDT000720475.1':0.0823951)'':0.00856106,(('PDT000652927.1':0.07681,('PDT000645522.2':0.0867467,('PDT000719107.1':0.052624,('PDT000494409.1':0.0421918,'PDT000587571.1':0.0284551)'':0.0222463)'':0.00726542)'':0.000537384)'':-0.000505034,(('PDT000497244.1':0.0163081,'PDT000497247.1':0.0104607)'':0.0493449,(('PDT000525102.1':0.0507206,('PDT000027740.2':0.0225636,'PDT000494255.1':0.0193875)'':0.0333861)'':-0.00115546,('PDT000682555.1':0.0450677,('PDT000395462.1':0.0526724,('PDT000602267.1':0.01057,'PDT000645172.1':0.016748)'':0.0249853)'':0.00529281)'':0.00820905)'':0.00481955)'':0.00796638)'':0.00630873)'':0.0489176)'':0.00895525,((('PDT000417129.1':0.0493602,'PDT000346964.1':0.044851)'':0.0377941,(('PDT000652552.1':0.0440329,('PDT000092532.1':0.0434818,('PDT000027907.2':0.0332713,'PDT000604446.1':0.0345696)'':0.012094)'':0.00480292)'':0.00625473,(('PDT000586240.1':0.0668064,('PDT000652972.1':0.0629886,'PDT000459424.1':0.0616814)'':0.00273263)'':0.00465921,('PDT000468898.1':0.079024,('PDT000604311.1':0.0546098,(('PDT000713363.1':0.0539364,'PDT000287994.1':0.0541286)'':0.00343894,('PDT000586239.1':0.0347708,'PDT000468841.1':0.0389828)'':0.00403046)'':0.0148534)'':0.00280227)'':-0.00299264)'':0.00552485)'':0.00953901)'':0.057805,(('PDT000034486.2':0.107559,(('PDT000063313.2':0.0391302,'PDT000637160.1':0.0421126)'':0.00592369,('PDT000228340.1':0.0112763,'PDT000228373.1':0.00618702)'':0.0436359)'':0.0563578)'':0.00991757,((('PDT000176001.2':0.0188933,'PDT000394243.1':0.0264122)'':0.0352152,('PDT000290387.2':0.0667757,('PDT000188128.2':0.0414508,'PDT000581028.1':0.0565251)'':0.0120643)'':0.000516295)'':0.0171639,(('PDT000212794.1':0.0740893,('PDT000616944.1':0.0524498,('PDT000687909.1':0.0434899,'PDT000359699.1':0.0350201)'':0.0183547)'':0.00496358)'':0.00561318,('PDT000346148.1':0.0594586,('PDT000312792.2':0.0495028,('PDT000566316.1':0.0566844,('PDT000367302.1':0.0363315,'PDT000485558.1':0.0342965)'':0.00831329)'':0.00987817)'':0.0112323)'':0.00689798)'':0.0134853)'':0.0316414)'':0.0142779)'':0.00317078)'':-0.000219559,((('PDT000626916.1':0.0942788,'PDT000502413.1':0.0666462)'':0.00368637,(('PDT000566190.1':0.0648767,('PDT000312280.2':0.064283,('PDT000025252.2':0.0510429,'PDT000675221.1':0.0592381)'':0.00437202)'':0.0060106)'':0.0145991,(('PDT000535484.1':0.0607186,(('PDT000427755.1':0.052436,'PDT000502174.1':0.050055)'':0.0050521,('PDT000557665.1':0.0415871,'PDT000593091.1':0.0407704)'':0.0120942)'':0.00458161)'':-0.00389748,(('PDT000430636.1':0.0587622,('PDT000181763.2':0.0479415,'PDT000426687.1':0.0533385)'':0.00161975)'':0.00336323,('PDT000188160.2':0.0614308,('PDT000502486.1':0.0418457,('PDT000634319.1':0.0446372,'PDT000252490.1':0.0394306)'':0.00282137)'':0.00536473)'':0.00299745)'':0.00532851)'':0.00773905)'':0.00894977)'':0.0159895,((('PDT000394830.1':0.0568736,'PDT000395068.1':0.0450704)'':0.0214074,('PDT000590191.1':0.0518298,('PDT000485257.1':0.039118,'PDT000502034.1':0.0367315)'':0.0118335)'':0.006718)'':0.0351871,((('PDT000081772.1':0.0730193,('PDT000668475.2':0.030472,'PDT000395453.1':0.0328389)'':0.0350607)'':0.0181282,(('PDT000633723.1':0.0824479,'PDT000468837.1':0.0705721)'':0.0215941,(('PDT000320224.1':0.0339023,'PDT000532319.1':0.0274336)'':0.0432123,('PDT000600637.1':0.060523,('PDT000228351.1':0.0645108,('PDT000372161.1':0.0388564,'PDT000405224.1':0.0634096)'':0.00296973)'':0.00767818)'':0.0072784)'':0.0175642)'':0.00972068)'':-0.00807936,(('PDT000395137.1':0.0782721,('PDT000610090.1':0.0371293,'PDT000645028.1':0.0349768)'':0.0265853)'':0.0111813,(('PDT000395180.1':0.0468512,('PDT000395003.1':0.0200548,('PDT000024555.2':0.0202252,'PDT000397633.1':0.0152901)'':0.00901339)'':0.00882832)'':0.0247243,(('PDT000644661.1':0.0495565,('PDT000565999.1':0.0488571,('PDT000303147.2':0.0239065,'PDT000027899.2':0.0152588)'':0.0176274)'':0.00673492)'':0.00784018,('PDT000643429.1':0.0644877,('PDT000060704.2':0.0362507,(('PDT000652338.1':0.0282856,('PDT000026894.2':0.0244128,'PDT000612016.1':0.0227773)'':0.0109434)'':0.00571354,('PDT000652979.1':0.0255162,('PDT000026909.2':0.0187043,('PDT000034507.2':0.0190838,'PDT000644945.1':0.0184273)'':0.00162611)'':0.0061273)'':0.00243986)'':0.00850517)'':0.0178837)'':-0.00759909)'':0.00591578)'':0.0191877)'':0.01858)'':0.00573486)'':0.00140869)'':0.038766)'':0.011061,((('PDT000223029.2':0.0958519,(('PDT000162004.3':0.061958,'PDT000222000.3':0.063817)'':0.0151356,('PDT000702545.1':0.0319256,(('PDT000378029.2':0.0270175,'PDT000532046.2':0.0178606)'':0.00340002,('PDT000376201.2':0.0161656,'PDT000466235.2':0.018504)'':0.00142493)'':0.0121724)'':0.0600657)'':0.0204235)'':0.0465838,(('PDT000390616.1':0.0457709,'PDT000719104.1':0.0454073)'':0.0493048,(('PDT000509506.1':0.0572433,'PDT000395321.1':0.0438707)'':0.0431477,(('PDT000350438.1':0.0995905,('PDT000095742.2':0.0833522,'PDT000417177.1':0.0957038)'':0.0179005)'':0.00162972,('PDT000292788.2':0.0869152,(('PDT000333766.1':0.0671574,('PDT000607503.1':0.0461768,('PDT000317614.1':0.042082,('PDT000519119.1':0.0421468,('PDT000594467.1':0.0203443,'PDT000468870.1':0.0246272)'':0.00430336)'':0.0110722)'':0.00438512)'':0.0207395)'':0.0149169,(('PDT000471409.1':0.100092,(('PDT000640347.1':0.0692191,(('PDT000272321.2':0.0385534,'PDT000251826.1':0.0502225)'':0.00158942,('PDT000397551.1':0.0270265,'PDT000687836.1':0.0224979)'':0.0107141)'':0.00860061)'':0.000523794,(('PDT000566171.1':0.0222206,'PDT000046580.1':0.0386404)'':0.011758,('PDT000306268.1':0.0324429,('PDT000203706.1':0.044251,('PDT000343616.1':0.0248587,'PDT000415746.1':0.0226161)'':0.00381639)'':0.00784527)'':0.000630535)'':0.00493255)'':0.0161709)'':-0.00295264,(('PDT000620739.1':0.0609516,('PDT000395406.1':0.0652255,('PDT000397989.1':0.0390379,('PDT000397580.1':0.0198701,'PDT000399705.1':0.0216493)'':0.0166303)'':0.0261782)'':0.00875041)'':0.00876779,('PDT000652998.1':0.0834392,(('PDT000283584.1':0.0397518,('PDT000520909.1':0.0350852,('PDT000426267.1':0.0401156,'PDT000346703.1':0.03286)'':0.00153728)'':0.00821918)'':0.0203017,((('PDT000395059.1':0.0381264,'PDT000639534.1':0.0284815)'':0.0233105,('PDT000532970.1':0.0430999,'PDT000568768.1':0.0465519)'':0.00392079)'':0.00751431,('PDT000027895.2':0.0617441,('PDT000653009.1':0.0475987,('PDT000034492.2':0.0370638,'PDT000042702.1':0.0393224)'':0.00454236)'':0.00592628)'':-0.000680405)'':0.00592473)'':0.00430122)'':0.00171712)'':-0.000731287)'':0.00645947)'':0.00107396)'':0.0122562)'':0.0137418)'':0.00428744)'':0.0428804)'':-0.00109603,(('PDT000346978.1':0.0915999,(('PDT000627349.1':0.0513253,'PDT000340388.1':0.0392224)'':0.0105756,('PDT000193067.3':0.0521865,(('PDT000184371.2':0.0478189,'PDT000502061.1':0.0423726)'':0.00114697,('PDT000517999.1':0.0362641,('PDT000687932.1':0.0325172,('PDT000407488.1':0.0272516,'PDT000672635.1':0.0292401)'':0.0017394)'':0.00478122)'':0.00389446)'':0.00384432)'':0.00263837)'':0.0187384)'':0.0575381,(((('PDT000364514.2':0.0832204,'PDT000124665.1':0.0946026)'':0.00549615,('PDT000346542.1':0.107408,('PDT000417141.1':0.0838377,'PDT000417151.1':0.0710523)'':0.0132537)'':0.00236328)'':0.00999409,('PDT000422119.1':0.103203,('PDT000661112.1':0.0881996,(('PDT000620252.1':0.0409435,'PDT000675248.1':0.0483353)'':0.0281733,('PDT000346973.1':0.0808761,('PDT000535534.1':0.0693569,('PDT000535459.1':0.0517923,'PDT000395274.1':0.0368178)'':0.0157895)'':0.00376458)'':0.00342782)'':0.00650881)'':0.0263101)'':-0.0123539)'':0.0398634,(('PDT000029746.3':0.0527722,'PDT000395427.1':0.035753)'':0.0710367,((('PDT000409855.1':0.0527222,('PDT000257708.2':0.0639616,'PDT000389797.2':0.0519084)'':0.00592133)'':0.0129118,('PDT000687692.1':0.0588284,('PDT000343397.1':0.0398016,('PDT000480401.1':0.0243575,'PDT000719123.1':0.0244829)'':0.0116764)'':0.0155517)'':0.00898912)'':0.0163161,((('PDT000652570.1':0.0451614,('PDT000378554.2':0.0443943,('PDT000652535.1':0.0279617,'PDT000661346.1':0.0427145)'':0.00934647)'':0.00868245)'':0.0270524,(('PDT000519277.2':0.0460726,'PDT000395038.1':0.0302128)'':0.039204,('PDT000408385.2':0.0769166,(('PDT000652509.1':0.0414096,'PDT000248698.1':0.0399411)'':0.0239085,(('PDT000672186.1':0.0444216,('PDT000498755.1':0.030455,'PDT000504750.1':0.0334786)'':0.00396913)'':0.00766644,('PDT000228157.1':0.0355738,('PDT000502696.1':0.019093,'PDT000249777.1':0.0229998)'':0.0141535)'':0.00901025)'':0.0135734)'':0.00777499)'':0.00247376)'':0.00294136)'':-0.00256071,(('PDT000644903.1':0.0616552,(('PDT000565845.1':0.0466874,'PDT000669191.1':0.0479098)'':0.00135877,(('PDT000557102.1':0.0374909,'PDT000395140.1':0.0395287)'':0.00344587,('PDT000228044.1':0.0358684,('PDT000354948.1':0.0345233,'PDT000251876.1':0.030432)'':0.00636737)'':0.00158542)'':0.00701477)'':0.0141948)'':-0.00516784,((('PDT000631584.1':0.0408289,'PDT000395299.1':0.0389476)'':0.012144,('PDT000395506.1':0.0371395,'PDT000395282.1':0.0469772)'':0.00711586)'':0.00377935,((('PDT000426874.1':0.0214905,'PDT000620189.1':0.0225119)'':0.00238407,('PDT000395320.1':0.0203633,('PDT000565984.1':0.0207235,'PDT000644856.1':0.0167049)'':0.00154564)'':0.00715278)'':0.0172545,(('PDT000032371.1':0.00110834,'PDT000032372.1':0.0005903)'':0.0407794,(('PDT000639570.1':0.0425805,('PDT000368355.1':0.0279517,'PDT000397949.1':0.035665)'':0.00422558)'':0.00404167,('PDT000191216.1':0.0287571,('PDT000617812.1':0.0226722,'PDT000662000.1':0.0199206)'':0.00798494)'':0.00477413)'':0.00197114)'':0.000561095)'':0.002904)'':0.0098321)'':0.020301)'':0.00580442)'':0.0354796)'':0.00772867)'':0.00216506)'':0.00649978)'':0.000966125)'':0.00440838)'':0.000941562,(((('PDT000651035.1':0.0777115,'PDT000124697.1':0.0695145)'':0.01769,('PDT000357644.1':0.0719264,('PDT000425553.1':0.0801632,('PDT000309239.2':0.0617053,'PDT000228149.1':0.0810567)'':0.00635632)'':0.00631362)'':0.0113719)'':0.000258424,((('PDT000200200.2':0.0196525,'PDT000526139.1':0.0228777)'':0.0087681,(('PDT000200188.2':0.0292123,'PDT000200206.2':0.019194)'':0.0239265,('PDT000216367.2':0.0169382,'PDT000494564.1':0.0269633)'':0.0135066)'':0.00498705)'':0.063344,(('PDT000065194.3':0.0824212,('PDT000293438.1':0.043155,('PDT000224229.3':0.035973,'PDT000480970.2':0.034026)'':0.00751991)'':0.0268081)'':0.00747352,(('PDT000181687.2':0.0742437,('PDT000426758.1':0.0551845,'PDT000252617.1':0.0434819)'':0.0325916)'':0.00050931,('PDT000661092.1':0.0930895,((('PDT000536935.1':0.0331691,'PDT000252553.1':0.0277768)'':0.0291162,(('PDT000636173.1':0.0447661,('PDT000417498.1':0.0336944,'PDT000487014.1':0.029117)'':0.00971457)'':0.00746039,('PDT000557021.1':0.0416384,('PDT000557270.1':0.0106275,'PDT000251406.1':0.00919018)'':0.0305653)'':0.00994962)'':0.00466734)'':0.00235279,('PDT000497371.1':0.0665916,('PDT000535176.1':0.0700589,('PDT000251557.1':0.0441755,('PDT000397757.1':0.0524879,(('PDT000343428.1':0.0242028,'PDT000250049.1':0.0356458)'':0.00101191,('PDT000619939.1':0.0189636,('PDT000310574.2':0.0172441,('PDT000373236.1':0.00753254,'PDT000249892.1':0.00639506)'':0.00533953)'':0.00353242)'':0.00757785)'':0.0106014)'':0.00908185)'':0.00487539)'':0.00223326)'':-0.00810949)'':0.0240778)'':-0.0139235)'':0.0148911)'':0.0114177)'':0.00635667)'':0.0672551,((('PDT000400819.1':0.133684,('PDT000557005.1':0.115504,(('PDT000394801.1':0.0982198,('PDT000248794.1':0.0321948,'PDT000252554.1':0.0314398)'':0.0620559)'':0.00192485,(('PDT000088281.2':0.026508,'PDT000131773.2':0.0356489)'':0.0347911,('PDT000652897.1':0.0450207,('PDT000128813.2':0.0252333,'PDT000128864.2':0.0224232)'':0.00929545)'':0.0243636)'':0.0360421)'':0.0192037)'':0.0201908)'':0.021714,('PDT000346916.1':0.123884,(('PDT000280980.2':0.062635,('PDT000357013.1':0.0340742,'PDT000494424.1':0.0389649)'':0.032348)'':0.0408234,('PDT000568598.1':0.118737,(('PDT000687070.1':0.0739597,'PDT000011110.2':0.0705023)'':0.0181468,(((('PDT000181775.2':0.0809669,'PDT000417116.1':0.0791141)'':0.0129712,('PDT000204900.2':0.0787496,'PDT000313015.2':0.0723244)'':0.00819277)'':0.00261464,((('PDT000713626.1':0.0610639,('PDT000003959.2':0.0761668,('PDT000077059.2':0.0575441,'PDT000611905.1':0.0455179)'':0.00738769)'':0.00380841)'':0.00869279,('PDT000639568.1':0.0775797,('PDT000425805.1':0.0592016,('PDT000251396.1':0.0502606,('PDT000088284.2':0.0411742,'PDT000425871.1':0.0465097)'':0.00636396)'':0.00282294)'':0.0059791)'':-6.3592e-05)'':0.00266358,((('PDT000399245.1':0.0549646,'PDT000522771.1':0.0503334)'':0.0101735,('PDT000425918.1':0.0589571,('PDT000425977.1':0.0556663,('PDT000497395.1':0.0198261,'PDT000598037.1':0.0200641)'':0.024039)'':0.00938103)'':0.00927635)'':0.00104658,(('PDT000030305.1':0.0453812,'PDT000251512.1':0.0475641)'':0.0149963,('PDT000201014.2':0.0691554,(('PDT000553120.1':0.0446409,('PDT000444973.1':0.0275357,'PDT000346920.1':0.0332896)'':0.0118249)'':-0.000584999,('PDT000537538.1':0.0471889,('PDT000425865.1':0.0255454,'PDT000019320.2':0.0226994)'':0.0101064)'':0.00959464)'':0.0201327)'':0.000155906)'':0.00165665)'':0.000776713)'':0.00717056)'':0.000395941,(('PDT000167702.2':0.076418,('PDT000378024.1':0.0407434,'PDT000500527.1':0.0334674)'':0.0206801)'':0.0137753,(('PDT000539293.1':0.059172,'PDT000501932.1':0.061475)'':0.0232982,((('PDT000428476.1':0.0037398,'PDT000551870.1':0.00513801)'':0.0559966,('PDT000497303.1':0.0211614,'PDT000563559.1':0.0245697)'':0.0388087)'':0.0224403,(('PDT000236266.3':0.0568419,('PDT000251698.1':0.062126,(('PDT000270645.2':0.0384495,('PDT000049097.1':0.00891368,'PDT000019458.2':0.0150504)'':0.0270437)'':0.00225206,('PDT000372691.1':0.033551,('PDT000602933.1':0.0232435,'PDT000249855.1':0.0240307)'':0.0126296)'':0.00883566)'':0.0106051)'':0.00104253)'':0.00596417,('PDT000671191.1':0.0805564,('PDT000249751.1':0.0891974,(('PDT000468343.1':0.0594108,'PDT000102985.1':0.0560982)'':0.00572218,(('PDT000355605.1':0.0566342,'PDT000385028.1':0.0543188)'':0.00175644,('PDT000518078.1':0.066591,('PDT000636817.1':0.064205,((('PDT000085586.2':0.0339278,'PDT000247118.3':0.0430935)'':0.0153497,('PDT000129822.2':0.0320337,'PDT000502249.1':0.0392421)'':0.0108256)'':0.00267931,(('PDT000713401.1':0.023013,'PDT000278221.1':0.0184403)'':0.0242186,('PDT000661794.1':0.0400737,('PDT000210715.1':0.0362154,('PDT000354925.1':0.0401683,'PDT000395049.1':0.038244)'':0.00543279)'':0.00134379)'':0.00166098)'':0.00401906)'':0.00563879)'':0.00228379)'':-0.00298862)'':0.00174793)'':0.00780233)'':0.00260139)'':-0.00967807)'':0.0101679)'':0.0026405)'':-0.00156772)'':-0.00317057)'':0.00857254)'':0.0146625)'':-0.0100566)'':0.0420491)'':0.017119)'':0.00190097,(((('PDT000311060.3':0.128662,(('PDT000535247.1':0.080454,'PDT000468883.1':0.10016)'':0.0210786,('PDT000719035.1':0.0661032,('PDT000502480.1':0.0608823,('PDT000510904.1':0.0171016,'PDT000346792.1':0.00894369)'':0.0310017)'':0.00784383)'':0.0324698)'':0.00472207)'':0.0312695,(('PDT000645020.1':0.0496504,('PDT000390617.1':0.0122895,('PDT000714570.1':0.0108651,('PDT000395073.1':0.00682868,'PDT000251411.1':0.00375122)'':0.00182323)'':0.00385017)'':0.0306704)'':0.0434,('PDT000354153.1':0.0439678,('PDT000395601.1':0.0462627,('PDT000502423.1':0.0349018,('PDT000510195.1':0.0215955,'PDT000251504.1':0.0185971)'':0.0146382)'':0.00837159)'':0.00237239)'':0.0588888)'':0.0319596)'':0.0163281,((('PDT000480407.2':0.115504,(('PDT000109419.2':0.0449291,'PDT000604402.1':0.0436494)'':0.0663431,('PDT000248763.1':0.0883849,('PDT000275045.2':0.0505427,'PDT000248803.1':0.0257665)'':0.067551)'':0.00448873)'':0.0102092)'':-0.00453423,(('PDT000600649.1':0.0693399,'PDT000278219.1':0.0457481)'':0.0085038,(('PDT000426097.1':0.0610482,('PDT000577494.1':0.0568128,('PDT000541635.1':0.0384321,('PDT000563563.1':0.0419138,'PDT000252530.1':0.0302592)'':0.000335524)'':0.00909454)'':0.00189101)'':0.00822773,(('PDT000566083.1':0.0368432,'PDT000069652.1':0.04746)'':0.00628347,(('PDT000490600.1':0.0388597,'PDT000556875.1':0.0495467)'':0.00488471,('PDT000490907.1':0.0306557,'PDT000517922.1':0.02921)'':0.0135512)'':0.00661652)'':0.00581792)'':0.00535703)'':0.045631)'':0.0353519,(('PDT000607513.1':0.0729703,(('PDT000548169.2':0.0250028,'PDT000112426.3':0.0192511)'':0.0432584,('PDT000136015.2':0.0558042,('PDT000717152.1':0.044354,('PDT000097097.2':0.0294689,'PDT000252585.1':0.0309488)'':0.00874774)'':0.00624261)'':0.00304521)'':0.0113682)'':0.0655224,((('PDT000582437.1':0.0358248,('PDT000627600.1':0.0375103,('PDT000216027.1':0.0280034,('PDT000645109.1':0.0202214,'PDT000354166.1':0.0216578)'':0.00313782)'':0.00543279)'':0.00412517)'':0.0725103,((('PDT000258418.2':0.0594893,('PDT000352535.1':0.0401612,('PDT000076957.2':0.0215996,'PDT000393060.1':0.0188362)'':0.0144433)'':0.0138848)'':0.0329178,('PDT000264780.3':0.0836721,(('PDT000221015.1':0.0369819,'PDT000221042.1':0.0321385)'':0.0118099,(('PDT000196610.2':0.0230002,'PDT000632004.1':0.0225434)'':0.0149716,('PDT000541632.1':0.0482897,('PDT000034464.2':0.0245724,'PDT000221014.1':0.0224505)'':0.0124291)'':0.00663659)'':0.00429389)'':0.0263146)'':0.00928684)'':0.0173822,((('PDT000470079.1':0.0280127,'PDT000395111.1':0.0299626)'':0.0280132,('PDT000505381.1':0.00200586,'PDT000712991.1':0.00228684)'':0.053111)'':0.0414564,(('PDT000640340.1':0.0742414,('PDT000565907.1':0.056416,('PDT000607515.1':0.044151,'PDT000251688.1':0.0445486)'':0.00724269)'':0.0105413)'':0.0228205,(('PDT000652357.1':0.0477954,('PDT000548533.2':0.0536809,('PDT000251567.1':0.0412902,('PDT000517939.1':0.0411759,('PDT000668395.2':0.0213377,'PDT000502246.1':0.0269334)'':0.0102176)'':0.00331554)'':0.00872566)'':0.00258345)'':0.0190222,(('PDT000366989.1':0.083221,('PDT000147502.3':0.00687972,'PDT000096855.3':0.00431128)'':0.072356)'':0.00407904,(('PDT000490603.1':0.0332126,'PDT000249816.1':0.0350045)'':0.0369952,(('PDT000251920.1':0.0425506,('PDT000111294.2':0.0342558,'PDT000641739.1':0.0344814)'':0.0101943)'':0.0227741,(('PDT000155900.2':0.0425318,'PDT000581712.1':0.0601642)'':0.0145655,('PDT000710122.1':0.0539515,(('PDT000661301.1':0.0395165,('PDT000184369.2':0.0341969,'PDT000309350.2':0.0369681)'':0.00328047)'':0.00736629,('PDT000346582.1':0.033204,('PDT000251470.1':0.0381786,('PDT000251575.1':0.0140513,('PDT000160573.2':0.0116746,'PDT000571132.1':0.0101227)'':0.00530339)'':0.0115734)'':0.00345508)'':0.00290331)'':0.0083478)'':0.0142333)'':0.0022951)'':0.00375658)'':0.00198121)'':0.000593006)'':0.0102077)'':0.0138633)'':0.005374)'':0.0119686)'':0.0102495,(('PDT000120825.2':0.134495,('PDT000645139.1':0.0958497,('PDT000502196.1':0.0500693,'PDT000011791.2':0.0397539)'':0.0580097)'':0.0250079)'':0.000744378,((('PDT000171831.1':0.107656,'PDT000354232.1':0.0929296)'':0.0252789,(('PDT000346519.1':0.08672,('PDT000150090.3':0.0818634,(('PDT000535500.1':0.0409555,'PDT000228331.1':0.0419301)'':0.0240345,('PDT000196645.2':0.0462828,('PDT000228348.1':0.0467994,(('PDT000526147.1':0.0342575,'PDT000590207.1':0.0257152)'':0.00156689,('PDT000497394.1':0.028516,('PDT000374999.1':0.023808,('PDT000346758.1':0.0140553,('PDT000713690.1':0.00721107,'PDT000228289.1':0.00580563)'':0.00628564)'':0.00647224)'':0.00208852)'':0.00631824)'':0.0083924)'':0.0104966)'':0.0074011)'':0.0187898)'':0.0055431)'':0.0185923,(('PDT000509156.1':0.0586202,('PDT000081756.1':0.0350369,'PDT000228085.1':0.0316103)'':0.0239302)'':0.0412964,(('PDT000407742.2':0.00296741,'PDT000679127.1':0.00632887)'':0.0427177,('PDT000425916.1':0.0502317,('PDT000046608.1':0.0571799,('PDT000553434.1':0.0403104,('PDT000143591.1':0.0415748,('PDT000502004.1':0.0342201,('PDT000672181.1':0.037492,('PDT000390715.1':0.0149908,'PDT000430752.1':0.0146848)'':0.0155203)'':0.00424611)'':0.00144213)'':0.00116759)'':0.00403417)'':0.0025995)'':0.000954436)'':0.0547045)'':0.00223406)'':0.0137233)'':-0.00526387,((('PDT000356787.1':0.0850581,('PDT000521354.1':0.0253593,'PDT000566131.1':0.0224923)'':0.0628576)'':-0.000526217,(('PDT000309383.2':0.0171764,'PDT000589382.1':0.0219286)'':0.0365432,('PDT000652411.1':0.046414,('PDT000305085.2':0.0274372,('PDT000573629.1':0.0340737,(('PDT000095809.2':0.0194513,'PDT000619659.1':0.0195736)'':0.00273248,('PDT000563586.1':0.0162158,('PDT000373248.1':0.0103748,'PDT000550385.1':0.0110283)'':0.00510286)'':7.54459e-05)'':0.0095824)'':0.00342972)'':0.00763812)'':0.0194128)'':0.0226)'':0.0142912,(('PDT000679693.1':0.0663869,('PDT000409786.1':0.0663419,('PDT000502436.1':0.0369839,'PDT000289874.1':0.0440447)'':0.0245973)'':0.0240594)'':0.0103692,('PDT000424012.1':0.106653,(('PDT000375015.1':0.047972,'PDT000689854.1':0.051569)'':0.0328666,('PDT000128811.2':0.0839238,(('PDT000675153.1':0.0649306,('PDT000633685.1':0.0571484,'PDT000435618.1':0.0559436)'':0.00474489)'':0.00311188,('PDT000485414.1':0.0573563,('PDT000502558.1':0.0510878,('PDT000390631.1':0.0298114,('PDT000651042.1':0.0273887,'PDT000720675.1':0.0217044)'':0.00281533)'':0.0132377)'':0.0173117)'':-0.00308418)'':0.00996371)'':-0.00215894)'':0.0118059)'':0.00360614)'':0.0200147)'':0.0140889)'':0.0104625)'':0.00250139)'':0.00673844)'':0.00394311)'':0.00174534)'':-0.00262462,((((('PDT000536898.1':0.081733,(('PDT000303034.2':0.067378,'PDT000095769.2':0.065887)'':0.014142,(('PDT000333152.1':0.0359392,'PDT000539292.1':0.0527563)'':0.0219512,('PDT000413228.1':0.0385241,'PDT000251692.1':0.0542543)'':0.00823903)'':0.0193603)'':0.0137594)'':-0.00198174,(('PDT000557030.1':0.0255104,'PDT000689868.1':0.035706)'':0.0589968,(('PDT000244673.3':0.0378396,('PDT000384072.2':0.0167337,'PDT000638831.2':0.0152347)'':0.0298656)'':0.0317855,('PDT000352480.1':0.075704,(('PDT000196623.2':0.0532937,'PDT000150891.1':0.0610523)'':0.00806945,('PDT000556884.1':0.042745,('PDT000288554.2':0.0513276,('PDT000252486.1':0.0364734,('PDT000312785.2':0.0125401,'PDT000501933.1':0.0183652)'':0.0219971)'':0.00771761)'':0.00477087)'':0.0015671)'':0.0174321)'':0.00303856)'':0.0193959)'':-0.00308566)'':0.00817028,(('PDT000374996.1':0.054439,('PDT000095760.2':0.0473933,'PDT000645001.1':0.0387148)'':0.015556)'':0.0125469,(('PDT000358621.1':0.0488126,'PDT000140548.3':0.0654314)'':0.00317334,('PDT000140466.3':0.0670369,(('PDT000486498.1':0.0248904,(('PDT000523151.2':0.0234996,'PDT000502356.1':0.0269411)'':0.0017575,('PDT000707453.1':0.0258744,('PDT000081061.1':0.0134901,'PDT000497823.1':0.0170737)'':0.00671318)'':0.000642043)'':0.0019239)'':0.00800815,(('PDT000390235.2':0.031882,'PDT000598431.2':0.0391021)'':0.00461253,('PDT000501655.2':0.0402335,('PDT000095247.3':0.025419,(('PDT000717157.1':0.0130745,('PDT000288555.3':0.014634,('PDT000288543.3':0.0107042,'PDT000123256.1':0.00921255)'':0.00339615)'':0.00248513)'':0.00485479,('PDT000542082.2':0.0119752,('PDT000502287.1':0.0175912,('PDT000140575.4':0.0129122,('PDT000413918.2':0.0116716,('PDT000623975.2':0.00541157,'PDT000140454.3':0.00426231)'':0.00340725)'':0.00128115)'':0.00159565)'':0.00351174)'':0.000941414)'':0.00416986)'':0.00381012)'':0.00703089)'':-0.00213639)'':0.037675)'':-0.00561072)'':0.00829084)'':0.0259857)'':0.00617154,((('PDT000046598.1':0.0704516,('PDT000645159.1':0.0503277,'PDT000668328.2':0.0530453)'':0.0124699)'':0.0426065,('PDT000405239.1':0.0574126,('PDT000430823.1':0.0588905,('PDT000277723.2':0.0354952,('PDT000245421.5':0.0268474,('PDT000252968.2':0.0189305,'PDT000095737.2':0.0331436)'':0.00807788)'':0.0102504)'':0.00884372)'':0.00901333)'':0.0322264)'':0.0101178,((('PDT000292792.2':0.0640105,'PDT000420663.1':0.0610835)'':0.0318007,((('PDT000155784.2':0.0641648,'PDT000487030.1':0.0395662)'':0.0200905,('PDT000632001.1':0.081738,'PDT000289928.1':0.080092)'':0.00726701)'':0.00562501,(('PDT000332096.1':0.0113034,'PDT000352091.1':0.00770906)'':0.0616076,((('PDT000171814.1':0.0533185,'PDT000225584.1':0.0614265)'':0.0109732,(('PDT000541645.1':0.061159,'PDT000713766.1':0.060991)'':0.00675144,('PDT000426732.1':0.0494338,('PDT000600640.1':0.0430309,'PDT000682354.1':0.0412717)'':0.00883524)'':0.00425893)'':0.0079123)'':-0.00106168,(('PDT000502141.1':0.0941858,('PDT000280809.2':0.035989,'PDT000251696.1':0.0450579)'':0.0441713)'':0.00135219,('PDT000251846.1':0.0572963,('PDT000252564.1':0.0619147,('PDT000627436.1':0.0464619,('PDT000502906.1':0.0224389,'PDT000571193.1':0.0186691)'':0.0198099)'':0.00908615)'':0.00418572)'':0.00882394)'':0.000569017)'':0.00648951)'':0.00127339)'':0.00510869)'':-0.00574787,(((('PDT000619140.2':0.0332833,'PDT000132430.3':0.0307857)'':0.0116486,('PDT000405103.2':0.0524859,('PDT000201227.3':0.026234,'PDT000342265.2':0.0241222)'':0.00892209)'':0.0037258)'':0.027702,(('PDT000372686.1':0.0564811,('PDT000303767.3':0.0480319,'PDT000598365.2':0.0399681)'':0.0114034)'':0.00604311,('PDT000660401.1':0.0640238,('PDT000565865.1':0.0707411,('PDT000277739.2':0.0557447,'PDT000250026.1':0.0469133)'':0.00472444)'':0.00241523)'':0.00432539)'':0.00180016)'':0.0107182,(('PDT000372009.1':0.0670219,('PDT000644878.1':0.0456453,'PDT000607567.1':0.0520948)'':0.0200756)'':0.00672083,(('PDT000668484.2':0.0865083,(('PDT000395410.1':0.0559023,('PDT000411066.1':0.0522457,'PDT000651576.1':0.0571383)'':0.0101627)'':0.00225089,(('PDT000178607.2':0.0546538,'PDT000045593.1':0.0444974)'':0.00546303,('PDT000109469.2':0.0451303,('PDT000417516.1':0.0309358,'PDT000677991.1':0.0287352)'':0.0114074)'':0.00122001)'':0.0156249)'':0.0139564)'':-0.00215552,(('PDT000397876.1':0.0582383,'PDT000687839.1':0.0606097)'':0.0186293,('PDT000526207.1':0.100158,(('PDT000252644.1':0.0469806,'PDT000279211.1':0.0600394)'':0.0106634,(('PDT000719014.1':0.0412151,'PDT000228043.1':0.0374474)'':0.00971058,('PDT000510863.1':0.049544,'PDT000346517.1':0.0436356)'':0.0109596)'':0.00424868)'':0.0135208)'':-0.000611946)'':-0.000473439)'':0.00249359)'':0.000701307)'':0.000960944)'':0.0187937)'':0.0104548)'':0.00340547,((('PDT000346963.1':0.0419833,('PDT000535262.1':0.0380836,'PDT000215981.1':0.0405724)'':0.0151345)'':0.0824848,((('PDT000499515.1':0.0702813,'PDT000045580.1':0.0732167)'':0.00108345,('PDT000045893.3':0.0688004,'PDT000135924.3':0.0637816)'':0.0163008)'':0.00391847,(('PDT000443040.1':0.0785917,('PDT000717225.1':0.0745774,('PDT000645033.1':0.0575504,('PDT000538568.1':0.0321438,'PDT000639531.1':0.0297531)'':0.0114981)'':0.0133424)'':0.00464849)'':0.0015397,((('PDT000277713.2':0.0246555,'PDT000534962.1':0.0244092)'':0.0328776,('PDT000706410.1':0.0420129,('PDT000604969.1':0.0326118,'PDT000488278.1':0.0216815)'':0.00745602)'':0.0145509)'':0.00131089,((('PDT000368385.2':0.0528104,'PDT000421168.2':0.0455913)'':0.0122929,(('PDT000566165.1':0.0358666,'PDT000713725.1':0.0332529)'':0.00398482,('PDT000556973.1':0.0312274,('PDT000539271.1':0.0273793,('PDT000682553.1':0.02006,'PDT000713318.1':0.0229065)'':0.00557389)'':0.00250095)'':0.00705047)'':0.00869414)'':-0.00237181,(('PDT000557074.1':0.0435554,'PDT000645083.1':0.0522795)'':0.00272123,(('PDT000403079.1':0.0223962,('PDT000668473.2':0.0159738,'PDT000502155.1':0.0153316)'':0.00866175)'':0.0063957,('PDT000233319.1':0.0382513,('PDT000566186.1':0.0228995,('PDT000584791.1':0.0154933,'PDT000346693.1':0.0130103)'':0.00730374)'':0.010797)'':-0.000451096)'':0.00609885)'':0.0025913)'':0.00848591)'':0.0140536)'':0.00281965)'':0.041659)'':-0.0280821,(((('PDT000003796.2':0.0841201,'PDT000394845.1':0.0826849)'':0.00461917,(('PDT000150228.2':0.0229388,'PDT000470054.1':0.0254247)'':0.0149155,('PDT000258402.2':0.0246717,'PDT000311238.2':0.0255526)'':0.0214577)'':0.0199711)'':0.00920075,(('PDT000618739.1':0.0588201,('PDT000617800.1':0.0205417,'PDT000644908.1':0.0298967)'':0.0267272)'':0.0240741,('PDT000188178.2':0.0650815,('PDT000562855.1':0.0790084,(('PDT000188115.2':0.0576016,'PDT000417488.1':0.0415647)'':0.0100408,('PDT000095724.2':0.0279824,'PDT000672197.1':0.0332121)'':0.0200856)'':0.0161475)'':0.00283232)'':0.000750201)'':0.000711318)'':0.0065024,((('PDT000140456.3':0.0272684,('PDT000283358.3':0.0242375,'PDT000603611.2':0.0255084)'':0.00692333)'':0.0151023,('PDT000498114.1':0.028702,((('PDT000500762.2':0.0127387,'PDT000130969.1':0.0154555)'':0.00648575,('PDT000137339.3':0.0303713,('PDT000495634.2':0.0245116,'PDT000691139.1':0.0212854)'':0.00197832)'':0.00356983)'':0.00232198,(('PDT000253423.3':0.0184201,('PDT000224238.3':0.020715,'PDT000029750.3':0.0132068)'':0.00669547)'':0.00328733,('PDT000063253.3':0.0259871,('PDT000063170.3':0.0140092,'PDT000137671.3':0.0148891)'':0.00767567)'':0.00518797)'':0.00242116)'':0.00402751)'':0.00343535)'':0.0441136,((('PDT000281214.2':0.0457682,(('PDT000341205.1':0.0225185,'PDT000376078.1':0.0232427)'':0.0163331,('PDT000435581.1':0.026323,('PDT000150239.2':0.0228901,'PDT000178587.2':0.0224844)'':0.00140344)'':0.00964341)'':0.00421855)'':0.00446201,(('PDT000333670.1':0.0396448,('PDT000258437.2':0.0383029,'PDT000551887.1':0.0316435)'':0.00340852)'':0.00511391,(('PDT000471040.1':0.0178767,'PDT000045583.1':0.0170196)'':0.0193749,('PDT000565979.1':0.0353515,'PDT000660677.1':0.0390214)'':0.00795506)'':0.0052163)'':0.00256322)'':0.0317262,(('PDT000346147.1':0.0727048,('PDT000502308.1':0.038725,'PDT000346915.1':0.0504513)'':0.00676708)'':0.0342088,(('PDT000181784.2':0.0936711,'PDT000233302.2':0.107409)'':0.0102595,((('PDT000008588.2':0.0580156,('PDT000395416.1':0.0424686,('PDT000317677.1':0.0182545,'PDT000264643.2':0.0142425)'':0.0394512)'':0.00834611)'':0.006997,('PDT000541626.1':0.0542601,('PDT000517544.1':0.0598469,('PDT000333063.1':0.0404592,'PDT000566227.1':0.0377305)'':0.0129812)'':0.000215168)'':0.00458455)'':0.00192369,(('PDT000181696.2':0.0617961,'PDT000537836.1':0.0740809)'':0.00605017,(((('PDT000147470.3':0.0150845,'PDT000517422.2':0.0170274)'':0.00377278,('PDT000233965.3':0.0191786,'PDT000115351.3':0.0150558)'':0.00403197)'':0.0421344,('PDT000607492.1':0.0715418,('PDT000535506.1':0.0673023,('PDT000342670.1':0.0649711,(('PDT000271784.1':0.0320231,'PDT000287623.1':0.0385248)'':0.000631813,('PDT000136527.2':0.0210701,'PDT000395241.1':0.021529)'':0.0126734)'':0.0113483)'':0.00257294)'':0.00487503)'':-0.00199722)'':0.00396095,(('PDT000439490.1':0.0585699,('PDT000668410.1':0.0382986,('PDT000694374.1':0.0281507,'PDT000107543.1':0.0293809)'':0.00578836)'':0.0112887)'':0.00816912,((('PDT000405292.1':0.039507,'PDT000661240.1':0.0375054)'':0.00907502,(('PDT000399584.1':0.0455116,'PDT000555722.1':0.0410806)'':0.00942968,('PDT000607579.1':0.0468274,('PDT000372160.1':0.0415973,'PDT000682304.1':0.0321539)'':0.00595036)'':0.00576912)'':0.00086933)'':0.00374692,(('PDT000325583.1':0.0456533,'PDT000619397.1':0.0699487)'':0.0104715,(('PDT000367273.1':0.0342085,'PDT000455211.1':0.0458032)'':0.0185494,('PDT000501830.1':0.0596648,('PDT000433308.1':0.0474073,('PDT000132149.2':0.0373066,('PDT000618730.1':0.0265165,('PDT000188102.3':0.0261925,('PDT000242903.1':0.0294274,('PDT000719204.1':0.0104163,'PDT000251426.1':0.0124761)'':0.00821047)'':0.00440562)'':0.00485217)'':0.00508537)'':0.00547918)'':0.00509202)'':0.00184132)'':0.00366155)'':-0.00392287)'':0.00408703)'':0.00115585)'':0.00833457)'':-0.00510318)'':0.0172544)'':0.00295683)'':-0.0154763)'':0.00349508)'':0.00653182)'':0.00645718)'':0.018771)'':0.0302053)'':0.00362976)'':0.00773345)'':-0.00888336)'':0.000746337)'':0.00697069)'':-0.00145595)'':0.00049937)'':0.00672929)'':-0.00282949)'':-0.00514713,((((('PDT000594314.1':0.069423,('PDT000556897.1':0.0596528,'PDT000017731.2':0.0736802)'':0.0106255)'':0.00199181,(('PDT000008481.2':0.0996577,('PDT000543323.1':0.0905609,('PDT000718831.1':0.0748558,(('PDT000652523.1':0.07171,('PDT000496983.1':0.0249133,'PDT000502316.1':0.0318355)'':0.0252106)'':0.00265311,('PDT000627607.1':0.0558014,('PDT000557037.1':0.0555239,('PDT000719044.1':0.0305365,'PDT000640344.1':0.0303999)'':0.0162592)'':0.001642)'':0.00203816)'':0.00791922)'':0.00744124)'':0.0120487)'':-0.0152804,(('PDT000526091.2':0.0283306,('PDT000189615.3':0.0179525,'PDT000098737.3':0.0145549)'':0.0118415)'':0.00308655,('PDT000486519.1':0.0538937,(('PDT000081057.1':0.023163,('PDT000480998.2':0.00790112,'PDT000498113.1':0.00673158)'':0.0200308)'':0.00514191,((('PDT000258346.3':0.00447826,'PDT000568704.2':0.00569524)'':0.00761697,('PDT000565852.2':0.0164803,('PDT000140476.3':0.0144858,'PDT000140533.3':0.0126456)'':0.00408612)'':0.00162229)'':0.00310055,('PDT000383953.2':0.030197,('PDT000104681.3':0.0232689,('PDT000668369.2':0.013096,('PDT000083131.3':0.0122964,('PDT000052126.3':0.00951492,'PDT000598382.2':0.00845388)'':0.00246084)'':0.0036606)'':0.00624795)'':0.00221402)'':-0.00186907)'':0.0127298)'':0.0139934)'':-0.00919593)'':0.0323061)'':0.0118335)'':0.0406369,(((('PDT000668326.1':0.069558,('PDT000343402.1':0.048382,('PDT000095802.2':0.0282369,'PDT000396476.1':0.0260312)'':0.0302256)'':0.0171019)'':-6.09384e-05,(('PDT000500735.1':0.0770735,'PDT000682556.1':0.0677735)'':0.00789612,('PDT000661991.1':0.0713589,('PDT000317611.1':0.0526552,('PDT000203081.2':0.0535056,'PDT000550332.1':0.0394792)'':0.00728291)'':0.0103128)'':0.00394758)'':0.00048814)'':0.00262306,(('PDT000426543.1':0.0670507,('PDT000083501.1':0.0431858,('PDT000216008.1':0.0337714,'PDT000228303.1':0.0381501)'':0.00842444)'':0.00548025)'':0.00293796,(('PDT000565990.1':0.0513212,'PDT000215940.1':0.0438892)'':0.0143052,('PDT000497393.1':0.0526269,('PDT000251905.1':0.0360188,('PDT000557314.1':0.0229304,'PDT000672648.1':0.0187773)'':0.0138294)'':0.00450426)'':0.00291538)'':0.00765644)'':0.00738829)'':0.0293515,(('PDT000316907.1':0.0513648,('PDT000625515.1':0.0426764,('PDT000032342.1':0.00360547,'PDT000032343.1':0.00367487)'':0.0361888)'':0.00742397)'':0.0920144,((('PDT000598379.1':0.0653306,('PDT000149344.2':0.0664454,'PDT000494514.1':0.0829386)'':0.00325536)'':0.028293,((('PDT000201874.2':0.0324665,'PDT000238219.2':0.0383498)'':0.0249724,('PDT000216614.3':0.0586445,(('PDT000625419.1':0.0162428,'PDT000553144.1':0.0173315)'':0.0092866,('PDT000625457.1':0.0196606,'PDT000707302.1':0.0239804)'':0.0130306)'':0.0146134)'':0.0160417)'':0.0214426,('PDT000323906.1':0.088032,('PDT000354896.1':0.083018,('PDT000425892.1':0.078111,(('PDT000325063.1':0.0232607,'PDT000713358.1':0.0157864)'':0.0411326,(('PDT000413141.1':0.0516999,'PDT000641738.1':0.0407675)'':0.0016523,('PDT000566019.1':0.03553,('PDT000426815.1':0.037956,'PDT000251903.1':0.0446012)'':0.00374337)'':0.0116053)'':0.00838969)'':0.0183225)'':0.00377476)'':0.00120105)'':-0.00359196)'':0.00685666)'':0.000579047,((('PDT000103593.2':0.0901994,('PDT000581014.1':0.0541977,'PDT000502521.1':0.0560683)'':0.0131221)'':0.0091884,(('PDT000694407.1':0.0365799,'PDT000714677.1':0.0432237)'':0.0316469,(('PDT000653341.1':0.0524973,'PDT000687375.1':0.0841387)'':0.0187318,('PDT000468341.1':0.0541244,('PDT000279205.1':0.0722521,('PDT000637616.1':0.0387687,'PDT000346097.1':0.0553951)'':0.0104915)'':0.0067196)'':0.0117737)'':0.00639168)'':0.00105286)'':0.0025769,(('PDT000566010.1':0.0889237,('PDT000282210.2':0.060493,('PDT000204905.2':0.0522181,'PDT000026889.2':0.0638389)'':0.0092325)'':0.017417)'':0.0307053,(('PDT000160301.2':0.0722496,('PDT000520857.1':0.0626706,('PDT000687852.1':0.0526414,('PDT000228200.1':0.0342677,'PDT000228321.1':0.0245673)'':0.0114275)'':0.00770663)'':0.0048852)'':0.00962742,('PDT000619418.1':0.0700003,('PDT000523070.1':0.0734373,('PDT000687847.1':0.051028,(('PDT000439438.1':0.0553454,('PDT000584734.1':0.0255492,'PDT000216025.1':0.0247498)'':0.0175562)'':0.00326817,('PDT000495094.1':0.0277275,('PDT000617797.1':0.0368476,'PDT000252528.1':0.02897)'':0.00541563)'':0.0108947)'':0.00194118)'':0.0108686)'':0.00975755)'':-0.00871511)'':0.0183255)'':-0.00168571)'':0.00940379)'':0.0248333)'':-0.0194409)'':0.0241326)'':0.0196277,(((('PDT000495026.1':0.0777945,'PDT000645075.1':0.0890285)'':0.0154305,(('PDT000104578.2':0.00459271,'PDT000713219.1':0.00758779)'':0.062687,('PDT000619160.2':0.0602312,('PDT000653433.2':0.048499,('PDT000366168.2':0.0460872,('PDT000275625.3':0.0243045,('PDT000247387.3':0.0174659,('PDT000140516.3':0.0105852,('PDT000258480.3':0.00963152,'PDT000372766.2':0.00949628)'':0.00219359)'':0.00415678)'':0.00367817)'':0.0285416)'':0.00556636)'':0.00424169)'':0.0193793)'':0.0105774)'':0.0156393,(('PDT000368899.1':0.11327,('PDT000251807.1':0.116971,(('PDT000482534.1':0.0168086,'PDT000523263.1':0.0192079)'':0.0602137,(('PDT000309310.2':0.0564825,'PDT000368900.1':0.0502395)'':0.0115826,('PDT000525372.1':0.0317276,'PDT000502318.1':0.0300511)'':0.0298648)'':0.019721)'':0.0145553)'':0.0139104)'':-0.011222,((((('PDT000379417.1':0.0475618,'PDT000499501.1':0.0538482)'':0.0137239,('PDT000502328.1':0.0626045,'PDT000648086.1':0.0670235)'':0.00734056)'':0.0112813,('PDT000228022.1':0.0632089,('PDT000252698.1':0.0423799,('PDT000045617.1':0.0340422,'PDT000251402.1':0.0229061)'':0.010034)'':0.0196977)'':0.0339493)'':0.0165987,(('PDT000633613.1':0.0359334,('PDT000538421.1':0.0266124,'PDT000618793.1':0.0376276)'':0.00936904)'':0.0564671,(('PDT000345978.1':0.0885047,('PDT000129555.1':0.0757571,(('PDT000400856.2':0.0206092,'PDT000562635.2':0.025344)'':0.0281708,('PDT000719106.1':0.0493625,'PDT000501967.1':0.0374343)'':0.00649774)'':0.0130799)'':0.0051964)'':0.00475775,((('PDT000354947.2':0.0288725,'PDT000502470.1':0.0302715)'':0.000281245,('PDT000409221.2':0.0296926,'PDT000709062.1':0.0191072)'':0.0106508)'':0.0328982,('PDT000156984.3':0.0673091,('PDT000548827.2':0.0461921,(('PDT000040428.4':0.0257263,('PDT000148556.3':0.022748,'PDT000399357.2':0.0238117)'':0.00448513)'':0.00357917,('PDT000571594.2':0.0334653,('PDT000288526.3':0.0228914,'PDT000288557.3':0.0313661)'':0.00580811)'':0.00246434)'':0.0141056)'':0.0110848)'':-0.000969441)'':0.0243143)'':0.00937335)'':0.00141191)'':-0.00257045,(('PDT000652477.1':0.0623181,'PDT000716411.1':0.0526279)'':0.0580812,(('PDT000425931.1':0.0574954,('PDT000004596.2':0.0629977,'PDT000233093.1':0.0349404)'':0.00748597)'':0.0286953,((('PDT000188176.2':0.0134398,'PDT000191803.2':0.0122875)'':0.0464022,('PDT000674993.1':0.0391951,'PDT000391856.1':0.0449379)'':0.0221806)'':0.00751684,('PDT000399973.1':0.109295,(('PDT000149346.2':0.0651861,('PDT000413213.1':0.0329987,('PDT000191873.2':0.0160259,'PDT000095727.2':0.0106314)'':0.013951)'':0.0293945)'':0.0146783,(('PDT000689102.1':0.0439362,(('PDT000509959.2':0.02521,'PDT000502416.1':0.0191401)'':0.00413874,(('PDT000337023.2':0.0122852,'PDT000498125.1':0.0130925)'':0.00529038,('PDT000295968.3':0.0177895,'PDT000137588.3':0.0123932)'':0.00292309)'':0.00572625)'':0.0210152)'':0.017685,(('PDT000318983.1':0.0613864,(('PDT000281522.3':0.0218421,'PDT000645037.1':0.0140681)'':0.0254214,('PDT000502156.1':0.0490492,('PDT000420667.1':0.0149424,'PDT000639559.1':0.0152054)'':0.0254161)'':0.00111666)'':0.00871515)'':-0.00193887,('PDT000360989.1':0.0725877,(('PDT000502132.1':0.0479471,'PDT000228365.1':0.0470491)'':0.00867113,(('PDT000581877.1':0.0536632,'PDT000251588.1':0.0450398)'':0.00263666,(('PDT000408769.1':0.0449945,'PDT000318967.1':0.0411558)'':0.00413344,(('PDT000468257.1':0.0338254,'PDT000505384.1':0.0303903)'':0.00313622,('PDT000083149.2':0.0303931,('PDT000671188.1':0.0266216,('PDT000310640.2':0.0201728,'PDT000249867.1':0.0264952)'':0.00491221)'':0.00161255)'':0.00450258)'':0.00438041)'':0.003269)'':0.00255584)'':0.00720758)'':-0.00325826)'':0.00322692)'':0.00337155)'':0.0174818)'':-0.00753628)'':0.00862968)'':0.00583265)'':-0.00518603)'':0.00546074)'':0.0104109)'':-0.00413185,((('PDT000414202.1':0.118184,('PDT000625975.1':0.098428,'PDT000168845.1':0.103178)'':0.00745631)'':0.0256781,(('PDT000459427.1':0.0974182,('PDT000414189.1':0.0790799,('PDT000417173.1':0.0469865,'PDT000225233.1':0.038463)'':0.0153259)'':0.0332654)'':0.00959437,(((('PDT000417114.1':0.0394689,'PDT000417189.1':0.0325812)'':0.0363643,('PDT000168744.1':0.0107864,'PDT000168791.1':0.00752571)'':0.0522191)'':0.0388998,(('PDT000168869.1':0.065956,'PDT000298023.1':0.055224)'':0.0063741,(('PDT000417089.1':0.0803366,'PDT000425677.1':0.0675464)'':0.0073939,('PDT000219683.1':0.0674773,'PDT000346965.1':0.0705597)'':0.0034046)'':0.00187252)'':0.0156147)'':0.00303825,('PDT000720464.1':0.0907879,(('PDT000417120.1':0.0659887,('PDT000276841.1':0.0475372,'PDT000396736.1':0.0403131)'':0.00797667)'':0.0108377,((('PDT000417090.1':0.0379054,'PDT000168729.1':0.0374258)'':0.00462735,('PDT000168717.1':0.0211227,'PDT000168747.1':0.020356)'':0.0137907)'':0.0217944,(('PDT000661839.1':0.0173167,'PDT000661843.1':0.0310912)'':0.0201997,('PDT000168743.1':0.036696,'PDT000414210.1':0.0455978)'':0.00266109)'':0.00578474)'':0.00896746)'':0.0343064)'':0.00176345)'':0.00538143)'':0.0251481)'':-0.0163608,(((('PDT000004355.2':0.0547907,'PDT000027901.2':0.0773303)'':0.0418204,('PDT000490589.1':0.0876523,'PDT000521032.1':0.0873407)'':0.00895157)'':0.00631004,('PDT000300830.2':0.0797534,(('PDT000520786.1':0.0680486,('PDT000088292.2':0.0621845,('PDT000088305.2':0.0330092,'PDT000131798.2':0.0308102)'':0.0169003)'':0.00679622)'':-0.000296477,(('PDT000175888.2':0.0131553,'PDT000652953.1':0.0181905)'':0.0358745,('PDT000289898.1':0.0535183,('PDT000667966.1':0.0253719,'PDT000233277.1':0.0230421)'':0.0207018)'':0.00469242)'':0.0123241)'':0.00766243)'':0.026415)'':0.00013288,(((('PDT000423984.1':0.0300001,'PDT000534792.1':0.0285183)'':0.0569527,(('PDT000423930.1':0.0676909,'PDT000423998.1':0.0655361)'':0.0210404,('PDT000423848.1':0.0935583,'PDT000687959.1':0.0827217)'':0.0025186)'':0.00676531)'':0.00107867,(('PDT000674809.1':0.0662915,('PDT000656991.1':0.025878,('PDT000656984.1':0.0243408,'PDT000173623.1':0.0235482)'':0.00404944)'':0.0498402)'':0.0163964,(('PDT000708209.1':0.042637,'PDT000168775.1':0.058082)'':0.00710619,('PDT000168714.1':0.0316555,'PDT000168790.1':0.0288232)'':0.0178149)'':0.0466842)'':0.0019279)'':0.00137928,(('PDT000015785.3':0.0183416,('PDT000014119.2':0.0176975,(('PDT000014120.3':0.0169156,(('PDT000015750.3':0.0131836,'PDT000014118.2':0.0141934)'':0.00195567,('PDT000015772.3':0.0148444,'PDT000014121.3':0.0147326)'':0.00158098)'':0.000861666)'':0.00055654,(('PDT000014147.3':0.00953479,'PDT000014136.2':0.0123275)'':0.00374388,('PDT000015752.3':0.0170555,(('PDT000014124.3':0.0134192,'PDT000015766.3':0.0144901)'':0.00113313,('PDT000015771.3':0.0143025,('PDT000014122.3':0.0164137,('PDT000014123.3':0.0127536,'PDT000014117.3':0.0129305)'':0.00189042)'':0.00101673)'':0.000500972)'':0.000477072)'':-0.000213581)'':0.00112231)'':0.00148744)'':0.00110029)'':0.0736576,((('PDT000719226.1':0.0669403,('PDT000428368.1':0.0577811,('PDT000354888.1':0.030753,'PDT000045624.1':0.0199414)'':0.0336881)'':0.0117381)'':0.00959041,('PDT000030565.1':0.0854582,(('PDT000014172.3':0.0778453,('PDT000045586.1':0.0726711,(('PDT000215772.2':0.0677289,'PDT000389332.1':0.0673281)'':0.00203331,(('PDT000674963.1':0.0447988,'PDT000391862.1':0.0605512)'':0.00701229,('PDT000668904.1':0.0357085,'PDT000346045.1':0.0411847)'':0.0119185)'':0.00292801)'':0.00811998)'':0.0021349)'':-0.000430105,('PDT000350402.1':0.0666684,('PDT000343389.1':0.0625642,(('PDT000298070.1':0.0534115,('PDT000589998.1':0.0439135,'PDT000088795.1':0.0378942)'':0.00935064)'':0.000516811,('PDT000346969.1':0.0518007,('PDT000426011.1':0.0481757,('PDT000714557.1':0.0372837,('PDT000437914.1':0.00987125,('PDT000102022.2':0.000770526,'PDT000102023.2':0.000628774)'':0.0110153)'':0.0239669)'':0.00427379)'':0.00719313)'':0.000638694)'':0.00938158)'':0.00781874)'':-0.005251)'':0.00939763)'':-0.00669818)'':0.00469044,(('PDT000584541.1':0.0996089,('PDT000289876.1':0.0853027,('PDT000139180.2':0.0716695,('PDT000672223.1':0.074799,'PDT000393655.1':0.064232)'':0.00584846)'':0.00896917)'':0.00224269)'':0.0121223,(('PDT000188145.3':0.058731,('PDT000675054.1':0.067434,'PDT000346035.1':0.073531)'':0.00824004)'':0.0200023,(('PDT000661161.1':0.100468,('PDT000168799.1':0.0898246,(('PDT000168854.1':0.0842719,('PDT000215983.2':0.0321508,('PDT000676119.2':0.0121166,'PDT000130968.1':0.0218054)'':0.00754167)'':0.0506601)'':-0.00249018,('PDT000707292.2':0.0541356,('PDT000318154.1':0.0480791,('PDT000289929.1':0.0391307,('PDT000589916.1':0.0187124,'PDT000396761.1':0.0115185)'':0.0123922)'':0.00350511)'':0.00476394)'':0.020033)'':0.0105452)'':0.00853026)'':-0.00443971,(((('PDT000425628.1':0.0395852,'PDT000396739.1':0.032711)'':0.0315739,('PDT000425603.1':0.0639356,('PDT000520937.1':0.060072,'PDT000251735.1':0.06003)'':0.0126574)'':0.00600653)'':0.00392249,(('PDT000233331.1':0.0358988,('PDT000534869.1':0.0245785,'PDT000535158.1':0.0218279)'':0.0109428)'':0.0217598,('PDT000231455.2':0.0408875,('PDT000622392.1':0.0326826,('PDT000294787.1':0.0282357,'PDT000476468.1':0.02654)'':0.00439654)'':0.00716044)'':0.00883671)'':0.0196051)'':0.004244,(('PDT000643768.1':0.0759103,('PDT000346977.1':0.0561513,('PDT000168871.1':0.0571632,('PDT000168877.1':0.0379242,'PDT000298052.1':0.0510144)'':0.012616)'':0.00887372)'':0.0300591)'':0.00947264,((('PDT000668915.1':0.00320943,'PDT000668916.1':0.00247579)'':0.0666115,(('PDT000425459.1':0.0492291,'PDT000008470.2':0.0571049)'':0.00629103,('PDT000584544.1':0.051832,'PDT000259738.1':0.0478326)'':0.0107357)'':0.00491576)'':0.00572031,(('PDT000168745.1':0.0539839,'PDT000171423.1':0.0416143)'':0.0360743,(('PDT000661685.1':0.0686201,('PDT000136094.2':0.0517264,('PDT000445977.1':0.0605131,'PDT000250020.1':0.0530669)'':0.0054896)'':0.00773473)'':0.000691982,('PDT000251736.1':0.0724762,('PDT000468896.1':0.067515,('PDT000520871.1':0.0522439,('PDT000521056.1':0.0227779,'PDT000521112.1':0.0327393)'':0.0241373)'':0.0101724)'':0.00167903)'':0.00603784)'':0.0102523)'':-0.00340751)'':0.00646502)'':-0.00199734)'':0.00325099)'':0.00442185)'':-0.000983788)'':-0.0021157)'':0.00134345)'':0.00371604)'':0.00774922)'':0.000321326)'':0.0109935)'':0.0433375)'':0.00506125,((((('PDT000168709.1':0.0383451,'PDT000215974.1':0.0477998)'':0.0193428,(('PDT000168865.1':0.0138818,('PDT000267153.1':0.0176114,'PDT000304333.1':0.0167694)'':0.00184301)'':0.0174955,(('PDT000168706.1':0.0267836,'PDT000168727.1':0.0282417)'':0.0074488,('PDT000168707.1':0.0297916,'PDT000168773.1':0.033176)'':0.000663273)'':0.00566665)'':0.0119093)'':0.0689012,((('PDT000633767.1':0.0872579,('PDT000145739.1':0.0862155,('PDT000432061.1':0.0857535,(('PDT000432051.1':0.0726499,('PDT000097046.1':0.0151821,'PDT000297983.1':0.0153033)'':0.0315219)'':-0.000356802,('PDT000191144.1':0.0510584,('PDT000424025.1':0.0401836,'PDT000534803.1':0.0411733)'':0.00610911)'':0.00546482)'':0.0141457)'':0.00783632)'':0.0123225)'':0.00188627,(('PDT000445085.1':0.0889332,(('PDT000427159.1':0.0404228,'PDT000136062.3':0.0678702)'':0.00745058,('PDT000664535.1':0.0507252,'PDT000668942.1':0.0449503)'':0.00326302)'':0.0103944)'':0.0139549,('PDT000253678.1':0.0615258,('PDT000427085.1':0.0728567,(('PDT000540589.1':0.0600245,('PDT000136099.2':0.0230956,'PDT000158301.1':0.0231171)'':0.0346886)'':0.0106855,('PDT000145741.1':0.0527971,('PDT000197012.1':0.0402462,'PDT000216782.1':0.0397577)'':0.027801)'':0.00908989)'':0.00168199)'':0.00550834)'':0.0022761)'':0.0222701)'':0.0120836,('PDT000466236.1':0.121765,((('PDT000417160.1':0.0460476,'PDT000672222.1':0.049736)'':0.0266475,(('PDT000710426.1':0.0726368,'PDT000171808.1':0.0540802)'':0.00580203,('PDT000280682.2':0.0603514,('PDT000651117.1':0.0668744,'PDT000251422.1':0.0551296)'':0.00835405)'':0.00618862)'':0.000932646)'':0.00468825,(('PDT000168719.1':0.11395,('PDT000251528.1':0.0761097,'PDT000346927.1':0.0859163)'':0.0197815)'':-0.00141041,((('PDT000218030.2':0.0411218,'PDT000006340.2':0.0390751)'':0.0157714,('PDT000188148.2':0.0548093,('PDT000042609.1':0.0526024,'PDT000557184.1':0.0420625)'':0.00720664)'':0.00553473)'':0.00744699,('PDT000289887.1':0.0919286,('PDT000396801.1':0.0890598,('PDT000426558.1':0.078914,(('PDT000299829.2':0.0632744,'PDT000556932.1':0.0445526)'':0.012594,('PDT000251393.1':0.0527541,('PDT000252470.1':0.0569676,(('PDT000251524.1':0.0369169,'PDT000251527.1':0.0355989)'':0.00537063,('PDT000251390.1':0.027827,('PDT000248840.1':0.0219872,'PDT000250175.1':0.0222155)'':0.00907727)'':0.00562386)'':0.00839282)'':0.00314347)'':0.0047914)'':0.00534059)'':0.00588326)'':0.0026981)'':-0.00394675)'':0.0154967)'':0.00267063)'':0.0109521)'':-0.00618265)'':0.00345154)'':0.0143008,(((('PDT000557289.1':0.109588,(('PDT000426650.1':0.0471541,('PDT000354231.1':0.0553806,('PDT000128800.2':0.0431959,'PDT000652847.1':0.0326506)'':0.0079361)'':0.0162684)'':0.0346193,(('PDT000417187.1':0.08291,('PDT000433723.1':0.0753775,'PDT000168873.1':0.0852355)'':0.00554198)'':0.00388619,('PDT000019609.1':0.0790196,('PDT000129099.2':0.0663311,'PDT000141262.1':0.0629439)'':0.0137294)'':0.0117672)'':0.00650755)'':0.00869099)'':-0.00496939,((('PDT000423981.1':0.0950637,'PDT000049402.1':0.0849583)'':0.00939378,(('PDT000590027.1':0.0669046,('PDT000708196.1':0.0618658,'PDT000215979.1':0.0607242)'':0.00212536)'':0.00638697,(('PDT000645481.2':0.0669749,'PDT000248851.1':0.0539871)'':0.0123817,('PDT000668902.1':0.0603473,('PDT000661174.1':0.0503792,'PDT000124683.1':0.0483132)'':0.00680451)'':0.0102534)'':0.00191279)'':0.00955053)'':-0.00244802,((('PDT000417172.1':0.0681534,('PDT000417096.1':0.0514335,'PDT000459414.1':0.0509645)'':0.00282155)'':0.00435233,('PDT000168837.1':0.0687828,(('PDT000417095.1':0.0476399,'PDT000168816.1':0.0570131)'':0.0131076,('PDT000168713.1':0.0511603,('PDT000168766.1':0.0211183,'PDT000270538.1':0.0220649)'':0.0227539)'':0.00348062)'':0.00649796)'':0.00686816)'':0.0256774,(('PDT000242482.1':0.0717682,('PDT000427684.1':0.0468194,'PDT000520934.1':0.0524999)'':0.0147381)'':0.0103129,(('PDT000426002.1':0.0693981,('PDT000425767.1':0.0717295,'PDT000520848.1':0.0700105)'':0.00444442)'':0.0010572,(('PDT000103026.1':0.0510244,('PDT000093007.2':0.0413252,'PDT000396733.1':0.0512978)'':0.0121537)'':0.0103144,('PDT000289909.1':0.0750386,('PDT000652948.1':0.0626786,'PDT000645620.2':0.0669804)'':0.00480144)'':0.00422869)'':0.00223655)'':0.00325733)'':0.000954242)'':0.00547726)'':0.0024959)'':0.0203996,((('PDT000625084.1':0.112838,('PDT000661160.1':0.124414,('PDT000503007.1':0.0808726,('PDT000298087.1':0.0633829,('PDT000720470.1':0.0556952,'PDT000298062.1':0.0608368)'':0.0190341)'':0.0029334)'':0.00450681)'':0.00575743)'':0.0189278,('PDT000250021.1':0.125347,(('PDT000251431.1':0.0551097,('PDT000397849.1':0.0545133,('PDT000482512.1':0.0391336,'PDT000662553.1':0.051395)'':0.00485011)'':0.00800958)'':0.00902254,(('PDT000651129.1':0.0743899,('PDT000425906.1':0.0604577,'PDT000352502.1':0.0543543)'':0.00880563)'':0.00996549,('PDT000346981.1':0.0829173,('PDT000645035.1':0.0564987,('PDT000346925.1':0.0648967,('PDT000463681.1':0.0522164,('PDT000346931.1':0.0410882,('PDT000346928.1':0.0214431,'PDT000346980.1':0.0227222)'':0.00647279)'':0.0231189)'':0.00741595)'':0.00241453)'':0.00845381)'':-0.00333362)'':0.00733466)'':0.0209006)'':-0.00512509)'':-0.000655419,(((('PDT000391661.1':0.0555314,('PDT000250234.1':0.0420084,'PDT000251818.1':0.0538966)'':0.00770402)'':0.0046914,('PDT000583354.2':0.0541627,('PDT000559296.2':0.0491095,'PDT000252475.1':0.045751)'':0.014246)'':0.00212574)'':0.0192561,('PDT000359186.1':0.0877248,(('PDT000482831.1':0.067555,('PDT000399647.1':0.0564992,'PDT000541629.1':0.0522278)'':0.015474)'':0.00479631,('PDT000109447.2':0.040074,('PDT000325538.1':0.0339431,('PDT000667752.2':0.0313437,'PDT000228057.1':0.026979)'':0.00322955)'':0.00484159)'':0.0239175)'':0.0155881)'':-0.00251455)'':0.00442345,(('PDT000032350.1':0.00105503,('PDT000032358.1':0.00118235,'PDT000032366.1':0.00111512)'':0.000343553)'':0.0832686,((('PDT000146000.2':0.036979,'PDT000483978.1':0.0448901)'':0.0267891,('PDT000242923.1':0.0528811,'PDT000463708.1':0.0483279)'':0.0210384)'':0.00588487,((('PDT000502122.1':0.0437502,'PDT000708220.1':0.0431267)'':0.0211404,('PDT000346926.1':0.0356943,('PDT000294391.1':0.00788627,'PDT000414204.1':0.00959483)'':0.0268175)'':0.0234825)'':0.00638395,(('PDT000553122.1':0.0564352,(('PDT000294399.1':0.0336027,'PDT000395144.1':0.0261003)'':0.0226911,('PDT000396847.1':0.055935,('PDT000249761.1':0.0453201,('PDT000625590.1':0.0312722,('PDT000539303.1':0.017323,'PDT000660635.1':0.0148968)'':0.00561195)'':0.00726227)'':0.00630229)'':0.00391793)'':0.0089776)'':-0.00348308,('PDT000556949.1':0.0869302,(('PDT000523152.1':0.0631562,('PDT000397966.1':0.0367549,'PDT000645026.1':0.0384403)'':0.0147612)'':0.00271225,(('PDT000243039.1':0.0407978,'PDT000306267.1':0.0426554)'':0.00711922,(('PDT000435645.1':0.0520688,(('PDT000258745.2':0.0249681,'PDT000332083.1':0.0192587)'':0.0102731,('PDT000344373.1':0.0316091,'PDT000426275.1':0.0327212)'':0.00761408)'':0.00195847)'':0.00626563,(('PDT000325548.1':0.0335207,('PDT000325649.1':0.0236419,'PDT000355757.1':0.0269513)'':0.00387559)'':0.0075789,(('PDT000309335.2':0.0235023,'PDT000251791.1':0.0251607)'':0.00427074,('PDT000509505.1':0.00675143,'PDT000669187.1':0.00809957)'':0.018299)'':0.00962895)'':0.00538211)'':0.0047117)'':8.63888e-05)'':0.0126286)'':-0.0109985)'':0.00693529)'':0.00804972)'':0.0133243)'':-0.00198499)'':0.00587221)'':0.00429061)'':0.00160396,((('PDT000213995.2':0.0590512,'PDT000417505.1':0.0442018)'':0.0310939,('PDT000248622.1':0.0858266,(('PDT000690260.1':0.058361,('PDT000324602.2':0.0443245,(('PDT000490365.2':0.0164755,('PDT000480994.2':0.00813473,'PDT000131007.3':0.00945967)'':0.00204314)'':0.0173891,('PDT000141122.3':0.0300409,('PDT000135822.3':0.0115211,('PDT000400785.2':0.00715957,('PDT000152931.3':0.00435716,'PDT000342271.2':0.00560598)'':0.00629216)'':0.00278659)'':0.0113085)'':0.0100458)'':0.0193696)'':0.00339326)'':0.00266895,('PDT000426898.2':0.0847828,('PDT000518477.2':0.074354,(('PDT000161983.3':0.0622415,(('PDT000318852.2':0.0246991,('PDT000092545.1':0.020013,'PDT000406994.1':0.0303382)'':0.0047501)'':0.00685223,('PDT000590227.2':0.0221117,('PDT000502396.1':0.0121851,'PDT000502439.1':0.0228763)'':0.00523307)'':0.0105849)'':0.0156796)'':-0.00646774,(('PDT000372143.2':0.0593068,('PDT000162958.4':0.0400485,(('PDT000536926.2':0.0262656,'PDT000502382.1':0.0280304)'':0.00676743,('PDT000502445.1':0.0318708,'PDT000502564.1':0.0365293)'':0.00707402)'':0.00620902)'':0.0146105)'':0.00638607,(('PDT000123299.1':0.0607214,('PDT000155920.3':0.0355334,'PDT000431844.2':0.0277465)'':0.019247)'':0.00479804,(('PDT000576902.2':0.030801,('PDT000604126.2':0.0308578,('PDT000584516.2':0.00896806,'PDT000672419.1':0.0124847)'':0.0126781)'':0.00406529)'':0.00279573,(('PDT000502378.1':0.0511892,(('PDT000107027.9':0.0231306,'PDT000325112.2':0.021493)'':0.00469958,('PDT000196472.3':0.0223023,'PDT000222746.3':0.0183168)'':0.0156558)'':0.00787415)'':-0.00304346,(('PDT000137733.3':0.0356894,('PDT000469847.2':0.0185847,('PDT000197861.3':0.0146506,('PDT000574369.2':0.0140669,('PDT000315908.2':0.00475552,'PDT000342261.2':0.00371906)'':0.00479111)'':0.00435948)'':0.00680619)'':0.00790714)'':-0.00290603,(('PDT000664980.1':0.0382936,('PDT000325718.2':0.0131267,'PDT000376003.2':0.0140499)'':0.00972139)'':0.00140539,('PDT000325666.2':0.0376631,(('PDT000275774.3':0.0189387,('PDT000247407.3':0.0136342,'PDT000386077.2':0.0188732)'':0.00891219)'':0.00597233,(('PDT000386296.2':0.0158349,('PDT000544877.2':0.0143965,'PDT000502369.1':0.00986757)'':0.00129482)'':0.00462084,('PDT000605102.2':0.0268869,('PDT000131983.3':0.0226078,(('PDT000317248.2':0.0202619,('PDT000710300.1':0.0104823,('PDT000593617.2':0.00820779,('PDT000630459.2':0.00387904,'PDT000679485.1':0.00211796)'':0.00424091)'':0.00213609)'':0.00945839)'':-0.00216216,(('PDT000221955.3':0.0159747,'PDT000130543.3':0.0215241)'':0.00335983,('PDT000163011.4':0.0187069,('PDT000343885.2':0.0139706,(('PDT000146535.3':0.00638041,'PDT000443303.2':0.00773909)'':0.00293211,(('PDT000107657.2':0.0036954,'PDT000141123.3':0.00408127)'':0.00434142,(('PDT000188052.3':0.0108292,'PDT000063257.3':0.0115271)'':0.000523414,('PDT000334219.2':0.0138897,(('PDT000205375.3':0.0115162,(('PDT000151642.3':0.00559921,'PDT000285825.3':0.00725969)'':0.000717864,('PDT000239293.3':0.000733844,'PDT000555924.2':0.000765556)'':0.00781934)'':0.00148529)'':-0.000582382,('PDT000063173.3':0.00808438,(('PDT000579284.2':0.00231011,'PDT000619588.2':0.00297615)'':0.000757651,('PDT000186212.1':0.00373782,('PDT000168321.3':0.0013768,'PDT000399330.2':0.00142124)'':0.00155015)'':0.000288136)'':0.00195115)'':0.0019241)'':0.00294663)'':-0.000923058)'':0.000833115)'':0.000718416)'':0.00111606)'':0.000781138)'':0.00205162)'':-0.000520208)'':0.00677709)'':0.00104707)'':-0.00571508)'':0.00463306)'':0.00506002)'':-0.000587467)'':-0.00194763)'':0.00397408)'':0.0084389)'':0.0134102)'':-0.0093152)'':0.000251993)'':0.0353807)'':0.00287413)'':-0.036785)'':0.0393628)'':0.00293221)'':0.0101874,((((('PDT000496892.1':0.107315,'PDT000667759.2':0.0960155)'':0.0170153,('PDT000716284.1':0.0607202,('PDT000691090.1':0.0485014,'PDT000720331.1':0.0445954)'':0.00698693)'':0.0322982)'':0.0126163,(('PDT000599142.2':0.0793552,(('PDT000502507.1':0.0272542,'PDT000502520.1':0.0185828)'':0.027637,(('PDT000502517.1':0.00983516,('PDT000502509.1':0.00582619,'PDT000502511.1':0.00700451)'':0.00147429)'':0.0230954,(('PDT000288537.3':0.0158736,'PDT000137527.3':0.0208418)'':0.00718899,('PDT000399989.2':0.0167973,'PDT000140540.3':0.0149362)'':0.0174338)'':0.00559952)'':0.0136174)'':0.0207373)'':0.0206458,(('PDT000550343.1':0.0746171,(('PDT000397555.1':0.0226408,'PDT000645096.1':0.0289219)'':0.0266875,('PDT000662175.1':0.0759657,(('PDT000655480.1':0.0466753,'PDT000502483.1':0.0345204)'':0.00923843,('PDT000425534.1':0.0546813,('PDT000645578.2':0.0427262,('PDT000488281.1':0.0263866,('PDT000639538.1':0.0228613,'PDT000488276.1':0.0237597)'':0.00882838)'':0.00965431)'':0.00423057)'':-0.00113581)'':0.00941293)'':-0.00208838)'':0.00918237)'':0.0158834,(('PDT000651228.1':0.0851775,('PDT000181666.2':0.0570227,'PDT000411303.1':0.0613623)'':0.00674097)'':0.0149086,(((('PDT000252688.1':0.037345,('PDT000281074.2':0.0363654,'PDT000642595.1':0.0363373)'':0.0276362)'':0.0201377,(('PDT000426568.1':0.0588509,'PDT000639583.1':0.0476511)'':0.00488533,('PDT000675055.1':0.0597446,'PDT000463688.1':0.0582644)'':0.00704792)'':0.00816828)'':0.00226513,((('PDT000350388.1':0.0302507,'PDT000637789.1':0.0325043)'':0.010126,('PDT000394949.1':0.0382047,('PDT000025151.2':0.0256729,'PDT000565844.1':0.0292486)'':0.0152888)'':0.00668643)'':0.0196356,(('PDT000540523.1':0.0573425,'PDT000386715.1':0.0652655)'':0.00633893,(('PDT000380320.1':0.0346058,'PDT000251508.1':0.0373888)'':0.00891496,('PDT000523910.1':0.0272396,'PDT000557280.1':0.0294185)'':0.017157)'':0.0166409)'':0.000676736)'':0.00399836)'':0.00092331,((('PDT000150277.2':0.033788,'PDT000645101.1':0.0372417)'':0.0169518,('PDT000539288.1':0.0573211,('PDT000521374.1':0.0421036,('PDT000381352.1':0.0417511,('PDT000455357.1':0.0100396,'PDT000651555.1':0.00861121)'':0.0252908)'':0.000473914)'':0.0119648)'':-0.00267405)'':0.00413817,(('PDT000557201.1':0.0385782,'PDT000251792.1':0.0556112)'':0.013289,('PDT000502700.1':0.0742268,('PDT000145187.2':0.0610504,('PDT000175380.2':0.0539844,(('PDT000652329.1':0.0357073,('PDT000395373.1':0.0314289,('PDT000588878.1':0.0280622,'PDT000228146.1':0.026652)'':0.00703533)'':0.00747583)'':0.000565637,('PDT000278220.1':0.0518789,('PDT000365304.1':0.0482552,('PDT000502886.1':0.0387838,('PDT000047182.2':0.0227055,'PDT000252501.1':0.0200648)'':0.00703145)'':0.00618016)'':0.0110925)'':-0.00358734)'':0.00739179)'':0.00370062)'':0.0047865)'':-0.00278564)'':0.000191236)'':0.00629066)'':0.00852369)'':0.0005399)'':0.00639258)'':0.0177004)'':-0.0190907,((('PDT000253674.1':0.0596338,('PDT000188089.2':0.031512,'PDT000517932.1':0.0277286)'':0.0227934)'':0.0321966,(('PDT000393718.1':0.075346,('PDT000188939.2':0.0538928,('PDT000251591.1':0.0498943,'PDT000252157.1':0.0459462)'':0.00888246)'':0.0045083)'':0.00725434,(('PDT000285323.1':0.0559929,('PDT000557672.2':0.041513,'PDT000648046.1':0.0513324)'':0.0101694)'':0.0170772,('PDT000644968.1':0.0203463,('PDT000046341.1':0.0234716,'PDT000019129.2':0.0188251)'':0.00685395)'':0.0409762)'':0.00539534)'':0.0118189)'':0.00947058,(((('PDT000661107.1':0.0887361,'PDT000661111.1':0.115283)'':0.00935095,(('PDT000150269.2':0.028968,('PDT000294401.1':0.032746,'PDT000395448.1':0.0403985)'':0.0010242)'':0.0169243,('PDT000400843.1':0.0482387,(('PDT000333672.1':0.0250696,'PDT000415603.1':0.0323212)'':0.00651054,('PDT000258722.2':0.0222629,('PDT000502334.1':0.027464,('PDT000502527.1':0.0153544,'PDT000502894.1':0.0174891)'':0.00311891)'':0.00267281)'':0.00851823)'':0.00636138)'':0.00880525)'':0.0359087)'':-0.00362507,(('PDT000109474.2':0.0279684,('PDT000502310.1':0.0279889,'PDT000502419.1':0.0392906)'':0.00674582)'':0.0461255,((('PDT000260220.2':0.0323358,'PDT000346132.1':0.0385834)'':0.0237192,('PDT000557315.1':0.0322688,'PDT000042650.1':0.0387377)'':0.0194461)'':0.00825149,(('PDT000382197.1':0.0594376,('PDT000644987.1':0.0530604,('PDT000250045.1':0.0375853,('PDT000150856.1':0.032325,'PDT000501840.1':0.0313167)'':0.0136728)'':0.0131182)'':0.00186471)'':0.000373453,('PDT000720185.1':0.0608787,('PDT000226172.2':0.0484544,(('PDT000485559.1':0.0313324,('PDT000262174.2':0.0153001,'PDT000499664.1':0.0172379)'':0.0131613)'':0.00673572,('PDT000190104.3':0.0380857,('PDT000389657.1':0.03228,'PDT000719017.1':0.0308462)'':0.00287817)'':0.00155812)'':0.00568629)'':0.00550105)'':0.0127547)'':0.0041284)'':0.00621461)'':0.00469047)'':0.00154339,(('PDT000270452.1':0.0864224,(('PDT000252633.1':0.0295504,'PDT000395150.1':0.0399089)'':0.00401629,('PDT000468847.1':0.0456451,('PDT000397557.1':0.0320397,('PDT000354889.1':0.0270385,'PDT000718994.1':0.0251813)'':0.00783403)'':0.0127922)'':0.0032151)'':0.019329)'':0.0214693,((('PDT000691094.1':0.00534806,'PDT000346084.1':0.00835054)'':0.0558175,('PDT000636200.1':0.0875586,('PDT000577098.1':0.0437935,('PDT000258734.2':0.0271228,'PDT000352523.1':0.0236535)'':0.014627)'':0.0215881)'':-0.00199421)'':0.00352519,(('PDT000706049.1':0.049148,('PDT000372355.1':0.0460779,('PDT000027897.2':0.0388549,('PDT000339383.1':0.0255286,('PDT000662171.1':0.019456,'PDT000714238.1':0.0192549)'':0.00228474)'':0.00624364)'':0.00298832)'':0.008394)'':0.00514529,((('PDT000399619.1':0.0446401,'PDT000645533.2':0.0481523)'':0.00779926,('PDT000625974.1':0.0378274,('PDT000011803.2':0.0262068,('PDT000397910.1':0.0249174,('PDT000004347.2':0.00706626,('PDT000395177.1':0.00148047,'PDT000395440.1':0.00211701)'':0.0061339)'':0.0126195)'':0.00719408)'':0.00881905)'':-0.000312155)'':0.0112059,(('PDT000520702.1':0.043468,'PDT000497259.1':0.0346145)'':0.0038574,('PDT000619604.1':0.046033,(('PDT000556956.1':0.0339607,'PDT000557109.1':0.0305176)'':0.00400252,('PDT000356791.1':0.0324002,('PDT000439417.1':0.0201141,('PDT000557091.1':0.00674888,'PDT000228381.1':0.00982952)'':0.00358861)'':0.0159479)'':0.00261175)'':0.0055679)'':0.0049915)'':-0.00101055)'':0.00407722)'':0.0116372)'':0.0139241)'':0.00834649)'':0.00511218)'':-0.00145656)'':0.00484985,((('PDT000140553.3':0.052012,('PDT000421954.2':0.018362,'PDT000510650.2':0.0179712)'':0.0572264)'':0.0088072,('PDT000524501.1':0.0564043,('PDT000633199.1':0.0460856,'PDT000345959.1':0.0517311)'':0.0081353)'':0.00878546)'':0.0231867,((('PDT000644677.1':0.0797457,('PDT000367280.1':0.0832569,('PDT000417380.1':0.0340439,('PDT000311242.2':0.0230782,'PDT000417401.1':0.0177039)'':0.0163618)'':0.0423666)'':0.00647904)'':0.00461339,(('PDT000439500.1':0.0824565,'PDT000672168.1':0.0642595)'':0.0202734,(('PDT000661278.1':0.0759034,(('PDT000557140.1':0.0248053,'PDT000252584.1':0.041018)'':0.0162819,('PDT000719108.1':0.0363262,'PDT000045621.1':0.0359014)'':0.0163101)'':0.0305899)'':0.000407606,('PDT000415763.1':0.111942,(('PDT000482528.1':0.055038,('PDT000566187.1':0.0605555,('PDT000374994.1':0.0242669,(('PDT000409865.1':0.016988,'PDT000490607.1':0.0162205)'':0.00523749,('PDT000566193.1':0.0197631,('PDT000232208.2':0.00791601,'PDT000422079.1':0.0136151)'':0.0126011)'':0.00167058)'':0.00279003)'':0.0212811)'':0.00151378)'':0.00426219,('PDT000494592.1':0.0582483,(('PDT000266491.2':0.0194369,'PDT000485588.1':0.0277815)'':0.0138583,((('PDT000319751.1':0.0226702,'PDT000384907.1':0.0302686)'':0.000800243,('PDT000240696.2':0.0188025,'PDT000362852.1':0.022786)'':0.00743628)'':0.00191481,('PDT000499660.1':0.0389156,(('PDT000475410.1':0.0238673,('PDT000095715.2':0.0134141,'PDT000042689.1':0.00994763)'':0.00636265)'':0.00544367,('PDT000305152.2':0.0179006,('PDT000292816.2':0.0159685,('PDT000288643.2':0.0164186,'PDT000470095.1':0.0155404)'':0.00601392)'':0.00190902)'':0.00211745)'':0.00692119)'':-0.00368485)'':0.00630073)'':0.0268828)'':-0.018348)'':0.040508)'':-0.0154102)'':0.0112464)'':-3.50856e-06)'':0.00149545,(((('PDT000551405.2':0.09421,'PDT000651642.1':0.074844)'':0.00183775,(('PDT000396824.1':0.0546286,('PDT000682566.1':0.0437609,'PDT000415829.1':0.0364478)'':0.00358575)'':0.0105599,(('PDT000188083.2':0.0717186,'PDT000557178.1':0.0542294)'':0.00900524,('PDT000346597.1':0.0683777,(('PDT000319736.1':0.0545185,'PDT000645144.1':0.0598645)'':0.0111693,(('PDT000619675.1':0.0481955,'PDT000318964.1':0.0461369)'':0.00406171,('PDT000677943.1':0.0268186,'PDT000395349.1':0.0301701)'':0.0145211)'':0.00386461)'':0.00596496)'':0.00300977)'':0.000186051)'':0.0155203)'':-0.0029292,((('PDT000601908.1':0.0495247,('PDT000557195.1':0.0312214,'PDT000624694.1':0.0331667)'':0.0139062)'':0.0173244,('PDT000226324.1':0.0498296,('PDT000131794.2':0.0360486,'PDT000398144.1':0.0243554)'':0.0249367)'':0.014817)'':0.00383695,(('PDT000303035.2':0.0401423,'PDT000395224.1':0.0457809)'':0.0103791,(('PDT000233051.1':0.030898,('PDT000367296.1':0.0273761,'PDT000371788.1':0.0213368)'':0.0110673)'':0.0107998,(('PDT000238516.1':0.0462381,('PDT000604852.1':0.0339507,('PDT000259567.2':0.0213979,'PDT000566101.1':0.0252576)'':0.00998221)'':0.00999748)'':-0.00291755,(('PDT000025248.2':0.0223874,'PDT000395386.1':0.0167662)'':0.00956827,('PDT000397783.1':0.0313942,('PDT000024332.2':0.015411,('PDT000271393.2':0.0203042,('PDT000060380.2':0.0129862,'PDT000397832.1':0.0119467)'':0.00797999)'':0.00601593)'':0.00159713)'':0.00798099)'':0.0152369)'':0.00533595)'':0.00383957)'':0.0207743)'':0.00380056)'':0.000420125,((('PDT000252489.1':0.0794888,('PDT000252721.1':0.0714265,('PDT000145278.2':0.0548665,((('PDT000251954.1':0.0330524,'PDT000556890.1':0.0389132)'':0.00285546,('PDT000372059.1':0.0363722,'PDT000718982.1':0.0403191)'':0.0110349)'':0.0044713,('PDT000399636.1':0.0345366,('PDT000027746.2':0.0407337,('PDT000497209.1':0.0268476,('PDT000167645.2':0.0217315,('PDT000167688.2':0.0050963,'PDT000026888.2':0.0076502)'':0.00794727)'':0.0104552)'':0.00413048)'':0.0102821)'':0.0131834)'':0.00413561)'':0.0121986)'':0.0109538)'':-0.00226802,((('PDT000557269.1':0.0699273,'PDT000248673.1':0.0735947)'':0.00941063,(('PDT000619597.1':0.0360558,'PDT000248849.1':0.0474956)'':0.0306355,(('PDT000494549.1':0.05528,('PDT000415667.1':0.0409143,'PDT000252565.1':0.0458234)'':0.00966263)'':0.00722301,(('PDT000557241.1':0.0533355,'PDT000251448.1':0.040926)'':0.00995244,('PDT000429709.1':0.0439814,('PDT000202784.2':0.0292091,'PDT000520912.1':0.0325733)'':0.0146318)'':0.00546842)'':0.00424501)'':0.00649447)'':0.0027582)'':0.00136941,('PDT000325633.1':0.0961697,('PDT000095789.2':0.073941,((('PDT000555541.1':0.0514575,'PDT000411311.1':0.0443087)'':0.00766167,('PDT000384291.1':0.0510287,('PDT000492250.1':0.0377929,'PDT000651588.1':0.0339191)'':0.0068919)'':0.00295099)'':0.00487476,('PDT000589279.1':0.0635152,(('PDT000610209.1':0.0433273,'PDT000584745.1':0.0465593)'':0.00980709,('PDT000565913.1':0.0517897,('PDT000196625.2':0.0461136,('PDT000550371.1':0.0323538,'PDT000228311.1':0.0419012)'':0.00748869)'':0.00450082)'':0.00310543)'':0.00983498)'':-0.0023967)'':0.00964074)'':0.00790898)'':-0.00868522)'':0.00190411)'':0.00108803,('PDT000661149.1':0.115612,((('PDT000298033.1':0.0608742,'PDT000298045.1':0.0764488)'':0.0161766,(('PDT000298061.1':0.0709671,('PDT000081757.1':0.0297094,'PDT000501869.1':0.0255852)'':0.0232576)'':0.0120761,('PDT000346933.1':0.0614976,('PDT000538579.1':0.0433957,('PDT000661163.1':0.0241791,'PDT000313448.1':0.0177181)'':0.0420647)'':0.0112633)'':0.00815739)'':0.00150306)'':0.0135885,(('PDT000645688.2':0.123099,(('PDT000298080.1':0.0903948,('PDT000652976.1':0.0470579,'PDT000502247.1':0.0388281)'':0.0353212)'':0.00253303,('PDT000081740.1':0.0771485,(('PDT000471026.1':0.0467712,('PDT000565869.1':0.0300991,'PDT000612019.1':0.0331254)'':0.00853742)'':0.0168727,('PDT000131747.2':0.0375669,(('PDT000395053.1':0.0140594,('PDT000619350.1':0.00899094,'PDT000644894.1':0.0136799)'':0.00161309)'':0.00763663,('PDT000395057.1':0.0113874,('PDT000565934.1':0.00762304,'PDT000594312.1':0.00769886)'':0.00776136)'':0.00151777)'':0.0177966)'':0.0121623)'':0.0161274)'':-0.000860853)'':0.0217491)'':-0.0226269,(('PDT000157753.2':0.0544684,('PDT000312628.2':0.0407281,('PDT000565858.1':0.035913,('PDT000372690.1':0.0375506,('PDT000566687.1':0.030645,'PDT000644998.1':0.0297935)'':0.00269544)'':0.00464688)'':0.00436394)'':0.00489802)'':0.0168672,('PDT000645117.1':0.0719928,((('PDT000482530.1':0.0295189,'PDT000250056.1':0.0410798)'':0.0183559,('PDT000566207.1':0.0342609,'PDT000081748.1':0.0297853)'':0.0196448)'':0.00420565,((('PDT000479645.1':0.0401148,('PDT000126457.2':0.0419993,('PDT000504790.1':0.0280424,'PDT000557034.1':0.0217358)'':0.0160013)'':0.00163889)'':0.0103787,('PDT000288660.2':0.0530907,('PDT000426409.1':0.0571567,('PDT000394893.1':0.0355007,('PDT000150257.2':0.0209084,'PDT000426811.1':0.0305767)'':0.00715277)'':0.0111789)'':0.00373276)'':0.00528737)'':0.00276468,(('PDT000231466.2':0.0478384,'PDT000395040.1':0.0525066)'':0.0213357,('PDT000427288.1':0.0810595,('PDT000032201.1':0.05177,(('PDT000399658.1':0.047406,'PDT000627426.1':0.0403048)'':0.00177756,('PDT000215937.1':0.0215254,('PDT000604409.1':0.020894,'PDT000394833.1':0.0180887)'':0.00883094)'':0.0212632)'':0.00533283)'':0.00394641)'':0.00159268)'':0.00018139)'':0.000469626)'':0.0114774)'':-0.0121612)'':0.00716643)'':0.00493925)'':0.0157787)'':-0.0166479)'':0.000705058)'':0.00617048)'':0.00674762)'':0.00064131)'':0.00442724)'':0.00152205)'':0.00405234)'':0.0446827,(((('PDT000288553.3':0.0757964,('PDT000258443.2':0.0563545,'PDT000285361.2':0.0569975)'':0.0127511)'':0.0297289,('PDT000339382.1':0.0800879,('PDT000627422.1':0.0616078,('PDT000673457.1':0.0548436,(('PDT000313312.2':0.0196123,'PDT000407463.1':0.0179246)'':0.0177415,('PDT000395019.1':0.0438902,('PDT000627421.1':0.045718,('PDT000718843.1':0.030275,('PDT000178581.2':0.0210271,'PDT000459772.1':0.0200634)'':0.00843812)'':0.0039237)'':0.00222204)'':0.00487007)'':0.00728483)'':0.00489819)'':0.0352786)'':0.00345558)'':0.0120923,((('PDT000557053.1':0.0621701,(('PDT000557227.1':0.0348641,'PDT000424777.1':0.0208019)'':0.0101055,('PDT000413226.1':0.0403961,('PDT000228042.1':0.0327541,'PDT000394911.1':0.0244337)'':0.00861415)'':0.00954057)'':0.00450284)'':0.00345081,(('PDT000146530.3':0.0417158,'PDT000351875.2':0.0374605)'':0.0180767,(('PDT000627439.1':0.0401502,('PDT000109428.2':0.0398955,'PDT000400003.1':0.0475141)'':0.00808736)'':0.00336738,('PDT000577419.1':0.0663053,('PDT000500524.1':0.0420447,('PDT000550387.1':0.0269303,('PDT000500534.1':0.019614,('PDT000394827.1':0.0137303,('PDT000258393.2':0.0151621,'PDT000258429.2':0.0133529)'':0.00689077)'':0.00160487)'':0.0127673)'':0.0173483)'':0.00950476)'':-0.00458543)'':0.013158)'':0.00440709)'':0.0516639,((((('PDT000271755.2':0.0225334,'PDT000395023.1':0.0228944)'':0.0559382,(('PDT000332081.1':0.041099,'PDT000645056.1':0.0407235)'':0.012017,('PDT000025498.2':0.0602096,('PDT000557233.1':0.0334493,'PDT000502541.1':0.0292817)'':0.0130779)'':0.00036118)'':0.0182534)'':-0.00490865,(('PDT000250542.2':0.0599668,'PDT000687815.1':0.0536502)'':0.027947,('PDT000362324.1':0.0869949,('PDT000109423.2':0.0523539,(('PDT000135930.2':0.0264398,'PDT000427615.1':0.0340948)'':0.00343027,(('PDT000397807.1':0.0162599,'PDT000556985.1':0.0126138)'':0.00392825,('PDT000694447.1':0.0174684,'PDT000714237.1':0.0178732)'':0.00792478)'':0.00662709)'':0.00828341)'':0.0307537)'':-0.00299026)'':-0.00634281)'':0.0133214,(('PDT000346805.1':0.0827159,('PDT000501056.1':0.052614,'PDT000501940.1':0.074262)'':0.0103936)'':0.00357291,(('PDT000668923.1':0.0659697,('PDT000394894.1':0.0535938,'PDT000668928.1':0.0616992)'':0.00916234)'':0.00352341,((('PDT000362851.1':0.0345466,'PDT000717100.1':0.0297057)'':0.0258993,(('PDT000485552.1':0.03762,'PDT000718957.1':0.0356475)'':0.01866,('PDT000357602.1':0.0451441,('PDT000171816.1':0.0508715,('PDT000095679.2':0.0376846,('PDT000141395.2':0.0243797,'PDT000502918.1':0.0320861)'':0.0054283)'':0.00935387)'':0.00017439)'':0.00473856)'':0.00487736)'':-0.00323882,((('PDT000500583.1':0.0299669,'PDT000562852.1':0.0348434)'':0.0118606,('PDT000356788.1':0.0383386,('PDT000359339.1':0.0315143,('PDT000495055.1':0.0293646,('PDT000539281.1':0.0200011,'PDT000346515.1':0.0261315)'':0.00214749)'':0.00297936)'':0.00367038)'':0.00421342)'':0.00169042,(('PDT000128839.2':0.0435031,'PDT000133500.1':0.0462257)'':0.00440212,(('PDT000661164.1':0.047496,('PDT000388468.1':0.0483698,('PDT000720179.1':0.0424358,('PDT000336243.1':0.0286547,('PDT000228309.1':0.0314519,'PDT000233053.1':0.0267948)'':0.00378006)'':0.00399129)'':0.00335628)'':0.00630126)'':-0.00226336,(('PDT000496885.1':0.0242698,'PDT000570973.1':0.0260618)'':0.0253573,('PDT000607571.1':0.0347863,(('PDT000439484.1':0.00718708,'PDT000556979.1':0.00742092)'':0.0134122,('PDT000217972.2':0.0273494,('PDT000181796.2':0.0222155,('PDT000534892.1':0.0100646,'PDT000557666.1':0.0141097)'':0.00456897)'':0.00160253)'':-0.000128597)'':0.00957831)'':0.00815435)'':-0.00369478)'':0.00834607)'':-0.00231386)'':0.011857)'':0.00938504)'':0.0069031)'':0.00232214)'':0.000810664,('PDT000487029.1':0.112203,((('PDT000412038.1':0.0409985,('PDT000600656.1':0.026372,'PDT000395362.1':0.0190915)'':0.0103489)'':0.0252702,(('PDT000231463.2':0.0494291,('PDT000710815.1':0.0340167,'PDT000395160.1':0.0378495)'':0.00822868)'':0.0154597,('PDT000181794.2':0.0665197,('PDT000145176.2':0.0626317,('PDT000232226.2':0.0562409,'PDT000271468.2':0.0558921)'':0.00501577)'':0.00351554)'':-0.000609644)'':0.00410773)'':0.00588529,('PDT000212787.1':0.0939819,((('PDT000409834.1':0.0569128,'PDT000645526.2':0.0677062)'':0.0168464,('PDT000463694.1':0.0633912,(('PDT000285521.2':0.0420639,'PDT000329038.1':0.0459307)'':0.0152561,(('PDT000003871.2':0.0425569,'PDT000428411.1':0.0302849)'':0.0198704,('PDT000025254.2':0.0513119,'PDT000397766.1':0.046872)'':0.0129775)'':0.00568852)'':0.00404429)'':0.00485612)'':-4.56651e-05,((('PDT000329820.1':0.044709,('PDT000069154.2':0.0537743,'PDT000397612.1':0.0359554)'':0.0106355)'':0.00988558,('PDT000713286.1':0.0684377,('PDT000640010.1':0.0473735,('PDT000621057.1':0.0384048,'PDT000015965.2':0.0303348)'':0.00439758)'':0.00981694)'':0.00452426)'':0.0153082,((('PDT000630064.1':0.0572814,'PDT000354195.1':0.0582486)'':0.003154,(('PDT000352469.1':0.0548334,(('PDT000232215.2':0.0399435,'PDT000281131.2':0.0259147)'':0.0121499,('PDT000468209.1':0.0454532,('PDT000397542.1':0.0341314,'PDT000604833.1':0.030713)'':0.0090968)'':0.00285449)'':0.006283)'':-0.00183717,('PDT000713657.1':0.0485882,('PDT000550329.1':0.0355201,('PDT000131791.2':0.0394658,('PDT000124711.1':0.0261181,('PDT000362830.1':0.0284309,'PDT000024318.2':0.0342583)'':0.00187995)'':0.00230255)'':0.00385986)'':0.0101324)'':-0.000918104)'':0.0162349)'':-0.00346258,(('PDT000645121.1':0.0768416,('PDT000372174.1':0.0330841,'PDT000345950.1':0.04681)'':0.0213319)'':0.00334582,(('PDT000334826.1':0.0379035,'PDT000557013.1':0.03757)'':0.0216024,(('PDT000719197.1':0.0533146,'PDT000346552.1':0.0466459)'':0.00204005,(('PDT000485242.1':0.0271338,('PDT000482194.1':0.0234231,'PDT000249818.1':0.0229313)'':0.00116112)'':0.0116535,(('PDT000228192.1':0.0371528,'PDT000252662.1':0.0374669)'':0.00324448,(('PDT000487008.1':0.0235161,'PDT000620740.1':0.0238749)'':0.00422994,('PDT000566016.1':0.0279696,('PDT000104642.2':0.00845492,'PDT000566217.1':0.00862378)'':0.00998902)'':0.000555755)'':0.0146768)'':0.0035549)'':0.0012853)'':0.00691096)'':0.00908315)'':-0.00729774)'':0.00794234)'':0.00295301)'':0.00906546)'':-0.00760236)'':0.0201146)'':-0.0111757)'':0.0272122)'':0.00349894)'':0.0316125,((((('PDT000394976.1':0.0164416,'PDT000661419.1':0.01742)'':0.0476435,('PDT000468330.1':0.0549652,'PDT000538663.2':0.0704098)'':0.00437525)'':0.0115003,(('PDT000619551.1':0.0605336,('PDT000397875.1':0.066928,'PDT000714644.1':0.068569)'':0.00310743)'':0.0135602,('PDT000215882.3':0.0712293,('PDT000396820.1':0.0497391,'PDT000228007.1':0.0531299)'':0.0131492)'':0.0169803)'':0.00365897)'':0.0685258,(((('PDT000344857.1':0.0842274,('PDT000709970.1':0.0774249,('PDT000405262.1':0.0433019,('PDT000271401.2':0.0334676,'PDT000588013.1':0.0339765)'':0.0067372)'':0.0100326)'':0.00715902)'':-0.00372035,((('PDT000502259.1':0.0289404,'PDT000502264.1':0.0368401)'':0.0272187,('PDT000502315.1':0.0394466,('PDT000432465.1':0.022735,'PDT000444818.1':0.0289807)'':0.0155702)'':0.0131265)'':0.00738853,('PDT000455216.1':0.072694,((('PDT000502913.1':0.0278621,('PDT000355828.2':0.0324281,'PDT000502526.1':0.0298472)'':0.00356883)'':0.0176952,(('PDT000150279.2':0.0310154,'PDT000311258.2':0.0361335)'':0.00830095,('PDT000258379.2':0.0321773,'PDT000566128.1':0.0528262)'':0.00399625)'':0.00347217)'':0.00922671,('PDT000469961.1':0.0814786,(('PDT000264625.2':0.0486774,('PDT000395157.1':0.0350703,('PDT000095680.2':0.0254441,'PDT000425964.1':0.0225145)'':0.00815426)'':0.0118465)'':0.0099263,(('PDT000306640.2':0.0436521,('PDT000367705.1':0.028013,'PDT000250181.1':0.0321572)'':0.0158604)'':0.00953896,((('PDT000216077.2':0.0361745,'PDT000282206.2':0.0405481)'':0.00225079,('PDT000600634.1':0.0271072,'PDT000502655.1':0.0295797)'':0.00420559)'':0.0110466,(('PDT000620270.1':0.0442119,('PDT000364397.1':0.0285694,'PDT000251898.1':0.0264836)'':0.00725398)'':0.00562082,((('PDT000281089.2':0.00954294,'PDT000443388.1':0.0107032)'':0.0248753,('PDT000411091.1':0.024068,'PDT000502257.1':0.0229092)'':0.0102379)'':0.0067499,('PDT000564922.1':0.0457389,('PDT000196605.2':0.0377481,('PDT000191847.2':0.0355654,('PDT000539274.1':0.0280918,'PDT000379416.1':0.0259807)'':0.00609358)'':0.00398093)'':0.0060813)'':-0.00366023)'':0.00409015)'':0.0037291)'':0.0018762)'':0.00021062)'':0.01677)'':-0.0137108)'':0.00840387)'':0.000969464)'':0.00429802)'':0.000656401,(('PDT000630063.1':0.0798316,('PDT000661992.1':0.0564077,('PDT000642311.1':0.0198351,('PDT000430845.1':0.0167639,('PDT000367279.1':0.00743895,'PDT000628868.1':0.00776825)'':0.00559953)'':0.00374876)'':0.035334)'':0.0109197)'':-0.000743232,(('PDT000652286.1':0.0584838,'PDT000652562.1':0.0434962)'':0.0639514,((('PDT000396546.1':0.00499852,'PDT000497358.1':0.00447212)'':0.0282245,('PDT000566156.1':0.0291392,'PDT000319011.1':0.032498)'':0.00563063)'':0.016899,(('PDT000252680.1':0.0650376,('PDT000145172.3':0.0472893,('PDT000499452.1':0.038815,('PDT000482521.1':0.0244261,('PDT000292805.2':0.0214994,'PDT000387598.1':0.0232109)'':0.00822827)'':0.00596107)'':0.00566154)'':0.00711869)'':-0.00202834,(('PDT000333763.1':0.0375805,'PDT000564935.1':0.0319408)'':0.0230286,((('PDT000147455.3':0.0281369,'PDT000502677.1':0.0277874)'':0.0102154,('PDT000542096.1':0.0355389,'PDT000669487.1':0.0347366)'':0.0104614)'':0.00316754,(('PDT000336998.1':0.0621986,'PDT000502903.1':0.0451824)'':0.00235994,('PDT000502907.1':0.0423885,(('PDT000375011.1':0.0302439,'PDT000502664.1':0.0327498)'':0.00845309,((('PDT000411079.1':0.0262351,('PDT000175378.2':0.0283906,'PDT000252660.1':0.0229242)'':0.00450675)'':0.000175107,('PDT000235601.2':0.0304949,('PDT000034508.2':0.0262399,('PDT000337224.1':0.0205675,'PDT000396784.1':0.0213487)'':0.00237262)'':0.00368451)'':0.00290335)'':0.00212176,(('PDT000413220.1':0.0378041,'PDT000602924.1':0.0433369)'':0.0023254,('PDT000288595.2':0.029017,('PDT000258366.2':0.0320229,(('PDT000258415.2':0.021201,('PDT000497312.1':0.0209525,'PDT000502403.1':0.0208897)'':0.00264102)'':0.00320634,('PDT000395092.1':0.019097,('PDT000312617.2':0.01583,'PDT000095775.2':0.0247681)'':0.00167553)'':0.00418487)'':0.00129192)'':0.0022769)'':0.00337817)'':-0.00176399)'':0.00647358)'':0.0014018)'':0.00715483)'':-0.0046042)'':0.00845856)'':-0.00625655)'':0.00638109)'':0.0189413)'':-0.00859665)'':0.00371667)'':0.0280185,(((('PDT000249919.1':0.0326775,'PDT000406993.1':0.0259727)'':0.0759426,(('PDT000501953.1':0.0558676,('PDT000161717.3':0.0493707,'PDT000672613.1':0.047465)'':0.00728404)'':0.0093157,('PDT000706512.1':0.0379669,('PDT000115019.3':0.0463716,('PDT000103579.3':0.0104412,'PDT000029742.3':0.0108417)'':0.0127206)'':0.0133661)'':0.018986)'':0.0253256)'':-0.00902827,(('PDT000683623.1':0.0492042,('PDT000663993.1':0.0450717,('PDT000719093.1':0.0372602,'PDT000588039.1':0.0323447)'':0.0042264)'':0.0027179)'':0.00691337,(('PDT000375964.2':0.0558823,('PDT000309220.2':0.0529924,'PDT000485557.1':0.0629176)'':0.0129302)'':0.00349557,(('PDT000501984.1':0.0365325,('PDT000502917.1':0.0228724,'PDT000228306.1':0.0246434)'':0.0114486)'':0.0120792,('PDT000619602.1':0.0465927,('PDT000155951.2':0.0242167,'PDT000502550.1':0.0317833)'':0.0177463)'':0.00400426)'':0.00682499)'':0.00245706)'':0.0137429)'':0.00971471,((('PDT000206131.2':0.00722968,'PDT000216397.2':0.00929432)'':0.106207,(('PDT000443257.2':0.111135,'PDT000502414.1':0.102469)'':0.00405604,((('PDT000206137.2':0.098103,('PDT000310406.2':0.0387498,'PDT000477484.1':0.0519215)'':0.0218099)'':0.00385304,(('PDT000394935.1':0.0171514,'PDT000566209.1':0.0215335)'':0.0321948,('PDT000628128.1':0.017869,'PDT000412039.1':0.0177994)'':0.0269898)'':0.0306253)'':0.00112383,((('PDT000309298.2':0.0386055,'PDT000482524.1':0.0383917)'':0.0290446,('PDT000495090.1':0.0477217,'PDT000590208.1':0.0450175)'':0.012738)'':0.00316528,('PDT000640339.1':0.0796345,('PDT000367257.1':0.0503008,('PDT000395304.1':0.0439681,('PDT000312777.2':0.0295099,('PDT000396541.1':0.0190986,('PDT000502412.1':0.0165043,('PDT000235744.2':0.00906127,'PDT000274858.2':0.0107938)'':0.0108131)'':0.00622968)'':0.00271189)'':0.0100076)'':0.0128934)'':0.0154404)'':-0.00730179)'':0.0149964)'':0.0118231)'':0.00641056)'':-0.013324,((('PDT000252616.1':0.103958,(('PDT000243010.1':0.0851194,('PDT000687971.1':0.0322468,('PDT000258367.2':0.00871762,('PDT000485249.1':0.00612201,'PDT000719199.1':0.00867589)'':0.00217403)'':0.0218031)'':0.0222827)'':-0.00228123,('PDT000661232.1':0.0723938,('PDT000510902.1':0.0671046,('PDT000490593.1':0.0582221,('PDT000439506.1':0.0290075,'PDT000671254.1':0.0363551)'':0.00778856)'':0.00522077)'':0.00952196)'':-0.00480696)'':0.0231869)'':-0.00854171,(('PDT000550399.1':0.0486754,'PDT000718869.1':0.0726136)'':0.0108039,('PDT000557332.1':0.0781164,((('PDT000356782.1':0.0523645,('PDT000339356.1':0.0357999,'PDT000672170.1':0.0330417)'':0.0135512)'':-0.000707705,(('PDT000443383.1':0.0335059,'PDT000045626.1':0.0326371)'':0.0101619,('PDT000550360.1':0.0329873,('PDT000094200.2':0.0291426,'PDT000494441.1':0.0178613)'':0.0156288)'':0.000828703)'':0.0100795)'':0.00841105,(('PDT000502261.1':0.0481195,('PDT000275047.4':0.0392959,'PDT000557320.1':0.0365601)'':0.00625174)'':0.00886063,('PDT000502408.1':0.0561404,(('PDT000502321.1':0.0370216,('PDT000042499.1':0.0238029,'PDT000251810.1':0.0132894)'':0.0169998)'':0.00588021,('PDT000346522.1':0.0321322,('PDT000345945.1':9.94107e-05,'PDT000345947.1':0.000100569)'':0.0387362)'':0.0083783)'':0.0012594)'':0.00207604)'':0.00422274)'':0.00916495)'':6.79099e-05)'':0.00803298)'':0.00235137,((('PDT000201785.1':0.049509,'PDT000346923.1':0.056115)'':0.00781548,('PDT000652406.1':0.0592907,'PDT000318853.2':0.0732493)'':0.00448002)'':0.0136127,(('PDT000150304.2':0.0535062,(('PDT000279722.2':0.0202645,'PDT000326057.1':0.0246628)'':0.0125836,('PDT000362865.1':0.0304892,'PDT000502665.1':0.0341831)'':0.00790996)'':0.00573529)'':-0.0014245,(('PDT000583902.1':0.050138,(('PDT000539283.1':0.0236137,'PDT000386760.1':0.0271138)'':0.011032,('PDT000376084.1':0.0335876,('PDT000092531.1':0.0260158,('PDT000408760.1':0.0257358,'PDT000251906.1':0.0204014)'':0.00581487)'':0.00743099)'':0.00580048)'':0.00370354)'':-0.00097205,('PDT000490597.2':0.0614038,((('PDT000617220.1':0.0370351,'PDT000502889.1':0.0349657)'':0.00812674,(('PDT000417413.1':0.0321773,'PDT000395379.1':0.0319017)'':0.00586952,('PDT000395422.1':0.0342734,('PDT000252723.1':0.0324286,('PDT000556883.1':0.0240642,'PDT000714700.1':0.0267932)'':0.00390872)'':0.00274829)'':0.000522295)'':0.00363356)'':0.000607449,('PDT000557191.1':0.053381,(('PDT000385066.1':0.0354375,('PDT000470067.1':0.0306706,('PDT000499671.1':0.0259705,'PDT000501826.1':0.0273381)'':0.00603094)'':0.00363724)'':0.00535526,(('PDT000394856.1':0.0191922,'PDT000252593.1':0.0262758)'':0.0174523,(('PDT000551890.1':0.0207748,'PDT000586258.1':0.0273419)'':0.00642937,('PDT000502559.1':0.031128,'PDT000150855.1':0.0265024)'':0.00248528)'':0.00360157)'':0.00129608)'':0.00454534)'':-0.00376099)'':0.0109306)'':-0.0071284)'':0.00276064)'':0.0270039)'':-0.000670378)'':0.00285584)'':0.0022597)'':0.00998036)'':0.0538073)'':-0.0135369,(((('PDT000004349.2':0.0548822,'PDT000354269.1':0.0672248)'':0.032745,('PDT000027747.2':0.0671524,'PDT000184432.3':0.0735046)'':0.0203828)'':0.0606486,((('PDT000448174.1':0.124872,(('PDT000417113.1':0.0467005,'PDT000540591.1':0.0473518)'':0.034823,('PDT000014140.3':0.0190971,'PDT000015746.3':0.0207098)'':0.0847381)'':0.0291426)'':-0.00134372,(('PDT000707862.1':0.0909073,(('PDT000594974.1':0.00631755,'PDT000395138.1':0.00819105)'':0.0442773,('PDT000600632.1':0.052511,'PDT000019709.2':0.041122)'':0.00218456)'':0.0192513)'':0.0237445,(('PDT000645124.1':0.0749603,('PDT000349614.1':0.0624795,'PDT000174334.1':0.0508695)'':0.0305072)'':0.0138988,('PDT000305147.2':0.083497,(('PDT000305445.2':0.0477705,'PDT000069123.2':0.0690635)'':0.028963,('PDT000617228.1':0.0333874,('PDT000542701.1':0.0290595,'PDT000661735.1':0.0335259)'':0.00157418)'':0.0254481)'':0.0130427)'':0.00630643)'':0.00667377)'':0.0112691)'':-0.000827901,((('PDT000286090.1':0.00414951,'PDT000286095.1':0.00474515)'':0.0985552,(('PDT000535441.1':0.0373355,'PDT000414199.1':0.059636)'':0.0406973,((('PDT000632747.1':0.0471455,'PDT000168464.1':0.0607905)'':0.0393289,((('PDT000004397.3':0.0229594,'PDT000024558.3':0.0283753)'':0.0407623,('PDT000633169.1':0.084561,('PDT000555542.1':0.0482939,'PDT000206608.2':0.0445311)'':0.0161025)'':0.000713332)'':0.00497432,(('PDT000353636.1':0.062246,'PDT000020966.1':0.061351)'':0.0179015,(('PDT000168734.1':0.0673886,'PDT000290007.1':0.0665504)'':0.00956691,('PDT000124725.1':0.0732414,('PDT000423887.1':0.0332708,('PDT000720454.1':0.0150865,'PDT000720467.1':0.00955577)'':0.013983)'':0.0375726)'':0.00304377)'':0.00370544)'':0.000195238)'':0.0108244)'':-0.00633769,(('PDT000639545.1':0.0977495,(('PDT000669457.1':0.0470031,'PDT000231011.1':0.0472804)'':0.00435641,(('PDT000652937.1':0.0673736,'PDT000346139.1':0.0575904)'':0.00052144,('PDT000313320.2':0.0250616,('PDT000708756.1':0.0171179,('PDT000069120.3':0.00725028,'PDT000069129.3':0.00676842)'':0.00642327)'':0.00735762)'':0.0187974)'':0.0116273)'':0.0178581)'':-0.00686339,((('PDT000161987.2':0.0403286,'PDT000299874.2':0.0341631)'':0.0144752,(('PDT000095785.2':0.0521524,'PDT000425741.1':0.0608206)'':0.00445789,('PDT000346970.1':0.0513469,('PDT000270453.1':0.0522146,'PDT000270471.1':0.0418372)'':0.0159877)'':0.00547057)'':0.00923111)'':0.00267272,(('PDT000523065.1':0.0385443,'PDT000644930.1':0.0249651)'':0.025016,('PDT000396606.1':0.0655696,(('PDT000027903.3':0.0255945,'PDT000568764.1':0.0203621)'':0.0461469,('PDT000127575.1':0.0505733,('PDT000546953.1':0.036995,'PDT000645059.1':0.0310362)'':0.0156786)'':0.00252723)'':0.00755404)'':-0.000491348)'':0.00017962)'':0.00196226)'':0.00400659)'':0.0162498)'':0.00396969)'':0.00205221,((('PDT000092373.1':0.1125,('PDT000488285.1':0.0836962,(('PDT000497385.1':0.0452922,'PDT000557028.1':0.0458168)'':0.0127059,('PDT000228031.1':0.0310125,'PDT000252495.1':0.0355699)'':0.0230184)'':0.0253943)'':0.0147013)'':-0.00464104,('PDT000124604.1':0.0693198,(('PDT000258439.2':0.0506463,'PDT000502312.1':0.0662417)'':0.000806735,('PDT000661233.1':0.0712971,('PDT000502668.1':0.051538,('PDT000393065.1':0.0340265,'PDT000557166.1':0.0323238)'':0.00961343)'':0.00753632)'':0.00498244)'':0.00622915)'':0.0286495)'':0.00332093,(('PDT000250244.1':0.105214,('PDT000121600.1':0.0823905,('PDT000204894.2':0.0536776,'PDT000488303.1':0.0559514)'':0.0237075)'':0.0218946)'':0.000252565,(('PDT000645100.1':0.0965386,(('PDT000565982.1':0.0656752,'PDT000395401.1':0.0601948)'':0.00852754,('PDT000395248.1':0.0665451,('PDT000565849.1':0.0537352,('PDT000639522.1':0.0333823,'PDT000395490.1':0.0245436)'':0.0134188)'':0.0203943)'':0.00549386)'':0.0104554)'':-0.00276803,(('PDT000046602.1':0.0843279,('PDT000228300.1':0.0738069,(('PDT000502007.1':0.0690389,'PDT000627931.1':0.0698941)'':0.00201818,('PDT000604734.2':0.0670758,('PDT000309252.2':0.0420616,('PDT000319043.1':0.0522507,('PDT000397937.1':0.0265223,'PDT000497381.1':0.034129)'':0.00753774)'':0.00554644)'':0.00437712)'':0.00670968)'':0.00677904)'':0.00361393)'':-0.00362209,(('PDT000228294.1':0.0785427,('PDT000395314.1':0.0551555,('PDT000397651.1':0.0677919,'PDT000720325.1':0.0491061)'':0.00766199)'':0.0109594)'':-0.00254384,(('PDT000027737.3':0.0554485,('PDT000682533.1':0.0166951,'PDT000687778.2':0.0143633)'':0.0469978)'':0.012635,('PDT000640376.1':0.072572,(('PDT000313331.2':0.0535108,('PDT000565910.1':0.00764068,'PDT000687444.1':0.00958182)'':0.0297531)'':0.00169877,('PDT000206121.2':0.0157659,('PDT000204887.2':0.00792583,'PDT000206128.2':0.00967557)'':0.00289326)'':0.0421449)'':0.0155424)'':0.00914111)'':0.00487697)'':0.00209218)'':0.00412769)'':0.0122427)'':0.00235018)'':0.00103794)'':0.00787732)'':0.0389499)'':-0.00295616,((('PDT000137202.1':0.113776,(('PDT000653454.1':0.0208158,'PDT000717057.1':0.0184907)'':0.0300366,(('PDT000249917.1':0.023891,'PDT000251577.1':0.034857)'':0.0191166,('PDT000319020.1':0.038078,('PDT000150877.1':0.0248459,('PDT000343614.1':0.00104114,'PDT000373245.1':0.00105676)'':0.0192886)'':0.00738694)'':0.00541835)'':0.0033155)'':0.0354116)'':-0.0274901,('PDT000395171.1':0.0973128,(('PDT000124728.1':0.0223768,('PDT000471016.1':0.0197674,'PDT000479792.1':0.0145584)'':0.00981707)'':0.0101236,(('PDT000372019.1':0.027503,'PDT000523153.1':0.0382372)'':0.00133661,('PDT000650299.1':0.0298136,('PDT000566211.1':0.0169707,'PDT000626548.1':0.0120234)'':0.00894789)'':0.00642213)'':0.0104402)'':0.039225)'':-0.0165105)'':0.0689517,((('PDT000502568.1':0.119789,'PDT000252504.1':0.0791014)'':0.017831,((('PDT000502242.1':0.0572591,('PDT000196589.2':0.0165224,'PDT000045629.1':0.017685)'':0.0285997)'':0.00745569,(('PDT000312781.2':0.0317554,'PDT000566699.1':0.0328109)'':0.0135628,('PDT000502672.1':0.0528689,'PDT000228283.1':0.0546321)'':0.00379378)'':0.00166999)'':0.0123664,('PDT000354899.1':0.0914394,('PDT000242787.3':0.0622835,(('PDT000396764.1':0.0444573,('PDT000299984.2':0.0268801,'PDT000171810.1':0.0218164)'':0.00981901)'':0.00333277,('PDT000631952.1':0.0321799,('PDT000439437.1':0.0217444,'PDT000466989.1':0.0189618)'':0.00742915)'':0.00840149)'':0.0124088)'':0.0145421)'':-0.0128655)'':0.0431342)'':0.0187053,(('PDT000364401.1':0.0389353,'PDT000501836.1':0.0622957)'':0.082165,(((('PDT000124601.1':0.106483,('PDT000339360.1':0.0480887,'PDT000502522.1':0.0426377)'':0.0303129)'':0.0172476,('PDT000691305.1':0.0716381,((('PDT000303140.2':0.0452124,'PDT000557318.1':0.0441353)'':0.0071793,('PDT000502406.1':0.0196808,'PDT000502657.1':0.0216578)'':0.0247627)'':0.00553467,(('PDT000354878.1':0.0378726,'PDT000556881.1':0.0293576)'':0.00771597,('PDT000683614.1':0.0391493,'PDT000706409.1':0.0320575)'':0.0105906)'':0.00492902)'':0.0160279)'':0.0130625)'':-0.00142094,((('PDT000550369.1':0.0650196,'PDT000228126.1':0.0786944)'':0.00383845,('PDT000250043.1':0.0997217,('PDT000497870.1':0.0881042,('PDT000672704.1':0.0877784,('PDT000557211.1':0.0744646,(('PDT000258444.2':0.0364072,'PDT000651939.1':0.0288444)'':0.0234936,(('PDT000249953.1':0.0393868,'PDT000252513.1':0.0399524)'':0.0094448,('PDT000379440.1':0.0572586,('PDT000620256.1':0.0411933,('PDT000430751.1':0.0436202,'PDT000509497.1':0.0432987)'':0.00707898)'':0.00918132)'':-0.00103586)'':0.00192038)'':0.0121045)'':0.0046997)'':0.00222545)'':0.00273879)'':-0.000832964)'':-0.000795336,((('PDT000190947.3':0.00667569,'PDT000208699.1':0.00559611)'':0.0839915,('PDT000573941.2':0.0567608,'PDT000600061.2':0.0534032)'':0.0301559)'':0.0214614,(((('PDT000502450.1':0.0388388,('PDT000689072.1':0.0265687,'PDT000103614.3':0.0330657)'':0.010178)'':0.0025213,('PDT000288574.3':0.041685,('PDT000502370.1':0.0471541,('PDT000625733.2':0.0300832,'PDT000502548.1':0.0342978)'':0.00403969)'':0.00580019)'':0.0023366)'':0.0367242,(('PDT000343800.2':0.0412786,'PDT000486521.1':0.0455055)'':0.000981063,(('PDT000203843.3':0.0222025,'PDT000248675.1':0.0240675)'':0.0092966,(('PDT000354926.2':0.0318034,('PDT000639316.1':0.0236256,('PDT000394034.2':0.022324,'PDT000430579.2':0.0173288)'':0.00222213)'':0.00367463)'':0.00598597,(('PDT000502506.1':0.0291589,('PDT000077521.1':0.011834,'PDT000248526.1':0.015547)'':0.0121379)'':-0.000610221,('PDT000110809.3':0.0224567,('PDT000081054.1':0.019164,('PDT000190942.3':0.0173831,'PDT000497764.1':0.0159338)'':0.000670693)'':0.00510068)'':0.00700528)'':0.00698813)'':0.00144517)'':0.0133499)'':0.0222018)'':0.00150142,(('PDT000568709.1':0.0805232,('PDT000367277.1':0.0374416,'PDT000640343.1':0.0438986)'':0.0177722)'':0.00129637,(('PDT000177795.1':0.0770414,(('PDT000251721.1':0.0269184,('PDT000551252.2':0.0143879,'PDT000586297.2':0.012251)'':0.0135465)'':0.00306813,(('PDT000480403.2':0.0201627,'PDT000708516.1':0.020423)'':0.00434196,('PDT000653525.2':0.0231558,('PDT000143056.3':0.0190319,'PDT000547357.2':0.0145683)'':0.00435375)'':0.00505587)'':0.00450763)'':0.0201102)'':-0.0153597,(('PDT000275703.3':0.0285637,'PDT000366330.2':0.0345757)'':0.000320141,('PDT000638793.2':0.0241884,(('PDT000313233.3':0.00862015,'PDT000334244.2':0.0143432)'':0.00942238,('PDT000107742.3':0.0247449,('PDT000148020.3':0.0208942,(('PDT000288556.3':0.00894987,'PDT000288577.3':0.0134649)'':0.00554291,('PDT000173973.3':0.0222322,('PDT000497958.1':0.0146123,('PDT000589640.2':0.00833852,('PDT000465862.2':0.00666751,'PDT000140207.3':0.00723929)'':0.00308838)'':0.00372006)'':0.00164663)'':0.00126444)'':0.00232127)'':0.00128306)'':-0.000451061)'':0.00653257)'':0.0077889)'':0.00101422)'':0.0351783)'':-0.00231429)'':0.0134822)'':-0.00454169)'':0.00584555)'':0.0332595,((('PDT000218928.1':0.0805863,'PDT000252569.1':0.0903277)'':0.0484824,(('PDT000161632.3':0.113694,'PDT000652258.1':0.120989)'':0.0226232,(('PDT000040379.4':0.0661222,'PDT000251895.1':0.0859778)'':0.0471145,('PDT000672677.1':0.0714676,'PDT000473267.2':0.102072)'':0.0078053)'':0.0195459)'':0.00953061)'':-1.70033e-05,(((('PDT000392397.2':0.0150227,'PDT000486512.1':0.0157162)'':0.00526202,('PDT000077518.1':0.0158048,('PDT000274849.3':0.00420229,'PDT000127287.3':0.00396504)'':0.0104816)'':0.00491593)'':0.0820337,('PDT000248585.1':0.0972243,('PDT000288593.3':0.05954,(('PDT000265932.5':0.0460707,('PDT000714483.1':0.0435488,('PDT000038896.1':0.0342762,('PDT000390630.1':0.0261075,'PDT000611898.2':0.0393158)'':0.00614962)'':0.000509314)'':0.010945)'':-0.00620728,((('PDT000220294.1':0.0175638,('PDT000190975.3':0.0245659,'PDT000238404.3':0.0214394)'':0.00237719)'':0.0110809,('PDT000502578.1':0.0497607,('PDT000510017.2':0.0336534,('PDT000384016.2':0.0261663,('PDT000404012.2':0.0187416,'PDT000140491.3':0.0154591)'':0.0063526)'':0.00418391)'':0.0103433)'':0.0020873)'':0.00115731,(('PDT000682724.1':0.036262,'PDT000502306.1':0.0395701)'':0.00839181,(('PDT000551536.2':0.0396784,('PDT000141125.3':0.0272581,('PDT000656408.2':0.0262125,'PDT000502368.1':0.019718)'':0.000561403)'':0.0117713)'':-0.00078528,('PDT000502479.1':0.033852,('PDT000147979.3':0.0360261,('PDT000502305.1':0.0298792,'PDT000502402.1':0.0297048)'':0.00434182)'':0.00440201)'':0.00337237)'':0.00207515)'':0.00423356)'':0.00770757)'':0.0253845)'':0.0394918)'':0.0038075)'':-0.0137966,((('PDT000357704.1':0.0867468,('PDT000249968.1':0.07945,(('PDT000656749.1':0.0278772,'PDT000502017.1':0.0311462)'':0.0317009,('PDT000228378.1':0.0574846,'PDT000249918.1':0.0454994)'':0.00491763)'':0.0156496)'':0.0139377)'':-0.00961873,(('PDT000252581.1':0.0639899,('PDT000253426.2':0.043751,('PDT000496883.1':0.0378035,'PDT000162892.4':0.0301406)'':0.00677881)'':0.0257308)'':-0.00519841,('PDT000252536.1':0.0460968,('PDT000720547.1':0.00887995,('PDT000337887.2':0.00848662,'PDT000141141.3':0.00798488)'':0.00352975)'':0.0452653)'':0.0143692)'':0.0217564)'':0.00586725,((('PDT000691273.1':0.0411281,('PDT000137570.4':0.0203595,'PDT000137159.1':0.0178182)'':0.0186174)'':0.00200415,(('PDT000381310.2':0.0186666,('PDT000501716.2':0.016833,'PDT000549847.2':0.0165036)'':0.00360297)'':0.000737216,(('PDT000550779.2':0.00929522,('PDT000386726.2':0.00923135,'PDT000557054.2':0.0153752)'':0.00173193)'':0.00512647,('PDT000497828.1':0.0174239,('PDT000501648.2':0.00736168,('PDT000503694.2':0.0122294,('PDT000136919.1':0.00960903,('PDT000363931.2':0.00602404,'PDT000497969.1':0.00553546)'':0.00284792)'':0.00104467)'':0.0014044)'':0.00279691)'':0.00509379)'':0.00151134)'':0.0170754)'':0.0151955,((('PDT000136751.3':0.0385777,('PDT000146147.3':0.0170911,'PDT000481007.2':0.0222231)'':0.0046539)'':0.000215908,('PDT000051368.3':0.0279454,('PDT000231309.3':0.0221387,'PDT000309630.3':0.0172765)'':0.00291235)'':0.00446214)'':0.0225121,(('PDT000679465.1':0.0666194,('PDT000367713.1':0.0477674,'PDT000455274.2':0.0450418)'':0.00755845)'':0.001292,(('PDT000538891.2':0.0443823,('PDT000320187.2':0.035195,('PDT000356226.2':0.0282826,('PDT000150689.3':0.0150694,'PDT000356215.2':0.0115669)'':0.00801096)'':0.00998324)'':0.00786678)'':5.21108e-05,(('PDT000647195.2':0.018633,'PDT000137558.3':0.015066)'':0.0233455,(('PDT000191659.3':0.026529,'PDT000502358.1':0.0348659)'':0.00523397,('PDT000316837.2':0.0343731,('PDT000234196.3':0.0184363,('PDT000288579.3':0.014714,('PDT000288588.3':0.0149098,'PDT000084095.3':0.00713294)'':0.00184556)'':0.00544231)'':0.00985814)'':0.00599062)'':0.00438743)'':0.00140758)'':0.00629345)'':0.00480501)'':-0.00183906)'':0.0469757)'':0.00760677)'':0.032108)'':-0.00286551)'':0.00914493)'':-0.000499051)'':0.00565157)'':0.00217711)'':0.0019572)'':0.00768033)'':-0.00149606)'':0.00652201)'':0.00191899)'':0.00265939)'':0.00189866)'':-0.00171511)'':0.000936113)'':0.00269291)'':0.000665636)'':0.0010202)'':0.000323578)'':0.0106908)'':-0.00280869)'':0.00498922)'':0.0149145)'':0.0101073)'':0.017423)'':0.00666686)'':0.00948076)'':0.0139023)'':0.0437958)'':0.00952451)'':0.0258649)'':0.0704895)'':0.00913465)'':-0.0346075)'':0.0488513)'';

In [ ]:
%%time
!pip --quiet install treeswift
import urllib.request
from treeswift import read_tree_newick

# when NCBI discontinues supporting k-Mer trees, use the cached Newick file from the previous code cell
try:
  url =\
    'http://ftp.ncbi.nlm.nih.gov/pathogen/Results/Escherichia_coli_Shigella/latest_snps/Trees/PDG000000004.'\
    + get_version() + '.reference_target.tree.newick'
  urllib.request.urlretrieve(url, 'tree.newick') # overwrite the cached Newick file with the latest from NCBI
except:
  pass

tree = read_tree_newick('tree.newick')

gene_families_no_mcr = list(gene_families-{'mcr'})
db.execute('DROP TABLE IF EXISTS isolate_for_mcr_survival')
db.execute('CREATE TABLE isolate_for_mcr_survival AS SELECT target_creation_date AS start_date,'
           + ' target_creation_date AS event_date, mcr as event, '
           + (', '.join(['"' + f + '"' for f in gene_families_no_mcr]))
           + ' FROM isolate_for_mcr LIMIT 0')
mcrdf = pd.read_sql('SELECT * FROM isolate_for_mcr', con=db)

skip = dict()
feature = dict()
for u in tree.traverse_postorder():
  if u.is_leaf():
    f = mcrdf.loc[mcrdf['target_acc'] == u.get_label().strip("'")]
    skip[u] = f.empty
    if not f.empty:
      feature[u] = tuple(f[['target_creation_date', 'target_creation_date', 'mcr']
                           + gene_families_no_mcr].iloc[0])
  else:
    skip[u] = min([skip[v] for v in u.children])
    if not skip[u]:
      feature[u] = tuple([min(feature[v][i] for v in u.children if not skip[v])\
                          for i in range(3 + len(gene_families_no_mcr))])
      if feature[u][2] == 0: # non-mcr event
        feature[u] = (feature[u][0], max(feature[v][1] for v in u.children if not skip[v]))\
                     + feature[u][2:] # extend mcr-survival
      for v in u.children:
        if not skip[v]:
          # (u's start date, v's event date, event = {0:no_mcr, 1:mcr}), u's gene families
          db.execute('INSERT INTO isolate_for_mcr_survival VALUES ("' + feature[u][0]\
                     + '", "' + feature[v][1] + '", ' + str(feature[v][2]) + ', '\
                     + (', '.join(map(str, feature[u][3:]))) + ')')

db.execute('DELETE FROM isolate_for_mcr_survival WHERE start_date = event_date')

---
# Survival Analysis, Kaplan-Meier
---
We generate the survival plots for the Kaplan-Meier estimate on the dataset `isolate_for_mcr_survival` with days as the unit of time, stratifying on presence or absence of the gene family `tet(A)`.

In [ ]:
from datetime import datetime
def to_datetime(s):
  '''
  converts string to datetime (y-m-d)
  '''
  return datetime.strptime(s, '%Y-%m-%d')

In [ ]:
survival_df = pd.DataFrame(pd.read_sql_query('SELECT * FROM isolate_for_mcr_survival', con=db))

# compute time-to-event in days
days = [] 
for i in range(len(survival_df)):
  days.append((to_datetime(survival_df['event_date'].iloc[i]) - to_datetime(survival_df['start_date'].iloc[i])).days)
survival_df['days'] = days

# two dfs; with tet(A) and without tet(A)
survival_df1 = survival_df[survival_df['tet(A)'] == 1]
survival_df0 = survival_df[survival_df['tet(A)'] == 0]

In [ ]:
!pip --quiet install lifelines
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
%matplotlib inline
kmf_0 = KaplanMeierFitter()
kmf_1 = KaplanMeierFitter()
kmf_0.fit(survival_df0['days'], survival_df0['event'], label='tet(A) absent')
survival_plot = kmf_0.survival_function_.plot(title = 'Survival Function')
# survival_plot.title('Survival Function')
kmf_1.fit(survival_df1['days'], survival_df1['event'], label='tet(A) present')
kmf_1.survival_function_.plot(ax=survival_plot)
plt.ylabel('Survival probability')
plt.xlabel('Days')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
km_plot = kmf_0.plot(title = 'KM estimates')
kmf_1.plot(ax=km_plot)
plt.ylabel('Survival probability')
plt.xlabel('Days')

---
# Survival Analysis, Cox Proportional Hazards
---
Here we show (a summary of) the covariates and the survival plots for the Cox Proportional Hazards (Cox PH) model on the dataset `isolate_for_mcr_survival` with days as the unit of time, again stratifying on presence or absence of the gene family `tet(A)`.


In [ ]:
survival_cox = pd.get_dummies(survival_df.drop(columns=['start_date', 'event_date']), drop_first = True)

In [ ]:
survival_cox = survival_cox[['days', 'event', 'tet(A)']]

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(survival_cox[['days', 'event', 'tet(A)']], duration_col = 'days', event_col='event', step_size = 0.5)
cph.print_summary()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
cph.plot_covariate_groups('tet(A)', values= [0, 1], plot_baseline=True)
plt.ylabel('Survival probability')
plt.xlabel('Days')
plt.show()
plt.savefig(r'''/content/sample_data/CoxPH.png''')

In [ ]:
# filter down to just censored subjects to predict remaining survival
censored_subjects = survival_cox.loc[~survival_cox['event'].astype(bool)]
censored_subjects_last_obs = censored_subjects['days']

survival_func = cph.predict_survival_function(censored_subjects, times=[365., 1825.], conditional_after=censored_subjects_last_obs)


In [ ]:
survival_func